In [1]:
import os
import sys
import time
import json
import traceback
from typing import List, Dict, Tuple, Generator, Set

import torch
from transformers import (
    PreTrainedTokenizer,
    PreTrainedModel,
    AutoModelForCausalLM,
    AutoTokenizer,
)
from IPython.display import clear_output, display, HTML
from ipywidgets import Output

# Enable efficient transfer for Hugging Face models
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = "1"

# Set the device to 'cuda' if available, else 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Specify the model name (replace with your preferred model)
model_name = "unsloth/Llama-3.2-1B-Instruct"

# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [2]:
%%capture
word_penalties_list = [["a kaleidoscope of", 2000], ["testament", 2000.015048553091537], ["technopolis", 1682.682059035117], ["understandingly", 762.9294022671543], ["paperbound", 659.2264970913199], ["hesitantly", 496.5646879026894], ["piqued", 482.3001178804444], ["delved", 473.4940223966827], ["curveballs", 462.50687039417824], ["bustling", 454.70303449492854], ["marveled", 428.19439049963717], ["inclusivity", 399.28185144068397], ["birdwatcher", 382.93952575702605], ["elara", 382.02399833524635], ["camaraderie", 325.065910926091], ["newfound", 289.3537643476301], ["marveling", 281.4117889244332], ["hiroshi's", 277.20734354116485], ["greentech", 268.92005660614404], ["thoughtfully", 266.9326102346037], ["intently", 251.51633784078055], ["birdwatching", 250.1588231011304], ["amidst", 249.22122673588677], ["cherishing", 247.91009553317267], ["attentively", 246.79285812826976], ["interjected", 235.9610251843368], ["serendipitous", 233.0266906486917], ["marianne's", 232.87022859034334], ["maya's", 230.37564440034032], ["excitedly", 228.84649139211055], ["steepled", 228.70847810531137], ["engrossed", 228.4472196666415], ["fostering", 222.59645412759878], ["brainstormed", 218.63487421031408], ["furrowed", 217.1288191216257], ["nodded", 215.746532494257], ["contemplatively", 213.9394730581084], ["jotted", 212.19052857841066], ["mia's", 209.54359039341304], ["yesteryears", 205.40375361782048], ["conspiratorially", 204.18903883197223], ["poring", 203.12158920489887], ["stumbled", 201.95286430826962], ["strategized", 198.31538808865406], ["hesitated", 194.35575102206053], ["intrigued", 191.32480777377384], ["sarah's", 188.1056806342427], ["lykos", 186.8984432180823], ["adaptability", 185.2743410645729], ["yoing", 184.27349339389698], ["geocaches", 182.61995131301913], ["furrowing", 181.28300434012144], ["quandaries", 178.45513005800902], ["chimed", 177.6317240627814], ["headfirst", 177.55430176035384], ["gruffly", 173.5169670342752], ["skeptically", 173.2196196510284], ["nestled", 170.24886793134038], ["fruitville", 168.93895251122095], ["gastronomical", 168.77834340202367], ["sighed", 167.83599102428747], ["warmly", 166.64524795750117], ["approvingly", 165.14554435242388], ["questioningly", 164.2217764827755], ["timmy's", 162.85237720972512], ["undeterred", 159.81034467083455], ["starlit", 158.81973280586473], ["unearthing", 157.53953848282245], ["grappled", 155.11380760257956], ["couldn", 154.07882616965472], ["yumi's", 153.362396079487], ["seabrook's", 152.65396517679832], ["geocachers", 152.48899331241418], ["animatedly", 150.64516344395025], ["bakersville", 149.48324667712868], ["minji", 148.7787149817242], ["fateful", 147.881376001738], ["sparkled", 145.48284973440963], ["resonated", 144.91492949803347], ["harmoniously", 144.8378436549682], ["fidgeted", 143.88462648395776], ["mwanga", 141.271194443305], ["gleamed", 140.84454272803274], ["embracing", 140.8134127640521], ["pleasantries", 138.9683910665212], ["iostream", 137.02499195670802], ["navigated", 136.8749045617025], ["interconnectedness", 136.6775722710472], ["tanginess", 136.0248012468762], ["mouthwatering", 135.40207079890078], ["amelia's", 135.12735430462644], ["delving", 134.62133115310917], ["mischievously", 134.53400914082113], ["tirelessly", 134.50459651470058], ["transcended", 132.75875026522667], ["sympathetically", 132.28731274201124], ["pondered", 132.24181930810568], ["lingered", 131.6820547398057], ["println", 131.53501960158073], ["empathizing", 130.38734974729505], ["niche", 128.82354262722254], ["regaled", 128.21211629309659], ["greenthumb", 127.87603715586023], ["savored", 127.44044593637169], ["amira's", 127.26977675143385], ["meticulously", 125.67264078678225], ["firsthand", 123.40718461639742], ["empathetically", 122.76583436768932], ["unshed", 122.234447281337], ["jenkin's", 122.13793091468249], ["empathy", 120.78510640297107], ["enigmatically", 120.10278606401909], ["marla's", 119.96311948139385], ["bayville", 119.86205591147561], ["adversities", 119.16540991510242], ["eagerly", 118.7736944560049], ["labyrinthine", 117.30247757246565], ["quizzically", 116.99368259297609], ["transcending", 116.98548707285242], ["resilience", 115.03290831887757], ["lily's", 114.79275367950875], ["commiserated", 114.67810631179985], ["savoring", 114.09168105940152], ["amara's", 113.35572623503091],  ["somberly", 111.33701848917218], ["cinephile", 110.08462885614495], ["solace", 109.58942409221955], ["twinkled", 108.8491511689895], ["aquascaping", 108.490673606918], ["rippled", 107.44694800406151], ["reveled", 107.0468756808211], ["greenhaven", 106.94331659047654], ["birdwatchers", 106.78153731714639], ["adwoa", 105.93106020380063], ["appreciatively", 105.79259888560438], ["awestruck", 105.76861040590829], ["ecotech", 105.3920442928442], ["navigating", 105.00722107822565], ["wasn", 104.19334186895765], ["lightheartedness", 103.97167324805294], ["disapprovingly", 103.7632507571004], ["exclaimed", 103.50051962551075], ["samir's", 103.32935174824694], ["fishkeeping", 102.85891331779236], ["sparked", 101.88166406410981], ["welled", 101.1238612235024], ["jotting", 101.00457387540926], ["resourcefulness", 100.7218012566337], ["flickered", 99.7346200354375], ["reminisced", 99.40737436442414], ["abernathy's", 97.97170234043274], ["unbeknownst", 96.90280381444087], ["pattered", 96.45923446705675], ["reassuringly", 95.99564132547866], ["miscommunications", 95.92537976660182], ["wafted", 95.46404472730974], ["absentmindedly", 94.91609082523561], ["weightiness", 94.72530273625273], ["allyship", 94.20061224332532], ["perseverance", 93.51317590165482], ["timmy", 93.37738761768523], ["mindfully", 92.98616009115628], ["hadn", 92.85616357020969], ["disheartened", 92.79027445159294], ["leaned", 92.64926718281589], ["birder", 92.44448100845747], ["captivated", 92.43721619730242], ["ravi's", 92.31017082428738], ["abuela's", 91.35242003061113], ["apprehensions", 91.29401570563567], ["gestured", 91.25838311067005], ["sagely", 90.85405765687906], ["jamie's", 90.7101431289521], ["emily's", 90.30345237120699], ["piquing", 90.28960468143727], ["bated", 89.97164617298913], ["\u00e9lise", 88.83753859369634], ["cinephiles", 88.7355702396338], ["alex's", 88.2556817444544], ["wholeheartedly", 88.16272402677548], ["enthusiasts", 87.63270532947693], ["enchantingly", 87.09964533268402], ["wambui", 86.62912301520548], ["blankly", 86.48383021818377], ["eadric", 86.34210211634532], ["immersing", 85.82226232320427], ["adversity", 85.78100322441296], ["tldr", 85.71508797142025], ["cleanups", 85.15605680775293], ["candidness", 83.51042672731568], ["todayilearned", 83.43565257213649], ["windowpanes", 82.63160794617292], ["chuckled", 82.17901382209192], ["jake's", 81.84386060155019], ["cobblestone", 81.79712032621906], ["scrolling", 81.78887253113474], ["curiosity", 81.51530210405123], ["homebrewer", 81.04455356211855], ["worriedly", 80.79276900031643], ["intriguingly", 80.58716425837639], ["brainstorming", 80.17302665340826], ["shimmered", 79.93482088716453], ["supportively", 79.81980356155465], ["aldric", 79.76253783791124], ["captivating", 79.53261479326709], ["grumbled", 79.50446451638021], ["flytraps", 79.31283233215875], ["evergreen's", 79.04051980014812], ["jingled", 78.97208486964455], ["csharp", 78.31214402315507], ["etched", 78.2484104971739], ["intricate", 78.21927462240582], ["vibrantly", 78.21788362848538], ["insights", 78.13599417565045], ["etiquettes", 78.13242301857765], ["jamal's", 77.58542678683071], ["serendipity", 77.5119644921462], ["aback", 77.48534961672479], ["vibrant", 77.46467368570217], ["tightknit", 76.78236438341992], ["fostered", 76.37582020518892], ["unease", 76.31779012630413], ["stammered", 76.15241103671885], ["passions", 75.86883777669632], ["johann's", 75.25688694533872], ["maplewood", 75.12232429540036], ["user1", 74.66719089614045], ["appreciating", 74.64547548366207], ["bibliophiles", 74.58982689947247], ["reverberated", 74.49733463113604], ["insightfulness", 74.4353052101276], ["amina's", 74.37594691251186], ["unwavering", 74.36354568386292], ["makena", 74.13148763784136], ["strummed", 73.90269656900801], ["maya", 73.8392870217604], ["dataisbeautiful", 73.65993744752211], ["geocache", 73.44283447399256], ["conlangs", 73.3074975554287], ["geocaching", 72.83344653573383], ["advancements", 72.75299212520173], ["maria's", 72.70713211768961], ["shying", 72.68113986332979], ["quaint", 72.62177837440774], ["unforeseen", 72.5794519923886], ["strategizing", 71.9616017940079], ["dialogues", 71.85996322464466], ["insurmountable", 71.76861982097972], ["clinked", 71.75777624573452], ["trusty", 71.45209585957218], ["persevered", 71.26846357550468], ["collaboratively", 71.12793274132686], ["fascinated", 71.00903688990262], ["thrived", 70.72570258119151], ["anika's", 70.40087132339981], ["chanoyu", 70.30001047623162], ["profusely", 70.22379980888442], ["eliab", 70.12200110026922], ["zara's", 69.93220781961107], ["yumi", 69.79186886111535], ["solidifying", 69.66883391158434], ["naledi", 69.38661708568878], ["murmured", 69.16906099425256], ["prided", 68.88742307155565], ["curveball", 68.78373574046977], ["belongingness", 68.65677493723611], ["hometown's", 68.63745320566527], ["glanced", 68.53580433097356], ["dismissiveness", 68.38075885332297], ["kavarna", 68.17328747022798], ["echoed", 68.15970713407852], ["arben", 67.91829009151938], ["clara's", 67.56435395996198], ["wonderment", 67.27027413037723], ["ayla's", 67.01057148335981], ["aquarist", 66.95076350167123], ["twinkling", 66.8823110049823], ["yearned", 66.52492167836711], ["sqrt", 66.27978481319188], ["paused", 65.93468997276268], ["nurturing", 65.93279047744063], ["avid", 65.90861536910037], ["brimming", 65.8868500882794], ["freydis", 65.8102619080017], ["gesturing", 65.71995864280908], ["seasoned", 65.65967372807557], ["zizzi", 65.54158913859789], ["claudette's", 65.2806182475027], ["breadmaking", 65.1574003584379], ["hyperparameters", 64.887639427456], ["sarah", 64.83972349190283], ["naturedly", 64.23986170183434], ["transformative", 64.05481645093263], ["weren", 63.98985703043479], ["blossomed", 63.34656967116295], ["pastimes", 63.26729479311575], ["meera's", 63.11493587608736], ["dataframe", 62.85647995521886], ["slipups", 62.82128595748357], ["\u00e9mile", 62.81655297158125], ["intricacies", 62.69146579035666], ["enthusiast", 62.68374524810508], ["clinking", 62.56300742571657], ["alexei's", 62.549039194897794], ["underscored", 62.51084338044547], ["ramesh's", 62.37724953745895], ["huddled", 62.3377958443102], ["jaspreet", 62.272673639845316], ["ofrenda", 62.02942100843967], ["gnawed", 62.018344326116726], ["quirks", 61.996773944751006], ["whirred", 61.93374580894079], ["sipped", 61.842944252172124], ["fatima's", 61.816051992685814], ["empathized", 61.720377629749926], ["cheerily", 61.5966576060552], ["unexpected", 61.325131664730904], ["amelia", 61.101950116290936], ["cherished", 61.101151929846445], ["reflecting", 60.927032097356665], ["nervously", 60.9048876340286], ["melodia", 60.851913355398096], ["unlikeliest", 60.84620083891452], ["ahmose", 60.58206785157607], ["intertwine", 60.113328945875296], ["buzzed", 60.08346945794633], ["perusing", 59.98845296235552], ["towering", 59.939238608391236], ["hiroshi", 59.523461555713276], ["prioritizing", 59.1951913370659], ["teemed", 58.95817872069236], ["astonishment", 58.87654436049688], ["gaud\u00ed's", 58.26487683689298], ["jaxon", 58.22494985325537], ["showcasing", 58.126949032849325], ["diligently", 58.0567812007436], ["setbacks", 57.993771930782145], ["millfield", 57.492411928965225], ["exhilarated", 57.45992032748462], ["murmurs", 57.45979798148657], ["gleaming", 57.418567380145646], ["defne", 57.340115496796855], ["coexisted", 57.307181215539096], ["brainteasers", 57.19112616978138], ["yellowed", 57.16422184259698], ["amara", 57.1135530765296], ["max's", 57.111830827461304], ["seamlessly", 57.052298418000575], ["ominously", 56.90590327318063], ["amira", 56.74576385203732], ["quietude", 56.67195414671209], ["adorning", 56.665186571935166], ["teeming", 56.6525883033453], ["delve", 56.596902296062844], ["countless", 56.51125106846517], ["obaa", 56.42379925804734], ["peculiar", 56.18036470693872], ["precariously", 56.08031688405674], ["deepened", 56.03360323513692], ["embarked", 56.01649308334308], ["complexities", 55.98561197719851], ["tapestry", 55.94768582220989], ["humanity's", 55.811883749711356], ["empathetic", 55.708259804410844], ["lily", 55.625562062443784], ["triumphantly", 55.49247433293487], ["remorsefully", 55.47346594250702], ["grumble", 55.39591018490637], ["nuances", 55.36617782480204], ["researching", 54.99205971726203], ["windowpane", 54.890398554195606], ["deciphering", 54.8627833076834], ["pulsated", 54.8443464082954], ["aquafaba", 54.82600437520151], ["breathwork", 54.7877086993204], ["lila's", 54.63315099867143], ["hunched", 54.534772739527725], ["reminiscing", 54.52314396328542], ["shockwaves", 54.474490710921955], ["considerately", 54.12983172465151], ["sparking", 54.01108451941536], ["emboldened", 53.8762318572456], ["delectable", 53.82568959921968], ["vigor", 53.51341918961744], ["homebrewing", 53.459566790168395], ["dismissively", 53.455227623475594], ["aisha's", 53.12263235081756], ["pastime", 53.03810874416869], ["thompson's", 52.62545446543177], ["enriching", 52.46110086597417], ["symbolizing", 52.232380464377385], ["mindedness", 52.20923836204829], ["h\u00fcseyin", 52.1335166949871], ["terrariums", 52.07499895439067], ["hassan's", 51.94861311739921], ["thoren", 51.82316166520704], ["brushstroke", 51.81404946715164], ["trudged", 51.791107652399816], ["dimly", 51.73273818885395], ["preserving", 51.50105302476137], ["flocked", 51.465345199302455], ["proactivity", 51.26042430558556], ["indelible", 51.117498051009456], ["wafting", 51.028249764997334], ["nonna's", 50.99071588203979], ["enthusiastically", 50.97424487320517], ["samantha's", 50.904144208216294], ["sini", 50.83488576920611], ["nov\u00e1k", 50.78232818822808], ["scrolled", 50.76840660448693], ["apiarists", 50.75134446145064], ["enriches", 50.670283286269154], ["meetups", 50.42435427710139], ["thoughtfulness", 50.2945029382233], ["tagines", 50.127841042592706], ["klara's", 50.08730817877745], ["basked", 50.006051569637116], ["compromising", 49.977742192136375], ["pemdas", 49.95373144329856], ["ambled", 49.85097801711601], ["solution's", 49.766544117434016], ["expectantly", 49.73558995437988], ["memoization", 49.53555181241352], ["buend\u00eda", 49.527367161777406], ["bleakest", 49.49804375846876], ["horology", 49.433973211140206], ["scoffed", 49.424866063308336], ["loomed", 49.37974447061713], ["grinned", 49.320505087385264], ["maycomb", 49.23585292544899], ["intertwined", 49.105316327440725], ["coursed", 49.09384736892685], ["surmountable", 48.91773157864695], ["mei's", 48.73256875953959], ["chatbot", 48.7303131442302], ["skillset", 48.51810977088149], ["jiyeon", 48.47057359098148], ["nodding", 48.425368506756456], ["dawned", 48.30874496705882], ["greenwashing", 48.23103960043982], ["browsing", 48.04007219005388], ["seemingly", 47.816609926836044], ["ventured", 47.77434558569385], ["factorials", 47.6998395729457], ["nalini", 47.66197815287227], ["jsem", 47.60235342557785], ["nonno", 47.560413832291594], ["skillsets", 47.53934826086816], ["mariama", 47.51074430078741], ["marianne", 47.49747840168747], ["grueling", 47.442666916537256], ["aficionados", 47.392106431743535], ["confidently", 47.36682440819566], ["overwatering", 47.36418703492115], ["smiled", 47.303430838646], ["townsfolk", 47.24488793502313], ["gamifying", 47.23204105702876], ["perspectives", 47.06114618839279], ["verdant", 47.05816833710129], ["mira's", 47.03448271248642], ["comfortingly", 46.929983660664995], ["lighthearted", 46.887045498727474], ["thrummed", 46.7836356039557], ["immortalizing", 46.61875430226617], ["puzzled", 46.57402757497557], ["nonchalantly", 46.567771645493856], ["nums", 46.55301856122418], ["learnings", 46.40542923819422], ["ravi", 46.374435734329666], ["incorporating", 46.372182420668544], ["aquarists", 46.36597421549888], ["scouring", 46.31445449051849], ["vermicomposting", 46.25759923265036], ["armfuls", 46.25505169238614], ["predicament", 46.236275753777775], ["regaling", 46.218313436958525], ["pi\u00e8ce", 46.102628719034584], ["revolutionize", 45.98587923574832], ["introspection", 45.8887810915839], ["fprintf", 45.85312101996422], ["tifu", 45.74958040176368], ["preconceived", 45.72185195492253], ["exploring", 45.705327066842976], ["lockdowns", 45.686017038389906], ["luminara", 45.6720329499178], ["peasy", 45.65691856858046], ["elior", 45.65086602306514], ["jamie", 45.62163326920453], ["honing", 45.61424457042017], ["coursing", 45.613144852573434], ["sipping", 45.60713179645527], ["samira's", 45.25193951663313], ["upvoting", 45.20629015918102], ["quirky", 45.149367477847996], ["ayla", 45.02322254830481], ["zucchinis", 44.99761388408844], ["nonna", 44.97515920772422], ["gadgetry", 44.97133023111876], ["ttrpg", 44.953641232732316], ["makeshift", 44.79510094891597], ["miscommunication", 44.72308873586542], ["jos\u00e9's", 44.71365514384805], ["pedaled", 44.69043601486939], ["responsibly", 44.37313845840518], ["uncovering", 44.316698305833256], ["pondering", 44.20009776324716], ["divemaster", 44.14057905689858], ["morwen", 44.017915056359406], ["jemaa", 44.007359499230844], ["societal", 43.94298129517164], ["frac", 43.91904541043987], ["steadfast", 43.80086202600298], ["solemnly", 43.790452375098276], ["nerdiness", 43.75545790415384], ["creaked", 43.747658925770445], ["taxidermied", 43.724412278194], ["intertwining", 43.66979587327791], ["saranya", 43.63448926110928], ["plaguing", 43.59820031306868], ["approached", 43.579218908045206], ["dilemmas", 43.38001471085552], ["intertwines", 43.28368766463277], ["conversed", 43.23872834064741], ["yoyos", 43.08126526931893], ["instagrammable", 43.072114812601924], ["lurked", 43.067093704461826], ["acknowledging", 42.96959779700971], ["bloomed", 42.96182130763966], ["raptly", 42.94344531353515], ["ethical", 42.90251649898941], ["rummaged", 42.88891395440686], ["timidly", 42.81137717814631], ["tilework", 42.80795844903429], ["saut\u00e9ing", 42.76681417599876], ["vowed", 42.7025725100713], ["taxidermy", 42.667706893176344], ["seabrook", 42.65338870922443], ["preprocess", 42.6400920574572], ["axolotls", 42.63221117750313], ["punctuated", 42.616485562363806], ["johnsons", 42.441518497702575], ["crackled", 42.42309455725854], ["harmonious", 42.244235331605864], ["aarav", 42.236214175325905], ["reassured", 42.20212530408307], ["harbored", 42.107372303293495], ["eager", 42.02904870414461], ["ensuring", 41.99590296418592], ["godric", 41.98914652878993], ["rulebooks", 41.96733398513861], ["skeptical", 41.96685299350538], ["eleanor's", 41.96188084922578], ["microaggressions", 41.926348958935954], ["apologetically", 41.91710013047142], ["mesmerizing", 41.90250624438543], ["festivalgoers", 41.7846539320588], ["fosters", 41.77032359814835], ["retrospect", 41.764568214779885], ["cohesively", 41.759267899184096], ["conlang", 41.73466685285787], ["greenfield's", 41.714283772018646], ["enlightening", 41.66763318212454], ["meticulous", 41.596787131889506], ["emily", 41.55629596261353], ["gearheads", 41.52785406486475], ["mathletes", 41.471098617071085], ["lockpicking", 41.469107303395056], ["upvotes", 41.466667944141925], ["solving", 41.36784625140943], ["jaxson", 41.30144010917815], ["unconventional", 41.28565052732071], ["cout", 41.27024144428185], ["joaquim", 41.26417923210651], ["immerse", 41.22998048062693], ["artistry", 41.174377793632445], ["profound", 41.16973146779899], ["adorned", 41.13451895045142], ["assertively", 41.10069436019212], ["grappling", 41.05525594094738], ["palindromes", 41.0348477440447], ["lessons", 41.02155515722218], ["squinted", 40.98123545276688], ["blared", 40.96895568509801], ["reminding", 40.92116712344851], ["flaxseeds", 40.88631875031004], ["succumbing", 40.85111975737311], ["sternly", 40.808661759588375], ["greenery", 40.7224941678239], ["gracefully", 40.71392267625715], ["reigniting", 40.71323323647863], ["scribbled", 40.66756718567232], ["concertgoer", 40.644039670291896], ["sari's", 40.570594281195675], ["riya", 40.555235645130814], ["patel's", 40.499492893879115], ["sublist", 40.486770739292375], ["tinkering", 40.46467514776435], ["sweltering", 40.45004002143324], ["watchful", 40.34131304186278], ["hushed", 40.27320293253006], ["glistened", 40.27116276295349], ["greybeard", 40.25353363327142], ["melodiously", 40.18320261343541], ["farsightedness", 40.17885612054208], ["thriving", 40.030545533767864], ["imparted", 39.82023934417236], ["disparate", 39.70837243324884], ["reenactments", 39.61964295049699], ["friedrich's", 39.58544233173004], ["hauntingly", 39.576891267350604], ["caf\u00e9's", 39.55332833439393], ["gleaned", 39.416181354162035], ["serendipitously", 39.41095809928487], ["raved", 39.40831190396945], ["aficionado", 39.40535691589909], ["swirled", 39.393984000274436], ["ambiance", 39.362135302140544], ["writingprompts", 39.35659815707896], ["heikki", 39.256741306722446], ["passersby", 39.214075101273735], ["caringly", 39.15944317576278], ["whispered", 39.14033909479449], ["pelted", 39.073045906024205], ["cacophony", 39.05262614054688], ["contentedly", 39.0402454692624], ["curiously", 38.9835281474433], ["mabel's", 38.93785746499233], ["biophilic", 38.92880904667593], ["mundane", 38.901440085374674], ["abuela", 38.885110159054115], ["hartfield", 38.79996795340342], ["innovision", 38.76838813027479], ["skyrocketed", 38.750817213915354], ["marla", 38.74300242416813], ["wrongdoings", 38.72961974214452], ["embark", 38.728106075994404], ["nature's", 38.69490712384941], ["grumpiest", 38.69344552124972], ["stringbuilder", 38.63098118500293], ["trembled", 38.615498712296244], ["quirkiest", 38.28449768383303], ["enviously", 38.28012127724866], ["kenji's", 38.2789229350017], ["confided", 38.27247752357381], ["monstera", 38.2446297740777], ["murkiest", 38.244346470031076], ["ecolodge", 38.23838991279762], ["houseplants", 38.22948003995324], ["playstyles", 38.1919367046728], ["likeminded", 38.03045191482956], ["liveaboard", 38.018032230979145], ["skepticism", 37.981281732219486], ["jihoon", 37.96952437486307], ["unforgettable", 37.933098742520365], ["jasmeet", 37.902115181708645], ["jolting", 37.897975287091846], ["prioritize", 37.81605407832169], ["unfolded", 37.7113212384689], ["viewpoints", 37.69543819662624], ["misadventure", 37.640211106256665], ["twinge", 37.61288431414412], ["unassuming", 37.561113544384355], ["misunderstandings", 37.54943788022523], ["oohed", 37.439186251522514], ["wayan", 37.4206579829096], ["teachable", 37.33841784950501], ["meticulousness", 37.281436071396385], ["concoctions", 37.275970864131565], ["valuable", 37.24319187902221], ["mihai", 37.24246437346717], ["escapade", 37.21015660252502], ["simmered", 37.12526836100867], ["shrilly", 37.10727996981746], ["realized", 37.09802671395704], ["overcomplicating", 37.08132420724305], ["exchanging", 37.065494965496086], ["duofold", 37.050005521257205], ["escargots", 36.9591966841953], ["effortlessly", 36.93590451204956], ["overhearing", 36.90160916722612], ["lnwr", 36.89955301014872], ["kahlo's", 36.86900948932549], ["juan's", 36.79646517846328], ["acing", 36.709602109185155], ["impulsively", 36.639486104797776], ["eyebrow", 36.63340968240311], ["ripples", 36.632141247881336], ["aquaponic", 36.55938119844362], ["clutched", 36.524512455716824], ["relatability", 36.51836476906051], ["alderwood", 36.497668254072984], ["mindfulness", 36.49405647983694], ["invigorated", 36.435451092686186], ["varchar", 36.350574676988835], ["nilofar", 36.31540648130468], ["cranny", 36.271813915928014], ["matplotlib", 36.226904908401224], ["maple's", 36.2221186876444], ["wellbeing", 35.98793338857624], ["ethan's", 35.986718569487756], ["crocheter", 35.985280002247116], ["exoplanetary", 35.934285273854705], ["trepidation", 35.89672131577932], ["mishaps", 35.881107069522834], ["drumheads", 35.86918693096729], ["greeted", 35.85029070738541], ["riverbanks", 35.80098066719363], ["terrains", 35.79623265246362], ["fixating", 35.77209740243234], ["triumphs", 35.765566279528876], ["seraphina", 35.73086493656254], ["pierre's", 35.670842992574855], ["daunting", 35.6468996172963], ["catan", 35.640470427484864], ["bustled", 35.62103733058621], ["teamwork", 35.6143358487993], ["perfecting", 35.606015466499066], ["encouragingly", 35.486598995525945], ["bubbled", 35.383583845563535], ["trivia", 35.371068282026116], ["freewriting", 35.36256711696093], ["tarnishing", 35.29347464725098], ["misinterpretations", 35.28784839591234], ["r\u00e9sistance", 35.26687226320418], ["intriguing", 35.18561940908936], ["igniting", 34.96847169749946], ["anglerfish", 34.96203729566599], ["samuel's", 34.93937573998746], ["sipho", 34.92366514173393], ["oenophiles", 34.906752606710164], ["squeezy", 34.896000337698545], ["glances", 34.844745299577305], ["anika", 34.840234068958516], ["geekiness", 34.76477816214451], ["mahal's", 34.73647576472621], ["glinted", 34.72018795176995], ["harold's", 34.67782976668187], ["cautiously", 34.55246504211744], ["reskilling", 34.535299264158304], ["anxiously", 34.40464357754285], ["enchanting", 34.38805518714689], ["homeland's", 34.34238284681197], ["pulsating", 34.32447223035281], ["commiserating", 34.296200604002955], ["ensued", 34.287258131304355], ["aromas", 34.25128076199041], ["mused", 34.224865582904464], ["gratefully", 34.20161129615044], ["cityscape", 34.19898429462966], ["meandered", 34.14405994610628], ["begrudgingly", 34.12135067305919], ["megapolis", 34.086070185220244], ["exchanged", 34.06024229496875], ["hummed", 34.031354995997795], ["craftsmanship", 33.883817374356234], ["enigmatic", 33.86998726350755], ["pedas", 33.86471092893193], ["ambiguities", 33.848475632394866], ["rallied", 33.84738739693858], ["creations", 33.83951935828559], ["readline", 33.809442659545134], ["tinging", 33.791265539835706], ["navigate", 33.75203388893048], ["thoroughness", 33.75003566674478], ["innocently", 33.74923544090283], ["leveraging", 33.72510082409337], ["exuded", 33.70265208125222], ["symbiotically", 33.69176972668933], ["tangy", 33.68325310137925], ["heatedly", 33.68088737212645], ["murmur", 33.664380723739], ["xochitl", 33.65493714201496], ["vowing", 33.604665452380154], ["json", 33.59743016898405], ["discussing", 33.58995313712198], ["yiayia", 33.53866625559772], ["flavorsome", 33.4761954648722], ["endl", 33.47284235842425], ["hesitation", 33.46862934546639], ["passionately", 33.44255363677446], ["insistently", 33.42772314564705], ["abuzz", 33.427654981448136], ["retorted", 33.42259218625423], ["culinary", 33.42240896705431], ["strolled", 33.39600946327264], ["deliberation", 33.393403953326086], ["thanked", 33.36416353976146], ["mentorship", 33.327368108291715], ["neha's", 33.31209646749537], ["charat", 33.2755959428452], ["amatriciana", 33.262153294380695], ["groundbreaking", 33.169802124395], ["creatively", 33.1611253899087], ["jitka", 33.10558986405488], ["seema's", 33.08235787116782], ["upskilling", 33.042209378605726], ["elusive", 33.034136129186116], ["ostracization", 33.01645675987068], ["amiss", 32.95795887564757], ["tugged", 32.95650107491881], ["queried", 32.943261593754954], ["tumultuous", 32.93384685159399], ["deeper", 32.90059099113942], ["pawsome", 32.883066558690906], ["groggily", 32.87662181475508], ["drumbeats", 32.869685570567455], ["thabo", 32.86307999478832], ["gasps", 32.84577129744973], ["fascination", 32.821368739949094], ["sergei's", 32.81229780691339], ["somchai", 32.80513140856027], ["proudly", 32.64899832946364], ["overreliance", 32.61187523677248], ["jimbob", 32.6015096462962], ["ivan's", 32.58006029092496], ["philately", 32.503416608422384], ["harnessing", 32.47595694378624], ["avians", 32.452478540839294], ["selflessness", 32.41892569443487], ["embodying", 32.41169789188854], ["curie's", 32.40720771053174], ["friendships", 32.39340716518092], ["emphasizing", 32.29647129117152], ["clutching", 32.28044598695961], ["tom\u00e1s", 32.277295172973176], ["obscure", 32.251794437326], ["thandiwe", 32.23211035578734], ["passionate", 32.22988199090224], ["heartful", 32.177762148128075], ["aquascape", 32.093483043497045], ["se\u00f1ora", 32.01840264315037], ["authentically", 31.978173744935475], ["creativity", 31.975436771956428], ["soapmaking", 31.93719932947355], ["flightradar24", 31.935939529097645], ["jamila", 31.934205450933177], ["gutenberg's", 31.858010789833063], ["contemplation", 31.679877061416715], ["confines", 31.603633512874673], ["modernity's", 31.59789309489747], ["determination", 31.56903963348008], ["prioritized", 31.54919326231627], ["mariam's", 31.525541030171688], ["discovering", 31.51301039159474], ["earnestly", 31.501905146958183], ["aloud", 31.483377240728043], ["brainfuck", 31.45777779713726], ["numismatics", 31.428239977609437], ["overwatered", 31.42823997760943], ["ensconced", 31.397445229591053], ["tokenize", 31.37488133588174], ["scikit", 31.370770624812085], ["ethel's", 31.31977323049085], ["zara", 31.281437014556122], ["heartwarming", 31.27944321173799], ["polyglots", 31.258494254908022], ["wei's", 31.240728637448967], ["echoing", 31.24003105061801], ["grizzled", 31.195843248762383], ["revolved", 31.17350582200144], ["emiko", 31.1724124220379], ["neurodiverse", 31.146687995727152], ["jangled", 31.13055492376021], ["waitstaff", 31.119137138914173], ["crafting", 31.115029691282864], ["tranquility", 31.082056732743283], ["knitter", 31.069847767338448], ["apprehension", 31.04396014989861], ["mentor's", 31.00254787264955], ["edna's", 30.99019294650899], ["willowbrook", 30.980487375387597], ["halfheartedly", 30.96444355040424], ["horologist", 30.924812792613398], ["keepsakes", 30.876056504318615], ["colloquialisms", 30.78497190789228], ["excitement", 30.783400792330294], ["wistfully", 30.755600264564325], ["regex", 30.743588531600164], ["tailored", 30.71259475671867], ["reverie", 30.693407028492665], ["dejectedly", 30.655657967444434], ["conundrum", 30.64005280132886], ["ahhs", 30.622703010857947], ["priya's", 30.61142324688942], ["awaited", 30.60988691448054], ["softly", 30.565866035055194], ["destigmatizing", 30.544790648095287], ["stargazing", 30.53694344273088], ["tactfulness", 30.52820524140854], ["cybertech", 30.444325023682456], ["kindhearted", 30.394416294135436], ["ahau", 30.347727482623707], ["gaze", 30.292891094081703], ["showcased", 30.281399338580794], ["discoveries", 30.266892121335964], ["gasped", 30.248422823334863], ["lush", 30.23890591470583], ["crocheters", 30.233599276706148], ["equaled", 30.23018084682644], ["priya", 30.215485271995703], ["blurted", 30.214426847084567], ["overwhelmed", 30.212496015920962], ["paled", 30.209302315209726], ["rohan's", 30.140332264465705], ["admiring", 30.071243010681464], ["unfazed", 30.061675691374933], ["apiculture", 30.029455718627165], ["thoughtful", 30.01911994513467], ["interjecting", 30.007531761041562], ["alleyways", 29.896078233830348], ["tadesse", 29.855919122743487], ["cozily", 29.849965917598475], ["enthusiasm", 29.849502599486055], ["adverted", 29.846040253336188], ["aldwyn", 29.826913080653966], ["typewriters", 29.817651502302578], ["intercropping", 29.808582873500175], ["numismatist", 29.79346849216283], ["gastronomes", 29.77412208405104], ["wove", 29.678429313349312], ["alex", 29.655239798031985], ["hesitatingly", 29.650063242034157], ["flourished", 29.642940246320272], ["predicaments", 29.630285745480748], ["jovial", 29.627507088940177], ["neurodivergent", 29.61600787363737], ["sfumato", 29.590331206988992], ["steaming", 29.58654986358533], ["reusables", 29.574741802238197], ["sheepishly", 29.57337962124056], ["soulfulness", 29.571730015503867], ["sustainability", 29.564759624008968], ["sustainably", 29.559155823353734], ["homebrewers", 29.51659474732937], ["tantalizing", 29.51294557454182], ["perked", 29.510813521402973], ["impromptu", 29.50679528899531], ["whispers", 29.476981147929965], ["indomitable", 29.458182758031825], ["jasmine's", 29.44900236014474], ["instilling", 29.43481584947496], ["aroma", 29.434054905345945], ["rhythm's", 29.433568400083136], ["scribbling", 29.405218093401324], ["sneered", 29.333586604236], ["captivate", 29.30562192537702], ["compostable", 29.29029260018521], ["challenges", 29.28182126825461], ["keycap", 29.215857294975084], ["fueled", 29.21418983742109], ["languagelearning", 29.210218256701587], ["lemonades", 29.191584912841346], ["appreciation", 29.190857558629258], ["factoria", 29.18336569349447], ["commonalities", 29.166945878409514], ["jamukha", 29.12685856048471], ["financia", 29.091527582124105], ["introverted", 29.089596224993407], ["smoothly", 29.070690320130698], ["amicably", 29.02031692970086], ["rummaging", 29.0167102064743], ["greywater", 28.971246147411584], ["sofia's", 28.969784536908637], ["spiraled", 28.964293531996407], ["nonrenewable", 28.947063137271844], ["cachers", 28.947063137271844], ["missteps", 28.855640168861804], ["mattered", 28.84788925471558], ["pursuits", 28.798266371400405], ["riverbank", 28.779966338177235], ["contrasting", 28.7420214400495], ["ionut", 28.740298400577043], ["artisanal", 28.73460147914461], ["handsomely", 28.706791071705556], ["conundrums", 28.6960935258354], ["tribulations", 28.656491391916727], ["jake", 28.651221916714473], ["piecing", 28.65017018201777], ["innovation's", 28.613571626473778], ["shared", 28.580722643037728], ["overharvesting", 28.562384557374543], ["teetered", 28.499159677429205], ["lils", 28.49075475284194], ["savasana", 28.46586520460031], ["\u00e9touff\u00e9e", 28.46055787446055], ["coexistence", 28.44603624381996], ["gastronomic", 28.41578289857809], ["zerowaste", 28.41100641250755], ["adventurous", 28.367984944237204], ["archrivals", 28.328007039383223], ["shaping", 28.278563205718033], ["sigurdur", 28.273875622451566], ["realization", 28.24146370439108], ["overshadow", 28.209481328009694], ["admiration", 28.201859638952797], ["ixchel", 28.157175379111937], ["laughter", 28.06978989729534], ["twist", 28.06308160936643], ["snippet", 28.049516212073968], ["perused", 28.04275143172756], ["firesides", 28.03729829581473], ["\u00e9milie", 28.01328690703727], ["oddities", 28.004493164406167], ["homebrewed", 27.992764352526617], ["workspace", 27.983329195050324], ["jorge's", 27.96124984157528], ["warier", 27.940208767279778], ["martina's", 27.913239453797846], ["nepenthes", 27.90614226603945], ["maneuverings", 27.867915242205044], ["birdhouses", 27.826820755798476], ["frowned", 27.81841286765851], ["potlucks", 27.79182955502688], ["tabletop", 27.78918061178097], ["amiably", 27.789180611780967], ["transcends", 27.780666769681773], ["inkwells", 27.777318411747483], ["herpetology", 27.7480723564381], ["reminded", 27.747352664371224], ["crafters", 27.74406830559301], ["resonating", 27.729819122858917], ["sifted", 27.729819122858917], ["handwoven", 27.72179062352489], ["aamir's", 27.69685775260562], ["innocuous", 27.69640617340313], ["swirling", 27.69529824484928], ["connections", 27.690408404548393], ["reenactor", 27.64652818410554], ["basking", 27.642729067054855], ["taxidermists", 27.642641308459016], ["realizing", 27.615524826492603], ["unboxed", 27.54742491964551], ["bakery's", 27.54106292774721], ["reluctantly", 27.53314427825677], ["amina", 27.523234112128016], ["coexisting", 27.49059831404543], ["demeanor", 27.480603871827594], ["embarking", 27.435826806588064], ["preeti", 27.435541613847796], ["unearthed", 27.424207502363572], ["sreekumar", 27.41880203996246], ["kofi", 27.412651143905176], ["serene", 27.388990798823293], ["counterarguments", 27.38512518824612], ["graying", 27.32702734316864], ["throngs", 27.301967704951043], ["thompson", 27.281578595884678], ["nia's", 27.188943819268165], ["witnessing", 27.16888640169657], ["puzzlers", 27.168034739165567], ["wavered", 27.16040374583217], ["collaboration", 27.141346637814294], ["importantly", 27.12380160301239], ["kavi", 27.115093554805586], ["scoured", 27.10370294233177], ["fueling", 27.102897655942915], ["askreddit", 27.080878847103403], ["enthusiast's", 27.070864600596817], ["secretively", 27.046161525722887], ["dejected", 27.042300045071542], ["overheard", 27.039390717997676], ["renewed", 27.024301460364175], ["datasets", 27.01865329532353], ["positivity", 27.017580991573183], ["deliciosa", 26.982798138671253], ["marco's", 26.918987968734342], ["hesitant", 26.91787240228499], ["trebuie", 26.916338044733642], ["parseint", 26.916338044733642], ["strived", 26.897639103049503], ["babbel", 26.88746109081782], ["chirpy", 26.88428082295197], ["jayden's", 26.879415770323853], ["struggled", 26.85679327560313], ["sandcastles", 26.848401059819636], ["yiannis", 26.775134424860394], ["disagreements", 26.757424575673923], ["microgreens", 26.742485481122003], ["codebases", 26.73681940018951], ["indulging", 26.69861683777212], ["fascinating", 26.69544110238475], ["composting", 26.69069595610424], ["playtesting", 26.688832621718213], ["fived", 26.684974759657226], ["unyielding", 26.680603523501933], ["insatiable", 26.65460611115387], ["impulsiveness", 26.652641542981172], ["cuisines", 26.652565316350135], ["profoundness", 26.646101475998986], ["twists", 26.635354862769553], ["rohit's", 26.628968342778045], ["knickknacks", 26.607356906251752], ["gracefulness", 26.603427828564936], ["subtly", 26.556656016343275], ["boundless", 26.548592191612176], ["edifices", 26.548592191612176], ["respecting", 26.53892931698272], ["rulesets", 26.535976038186732], ["shuteye", 26.526036038518196], ["subarray", 26.52292484498799], ["misha's", 26.489183732054798], ["whimsical", 26.472778454824084], ["martha's", 26.466752327245022], ["budi", 26.461556145380438], ["mergesort", 26.429927212291684], ["sitars", 26.40949591419931], ["thip", 26.402266377951243], ["crafted", 26.39705723022243], ["openness", 26.379061582800265], ["haikus", 26.368782210065447], ["substr", 26.362503928586857], ["doctype", 26.343359045559563], ["masha's", 26.338646458968228], ["reminders", 26.334604232674764], ["sacrificing", 26.327993014353073], ["immersed", 26.313025207548996], ["marvels", 26.30851070787101], ["knitters", 26.278581770751913], ["longboarder", 26.271284191809738], ["infusing", 26.260430197813477], ["elysia", 26.259594706090017], ["breaktime", 26.237732104719306], ["nogizaka46", 26.209614510608308], ["flickering", 26.202192477157677], ["megacorp", 26.179717611927007], ["dove", 26.17774108054035], ["sneakerhead", 26.170337854857543], ["journaling", 26.163397145102977], ["forums", 26.151533807416907], ["primality", 26.13599199793806], ["overline", 26.111948407046217], ["transformed", 26.069931826163717], ["skyscrapers", 26.04765762498341], ["noa's", 26.033560029299675], ["shyly", 26.033488558599124], ["tinkerer", 26.023117365830245], ["pored", 26.00524587088002], ["graciously", 25.988071787226822], ["plath's", 25.987062696167353], ["standups", 25.982825319169418], ["interconnected", 25.950600694699354], ["festered", 25.93698637344859], ["pytorch", 25.902395585941836], ["tom\u00e1\u0161", 25.884323233492978], ["bashfully", 25.871224079002108], ["datetime", 25.857826686654285], ["amal's", 25.857208083293386], ["imperceptibly", 25.83846226834314], ["quirkiness", 25.821350703927024], ["pastries", 25.762147746683745], ["jamal", 25.753696648318837], ["stricter", 25.73783117113716], ["humility", 25.732163822555076], ["basslines", 25.717937336368724], ["beekeeping", 25.715742022448534], ["guildmates", 25.663152613295626], ["solidified", 25.642423258619317], ["despre", 25.629738790300344], ["embodied", 25.618434115850913], ["hyperconnected", 25.612148029291212], ["vytautas", 25.612148029291212], ["dedication", 25.611757493179887], ["browsed", 25.598848669948215], ["mulling", 25.586095403883903], ["adapting", 25.576536201631818], ["tensions", 25.53493064166663], ["geeking", 25.508860944232616], ["cozy", 25.49139510312051], ["prowess", 25.482461518029616], ["respites", 25.47003980999604], ["caf\u00e9", 25.434711556706173], ["quizmaster", 25.432062613460264], ["realizations", 25.419003787984803], ["advocating", 25.395745622237524], ["bharatanatyam", 25.394950994053502], ["arrabbiata", 25.391482618408013], ["upcycling", 25.332383809997367], ["brimmed", 25.30349254082459], ["substrings", 25.27698906093916], ["howled", 25.263638357741982], ["erupted", 25.248759344847567], ["frida's", 25.24252827149003], ["gnarled", 25.235901196595968], ["massaman", 25.229944275342874], ["expanse", 25.2279151519135], ["mindful", 25.224093421988602], ["dappled", 25.191540368733662], ["dplyr", 25.17935015749995], ["vinegared", 25.17935015749995], ["i\u00e7in", 25.159596932395157], ["gamified", 25.156376297867197], ["sunbaked", 25.12755454669156], ["either's", 25.12584142114012], ["overzealousness", 25.12584142114012], ["esteemed", 25.098207928660912], ["trinkets", 25.072944912885088], ["orderliness", 25.040659255805533], ["journey", 25.020392635018123], ["rustling", 24.992249487100985], ["compassion", 24.959914989128908], ["eerily", 24.955604741946164], ["tashi", 24.94723495500349], ["hues", 24.925039519999974], ["clockmaking", 24.922535226605227], ["mwangi", 24.9218798016157], ["thingvellir", 24.903663841906887], ["nali", 24.899753397714075], ["awakenings", 24.870643786027948], ["cubing", 24.847520231046435], ["oauth2", 24.811768403375865], ["bioluminescent", 24.794171404508223], ["mpho", 24.783379652342255], ["starkly", 24.774174814885903], ["steeled", 24.77238464400543], ["narratives", 24.764217594150725], ["furballs", 24.725616429753032], ["whiskers", 24.701296353678135], ["giuseppe's", 24.69689910521209], ["melody's", 24.694573815775577], ["eagerness", 24.688334542930857], ["highlighting", 24.659345680812393], ["forkful", 24.652309737544403], ["experiences", 24.616978548333023], ["eilish's", 24.610046709039477], ["aspiring", 24.599535508355256], ["sunny", 24.561848154970924], ["greener", 24.537270390867253], ["hobbies", 24.507624145528034], ["handcrafted", 24.505054661120113], ["underscores", 24.49565693092901], ["fervently", 24.490820155372], ["wanly", 24.488938702960194], ["resorting", 24.478092897261497], ["tencel", 24.46393987435658], ["reevaluate", 24.41646755916591], ["abernathy", 24.40355421884732], ["trello", 24.401931157427246], ["berbere", 24.400977535770306], ["sprintf", 24.387635610155765], ["kaarina", 24.360645341496305], ["combatting", 24.333006111648757], ["algorithm's", 24.329986492630706], ["breathtaking", 24.325195909937804], ["meaningfully", 24.312854301877486], ["flavorful", 24.308110711426995], ["serenity's", 24.292150216917733], ["jokester", 24.291902779686087], ["paddleboarding", 24.288992083693714], ["oversights", 24.272754124268047], ["subcultures", 24.224960513116077], ["emre", 24.21892083975538], ["smirked", 24.21858216648282], ["mulled", 24.204282628307958], ["swelled", 24.200496654529058], ["trattoria", 24.17470948491081], ["meaningful", 24.156414085764958], ["tteokbokki", 24.107888448670167], ["raced", 24.09675574578758], ["collaborate", 24.088857620265163], ["tucked", 24.079612983223495], ["anna's", 24.066775872707492], ["jahan's", 24.042411243581267], ["shodo", 24.042411243581267], ["grunted", 23.97388407771213], ["d\u00fcrer's", 23.973532984342896], ["channelize", 23.963502816935666], ["artforms", 23.94910758140419], ["curtly", 23.93907861815368], ["mesmerizingly", 23.925633817541012], ["calloused", 23.914856505010587], ["jack's", 23.88944612241268], ["oddball", 23.871078749126138], ["wipeouts", 23.86203564152416], ["gastronomically", 23.843933465655528], ["naysayers", 23.841564638884886], ["feijoada", 23.808056089323443], ["discontentment", 23.80366172695346], ["cubers", 23.77794471990187], ["ensnared", 23.761126535577414], ["overfitting", 23.730116577961795], ["flowed", 23.724804250190594], ["flavors", 23.719928668475468], ["overfishing", 23.71795382446648], ["detours", 23.713123738375128], ["flashiness", 23.710311394814788], ["weathered", 23.704917174601157], ["lackadaisical", 23.690754770693218], ["seitan", 23.678723251828515], ["transforming", 23.669150388647694], ["tom's", 23.653885877884992], ["meanwhile", 23.64319162229833], ["tiktok", 23.6357939634242], ["anticipation", 23.635239457993972], ["euskara", 23.630255622262727], ["peered", 23.59561170264147], ["recounted", 23.59080954681734], ["patagonia's", 23.579395800559183], ["impactful", 23.54397409679986], ["unengaging", 23.539370023715566], ["cunningness", 23.536397130305144], ["hyperparameter", 23.528401072166766], ["pe\u00f1arol", 23.482566524623586], ["vigilance", 23.418887059903593], ["fibonacci's", 23.409364102315493], ["ignited", 23.401876151748745], ["eateries", 23.398679117164555], ["cornstalks", 23.395571576699158], ["forebearers", 23.385804702032424], ["sethe", 23.37943167990352], ["aling", 23.366213200744404], ["regally", 23.354849618124256], ["stakeouts", 23.30802486377733], ["lepidopterist", 23.291047114136703], ["ingenuity", 23.25957336237644], ["sceneries", 23.258335946816683], ["craftswoman", 23.228038505288044], ["sharing", 23.220937711937538], ["rockhound", 23.219121107213226], ["ggplot2", 23.209094367751113], ["recognizing", 23.205743496111513], ["venturing", 23.198353934423363], ["checkups", 23.18844525783585], ["brewed", 23.16556915930791], ["bonded", 23.14053463735392], ["admired", 23.12953050562698], ["unconvinced", 23.127900187990885], ["skillfully", 23.117612251555766], ["astrophotographer", 23.084366805672484], ["integrating", 23.070821813743226], ["kintetsu", 23.065755071286453], ["gems", 23.06368660634443], ["kalliope", 23.055684338296754], ["confessed", 23.0386741529484], ["preprocessing", 23.01734586706029], ["mateo's", 23.010914245066324], ["underscoring", 23.009849101077464], ["revelation", 22.992238720461632], ["farid's", 22.98454514886369], ["poster's", 22.968169512722554], ["rajan's", 22.950885773122675], ["visualizations", 22.93399060008536], ["relentless", 22.931357992434833], ["mirroring", 22.930209299453196], ["steeling", 22.876394773707375], ["func", 22.864619690925508], ["thandi", 22.859859240024054], ["boasted", 22.82159553415328], ["inclusiveness", 22.82134029927213], ["samir", 22.81907325347974], ["codebase", 22.817763271415068], ["proactive", 22.817172546513042], ["astrobiologist", 22.7990143293557], ["geometria", 22.793586284440988], ["jaida", 22.790585022201988], ["endeavors", 22.747286812300718], ["kadek", 22.730781375995956], ["rekindling", 22.702768089088917], ["rustled", 22.698975897411543], ["moshe's", 22.698678237154294], ["codey", 22.69217009755984], ["attendee", 22.685045397372225], ["scowled", 22.67323026956109], ["symbolized", 22.668774564581465], ["chatted", 22.66085631813727], ["quicksort", 22.657062199924802], ["bridging", 22.65676874317255], ["conversations", 22.636086461504842], ["compassionate", 22.62903170693241], ["underrepresented", 22.61899403709966], ["j\u00f6kuls\u00e1rl\u00f3n", 22.596431938788736], ["steeped", 22.59421660232415], ["protectively", 22.591115131273725], ["trickled", 22.568462499058846], ["gently", 22.560100420769505], ["rockhounds", 22.556153093978057], ["hoarsely", 22.553897478668663], ["biscuit's", 22.516679826063598], ["agroforestry", 22.510262989407554], ["rania's", 22.50086840502223], ["alocasia", 22.474427901608575], ["kenji", 22.458216392949744], ["benny's", 22.452331548195026], ["beloved", 22.43989746168018], ["palates", 22.431323101428053], ["scrutinizing", 22.425739937446505], ["collaborations", 22.411912430771835], ["remorseful", 22.405400915008258], ["reminder", 22.398729942176338], ["compassionately", 22.396380342895718], ["tagine", 22.370965742392887], ["punctuating", 22.353780131639564], ["melodious", 22.340385682144873], ["sonra", 22.311357634043414], ["gatsby's", 22.26977840518687], ["harmonia", 22.264802783180844], ["unraveling", 22.256926393689525], ["bakso", 22.22931903894287], ["crestfallen", 22.227922176541554], ["imbuing", 22.221987862956166], ["amarjeet", 22.220666006820792], ["tireless", 22.2054049133667], ["prompting", 22.170018948371045], ["glimmered", 22.15068750210746], ["surged", 22.149465653693635], ["shooto", 22.145083014227993], ["instilled", 22.143107139411484], ["isabelle's", 22.13756646038641], ["dissecting", 22.10543138713576], ["kallisto", 22.094289006815654], ["numpy", 22.080337564381846], ["serenity", 22.052079462710388], ["risottos", 22.04392172889569], ["glinting", 22.03720916637674], ["passion", 22.027075967005707], ["subarrays", 22.01436314221094], ["recalculations", 22.006959801255114], ["seeped", 22.002502063489505], ["hastiness", 21.994711728756773], ["specializing", 21.993258724097323], ["rhythmically", 21.985983668546947], ["momentarily", 21.97859328237593], ["whimsy", 21.978364197512967], ["pandemic's", 21.96571261592981], ["hobbyist", 21.94040649540455], ["reevaluating", 21.91706208964868], ["revitalized", 21.90115710990293], ["observing", 21.889321767323437], ["diligent", 21.879601184412213], ["eatery", 21.877972238930262], ["positively", 21.87640191075741], ["reprimanding", 21.86404549049617], ["dataset", 21.858704192002115], ["laoshi", 21.83050940917179], ["reenactors", 21.828595585391202], ["kinkaku", 21.820278595876648], ["debugging", 21.788633477423307], ["microplastics", 21.78783412921443], ["setback", 21.778688765860625], ["siti", 21.772002012595784], ["nwoye", 21.771110510805293], ["dampen", 21.769830233797894], ["smoggy", 21.766874768467382], ["heatmaps", 21.765680764568565], ["teletherapy", 21.761988537127582], ["oenophile", 21.7552462087571], ["lila", 21.754604082245624], ["dmitri's", 21.748587119008477], ["masterpieces", 21.745946445487466], ["heartrending", 21.744630980818332], ["adira", 21.73928306370549], ["mutta", 21.732293601538434], ["bettas", 21.732293601538434], ["apologized", 21.71407432004105], ["wallstreetbets", 21.68332803947195], ["patel", 21.682996378011655], ["ada's", 21.675378026900006], ["pirouetted", 21.674648260420298], ["reverberate", 21.65622042181832], ["ganesha's", 21.64948419510247], ["jovana", 21.63177909851282], ["mischievous", 21.619672808785523], ["vibrancy", 21.619130192393925], ["exhilarating", 21.613493862592225], ["sportsmanship", 21.609347510920543], ["patiently", 21.60431330911165], ["discovered", 21.599930095865044], ["hallowed", 21.589716796227197], ["reinforcing", 21.581807838007826], ["neglecting", 21.573914784908837], ["abuelita", 21.545130255594135], ["unity", 21.543714727137242], ["yodelers", 21.541126204749048], ["storeowner", 21.53988685567795], ["painstakingly", 21.5310215696925], ["playtimes", 21.530211937122928], ["categorizing", 21.522600257683386], ["henri's", 21.52252561921226], ["felines", 21.51429360627294], ["adhering", 21.498937844332534], ["lightbulb", 21.498696014231136], ["unfurl", 21.498667563630974], ["absurdity", 21.47213500809432], ["unicycling", 21.471722656767575], ["practicality", 21.466804669609353], ["morphed", 21.45775444577315], ["reconvened", 21.45597672681928], ["thida", 21.445830829274488], ["intersected", 21.406820078127645], ["infty", 21.4035584578572], ["harmonizers", 21.400150247911686], ["heartfelt", 21.39199498538866], ["ensured", 21.386028275981808], ["other's", 21.377147374219692], ["carpooling", 21.345418405845415], ["paves", 21.345187469802937], ["fidgeting", 21.321299289002653], ["namgyal", 21.312672859310034], ["eeva", 21.31216873950827], ["bitingly", 21.29254819106031], ["punny", 21.280862899818533], ["repurposing", 21.276091405894807], ["stormiest", 21.275511691679775], ["palindromic", 21.273394474546613], ["gullfoss", 21.267230060036457], ["drizzled", 21.26391740737913], ["proactively", 21.24390367305684], ["bibimbap", 21.237191599499685], ["soundwaves", 21.232682902693966], ["izakaya", 21.22625273814814], ["blending", 21.222633323062077], ["thompsons", 21.215859939118495], ["voivodeship", 21.212530394244183], ["overexertion", 21.1912756669649], ["pothos", 21.164812494839463], ["windborne", 21.16297893229118], ["somber", 21.15424790034251], ["scrumptious", 21.14551045473386], ["br\u00f8ndby", 21.11465059492582], ["authenticity", 21.113069359726893], ["breathability", 21.11001263351737], ["attentiveness", 21.102409027071175], ["customerid", 21.10060646269999], ["uncover", 21.090413933737093], ["landscapes", 21.05526666710476], ["obstacles", 21.055024760535343], ["eccentricities", 21.05303196478499], ["thunderous", 21.041559112879874], ["tenzin's", 21.039709017727507], ["technophiles", 21.017027353447794], ["philatelists", 21.01404874979793], ["mbsr", 20.9888071085959], ["samnang", 20.9856545841637], ["sentence's", 20.964373935763785], ["helplessly", 20.962208773121514], ["mzee", 20.960036736947043], ["farzana", 20.95698600372666], ["gaudi's", 20.953479058353302], ["quetzals", 20.950334247883596], ["amazement", 20.94609488612991], ["coexist", 20.94113253244923], ["offbeat", 20.940458528553723], ["escapades", 20.926376372823242], ["binoculars", 20.912776225702515], ["tending", 20.881329808605194], ["kente", 20.873058139002957], ["breathtakingly", 20.865171584521743], ["apiarist", 20.846417288681092], ["mutual", 20.846384295963812], ["lastly", 20.844417271938116], ["nyima", 20.83476246312184], ["chirped", 20.83034510143881], ["thornfield", 20.79733823372737], ["constructively", 20.789628500506893], ["synthwave", 20.783053430662775], ["welling", 20.775720743093395], ["pitfalls", 20.774137013195304], ["chinwe", 20.772643314454214], ["sparky's", 20.762625643102723], ["encountered", 20.759575114088875], ["sulla's", 20.75376889815084], ["striving", 20.749233918595277], ["lively", 20.744728879521404], ["didn", 20.73245919818494], ["innovative", 20.73033096041896], ["clung", 20.71457900315349], ["meera", 20.704557165126044], ["treasures", 20.70346451534783], ["sustenance", 20.6912879290179], ["abounded", 20.67003741308472], ["stumble", 20.654617929251316], ["intricately", 20.637199920610485], ["bakeshop", 20.628804565919125], ["olivia's", 20.61775498292608], ["sleek", 20.593767774801965], ["andrena", 20.586183391447225], ["sundews", 20.585588070932722], ["reassess", 20.57882566387759], ["blanketing", 20.578679537259376], ["disquietude", 20.575612822311694], ["disappointedly", 20.566408790630945], ["raoni", 20.558322391368574], ["sniffled", 20.548337776316913], ["vondelpark", 20.543077065160663], ["claudette", 20.53516711575882], ["gazed", 20.522929441934345], ["maria", 20.520805314291582], ["virtuosos", 20.510833410801734], ["lucy's", 20.50746597165979], ["craftspeople", 20.503238349546407], ["birdfeeder", 20.49667824626702], ["intersectionality", 20.496104431086554], ["upbringings", 20.49083350340716], ["implementing", 20.47474453153854], ["individuality", 20.45871637716334], ["jabali", 20.457049867273163], ["flashcards", 20.436359781705285], ["pang", 20.436049557044075], ["muttered", 20.399971713899863], ["calmly", 20.393154716223712], ["replied", 20.36422644093409], ["panted", 20.35089817339172], ["garnering", 20.32401705425993], ["kindness", 20.32145861657492], ["dwindled", 20.31612647211515], ["heatwaves", 20.304297143429253], ["negatron", 20.300537784580253], ["lightheartedly", 20.297956989031743], ["fluffy's", 20.293068859714698], ["wouldn", 20.29177984875137], ["whispering", 20.291585537048793], ["astrophotographers", 20.27032865097225], ["glimmers", 20.265612128176677], ["strumming", 20.263960239022452], ["wisdom", 20.260421666302612], ["wargamer", 20.249604535658367], ["resiliency", 20.240322233651035], ["burgeoning", 20.23098964587382], ["delicacies", 20.225411537241317], ["leibniz's", 20.21699647682478], ["quirkier", 20.20853242327587], ["newtonville", 20.19255620061972], ["utzon", 20.17870671077019], ["teashop", 20.16424682932963], ["anjali's", 20.15956182774289], ["mirage's", 20.125101038293757], ["frantically", 20.111499296233042], ["emma's", 20.07651184651374], ["amaranta", 20.068342090965775], ["gamepro", 20.06834209096577], ["cemre", 20.0605787091124], ["collaborating", 20.05272027748881], ["funmi", 20.04027447964974], ["ethan", 20.036369664263056], ["clara", 20.035797895810045], ["crucial", 20.029332519761823], ["nisha's", 19.993411592898855], ["embraced", 19.987822007299595], ["woven", 19.98030876262484], ["everlane", 19.976654560666724], ["naima", 19.975576256955147], ["doctora", 19.97347356471757], ["dup\u0103", 19.962195488170586], ["backgrounds", 19.953366783191335], ["tactfully", 19.952345639657928], ["preetam", 19.943636628029967], ["twirled", 19.937456481551383], ["teaware", 19.933049897094094], ["kindred", 19.925081817831554], ["candidly", 19.903353349664552], ["persistence", 19.886938693434196], ["graced", 19.87227100884505], ["petunia's", 19.849414722700693], ["bhature", 19.849414722700693], ["transcend", 19.84646094074791], ["darted", 19.846150016331826], ["unappetizing", 19.837008838499006], ["marek's", 19.83096731124837], ["grandeur", 19.826247107625253], ["oneself", 19.821108339029237], ["mastering", 19.809752788078196], ["subqueries", 19.804574177393388], ["ecosystems", 19.80293950696023], ["attendees", 19.79245249383097], ["exacerbating", 19.77952241733907], ["martha", 19.776668176596555], ["svetlana's", 19.7366339572308], ["divisibility", 19.733793131904108], ["paintbrushes", 19.729672181584515], ["carvings", 19.71915293858297], ["tsering", 19.712234272393566], ["fluffers", 19.697505936557572], ["panicked", 19.68383818056661], ["bonaventure's", 19.66562384563865], ["homecooked", 19.653532340568777], ["niches", 19.65114210912014], ["curating", 19.637208808724452], ["timeless", 19.618130981831467], ["morsel", 19.613415897270123], ["jiaozi", 19.58183682815448], ["pythonic", 19.57372840710763], ["baatar", 19.563125087277122], ["deadlines", 19.548055118968023], ["recursions", 19.539267617658496], ["kiyomizu", 19.52102367030307], ["homestays", 19.511786228604123], ["galician", 19.511072426291022], ["livelihoods", 19.503822234225098], ["flawlessly", 19.503481413230546], ["thrain", 19.49496088836675], ["origami", 19.49190876005733], ["unimagined", 19.451422483860416], ["pintxos", 19.45120115573293], ["experimenting", 19.44905963961377], ["rekindled", 19.440965463971903], ["fiona's", 19.438737176713783], ["ancestor's", 19.43029701318421], ["sa\u00ebns", 19.393688364271338], ["philatelist", 19.38883698587156], ["nuanced", 19.38317160215374], ["incredulously", 19.371115572756096], ["pattering", 19.35252555309422], ["differing", 19.35226491599423], ["paths", 19.347744395840056], ["tantalizingly", 19.328367586229803], ["emberiza", 19.328007995383373], ["pirouetting", 19.315490592501465], ["reassurances", 19.30939388098016], ["parchments", 19.287869164739913], ["pristine", 19.277816506679], ["headbanging", 19.27766527688377], ["cochinita", 19.266714101106256], ["strengthened", 19.264312714739454], ["linda's", 19.262954742257264], ["rina's", 19.260276842803183], ["mirrored", 19.251676665710274], ["picado", 19.25050996813645], ["championing", 19.25004811754492], ["palpable", 19.24352734588277], ["encapsulated", 19.229883482124396], ["tinseltown", 19.228408349344097], ["jada's", 19.213215532870542], ["\u043a\u043e\u0433\u0434\u0430", 19.206700239962643], ["numbeo", 19.202151199134363], ["cheerfully", 19.17131298937283], ["fisherwoman", 19.167091091607855], ["enveloped", 19.16351247116506], ["vying", 19.162021379313895], ["renn", 19.159000297563278], ["aisha", 19.128691926221684], ["subgenres", 19.12793776912735], ["tribespeople", 19.113867517423163], ["analyzing", 19.10941461242457], ["birdcalls", 19.094273945206645], ["vehemently", 19.09416461015199], ["crept", 19.07690647241119], ["zemun", 19.06830349518701], ["grandmother's", 19.060240411548158], ["hab\u00eda", 19.03368535053491], ["trove", 19.032881980698686], ["donning", 19.010325827604706], ["unsuspecting", 19.007972142064467], ["mishap", 19.004574008565747], ["rfk's", 18.986396691278923], ["milo\u0161", 18.984220493072886], ["fantastical", 18.97882635766996], ["insignificant", 18.978075186293054], ["aimlessly", 18.974809270660366], ["whirring", 18.960766279884748], ["renowned", 18.92713517349626], ["monteux", 18.920389604400775], ["underestimate", 18.91542886923076], ["susan's", 18.912857830854957], ["mit\u00e4", 18.90827867981402], ["unwaveringly", 18.904204497810184], ["breakthroughs", 18.896820042928226], ["shrugged", 18.88940798260746], ["diversifying", 18.88401137027843], ["chatbots", 18.862279850738428], ["unique", 18.848842635138578], ["footfall", 18.829753007493466], ["geekdom", 18.829643011623435], ["juggled", 18.800275132804863], ["rectifying", 18.79664446972877], ["demotivated", 18.784309289394646], ["importance", 18.780150538077525], ["soared", 18.77819359119842], ["bakr's", 18.77064218342348], ["lacework", 18.742222906851122], ["mentees", 18.73439193210364], ["unfold", 18.730685519690322], ["spontaneity", 18.69643847909749], ["enrich", 18.695667491943716], ["rumi's", 18.694978276154735], ["knit", 18.689450976665405], ["cemil", 18.682545478603053], ["botticelli's", 18.674120429909202], ["yesteryear's", 18.652818799701002], ["mycologist", 18.651409429082317], ["revitalizing", 18.6490813840123], ["decomposers", 18.641473366220552], ["rechecked", 18.641302439185534], ["widened", 18.63984101303612], ["quietness", 18.639621050550275], ["gratitude", 18.619413054675167], ["shimmering", 18.61148565605852], ["unshaken", 18.608826302531902], ["innocuously", 18.6088263025319], ["recounting", 18.599518565188728], ["haphazardly", 18.58022484039544], ["exuding", 18.570536536477306], ["reverence", 18.562491072381015], ["crosschecked", 18.54679688152346], ["keycaps", 18.546485175890254], ["skyhigh", 18.530308048090838], ["boundlessness", 18.510366904105805], ["turmoil", 18.473523456706847], ["sourdough", 18.470983144735367], ["cityville", 18.470983144735364], ["dwindling", 18.462578074138015], ["proving", 18.459955692111645], ["homeworkhelp", 18.44977650507436], ["hinduism's", 18.44977650507436], ["ahmed's", 18.449561572915634], ["enchant\u00e9", 18.441179218725303], ["persisted", 18.43718324713868], ["mkhize", 18.41958400115022], ["puzzles", 18.387966617589175], ["functionalities", 18.387457365104012], ["rnns", 18.38462357599537], ["guiltily", 18.374458213079357], ["aiden's", 18.366253147750292], ["giacomo's", 18.356674184611414], ["windowsills", 18.35165623201598], ["deeply", 18.345144872863298], ["imparting", 18.338441646854083], ["richness", 18.331398012966684], ["diehard", 18.330366053422804], ["salla", 18.319254066865962], ["substring", 18.315839833531175], ["fibonacci", 18.315135982655207], ["brunelleschi's", 18.313448107253613], ["colorful", 18.311360331103156], ["memrise", 18.30905586643163], ["maintaining", 18.307948762651638], ["spectatorship", 18.307294394851986], ["szigeti", 18.300878822348153], ["rajesh", 18.28874566031753], ["sensing", 18.273839040335293], ["hana's", 18.257407975343114], ["refining", 18.25547582643713], ["budding", 18.244444049278137], ["palindrome", 18.240002718157395], ["ephemerality", 18.225928641985977], ["coders", 18.220860469315475], ["immortalize", 18.220468188392104], ["se\u00f1or", 18.206847931750946], ["voila", 18.187521891922287], ["crestwood", 18.184625100796765], ["busied", 18.17683569193741], ["imbued", 18.172407743899335], ["extroverted", 18.16890133084084], ["mathlete", 18.168617508278455], ["souffl\u00e9s", 18.167260932641312], ["fearmongering", 18.15770324065234], ["herpetologist", 18.15597993523769], ["valuing", 18.146707116019023], ["crunchiness", 18.141938187414613], ["outcompeting", 18.138906446407354], ["wanjiru", 18.119120347202113], ["rabbit", 18.117132374216787], ["haven's", 18.103190379271368], ["appreciated", 18.08991811161302], ["noura", 18.08583314500976], ["poipu", 18.08568660728], ["posthaste", 18.075403763147282], ["detailing", 18.066775438851433], ["foreach", 18.066068868708054], ["revealing", 18.058739025550302], ["hobby", 18.056899766578507], ["melodies", 18.056238782497392], ["lena's", 18.05409164310683], ["malala's", 18.05125847324255], ["demeanors", 18.05125847324255], ["ergs", 18.042822274708147], ["munched", 18.024549175613696], ["trio's", 18.01893867137321], ["filled", 17.983356783469155], ["overhear", 17.981593678464453], ["concoction", 17.978930014892015], ["simmering", 17.97653745302598], ["astrophotography", 17.970552550126214], ["balancing", 17.968084993287828], ["strategize", 17.96653584386295], ["naila", 17.941843280935775], ["acclimatizing", 17.939881566166374], ["harjeet", 17.932141709712557], ["craved", 17.929158843119524], ["first_name", 17.92850362050385], ["resonate", 17.927703240877936], ["godric's", 17.926205879950984], ["aminata", 17.92307390947377], ["microplastic", 17.922997323323106], ["rivian", 17.919610513549237], ["regale", 17.91516396007193], ["onwards", 17.914462562692954], ["soundscapes", 17.89652673114312], ["companionship", 17.884482741078507], ["katya's", 17.873354420328923], ["encounter", 17.86522305661864], ["warmth", 17.863490216817176], ["handpicking", 17.862902885341803], ["endeavor", 17.860372131686262], ["captivatingly", 17.85441442540223], ["pomodoro", 17.844535222249338], ["chalkboards", 17.84066009809302], ["bessie's", 17.840162007614634], ["thunderclaps", 17.833458539926404], ["jalebis", 17.824546266793007], ["hanok", 17.824546266793007], ["mailto", 17.824546266793007], ["fascinations", 17.814608432245556], ["chatuchak", 17.80175205149272], ["tokenizing", 17.796026992766137], ["exhilaration", 17.79108632980092], ["delightful", 17.775167734275858], ["upholding", 17.76807617724183], ["entranced", 17.76391716265675], ["cradling", 17.75537185745124], ["gloomiest", 17.74512550368021], ["plov", 17.735819636487193], ["gulnara", 17.7319706128838], ["unraveled", 17.72468863970738], ["mirthlessly", 17.709849186467554], ["nanjing's", 17.707544116939186], ["grandfather's", 17.704340560735076], ["axolotl", 17.699061461074784], ["optimizing", 17.699061461074784], ["yaml", 17.693967743088134], ["glistening", 17.693034310855342], ["gelateria", 17.688772689488058], ["paddleboards", 17.686767341595633], ["kapurthala", 17.681283558480466], ["milkweeds", 17.678384987405305], ["inputted", 17.668986590282813], ["intrigue", 17.664570574866307], ["reenactment", 17.649225857871425], ["heikkinen", 17.64392419795617], ["intervened", 17.642939603971907], ["nfts", 17.63612844095997], ["nerdier", 17.622717558295168], ["tales", 17.618817051357517], ["lettera", 17.61807860586932], ["yara's", 17.616355566396866], ["grapevines", 17.616355566396862], ["elated", 17.614228083575565], ["hiccups", 17.614140229932275], ["rangoli", 17.61354061698319], ["swiveled", 17.608351770137713], ["amusedly", 17.59379941330289], ["pedro's", 17.584545606905355], ["nikola's", 17.583992390218548], ["espressos", 17.57997292522845], ["riddles", 17.57868324787865], ["khwarizmi", 17.570056094256596], ["uniqueness", 17.56504727988371], ["gatherings", 17.553560350067333], ["immersive", 17.543871748589254], ["ujjayi", 17.514189461206495], ["vistas", 17.510545595877744], ["pawsitive", 17.50736929397082], ["jukka", 17.503574800926973], ["apiary", 17.465538934925366], ["jamies", 17.46459426451214], ["permeated", 17.46406621544504], ["berrington", 17.458462494739017], ["listened", 17.450867197508916], ["unpredictability", 17.44944003350143], ["multifaceted", 17.44491450421189], ["celie", 17.434823136552957], ["marginalized", 17.431683500198222], ["donatello's", 17.408256863658874], ["playdates", 17.407465974700457], ["respectfulness", 17.396757156389974], ["durga's", 17.39256314550367], ["individualities", 17.3919812014094], ["amir's", 17.391027107073437], ["concertgoers", 17.389750976354474], ["gl\u00fchwein", 17.38703467660809], ["nyoman", 17.36140268445584], ["lurking", 17.360918776639394], ["journeys", 17.35038768926715], ["fearlessly", 17.350108197725984], ["struggles", 17.343839643433117], ["marigolds", 17.341558561499262], ["factoid", 17.33304388462782], ["marisol's", 17.325458971322803], ["brow", 17.303587499903603], ["carlos's", 17.29457271301435], ["stubbornness", 17.291159903595226], ["prakash's", 17.283411323719083], ["carla's", 17.28139669295129], ["revelries", 17.28134842102833], ["oversimplifying", 17.279127154090602], ["treacherous", 17.278013269987195], ["adinkra", 17.27401597703383], ["rectify", 17.272679044593346], ["nasrid", 17.27093682980085], ["punctuations", 17.268492079736472], ["horizons", 17.263523808931907], ["misconceptions", 17.259579448320736], ["filepath", 17.254160786255632], ["trembling", 17.252849819626977], ["epazote", 17.250086604251795], ["collective", 17.246598187765553], ["jellybeans", 17.237118781044455], ["isabela", 17.235696384481823], ["wildflowers", 17.22547973352683], ["nimbly", 17.211295973787536], ["customizability", 17.206150953094824], ["seeping", 17.194555503539473], ["reginald's", 17.190522736774188], ["mesmerized", 17.182689005607426], ["trailed", 17.175570741352043], ["generations", 17.17419712946491], ["kepi", 17.1608772818738], ["initially", 17.156976514501352], ["brushstrokes", 17.153895380449125], ["seeta", 17.15175936609204], ["composters", 17.149310514098023], ["poignant", 17.141463655020424], ["recyclables", 17.13723176274547], ["gerhard's", 17.13193532614048], ["larpers", 17.11308139878229], ["p\u00e4rnu", 17.111564416121286], ["plummeted", 17.10577637157709], ["uncharted", 17.10335548994869], ["rheumy", 17.09756404523537], ["kokum", 17.09255156677004], ["dawning", 17.086742940995794], ["whizzes", 17.083793576081312], ["precipice", 17.066181571365487], ["adho", 17.05809077732091], ["ajaw", 17.05809077732091], ["grep", 17.050878053947834], ["sprawling", 17.047285652371052], ["bot's", 17.044836627532778], ["microcosms", 17.041416201781004], ["relishing", 17.030609388266534], ["sara's", 17.030609388266534], ["sensationalized", 17.029895585953433], ["cultivating", 17.017692128766694], ["toate", 17.01617900636926], ["ichiran", 17.013784048029166], ["playfully", 17.005039298826848], ["impermanence", 16.98240894786085], ["debates", 16.982291474371326], ["hopefulness", 16.97997226747641], ["morsels", 16.975915048634512], ["disheveled", 16.967817412452717], ["cakebread", 16.96687104054379], ["simit", 16.964672974597352], ["singalongs", 16.962366362184426], ["pibil", 16.95994295926958], ["pixelated", 16.954708408973506], ["heated", 16.949665366615097], ["embrace", 16.93865205769638], ["chattered", 16.934347874630046], ["tattered", 16.92595709943311], ["emerged", 16.920513799415293], ["sustainable", 16.918824722411635], ["peacefulness", 16.908760689708], ["bratwursts", 16.908760689707997], ["ipay", 16.906057882692377], ["const", 16.902120845883502], ["overjoyed", 16.89886198491446], ["pippin's", 16.891940542476874], ["underbrush", 16.881424704828515], ["earthiness", 16.881332438959607], ["greta's", 16.872002514295588], ["jakub", 16.864418130940848], ["habitats", 16.86397281578317], ["shone", 16.85740735641125], ["traditionalists", 16.850582282580444], ["whakapapa", 16.848531922562664], ["misinformation", 16.84465621041015], ["chiar", 16.843761477088496], ["misl", 16.842425571827988], ["sleuthing", 16.84098780379137], ["misconfigured", 16.839926463888183], ["neared", 16.83041053257895], ["beneath", 16.828296726849413], ["stubbornly", 16.816533897432272], ["picturesque", 16.812721232192587], ["g\u00e0idhlig", 16.811712235932244], ["pixel's", 16.801346485067434], ["ioexception", 16.784431566989557], ["seema", 16.782404461727843], ["firstly", 16.775897340591637], ["aethelred", 16.767770427852184], ["clicked", 16.765031667046873], ["ai's", 16.76384865202446], ["dal\u00ed's", 16.75535754729964], ["rashed", 16.749506126208395], ["significance", 16.734223257300684], ["weightlessly", 16.729146878033728], ["melded", 16.72245212939853], ["introspective", 16.7010700543882], ["scurried", 16.69832013547196], ["organically", 16.68765339391537], ["tuuli", 16.662805251286734], ["reconnecting", 16.660666094980403], ["comprehending", 16.654975743985965], ["args", 16.631888626520983], ["alike", 16.626182216225107], ["fyne", 16.62388483026183], ["kuin", 16.617758467693097], ["craftswomen", 16.61309710486906], ["larp", 16.611615274457964], ["norms", 16.60640871512554], ["coding", 16.606024966008626], ["contemplated", 16.600188197517035], ["timepieces", 16.59461251922414], ["hanauma", 16.59419553473642], ["intermittency", 16.589544955863396], ["recalling", 16.57943041187245], ["diverse", 16.5583617981851], ["ebird", 16.5576542134082], ["theodorus", 16.555220683406986], ["slipup", 16.54117893558391], ["donned", 16.538557512614723], ["change's", 16.534158061672375], ["practicing", 16.531697809479585], ["essence", 16.526724727822444], ["momentary", 16.526705404015274], ["shuka", 16.517946942696856], ["mere", 16.515723899110817], ["engaging", 16.51517252647963], ["posed", 16.51403706302707], ["empathize", 16.513850031255554], ["gathered", 16.51349914018767], ["colorwork", 16.513237754949476], ["injera", 16.511746232851554], ["caravaggio's", 16.504583406965363], ["amorosa", 16.501606737173425], ["enveloping", 16.49372473371953], ["sunshine's", 16.489671705802287], ["inspiring", 16.484375023375577], ["spreadsheets", 16.477207525335796], ["cybercriminals", 16.476852128612194], ["locals", 16.466206042058374], ["thriftiness", 16.464599403474725], ["dismissive", 16.464380604811556], ["dijkstra's", 16.46388370691296], ["cotto's", 16.46388370691296], ["javascript's", 16.44928349705289], ["armloads", 16.447194964359003], ["escalated", 16.443886728571886], ["noticed", 16.437099281005036], ["ceaseless", 16.431126737020552], ["admiringly", 16.426719884913577], ["untapped", 16.425333709973046], ["jeanette's", 16.419552619881088], ["heartily", 16.416051936255695], ["patience", 16.415183309957484], ["sam's", 16.409693441800034], ["smallbusiness", 16.406148903456696], ["chamomile", 16.398377390816282], ["accolades", 16.39504656086583], ["yui's", 16.388019871365543], ["accusingly", 16.379299070555955], ["terminologies", 16.358739462029675], ["rami's", 16.350319178634866], ["eiffel's", 16.349476530701317], ["munching", 16.348307481591927], ["amela", 16.34794086390653], ["healthier", 16.347152955112723], ["aromatica", 16.34154401739583], ["boomed", 16.3412464249712], ["equitably", 16.32041021273847], ["addressing", 16.317297674599246], ["numismatists", 16.316891763575995], ["underprivileged", 16.305089392482877], ["seemed", 16.294762915948972], ["detectorist", 16.29206479498777], ["delicate", 16.27530380945754], ["pranav", 16.271034053470103], ["weeklong", 16.261099212571036], ["thrive", 16.25771487756769], ["retraced", 16.257554008808846], ["reforestation", 16.257204582022066], ["doorframe", 16.255186589501385], ["geometridae", 16.252670000660938], ["demetrios", 16.220937255854594], ["direst", 16.218218735653156], ["dusty", 16.21751259775782], ["unchecked", 16.213874756645758], ["stomachaches", 16.211904881831067], ["cryptic", 16.21157162002926], ["size_t", 16.2012916971815], ["innovatively", 16.178285724242016], ["burgerville", 16.17794358733629], ["ioana", 16.17476041485895], ["pausing", 16.17276176838227], ["reverberating", 16.147419795583456], ["commending", 16.145246461061955], ["indexof", 16.137735546911134], ["stewardship", 16.129891489459677], ["scampered", 16.127649462194313], ["learned", 16.111686523624062], ["beadwork", 16.108415533199445], ["contemplative", 16.10526771234591], ["marta's", 16.105093309100333], ["encroaching", 16.10076364503895], ["compelled", 16.099293155447597], ["liisa", 16.097120440668906], ["webinars", 16.09471348643762], ["sophia's", 16.087995951047365], ["leatherbound", 16.058083756390456], ["xlsx", 16.058083756390456], ["frevo", 16.05467367277262], ["chuckling", 16.052406673397094], ["minimizing", 16.0436096014732], ["nagging", 16.040274686365223], ["decided", 16.039033121872336], ["subpar", 16.027736300838438], ["gastronomy", 16.01977220826659], ["averting", 16.01379799325477], ["teetering", 16.011451097770788], ["alvaro's", 16.00011233488724], ["newbridge", 15.996071902479441], ["enki", 15.988690165468254], ["fraught", 15.986673505356949], ["biases", 15.986376713868873], ["hurdles", 15.983927114958622], ["unfamiliar", 15.97205527904125], ["monotonous", 15.95638978129559], ["demonstrating", 15.955261610417816], ["samurais", 15.952457297233622], ["chiming", 15.952072702542862], ["trivializing", 15.948339972557696], ["unexplored", 15.946219535134054], ["artifact's", 15.939205651535763], ["rag\u00f9", 15.938407160812632], ["makefile", 15.931488045330015], ["streetlights", 15.929683224933337], ["liszt's", 15.926011123929964], ["gnawing", 15.922356360636133], ["lackluster", 15.91813350587103], ["majestic", 15.910546488664773], ["uncrackable", 15.908023761016107], ["ripple", 15.900288705034972], ["beatrice's", 15.881216787899325], ["storied", 15.878769901955742], ["looming", 15.877623180591064], ["combining", 15.872969471655884], ["raj's", 15.858924922247539], ["squinting", 15.858900548415953], ["interpretability", 15.858654562028264], ["opencv", 15.856921533572345], ["subforum", 15.847337537279216], ["samira", 15.841057833484433], ["pave", 15.838944626142686], ["tensorflow", 15.838178830821596], ["stared", 15.829658876847237], ["deceitfulness", 15.827435894324283], ["intentioned", 15.825513536987152], ["tokenization", 15.823831889908078], ["replying", 15.82291901081764], ["hanout", 15.820987156085172], ["critters", 15.815761913240543], ["sarita", 15.811299415051272], ["cautioned", 15.810265591367797], ["sourcing", 15.810165543914557], ["bj\u00f6rk's", 15.802307830392124], ["silently", 15.792690588748739], ["needn", 15.789307165784644], ["oakwood", 15.78481706505165], ["gushed", 15.771554637886604], ["terrarium", 15.770099961184966], ["algo", 15.767704089581958], ["thangka", 15.765811172978415], ["tidbits", 15.761168252244246], ["mira", 15.752805004057292], ["horemheb", 15.736077305975844], ["caulfield's", 15.729948868168911], ["reinforced", 15.722623035668793], ["fluttered", 15.721981765865353], ["welcomed", 15.71425601823675], ["gama's", 15.714119988804715], ["tranquil", 15.70872612610833], ["unsung", 15.704964354707528], ["tempers", 15.70323763424183], ["fleeting", 15.699676136384888], ["tale", 15.664814551592878], ["ecotourism", 15.65724047016005], ["allure", 15.654290192654729], ["conjugations", 15.652090567796275], ["hilarity", 15.650839677930152], ["sifting", 15.64882586142741], ["patrons", 15.64850004242852], ["workshops", 15.643999773751695], ["clamored", 15.643229221937927], ["rpgs", 15.638098722698228], ["nanosensors", 15.63343131107016], ["murky", 15.633293773551292], ["abuelo", 15.631414094126795], ["mellotron", 15.623548692039268], ["confronted", 15.621016098697636], ["owino", 15.611232272937446], ["overhunting", 15.602160044044213], ["understanding", 15.601046891950805], ["toru\u0144", 15.586880150838686], ["generously", 15.582426756509873], ["overindulging", 15.574053554269527], ["sagacious", 15.571252436983903], ["worldbuilding", 15.560523327260007], ["vishnu's", 15.559046436283616], ["oryol", 15.556584951322963], ["gracing", 15.5505402920525], ["overdependence", 15.5422032414405], ["wordnet", 15.54204732202515], ["okonomiyaki", 15.540199877454778], ["larping", 15.525577761219328], ["tostring", 15.525492509714722], ["blanketed", 15.524952250977558], ["eeriness", 15.515899249493998], ["eduardo's", 15.507355252109916], ["jayla", 15.507355252109916], ["seashells", 15.501992477959329], ["wordsmiths", 15.499917671653266], ["sugarcoating", 15.496077175562926], ["intergenerational", 15.487718377675215], ["hezekiah's", 15.483312840866335], ["alleyway", 15.483073272355723], ["blockchain's", 15.478369541358308], ["factorial", 15.471749240050691], ["fifi's", 15.471706159576332], ["propelling", 15.46613634998113], ["cautionary", 15.464888420667549], ["tringa", 15.446542094258504], ["millville", 15.444426853985414], ["unafraid", 15.443270275239598], ["hangi", 15.43963754358542], ["indulged", 15.425580131166313], ["duo's", 15.42173717804193], ["touchpoints", 15.410666706173497], ["harmony", 15.3991644015138], ["struck", 15.392494689253985], ["sleepless", 15.382474828357825], ["honed", 15.382168602438338], ["dimmed", 15.377115542519242], ["flytrap", 15.374435064234689], ["explore", 15.368762065366548], ["wandered", 15.366927535104699], ["glancing", 15.365052462737614], ["nutritiously", 15.36357844844797], ["restituta", 15.36357844844797], ["unknowingly", 15.3602156952992], ["resilient", 15.347570604322355], ["subculture", 15.345624091574821], ["unearth", 15.345319213557259], ["velvety", 15.345319213557259], ["sammy's", 15.343369426455421], ["downpour", 15.342188154750167], ["billowed", 15.339996883143401], ["ardeshir", 15.336944754833983], ["hobbyists", 15.332764433253208], ["confidante", 15.33035613303699], ["dockerfile", 15.309389014848938], ["chalkboard", 15.306207214544349], ["insightful", 15.305095894891734], ["focusing", 15.304888434099222], ["somberness", 15.304455276848666], ["glimmer", 15.300590515415118], ["legaladvice", 15.293071797717124], ["compasses", 15.292560816902698], ["nurturance", 15.291975318052835], ["gmina", 15.290973550917684], ["misadventures", 15.290337718554218], ["infused", 15.285600072004746], ["deepfakes", 15.281420274926857], ["backlashes", 15.277616655782364], ["gazing", 15.274553474497996], ["quokkas", 15.273911194551273], ["ingrained", 15.26750815754395], ["routines", 15.26669680224806], ["atomica", 15.263784227207667], ["heathcliff's", 15.261206756044679], ["adventure", 15.26087445557499], ["hashmap", 15.253927656550145], ["lacewings", 15.238277855041343], ["gogh's", 15.234618887364638], ["countertop", 15.224887179325489], ["macondo", 15.213724856803692], ["downcast", 15.213271542946396], ["jamyang", 15.207212892391658], ["glided", 15.202464877661656], ["radiated", 15.186694685879846], ["cnns", 15.177135996219477], ["ribcages", 15.175054782421848], ["dualities", 15.174428981160665], ["rechecking", 15.170624109492671], ["lingering", 15.167922881387154], ["usta\u0161e", 15.162747357618585], ["kelps", 15.162747357618585], ["curiosities", 15.140918103092089], ["verifying", 15.129127075229185], ["disrupting", 15.12804890755256], ["reverentially", 15.124457591563992], ["cuisine's", 15.12336359824815], ["whiz", 15.118304223618694], ["lattes", 15.117133803584096], ["deforestation", 15.115549162449014], ["jarmo", 15.1143813373391], ["rakesh", 15.11194111818905], ["skylar's", 15.10786834865978], ["illuminating", 15.09459157404141], ["enhancing", 15.093163374529535], ["hassan", 15.092500363741504], ["overstocking", 15.089606090216341], ["gardening", 15.086933620830493], ["appreciators", 15.075504852684071], ["painstaking", 15.075117336034657], ["demonstrated", 15.073298773535198], ["calculating", 15.068146932388867], ["ecofriendly", 15.067510266777344], ["h\u00f6lder", 15.06428795919249], ["shizen", 15.06428795919249], ["stepping", 15.05931447083639], ["lovelace's", 15.058176686186732], ["sank", 15.053215698698825], ["pickleball", 15.049761162703394], ["revolutionizing", 15.046519707856984], ["marama", 15.045434031834303], ["djembe", 15.040414285723086], ["diyas", 15.039535596459674], ["terence's", 15.028266228060993], ["edtech", 15.023525768244086], ["needn't", 15.019869626048626], ["immense", 15.018311700970912], ["parallelly", 15.012917838274529], ["absorbing", 15.009942110225783], ["fervor", 15.006689211113224], ["koji's", 15.005212320136833], ["peculiarities", 14.988710744981622], ["legacies", 14.988671141201252], ["tinkerers", 14.986974203381394], ["unfolding", 14.98677188701105], ["praising", 14.983689788320072], ["arjun's", 14.98115756890459], ["zawadi", 14.972618864106126], ["breakroom", 14.969527902325757], ["jane's", 14.961284280839038], ["darauf", 14.960036831447212], ["zden\u011bk", 14.960036831447212], ["zephyr's", 14.958290999164399], ["amigurumi", 14.95701130120585], ["sheepish", 14.953099969334883], ["montelena", 14.95278758084241], ["flitted", 14.936684578832272], ["tlaloc", 14.930975676367778], ["dedicating", 14.927331451476025], ["verdura", 14.918468343801944], ["frustration", 14.918042510610656], ["despite", 14.910619816124523], ["expressing", 14.906488885248224], ["seljuq", 14.901320679038955], ["reactively", 14.900922365155898], ["solely", 14.900330627026058], ["passphrases", 14.898655326326162], ["w\u00e4hrend", 14.898655326326162], ["drava", 14.898390616638931], ["sudsy", 14.897664361856021], ["showerheads", 14.893439388658779], ["emphasized", 14.888028362431111], ["longingly", 14.88706104202552], ["spacy", 14.887061042025518], ["excelled", 14.879857625392281], ["scorching", 14.877756628874254], ["reverently", 14.872465884141182], ["lunchtime", 14.869089901359523], ["comprehensions", 14.866555448303721], ["vastness", 14.854485416112993], ["duolingo", 14.8535316252642], ["levenshtein", 14.84464776270351], ["dish's", 14.841948735837562], ["beachgoers", 14.841502710211412], ["tommys", 14.838410515744389], ["offred", 14.834082177107279], ["juanito", 14.834044442873006], ["samantha", 14.828563545344963], ["teow", 14.814793755413746], ["pandemic", 14.807290429622181], ["seljalandsfoss", 14.795165603494496], ["effectuating", 14.78367867367812], ["squawked", 14.783059617580232], ["nurtured", 14.78269407969386], ["santer\u00eda", 14.775547476167649], ["disparities", 14.77500789439737], ["aureliano", 14.764417319879554], ["thrilling", 14.756601129536024], ["gauss's", 14.752943374980244], ["jurchen", 14.735373374930646], ["exploration", 14.729306217757603], ["preprocessed", 14.722936503948715], ["resiliently", 14.722172707699288], ["entwined", 14.716153399052509], ["lepidopterists", 14.713955681071734], ["iterating", 14.707389615656247], ["lyra's", 14.706313060279074], ["resemblances", 14.706313060279074], ["commenter's", 14.706313060279074], ["timeworn", 14.705373035410409], ["commotion", 14.705108073734095], ["takeouts", 14.703270164963476], ["haggled", 14.699871665017804], ["croquembouche", 14.696201285076475], ["xolani", 14.693494054481453], ["cucina", 14.689346613670898], ["disdainfully", 14.68268664910754], ["calligraphy", 14.681015487023572], ["overhydration", 14.680296305330721], ["kusama's", 14.680296305330721], ["thalis", 14.680296305330721], ["slacklining", 14.679213767442267], ["idiosyncrasies", 14.671752307946637], ["hurston's", 14.668133673760437], ["contemplations", 14.660046150705973], ["westbridge", 14.65530276573025], ["mistranslations", 14.643654453258225], ["curious", 14.643562570194517], ["wireframes", 14.64301086100871], ["tentatively", 14.63894335799176], ["microfinancing", 14.63258136609346], ["mallie", 14.630046707966647], ["fagin's", 14.627025041291448], ["atat\u00fcrk's", 14.61778603609741], ["gaud\u00ed", 14.617459483822454], ["hidden", 14.613399679016014], ["excelling", 14.608851281896133], ["confidantes", 14.607408016855574], ["protractors", 14.606928172955145], ["roadblock", 14.603497974558367], ["faced", 14.59811234105611], ["gulal", 14.595157884338743], ["joyous", 14.5873487928539], ["matchboxes", 14.577657695028986], ["decipher", 14.576913936983322], ["taro's", 14.573579021875338], ["bruckner's", 14.572850874471852], ["accessibility", 14.568107290790765], ["trepidations", 14.566909191659379], ["safeguarding", 14.56523781067585], ["pickiest", 14.556653070824785], ["chord", 14.554849780517232], ["influencers", 14.554268275345319], ["kaiseki", 14.533754307479068], ["boutique's", 14.532607207691578], ["breathlessly", 14.528564008060393], ["kasia", 14.527718189286976], ["expanses", 14.51688546503967], ["gelatos", 14.510453843045708], ["strada", 14.506880719715728], ["bippy", 14.50534152812743], ["corals", 14.493309065189338], ["relentlessly", 14.48884747505776], ["reflectively", 14.478611783296726], ["eerie", 14.473724806707601], ["wikisource", 14.473531568635922], ["alcazaba", 14.473531568635922], ["nyepi", 14.473531568635922], ["luna's", 14.468229908720671], ["fatima", 14.464342024782821], ["reignited", 14.462525080751027], ["larper", 14.459838539715737], ["tackled", 14.457839602011761], ["ambitious", 14.450696944516094], ["bitwise", 14.449014010925078], ["discomfort", 14.446332985822377], ["soothing", 14.4352580960988], ["depicting", 14.4283650930121], ["rosebush", 14.427895888210923], ["technophile", 14.42706758286181], ["discovery", 14.423292136868803], ["cyberbullying", 14.416143804981857], ["uncontrollably", 14.415507537814765], ["larps", 14.410328374013057], ["tangible", 14.40548470561864], ["enriched", 14.402603412124163], ["bella's", 14.395994792375374], ["dosas", 14.392324874465759], ["olsztyn", 14.382645970088756], ["flawlessness", 14.38069842154846], ["flashier", 14.38043331022884], ["collaborative", 14.377247094234763], ["lakshmi's", 14.376230516073662], ["uneasy", 14.375868224920504], ["grandpa's", 14.373102982241306], ["salesy", 14.370149200288521], ["gloominess", 14.345447218471001], ["composter", 14.344303608201672], ["marcy's", 14.339881358605064], ["interschool", 14.329581435493973], ["robo's", 14.329277101174434], ["contraptions", 14.327106735645831], ["grumpiness", 14.323367234723499], ["cheesemaking", 14.323367234723499], ["erechtheion", 14.322240297883631], ["katerina's", 14.322240297883631], ["interpersonal", 14.316072269152961], ["clacking", 14.311073561232867], ["inconsolably", 14.307930520939859], ["enamored", 14.302770641813352], ["longboarding", 14.301488293019977], ["mealtime", 14.291092095081687], ["kandinsky's", 14.289500127598666], ["kerameikos", 14.285563626186104], ["chiptune", 14.283090363785451], ["rediscovering", 14.282380645009608], ["amplifying", 14.281783265332363], ["furiously", 14.275571007826192], ["testaments", 14.256762086617181], ["theodoros", 14.254443105912957], ["cockaded", 14.250861852195369], ["moonlit", 14.250787355803118], ["biodiversity", 14.24826314025047], ["capturing", 14.237208464364926], ["negotiables", 14.234761479578045], ["chime", 14.234403655762806], ["affordability", 14.233399281329685], ["ashenafi", 14.23310745620011], ["eesha", 14.23310745620011], ["unbothered", 14.229022178514633], ["eras", 14.224648089281072], ["easygoing", 14.215422568466575], ["reconnected", 14.213672755503836], ["tidbit", 14.213408190213432], ["bountiful", 14.210376449206178], ["uncontainable", 14.206738359997471], ["vulnerability", 14.19990877336915], ["hargrove's", 14.191579654961194], ["telemedicine", 14.184953446918001], ["typeerror", 14.178153373357638], ["word2vec", 14.178153373357638], ["insisting", 14.175906834971014], ["pelmeni", 14.159595580984135], ["horological", 14.155684033621728], ["encouragement", 14.154772058839985], ["unintentionally", 14.15419491974062], ["tartness", 14.153674517395348], ["foresight", 14.152119350353109], ["ladoos", 14.151897533777346], ["perpetuating", 14.146771329683242], ["ava's", 14.140910035692114], ["eloise's", 14.133285799391317], ["bristled", 14.132580737514704], ["kyudo", 14.131920264531471], ["adherence", 14.131378415310829], ["vulnerably", 14.12751711130993], ["chaos", 14.12743407872969], ["perched", 14.127022389209465], ["memorable", 14.117843034450335], ["delights", 14.110477016262996], ["jazmine", 14.110243993882442], ["sketchbooks", 14.108484861092], ["shrouded", 14.103598344743897], ["orlaith", 14.097595683736289], ["konnte", 14.097595683736287], ["jenkins", 14.097351216759112], ["inkwell", 14.09519609298161], ["alistair's", 14.090992594423064], ["nabateans", 14.090633908089998], ["marja", 14.083632350868585], ["repurpose", 14.080919928530623], ["kpis", 14.07426173225976], ["synthesizers", 14.063661648108178], ["musings", 14.060996156480433], ["m\u00f8re", 14.060002095246325], ["zainab's", 14.060002095246324], ["opulence", 14.056626344443146], ["francophiles", 14.054767544950254], ["ch\u00e8re", 14.051972396733905], ["poured", 14.051306089678826], ["crunchers", 14.038237386120556], ["decluttering", 14.031151201754026], ["chessboxing", 14.029084003777943], ["knitting", 14.025687947454422], ["hardships", 14.022707147412591], ["detracted", 14.020143832750941], ["bataar", 14.014054375980814], ["malfunctioned", 14.009978368626614], ["bookstore", 14.00591306906913], ["creating", 14.003863359389435], ["macarons", 13.993837379503988], ["heartstrings", 13.985450302860595], ["giovanni's", 13.983825613306136], ["bridged", 13.979797980300654], ["artisanship", 13.976935039456274], ["mrinalini", 13.975303287443635], ["succulents", 13.974957473469544], ["scuba", 13.971214884783263], ["jenny's", 13.970287226443157], ["respectfully", 13.965240765072979], ["geekery", 13.96441236772913], ["takoyaki", 13.963657860901394], ["do\u00f1a", 13.959630863841083], ["transpired", 13.956080340629287], ["softened", 13.952341835794895], ["basie's", 13.945924999138848], ["challenging", 13.942546385143245], ["herg\u00e9", 13.93970261207844], ["yakitori", 13.93941663866362], ["blurting", 13.934466362253053], ["chopin's", 13.934145298997315], ["jaynie", 13.934145298997315], ["nalani", 13.916165756676028], ["stablehand", 13.916165756676028], ["birdsongs", 13.916165756676028], ["polyhedrons", 13.913141160771513], ["introspections", 13.913141160771513], ["carelessness", 13.912575167602991], ["dripped", 13.906210142332837], ["subsided", 13.903104147989684], ["grogginess", 13.902313813564689], ["gti's", 13.901629105437541], ["inspired", 13.89209169689436], ["gravely", 13.89156448047544], ["succulent", 13.888387363789642], ["skeptics", 13.888228313992183], ["pouring", 13.885165946345003], ["ethnomusicologist", 13.881677117873858], ["fisherfolk", 13.868986024000025], ["ella's", 13.864598058369921], ["whirlwind", 13.856802267804884], ["purposelessness", 13.84842887630281], ["quackers", 13.841275762213808], ["entomophagy", 13.83733237880577], ["technophobe", 13.83733237880577], ["longtime", 13.836141726690727], ["component's", 13.835714923238406], ["radiating", 13.83314059909708], ["conversing", 13.831613314408354], ["wcag", 13.821004072602905], ["pitmaster", 13.817936062042657], ["pesky", 13.817791975118132], ["began", 13.814630671048665], ["sprouted", 13.809004106422787], ["splurging", 13.807289639286015], ["sunlit", 13.80436569351457], ["dialed", 13.80215430595634], ["grasped", 13.800069283401434], ["seetha", 13.798381407999843], ["olena", 13.788748035530963], ["barista", 13.785891130028077], ["amdavad", 13.784315779653259], ["outrightly", 13.779953654406533], ["techy", 13.779953654406532], ["wordsworth's", 13.773063276975991], ["bananaman", 13.771125046849763], ["interdependently", 13.754349875784447], ["alphonse's", 13.749854990204126], ["olivia", 13.748142146231507], ["impeccably", 13.745254837982579], ["strengths", 13.744104728296046], ["kiitos", 13.743773674418984], ["hyejin", 13.741902500331248], ["banter", 13.740132939670794], ["intuitiveness", 13.722427014928284], ["normalcy", 13.721715410653946], ["psychotria", 13.720414978025728], ["waban", 13.708785282230501], ["hempcrete", 13.705548260912382], ["wingbeats", 13.700411248820586], ["dottie's", 13.692420341122236], ["yodeling", 13.681666619592011], ["encapsulate", 13.68093341130586], ["iname", 13.678282581348233], ["ntombi", 13.678282581348233], ["hieroglyphics", 13.6771044347004], ["japchae", 13.676366318728265], ["thatha", 13.676366318728265], ["outmaneuver", 13.669077126833008], ["seismo", 13.663237330237385], ["wira", 13.660954198745856], ["hiit", 13.653461852392887], ["inspire", 13.647341033750841], ["thead", 13.646472621856727], ["zongzi", 13.646472621856727], ["dave's", 13.640903201339693], ["feline", 13.638953904158733], ["healthtech", 13.632321423147577], ["quadratics", 13.625265982195721], ["railfans", 13.624319257210855], ["mingling", 13.622483343013684], ["wollstonecraft's", 13.62214735871616], ["disheartening", 13.615629263344243], ["escalating", 13.614457436820112], ["forebears", 13.613747267847478], ["cherish", 13.612740610432196], ["wilted", 13.605950889539656], ["jeopardizing", 13.600323818480645], ["woks", 13.598757682619464], ["upon", 13.595946838926213], ["baristas", 13.594781437683029], ["skole", 13.593050154002569], ["molokini", 13.59071583892779], ["kaida", 13.59071583892779], ["volunteering", 13.590627402942783], ["keras", 13.587396982801069], ["potluck", 13.582119332049698], ["awry", 13.579413788334088], ["kaggle", 13.572249383043209], ["cerulean", 13.565834374545755], ["kibet", 13.565312631752226], ["wastefulness", 13.557546777402793], ["luxembourgish", 13.540136471556544], ["unconventionality", 13.539755338401346], ["extraordinary", 13.537040804881137], ["theremin", 13.536000572158995], ["unsettling", 13.53386536525679], ["impassioned", 13.532526498305058], ["constructive", 13.530437486039943], ["heartbreaks", 13.528321343745413], ["urbex", 13.528024481237596], ["democratized", 13.52281268286884], ["arraylist", 13.51988196673746], ["hammurabi's", 13.505139763862813], ["brainteaser", 13.503750060244387], ["chuckle", 13.499925772223495], ["algebraic", 13.491198974429148], ["horace's", 13.48857955019888], ["civilizations", 13.481783154725147], ["tama's", 13.475356977695514], ["cybercriminal", 13.474333558771106], ["russula", 13.473057036241734], ["saki's", 13.473057036241734], ["triumphant", 13.47098767866191], ["ekstraklasa", 13.469245704689754], ["relying", 13.46499335605804], ["\u00e9clairs", 13.457230320475047], ["photosynthesis", 13.454007502466055], ["confront", 13.450062505890225], ["overshadowing", 13.45004612199667], ["frustrated", 13.449425827786582], ["kabir's", 13.449369788745795], ["yearning", 13.445411739967302], ["ani's", 13.443467244010922], ["responding", 13.4433130139043], ["teahouse", 13.442540278815281], ["tomatina", 13.439707885161928], ["academically", 13.436969279377891], ["geeked", 13.43473757899138], ["moreover", 13.434374780918903], ["foodie", 13.433954431619116], ["levity", 13.431395734943042], ["reevaluated", 13.43037012285958], ["recalculated", 13.418769683977644], ["lifelessly", 13.417711636577373], ["trickling", 13.40914266321574], ["dwelled", 13.406149210619839], ["tortuguero", 13.402785610752142], ["kapok", 13.398354937822967], ["seamus", 13.39603608096284], ["elif", 13.395214207645324], ["buzzy", 13.392324299669369], ["overlooked", 13.389533809873205], ["ethically", 13.388737973325535], ["prioritization", 13.388560818716371], ["life's", 13.38412653970505], ["ljuba", 13.381627902944286], ["conversation", 13.376214187192554], ["kathak", 13.375798784728987], ["delight", 13.374420756083376], ["symbolisms", 13.373719139408268], ["repercussions", 13.365441367491961], ["dismissing", 13.362403191769097], ["wilting", 13.361668276370507], ["londolozi", 13.36018298643316], ["prized", 13.358134947945347], ["speedrunning", 13.355088864806405], ["aspired", 13.349171846609176], ["skytree", 13.348103254980687], ["elderly", 13.343980211321952], ["ambiguity", 13.340036678755204], ["guruji's", 13.336325516814528], ["stranger's", 13.334592851423231], ["truthseeker", 13.333426945739367], ["midsentence", 13.327943162624198], ["transformations", 13.319273604266392], ["fondly", 13.315910770659851], ["prickled", 13.311931924283119], ["hackathons", 13.310064400739144], ["whiteboards", 13.309624110420167], ["parameswara", 13.308130325447054], ["hollowly", 13.29625866592391], ["contrarily", 13.294739773968544], ["pentru", 13.292018787522785], ["akosua", 13.292018787522785], ["samosas", 13.286031391672548], ["untold", 13.281419091519977], ["linguistics", 13.269684242943008], ["fryderyk", 13.268902233109701], ["stirred", 13.267988019093364], ["rewarding", 13.266401442223197], ["resolute", 13.255499301561105], ["stumper", 13.248259054888967], ["mridula", 13.248259054888967], ["sanitizers", 13.247326782324159], ["despair", 13.241451470461227], ["njeri", 13.236017717042515], ["towards", 13.224057658651947], ["pliny's", 13.214963606145842], ["creaminess", 13.211073801316715], ["interject", 13.206434848890872], ["politely", 13.19901971370093], ["alaric's", 13.196697537253696], ["staycation", 13.194653382412536], ["theodore's", 13.194393790778268], ["clinks", 13.19242441044048], ["machiya", 13.191066746098562], ["cdot", 13.190446048434994], ["klara", 13.188849227673424], ["eloquently", 13.188053435895231], ["bandmates", 13.185894885027592], ["adara", 13.172650420175776], ["sparkling", 13.16698348866201], ["confections", 13.165670411169298], ["pieter's", 13.16102497918198], ["tidyverse", 13.16102497918198], ["thuringian", 13.157755971487202], ["elisha's", 13.157755971487202], ["unturned", 13.156705871250052], ["civilization's", 13.15586208793607], ["misconfiguration", 13.155690389702038], ["ukiyo", 13.154846251015842], ["manong", 13.153226623476362], ["charset", 13.151807618878031], ["unsure", 13.151782470682656], ["fellow", 13.15153178083634], ["boundaries", 13.149819077917243], ["disbelief", 13.148392512643976], ["neurodiversity", 13.14811658982311], ["herder's", 13.14811658982311], ["dreamcatchers", 13.139565525443668], ["webhooks", 13.135642095904869], ["collaborated", 13.127970282145153], ["yanow", 13.12622586501175], ["tutelage", 13.117874077610892], ["surreal", 13.113561760647972], ["complemented", 13.109892914921929], ["vongole", 13.10154125972651], ["sketching", 13.09550541172294], ["ben's", 13.095444598565088], ["monoculture", 13.08820783278077], ["thrives", 13.08722323879651], ["dewdrops", 13.08454307486561], ["sm\u00f8rrebr\u00f8d", 13.082568794507274], ["fearing", 13.08071327548669], ["popmatters", 13.076472536914308], ["shelob", 13.074828938053459], ["aquaponics", 13.073893351009591], ["pineville", 13.073893351009591], ["rodriguez's", 13.067076931118553], ["footprint", 13.067031625608704], ["sourced", 13.06561687080856], ["hagia", 13.064679431708605], ["bustle", 13.058005025692891], ["ethica", 13.055974203523109], ["blossoming", 13.055865036436316], ["stargazers", 13.050990180175706], ["geysir", 13.047567867292482], ["crunching", 13.044655260583355], ["kombucha", 13.03497691120615], ["highlighted", 13.03465150278637], ["scoby", 13.02747338924641], ["plyometrics", 13.024940299576553], ["noticing", 13.023055649158634], ["shortcuts", 13.022756098889106], ["benny", 13.017762359675707], ["obsession", 13.009772645510651], ["innovation", 13.00825220589238], ["traditions", 13.003828128329607], ["matlab", 13.003055613017157], ["conserving", 13.002687228790629], ["throbbed", 13.001000418005068], ["thrilled", 12.996694852303335], ["charismatic", 12.996190678437124], ["enormity", 12.993803974920326], ["transparency", 12.989304499031796], ["revolutionized", 12.986767453673046], ["explored", 12.983054683313062], ["kaija", 12.971932150053922], ["resented", 12.969980816670667], ["recreating", 12.966581517077946], ["mutianyu", 12.966149939828679], ["warung", 12.960957320331687], ["parvaneh", 12.959718317834717], ["panicking", 12.953686196858852], ["counterintuitive", 12.952755831803158], ["villagers", 12.950683529263836], ["researched", 12.942105477512957], ["granddaughter's", 12.939221279517506], ["unwittingly", 12.934989861230008], ["rosa's", 12.92613095853291], ["jsme", 12.925502388146066], ["birdbrain", 12.925502388146066], ["microloans", 12.922796043424931], ["expertise", 12.916467244527837], ["bigos", 12.914843553552053], ["cubie", 12.914843553552053], ["microclimates", 12.912629849865805], ["mus\u00e9e", 12.909984971842974], ["solara", 12.909193100221325], ["guiding", 12.904537870769424], ["pieced", 12.902962074624835], ["okonkwo", 12.901536449463949], ["mingled", 12.901449886397735], ["pinecones", 12.896512390455252], ["sven's", 12.89451859037586], ["unha", 12.89375605231611], ["carlos", 12.88981454640093], ["g\u00fcell", 12.87699372833431], ["kyoto's", 12.87207609235679], ["approachability", 12.87207609235679], ["oblivious", 12.871104859251231], ["kova\u010d", 12.86965557371365], ["brusquely", 12.868340176791373], ["microgrid", 12.86536139434304], ["chilliness", 12.860550114838802], ["phidias", 12.860104218225704], ["recalculating", 12.854073659841399], ["handiwork", 12.851756762381743], ["derail", 12.849394561898276], ["elijah's", 12.84895149460536], ["respectful", 12.839485573493516], ["blindly", 12.83801260833046], ["entangled", 12.833928285732927], ["elseif", 12.833673312090964], ["sundarban", 12.830016994908032], ["xiongnu", 12.82707162828845], ["mustered", 12.814943086176022], ["tsukiji", 12.814510163535362], ["grapples", 12.81097429806958], ["vaishnavism", 12.81097429806958], ["unfurling", 12.81001527795504], ["devouring", 12.801185045897888], ["sitar", 12.796886777694441], ["sensationalism", 12.795676333704716], ["jill's", 12.795088411936966], ["bonsai", 12.793945085144298], ["leclair", 12.79062364844914], ["fallingwater", 12.788781862233858], ["graphql", 12.785933170667432], ["raka", 12.782363346536846], ["underhanded", 12.77559598253294], ["disgruntled", 12.769405459690882], ["unfurled", 12.762334959032207], ["bu\u00f1ol", 12.75371273070722], ["bcis", 12.750492096179265], ["queijo", 12.746705196239798], ["jsou", 12.744226498097605], ["stakeholders", 12.740467139248608], ["mankind's", 12.738032588781179], ["retaliating", 12.736707780399612], ["herpetologists", 12.73092920260552], ["pasticceria", 12.729515963471094], ["signorina", 12.721670345003625], ["f\u00fcrth", 12.716031306730132], ["dirndls", 12.708466743192519], ["epidendrum", 12.704821170403305], ["gained", 12.68977419259452], ["mildred's", 12.680215793371449], ["swapping", 12.678338787744142], ["shintoism", 12.677347750959003], ["cookoff", 12.672677410326381], ["neckpiece", 12.66983918470257], ["invitingly", 12.662912189291964], ["mindsets", 12.661217836264061], ["andrei's", 12.65400188572169], ["gazes", 12.64363824781365], ["seamless", 12.643055517308436], ["aeiou", 12.64143896501112], ["extracurricular", 12.633177269456832], ["kneading", 12.631819178870991], ["idolized", 12.629539578845124], ["clashed", 12.627686373778715], ["moli\u00e8re's", 12.619778756889449], ["tangibility", 12.616153425445356], ["discrepancies", 12.61318252480001], ["zzz's", 12.612648938382732], ["anytown", 12.612648938382732], ["gearhead", 12.607755098460961], ["trivial", 12.606767228148524], ["inviting", 12.605509272060399], ["megamart", 12.602802998540122], ["maryam's", 12.602802998540122], ["wordle", 12.598223491636583], ["picchu", 12.597234600685692], ["snorkeling", 12.595908686177683], ["airbnbs", 12.586333426439756], ["resultset", 12.585679624900802], ["promoting", 12.583658573729922], ["nuance", 12.583399292703955], ["inconveniences", 12.582420175708053], ["rosebushes", 12.581854190364357], ["mixology", 12.578487807972287], ["heritages", 12.57841636608227], ["tension", 12.575157260223012], ["empowering", 12.566092640054103], ["emanating", 12.562407039746613], ["heron's", 12.558290641512109], ["landmarks", 12.556857504345762], ["nourishing", 12.556369779282374], ["ladybugs", 12.55535268604562], ["homemade", 12.553180071389088], ["tagore's", 12.550699970579233], ["muertos", 12.54527615901027], ["jollof", 12.541615976323468], ["greenwood's", 12.540313287464738], ["assertiveness", 12.53381058554227], ["pompadours", 12.527510517390755], ["menacingly", 12.52656400909735], ["personalization", 12.522606230467256], ["beatmatching", 12.518664967157825], ["angelfish", 12.517272612028565], ["heartier", 12.516898825416682], ["ariana's", 12.516898825416682], ["innovate", 12.514320819154538], ["truths", 12.510194901506209], ["saltado", 12.506745048856128], ["sita's", 12.505931654927352], ["adem\u00e1s", 12.49986817291284], ["prejudices", 12.495565292302544], ["cz\u0119stochowa", 12.489707743591229], ["delightfully", 12.486758969546415], ["tabanus", 12.472582503847544], ["leclerc's", 12.470836474995199], ["pursuing", 12.468593769856552], ["untouched", 12.465319236266364], ["woodcarvings", 12.462532074755], ["noor's", 12.457575385861633], ["leila's", 12.456520463735638], ["linguistic", 12.453766561764624], ["nephrons", 12.448960188499349], ["disregarding", 12.442612148337666], ["fulfillment", 12.437462816019503], ["crumbled", 12.435762247409276], ["rollercoaster", 12.432279699989396], ["util", 12.43069597009131], ["patricia's", 12.427459652473479], ["spiraling", 12.425974702419412], ["jebediah", 12.423811780014072], ["woolf's", 12.417478485988575], ["echolocation", 12.411681343838254], ["environmentalist", 12.407375293539097], ["underprepared", 12.405884201687932], ["boqueria", 12.405884201687932], ["gnawa", 12.405884201687932], ["kalea", 12.405884201687932], ["validating", 12.39364155280469], ["idyllic", 12.392350509831546], ["recommitted", 12.3898972375105], ["afterschool", 12.38828720282029], ["tingled", 12.385777420161697], ["negativity", 12.385408470481263], ["diversity", 12.385168155820041], ["anticipating", 12.378034257561696], ["ad\u00e9lie", 12.376830608709742], ["scriptwriting", 12.375365421363979], ["romanticizing", 12.37386901665132], ["genuine", 12.367288117504904], ["dissimilarities", 12.366562857150095], ["atahualpa", 12.3617351475894], ["hydroponics", 12.36157747239619], ["taught", 12.361544172975428], ["progressiveness", 12.361151446152999], ["colleague's", 12.36056590323428], ["sketchpad", 12.359679791253342], ["rohan", 12.359558431567706], ["drumroll", 12.358773249023294], ["stepped", 12.355161409319004], ["destinies", 12.354005049571782], ["connecting", 12.351469426844668], ["shouldn", 12.342115638034397], ["conchs", 12.33772000277756], ["microbrewery", 12.336962622789667], ["optimistically", 12.333704511787202], ["passionflower", 12.32736594724687], ["curriculums", 12.32186028478407], ["cascading", 12.316246310057238], ["stumped", 12.312596408027652], ["metalhead", 12.310663599262696], ["explaining", 12.308315571231129], ["tamagoyaki", 12.305023354519738], ["afterall", 12.300641482239167], ["beretta's", 12.299851003382907], ["dampened", 12.29800694776021], ["tackling", 12.295173462926035], ["getname", 12.293103436218043], ["saplings", 12.291478353142105], ["avian", 12.290815130831698], ["dannebrog", 12.287167606456471], ["stood", 12.286861081484103], ["minimalistic", 12.284327987607993], ["adrenaline", 12.278826958655541], ["butterbur", 12.278644363721902], ["surprises", 12.278148357910244], ["chanterelles", 12.276431496974666], ["rakija", 12.275979655073401], ["majestically", 12.273906710180613], ["grapple", 12.268667603699566], ["marae", 12.265704719182985], ["overexploitation", 12.264102667954356], ["tantalized", 12.262876891005652], ["weighty", 12.260447988589048], ["neha", 12.258789825247337], ["rigidness", 12.256415717330247], ["harrumphed", 12.252725137469564], ["feathered", 12.247487644469953], ["todoist", 12.245290231439546], ["november24", 12.242648883244671], ["drawbacks", 12.242004002974278], ["interfaith", 12.23245181549305], ["protagonist's", 12.22952604391884], ["ultimately", 12.224731613022424], ["fervent", 12.219795938662614], ["establishment's", 12.217099007314422], ["integers", 12.210934592804268], ["dioramas", 12.209101210902539], ["reputable", 12.207003879077215], ["freshly", 12.202269198082835], ["kaito's", 12.201391604956813], ["acrophobia", 12.198687805835108], ["ethereal", 12.198596009963527], ["bokashi", 12.193817805077883], ["hatte", 12.187690258442167], ["quickened", 12.18619666894971], ["dupont's", 12.185661405208267], ["tim's", 12.182453289743176], ["discordance", 12.180663913608878], ["breakneck", 12.179974581965839], ["patels", 12.177354755867366], ["keoni", 12.175291558533884], ["pollinators", 12.174862149514972], ["trivialized", 12.174634167226644], ["arjuna's", 12.170329438364744], ["threads", 12.168805651901438], ["crackling", 12.167837715161081], ["cybersecurity", 12.167735531744816], ["raindrops", 12.167479309555853], ["makhani", 12.165770313913328], ["communities", 12.165359982838055], ["enshrouding", 12.162631570282286], ["wearier", 12.162631570282286], ["soulful", 12.162361289580724], ["cluttered", 12.160900036051844], ["entwining", 12.156519695121341], ["contemplating", 12.15551771689254], ["coffees", 12.153040466529722], ["biodegradable", 12.151312589456763], ["scampering", 12.149762721395021], ["unanticipated", 12.145951389843045], ["calmness", 12.142099292760289], ["kai's", 12.142080916939397], ["carelessly", 12.140267033641717], ["idlis", 12.139090993049482], ["tenacity", 12.136609195759823], ["mohan's", 12.135013367589943], ["determinedly", 12.134951098432678], ["nestl\u00e9's", 12.131070311144214], ["metropolises", 12.12699223126239], ["tiptoed", 12.116027094171859], ["vocalizations", 12.115028543370848], ["resplendent", 12.114292906349114], ["nura", 12.114292906349114], ["calidris", 12.113981044001157], ["ramesh", 12.106794973056408], ["ehrs", 12.103827890690313], ["philodendrons", 12.103301660183348], ["interacted", 12.101026427119324], ["forevermore", 12.096943894719828], ["chasms", 12.095737096645735], ["tasked", 12.093358387285528], ["aisling", 12.091242211065413], ["jasmine", 12.087964368214703], ["pushback", 12.085732351321793], ["selene's", 12.085732351321793], ["shopkeeper", 12.080617936057381], ["mysteries", 12.075060622976254], ["whitespace", 12.070590034074746], ["plyometric", 12.070590034074746], ["fingerpicking", 12.070590034074746], ["polyglot", 12.069891504558885], ["commended", 12.06968473982219], ["guidance", 12.068490528539225], ["rania", 12.068132380490146], ["disruptions", 12.061755761658503], ["wildebeests", 12.052981587635541], ["climatologist", 12.049889263726454], ["bathed", 12.046573037031639], ["kowalska", 12.044045912472036], ["magritte's", 12.041005254579465], ["bookshelves", 12.036239488739682], ["monotony", 12.035032147260251], ["tastings", 12.033707675637295], ["alina's", 12.033707675637295], ["composure", 12.032064512166873], ["hiker", 12.029387218452628], ["abasi", 12.028187492971439], ["frolicked", 12.026112236330137], ["splendor", 12.024846329778946], ["faraway", 12.023676305607271], ["leif's", 12.022986541142009], ["groomers", 12.020295908932766], ["sofia", 12.020173983780907], ["spellbound", 12.017691326230302], ["marbles", 12.016899703493072], ["qubits", 12.013744183818487], ["expertly", 12.01373580565338], ["noora", 12.012278004924626], ["corsica's", 12.012046607983553], ["hilariously", 12.006532881671694], ["organizing", 12.001130594936184], ["reignite", 11.995649783863193], ["rsvped", 11.995093334082373], ["contrastingly", 11.991491409982283], ["peering", 11.989391876390734], ["patisseries", 11.981751408467833], ["christgau", 11.980192096728052], ["encapsulating", 11.9770388218765], ["exfoliants", 11.976862269641089], ["diyers", 11.974033802262088], ["prevailed", 11.971919614243674], ["stemmed", 11.966317687381212], ["interpersonally", 11.962816908770506], ["seesawed", 11.962816908770506], ["blueprints", 11.955496666539698], ["knack", 11.955318846890366], ["seizing", 11.954761139808372], ["eleanora", 11.953264877579915], ["adversaries", 11.95253337160156], ["idealistic", 11.951900758075439], ["grinning", 11.950468194542099], ["reconnect", 11.947369427476898], ["arcadio", 11.943291909421603], ["urbanization", 11.941227447951984], ["darting", 11.939692311019074], ["bullseyes", 11.934774675041556], ["disarrayed", 11.934774675041556], ["grafana", 11.934774675041556], ["bijay", 11.93327908924268], ["gruff", 11.929621154935825], ["sneakerheads", 11.927420348179307], ["intersect", 11.927257185390554], ["printouts", 11.925081991000505], ["cloudwatch", 11.925035976816309], ["nostalgically", 11.925035976816307], ["mi'kmaq", 11.923321445058255], ["assistive", 11.91944295272701], ["exquisite", 11.918488407130218], ["delegating", 11.91514511143129], ["humbled", 11.906240623674897], ["timbaland's", 11.902942950268152], ["guided", 11.90145016232191], ["diluting", 11.89824736455103], ["coder", 11.894457955006104], ["truthfulness", 11.892427297710617], ["vigilant", 11.886423205662917], ["evolving", 11.886269323494993], ["rani's", 11.881164023973035], ["misha", 11.88101987007806], ["merely", 11.88080780368145], ["linh", 11.866869922580115], ["strengthening", 11.863666358137609], ["marcel's", 11.861993662574267], ["\u0435\u0441\u043b\u0438", 11.854511570501803], ["greenspaces", 11.845618463547188], ["lejla", 11.833304930840798], ["amends", 11.82654721695839], ["ramanujam", 11.826169986655787], ["treehouses", 11.82549195833411], ["gameworld", 11.821313741922534], ["briskly", 11.819877611256443], ["enigmas", 11.816604702107757], ["deviating", 11.815979045123521], ["sprang", 11.810965663834262], ["aoife", 11.798787740754268], ["bajram", 11.79775262317382], ["strategies", 11.797614156291326], ["accordions", 11.79453654271052], ["florae", 11.793247944814455], ["punchlines", 11.785589991603537], ["influencer", 11.784893814038906], ["machu", 11.784102475512206], ["soil's", 11.782124660820942], ["visibly", 11.782045453260197], ["microdermabrasion", 11.779180522153755], ["xeriscaping", 11.77862039373742], ["wildest", 11.772756318291444], ["brainstorm", 11.77157204335304], ["platform's", 11.769351923276718], ["microbusiness", 11.765580500955652], ["playstyle", 11.765400842229296], ["guanciale", 11.76022480085659], ["retrocomputing", 11.758620678121606], ["mercilessly", 11.755768154580247], ["algorithms", 11.750830470852677], ["smartypants", 11.747612795067756], ["bishnu", 11.747612795067756], ["cardistry", 11.741283262311793], ["cryptozoology", 11.737651563189521], ["codecademy", 11.73592025038635], ["cliffside", 11.73529586646156], ["delighting", 11.732798995000609], ["wincing", 11.731343729885129], ["craftsperson", 11.730077280195205], ["scatterplots", 11.72919960886859], ["\u0442\u043e\u043b\u044c\u043a\u043e", 11.72919960886859], ["nudged", 11.727398718402005], ["bakery", 11.727006649540009], ["greensville", 11.726109724883115], ["victorious", 11.723894062105893], ["chiseling", 11.720717859174027], ["shaka's", 11.71392253306789], ["clasped", 11.710501745119634], ["jasper's", 11.708053215342987], ["reverend's", 11.70753972675848], ["dolma", 11.706961148071711], ["luca's", 11.704399649969453], ["savor", 11.696605377172581], ["misstep", 11.695195357973148], ["hamish's", 11.693584630299151], ["millbrook", 11.692121001042874], ["wrestled", 11.690382043078955], ["busan's", 11.682207623256136], ["zork", 11.682040066743419], ["conflicted", 11.679467367709579], ["participant's", 11.67864271400278], ["amongst", 11.677342570628023], ["documenting", 11.677320456752442], ["pounded", 11.67477338553228], ["harmonies", 11.67403320343332], ["whitepapers", 11.673973629316965], ["veered", 11.673544296299651], ["zesty", 11.672150972638686], ["bewildered", 11.671377089429265], ["ahalya", 11.670720693439758], ["tokyo's", 11.66852038012282], ["seabrooke", 11.658541779899503], ["charango", 11.658541779899503], ["milda", 11.658541779899503], ["mezquita", 11.654012431888665], ["teufelsberg", 11.650743424193884], ["checkup", 11.637667393414766], ["policymakers", 11.6372223224347], ["pedrera", 11.636527041893332], ["incorporate", 11.635597945323243], ["staycations", 11.633533434267674], ["myrtle's", 11.633533434267674], ["cacti", 11.633275754963595], ["rejoiced", 11.631557201179895], ["buzzing", 11.629891981823961], ["moonlighted", 11.626620118667335], ["radhika", 11.626540194146], ["sensationalizing", 11.626085766153263], ["delicately", 11.62276276145638], ["primatology", 11.622354673160274], ["arithmetica", 11.620701657277305], ["carefulness", 11.613370714602217], ["lucius", 11.607306755062723], ["yoyo", 11.602212525897352], ["deepfake", 11.601799114541494], ["marketplaces", 11.598915755640217], ["teacups", 11.598754386156429], ["euler's", 11.597307019082574], ["spice's", 11.596804797230025], ["melina's", 11.594284300642927], ["unengaged", 11.588730152474955], ["hanami", 11.586017071837253], ["hurried", 11.578825254908738], ["louvre's", 11.578825254908738], ["nuremberg's", 11.578825254908738], ["slyly", 11.574517656227595], ["yoshiko", 11.568860689284891], ["splurged", 11.56787083177259], ["vladimir's", 11.564441621051708], ["cybercrimes", 11.564278992025686], ["rooted", 11.563184957125468], ["wordcount", 11.556166105681909], ["eliciting", 11.554902062233307], ["trio", 11.55057611177822], ["diplomatically", 11.550531133045238], ["tarantulas", 11.550501318375204], ["inputting", 11.543675249670622], ["reallocations", 11.542866170266164], ["eilidh", 11.542588136106557], ["eumaeus", 11.542183402836242], ["adventures", 11.538440142507499], ["dauntingly", 11.536312879139714], ["illustrating", 11.530942894832046], ["otherworldly", 11.522054590209299], ["desdemona's", 11.519749615853081], ["c\u0153ur", 11.517767211857924], ["troubleshooting", 11.517612010492515], ["macbeth's", 11.511112826517458], ["cornfields", 11.510360110307811], ["cuisine", 11.509818324209977], ["sollte", 11.506907085623592], ["bridalveil", 11.506907085623592], ["naans", 11.506907085623592], ["steelpan", 11.506907085623592], ["bumble's", 11.50363807792881], ["kaanapali", 11.50292427561571], ["schoolwork", 11.500828592667997], ["incan", 11.4965523753862], ["cultures", 11.495857632716014], ["urging", 11.493702068945868], ["resurfaced", 11.490562047149695], ["cooped", 11.48911038610557], ["sprawled", 11.48904015003918], ["nara's", 11.486929816377716], ["crispr", 11.484462824739298], ["rumbled", 11.476357774776801], ["gandhiji's", 11.475442886561337], ["profoundly", 11.475059988900792], ["brew's", 11.472107971453356], ["equitable", 11.467587632372496], ["homestay", 11.465828682386666], ["solutions", 11.463625889267957], ["amsterdammers", 11.460673976797425], ["sevillanas", 11.460673976797425], ["osteria", 11.457871970201898], ["caipirinhas", 11.451585416942708], ["hofbr\u00e4uhaus", 11.451585416942706], ["holistic", 11.445877461594698], ["galettes", 11.444187751944682], ["srey", 11.440445742412608], ["unintended", 11.43549723847737], ["hawked", 11.43484333299981], ["beekeeper", 11.433434489048418], ["scania's", 11.432873676065348], ["coreference", 11.432873676065348], ["membrana", 11.432873676065348], ["overzealous", 11.432728015807022], ["spices", 11.430187618839687], ["carefully", 11.429911568501671], ["overpoweringly", 11.415300292047451], ["l'orangerie", 11.414780505134352], ["mientras", 11.414780505134352], ["putu", 11.413413465552898], ["seawalls", 11.407268135917388], ["creatives", 11.397291714292384], ["juana's", 11.397275730005989], ["pomelos", 11.397275730005989], ["dedicated", 11.393009050747489], ["evasively", 11.392445773944411], ["opportunities", 11.387321483054999], ["zappa's", 11.378998932196984], ["deadlifts", 11.378907032674032], ["imperfection", 11.374621072228734], ["gran's", 11.373815736691823], ["democratizes", 11.372060518213939], ["durians", 11.372060518213937], ["platypuses", 11.366284966909616], ["acme's", 11.365390687997978], ["veering", 11.364488851799763], ["trolleybuses", 11.362023395073415], ["responded", 11.355882881672544], ["exasperatedly", 11.351384044544458], ["congestions", 11.350064269629385], ["considerations", 11.350030675358822], ["corleones", 11.348018106970356], ["interning", 11.344544298501702], ["rehabilitators", 11.34252269868611], ["archaeologist's", 11.34252269868611], ["figurines", 11.341797102300575], ["acquaintanceship", 11.338711367134131], ["inherent", 11.337620119764479], ["aysel", 11.336411425680351], ["elderton", 11.336411425680351], ["savory", 11.335256395082265], ["eias", 11.333770752159348], ["seethed", 11.332298068849553], ["indulge", 11.330469225357175], ["factorizations", 11.328984531402522], ["fascinatingly", 11.326559578204336], ["acupuncturist", 11.32340999193281], ["tarnish", 11.322997699337204], ["yokocho", 11.322830819000892], ["pavel's", 11.320369334040238], ["fiercely", 11.318707831691798], ["exclamations", 11.317367910443057], ["meditation", 11.315391664453143], ["envisioned", 11.312681927163124], ["celje", 11.311247360362527], ["railfan", 11.311247360362527], ["jana's", 11.306628639513054], ["frequented", 11.303761456802494], ["sketchbook", 11.303138939315673], ["blerim", 11.303138939315673], ["unenthused", 11.3016955554644], ["rediscover", 11.29898835294779], ["tightly", 11.290864905091007], ["fairness", 11.285352725406442], ["freshbooks", 11.284870569005289], ["erupting", 11.284290997331249], ["cryptically", 11.279676274158815], ["shrouding", 11.276689329701577], ["knelt", 11.275395546225086], ["ahhing", 11.272926740346568], ["speakeasies", 11.266925906335228], ["miscounted", 11.26656830561455], ["colonizers", 11.264370551185397], ["promising", 11.259445863949004], ["indirectness", 11.257191220050162], ["overconfidence", 11.253909240102626], ["tiraspol", 11.252573358817447], ["bombonera", 11.251848461996033], ["temujin", 11.250680409627924], ["gamify", 11.247358050557544], ["masterpiece", 11.24710548229188], ["noi's", 11.24645577162364], ["adored", 11.24483590032422], ["overspending", 11.244377115075716], ["tablas", 11.241577917387124], ["bapak", 11.238271570940833], ["armful", 11.236743282157134], ["bookworms", 11.236186548385929], ["brows", 11.236105622852973], ["lekan", 11.232354615041778], ["flitting", 11.231755875456754], ["rainwater", 11.231046967788087], ["learning", 11.230993288481443], ["strides", 11.230973991998665], ["rawness", 11.226559407206487], ["chirping", 11.225567283713575], ["tento", 11.224371420574796], ["deciphered", 11.223402967475524], ["educating", 11.222754613611169], ["choucroute", 11.219234408482999], ["holistically", 11.217609751044328], ["unparalleled", 11.209266004006135], ["lhakpa", 11.208074692559443], ["streamline", 11.206374868279696], ["coaxing", 11.202775794213062], ["nigiri", 11.200454157799227], ["peals", 11.200454157799227], ["daisy's", 11.197800281610899], ["comforting", 11.19486263084503], ["ashwagandha", 11.193998341368804], ["xanthippe", 11.193815055546008], ["pachamama", 11.190613912542993], ["nostalgia", 11.189393542198678], ["devastated", 11.188217129472735], ["wulfstan", 11.186121006797752], ["adapt", 11.182053468032414], ["incentivizing", 11.176083506911912], ["calming", 11.174710961493634], ["harvests", 11.174451415616327], ["sirkka", 11.17409428095296], ["nisha", 11.173621878298796], ["rejuvenated", 11.171614568039523], ["blockquote", 11.170238364866426], ["gamification", 11.165295781519141], ["mustafa's", 11.16529578151914], ["jubilance", 11.16529578151914], ["natalia's", 11.16128093226293], ["squabbles", 11.158145415985024], ["omakase", 11.156779156209057], ["maasai", 11.153103266333206], ["enticing", 11.150258346123154], ["peeked", 11.145423655795613], ["lucy", 11.14143831190051], ["ecosystem", 11.139990246580375], ["audiophiles", 11.136444888026842], ["burnout", 11.136260733387529], ["rhythmic", 11.133901299049564], ["zapier", 11.133485822027634], ["birder's", 11.130512928617211], ["sumimasen", 11.125912022148702], ["daimyos", 11.125276800223372], ["carbonara", 11.123744973188126], ["zoey's", 11.118247518079604], ["concocting", 11.11500165637716], ["teasingly", 11.113604597345441], ["upskill", 11.112876552497923], ["uncertainty", 11.110222931091087], ["evergreen", 11.108334912421896], ["caribous", 11.106220142463483], ["sandcastle", 11.105267309575488], ["begrudged", 11.102859165186794], ["torrential", 11.102797624331794], ["starry", 11.101066735652244], ["codebreakers", 11.100001654141835], ["jisoo", 11.097627322237205], ["mabel", 11.096296762644585], ["swaying", 11.096029594726383], ["overindulgence", 11.094953139138436], ["lhamo", 11.091353425350139], ["elotes", 11.09010860453921], ["abacuses", 11.09010860453921], ["farzaneh", 11.086109286614748], ["unusual", 11.07915847206203], ["eastville", 11.076682322935655], ["sleuths", 11.074921790645917], ["gumballs", 11.073330830553527], ["breathed", 11.070383293952009], ["prickling", 11.068723030248515], ["ignite", 11.065605640612718], ["gertie's", 11.064707531235184], ["reusable", 11.063445003813925], ["harrowing", 11.061383247180213], ["marring", 11.059262583214116], ["constraints", 11.0587349378216], ["weaved", 11.054255459960823], ["kubernetes", 11.05378221566127], ["nondescript", 11.051740777384696], ["translating", 11.051675547568404], ["nomsa", 11.051218685411664], ["aquariums", 11.0487883472772], ["unwrapped", 11.046478862404381], ["fromagerie", 11.045238837631837], ["olexander", 11.045238837631837], ["patatas", 11.042984247136301], ["niched", 11.042600223480468], ["kufic", 11.038705126399876], ["chilies", 11.038569007415875], ["tamara's", 11.038569007415875], ["eleni", 11.03695904839823], ["juggling", 11.036858392136958], ["imperfections", 11.033602549224298], ["o'mally", 11.027452623722606], ["nightowl", 11.027452623722606], ["whiteboard", 11.026500668212963], ["seeking", 11.023346348600878], ["beauvoir's", 11.021298911320978], ["kway", 11.01525411418309], ["hinting", 11.014652901927214], ["cacio", 11.01196462846457], ["questioning", 11.01135003665274], ["justinian's", 11.006245984061604], ["senet", 11.005219856336069], ["handwashing", 11.001807385062786], ["frugality", 11.000383725665886], ["trainspotters", 10.996124633314304], ["inaturalist", 10.99383234133321], ["marissa's", 10.98806886435217], ["miscommunicated", 10.988068864352169], ["felt", 10.984243388525321], ["drizzles", 10.977868754011622], ["parched", 10.975693335221933], ["absently", 10.974436024570094], ["unilever's", 10.971870705256261], ["alongside", 10.970453011557169], ["warily", 10.967814359533087], ["marisco", 10.967520815984983], ["naysayer", 10.964438608729905], ["strive", 10.963257095948794], ["phyllo", 10.962750733328315], ["testimonials", 10.960614316730585], ["sylas", 10.95853104482434], ["tasha's", 10.956242136300045], ["jolted", 10.955792439040303], ["stag's", 10.955509430718156], ["tutorials", 10.953792686275085], ["traversing", 10.947086502202858], ["brewskis", 10.946368413254058], ["murals", 10.93965252538699], ["impulsive", 10.93785457115486], ["empowered", 10.928978038756908], ["masterfully", 10.928516808852516], ["workdays", 10.921815811205638], ["k\u00f6nnen", 10.921419425417582], ["scholastic's", 10.921419425417582], ["cultivate", 10.92000833722721], ["lef\u00e8vre", 10.918368110358443], ["umami", 10.917178097485383], ["valea", 10.9119819156208], ["w\u0142adys\u0142aw", 10.911199358111075], ["clatter", 10.907186781349793], ["lizzy's", 10.904010989874966], ["impatiently", 10.903644405628993], ["thankfully", 10.901252952947557], ["\u0431\u044b\u0442\u044c", 10.898627242604352], ["dant\u00e8s", 10.898627242604352], ["blossom's", 10.898224663288358], ["joyrides", 10.896423006045294], ["miniature", 10.89378414441934], ["crannies", 10.893453697418407], ["erratically", 10.89053458779048], ["alliterations", 10.873683364029263], ["oversimplifications", 10.873683364029263], ["mysterious", 10.867973678388147], ["wisely", 10.864222524634052], ["creased", 10.863763873839225], ["foliage", 10.863113543765962], ["chang'an", 10.8567359185703], ["westpac's", 10.855148676476942], ["paraphrases", 10.855148676476942], ["whodunits", 10.855148676476942], ["guqin", 10.855148676476942], ["surprise", 10.854177397286774], ["blushed", 10.85301952014942], ["spark", 10.850386950196867], ["guzzlers", 10.8503476376992], ["luisa's", 10.849252343681462], ["kavita", 10.848731839820896], ["piques", 10.847029642313533], ["microcosm", 10.84679289076265], ["worrywart", 10.844878904786803], ["scalability", 10.843755517516207], ["cutthroat", 10.842599371648644], ["nonperishable", 10.833307331051433], ["lacing", 10.833049742702437], ["verne's", 10.832128311055456], ["abode", 10.83079638526622], ["vintage", 10.830336908073566], ["ruffling", 10.828345840238727], ["valjean's", 10.826953485109469], ["jan's", 10.823064493196714], ["gripped", 10.818290614718304], ["lifestyles", 10.816268991538639], ["shook", 10.811506548124576], ["handcrafting", 10.805124949857232], ["jimothy", 10.805124949857232], ["initiatives", 10.804343626889077], ["excessiveness", 10.803457492303243], ["saut\u00e9ed", 10.799886101396694], ["optimize", 10.796017826543663], ["stumbling", 10.794780757816941], ["misinterpreting", 10.793119255468502], ["iago's", 10.792110646996822], ["envisioning", 10.790621426434605], ["blinked", 10.789397490207996], ["backlash", 10.78524681253964], ["entrenched", 10.783997281790791], ["quena", 10.782684399597922], ["coolidge's", 10.782077369006267], ["ynab", 10.780659197099995], ["leisurely", 10.77982723668098], ["afar", 10.770965890822628], ["wilhelm's", 10.768418501688855], ["familial", 10.767709613127694], ["overexerting", 10.766025945142976], ["muchmusic", 10.765784256380037], ["epiphany", 10.765740752402706], ["revered", 10.764795318934967], ["remained", 10.764618447158629], ["resided", 10.76297411628683], ["hutongs", 10.76272073126569], ["misclassifying", 10.76272073126569], ["receptiveness", 10.762369627960044], ["alleviate", 10.761261244678668], ["lunged", 10.76027077030722], ["radha's", 10.757000858425613], ["secondly", 10.755446019545296], ["influencing", 10.755037902064588], ["daniyar", 10.751766308129541], ["pensive", 10.746924042633177], ["twitchers", 10.738426647697619], ["progressed", 10.733268853933428], ["carnivore's", 10.732074428444323], ["pachacuti", 10.72471297809471], ["renewable", 10.723841624133122], ["practices", 10.723786088549772], ["corbusier's", 10.723730411628551], ["environmentally", 10.72364630393905], ["stunned", 10.721895494394092], ["drishti", 10.721641644505135], ["hypotenuse", 10.720395106699986], ["preferences", 10.717768628103835], ["leafcutter", 10.717741731205082], ["friendship", 10.716304045942131], ["undeniably", 10.716117215595954], ["couldn't", 10.715375714471257], ["lopsidedly", 10.714172719639578], ["unnoticed", 10.711042302955434], ["jargon", 10.70970037893991], ["kibbeh", 10.706448009675887], ["adventurers", 10.704388428065702], ["tradici\u00f3n", 10.703115781848412], ["divides", 10.702494711300142], ["interdependencies", 10.700257349046936], ["sanctity", 10.698383412473794], ["unethical", 10.693993808170703], ["causing", 10.69306708022539], ["jokingly", 10.692886283772744], ["bytecode", 10.688146389146528], ["greenfield", 10.687883134309356], ["buddhism's", 10.684251292065934], ["patting", 10.683099052400642], ["untangled", 10.678335840892135], ["artifact", 10.67581848266655], ["culinarily", 10.674830592150082], ["harmony's", 10.670858367683753], ["concerns", 10.66746582679331], ["struggling", 10.66540821389692], ["speechless", 10.664970561517], ["strangers", 10.66078982398383], ["longstanding", 10.660256237566553], ["deceitful", 10.66001861146111], ["tantalize", 10.657686597714932], ["calligraphers", 10.657686597714932], ["crinkling", 10.65364067037608], ["honesty", 10.65229913694576], ["piece's", 10.652172822391732], ["outsmarting", 10.651516738822972], ["selfhood", 10.651104528422865], ["remy's", 10.650710874117937], ["mineshafts", 10.649800136608818], ["trainings", 10.647793183391586], ["miraculously", 10.64664978392599], ["egusi", 10.646184314923687], ["freecodecamp", 10.644554963176683], ["pedaling", 10.639223476764016], ["mqtt", 10.638379194458201], ["ewell's", 10.633615030018229], ["discord", 10.629467165999426], ["gameboard", 10.626554196664962], ["explained", 10.626102248876627], ["oohs", 10.624686721848608], ["nerdiest", 10.623002639768709], ["nutritious", 10.622230254544588], ["ticked", 10.621594421723424], ["mitra's", 10.62018874841467], ["swooped", 10.616225332509153], ["frustrations", 10.61384741233422], ["sprinted", 10.610828712096762], ["aced", 10.61053634055914], ["trajan's", 10.607261585086338], ["fattoria", 10.603319830502505], ["nyambura", 10.601391954169689], ["enigma", 10.599901173936464], ["named", 10.599205956616753], ["deepen", 10.598627006371073], ["reef's", 10.59455675626242], ["cradled", 10.593266538790013], ["devoured", 10.591523637191072], ["nook", 10.589366985136143], ["gathering", 10.589119125927263], ["h\u00e4rtel", 10.58148946614559], ["yusuf's", 10.580809989521553], ["voltaire's", 10.57883580107571], ["ajji", 10.571970884916672], ["cuttlebone", 10.571970884916672], ["suggested", 10.571341413257533], ["levelheaded", 10.570694231024039], ["misdiagnoses", 10.569363579685792], ["lounging", 10.568701877221894], ["genealogist", 10.567154936080616], ["yianni", 10.565776975600249], ["pivoted", 10.565129112174946], ["settimeout", 10.565011062082627], ["quinoa", 10.564461781947195], ["query", 10.564022034627262], ["genres", 10.563433170820108], ["startled", 10.560204860897597], ["fermenters", 10.556060301274172], ["textarea", 10.55213138994146], ["weightage", 10.550303231349995], ["attributing", 10.548646073256627], ["exemplifying", 10.541820575485593], ["inconsistencies", 10.53942589314185], ["throughout", 10.537738351615534], ["glint", 10.535287791858476], ["deescalating", 10.535155631592133], ["impersonal", 10.53456392847659], ["remnants", 10.532365948636727], ["concatenate", 10.532342505922816], ["hesitating", 10.532004930842499], ["bellowed", 10.52838654795034], ["\u00farsula", 10.52620477718976], ["sudokus", 10.523612115914592], ["reassessing", 10.523612115914592], ["leatherworking", 10.522848206788872], ["mistrust", 10.521268575570645], ["nonverbal", 10.519212534253992], ["underfoot", 10.516497898761282], ["eli's", 10.515024527824718], ["ponderings", 10.515024527824716], ["steadfastly", 10.512088001103734], ["flashlights", 10.508141983399168], ["repurposed", 10.503768141284624], ["signboard", 10.499889265246786], ["swainson's", 10.497286632197483], ["quandary", 10.495128670121424], ["memorizing", 10.495070768146215], ["idiomatic", 10.493310387261044], ["spotting", 10.492846221525607], ["hurdle", 10.484661667730036], ["taonga", 10.480833204874287], ["tupa", 10.479856517407615], ["workflows", 10.479012825681083], ["slurped", 10.478416317244893], ["maslenitsa", 10.477942737912105], ["longjing", 10.477942737912105], ["uttering", 10.474110804567955], ["neatly", 10.472332799897348], ["transitioning", 10.471384236611541], ["monet's", 10.468965506972784], ["creatures", 10.465894914765954], ["dishes", 10.46532822622835], ["lazily", 10.463983552158416], ["mustering", 10.463448946204522], ["capybaras", 10.456388112851256], ["underappreciated", 10.454802189655132], ["interspecies", 10.450779666105467], ["fertilizers", 10.450133045186542], ["surprising", 10.448482493300144], ["golang", 10.443498882511841], ["oma's", 10.442927840661362], ["alphabetizing", 10.438608066145187], ["rajan", 10.438560553136238], ["interconnectivity", 10.437642958150905], ["ethnobotanist", 10.436696233166037], ["artisans", 10.436275844723323], ["unpleased", 10.434855870578636], ["borland's", 10.434855870578636], ["joys", 10.434614520170745], ["hornworms", 10.434088037181242], ["undeniable", 10.432838782761946], ["mycologists", 10.431069981827404], ["reallocating", 10.430318296425888], ["strlen", 10.429122433287107], ["aragonese", 10.423879033370927], ["chatbox", 10.420942729417863], ["offhandedly", 10.419176767253921], ["yuki's", 10.41909455300048], ["polyrhythms", 10.414413316680132], ["farsi", 10.410627114545564], ["transglobal", 10.409535019807116], ["complexity", 10.408064240103732], ["fragrant", 10.407924208959303], ["roastery", 10.404935136899557], ["predefined", 10.402070243383974], ["montagues", 10.401186286192694], ["tapestries", 10.396563495804404], ["cloudberries", 10.391253433892455], ["freikorps", 10.387299653616676], ["r\u00eaves", 10.382229331909048], ["plutarch's", 10.379425029300661], ["nudging", 10.377569972311706], ["combinatorics", 10.377157255764848], ["radicle", 10.375830423229907], ["gravesites", 10.375380200613261], ["unfamiliarity", 10.374591074158811], ["problemo", 10.374214322189399], ["inclines", 10.370478115716082], ["personalfinance", 10.369251545244165], ["letterforms", 10.363637908043973], ["occurrences", 10.358913308409425], ["unwrapping", 10.358246325387828], ["galen's", 10.356534599049217], ["irezumi", 10.35621637706123], ["nzinga", 10.35192986366013], ["belied", 10.350051962551076], ["amphitheaters", 10.349119189302828], ["coworkers", 10.346507424207736], ["communication", 10.344046967006985], ["anjali", 10.340599860302172], ["shifted", 10.339492578690551], ["etouffee", 10.338236834739945], ["zahra's", 10.338236834739945], ["interiordesign", 10.338236834739943], ["zetian", 10.338236834739943], ["microhabitats", 10.338236834739943], ["startup", 10.336707509764393], ["flourish", 10.336041103022831], ["arduous", 10.330568738544642], ["melkite", 10.329999195827801], ["grouses", 10.329797457731992], ["ingredients", 10.32798849561681], ["honeybees", 10.327984037879045], ["saddened", 10.327542106979868], ["pasteur's", 10.3274114558554], ["antique", 10.32207263018618], ["append", 10.319311000717537], ["meetup", 10.31867800829584], ["encouraging", 10.314054350317813], ["correspondences", 10.311433998501728], ["clownfish", 10.310850776899574], ["rastafarianism", 10.30991289820641], ["retrogaming", 10.30991289820641], ["outweighed", 10.307508574122833], ["jameson's", 10.301430546532492], ["sincerity", 10.300282759785008], ["goosebumps", 10.296883887400986], ["pok\u00e9mons", 10.296883887400984], ["clearbrook", 10.296883887400984], ["cryptokitties", 10.294244337570838], ["nonverbally", 10.293091695723612], ["beschloss", 10.292794035466361], ["gratitudes", 10.287806411155847], ["orion's", 10.28712645038842], ["satiate", 10.28712645038842], ["badaling", 10.285891331779236], ["clickthrough", 10.285891331779236], ["blazingly", 10.283825061925523], ["stories", 10.283670437224327], ["euclid's", 10.277779309390587], ["bromeliads", 10.269914573919054], ["strutted", 10.268484875373023], ["lumi\u00e8res", 10.262459182548133], ["ishiguro's", 10.261657302630761], ["classmates", 10.261114061098635], ["isabella's", 10.253098413747969], ["cues", 10.25260329777254], ["llamado", 10.248339123133508], ["artwork's", 10.245931148715481], ["underrepresentation", 10.24380101749953], ["erlewine", 10.238917528904015], ["adanna", 10.238591178501487], ["revisiting", 10.238576231653052], ["tendrils", 10.237085740529539], ["disillusionment", 10.234385257459046], ["thermoses", 10.23364766831845], ["microgrids", 10.232203636434917], ["twinkle", 10.23066328929738], ["agreed", 10.22522735683141], ["mariana's", 10.225022713783138], ["exfoliators", 10.22462983655599], ["determined", 10.22435062851011], ["danya", 10.223944769230256], ["plopped", 10.221600316604416], ["implications", 10.220676313019187], ["shamash", 10.219950372099898], ["wiser", 10.216368877350295], ["scanned", 10.209420574002653], ["raza's", 10.207373077338174], ["birdseed", 10.199683145202194], ["plot", 10.196722696004198], ["furball", 10.19544350829326], ["uneventfully", 10.194138233801915], ["amar's", 10.192194827409047], ["creaky", 10.190547737100802], ["convolutional", 10.190547737100802], ["frostings", 10.190547737100802], ["opa's", 10.190547737100802], ["techworld", 10.190547737100802], ["eclectic", 10.189790357112908], ["bonding", 10.187587788588619], ["balmy", 10.18718714685512], ["discussions", 10.185555690952329], ["souks", 10.184157343452954], ["tonkatsu", 10.180750180254963], ["vulnerabilities", 10.177933350643624], ["ancestral", 10.177898083533494], ["sof\u00eda", 10.177419817310655], ["illuminated", 10.176750958106158], ["broached", 10.172825045384105], ["astray", 10.171906090998794], ["overcoming", 10.171427199276874], ["stmt", 10.169330430116025], ["moments", 10.168362497110117], ["goldmines", 10.168113950117641], ["uluwatu", 10.168113950117641], ["pepsico's", 10.168113950117641], ["upvoted", 10.163361186789118], ["morale", 10.163323821159617], ["untangling", 10.162966064561571], ["intimidatingly", 10.157083440052658], ["cheerfulness", 10.154216219081574], ["yannis", 10.151030924489246], ["replicas", 10.1492538654009], ["timestamps", 10.148910087886874], ["achebe's", 10.146076298778235], ["regarding", 10.138576322331582], ["wryly", 10.13710382239092], ["questioned", 10.136079997891487], ["\u00e9galit\u00e9", 10.135164325486125], ["tillage", 10.135164325486123], ["savviness", 10.130569195264819], ["hasty", 10.128927301285826], ["gilled", 10.12725240954117], ["shocker", 10.126977212464825], ["utilizing", 10.122852865461615], ["alexei", 10.121363599806733], ["bonaparte's", 10.118765885175632], ["rajani", 10.117767230771426], ["zookeepers", 10.116975677391267], ["xabier", 10.115567118299392], ["kelechi", 10.111154175942188], ["semblance", 10.110795624375665], ["bihu", 10.110795624375665], ["groaned", 10.109529717824474], ["hanuman's", 10.108498238412391], ["troves", 10.106986800278657], ["pricier", 10.105455343096793], ["pickpocketed", 10.105455343096793], ["innovations", 10.103287407680044], ["seagrasses", 10.102995517542269], ["overshadowed", 10.098389740173976], ["factorize", 10.097812722304132], ["parchment", 10.096930224385543], ["bolstering", 10.096922720448328], ["circe's", 10.095823005511559], ["reenacting", 10.09235444515694], ["buoyed", 10.091229296943471], ["crocheting", 10.090119150706185], ["acidification", 10.088733018393148], ["sarde", 10.087027341559347], ["neutro", 10.086084716819459], ["reveling", 10.082821571764015], ["dreamtime", 10.075735516849155], ["farfetched", 10.07409758995706], ["han\u0131m", 10.071443626101493], ["equations", 10.066669431841648], ["krumlov", 10.06377037010083], ["bravas", 10.06377037010083], ["witnessed", 10.061961552927203], ["birthed", 10.061010372132394], ["untamed", 10.057154523645949], ["breaths", 10.057036792835017], ["superficiality", 10.053044094471257], ["weavings", 10.046645539401124], ["aligns", 10.044026142209797], ["birders", 10.04285863946166], ["untarnished", 10.040678726581378], ["unimaginable", 10.039967703933403], ["sibling's", 10.038880481949494], ["vilaine", 10.036220927208216], ["theodoric", 10.031918706303205], ["phrasings", 10.027174841482282], ["defensiveness", 10.02543889974012], ["recklessness", 10.02346066842846], ["zafir", 10.023297169575681], ["instrumentations", 10.022220861625623], ["cheered", 10.021369875755166], ["plze\u0148", 10.020137239824868], ["prestigious", 10.015352564978702], ["denims", 10.01176619785342], ["reassessed", 10.011386028261702], ["rekindle", 10.009347896839445], ["visualizing", 10.004175658727707], ["meditating", 10.002389066164655], ["kaleidoscopic", 9.995937318489224], ["saladin's", 9.993628940248612], ["whenever", 9.98217536826121], ["detour", 9.9804553220035], ["rohit", 9.97407108906387], ["campeonato", 9.973357887631476], ["excel's", 9.973357887631476], ["sondern", 9.970025659804], ["nefariously", 9.969014090642089], ["prussia's", 9.967120640672356], ["nammu", 9.967120640672356], ["aren", 9.966956502594485], ["nerding", 9.96635781190757], ["techniques", 9.966213467753523], ["crumpled", 9.964067981026679], ["shivers", 9.962300949840309], ["brighter", 9.957050448654561], ["playful", 9.954715694817555], ["onclick", 9.953009758768646], ["bartolomeu", 9.951973040914496], ["amardeep", 9.945212955072146], ["aryan's", 9.944399241035565], ["bottura", 9.944399241035565], ["reminiscent", 9.943910572768193], ["brews", 9.9386750917719], ["soundscape", 9.9343617459431], ["pembleton", 9.932559186794453], ["caelum", 9.929659810133456], ["daunted", 9.929649944697632], ["petrovi\u0107", 9.924707361350347], ["guell", 9.924707361350347], ["graywater", 9.924707361350347], ["logistica", 9.924707361350347], ["zrinka", 9.924707361350347], ["idiomas", 9.924707361350345], ["spellbooks", 9.924707361350345], ["ketut", 9.919527451454027], ["baguettes", 9.91916902018888], ["chorused", 9.915483655624184], ["synthia", 9.912544729780063], ["interdependence", 9.91204829583842], ["nooks", 9.91195066808383], ["pivotal", 9.911359258095366], ["confidants", 9.911074521568272], ["grammarly", 9.908419548394082], ["shandor", 9.905473432355482], ["herself", 9.904606948638822], ["ryohei", 9.903256264748004], ["sanitization", 9.903131910564802], ["gaps", 9.9026029544164], ["unbreakable", 9.900814547332281], ["sunlight", 9.900546730515567], ["citrusy", 9.900382098209782], ["pitter", 9.899560298779358], ["leo's", 9.898145644545725], ["alexios", 9.896023236028526], ["glitz", 9.892131735437818], ["geeky", 9.890523524284847], ["tetras", 9.889388118071519], ["pasang", 9.887743833002673], ["hargrove", 9.88419835171218], ["devised", 9.880058934917535], ["retorting", 9.87822161258524], ["cedric's", 9.87822161258524], ["boulangerie", 9.87593730060661], ["recipes", 9.87518540803757], ["mariam", 9.875182873519256], ["misuses", 9.873016177176648], ["wellspring", 9.873016177176648], ["mithu", 9.872745542704534], ["toucans", 9.872155960798565], ["abuelito", 9.8683169786154], ["thread", 9.867956080165897], ["dollhouses", 9.867504437077434], ["rapt", 9.866154515560668], ["guelaguetza", 9.864919967607273], ["declensions", 9.864641627785376], ["djurg\u00e5rden", 9.864400979814365], ["neighbors", 9.862820536712041], ["orb's", 9.862677940341905], ["recoiled", 9.86108744236733], ["bubbe", 9.859698361166828], ["recuerda", 9.858247267412732], ["boisterous", 9.856166564655938], ["bcrypt", 9.854319365879775], ["hadn't", 9.852792773647334], ["backaches", 9.849168019607047], ["relished", 9.846997124740222], ["motioning", 9.84401868361579], ["ramirez's", 9.84224050105011], ["frequenting", 9.838555387727515], ["catered", 9.838063090735382], ["zooxanthellae", 9.836380677713734], ["boasting", 9.830526565054809], ["murkier", 9.827603274481534], ["\u0142\u00f3d\u017a", 9.823164536923718], ["lifelong", 9.822723046382734], ["semrush", 9.821324993002946], ["lviv's", 9.821324993002946], ["clarissa's", 9.819763325807367], ["friulian", 9.816830107422625], ["tuscany's", 9.81564464309375], ["scribe's", 9.81564464309375], ["altostratus", 9.811926595880456], ["shiitakes", 9.811926595880456], ["geogebra", 9.809303787381156], ["coveted", 9.805746827909504], ["gulped", 9.800648519333468], ["cavaradossi", 9.799200361644555], ["khalil's", 9.799200361644555], ["creation's", 9.796000781736293], ["bookworm", 9.787589693858006], ["shopkeeper's", 9.782283313113446], ["weiwei's", 9.778755782506959], ["crispiness", 9.775522677912328], ["nerd", 9.775402409251257], ["uneasily", 9.773981665441989], ["oliver's", 9.770994103149608], ["buzzworthy", 9.769286888130093], ["dataviz", 9.764631436167276], ["mondavi", 9.76063493870994], ["dwindle", 9.760108406340237], ["retiree", 9.757631211884888], ["gyeongbokgung", 9.757437012563543], ["yelped", 9.755536213145511], ["jubilation", 9.752708376843168], ["interests", 9.750563494492955], ["clasping", 9.750467414697425], ["brightly", 9.749671613341077], ["stifled", 9.742794155508287], ["thrumming", 9.741642930442698], ["peuvent", 9.740916484288302], ["strolls", 9.740801037757485], ["vinci's", 9.7407417781314], ["albeit", 9.740373928193655], ["s\u00fcper", 9.73919881254006], ["insensitivity", 9.737397512353308], ["squealed", 9.736739418900529], ["latinx", 9.73010525622583], ["kickflips", 9.73010525622583], ["mikhail's", 9.73010525622583], ["souffl\u00e9", 9.727031184510265], ["hydroponic", 9.72659057942225], ["resourceful", 9.725179390439866], ["m\u00f6chte", 9.723530860782432], ["hokkien", 9.718638802220177], ["bronte's", 9.717156446949485], ["raymi", 9.717156446949485], ["swiftly", 9.712699023263864], ["tweaking", 9.705996217646515], ["longed", 9.703097976892844], ["questline", 9.697175860708036], ["transience", 9.694968765022791], ["extremophiles", 9.69404519277625], ["seabreeze", 9.689906728154558], ["symphonies", 9.68945801749338], ["belying", 9.688805997922664], ["astrobiology", 9.687867753863577], ["chaotic", 9.68719299714709], ["cyberbullies", 9.68678629446866], ["councilmembers", 9.684472387399355], ["babbage's", 9.681227391036845], ["patted", 9.68091502521575], ["interpretations", 9.680076108473973], ["woodworker", 9.678232840787011], ["eero", 9.676589677316588], ["python", 9.676383941260175], ["suspiciously", 9.673488206266166], ["hilbert's", 9.671526051111817], ["sidelong", 9.669594651398553], ["calmer", 9.668835999690533], ["akbar's", 9.664102354050119], ["carneros", 9.661219555296798], ["entailment", 9.660531956637994], ["polymaths", 9.657745296250733], ["rousseau's", 9.656126363169474], ["platforms", 9.655219541304646], ["huitzilopochtli", 9.65403352422261], ["apiaries", 9.65299347105401], ["provoking", 9.650095755123152], ["spaceless", 9.649021045757282], ["cgms", 9.64902104575728], ["outshine", 9.645192069151822], ["cautious", 9.644619894689871], ["ana's", 9.642980110555042], ["maxine's", 9.641144293883194], ["recyclable", 9.641068555884404], ["wiradjuri", 9.639707318879138], ["soukous", 9.639707318879138], ["thrill", 9.638781268320967], ["azulejos", 9.638037527207757], ["technological", 9.637322356886896], ["rustle", 9.636418242758742], ["stillness", 9.633721176933792], ["stifling", 9.633508150185325], ["wielded", 9.633508150185325], ["maleficence", 9.632804203663571], ["astrolabes", 9.625254984068224], ["fragility", 9.624627858670776], ["faded", 9.619527230429867], ["mementos", 9.619235840806272], ["trainspotter", 9.61905514188847], ["hazel's", 9.617211086265774], ["clean's", 9.616964497432505], ["divisors", 9.609637286386842], ["emphatically", 9.605547434452221], ["libations", 9.604335626471592], ["vendor's", 9.603073326491769], ["hatshepsut's", 9.601075599567181], ["holst's", 9.594883249643841], ["dreamt", 9.594849972997055], ["interwoven", 9.59247547793053], ["gradually", 9.591356658079064], ["exhaustion", 9.590270418308078], ["sidled", 9.583492889249593], ["comorian", 9.581780480978486], ["calligrapher", 9.580099466859016], ["pandemics", 9.57915533564123], ["snowdrifts", 9.576472015338053], ["moshav", 9.575245834542235], ["ojibwe", 9.575086988393688], ["rigorous", 9.572473488400544], ["humankind's", 9.57199104581216], ["colleagues", 9.571934910589528], ["metropolis", 9.566870970264025], ["calculations", 9.56542223993574], ["selam", 9.56431295996053], ["oddly", 9.562783632160608], ["bali's", 9.559828414241878], ["tempestuous", 9.559356079035265], ["revitalize", 9.558055723000864], ["unlocking", 9.556532733429286], ["elegance", 9.547813769063326], ["shoreline", 9.545841387936054], ["adopting", 9.545330073325262], ["tinkled", 9.540299681861425], ["rephrasing", 9.537118851236112], ["reflected", 9.533734041054725], ["jeanne's", 9.530968971070076], ["am\u00e9lie", 9.527006767324943], ["manageable", 9.52668524321286], ["marination", 9.525946797724663], ["midday", 9.523371705765827], ["souvenirs", 9.518845984156053], ["encouraged", 9.516440990349343], ["collecting", 9.514080810091828], ["heeding", 9.51117788796075], ["fina's", 9.511177887960748], ["verdurous", 9.511177887960748], ["contributing", 9.510867740855707], ["billowing", 9.507518335098894], ["diophantine", 9.505134659748375], ["worku", 9.502379388526926], ["daria's", 9.500708787368607], ["underripe", 9.496918250947314], ["togethers", 9.495758144885206], ["lorsque", 9.492381093715768], ["personas", 9.492172571434127], ["coastlines", 9.490352662682136], ["armed", 9.488683585500791], ["greenville's", 9.486852624820184], ["deliberating", 9.486191212831468], ["droned", 9.481938430246334], ["\u043f\u043e\u0441\u043b\u0435", 9.48091963381029], ["emine", 9.480122314860344], ["rigidly", 9.478834800656534], ["skillshare", 9.47504424465486], ["puzzle", 9.474994059039158], ["rachel's", 9.474555351572002], ["luka's", 9.473584299470785], ["miscalculations", 9.468764608638738], ["judiciously", 9.467151936421171], ["encountering", 9.467149161676327], ["multe", 9.465230168695237], ["astrophysicist", 9.463577373038206], ["refine", 9.462896698135681], ["continued", 9.461119056971482], ["galangal", 9.459177175738105], ["chiaroscuro", 9.458386891357822], ["contraption", 9.455498427852316], ["sequoias", 9.45352233638239], ["downpours", 9.453392110019843], ["vologda", 9.452102248905092], ["xiaolongbao", 9.452102248905092], ["smoothening", 9.452102248905092], ["eslint", 9.45210224890509], ["possessed", 9.448622793892916], ["g\u00f3ra", 9.448189954927896], ["benefiting", 9.44627948256755], ["tambov", 9.444963198868155], ["minho", 9.442557276655924], ["lounged", 9.440705019505288], ["scala's", 9.440334193316316], ["sturdier", 9.438599245692371], ["obon", 9.437809755585173], ["lobsang", 9.43605453710729], ["gamemaster", 9.433641111700199], ["overburdening", 9.432573112027189], ["encyclopedic", 9.429155513073555], ["ornithologist", 9.427108709304623], ["jargons", 9.426305026610045], ["nighters", 9.4261531364227], ["imitates", 9.425699728656753], ["collectibles", 9.4255869039336], ["frescoes", 9.425523226239983], ["reaped", 9.425523226239983], ["terez", 9.422190532927544], ["tonya's", 9.422190532927544], ["earsplitting", 9.422190532927544], ["lesson", 9.422037755035852], ["octopuses", 9.422015399274516], ["wistful", 9.421700750443701], ["martinez's", 9.420468197023677], ["tasha", 9.420308309201811], ["interwove", 9.416514514534216], ["meurthe", 9.416514514534216], ["mukasa", 9.415748009486226], ["skills", 9.413812408555538], ["flock's", 9.411360428866706], ["michelangelo's", 9.410446349570975], ["copycatting", 9.408957119257702], ["melodifestivalen", 9.408061284056403], ["familiarizing", 9.40779551961335], ["trivialize", 9.402872549692045], ["contribute", 9.400714535479965], ["lamplight", 9.40055519301267], ["uncanny", 9.400065996746255], ["curries", 9.399089635963042], ["blend", 9.39867559351671], ["tempeh", 9.398635056595225], ["peacefully", 9.398546792515292], ["pandemonium", 9.398090236054205], ["idioms", 9.397909685362949], ["fluently", 9.395206522729717], ["sharding", 9.394747259530668], ["solti", 9.393929293638061], ["silence", 9.393881975873262], ["korn's", 9.391370283520772], ["fattoush", 9.391370283520772], ["billy's", 9.38963618186885], ["chatter", 9.38866978146908], ["pastilla", 9.388236693169246], ["dimmesdale", 9.386961810402656], ["servername", 9.38532109171174], ["grateful", 9.384719098107235], ["marinades", 9.384524949616333], ["purrfect", 9.384178669960615], ["lanikai", 9.383938049994718], ["overemphasizing", 9.383359687094872], ["subforums", 9.380058786642095], ["oluwole", 9.380058786642095], ["thunderbolts", 9.377835731235125], ["mealworms", 9.377388940687643], ["await", 9.372689094764702], ["supportive", 9.370762478380334], ["bioplastic", 9.37040189701961], ["asana", 9.370091361745475], ["rbac", 9.36845532634841], ["swayed", 9.368012937363211], ["croissants", 9.366079827122293], ["\u00e9toiles", 9.3644416232096], ["automating", 9.363637221575516], ["declutter", 9.362793782803305], ["jaden's", 9.362061312054463], ["stressors", 9.361781401909592], ["tightropes", 9.360803534000896], ["caymus", 9.360803534000896], ["dali's", 9.35882492408037], ["celestial", 9.358208578335216], ["sato's", 9.357429750418461], ["technicalities", 9.356406623066396], ["schr\u00f6dinger", 9.35432188081297], ["fandoms", 9.353968336093628], ["jayden", 9.353910312342293], ["\u0431\u044b\u043b\u043e", 9.353642850478996], ["washoku", 9.353642850478996], ["chiron's", 9.3531273562464], ["rina", 9.352327317806916], ["glowworms", 9.340062243799535], ["overconsumption", 9.339714447774819], ["charlemagne's", 9.338346094924834], ["clackety", 9.33395097079378], ["varietals", 9.333398862017557], ["enigma's", 9.333398862017557], ["parallels", 9.332759929683784], ["poacher's", 9.330045143418197], ["sawm", 9.328455562509532], ["sambadrome", 9.328455562509532], ["kemal's", 9.326409399850501], ["bulgogi", 9.325340756194574], ["moko", 9.32508962493543], ["penelope's", 9.322340729592089], ["ornamentations", 9.319252725669882], ["impracticality", 9.319158655467639], ["crisscrossing", 9.318838598012098], ["admitting", 9.317979015131504], ["aletsch", 9.31795669297085], ["rhythms", 9.317495720060457], ["eluded", 9.317383632903631], ["lstm", 9.317072216777875], ["dragonfruit", 9.314450274406473], ["s\u00f8ren", 9.313811548388442], ["drawled", 9.310603712244836], ["sanna", 9.307660764931313], ["jameson", 9.306814290143697], ["carnivorous", 9.306165394797262], ["lasting", 9.304605490555899], ["cascaded", 9.30441315126595], ["aditi's", 9.30441315126595], ["concat", 9.30441315126595], ["subcommunity", 9.30441315126595], ["reblochon", 9.30441315126595], ["knowledgeable", 9.301485508976466], ["comfort", 9.300744485120703], ["bangkok's", 9.30013526016192], ["aligning", 9.297324074579272], ["inti", 9.297115572323781], ["reran", 9.295264269111312], ["integrity", 9.293053546454765], ["schr\u00f6dinger's", 9.292250519695667], ["caltech's", 9.290251182999182], ["alima", 9.290251182999182], ["probed", 9.287250860868358], ["brewing", 9.287152790637515], ["boisterously", 9.287086497352977], ["pecola", 9.283873607885672], ["fallibility", 9.282416902681396], ["essential", 9.282161811051619], ["vanquishing", 9.281856998171971], ["strolling", 9.280206547945582], ["thaddeus", 9.279601382862573], ["boxty", 9.279197939473901], ["shied", 9.277610315027735], ["teachings", 9.27550204721332], ["congregants", 9.273398440761731], ["foodie's", 9.267925256555102], ["resolutely", 9.267814854453103], ["ayd\u0131n", 9.266123385211358], ["clickety", 9.257563135398549], ["ajvar", 9.255183452052902], ["thobe", 9.252721967092251], ["sarracenia", 9.248022768531003], ["retracing", 9.24755284867488], ["reminisce", 9.247044900622306], ["underwhelmed", 9.245524460435151], ["juanita's", 9.245449823311159], ["dukla", 9.245449823311159], ["papayas", 9.243879273129126], ["lacked", 9.241499000005408], ["nearby", 9.238469761157258], ["whizzed", 9.23842440551229], ["citro\u00ebn's", 9.23842440551229], ["trailblazers", 9.237672634991046], ["tuomas", 9.236316980498202], ["downsizing", 9.235669205302989], ["synthpop", 9.234292936212931], ["codebreaking", 9.23415328928228], ["devadatta", 9.23415328928228], ["resolving", 9.233577084064954], ["tinged", 9.228944022372348], ["untrodden", 9.227833619156765], ["matariki", 9.22488825253718], ["masha", 9.223322057587685], ["skincare", 9.221905117580084], ["thrift", 9.221872009366272], ["centered", 9.221241665573977], ["disoriented", 9.220040639063035], ["mentors", 9.21852626063888], ["kikongo", 9.215799692682465], ["tatsuo", 9.208595346456166], ["mehrangarh", 9.201715434364559], ["jada", 9.201030782918549], ["arithmetic", 9.199136125968758], ["categorize", 9.198545629833275], ["attracting", 9.197570638345292], ["deliciousness", 9.192415585556267], ["purists", 9.191755778318456], ["takeshi's", 9.190838155053314], ["houseplant", 9.190692546083811], ["vivid", 9.190067101150747], ["zuzana", 9.189543853102172], ["boastfully", 9.188219711336597], ["relu", 9.186635769604356], ["misremembered", 9.186460113554151], ["townspeople", 9.186180276968095], ["tactility", 9.184819151892352], ["seep", 9.18276712303222], ["abodes", 9.182673166109199], ["fredrikstad", 9.18035430924907], ["deceit", 9.178928345547726], ["sagrada", 9.178783944160248], ["glow's", 9.178337092305707], ["longboarders", 9.178337092305707], ["aleksandre", 9.178337092305707], ["khanom", 9.176955436865047], ["bonds", 9.172083719781277], ["letterboxing", 9.171871653384668], ["skeins", 9.171164765395968], ["kinesthetic", 9.171164765395968], ["leitmotifs", 9.171016546946724], ["tearfully", 9.169915701182926], ["breeze's", 9.169566583856298], ["playtested", 9.169566583856298], ["meshack", 9.169566583856298], ["colosseum", 9.168163555987297], ["johnson's", 9.167431513205646], ["woodworking", 9.16525149369919], ["liam's", 9.162582794690177], ["angelou's", 9.161268333554165], ["burdening", 9.160416281067787], ["marvella", 9.159484597507914], ["glassblowing", 9.155254510366715], ["arun's", 9.154686962624888], ["strewn", 9.154508717162221], ["birdhouse", 9.153049492345833], ["roleplaying", 9.152092383700847], ["subtracting", 9.152092383700847], ["refactored", 9.15125408704758], ["disappointment", 9.151163993480393], ["experiencing", 9.146971202669983], ["organizer's", 9.143779192479155], ["timelessly", 9.141177832822688], ["unsorted", 9.140410569281238], ["plight", 9.140293641514454], ["versed", 9.139997818472496], ["stracciatella", 9.136766067459357], ["wonders", 9.133832243492739], ["ragas", 9.133061159529928], ["visionaries", 9.129145745468149], ["crinkled", 9.128663125075372], ["farmlands", 9.127801605339142], ["exotic", 9.124047159551372], ["nanuk", 9.123393440151826], ["distractions", 9.12228421298585], ["siloed", 9.121278670193412], ["sumaya", 9.120307563797978], ["pichola", 9.119557260843449], ["relatable", 9.118904602455665], ["symbiotic", 9.11708575762119], ["alleys", 9.11454918435316], ["instrument's", 9.113107460305342], ["manneken", 9.112964320992988], ["ptolemy's", 9.112505760920477], ["humor", 9.111201509569984], ["websockets", 9.107643084622863], ["unitedly", 9.106446914004971], ["exponentially", 9.105919004038943], ["downturns", 9.105919004038943], ["kaitlyn's", 9.10393783621966], ["gizmos", 9.103313201877858], ["natalya's", 9.097648414571152], ["workplace's", 9.097648414571152], ["milongas", 9.097648414571152], ["imagined", 9.09510361781183], ["laypeople", 9.090518596064433], ["glamourous", 9.089918891704055], ["cola's", 9.088031450073718], ["wagged", 9.08756232985433], ["transportations", 9.08599969701088], ["wisps", 9.085618466254362], ["uneaten", 9.084562038830972], ["antiheroes", 9.084400330755466], ["zulkifli", 9.083630466320656], ["downsides", 9.08356536748977], ["accra's", 9.082879504807238], ["microloan", 9.081819535206956], ["ania", 9.080051415703508], ["answering", 9.078138306083027], ["heritage", 9.07796513396003], ["practiced", 9.07562687529231], ["conflicts", 9.074018158948888], ["startup's", 9.071354633026148], ["forum", 9.070510542879958], ["paradoxical", 9.069861849165527], ["discreetly", 9.069829159088577], ["laidback", 9.063890906539347], ["unbeatables", 9.062454416835866], ["corners", 9.060934189151084], ["heirloom", 9.059543080897553], ["parted", 9.056295467232191], ["campfires", 9.055812748002555], ["allocating", 9.055479290639974], ["trailblazing", 9.051700695305641], ["rifts", 9.050556090199382], ["javier's", 9.04769183389321], ["honoring", 9.046550663931772], ["canva", 9.045157057267827], ["hluhluwe", 9.043541754501485], ["antonyms", 9.043541754501483], ["flexitime", 9.04284330364], ["casanova's", 9.03923161163879], ["ethnobotany", 9.038942769079517], ["katya", 9.038806276065122], ["callused", 9.038572775515494], ["taoism", 9.037425675728006], ["aeroponics", 9.037399087322335], ["subaquatic", 9.037399087322335], ["huipil", 9.036384788883803], ["tobi's", 9.036384788883803], ["hungrily", 9.036058918534401], ["choices", 9.034455941918804], ["riddle", 9.034190437200094], ["krakow's", 9.034028495588137], ["pierogi", 9.033739314138213], ["cathedral's", 9.031859634713715], ["reigned", 9.02726041910058], ["cataloguing", 9.026591237453502], ["pristinely", 9.022461237591225], ["craftwork", 9.022461237591225], ["boerewors", 9.022461237591223], ["affectionately", 9.021674395041435], ["loggers", 9.01784448111], ["inclusive", 9.017734841096182],  ["facades", 9.00990824804327], ["ensnaring", 9.007241161279499], ["kad\u0131k\u00f6y", 9.005752976040128], ["scatterplot", 9.002881243586035], ["windowsill", 9.00192399943467], ["reconciling", 9.001304180226223], ["lgbtq", 8.995955563527485], ["peers", 8.991075961714747], ["hypercompetitive", 8.990781247290693], ["perfect's", 8.990781247290693], ["ataturk's", 8.98977116064343], ["joyously", 8.988727705062283], ["henceforth", 8.987029280439433], ["barriers", 8.983076425179092], ["treasured", 8.982987969676762], ["diners", 8.981860162022063], ["kierkegaard's", 8.981161238968447], ["absurdities", 8.977348931403267], ["harmonicas", 8.9753875267864], ["hostelworld", 8.972112681577881], ["marcolini", 8.972112681577881], ["botanist", 8.96868690353986], ["grigori", 8.966529313252497], ["cocina", 8.96547448183088], ["haunting", 8.965219036385962], ["maximizing", 8.96072421115993], ["nonbinary", 8.957778151512905], ["unforgiving", 8.957020546516103], ["app's", 8.955402404068586], ["consolingly", 8.955240913801223], ["idolizing", 8.953812076000856], ["flyboarding", 8.953812076000856], ["aimed", 8.953215114792323], ["centerstage", 8.953113841347601], ["spinoza's", 8.953113841347601], ["efficiently", 8.950533796404187], ["bren\u00e9", 8.950363944596774], ["plummeting", 8.948294345893906], ["eldridge's", 8.942574862050051], ["merriment", 8.940660373747322], ["guzzler", 8.939534204157482], ["aspects", 8.939028454224463], ["itinerary", 8.937344930474831], ["richer", 8.934896756330684], ["gamers", 8.93393995027733], ["managed", 8.932107172510598], ["nuestras", 8.930305748296762], ["cogsworth", 8.929932126601996], ["dared", 8.929613024326738], ["eswatini", 8.928477266366315], ["interplay", 8.91903887606458], ["meals", 8.919030361387707], ["spellcasting", 8.917451213876241], ["quietened", 8.917451213876241], ["waitlists", 8.916729269963202], ["anxieties", 8.916614400665038], ["yonne", 8.914161827040669], ["nonni", 8.912273133396504], ["memorabilia", 8.90973575681029], ["byval", 8.909680472121334], ["jovie", 8.906788657622105], ["alleviating", 8.903170207221544], ["pachyderms", 8.902205089972034], ["ponder", 8.901775748827239], ["yogis", 8.898461443095533], ["finned", 8.898461443095533], ["fairgoers", 8.898461443095533], ["clumsiness", 8.896185337791602], ["intermingled", 8.895398191777986], ["infuse", 8.890570871920687], ["finalizing", 8.88725622635539], ["cheesemonger", 8.887124319027354], ["matcha", 8.8855820179611], ["candies", 8.883691860947838], ["connoisseurs", 8.883446097419705], ["inconspicuously", 8.882240778914834], ["tasks", 8.881080596830705], ["bouchard's", 8.880001323313468], ["\u043c\u0435\u043d\u044f", 8.878721046306069], ["efforts", 8.878569065035514], ["electrifying", 8.878327114919179], ["enthralled", 8.877510834209524], ["artifacts", 8.877414432171664], ["webcrawler", 8.875754550801123], ["possibilities", 8.874246329295794], ["alarmingly", 8.874143425671303], ["oenology", 8.873361242563233], ["voraciously", 8.871191798191134], ["topics", 8.870874187228468], ["petrolheads", 8.870874187228468], ["jabu", 8.870874187228468], ["plumages", 8.8680845244005], ["valiantly", 8.864251217646343], ["disagreed", 8.861575278306022], ["gabriela's", 8.861345858348523], ["aspirations", 8.86084171175702], ["musicians", 8.859314232712707], ["displaying", 8.851948021822341], ["flours", 8.851218605938982], ["confessing", 8.849530730537392], ["creeping", 8.846618551147325], ["unethically", 8.84649962444972], ["amma's", 8.845282995765414], ["alienating", 8.844154847383328], ["tabla", 8.843383670797817], ["wattson", 8.843168738639092], ["cloudformation", 8.843168738639092], ["affirmations", 8.838420983491105], ["unspoken", 8.837516022864586], ["marquez's", 8.83584558249716], ["stacks", 8.835478758140658], ["storefront", 8.834797696667643], ["liko", 8.833761902369229], ["pygame", 8.833509169788268], ["americanah", 8.830389191594467], ["entrusting", 8.829294181839604], ["waned", 8.828100863090482], ["istanbul's", 8.825324127217026], ["containerization", 8.82435937128759], ["resist", 8.823096251542234], ["starlight's", 8.821962098978085], ["rellenos", 8.820913862036667], ["layers", 8.819493961355377], ["handspun", 8.817907888454657], ["bibliophile", 8.815696500896426], ["riley's", 8.814009609105208], ["glumly", 8.809975737430563], ["kathakali", 8.80609526067057], ["samual", 8.805962446733846], ["beautifully", 8.804300944385405], ["reflective", 8.804260135555793], ["wida", 8.804175885068856], ["coursework", 8.803215429517758], ["amid", 8.801772949603846], ["zamenhof", 8.801471899846169], ["krathong", 8.801471899846169], ["hailing", 8.801138983651375], ["ginkaku", 8.800755459317081], ["rmds", 8.800755459317081], ["nmai", 8.800755459317081], ["fatoumata", 8.798499433821227], ["korg's", 8.798499433821227], ["huayna", 8.796899706651443], ["memorization", 8.795740007263579], ["hina's", 8.79404449739904], ["patterns", 8.793937565924121], ["aliza", 8.793630698956663], ["existential", 8.792617860640382], ["secluded", 8.791066218782312], ["scrimping", 8.790596590018396], ["olga's", 8.790357176057887], ["signora", 8.787894398381985], ["indelibly", 8.786085112702276], ["freja", 8.784929608823793], ["turned", 8.784223331995985], ["anagrams", 8.783672332923492], ["feasted", 8.782970608127274], ["falafels", 8.782926868451634], ["oversimplify", 8.78141999374381], ["conclusion", 8.780054483952375], ["turmeric", 8.776200402867497], ["panting", 8.775544085758987], ["debussy's", 8.774343553557465], ["aubrey's", 8.77220805977342], ["terroir", 8.770489021066721], ["gleefully", 8.768470724310774], ["toho's", 8.762886459922429], ["chengdu's", 8.762886459922429], ["marimbas", 8.760073334253113], ["bluestones", 8.760073334253113], ["dadaism", 8.760073334253113], ["faltered", 8.758829495803445], ["shadows", 8.756396701313127], ["sentiments", 8.7550596311236], ["mentee", 8.75472153419929], ["diwali", 8.754312135926652], ["hartley's", 8.75143769266358], ["phonenumber", 8.75143769266358], ["eyck's", 8.750579035119168], ["reinterpretations", 8.748999159668985], ["nkosi", 8.746148362189993], ["empathically", 8.746148362189992], ["tokenized", 8.745131486435756], ["abena", 8.745131486435756], ["hartwell's", 8.740509323916497], ["kashubian", 8.740509323916497], ["nonsensical", 8.738678850759657], ["krok", 8.737218056441696], ["shuffled", 8.734910688169018], ["smoother", 8.730641006937883], ["preloved", 8.730066660447063], ["acumen", 8.729897111667485], ["irreparable", 8.729897111667485], ["aamir", 8.727699330648841], ["lebkuchen", 8.726897852221855], ["querida", 8.726172785933038], ["laplace's", 8.725017460527777], ["stopwatches", 8.72344595161918], ["dotting", 8.719564324614947], ["messiness", 8.717648357942872], ["vaporwave", 8.71648211735987], ["quokka", 8.71552624295798], ["erhu", 8.71552624295798], ["sp\u00e4tzle", 8.71552624295798], ["humbert's", 8.71552624295798], ["shyest", 8.71487733176425], ["jupyter", 8.714647962706959], ["gadgets", 8.714097170357869], ["joked", 8.713347464169823], ["percussions", 8.709677815131148], ["teapots", 8.708766393237887], ["eira", 8.70851780572345], ["decluttered", 8.705145863557295], ["birdsong", 8.702407713960454], ["melancholic", 8.701778562820968], ["communal", 8.701471639334228], ["tenzin", 8.69894669122341], ["pueden", 8.69894669122341], ["treetops", 8.698600518070684], ["grimsby's", 8.698378578194987], ["terroirs", 8.696281572751834], ["mountainsides", 8.69608603205135], ["impending", 8.695097599767116], ["huitlacoche", 8.694722261012055], ["truly", 8.692171883558087], ["detracting", 8.691592365399437], ["brigid's", 8.687093973364211], ["floorboards", 8.686537242195527], ["tarnished", 8.685511296310812], ["divisiveness", 8.684118941181554], ["ultramarathons", 8.684118941181554], ["erik's", 8.684118941181554], ["sagacity", 8.684118941181552], ["burj", 8.684118941181552], ["ardent", 8.68346941844848], ["collector", 8.681677438755637], ["handmade", 8.681280066078193], ["awaiting", 8.677275837796278], ["roared", 8.677162370601167], ["biodiverse", 8.676947909851098], ["clandestine", 8.676460987970636], ["eleanor", 8.67278848966608], ["plotlines", 8.671713056979865], ["nonjudgmental", 8.671619309744084], ["stronger", 8.671398541605738], ["schnitzels", 8.67028338259231], ["revelers", 8.67017974544932], ["thorniest", 8.667124579261431], ["memories", 8.666687888695636], ["hitsville", 8.664427061496333], ["overcommitment", 8.664427061496333], ["chastened", 8.664109450533669], ["various", 8.663649232248767], ["homesickness", 8.660533609999636], ["mothing", 8.655268047689255], ["disbelievingly", 8.655268047689255], ["nada's", 8.65348712833788], ["monotremes", 8.65194229627804], ["rainfalls", 8.650888894212747], ["onesource", 8.646525352691588], ["knut's", 8.644099959885786], ["abuelos", 8.644099959885786], ["community's", 8.640765044777805], ["ramprasad", 8.640674210514165], ["afterward", 8.640624938876819], ["intermingling", 8.640126444012447], ["sarawat", 8.639812211889812], ["letdowns", 8.637430129669823], ["blissfully", 8.635554981673167], ["paving", 8.63458974569953], ["diferentes", 8.632427757007854], ["syntaxes", 8.632231212581338], ["keychains", 8.630513268705124], ["upcycled", 8.62898167806294], ["ardabil", 8.626608219716775], ["disassembling", 8.625442461849245], ["faline", 8.625043302125897], ["dialects", 8.622771162162218], ["ten\u00eda", 8.622529445144803], ["kilmarnock's", 8.622529445144803], ["jaymee", 8.622089520173112], ["sysadmin", 8.618330161324117], ["within", 8.617271525872239], ["sarees", 8.613452512184596], ["magical", 8.613420348080368], ["outmaneuvering", 8.610476706194364], ["capulets", 8.601413046503634], ["ethics", 8.60103953601154], ["ulithi", 8.60009502427769], ["arborio", 8.599949225358891], ["hamilcar", 8.598650427892263], ["assuring", 8.598593527366885], ["flared", 8.597653687654637], ["cyberstalking", 8.59354966772271], ["invigorating", 8.591710120312369], ["acknowledged", 8.589420691775334], ["pursuit", 8.589101146273169], ["appreciative", 8.587153409490199], ["bombshell", 8.586455597466138], ["deflated", 8.585684343611485], ["permaculture", 8.584871867568049], ["blurrier", 8.57804632533069], ["sondre", 8.57804632533069], ["remembering", 8.577864180969621], ["superficialities", 8.576907596228693], ["companion's", 8.576241687253832], ["tourists", 8.576133809999904], ["significantly", 8.574003311956195], ["hanfu", 8.5737000765592], ["scanning", 8.573529344869364], ["skyrocketing", 8.573228356138532], ["archaeoastronomy", 8.57317200929654], ["situaci\u00f3n", 8.57317200929654], ["desperation", 8.572640714129767], ["mathematical", 8.569759244995083], ["karim's", 8.568092686057852], ["echoes", 8.565481693802942], ["serenaders", 8.564061997294251], ["daises", 8.564061997294251], ["humanizing", 8.562275435629262], ["precious", 8.56197315607428], ["workspaces", 8.561591689806857], ["ngozi", 8.559493620434003], ["perrotin", 8.55909840271493], ["individuals", 8.558965462323348], ["sizzled", 8.558867225683741], ["nods", 8.558819510744504], ["audiophile", 8.556540699391146], ["stroopwafels", 8.555782208060643], ["rada's", 8.555782208060643], ["unravel", 8.55365706215735], ["enron's", 8.553530686426942], ["beyond", 8.55278544455426], ["rainstorms", 8.552665863547775], ["lyocell", 8.55254138146668], ["encapsulates", 8.551577443300271], ["dilemma", 8.551512283234274], ["customizable", 8.550232246008342], ["snorkeler", 8.550001274136278], ["ashwood", 8.548455121848207], ["exclaiming", 8.544869220550362], ["surroundings", 8.544175741758947], ["neuroplasticity", 8.542913755146081], ["macronutrients", 8.54264833186406], ["relics", 8.539814385363309], ["yared", 8.538167362338166], ["ramose", 8.537889887853673], ["nerdy", 8.537427742850783], ["meena's", 8.535672463554517], ["humboldt's", 8.535672463554517], ["grader", 8.53338077830083], ["user", 8.523424763573159], ["fennec", 8.521798960972552], ["hurriedly", 8.521254010819037], ["reality's", 8.521213391058378], ["tariq's", 8.520372371181645], ["emblazoned", 8.517019276424124], ["pulmonologist", 8.51662531008325], ["ladled", 8.5150927554002], ["clubgoers", 8.513842099197602], ["eddie's", 8.510375599781993], ["valencian", 8.51020467667191], ["neighboring", 8.508829322299404], ["tkinter", 8.508747803251934], ["ovid's", 8.506892024014583], ["utrecht's", 8.506892024014583], ["burly", 8.505819546452301], ["gentrification", 8.504407534521588], ["crunched", 8.504033525350598], ["exquisiteness", 8.50124335733197], ["pounding", 8.498697661177829], ["reconsider", 8.497594874261562], ["esoteric", 8.497192442737202], ["pandals", 8.497180960060227], ["discordantly", 8.497180960060227], ["otakus", 8.495858441622865], ["digitalization", 8.492483331561983], ["panditji", 8.49071922548984], ["octogenarian", 8.48733595039616], ["motown's", 8.485809268069069], ["gr\u00eamio", 8.484024034702715], ["aili", 8.484024034702715], ["favoring", 8.483913637512934], ["iterations", 8.483124383185213], ["waterman's", 8.482655864402005], ["overexpansion", 8.482655864402005], ["preheated", 8.48157389299073], ["eccentric", 8.480401263764362], ["simultaneously", 8.477996851641347], ["microlearning", 8.477354204486755], ["imprints", 8.476602332716956], ["praised", 8.474282271255861], ["legalese", 8.472759432560203], ["sushi's", 8.472311162128344], ["overmix", 8.472311162128344], ["undervalued", 8.470487478913865], ["nala's", 8.470224385980037], ["uplifting", 8.469835486788762], ["underserved", 8.469032403938666], ["applause", 8.468484119628545], ["folklores", 8.467508264644144], ["meeples", 8.466177732232982], ["nilu", 8.463827539469337], ["caff\u00e8", 8.463336256236259], ["atomium", 8.462928757740606], ["subtlest", 8.461816969937434], ["campfire", 8.459631512770057], ["fruitful", 8.459572437131001], ["autoencoders", 8.458557410241774], ["aesthetics", 8.4583254703642], ["overlook", 8.456128797887995], ["photosynthesize", 8.454595556653839], ["redbeard", 8.454543859869178], ["alessandro's", 8.453028941346188], ["unsplash", 8.452822795048387], ["anna", 8.452330369686768], ["colomba", 8.451817523859111], ["patronizingly", 8.451170023857282], ["geopolitical", 8.449199006298526], ["athena's", 8.447279333694782], ["comprehension", 8.446672985493333], ["affecting", 8.445120112202027], ["subtextual", 8.443795529944667], ["collectively", 8.439942612189178], ["staunchest", 8.439760615996791], ["zephyrus", 8.43969073943826], ["helpfulness", 8.439064438432162], ["allegiances", 8.437111403385082], ["brightened", 8.435658553164322], ["ourselves", 8.435316999745783], ["kostas", 8.435078886575178], ["params", 8.43442840336628], ["expansively", 8.434039852530926], ["disillusioned", 8.431893348471739], ["councilwoman", 8.431183438040344], ["recyclability", 8.425663020313054], ["narong", 8.425663020313054], ["playfulness", 8.42551741134355], ["aromatherapy", 8.425027993480454], ["cityscapes", 8.424265961281334], ["carefree", 8.419193285003573], ["thuli", 8.41735358152104], ["potterheads", 8.41442580636225], ["changemakers", 8.412371001525532], ["goong", 8.41141993860575], ["alderton", 8.410768950296903], ["raucously", 8.409357747452894], ["granduncle", 8.409357747452894], ["volcanology", 8.407923979249755], ["negre", 8.401453225193727], ["alisa's", 8.401453225193727], ["chatting", 8.398720416677525], ["dr\u00f4me", 8.397829305757986], ["sokrates", 8.396995857037684], ["earthy", 8.39228528424661], ["steamtown", 8.392215783494779], ["mispronunciations", 8.390888950959837], ["modernity", 8.390698871018035], ["stark", 8.385647962446825], ["kaito", 8.385364995100089], ["digitize", 8.385338483968125], ["yellowstone's", 8.384881533301643], ["artistic", 8.38327624929062], ["plateful", 8.382923450376566], ["farofa", 8.382354190329684], ["wordplay", 8.382062630183933], ["resonates", 8.381208601923674], ["kayapo", 8.380133699153438], ["coalwood", 8.380133699153438], ["hana", 8.380098198708088], ["microexpressions", 8.37662266609698], ["multilingualism", 8.373971836139356], ["timur's", 8.373971836139354], ["montserrat's", 8.373971836139354], ["bistro's", 8.373971836139354], ["tygra", 8.373971836139354], ["permeating", 8.370406926885996], ["tousled", 8.370235124030414], ["compromise", 8.369308631439429], ["sungwoo", 8.36904886621805], ["slinked", 8.36871510554542], ["penned", 8.368285805880248], ["karthik's", 8.36675911276628], ["djellaba", 8.36675911276628], ["w3schools", 8.36675911276628], ["swiveling", 8.365852801975945], ["meditative", 8.365404236552555], ["mobil's", 8.364573439016864], ["theodora", 8.36261433651809], ["interaction", 8.359593734449193], ["maximilian's", 8.358574462130168], ["trendiness", 8.358042830744438], ["janie's", 8.35764830429503], ["attending", 8.355180570363268], ["basecamp", 8.352862347314494], ["backstories", 8.351497408237748], ["prebiotics", 8.349699454005618], ["chloroplasts", 8.34964657299879], ["sundaes", 8.346490447084982], ["m\u00fdvatn", 8.345776644771883], ["leaped", 8.345010731254261], ["poate", 8.342821690218086], ["enticingly", 8.339896083443843], ["showdowns", 8.33987189265304], ["pandanus", 8.33755780356355], ["ungracefully", 8.337287769951567], ["rivulets", 8.337073305957485], ["printf", 8.332060065187706], ["overgrazing", 8.331105976092871], ["latte", 8.326209181962858], ["straying", 8.324601480642842], ["incantations", 8.324601480642842], ["collaging", 8.323948109519645], ["novalee", 8.323948109519645], ["gibberellins", 8.323948109519645], ["planet's", 8.323350124741662], ["gherardini", 8.322934970752664], ["dishware", 8.321391614400014], ["sheer", 8.319118367757381], ["braving", 8.314407160336549], ["audacious", 8.31375247679895], ["hastily", 8.311942415130915], ["finneas", 8.311942415130915], ["wilma's", 8.311942415130915], ["webdriver", 8.311942415130915], ["cumulonimbus", 8.310571599197027], ["formidable", 8.308889512978373], ["clanging", 8.307701087198716], ["harmoniums", 8.307511742201742], ["althea's", 8.307511742201742], ["anishinaabe", 8.307184996410504], ["seasonings", 8.306150970417596], ["biologist", 8.304742571621006], ["remarkable", 8.304626124447868], ["barista's", 8.304346975823758], ["esthetician", 8.303823753827858], ["bacalhau", 8.3027707497678], ["brainbox", 8.301335153545828], ["conferencing", 8.301308800100898], ["distancing", 8.301154689738143], ["adamantly", 8.299283349618989], ["negatively", 8.298523805688273], ["exemplified", 8.295401236195332], ["craftily", 8.295058667400808], ["diocletian's", 8.29382146067901], ["intellectus", 8.29382146067901], ["probiotics", 8.292875487315944], ["shakshuka", 8.292354176917723], ["overconfident", 8.292127461197664], ["glowed", 8.291895435975963], ["chartbusters", 8.289913274959693], ["psychographics", 8.288647523398488], ["elegantly", 8.287875265714993], ["enthused", 8.286676426711], ["scattered", 8.283221397043405], ["app\u00e9tit", 8.278108185489948], ["mark's", 8.277094425800332], ["fixit", 8.274919619345773], ["heartbroken", 8.272359438300628], ["kirtans", 8.270589467791956], ["vatnaj\u00f6kull", 8.270589467791956], ["thingamajigs", 8.270589467791956], ["kmart's", 8.270589467791956], ["ballybunion", 8.270589467791956], ["bhasha", 8.270589467791956], ["gyokuro", 8.270589467791956], ["nikhil's", 8.270589467791956], ["dechen", 8.270589467791956], ["tawaf", 8.270589467791954], ["realmente", 8.270589467791954], ["atun", 8.270589467791954], ["chilly", 8.270327740277152], ["facing", 8.26502775810238], ["homophones", 8.260251230957216], ["fritz's", 8.260199280018348], ["jackfruit", 8.260120367199812], ["oversimplified", 8.259259893178543], ["clientele", 8.25738641834036], ["guruji", 8.25689643887177], ["kael", 8.25632983077852], ["tommy's", 8.25441792413985], ["sympathized", 8.25335907306739], ["bookstore's", 8.250803368586713], ["bouillabaisse", 8.249273515555378], ["bioplastics", 8.242835811859766], ["subtleties", 8.24262840019622], ["mindlessly", 8.24248552299849], ["knowledge", 8.240894242778486], ["exorbitant", 8.238966625709223], ["oasis", 8.235046438674171], ["doubs", 8.23447335657889], ["allocate", 8.234101573081109], ["monterrey's", 8.232995879301992], ["intermission", 8.23288530992408], ["roleplayed", 8.23194185345648], ["sylvia's", 8.230245128924677], ["grumpy", 8.228675802522975], ["crosswords", 8.225732033390372], ["vinyasa", 8.225101225719099], ["spectacles", 8.223466341754538], ["potential", 8.221938941510826], ["salinger's", 8.221651068574252], ["aquatica", 8.220363215963259], ["region's", 8.220290780824568], ["staring", 8.21826899963701], ["seulement", 8.217572868639442], ["facets", 8.216512536656392], ["qanun", 8.215817352111214], ["w\u00fcrde", 8.21470710652309], ["tmbg", 8.213550919738218], ["glance", 8.213061566881299], ["winding", 8.212066583253778], ["sunhat", 8.210439726208014], ["tempests", 8.21007295949104], ["microbreweries", 8.209776309940544], ["chewiness", 8.209776309940544], ["timothy's", 8.208753284855193], ["garlicky", 8.20782160129532], ["kibo", 8.207760404155415], ["betrayals", 8.207695251306845], ["aniket", 8.20696954880894], ["aglio", 8.20515758909107], ["diner", 8.204725998685864], ["qawwali", 8.204688356494808], ["microbiomes", 8.20434904871353], ["nostalgic", 8.203280947123222], ["resoluteness", 8.203074451728348], ["gravelly", 8.202237488719295], ["neoclassicism", 8.198671298506808], ["bari's", 8.198671298506808], ["fruitopia", 8.198671298506808], ["barren", 8.194803189800236], ["sunflowers", 8.192387978875617], ["ahimsa", 8.19182194905108], ["portrayals", 8.190636067231685], ["wracking", 8.189649535277978], ["hardier", 8.187883573114036], ["rainforest", 8.184339034770696], ["pasi\u00f3n", 8.183986436715601], ["nefertari", 8.182604473453743], ["vigen\u00e8re", 8.182604473453743], ["chickpeas", 8.181445389695993], ["rawer", 8.181338502312185], ["poppop", 8.18069175618552], ["stigwood", 8.178694029260933], ["bufala", 8.176605496567046], ["pectore", 8.176605496567046], ["commiseration", 8.17349000580821], ["competitor's", 8.173288415229697], ["findings", 8.171342394178453], ["danced", 8.169753888476913], ["kenjiro", 8.167207099444555], ["taizong", 8.167207099444555], ["faintly", 8.16701386137288], ["summiting", 8.166687590055876], ["fujitsu's", 8.165898461870539], ["clarity", 8.164290518775813], ["eirene", 8.164059626871918], ["strasbourg's", 8.163872184336576], ["arccos", 8.163872184336576], ["linguist", 8.163440130557404], ["elation", 8.161635594563558], ["hurtling", 8.157808702322065], ["thalia's", 8.157293721657819], ["attire", 8.154560324287496], ["iconic", 8.153974156296089], ["cormac's", 8.152438189680643], ["sranan", 8.148963152089133], ["zaporizhia", 8.148963152089133], ["deescalate", 8.148162978959508], ["perfectionism", 8.147555905461166], ["amassing", 8.146402598074337], ["figments", 8.145277506158743], ["hopeful", 8.14405078938684], ["annan's", 8.141361507357706], ["affable", 8.141191842639412], ["samwise", 8.140384973651654], ["concepts", 8.13935066348985], ["septuagenarian", 8.139120560066939], ["history's", 8.137327802234324], ["hydration", 8.136123923795736], ["dismay", 8.134715497963946], ["scathing", 8.132303084407653], ["interplays", 8.130883564619795], ["blazed", 8.129955962671605], ["gentle", 8.128117387736985], ["roundedness", 8.12799309765761], ["chuckles", 8.125349847869755], ["gust", 8.122900370152813], ["warmed", 8.120176883064145], ["kumbh", 8.119857647063574], ["lucienne", 8.117995566172178], ["impalas", 8.11236949536463], ["slunk", 8.111028256194682], ["unscathed", 8.10890240654853], ["vexillology", 8.104346463414227], ["flashy", 8.104248398720635], ["comptoir", 8.101801927632936], ["jumpstarting", 8.10064584859075], ["proved", 8.094592967080814], ["divisible", 8.09420648832578], ["journeyed", 8.0929795643102], ["frigatebirds", 8.090794044579086], ["rudds", 8.090794044579086], ["lexicographically", 8.090794044579086], ["emilys", 8.090794044579086], ["telophase", 8.088149994237721], ["exploitations", 8.087611824699215], ["johnstons", 8.087380207429476], ["seahorses", 8.087380207429474], ["quirk's", 8.085978095743029], ["hyperrealistic", 8.085978095743029], ["anchorage's", 8.085978095743029], ["fraction's", 8.085978095743029], ["rollercoasters", 8.085978095743027], ["jorma", 8.085771826053588], ["benim", 8.083834221745041], ["hostel's", 8.083834221745041], ["vscode", 8.083834221745041], ["suryavarman", 8.083834221745041], ["prodigies", 8.082068678452579], ["foster", 8.077158158042897], ["liang's", 8.075102807644145], ["corcovado", 8.07483587092114], ["caligula's", 8.074338192285028], ["meiling", 8.074338192285028], ["queries", 8.07394300544605], ["mwamba", 8.073175900093403], ["stargaze", 8.072561269267359], ["crumbling", 8.072095320564948], ["ached", 8.071065519541722], ["unlikely", 8.070270925829405], ["selflessly", 8.069187505247742], ["debriefings", 8.068608491072101], ["cryptocurrencies", 8.063824731097156], ["overexert", 8.061411134948969], ["vindicating", 8.058430868400771], ["euphorbias", 8.058027588946835], ["victoriously", 8.058027588946834], ["immortalized", 8.057778978562222], ["agritech", 8.055209533734873], ["sternness", 8.051016296080665], ["tilling", 8.050281189392257], ["cappuccinos", 8.049399284351008], ["connection", 8.047794171163497], ["rothko's", 8.043997975523682], ["santiago's", 8.041828482512603], ["puzzling", 8.04176982584971], ["istv\u00e1n", 8.041214957987899], ["boardgames", 8.039782319853575], ["edmund's", 8.039339433330667], ["towered", 8.03788086679705], ["chonky", 8.034286911569328], ["drenching", 8.031177667408503], ["waterscape", 8.02733683638631], ["innerhalb", 8.02733683638631], ["grandaunt", 8.02733683638631], ["despu\u00e9s", 8.025847534561377], ["glow", 8.024760692282493], ["rocketry", 8.021045820056854], ["customized", 8.01916354797108], ["rallying", 8.018193892654166], ["discarding", 8.017784000254185], ["corvid", 8.017776689300499], ["fluffernutter", 8.016109791859897], ["lichtenstein's", 8.016109791859897], ["authentications", 8.015037546034339], ["rayel", 8.015037546034339], ["overusing", 8.01239808727235], ["afternoon", 8.012239883073756], ["odori", 8.012133546923456], ["heirlooms", 8.011203371372773], ["wingspans", 8.008409398207771], ["lasagnas", 8.006985022603367], ["cinemagoers", 8.006634484777319], ["colossal", 8.003915974054816], ["ancash", 8.003796259153505], ["adesina", 8.003796259153505], ["drachm", 8.003796259153505], ["s\u00e9amus", 8.001795310088717], ["etwas", 8.000529403537524], ["quietly", 8.000475535173644], ["deter", 8.000152146618294], ["kael's", 8.00005616744362], ["talents", 7.996941989902633], ["taro", 7.996169220238353], ["embeddings", 7.995320226806444], ["transfixed", 7.994486707613402], ["smithson's", 7.9891296167938], ["receptionist's", 7.9891296167938], ["hindering", 7.988747993566378], ["autoscaling", 7.988637554117229], ["folktales", 7.987350102456615], ["quilla", 7.987350102456614], ["arachnids", 7.987159154569872], ["dinnertime", 7.986287954836607], ["encounters", 7.986039837152573], ["soilless", 7.985396727523266], ["farid", 7.984986378256694], ["understood", 7.984133248633155], ["blurring", 7.983567366195886], ["spitfire's", 7.982916790651365], ["rainforests", 7.982622579958836], ["giulia's", 7.98208053286898], ["reams", 7.97894236550666], ["macronutrient", 7.97777235513169], ["enduring", 7.977672757474324], ["francesco's", 7.977239040444643], ["friedrich", 7.97545404911253], ["cajeta", 7.975211272513672], ["simran's", 7.975211272513671], ["contentious", 7.974429389055581], ["huffed", 7.9736965125378845], ["ergonomic", 7.972571020488838], ["grassroots", 7.972286168055575], ["fairer", 7.970024143230735], ["disinterest", 7.969840759872247], ["warmhearted", 7.9697243710533945], ["undiscovered", 7.969540011164328], ["fretboards", 7.968006926287372], ["impacting", 7.966859199405802], ["thanking", 7.966001897966022], ["edna", 7.9645776574836535], ["chagrined", 7.964271339355216], ["rekha", 7.963777277857738], ["soundboards", 7.9636025224809], ["reefs", 7.963602522480899], ["boutiques", 7.963106856779329], ["glued", 7.962936412102885], ["timelessness", 7.962641987608212], ["scrabble", 7.9623220421742555], ["langar", 7.960684476024575], ["compartir", 7.960442362749757], ["dabei", 7.960442362749757], ["quillwork", 7.960442362749757], ["unrelated", 7.959229954626507], ["sneakily", 7.956653480140167], ["undeciphered", 7.954361046964616], ["ananse", 7.954361046964616], ["amundsen's", 7.95248987287688], ["entomology", 7.952440631453518], ["evoking", 7.951681145093198], ["crawlies", 7.951043965627267], ["connect", 7.949432140520758], ["sissinghurst", 7.945762691111304], ["tintin's", 7.94534156737317], ["flocking", 7.9429199274366376], ["taxidermist", 7.942617816482964], ["impishly", 7.942084745940406], ["agumon", 7.939765889080277], ["marek", 7.937138319052687], ["maneuvered", 7.937097956993893], ["downtimes", 7.937097956993892], ["predispositions", 7.936246489306748], ["impressed", 7.9352340592349115], ["strained", 7.934276559787494], ["unaddressed", 7.930490461639763], ["raged", 7.930056936226782], ["laced", 7.929586702042996], ["claude's", 7.929427652245538], ["envious", 7.929303095175239], ["differed", 7.927424957151897], ["devastating", 7.927324960007963], ["unsaid", 7.9271068235683515], ["commodifying", 7.925981573300624], ["sa\u00f4ne", 7.925134870857157], ["apoapsis", 7.923758296561971], ["initialize", 7.923048740156016], ["commenter", 7.922354121779662], ["redoubled", 7.921829670957354], ["ljubica", 7.921829670957354], ["vita's", 7.920838171890305], ["mlk's", 7.919545772215047], ["harsh", 7.916841972620636], ["judgmentally", 7.91659536605767], ["designing", 7.916265051739451], ["duxelles", 7.916135633458015], ["elderlies", 7.916135633458015], ["mesoamerican", 7.914356246739471], ["jubilant", 7.9141987272594685], ["schmidts", 7.914098542456095], ["firelight", 7.9129423556712215], ["rehearsals", 7.912087422335476], ["baking", 7.910391379612338], ["shor's", 7.908133355657647], ["obstacle", 7.905738320984219], ["monsieurs", 7.905710520683487], ["assimilations", 7.905710520683487], ["continents", 7.903771351819042], ["geotags", 7.9030077136678685], ["nduja", 7.9030077136678685], ["curated", 7.902322674944274], ["adichie's", 7.899473273724367], ["yama's", 7.89465358289232], ["hoodoos", 7.89207278734381], ["cyberspace", 7.891767688177125], ["layovers", 7.890769055953059], ["spoke", 7.889776576817947], ["debating", 7.888225785611887], ["suddenly", 7.888167538053666], ["m\u00e1rquez's", 7.887184658026765], ["ahrefs", 7.887184658026765], ["friend's", 7.886577923142625], ["clickbait", 7.886079606570048], ["naadam", 7.8859108878946556], ["biosolutions", 7.8859108878946556], ["stouts", 7.885418719803692], ["ancestors", 7.8846863613157145], ["\u00e9ponine", 7.884113324437192], ["wargaming", 7.882805019983033], ["seagrass", 7.882341272251503], ["taqueria", 7.881385257542922], ["monika's", 7.881385257542922], ["magdalene's", 7.879412938909903], ["tanya's", 7.878308544418425], ["authentic", 7.877059783479019], ["mindlink", 7.876751874087576], ["tanaka's", 7.875521131607251], ["biomimicry", 7.875039536723644], ["initial", 7.873861106149786], ["abandoning", 7.873524593805832], ["unicycles", 7.8735236970899996], ["j\u00f8rn", 7.873463042407373], ["exploiting", 7.873217768528177], ["metaphorical", 7.8718289041662715], ["championed", 7.868600351799277], ["whimsicality", 7.867146079119178], ["stagnancy", 7.86540208752549], ["ribicoff", 7.864444449284315], ["gongo", 7.864444449284315], ["tirana's", 7.864444449284315], ["thingamajig", 7.8636840363102], ["advocated", 7.860904363878592], ["clinging", 7.860671605523664], ["sartorial", 7.860671605523664], ["collatz", 7.859225070179267], ["preheating", 7.858389671165669], ["symphony", 7.8578322905326266], ["cobblestones", 7.857059994402358], ["reframe", 7.857059994402357], ["tzolk'in", 7.857059994402357], ["coronis", 7.857059994402357], ["democratizing", 7.8570599944023565], ["nermin", 7.855254188841704], ["rizal's", 7.855254188841704], ["maestro's", 7.854749773880628], ["scriptorium", 7.854749773880628], ["kneaded", 7.854265876338914], ["enlightenment", 7.853613915457443], ["antonym", 7.852882929014584], ["reshaping", 7.852037369219489], ["flutterby", 7.851825444106287], ["virtual", 7.8515594588233], ["trodden", 7.850280822707446], ["consumerism", 7.8501317519162495], ["takeout", 7.849463586165579], ["typed", 7.849439058487926], ["kragujevac", 7.849402041191438], ["anticipated", 7.849202934407955], ["audiences", 7.848316799822121], ["autoencoder", 7.846456674571855], ["takeaways", 7.846177639839474], ["joka", 7.84536635890805], ["mutualistic", 7.845244866591226], ["cyto", 7.844897362832075], ["professionally", 7.843539378576749], ["pranayama", 7.843326543256579], ["documentaries", 7.843230035192971], ["mismatched", 7.838543749325212], ["confrontations", 7.8380158739173105], ["celebrating", 7.837417344416352], ["marisol", 7.836800625389033], ["adaptable", 7.835443681020628], ["culminated", 7.833950994418821], ["concocted", 7.8336020826217485], ["mealworm", 7.8314431243693745], ["ideologies", 7.83066056819936], ["storytelling", 7.8300946151747635], ["realm", 7.82929958105829], ["lentil", 7.829008760977681], ["welcoming", 7.82813399388682], ["playtest", 7.82789418771493], ["rebecca's", 7.8269851236103865], ["laborious", 7.82494756302979], ["dungeons", 7.8217883628485385], ["alphabetization", 7.821100909759784], ["drifted", 7.8210169585894125], ["cautioning", 7.8198888057830676], ["longing", 7.819240368513245], ["fortuitous", 7.819073988496007], ["virality", 7.814382575440442], ["goi\u00e1s", 7.811112275136846], ["aiden", 7.808155639288458], ["strife", 7.807304655373011], ["taisha", 7.8069065342429775], ["rediscovered", 7.8059642060673085], ["lissitzky", 7.805949610050833], ["traversals", 7.805026484505653], ["absentmindedness", 7.805026484505653], ["layoffs", 7.802983063266795], ["nanotech", 7.802873373751307], ["solitude", 7.799669021859695], ["vardar", 7.798777316944762], ["socioeconomic", 7.798712685143276], ["sza's", 7.797984355346701], ["kushites", 7.797984355346701], ["logbook", 7.796862665997542], ["headcounts", 7.795589397006606], ["sabah's", 7.794272796872], ["handwritten", 7.79193876424708], ["impeccable", 7.789919534958766], ["creature's", 7.787528490027115], ["linkedlist", 7.785761809335185], ["jumbled", 7.785325289834774], ["bravely", 7.78435884504838], ["overwhelming", 7.784278807085788], ["pierogis", 7.7840842049806644], ["reshape", 7.781720616659314], ["quidditch", 7.779617202113032], ["amenhotep", 7.778489394458335], ["hindsight", 7.778319450839132], ["v\u00e9lo", 7.778292475661481], ["lifelike", 7.776855879456688], ["unwitting", 7.776574392199013], ["stereotypes", 7.776080736356545], ["hardworking", 7.773717900534608], ["restoring", 7.7736451920557705], ["khafre", 7.773307189665225], ["algunos", 7.77177959716637], ["kalalau", 7.77071867578255], ["accessorizing", 7.770099938727389], ["belonged", 7.7677862786764855], ["hartman", 7.767775221738694], ["bundren", 7.7671622827959235], ["avial", 7.7671622827959235], ["poodle's", 7.7671622827959235], ["worthington's", 7.766931775843086], ["milo\u0161evi\u0107", 7.766931775843086], ["jovially", 7.765965546223032], ["ramya", 7.765396030527586], ["progressions", 7.764651173533173], ["redemption", 7.762967962938219], ["traversed", 7.756957677694652], ["occitan", 7.755418069966531], ["delicious", 7.754527344150963], ["eatery's", 7.753677626054959], ["carnavalet", 7.753677626054959], ["trishula", 7.753677626054959], ["devoid", 7.751554946248586], ["joonas", 7.7496911594091715], ["abdul's", 7.7496911594091715], ["prosperity", 7.749481293760414], ["hailed", 7.748659064484696], ["shamisen", 7.747348093298994], ["slowly", 7.746530442182323], ["hygge", 7.7449164083983995], ["ondra", 7.743270005080387], ["c\u00e9line's", 7.742679501762681], ["cuon", 7.742679501762681], ["positivo", 7.742679501762681], ["backbreaking", 7.741656420433167], ["courage", 7.7405594422818815], ["recursion", 7.740494639487168], ["mandalay's", 7.74042347626683], ["someday", 7.740274840444206], ["winemaking", 7.736150885475178], ["massenet's", 7.735988627668521], ["microtones", 7.735433678699534], ["nat\u00fcrlich", 7.734532743027662], ["hartman's", 7.732563224047547], ["regimen", 7.732364953195648], ["teff", 7.72739397308528], ["approachable", 7.727355178025547], ["fulfilling", 7.726471739647747], ["dissected", 7.724303299993151], ["deafening", 7.7241243417001515], ["crisp", 7.72086765286686], ["outcomes", 7.719216836605826], ["commitment", 7.718858136938696], ["unpredictable", 7.7178067500229375], ["replayability", 7.714204358140496], ["exorbitantly", 7.713647416088794], ["weightier", 7.713647416088794], ["yielded", 7.71309047403709], ["marjorie's", 7.71176585510331], ["tribe's", 7.710677879928163], ["mentorships", 7.710430021766205], ["catalysts", 7.710355490747476], ["aprender", 7.710199059927547], ["tunes", 7.708727410225847], ["feathery", 7.7085105719225995], ["leclerc", 7.708308385269409], ["broader", 7.707824505034993], ["willingness", 7.7070382580652534], ["animosity", 7.706736442589111], ["paced", 7.7063438805471405], ["mycology", 7.7051003686387105], ["costliness", 7.700203987254579], ["crosschecking", 7.6976269444208265], ["instead", 7.69653213072362], ["frayed", 7.6951871260549245], ["shaivism", 7.694788935224161], ["simplicity", 7.694386345872018], ["intolerances", 7.694033952008381], ["savvy", 7.693357004523334], ["comunidad", 7.693034337359556], ["antoinette's", 7.69216619603615], ["boldness", 7.691960695832455], ["rollo's", 7.691648205046519], ["agility", 7.691068490831487], ["microservices", 7.688921637090104], ["gumboot", 7.688817965304688], ["herr", 7.688729173469651], ["grumbles", 7.687442820571371], ["reznor's", 7.6874068771143165], ["quest", 7.685245168039194], ["offhand", 7.684675342731753], ["playtime", 7.682964086105337], ["shaken", 7.682297036050272], ["majapahit", 7.681284567138475], ["smokestacks", 7.680860479653746], ["v\u00e1clav", 7.680860479653746], ["saara", 7.680790543022351], ["fairey's", 7.679833077235388], ["alas", 7.6797194702372025], ["transform", 7.679693443906711], ["impressing", 7.679485573476236], ["transgressions", 7.6791230334967375], ["matej", 7.676569229773748], ["minimalism", 7.676336156535618], ["hinder", 7.675950963811729], ["petrolhead", 7.675235751238838], ["awareness", 7.6737386194898995], ["cemal", 7.672234924302184], ["vigilantly", 7.671876762211056], ["retries", 7.670304748355442], ["greenleaf's", 7.669092051952541], ["ridhwan", 7.669092051952541], ["outputted", 7.66909205195254], ["mathematician", 7.668119803974352], ["actionable", 7.666836436643143], ["cohesive", 7.666106678748926], ["misaligned", 7.665581366646085], ["reinvigorated", 7.665581366646085], ["quantifiable", 7.665559768470262], ["burdens", 7.664975554012889], ["toyland", 7.662457889277842], ["seeress", 7.662457889277841], ["gnudi", 7.662457889277841], ["ylang", 7.656855858360107], ["seashell", 7.656202821613125], ["whiskered", 7.656024255449733], ["maija", 7.655272089444195], ["emma", 7.654941382967406], ["baklava", 7.654809771609192], ["quaffle", 7.654694507424469], ["garuda's", 7.654694507424469], ["recipe's", 7.653799744770183], ["reassuring", 7.6535903132724155], ["foursome", 7.653238527980795], ["kirigami", 7.652227638424333], ["yele", 7.652227638424333], ["sensitively", 7.648925954815541], ["urgency", 7.648664437249121], ["disapproving", 7.646829926924964], ["sandefjord", 7.646075569203583], ["magnificent", 7.6443930643146345], ["surrounding", 7.643108677538943], ["supportiveness", 7.642910802825602], ["bemusement", 7.642298831426544], ["introspectively", 7.641647302940856], ["orishas", 7.641647302940856], ["greenmarket", 7.641647302940856], ["playtesters", 7.6407101374634285], ["merits", 7.639426854881293], ["rejuvenating", 7.638808327891182], ["mooncakes", 7.637709578082658], ["estaba", 7.636788013601869], ["anangu", 7.636788013601869], ["shamefacedly", 7.6343902779618045], ["wargamers", 7.6343902779618045], ["mysterio's", 7.634390277961804], ["frolicking", 7.632228436054616], ["dalloway", 7.631459984866375], ["rephrased", 7.629618784038079], ["ecstatically", 7.629039069823046], ["campiness", 7.628905802187408], ["gritted", 7.628567437919292], ["volcanologist", 7.628125734517088], ["dermatologist's", 7.627942313200013], ["spanakopita", 7.6269638282750725], ["escalate", 7.625188111108905], ["subtle", 7.624036136147319], ["motivations", 7.6194770916298555], ["veganism", 7.617267192546246], ["rituals", 7.616894800241475], ["polyrhythmic", 7.616270680783099], ["evoke", 7.61561214058456], ["beehives", 7.614646165173973], ["rushed", 7.612666240704882], ["jurgis", 7.612041360286272], ["starstruck", 7.611954776731039], ["topsoil", 7.611272053880653], ["alarming", 7.6085763550824135], ["soirees", 7.605751086136654], ["basils", 7.60360644619583], ["genau", 7.603606446195829], ["rivaled", 7.602911672215002], ["powerhouses", 7.600838562763269], ["otherworldliness", 7.5994159769502], ["undertones", 7.599084177411633], ["conveyed", 7.598727638117541], ["carthage's", 7.59686167406733], ["maximos", 7.59686167406733], ["glimpses", 7.594001890684845], ["fushimi", 7.593807984280629], ["riverbeds", 7.593637304947774], ["infamous", 7.59253357859531], ["learners", 7.592236815085974], ["brassware", 7.590814990987138], ["lightheaded", 7.589482099856148], ["sported", 7.589074413881724], ["stifle", 7.588679366172508], ["columbanus", 7.588065094236309], ["sown", 7.586943099326324], ["friendly", 7.586013662450453], ["serenading", 7.584506477850122], ["scss", 7.583315131736005], ["respite", 7.58319941599865], ["navigates", 7.58309671828175], ["risks", 7.582709034400446], ["salma's", 7.581373678809293], ["rajah's", 7.5813736788092925], ["shamefaced", 7.579387466160927], ["commendable", 7.578192682860141], ["affordably", 7.578168023976814], ["monument's", 7.5715627778985075], ["entonces", 7.570962564172093], ["shivering", 7.570286294377833], ["gasping", 7.568697031261932], ["laughters", 7.567589363029638], ["honeywood", 7.5667095130862565], ["grpc", 7.5667095130862565], ["meadowbrook", 7.565156836715583], ["szlachta", 7.564563537614594], ["invasives", 7.5634386954528665], ["puzzler", 7.561307902674353], ["pragmatic", 7.55996114944284], ["cipher's", 7.558551963942317], ["luigi's", 7.558399819176537], ["theyyam", 7.556311286482651], ["kallistos", 7.556311286482651], ["paneer", 7.552994205145301], ["enlightenment's", 7.551407774940481], ["routeburn", 7.551407774940481], ["menkaure", 7.551407774940481], ["healthily", 7.551265831185313], ["dizi", 7.54815846006314], ["soothingly", 7.547510475304364], ["planck's", 7.546686669735767], ["contentment", 7.546543666616061], ["opulent", 7.5442424960818135], ["chardonnays", 7.543635638490357], ["farmstead", 7.54174377094279], ["campsite", 7.540356934294226], ["salesperson", 7.540339443973909], ["became", 7.539794903318197], ["shelves", 7.538894013484865], ["goodheart", 7.538297691997876], ["knowing", 7.538117297256586], ["compost", 7.536957550185966], ["delphine's", 7.536908144036217], ["l\u00f6sung", 7.536284795417904], ["monotreme", 7.535425959543781], ["lukather", 7.535425959543781], ["lw\u00f3w", 7.534463703642933], ["discerning", 7.533325492377358], ["outdo", 7.533325492377358], ["merging", 7.530195740435898], ["extremophile", 7.529088343093366], ["aromatherapist", 7.52835707965678], ["confronting", 7.526236415690679], ["cherishes", 7.524791771242157], ["makeupaddiction", 7.523568483604294], ["andorra's", 7.523568483604294], ["transformation", 7.521335325635691], ["raving", 7.520077466087004], ["mathematicians", 7.5188847806081975], ["uilleann", 7.5173201668220555], ["churrasco", 7.5173201668220555], ["anthill", 7.516506310434453], ["keyboardist", 7.51474254513121], ["codenames", 7.513873163393465], ["temptingly", 7.5121057603185735], ["straightforward", 7.507945688983063], ["binges", 7.507698887573215], ["pursed", 7.506345124565609], ["miraculous", 7.504868164091737], ["diyer", 7.502959906410068], ["raclette", 7.501848780080523], ["ser\u00eda", 7.5012323079973555], ["spus", 7.5012323079973555], ["o'malleys", 7.5012323079973555], ["statista", 7.500135315849358], ["yosemite's", 7.498493299121504], ["bei's", 7.498061880141058], ["underqualified", 7.497081100300622], ["sapling", 7.496933333801483], ["infographics", 7.496698596162851], ["fiving", 7.496547120165272], ["regexp", 7.494166783060465], ["serverless", 7.492181047293889], ["eggplants", 7.489478240278271], ["unapproachable", 7.488848819466415], ["frigid", 7.486399021395556], ["demotivation", 7.486309432053063], ["sender's", 7.485281092845363], ["fraternit\u00e9", 7.482148059383773], ["lukas", 7.481477931512041], ["borneo's", 7.480018415723606], ["smokiness", 7.4797227277599845], ["minjoo", 7.4791454995821995], ["decomposer", 7.4791454995821995], ["antiquated", 7.479130014808908], ["deepening", 7.478747224553035], ["appetito", 7.478724518748044], ["sphere's", 7.478476673693571], ["vijayanagara", 7.478197662494522], ["gradgrind", 7.476035021104519], ["thoughtlessness", 7.47580016199981], ["dreamscapes", 7.475340480504267], ["handshakes", 7.474758147149732], ["beginner's", 7.474728750514046], ["kyubey", 7.473424217884297], ["arabella's", 7.473424217884297], ["twirling", 7.472227406176807], ["deadstock", 7.472131983149217], ["cultural", 7.4712119779325405], ["academia's", 7.471201638191246], ["dearly", 7.468999554804305], ["downtime", 7.4686563624339], ["kuya", 7.467855784153325], ["overcomplicate", 7.467330778474031], ["mwalimu", 7.465487838183889], ["chocoholic", 7.465247604954445], ["reflection", 7.461636406063872], ["ridzuan", 7.461510063334047], ["discover", 7.461253212729457], ["thundered", 7.461127519880401], ["akwasi", 7.459234171900972], ["bernini's", 7.458522526090329], ["bedtimes", 7.455459255822075], ["hammurabi", 7.4525202921734035], ["landlubber", 7.452145718375044], ["emotions", 7.44543911858225], ["colleague", 7.444309948292447], ["cinnamon's", 7.443530521012761], ["lifespans", 7.44353052101276], ["shilin", 7.44353052101276], ["flamengo's", 7.44353052101276], ["kekoa", 7.44353052101276], ["dynamodb", 7.44353052101276], ["nosiness", 7.443530521012759], ["chaac", 7.443530521012759], ["emile's", 7.443530521012759], ["histories", 7.443035276134449], ["reconcile", 7.4419601559239394], ["expressionism", 7.44019560590478], ["alaric", 7.4362114152890495], ["milliliters", 7.435314703660649], ["subquery", 7.433876136420007], ["experimented", 7.432191809645626], ["velociraptors", 7.431734117651408], ["discourse", 7.431051229212246], ["noctuidae", 7.430773827746243], ["generosity", 7.4307378826273265], ["challenge", 7.430634285656854], ["improvements", 7.430324257185156], ["resentment", 7.429509128548935], ["nourishment", 7.428457951421924], ["desolate", 7.428442283470167], ["cleverness", 7.4283396423984485], ["roamed", 7.425219252818387], ["warmups", 7.424921694710228], ["finserv", 7.424733726767777], ["jules", 7.423270352182262], ["participant", 7.4231626514279], ["sweetgrass", 7.422323881351755], ["paddleboard", 7.422017427021394], ["analyze", 7.42165887545487], ["remo's", 7.421377156366889], ["inkstone", 7.421377156366889], ["dingle's", 7.420341952411475], ["categorizations", 7.419541242169717], ["nehru's", 7.416743186704122], ["clapped", 7.415641323970206], ["debunking", 7.414833635848713], ["synesthetic", 7.413875020929044], ["francesca's", 7.413604045964829], ["chickpea", 7.412123219236333], ["waterings", 7.407830134820852], ["mellah", 7.40506266302303], ["telenovelas", 7.404302428675406], ["inclement", 7.404209176474922], ["unfulfilled", 7.404169901336429], ["smarty", 7.403937273560564], ["irrevocably", 7.403511539716993], ["corridors", 7.403407401103955], ["percival's", 7.400628327808075], ["tunde", 7.397582801747249], ["loaves", 7.397306673626663], ["cichlids", 7.397153383810188], ["differences", 7.3949939045741315], ["align", 7.394429948879616], ["diverged", 7.393906984206009], ["listlessly", 7.3929802288496855], ["morality", 7.392467627475425], ["worthiness", 7.39243076915226], ["bienvenida", 7.390739524409832], ["number2", 7.390739524409832], ["sikhism", 7.390108053158601], ["tokenism", 7.389591894048899], ["clucked", 7.387322631425825], ["sessions", 7.386101957721503], ["saguaros", 7.38560421345624], ["grandma's", 7.385260458853317], ["melanie's", 7.384454881957104], ["poisson's", 7.384454881957103], ["sarun", 7.384454881957103], ["spectacle", 7.38439682236098], ["pleadingly", 7.383562500702485], ["speediness", 7.380715917459909], ["brauhaus", 7.380715917459909], ["iridescent", 7.380616369752219], ["insignificance", 7.380449753885533], ["frideric", 7.378964580929626], ["agaves", 7.376471687490122], ["onto", 7.372633736254047], ["unaffordability", 7.370554731591066], ["uncovered", 7.370296530473931], ["collectors", 7.369999606727003], ["harmonizer", 7.366990361413657], ["raihan", 7.365993744752211], ["fosse's", 7.365993744752211], ["impacts", 7.365385044826427], ["taiko", 7.363257152648897], ["verschiedene", 7.362841843278205], ["pooling", 7.362420238450813], ["revelations", 7.36082462633484], ["intrepid", 7.358944946910342], ["notions", 7.3562020914359225], ["uncrossed", 7.352686517705204], ["mangosteens", 7.351635082481738], ["ivan", 7.351171797467044], ["forestland", 7.351000445475307], ["countered", 7.3509315767322185], ["gestures", 7.350087742835521], ["triangle's", 7.3483294911676715], ["crafter", 7.347229136798744], ["fragile", 7.343289605847188], ["typewriter", 7.342647506625407], ["panathenaic", 7.342257996917348], ["agribusinesses", 7.342257996917348], ["alfama", 7.342257996917348], ["verona's", 7.3401481526653605], ["lifelink", 7.3401481526653605], ["lenguas", 7.3396068837211335], ["wagashi", 7.3396068837211335], ["banh", 7.3377439115410015], ["mundanity", 7.3368132375573785], ["joads", 7.335653267085039], ["disconnected", 7.332589683438753], ["uplifted", 7.332162458241866], ["southport's", 7.330749755542871], ["thrasymachus", 7.330749755542869], ["unoptimized", 7.330749755542869], ["amsterdam's", 7.330143407341418], ["congruences", 7.329124672466934], ["beekeepers", 7.328720281614106], ["chugged", 7.327651382865127], ["solidarity", 7.327625030726371], ["arjun", 7.3275876780063305], ["frothy", 7.3273161490999925], ["engage", 7.326135101514692], ["erasmus's", 7.325379242901447], ["beachcombing", 7.325082380393629], ["ambition", 7.324507277749333], ["chromatica", 7.323955733526611], ["spectators", 7.322186314488591], ["partygoers", 7.319471678995882], ["braga's", 7.31947167899588], ["spiciness", 7.31947167899588], ["energized", 7.317314133917326], ["uplift", 7.316803746909496], ["ramanathan", 7.316730047128104], ["resisted", 7.316118737319749], ["epochs", 7.315023353983323], ["marko's", 7.3129422662581485], ["harshly", 7.31280184877992], ["khao", 7.310989023131478], ["pania", 7.310610333137533], ["skewers", 7.310513381869935], ["braved", 7.309883993313077], ["melody", 7.309485518338175], ["kababs", 7.308893018048705], ["agritourism", 7.308193602448891], ["marshlands", 7.308193602448891], ["washingtonville", 7.308193602448891], ["qazvin", 7.308193602448891], ["delftware", 7.307575625651796], ["bluntness", 7.307202123192014], ["grin", 7.3070339030610265], ["pinpointing", 7.303964323743771], ["nist's", 7.30275453007162], ["revolutionibus", 7.3021976630188465], ["unassumingly", 7.3021976630188465], ["whitmore's", 7.301070606735004], ["sisig", 7.301070606735004], ["methodically", 7.300304693217382], ["muerto", 7.299548599103832], ["canyoneering", 7.297578942169372], ["watashitachi", 7.2975789421693715], ["dough's", 7.2975789421693715], ["cmbr", 7.2975789421693715], ["nerd's", 7.295909015637983], ["uninspired", 7.294957057519491], ["validations", 7.294109411768974], ["energetic", 7.293837688247754], ["lerato", 7.293372844796364], ["waiata", 7.292648145586825], ["sparkle's", 7.292648145586825], ["funghi", 7.291621734869643], ["seatmate", 7.291621734869643], ["rogaland", 7.291621734869643], ["adhered", 7.291276487628407], ["rollerblading", 7.290704411281823], ["specks", 7.289782383470474], ["john's", 7.286204503482974], ["experienced", 7.286118154256916], ["cryptozoologist", 7.285995483531008], ["wittgenstein's", 7.285027823595616], ["borsch", 7.285027823595616], ["shuffling", 7.283872185199733], ["minhee", 7.2834545958296895], ["philosophies", 7.282983784285033], ["heightened", 7.282043757167059], ["orhan", 7.27811873165692], ["genuinely", 7.275460327899416], ["habaneros", 7.275315142006822], ["scribbles", 7.275055550372553], ["knickknack", 7.274654610413866], ["rollerbladers", 7.274654610413866], ["jack", 7.27450269480943], ["reputational", 7.273586901811555], ["scrambling", 7.273496931660214], ["digitus", 7.272414876851546], ["wanderlust", 7.271457854233195], ["akershus", 7.271122462620723], ["punchline", 7.270712233626062], ["choreographies", 7.268093774726264], ["experience", 7.266966775535834], ["straightforwardness", 7.266064431218035], ["encontrar", 7.265597520510463], ["refillable", 7.263445105181806], ["bromeliad", 7.26339824454797], ["amassed", 7.263146112792814], ["tordesillas", 7.262741253751981], ["rimmed", 7.262296734414188], ["duckduckgo", 7.261617315170701], ["kinsolving", 7.2597396439507165], ["hitachi's", 7.255226921522854], ["greying", 7.255226921522853], ["adjusting", 7.253210231212986], ["neo4j", 7.252670764063715], ["conveniences", 7.252535976099767], ["discography", 7.2522248300521515], ["endless", 7.2514447113204], ["taverna", 7.250550100097614], ["semicolons", 7.2503687275215665], ["harboring", 7.249912675090462], ["enhance", 7.249885946700958], ["digitization", 7.24863445500437], ["avgeeks", 7.24642768790183], ["ria's", 7.24642768790183], ["betraying", 7.245955328171063], ["gingerly", 7.245801775381744], ["para\u00edba", 7.241712309119272], ["arborists", 7.240444872159506], ["brexit's", 7.236765784317961], ["postgresql", 7.234442585029255], ["oradea", 7.233261297255337], ["gaming", 7.230788567912199], ["liberec", 7.229919269857868], ["enjambment", 7.229919269857868], ["jocelin", 7.229919269857868], ["enthusiastic", 7.229654511581926], ["devdutt", 7.227776013157317], ["hayworth's", 7.227776013157317], ["flourishing", 7.226586597280678], ["conflicting", 7.225628573676321], ["frankl's", 7.223679408577784], ["smugly", 7.220730968002854], ["together", 7.220306449757318], ["fearmongers", 7.21796899007298], ["rustam", 7.217968990072979], ["pheromonal", 7.214769535733407], ["paella", 7.214007136485043], ["embody", 7.210501074521595], ["invention's", 7.2097023371066005], ["cosines", 7.207888027796342], ["mcallister's", 7.207227964790133], ["opechancanough", 7.207227964790132], ["estella's", 7.207227964790132], ["mandolins", 7.206886487107729], ["jainism", 7.205847692849579], ["ultrarunning", 7.205164187006528], ["another's", 7.203968619187061], ["blanche's", 7.203416633238155], ["herra", 7.203416633238154], ["everyone's", 7.202616253612239], ["halima", 7.201464487809092], ["peeking", 7.201398131988587], ["intimidated", 7.2008263137215565], ["opted", 7.200048487659347], ["reeta", 7.199843509908177], ["clunker", 7.199730135472936], ["zoli", 7.198907452247083], ["mangroves", 7.1965483870203455], ["trouts", 7.1958060504243795], ["uncertainly", 7.195412836979002], ["mandalas", 7.195412836979001], ["habits", 7.195259677914782], ["luminescent", 7.194490466406386], ["rawest", 7.194133879844808], ["m\u00fcssen", 7.193689797506545], ["comedogenic", 7.19358395970111], ["camelcase", 7.191816928514744], ["navaratri", 7.19034921077423], ["gr\u00e0cia", 7.1890508450807005], ["eightfold", 7.188840183097313], ["algorithmics", 7.1884562663986165], ["krishnadevaraya", 7.1884562663986165], ["darbuka", 7.187536085104914], ["r\u00f6sti", 7.187536085104914], ["settling", 7.187352516734967], ["aquatic", 7.18678524364929], ["mitzy", 7.1865801686153405], ["sensitivities", 7.185499792023481], ["offering", 7.185001868829847], ["figma", 7.18359770916787], ["tubbataha", 7.1823540115035405], ["azman", 7.1823540115035405], ["melvin's", 7.18235401150354], ["bolstered", 7.181038260825912], ["rodriguez", 7.180177540590964], ["sobering", 7.179475806896227], ["ambling", 7.1770337492727965], ["knotwork", 7.1759526264665485], ["canyoning", 7.1756399624159], ["unwind", 7.174232059073679], ["est\u00e1s", 7.174221291011705], ["signified", 7.17417931092857], ["turdus", 7.172723609235501], ["disposables", 7.170777038564301], ["spritzed", 7.169940679302532], ["irreplaceable", 7.169871310681409], ["joviality", 7.169827560208134], ["activism", 7.16943470339427], ["refocused", 7.167242270232809], ["jimbaran", 7.16396129956627], ["tainting", 7.1615786073380345], ["profounder", 7.161120148941816], ["traditionalist", 7.160976574782146], ["esperanto", 7.160902463660157], ["lotuses", 7.15912685156695], ["language's", 7.157240885589192], ["samothrace", 7.157240885589192], ["humbling", 7.152261935407912], ["richard's", 7.152090701671519], ["kana's", 7.151627363325985], ["julianne's", 7.151627363325985], ["galbi", 7.1491536077523685], ["channeled", 7.1486949263228174], ["chatterbox", 7.14739005942408], ["anonymization", 7.145161154136721], ["misal", 7.145161154136721], ["lait", 7.1440781437306375], ["goldmine", 7.143052270564203], ["regexes", 7.142781813093052], ["invaluable", 7.140712540643029], ["directness", 7.139545278984645], ["gda\u0144sk", 7.137907441813964], ["broadened", 7.137894646589357], ["montague's", 7.137632006450592], ["toolshed", 7.136718331078542], ["abilities", 7.136293438190711], ["kintsugi", 7.1361280806169525], ["thaung", 7.133383415970562], ["ruleset", 7.1304848448954], ["sinulog", 7.126784541395195], ["valenciana", 7.126784541395195], ["explorations", 7.125506664096474], ["escargot", 7.1254309260976845], ["blooming", 7.125267416960112], ["eavesdropping", 7.1248158163837605], ["wurden", 7.122995713887808], ["amusing", 7.12162479529244], ["misinterpretation", 7.117196197544559], ["tricorn", 7.117059884126235], ["pressings", 7.116618707589903], ["persevere", 7.115699589805875], ["microcosmic", 7.114394817702672], ["matchsticks", 7.114067556314584], ["growled", 7.111382133919513], ["smetana", 7.1111076570448795], ["finist\u00e8re", 7.108989823439152], ["ceramicists", 7.108989823439152], ["wordsmith", 7.108429051197051], ["upcoming", 7.108401703947984], ["d\u00e9cor", 7.1082363533995725], ["iterate", 7.106580579732347], ["vlookup", 7.106283183491147], ["briny", 7.105902026916189], ["bardzo", 7.10518822460309], ["orfeu", 7.10518822460309], ["donato's", 7.10422428643668], ["chromatophore", 7.10422428643668], ["perkin's", 7.10422428643668], ["opole", 7.10422428643668], ["evolved", 7.102519996737094], ["acroyoga", 7.10191921690831], ["arara", 7.10191921690831], ["featherstone's", 7.10191921690831], ["kibali", 7.101299232690334], ["adebola", 7.101299232690334], ["respective", 7.099927547607299], ["eigenen", 7.099753006990082], ["hurrying", 7.09930766327505], ["multiplying", 7.09926723441592], ["nuttiness", 7.09821209478021], ["conservationists", 7.095997262391621], ["famished", 7.09414031288359], ["woodcarving", 7.093146191091378], ["afternoons", 7.092697451653198], ["acupuncture", 7.091654532746702], ["orecchiette", 7.08907668667882], ["medinas", 7.089076686678819], ["gizmo's", 7.089076686678818], ["bharta", 7.089076686678818], ["siguiente", 7.089076686678818], ["desalinating", 7.089076686678818], ["elise's", 7.085923182814637], ["counterpart's", 7.0837223991511165], ["pli\u00e9", 7.08263207150911], ["afloat", 7.082368565047734], ["hypervigilant", 7.081692231796862], ["alfredo's", 7.080576594728364], ["worsened", 7.079624584429913], ["mahotsav", 7.078651573904292], ["silhouetted", 7.077715986860423], ["fallouts", 7.077715986860422], ["grudgingly", 7.077455249487541], ["eyeing", 7.077343042507764], ["kalaripayattu", 7.075948766888673], ["caecilians", 7.075948766888673], ["quacky", 7.075948766888673], ["gongfu", 7.075717873827037], ["jade's", 7.075443846896745], ["unquenchable", 7.07257025811915], ["refocus", 7.072535507743235], ["frisbees", 7.072513423392186], ["spicewood", 7.072513423392185], ["location's", 7.071353994962122], ["kealakekua", 7.071353994962121], ["haka", 7.0697108314917], ["jeopardize", 7.069384806993599], ["sturdy", 7.0672938874052065], ["anthropomorphizing", 7.066642899695658], ["myco", 7.065602922815644], ["beneatha", 7.065602922815644], ["nimbostratus", 7.065602922815644], ["dockworker", 7.065045918249399], ["roboticist", 7.065045918249399], ["defying", 7.0623165958748], ["viniculture", 7.06181100711467], ["shenzhen's", 7.06181100711467], ["ragtag", 7.061811007114669], ["gioachino", 7.0602593017736215], ["climatechange", 7.0602593017736215], ["umno's", 7.0602593017736215], ["bol\u00e9ro", 7.0602593017736215], ["regardless", 7.059728613408885], ["caelan", 7.058520321650031], ["footprints", 7.057993811975688], ["hartley", 7.057020503254993], ["noblewoman", 7.055172406872964], ["oaxaca's", 7.0543263107637255], ["xmlhttprequest", 7.0543263107637255], ["stigmas", 7.051539041028871], ["supermarine", 7.051207687284166], ["concatenating", 7.050826272901775], ["hanbok", 7.050416394759503], ["amplify", 7.0491089612211635], ["bankaccount", 7.048797841868144], ["lemongrass", 7.046911174787793], ["forming", 7.045699851705544], ["davonte", 7.0433407080550845], ["worktable", 7.043110051102371], ["town's", 7.042988782340172], ["scenarios", 7.04144257455489], ["taprooms", 7.041177519876935], ["kimonos", 7.040955470759311], ["chloro", 7.040339284457903], ["rabbitmq", 7.039293844777984], ["quantum's", 7.039293844777984], ["mimicking", 7.038483703487564], ["kulit", 7.038440424631112], ["vivacious", 7.0369075941515895], ["nefarious", 7.036617519197396], ["docent", 7.034136342357058], ["warblers", 7.034015896879372], ["menorahs", 7.033693275064141], ["texian", 7.033693275064141], ["commutes", 7.033352177877535], ["grew", 7.032591936364604], ["lumina", 7.030001047623162], ["eventually", 7.029004000276254], ["mbira", 7.027383772475127], ["banyan", 7.026666132515181], ["vorpommern", 7.026341494761307], ["cavalries", 7.025018764811239], ["kushite", 7.025018764811239], ["waddling", 7.0250187648112385], ["yore", 7.023377005715319], ["tessellations", 7.022992073497915], ["hubris", 7.021929863871586], ["beringer", 7.021929863871586], ["hometowns", 7.0217304581553694], ["pushkin's", 7.021674950843505], ["troubleshoot", 7.020682073574946], ["banquo's", 7.02038408312573], ["whodunit", 7.0202709423669365], ["acclaim", 7.019821860585879], ["readability", 7.019699601666838], ["scurrying", 7.019610859849554], ["tonkotsu", 7.019118693060277], ["buff", 7.018094800574413], ["confusion", 7.017968673913567], ["rashid's", 7.017266752656544], ["wiping", 7.014512489165297], ["configuring", 7.012150854527209], ["forth", 7.012141846461635], ["pijp", 7.012021505301876], ["venice's", 7.012021505301875], ["bioluminescence", 7.011621959916957], ["grandiose", 7.010055896173503], ["microservice", 7.008762373763274], ["lner", 7.008762373763274], ["aiyana", 7.007027187990407], ["ostracized", 7.00620079016189], ["newfangled", 7.004553080029956], ["weaving", 7.003425522388038], ["fa\u00e7ade", 7.0014348668955915], ["arrogantly", 7.00142992037079], ["airbnb's", 6.999793604982612], ["diving", 6.999518017870444], ["marco", 6.999483367735997], ["coiffed", 6.998191088131656], ["hemingway's", 6.998191088131655], ["importantes", 6.998191088131654], ["ecosystem's", 6.998191088131654], ["aviva's", 6.998191088131654], ["loman's", 6.998191088131654], ["headwrap", 6.998191088131654], ["listeners", 6.998079474238702], ["\u00e9lys\u00e9es", 6.994099775013776], ["pierre", 6.993939049775383], ["ualberta", 6.9924074591332], ["bereits", 6.9924074591332], ["lleg\u00f3", 6.9924074591332], ["sophia", 6.9908767022366005], ["mihailo", 6.990617288252724], ["\u00e9lisabeth", 6.990617288252724], ["trapezoids", 6.989745966142759], ["helpfully", 6.989482309596514], ["culture's", 6.989230536162216], ["marrakech", 6.988288509437776], ["floatie", 6.987804162583407], ["pheidippides", 6.987804162583407], ["amit's", 6.987222136582858], ["cotto", 6.98695708756774], ["speakeasy", 6.986471629371625], ["cottbus", 6.986471629371625], ["unboxing", 6.985143613221578], ["lukewarm", 6.984613666397474], ["dampening", 6.9844837519573275], ["koska", 6.984053328357652], ["underwater", 6.983081357373188], ["athletica", 6.981828930450697], ["twangy", 6.9814490041887565], ["reputations", 6.980001574931511], ["c\u00f3mo", 6.979570624039065], ["galileo's", 6.979570624039065], ["wayang", 6.97933142835408], ["assortment", 6.978650684444014], ["duckpin", 6.978309863449463], ["index1", 6.978309863449463], ["treasuring", 6.9744523123917235], ["triumph", 6.973276774464128], ["frantic", 6.972802578406657], ["discussion", 6.972082738338479], ["burnaby's", 6.9709254085675045], ["mezi", 6.9709254085675045], ["fortress's", 6.9709254085675045], ["bumbly", 6.9709254085675045], ["crear", 6.969597866116816], ["taiyaki", 6.969597866116816], ["cabernets", 6.967688387249387], ["metaphorically", 6.967427508886578], ["functionality", 6.965866835881901], ["pentameter", 6.964406717361365], ["abaci", 6.963302745463549], ["matryoshka", 6.962986389880185], ["constellations", 6.961584653072841], ["idly", 6.960998479795521], ["deterring", 6.960353978420703], ["skeptic", 6.959125506436544], ["typewriting", 6.958190540268083], ["aquellos", 6.957353977973638], ["tewodros", 6.957353977973638], ["slathering", 6.956890864482459], ["discernment", 6.954935729925213], ["paved", 6.951162694782699], ["asheville's", 6.950284522632396], ["undercurrents", 6.9499630850316265], ["neuralink", 6.949893243877534], ["experimentation", 6.9493978451828164], ["maintainability", 6.947295152945243], ["exhaling", 6.947295152945243], ["industrialization", 6.947295152945242], ["bumblebee's", 6.947295152945242], ["globalization", 6.946433633209014], ["reaffirmed", 6.946355313232993], ["mandela's", 6.944920821040613], ["lamented", 6.944475633808495], ["marley's", 6.9420160532849495], ["broaden", 6.9417125050544835], ["puzzlement", 6.9392913566860885], ["jostled", 6.939179153934792], ["singlehandedly", 6.937998003192221], ["powwows", 6.93772587587507], ["successes", 6.93714087050258], ["bydgoszcz", 6.936623424599704], ["pangs", 6.93545033888333], ["hiccup", 6.933866889602703], ["biryani", 6.93364466547413], ["unity's", 6.933428296352538], ["isabelle", 6.9330283861133015], ["guidebook", 6.932536188575109], ["investing", 6.9324850276191965], ["bruschetta", 6.931113564856084], ["spirited", 6.930870461185261], ["activewear", 6.929140200454967], ["indeed", 6.928857234285685], ["fears", 6.927009855804644], ["considerate", 6.9243442671721205], ["oases", 6.923458519544619], ["h\u00e9rault", 6.922926451834784], ["breath", 6.922745744630724], ["ravana's", 6.922123793695441], ["clattering", 6.921826119153698], ["subtopics", 6.919865057222918], ["mudras", 6.9192523680175935], ["meditator", 6.918666189402886], ["bore", 6.917847578224432], ["bistrot", 6.916554908905662], ["tasted", 6.914361368537567], ["interrupting", 6.9128704480923915], ["pirouettes", 6.912503743375933], ["relationships", 6.911631473309075], ["complacency", 6.9107421012795545], ["knew", 6.908200914059037], ["barked", 6.906679454262244], ["polly's", 6.906081441119209], ["fourthly", 6.905942205606284], ["xiaolong", 6.904144251374154], ["assefa", 6.903274273519898], ["mumbled", 6.902938918928034], ["grumman's", 6.902884983818578], ["unsustainably", 6.902074663768825], ["banchan", 6.899901887455648], ["bront\u00eb's", 6.898599158882543], ["mathematician's", 6.896830539243461], ["intentionality", 6.8966044433039375], ["harmful", 6.893842147060155], ["discarded", 6.893398123894208], ["intricacy", 6.89280201673222], ["reclusiveness", 6.892157889826629], ["understaffing", 6.892157889826629], ["3arena", 6.892157889826629], ["songket", 6.892157889826629], ["alg\u00fan", 6.892157889826629], ["amitava", 6.892157889826629], ["tinkered", 6.891498353186455], ["favoritism", 6.890100529262502], ["skylar", 6.889303149872264], ["typography", 6.888914521407888], ["gertrude's", 6.888694493902094], ["intern", 6.8879165618944285], ["festivities", 6.887465356795258], ["inna's", 6.885265731936803], ["workouts", 6.883351243634073], ["caprese", 6.883089261024226], ["replicating", 6.882342879637975], ["composted", 6.881554569996127], ["meursault", 6.881261197510698], ["specters", 6.879626693663308], ["traducci\u00f3n", 6.879626693663308], ["respected", 6.879110874658447], ["cramped", 6.877796220506467], ["heartening", 6.876209094709675], ["redefine", 6.875683951455824], ["anansi", 6.874797290607923], ["understandings", 6.873567200781702], ["redstarts", 6.873530436070341], ["yesteryear", 6.872324341942237], ["scorm", 6.871886837209492], ["cuber", 6.871660765990714], ["hypotheticals", 6.871628057814379], ["belongings", 6.870399176488498], ["nabu", 6.868794642742471], ["romanticized", 6.868423306364976], ["monnieri", 6.867543040220107], ["complex", 6.867445124796258], ["spearfishing", 6.866725572890001], ["technology's", 6.8659562978487765], ["irina's", 6.865392228118566], ["proven\u00e7al", 6.864589258267323], ["napol\u00e9on", 6.863816305980613], ["instill", 6.862935140373764], ["botanicals", 6.862829558380558], ["frackville", 6.862829558380558], ["tiramisu", 6.862004699058639], ["sensationalize", 6.861354949536902], ["abenaki", 6.861354949536902], ["marcia's", 6.860207489012864], ["signaled", 6.858681694361757], ["nutritionist", 6.857610297381402], ["overachiever", 6.856895686669377], ["routine", 6.8556824696097], ["symbolism", 6.855543301036926], ["biko's", 6.8543926411152505], ["meditech", 6.8543926411152505], ["nitty", 6.854189115822794], ["flipping", 6.853917529921785], ["mediate", 6.853561805643601], ["relieved", 6.852893475181557], ["chord's", 6.852774130456191], ["luminis", 6.852774130456191], ["clipboards", 6.851010678544082], ["cove's", 6.849081903015213], ["natto", 6.848334920584958], ["francophile", 6.847512332645567], ["math", 6.841273234767696], ["redwood's", 6.840627737379327], ["disappointingly", 6.839667945632586], ["undisturbed", 6.839108731861946], ["progress", 6.837967012567515], ["uppercase", 6.8379551565913825], ["moderation", 6.837114079696352], ["disrupted", 6.836318879722871], ["entailed", 6.835627617122763], ["conveying", 6.83556003695653], ["teasers", 6.832258772165954], ["quizzes", 6.832097656786711], ["fluctuating", 6.831920429869545], ["peque\u00f1a", 6.83108813637247], ["wonderlands", 6.83108813637247], ["embraer's", 6.83108813637247], ["taglines", 6.828677488209805], ["eavesdrop", 6.827344219604419], ["revamped", 6.826522637869208], ["kaita", 6.826204225809151], ["sincerely", 6.825954094838527], ["rippling", 6.82447442312414], ["fruitiness", 6.824289442015094], ["\u0431\u0443\u0434\u0435\u0442", 6.823236310928364], ["stepwell", 6.823236310928364], ["thumbed", 6.823236310928363], ["coursera", 6.823236310928363], ["alternatives", 6.823123530162893], ["colmore", 6.819988697263001], ["abcdefghijklmnopqrstuvwxyz", 6.819128402252308], ["riads", 6.819128402252308], ["kaibab", 6.816227336803116], ["aristotle's", 6.816106492421646], ["prudence", 6.815999545144045], ["homogenization", 6.815762886710478], ["reassurance", 6.815232514669209], ["reluctance", 6.8152172983265125], ["carla", 6.813762726628892], ["companionable", 6.8124797639326795], ["tshepo", 6.812159628605428], ["coltrane's", 6.811958234381374], ["astronomica", 6.81107367935808], ["worldviews", 6.810723171243663], ["elmwood", 6.810038561777631], ["dividing", 6.809581122228614], ["dusk", 6.809251496010098], ["remedios", 6.808554791163052], ["giulini", 6.80733133118261], ["yodeler", 6.80733133118261], ["inseparable", 6.80669513199278], ["whims", 6.805432229123415], ["goya's", 6.8032268202804795], ["coursera's", 6.800262451295608], ["soldiered", 6.798424542524987], ["starlights", 6.797744768048181], ["adriaan", 6.794459775408985], ["repotting", 6.794145357051486], ["diplomacy", 6.793007548253685], ["equation", 6.791453617116421], ["fiery", 6.790554142870153], ["personalized", 6.790300335171668], ["aaro", 6.789706894167044], ["andronikos", 6.789706894167044], ["empowerment", 6.787126964981088], ["unseren", 6.7861246915216045], ["channelizing", 6.7861246915216045], ["laza", 6.7861246915216045], ["bigquery", 6.785259114392584], ["mending", 6.784467922798088], ["foulbrood", 6.782656315876113], ["restlessly", 6.780457399888061], ["josiah's", 6.779553620077349], ["heartwood", 6.77892958163662], ["mentor", 6.778773388701397], ["audience's", 6.778124004740406], ["mihrab", 6.778049315491752], ["embarrassed", 6.777600379757869], ["moksha", 6.776179508784031], ["antonio's", 6.7749713435730925], ["bella", 6.772484966466912], ["woodcarver", 6.7718920474538145], ["suiza", 6.7718920474538145], ["picnicked", 6.7713391857818594], ["wizened", 6.770343006192484], ["factcheck", 6.770172794431468], ["petunia", 6.769833498139641], ["descended", 6.769101543502816], ["mythologies", 6.769035845969533], ["selim", 6.768273532819619], ["gossiped", 6.768000286079498], ["kendal's", 6.766845928193418], ["kioko", 6.766845928193418], ["ameya", 6.766845928193418], ["easels", 6.763801912931074], ["unrewarded", 6.762423153076951], ["hushpuppies", 6.761206889919923], ["ryukyuan", 6.761206889919923], ["arcsin", 6.759067944367908], ["subdirectories", 6.75825310796714], ["polyculture", 6.758069214927279], ["galore", 6.7578409523458225], ["mnist", 6.757676760269037], ["antioch's", 6.756776216990749], ["metropolis's", 6.756776216990749], ["laoco\u00f6n", 6.756776216990749], ["create", 6.755866452149292], ["disagreement", 6.755370423610595], ["bounty's", 6.755036423955733], ["drastic", 6.751764472202011], ["min\u00e4", 6.75026052150667], ["mbct", 6.75026052150667], ["procrastination", 6.749780417628895], ["neighbor", 6.7497627021679785], ["sometimes", 6.748157978212493], ["stunning", 6.747933927790161], ["bosniak", 6.747059828988175], ["livelihood", 6.746348131275317], ["broadening", 6.745564688100404], ["overextending", 6.744607467396538], ["misidentifying", 6.743711412199594], ["whimsies", 6.743711412199594], ["influxes", 6.743610588239241], ["globalized", 6.743198348336827], ["nandini's", 6.742328370482572], ["confucianism", 6.739412769025067], ["quizzers", 6.73899882560826], ["microtransactions", 6.7380978899363875], ["sacrilegious", 6.737917032961365], ["newly", 6.737385705426188], ["embouchure", 6.737064554992413], ["arduinos", 6.736970228731195], ["bandhavgarh", 6.736528518120867], ["countermoves", 6.736528518120867], ["stringent", 6.736072291816132], ["doorjamb", 6.734622852344878], ["bylo", 6.734622852344877], ["irena's", 6.734622852344877], ["uoft", 6.733882865216833], ["cheesemaker", 6.733108092369092], ["drastically", 6.732204319470787], ["gastronomique", 6.731875148202755], ["teary", 6.7318056614650565], ["bourguignon", 6.7315406450898], ["curmudgeonly", 6.730317745045275], ["reaffirming", 6.730261563555533], ["guppies", 6.728367784680162], ["brushwork", 6.72674610047079], ["trickier", 6.726108391911589], ["unscrupulous", 6.7256857684877405], ["astana's", 6.724684894372897], ["antipasti", 6.722876818848431], ["kuchen", 6.721913352309], ["lindas", 6.719853942580964], ["cinephilia", 6.719853942580964], ["septa's", 6.719853942580964], ["franz's", 6.719853942580964], ["mehmet", 6.718976333342021], ["cafes", 6.717148609577481], ["wildfire", 6.715429718957362], ["heliocentrism", 6.713310754710876], ["scarpia", 6.713310754710876], ["marge's", 6.710656222977103], ["preachy", 6.709515705746225], ["hearty", 6.709515705746224], ["lovingly", 6.709066217188192], ["ecological", 6.7088470635628825], ["cyra", 6.708367012764586], ["heraus", 6.708367012764586], ["wordplays", 6.708367012764586], ["sightings", 6.708270787174501], ["triumphed", 6.708027915205431], ["cop\u00e1n", 6.7074876305776865], ["neeps", 6.7074876305776865], ["laserdiscs", 6.7074876305776865], ["macaws", 6.706369285839999], ["vermicompost", 6.705883352263747], ["parting", 6.705539460809786], ["improvisational", 6.705174270942511], ["reclusive", 6.703492559068593], ["overindulged", 6.703179367041059], ["prickles", 6.701570887085444], ["across", 6.700703810844164], ["customizing", 6.699177468911484], ["overdevelopment", 6.699177468911484], ["circ", 6.699177468911484], ["ristorante", 6.699177468911484], ["moussaka", 6.696833577159478], ["disrupt", 6.6966261302581795], ["yanna", 6.695891141970639], ["ayudar", 6.6952390929744405], ["constructing", 6.692480204946563], ["encantado", 6.690813951472143], ["among", 6.690754846549063], ["undercurrent", 6.690694813047082], ["superfans", 6.690084150648466], ["councilmember", 6.689839706609138], ["line2", 6.689447363655257], ["eryx", 6.689447363655257], ["saag", 6.688581695927836], ["wrigglers", 6.688389743518712], ["arrernte", 6.688389743518712], ["semanas", 6.686859569704134], ["tastebud", 6.686859569704134], ["lisa's", 6.686561315486038], ["connoisseur", 6.685748419002441], ["pollination", 6.68358892331774], ["cuppa", 6.683329632805534], ["cosplayers", 6.68235593101089], ["balance", 6.681032514800841], ["victories", 6.68009149321658], ["gridlines", 6.68009149321658], ["miyuki's", 6.68009149321658], ["splittingly", 6.68009149321658], ["colcannon", 6.680091493216579], ["mcallister", 6.678500995242004], ["eliza's", 6.678307757170327], ["hargraves", 6.676851089872793], ["algorithmic", 6.676851089872792], ["culminating", 6.675794650520868], ["texians", 6.674051627490344], ["bioinformatician", 6.672861956968509], ["daragh", 6.672861956968509], ["enoteca", 6.672487518836139], ["beauty", 6.6684718017859534], ["quixote's", 6.668162758407264], ["arctan", 6.668162758407264], ["outmatched", 6.666294402352793], ["quilters", 6.665848227772621], ["tatin", 6.662766855672037], ["spiaggia", 6.662419293499076], ["hatha", 6.65834797660213], ["lucija", 6.656815913100843], ["compositions", 6.65574267983006], ["distant", 6.6553166654113225], ["trumped", 6.654239745951808], ["mouthfuls", 6.654065162723528], ["disappointments", 6.654065162723527], ["zainab", 6.654065162723527], ["protocols", 6.65343751324613], ["knaw", 6.651063393300374], ["sally's", 6.650367432708122], ["studying", 6.6488935428316385], ["sacr\u00e9", 6.648457279357622], ["adelard", 6.648129332961955], ["xerox's", 6.648129332961955], ["markdown", 6.647774602099709], ["desserts", 6.64687815315927], ["honeybee", 6.646264763959039], ["sparky", 6.646259714265866], ["moshavim", 6.646009393761393], ["aceh's", 6.646009393761393], ["jaanu", 6.646009393761393], ["identities", 6.643628733680045], ["po'boy", 6.643150895097409], ["losar", 6.640796837374129], ["accountability", 6.640734004591109], ["renditions", 6.640644487083061], ["outpouring", 6.640244826864131], ["steadfastness", 6.639192973870356], ["bellies", 6.637834720460333], ["aristides", 6.637678213894569], ["tracklists", 6.635705503228429], ["ardashir", 6.635705503228429], ["harmless", 6.6352379036580516], ["nasser's", 6.6352379036580516], ["online", 6.63453881032256], ["congregated", 6.634068573101207], ["tradition's", 6.633962246824061], ["ibises", 6.633350328249466], ["bhajji", 6.6307312112469985], ["gridlocked", 6.628286702044695], ["tiko", 6.627074894064067], ["rejections", 6.622088273362795], ["netlink", 6.622059810360451], ["hitomi's", 6.622059810360451], ["traction", 6.621409239587471], ["floodwaters", 6.619806489341545], ["greediness", 6.619740581928343], ["petra's", 6.619740581928343], ["distraught", 6.618231274120329], ["callouses", 6.6164715742335645], ["aretha's", 6.6164715742335645], ["kood", 6.616471574233564], ["preservation", 6.615620190023644], ["rang", 6.615526364008675], ["harnessed", 6.614945635217366], ["workarounds", 6.61453919351679], ["henrietta's", 6.613732968449527], ["pyramid's", 6.61325344603598], ["tokenistic", 6.611927294306206], ["funkos", 6.611927294306206], ["bppv", 6.611927294306206], ["landscape", 6.610981925329178], ["derisively", 6.610109582335262], ["magda's", 6.609248351990951], ["pseudocode", 6.607315940136378], ["vespas", 6.6038927309365425], ["akhenaten's", 6.6038927309365425], ["nyota", 6.6038927309365425], ["doorsteps", 6.602968571020843], ["argued", 6.602909974473058], ["divisive", 6.602876084697468], ["frowning", 6.601961768149719], ["feared", 6.599759476112653], ["aelia", 6.5988745753659215], ["principles", 6.597902056816336], ["kettlebells", 6.597427453748518], ["chernivtsi", 6.595965980511766], ["rearranging", 6.595615305140871], ["expressvpn", 6.595533373049281], ["metalheads", 6.593555583264871], ["remembered", 6.593495567511123], ["intonations", 6.5920565499744], ["boastful", 6.591607460327227], ["rithy", 6.590625982146715], ["saoirse", 6.589502260751634], ["seafood", 6.58927299142002], ["feedback", 6.589008468536686], ["ziplining", 6.588331994683385], ["nambia", 6.5879523002066955], ["lifeblood", 6.587777692406532], ["earnestness", 6.587777692406532], ["whisper", 6.586622264972533], ["async", 6.586269983480391], ["campouts", 6.585839761389891], ["ambrose's", 6.584661614742057], ["chose", 6.5846589265764655], ["jamilah", 6.584352780183887], ["sizzling", 6.584308170747707], ["petroglyphs", 6.584086735956066], ["acceptance", 6.583417735636424], ["extraterrestrial", 6.582010784784431], ["similes", 6.580512489590991], ["haji's", 6.58051248959099], ["seismograph", 6.580388635108869], ["tangents", 6.580337930927678], ["partnerships", 6.580186083376383], ["mateusz", 6.579916482663213], ["remainders", 6.579300385614274], ["ignoring", 6.579065953686051], ["ajumma", 6.578877985743601], ["armyworms", 6.578877985743601], ["clouded", 6.578354944512436], ["helsinki's", 6.5783362913849555], ["kukulkan", 6.5783362913849555], ["buco", 6.576988860693854], ["doubters", 6.576851983968692], ["eurotech", 6.576613311738181], ["droplets", 6.5751186268946045], ["trogon", 6.575118626894604], ["estheticians", 6.575118626894604], ["tradition", 6.5749223312585015], ["infernos", 6.5748211236763385], ["clearcutting", 6.5748211236763385], ["unpacked", 6.574528806726538], ["tvnz's", 6.574058294911555], ["limoncello", 6.570913242419456], ["traumas", 6.570896202774096], ["backpropagation", 6.569876703992483], ["newcomer's", 6.569876703992483], ["levinsky", 6.569876703992483], ["shortchanged", 6.569211062989038], ["flicker", 6.566257281036255], ["pizzerias", 6.566145369664911], ["heartstone", 6.564780390059864], ["historical", 6.564434375602539], ["marmottan", 6.563112932505875], ["touched", 6.56296086037695], ["ukuleles", 6.562896905616289], ["maudie", 6.561537769650445], ["weatherwise", 6.559433026179827], ["ingredient", 6.559103758197881], ["bongo's", 6.55672206857271], ["simplicities", 6.55672206857271], ["avgeek", 6.555074185171353], ["therapist's", 6.553225890068097], ["partnering", 6.549627084014423], ["finding", 6.547801150241453], ["soundproofing", 6.547549995335299], ["structure's", 6.545868981215828], ["unfulfilling", 6.545580807366776], ["cups", 6.5445366797927695], ["community", 6.5445117810764675], ["winced", 6.544103916390386], ["claypot", 6.541284397253637], ["caj\u00f3n", 6.541284397253637], ["marguerite's", 6.541284397253637], ["peshwas", 6.541284397253637], ["gamelan", 6.539176345562611], ["offerings", 6.539083380801956], ["adaptations", 6.538934797973015], ["misappropriations", 6.538236268457154], ["interactions", 6.537216511712896], ["milestones", 6.537164551939668], ["calculate", 6.534267942073932], ["desensitize", 6.533276295563468], ["sammy", 6.531479838245451], ["betrayed", 6.5298101802449615], ["charm", 6.529630384821749], ["virtuously", 6.529412737730491], ["sprouting", 6.529224026379689], ["ingenious", 6.529150395254421], ["pride", 6.528385781552324], ["akeem", 6.528061824750272], ["puns", 6.527472839743195], ["bravery", 6.527281027145165], ["tense", 6.526386197370578], ["limitations", 6.524608955502018], ["awkwardly", 6.524240562471952], ["sydneysiders", 6.524210018417559], ["knockoffs", 6.522487603008656], ["revealed", 6.5221091440641255], ["equilibrium", 6.52119355234958], ["reshaped", 6.5196262643062575], ["quieten", 6.519170521671306], ["khachapuri", 6.517015118608218], ["petm", 6.517015118608218], ["townhall", 6.516825917995107], ["ebbed", 6.516737995357249], ["endeavored", 6.5165480955334765], ["susie's", 6.516071389588493], ["diese", 6.5159877769613255], ["hija", 6.51543288678069], ["harmonium", 6.515090154950954], ["experimentations", 6.514821871277462], ["blinis", 6.5145054027128415], ["si\u00e2n", 6.513089205886166], ["celluloid", 6.51100768169125], ["boardrooms", 6.510889581027709], ["\u00e9taient", 6.510889581027709], ["thenceforth", 6.51043837592854], ["connexus", 6.509260229280706], ["prediabetes", 6.5091723078890755], ["accommodations", 6.508570506342504], ["bursting", 6.508070644315902], ["qualities", 6.507803220443706], ["vodou", 6.507710932388323], ["mariscos", 6.507007890101024], ["perspective", 6.506561591884725], ["satisfaction", 6.506330137941797], ["proffered", 6.5048343399724935], ["grandpa", 6.5043402067264955], ["rhythm", 6.503563259088441], ["unicyclist", 6.502394478126089], ["biosignatures", 6.5018790695593385], ["raito", 6.5018790695593385], ["morels", 6.5013436143953145], ["delectably", 6.5009000183517545], ["frieda's", 6.5009000183517545], ["atsa", 6.498320296122251], ["hossein's", 6.49832029612225], ["graphene's", 6.49832029612225], ["courageousness", 6.49832029612225], ["streamlining", 6.4976675266299235], ["rows", 6.496125358704298], ["rayden", 6.495227330726666], ["logbooks", 6.494240486795203], ["arctis", 6.492799208360039], ["mofongo", 6.492799208360039], ["aethelwulf", 6.492799208360039], ["afib", 6.492514336509902], ["togetherness", 6.491953983197077], ["frustratedly", 6.49049570816786], ["escalus", 6.49049570816786], ["grandmother", 6.4891764145988535], ["mournfully", 6.488603908119676], ["earshot", 6.487667312030247], ["fermat's", 6.487287494261262], ["aligned", 6.48133722017186], ["tabbouleh", 6.478951234136789], ["bond", 6.47657317141076], ["grimly", 6.476405139698378], ["valued", 6.476189652876016], ["pyrography", 6.474784184944698], ["sabermetrics", 6.473615937971703], ["defied", 6.4731788032218205], ["artform", 6.47263523566327], ["intercropped", 6.472635235663269], ["kibou", 6.472635235663269], ["imposter", 6.471000731815881], ["johan's", 6.470970463226114], ["scala", 6.470519995390177], ["jean's", 6.470089459759371], ["flugelhorn", 6.468025096606528], ["fastpasses", 6.468025096606528], ["smarthome", 6.466897083858603], ["pairings", 6.464050239539179], ["emmanuel's", 6.463907302497596], ["challenged", 6.4634656690794134], ["portioned", 6.463249804154717], ["forebear", 6.462444401958896], ["cultivated", 6.462281187673222], ["birthdates", 6.462033048545066], ["kitchenette", 6.461966056703385], ["mycological", 6.461398021712465], ["pouted", 6.458936536751813], ["techsupport", 6.457857529645772], ["flustered", 6.455141626918234], ["enjoyability", 6.4550942187644536], ["malevich's", 6.4550942187644536], ["rulebook", 6.454770946818401], ["overtraining", 6.454722038516601], ["plagued", 6.45325940973618], ["inspiration", 6.452749302181457], ["values", 6.451908050464165], ["spirulina", 6.451059784877726], ["realities", 6.450264535890438], ["chai", 6.449151187308234], ["kayaking", 6.448696759315499], ["environments", 6.4473970952562745], ["pollinator", 6.4465279550323595], ["rikyu", 6.446194732249611], ["methodical", 6.44515222097216], ["roadblocks", 6.444937021816963], ["figurine", 6.444751708165001], ["kapampangan", 6.44336621327978], ["stunningly", 6.44299946463369], ["bhajans", 6.441516797030272], ["underdressed", 6.439945158715474], ["kiko's", 6.438496864167155], ["bhanwar", 6.438496864167155], ["caoimhe", 6.434827786856825], ["crms", 6.434827786856825], ["chainmail", 6.432680697171521], ["bistro", 6.432229424928734], ["layla's", 6.431551760580231], ["arrived", 6.430490547376385], ["tina's", 6.430473601486278], ["sichuanese", 6.430275057419401], ["chastanet", 6.428503631783747], ["yanni's", 6.428503631783747], ["sogginess", 6.428503631783747], ["sanitaria", 6.428503631783747], ["seribu", 6.428503631783747], ["digestible", 6.428268181333915], ["fearful", 6.428206840295668], ["techfest", 6.42561181728452], ["calculated", 6.425558800685367], ["intimidating", 6.424744272934751], ["biohacker", 6.4244757473026795], ["capulet", 6.4244757473026795], ["ecotourists", 6.4244757473026795], ["index2", 6.4244757473026795], ["sch\u00f6nbrunn", 6.423758809935499], ["heraclius", 6.423600120241578], ["emerge", 6.422922727925443], ["tatoosh", 6.422904586689497], ["voil\u00e0", 6.4227932132789425], ["kgalagadi", 6.421869469109048], ["minimize", 6.419799315970144], ["isolating", 6.4196801954263965], ["lifelines", 6.419671403162611], ["mija", 6.419552777381375], ["marshland", 6.418204018498826], ["yoshida's", 6.415857733596048], ["firebreaks", 6.415210081481213], ["drysuit", 6.415008497454016], ["knowledgeably", 6.415008497454016], ["sauerbraten", 6.415008497454016], ["expressed", 6.41391709229805], ["ribcage", 6.413399064979744], ["tony's", 6.412798646685604], ["teacup", 6.410915988045753], ["onboarding", 6.410789375427219], ["misused", 6.410076060282862], ["piata", 6.409706837538766], ["keats's", 6.409706837538766], ["furoshiki", 6.409706837538766], ["hauwa", 6.409706837538766], ["sunscreens", 6.409706837538765], ["backdrop", 6.409598013993136], ["hesitance", 6.408943868030668], ["muffin's", 6.408337534646747], ["showstopping", 6.407908883306637], ["gasthaus", 6.407434697575086], ["obsessively", 6.407413092440725], ["mistranslated", 6.406986248898045], ["openhearted", 6.405947478689769], ["pablo's", 6.405112065612214], ["cesena", 6.403037007322805], ["baden's", 6.403037007322803], ["mathy", 6.403037007322803], ["impetuousness", 6.403037007322803], ["elevate", 6.402470071754448], ["involving", 6.402093513459143], ["unrolled", 6.402048884327848], ["fruits", 6.401966414062499], ["harold", 6.401088419541955], ["kolas", 6.399860897696156], ["quiltmaking", 6.399860897696156], ["senhora", 6.39974519922562], ["accessibly", 6.399237736946623], ["depths", 6.396333993871938], ["drenched", 6.396327942811455], ["meinen", 6.395145940588428], ["punctuality", 6.392380183625793], ["easing", 6.389727323655897], ["scolding", 6.389388915435808], ["naivety", 6.389030363869286], ["didgeridoos", 6.389030363869286], ["alkan", 6.3886589302105525], ["morag", 6.38850019787776], ["enjoying", 6.387945438634918], ["worlds", 6.385454550187623], ["laboriously", 6.384808466104313], ["meowed", 6.384314676892036], ["pests", 6.38396181195809], ["discordant", 6.383731368104746], ["lived", 6.382950009556996], ["masterji", 6.382737524056836], ["bia\u0142ystok", 6.382737524056836], ["spiciest", 6.38197438558819], ["impressionism", 6.381791232355715], ["mateo", 6.381732784927116], ["traveler", 6.381026567610132], ["flora", 6.380735012157578], ["flexitarian", 6.378912089520391], ["flutists", 6.37685636535361], ["ousmane", 6.37685636535361], ["azure's", 6.37685636535361], ["sncf's", 6.37685636535361], ["capetonians", 6.37685636535361], ["macaron", 6.376047461797751], ["tainos", 6.375932517998978], ["takeshi", 6.375157459941947], ["raman's", 6.3705891846505605], ["amorphophallus", 6.3705891846505605], ["kerning", 6.368109198203718], ["niin", 6.3680394191173795], ["chefchaouen", 6.3680394191173795], ["evolve", 6.367698534457508], ["kitchenware", 6.365322784112133], ["selby's", 6.365322784112133], ["blisteringly", 6.364757981735549], ["cycloalkanes", 6.364757981735547], ["file2", 6.364757981735547], ["hyderabad's", 6.3643569510518025], ["python's", 6.363825325361533], ["librarian", 6.363676436002083], ["prying", 6.363666106695795], ["superficial", 6.363569251664719], ["companions", 6.3625899461622835], ["moai", 6.361991898301504], ["dodgeballs", 6.361991898301503], ["factly", 6.361556145431758], ["hainanese", 6.358794917448086], ["alara", 6.358219966345989], ["pappardelle", 6.356734053757453], ["l\u00fcneburg", 6.356734053757453], ["wapiti", 6.356538762388674], ["crisscrossed", 6.356273253898536], ["everest's", 6.356101165062336], ["shooing", 6.355854928624686], ["tersely", 6.352496388005531], ["saxony's", 6.352410585201652], ["pushing", 6.351767598958033], ["multiplication", 6.350176770490373], ["overcommit", 6.348893679687354], ["businesswoman", 6.3477470342867885], ["exacerbate", 6.34608691855575], ["shahryar", 6.344561783511637], ["delilah's", 6.343461824811306], ["dorjee", 6.343461824811306], ["scrambled", 6.342838241841724], ["exhilaratingly", 6.342334058166302], ["misplacing", 6.342073512451682], ["beliefs", 6.3416370983796915], ["socca", 6.3407852586405], ["kurmanji", 6.3407852586405], ["democratize", 6.340785258640499], ["escapism", 6.339208105805298], ["seoul's", 6.337788668253618], ["classname", 6.3363387051631905], ["wellness", 6.335338230630798], ["savvier", 6.334919592351285], ["garment's", 6.334919592351285], ["steely", 6.332443766489423], ["homegrown", 6.332394805557231], ["chaturanga", 6.332170061278216], ["housemade", 6.331026451008887], ["websocket", 6.330691480495215], ["communaut\u00e9", 6.329532755963231], ["rovs", 6.327624354127263], ["kommissar", 6.327000942860845], ["travelers", 6.326821667077585], ["cukes", 6.326506684526118], ["numismatic", 6.324645395227613], ["sadza", 6.324568416546788], ["slough's", 6.324568416546788], ["environmentalists", 6.323899471810424], ["terraform", 6.323572499939193], ["searching", 6.322992054516195], ["asado", 6.322888495059181], ["examining", 6.322648001035692], ["reeling", 6.3219579005024364], ["matchbooks", 6.320868756753548], ["veridian", 6.32071948250556], ["incredible", 6.319301740721625], ["reaping", 6.31856711807461], ["wailed", 6.318238422420992], ["serifs", 6.318167031509799], ["veggies", 6.317749097015518], ["yara", 6.316956791537817], ["montmartre", 6.316329214515307], ["doorstep", 6.316008387239097], ["l\u00ednea", 6.315722866313857], ["ladyfinger", 6.315722866313857], ["trebuchets", 6.314563166925993], ["windbreaks", 6.314372198463918], ["dashikis", 6.313708924073323], ["duckies", 6.3130534990838], ["frustratingly", 6.312728686699612], ["ramaswamy", 6.31231387592182], ["regained", 6.311744515687249], ["forgiveness", 6.311430804793949], ["labyrinth", 6.311189521819479], ["guidebooks", 6.309747726421413], ["rewards", 6.309278251144149], ["enlightened", 6.30813115135666], ["oupa", 6.308076712722678], ["mulching", 6.307486068835718], ["marcus's", 6.307368735538309], ["tasi", 6.30706822723703], ["giselle's", 6.306324469191366], ["strode", 6.305915843229519], ["bhosle", 6.303313332249209], ["personalizing", 6.302632241750387], ["kinfolk", 6.301401499270062], ["ptarmigans", 6.301401499270061], ["harly", 6.301401499270061], ["eatonville", 6.300883291909923], ["anara", 6.300370510804867], ["regaining", 6.298952856839638], ["trowels", 6.297954343088841], ["brecht's", 6.297954343088841], ["layering", 6.296906832770774], ["accomplishment", 6.296872366942461], ["lethargic", 6.295523326229697], ["ndlovu", 6.295523326229697], ["dietary", 6.2938195360140154], ["devastation", 6.293578697751275], ["increasingly", 6.293505055516288], ["hemo", 6.2934226177120305], ["tartufo", 6.292839812450401], ["asuka's", 6.292321670020392], ["jatiya", 6.291979547267564], ["decimals", 6.290771369528482], ["succumbed", 6.288499922924572], ["mosaics", 6.28832672895275], ["tambi\u00e9n", 6.288080521835942], ["swarmed", 6.2873608394945055], ["imperceptible", 6.287253935224371], ["etty", 6.286868746734845], ["parasailing", 6.285478689698595], ["silesian", 6.285100274365079], ["lineage", 6.2847864757856575], ["suspenseful", 6.284617850926254], ["doenjang", 6.28146035528503], ["thog", 6.28146035528503], ["superfoods", 6.280177177585993], ["neurotechnology", 6.278896493915526], ["keepsake", 6.278481442587266], ["inconsideration", 6.27767634302281], ["pa\u00edses", 6.27767634302281], ["miscalculated", 6.277534442562977], ["treehouse", 6.27618717362146], ["path", 6.274451261827185], ["enraptured", 6.27438960148196], ["weighing", 6.273935311015475], ["xiv's", 6.273913978427994], ["gripping", 6.273368517709444], ["commensalism", 6.272638079505135], ["guido's", 6.272326240674436], ["casually", 6.2713509152602755], ["bourgh", 6.271106299754339], ["marinated", 6.2708349994601695], ["tohunga", 6.269640403003579], ["songhua", 6.269640403003579], ["mitigate", 6.2694405003112275], ["dramatically", 6.266572730251053], ["lead", 6.2664241095595665], ["bioethicists", 6.266022867971193], ["dialogue", 6.265925820637151], ["fiona", 6.265697861676861], ["jungles", 6.26516199090361], ["kuchipudi", 6.264971521852407], ["motioned", 6.2645315968807145], ["blooms", 6.2638120934223425], ["quetzal", 6.263755258695378], ["fandom", 6.263439931210222], ["refreshed", 6.262880642492571], ["glowing", 6.2627640691474795], ["sacredness", 6.262729494587041], ["jmeter", 6.2620177398996235], ["hanaa", 6.2620177398996235], ["lifeless", 6.261322732381321], ["monikers", 6.260913522347181], ["aldrin's", 6.260913522347181], ["noblewomen", 6.259162119884849], ["marquita", 6.258325512458645], ["fencepost", 6.257835393771789], ["jane", 6.257565727861285], ["castleford's", 6.255958699996478], ["theravadin", 6.255958699996478], ["feeling", 6.25570846164848], ["conducive", 6.2557027577936735], ["comrade's", 6.255361329865183], ["elias's", 6.2546332850176665], ["popped", 6.254458060664535], ["surge", 6.253053614941556], ["clippings", 6.252565637650719], ["usfl", 6.252093781516264], ["arborist", 6.2515926271250954], ["brightside", 6.2506570400812285], ["karima", 6.249480787559838], ["telehealth", 6.249319238046539], ["seurat's", 6.248889820109477], ["pasmo", 6.248889820109477], ["maximillian", 6.248408692159053], ["carabiners", 6.2470911549425], ["armrests", 6.246934598013072], ["simplifying", 6.246141161903415], ["miscalculating", 6.246018087655383], ["goodall's", 6.246018087655383], ["benefactor's", 6.245721011884269], ["clattered", 6.244853871795615], ["pr\u00edncipe", 6.243804697028709], ["meditations", 6.2435754984339376], ["excited", 6.24315972181053], ["niuean", 6.242961082139734], ["pureeing", 6.242961082139734], ["cubism", 6.2424827676222545], ["alienate", 6.242362667466153], ["humming", 6.242349027131681], ["inari", 6.242325860214405], ["prewritten", 6.242201228064498], ["altering", 6.242168932638506], ["parfaits", 6.2417914250663715], ["handpicked", 6.241287561103729], ["implored", 6.240946997741916], ["kahlo", 6.2399132789284675], ["believing", 6.239895798466016], ["eloquence", 6.239645900257245], ["tongariro", 6.23921661605358], ["eyebrows", 6.237599108311316], ["wearily", 6.236930824684208], ["protagonists", 6.236660657904965], ["kangxi", 6.236291251923772], ["kamau", 6.235305277022283], ["l\u00e9onie", 6.234752060335474], ["brahma's", 6.234752060335474], ["outweighing", 6.234686942711131], ["tamales", 6.233700491426664], ["firmly", 6.23221998755995], ["shashlik", 6.2312660373775], ["rumors", 6.2307945271399126], ["tablecloths", 6.2306132180224525], ["ziggurats", 6.230029184253765], ["fogginess", 6.227655017580397], ["schedules", 6.2267552531815715], ["boardgamegeek", 6.225498253937944], ["kamila", 6.224809111775261], ["resembled", 6.223811812585124], ["wetsuits", 6.223618574513447], ["handlooms", 6.223016347125014], ["dickinson's", 6.222176029838831], ["scipy", 6.221682107190928], ["landfills", 6.220636705885993], ["squeezebox", 6.220415177184372], ["countee", 6.220415177184372], ["whopping", 6.218636291701523], ["pedro", 6.218394269778453], ["oohing", 6.218182744826383], ["whiteflies", 6.218182744826383], ["piled", 6.215975520044183], ["mongodb", 6.214536385144609], ["achieving", 6.214072613772585], ["mediums", 6.2132803376787065], ["na\u00efvet\u00e9", 6.213028185560786], ["dived", 6.212249838187141], ["walter's", 6.2118671973919435], ["eased", 6.2113056182833075], ["aravind", 6.209903876490256], ["surfaced", 6.208937509565971], ["muttering", 6.2085115213609985], ["diorama", 6.206956950100177], ["exploitative", 6.206446587906591], ["infectious", 6.205246599457284], ["cloaked", 6.204616309238256], ["aaliyah's", 6.202942100843967], ["maeve's", 6.202942100843967], ["variable's", 6.202942100843967], ["lumea", 6.202942100843967], ["slickrock", 6.202942100843967], ["marsha's", 6.202942100843967], ["farhad's", 6.202942100843967], ["onlookers", 6.202942100843966], ["germaphobe", 6.202942100843966], ["pasturelands", 6.202942100843966], ["ivanov's", 6.202942100843966], ["surrealistic", 6.202942100843966], ["zayde", 6.202942100843966], ["kerman's", 6.202942100843966], ["asif's", 6.202942100843966], ["codered", 6.202942100843966], ["pirozhki", 6.202942100843966], ["angelotti", 6.202942100843966], ["babel's", 6.202942100843966], ["erd\u0151s", 6.202942100843966], ["setsubun", 6.202942100843966], ["festas", 6.202942100843966], ["abdulsalami", 6.202942100843966], ["antiviruses", 6.202942100843966], ["botolph's", 6.202942100843966], ["assumptions", 6.200853568150079], ["piotr", 6.199282547982111], ["obscuring", 6.198527195434114], ["legacy", 6.198223623519393], ["crediting", 6.1975638273461255], ["subzero", 6.197277313537261], ["offered", 6.196859105137338], ["sturdiness", 6.194571058872517], ["misting", 6.193543703721476], ["junkyards", 6.193111764709507], ["notebook", 6.192791831951676], ["soaring", 6.192566270420737], ["slouched", 6.191664024296977], ["indulgences", 6.190749585658033], ["metrics", 6.189540682724859], ["triumphing", 6.187995252408197], ["algal", 6.187751222229656], ["piazzas", 6.185806901670364], ["mended", 6.184037896346156], ["strengthens", 6.183456418851787], ["khateeb", 6.182867854562918], ["fotheringay", 6.181915178468224], ["tatami", 6.181552645323815], ["syncopation", 6.181366650058423], ["symbols", 6.1804564357284075], ["earthier", 6.180385947749988], ["suica", 6.180385947749988], ["cephalopods", 6.179386624511647], ["collided", 6.178402989236233], ["emporium", 6.177942837969028], ["reinvention", 6.177392224913717], ["blunders", 6.177001137568604], ["fixated", 6.1744555699560495], ["imbibed", 6.174422826817098], ["grumbling", 6.1741655653245875], ["unmistakable", 6.173995037706695], ["subsequence", 6.173120263820678], ["moctezuma", 6.1727819285546115], ["rasputin's", 6.171771537523143], ["dimitri's", 6.171771537523143], ["timekeeping", 6.168159247942037], ["asam", 6.16733393723407], ["nearsightedness", 6.165798734970649], ["nikos", 6.165348512354003], ["cherrywood", 6.164092776621562], ["served", 6.1637464202624725], ["fives", 6.163512546404494], ["fred's", 6.163316426489681], ["brixton's", 6.162923119548199], ["hanafuda", 6.162923119548199], ["consequences", 6.162696571416796], ["phishing", 6.162563802095487], ["tala", 6.162433091205802], ["ramu", 6.162043581497743], ["komang", 6.16186301408341], ["petrov's", 6.16186301408341], ["pessoas", 6.16186301408341], ["perplexing", 6.160215808390428], ["wonderings", 6.160163189803663], ["microclimate", 6.159156627190951], ["sqlite", 6.158635371552224], ["summarizer", 6.156994381578456], ["restfulness", 6.156994381578456], ["washed", 6.155287752005737], ["quieter", 6.154791408736958], ["deep", 6.1544705389988], ["communally", 6.153647196731299], ["brightfield", 6.152511677259869], ["ashamedly", 6.152511677259869], ["cantopop", 6.152511677259869], ["subconsciously", 6.151666345245716], ["growth", 6.150362021540551], ["chhath", 6.149925501691453], ["pique", 6.149003473880106], ["grit", 6.148983422345918], ["user's", 6.146130107770817], ["jedes", 6.1449706793407515], ["started", 6.144664826853262], ["freyja", 6.14386646178831], ["ruffs", 6.14386646178831], ["bossche", 6.14386646178831], ["kutchi", 6.14386646178831], ["mathbb", 6.14386646178831], ["ucsd's", 6.14386646178831], ["benjamin's", 6.143631100676534], ["canvases", 6.143298426797389], ["dire", 6.14294865645506], ["kali's", 6.14248457549461], ["cath\u00e9drale", 6.142128942992555], ["konmari", 6.140912679835527], ["mollified", 6.1404439436566225], ["steeds", 6.140013702719463], ["holden's", 6.137647973466661], ["playdate", 6.1372638903644425], ["showcase", 6.1359920767266845], ["mark\u00e9ta", 6.134777901933592], ["perpetuated", 6.134092691138264], ["salsas", 6.132852359591492], ["edamame", 6.132554105373397], ["logistician", 6.132186107298218], ["visiting", 6.1321608924350635], ["undreamt", 6.131643915776794], ["hackerrank", 6.130534216398317], ["lucien's", 6.130534216398317], ["jillian's", 6.130308118398487], ["homebrew", 6.129646942540778], ["meandering", 6.129506766148836], ["quito's", 6.128207858665124], ["visvesvaraya", 6.128207858665124], ["vastly", 6.127673786622695], ["mapmaking", 6.1269877077724075], ["synchronicities", 6.126362568734782], ["sleeker", 6.1262765243166815], ["bool", 6.126251180324441], ["recalculate", 6.125940060971422], ["expectations", 6.125439158813058], ["jazzie", 6.124423846402904], ["neilston", 6.124423846402904], ["elizabeth's", 6.124028932105419], ["discussed", 6.123315680967885], ["irritations", 6.122904138252431], ["satchmo", 6.122084382583431], ["resubmitted", 6.122034160398176], ["oppressive", 6.121116056109429], ["chromaticism", 6.120783927322854], ["surrealism", 6.120647678278873], ["guessable", 6.11911855894067], ["redis", 6.11850414554452], ["sinigang", 6.117970291243363], ["tenochtitl\u00e1n", 6.113904654420369], ["wandering", 6.113486042020385], ["divisor", 6.113317842352685], ["mathayus", 6.113044389237532], ["pythagoras", 6.112293677658355], ["akua", 6.1122936776583545], ["wurde", 6.110360875458235], ["cisgender", 6.110254460256643], ["bludgers", 6.109430410881495], ["solche", 6.108958129619058], ["carpoolers", 6.108958129619058], ["titled", 6.108519537753342], ["theo's", 6.107512222369444], ["bioacoustics", 6.107512222369444], ["obsessions", 6.106133177882818], ["innovating", 6.10605805279269], ["raju's", 6.105804305081309], ["authentication", 6.105467296402132], ["surrounded", 6.104399291399118], ["mettle", 6.103695027230463], ["allowing", 6.103397524012197], ["shake", 6.102303713114387], ["tumbled", 6.101992258810624], ["akash", 6.101669576748555], ["resorted", 6.100775995653595], ["lemons", 6.099877832091482], ["storeowners", 6.099559732496567], ["ithaca's", 6.099559732496567], ["quipus", 6.099559732496567], ["priceless", 6.098797967677165], ["miguel's", 6.098314161793586], ["acclimating", 6.097807488965255], ["mater's", 6.096908902538941], ["murmurings", 6.096699350763635], ["flurry", 6.095232098472723], ["counteroffers", 6.09217527761461], ["jubilations", 6.09217527761461], ["farhana", 6.09217527761461], ["pixote", 6.09217527761461], ["innovators", 6.09160724262369], ["tempura", 6.0905263216700956], ["braced", 6.090477543127731], ["pair's", 6.090161335374076], ["pici", 6.090161335374076], ["discographies", 6.089787269816962], ["rao's", 6.088496674629134], ["prediabetic", 6.087645779266942], ["passerby", 6.0876197969972905], ["mkdir", 6.086999257837538], ["oskar's", 6.086999257837538], ["exponentiation", 6.086394128176752], ["plunging", 6.084277121523473], ["numerators", 6.082262293434551], ["jirka", 6.081315785141143], ["revelry", 6.081238806460319], ["streetlamp", 6.080783503671984], ["asymptotes", 6.080630566461128], ["mocktails", 6.080630566461128], ["hiking", 6.08005520470309], ["mordecai's", 6.079704840562298], ["spur's", 6.079704840562298], ["finalmente", 6.079377517161816], ["sandboarding", 6.079377517161816], ["teas", 6.078883258827087], ["snowstorms", 6.077489339253864], ["geographical", 6.076254893530119], ["hoping", 6.076099887371581], ["technophobes", 6.0746053677230565], ["expressions", 6.074111764903361], ["sch\u00e4fer", 6.073237284467156], ["chichen", 6.071822999525313], ["xenophon's", 6.071762056415243], ["recycled", 6.071364541129094], ["skillfulness", 6.070964609336648], ["studious", 6.070660338462856], ["hasn", 6.070519741387748], ["paddled", 6.070146720656885], ["secrets", 6.070126164096483], ["sightseeing", 6.068095533434316], ["erasers", 6.067605182280097], ["scents", 6.066665342567848], ["zzzs", 6.065098943047434], ["flooded", 6.065009579573984], ["linguists", 6.06451486187033], ["meryton", 6.0635501435216295], ["anuradha", 6.062762618339018], ["wreaked", 6.061430113752469], ["contorting", 6.060634387329328], ["empower", 6.060504171120883], ["caravanserais", 6.0603457307096225], ["weightiest", 6.05868763338248], ["lipa's", 6.05868763338248], ["kachori", 6.056990522000579], ["superfan", 6.056616287183031], ["unintentional", 6.055387266020859], ["gather", 6.054499279534114], ["misusing", 6.051033314700849], ["wreaking", 6.050535661019791], ["uncomfortable", 6.050469782107092], ["andr\u00e9's", 6.049783036625597], ["uluru", 6.049692943058409], ["feats", 6.048233427269976], ["refactor", 6.047664235341944], ["swapped", 6.0476034653271045], ["jahan", 6.04727891504332], ["stalls", 6.04671985534123], ["emojis", 6.045147959682146], ["tackle", 6.044921051012538], ["casting", 6.044217171057831], ["puppeteer's", 6.0438923033864285], ["megatech", 6.0438923033864285], ["babushka", 6.042821260979308], ["buffs", 6.042256362612581], ["tambura", 6.0411262199523845], ["blackacre", 6.0411262199523845], ["precautions", 6.040294261695872], ["detectorists", 6.039315330797723], ["suspicions", 6.039136251190611], ["grandchildren's", 6.037775003484216], ["smartboard", 6.036791865999932], ["motifs", 6.036621455502656], ["gochujang", 6.0358263865581545], ["reconquista", 6.035295017037373], ["hozier's", 6.035295017037373], ["sadness", 6.033117151100175], ["managing", 6.0329666595308], ["spurred", 6.032531603568034], ["downvoted", 6.029951683688955], ["sagte", 6.029027836334322], ["rith", 6.029027836334322], ["evenki", 6.029027836334322], ["nkiru", 6.029027836334322], ["accidentally", 6.028063974145474], ["humdrum", 6.027691692046888], ["flashiest", 6.027085494892205], ["atticus", 6.0263458844498325], ["paganini's", 6.025715183676996], ["yamada's", 6.025715183676996], ["retrowave", 6.025715183676996], ["oversharing", 6.023435737797035], ["takashi's", 6.0231466776310985], ["brilliance", 6.022677007954134], ["demoiselles", 6.022543337284746], ["nile's", 6.022543337284746], ["manor's", 6.022543337284746], ["zealousness", 6.022273884314531], ["incentivize", 6.022022956236018], ["recreate", 6.0215388394053875], ["paradises", 6.020717009749842], ["mislabeled", 6.020502627289733], ["lusciously", 6.020502627289733], ["rubik's", 6.019701495868935], ["societally", 6.018415219125496], ["familiarity", 6.01759522532074], ["datatable", 6.016853837818648], ["bloodcurdling", 6.016165714289235], ["darlington's", 6.01497415839415], ["egyptology", 6.014156906470455], ["radiantly", 6.012199723819244], ["ahab's", 6.0107382610995055], ["breathless", 6.006849092494706], ["fauvism", 6.006307048655829], ["joconde", 6.0060233039917765], ["extraordinaire", 6.005891143725433], ["dispelling", 6.005027216692525], ["reusing", 6.002928865755529], ["hokusai's", 6.002847194365129], ["multicollinearity", 6.0028471943651285], ["choosing", 6.000630758477979], ["barny", 6.000397052653144], ["ficus", 6.000108252679486], ["monocles", 5.9999367229981635], ["optimism", 5.998801511792121], ["austen's", 5.998184788777273], ["infectiously", 5.997975318381296], ["purples", 5.9975466670411866], ["petru", 5.9975466670411866], ["thura", 5.9975466670411866], ["mindscape", 5.997386514656155], ["aws's", 5.996177364149167], ["tootsie's", 5.996177364149167], ["garments", 5.995653909119561], ["neighborliness", 5.995645835263063], ["headsets", 5.995512525767512], ["nabatean", 5.995188059188906], ["dissect", 5.993573274266361], ["sabor", 5.99342614452068], ["panic", 5.991496755823989], ["leilani", 5.991368881900451], ["plummet", 5.98916839002392], ["keisha's", 5.989047545642451], ["helmi", 5.987562166786884], ["marketer", 5.987127055808991], ["omar's", 5.986884566994346], ["solvers", 5.985764751509861], ["chessboards", 5.984014732578886], ["freefalling", 5.982979614998436], ["hajja", 5.982979614998436], ["fern's", 5.982125391752434], ["stamp's", 5.982125391752434], ["crispy", 5.981862882377989], ["garlic's", 5.981408454385253], ["spiderland", 5.981408454385253], ["stealthily", 5.980798168857819], ["nijo", 5.980525850216095], ["zee's", 5.980525850216095], ["fugu", 5.9792783231693045], ["ensures", 5.978514044934034], ["euskera", 5.977380569904186], ["aristaeus", 5.977380569904186], ["visually", 5.976062047385122], ["contorted", 5.975792108418694], ["sulked", 5.9756091442685335], ["delacroix's", 5.9756091442685335], ["cautiousness", 5.975078921629289], ["hyperaware", 5.975078921629289], ["bissau's", 5.9732035045164125], ["wilis", 5.9732035045164125], ["clio's", 5.9732035045164125], ["augustiner", 5.9732035045164125], ["unapologetic", 5.971714113292246], ["vintner", 5.971056414831107], ["apprehensive", 5.969896986401044], ["nimbleness", 5.967700783646292], ["ladur\u00e9e", 5.967387337520778], ["li's", 5.964618535916803], ["swaths", 5.9642606598271515], ["ikebana", 5.963049312413537], ["droughts", 5.962951096328203], ["witty", 5.962684894403563], ["malpractices", 5.9624049805997235], ["miriam's", 5.9624049805997235], ["sanctuary", 5.962259200526848], ["barnsley's", 5.95968946943832], ["katarina's", 5.95968946943832], ["traditional", 5.95950351475333], ["graf's", 5.959322018333478], ["nyc's", 5.958513899100673], ["exceptional", 5.956264385512189], ["integer", 5.955837142051163], ["antics", 5.9553413286519445], ["diaphragmatic", 5.955332334280083], ["runoffs", 5.954824416810208], ["phrases", 5.954025068601336], ["thorough", 5.952298030959917], ["cryptopia", 5.951471475134076], ["falter", 5.951446112489022], ["vivaldi's", 5.94956927675503], ["potager", 5.949290193374511], ["crewmates", 5.947867846229822], ["vibraphone", 5.94715067400504], ["devout", 5.944061320927464], ["profundity", 5.943831861188459], ["timelines", 5.9433222725172525], ["discrepancy", 5.9427366322034345], ["kettlebell", 5.942640066254979], ["rapport", 5.942250885524713], ["achievements", 5.94198702359281], ["buzzwords", 5.941976899190338], ["tahlia", 5.941976899190338], ["emboldening", 5.940105571147188], ["fermenter", 5.940105571147188], ["ergonomically", 5.940105571147188], ["janek", 5.940105571147188], ["interrupted", 5.939930346794057], ["uninterested", 5.939464750655738], ["fingerstyle", 5.9394416901307325], ["pyr\u00e9n\u00e9es", 5.939282369488183], ["themes", 5.938896862254492], ["arundel's", 5.937859105081403], ["carpooled", 5.936148892205516], ["fullname", 5.935829761573173], ["khepri", 5.935829761573173], ["locket", 5.934883469872054], ["heartbreaking", 5.934814926162325], ["oliver", 5.933898829000132], ["olympism", 5.933248966024665], ["tuvaluan", 5.933248966024663], ["clingmans", 5.933248966024663], ["bickered", 5.932268263716229], ["absurdly", 5.931432850638674], ["sputtered", 5.93103231012204], ["kameni", 5.930285305202473], ["hamza's", 5.929592109959317], ["recycling", 5.92914963037541], ["distanced", 5.928666667111665], ["hometown", 5.927357514518648], ["knowingly", 5.927343027755836], ["haudenosaunee", 5.926932967551143], ["etiquette", 5.926651691730678], ["humble", 5.92664671083273], ["teslas", 5.925198424686773], ["monarchy's", 5.924442659581584], ["parabolas", 5.924442659581584], ["payasam", 5.924158186199294], ["positionality", 5.924158186199294], ["arroz", 5.923809706305988], ["awkwardness", 5.9225691178858195], ["bambaataa", 5.921842005639557], ["sal's", 5.920990187169241], ["novell's", 5.920990187169241], ["leaning", 5.9201327045451775], ["narrating", 5.918143557271563], ["nanay", 5.917905374109343], ["rehearsing", 5.917147287012489], ["syncopated", 5.9166524654204], ["homographs", 5.916652465420399], ["jitteriness", 5.916652465420399], ["option's", 5.915388493520075], ["organize", 5.91533343463939], ["shiver", 5.913840692215346], ["pennekamp", 5.913084993327893], ["zimmy", 5.913084993327893], ["dismissed", 5.912974187193121], ["strategically", 5.912591619527866], ["environmentalism", 5.911783594069658], ["paintbrush", 5.9113889469433865], ["tilbury's", 5.911038943157192], ["mixologist", 5.911038943157191], ["march\u00e9", 5.910656870961267], ["jamari", 5.910656870961267], ["ladyland", 5.909712110622252], ["stardom", 5.90943703558452], ["kefir", 5.908876697544697], ["angkor", 5.907687236753691], ["ljiljana", 5.907563905565683], ["kalamkari", 5.907563905565683], ["kaqchikel", 5.907563905565683], ["february20", 5.907563905565683], ["ebuka", 5.907563905565682], ["bize", 5.907563905565682], ["shorelines", 5.9071726761679635], ["doesn", 5.90624715939396], ["unmindful", 5.90509728180553], ["interspersing", 5.90509728180553], ["reimagined", 5.904860214991053], ["legumes", 5.90478963522329], ["sardana", 5.904005132128595], ["nietzsche's", 5.903489723561844], ["cephalopod", 5.903133232636509], ["river's", 5.901970945655087], ["embellishments", 5.899970635764183], ["forgetfulness", 5.899324408539498], ["kiddo", 5.897279050332498], ["cruncher", 5.896711893798856], ["pledged", 5.896627477191306], ["inuk", 5.896623972407228], ["closeby", 5.896623972407228], ["tadi\u0107", 5.896623972407228], ["fido's", 5.896623972407228], ["puebloan", 5.896332795479055], ["shovelful", 5.896332795479055], ["demanding", 5.895459963519167], ["strengthen", 5.895276172642106], ["clenched", 5.893176480555818], ["apparitions", 5.8927949958017685], ["sappho's", 5.892794995801768], ["vixen's", 5.892794995801768], ["havisham's", 5.892794995801768], ["shocking", 5.892105780012786], ["confidentially", 5.892032963602648], ["astrophysics", 5.891842650545754], ["melodica", 5.891236467635728], ["unpredictably", 5.8911711889690865], ["parlophone", 5.891062330410471], ["friendliness", 5.890972744890827], ["nicolae", 5.890794046736979], ["humanize", 5.890595370943312], ["grassroot", 5.890124602523422], ["benvolio", 5.890098064453575], ["inclusively", 5.889514944162074], ["microhabitat", 5.8891746015794855], ["anticlimactic", 5.889146206330683], ["wasn't", 5.88878744053871], ["resurface", 5.886257937381675], ["bassline", 5.884724253291377], ["churned", 5.884377642220247], ["kanchenjunga", 5.884361992945636], ["beeline", 5.883332880732684], ["yarn", 5.882508709905896], ["consisted", 5.882494907442433], ["alfajores", 5.881308065985391], ["marathoner", 5.8801873899057435], ["digitizing", 5.880049224361678], ["breakdancing", 5.879310339060804], ["crowdsource", 5.879310339060803], ["sabzi", 5.879197732323926], ["daring", 5.87905195042864], ["diesem", 5.878270200248828], ["tybalt", 5.878026086037854], ["sauces", 5.877187593096418], ["beignets", 5.87717129383287], ["latecomers", 5.876471463957442], ["underpinned", 5.8763382106362645], ["modernization", 5.876271874921791], ["estranged", 5.875683431385647], ["engulfing", 5.874309406759519], ["chic's", 5.874309406759518], ["validated", 5.8716016102905515], ["nour", 5.870888694702134], ["glenhaven", 5.870641631155896], ["companionably", 5.870641631155896], ["\u00e9douard", 5.870015829894714], ["pythia", 5.870015829894714], ["brainiacs", 5.869450590045903], ["dandiya", 5.869450590045903], ["afua", 5.869450590045903], ["loseit", 5.8685226136680315], ["hypnotizing", 5.868401403270359], ["unnecessary", 5.868394018265998], ["benefited", 5.86814227719585], ["steve's", 5.867863556489694], ["jacquemart", 5.86764793323078], ["wakanda's", 5.86764793323078], ["misclassify", 5.864599804434295], ["meer's", 5.864599804434295], ["romsdal", 5.864599804434295], ["housecats", 5.864599804434295], ["platters", 5.864347498471275], ["listenership", 5.863612664255128], ["labored", 5.862544849436452], ["goldfishes", 5.8621211062921], ["carraway", 5.861815094175779], ["seamstress", 5.861727628743211], ["belittled", 5.861679914066143], ["setups", 5.861482066927315], ["perishables", 5.859800622924938], ["theravada", 5.859164586821288], ["genealogy", 5.8590111147168145], ["thoreau's", 5.858880336137882], ["mandapa", 5.858334206352636], ["monastery's", 5.858334206352636], ["tucum\u00e1n", 5.858334206352636], ["guide's", 5.857461781303289], ["tradeoffs", 5.856975189303937], ["sensationalist", 5.854026607671494], ["beyonc\u00e9's", 5.854026607671494], ["radiant", 5.853248730628103], ["flyaways", 5.853032546437384], ["pulp's", 5.853032546437384], ["coexists", 5.852493394581596], ["neon", 5.852341025578873], ["doubts", 5.8522078437839005], ["programmer", 5.85161292841049], ["healing", 5.8508339168842936], ["f\u00eate", 5.850775065441212], ["westville", 5.850577758566656], ["farouk's", 5.849929135755286], ["d\u00f4me", 5.8491621331152235], ["frugally", 5.848488266510026], ["improving", 5.8477308865221325], ["cementing", 5.847665231290984], ["spent", 5.847105896556123], ["blini", 5.846881980251715], ["disgruntlement", 5.846881980251715], ["jacques", 5.846484531384161], ["refactoring", 5.8457045976540005], ["blunder", 5.844549890572982], ["letterform", 5.84335125441823], ["kant's", 5.84335125441823], ["\u00fanica", 5.84335125441823], ["paikea", 5.84335125441823], ["kindled", 5.842203624057296], ["marija", 5.8410509035582905], ["clashing", 5.839768614988896], ["reservations", 5.839612558664872], ["ripest", 5.839219663868963], ["luthier", 5.838642326667416], ["chowpatty", 5.838063153735498], ["starfishes", 5.838063153735498], ["gozon", 5.838063153735497], ["droysen", 5.838063153735497], ["wriggling", 5.8373382286750175], ["chennai's", 5.836187169431599], ["however", 5.834741987150606], ["walked", 5.834582769932309], ["cleo's", 5.8343462204882135], ["misconstrued", 5.833841413851599], ["trombonists", 5.833230319998962], ["g\u00f6bekli", 5.833230319998962], ["sencha", 5.831825906776378], ["shisa", 5.831825906776378], ["chihiro's", 5.831825906776378], ["castoffs", 5.831063078011594], ["bolivar's", 5.830765574793328], ["kolya", 5.830169138052863], ["balked", 5.829667708934773], ["zers", 5.829270889949751], ["draped", 5.829122411322906], ["birbal", 5.828895340994079], ["resolve", 5.8273288095752935], ["echinopsis", 5.8270062159443325], ["iss's", 5.8270062159443325], ["performant", 5.82596771902472], ["decimus", 5.82596771902472], ["dissipated", 5.825463914878077], ["sandstorms", 5.8248580108877634], ["eons", 5.82480867841256], ["treasure", 5.824143514982968], ["asakusa", 5.823984223300444], ["trustworthiness", 5.823873416903501], ["hakim's", 5.823170135486173], ["sauerkraut", 5.822893131864467], ["honking", 5.822877440329828], ["feasting", 5.822705254549294], ["phoenixes", 5.822330396480406], ["backyard", 5.82055290358816], ["aunque", 5.820044440298043], ["trollish", 5.819487498246339], ["untimely", 5.81854765853409], ["valera's", 5.817931901481238], ["exfoliant", 5.817448523955359], ["erevan", 5.81711316634171], ["getmotivated", 5.81711316634171], ["transparently", 5.817038813568079], ["miserliness", 5.815258219541219], ["flavortown", 5.815258219541219], ["mbali", 5.815258219541219], ["cyberattacks", 5.8114088760389215], ["interacting", 5.8112201992458115], ["spoonbills", 5.811177336580137], ["playthroughs", 5.810859861082354], ["vimeo's", 5.8103508286386525], ["musicianship", 5.809422118102253], ["sparta's", 5.809401967557793], ["luna", 5.809047839859918], ["rigorously", 5.80843498323029], ["zoey", 5.807577006192043], ["ethicality", 5.807009626322011], ["file1", 5.807009626322011], ["scent", 5.805428292384434], ["equilateral", 5.804181537218283], ["\u0447\u0435\u0440\u0435\u0437", 5.802752287886291], ["taquerias", 5.802752287886291], ["earthworms", 5.80106134501464], ["item's", 5.800589099708142], ["playdough", 5.799898319331126], ["mechelen", 5.7996548435135535], ["hideaways", 5.799422418035659], ["transcribing", 5.798118511104676], ["affirmatively", 5.797142150321464], ["amused", 5.796365777891892], ["altercations", 5.795077414761075], ["saarbr\u00fccken", 5.794365076237478], ["floorings", 5.793956907381727], ["quills", 5.793427476710579], ["counterfeits", 5.793104854895347], ["hua's", 5.792630755651953], ["lina's", 5.792220977018983], ["superliga", 5.792151233238406], ["sana's", 5.791796082632406], ["bauhaus", 5.791490665009091], ["whirrs", 5.789412627454369], ["indignantly", 5.789412627454369], ["neza", 5.789412627454369], ["headgears", 5.789412627454369], ["scrapy", 5.789412627454368], ["sollozzo", 5.789412627454368], ["hfmd", 5.789412627454368], ["omondi", 5.789412627454368], ["rafting", 5.788189167473926], ["cushioning", 5.787875343166675], ["amir", 5.787765099671939], ["exertion", 5.787329355548124], ["finally", 5.786976156503046], ["openly", 5.7868160470400625], ["mantises", 5.78543638251793], ["keiko's", 5.784766228876958], ["modernizing", 5.783240545761987], ["humanity", 5.782566637129354], ["avellino", 5.782173599473808], ["funland", 5.781665002660184], ["quilling", 5.78138292894195], ["greed", 5.78017174536745], ["aesthetical", 5.779739423398471], ["laughed", 5.779716074285234], ["smiles", 5.779494023849083], ["pizzeria", 5.779367377088629], ["outdated", 5.778644573333283], ["limply", 5.778273786420306], ["agtech", 5.775152990440934], ["daha", 5.7743751920583835], ["rsvps", 5.772890885861835], ["paradise's", 5.772533873438466], ["rgba", 5.7721822327298025], ["trnava", 5.771921954863872], ["overcome", 5.7702035164323995], ["minification", 5.770178698459503], ["siteground", 5.770178698459503], ["catchy", 5.769739866098941], ["uninviting", 5.768736153784889], ["sabermetric", 5.7687361537848885], ["macanese", 5.7687361537848885], ["flexibility", 5.768263548672444], ["cleanings", 5.768205987793364], ["capuchin", 5.768205987793364], ["yasmine", 5.767994766282795], ["schoolmates", 5.767503781182072], ["survivalism", 5.766554214352431], ["pirin", 5.766554214352431], ["embodies", 5.765934825094185], ["airness", 5.765087364313804], ["schmidt's", 5.763667601873694], ["lightbulbs", 5.76356703536752], ["demystifying", 5.763097115511394], ["reducing", 5.762831169601553], ["shanti's", 5.762733306590524], ["geralt's", 5.762733306590524], ["beginners", 5.761052751182635], ["fledgling", 5.759874807926541], ["hareton", 5.759874807926541], ["fopen", 5.759874807926541], ["ramdas", 5.759276876357151], ["preserve", 5.759131598273904], ["coalesced", 5.758780814610695], ["potentially", 5.756988540989821], ["chronicling", 5.756330269583201], ["repotted", 5.756044254961836], ["arcane", 5.755974727354289], ["yaroslav", 5.755614545494641], ["bistros", 5.755307103875846], ["backyards", 5.7552366379180375], ["momos", 5.7550272445466994], ["comfort's", 5.754820808387557], ["golestan", 5.754820808387557], ["h\u00e4ndel", 5.754536647770909], ["customizations", 5.7537635349207825], ["deepl", 5.753453542811796], ["deftly", 5.752615513296819], ["lutherie", 5.751819038964405], ["friendlier", 5.751604218458748], ["preparing", 5.751344727333925], ["visualization", 5.751225455988248], ["antagonist's", 5.75107214647785], ["captivates", 5.750294974566164], ["opportunity", 5.748936863846842], ["competitions", 5.747631469122254], ["foolproof", 5.747557417597123], ["gerald's", 5.746999348132358], ["whizzing", 5.746964237801794], ["slathered", 5.746633716414065], ["fate", 5.746253874554756], ["bijoy", 5.745883209202832], ["disapproved", 5.745733280092895], ["arepas", 5.7450067779628675], ["pihu", 5.744358832688187], ["porridges", 5.743464908188858], ["electroscope", 5.743464908188858], ["perceptions", 5.742707106555877], ["devising", 5.7425347924142525], ["vast", 5.742198000201343], ["specialized", 5.741909754613718], ["nudibranchs", 5.741566242103506], ["typeof", 5.741566242103506], ["blockchain", 5.741454067083146], ["shedding", 5.741304816207451], ["pawe\u0142", 5.741234436379853], ["mathematics", 5.740317000613646], ["uzun", 5.73917072881825], ["mithai", 5.73917072881825], ["charming", 5.737880493078126], ["shakeups", 5.73772144328067], ["insisted", 5.7372359956379935], ["responsibility", 5.736198300914269], ["burpees", 5.736053985726679], ["perceptiveness", 5.736053985726678], ["stormed", 5.7358605606504165], ["retraining", 5.734962127399511], ["funko", 5.734640511773628], ["timesaver", 5.73444194216995], ["pisac", 5.733022244719423], ["extrovert", 5.732719070618698], ["indecipherable", 5.732374079400631], ["vedder's", 5.732374079400631], ["distress", 5.73218683164187], ["portioning", 5.731169039653017], ["hartford's", 5.731169039653017], ["narrowed", 5.731149552950778], ["tyumen", 5.7304821373890205], ["rinc\u00f3n", 5.729033111815141], ["graciousness", 5.728599469602958], ["basmati", 5.728149001767021], ["wilhelmshaven", 5.727880889619593], ["convoluted", 5.727752429190026], ["viticulture", 5.727576444517606], ["grammatically", 5.7275070176655065], ["wicking", 5.725792708471355], ["kickstarting", 5.725792708471354], ["pastillas", 5.725792708471354], ["tuxedoed", 5.725792708471353], ["rubert", 5.725792708471353], ["instincts", 5.725301032282492], ["uniq", 5.7235578167038], ["participating", 5.722794363257347], ["vltava", 5.7218976113907605], ["caramelizing", 5.7218976113907605], ["persephone's", 5.721435179773887], ["hotspots", 5.7211872261207315], ["yellowing", 5.7211601900988045], ["waddled", 5.720491048556103], ["flamenco", 5.720410059626727], ["water's", 5.719697187437903], ["kilkenny's", 5.719024631983799], ["ard\u00e8che", 5.719024631983799], ["doneness", 5.718897486038224], ["overemphasis", 5.718897486038224], ["poverty's", 5.71869755022046], ["mitali", 5.71869755022046], ["padma's", 5.717494458169221], ["ossobuco", 5.717494458169221], ["suparman", 5.717494458169221], ["slices", 5.716875490318817], ["holi", 5.7168635970351405], ["bennington's", 5.716436838032674], ["siale", 5.716436838032674], ["leetspeak", 5.716436838032674], ["bulldust", 5.716436838032674], ["modenese", 5.716436838032674], ["moeran", 5.716436838032674], ["koffie", 5.715800051039467], ["palau's", 5.715800051039467], ["sasha's", 5.7157703924671805], ["belacan", 5.715070250215789], ["marred", 5.713766992078223], ["kolkata's", 5.712061790705235], ["oddballs", 5.710645108713493], ["hakka", 5.710645108713493], ["fingal's", 5.710645108713492], ["nihari", 5.710645108713492], ["stealthiness", 5.710645108713492], ["meines", 5.710645108713492], ["carlotta's", 5.710645108713492], ["quan's", 5.709993059717293], ["mediacorp", 5.708902464493562], ["zapotec", 5.707440808764716], ["nitra", 5.704213087710784], ["squawks", 5.702956843345208], ["spatulas", 5.702956843345208], ["fenerbah\u00e7e", 5.702956843345208], ["adjustments", 5.7023900105882666], ["protagonist", 5.701405072861197], ["validation", 5.701267827385313], ["stravinsky's", 5.699633202310575], ["blended", 5.698462316060138], ["yakshagana", 5.698396571110247], ["unapologetically", 5.696952720627702], ["chattering", 5.696429668940732], ["productivity", 5.696060548461525], ["prague's", 5.695970871448065], ["vel\u00e1zquez", 5.69542865622946], ["depended", 5.69542865622946], ["originality", 5.695377158163034], ["balinese", 5.6953329170397575], ["asako", 5.695316100575299], ["schiller's", 5.695316100575299], ["frisbee", 5.694192025029132], ["intense", 5.69389575847838], ["elasticsearch", 5.690219171924935], ["methods", 5.689923824712371], ["cookbooks", 5.688689109730125], ["reggiano", 5.688073388916206], ["harshest", 5.687887427400635], ["pervaded", 5.687620757081545], ["pythagorean", 5.686030259106969], ["varanasi's", 5.686030259106969], ["enums", 5.686030259106969], ["meritocracy", 5.685739041167963], ["\u0432\u0440\u0435\u043c\u044f", 5.683970849378934], ["elote", 5.683970849378934], ["yield", 5.683691678636766], ["familiarized", 5.683236141043526], ["vinaigrettes", 5.682695343998989], ["benito's", 5.682695343998989], ["workweeks", 5.682311468878646], ["alerting", 5.682024868919688], ["kito", 5.6819342857176105], ["clucking", 5.681535373526648], ["hydrogen's", 5.681199307315034], ["minelayers", 5.681199307315034], ["euphrosyne", 5.681199307315034], ["realms", 5.680751159446676], ["nannerl", 5.679802405592066], ["milkweed", 5.679431384531603], ["elders", 5.678874401263097], ["javascript", 5.678832752449872], ["brewmaster", 5.677719779336278], ["guzzling", 5.676996848280498], ["avait", 5.676728503600134], ["somersaulting", 5.676728503600134], ["fabio's", 5.675032134814693], ["geschichten", 5.674009053485178], ["candlelit", 5.673252663019089], ["juliette's", 5.67304252438996], ["katalin", 5.67304252438996], ["achievable", 5.673011738648408], ["grantville", 5.672776109318841], ["darma", 5.672776109318841], ["application's", 5.672427315903365], ["souk", 5.671261349343055], ["taxations", 5.671261349343055], ["willowdale", 5.670553856659755], ["convictions", 5.670432217566836], ["approaches", 5.669628472128708], ["osaka's", 5.66951956145935], ["others", 5.669493797237622], ["overcomplicated", 5.6693556835670655], ["dif\u00edcil", 5.6693556835670655], ["breakdancers", 5.6693556835670655], ["soneva", 5.668913972956738], ["jules's", 5.668913972956738], ["envirotech", 5.666885376079674], ["attic", 5.666322238536763], ["rome's", 5.666149034424777], ["entr\u00e9es", 5.666149034424777], ["grandfather", 5.666066767288161], ["codie", 5.665845246611298], ["navona", 5.665114751059808], ["captivation", 5.664787306706819], ["hyperbolas", 5.664787306706819], ["unclenched", 5.664787306706819], ["elevating", 5.664172272656377], ["embroiled", 5.661763658176551], ["blacksmithing", 5.661075894333459], ["elyria", 5.661075894333459], ["zanele", 5.660479483659893], ["mourned", 5.660286823510582], ["aconcagua", 5.659385318811249], ["capuchins", 5.658824372699759], ["juiciness", 5.658824372699758], ["barracudas", 5.657741551350218], ["ancient", 5.656487506331621], ["scrutinized", 5.656000707795956], ["glean", 5.653109616672323], ["finicky", 5.65303588623014], ["landmasses", 5.652961824787094], ["amstrad", 5.652768105812589], ["podcasts", 5.652482338252515], ["eiffel", 5.651974890710179], ["equals", 5.651895280758082], ["dreaded", 5.6518771552779175], ["initializing", 5.6515694696578365], ["battling", 5.65151320714445], ["mitigating", 5.650970151580459], ["cherubic", 5.650005608872462], ["adventuring", 5.649346192919182], ["anohni", 5.649107984697184], ["frazzled", 5.64889290348703], ["caching", 5.648691741937658], ["kl's", 5.648207441418896], ["tatau", 5.647814986760655], ["fauna", 5.647113512593115], ["discoverability", 5.646101786989498], ["fluctuations", 5.643625965649222], ["fretboard", 5.643086813793435], ["etching", 5.642379925804734], ["br\u00fbl\u00e9e", 5.642379925804734], ["halfhearted", 5.640333115807602], ["softening", 5.640138573645441], ["believed", 5.639357350749137], ["exhalations", 5.639038273494515], ["capsizing", 5.639038273494515], ["ion's", 5.639038273494515], ["dacian", 5.639038273494514], ["kupala", 5.639038273494514], ["amal", 5.638896232228181], ["nana's", 5.6378573230707465], ["saroja", 5.637167945708944], ["hien", 5.636327197401489], ["hakimi", 5.63600653248726], ["eggless", 5.635711407256466], ["seaview", 5.634822170112463], ["wriggly", 5.633992154234117], ["suriname's", 5.63255662030659], ["antidisestablishmentarianism", 5.63255662030659], ["costochondritis", 5.63255662030659], ["allergens", 5.632223061546042], ["existentialism", 5.632080077167323], ["brainy", 5.631735921773443], ["folklore", 5.631629051685327], ["hectic", 5.631343884822271], ["acclimatization", 5.631298417040699], ["desertification", 5.6310396376455865], ["remorse", 5.630424596908465], ["aerotech", 5.628595610025081], ["kalua", 5.628595610025081], ["rama's", 5.627411390456382], ["spanning", 5.6264995056114575], ["chaat", 5.625340204813962], ["backpacking", 5.624827897045309], ["promptly", 5.624598424042735], ["forgave", 5.624376773983429], ["fettuccine", 5.6222411382117645], ["struct", 5.6220019040881875], ["wawel", 5.6220019040881875], ["procrastinated", 5.619759510166329], ["napoleon's", 5.619759510166329], ["basquiat's", 5.619759510166328], ["algunas", 5.619759510166328], ["reggae's", 5.619759510166328], ["jeanine's", 5.619759510166328], ["mysqli", 5.619135785470417], ["amoah", 5.619135785470417], ["diffusing", 5.618851832982395], ["catering", 5.61841771706192], ["tver", 5.616594340067671], ["marjolein", 5.616177307520889], ["lizzo", 5.61568131023152], ["meatless", 5.614774682516929], ["facilitated", 5.614689578432803], ["doodling", 5.614055192535995], ["helfen", 5.614055192535995], ["tropica", 5.612945037267548], ["stetten", 5.612185710287398], ["oriya", 5.611876413747839], ["brokenly", 5.610701397748311], ["fatefully", 5.610701397748311], ["candombl\u00e9", 5.610701397748311], ["primatologist", 5.609792045819988], ["waggle", 5.609156190335826], ["propylaea", 5.60813943363975], ["chyme", 5.60813943363975], ["rinchen", 5.60813943363975], ["winked", 5.606307537247035], ["apartheid's", 5.6056217503923245], ["dubrovnik's", 5.6056217503923245], ["furry", 5.605545803748911], ["ballpoints", 5.605068163413223], ["recklessly", 5.604869280066586], ["boosting", 5.604011497995105], ["swaminarayan", 5.6032915966419345], ["multiverses", 5.602657381407453], ["ancestrydna", 5.602657381407453], ["procrastinating", 5.601219573696228], ["necessitating", 5.601219573696228], ["avocados", 5.600540992973543], ["maddy's", 5.600370582476267], ["extolling", 5.600244889627212], ["festa", 5.599157622586911], ["hephaestion", 5.598552870505324], ["amunet", 5.598552870505324], ["worthwhile", 5.597986074863475], ["stews", 5.595137035929055], ["intellectually", 5.595110422834324], ["lingue", 5.594810522329852], ["cryptocat", 5.594810522329852], ["geotagging", 5.593925967306559], ["element's", 5.593925967306559], ["rstudio", 5.593925967306559], ["jamir", 5.593724573082506], ["animistic", 5.593585068267355], ["unheard", 5.593251210590072], ["ctas", 5.592583953621333], ["stargazer", 5.59239073175566], ["unsere", 5.591594441866556], ["shads", 5.590305843970489], ["flibbertigibbet", 5.590305843970489], ["outcompete", 5.5896852642937525], ["zoroastrianism", 5.589464310650607], ["pouvez", 5.589464310650607], ["rambla", 5.588409137292862], ["heatmap", 5.588409137292862], ["catapulting", 5.5870921686209725], ["siobhan's", 5.586755799435626], ["orville's", 5.586755799435626], ["unmatched", 5.586323708300811], ["tauntingly", 5.585895504424934], ["vielleicht", 5.585895504424934], ["railay", 5.585895504424934], ["curandera", 5.585895504424934], ["hieroglyphs", 5.58495381719111], ["empire's", 5.584855343464497], ["oversaturation", 5.584521891394266], ["terracing", 5.58264789075957], ["nonchalance", 5.58264789075957], ["ilonggo", 5.58264789075957], ["vadas", 5.58034196432803], ["combinations", 5.5802621437977065], ["vesak", 5.579679975878784], ["hiker's", 5.579530834427488], ["uncomfortably", 5.578926125499064], ["oversimplifies", 5.578289851812841], ["recess", 5.578256427325344], ["nutrients", 5.578053118833019], ["streetwear", 5.577864130784626], ["\u0435\u0441\u0442\u044c", 5.577839408510854], ["fishbowls", 5.577839408510854], ["modulo", 5.576850748609248], ["neem", 5.5757301114649485], ["spaces", 5.575697997515692], ["sara", 5.575574360293695], ["joaquin's", 5.574796065315464], ["vied", 5.573849391325749], ["callously", 5.573658119598926], ["pollock's", 5.572905049763479], ["quickly", 5.572413036293177], ["solidify", 5.571854644524734], ["sought", 5.571125273394248], ["hurdy", 5.570242006557883], ["steller's", 5.569988825247643], ["maxentius", 5.569988825247643], ["engagement", 5.569877127610773], ["coziness", 5.569677409121889], ["keystroke", 5.569015050977496], ["rigor", 5.5688635749799165], ["embers", 5.568754608056757], ["mealtimes", 5.568388253746135], ["consuming", 5.56825962299988], ["preferring", 5.567575321168734], ["freshwaters", 5.566742911013816], ["charadrius", 5.566742911013816], ["rana's", 5.5652564643086055], ["elinor's", 5.5652564643086055], ["nalia", 5.5652564643086055], ["astronomy", 5.565215734972655], ["fanbase", 5.564039064457038], ["sketched", 5.563603770274523], ["lunges", 5.563310181572287], ["haltingly", 5.562335989673683], ["millennia", 5.56183802048577], ["mici", 5.561258435239418], ["lululemon's", 5.561258435239418], ["lira's", 5.561258435239418], ["organized", 5.559980349255251], ["tanika", 5.55680229867272], ["scrawled", 5.554537881239129], ["weaknesses", 5.553943309246948], ["wirings", 5.553110071231742], ["trails", 5.552975162921397], ["marathons", 5.551381028137429], ["snowflake's", 5.551271140833122], ["ahmad's", 5.550000827070917], ["arancini", 5.549161570934962], ["garcias", 5.5488136611186025], ["sunroom", 5.5488136611186025], ["othello's", 5.5488136611186025], ["reaching", 5.547540961508266], ["barbra's", 5.547346594250702], ["borte", 5.547346594250702], ["harina", 5.547041878700345], ["reached", 5.546288837484848], ["itza", 5.545054302269607], ["grasp", 5.545018038357879], ["eureka", 5.5446585802855015], ["spoonful", 5.543303993493758], ["neruda's", 5.543054643307374], ["quadratic", 5.543027004042138], ["sommelier", 5.541568803999015], ["byte", 5.541294943420609], ["disorientating", 5.540706289365962], ["parvathi", 5.5404919735693685], ["pantheon's", 5.5383411614678275], ["pinguicula", 5.5383411614678275], ["taros", 5.5383411614678275], ["oceanus", 5.537089558945461], ["tidying", 5.537075254916635], ["guanyin", 5.536510304885524], ["zhang's", 5.535959079247841], ["glaucon", 5.53595907924784], ["omnicorp", 5.53595907924784], ["kielce", 5.535806451554112], ["distractible", 5.5349329515223085], ["uproariously", 5.534669333073484], ["guillaume's", 5.533997364478441], ["combating", 5.532466786123529], ["decadently", 5.532353765617592], ["pushbacks", 5.532353765617592], ["intangible", 5.531081263656168], ["grunty", 5.53033392123438], ["applying", 5.530101128365064], ["regurgitating", 5.5290422182831405], ["dumplings", 5.5280620002721435], ["gridiron", 5.526997863159001], ["romanticize", 5.526821411851975], ["samgyeopsal", 5.526257508024625], ["dustings", 5.526257508024625], ["staffordshire's", 5.526257508024625], ["zdenka", 5.525765015598992], ["beddings", 5.525765015598992], ["aspens", 5.52488770115657], ["commercializing", 5.523572251703913], ["accomplishments", 5.523046667726549], ["comt\u00e9", 5.522977530505366], ["debated", 5.522800409183588], ["ethnomusicology", 5.522794940663707], ["odessey", 5.522619418815919], ["whisper's", 5.522619418815919], ["conscious", 5.5219330005274685], ["minelayer", 5.521300111740234], ["godwinson", 5.520943231117667], ["lugo's", 5.52061846975113], ["backpacked", 5.520182665856457], ["kondo's", 5.519352563199938], ["backstage", 5.518436530318673], ["garnishes", 5.517664687798347], ["saxophones", 5.516969680280046], ["watchmaking", 5.515589057236932], ["sancocho", 5.513726311861304], ["bridgerton", 5.513726311861303], ["objetivo", 5.513726311861303], ["timestamped", 5.513726311861303], ["tomatoey", 5.513726311861303], ["emilien", 5.513726311861303], ["saltwater", 5.512723816168238], ["cornerstones", 5.5123880287759], ["plunge", 5.510380604147795], ["whisperer", 5.509672101337876], ["keti", 5.509053662444472], ["outgoing", 5.507383400487119], ["limericks", 5.50728504280539], ["velma's", 5.505982314232285], ["slackline", 5.505781461267843], ["dabbling", 5.5047214824575095], ["esperanza's", 5.5045976259145135], ["willem's", 5.5045976259145135], ["micromanaged", 5.503809537919107], ["lenape", 5.503701354930646], ["toshiba's", 5.502334315349193], ["hawkin's", 5.5017399503137785], ["closer", 5.501582252652078], ["swordplay", 5.5013679597829945], ["ordinary", 5.50127849818196], ["archaeologist", 5.5008818357938996], ["vividly", 5.500850852067122], ["veins", 5.500445423266646], ["agnes's", 5.500343481007271], ["protectiveness", 5.499837832486338], ["competitors", 5.499268808566829], ["joris", 5.499062146138268], ["thrifting", 5.498608030038458], ["rosalind's", 5.4962778108744015], ["siddhartha", 5.496277810874401], ["conquistadors", 5.495806701347754], ["unconventionally", 5.495589054256497], ["grasping", 5.49552791743311], ["reconvening", 5.4952238745731785], ["bumbo", 5.4952238745731785], ["niam", 5.4952238745731785], ["precision", 5.495076943453537], ["seesaws", 5.494843687505615], ["agonizingly", 5.494034432176085], ["equate", 5.493922263241169], ["herbs", 5.493281118657925], ["vitals", 5.492728091263307], ["hardship", 5.490714478906841], ["uffizi", 5.490441994665943], ["userdata", 5.489960250172246], ["clamp's", 5.489960250172246], ["mumbai's", 5.4896037592469105], ["emilia's", 5.489543301721561], ["apprehensively", 5.489543301721561], ["mastercard's", 5.488104907066409], ["ugali", 5.488104907066409], ["disastrously", 5.48802674006873], ["kaffee", 5.487218012285047], ["ramakant", 5.4862675154874925], ["bittersweet", 5.485682359068216], ["halen's", 5.485378104602376], ["incident", 5.485188408529594], ["ollie's", 5.483760407992492], ["espanyol's", 5.4837604079924915], ["jemand", 5.4837604079924915], ["osijek", 5.483563695275629], ["etymologist", 5.482957749853149], ["matcher", 5.4822193043649525], ["fridge's", 5.481669763536528], ["widgets", 5.481395428574637], ["respect", 5.4794587604838485], ["asita", 5.47926552241217], ["flickr's", 5.47926552241217], ["sigurd's", 5.47926552241217], ["birdemic", 5.47926552241217], ["catalana", 5.47902894948689], ["irreversible", 5.478724253467943], ["geothermal", 5.477485428652546], ["rooftops", 5.475849409371126], ["stashes", 5.475700613158812], ["pomona's", 5.4757006131588115], ["crete's", 5.4757006131588115], ["fadh2", 5.4757006131588115], ["dimitra", 5.475436545806711], ["namesakes", 5.474661449327874], ["rigid", 5.474261962947669], ["curbside", 5.474036064091096], ["consumed", 5.473208777599899], ["alrededor", 5.473184206627029], ["racewalking", 5.473184206627029], ["chenonceau", 5.473184206627029], ["strum", 5.471734356506069], ["binomials", 5.471590848779394], ["anxious", 5.4706923504028735], ["spanned", 5.47012940613961], ["timbres", 5.469413930480099], ["mealybugs", 5.469260777088228], ["minds", 5.468399375611401], ["aelius", 5.468158325812863], ["tools", 5.465956039104104], ["weathering", 5.464971497206581], ["professionals", 5.46365121988246], ["innermost", 5.463360542666417], ["dreamed", 5.46101724671811], ["motionless", 5.46098246691665], ["caramelized", 5.460944596375922], ["ryo's", 5.460709712708791], ["yuccas", 5.460194988445175], ["amateur", 5.4591869226801215], ["rodin's", 5.458589048742691], ["parmigiana", 5.458589048742691], ["prospera", 5.458589048742691], ["unannounced", 5.45858904874269], ["transitioned", 5.45858904874269], ["torrenting", 5.457396175261759], ["piles", 5.457313379416038], ["naidu's", 5.4559909578104], ["pune's", 5.4559909578104], ["bookshelf", 5.455905083410594], ["farnham's", 5.455599679055537], ["bucovina", 5.455599679055537], ["eorzea", 5.455069648969162], ["limerick's", 5.455069648969162], ["drizzling", 5.4549402592716065], ["elamites", 5.45472729490559], ["betrayal", 5.453150143351555], ["kapoors", 5.45313591282986], ["banaue", 5.45313591282986], ["adus", 5.45313591282986], ["langston's", 5.45313591282986], ["exoskeletons", 5.452818870044231], ["immersion", 5.452280972029968], ["s\u00e1mi", 5.452005494937483], ["fort\u00e9", 5.451599818206528], ["patron's", 5.45155478450462], ["isolde", 5.451194813125791], ["shortcomings", 5.450670399252251], ["beaming", 5.45047304973224], ["craft", 5.450463194590585], ["spoilage", 5.449727702884342], ["resounding", 5.449313621302177], ["anniversary's", 5.449313621302176], ["coastline", 5.4490166072290425], ["inferno's", 5.448530223714295], ["supermans", 5.448530223714295], ["microcarpa", 5.448530223714295], ["registan", 5.448530223714295], ["quizzical", 5.447086904449735], ["tita's", 5.446485747082507], ["clues", 5.446201932898342], ["musician", 5.4458182855938935], ["newcomers", 5.4450553568863045], ["hypoallergenic", 5.443000503148946], ["icebreaker", 5.442823628577156], ["baingan", 5.442581456224384], ["entomologists", 5.442581456224383], ["panoramic", 5.442507346999761], ["package's", 5.441986378435598], ["emotion", 5.441621000673284], ["giuseppe", 5.441508483537468], ["calamities", 5.4414953460848325], ["attires", 5.4411772814420765], ["eavesdropped", 5.441177281442076], ["demographics", 5.441159294079162], ["evelyn's", 5.439503073047786], ["seeing", 5.437228371762693], ["spacious", 5.43590124494972], ["overthinker", 5.434958793120428], ["surds", 5.434958793120428], ["stylesheets", 5.434277805926703], ["brunel's", 5.434201413132534], ["gilan", 5.433982336276532], ["luxurious", 5.4337772803393145], ["tara's", 5.433668932005675], ["tilled", 5.433179406401884], ["latticework", 5.4319549470667505], ["parco", 5.4319549470667505], ["hearted", 5.431628892922384], ["abacus", 5.431390465929146], ["vintages", 5.429343238077114], ["prevails", 5.4293394030639135], ["paradiso", 5.429151357416651], ["chats", 5.428899753217284], ["aslak", 5.427574338238471], ["inequities", 5.427211168326243], ["d\u00fcrer", 5.42612505270089], ["majorelle", 5.424945972941503], ["literature's", 5.424435197499178], ["isabella", 5.424061345139288], ["reframing", 5.423598093302032], ["willpower", 5.423143665309296], ["complementing", 5.422931417504306], ["prominently", 5.4228678257495035], ["irrefutable", 5.422499203792325], ["jungle's", 5.422439452393402], ["semira", 5.422439452393402], ["ustadh", 5.422439452393402], ["teahouses", 5.422251447419669], ["crawly", 5.422018990911717], ["shivered", 5.422013931972773], ["opeth's", 5.421830873330281], ["blackwood's", 5.421715136002158], ["calcio", 5.421371396137626], ["disp", 5.420832009867989], ["fluidity", 5.420152682072474], ["burdensome", 5.419689242347567], ["gourmet", 5.418500165296188], ["gensokyo", 5.418080039104526], ["banters", 5.417759556433339], ["tableaus", 5.417759556433339], ["\u00e9amon", 5.417759556433339], ["perplexed", 5.417605324147829], ["airier", 5.4174166819597955], ["guten", 5.417236101403731], ["clockmaker", 5.417083507502111], ["channelized", 5.4169942713023325], ["environmental", 5.4169884789645755], ["hatshepsut", 5.4162614528132496], ["intuition", 5.416044373814712], ["instinctual", 5.415902135360539], ["edifice", 5.415729707317352], ["trinket", 5.415502744329642], ["heerenveen", 5.415438147171602], ["bowerbirds", 5.415266913435208], ["philon", 5.415266913435208], ["tower's", 5.414651542195046], ["currywurst", 5.414558572439337], ["churros", 5.41405510545458], ["haggling", 5.413734822109586], ["chaudhary's", 5.413476742554734], ["frostbitten", 5.412663419726826], ["misclassified", 5.412440868704264], ["jeho", 5.412371048775618], ["mouthguards", 5.411077151800056], ["nervousness", 5.408965511935939], ["humbler", 5.408965511935939], ["optimally", 5.4088009230908085], ["weightlifting", 5.40874397828948], ["ingeniously", 5.407952786694985], ["parvathy", 5.407693113556278], ["lennie's", 5.407693113556278], ["karak\u00f6y", 5.407693113556278], ["martyna", 5.407693113556278], ["anthemius", 5.407693113556278], ["testaccio", 5.407693113556278], ["saxophonist", 5.406709896626541], ["houdini's", 5.406684873952743], ["usability", 5.406289732990476], ["perilous", 5.406007817689179], ["lena", 5.405930979109627], ["displayed", 5.403460914310024], ["nutrient", 5.403068887963531], ["doshas", 5.402562474928616], ["gastronome", 5.402562474928616], ["java's", 5.402337901520672], ["tetradrachm", 5.401605083031589], ["dicots", 5.401605083031589], ["tocqueville's", 5.401605083031589], ["alphabets", 5.4010629346458705], ["picasso's", 5.399943050698347], ["adaptation", 5.398870527732604], ["metalworks", 5.398857013697526], ["schoolteacher", 5.398832957300005], ["embarrassingly", 5.398349950581693], ["rivalries", 5.398236098572317], ["disconnections", 5.398197603260305], ["commercialization", 5.39753391183386], ["evening's", 5.396910076440514], ["hydrated", 5.396559627734251], ["nanorobots", 5.39655962773425], ["hans's", 5.39655962773425], ["robust", 5.396061113443713], ["overworking", 5.39605532349841], ["undp's", 5.395867849804789], ["shipwrecks", 5.395624510834626], ["candlelight", 5.394127101420194], ["librarian's", 5.393862696386058], ["koper", 5.393862696386058], ["qualcosa", 5.393862696386058], ["brandwatch", 5.393862696386058], ["moriori", 5.393862696386058], ["lasagna", 5.393740108597503], ["pretenses", 5.393733347160725], ["deadlifting", 5.392507454502544], ["achiote", 5.391342199798961], ["ahtisaari", 5.391342199798961], ["huddling", 5.390142791078206], ["haring's", 5.390142791078206], ["moros", 5.388414350228092], ["notepads", 5.388045785635053], ["cheekily", 5.387698281875902], ["localhost", 5.387698281875902], ["franti\u0161ek", 5.387415848541772], ["influx", 5.387091979264542], ["renewability", 5.385500118562203], ["vocalist's", 5.385500118562203], ["culturas", 5.385500118562203], ["fester", 5.385232516071841], ["workings", 5.385187567216037], ["burdened", 5.385072697917873], ["krystian", 5.384498351427054], ["heartiness", 5.384498351427054], ["empowers", 5.384237284840318], ["quipped", 5.383156285506297], ["accipiter", 5.381360365632845], ["spread", 5.381253916110806], ["archibald's", 5.381184813980022], ["misunderstood", 5.38086176443236], ["shifts", 5.380140075114369], ["hunahpu", 5.377455509476898], ["delves", 5.375883154064772], ["youthfulness", 5.375883154064772], ["seamstresses", 5.375883154064771], ["majestic's", 5.375883154064771], ["hunslet", 5.3749782755672015], ["andalus", 5.3731625654240505], ["oversaturated", 5.371305337016178], ["kimchi", 5.371102466511133], ["formulas", 5.368364436366779], ["carnitas", 5.368364436366778], ["isosceles", 5.3682252008538525], ["ayumi", 5.368153631197676], ["encantada", 5.3674437770568195], ["quinton's", 5.367084654630949], ["volman", 5.367084654630949], ["phish's", 5.366590356909948], ["splatters", 5.366484756942281], ["bleary", 5.365918588440925], ["outdoorsy", 5.3652798342342685], ["aici", 5.364706681810999], ["interdimensional", 5.364706681810998], ["oversleeping", 5.364110072971829], ["zookeeper", 5.363853205747982], ["tech", 5.36369669366529], ["grumpy's", 5.363242515140199], ["chordates", 5.362774150585562], ["motley", 5.362771243932905], ["handicrafts", 5.362543493632849], ["minari", 5.362543493632849], ["quietening", 5.362543493632849], ["chills", 5.362360897520473], ["groggy", 5.361658553014515], ["trabajar", 5.360567247642934], ["basilica's", 5.360567247642934], ["citibank's", 5.360567247642934], ["friedan's", 5.360567247642934], ["gloomily", 5.360345919515451], ["hendrix's", 5.357504066358566], ["benefitting", 5.357366908987625], ["ikigai", 5.357086359819789], ["harrogate's", 5.357086359819789], ["value1", 5.357086359819789], ["bezukhov", 5.357086359819789], ["sturgeons", 5.356455594912239], ["ensure", 5.355231442639207], ["vaquita", 5.354118444939003], ["crevice", 5.352082896603499], ["dish", 5.351607442386159], ["swiftie", 5.351557890924206], ["toekomst", 5.351557890924206], ["proxi", 5.351557890924206], ["menus", 5.349417267767837], ["puppetry", 5.349153112058599], ["acquaintances", 5.3488636984329405], ["brainwave", 5.3469360909275], ["allfather", 5.346761360164095], ["quivered", 5.34595667901684], ["maka", 5.345681563311598], ["nourish", 5.34515489344766], ["overflowed", 5.344324325832408], ["trigonometry", 5.344239159579306], ["lavenders", 5.342947178308077], ["salinization", 5.342947178308077], ["picnics", 5.342418341189312], ["pollinating", 5.342246785415856], ["uncertain", 5.340752195731776], ["cozier", 5.340437770631377], ["ryazan", 5.340402814631936], ["octavian's", 5.34028127887229], ["unplanned", 5.339924069422198], ["kingda", 5.3397495107588835], ["shinier", 5.338532104855388], ["persia's", 5.337990015068911], ["brushed", 5.336531155790599], ["tradicional", 5.3358641727690035], ["pregunta", 5.3358641727690035], ["bigdata", 5.3358641727690035], ["aidar", 5.3358641727690035], ["expectant", 5.335504855316292], ["v\u00e9ritable", 5.3333707782957465], ["motionlessly", 5.3333707782957465], ["keyspace", 5.3333707782957465], ["gogreen", 5.3333707782957465], ["kanoa", 5.3333707782957465], ["shogi", 5.332353735813235], ["babur", 5.332353735813235], ["smithson", 5.332274591416413], ["tardiness", 5.331301909721654], ["profitability", 5.330604294003758], ["suggestions", 5.330466210176983], ["nyame", 5.330284847113067], ["trudging", 5.330284847113066], ["obsessed", 5.330084764886873], ["impact", 5.330027109335295], ["deities", 5.329421901466293], ["penchant", 5.32922854681056], ["rendang", 5.328563861089841], ["parkour", 5.328297104452678], ["outsmart", 5.32816821482751], ["penida", 5.327232627783642], ["softmax", 5.326086411195867], ["jamshid", 5.325919438770913], ["navratri", 5.325527166530245], ["tuning", 5.325252301874121], ["aida's", 5.324900068304409], ["looked", 5.324038098459113], ["paco's", 5.3230921574618435], ["insincerity", 5.322866554912258], ["ladling", 5.322277481588341], ["mazandaran", 5.322161802536818], ["walkthroughs", 5.321776493995103], ["stung", 5.31959118386514], ["dilapidated", 5.319022851473701], ["eggshells", 5.3187408995600265], ["mustn't", 5.317052303568737], ["bereit", 5.316807515009114], ["montevideo's", 5.316807515009114], ["sooyoung", 5.316807515009114], ["lophophora", 5.316807515009114], ["ramanna", 5.316807515009114], ["chashu", 5.316807515009113], ["walsall's", 5.316807515009113], ["exercises", 5.31479357276858], ["treats", 5.314788669257037], ["persuasion", 5.314240209199687], ["eyes", 5.314190682997611], ["opposites", 5.31341380808464], ["lemonade", 5.313327422817472], ["hootsuite", 5.3122632350817565], ["prototypes", 5.3122274935542375], ["ubud", 5.309991095118078], ["conversational", 5.309526099032487], ["banja", 5.309297899874921], ["typing", 5.309252321285956], ["protists", 5.308381512608149], ["sorting", 5.308316299076053], ["signaling", 5.308164233590232], ["fingerings", 5.307672106907725], ["regenerative", 5.30756089324388], ["bazaars", 5.306699515931149], ["emulating", 5.306179644773818], ["ostro", 5.306131194697851], ["fondest", 5.305571226312649], ["globetrotter", 5.305280561047034], ["mistakes", 5.305275196108355], ["papadakis", 5.303744105881328], ["sanitizing", 5.302515021689198], ["lyres", 5.302515021689197], ["foodies", 5.301956097369424], ["sagra", 5.3016599152512525], ["catania's", 5.3016599152512525], ["chemosynthesis", 5.3016599152512525], ["banoo", 5.3016599152512525], ["trafford's", 5.3016599152512525], ["nieuport", 5.300948283047863], ["architeuthis", 5.300695977084844], ["itsukushima", 5.300695977084844], ["gruy\u00e8re", 5.300346810628281], ["tessa's", 5.300346810628281], ["azlan", 5.299120197926582], ["dilute", 5.298380610376522], ["corydoras", 5.296894378248781], ["raghav", 5.2966966591603795], ["integrations", 5.296184746466048], ["listy", 5.295557285132978], ["architectural", 5.294129465411103], ["graceful", 5.29351483446717], ["languages", 5.2933502184821855], ["hoped", 5.293155608629082], ["educate", 5.292630525708091], ["tanaka", 5.292510276365255], ["transferrable", 5.291816645373348], ["intech", 5.291270495244421], ["whirled", 5.2911843462620824], ["satisfying", 5.290504624717471], ["ikea's", 5.289417900537855], ["kinships", 5.289330473587878], ["spiralized", 5.289330473587878], ["gew\u00fcrztraminer", 5.288564181596051], ["functioned", 5.2878755994716], ["siblings", 5.287565073676564], ["saxophonists", 5.2874166498072945], ["frites", 5.285985442458337], ["coulomb's", 5.285985442458336], ["cyberghost", 5.285985442458336], ["belittling", 5.285455333102426], ["eulex", 5.284681789842896], ["emilio's", 5.283987715533749], ["genoa's", 5.283114184749233], ["hoku", 5.283114184749233], ["monumental", 5.28270815879471], ["trekking", 5.282543644356834], ["alder's", 5.282505531041314], ["bellingham's", 5.282505531041314], ["backbends", 5.282505531041314], ["differently", 5.282425493078722], ["quests", 5.282389533994079], ["nature", 5.282022788984217], ["saguaro", 5.2818991828398625], ["joyfully", 5.281569414519775], ["instances", 5.280591579761951], ["subtotals", 5.2790996602927365], ["underinflated", 5.2790996602927365], ["rehabilitator", 5.2790996602927365], ["mutexes", 5.2790996602927365], ["yvelines", 5.278139823990866], ["focused", 5.277857241312284], ["joe's", 5.277237577867107], ["martinez", 5.276551686681188], ["sentier", 5.27606569497073], ["presentations", 5.276006133833327], ["bah\u00e1'\u00ed", 5.275698178552859], ["uniformity", 5.275399356792533], ["heptagon", 5.275399356792533], ["leonardo's", 5.274554740055399], ["parthenon", 5.274536762336905], ["filename", 5.274536762336905], ["behaviors", 5.274419221418664], ["accessible", 5.274390423708358], ["diver", 5.274190303021103], ["industrious", 5.274008794331839], ["hardiest", 5.27375644201309], ["dostoevsky's", 5.273262817916493], ["colette's", 5.273262817916493], ["quilting", 5.273148274454412], ["tapping", 5.27304017198701], ["judgment", 5.272955451630523], ["dem\u00e1s", 5.272500785717371], ["rifugio", 5.272500785717371], ["godfred", 5.272500785717371], ["eamon", 5.271791066941531], ["wield", 5.270969195075188], ["marsupials", 5.26960221464221], ["eyesores", 5.269547003764589], ["arashiyama", 5.269165870609391], ["revamping", 5.26895219183817], ["harm's", 5.26873381683022], ["avenues", 5.2677896904509085], ["excel", 5.267071767491307], ["animalism", 5.266171252961408], ["occasional", 5.265545971846729], ["flashed", 5.264747108091316], ["hued", 5.264708144887166], ["quechua", 5.263885588355088], ["promised", 5.26366811735371], ["truffula", 5.26310238859488], ["bigtable", 5.26310238859488], ["amusement", 5.263027201417901], ["grammaticality", 5.260723047551212], ["accessability", 5.260723047551212], ["j\u00fcri", 5.260723047551212], ["tool's", 5.259711214375425], ["seanad", 5.259711214375425], ["incas", 5.25944196024191], ["nitpicking", 5.259016128976406], ["namespace", 5.2587774624854156], ["skaftafell", 5.258123105351177], ["entomologist", 5.257386404380032], ["mike's", 5.254709401397122], ["sari", 5.25362226628002], ["edison's", 5.253355902690076], ["masala", 5.253316428704451], ["logistical", 5.2532502757492345], ["sacrifices", 5.253242127385228], ["pronunciations", 5.252871222107105], ["radiohead's", 5.252871222107105], ["hemline", 5.249230215720356], ["swam", 5.2489919183945375], ["acquaintance's", 5.248643316098741], ["nachtmusik", 5.248643316098741], ["krai", 5.2486433160987405], ["cubicles", 5.2483683020397365], ["harshness", 5.248216336776706], ["flutist", 5.2478859361108485], ["prot\u00e9g\u00e9", 5.2471496310443575], ["reciting", 5.246185273774397], ["eventful", 5.245916748142327], ["earth's", 5.245890531142745], ["hamlet's", 5.2456979494791565], ["cyberthreats", 5.2456979494791565], ["fluttering", 5.245517670130282], ["tumelo", 5.244764052746118], ["photowalk", 5.244764052746118], ["skyrocket", 5.243848622042484], ["gliding", 5.243733518003313], ["couchsurfing", 5.242734345295365], ["myself", 5.242343540738857], ["narwhals", 5.240272274127292], ["friendsgiving", 5.23913834812319], ["saltfish", 5.23913834812319], ["undergrowth", 5.238039996268238], ["mehndi", 5.238039996268238], ["gopal's", 5.238039996268238], ["zeppole", 5.238039996268238], ["apologizing", 5.236979664285188], ["darkest", 5.236589015659853], ["ascended", 5.236523721532477], ["mancino", 5.235943522385401], ["reya", 5.235816719529584], ["listening", 5.235391718532016], ["inspirations", 5.235283133112309], ["costlier", 5.235153228565693], ["stoyan", 5.235107163123631], ["securely", 5.234503705832621], ["armchair", 5.23421101966278], ["enabling", 5.234179725142446], ["swatching", 5.233160592452432], ["materialized", 5.232658210510439], ["lorenzo's", 5.232046815494477], ["pippin", 5.231867914575856], ["pointed", 5.231664750220149], ["iiif", 5.231396952519008], ["b\u00fcy\u00fck", 5.231396952519008], ["gruffness", 5.231396952519008], ["geekier", 5.231396952519008], ["skybound", 5.230244239272034], ["kitesurfing", 5.229931575221383], ["lockpicks", 5.229931575221383], ["cnot", 5.229931575221383], ["salesperson's", 5.229931575221383], ["grumbly", 5.229931575221383], ["variolation", 5.229931575221383], ["cohosh", 5.22892639905855], ["diagrams", 5.22792450978355], ["evening", 5.227692239878402], ["ganesha", 5.226529085387526], ["highbrow", 5.226509389912635], ["trevi", 5.226156965423709], ["midpoints", 5.224833466179948], ["depicted", 5.2245833212711235], ["kaleidoscope", 5.224096324236798], ["nguyen's", 5.223530190184392], ["gaulle's", 5.223530190184392], ["nurtures", 5.22287724891062], ["cowrie", 5.221259090101704], ["anemometers", 5.220809601543672], ["stickball", 5.21976683990328], ["striking", 5.219579482224095], ["djoser", 5.219027146916992], ["chawan", 5.219027146916992], ["scarcity", 5.218954020573334], ["coin's", 5.218926608935699], ["kt\u00f3re", 5.218348116583019], ["caught", 5.2183305079752405], ["turntable", 5.218089012902951], ["margaret's", 5.2176873152513], ["\u00e5land", 5.2176873152513], ["tallying", 5.217538007608627], ["evoked", 5.217050242694675], ["defensively", 5.216966592039658], ["caches", 5.216546411096357], ["subproblems", 5.215534990913702], ["bizarre", 5.215433718389607], ["esperantists", 5.214067273173189], ["wolverhampton's", 5.214067273173189], ["riley", 5.213283781782363], ["expos\u00e9", 5.213003177811317], ["stratovolcano", 5.212194404181388], ["alarmed", 5.212114528179354], ["probing", 5.212002955351116], ["analytical", 5.211395452917624], ["interlingua", 5.211393735281549], ["riverbed", 5.2104713647089325], ["legalities", 5.210471364708932], ["roiling", 5.210471364708932], ["jalape\u00f1os", 5.210471364708932], ["nebulas", 5.210471364708932], ["orlan", 5.209721764455053], ["conquer", 5.2079438501007775], ["parlamento", 5.2074081834245645], ["prompted", 5.207365807398891], ["forgotten", 5.207357537200142], ["cosmos", 5.2070252857640185], ["waterholes", 5.206788106241495], ["shoutouts", 5.206230036776731], ["farmer's", 5.206193473604901], ["sunrises", 5.206137431363365], ["thusly", 5.206137431363365], ["sitten", 5.206040691779758], ["atli", 5.206040691779758], ["fabric", 5.205405628659909], ["akhil", 5.205077502012546], ["frontend", 5.204720292562454], ["hackathon", 5.20348213417277], ["garibaldi's", 5.202467568449778], ["revathy", 5.202467568449778], ["micronations", 5.202467568449778], ["tracht", 5.202467568449778], ["raincloud", 5.202467568449778], ["vegan", 5.2022564694463105], ["while", 5.202198490547916], ["farming", 5.201971036644812], ["tarkan", 5.200720014681404], ["quartet", 5.19974581640776], ["tzu's", 5.19934718004465], ["dreams", 5.199245474634397], ["ravelry", 5.199036144544461], ["kayaker", 5.1986562368978015], ["cutworms", 5.198656236897801], ["neden", 5.198656236897801], ["\u0433\u043e\u0440\u043e\u0434\u0430", 5.198656236897801], ["strums", 5.1986562368978], ["\u00f6rebro", 5.19835250610574], ["waterbirds", 5.1979961738915925], ["diegetic", 5.1979961738915925], ["hush", 5.196593595940941], ["resources", 5.195822350153433], ["guaid\u00f3", 5.195626716946228], ["garnie", 5.195626716946228], ["arboreal", 5.194609960250153], ["impacted", 5.194530843108997], ["ideas", 5.1944799648420155], ["sosyal", 5.1931608286135535], ["esperan\u00e7a", 5.1931608286135535], ["xbalanque", 5.1931608286135535], ["rona's", 5.1931608286135535], ["connect's", 5.1931608286135535], ["amman's", 5.1931608286135535], ["bureaucratic", 5.192183806323973], ["livia's", 5.191114665954524], ["bubbling", 5.190767901565074], ["traceability", 5.190325057030977], ["uproarious", 5.188895913923387], ["abigail's", 5.188810297055191], ["gertie", 5.1884422245377095], ["rodya", 5.187915211614954], ["elegant", 5.187636740589102], ["adeline", 5.187590194999767], ["littering", 5.186605982165054], ["tanisha", 5.185739376911355], ["throes", 5.185538955616399], ["zipping", 5.182506062191938], ["mughlai", 5.18220479311015], ["slurping", 5.182041213413397], ["agonizing", 5.182027986395953], ["illuminate", 5.18193783104505], ["muse's", 5.181404946715164], ["cicero's", 5.181404946715164], ["hovered", 5.1813750899060045], ["montag's", 5.181281048940255], ["comprehend", 5.181251916888589], ["purred", 5.1804791171883675], ["pointing", 5.180163318494681], ["alvah", 5.17990989632064], ["distinctions", 5.178848522626198], ["crowds", 5.178159142699137], ["phytates", 5.178108188530615], ["asked", 5.177874903968997], ["prospect", 5.177664693153357], ["bernoulli's", 5.175964931830065], ["remarkables", 5.175964931830065], ["resumed", 5.175303379928494], ["detangling", 5.175025981275538], ["joyful", 5.174884764608472], ["marlowe's", 5.174830150427839], ["grandson's", 5.174352967666042], ["crops", 5.173475245750327], ["materialistic", 5.173139430673146], ["greatness", 5.172987698992331], ["asimov's", 5.1728106448109505], ["touristy", 5.172453332477953], ["microstates", 5.17203059676004], ["spirituality", 5.172030596760039], ["bouts", 5.171415803333248], ["sudden", 5.1713261045984265], ["paywalls", 5.170756806725715], ["debrett's", 5.170756806725715], ["whisking", 5.169922949419369], ["minh's", 5.1691184173699725], ["phrase's", 5.169118417369972], ["kandu", 5.169118417369972], ["rigorousness", 5.169118417369972], ["whinnied", 5.169118417369972], ["yucatecan", 5.169118417369972], ["kathmandu's", 5.169118417369972], ["value2", 5.169118417369972], ["dynamics", 5.167154152371371], ["nobleman's", 5.166752688117171], ["nestle's", 5.166752688117171], ["crowdfunding", 5.165952641497133], ["pulsate", 5.165917724851477], ["copywriter", 5.165289440764513], ["gadget", 5.165159092624752], ["frailer", 5.165095757122992], ["vendors", 5.164976101762164], ["cualquier", 5.164898728865996], ["clamor", 5.164655869095984], ["tilly's", 5.163789429310828], ["bede's", 5.162180003386925], ["avoiding", 5.16212022628184], ["measly", 5.161410817784915], ["rieslings", 5.160574419985889], ["untainted", 5.160057014566615], ["accurately", 5.159679617721468], ["nikolai's", 5.159456513786103], ["sagarmatha", 5.159456513786103], ["escalations", 5.159456513786102], ["gutenberg", 5.159190256735358], ["phonetics", 5.15911367204603], ["camu", 5.158236062807088], ["jax's", 5.157694951254237], ["midgar", 5.157502420926444], ["throng", 5.156994336137472], ["shimokitazawa", 5.156662710340165], ["deckbuilding", 5.156662710340165], ["mis\u00e9rables", 5.156662710340164], ["disproving", 5.156662710340164], ["netanya", 5.156532737745072], ["blossoms", 5.155522927833875], ["footsteps", 5.155277839076932], ["multiplied", 5.155261932397756], ["jjigae", 5.154956449103205], ["gershwin's", 5.154010346839037], ["leora", 5.151791763457], ["elgar's", 5.151096134456796], ["translate", 5.15099309497581], ["dieses", 5.150981159765165], ["iftar", 5.150797491333724], ["lexia", 5.150657280165079], ["spurring", 5.149753744144779], ["encryption", 5.1486128236481745], ["gritting", 5.14772896184982], ["passed", 5.147711308693421], ["store's", 5.147536912866474], ["minun", 5.146545847861806], ["unplugging", 5.1463644032839415], ["brampton's", 5.146144557737216], ["dodecagon", 5.146144557737216], ["enigmatica", 5.146144557737216], ["parabolica", 5.146144557737216], ["appreciate", 5.146096103778718], ["untangle", 5.144688001795082], ["weddingplanning", 5.1439032055779235], ["connaissances", 5.1439032055779235], ["merhaba", 5.1439032055779235], ["clearwater's", 5.1439032055779235], ["trigonia", 5.1439032055779235], ["anthos", 5.1439032055779235], ["psyches", 5.14310166242162], ["placekicker", 5.142945665889618], ["distressful", 5.142802905426997], ["bornean", 5.142610117793716], ["whirs", 5.142610117793716], ["despairingly", 5.1424859571399635], ["risking", 5.142326366981351], ["adama's", 5.141602197657736], ["amateurish", 5.140947386406413], ["citymapper", 5.14028668117747], ["vestfold", 5.14028668117747], ["hacia", 5.139580597842143], ["reimu", 5.1390070479484], ["groans", 5.137289768125532], ["kamari", 5.137084444079228], ["lingwood", 5.135769266290166], ["hygrometers", 5.135769266290166], ["battlestation", 5.135769266290166], ["finnian", 5.135769266290166], ["creators", 5.134806993571702], ["reactjs", 5.1345690397731545], ["society's", 5.133916066628895], ["astrobiologists", 5.133469324836386], ["guidelines", 5.132620956544407], ["featuring", 5.132516659725275], ["canal's", 5.132342080803704], ["tutoring", 5.13195311026787], ["grooming", 5.131724794776232], ["gamer", 5.131335508650204], ["chomper", 5.1308286513153805], ["commiserate", 5.13082865131538], ["coasteering", 5.130569059681111], ["delice", 5.130569059681111], ["depleting", 5.129453345432602], ["supplier's", 5.129355968005587], ["flashbulbs", 5.127229289190312], ["grandad's", 5.127155094424533], ["hoarding", 5.126780876046752], ["objectivity", 5.126780876046752], ["chameleons", 5.126705138047963], ["exchanges", 5.12651235041468], ["visuals", 5.125431029455426], ["overlooking", 5.125261846914458], ["tumbo", 5.124169561566755], ["workforce's", 5.124169561566755], ["eszter", 5.123374006596787], ["cryptography", 5.12257727737594], ["uplifts", 5.1221264317575175], ["picnickers", 5.121290275429893], ["bioethicist", 5.120856378092316], ["bellefontaine", 5.1201800181522685], ["faithful's", 5.119888718156925], ["appeasing", 5.119341721499002], ["trailhead", 5.11924308275741], ["rajinikanth's", 5.117427233196273], ["patisserie", 5.117427233196272], ["basketful", 5.116823834159225], ["wenchuan", 5.116823834159225], ["aquarium", 5.115803873693297], ["disrespectfully", 5.115512744893543], ["pencils", 5.115275192059245], ["drumhead", 5.113332294381134], ["decagon", 5.113236056101107], ["beatrix's", 5.113236056101107], ["mustelids", 5.113236056101107], ["wireframing", 5.113236056101107], ["guitaring", 5.113236056101107], ["hierarchies", 5.1129539576427785], ["weezer's", 5.112511356891569], ["subprocess", 5.112314918277995], ["fiercer", 5.111743909281886], ["madhubani", 5.109975186049949], ["consume", 5.1097859277097255], ["finley's", 5.10935982294951], ["spirits", 5.1085719986109295], ["productid", 5.108305259518561], ["chairo", 5.10830525951856], ["watering", 5.107422487872331], ["optimistic", 5.107172706781247], ["wavering", 5.1071225076640765], ["poised", 5.106704675289609], ["gehry's", 5.106672691522549], ["cheeses", 5.10611598583591], ["nike's", 5.105360991357454], ["lille's", 5.10481359495293], ["belonging", 5.1040244475855205], ["olomouc", 5.10351472629613], ["madiba's", 5.103129671616312], ["myvatn", 5.103129671616312], ["workbench", 5.102677306606862], ["gramophones", 5.102420115210359], ["carim", 5.101485092282888], ["littlerock", 5.101485092282888], ["maja's", 5.101485092282888], ["marinara", 5.101416689425657], ["additionally", 5.101095436272531], ["asma", 5.100746014399181], ["diver's", 5.100196838471706], ["recognized", 5.099830367236477], ["prickle", 5.099534130982299], ["beidou", 5.099215748380627], ["tirta", 5.098308576036136], ["ghostly", 5.098099225200889], ["muralists", 5.0976905992390416], ["critiquing", 5.09754600851408], ["chores", 5.09749558926993], ["salvage", 5.097236448505605], ["matsuri", 5.096991183639228], ["filenames", 5.096750759526794], ["entanglement", 5.09638299616113], ["intersectional", 5.096367861866247], ["zones", 5.095716935843318], ["janeth", 5.095273868550401], ["patience's", 5.095273868550401], ["m\u00f6glich", 5.095273868550401], ["montpellier's", 5.095273868550401], ["comparing", 5.095234484791031], ["soaking", 5.095175043131906], ["nosedive", 5.094895178556454], ["sheet1", 5.093806758308953], ["jeers", 5.092941935429782], ["reshooting", 5.092371681042208], ["piedmontese", 5.092320086597619], ["rushden", 5.0918063853884385], ["punctilious", 5.0918063853884385], ["echelons", 5.091117349036006], ["contextual", 5.090907408272375], ["ppsspp", 5.090860847704611], ["outspokenness", 5.089593518641204], ["numerator", 5.089593518641203], ["kurtas", 5.089593518641203], ["frederic's", 5.089593518641203], ["tatemae", 5.089593518641203], ["nutzen", 5.089593518641203], ["glitches", 5.088172222578817], ["fatimids", 5.086778477978238], ["treks", 5.086778477978237], ["witted", 5.085727114725109], ["snowshoeing", 5.08553267274867], ["margherita", 5.085387243005962], ["wariness", 5.084880932049869], ["upenn", 5.084845763493217], ["conall", 5.084378771183579], ["cheesecakes", 5.083386697277007], ["stirring", 5.08330457675426], ["invitations", 5.082068049625313], ["giotto's", 5.081928468161323], ["sugarcoated", 5.081586435016339], ["dissipating", 5.08105195544441], ["maranh\u00e3o", 5.080801821294304], ["banjos", 5.080131062336767], ["ample", 5.0796412798529245], ["unmotivated", 5.0794593722545285], ["consulted", 5.079149124447505], ["lychees", 5.078919035590362], ["relaxation", 5.0789157895487325], ["proposed", 5.0782870312850985], ["misuse", 5.076759529220999], ["mangeshkar", 5.076759529220999], ["expansive", 5.076074285857313], ["niamh's", 5.075134446145063], ["laterano", 5.075134446145063], ["cadaqu\u00e9s", 5.0751344461450625], ["barries", 5.0751344461450625], ["yudhisthira", 5.0751344461450625], ["utila", 5.0751344461450625], ["succumb", 5.074546024760003], ["fortifying", 5.074351246384857], ["modifying", 5.074259819800603], ["guffaws", 5.073897814944736], ["boardroom", 5.073897814944735], ["stalactites", 5.073867246533042], ["calligraphic", 5.073300612560187], ["nanobots", 5.073300612560187], ["mohanlal", 5.0731465360604], ["chirps", 5.071817364807714], ["bodyweight", 5.071131788497906], ["sarsens", 5.070230934602894], ["responsibilities", 5.069519403779116], ["covertly", 5.069333317512725], ["predictability", 5.069240536085196], ["chuffed", 5.068831329512015], ["pacing", 5.068689830975355], ["ponch", 5.06855191119546], ["bohr's", 5.06855191119546], ["confident", 5.068472415054667], ["faint", 5.068307749727731], ["throws", 5.06821722586291], ["drumming", 5.066981619725554], ["spruces", 5.066908944611603], ["purring", 5.066313604153005], ["soren's", 5.0662773179668], ["thali", 5.066117533776622], ["thanissaro", 5.065736049022572], ["asilomar", 5.0644274114485555], ["leftover", 5.064170897629265], ["misfortune", 5.063668401655625], ["ratm", 5.063626204770585], ["coubertin", 5.063085219064947], ["thoughts", 5.062748629695239], ["suffusing", 5.062171139769214], ["persistent", 5.061569505210083], ["ramon's", 5.06029487174113], ["bts's", 5.06029487174113], ["slumbered", 5.059893802162645], ["binta", 5.059893802162645], ["technologies", 5.059820005360204], ["burek", 5.058710062824206], ["dika", 5.058710062824206], ["banksy's", 5.05856852877287], ["eliot's", 5.057783559149696], ["caf\u00e9s", 5.056961899679436], ["complexions", 5.056467284963841], ["creamy", 5.056435479082026], ["ultramarathon", 5.056109350755406], ["charente", 5.055577087971094], ["visayan", 5.055325178205387], ["lutefisk", 5.055325178205387], ["sizeof", 5.055066694282094], ["haruto", 5.055029216307477], ["gerade", 5.054249119206196], ["sarawak's", 5.054249119206196], ["mixologists", 5.054249119206195], ["terrence's", 5.054249119206195], ["fran\u00e7ois", 5.052745996791561], ["juan", 5.051638453338835], ["redefining", 5.0514644089476795], ["rehydrating", 5.0511653796581735], ["innate", 5.051084508077975], ["workday", 5.050661926427717], ["methodologies", 5.050573301664953], ["hesse's", 5.05018294935969], ["multitasking", 5.049821724400353], ["upended", 5.0493108129299955], ["immigrated", 5.0483639236716], ["llamada", 5.047911502755779], ["smoothies", 5.047644134561778], ["unrelatable", 5.047260872930072], ["chasm", 5.046691928537525], ["klimt's", 5.046461370178142], ["amadou", 5.045988855068575], ["imaginable", 5.0450595753530925], ["groundswell", 5.0450595753530925], ["karwan", 5.044150719367621], ["coffee", 5.0439172287415195], ["guilt", 5.042854084828348], ["share", 5.042175338139958], ["cuny's", 5.041660702969068], ["highlighters", 5.041410785882007], ["songbirds", 5.040755179347551], ["primes", 5.040689805144595], ["zinnias", 5.040533382112012], ["kmph", 5.0402964086438935], ["formed", 5.040078228669465], ["cawl", 5.039890456935723], ["foreperson", 5.039890456935723], ["ribbing", 5.039204290774125], ["rachel", 5.0385486857720645], ["saraswati", 5.038308073746732], ["luthiers", 5.038308073746732], ["reprioritize", 5.0381975142226105], ["viewings", 5.0375408576551], ["peculiarity", 5.036946022394183], ["function's", 5.036280737370521], ["lyra", 5.035947908990271], ["computing's", 5.035329470096867], ["plumage", 5.034994334439362], ["veluwe", 5.03427184996032], ["strongheart", 5.03427184996032], ["gloved", 5.0341232587014675], ["anneke", 5.033966594572862], ["vendor", 5.033933822908575], ["meze", 5.0332444475419615], ["insomniacs", 5.0330889853339205], ["haugesund", 5.032245986036796], ["opinions", 5.032010065511888], ["aaliyah", 5.030912514421343], ["distances", 5.030806266580937], ["rivaling", 5.030331128355655], ["coolly", 5.0302195679929405], ["programmatically", 5.030022139957107], ["conciseness", 5.02991976709889], ["booming", 5.029686732476156], ["tumbleweeds", 5.0296724321572], ["calpurnia", 5.028420519619074], ["sowing", 5.028104866944119], ["sysadmins", 5.027647808052478], ["horseflies", 5.027647808052478], ["smile", 5.027051644280772], ["unmik", 5.026919620758307], ["gmos", 5.026892962649272], ["thanh", 5.026591268774529], ["hillsides", 5.026369809118047], ["rivi\u00e8res", 5.026271585227196], ["nurture", 5.026207496419156], ["liechtenstein's", 5.0259735996581885], ["madikeri", 5.0259735996581885], ["subsequences", 5.025168284228024], ["streamed", 5.025059786276432], ["osnabr\u00fcck", 5.024512599431021], ["mpas", 5.024383101683613], ["offputting", 5.023870461840567], ["plagiarizing", 5.023701459694509], ["copious", 5.022413913715091], ["mural's", 5.02142931973083], ["roundtrips", 5.02142931973083], ["aada", 5.02142931973083], ["cadence's", 5.02142931973083], ["topic", 5.021171222278645], ["gondolier", 5.019721931318817], ["wagging", 5.019276766081029], ["identifying", 5.018933788062311], ["array", 5.018672456574899], ["drumsticks", 5.01856157026211], ["tavernas", 5.018561570262109], ["technophobic", 5.018110463604108], ["francey", 5.018110463604108], ["arise", 5.01643808771191], ["artwork", 5.0161125122158206], ["appetizers", 5.015845055342485], ["mugs", 5.0158273194755525], ["fundraisers", 5.015698982742431], ["microscopes", 5.015521755825265], ["prospero's", 5.0151446772781], ["suhoor", 5.0151446772781], ["protecting", 5.014582453164279], ["lovably", 5.014044864848874], ["cacher", 5.0129899427228795], ["reinterpreting", 5.0127841042592705], ["amphibians", 5.012205109729028], ["arpeggios", 5.011977217481926], ["guise", 5.011977217481925], ["frescas", 5.0116485847878405], ["inca's", 5.0116485847878405], ["punc", 5.0116485847878405], ["carps", 5.011416499551905], ["baroque", 5.011027484720074], ["lisinopril", 5.010068619912435], ["enthrall", 5.007465986863129], ["pilsners", 5.007465986863129], ["flexbox", 5.007002107107983], ["luscious", 5.006796459289204], ["simran", 5.006268315017431], ["bumped", 5.006115537567859], ["transpire", 5.005693014130851], ["mochi", 5.004807276802622], ["saraswathi", 5.004407525426658], ["revisited", 5.003474308085263], ["improvisation", 5.002722034029872], ["illustrious", 5.002490364857104], ["careening", 5.0023726619709405], ["gij\u00f3n", 5.0023726619709405], ["php's", 5.0023726619709405], ["paystubs", 5.0023726619709405], ["encroachment", 5.001764638774459], ["enchanted", 5.001601387058695], ["kommission", 5.001325044659533], ["vinification", 5.001325044659533], ["basescu", 5.000821538664903], ["waverly's", 5.000821538664903], ["deserialization", 5.000821538664903], ["chatterboxes", 5.000821538664903], ["temos", 5.000821538664903], ["celeste's", 4.998931226232975], ["dilan", 4.9980540668670805], ["giggled", 4.997525001865526], ["haversham", 4.997299833355984], ["wakame", 4.996814470124306], ["binging", 4.99614402674981], ["unwinding", 4.995079034684422], ["chessboard", 4.9944724747248515], ["disaster", 4.99400134445499], ["jako", 4.993810756305443], ["lemak", 4.993810756305443], ["partaking", 4.993791593856839], ["grayness", 4.99200918075889], ["stifles", 4.992009180758889], ["savouring", 4.9918915002030015], ["zoe's", 4.991791372001213], ["aiming", 4.99176969476165], ["enquired", 4.991109041407563], ["ghanoush", 4.990872954702042], ["quizzer", 4.990872954702042], ["sant'andrea", 4.990872954702042], ["kusatsu", 4.990872954702042], ["devise", 4.990366967582211], ["rodrigo's", 4.990076326935928], ["enjoyable", 4.9898205829667335], ["mutely", 4.989694472304238], ["maintain", 4.989607770800581], ["encasing", 4.989322994157104], ["karen's", 4.988501914355546], ["newsfeeds", 4.987671811699025], ["xylophones", 4.987290131331832], ["petah", 4.987290131331832], ["zurich's", 4.987290131331832], ["spires", 4.9870420074447015], ["alvalade", 4.986678943815738], ["visarjan", 4.986678943815738], ["topkapi", 4.9864818185066575], ["whim", 4.986386021798862], ["harming", 4.986190314075796], ["skies", 4.986158329327859], ["caleb's", 4.985909157007492], ["oxybenzone", 4.985542249276459], ["quindi", 4.985542249276459], ["dialogued", 4.985542249276459], ["concacaf's", 4.985542249276459], ["polyhymnia", 4.985542249276459], ["rackers", 4.985542249276459], ["bremen's", 4.985542249276459], ["littered", 4.984999342313174], ["nescopeck", 4.9845070453210445], ["aeroponic", 4.9845070453210445], ["namche", 4.9845070453210445], ["brunelleschi", 4.9842280356907445], ["scranton's", 4.983560320336178], ["hanover's", 4.983560320336178], ["fluent", 4.983342487783769], ["modernize", 4.982063963986267], ["necessity", 4.981580512279265], ["recalled", 4.981579553574124], ["hetfield's", 4.981150474920154], ["steinem's", 4.981150474920154], ["epiphanies", 4.980732768381377], ["disappearing", 4.980432607401251], ["product's", 4.980203873771127], ["lifeline", 4.977678596453728], ["infusers", 4.977669587097011], ["faren", 4.977669587097011], ["baobab", 4.976948838559512], ["eroding", 4.976371628925667], ["meditated", 4.9763321417474975], ["kert\u00e9sz", 4.975986520457248], ["caecilia", 4.975986520457248], ["thrissur", 4.974561192804756], ["arachnid", 4.974516312245456], ["interest", 4.974407506000867], ["hugo's", 4.974225340198319], ["arranging", 4.973885911059841], ["setlists", 4.9736317572221616], ["advocacy", 4.9725096312181165], ["ryokan", 4.972398931040913], ["crises", 4.972358425999115], ["embla", 4.972008065267927], ["ushering", 4.9714980670646245], ["rectified", 4.971300231782159], ["holika", 4.971152180108994], ["sheela", 4.971100485047691], ["madeline's", 4.970778729369019], ["sanctuaries", 4.97034716276389], ["crumble", 4.969988070953135], ["malfunctioning", 4.969475313397152], ["engaged", 4.968987642921998], ["sbdc", 4.968303745040491], ["forward", 4.967879938183198], ["infestations", 4.9677829516605945], ["introvert", 4.966931497723766], ["subtler", 4.966368529931383], ["sudoku", 4.965192555778534], ["detractors", 4.962353680675173], ["seafloor", 4.962353680675173], ["arav", 4.962353680675173], ["seafarer", 4.962353680675173], ["carcassonne", 4.962353680675173], ["ideate", 4.962353680675173], ["whiterun", 4.962353680675173], ["hulas", 4.962353680675173], ["anastasia's", 4.962353680675173], ["galicia's", 4.962353680675173], ["cu\u00e1l", 4.962353680675173], ["interclass", 4.962353680675173], ["eugenia's", 4.962353680675173], ["cotija", 4.9623536806751725], ["facepalming", 4.9623536806751725], ["taroko", 4.9623536806751725], ["guangdong's", 4.9623536806751725], ["timpanist", 4.9623536806751725], ["convey", 4.9597844147162435], ["metamorphosis", 4.958875395384981], ["punctuation", 4.957900286346362], ["intentions", 4.957449028781483], ["renewables", 4.95707458101488], ["grins", 4.956723173473197], ["chutneys", 4.956522477760161], ["coral's", 4.956119568011008], ["superposition", 4.955975334041915], ["kora", 4.9558584533444465], ["souad", 4.955518482767907], ["regrouped", 4.955364450139011], ["kafka's", 4.954695727464254], ["tupac's", 4.9544894117676055], ["selecting", 4.953327727773016], ["mispronounced", 4.953164136822071], ["bienvenidos", 4.952919548202407], ["carwashes", 4.952919548202407], ["housewarming", 4.952468513980601], ["roots", 4.952270359269235], ["insincere", 4.95149513870433], ["conquering", 4.951407312261919], ["conventions", 4.951038197572473], ["leila", 4.950981620156959], ["family's", 4.950527510688518], ["demosthenes", 4.949014020243251], ["holdouts", 4.948094043661739], ["caramelization", 4.947361675597605], ["akron's", 4.947361675597605], ["demanded", 4.9468463254230635], ["frenetically", 4.946650029786961], ["kubb", 4.946650029786961], ["unverified", 4.9458125017395895], ["stubbled", 4.945588972294513], ["duvalier's", 4.945588972294513], ["confiding", 4.945359318755052], ["mycorrhizae", 4.9446940590357595], ["manon's", 4.944374138353886], ["troms\u00f8", 4.943974592968969], ["professor's", 4.943837435598027], ["salads", 4.943501601741236], ["wildlife", 4.943281083000616], ["macroevolution", 4.943218640775397], ["securing", 4.942964930982758], ["natured", 4.942643397364081], ["informationen", 4.942583347286029], ["appalled", 4.942390188856365], ["neptune's", 4.942344190027289], ["loved", 4.942305771805245], ["perils", 4.942158055230565], ["analytics", 4.942133563809943], ["langley's", 4.9413267582994305], ["congas", 4.9413267582994305], ["atwood's", 4.941147041014168], ["rexes", 4.940396363504044], ["stoic", 4.94035743209062], ["titel", 4.940150755929422], ["asmr", 4.940029467458769], ["geishas", 4.939797527581195], ["spellbook", 4.939797527581195], ["aphrodite's", 4.93911080629262], ["clucks", 4.938553423213901], ["assalamu", 4.938264585137915], ["kaleidoscopes", 4.938264585137915], ["palmeiras", 4.938028417534609], ["watercooler", 4.937665353905644], ["seafaring", 4.937417230018514], ["miyuki", 4.936508088588323], ["insecticidal", 4.936372771352266], ["lesser", 4.936163480693833], ["minuscule", 4.935688013506756], ["rhin", 4.934828739498686], ["imbibing", 4.934785049115867], ["tejanos", 4.934682563496687], ["outlining", 4.934379628063525], ["smiling", 4.931382962668122], ["neighbor's", 4.93096529896006], ["chanced", 4.930775066343603], ["dropdown", 4.930338495638559], ["festivals", 4.930041458665379], ["abbiamo", 4.929849180583414], ["mila's", 4.92970661698652], ["woodstove", 4.92970661698652], ["sane's", 4.929490411266728], ["ramirez", 4.9280159654740725], ["polygram", 4.92768653919341], ["moma's", 4.92768653919341], ["radom", 4.92768653919341], ["chameleon's", 4.927570827773245], ["scathingly", 4.927366463185102], ["genseric", 4.927159682939887], ["zabit", 4.927159682939887], ["nebulae", 4.926220037369286], ["galactic's", 4.9258657859643264], ["refilling", 4.925758152056625], ["photographing", 4.924735838257152], ["genealogists", 4.923382316690813], ["automate", 4.923068380703161], ["sinaloan", 4.922969921304735], ["poble", 4.922969921304735], ["tchaikovsky's", 4.922334699379405], ["sprightly", 4.922205188113077], ["renoir's", 4.921342493231577], ["academia", 4.9212390788540175], ["templo", 4.920914401504608], ["kahoot", 4.920769711284041], ["mimicked", 4.920703230117948], ["artisan's", 4.9202998359236885], ["wonka's", 4.9198565806430405], ["serenades", 4.91957476963487], ["ariston", 4.917309623059362], ["laura's", 4.917241374487217], ["kastor", 4.917241374487217], ["manipulating", 4.91572163367592], ["ainu", 4.915664869163445], ["lucian's", 4.915093169430647], ["sibiu", 4.913523632156845], ["legged", 4.913279771649509], ["clearer", 4.912452917405814], ["scolded", 4.912022790821834], ["nitpicky", 4.911717418627467], ["unseasonably", 4.91145774548876], ["ajani", 4.911326907609105], ["tensed", 4.911253928814674], ["ampat", 4.9100572133144285], ["signifying", 4.907836933379439], ["gayageum", 4.907822321546875], ["biohacking", 4.907390902566429], ["ayam", 4.90721641755656], ["crayons", 4.906405575451875], ["expeditions", 4.906012547182791], ["undercooking", 4.904651893690578], ["installing", 4.9045562864411885], ["misguided", 4.903862059344021], ["shwedagon", 4.903800834885496], ["qubit", 4.903677201342865], ["rostam", 4.902924295277866], ["dreamlike", 4.902385660364899], ["humbly", 4.902000730504799], ["anxiety", 4.901776012073315], ["warbler", 4.901272077644792], ["taganrog", 4.901090054987826], ["xii's", 4.901090054987826], ["diets", 4.900731454990639], ["immunotherapies", 4.900545004937226], ["surya's", 4.900324259666734], ["resultantly", 4.900324259666734], ["mythology", 4.8996145408908935], ["tarangire", 4.898733761692158], ["trey's", 4.898541265719441], ["slicked", 4.898100097160454], ["ladyfingers", 4.898000390868146], ["housecat", 4.89762732831854], ["palakkad", 4.89762732831854], ["webdev", 4.89762732831854], ["dal\u00ed", 4.897344680491655], ["fate's", 4.8970595532978685], ["ch\u00e2teau", 4.895385344903579], ["roommate's", 4.894101243319609], ["palak", 4.893210488983022], ["completionist", 4.893157513813169], ["aelfric", 4.893157513813169], ["outsiders", 4.892976421916422], ["livelier", 4.8927879748713154], ["laden", 4.891076236898202], ["plotline", 4.89086214459765], ["exuberance", 4.889969267180722], ["capoeira", 4.889945796229524], ["fai's", 4.8893778912534795], ["tcja", 4.8893778912534795], ["hindered", 4.8888373298503565], ["codebreaker", 4.88868851132642], ["elderflowers", 4.887166503695246], ["samuelson's", 4.887166503695246], ["pleasures", 4.8865598060135635], ["airsoft", 4.886343886879274], ["subtraction", 4.886056729865285], ["passageways", 4.88581352107607], ["liberating", 4.884269585993961], ["cleanliness", 4.883964851928244], ["pompeii's", 4.882315718083638], ["rowling's", 4.882163554656658], ["tikka", 4.881843517714366], ["improved", 4.881364898552606], ["consultations", 4.881335661398844], ["hyderabadi", 4.881003620336236], ["wonderworld", 4.878718506281771], ["swallowing", 4.878383722104796], ["xiii's", 4.878245991172204], ["kolya's", 4.877527122031153], ["neighed", 4.877527122031153], ["pruners", 4.877527122031153], ["roygbiv", 4.877527122031153], ["frenetic", 4.876885167385883], ["mahal", 4.876768783334414], ["halina", 4.876301304709708], ["sojourns", 4.875870353307637], ["mandap", 4.875464219340395], ["basketballs", 4.874782733369141], ["carpools", 4.874565300849449], ["seismographs", 4.874565300849449], ["clipboard", 4.874425377699293], ["br\u00fccke", 4.873740222091688], ["monza's", 4.873740222091688], ["nibbling", 4.873541981692172], ["sushi", 4.8724822439740105], ["tyrone's", 4.872129068299261], ["empathising", 4.872129068299261], ["mockingbirds", 4.872129068299261], ["turin's", 4.872129068299261], ["kalimba", 4.872129068299261], ["explorers", 4.8720851847329305], ["endeared", 4.871493683705065], ["kuti's", 4.871420498045], ["tostones", 4.871420498045], ["isla's", 4.871420498045], ["loopholes", 4.870992517949564], ["tactile", 4.869801846154644], ["baati", 4.86959940627003], ["jajangmyeon", 4.86959940627003], ["retreat's", 4.86959940627003], ["jeeto", 4.86959940627003], ["churning", 4.869309549162513], ["simplemente", 4.868976057651715], ["clacked", 4.868131775345899], ["feverfew", 4.868131775345899], ["nuwa", 4.868131775345899], ["nong", 4.867871781125895], ["marie's", 4.867861687984184], ["zusammen", 4.867380404490003], ["delicias", 4.867380404490003], ["fermentation", 4.866828372322177], ["salvaging", 4.866476823847139], ["invited", 4.866217226994246], ["n\u00eemes", 4.865465620695947], ["excruciating", 4.865264152140225], ["exuberantly", 4.865052628112915], ["covey's", 4.865052628112915], ["veliky", 4.865052628112915], ["manty", 4.865052628112915], ["atanarjuat", 4.865052628112915], ["erosion", 4.862425277036864], ["onsens", 4.861765430391216], ["seismologist", 4.860959434795992], ["pericles", 4.860892917687391], ["janet's", 4.86038750915445], ["gluten", 4.86031007680997], ["nonconformity", 4.859684294178446], ["quirinus", 4.858971312327774], ["primase", 4.858971312327774], ["headlong", 4.8583805559372175], ["heisenberg's", 4.858373726383569], ["griots", 4.857967600013722], ["oshea", 4.857725741624794], ["flowcharts", 4.856953516389879], ["eratosthenes", 4.856250723687053], ["defi", 4.8544764267474525], ["tunings", 4.8544764267474525], ["grumpily", 4.8544764267474525], ["stagehand", 4.854476426747452], ["krewes", 4.854476426747452], ["regretful", 4.854020607364659], ["paolo's", 4.851950027358044], ["benefitted", 4.85193481605282], ["advice", 4.851207419052672], ["contagious", 4.849721310949322], ["overpacking", 4.849572915205283], ["caliphate's", 4.849572915205283], ["riter", 4.849572915205283], ["enchantment", 4.849336725120529], ["genre's", 4.848587930354017], ["clanged", 4.847543441276519], ["baul", 4.847332237745616], ["\u00e9tienne", 4.846950106705983], ["tablecloth", 4.8458902779654505], ["hearts", 4.845648144566931], ["immeasurable", 4.844953364077279], ["harm", 4.8447290769052715], ["needlepoint", 4.844497780759138], ["suficiente", 4.84420240256386], ["alice's", 4.844008074803808], ["individual's", 4.8437680228649205], ["eclipse's", 4.8437316803403085], ["skillets", 4.843607182019218], ["inuktitut", 4.843215753059798], ["feasts", 4.842841497789803], ["chaucer's", 4.842647780483448], ["doodler", 4.842445506235067], ["darkwood", 4.842445506235067], ["receptive", 4.841856336515238], ["optimal", 4.841713222364844], ["heartache", 4.841435568835937], ["platys", 4.84132066407334], ["cherishable", 4.84132066407334], ["oportunidades", 4.84132066407334], ["unexplainable", 4.8411650946432605], ["buzzword", 4.840428528815832], ["tablescape", 4.839658122636501], ["breadman", 4.839658122636501], ["endangering", 4.839328662341768], ["mj\u00f6lnir", 4.839174688601675], ["austinites", 4.839174688601675], ["parvati's", 4.839174688601675], ["resentments", 4.8391164203935055], ["centuries", 4.838532271848757], ["intruding", 4.838294838658294], ["narada", 4.838294838658294], ["slithered", 4.837701255682137], ["senhor", 4.837333142208551], ["weakly", 4.836618367820227], ["amplified", 4.836595402466281], ["horizon", 4.835537975502364], ["leak's", 4.835113842709143], ["fortitude", 4.8347754388847655], ["categorization", 4.8347288507637085], ["dorian's", 4.834321423789426], ["excursion", 4.8342368391530695], ["bakeries", 4.834232458441115], ["grievances", 4.832724849833046], ["miniatures", 4.831205761971615], ["elder's", 4.8307761209603015], ["freshest", 4.830743430883353], ["homogenizing", 4.828637563531232], ["intercultural", 4.828417009131984], ["romanticization", 4.827192296376627], ["vianne", 4.827192296376627], ["takraw", 4.827192296376627], ["participants", 4.827124600293427], ["kleon", 4.827016762111305], ["advisor's", 4.826549622846045], ["earthbound", 4.826496735527005], ["jeopardized", 4.826360766607453], ["customer", 4.825370772561204], ["elysium", 4.825086068422676], ["b\u00e1nh", 4.824510522878641], ["dmards", 4.824510522878641], ["architettura", 4.824510522878641], ["siree", 4.82451052287864], ["chelmsford's", 4.82451052287864], ["turbulent", 4.823575992995274], ["buildups", 4.823467836359756], ["martina", 4.8227874834061835], ["memorability", 4.822569069951929], ["morbihan", 4.822569069951929], ["lightyears", 4.821720175554825], ["kharis", 4.821500846944218], ["regain", 4.821199206797495], ["neighborhood's", 4.8190744546838475], ["capitalizing", 4.818830773112064], ["algorithm", 4.818445423935593], ["heathenry", 4.81835681047701], ["stern", 4.818139182965376], ["fencers", 4.817618364988815], ["ominous", 4.817535988599692], ["overactive", 4.817094909959207], ["coprime", 4.816402101831786], ["roat\u00e1n", 4.816402101831786], ["instantly", 4.816256150252943], ["biking", 4.815607034865324], ["eared", 4.815347538400837], ["repellents", 4.815320979025538], ["stickhandling", 4.815069210700796], ["hereon", 4.814223720058004], ["cons", 4.813267315747063], ["undetected", 4.81278741319688], ["pycharm", 4.811627984766815], ["clawfoot", 4.811627984766815], ["marijke", 4.811407561110909], ["wigglers", 4.811407561110909], ["svetlana", 4.811325525176364], ["proficient", 4.810938273495597], ["unmarred", 4.810444894532055], ["dangers", 4.809678172780689], ["riva's", 4.809022527620603], ["ashna", 4.809022527620603], ["channeling", 4.808313951837548], ["jenny", 4.8082393460047195], ["roberto's", 4.808197723139406], ["factorization", 4.807883527191121], ["admitted", 4.807500944863166], ["poachers", 4.807102087336025], ["firewalls", 4.806956383785554], ["fretted", 4.806084188160338], ["adornments", 4.805411772099603], ["faiths", 4.805317171793048], ["conservation", 4.804613442764927], ["burgeoned", 4.804460609017327], ["mendez's", 4.804239470261503], ["yael", 4.803739636087382], ["lobstering", 4.802277755492103], ["jerrycan", 4.802277755492103], ["cosine", 4.801536663245885], ["tasting", 4.801104939709162], ["appetizing", 4.7997058415270795], ["ella", 4.798781952207953], ["possessions", 4.798734279129691], ["shattered", 4.798339944699121], ["iteratively", 4.79803733363295], ["monsoons", 4.798002223302384], ["chore", 4.797821741262716], ["approach", 4.7976659323479485], ["dictate", 4.797486009047478], ["sanjay's", 4.7974416248219205], ["mellifluous", 4.796941891319334], ["leading", 4.796573850088017], ["luther's", 4.796326977604257], ["anie", 4.79518219143257], ["musician's", 4.79435733211065], ["qingming", 4.794305846867449], ["gozaimasu", 4.794305846867449], ["bwindi", 4.79401424902985], ["kawakawa", 4.793182532470337], ["yupanqui", 4.793182532470337], ["nested", 4.793077129885787], ["jira", 4.792700563387133], ["astounded", 4.79143449534767], ["soundhole", 4.79123803651396], ["nairobi's", 4.790172369690892], ["walkability", 4.789701644647193], ["hans", 4.789684793147898], ["tacoma's", 4.789613520904835], ["pasts", 4.789182299370147], ["consuela", 4.788236007669027], ["cartography", 4.787398903471882], ["harmonize", 4.787398903471882], ["illicit", 4.787103271878094], ["manny's", 4.78632424267825], ["overfished", 4.785414003244291], ["gramps", 4.785126763508202], ["napkin", 4.783537832526705], ["tantan", 4.78299149944595], ["jaku", 4.78299149944595], ["breakouts", 4.7825140816826455], ["consulting", 4.782090789361951], ["kauai's", 4.781904455923348], ["philodendron", 4.781904455923348], ["manicured", 4.781589300091696], ["zahm", 4.781434536067224], ["pantheons", 4.781215133757488], ["mariah's", 4.78072056943499], ["halls", 4.780470802125003], ["wrappers", 4.780128589581049], ["poetess", 4.779914207120939], ["intermissions", 4.779914207120939], ["pause", 4.778746594490191], ["quienes", 4.77856280361313], ["trumpet's", 4.77856280361313], ["automail", 4.77856280361313], ["morecambe's", 4.77856280361313], ["unpopped", 4.77856280361313], ["localstorage", 4.77856280361313], ["adire", 4.77794188848792], ["conocimiento", 4.77794188848792], ["ramifications", 4.77785591562443], ["monocrystalline", 4.7777584671708455], ["brodo", 4.776978399500526], ["nicomedes", 4.776978399500526], ["varroa", 4.776845131864886], ["systemic", 4.776490979180794], ["maneki", 4.776265417649854], ["asymptote", 4.775778093354615], ["unopened", 4.774342576814919], ["meager", 4.774050081185266], ["fanbases", 4.773909870016622], ["simple", 4.7738566084673595], ["calcifying", 4.773628820336296], ["stringed", 4.772213244418147], ["safeguarded", 4.771951620984999], ["bogot\u00e1's", 4.771493923726128], ["samba's", 4.771493923726128], ["liri", 4.771493923726128], ["solitary", 4.771298443528136], ["pollution", 4.7708476580934205], ["trilobite", 4.77062637937636], ["each", 4.770439715701266], ["geek", 4.770321566496711], ["restocking", 4.769096188086064], ["introducing", 4.768736329306761], ["reza's", 4.766471298543259], ["snorkelers", 4.766471298543259], ["gardener", 4.76602755204652], ["bumblebee", 4.765529556321471], ["fika", 4.765307221718055], ["sultanahmet", 4.765174064224504], ["tearjerkers", 4.765174064224504], ["openai", 4.764789248423469], ["cafe's", 4.764578715141018], ["konak", 4.764578715141018], ["gunnar's", 4.764236898747077], ["ameera", 4.764236898747077], ["etti", 4.762973398862331], ["cryptozoologists", 4.762973398862331], ["kitesurfers", 4.762973398862331], ["luat", 4.762973398862331], ["thinnes", 4.762973398862331], ["simbas", 4.762973398862331], ["jambalaya", 4.762258774196336], ["parsing", 4.761871850344028], ["exciting", 4.761392657338897], ["goodbyes", 4.761001633422647], ["begrudging", 4.760929730537067], ["enchantments", 4.7602353425577855], ["uberpool", 4.75980863248435], ["stuttered", 4.759432154699609], ["sculptures", 4.759257350599153], ["sternest", 4.757882689078414], ["selena's", 4.757386898212503], ["standpoints", 4.755588943980375], ["outshone", 4.755588943980375], ["handmaid's", 4.755588943980374], ["zermatt", 4.755588943980374], ["yongala", 4.755588943980374], ["mensaje", 4.755588943980374], ["guling", 4.755588943980374], ["laverbread", 4.755588943980374], ["aromatic", 4.75519322199627], ["volunteered", 4.754911026810883], ["encryptions", 4.754350831784597], ["filling", 4.754014864049408], ["kitap", 4.7536565632636005], ["teairra", 4.7536565632636005], ["yarralumla", 4.752676764590307], ["cheerful", 4.752575789526028], ["pandal", 4.7518682417776406], ["misterioso", 4.751189694263463], ["bezae", 4.751189694263463], ["xampp", 4.751189694263463], ["quintet's", 4.751189694263463], ["murmuring", 4.75091426297684], ["imbalances", 4.750862892855922], ["lacquerware", 4.750612112243738], ["barong", 4.750612112243738], ["matchdays", 4.750612112243738], ["corvids", 4.750612112243738], ["overhead", 4.750389454714489], ["torn", 4.75021306082631], ["sufism", 4.749957219634602], ["plush", 4.749018849543344], ["overcooking", 4.748394407000071], ["cebu's", 4.747930990769456], ["hourglasses", 4.747930990769456], ["proclaimed", 4.74730431276476], ["chaturthi", 4.746946045018856], ["synergistically", 4.746946045018856], ["susans", 4.746023315592359], ["sunsets", 4.7458146109729835], ["retreated", 4.745767618987371], ["soliloquies", 4.7456703403750895], ["underestimating", 4.744706589417491], ["altogether", 4.743925292166821], ["bycatch", 4.743426312410092], ["advocacies", 4.743426312410092], ["sphinx's", 4.743426312410092], ["serait", 4.743426312410092], ["ninths", 4.743426312410092], ["tuktoyaktuk", 4.743426312410092], ["gell\u00e9rt", 4.743426312410092], ["webflow", 4.743426312410092], ["hypogeum", 4.743426312410092], ["elongated", 4.7430353706810475], ["gateways", 4.7421308841955065], ["misunderstanding", 4.741969709826825], ["trung", 4.741804628200721], ["dormant", 4.741750992730761], ["ingrid's", 4.740921422613009], ["marenghi's", 4.740459816905145], ["ecstatic", 4.740214130226146], ["plights", 4.740061157430172], ["filmmaking", 4.739777522939008], ["earhart's", 4.739617555133386], ["alberobello", 4.739326548959435], ["bacalao", 4.739326548959435], ["apian", 4.739326548959435], ["powerbi", 4.739326548959435], ["ilo's", 4.739326548959435], ["miro's", 4.739326548959435], ["soundbites", 4.7393265489594345], ["proficiency", 4.738678816815744], ["versatility", 4.738572898663865], ["raion", 4.738358549255808], ["untended", 4.7378523684472125], ["schematics", 4.737017261043715], ["nonessential", 4.736792149735393], ["williamsii", 4.736792149735393], ["heseltine's", 4.736792149735393], ["himeji", 4.7359520695456725], ["aneta", 4.735735655445277], ["ijeoma", 4.73557945333249], ["virtues", 4.735186330889298], ["breadcrumbs", 4.734986052067725], ["awning", 4.7345410366521605], ["unsustainable", 4.734472545339203], ["nightfall", 4.7344316220860225], ["solemnity", 4.733541983583522], ["ridicule", 4.732929794753546], ["reclaiming", 4.732786851522444], ["workflow", 4.732203139278342], ["valpara\u00edso", 4.7319292974929095], ["cusco", 4.731057534542009], ["fabrics", 4.730559528485741], ["marmots", 4.730410950213111], ["ismaili", 4.730410950213111], ["titian's", 4.730301170427772], ["messaged", 4.7301884539386485], ["damselflies", 4.729588587869053], ["onsen", 4.729404763350902], ["circulating", 4.7289316721420365], ["mentoring", 4.728728852017219], ["greenie", 4.728525496769013], ["conceded", 4.727019577551819], ["catastrophes", 4.726452317247491], ["detalles", 4.726051124452546], ["chatchai", 4.726051124452546], ["\u0434\u0440\u0443\u0433", 4.726051124452546], ["insalata", 4.726051124452545], ["dolby's", 4.726051124452545], ["slova", 4.726051124452545], ["se\u00f1orita", 4.725387353227202], ["sugary", 4.725263449265137], ["gatsby", 4.725228552263164], ["improvisations", 4.724205010732057], ["facade", 4.72414183573782], ["insightfully", 4.724094977463948], ["emissions", 4.723566918092256], ["reformations", 4.723465757316631], ["ashtanga", 4.723274361159683], ["experiencia", 4.722481599434078], ["specifically", 4.7218960056786985], ["brownstones", 4.721267671897432], ["trina's", 4.7208231917927534], ["helped", 4.720398076146074], ["schnauzers", 4.719976508611348], ["thread's", 4.719976508611348], ["debilitating", 4.719838654022822], ["marigold", 4.719533883133239], ["microaggression", 4.719241252515295], ["hers", 4.718623929960339], ["mechanica", 4.718477324573615], ["sarla", 4.718477324573615], ["bah\u00e1", 4.718477324573615], ["kermes", 4.718477324573615], ["bean's", 4.718417755361084], ["pelting", 4.717925478931879], ["op's", 4.717730611909496], ["codice", 4.717730611909495], ["chefs", 4.717074871744776], ["membre", 4.7170662363832445], ["tereza", 4.716205184609937], ["haircare", 4.715995696528179], ["cardamom", 4.715686977249799], ["minded", 4.71493987659612], ["cioppino", 4.714235996641414], ["fleetwood's", 4.714235996641414], ["bindis", 4.7131525133050225], ["handpick", 4.712849864328377], ["verdure", 4.712532671579398], ["hikers", 4.712312603741928], ["bemusedly", 4.712312603741928], ["twelfths", 4.712312603741928], ["poseidon's", 4.712179772740582], ["jaden", 4.71207845156286], ["aurelia's", 4.711095266463772], ["spiritualities", 4.711095266463772], ["rova", 4.711095266463772], ["personalities", 4.70892365328049], ["discouraged", 4.708583435494362], ["faucets", 4.708515704214123], ["solopreneur", 4.708257257267108], ["negin", 4.708257257267108], ["lyrebird", 4.708257257267108], ["dumbbells", 4.708205358487817], ["humpbacks", 4.707589987247653], ["centrioles", 4.707589987247653], ["alessandria", 4.706755061444417], ["hives", 4.705777439231173], ["compotes", 4.705680214433353], ["manzikert", 4.705680214433353], ["amala", 4.705046452788313], ["brandishing", 4.704826343953507], ["beloved's", 4.704826343953507], ["jareth", 4.704826343953507], ["pruning", 4.704266982550772], ["dive", 4.703897759806675], ["euronymous", 4.703897759806675], ["users", 4.70377651899435], ["preconceptions", 4.703595472179928], ["microcytic", 4.703329724815755], ["walks", 4.703076636689657], ["iambic", 4.702572344827862], ["statures", 4.701773464566659], ["coup's", 4.701773464566659], ["atka", 4.701177171165954], ["batches", 4.70110035454551], ["omelets", 4.699640838757076], ["visigothic", 4.698900672271657], ["kwame", 4.697694817705831], ["compatibility", 4.696057708736627], ["greta", 4.69569846852395], ["fresno's", 4.695454179921727], ["twitcher", 4.695454179921727], ["yearn", 4.695088651249742], ["lowercase", 4.694968537817874], ["midair", 4.69492144638729], ["permissions", 4.693078674747063], ["mantras", 4.6929983660665], ["prothonotary", 4.691503967110057], ["signor", 4.690836997969192], ["deborah's", 4.690782086210362], ["sounded", 4.690082116036613], ["conditionals", 4.690029393321048], ["thudding", 4.689800164122938], ["bhavna", 4.688084775445055], ["tanagers", 4.688084775445055], ["maurice's", 4.687361789050857], ["cosplayer", 4.687203719781706], ["kizito", 4.686667365082108], ["tears", 4.686101974502671], ["sprechen", 4.685001586740156], ["keyboards", 4.684772021662406], ["inflatables", 4.684090857459743], ["wholesome", 4.6839870814582865], ["umayyad", 4.683670774695227], ["suggesting", 4.683639462009162], ["modding", 4.6836138774094], ["slangs", 4.683489424844724], ["waterman", 4.6831068407109795], ["interesante", 4.683015890703392], ["canoed", 4.683015890703392], ["bioindicators", 4.683015890703392], ["activit\u00e9s", 4.683015890703392], ["aden's", 4.683015890703392], ["marvin's", 4.68261315455868], ["svms", 4.68237652748533], ["avea", 4.68237652748533], ["gravlax", 4.68237652748533], ["abab", 4.68237652748533], ["assuaging", 4.681872820463099], ["uncorked", 4.680401767000448], ["similarities", 4.6802555358620745], ["alrighty", 4.67972562332257], ["heavily", 4.679663979283973], ["fiordland", 4.679240000002738], ["gawked", 4.679240000002738], ["grimmest", 4.6776284694888925], ["adrian's", 4.677018344036351], ["fost", 4.6765636781232], ["phaedrus", 4.6765636781232], ["hiroshima's", 4.6765636781232], ["xanthia", 4.6765636781232], ["smallholder", 4.676064045251605], ["recesses", 4.675902025084416], ["simpler", 4.67588052625454], ["preikestolen", 4.674681003534583], ["obstinance", 4.674681003534583], ["task's", 4.674681003534583], ["path\u00e9tique", 4.674681003534583], ["whisked", 4.674064562083569], ["getups", 4.671836139243241], ["overextension", 4.671836139243241], ["perlis", 4.67173724975402], ["auroras", 4.6714980223580875], ["soundcloud's", 4.671351458660271], ["recreationally", 4.670890136177686], ["annals", 4.669702053353304], ["lighthouse's", 4.669630570298267], ["corleone's", 4.669630570298267], ["deque", 4.669630570298267], ["scrimped", 4.668881151172878], ["misremembering", 4.668881151172878], ["leatherwork", 4.668881151172878], ["skanderbeg", 4.668881151172878], ["chokepoints", 4.668881151172878], ["allsvenskan", 4.667976767414782], ["secondhand", 4.667713930885085], ["sita", 4.667443596641387], ["herpetological", 4.667117494144619], ["entrepreneurship", 4.666213219086494], ["toastmasters", 4.666213219086494], ["sowed", 4.665760562228649], ["az\u00facar", 4.665460725421103], ["hordaland", 4.665460725421103], ["kiteboarding", 4.665460725421103], ["moviegoer", 4.664773314086872], ["tangled", 4.6642678519401715], ["dacians", 4.6632046999252506], ["dulse", 4.6632046999252506], ["brainwaves", 4.663152944046229], ["poise", 4.661229036870566], ["beantown", 4.660774175219776], ["beethoven's", 4.659921677748453], ["threatening", 4.659890220041762], ["minimalist", 4.6596760870633105], ["intellect", 4.659634046756962], ["multiprocessing", 4.658536108388938], ["shop's", 4.657290954404158], ["policymaking", 4.657063465421107], ["options", 4.656511151437849], ["mocktail", 4.655755169512176], ["accordingly", 4.655278508863869], ["prophase", 4.655160357585757], ["gelato", 4.653749596056071], ["recipe", 4.652790381948349], ["mangoes", 4.65270924517113], ["optometrist's", 4.652206575632975], ["karani", 4.652206575632975], ["overwrote", 4.652206575632975], ["current's", 4.652206575632975], ["papiamento", 4.652206575632975], ["polonica", 4.652206575632975], ["disfluencies", 4.652206575632975], ["zaanse", 4.652206575632975], ["molcajete", 4.652206575632975], ["donwood", 4.652206575632975], ["matilda's", 4.650582768800294], ["pollutants", 4.648707480088909], ["disconnection", 4.6474559635207315], ["stalagmites", 4.646567537359481], ["eski", 4.646567537359481], ["penning", 4.64656753735948], ["trusting", 4.645944557893076], ["trailblazer", 4.645708255336853], ["armload", 4.645125591499591], ["mika's", 4.6446665325606356], ["doha's", 4.644254085760099], ["sunbathers", 4.6435432486764885], ["observant", 4.643345229774626], ["kepler's", 4.643345229774626], ["cooperation", 4.643336368428768], ["padlet", 4.642201830309033], ["ramens", 4.642201830309033], ["oauth", 4.641990926586789], ["cohousing", 4.641936803942836], ["tikal", 4.641868338798235], ["servings", 4.641616186680314], ["addison's", 4.641260207219721], ["implemented", 4.640895328272613], ["bhima", 4.6398006914312875], ["tainted", 4.639800691431287], ["zain's", 4.639598969736951], ["xiaohua", 4.639598969736951], ["wondered", 4.639486618413522], ["neighborly", 4.638952425844847], ["amrita", 4.638952425844847], ["obsidian's", 4.63872191889201], ["lavaux", 4.63872191889201], ["guincho", 4.63872191889201], ["youthful", 4.637841867609968], ["anvar", 4.637713720257171], ["amphorae", 4.637557252182939], ["smartwatches", 4.636809201623787], ["turbines", 4.636652370364997], ["malfunctions", 4.636623819950142], ["celebratory", 4.636564373813456], ["cleaner", 4.636113300157233], ["ablaze", 4.635806317434504], ["wildernesses", 4.635784262960543], ["attuned", 4.6354529111972305], ["breakdancer", 4.635165525905381], ["sachertorte", 4.635165525905381], ["rhoda's", 4.635165525905381], ["samar's", 4.635165525905381], ["epicureanism", 4.634860540675358], ["callaloo", 4.634860540675358], ["geshe", 4.634860540675358], ["linda", 4.634730460496684], ["thaddaeus", 4.634382029366182], ["breton's", 4.634382029366182], ["awadhi", 4.634382029366182], ["turning", 4.6341660500687265], ["bewilderment", 4.634048080021605], ["prioritizes", 4.633720986590725], ["satiated", 4.633650765929595], ["theatergoers", 4.633217977365085], ["message's", 4.632577012022709], ["phrasing", 4.632396132274258], ["collector's", 4.632268547451691], ["peeks", 4.631945709474439], ["budgeting", 4.631297782034625], ["microbiologist", 4.631200596407009], ["webs", 4.630496278280021], ["barbirolli", 4.629947214027076], ["superbugs", 4.628678174560808], ["flaunted", 4.628349106014344], ["yuki", 4.627837874522516], ["snowflakes", 4.627683316164522], ["sofija", 4.627591726026451], ["lyso", 4.627591726026451], ["formalwear", 4.627366418478565], ["fluffiness", 4.627366418478565], ["apples", 4.626360983546125], ["disneysea", 4.625922922663297], ["skinks", 4.625315208143767], ["ensnare", 4.624601859477386], ["miserably", 4.624544806804886], ["secretly", 4.624354149337028], ["leatherworker", 4.624011384265502], ["gelug", 4.624011384265502], ["blinded", 4.623399691978208], ["ebbs", 4.623090724955545], ["zoodles", 4.622724911655579], ["amita", 4.622048975859169], ["paellas", 4.621799996707269], ["entscheidung", 4.621799996707269], ["antigua's", 4.621799996707269], ["cubbon", 4.620122392352747], ["mfa's", 4.620122392352747], ["lidia's", 4.620122392352747], ["overwhelmingly", 4.619136135037698], ["implicating", 4.617915126672536], ["juma", 4.617745786183842], ["underutilized", 4.617537563891098], ["rashness", 4.617015905620622], ["considering", 4.616834191845096], ["beansprouts", 4.616142958767603], ["kt\u00f3ry", 4.616142958767603], ["chenda", 4.616142958767603], ["marginalization", 4.615432981522826], ["nimble", 4.614564790234691], ["resolved", 4.614398166637355], ["athleticism", 4.6142094957482245], ["baade", 4.613758587404603], ["oaxacan", 4.6135761532967585], ["halfling", 4.6129925738460305], ["myst\u00e8re", 4.61244412626859], ["camelid", 4.611611404815934], ["dupont", 4.611520611315612], ["pozole", 4.611299883121413], ["elaborated", 4.610413703322323], ["tastefully", 4.61034098845097], ["dilla's", 4.610294804681326], ["wuwei", 4.610294804681326], ["kiel's", 4.610294804681326], ["panucci", 4.610294804681326], ["bladesmith", 4.610294804681326], ["riitta", 4.610294804681326], ["perfectionistic", 4.610041561352314], ["amirite", 4.609765392837726], ["solveig", 4.609637365136987], ["moviemaking", 4.6093685776989695], ["telecommuting", 4.60902153569039], ["tevita", 4.607899846341232], ["bipv", 4.607899846341232], ["contexto", 4.607899846341232], ["mounting", 4.607141001806402], ["yorke's", 4.607106884743169], ["refurbishing", 4.606923389497337], ["kaisa", 4.606525994270171], ["emir's", 4.606326826366377], ["mural", 4.605433581798132], ["konrad's", 4.60521459002052], ["djibouti's", 4.60521459002052], ["disrespected", 4.605192502130691], ["druidry", 4.604804179665827], ["erasure", 4.604491636395713], ["flaky", 4.604073084297141], ["labeled", 4.603758011868695], ["denizens", 4.603301143899773], ["dissatisfaction", 4.6032359800999965], ["bisector", 4.603235980099996], ["nonviolence", 4.602836220136462], ["oversimplification", 4.602836220136462], ["modifications", 4.602751954042282], ["jigsaws", 4.602674482690118], ["shoebox", 4.602408308062819], ["reacquainting", 4.602182849013265], ["iona's", 4.602182849013265], ["biko", 4.601840806438088], ["akhenaten", 4.601615203888502], ["telescreens", 4.600857717182279], ["musikverein", 4.600857717182279], ["pudd'nhead", 4.600857717182279], ["gambas", 4.600515391459275], ["nightscape", 4.600515391459275], ["shapes", 4.599700607184684], ["nonprofit's", 4.598977667444954], ["gloating", 4.598780787963514], ["afforestation", 4.598053906498623], ["cromwell's", 4.598053906498623], ["marissa", 4.597695872322528], ["geocentrism", 4.597474733566705], ["hellfighters", 4.597474733566705], ["tallinn's", 4.597474733566705], ["m\u00e1gico", 4.597474733566705], ["hurtled", 4.596823164018297], ["bugatti's", 4.596823164018297], ["devtools", 4.596823164018297], ["topiaries", 4.596823164018297], ["inheriting", 4.596370712546709], ["oddity", 4.595864062492882], ["perpetuate", 4.594979432380027], ["anja's", 4.594771926551086], ["achaemenids", 4.594771926551086], ["frybread", 4.594771926551086], ["jadwiga", 4.594771926551086], ["talented", 4.594643272937143], ["johann", 4.593358150573686], ["arduously", 4.593317884802178], ["celebrated", 4.592815187472021], ["xiao", 4.592658331464873], ["chanin", 4.592409575946176], ["greenwood", 4.592154187963449], ["contextualizing", 4.592072295571914], ["decode", 4.5913586674056495], ["focaccia", 4.591237486607586], ["rambunctious", 4.5906598738541735], ["quer\u00e9taro", 4.590467690554317], ["countermove", 4.590177154624535], ["humanlike", 4.58946005149149], ["migratory", 4.589411359303444], ["midflight", 4.588477718432523], ["strange", 4.587906054541846], ["splurge", 4.587755799499062], ["marinating", 4.587113973340168], ["falconry", 4.586996158675384], ["milonga", 4.585935826692334], ["bobbed", 4.585008036207166], ["ironworking", 4.584783291928149], ["ieva", 4.584783291928149], ["wayuu", 4.584783291928149], ["imposing", 4.584401603320211], ["grime", 4.584125503794443], ["designs", 4.583715756602823], ["emanated", 4.583410308696341], ["nasturtiums", 4.582581715317379], ["logger", 4.5817601830422685], ["gregor's", 4.581015124375784], ["torchbearers", 4.581015124375784], ["focke", 4.581015124375784], ["adalbert", 4.580936638658144], ["summoning", 4.580634166777083], ["trendy", 4.580084231884179], ["chmod", 4.579353228810982], ["atmosphere", 4.579322005948059], ["steadying", 4.5790257980387485], ["tankards", 4.578899078077546], ["turns", 4.578634288454269], ["photosynthesizing", 4.577946001186251], ["preservationists", 4.577675100777872], ["diner's", 4.577494288284384], ["discomforts", 4.577343481312445], ["onward", 4.5772543585811105], ["harkened", 4.576928152079043], ["unsolvable", 4.576524746172916], ["professionalism", 4.576337921252135], ["lili's", 4.576233941833013], ["informed", 4.575615638061561], ["notch", 4.575569894619231], ["destigmatize", 4.575554249291748], ["decadent", 4.575515655113449], ["muisca", 4.575219705587038], ["fitfam", 4.575219705587038], ["friedrichshain", 4.575219705587038], ["galer\u00eda", 4.575219705587038], ["subside", 4.574976268843811], ["alphanumeric", 4.574669799372425], ["ordeal", 4.572935078276426], ["vanished", 4.572927068020284], ["silks", 4.572282119066185], ["overfeeding", 4.572088207510707], ["haki", 4.572088207510707], ["plumes", 4.571687615670096], ["connected", 4.571658976745189], ["carpool", 4.571634551903151], ["colonialism", 4.5710465855284745], ["fatuma", 4.570588916411344], ["tuku", 4.570588916411344], ["loosestrife", 4.568833697933459], ["showerhead", 4.568148014453313], ["deforested", 4.568148014453313], ["spine", 4.56801858354749], ["cleverly", 4.567994447773835], ["subtract", 4.567531492986533], ["staccato", 4.567324210042478], ["historian", 4.567209081858186], ["madame's", 4.567001326995008], ["misidentifications", 4.567001326995008], ["disembarked", 4.565912202053741], ["placita", 4.565734334450206], ["interdisciplinary", 4.563846298359728], ["mixups", 4.56308384429901], ["waterhole", 4.562163867717498], ["hillman's", 4.562163867717498], ["protium", 4.562163867717498], ["bladderwort", 4.562163867717498], ["interruptions", 4.561981963257063], ["fearlessness", 4.561747003329001], ["generational", 4.561598154369339], ["tracing", 4.561469323414018], ["bosco's", 4.561414097576125], ["gorbachev's", 4.560986838855857], ["sestina", 4.560986838855857], ["atop", 4.560833025716561], ["bartolom\u00e9", 4.560710833600859], ["immemorial", 4.560517842779883], ["exoplanets", 4.560253413836544], ["aesculapius", 4.559778630421724], ["sumita", 4.559778630421724], ["beekeeper's", 4.559427527116077], ["stormchaser", 4.559427527116077], ["rosal\u00eda", 4.559427527116077], ["completamente", 4.558788772909421], ["edutech", 4.558788772909421], ["floorboard", 4.5582226044080665], ["tailoring", 4.556947107655728], ["jorge", 4.5564945120339315], ["arrays", 4.5564585975635366], ["paninis", 4.556252880460239], ["cinematic", 4.556201240604837], ["leaving", 4.556090201592881], ["seafarers", 4.55478284523067], ["heaviness", 4.554139496146624], ["abayas", 4.554058757581646], ["dishoom", 4.554058757581646], ["nidra", 4.554058757581646], ["\u0436\u0438\u0437\u043d\u0438", 4.554058757581646], ["concisely", 4.553821542311432], ["pongal", 4.5536048948392125], ["gladiatorial", 4.553154358839393], ["thicket", 4.552199958088756], ["astonishing", 4.551740120238963], ["discolorations", 4.550902244840297], ["retriever", 4.5496707364020335], ["greenfields", 4.549526294676729], ["wit's", 4.548824207285575], ["matriarch", 4.548824207285575], ["rainclouds", 4.548824207285575], ["biologist's", 4.548824207285575], ["xeriscape", 4.548824207285575], ["hosseini's", 4.548824207285575], ["owlets", 4.548824207285575], ["aulos", 4.548824207285575], ["friends", 4.547167014823442], ["appam", 4.546659131508667], ["strayed", 4.546164853758954], ["cater", 4.546145002246713], ["impart", 4.545798381870529], ["backstrap", 4.545406608331942], ["serves", 4.544958972615934], ["mapmaker", 4.544132624929453], ["catharsis", 4.543522547370324], ["performances", 4.543289274334053], ["customers", 4.542528683959345], ["luca", 4.541587441501258], ["kayaks", 4.54155107584405], ["gedacht", 4.541439752403619], ["suneo", 4.541439752403619], ["inquiringly", 4.541439752403619], ["sogdian", 4.541439752403619], ["\u00e1reas", 4.541439752403619], ["vasco's", 4.541439752403619], ["challah", 4.541439752403618], ["accusatory", 4.541439752403618], ["cyberattack", 4.540909767603478], ["peaceful", 4.54061857009109], ["electronica", 4.540594765029066], ["immutability", 4.540553617817784], ["worn", 4.5404854536188735], ["meadow's", 4.540482114162442], ["quell", 4.539313029397615], ["driven", 4.538873281917266], ["vila's", 4.538738122568756], ["eeyore's", 4.538738122568756], ["calment", 4.538738122568756], ["mussie", 4.538738122568756], ["debate", 4.5382825177350945], ["weariness", 4.538171223193571], ["brought", 4.537935548462117], ["idioma", 4.53756075859403], ["favorite", 4.537375690353471], ["lenny's", 4.537104748525547], ["rollerblades", 4.537009079474444], ["tale's", 4.536779659516947], ["skimping", 4.536661575715294], ["haven", 4.536602113960949], ["restlessness", 4.53571709243963], ["abruptly", 4.535495571366406], ["spontini", 4.535484546853653], ["esteban's", 4.535484546853653], ["waded", 4.534677146353825], ["calanques", 4.53456457027214], ["tegus", 4.53456457027214], ["milica", 4.532919227539821], ["soggy", 4.532642619196418], ["ranjana", 4.5324211592805606], ["fumaroles", 4.532360504597934], ["forest's", 4.531829845365455], ["closely", 4.531807286477951], ["alexandrinus", 4.531227208417933], ["tooltips", 4.530844664964289], ["def's", 4.530844664964288], ["woodblock", 4.530503281249326], ["opting", 4.530374430780501], ["orchid's", 4.530238612975931], ["sedona's", 4.530238612975931], ["toolsets", 4.530238612975931], ["dabke", 4.530238612975931], ["naram", 4.530238612975931], ["neglected", 4.529896490095604], ["paperclips", 4.529780086800528], ["nichiren", 4.529358411178832], ["empanadas", 4.528814716637691], ["pingyao", 4.528147733616096], ["svenja", 4.528147733616096], ["revitalization", 4.527508255048999], ["centerpieces", 4.527434751765424], ["entrepreneurs", 4.527054015865317], ["subtractor", 4.5264712627780295], ["mollusks", 4.526181383094214], ["harajuku", 4.526102807648785], ["ethicists", 4.526008788064081], ["bismarck's", 4.52527804509969], ["griot", 4.52527804509969], ["unexpectedly", 4.525124857348628], ["rivalry", 4.524903722691724], ["skylines", 4.524824728919215], ["homeschooling", 4.524764310651999], ["johnson", 4.524697716295245], ["nerds", 4.524084671104625], ["fluency", 4.5240124388822], ["buongiorno", 4.523692581830959], ["chinatown's", 4.522978615198726], ["guetta's", 4.522978615198726], ["fluidly", 4.522651455805221], ["neurofeedback", 4.5219546218513855], ["rakia", 4.521770877250742], ["usage", 4.521579912568143], ["embedded", 4.520793730094477], ["serenely", 4.520500270752042], ["mehmed", 4.519973313793278], ["resplendently", 4.519973313793278], ["interpreting", 4.519789159153966], ["storefronts", 4.519073885458986], ["akihabara", 4.518699543661167], ["zestful", 4.518699543661167], ["jaffa's", 4.518699543661167], ["surmount", 4.518192394441902], ["longbourn", 4.518192394441901], ["escapements", 4.518192394441901], ["nkechi", 4.518192394441901], ["flavor", 4.517433560896456], ["stovetop", 4.517201365202842], ["headbangers", 4.516779487945422], ["nectar", 4.514208537883862], ["calculation", 4.513800058843708], ["incredibly", 4.513312363758243], ["pseudonyms", 4.512593528348118], ["elena's", 4.512578596072145], ["goalpost", 4.51246996786671], ["circumstances", 4.511946733167619], ["hasting's", 4.511230618795612], ["qled", 4.511230618795612], ["aster's", 4.511230618795612], ["regressor", 4.5112306187956115], ["depictions", 4.509394652846102], ["risotto", 4.508643205822619], ["craiova", 4.508053695824629], ["risked", 4.508001425938141], ["implement", 4.507968542224742], ["superior's", 4.5074010512075], ["shinto", 4.50717375672835], ["pasos", 4.50662732224582], ["shattering", 4.50503873363256], ["aleksi", 4.504730286491872], ["anew", 4.504592257283777], ["commuting", 4.504290263997465], ["retrofitting", 4.503966772883992], ["merrymaking", 4.50391906187698], ["botafogo", 4.503620580639749], ["planchette", 4.503306911619742], ["roughhousing", 4.503306911619742], ["deviously", 4.503203518050424], ["disengaged", 4.502528925847103], ["midway", 4.501816877351289], ["beginnings", 4.50160425139967], ["equally", 4.501513059779002], ["everyone", 4.5013678452783505], ["accademia", 4.500341441439899], ["anagrammatic", 4.500173681004446], ["permissioned", 4.500173681004446], ["transforma", 4.500173681004446], ["chikankari", 4.500173681004446], ["bibbo", 4.500173681004446], ["sasanians", 4.500173681004446], ["boosted", 4.499803163751312], ["helpless", 4.499791426869381], ["shifting", 4.4997635901808195], ["sauntered", 4.499095979472902], ["woodworkers", 4.49813349764427], ["neonicotinoid", 4.498039885992116], ["sneaked", 4.498029402028182], ["moderators", 4.49774115469039], ["temples", 4.497566362380398], ["habitat", 4.497295403795144], ["reevaluation", 4.497133023111876], ["ramped", 4.496823495062931], ["shadowy", 4.49677653218654], ["tardes", 4.496734376447297], ["kweku", 4.496734376447297], ["situations", 4.495766273157453], ["amphora", 4.495218534809147], ["thorkell", 4.494885580321715], ["parmigiano", 4.494697312862853], ["harrisons", 4.494697312862853], ["meerkats", 4.494092365219305], ["potted", 4.4937502475689355], ["therapies", 4.49261631769864], ["path\u00e9", 4.492336933858646], ["betterment", 4.492111593792146], ["tummies", 4.492020708193598], ["didgeridoo", 4.491785659231838], ["puttu", 4.491785659231838], ["failures", 4.491260227366451], ["member's", 4.490930081011032], ["solve", 4.490458374387393], ["vital", 4.490258095616773], ["gourmands", 4.489748568229919], ["circumambulating", 4.489748568229919], ["cayugas", 4.489748568229919], ["mapo", 4.489422183483755], ["szczecin", 4.489422183483755], ["midsommar", 4.489288835630264], ["concern", 4.48897554345628], ["daws", 4.488795735341925], ["pla\u00e7a", 4.48856111819865], ["emira", 4.487234711248826], ["businesses", 4.486423786093132], ["riserva", 4.486056340788941], ["jafar's", 4.486056340788941], ["unmoved", 4.485801970213674], ["saxophone", 4.4854750113620625], ["proprietress", 4.485046029797627], ["factory's", 4.484461643333888], ["keyboardists", 4.484054530730578], ["ola's", 4.484054530730578], ["deterred", 4.482680324262302], ["prodding", 4.482583788207149], ["farzan", 4.4821259051259625], ["vedere", 4.481706858201401], ["namesake's", 4.481706858201401], ["toomas", 4.481128991001158], ["punctuate", 4.480846759605093], ["opus", 4.479807564140552], ["inequality", 4.4795381392681355], ["uninvited", 4.478734466033101], ["bob's", 4.478075791356271], ["ostracizing", 4.477054629259281], ["bronchioles", 4.477054629259281], ["interdepartmental", 4.476177137636051], ["storytime", 4.475615640226173], ["diotima", 4.475540503140583], ["lenguaje", 4.475540503140583], ["prot\u00e9g\u00e9s", 4.47539374004817], ["cooperatively", 4.47465307323781], ["moules", 4.47425331864155], ["nalin", 4.47425331864155], ["riana", 4.47425331864155], ["hoppy", 4.473719291987246], ["cheering", 4.473459072490312], ["extravagance", 4.472647725345386], ["leches", 4.472647725345386], ["photoshoots", 4.4723212547085005], ["antioxidants", 4.47224467517639], ["setting", 4.471257592310901], ["anecdotes", 4.470452245953223], ["beside", 4.469720020924275], ["pictish", 4.469673734896778], ["digging", 4.469419776161841], ["partake", 4.469037344184524], ["readymades", 4.468786244694041], ["geologist's", 4.468786244694041], ["marmee", 4.468786244694041], ["sponsorships", 4.468462204359662], ["horsemanship", 4.46809534594657], ["requiring", 4.4676465737049655], ["instructor's", 4.466118312607656], ["doodads", 4.466118312607656], ["speedrunners", 4.466118312607656], ["complimenting", 4.4654329046407115], ["fireflies", 4.4654329046407115], ["maintainable", 4.464586721965472], ["covid", 4.4642334406985595], ["figure's", 4.463997648641555], ["devanagari", 4.463705498824507], ["hypnotically", 4.462972605044628], ["coworker", 4.462301117468675], ["needlework", 4.462116414478079], ["disparity", 4.461401246371272], ["compromises", 4.460922130743074], ["hinted", 4.460681470925641], ["durations", 4.459347069768528], ["isidro", 4.458533929252912], ["local", 4.458301222004733], ["freediving", 4.458142083959038], ["strict", 4.458017999981848], ["marcus", 4.457847723139864], ["colors", 4.45771849517943], ["belinda's", 4.457707543657359], ["woodsmoke", 4.457707543657359], ["squarely", 4.457562530399596], ["curricula", 4.457095851370064], ["ibrahima", 4.456482674392753], ["reviewer's", 4.455363985745511], ["simbel", 4.454722510835751], ["myeongdong", 4.454722510835751], ["angoul\u00eame", 4.454722510835751], ["mozart's", 4.454716843538271], ["vegetables", 4.454240338371997], ["colliding", 4.454144058496038], ["shiny", 4.454022984136971], ["uncommons", 4.453394328811053], ["tirunesh", 4.453394328811053], ["ijma", 4.453394328811053], ["tezcatlipoca", 4.4533943288110525], ["fagin", 4.4533943288110525], ["stripe's", 4.4533943288110525], ["informative", 4.453011358782777], ["fisherman", 4.452869310062164], ["hardliners", 4.452826293820133], ["bickering", 4.452739417880346], ["bj\u00f6rk", 4.45252282307156], ["bangalore's", 4.452002208045561], ["jiri", 4.451523154723318], ["pla\u00eet", 4.451523154723317], ["battlestations", 4.451523154723317], ["urushi", 4.451523154723317], ["chompy", 4.451523154723317], ["berryville", 4.450698569532111], ["perceptibly", 4.450698569532111], ["ecosphere", 4.450268042865941], ["sociedad's", 4.449006748191534], ["croque", 4.448100689561332], ["regretfully", 4.448059114086211], ["weren't", 4.447988627970534], ["boutique", 4.44752736221896], ["showdown", 4.44724852110347], ["steered", 4.446820270516142], ["factoring", 4.446802133258537], ["gardeners", 4.446603438582529], ["fierce", 4.445988572616937], ["varying", 4.445641290452672], ["tahini", 4.445441838938176], ["taberna", 4.445441838938176], ["figaro's", 4.445441838938176], ["eigene", 4.445441838938176], ["manuscript's", 4.445441838938176], ["palmyra's", 4.445441838938176], ["zanzibari", 4.445441838938176], ["chimichurri", 4.44389881851508], ["babbling", 4.443689595406863], ["haughtily", 4.443598717298271], ["cleanup", 4.443288039597605], ["poodles", 4.443242214079721], ["se\u00e1n", 4.44279100898055], ["com\u00fan", 4.442647720874732], ["lahiri's", 4.442647720874732], ["meiner", 4.442488056985393], ["stoicism", 4.442294366365538], ["balanced", 4.442210284113012], ["jabari", 4.4416128623327165], ["dosomething", 4.4416128623327165], ["nunavut's", 4.4416128623327165], ["lojban", 4.4416128623327165], ["songkran", 4.441306544204281], ["presented", 4.441240320623133], ["dandelions", 4.441134240257035], ["maori", 4.440450965983474], ["paulaner", 4.439360523153034], ["number1", 4.439360523153034], ["during", 4.438693037932458], ["memberof", 4.437877275400561], ["misallocated", 4.437877275400561], ["y\u014dkai", 4.437877275400561], ["dsme", 4.437877275400561], ["snat", 4.437877275400561], ["xcloud", 4.437877275400561], ["festspielhaus", 4.437877275400561], ["chitarra", 4.437877275400561], ["gandee", 4.437877275400561], ["indie", 4.4370885435757055], ["neorealist", 4.436826641575893], ["wordlist", 4.436826641575893], ["jard\u00edn", 4.436826641575893], ["decorum", 4.436652244868155], ["veche", 4.436541370140056], ["groomer", 4.4361897203701215], ["bakers", 4.435103602103436], ["deniz", 4.435103602103436], ["counterargument", 4.434559484375292], ["hanseatic", 4.4344437146459], ["contexts", 4.434124190192776], ["surprisingly", 4.433237676430824], ["keone", 4.433226631150731], ["fabled", 4.433185166402144], ["treehugger", 4.4329509101147115], ["effortless", 4.432856159313276], ["amnh", 4.432441660882516], ["responses", 4.431890796194794], ["tsegaye", 4.430672929174262], ["woodpecker's", 4.430672929174262], ["trivialities", 4.4306729291742615], ["savoye", 4.4306729291742615], ["bingley's", 4.4306729291742615], ["t\u00edtulo", 4.4306729291742615], ["kraftwerk's", 4.4306729291742615], ["caledonia's", 4.4306729291742615], ["semaines", 4.4306729291742615], ["majuli", 4.4306729291742615], ["alyana", 4.4306729291742615], ["beamed", 4.430141248422761], ["charlie's", 4.429808407627106], ["dieser", 4.429745039555586], ["chunking", 4.428900660002592], ["spontaneous", 4.4288764769924525], ["manipulatives", 4.428822961354148], ["juhani", 4.428822961354148], ["easel", 4.428822961354147], ["aeropress", 4.428645161243289], ["masterful", 4.428361273732953], ["undying", 4.428275025005951], ["uniquely", 4.427863857000146], ["spaceship's", 4.427378748557775], ["metaphors", 4.426711386319941], ["irresponsibly", 4.42629695591088], ["extravaganza", 4.426296955910879], ["unperturbed", 4.425134588012794], ["foggia", 4.424855853118453], ["indisputable", 4.424414916562434], ["infuriated", 4.423834591534379], ["unburdened", 4.423803668818953], ["frigatebird", 4.423803668818953], ["meninas", 4.423651260506791], ["butterbeer", 4.423651260506791], ["stethoscopes", 4.4234919357720335], ["encroached", 4.422766895694695], ["forests", 4.42253866810429], ["seated", 4.422314030761489], ["pseudonym", 4.421811583315913], ["pedagogies", 4.421744621004641], ["safeguard", 4.421037775649409], ["unhoused", 4.4210060064197], ["dialoguing", 4.4210060064197], ["greenhouse", 4.420650644140442], ["woodcarvers", 4.420487474164666], ["unboxings", 4.420487474164666], ["itzel", 4.420487474164666], ["abebe", 4.419786754901106], ["assuredness", 4.419786754901106], ["unattainable", 4.419698403341789], ["surmounted", 4.419470579466787], ["rejuvenate", 4.41893353936192], ["rainstorm", 4.41792279124858], ["falsehoods", 4.417459901314163], ["decimating", 4.417422318544956], ["exit's", 4.417246647570703], ["worsley's", 4.417246647570703], ["spreading", 4.417215117464227], ["visage", 4.417153131181426], ["birria", 4.416754584894134], ["meroitic", 4.416754584894134], ["ameena", 4.416334181830655], ["slithering", 4.415482050559951], ["reimagining", 4.415427602966351], ["rajasthani", 4.415035260012471], ["backchannel", 4.414905409853357], ["warnings", 4.414890034560885], ["horrified", 4.4144655605411485], ["smirking", 4.41399731114761], ["wallowing", 4.413631879446668], ["stranger", 4.4133770666821555], ["coral", 4.413170547161225], ["ceiba", 4.413160387952229], ["resentful", 4.4118675006645836], ["poufs", 4.410981049489043], ["kokand", 4.410981049489043], ["etta's", 4.410981049489043], ["ensalada", 4.410981049489043], ["gurdy", 4.409727929872711], ["sarie", 4.409320288551735], ["unaccommodating", 4.409320288551735], ["disguised", 4.409044410081984], ["icebreakers", 4.408860385522942], ["kaftans", 4.408702650186621], ["argv", 4.408702650186621], ["chac", 4.408702650186621], ["converged", 4.40843781779538], ["shortness", 4.408256876410061], ["mehta's", 4.407353597968081], ["euterpe", 4.407353597968081], ["relied", 4.405871885546508], ["vivienne's", 4.405828034244313], ["mediu", 4.405828034244313], ["marcello's", 4.405828034244313], ["leatherbacks", 4.405828034244313], ["tonyo", 4.405828034244313], ["bichons", 4.405828034244313], ["galinda", 4.405828034244313], ["relleno", 4.405354798150409], ["puzzle's", 4.405354798150409], ["thessaloniki's", 4.40498786871528], ["indigenous", 4.4036611024888135], ["despondently", 4.4033230962781245], ["soundtracked", 4.4033230962781245], ["indre", 4.403215237782196], ["reframed", 4.402872628442188], ["adding", 4.402763476620404], ["graffitis", 4.402087942534427], ["pinpricks", 4.401307845433146], ["ophelia's", 4.401059176948419], ["reflections", 4.400907895650065], ["krishna's", 4.4003777296585405], ["floresta", 4.4003777296585405], ["panzanella", 4.4003777296585405], ["bigshot", 4.4003777296585405], ["pel\u00e9's", 4.399249716910614], ["lugares", 4.399249716910614], ["globo's", 4.399249716910614], ["bronchoconstriction", 4.398899179084566], ["warming's", 4.398899179084566], ["pilfered", 4.398449853325722], ["sbss", 4.398449853325722], ["pounamu", 4.398449853325722], ["jena's", 4.398449853325722], ["uninstallation", 4.398449853325722], ["soiree", 4.398058253445618], ["disastrous", 4.398002497471039], ["convenience", 4.397590571303094], ["woodpeckers", 4.397572181999108], ["grandmotherly", 4.3974748034801605], ["restful", 4.397245788213944], ["bygone", 4.397115169710694], ["madeiran", 4.39702224869952], ["clarinetists", 4.39702224869952], ["ta\u00edno", 4.39702224869952], ["fragments", 4.396675390449947], ["commiserations", 4.396415148794048], ["travels", 4.396224447149932], ["behind", 4.396104591766848], ["gurdwara", 4.3954622833795], ["reishi", 4.395049428236177], ["modest", 4.395045349844715], ["carve", 4.394193147391644], ["ticking", 4.394181414632591], ["wernicke's", 4.393750654764476], ["xylo", 4.393750654764476], ["whiffs", 4.393486114415584], ["visited", 4.3933318907407894], ["furtively", 4.393142523185962], ["arkhangelsk", 4.39283088280409], ["hole", 4.392823154304467], ["frontlines", 4.39265748772757], ["antiderivative", 4.392514023564149], ["salzburg's", 4.392514023564149], ["kebabs", 4.392280227205925], ["amphitheater", 4.39219418487652], ["voices", 4.390852256223308], ["geh\u00f6rt", 4.390846655653594], ["mycorrhizal", 4.390773675571227], ["tenochtitlan", 4.390773675571227], ["mortified", 4.390592672451614], ["metaverse", 4.389418991286234], ["echeveria", 4.389305466936271], ["barged", 4.389170423255414], ["diffusers", 4.389170423255414], ["tapas", 4.388324508359503], ["hydropower", 4.388324508359503], ["timestamp", 4.388172469421991], ["sarod", 4.3881075123874345], ["headbanger", 4.385077637285668], ["sartre's", 4.384329915968952], ["kotlin", 4.383412417929736], ["novice", 4.38260788588034], ["espresso", 4.382062117608464], ["erasing", 4.381532738505237], ["unfiltered", 4.381470428690061], ["waren", 4.380608828279637], ["roleplay", 4.3805346031376144], ["granny's", 4.380036667126556], ["panettone", 4.380036667126556], ["overnight", 4.379997464859164], ["wholes", 4.379816509465479], ["anomalies", 4.379731789108992], ["juliet's", 4.3796530590807405], ["recursive", 4.3795583649031835], ["altocumulus", 4.378547365301624], ["jyoti's", 4.378547365301624], ["erhalten", 4.378547365301624], ["karluk", 4.378547365301624], ["oncologist's", 4.378547365301624], ["dooky", 4.378547365301623], ["jaswinder", 4.378547365301623], ["reviving", 4.3780765537569675], ["ipas", 4.377050426031435], ["abled", 4.377050426031435], ["elias", 4.376494054116375], ["exclamatory", 4.376185689268559], ["circulated", 4.375957238691163], ["nudges", 4.375893700231743], ["flo's", 4.375893700231743], ["udhr", 4.3738694300822845], ["jamison's", 4.3735998541543895], ["lanky", 4.373247447634126], ["alan's", 4.373230033911601], ["delightedly", 4.3730272602016464], ["kulfi", 4.3718676591760675], ["conocido", 4.371597290118605], ["bonfim", 4.371597290118605], ["once's", 4.371597290118605], ["moovit", 4.371597290118605], ["unifiers", 4.371597290118605], ["kiboko", 4.371597290118605], ["hobbled", 4.371441827910563], ["mallets", 4.371249786359455], ["referencing", 4.370771779005163], ["pileated", 4.370756355754824], ["khuzestan", 4.370756355754824], ["mentioned", 4.37048656323636], ["disinterested", 4.370289342390436], ["catalyst", 4.369676047596274], ["comiket", 4.369167443782126], ["backroads", 4.3690287840727065], ["squawking", 4.368770704517855], ["denarius", 4.368090240921852], ["hanalei", 4.3677521301800715], ["plaintively", 4.3677521301800715], ["counteract", 4.367671618620068], ["midst", 4.367478743632895], ["mitosis", 4.367347244143378], ["answers", 4.367338797443965], ["plunged", 4.3672666456619345], ["ali's", 4.366871238994153], ["calum's", 4.365033330223532], ["lizzy", 4.364713693915598], ["quiches", 4.364441220439183], ["wroc\u0142aw", 4.364125272530537], ["cultivates", 4.3640557191753295], ["euthyphro", 4.363878864915353], ["sidamo", 4.3634489261109275], ["ethnomusicologists", 4.3634489261109275], ["arkit", 4.3634489261109275], ["waited", 4.363054043855172], ["ejemplo", 4.3625087302638885], ["surabhi", 4.3625087302638885], ["brusqueness", 4.3625087302638885], ["understatement", 4.361562170645831], ["synchronized", 4.360118669112222], ["joined", 4.360102292865485], ["hardcoded", 4.359962699065199], ["yellows", 4.359607769277051], ["sarthe", 4.358824178971435], ["farmwork", 4.358824178971435], ["tofu", 4.35860064952636], ["dumitru", 4.358241058679935], ["spellbinding", 4.35796445033653], ["almighty's", 4.35776312147899], ["handholds", 4.357323981353479], ["kosice", 4.357323981353479], ["successfully", 4.356752964645258], ["sato", 4.356237623964135], ["nahuatl", 4.35584378637043], ["understandably", 4.355692310372851], ["thales", 4.3553136203789045], ["saito's", 4.354718536102703], ["robespierre's", 4.354383196618943], ["yunnan's", 4.354383196618943], ["mnemonics", 4.354065164979507], ["soundwave", 4.353852136181734], ["grounded", 4.353712396502766], ["herbalist", 4.353136152913712], ["iguazu", 4.352340590094938], ["caution", 4.352165013991937], ["gerne", 4.351802311586867], ["freely", 4.351178325645009], ["villager's", 4.35067466795306], ["remotely", 4.35066853613999], ["dwelling", 4.350630808766488], ["trekked", 4.3505566515508365], ["propel", 4.3497596745780305], ["skansen", 4.349473345611705], ["bucharest's", 4.349473345611705], ["oolongs", 4.3491892890974935], ["rockface", 4.3491892890974935], ["breadboards", 4.3491892890974935], ["gooey", 4.348913550260217], ["overflowing", 4.348627291638729], ["silverwood", 4.34845425626175], ["workwear", 4.348140786375918], ["multicultural", 4.348135822036501], ["sturdiest", 4.347967034496342], ["tenuta", 4.347856612741098], ["nocturnes", 4.347780234056641], ["aspect", 4.3477072855775365], ["picnicking", 4.347597811752245], ["haifa's", 4.347361130506028], ["vanishings", 4.347361130506028], ["nicotine's", 4.347361130506028], ["cymru's", 4.347361130506028], ["silverstone's", 4.347361130506028], ["chiseled", 4.347123096795547], ["effectively", 4.346435656280045], ["neglectful", 4.346367069271919], ["wares", 4.3461673792668325], ["newcomer", 4.345519278933876], ["flipped", 4.345112372743317], ["unharmed", 4.3448789897275235], ["fingertips", 4.344727402677161], ["agroecological", 4.344473066738965], ["plack", 4.344473066738965], ["tubman's", 4.344473066738965], ["regulars", 4.343119802573827], ["forward's", 4.343064809667088], ["lassi", 4.342059470590777], ["armrest", 4.342059470590777], ["hapoel", 4.342059470590777], ["ningyo", 4.342059470590777], ["unsuspectingly", 4.34035067111396], ["soir\u00e9es", 4.34035067111396], ["juntos", 4.34013905507968], ["playthings", 4.3395881789171336], ["crevices", 4.338900077330584], ["sensations", 4.338878474641626], ["splattering", 4.337660220873866], ["basil's", 4.337464698664225], ["wakatipu", 4.337275710615833], ["smartie", 4.337275710615833], ["playlists", 4.337257192835285], ["savories", 4.337016428232367], ["smartboards", 4.337016428232367], ["reinjury", 4.337016428232367], ["hacerlo", 4.337016428232367], ["bloodshed", 4.336969877072136], ["brusque", 4.336890352173406], ["geofencing", 4.336037196706462], ["churrascaria", 4.335389640374816], ["ojisan", 4.335389640374816], ["genevieve's", 4.335141691296155], ["perplexingly", 4.334586046372892], ["farzad", 4.333392685220336], ["squidward's", 4.3331557259484175], ["melcombe", 4.3331557259484175], ["euthydemus", 4.3331557259484175], ["tagliatelle", 4.332467292084317], ["proofreading", 4.332265351484181], ["zielona", 4.332213530748167], ["xun's", 4.332213530748167], ["reintroducing", 4.331721233756037], ["cloyingly", 4.330781394043787], ["eliud", 4.330781394043787], ["yields", 4.330069247457701], ["instinctively", 4.329514194431766], ["humanized", 4.32940040507885], ["revere's", 4.32940040507885], ["acrobatics", 4.3288147543612165], ["what'cha", 4.328758815013918], ["spotted", 4.328453016950215], ["neorealism", 4.328342891006601], ["esmeralda's", 4.327634023844627], ["pupusas", 4.32674356416894], ["panya", 4.32674356416894], ["thespis", 4.32674356416894], ["techie", 4.326355819702564], ["buzzo", 4.326154490845023], ["pinecone", 4.326154490845022], ["lillian's", 4.32597114813902], ["grapefruits", 4.32573593874645], ["lechon", 4.325224763427735], ["ways", 4.324507841376737], ["lengai", 4.323262676345794], ["camii", 4.323262676345794], ["overfit", 4.323262676345794], ["relic", 4.322443328904347], ["consoled", 4.322049979942893], ["karl's", 4.322049979942893], ["username", 4.3219943013602204], ["anns", 4.320934899968708], ["aqueducts", 4.320832672527052], ["proficiently", 4.3205463765994105], ["mandarake", 4.319906105944906], ["suisham", 4.319906105944906], ["reimagine", 4.3193261749332335], ["agnieszka", 4.31892985597746], ["jakob's", 4.3185039942584575], ["kimberl\u00e9", 4.3185039942584575], ["bcms", 4.3185039942584575], ["moby's", 4.31827237698872], ["selfless", 4.317867996397485], ["greenwash", 4.317734207450212], ["communicating", 4.316784324509239], ["trellises", 4.316722188181883], ["along", 4.31618803291184], ["alpacas", 4.315551171014521], ["hypatia", 4.3150901571088465], ["waterlogging", 4.3150901571088465], ["samsa", 4.3150901571088465], ["visigoth", 4.3150901571088465], ["propres", 4.3150901571088465], ["tasbih", 4.3150901571088465], ["irreparably", 4.314578081536404], ["materializing", 4.313073759839169], ["waters", 4.313068414956335], ["classroom", 4.312848452470489], ["bloating", 4.312521651062949], ["rugged", 4.312447435435994], ["creamier", 4.312372845926788], ["sigrid", 4.3120824269807505], ["meanings", 4.311927016621687], ["likening", 4.311398540054285], ["marlon's", 4.3112647225724015], ["malbork", 4.3112647225724015], ["orkney's", 4.3112647225724015], ["afterparties", 4.3112647225724015], ["imperfectly", 4.3102495110992685], ["tragically", 4.310022297102901], ["fragrances", 4.3097898296037265], ["armchairs", 4.30822524094981], ["adebayo", 4.30822524094981], ["lufthansa's", 4.307598681141643], ["souls", 4.307368942545316], ["capabilities", 4.306380532305302], ["ramblas", 4.30617468157763], ["demotivating", 4.30617468157763], ["uchicago", 4.305687226893276], ["tipton's", 4.305571575879929], ["jasperware", 4.305571575879929], ["subtexts", 4.30538397744954], ["bandai's", 4.305238353097182], ["reflect", 4.304993726771856], ["schoolbag", 4.304532774607023], ["obscurity", 4.304370708459067], ["trombonist", 4.304302431716074], ["retrace", 4.3040822740549975], ["d\u00e9jeuner", 4.304082274054997], ["jaideep", 4.304082274054997], ["clack", 4.303957860634733], ["labyrinths", 4.303769704536184], ["fray", 4.302438583873344], ["deity's", 4.301816669489104], ["cheesemakers", 4.3017307448577355], ["timepiece", 4.30094486876962], ["oolong", 4.300706523251817], ["crossbreeding", 4.300706523251817], ["comforted", 4.300706523251816], ["approaching", 4.299644758387708], ["crafts", 4.298858346834432], ["hawthorne's", 4.298745118634949], ["individualism", 4.298638875884869], ["hern\u00e1n", 4.29853005233924], ["huangshan", 4.297861603041765], ["mastery", 4.297371608143836], ["symbolize", 4.2969471644028205], ["oxymorons", 4.296201143775199], ["charminar", 4.296201143775199], ["volleyballs", 4.296201143775199], ["kline's", 4.296201143775199], ["appropriative", 4.295888704144366], ["programming", 4.295722962931481], ["trills", 4.2953706590790475], ["acropolis", 4.294699949895319], ["clickity", 4.294344531353516], ["onomastics", 4.294344531353516], ["traje", 4.294344531353516], ["ufologists", 4.294344531353516], ["nonvenomous", 4.294344531353516], ["homier", 4.294344531353516], ["shakespearean", 4.29384160709278], ["haunted", 4.293674847995799], ["threadbare", 4.293635015528426], ["protractor", 4.293514706323302], ["shortsightedness", 4.292996651714045], ["iterable", 4.292530275108752], ["nonprofits", 4.2924359337840245], ["sips", 4.2919026204406565], ["poe's", 4.291765345448798], ["samuel", 4.290354946756644], ["client's", 4.289885191237883], ["kinara", 4.289885191237883], ["viaje", 4.289124078482918], ["chaque", 4.289124078482918], ["musicality", 4.2890231626653454], ["camelids", 4.289023162665345], ["beowulf's", 4.289023162665345], ["informing", 4.288518286141387], ["torii", 4.288453798114348], ["w\u00e4re", 4.288453798114347], ["malgr\u00e9", 4.288453798114347], ["comfortably", 4.288150264696169], ["poignantly", 4.287982539455213], ["persevering", 4.28771345218459], ["smothered", 4.2877098826595725], ["pufferfish", 4.2868500382796], ["kartikeya", 4.2868500382796], ["alguna", 4.2868500382796], ["drina", 4.2868500382796], ["brainpower", 4.286676058976099], ["existed", 4.286073941885723], ["freya's", 4.286026554624357], ["summited", 4.286026554624357], ["surrendering", 4.286003253086853], ["k\u00f6nigsberg", 4.285973489382067], ["through", 4.285844575335705], ["abogados", 4.2856690878558314], ["besan", 4.2856690878558314], ["potlatches", 4.2856690878558314], ["hanae", 4.2856690878558314], ["maestro", 4.285483185495167], ["benevolence", 4.285482055077275], ["skewing", 4.285199167999707], ["numerous", 4.285136722273859], ["uphold", 4.284715051169077], ["migraines", 4.284165344316233], ["nibbled", 4.284088049087561], ["moth's", 4.284046343028926], ["glitzy", 4.283741211523013], ["collectible", 4.28272698093053], ["backdrops", 4.2807958449034285], ["danielle's", 4.2807958449034285], ["missives", 4.280030049582337], ["alain's", 4.280030049582337], ["guangzhou's", 4.280030049582337], ["eradicating", 4.279924736473664], ["hornbill", 4.279433612841871], ["learn", 4.278487696411316], ["staunch", 4.278459684493516], ["crossroads", 4.278386886111915], ["grace's", 4.278096278663609], ["finches", 4.278017295508317], ["georg's", 4.277891104030322], ["unquenched", 4.277891104030322], ["submenus", 4.277891104030322], ["javelinas", 4.277891104030322], ["manmade", 4.277076267629554], ["distractedly", 4.277076267629554], ["hera's", 4.276765343213471], ["tundras", 4.276765343213471], ["changer", 4.276583970637424], ["amun", 4.276381260111252], ["creature", 4.27626550541079], ["liza's", 4.2757122121267415], ["silhouettes", 4.2752321938137285], ["staying", 4.2751375882070235], ["cogwheels", 4.275000637068139], ["digitally", 4.274750949922044], ["kinship", 4.274056846077821], ["hypersexualized", 4.273797428332686], ["hoarder's", 4.273797428332686], ["lore", 4.273215549809579], ["sunburns", 4.27313789169251], ["oranges", 4.273096247233961], ["linguine", 4.272225023097832], ["usando", 4.272225023097831], ["undrip", 4.272225023097831], ["teaching", 4.272161174460123], ["sweeter", 4.272016679404214], ["roger's", 4.271168703723989], ["providing", 4.270648838100298], ["harmlessly", 4.270575751213022], ["scraps", 4.26914325324208], ["lilting", 4.269083681169083], ["butterflyfish", 4.268691338215203], ["grampy", 4.268691338215203], ["exfoliation", 4.268636890621604], ["vineyards", 4.268382603716123], ["corresponded", 4.267624165380648], ["defarge", 4.267272225403296], ["panes", 4.266920738956842], ["antivirus", 4.265499228036279], ["hummingbirds", 4.265067449198568], ["chlorophyll", 4.264798625918343], ["wrapping", 4.263706517738011], ["kristiania", 4.263430627058951], ["wistfulness", 4.263322434635792], ["cramming", 4.263322434635791], ["mable", 4.263291951849901], ["dragonborn", 4.263112934761853], ["ndes", 4.263112934761853], ["kemar", 4.261977723555811], ["drone's", 4.260869128956693], ["livraria", 4.259851804194049], ["excursions", 4.259818215770599], ["engulf", 4.259263088063101], ["pillowy", 4.259105953473702], ["souvlaki", 4.2587363677436185], ["fust", 4.258312655041625], ["outdoors", 4.257902595304472], ["dendrobium", 4.257640731940238], ["polished", 4.2574837035114435], ["athleisure", 4.2573698551918], ["flaws", 4.257075659322152], ["suquet", 4.256921049598801], ["anhinga", 4.256921049598801], ["dysbiosis", 4.256921049598801], ["hunnic", 4.256921049598801], ["rustbucket", 4.256921049598801], ["poaching", 4.25671402608271], ["charisma", 4.256521182259503], ["finances", 4.256436995244114], ["chanel's", 4.256327750497811], ["hilarious", 4.256285454013515], ["merchant's", 4.255849088850233], ["farmer", 4.25574619114499], ["heartbreakingly", 4.255102338335955], ["storting", 4.254467492797879], ["percussionist", 4.254074476252565], ["tejano", 4.253808810200059], ["c\u00e9leste", 4.253446012007291], ["gordonia", 4.253446012007291], ["seagulls", 4.252306289067311], ["cachoeira", 4.2514546983312576], ["elicited", 4.251266099127009], ["machinations", 4.250933774779409], ["ashfall", 4.250621678665985], ["bunraku", 4.250621678665985], ["janice's", 4.250363804752213], ["kitschy", 4.250164032059756], ["solstices", 4.249874688739443], ["wordings", 4.249874688739443], ["indoors", 4.249482897527929], ["quench", 4.249335078361666], ["tweaks", 4.2487953765922715], ["forestlands", 4.2485904800301135], ["prefecture's", 4.247929220811432], ["saleswoman", 4.24787867429524], ["mockingbird", 4.24787867429524], ["yaqub", 4.247526114982925], ["preoccupied", 4.247311234105357], ["notebooks", 4.246461186448358], ["amah", 4.246309357624728], ["marlow's", 4.24535961274492], ["hydrilla", 4.24535961274492], ["cohabitants", 4.24535961274492], ["punkt", 4.24535961274492], ["mahayana", 4.24506801228667], ["harissa", 4.24411827952482], ["inventiveness", 4.2437419444967075], ["financially", 4.243721728803575], ["selma's", 4.242904634034534], ["misapplying", 4.242904634034534], ["copenhagen's", 4.242812396977273], ["facet", 4.2426480806302305], ["seagal's", 4.242012017351358], ["unconsidered", 4.241824574816015], ["kerouac's", 4.241648758822198], ["aldridge's", 4.241600511116954], ["chapada", 4.241327932201003], ["pgcps", 4.241327932201003], ["assured", 4.241009832606088], ["expedition", 4.2409838940857245], ["marta", 4.240903799407783], ["muthu", 4.240429345774689], ["nook's", 4.239985739817395], ["polvo", 4.239985739817395], ["ashurbanipal", 4.239985739817395], ["aku's", 4.239985739817395], ["composition's", 4.238677102243377], ["bianco's", 4.238677102243377], ["tigger's", 4.238677102243377], ["boise's", 4.238677102243377], ["elo's", 4.238677102243377], ["tidepool", 4.238677102243377], ["catlins", 4.238677102243377], ["misplays", 4.238677102243377], ["hotu", 4.238677102243377], ["hanoi's", 4.238309193459222], ["olen", 4.238309193459222], ["fiddles", 4.238182449763245], ["coalville", 4.238162762102345], ["gentileschi", 4.237822702504969], ["rooftop", 4.237570474075151], ["incense", 4.2373431362001845], ["rembrandt's", 4.236155581064172], ["santoor", 4.236155581064172], ["matlock's", 4.236155581064172], ["grevy's", 4.236155581064172], ["subplots", 4.23604555104908], ["gritty", 4.235723320290594], ["apis", 4.235723320290594], ["resembling", 4.234866634286162], ["spick", 4.2347008573069385], ["feynman's", 4.234343709558157], ["aerators", 4.234343709558157], ["exteriors", 4.234324160418223], ["hesitations", 4.234008221092204], ["acciones", 4.233754132322072], ["supertree", 4.233754132322072], ["enqueue", 4.233754132322072], ["palden", 4.233754132322072], ["mnemonic", 4.233470795923724], ["elixirs", 4.23308886611649], ["imaginations", 4.232662128084983], ["vegetarian", 4.231990680409963], ["wobbled", 4.2319137697346685], ["linnea", 4.2319137697346685], ["surely", 4.231734466577456], ["entrepreneur", 4.231714400260513], ["hiligaynon", 4.231464378870303], ["govan's", 4.231464378870303], ["slugger's", 4.231464378870303], ["odissi", 4.231145936337276], ["romanesco", 4.231145936337276], ["trivialization", 4.229923904008014], ["impressionist", 4.229915884586817], ["ericsson's", 4.228788353966669], ["suboptimal", 4.227190172427], ["dujiangyan", 4.227190172426999], ["luchadores", 4.226180332443142], ["publication's", 4.226069496347352], ["hwaseong", 4.225585011928641], ["rehearsed", 4.225552036348838], ["asking", 4.2254414385062695], ["inasal", 4.225192445502412], ["pagespeed", 4.225192445502412], ["frigidly", 4.225192445502412], ["kelantanese", 4.225192445502412], ["conversely", 4.224953779011421], ["neckar", 4.224914124413847], ["voiced", 4.224904772825271], ["scalene", 4.224545699375747], ["flyer", 4.224530041311628], ["imitations", 4.224438811991939], ["submerging", 4.224203570674741], ["whitefoot", 4.22327972823419], ["chords", 4.2231043151125665], ["hashtags", 4.222399878290807], ["anica", 4.221897764972334], ["roran", 4.221897764972334], ["method's", 4.221626565501322], ["tucson's", 4.221626565501322], ["limba", 4.22144670751881], ["puddles", 4.2207473188030535], ["introduced", 4.220505435669857], ["emotionally", 4.220481805414235], ["tzatziki", 4.219789938795294], ["organisms", 4.219573482355411], ["humid", 4.218785811118308], ["ceviche", 4.217139108837669], ["sensibilities", 4.216031440605375], ["niccol\u00f2", 4.215714999959461], ["significa", 4.215591719020172], ["philosophical", 4.215076682802455], ["paradoxes", 4.214381407899174], ["albatros", 4.212980906642579], ["faires", 4.2127709818378225], ["unsellable", 4.2127709818378225], ["legibility", 4.212547492661067], ["kindly", 4.2120458041570865], ["estaban", 4.211874266005163], ["unsettlingly", 4.211874266005163], ["resubmitting", 4.211874266005163], ["wordsmithing", 4.211874266005163], ["overpromising", 4.211874266005163], ["nyasha", 4.211874266005163], ["fitzgerald's", 4.211576869763962], ["insensitively", 4.211471215836166], ["swifter", 4.210703049631728], ["dubai's", 4.210621663992968], ["ghazals", 4.209708777950661], ["breathable", 4.209612308295176], ["riga's", 4.209139282715549], ["rubbing", 4.209052914822193], ["ppps", 4.20890731031088], ["overdoing", 4.208662866271552], ["unintuitive", 4.207843764315205], ["grammophon", 4.207843764315205], ["pierogies", 4.207843764315205], ["gubbio", 4.207685013701596], ["enunciating", 4.207383865548681], ["escape's", 4.207212903181125], ["resourcefully", 4.20659291896315], ["brabham's", 4.20659291896315], ["dense", 4.206179314831974], ["jaime's", 4.205897326913714], ["caffeinated", 4.204941803098436], ["maximilien", 4.20487902028365], ["unseasonal", 4.204112044142355], ["cedarwood", 4.204112044142355], ["panels", 4.203760831055438], ["universe's", 4.2033304313174025], ["wabi", 4.203309449914004], ["scarves", 4.203043179536401], ["sassanids", 4.202809749959585], ["comida", 4.202317338983433], ["frivolous", 4.20230643771634], ["honeycombs", 4.2022630696675725], ["firstname", 4.2022630696675725], ["gretchen's", 4.20199303605559], ["memes", 4.200782497475469], ["reapplying", 4.200674887791567], ["minding", 4.200143674041164], ["creative", 4.200077554664423], ["village's", 4.199541107223464], ["froggies", 4.198914652878993], ["camarones", 4.198914652878993], ["historian's", 4.198123797532519], ["probabilities", 4.197466751274551], ["yyyy", 4.19713091683274], ["belting", 4.197079462908329], ["rathaus", 4.197015550819798], ["ripe", 4.196793988912892], ["swindon's", 4.196786105403724], ["strengthener", 4.196786105403724], ["sleekness", 4.196786105403724], ["kinder", 4.196290331220943], ["no\u00e9mie", 4.196107891747389], ["camouflaging", 4.195916855649426], ["plastered", 4.195817921723197], ["rockslide", 4.195444475479919], ["weatherstripping", 4.195444475479919], ["dollops", 4.19505332831644], ["shostakovich's", 4.19505332831644], ["kaew", 4.19505332831644], ["tomek", 4.19505332831644], ["zlata", 4.194795377549156], ["albatrosses", 4.194370372951634], ["sabotaging", 4.193773649324809], ["aria's", 4.193188860170522], ["organism's", 4.193188860170522], ["cybercrime", 4.193188860170521], ["freelancing", 4.19301852766432], ["cybertruck", 4.192333281949715], ["polyhedral", 4.191685116630923], ["chocolatiers", 4.191461725188283], ["dojang", 4.191177095164842], ["grandiosely", 4.191177095164842], ["balboa's", 4.191177095164842], ["immeasurably", 4.190547106123398], ["appealing", 4.190431997014591], ["birding", 4.190179652560959], ["ornithologists", 4.190066849576719], ["ultrarunner", 4.190066849576719], ["merisi", 4.190066849576719], ["sepak", 4.190066849576719], ["reroute", 4.190002913254913], ["jigsaw", 4.1898454303856685], ["wise", 4.189359883565062], ["territories", 4.18858096032418], ["comprehensiveness", 4.1884818044284025], ["prepackaged", 4.18831133304849], ["clapton's", 4.187862039835333], ["timbales", 4.186985918069677], ["unhurt", 4.186517417911004], ["ableism", 4.185985443537283], ["agroecology", 4.185637452395581], ["powerful", 4.185601645679941], ["toppling", 4.185251314573915], ["zhen's", 4.185117562015206], ["cheesemongers", 4.185117562015206], ["pteropods", 4.185117562015206], ["bonaire", 4.1850229617086505], ["hone", 4.184471211812578], ["broca's", 4.183717388859631], ["coves", 4.183472924970494], ["tipoff", 4.18337955638314], ["petrov", 4.183122875836056], ["reena", 4.182926400987973], ["brochures", 4.182850623335781], ["allier", 4.182555245140503], ["logfile", 4.181758719670089], ["uikit", 4.181758719670089], ["leucistic", 4.181758719670089], ["mayan", 4.181456163483654], ["scowl", 4.181242453161489], ["conducting", 4.179934710382393], ["unhackable", 4.178824152147515], ["farabi", 4.178824152147515], ["altes", 4.178824152147515], ["taft's", 4.177817044784368], ["upselling", 4.177491618935733], ["mentioning", 4.177383498803246], ["sunscreen", 4.176011482045107], ["leipzig's", 4.175443226458074], ["stavropol", 4.175057183260362], ["mythical", 4.17469281463346], ["glorifying", 4.173960722691519], ["kolaches", 4.173942348231455], ["digitalizing", 4.173942348231455], ["sortino", 4.173942348231455], ["darkened", 4.173890818079006], ["mostrar", 4.172888322385941], ["nuka", 4.172888322385941], ["hiccoughs", 4.172888322385941], ["lilyan", 4.172888322385941], ["egos", 4.172709305297894], ["batik", 4.172238289155566], ["huck's", 4.171890262514526], ["importante", 4.171751992255262], ["scavenger", 4.171623491371326], ["meetings", 4.171622339621229], ["piercing", 4.171311817062168], ["prodigy", 4.171304322870915], ["prepping", 4.170348887931652], ["rarity", 4.17032736177265], ["xvi's", 4.169827466538115], ["bioengineer", 4.169470723432307], ["taxonomical", 4.169470723432307], ["tilly", 4.167579468902283], ["sacrilege", 4.16746865173407], ["typists", 4.167104693387485], ["shackled", 4.166732647077643], ["origins", 4.166572599519627], ["roboto", 4.165881233407043], ["misalignments", 4.165045055722567], ["nuna", 4.164832553423807], ["silliness", 4.164114454083584], ["pramod", 4.164062001610036], ["travelogues", 4.163853399185315], ["purifiers", 4.163618670429512], ["eritrea's", 4.163048389828165], ["loanwords", 4.162863365455284], ["reverberates", 4.162264047377908], ["balancers", 4.16210377593096], ["yazd", 4.16210377593096], ["ponchos", 4.161974054759823], ["atossa", 4.161974054759822], ["coup\u00e9s", 4.161974054759822], ["maestros", 4.1616102458389514], ["weisswurst", 4.161467485376332], ["robotron", 4.161467485376332], ["sping", 4.161467485376332], ["bagpiping", 4.161467485376332], ["doors", 4.161405485371384], ["alight", 4.160933561246133], ["robustness", 4.160375663495051], ["rivalrous", 4.1602814090252584], ["tili", 4.1602814090252584], ["colorblindness", 4.160056977811522], ["feyenoord's", 4.1596199970365415], ["julian's", 4.159354630529554], ["swimmingly", 4.159294212262338], ["fudgy", 4.158396939113274], ["pinpoint", 4.158186544030044], ["interlopers", 4.158110153117473], ["orion", 4.158001261343916], ["distressed", 4.157638665119448], ["acquiring", 4.157445757985627], ["lavender", 4.157099015220157], ["lighten", 4.157078875797751], ["nizam's", 4.155971207565457], ["anton\u00edn", 4.155971207565457], ["skipped", 4.155564012241551], ["culprit", 4.155229820063364], ["dismally", 4.154986613581197], ["offstage", 4.15459954960704], ["hemos", 4.153755871100871], ["automation", 4.152530589918396], ["tomos", 4.152036817838877], ["lilys", 4.151770011720265], ["ethel", 4.1513334938452005], ["dipped", 4.15114471082993], ["hexadecimal", 4.151018288017255], ["scowling", 4.150753779630168], ["glasswork", 4.150667576772914], ["verde's", 4.150667576772914], ["gendo", 4.150667576772914], ["owusu", 4.150554124057956], ["passau", 4.150554124057956], ["crash's", 4.150332169291962], ["cryptograms", 4.150332169291962], ["disapproval", 4.149800741601267], ["danceable", 4.149554370909413], ["viii's", 4.149554370909413], ["sigh", 4.1494967562144085], ["gado", 4.149336481719054], ["sue\u00f1os", 4.148987762816163], ["bemused", 4.1489651297105095], ["connectivity", 4.148623369815146], ["words", 4.148423133076453], ["bulkiness", 4.147212009786167], ["tatay", 4.147212009786167], ["hallways", 4.1463221865197], ["settled", 4.146091612510295], ["himba", 4.145925311618075], ["capriciousness", 4.144757650907182], ["suomen", 4.144757650907182], ["raucous", 4.144053300654607], ["dimitris", 4.14356532336377], ["combing", 4.143565323363769], ["follower", 4.143496971384697], ["wineries", 4.14345649981814], ["observation", 4.1431977416096455], ["printout", 4.142736138301446], ["oceans", 4.142702305332348], ["vinyl", 4.142076617259566], ["writhing", 4.14146681558836], ["clunky", 4.141270593338024], ["downfalls", 4.140998588701351], ["timeslots", 4.14050291366159], ["frameworks", 4.140384327414619], ["sophistication", 4.139296632025554], ["powerlifter", 4.139054092744974], ["bruno's", 4.1389063450172845], ["gemstones", 4.1373520944601045], ["headspace", 4.136382969352266], ["subatomic", 4.136382969352266], ["lapping", 4.135892319840182], ["twinkles", 4.135294733895978], ["ainsi", 4.135294733895978], ["grutas", 4.135294733895978], ["cammy's", 4.135294733895978], ["mishandled", 4.135294733895978], ["siendo", 4.135294733895978], ["rom\u00e2nia", 4.135294733895978], ["bringen", 4.135294733895978], ["osmoregulation", 4.135294733895978], ["radley's", 4.135294733895978], ["opel's", 4.135294733895978], ["k\u00fcche", 4.135294733895978], ["luc's", 4.135294733895978], ["stonewalls", 4.135294733895978], ["arepa", 4.135294733895978], ["castellers", 4.135294733895978], ["herd's", 4.135294733895978], ["cpi's", 4.135294733895978], ["novotna", 4.135294733895978], ["propia", 4.135294733895978], ["cello's", 4.135294733895978], ["bogleheads", 4.135294733895978], ["gonz\u00e1les", 4.135294733895978], ["fritto", 4.135294733895978], ["jet\u00e9", 4.135294733895978], ["alexia's", 4.135294733895978], ["stasi's", 4.135294733895978], ["anguilla's", 4.135294733895978], ["atlatl", 4.135294733895978], ["donnison", 4.135294733895978], ["blenny", 4.135294733895978], ["guimard", 4.135294733895978], ["seracs", 4.135294733895978], ["laureate's", 4.135294733895978], ["r\u00fcdesheim", 4.135294733895978], ["klinefelter's", 4.135294733895978], ["pitka", 4.135294733895978], ["annika's", 4.135294733895978], ["ahmir", 4.135294733895978], ["bombilla", 4.135294733895978], ["catio", 4.135294733895978], ["mabuza", 4.135294733895978], ["time2", 4.135294733895978], ["condiciones", 4.135294733895978], ["pfennigs", 4.135294733895978], ["vigo's", 4.135294733895978], ["ugadi", 4.135294733895978], ["algis", 4.135294733895978], ["tannerite", 4.135294733895978], ["jarabe", 4.135294733895978], ["p\u00e2tisserie", 4.135294733895977], ["kermanshah", 4.135294733895977], ["untraveled", 4.135294733895977], ["silkier", 4.135294733895977], ["purchasing", 4.13407212501813], ["gamer's", 4.132971534607272], ["resurfacing", 4.132931708333752], ["recommendations", 4.1328135570556395], ["simulations", 4.132614450272157], ["raised", 4.131966180186757], ["their", 4.1315895871096355], ["roasters", 4.131465757290519], ["bask", 4.1314072652859695], ["rickety", 4.130978142315711], ["excitingly", 4.13096458234216], ["silenced", 4.130399895231366], ["shelter's", 4.130324427725429], ["thunberg", 4.130324427725429], ["boules", 4.130324427725429], ["yeasts", 4.130060183599906], ["rushing", 4.129586077866021], ["batch", 4.129190977831186], ["sanremo", 4.1290386147826705], ["success", 4.12893716773117], ["aphids", 4.128515562201066], ["geodes", 4.127936558212888], ["pragmatism", 4.127316673316114], ["yippee", 4.127249413402017], ["seedlings", 4.126836176485735], ["demystify", 4.126496234462157], ["remarked", 4.126152437054974], ["restorative", 4.125772673653454], ["prototyping", 4.125729105507962], ["helping", 4.125686860061503], ["relented", 4.125621529840081], ["analogies", 4.125576062370958], ["neti", 4.125576062370957], ["tuks", 4.124878374364754], ["pieces", 4.12477927014407], ["rafael's", 4.124636757777689], ["effectuate", 4.124636757777689], ["ovations", 4.124469355011433], ["efficiency", 4.123767599825243], ["unprecedented", 4.123595791154093], ["chandra's", 4.123343015011885], ["andean", 4.123087221766395], ["auditorium", 4.1226591110979625], ["loyalty", 4.122609437887945], ["hao's", 4.122208358155801], ["beiderbecke", 4.122208358155801], ["astrid's", 4.121276785645483], ["quadraphonic", 4.121276785645483], ["irreconcilable", 4.120618322523069], ["redesigning", 4.120419572982683], ["outdoorsman", 4.1199788274741405], ["constant", 4.119969117101193], ["\u00e9ireann", 4.118949695422081], ["herons", 4.118610547649712], ["xpath", 4.118552649953079], ["punter's", 4.117236678289445], ["hosseini", 4.116497939650996], ["slinking", 4.116497939650995], ["multithreading", 4.115970926728239], ["homages", 4.115122564462339], ["senso", 4.114721128254704], ["countertops", 4.114618260226498], ["hinduism", 4.114618260226497], ["walkathon", 4.114514358348762], ["phytoremediation", 4.114514358348762], ["jam\u00f3n", 4.1140335784517825], ["glowworm", 4.1140335784517825], ["bouzouki", 4.1140335784517825], ["juho", 4.11353002477021], ["sistine", 4.113530024770209], ["epigenetics", 4.112229344941976], ["traits", 4.112169730449849], ["inhabitants", 4.112073463467177], ["gr\u00fcne", 4.112062741008921], ["elm's", 4.112062741008921], ["donegal's", 4.112062741008921], ["ironheart", 4.111460182115597], ["vinny's", 4.111111723756235], ["congratulated", 4.110789283621038], ["creation", 4.110653845440283], ["purees", 4.110532489980433], ["prut", 4.110383319836363], ["vanger", 4.110383319836363], ["vitebsk", 4.109689193748015], ["jalopy", 4.108649793600256], ["lian's", 4.107908676055607], ["nzrl", 4.107908676055607], ["typescript", 4.107655231818524], ["doge's", 4.1075410779637895], ["taj's", 4.107353553261545], ["bar\u0131\u015f", 4.107353553261545], ["umar's", 4.107353553261545], ["bhas", 4.107353553261545], ["appending", 4.107099542528505], ["suggestively", 4.106775459869109], ["numai", 4.106775459869109], ["niilo", 4.106775459869109], ["baptiste's", 4.106775459869109], ["bestowed", 4.1066008520689445], ["pried", 4.1066008520689445], ["orisha", 4.105544412069388], ["mischief", 4.105318361589133], ["fusion", 4.105284309255704], ["checkmarks", 4.105219863104007], ["berghain", 4.105219863104007], ["indecision", 4.105104976076364], ["dmitri", 4.104566473278866], ["lysandra", 4.104549048142104], ["foretelling", 4.104376642427597], ["cosplaying", 4.104280023391758], ["membres", 4.103847625653422], ["kareem's", 4.103113451920134], ["\u00e9poque", 4.102604656948183], ["chillier", 4.102604656948183], ["hoarded", 4.1025787945455345], ["juvy", 4.102344178247404], ["colca", 4.101537225864174], ["soundcheck", 4.101239365499188], ["tamriel", 4.101118744359646], ["clanked", 4.101118744359646], ["criticized", 4.10101314055198], ["roles", 4.100889081709963], ["sanitizer", 4.100627592414215], ["historians", 4.10051071162599], ["atalanta's", 4.099335649253404], ["muddiest", 4.099335649253404], ["aesthetician", 4.099099593993168], ["nudibranch", 4.099099593993168], ["gibi", 4.098699205277429], ["stanis\u0142aw", 4.098699205277429], ["oleg's", 4.0983724594861926], ["farooq's", 4.0983724594861926], ["exup\u00e9ry's", 4.0983724594861926], ["gamemasters", 4.0983724594861926], ["servals", 4.0983724594861926], ["amazfit", 4.0983724594861926], ["madhavrao", 4.0983724594861926], ["workload", 4.098341036273928], ["dominique's", 4.097004967841386], ["rehabilitating", 4.096685102473853], ["reeled", 4.096685102473853], ["artists", 4.09598292562439], ["philippos", 4.09572253548549], ["interactive", 4.095667822619861], ["alliances", 4.095609244111008], ["overwater", 4.095312602490906], ["senecio", 4.095312602490906], ["streets", 4.095217665238592], ["archways", 4.094950395028699], ["assigning", 4.094591309290091], ["couture's", 4.093941786557018], ["symlinks", 4.093941786557018], ["dortmunder", 4.093941786557018], ["sugriva", 4.093941786557018], ["brentford's", 4.093941786557018], ["putri", 4.09352408001824], ["carlitos", 4.092662829422824], ["gazprom's", 4.092218747084561], ["hardhats", 4.092218747084561], ["tjuta", 4.092218747084561], ["downvotes", 4.092128818093306], ["fedex's", 4.091993218357801], ["unaware", 4.091430473965583], ["confirming", 4.0912909565993925], ["hesitate", 4.0909257693280825], ["birthdate", 4.090713489552861], ["somsak", 4.089851934622395], ["wiremu", 4.089851934622395], ["scenic", 4.089170292633292], ["potala", 4.089090323461386], ["seabird", 4.088772668139648], ["papermaking", 4.088302748283524], ["graphs", 4.087651479074472], ["swig", 4.087540716084403], ["francesca", 4.087252339193363], ["confounding", 4.086968064668495], ["scarce", 4.0869310903251055], ["hurston", 4.086644207614849], ["simplement", 4.086644207614849], ["onload", 4.086644207614849], ["spiderwebs", 4.086148616515211], ["shiva's", 4.086065034682931], ["soundtracks", 4.084926251466744], ["lamenting", 4.084669347334107], ["mimic", 4.083503178490873], ["scorekeeping", 4.082949230935269], ["audible", 4.082943662264741], ["tapped", 4.082801808906523], ["hopped", 4.08238728656525], ["digesta", 4.082278134743465], ["harmonizing", 4.08214184528549], ["disposability", 4.08214184528549], ["dana's", 4.081936092168288], ["ufologist", 4.081936092168288], ["ammit", 4.081936092168288], ["cremini", 4.081936092168288], ["liam", 4.080967360071116], ["wrapped", 4.080656052411948], ["thermoregulation", 4.079600528725662], ["vineyard's", 4.079222940893999], ["friendship's", 4.079222940893999], ["customs", 4.079214801440175], ["moral", 4.078822115186211], ["sephora's", 4.0786468608289095], ["customization", 4.0779117114649495], ["elements", 4.077846328011914], ["tryouts", 4.0776234285738155], ["town", 4.077364958528901], ["leap", 4.077277165987587], ["leveraged", 4.07663125046164], ["merfolk", 4.0762190948403205], ["reconnection", 4.074481576044566], ["sylvie's", 4.074481576044566], ["raan", 4.074481576044566], ["tzotzil", 4.074481576044566], ["eno's", 4.0735739169721565], ["aloft", 4.073265312887538], ["kijiji", 4.073265312887538], ["inventor's", 4.072733542762907], ["unshakable", 4.072266078100537], ["veneer", 4.071886881309573], ["caprine", 4.071674814912963], ["dell'accademia", 4.071674814912963], ["cumberland's", 4.071674814912963], ["sunbeams", 4.070967926924262], ["clich\u00e9d", 4.070812171943702], ["seascape", 4.070680753678853], ["came", 4.070358828835645], ["wiry", 4.070202131603171], ["fa\u00e7ades", 4.069593789525667], ["asunci\u00f3n", 4.069560280033469], ["juicy", 4.068931920202317], ["pawel", 4.068596431736365], ["twilit", 4.068596431736364], ["returned", 4.068097758499947], ["seaborn", 4.067264033220838], ["mercat", 4.066942754823317], ["whippersnappers", 4.0662292686117025], ["catastrophic", 4.0661917824216625], ["roadwork", 4.065142412517385], ["befell", 4.064982806143855], ["nourished", 4.064945785020528], ["ciara's", 4.06440396702919], ["spirit", 4.064264177024293], ["spun", 4.064252491031611], ["engulfed", 4.064252491031611], ["plastics", 4.0642329586003445], ["taycan", 4.063996548828805], ["repellants", 4.063996548828805], ["clairo", 4.063996548828805], ["fell", 4.0636104510300095], ["fluorescent", 4.063192159048561], ["gilda's", 4.063062511469847], ["neytiri", 4.063062511469847], ["hanafi", 4.062927076052799], ["calientes", 4.062745703476749], ["immensely", 4.062593584542], ["antarctica's", 4.0623802171774095], ["gerunds", 4.0621036766588805], ["fermented", 4.061900098883784], ["lucrative", 4.0618163563928675], ["castle's", 4.0616922983512245], ["jill", 4.061294722868365], ["overripe", 4.06107149508246], ["coadjutor", 4.06107149508246], ["planks", 4.06085942868585], ["teased", 4.060701139892209], ["handeln", 4.060107556916051], ["galego", 4.060107556916051], ["ymir's", 4.060107556916051], ["warao", 4.060107556916051], ["cemile", 4.060107556916051], ["baystars", 4.060107556916051], ["ratu", 4.0601075569160505], ["manuel's", 4.0601075569160505], ["montenegro's", 4.0601075569160505], ["frustum", 4.059591397806349], ["rebirth", 4.059343703126015], ["horchata", 4.058997783086089], ["permeate", 4.058786939053406], ["engravings", 4.058670155003199], ["tenets", 4.058640490048149], ["amneris", 4.057999505225025], ["tetrarchy", 4.057999505225025], ["shikumen", 4.057999505225025], ["essling", 4.057999505225025], ["levente", 4.057999505225025], ["shanghai's", 4.057453891846171], ["ravaged", 4.057059428119567], ["reactions", 4.056652835766887], ["sleuth", 4.055881579755586], ["asturian", 4.055769835167208], ["laksa", 4.055514128097341], ["hooked", 4.055471616796976], ["expression", 4.055431126302763], ["commute", 4.0553229183809645], ["shaped", 4.055070016058396], ["soryu", 4.054997748771784], ["undertone", 4.054968864964185], ["rumored", 4.054951864780285], ["rajasthan's", 4.054841528956367], ["flamb\u00e9", 4.054841528956367], ["adobo", 4.0543867934501865], ["chromatophores", 4.054011299323084], ["hank's", 4.053818666648213], ["conserve", 4.0536491712011085], ["zephyr", 4.05293928792432], ["segued", 4.052686025933308], ["zofia", 4.052588839218058], ["ballads", 4.052018453737521], ["taino", 4.051753426140504], ["addressed", 4.050930177044572], ["nebulizers", 4.050900963816468], ["bujo", 4.050900963816468], ["corvin", 4.050900963816468], ["copland's", 4.050900963816468], ["heartaches", 4.050322924295376], ["vectorization", 4.050250112119197], ["huichol", 4.050250112119197], ["sentimentality", 4.050156312904002], ["yusuf", 4.0496350572652755], ["imbalanced", 4.0492068137851325], ["bowlful", 4.049142760273145], ["acum", 4.049142760273145], ["clts", 4.049142760273145], ["addie's", 4.049142760273145], ["heliocentric", 4.0486770739292375], ["panelists", 4.04831262374705], ["lia's", 4.048235897392905], ["foreshadowing", 4.047554567368098], ["condiments", 4.047554567368098], ["untaken", 4.0473097395577655], ["adowa", 4.0473097395577655], ["preeya", 4.0473097395577655], ["fanworks", 4.0473097395577655], ["tytus", 4.0473097395577655], ["overshadows", 4.046810261423968], ["subgenre", 4.046573865059664], ["surrealist", 4.046260808130602], ["lettuces", 4.045397022289543], ["malacology", 4.045397022289543], ["saxy", 4.045397022289543], ["obasan", 4.045397022289543], ["cicotte", 4.045397022289543], ["cartographer", 4.045166515336706], ["pressing", 4.044421632118613], ["enhancements", 4.044318249750266], ["mumtaz", 4.0440749971188605], ["nikias", 4.043690103714738], ["hugged", 4.043672590349434], ["gaman", 4.043399295364956], ["gurmukhi", 4.042989047871514], ["regularization", 4.0421369953851345], ["seti's", 4.041310762671069], ["repente", 4.041310762671069], ["wiggler", 4.040773711406927], ["ambitions", 4.040287047103494], ["bandmate", 4.03998310805897], ["unpolished", 4.039864855421455], ["paleontology", 4.039452408620917], ["rave", 4.039220968023892], ["isoroku", 4.039125088921653], ["bamboo's", 4.039125088921653], ["mpemba", 4.039125088921653], ["lohri", 4.039125088921653], ["gotovina", 4.039125088921653], ["cooking", 4.038467247626775], ["oozing", 4.038307151219552], ["gheorghe", 4.0381080186457785], ["sasb", 4.036835335469883], ["parrotlet", 4.036835335469883], ["fujianese", 4.036835335469883], ["prudencio", 4.036835335469883], ["steadily", 4.0363942601301135], ["chiptunes", 4.036364237869758], ["labneh", 4.036364237869758], ["skylink", 4.036364237869758], ["litterbugs", 4.036364237869758], ["marleen", 4.0362457582337985], ["flicks", 4.036047660282474], ["charly's", 4.03564907765752], ["loamy", 4.03564907765752], ["influential", 4.035543356046633], ["kaaba", 4.0352472806565585], ["rust's", 4.0352472806565585], ["fangirling", 4.035196828083064], ["offspring's", 4.035196828083064], ["initiated", 4.034896380684092], ["jests", 4.034842230238586], ["pros", 4.034842230238585], ["caffeine", 4.0348282785019665], ["ezekiel's", 4.034609296896772], ["overexcitement", 4.034433886727784], ["protecci\u00f3n", 4.034433886727784], ["lidars", 4.034433886727784], ["interactivity", 4.033470893714619], ["oeuvre", 4.033420374163046], ["everyday", 4.032879603977863], ["courageously", 4.032606206946882], ["slung", 4.031912365548578], ["breeze", 4.031912365548578], ["saltville", 4.031912365548578], ["dente", 4.031912365548578], ["sklodowska", 4.031912365548578], ["geographica", 4.031912365548578], ["ayran", 4.031912365548578], ["uncelebrated", 4.031912365548578], ["marlin's", 4.031912365548578], ["usherette", 4.031912365548578], ["cusi", 4.031912365548578], ["yasuni", 4.031912365548578], ["diff\u00e9rentes", 4.031912365548578], ["miri's", 4.031912365548578], ["replicate", 4.03068925020475], ["goethe's", 4.030160122017267], ["scones", 4.030119607715965], ["pizzas", 4.0299296899912305], ["patter", 4.029279537831751], ["tenemos", 4.029261535590953], ["misdelivered", 4.029261535590953], ["crates", 4.027690467783606], ["woreda", 4.027040945050533], ["formations", 4.027003562447192], ["ecologist", 4.026610705633327], ["trends", 4.0257984608941575], ["telemundo's", 4.025750502534494], ["vlora", 4.025750502534494], ["textfield", 4.025750502534494], ["timor's", 4.025508148040332], ["skillful", 4.025020207658752], ["bouchard", 4.024796979872846], ["lifestyle", 4.024277975270617], ["mateja", 4.023530011358249], ["waheguru", 4.023530011358249], ["corinthos", 4.023530011358249], ["absurdism", 4.023530011358249], ["mon\u00e1e's", 4.023530011358249], ["curieux", 4.023530011358249], ["oslofjord", 4.023530011358249], ["yoshida", 4.023151147892018], ["epidemiologist", 4.022878954722107], ["brigadeiro", 4.021998987761841], ["roshan's", 4.021998987761841], ["bloodstreams", 4.021998987761841], ["quoi", 4.021687735712023], ["twizy", 4.021687735712022], ["bolognese", 4.021547552269533], ["embodiments", 4.021217637788503], ["lacemaking", 4.021217637788502], ["kvas", 4.021217637788502], ["stunden", 4.021217637788502], ["subfolders", 4.02100266838629], ["ripeness", 4.020965997135324], ["acad\u00e9mie", 4.020702229221752], ["selfishly", 4.019847146359294], ["proposing", 4.0192533000366515], ["mulia", 4.0188075582932745], ["skink", 4.0184277088076135], ["angelenos", 4.018100146295687], ["sodas", 4.017797429648749], ["sisyphean", 4.017563922966555], ["coverups", 4.017563922966555], ["deserialize", 4.017143455784664], ["aouda", 4.017143455784664], ["wochen", 4.017143455784664], ["theories", 4.017114985597168], ["prepared", 4.016980192200365], ["takeaway", 4.016182416680409], ["linkedin's", 4.015623825198023], ["lifeforms", 4.015237790008675], ["pakoras", 4.014849256209687], ["icefields", 4.014849256209687], ["fintech", 4.014777166374976], ["mughal", 4.014777166374976], ["unseen", 4.014690437848446], ["congratulating", 4.014544127666215], ["fraulein", 4.01405049038908], ["explanations", 4.013681109634793], ["tabletops", 4.013668418193156], ["replanting", 4.013668418193155], ["thaipusam", 4.013668418193155], ["golkonda", 4.013668418193155], ["concr\u00e8te", 4.013668418193155], ["ghagra", 4.013668418193155], ["komarov's", 4.013668418193155], ["neuschwanstein", 4.0127280085356585], ["ellington's", 4.012637686704148], ["precomputed", 4.012311990880484], ["of's", 4.012311990880484], ["revelatory", 4.01211574182248], ["collide", 4.011872091068929], ["nanook", 4.011853100048336], ["twitched", 4.011622368022453], ["asteraceae", 4.01077714285949], ["musclebound", 4.010361358249574], ["doudna", 4.010361358249574], ["kairo", 4.010361358249574], ["contributes", 4.01025845857836], ["mufasa's", 4.009506300925758], ["yasna", 4.009506300925758], ["watermelons", 4.009250259752175], ["babilonia", 4.0088883446502495], ["sterile", 4.008622508590019], ["vibrated", 4.008567959792714], ["giocondo", 4.0080548959299485], ["voc's", 4.0080548959299485], ["dogtraining", 4.0080548959299485], ["cr\u00e9cy", 4.0080548959299485], ["tenacious", 4.007100597145202], ["correctly", 4.006947438080984], ["adriene", 4.006569605992601], ["sway", 4.0065177703706265], ["healthcare", 4.005502551044646], ["dobro", 4.005328327973532], ["patchi", 4.0044309764942065], ["lullabies", 4.004430976494206], ["junkanoo", 4.0042046991038855], ["apologetic", 4.003659456286802], ["pointedly", 4.003612164157391], ["canai", 4.003492511301683], ["joycean", 4.003317242388659], ["polyphasic", 4.003317242388659], ["filtering", 4.002350134599653], ["immunotherapy", 4.002246120718455], ["critiques", 4.0019671278203655], ["seg\u00fan", 4.001898129576753], ["silkiness", 4.001898129576752], ["pickiness", 4.001898129576752], ["phototropism", 4.001898129576752], ["communicated", 4.001721054438275], ["defy", 4.001487288323906], ["copd", 4.000946806725031], ["wrinkled", 4.0008976550443585], ["impressive", 4.0007504665877285], ["salty's", 4.000264701768762], ["stealthier", 4.000264701768762], ["almer\u00eda", 4.000264701768762], ["swallowtails", 4.000264701768762], ["retirees", 3.99979784261513], ["farewells", 3.9993206358661784], ["melville's", 3.999265301859926], ["quivering", 3.99920594356231], ["droplet", 3.99920594356231], ["misspoken", 3.9985907757506562], ["residents", 3.9976187542999764], ["quilter", 3.9974515760994453], ["mahler's", 3.997451576099445], ["threatened", 3.9973492471176746], ["debugged", 3.995489395917288], ["phenomena", 3.995145836005394], ["feng's", 3.995115251391029], ["springtails", 3.995115251391029], ["poblano", 3.994464263082183], ["cattails", 3.994464263082183], ["reaps", 3.9936750512283075], ["forties", 3.993566905288815], ["duchamp's", 3.9935132001624014], ["erosions", 3.9935132001624014], ["needless", 3.9935002733267218], ["mancala", 3.993048723393413], ["halakha", 3.993048723393413], ["verschiedenen", 3.992698363761633], ["rsis", 3.992698363761633], ["anglophiles", 3.992698363761633], ["belfort's", 3.992698363761633], ["glassmaking", 3.9924390976341164], ["dumpling", 3.992149916184194], ["confusedly", 3.991458395325683], ["timings", 3.9914583953256826], ["\u017ei\u017ekov", 3.9914583953256826], ["dvaita", 3.9914583953256826], ["jerrycans", 3.9914583953256826], ["kulen", 3.9914583953256826], ["sabin's", 3.9914583953256826], ["wielding", 3.991251931681802], ["lehenga", 3.9905141742847396], ["squealer", 3.9905141742847396], ["mcgillicuddy", 3.9905141742847396], ["matters", 3.9897796197740836], ["cutouts", 3.9896558191032403], ["glaring", 3.9893431550525897], ["streaked", 3.9893008502471283], ["cheers", 3.9890890911931223], ["cear\u00e1", 3.988530620166893], ["reciprocals", 3.988530620166893], ["slumber", 3.987942143061583], ["ornithology", 3.9878733506332207], ["vicario", 3.987605636256836], ["prim's", 3.987605636256836], ["topeka's", 3.987605636256836], ["hatillo", 3.987605636256836], ["plucking", 3.987360847697213], ["gujarati", 3.987174776652702], ["ceramicist", 3.986721270402709], ["purposefulness", 3.986466645786611], ["usmca", 3.986466645786611], ["gaius", 3.9858823448411895], ["inherently", 3.9858288111232847], ["meal", 3.985780551116474], ["yelena", 3.9846408737019976], ["fenugreek", 3.9840034631436856], ["hefty", 3.983968227913234], ["needed", 3.983233647737319], ["mend", 3.9832163714858786], ["smoky", 3.9832163714858786], ["workplace", 3.983191074865586], ["nagged", 3.982941770015599], ["halloweens", 3.9829417700155987], ["lull", 3.9822888287418263], ["alveoli", 3.9821356696776085], ["jaidee", 3.982135669677608], ["dear", 3.98153057435938], ["celiac", 3.981174893260002], ["heart", 3.980756337163971], ["manipulations", 3.980527046369989], ["clotheslines", 3.9803422380122875], ["khenpo", 3.9802211813748785], ["anastasios", 3.9802211813748785], ["themselves", 3.9791403008892603], ["embarrassment", 3.9789398718860225], ["vestiges", 3.9789353310222992], ["leyla", 3.9789054057777298], ["advocate", 3.978842749796913], ["stranded", 3.9785497299105788], ["jamar", 3.978438726748199], ["momentous", 3.977576516138085], ["worldview", 3.9773810064202797], ["alkanes", 3.977180523482308], ["solver", 3.9770149203293084], ["shug", 3.976839515120524], ["napkins", 3.9765289539338995], ["cixi", 3.97624493643844], ["pinacoteca", 3.97624493643844], ["terrain", 3.9752953854088435], ["spraining", 3.9749465707449088], ["suya", 3.9747007636475904], ["kari's", 3.9747007636475904], ["nuestros", 3.9744251447055166], ["redder", 3.9743941751589342], ["coffeehouse", 3.9742744079743644], ["fjords", 3.974246823405054], ["whinnies", 3.972670783686585], ["storytimes", 3.972670783686585], ["trochu", 3.972670783686585], ["jamiel", 3.972670783686585], ["macaw", 3.9710423729702025], ["upheaval", 3.9708018989254485], ["shrubbery", 3.9706487398612307], ["jos\u00e9", 3.9698829445401387], ["arawaks", 3.9698829445401387], ["brightwood", 3.9698829445401387], ["dory's", 3.9698829445401387], ["misdiagnosing", 3.9698829445401387], ["jessa's", 3.9698829445401387], ["morcilla", 3.9698829445401387], ["tenser", 3.9698829445401382], ["quirked", 3.9698829445401382], ["continuing", 3.968650571937322], ["veggie", 3.9685196605619315], ["logarithms", 3.968388259696561], ["opposing", 3.9680784522926205], ["andalusian", 3.9672983853314534], ["affordable", 3.967267246063879], ["elin", 3.9669553022506543], ["wooden", 3.966404659249946], ["insee", 3.9663043240973437], ["haste", 3.965641616607938], ["amplifies", 3.965615564225062], ["glucometer", 3.9653511146947733], ["soane's", 3.9653511146947733], ["bilbao's", 3.965171849273675], ["statistician", 3.964525477678411], ["clarifying", 3.964522377292496], ["moreau's", 3.9644147862143253], ["eternity", 3.9639847428594632], ["azure", 3.963790769262524], ["globally", 3.963314019471166], ["atlantans", 3.962990786650312], ["migas", 3.9625674354285785], ["venomously", 3.9620886717432664], ["router's", 3.9619590264871642], ["marvel", 3.961327162332078], ["fresh", 3.961129088380072], ["quedlinburg", 3.960914835478677], ["powerlifting", 3.960860483302547], ["preparedness", 3.960385616921622], ["otros", 3.960070380764792], ["ning's", 3.959324745219553], ["foal's", 3.959324745219553], ["nabla", 3.959324745219553], ["noelani", 3.959324745219553], ["aloof", 3.9592166776727713], ["nima", 3.9591984509693066], ["undiminished", 3.9587315879543516], ["karamanlis", 3.958297683028835], ["tecnolog\u00eda", 3.958297683028835], ["gulab", 3.9580678167290073], ["tariq", 3.9569376740688122], ["ridiculing", 3.956824119064678], ["transpiration", 3.9567251885686514], ["hepa", 3.9564711778356108], ["pragmatist", 3.9559160550415493], ["silent", 3.955673367954517], ["callousness", 3.9555934444125396], ["individually", 3.9555083683870493], ["krit", 3.955499310683109], ["shmuel", 3.9552093180650236], ["laptop", 3.9548499319192696], ["tahoe's", 3.954714177830651], ["bridges", 3.954326359588816], ["alstom's", 3.9540666778288234], ["blindsided", 3.9539221578479085], ["lauding", 3.9532626212077333], ["foremothers", 3.9530614066395446], ["atenci\u00f3n", 3.9530614066395446], ["something", 3.953060104662927], ["spreadsheet", 3.9528552603417437], ["laptop's", 3.9527713801240174], ["moncayo", 3.9515038568339342], ["nerdish", 3.9515038568339342], ["searchingly", 3.9515038568339342], ["janeiro's", 3.9513403418187547], ["mansion's", 3.9506399749205987], ["venus's", 3.9502947063269467], ["gimignano", 3.9501322831245154], ["everwood", 3.949628439721056], ["equestrianism", 3.949628439721056], ["woolen", 3.9495779045293915], ["multitool", 3.948915252931652], ["tobi", 3.9481965659870597], ["jostling", 3.94774605079735], ["menacing", 3.947700237689438], ["manipulate", 3.9473498079289095], ["jlo's", 3.947326791446161], ["b\u00e9chamel", 3.9473267914461605], ["cartwrights", 3.94732679144616], ["tocino", 3.94732679144616], ["factions", 3.947228721215317], ["warm", 3.947054182968222], ["visions", 3.946581457691751], ["detail", 3.946471840482114], ["noncompliance", 3.9461269960688212], ["indescribable", 3.945196172451137], ["hydroelectricity", 3.944853529045505], ["sm\u00e9agol", 3.944434976946933], ["plankton's", 3.944434976946933], ["artfully", 3.9444349769469325], ["emerging", 3.9443508344734384], ["pantry", 3.944341418242546], ["stagnant", 3.9443414182425456], ["oxfam's", 3.943845903623016], ["davida", 3.943845903623016], ["reef", 3.9436397279596447], ["groupmates", 3.9435923290133825], ["prodigy's", 3.9435923290133825], ["bottommost", 3.9435923290133825], ["bayek", 3.9435923290133825], ["consistency", 3.9434712361066575], ["toppings", 3.9431251550855175], ["selves", 3.942995666992754], ["neighborhood", 3.9424408146129863], ["lengthier", 3.9423143129808316], ["auditore", 3.942056662218596], ["myoho", 3.942056662218596], ["goldenrods", 3.942056662218596], ["jamil's", 3.942056662218596], ["ancud", 3.942056662218596], ["tampering", 3.9419491655429604], ["powwow", 3.9418696576331014], ["hornbills", 3.941830652778037], ["splendors", 3.941355041383458], ["mizo", 3.941019142370663], ["basilicas", 3.9409291791423575], ["treatments", 3.94073415970853], ["omoide", 3.940692628771461], ["expanding", 3.9397902929968547], ["meteorologist", 3.9394646707763843], ["streetart", 3.938375937043788], ["shawarmas", 3.938375937043788], ["pantocrator", 3.938375937043788], ["sk\u0142odowska", 3.938375937043788], ["rudimentary", 3.9380299986168854], ["agey", 3.9367315212036864], ["eumenes", 3.9367315212036864], ["canyon's", 3.9364824870740556], ["hildegard", 3.9361879504120973], ["sensitivity", 3.936048714899172], ["\u00e9dith", 3.9356598157078957], ["lopen", 3.9356598157078957], ["iara", 3.935401332861731], ["ectothermic", 3.9351998274171396], ["backpacker's", 3.9351998274171396], ["optimization", 3.935135610233441], ["isolation", 3.9349101662191903], ["dizzyingly", 3.934552271085493], ["pitting", 3.933905880355244], ["nginx", 3.9337645474972494], ["archegonia", 3.933573039559589], ["tiew", 3.933573039559589], ["egrets", 3.9333950498292922], ["d'orsay", 3.932837595882321], ["avant", 3.9325027694346155], ["bhante", 3.9322222246421576], ["lawncare", 3.9322222246421576], ["cryptanalyst", 3.9322222246421576], ["melinda's", 3.932086147463743], ["hartwell", 3.931442176591246], ["minted", 3.9312812168296665], ["smog", 3.9311145564098635], ["grocery", 3.931047133126159], ["louvre", 3.9303279514333074], ["diminish", 3.929822276805521], ["banff", 3.9295716331543007], ["cdns", 3.9292658147694874], ["behavior", 3.9289435266745687], ["graybeard", 3.9285299972011787], ["sluggishness", 3.9285299972011787], ["learner's", 3.9285299972011787], ["peshawar's", 3.9285299972011787], ["mestizaje", 3.9285299972011787], ["microfauna", 3.9285299972011787], ["internships", 3.9283027832048107], ["fatigue", 3.928244410548285], ["sangita", 3.9274909784238186], ["ensembles", 3.9274718358056067], ["saras", 3.927374886940314], ["locally", 3.926895113236615], ["niacinamide", 3.92673204296905], ["narrated", 3.9266213996316885], ["geolocation", 3.9262578572375], ["undimmed", 3.9261396534243604], ["insight", 3.926022141039977], ["nawsa", 3.9259127220531433], ["fantasyfootball", 3.9259127220531433], ["verdadera", 3.9259127220531433], ["wilhem", 3.9259127220531433], ["unbrushed", 3.9259127220531433], ["butterflies", 3.9250698526279195], ["suppressing", 3.9250658759581247], ["dabbled", 3.925064666418584], ["brevity", 3.925025510138555], ["newlines", 3.9247706383521823], ["tips", 3.924340290694468], ["paleontologist", 3.9241573290535547], ["playbooks", 3.923960610754885], ["marked", 3.9238319883104547], ["popescu", 3.923662454781547], ["tipster", 3.9235834723998675], ["cooperative's", 3.9232283372859276], ["cameraperson", 3.9232283372859276], ["xola", 3.9232283372859276], ["marienplatz", 3.9232283372859276], ["f\u00e1brica", 3.9232283372859276], ["descriptions", 3.923062660413576], ["duality", 3.9230162708893173], ["stewards", 3.922941761074292], ["universality", 3.922754445896855], ["pujas", 3.922683179454025], ["uva's", 3.922622433295613], ["morrissey's", 3.9225464517662814], ["luxuries", 3.9224783463710873], ["pandit's", 3.9218601669852173], ["insectarium", 3.9218601669852173], ["xanthan", 3.921571568562411], ["biplanes", 3.9214839603455394], ["jv's", 3.9214001786944612], ["saor", 3.9214001786944612], ["perdida", 3.9214001786944612], ["pensively", 3.920822397616121], ["grumblings", 3.920822397616121], ["questionable", 3.9205062910010824], ["dice", 3.9200436532255316], ["hooty", 3.9199147998388955], ["eiger", 3.9195402260405356], ["patagonia", 3.9190079369586552], ["humored", 3.9185482512917753], ["drizzly", 3.9185482512917753], ["larus", 3.9176476426382947], ["sassanian", 3.9176476426382947], ["guimar\u00e3es", 3.9176476426382942], ["marieke", 3.9176476426382942], ["l\u00fcbeck", 3.916897997088612], ["beefeater", 3.916806223471062], ["refreshments", 3.916367365630897], ["complications", 3.916362271334951], ["rainville", 3.9153322480504467], ["felidae", 3.9153322480504467], ["curtsied", 3.915164976198093], ["mechanicville", 3.915164976198093], ["wetlands", 3.9141830562876625], ["abundantly", 3.91410455045503], ["goldwater's", 3.9139691002508403], ["styles", 3.913849700895848], ["schubert's", 3.9134008701259497], ["rio's", 3.913214090779342], ["gorging", 3.912625017455425], ["whippersnapper", 3.912473781514895], ["giovanni", 3.9118630298326247], ["pigeonholed", 3.910550454879892], ["malevolent", 3.909642833753096], ["paramount", 3.909355166870823], ["sight", 3.9090333338448278], ["freshness", 3.9087933632439475], ["devesh", 3.908703241627705], ["interruption", 3.9086661360339927], ["notepad", 3.908538931498643], ["gloomy", 3.908106704841938], ["ludlow's", 3.907853523531699], ["bovines", 3.907191287720221], ["dukkha", 3.9068254115812824], ["medieval", 3.9067458552994054], ["rigidity", 3.9058853433359344], ["heatwave", 3.905750831294124], ["zebrina", 3.905556137568423], ["blockbusters", 3.9051565921835056], ["cluttering", 3.9046408443559626], ["voicing", 3.90428861427834], ["fado", 3.904204734060614], ["polarizing", 3.904163782439165], ["fortunate", 3.9040639447562637], ["minimized", 3.903422850602525], ["steady", 3.9030492840614373], ["unexplained", 3.9029821030336542], ["unsolved", 3.9029821030336542], ["muddiness", 3.9029748050254165], ["cestius", 3.9029748050254165], ["embroidery", 3.9028626505769966], ["destination's", 3.9025132422528266], ["destinations", 3.9023479536197074], ["sassanid", 3.902214485258204], ["vend\u00e9e", 3.902214485258204], ["nuevas", 3.9018506763373337], ["gobbled", 3.901727160962964], ["phra", 3.901650581430855], ["thunderclouds", 3.901604859498908], ["crispness", 3.901560683719249], ["57something", 3.9012214470716775], ["interweaving", 3.901127441735603], ["buntings", 3.9009055586953965], ["bennet's", 3.9008192592936286], ["grains", 3.9007962840157684], ["chihuly", 3.900715616089885], ["worldwide", 3.900667406169914], ["dionysus", 3.9006143920181278], ["dishonesty", 3.899471169341369], ["denke", 3.8989921776733505], ["nitpicked", 3.8989921776733505], ["en_us", 3.8989921776733505], ["spacesaver", 3.8989921776733505], ["machiavelli's", 3.89899217767335], ["finden", 3.89899217767335], ["obscurities", 3.89899217767335], ["vercelli", 3.89899217767335], ["hyperboles", 3.89899217767335], ["amarjit", 3.89899217767335], ["enlighten", 3.898885414470238], ["vpns", 3.898391408462615], ["lovejoy's", 3.89792132016781], ["persepolis", 3.897136398436], ["utopia's", 3.897136398436], ["corner", 3.8970370052775585], ["rare", 3.897027335960107], ["insatiably", 3.896949216092175], ["physiotherapist", 3.8967200377096716], ["bertha's", 3.896720037709671], ["juggle", 3.896629152111124], ["papes", 3.896320415102994], ["counterparts", 3.8961540241955874], ["reputation", 3.895868177777526], ["hosting", 3.8956187549741723], ["visionary", 3.895353298005283], ["untested", 3.8952373701062535], ["overregulation", 3.894403778523397], ["woes", 3.8940692077520453], ["bach's", 3.893980619604361], ["disorganization", 3.8939043235920017], ["defiance", 3.89388663403311], ["impatience", 3.893239262037564], ["kashrut", 3.8931814590777125], ["majoring", 3.893162344871805], ["keywords", 3.8925230582634534], ["geraldine's", 3.8920421024903322], ["diferente", 3.8920421024903322], ["kandy's", 3.8920421024903322], ["sydneysider", 3.8920421024903322], ["semiochemicals", 3.8920421024903322], ["lobbied", 3.8919855321109353], ["viewership", 3.89177182178877], ["stomata", 3.89141837779442], ["twirl", 3.8913967383661947], ["great's", 3.890936408711215], ["ole's", 3.890791446287732], ["kerala's", 3.890510284912211], ["coops", 3.8897616090709044], ["petros", 3.8894123443129733], ["micromanaging", 3.889146237830741], ["raphael's", 3.889146237830741], ["accuracy", 3.8882465226382266], ["engineer", 3.8867713722142736], ["predestination", 3.8864112545411276], ["cornelia's", 3.886180593299834], ["pawpaws", 3.886180593299834], ["temperaments", 3.8858897985831855], ["tomiko", 3.8855118305062875], ["granola", 3.885444989240692], ["tetrahedrons", 3.885359337891275], ["linq", 3.8852536569627327], ["campaigns", 3.8848796638989436], ["dessert", 3.884546610904268], ["city's", 3.8844282951273352], ["magnifying", 3.8842823435484926], ["typefaces", 3.8841695627830215], ["narva", 3.8841634747524973], ["cardstock", 3.8841459437745467], ["penang's", 3.884085240715381], ["lowri", 3.884085240715381], ["insulations", 3.884085240715381], ["voracious", 3.8839474478629117], ["regrettably", 3.8835811413979617], ["dukkah", 3.8835811413979617], ["infante's", 3.8835811413979617], ["sn\u00e6fellsnes", 3.8835811413979617], ["indulgence", 3.8834419449412807], ["followings", 3.881973698872549], ["frase", 3.8819287545763115], ["squirrels", 3.8817758485771305], ["diversify", 3.8817583877971136], ["temple's", 3.8815698705620214], ["laboring", 3.881223926337844], ["footrace", 3.881170476505722], ["circumstance", 3.8811143639284986], ["amrit", 3.881029990122644], ["tempos", 3.8808150579639173], ["dunrobin", 3.8808150579639173], ["excessively", 3.8807156518405064], ["rampant", 3.8797218706236465], ["styluses", 3.879658332164226], ["coping", 3.8795007542484], ["blacksmith's", 3.8793002979881313], ["prouder", 3.8792141190836014], ["cuneiform", 3.879126033566138], ["seller's", 3.878801769388505], ["staunchly", 3.8787376728542684], ["curie", 3.8786776298557926], ["unfolds", 3.877498700485016], ["feather's", 3.8768388130274793], ["verzasca", 3.8768388130274793], ["sahlan", 3.8768388130274793], ["zanzibar's", 3.8768388130274793], ["yuanzhang", 3.8768388130274793], ["initiative", 3.876595029276791], ["bolder", 3.876316679854007], ["individualistic", 3.875147101545431], ["twigs", 3.8740192938907314], ["vireos", 3.8739564049880535], ["timpani", 3.8738373894302964], ["flourishes", 3.873641420191993], ["watched", 3.8734006108344397], ["individual", 3.872670435935712], ["slicker", 3.8725312143463375], ["victor's", 3.8716564063879564], ["eucalyptus", 3.8713656288208522], ["paddington's", 3.8713397508813405], ["nulth", 3.8713397508813405], ["yeshi", 3.8713397508813405], ["bottlenecks", 3.8711863697501654], ["sindarin", 3.8711235715340724], ["trigonometric", 3.8706358709266344], ["polydactyl", 3.870211738133415], ["sanssouci", 3.870211738133415], ["quoc", 3.8702117381334147], ["dissipate", 3.869354904207718], ["cetaphil", 3.8692816223588093], ["exclusivity", 3.869151406150365], ["artisan", 3.8689999301527864], ["nights", 3.868869609447073], ["ulterior", 3.868845330938763], ["hertzsprung", 3.868501525257527], ["pivottable", 3.8679943138342603], ["jim's", 3.867791533095337], ["handholding", 3.86754183745667], ["indra's", 3.86754183745667], ["chinedu", 3.867266371513831], ["intervene", 3.8669168810317216], ["commercialized", 3.8665478907777953], ["odds", 3.86647892543497], ["zamba", 3.8656015990766748], ["creperie", 3.8656015990766748], ["pepperland", 3.8656015990766748], ["caterpillars", 3.865370350943925], ["oversight", 3.864910078218164], ["tommaso", 3.8644878221187193], ["meine", 3.8633849431740503], ["misplaced", 3.8630404316194795], ["lanterns", 3.8623129359558828], ["khufu's", 3.8614341554922706], ["kirstenbosch", 3.8614341554922706], ["pandava", 3.8614341554922706], ["underbellies", 3.8614341554922706], ["nikephoros", 3.8614341554922706], ["autopia", 3.8614341554922706], ["beneficial", 3.8611246930386742], ["buds", 3.8603508990123556], ["gourds", 3.8602237895430758], ["frida", 3.8600516438906824], ["kaelin", 3.8600351773053783], ["headdresses", 3.859608418302913], ["creases", 3.859608418302913], ["preachiness", 3.8596084183029125], ["undrinkable", 3.8592612063185383], ["tribbiani", 3.8588997131214398], ["diagnosed", 3.8585622871946352], ["cartographical", 3.8582893450225635], ["ningi", 3.8582893450225635], ["ecover", 3.8582893450225635], ["junction's", 3.8582893450225635], ["pilar's", 3.8582893450225635], ["nourishes", 3.8582893450225626], ["roleplays", 3.8577581745740996], ["growing", 3.856976579365879], ["accolade", 3.8568974333538124], ["scrimmages", 3.856823708044397], ["yoga", 3.8567381398905667], ["brigitte's", 3.855882927551655], ["restoran", 3.855882927551655], ["bolkonsky", 3.855882927551655], ["cichlid", 3.855554207779485], ["subheadings", 3.8554198140604767], ["lifesavers", 3.855161864825605], ["aimless", 3.854982808980881], ["osso", 3.854935768886081], ["resuscitating", 3.854935768886081], ["fondness", 3.854813873683903], ["corner's", 3.854782399649205], ["dearie", 3.8546854483816078], ["geeks", 3.854494881804009], ["siargao", 3.8542552859612997], ["oak's", 3.8534917647081373], ["envied", 3.853447830244967], ["udemy", 3.8534195418304154], ["valeria's", 3.853342820221252], ["stomachs", 3.8528843618250326], ["snorkels", 3.85285836676133], ["garde", 3.852621847291307], ["heinrich's", 3.852353515260989], ["madin", 3.852353515260989], ["reemerged", 3.8517316664288246], ["songwriting", 3.8514267599046668], ["zest", 3.8514267599046668], ["disciplines", 3.850381366107553], ["rehman's", 3.8501019936272893], ["bliss's", 3.8501019936272893], ["jasp", 3.8501019936272893], ["jahanara", 3.8501019936272893], ["teriyaki", 3.849849610671299], ["blaring", 3.8484695772688493], ["tortelli", 3.8483559156256444], ["safenet", 3.8473944676120806], ["nadu's", 3.8473944676120806], ["preparations", 3.84733089007812], ["punic", 3.847272260212172], ["tarantula", 3.847005615304373], ["filmographies", 3.8467857989730025], ["jacques's", 3.8467857989730025], ["geremy", 3.8467857989730025], ["tzedakah", 3.8463658441914554], ["demises", 3.8463658441914554], ["unhandled", 3.8458241025232596], ["g\u00e2teau", 3.8458241025232596], ["surstr\u00f6mming", 3.8458241025232596], ["chickadees", 3.845824102523259], ["plopping", 3.845824102523259], ["offaly", 3.8451745797901866], ["muffled", 3.84469629486856], ["reorganizing", 3.8440767948892187], ["bartholdi", 3.8440767948892183], ["sidetracked", 3.8437903510147855], ["lucca", 3.8437903510147855], ["upcycle", 3.8436366670217024], ["reconvene", 3.843321474586319], ["cosworth", 3.842900156751818], ["lunchboxes", 3.842765845967645], ["sculpting", 3.842718545213219], ["daniella", 3.842210738192671], ["negotiation", 3.8419151174514177], ["monoliths", 3.8418222043936825], ["numberphile", 3.8418222043936825], ["fueron", 3.8418222043936825], ["blends", 3.8412922726778937], ["desks", 3.841266838938966], ["minks", 3.8412293305967085], ["misprinted", 3.8409677065724206], ["farouq", 3.8409677065724206], ["superheroes", 3.840881519175973], ["schmancy", 3.840430239826873], ["ohm's", 3.840430239826873], ["searchability", 3.8399165386176937], ["mcilvain", 3.8399165386176937], ["udmurt", 3.8399165386176937], ["paddling", 3.839847119535372], ["molly's", 3.8397646732216635], ["evenly", 3.8394407616588557], ["dashed", 3.8393288751925327], ["gradual", 3.838401778641907], ["gitlab", 3.838284614886874], ["mismanagement", 3.8377552347498036], ["cort\u00e9s", 3.837708103512809], ["beings", 3.8372600405259645], ["maracas", 3.83716883447557], ["twisters", 3.836786703435937], ["tensing", 3.836786703435937], ["impasto", 3.8365017617936674], ["perhaps", 3.8363633550588823], ["marimba", 3.8362811146758067], ["bisi", 3.8360299834166636], ["agora", 3.835190487493241], ["mastered", 3.8348188826669234], ["catalyzing", 3.8345460259762705], ["longshan", 3.8345460259762705], ["supershuttle", 3.8345460259762705], ["r\u00edas", 3.8345460259762705], ["cawing", 3.83454602597627], ["meiosis", 3.83454602597627], ["inventive", 3.834236188708496], ["tanager", 3.8340159166203596], ["relais", 3.8338310924392185], ["cunning", 3.8335062033159093], ["martenot", 3.8327121923913943], ["alexandre's", 3.8327121923913943], ["imambara", 3.8327121923913943], ["lyosha", 3.8327121923913943], ["tamazight", 3.832184125128733], ["wilt's", 3.832184125128733], ["mobas", 3.8312289446389207], ["bisectors", 3.8312289446389207], ["gasson", 3.8312289446389207], ["suffocating", 3.8307212869733784], ["planner", 3.8304539805034685], ["rolled", 3.8303550370372035], ["tijuca", 3.8300045186419123], ["counterculture", 3.8295005706789325], ["safely", 3.829233100759556], ["tavo", 3.8289766054592387], ["nirupama", 3.8289766054592387], ["fearsome", 3.828101410806562], ["asterion", 3.8278378763572434], ["honeycrisp", 3.8277108280690038], ["anila", 3.8273472537122344], ["smudges", 3.8269937425742686], ["courtships", 3.8268998723850913], ["auvs", 3.8268998723850913], ["glimmering", 3.826309228498132], ["lapa", 3.8261138192121664], ["prasadam", 3.8261138192121664], ["zhiyi", 3.8261138192121664], ["leith's", 3.8261138192121664], ["slowdowns", 3.8258146118753755], ["enumerate", 3.8255595107993865], ["aimlessness", 3.8254455607510343], ["pulpo", 3.8254455607510343], ["abeba", 3.8254455607510343], ["lives", 3.8254263117597596], ["rono", 3.8251476288537796], ["morphos", 3.8251476288537796], ["dhaka's", 3.8251476288537796], ["netscape's", 3.8251476288537796], ["saltmarshes", 3.8251476288537796], ["trolltunga", 3.8251476288537796], ["loxodonta", 3.8251476288537796], ["caterina's", 3.8251476288537796], ["handed", 3.82478598710899], ["sandwiches", 3.8243402283539525], ["dives", 3.8241138051703056], ["overcame", 3.82401155887194], ["phrasebook", 3.8230377846017913], ["taib", 3.8230377846017913], ["contently", 3.8229612947799674], ["surmounting", 3.8226261076745742], ["afonso", 3.8226261076745742], ["permutations", 3.8222991681375627], ["trustworthy", 3.8220844475694125], ["overacted", 3.821455401412801], ["xinyi", 3.821455401412801], ["harriet's", 3.8210123341198834], ["fermentations", 3.820915602079324], ["pandas", 3.820652743273458], ["pecorino", 3.820652743273458], ["esports", 3.819787061606137], ["numerological", 3.819706451572337], ["scamper", 3.8192787607816565], ["polyamory", 3.8192787607816565], ["abdullah's", 3.819066313068638], ["receded", 3.8184612070203654], ["reginald", 3.818162333695171], ["gianni's", 3.8171951389809022], ["toulon's", 3.8171951389809022], ["jogs", 3.8171951389809022], ["\u00fanico", 3.8171951389809022], ["bitz", 3.8171951389809022], ["goldbach's", 3.8171951389809022], ["apizza", 3.8171951389809022], ["v\u011bra", 3.8171951389809022], ["maricela", 3.8171951389809022], ["axl's", 3.817195138980902], ["dweller", 3.816547058312145], ["luoyang", 3.816286282995431], ["sequins", 3.816079000051376], ["equanimity", 3.815674344104814], ["skilled", 3.815590503246553], ["converting", 3.815560213688601], ["lentils", 3.8148093920190393], ["excessive", 3.8147154080478143], ["cannolis", 3.8141067933992026], ["pressed", 3.81386955230679], ["merrymakers", 3.8134819141375362], ["all\u00ed", 3.8134819141375362], ["significant", 3.8129972235937664], ["paine's", 3.8123875884280296], ["engagingly", 3.8123875884280296], ["tiny", 3.8120872375719066], ["trendier", 3.8118638608538347], ["poulenc's", 3.8114463511209915], ["sesquipedalian", 3.8114463511209915], ["drizzle", 3.809750254844592], ["snowstorm", 3.809693563482261], ["diffused", 3.8095536403321044], ["html", 3.8091892766927713], ["compson", 3.8091306422084075], ["din\u00e9", 3.8091306422084075], ["seeds", 3.8078919749118154], ["zoomed", 3.8076251935406606], ["scoops", 3.8068414367103816], ["subtracted", 3.8064057959954787], ["comically", 3.8060157474448375], ["nnedi", 3.8057891774102424], ["kichwa", 3.8057891774102424], ["treadle", 3.804897474229031], ["inclusionary", 3.804897474229031], ["pocketknife", 3.8048599020453002], ["smudging", 3.8044711551842996], ["groceries", 3.8042060721885367], ["elaborate", 3.8038598507453756], ["restless", 3.802808725140522], ["hari's", 3.802808725140522], ["pulsed", 3.802439362974004], ["oxidization", 3.802250862709724], ["kyaw", 3.802038628870243], ["atp's", 3.8018032230979144], ["dwellers", 3.801007033941245], ["crampons", 3.8007117963353028], ["isaiah's", 3.80064660514139], ["cornstarch", 3.8002835149474423], ["polluting", 3.8002298272520982], ["agronomist", 3.8001932640802694], ["flatworms", 3.800181708458702], ["craving", 3.8000282765445763], ["mixup", 3.798767300378926], ["broadens", 3.798344792615565], ["shunning", 3.797968480418468], ["tidied", 3.7977196535779387], ["ganoush", 3.7977196535779387], ["proctoring", 3.7977196535779387], ["rolex's", 3.7977196535779387], ["tightrope", 3.797382078497621], ["experiential", 3.7971098975428093], ["joeys", 3.7966830873128887], ["gridlock", 3.796160352153455], ["binaural", 3.7959884993198973], ["weser", 3.795221153805848], ["overambitious", 3.795116246788854], ["lynxes", 3.795116246788854], ["inflexibility", 3.7937994268387616], ["dreamscape", 3.7937994268387616], ["guernica", 3.7937994268387616], ["waterfalls", 3.793132443817165], ["encompassed", 3.792981587726173], ["translations", 3.792656027373168], ["friction", 3.7916541598102356], ["dedicate", 3.791452634725738], ["glamorizing", 3.7912719122306076], ["extracting", 3.7911109721978664], ["aloe", 3.790806703020121], ["aiko's", 3.7906868394046467], ["centroids", 3.7906868394046467], ["appreciator", 3.7906868394046462], ["damselfly", 3.790686839404646], ["erato", 3.790686839404646], ["billie's", 3.7902818947577708], ["olivetti", 3.790211518170865], ["rhodiola", 3.7896937330804636], ["afin", 3.7894337197883137], ["kheer", 3.7887616556365384], ["rumblings", 3.7887379001555948], ["snowmass", 3.7872059515815018], ["crescendos", 3.7872059515815018], ["brew", 3.7870593878836853], ["vectorized", 3.787059387883684], ["rigatoni", 3.786879017366068], ["transitions", 3.7865054788339543], ["neuroscientist", 3.786147094686796], ["garden's", 3.7860028486057353], ["damp", 3.785728452433548], ["williamses", 3.7854812820860464], ["styrian", 3.7854812820860464], ["quiet", 3.785459900870751], ["dave", 3.784336029189075], ["gropius", 3.7839851505660227], ["drawing", 3.7838926227058858], ["zhongshan", 3.78379468151482], ["schoolbags", 3.7833547565431283], ["telah", 3.7833547565431283], ["saif's", 3.7833547565431283], ["britbox", 3.7833547565431283], ["forever", 3.7833435584529402], ["thus", 3.783077490177207], ["wildfires", 3.782956446095787], ["thirst", 3.7828010752286434], ["brinkmanship", 3.7827190268152515], ["silences", 3.7817384576139874], ["anonymized", 3.7816182106022427], ["perfection", 3.7815725425841182], ["biometric", 3.78144556321665], ["pressures", 3.7809716943348537], ["alafair", 3.7808408995620373], ["bulbuls", 3.7808408995620373], ["pulsar's", 3.7808408995620373], ["boltzmann's", 3.7808408995620373], ["kaspars", 3.7808408995620373], ["pioneers", 3.780824258536951], ["fanfiction", 3.7799471682601355], ["gizmo", 3.7796797076395667], ["hotcakes", 3.7792759819711588], ["labyrinthian", 3.7792759819711583], ["philopator", 3.7792759819711583], ["draiman", 3.7792759819711583], ["saskia's", 3.7792759819711583], ["korowai", 3.7792759819711583], ["albert's", 3.7791999095882685], ["bite", 3.7788080025710036], ["biomagnification", 3.7788038085601174], ["arrogance", 3.778028566322486], ["plucked", 3.777632818500736], ["hypnotic", 3.776902523624993], ["hawking's", 3.776902523624993], ["helpings", 3.7767280386151234], ["cassandra's", 3.776282007802115], ["childhood", 3.776021859005737], ["caped", 3.775939427325978], ["familiar", 3.7759000279319275], ["audacity", 3.7757908852556663], ["cheezits", 3.7757038874702404], ["sensical", 3.7757038874702404], ["archeologico", 3.7757038874702404], ["goldsmithing", 3.7757038874702404], ["cosecant", 3.7757038874702404], ["chestnut's", 3.7757038874702404], ["padmanabhaswamy", 3.7757038874702404], ["soshite", 3.7757038874702404], ["agbada", 3.7757038874702404], ["koffer", 3.7757038874702404], ["menanti", 3.7757038874702404], ["eudaimonia", 3.7744972067775366], ["hba1c", 3.7744653426490267], ["headbang", 3.774184489527596], ["simplicity's", 3.773958300837106], ["haci", 3.773958300837106], ["minarets", 3.7739395398592728], ["bureaucracy", 3.7734855664739806], ["pienza", 3.77345644468008], ["jossa", 3.77345644468008], ["khufu", 3.772923545874165], ["seminars", 3.7728021258930706], ["thanjavur", 3.77246238344597], ["duat", 3.7720593856483577], ["tankmates", 3.7720593856483577], ["twenties", 3.7718817463542584], ["haldi", 3.7716381618196984], ["inclusion", 3.7716250998693543], ["unkindly", 3.7712872138191567], ["dreamers", 3.7711870756187946], ["crackle", 3.7711419140454363], ["nightlife", 3.7710368573357784], ["rickshaws", 3.7705524455691974], ["pastas", 3.770522010644018], ["crowded", 3.77047946548683], ["frosty", 3.770446578259839], ["plasticware", 3.770415786787509], ["treebank", 3.770415786787509], ["azal", 3.770415786787509], ["ad\u00e9la\u00efde", 3.770415786787509], ["bloodwork", 3.7704157867875088], ["epicenter", 3.7702013323265717], ["airwaves", 3.7698299427537574], ["modernizers", 3.768876213171018], ["rochdale's", 3.768876213171018], ["propio", 3.768876213171018], ["saltshaker", 3.768876213171018], ["orleanian", 3.768876213171018], ["nontraditional", 3.768521105487616], ["ritter's", 3.7684540720181086], ["acadian", 3.768142397708951], ["waver", 3.768121767309841], ["ahmed", 3.767828528770044], ["pesar", 3.7677129797718907], ["eivor", 3.7677129797718907], ["trimurti", 3.7672318518214665], ["exploitation", 3.7669427578882497], ["slowhand", 3.7660719897981227], ["ikkoku", 3.7660719897981227], ["nocturnal", 3.7659701352480264], ["likud's", 3.764544171546683], ["tibble", 3.764544171546683], ["monohulls", 3.764544171546683], ["sliotar", 3.764544171546683], ["iktomi", 3.764544171546683], ["once", 3.7644100300055743], ["gharana", 3.76417853982839], ["chytrid", 3.76417853982839], ["pemberton's", 3.76417853982839], ["happenings", 3.7641028018296008], ["diacritical", 3.7640144017505186], ["simplify", 3.763979727581568], ["alona", 3.763861075162806], ["kinkajou", 3.7635828477030806], ["radu", 3.7635828477030806], ["violin's", 3.7635828477030806], ["amila", 3.7635828477030806], ["stance", 3.7635243528638473], ["reliance", 3.763438277097189], ["contactless", 3.7633492298975124], ["nudge", 3.7628690937047433], ["screens", 3.7623452555586305], ["schnitzel", 3.762274270144545], ["mariana", 3.76205787586229], ["geckos", 3.7619137530684768], ["ayatollah's", 3.761784241802147], ["ribeyes", 3.761784241802147], ["soundchecks", 3.761784241802147], ["geechee", 3.761784241802147], ["horrors", 3.761494073961936], ["purrs", 3.761305927284668], ["athanasius", 3.7605336486366547], ["humorously", 3.760055026560972], ["cptpp", 3.759358848996343], ["mairead", 3.759358848996343], ["hertfordshire's", 3.759358848996343], ["chateau's", 3.759358848996343], ["abner's", 3.759358848996343], ["hurrem", 3.759358848996343], ["quill", 3.7592714220463668], ["ashkenazi", 3.7592452732003014], ["spaciousness", 3.758589277072291], ["aloofness", 3.7585234359187885], ["wane", 3.7584690599196584], ["moisturizers", 3.758253155217227], ["sopwith", 3.757551464934326], ["thermostats", 3.75751708817695], ["altruism", 3.757373424116276], ["revering", 3.7572106439397737], ["cutlery", 3.757036892060199], ["fabric's", 3.756363343937382], ["rintaro", 3.756363343937382], ["improvement", 3.755690736721352], ["grudges", 3.755335918862731], ["darkness", 3.7549510007458946], ["atat\u00fcrk", 3.754128436684696], ["concoct", 3.7540482305479475], ["soil", 3.753935652238514], ["networking", 3.753812607755287], ["blackwood", 3.7536775028258345], ["riveting", 3.7535101023735447], ["duvalier", 3.7531725622828045], ["ayanami", 3.7526695983753533], ["treefrog", 3.752397073350054], ["hardcode", 3.752397073350054], ["tr\u00f8ndelag", 3.752397073350054], ["newsletters", 3.7516761179386933], ["joan's", 3.75072841428464], ["alamo's", 3.7506161539986778], ["residing", 3.7501682059155077], ["boule", 3.749960451873853], ["artnet", 3.74980115700737], ["heidi's", 3.7496884725359028], ["canvas", 3.7496512940737703], ["gears", 3.7493338920656867], ["violinist", 3.7493338920656862], ["lachaise", 3.749146668761379], ["edward's", 3.7491466687613784], ["receptionist", 3.7490309400705293], ["shuttlecocks", 3.749030940070529], ["lamarckism", 3.749030940070529], ["agnes", 3.7487780728810227], ["homebody", 3.7487345739883104], ["mantlepiece", 3.7480012693921245], ["undue", 3.747849427289416], ["arturo's", 3.7476108525932297], ["for\u00eat", 3.7476108525932297], ["tricks", 3.7475448638474758], ["julia's", 3.747382130539364], ["inflections", 3.747083391530233], ["serps", 3.7470833915302326], ["relief", 3.7467168135204787], ["monopolizing", 3.7464127125627136], ["clouding", 3.746227567382948], ["fronds", 3.746227567382948], ["stamps", 3.7461802847655443], ["swept", 3.746121244415549], ["gravitating", 3.7460905236469446], ["cashew", 3.7460472402605736], ["unrolling", 3.745978811761109], ["globe", 3.745927506176683], ["irritability", 3.745498256370479], ["sina's", 3.7443214136003578], ["sassiness", 3.7443214136003578], ["snuffles", 3.7443214136003578], ["farfalle", 3.7443214136003578], ["developer", 3.7442214812765235], ["shareable", 3.744185533160033], ["barreling", 3.7439186251522516], ["nirvana's", 3.743657997332888], ["symbiosis", 3.743529969632148], ["gion", 3.7433407112919244], ["thalia", 3.743216357108722], ["hummus", 3.7428552636492496], ["metamorphosing", 3.742792182882122], ["survival", 3.74265380057247], ["whovian", 3.7425572452019464], ["heralding", 3.7424417341758605], ["treadmills", 3.7415355938955237], ["linters", 3.741457140191599], ["splendorous", 3.741457140191599], ["danube's", 3.741457140191599], ["leicas", 3.741457140191599], ["curiouser", 3.740809380991427], ["strenuous", 3.740562054751361], ["exhausted", 3.740167322072217], ["reddi", 3.7398613638799922], ["zeppelin's", 3.739744802827667], ["cledus", 3.7395727497910998], ["singlehood", 3.7395727497910998], ["swamplands", 3.7392383368467854], ["puedes", 3.7392383368467854], ["surpasses", 3.739133498388743], ["allegories", 3.7383802839907836], ["charades", 3.7380175105522593], ["neurostimulation", 3.7380175105522593], ["quincy's", 3.7376702402521333], ["parakeets", 3.737468911394592], ["workloads", 3.7372174294408667], ["smoothie", 3.737098375811837], ["thirdly", 3.7368680760562607], ["berimbau", 3.7367121089421484], ["couponers", 3.7367121089421484], ["clink", 3.7366404214196747], ["donan", 3.735104920938302], ["automations", 3.735104920938302], ["zenny", 3.735104920938302], ["blackcomb", 3.7349031106443245], ["euphoria", 3.7345989338184706], ["ours", 3.733914471241826], ["vittorio", 3.7330433370533687], ["nonetheless", 3.7329305562878994], ["streetlight", 3.7328419428293156], ["dumbfounded", 3.7328025240310136], ["ramipril", 3.7327439190919445], ["sorrow", 3.732392274906336], ["com's", 3.7322787216942515], ["checkmark", 3.73210349734112], ["possess", 3.7320444217020645], ["showoffs", 3.7318513452231996], ["corna", 3.7318513452231996], ["helsingborgs", 3.7318513452231996], ["koda's", 3.7318513452231996], ["heathcliff", 3.7316504272009516], ["rinsing", 3.731438464562277], ["clients", 3.7312676398949063], ["invasive", 3.731132968985206], ["feigning", 3.730690357054357], ["jigme", 3.730595071326087], ["narasimha", 3.7305637599402006], ["technology", 3.730035849974172], ["claiming", 3.729301545301798], ["whisperings", 3.728936291836835], ["loudly", 3.7288026340405627], ["giraffe's", 3.7280840470267647], ["havens", 3.727968202607224], ["melos", 3.7277296279110375], ["ling's", 3.7272064377878222], ["milkshakes", 3.7268288867111505], ["integrate", 3.7267276141870553], ["alhambra", 3.726276491125175], ["shiniest", 3.725606091528575], ["benefits", 3.7256017622681736], ["resting", 3.725442262267189], ["anonymize", 3.725182859460013], ["swished", 3.724962653341867], ["prejudice", 3.724580209920053], ["meiji", 3.724241484897934], ["rami", 3.7231576156356376], ["rangda", 3.7217652605063805], ["supercontinents", 3.7217652605063805], ["trichologist", 3.7217652605063805], ["concierge's", 3.7217652605063805], ["peache", 3.7217652605063805], ["kendall's", 3.72176526050638], ["subsiding", 3.72176526050638], ["sprees", 3.72176526050638], ["homogenic", 3.72176526050638], ["transcriptionist", 3.72176526050638], ["gameplans", 3.72176526050638], ["literario", 3.72176526050638], ["qian's", 3.72176526050638], ["learnenglish", 3.72176526050638], ["debian's", 3.7217652605063796], ["anoush", 3.7217652605063796], ["sl\u00e1inte", 3.7217652605063796], ["oiliness", 3.7217652605063796], ["inupiaq", 3.7217652605063796], ["globale", 3.7217652605063796], ["posay", 3.721765260506379], ["celebration", 3.7214471609114645], ["jealousy", 3.719509645196982], ["plumeria", 3.71945503998465], ["jinnah's", 3.7189263854030186], ["adversary's", 3.7187902283237206], ["exacerbated", 3.7187758908192263], ["kumu", 3.718717131709896], ["advancement", 3.7169642217286367], ["foray", 3.716126222232885], ["brisk", 3.715483800151095], ["madame", 3.7152772386576367], ["diligence", 3.7152560002770993], ["sunrays", 3.7143365873317165], ["freiherr", 3.7143365873317165], ["peddled", 3.713549443154269], ["sentient", 3.713306703096138], ["projects", 3.7132002073268864], ["library's", 3.7130184561338617], ["lightheadedness", 3.7128187093993934], ["accordion", 3.712687784261242], ["prematurely", 3.7114270236716402], ["timid", 3.7113108637072045], ["tricky", 3.7112909942396035], ["environment", 3.7112512317902393], ["diversos", 3.7111619406758773], ["afrofuturism", 3.7111619406758773], ["ischgl", 3.7111619406758773], ["ramlal", 3.7111619406758773], ["dogsledding", 3.7111619406758773], ["propelled", 3.7110846979486354], ["taronga", 3.711008713510697], ["petersburg's", 3.711008713510697], ["libert\u00e9", 3.710977535113608], ["pemba", 3.710977535113608], ["catapulted", 3.710409760092935], ["mosque's", 3.710323733574652], ["synergizing", 3.7101709762057373], ["demna", 3.7101709762057373], ["tanh", 3.7101709762057373], ["gushi", 3.7101709762057373], ["livestock's", 3.7101709762057373], ["astoria's", 3.7101709762057373], ["larue's", 3.7101709762057373], ["fangirled", 3.7101709762057373], ["nouf", 3.7101709762057373], ["tangibly", 3.710170976205737], ["straining", 3.7098077817577653], ["harvest's", 3.7096026289360973], ["cheetah's", 3.7092970351780505], ["batur", 3.7092970351780505], ["spilling", 3.7085111107182516], ["harried", 3.708231568649993], ["intergalactic", 3.7080571011674985], ["stormy", 3.7075501848586128], ["csail", 3.707505623492945], ["eisenhower's", 3.7071701026220416], ["kannon", 3.7070214252921496], ["nyhavn", 3.706937510464522], ["rom\u00e2n\u0103", 3.706937510464522], ["tomorrows", 3.706773255428812], ["business's", 3.706449354084543], ["handing", 3.706389546640111], ["undergrad", 3.7059950687245733], ["necessitates", 3.7059759533405945], ["nanobot", 3.704534865781813], ["xiao's", 3.704292184165974], ["satisfyingly", 3.7037857181850935], ["imaginative", 3.703728336667046], ["gpas", 3.703386172800175], ["stoically", 3.703386172800175], ["einstein's", 3.703348968169191], ["sowerberry", 3.7022795785142], ["troubled", 3.701622865492569], ["suffice", 3.7010887868369], ["medications", 3.7010528277522576], ["gilead's", 3.7007383381306376], ["commodify", 3.7007383381306376], ["khanate", 3.700000551380612], ["feverishly", 3.6998079433352506], ["streamlined", 3.6997548207937077], ["phraya", 3.6996478008275195], ["minimoog", 3.699441047289975], ["panspermia", 3.699441047289975], ["waves", 3.699339239064867], ["sipadan", 3.699209107412402], ["simulating", 3.698382442115764], ["durability", 3.6980093120350626], ["blurred", 3.6977945689980336], ["podcasters", 3.6977538717289193], ["frodo's", 3.697233851068014], ["tupolev", 3.697233851068014], ["dampness", 3.697117808450047], ["chilliest", 3.6971178084500465], ["normal's", 3.6971178084500465], ["arsaces", 3.6971178084500465], ["nwankwo", 3.696447129482527], ["respeto", 3.696447129482527], ["narrative's", 3.696447129482527], ["reformative", 3.696447129482527], ["aspasia", 3.696206386556785], ["antacids", 3.696097913882198], ["umbanda", 3.695369762204916], ["pritchett's", 3.695369762204916], ["rotax", 3.6950995933379622], ["source's", 3.6950995933379622], ["containers", 3.6950343497955607], ["unwanted", 3.6948138371163113], ["ariadne's", 3.693886869041912], ["ladywell", 3.693886869041912], ["bobbing", 3.69372936400539], ["harlowe", 3.6936613157129123], ["marcy", 3.6931990797788092], ["tubas", 3.6930811351845585], ["evident", 3.6919251091512457], ["alleviated", 3.690685801128456], ["marv's", 3.6903579587299546], ["backfired", 3.6901866461748103], ["emmas", 3.689955301014872], ["twangs", 3.689482290464813], ["astronomer", 3.6892532881157494], ["combatted", 3.689118196817727], ["neap", 3.689118196817727], ["hostas", 3.689118196817727], ["parallelograms", 3.6890399064971313], ["wisdom's", 3.6890399064971313], ["seaweed", 3.6888821939476886], ["dhikr", 3.6887318410344294], ["cavernous", 3.68865346993248], ["tearoom", 3.688235843745061], ["ugga", 3.688235843745061], ["nopa", 3.688235843745061], ["gouramis", 3.688235843745061], ["bleaching", 3.6879310308654123], ["fiendishly", 3.6869824076044506], ["liters", 3.6865485569661045], ["chippy", 3.6860648743144724], ["insurmountably", 3.685806175863806], ["substance's", 3.685806175863806], ["firekeeper", 3.685806175863806], ["sorani", 3.685806175863806], ["propels", 3.6851697318878314], ["fluffy", 3.685089767365143], ["brightening", 3.684238564522627], ["\u00e9p\u00e9e", 3.6838433437326614], ["montju\u00efc", 3.6838433437326614], ["utterly", 3.6838387002326485], ["papel", 3.6823815011359424], ["rotterdam's", 3.682111749359432], ["apoyo", 3.682111749359432], ["turntables", 3.6817462792106124], ["crafter's", 3.6817462792106124], ["rabri", 3.6817462792106124], ["accepting", 3.681672071828077], ["durga", 3.681593825948534], ["preservations", 3.6808667411601554], ["koregaon", 3.6808667411601554], ["polymath", 3.6806861737458236], ["sciencedirect", 3.680214452175368], ["privacy", 3.6797261070543716], ["barceloneta", 3.679541019301283], ["gemology", 3.6793116643789308], ["shyamal", 3.6789863494660766], ["musescore", 3.6789863494660766], ["razia", 3.678598038624561], ["visualize", 3.6784113641026317], ["maharashtrian", 3.6777360263876546], ["consideration", 3.6772729532909025], ["arguing", 3.6772295301418936], ["nurturer", 3.676652954318423], ["incrementing", 3.676545714287866], ["huxley's", 3.676298669191293], ["programmer's", 3.6758175412408693], ["meses", 3.675817541240869], ["moana's", 3.675817541240869], ["trace's", 3.675817541240869], ["beccs", 3.675817541240869], ["badwan", 3.675817541240869], ["chewa", 3.675817541240869], ["weidemann", 3.675817541240869], ["laptops", 3.675287884534639], ["vinegars", 3.6751535568584193], ["ongoing", 3.674990921985183], ["giza", 3.6744459675314505], ["misfortunes", 3.6744459675314505], ["classmate", 3.674181047130042], ["meteorologists", 3.6741647455838367], ["attics", 3.673493337542614], ["soups", 3.672535561293332], ["similar", 3.671973248728988], ["undoubtedly", 3.6714225420067765], ["resenting", 3.6711289984586744], ["accompanied", 3.670817500105066], ["traveled", 3.670739319398568], ["duryodhana", 3.670379941327791], ["discuss", 3.670295771564585], ["ent\u00e3o", 3.6700740763326802], ["denen", 3.6700740763326802], ["tastes", 3.6699706939643324], ["buttery", 3.6698833339556556], ["tallied", 3.6694611373899093], ["thomas's", 3.6692138599546], ["judgmental", 3.669158030030868], ["basilica", 3.6690402526492063], ["probiotic", 3.6687753028609795], ["enhanced", 3.668523340807469], ["waystation", 3.6684066187786892], ["underperformers", 3.6684066187786892], ["table1", 3.6684066187786892], ["shahnameh", 3.6684066187786892], ["soulmates", 3.668362447717657], ["climactic", 3.6681905918891045], ["melancholy", 3.668090822735812], ["blanching", 3.668085953864461], ["lukasz", 3.668085953864461], ["restaurante", 3.6679936261956585], ["lyfts", 3.6678266335425196], ["millworkers", 3.6678266335425196], ["boatworks", 3.6678266335425196], ["couponing", 3.667826633542519], ["overanalyzing", 3.6675493141236823], ["noah", 3.667097733178143], ["jeb's", 3.6665163834948142], ["begonias", 3.6663818488917017], ["graphing", 3.665851416217082], ["affording", 3.665669088463965], ["serendip", 3.6653748777714346], ["moseley's", 3.6653748777714346], ["cervezas", 3.6653748777714346], ["anisha", 3.6648120995006273], ["llorona", 3.6648120995006273], ["taipei's", 3.6648120995006273], ["influences", 3.6646095797200324], ["meimei", 3.664371490178286], ["suds", 3.6636660782946184], ["lionheart's", 3.6626896214507236], ["eggstravaganza", 3.6626896214507236], ["timespans", 3.6626896214507236], ["stroll", 3.6620597082965274], ["tortes", 3.6619778667633054], ["smaug's", 3.6619778667633054], ["appenzeller", 3.6619778667633054], ["percussionists", 3.66189398994829], ["distinctiveness", 3.6618111732863268], ["nora's", 3.6617367885627288], ["chia", 3.66150810866961], ["bolsonaro's", 3.661458878970397], ["sophie's", 3.661307735157023], ["scientist", 3.6608845324795776], ["gambino's", 3.6607527152521775], ["gameplays", 3.6607527152521775], ["buteo", 3.6607527152521775], ["harvesting", 3.6606507277134037], ["organizers", 3.660462179322395], ["therein", 3.6604433994333982], ["sights", 3.6602736668477243], ["intimately", 3.65973583949794], ["katniss's", 3.65973583949794], ["hania", 3.6594342377750464], ["bristlecone", 3.659289584670541], ["honorifics", 3.6590798291700013], ["puris", 3.658684493379153], ["einfach", 3.658684493379153], ["csrf", 3.658684493379153], ["bologna's", 3.658145341523365], ["artificial", 3.658095981241395], ["culture", 3.6579966968528437], ["lithops", 3.6577545557802256], ["fates", 3.6576818851599127], ["doughy", 3.657596893945925], ["pandora's", 3.6575336115331436], ["vivaan", 3.657511676991662], ["hootie", 3.6574384535346645], ["varied", 3.657301039794805], ["racks", 3.6570850095403147], ["rich", 3.6569281117081363], ["consult", 3.6566385539685036], ["wildwood", 3.6564711331290747], ["colorations", 3.6564711331290742], ["misinterpret", 3.6563070775326496], ["koftas", 3.6553051665687666], ["milt's", 3.6553051665687666], ["sri's", 3.6553051665687666], ["microstate", 3.6553051665687666], ["gavels", 3.6553051665687666], ["pal\u00e1cio", 3.6553051665687666], ["logrolling", 3.6553051665687666], ["ropac", 3.6544465090243525], ["personales", 3.6544465090243525], ["park's", 3.654328146384384], ["gravitated", 3.6543012376573216], ["weatherfield", 3.6540968012244455], ["keypoint", 3.6540968012244455], ["sinister", 3.6538134324669835], ["scrapyards", 3.6521995547025226], ["meryl's", 3.6521995547025226], ["relative's", 3.6520692818452116], ["lyricism", 3.6520366373587194], ["gamgee", 3.652028041621152], ["dunstan's", 3.6518364769060514], ["makoto's", 3.6518364769060514], ["expat", 3.6512219973987428], ["redistributing", 3.650874493639592], ["dulled", 3.6505532363879296], ["oughtn't", 3.6497974239772515], ["hydroelectric", 3.6496380267756363], ["leafy", 3.649190837926506], ["expressiveness", 3.648789471084686], ["kendrick's", 3.648789471084686], ["hepta", 3.648789471084686], ["fasnacht", 3.648789471084686], ["sketch's", 3.6487894710846858], ["liverpudlians", 3.6487894710846858], ["quenelles", 3.6487894710846858], ["kulich", 3.6487894710846858], ["farrokh", 3.6487894710846858], ["maximus", 3.64872126941233], ["superstitions", 3.6482621893692113], ["traced", 3.647644526052555], ["tactics", 3.6474137788381555], ["george's", 3.6473299552962524], ["ackee", 3.6469559085364027], ["underdogs", 3.6469490728865512], ["randomness", 3.646767471881566], ["glassblower", 3.646686422398182], ["battles", 3.6465083735610415], ["roommate", 3.64610088936915], ["cors", 3.645588778566191], ["medians", 3.645421357726762], ["irrigating", 3.6447632206338345], ["else's", 3.6443525430878467], ["gravity", 3.643960455883448], ["idolize", 3.6437149956613237], ["canopy", 3.6434320339760844], ["luxembourgers", 3.642997741765504], ["playmats", 3.642997741765504], ["wftda", 3.642997741765504], ["\u043e\u0434\u0438\u043d", 3.642997741765504], ["rerouting", 3.6422403617776107], ["sidelines", 3.6422251417013], ["stuffy", 3.641764698831944], ["divide", 3.6415969330515328], ["nuzzling", 3.640857320060589], ["lightened", 3.6404152001674426], ["townscape", 3.640049855585082], ["interspersed", 3.6400439598127208], ["onscreen", 3.639925396139224], ["rani", 3.639228671651752], ["mouthful", 3.6390593658284605], ["turing's", 3.6390593658284605], ["secretive", 3.638604937835725], ["quantum", 3.6383171334403253], ["creasing", 3.637741343602517], ["couscous", 3.637604739540155], ["statistica", 3.6375277751862765], ["reclaim", 3.6374052479349017], ["mollusk", 3.6373273052069335], ["candor", 3.6372243697500277], ["barbecuing", 3.6371796864039623], ["balms", 3.636207438425773], ["tropea", 3.636207438425773], ["daigoro", 3.636207438425773], ["discontent", 3.6358783698793093], ["smoldering", 3.6356746050142394], ["pei's", 3.6355612313103616], ["lgbtqia", 3.635212580029487], ["peaceably", 3.6349240710945647], ["peque\u00f1o", 3.634399878804352], ["postures", 3.633796263439865], ["alphabetize", 3.6337601323021866], ["harpsichords", 3.6337601323021866], ["comedic", 3.633373335569354], ["analyzed", 3.633330368086287], ["spammy", 3.632084169891768], ["guest's", 3.6316370419471085], ["unicycle", 3.6316370419471085], ["matisse's", 3.6316370419471085], ["besan\u00e7on", 3.6315406481304673], ["squirrely", 3.6315406481304673], ["jenever", 3.630990498055005], ["usman's", 3.630990498055005], ["anaya's", 3.630990498055005], ["dappling", 3.630990498055005], ["coily", 3.630990498055005], ["sketches", 3.6305953122795502], ["plagiarism", 3.6305455237292623], ["geometric", 3.6299170689874565], ["guzheng", 3.6296691769130636], ["ashoka's", 3.6296691769130636], ["sicilia", 3.6295593644127533], ["lusher", 3.629539262843951], ["libertyville", 3.6290408682772695], ["dizzying", 3.628986211989483], ["impartiality", 3.6286829805183154], ["famiglia", 3.6285985412051662], ["proper", 3.6282839692647086], ["whitmore", 3.627923473806774], ["spirit's", 3.627543355177104], ["liaised", 3.627046219115466], ["wished", 3.626984305205484], ["endured", 3.6267623051724014], ["tended", 3.62662404892404], ["cymraeg", 3.6263353820318573], ["est\u00e1n", 3.625936927424448], ["reuniting", 3.6256927565875845], ["mettre", 3.625463876292364], ["climate", 3.625275050048807], ["troubling", 3.625187807543873], ["claire's", 3.624773802202274], ["itchiness", 3.6244642079441216], ["dragan", 3.6244642079441216], ["gratification", 3.6243857393533454], ["andra", 3.6241908903807447], ["zora's", 3.6241908903807443], ["hoto", 3.6241908903807443], ["hazel", 3.6234477344123115], ["chipped", 3.6227969708075216], ["outshining", 3.6227709383536917], ["wargames", 3.6225181868928766], ["boarded", 3.622518186892876], ["grabbed", 3.6223061204962663], ["outlined", 3.6209779151701293], ["garum", 3.620856154559636], ["freddie's", 3.618834343549144], ["geopolitics", 3.6185392951368134], ["proposal's", 3.6183828921589805], ["bottarga", 3.6183828921589805], ["chewier", 3.6183828921589805], ["directrix", 3.6183828921589805], ["perseverant", 3.6183828921589805], ["florence's", 3.6177159091373845], ["mmorpgs", 3.6174167018005936], ["catching", 3.6172574638453248], ["retrain", 3.6168212249634006], ["falafel", 3.6167268816820197], ["radha", 3.616662459402821], ["udon", 3.6166306486276687], ["miso", 3.6164521518333075], ["baked", 3.6160468482588226], ["flakiness", 3.615785345215578], ["manage", 3.6155108286694446], ["literalness", 3.614959634928934], ["eyed", 3.6147100974940063], ["lamppost", 3.6141343016789507], ["rectangles", 3.6136315630273255], ["umbrellas", 3.6130935616853925], ["pottery", 3.613059009717138], ["vaishali", 3.6130539040998366], ["mustiness", 3.6127025422497825], ["steelheads", 3.611839704288892], ["prahlada", 3.611839704288892], ["rattled", 3.6113956700223975], ["companion", 3.6112150479535607], ["jagannath", 3.6108183286213658], ["waved", 3.610577076450906], ["internalizing", 3.6102516946485106], ["camembert", 3.6102516946485106], ["raccoons", 3.6099333716690487], ["sinfully", 3.6092685775691593], ["nuong", 3.60898449503649], ["eggcellent", 3.60898449503649], ["mtoto", 3.60898449503649], ["spark's", 3.6089844950364895], ["sami's", 3.6089844950364895], ["shyne", 3.6089844950364895], ["kunle", 3.6089844950364895], ["railroading", 3.608984495036489], ["contextualization", 3.608442279817884], ["mediations", 3.608442279817884], ["gyoza", 3.608442279817884], ["zoltar", 3.6080858435188015], ["magellan's", 3.60797527118441], ["hiroshige", 3.60797527118441], ["stan's", 3.6079137915668387], ["lucie's", 3.6073847678667037], ["tassili", 3.6073847678667037], ["shoshones", 3.6073847678667037], ["shorthairs", 3.6073847678667037], ["discgolf", 3.6073847678667037], ["epimetheus", 3.6073847678667037], ["coalescing", 3.6072494063369525], ["algebra", 3.6067633458353843], ["briefly", 3.6064503248244253], ["mudslinging", 3.606199784777974], ["jars", 3.60606499295163], ["threats", 3.605630595832987], ["tread", 3.605202421028285], ["tarvaris", 3.6051287423708525], ["fiorelli", 3.6051287423708525], ["buffalos", 3.6047799489553753], ["pesticides", 3.604531662225057], ["saadian", 3.6044123018417644], ["medida", 3.6044123018417644], ["carving", 3.6038327810582356], ["kimbap", 3.6037658477910965], ["baseless", 3.6037580693195923], ["babywearing", 3.603268510821641], ["commuters", 3.603066985737144], ["sweetness", 3.603046699768373], ["westwood's", 3.6026455292957937], ["dealing", 3.602413173550945], ["nonviolent", 3.6020215086466094], ["dousing", 3.6019843095935307], ["distinct", 3.6019053331423794], ["touchscreens", 3.6017083166190775], ["cheesier", 3.601708316619077], ["manta's", 3.601708316619077], ["trapezoid", 3.601708316619077], ["crabcakes", 3.601708316619077], ["khula", 3.601708316619077], ["miko's", 3.601708316619077], ["jagua", 3.601708316619077], ["shyama", 3.601708316619077], ["garnishing", 3.6007678000198817], ["undulating", 3.6002643327578903], ["tardigrades", 3.600037377942845], ["sjaak", 3.5999217549540883], ["unfermented", 3.5997889410173616], ["coordinating", 3.5996199224794783], ["grimacing", 3.5992380091316845], ["kaylee's", 3.5992380091316845], ["lakeshores", 3.5992380091316845], ["calculator's", 3.5992380091316845], ["quincea\u00f1eras", 3.5992380091316845], ["phacelia", 3.5992380091316845], ["communicate", 3.598648637542793], ["arenal", 3.5983559412225734], ["venerating", 3.5981479090304855], ["pelota", 3.5980812186164397], ["leofric", 3.5979030252121897], ["shophouses", 3.597245233203193], ["dan's", 3.5970669399224033], ["j\u00e1nos", 3.596972342501235], ["renaissance", 3.596869314292355], ["bestowing", 3.596844898753272], ["aloo", 3.5967447220397575], ["avocado", 3.5959467186027365], ["countrymen's", 3.595908464257372], ["jeezy's", 3.595908464257372], ["remilia", 3.595908464257372], ["bagful", 3.595174605387115], ["morricone's", 3.595174605387115], ["naan", 3.594441712120635], ["compromiso", 3.5942281331993082], ["padhao", 3.5942281331993082], ["selloffs", 3.5942281331993082], ["chameleonic", 3.5942281331993082], ["victorien", 3.5942281331993082], ["sopes", 3.5942281331993082], ["horticulturists", 3.5942281331993073], ["tree's", 3.5940158808551312], ["sakura's", 3.5936440382723234], ["outskirts", 3.593605299745141], ["extravaganzas", 3.59353136130624], ["sugarplums", 3.59342852738547], ["bigram", 3.59342852738547], ["schans", 3.59342852738547], ["height's", 3.59342852738547], ["gittel", 3.59342852738547], ["beachside", 3.5932900843076703], ["lakshmi", 3.5922635948609236], ["alistair", 3.592130740196607], ["reintegrating", 3.5920992391893023], ["stalybridge", 3.5911770057517702], ["doordarshan", 3.5911770057517702], ["mris", 3.59117700575177], ["squabbled", 3.5911770057517693], ["perceptron", 3.5911770057517693], ["tugging", 3.5910049527152026], ["schwerin", 3.59012726093863], ["computations", 3.5900387563046223], ["walls", 3.589911834170934], ["abba's", 3.589302891877565], ["tenzing", 3.5888450726311523], ["migrations", 3.5884309910489867], ["hairdos", 3.587976313233275], ["sk\u00e5l", 3.5879763132332743], ["meadowlarks", 3.5879763132332743], ["keycards", 3.5879763132332743], ["moment", 3.5873040132882004], ["heathenish", 3.5872436245844623], ["shyamala", 3.5872436245844623], ["catalonian", 3.586428341942511], ["saffron", 3.5862194886731227], ["intruders", 3.5861660610925816], ["multilingual", 3.5861121341888578], ["luigi", 3.585858102117102], ["contrasted", 3.5855857270280813], ["berber", 3.5855729189708834], ["palettes", 3.585314457839105], ["pebbles", 3.585132887204006], ["m\u00e2ch\u00e9", 3.58487274517741], ["chianti", 3.5843583152345198], ["wrongdoers", 3.5839221027098476], ["taproom", 3.5839221027098476], ["habang", 3.583922102709847], ["mutis", 3.583922102709847], ["rooflines", 3.583922102709847], ["sanader", 3.583922102709847], ["lani's", 3.583922102709847], ["kreon", 3.583922102709847], ["chocolaty", 3.582805963780321], ["assistance", 3.5826854121681766], ["correlations", 3.582577291414271], ["contributed", 3.582161086040855], ["paddies", 3.5819171113237154], ["twain's", 3.5818014387437467], ["plantains", 3.5817766839114578], ["acorns", 3.5815749281962836], ["petunias", 3.5815248304003426], ["heinkel", 3.5815248304003426], ["reindeers", 3.5812566272828428], ["nces", 3.5808418086808746], ["rcaf", 3.5808181626161693], ["avro", 3.5807893036690173], ["aquarium's", 3.580645694884463], ["laugher", 3.5805600744709074], ["delicates", 3.580221615252222], ["wreckage", 3.579892841174256], ["lurker", 3.579265239270775], ["inputs", 3.5791121189834576], ["antelopes", 3.5790975921869688], ["profiteroles", 3.5790855006818987], ["catchier", 3.5790071116701005], ["gentleman", 3.578883697631767], ["microcontrollers", 3.5786204427945965], ["satiating", 3.5786204427945965], ["faring", 3.578620442794596], ["meditators", 3.5786204427945956], ["usual", 3.578576961555004], ["botanists", 3.5783806692305897], ["elphaba", 3.5782423529432705], ["utis", 3.5777683903082167], ["small", 3.5777306475388193], ["extravagant", 3.577441826765628], ["safer", 3.5770299448200205], ["worcester's", 3.5769826302349648], ["presentation", 3.576786692188615], ["ensuing", 3.576612840163864], ["affirmed", 3.576578493429857], ["ballistae", 3.5758136816629924], ["barbells", 3.5758136816629924], ["hyundais", 3.5758136816629924], ["peahens", 3.5758136816629924], ["raimunda", 3.5758136816629924], ["erratic", 3.575613747993344], ["monta\u00f1a", 3.575306905347564], ["superintelligent", 3.575306905347564], ["nyama", 3.575306905347564], ["penske's", 3.5745768038761843], ["remy", 3.574362012733636], ["unwelcoming", 3.573880548168378], ["greco's", 3.573880548168378], ["langstroth", 3.573880548168378], ["iconostasis", 3.5732158380266217], ["stereolab", 3.5732158380266217], ["ripened", 3.5732158380266212], ["discolored", 3.5732158380266212], ["greenhorns", 3.5725805770683605], ["ihram", 3.5725805770683605], ["jimmie's", 3.5725805770683605], ["stuffies", 3.5725805770683605], ["incongruous", 3.572388287465648], ["eilish", 3.572375031899666], ["salesforce's", 3.5721222993187047], ["cortado", 3.571874114624803], ["lalbagh", 3.571390906546526], ["minnewanka", 3.571390906546526], ["sayajirao", 3.571390906546526], ["merneptah", 3.571390906546526], ["devotion", 3.57113888248961], ["individualized", 3.5711223809144546], ["synergy", 3.5708020913051173], ["shocked", 3.570760304417017], ["entwine", 3.5706972979028], ["listener's", 3.5704739897540883], ["stratocumulus", 3.568816003225296], ["amper", 3.568816003225296], ["rijeka", 3.568168598961672], ["traveling", 3.5681590706327917], ["polo's", 3.567588086901588], ["retro", 3.56721150760267], ["greyback", 3.566691707985281], ["darkplace", 3.566691707985281], ["brutalist", 3.5666917079852807], ["cravings", 3.566358216474483], ["codemasters", 3.5660638798374227], ["predictive", 3.5658964589979933], ["backend", 3.5658349480266005], ["siku", 3.565763123838448], ["cornered", 3.565664730816267], ["cosplay", 3.565630368082095], ["garnished", 3.56562712982152], ["insufficient", 3.565269981018194], ["agreeing", 3.5652235560087497], ["scowls", 3.5650930115675377], ["synchronizing", 3.564792848158492], ["sartorially", 3.5645075734427296], ["pricked", 3.56445721875587], ["transpiring", 3.564429130900237], ["smartwatch", 3.5642105311449437], ["melodic", 3.5641962715079294], ["diversities", 3.5639711193445596], ["alfonso's", 3.563872188848533], ["saosin", 3.563872188848533], ["melissa's", 3.56365105009271], ["calvados", 3.5633922706975976], ["hypervisors", 3.5633922706975976], ["slyness", 3.5627154630488422], ["karanth", 3.5627154630488422], ["inhabiting", 3.5623949596786506], ["parse", 3.5619826681705042], ["updos", 3.561497856943904], ["hypnobirthing", 3.561497856943904], ["brim", 3.561028152154075], ["georgiadis", 3.560948243077092], ["facepalm", 3.5606012059225183], ["gastropod", 3.5604887658844366], ["multicolored", 3.5602144342927535], ["tusayan", 3.5599493796147983], ["huasteca", 3.5599493796147983], ["sroi", 3.5599493796147983], ["genders", 3.559688386258521], ["expressionless", 3.559384577238213], ["gloated", 3.559384577238213], ["homogenize", 3.559233775940319], ["intern's", 3.558900330143864], ["mammatus", 3.558660010045941], ["catchiness", 3.5585299420631173], ["mysql", 3.5584333531172936], ["calamares", 3.5582768640500273], ["chagatai", 3.5582768640500273], ["git's", 3.5582768640500273], ["scriabin's", 3.5582768640500273], ["gruit", 3.5582768640500273], ["splashes", 3.557745156878294], ["palate", 3.557276528010785], ["stubborn", 3.5571445710126777], ["tourist's", 3.557054368563066], ["embezzling", 3.5568532046531267], ["reconsidering", 3.5560081438241196], ["silhouette", 3.555745339572027], ["tracy's", 3.555615025662345], ["socializing", 3.5555556650925544], ["piling", 3.5555264122037618], ["updating", 3.5554251396796666], ["battered", 3.5552271262813866], ["tylko", 3.554494911719576], ["lowertown", 3.554494911719576], ["terpsichore", 3.5543029117783607], ["fdr's", 3.554118176699786], ["waving", 3.553557364390967], ["palpably", 3.5532902898661733], ["mountaintops", 3.552594112301545], ["trebuchet", 3.55211214321834], ["taskmaster", 3.55211214321834], ["kasra", 3.55211214321834], ["equips", 3.551950875996583], ["hangout", 3.551654272589295], ["ferns", 3.551654272589295], ["schooler", 3.5516274200260884], ["nuestro", 3.551488418522428], ["monstrosities", 3.5513547632304467], ["interesse", 3.5511267045723827], ["citigroup's", 3.5511267045723827], ["chocolatier", 3.550959608454155], ["chamorro", 3.5506496163451677], ["meant", 3.5503760474192263], ["jazz's", 3.550368176140955], ["synchronicity", 3.5503384969921465], ["preciousness", 3.550236958039312], ["databases", 3.550105903566957], ["messages", 3.5499419767216183], ["universes", 3.549461313260714], ["momo's", 3.5493217958945236], ["peacebuilding", 3.5487748433513686], ["bowls", 3.5482594778991428], ["suraj", 3.5479863378368215], ["counterattacks", 3.5475253138534595], ["bedraggled", 3.5471639272974382], ["hardcovers", 3.5469862289356384], ["flair", 3.546938823197291], ["offline", 3.54672951249711], ["intellectual", 3.546346057894512], ["preteen", 3.546106723137347], ["crowd", 3.5457552307546876], ["novelty", 3.5450424899309128], ["turrell's", 3.54453834333941], ["exercise's", 3.54453834333941], ["zlatni", 3.54453834333941], ["braose", 3.54453834333941], ["compartmentalize", 3.5445383433394095], ["rajinder", 3.5445383433394095], ["rotis", 3.5445383433394095], ["glenda's", 3.5445383433394095], ["dionna", 3.5445383433394095], ["pong's", 3.5445383433394095], ["aparthotel", 3.5445383433394095], ["coll's", 3.5445383433394095], ["amoudi", 3.544538343339409], ["seders", 3.544538343339409], ["frauenkirche", 3.544538343339409], ["lafitte's", 3.544538343339409], ["crescendoing", 3.544538343339409], ["gmps", 3.544538343339409], ["papou", 3.544538343339409], ["rendition", 3.544475496914882], ["struggle", 3.5444235920980787], ["shatter", 3.5438977641207337], ["illnesses", 3.5438945703497], ["omelettes", 3.543629487353938], ["fizzy", 3.543465865777128], ["ahura", 3.542455770869762], ["mersenne", 3.541916643381318], ["underbelly", 3.5413160357545554], ["pokal", 3.541316035754555], ["primeira", 3.541000879922903], ["productively", 3.5409759932456013], ["unsportsmanlike", 3.540933728074996], ["silencing", 3.540846115898431], ["signpost", 3.540846115898431], ["synesthesia", 3.5405557159873426], ["kvass", 3.540288297364182], ["enthralling", 3.5398846824947623], ["lupines", 3.539430939386471], ["banksia", 3.539325786952146], ["enhances", 3.5393257869521455], ["timotheus", 3.5387935486176274], ["disjointed", 3.538700829598732], ["chanterelle", 3.5386601702990452], ["ch\u00e9rie", 3.5385204514152337], ["nasdaq's", 3.5377996772874334], ["josef's", 3.5377996772874334], ["starfruit", 3.5377996772874334], ["gnomish", 3.5377996772874334], ["worthlessness", 3.5376779465458497], ["emptiness", 3.537677946545849], ["hikes", 3.5376596730384082], ["tourism", 3.5368461384005916], ["exterior", 3.5368214517432093], ["crux", 3.536784665713354], ["fictionalized", 3.5365385172172896], ["heorot", 3.5362567116960926], ["otters", 3.5362071634725862], ["kangaroo's", 3.535829403920885], ["addiction's", 3.5356769974810605], ["vasanth", 3.5356769974810605], ["greenberry", 3.5356769974810605], ["currents", 3.5355350537258934], ["unsold", 3.5354318219039524], ["entanglements", 3.535010014459465], ["augustine's", 3.535010014459465], ["landau's", 3.5350100144594645], ["pozna\u0144", 3.5346373982462826], ["globetrotting", 3.533887687259664], ["conspiratorial", 3.5337973180565627], ["tumaini", 3.533321449847829], ["rockfield", 3.533321449847829], ["wabanaki", 3.533321449847829], ["rooibos", 3.533119083212672], ["sufficed", 3.5330826110510496], ["frittatas", 3.532801461407822], ["stojan", 3.532801461407822], ["bribri", 3.532801461407822], ["kaitlyn", 3.5325229591247], ["mar\u00eda's", 3.532230918536148], ["esterbrook", 3.532230918536148], ["cellists", 3.5319243278826855], ["sameness", 3.5309055035573347], ["ariana", 3.530837822792459], ["cellist", 3.5307194804024133], ["franticly", 3.5301296508868107], ["karijini", 3.5301296508868107], ["wilde's", 3.529474055380217], ["bowie's", 3.528721753477826], ["hooted", 3.5284511799051907], ["lastname", 3.5278777635822323], ["informational", 3.527871970996556], ["recited", 3.5275862034364818], ["daisy", 3.527550423252758], ["millford", 3.5271631553818628], ["chhe", 3.5271631553818628], ["hwei", 3.5271631553818628], ["lagers", 3.5269084405845796], ["outrage", 3.526892484101253], ["ratatouille", 3.5268691944979893], ["giggling", 3.5259195604684974], ["mossy", 3.525882878374465], ["dvo\u0159\u00e1k", 3.5258828783744645], ["pharaohs", 3.525526728588771], ["rijksmuseum", 3.5254696244037222], ["marley", 3.524477240910093], ["absurd", 3.524190067664683], ["trustingly", 3.5238503763549383], ["decimal", 3.5236608269192167], ["scratching", 3.5235445211956633], ["userid", 3.5232711132793733], ["developers", 3.523121644795015], ["jiro", 3.5231211932285973], ["hephaestus", 3.5226584770224996], ["terrazza", 3.5226584770224996], ["exp\u00e9rience", 3.5226584770224996], ["beginner", 3.521996792847779], ["coyoac\u00e1n", 3.5216703540275422], ["namdaemun", 3.5216703540275422], ["faces", 3.521389501022562], ["chef", 3.5209358880178785], ["osteopenia", 3.5209080877171464], ["abcdef", 3.5205887599384673], ["calakmul", 3.5205887599384673], ["hansberry's", 3.5205887599384673], ["pqrs", 3.5205887599384673], ["chamorros", 3.5205887599384673], ["hotheaded", 3.520413213253853], ["model's", 3.5202572544055166], ["untreatable", 3.5200435661699907], ["title", 3.5200317990711545], ["irresistible", 3.5194862672517395], ["secondi", 3.5193997735284914], ["candlebox", 3.5193997735284914], ["leathercraft", 3.5193997735284914], ["significances", 3.5193997735284914], ["uncrossing", 3.5193997735284914], ["ngawang", 3.5193997735284914], ["diye", 3.5193997735284914], ["beyhive", 3.5193997735284914], ["tinkercad", 3.5193997735284914], ["dough", 3.5193985582661385], ["enchiladas", 3.519308122492723], ["vortexes", 3.5192603361365133], ["darry", 3.5192603361365133], ["raul's", 3.5192603361365133], ["sevenths", 3.519220212315556], ["anisa", 3.5187598826605773], ["arnav", 3.5187598826605773], ["nsf's", 3.5187598826605773], ["kois", 3.5187598826605773], ["shotover", 3.518086564657772], ["leapt", 3.51684663753207], ["decision", 3.516694986044006], ["outsmarted", 3.515903426591908], ["moher", 3.515521341788143], ["truffled", 3.515484750361218], ["felicia's", 3.515484750361218], ["obinna", 3.515484750361218], ["bluehost", 3.515484750361218], ["attempting", 3.515467411926698], ["rica's", 3.5150005238115813], ["paloma's", 3.5150005238115813], ["kohana", 3.5150005238115813], ["mtls", 3.5150005238115813], ["greensboro's", 3.5150005238115813], ["gastrique", 3.5150005238115813], ["mockumentaries", 3.5150005238115813], ["shamima", 3.5150005238115813], ["shitou", 3.5150005238115813], ["jaswant", 3.5143603853078824], ["maximization", 3.514214346105517], ["flickers", 3.5141959917621852], ["weeks", 3.51413077795142], ["jolt", 3.5138190110304683], ["gardenias", 3.5138053519231716], ["lisbon's", 3.513299756294703], ["cosmic", 3.513148899303866], ["inconvenience", 3.5128906795595936], ["lyric's", 3.5125093824056197], ["wehrmacht's", 3.5125093824056197], ["clanking", 3.5125093824056193], ["hypermasculine", 3.5121681301582277], ["unveiled", 3.51205187887089], ["speculations", 3.5116763319033044], ["sephardic", 3.5114960367489574], ["cohesion", 3.511319894990836], ["trekkie", 3.5108652290776847], ["creator's", 3.510829477250014], ["ubc's", 3.510537975537592], ["thuy", 3.5101920415628642], ["unflappable", 3.50969886389633], ["alexandra's", 3.509559346530139], ["serenade", 3.509540605391397], ["peux", 3.509234176573081], ["superstardom", 3.5090929599060154], ["feedings", 3.5089782499272664], ["sentence", 3.5088084640652997], ["deserved", 3.5087076839991305], ["checking", 3.508420947060252], ["inboxes", 3.5083306935956196], ["moistness", 3.5080906482922174], ["antone's", 3.5078707053048634], ["skippy's", 3.5078707053048634], ["gilgamesh's", 3.5078707053048634], ["ferragni", 3.5078707053048634], ["lodgings", 3.5075752708363535], ["leche", 3.5072542944829572], ["stretching", 3.5069402035675465], ["irritable", 3.506931656038125], ["affiliations", 3.5067299343437894], ["coelho's", 3.506010752650938], ["bakersfield's", 3.506010752650938], ["pooches", 3.5056021266890904], ["unripe", 3.5054575359641285], ["yolanda's", 3.505008175941572], ["raphson", 3.505008175941572], ["friezes", 3.5046622869768416], ["repast", 3.5046305421349744], ["uptick", 3.5046120200230253], ["scca", 3.504065850236375], ["equaling", 3.504046100675433], ["balki", 3.5036791117159], ["segmenting", 3.502837892241299], ["daffy's", 3.5028378922412986], ["altagracia", 3.5028378922412986], ["adopters", 3.5027348190923977], ["amma", 3.502601341870402], ["esme's", 3.5021395593281723], ["elementary's", 3.5021395593281723], ["hadestown", 3.5021395593281723], ["ikenna", 3.5021395593281723], ["unspoiled", 3.5016608633796587], ["stability", 3.501256750137162], ["olive's", 3.501145154857806], ["pilates", 3.500940521716335], ["satay", 3.5007408867981464], ["digestion", 3.5004053659272425], ["horizon's", 3.499896802491306], ["hegel's", 3.499896802491306], ["rewatched", 3.499714414873055], ["figuratively", 3.499628485795528], ["meditate", 3.4995499720585634], ["crouched", 3.4995381174152747], ["lorawan", 3.499095544065827], ["asylum's", 3.499095544065827], ["estense", 3.499095544065827], ["trays", 3.498601941246131], ["darker", 3.498575968934297], ["fledged", 3.498186688080356], ["berries", 3.4979924567608798], ["stroked", 3.4972941513321922], ["lapses", 3.4966103909805533], ["verbindung", 3.4962037295666], ["gaggan", 3.4962037295666], ["contrapposto", 3.4962037295666], ["cto's", 3.4962037295666], ["dibango", 3.4962037295666], ["abdou", 3.4958777735970332], ["workshop", 3.494763975113792], ["hakata", 3.4944003470929115], ["infatuations", 3.4940623226272978], ["aragon's", 3.493611068291429], ["kuusamo", 3.493611068291429], ["gigs", 3.4935967369979486], ["divers", 3.493404019697792], ["precedents", 3.4932358146858125], ["legendary", 3.492475330143418], ["rajalakshmi", 3.492026664178826], ["hockeyville", 3.492026664178826], ["qanat", 3.492026664178826], ["rvers", 3.492026664178826], ["mathcounts", 3.492026664178826], ["onam", 3.491932605388108], ["conversation's", 3.491722374647266], ["reebok's", 3.491722374647266], ["suzie's", 3.491722374647266], ["contributions", 3.4914350086156807], ["diego's", 3.4913702681893186], ["herbivores", 3.4913702681893186], ["exasperated", 3.4910243603142503], ["parrotfish", 3.4906752606710163], ["gogo's", 3.490629014923601], ["ni\u00e7oise", 3.490629014923601], ["crookedly", 3.490469114373215], ["kanun", 3.490469114373215], ["bironas", 3.490469114373215], ["c\u00e1diz", 3.489879574493521], ["standoffish", 3.4897853120195323], ["sommeliers", 3.4896000337698547], ["planting", 3.4895870252147283], ["briskets", 3.4891549317247317], ["anthropomorphize", 3.4891549317247312], ["serenaded", 3.4888163431821084], ["authenticate", 3.487825729845979], ["nonliving", 3.487775273784152], ["\u00e9clair", 3.487775273784152], ["deception", 3.486969331320071], ["jabir", 3.4869646273105914], ["drawn", 3.4867487757180617], ["critter", 3.4858638970786973], ["milanese", 3.4858031978806148], ["ferm\u00edn", 3.485641180534676], ["tom\u00e9", 3.485462704283753], ["adoring", 3.4852839580233725], ["misinterpreted", 3.4852418607732645], ["macarthur's", 3.4851581906803735], ["mulberries", 3.4851581906803735], ["luminescence", 3.4843687109679076], ["hennessy's", 3.484368710967907], ["upperclassmen", 3.484332070043453], ["amenities", 3.484260279772], ["syncretism", 3.484205775793207], ["commonality", 3.4838125467682315], ["polluted", 3.4837384870672956], ["confrontation", 3.483572283833972], ["actively", 3.4835722838339716], ["breakthrough", 3.48337408331714], ["sonorous", 3.483190564320074], ["populations", 3.483092056058422], ["identity", 3.4816961366929555], ["sailing", 3.481647276623147], ["careers", 3.4815046364670237], ["bhutan's", 3.4814931949400925], ["mismanaging", 3.4814931949400925], ["shrewd", 3.4813411480705674], ["fluminense", 3.4808878231447626], ["invasions", 3.4808509931058595], ["formatter", 3.4807876537110025], ["feeders", 3.4800095683709227], ["deterrents", 3.4800095683709227], ["tutankhamun's", 3.4799389915986754], ["observations", 3.4795551403781873], ["compelling", 3.478973662883817], ["internship", 3.478829395426643], ["decisiveness", 3.478676988986819], ["livorno", 3.478676988986819], ["salmons", 3.4782852901928782], ["onkar", 3.4782852901928782], ["audiologist", 3.4782852901928782], ["deskbound", 3.4782852901928782], ["hypercalcaemia", 3.4782852901928782], ["skimboarding", 3.4782852901928782], ["gargery", 3.4782852901928782], ["vigilia", 3.4782852901928782], ["shropshire's", 3.4782852901928782], ["healthyliving", 3.4782852901928782], ["thrombo", 3.4782852901928782], ["draggable", 3.4782852901928782], ["skya", 3.4782852901928782], ["windswept", 3.4781588070914164], ["possessiveness", 3.478078249401795], ["sneakers", 3.478039039906847], ["qlikview", 3.4774069353216173], ["townsperson", 3.4774069353216173], ["bendigo's", 3.4774069353216173], ["rxjs", 3.4774069353216173], ["bosques", 3.4774069353216173], ["hinterglemm", 3.4774069353216173], ["sacred", 3.4771411875409126], ["agile", 3.4770266110800847], ["respiration", 3.4769705811695015], ["legco", 3.476939353375225], ["belittle", 3.47673191232387], ["dosha", 3.4765752187621053], ["detoxes", 3.4765752187621053], ["longmen", 3.4763741444290357], ["snorkeled", 3.4763741444290357], ["raag", 3.475578453391172], ["birthstones", 3.4754072763593853], ["marzano", 3.475356768325896], ["rambutan", 3.475332586211391], ["coleridge's", 3.4748668279912143], ["continuous", 3.474566530857931], ["charred", 3.473647576472621], ["lupins", 3.473647576472621], ["industriousness", 3.473647576472621], ["monteverde", 3.473647576472621], ["teej", 3.473647576472621], ["jede", 3.473647576472621], ["norquay", 3.473647576472621], ["arthropods", 3.47291566590025], ["flecks", 3.47291566590025], ["cannoli", 3.47251461901128], ["arabica", 3.471760749978068], ["responsible", 3.4713919611632233], ["technologically", 3.4710080266424748], ["input", 3.470933789303502], ["stash", 3.470787430258976], ["betta", 3.4704304914284236], ["clamoring", 3.470108655464216], ["viewport", 3.470051668008364], ["sage's", 3.469613142585893], ["instance", 3.469246517295673], ["redundancies", 3.469206991523471], ["linzer", 3.469206991523471], ["afrobeat", 3.46905280454607], ["lionfish", 3.46905280454607], ["desperate", 3.469017629258594], ["echidnas", 3.4685839502678504], ["longbows", 3.4685839502678504], ["leaderboards", 3.4684784580552512], ["guessed", 3.468448920235723], ["coach's", 3.468392436417443], ["monstrous", 3.4683376649466067], ["asel", 3.468311712299852], ["frosti", 3.468311712299852], ["erie's", 3.4679272840453303], ["panth\u00e9on", 3.4679272840453303], ["castilian", 3.4670436837416845], ["assortments", 3.4669642718521834], ["artful", 3.466782660313585], ["illustrate", 3.4663926734306982], ["lining", 3.466292309159367], ["influenced", 3.4654031597563093], ["necessitate", 3.4654008904425972], ["physically", 3.465273604636482], ["darcy's", 3.4650917942645605], ["doubted", 3.46507623829691], ["fearfully", 3.463501500173843], ["frederick's", 3.4633093396378816], ["ibiza's", 3.4633093396378816], ["yarns", 3.4622767307883477], ["metalwork", 3.4622095272086533], ["cython", 3.4621072190757025], ["javeria", 3.4621072190757025], ["manolo's", 3.4621072190757025], ["sabemos", 3.4621072190757025], ["joyride", 3.4619370073146856], ["ocean's", 3.4613948513351516], ["leafed", 3.4609235926760182], ["sensed", 3.4605887509971605], ["roading", 3.46058875099716], ["lahore's", 3.4605887509971596], ["frugal", 3.460183826176216], ["ashgabat", 3.4600226169447557], ["chambord", 3.4600226169447557], ["martial's", 3.459932528611459], ["seraphine", 3.459932528611459], ["woodblocks", 3.4598083431002205], ["wolfgang's", 3.458889647310762], ["omnivores", 3.458859105238821], ["aditi", 3.458859105238821], ["pirouette", 3.458859105238821], ["relentlessness", 3.458170449983186], ["indispensable", 3.457786172013842], ["signore", 3.457639983954314], ["nuptials", 3.457201735744159], ["took", 3.4571714901393302], ["unamused", 3.456747919974966], ["unlock", 3.456304337853133], ["frau", 3.455621932760767], ["concise", 3.4555556620118266], ["coffeehouses", 3.4554609399465863], ["guesthouses", 3.4552684887664165], ["bookstores", 3.455096721591593], ["trip", 3.455023125630792], ["marketplace", 3.4548792191327], ["hamida", 3.4548031954067664], ["yongle", 3.4548031954067664], ["childlike", 3.453991870159958], ["berlin's", 3.4537796799857463], ["polonia", 3.4531842623255073], ["misappropriating", 3.4530407205596045], ["koalas", 3.45301634672802], ["thich", 3.452971102803142], ["confederacy's", 3.452770360340331], ["vocs", 3.452721988662545], ["childproofing", 3.452072125687077], ["ramadan's", 3.452072125687077], ["tipitina's", 3.452072125687077], ["alissa's", 3.452072125687077], ["ljubo", 3.452072125687077], ["sandburg's", 3.452072125687077], ["revolving", 3.451925211138927], ["roadkill", 3.4517749431693696], ["dottie", 3.451637136759949], ["zoho", 3.4510914233786427], ["measures", 3.4510890135332275], ["hablar", 3.4510373318844123], ["thirstier", 3.4510373318844123], ["gdpr", 3.450958348729121], ["alright", 3.4508169758885265], ["bol\u00edvar", 3.4506432878867104], ["ceausescu's", 3.45064328788671], ["\u00feingvellir", 3.45064328788671], ["atonality", 3.45064328788671], ["netsec", 3.45064328788671], ["sculpture's", 3.45064328788671], ["forcing", 3.4503673987114287], ["ancestor", 3.449959965860984], ["cette", 3.449950943727824], ["fitr", 3.449299579441271], ["pi's", 3.4492117439541454], ["everbright", 3.4484152696217305], ["tiago", 3.4483022216519688], ["taylor's", 3.448171207129869], ["certains", 3.448161167840755], ["reciprocated", 3.4480411250954717], ["seascapes", 3.4479802298484397], ["rolleiflex", 3.4475870757425984], ["avons", 3.4475870757425984], ["raising", 3.447360015153059], ["clouds", 3.447039093805553], ["intensified", 3.446509704781429], ["esoterica", 3.4460789449133147], ["baetica", 3.4460789449133147], ["tinsley's", 3.4460789449133147], ["selkies", 3.4460789449133147], ["trouver", 3.4460789449133147], ["autoload", 3.4460789449133147], ["guiraud", 3.4460789449133147], ["stdin", 3.4460789449133147], ["tamb\u00e9m", 3.4460789449133147], ["aava", 3.4460789449133147], ["motivated", 3.4451500018933814], ["vibrato", 3.444939745262104], ["sedentary", 3.4448803087585627], ["abstract", 3.4447005133353494], ["fragging", 3.4443071819596316], ["pascals", 3.4443071819596316], ["capsize", 3.4435024372909493], ["sentience", 3.4435024372909493], ["farro", 3.4435024372909493], ["acerca", 3.4434583525597686], ["ethereally", 3.4434583525597686], ["zuppa", 3.4434583525597686], ["ghasemi", 3.4434583525597686], ["pietro's", 3.4434583525597686], ["apprehending", 3.443458352559768], ["anand's", 3.4432658192439978], ["zusammenarbeit", 3.4427812617124407], ["protectants", 3.4427812617124407], ["quercetin", 3.442470485285133], ["recurse", 3.4416323914360074], ["diferencia", 3.4416323914360074], ["betty's", 3.4415221463084706], ["repositories", 3.4403022807860033], ["kolton", 3.4399796901435566], ["triangles", 3.4396314423712187], ["wore", 3.4392292003028104], ["decisively", 3.4388735071194048], ["scientist's", 3.438525895171039], ["wreak", 3.4383242781716037], ["fewer", 3.43782862649814], ["vipassana", 3.4377751402267767], ["camerawork", 3.4377751402267767], ["restringing", 3.4377751402267767], ["salesclerk", 3.4377751402267767], ["seychellois", 3.4377751402267767], ["cahal", 3.4377751402267767], ["cloudless", 3.437463747551032], ["saltimbocca", 3.4367652180351707], ["fallbacks", 3.4367652180351707], ["paired", 3.436409350261916], ["reptiles", 3.4362674013670986], ["plants", 3.436018812695182], ["sparrows", 3.4354756250828125], ["haumea", 3.435475625082812], ["attrib", 3.435475625082812], ["auxins", 3.435475625082812], ["bellbottoms", 3.435475625082812], ["rutland's", 3.435475625082812], ["cessna's", 3.435475625082812], ["sherries", 3.434962561220046], ["rylee", 3.434962561220046], ["hiroto", 3.434962561220046], ["leads", 3.4346259362319653], ["balos", 3.4337715201100534], ["tangaroa", 3.4337715201100534], ["mazurkas", 3.4337715201100534], ["bowings", 3.4337715201100534], ["europ\u00e4ische", 3.4337715201100534], ["faisal's", 3.4337715201100534], ["drooped", 3.4333627864450005], ["ivanov", 3.4332073427421617], ["dining", 3.4326542199800874], ["pauses", 3.4322946291336613], ["bachchan's", 3.4321754563747593], ["harp's", 3.431414779190279], ["hrafn", 3.431414779190279], ["nazira", 3.431414779190279], ["simone's", 3.4312477190744466], ["pledging", 3.4307790760898254], ["latin's", 3.429862102819605], ["gulping", 3.429862102819605], ["fungi", 3.4296929536388263], ["texture", 3.429621994464281], ["treading", 3.429586375655543], ["escribir", 3.429268803718616], ["bozz", 3.429268803718616], ["omelette", 3.4291405907773003], ["taoist", 3.429100351364264], ["xuanzang", 3.4287619652906347], ["painting's", 3.4286592051917752], ["nuestra", 3.4286592051917752], ["oboist", 3.428535270284665], ["moje", 3.428535270284665], ["junctures", 3.428442921653779], ["maia's", 3.4283239714266416], ["vahan", 3.4279416873085076], ["using", 3.4277415995419007], ["beeswax", 3.427523135209935], ["disasters", 3.4272603572837013], ["huseyin", 3.427039834720423], ["misshapen", 3.4269340618860187], ["surpassing", 3.426706393006775], ["famed", 3.4265800258581773], ["faulkner's", 3.426387065228096], ["thorgerson", 3.4263870652280954], ["vegan's", 3.4263870652280954], ["ramanujan's", 3.4263870652280954], ["occitania", 3.4263870652280954], ["dzie\u0144", 3.4263870652280954], ["michaelangelo's", 3.4263870652280954], ["hassans", 3.4263870652280954], ["halld\u00f3r", 3.4263870652280954], ["tambu", 3.4263870652280954], ["maimouna", 3.4263870652280954], ["akamba", 3.4263870652280954], ["vacherin", 3.4263870652280954], ["hargreeves", 3.4263870652280954], ["bedtime", 3.426142951017122], ["kibble", 3.4259958358303764], ["henna", 3.4259602181250925], ["iupac", 3.425831712569523], ["leniency", 3.4252252145356246], ["thirties", 3.425164810626944], ["fruitless", 3.425151847320568], ["firmness", 3.4248891092457026], ["brainstorms", 3.4247229627194855], ["unappreciated", 3.4246769809396422], ["crosscheck", 3.42402403966587], ["galaxies", 3.4240240396658694], ["distill", 3.42318296277084], ["romanesque", 3.4225132528310036], ["peepal", 3.422312883224257], ["harmandir", 3.422312883224257], ["michaelides", 3.422312883224257], ["hypervitaminosis", 3.422312883224257], ["culinaire", 3.422312883224257], ["souvent", 3.422312883224257], ["visibility", 3.421596354200199], ["entirely", 3.420644913111886], ["souvenir", 3.4205568929894152], ["overgrazed", 3.4203138686896635], ["peixe", 3.4203138686896635], ["castello", 3.419873021377852], ["amra", 3.4195706453370582], ["refuge", 3.4193241474959803], ["carmen's", 3.419182719001796], ["windbreakers", 3.418740531571623], ["exposing", 3.4183045772975955], ["longship", 3.417598953633039], ["trusted", 3.41742850629282], ["azamat", 3.4171896603451795], ["disturbances", 3.416999703405553], ["glimpse", 3.4169953188952538], ["day's", 3.4165678715718157], ["s\u00e9ances", 3.4161130410445035], ["ulmus", 3.4161130410445035], ["\u044f\u0437\u044b\u043a", 3.4161130410445035], ["biotechnologist", 3.4161130410445035], ["falkirk's", 3.4161130410445035], ["inflicted", 3.415931952129037], ["bartering", 3.415925754145323], ["irrigation", 3.4159222377382368], ["townsmen", 3.415744984906913], ["shockingly", 3.4155565314012257], ["ollies", 3.415544068186235], ["reblogging", 3.415544068186235], ["gaslit", 3.415544068186235], ["snacker", 3.415544068186235], ["noticeable", 3.4153708250625567], ["rather", 3.415369599873496], ["tenderly", 3.415218536451405], ["mauryan", 3.415218536451404], ["interessante", 3.4151029543972395], ["kilims", 3.4151029543972395], ["eclectus", 3.4151029543972395], ["customer's", 3.4147196265146036], ["ransomware", 3.414713435953624], ["citronella", 3.4146003391665105], ["immunologist", 3.4145719374169645], ["slumped", 3.414516726539343], ["spiritually", 3.4144810518184174], ["comicbooks", 3.413904252061544], ["lleida", 3.413904252061544], ["flamengo", 3.413757101016197], ["gareth's", 3.4136996796590955], ["harmonica", 3.41328025141832], ["array's", 3.4129725096346717], ["paleoclimatology", 3.4129725096346717], ["\u00fcbermensch", 3.412771118679952], ["microkernel", 3.412771118679952], ["phuket's", 3.412771118679952], ["rappelled", 3.4126218677782343], ["devasted", 3.4126218677782343], ["ristretto", 3.4126218677782343], ["kowalski's", 3.4126218677782343], ["wetland", 3.412517132580246], ["biri", 3.412415448793339], ["verifications", 3.411618155464182], ["vxlan", 3.411618155464182], ["hoopoes", 3.411618155464182], ["erfolg", 3.411618155464182], ["buren's", 3.411618155464182], ["vanuatu's", 3.411618155464182], ["glassblowers", 3.411618155464182], ["rockiest", 3.411618155464182], ["rabiya", 3.411618155464182], ["coulrophobia", 3.4116181554641813], ["onkel", 3.4116181554641813], ["employing", 3.411271622385922], ["barely", 3.41101592807575], ["cmu's", 3.4106811551468335], ["soaked", 3.4106680330417327], ["gifting", 3.4106208978916985], ["marsellus", 3.41038250962736], ["julies", 3.41038250962736], ["liis", 3.41038250962736], ["ollie", 3.4097572728339287], ["melburnians", 3.409564201126154], ["unenjoyable", 3.409564201126154], ["kindling", 3.4092262754767098], ["leatherback", 3.4089212241159887], ["contacted", 3.4088888609398103], ["deteriorating", 3.408830316317735], ["stove's", 3.4082099455186627], ["emilie's", 3.4082099455186627], ["destination", 3.40812454439589], ["yamuna", 3.4075550929527116], ["zora", 3.406942570966058], ["loyalties", 3.4067636790374336], ["estos", 3.406650124033299], ["belgaum", 3.406149566710289], ["inequity", 3.40603710216008], ["kuznetsov", 3.4057218226687023], ["susan", 3.405557809733472], ["p\u00e9riode", 3.40553683967904], ["dlss", 3.40553683967904], ["fibrodysplasia", 3.40553683967904], ["sanuk", 3.40553683967904], ["terracycle", 3.40553683967904], ["grosbeaks", 3.40553683967904], ["cafecito", 3.40553683967904], ["stevenage's", 3.40553683967904], ["fumbled", 3.4053021908384564], ["quartet's", 3.405223369793209], ["showcases", 3.404648557598065], ["unusually", 3.40460613395888], ["farmers", 3.404423664997775], ["dashboards", 3.4036656655913045], ["mashups", 3.403499121300773], ["breakbeats", 3.403384161525008], ["arthropod", 3.402922442238699], ["peckish", 3.402756809605833], ["wickham's", 3.402756809605833], ["smirk", 3.4025956942265903], ["visitors", 3.4016211808162193], ["clotho", 3.4016134101402393], ["facilitator", 3.4014622049212377], ["rockier", 3.401296302522999], ["jutted", 3.401296302522999], ["kaikeyi", 3.400131225647804], ["zhiyuan", 3.400131225647804], ["orangutans", 3.3997246086631536], ["hideaway", 3.399477165587085], ["melpomene", 3.399477165587085], ["aioli", 3.399455523893899], ["cropping", 3.399433804908952], ["structures", 3.3988836385684094], ["biohackers", 3.3988723840240906], ["nauru's", 3.3988723840240906], ["puebloans", 3.3988723840240906], ["mannequin's", 3.3988723840240906], ["lse's", 3.3988723840240906], ["rockslides", 3.3988723840240906], ["memcached", 3.3988723840240906], ["mongol", 3.398653807021903], ["zac's", 3.398643040408435], ["maneuvering", 3.3980019410964752], ["session", 3.397510892857691], ["placing", 3.3969490356311045], ["prevail", 3.396766040574837], ["inherited", 3.3967402500562534], ["rudeness", 3.3967402500562534], ["seabeds", 3.396287690195936], ["normandy's", 3.396287690195936], ["jingdezhen", 3.396287690195936], ["troupe", 3.396285040158725], ["corsican", 3.396220781454994], ["standing", 3.3955796090376897], ["supernovae", 3.395554759585502], ["incorporated", 3.3954433950493224], ["found", 3.3953610082545795], ["overwhelm", 3.3953409315116123], ["rumbling", 3.395294623619855], ["oompah", 3.3952946236198547], ["scrunching", 3.3951014457282067], ["superiors", 3.395076976528598], ["attentive", 3.394870211791903], ["budged", 3.394853447083522], ["flit", 3.394387760739615], ["academics", 3.394027722640893], ["conventional", 3.3937062889482617], ["structs", 3.3932943658883845], ["bouldering", 3.3932075967173843], ["inuit", 3.393169450001514], ["fuels", 3.3931313234843588], ["spotlessly", 3.3930623457608022], ["pueda", 3.3930623457608022], ["lontong", 3.3930623457608022], ["budtender", 3.3930623457608022], ["delivering", 3.393059759585234], ["crossfire", 3.3930093291616497], ["weitere", 3.392629557196292], ["hamid's", 3.392629557196292], ["paragliding", 3.3925476214971857], ["patron", 3.392096792055566], ["emergencies", 3.3919669614807915], ["chilaquiles", 3.3915366882312328], ["obsolescence", 3.3904115158031765], ["jana", 3.3895033184089987], ["portrayal", 3.3894273485118664], ["voice", 3.38942540705894], ["stamina", 3.3885786562324753], ["learner", 3.3883571225860165], ["stonehenge", 3.387928893621946], ["realize", 3.3877239820461376], ["pub's", 3.387584615959067], ["esp\u00edritu", 3.387422282021173], ["necessitated", 3.3871328576976922], ["errors", 3.386876083039467], ["minutiae", 3.3859460237269077], ["nani's", 3.3859460237269072], ["tesserae", 3.3859460237269072], ["accusing", 3.3859041409370465], ["exoplanet", 3.3855005045132596], ["gentler", 3.385249373254116], ["young", 3.38483712588827], ["hilda's", 3.384778502143222], ["chunks", 3.384095167334609], ["bakongo", 3.383422964096709], ["fossilization", 3.383422964096709], ["croquetas", 3.383422964096709], ["spirometry", 3.383422964096709], ["stalinization", 3.383422964096709], ["ddlj", 3.383422964096709], ["laika's", 3.383422964096709], ["nezuko", 3.383422964096709], ["coiba", 3.383422964096709], ["poitier's", 3.383422964096709], ["wachau", 3.383422964096709], ["lylah", 3.383422964096709], ["ohayou", 3.383422964096709], ["pumblechook", 3.383422964096709], ["yvette's", 3.383422964096709], ["befana", 3.383422964096709], ["callicles", 3.383422964096709], ["archimedes", 3.382502304786751], ["micromanagement", 3.3824590259302996], ["maeve", 3.3824590259302996], ["trastevere", 3.3820976907069333], ["tidiness", 3.3820976907069333], ["latkes", 3.381900956465537], ["seared", 3.3817884602493193], ["flawless", 3.3816944146761907], ["goddesses", 3.3816372686434364], ["sundials", 3.3816356302540798], ["kaleo", 3.3816356302540798], ["achaemenid", 3.3812115765384756], ["harness", 3.381133610951487], ["entities", 3.380437235364548], ["scrutiny", 3.3802651026635524], ["establishments", 3.380244893393439], ["synthesizer", 3.3799312995310773], ["zhang", 3.379891207349218], ["fractions", 3.3798614662397655], ["fossilized", 3.379721188424831], ["damaging", 3.3796497330748383], ["russets", 3.3790346074636397], ["robotics", 3.378865157350581], ["refocusing", 3.37877905022442], ["glutenfree", 3.3783881084953746], ["hackerspace", 3.3782613729996886], ["abysses", 3.3782613729996886], ["katarzyna", 3.3779392964077353], ["cyprinidae", 3.3775182119778666], ["varias", 3.3775182119778666], ["crocodilians", 3.3774026385378177], ["khajuraho", 3.3774026385378177], ["taxied", 3.3773789788089337], ["discriminant", 3.3771573660150485], ["captions", 3.376801641736864], ["obi's", 3.376284940965703], ["regionalisms", 3.376284940965703], ["decis", 3.376284940965703], ["incredulous", 3.3760904685088833], ["arianism", 3.375653910218186], ["trekkies", 3.3756342371996166], ["several", 3.3752258739304954], ["belief", 3.374874338714043], ["ustad", 3.374400502859118], ["mandolin", 3.374071325168857], ["canine", 3.3739367314870923], ["bassist", 3.373929393332472], ["hummingbird's", 3.3732011424529125], ["cotangent", 3.3732011424529125], ["monsieur", 3.372881414997687], ["unrecognizable", 3.3722429577805633], ["barbell", 3.372069195760814], ["politeness", 3.371893085605663], ["chowders", 3.371855706099797], ["ncgs", 3.371855706099797], ["fauvist", 3.371855706099797], ["zareen", 3.371855706099797], ["zulaikha", 3.371855706099797], ["perceived", 3.3718337681966997], ["sashimi", 3.371230448910775], ["soon", 3.3712188981069144], ["boldly", 3.3710229846471402], ["complimented", 3.3705446199315663], ["unicef's", 3.3703847253140626], ["ertegun", 3.370216785470258], ["diversification", 3.3697669798440293], ["abstractness", 3.36949941280413], ["housesit", 3.36949941280413], ["lagman", 3.36949941280413], ["numeri", 3.36949941280413], ["colosseo", 3.3684851143655976], ["daran", 3.3682642590604335], ["believin", 3.367311426172439], ["garrus", 3.3673114261724386], ["botany", 3.3671637370748], ["roaring", 3.3668191291803082], ["maghrebi", 3.3666525900491417], ["overgrown", 3.3665058492762254], ["pasaje", 3.3659375741013773], ["schritt", 3.3659375741013773], ["ctes", 3.3659375741013773], ["rajkumari", 3.3659375741013773], ["alphie", 3.3659375741013773], ["hyperx", 3.3651588725226373], ["claudia's", 3.3646261698517277], ["impenetrable", 3.364433382218445], ["risd", 3.3643075801187616], ["humanely", 3.364066997482883], ["gratefulness", 3.363373050235395], ["wiltshire's", 3.3630408980479336], ["rmnp", 3.3630408980479336], ["ranging", 3.3630351121711195], ["leste's", 3.3623424471864487], ["nazareth's", 3.3623424471864487], ["eggbeater", 3.3623424471864487], ["raketa", 3.3623424471864487], ["greyville", 3.3623424471864487], ["cirrostratus", 3.3623424471864487], ["quahogs", 3.3623424471864487], ["reacted", 3.362079784910531], ["astonished", 3.361594428844472], ["hyperfocus", 3.361594428844472], ["anush", 3.361594428844472], ["erzs\u00e9bet", 3.361594428844472], ["thick", 3.361520782802504], ["courland", 3.3614384094242156], ["kannur", 3.361410644774227], ["coaxed", 3.3613886757272704], ["certifications", 3.3610403198726844], ["matsya", 3.3609566761545], ["outmatch", 3.3609566761545], ["era's", 3.360866811002731], ["concertmaster", 3.3608668110027304], ["brushing", 3.3605812900774907], ["hatched", 3.360495006281402], ["parallelogram", 3.360269297013487], ["deft", 3.3601362473397685], ["floodings", 3.359926971290482], ["herzegovina's", 3.359926971290482], ["jennys", 3.359926971290482], ["bigcommerce", 3.359926971290482], ["freqs", 3.359926971290482], ["shouted", 3.3596698012199657], ["jwst", 3.3595359638459], ["bushy", 3.359331575451832], ["abin", 3.3592759488197803], ["thoroughly", 3.359271375783888], ["steadier", 3.3591317223031942], ["imo's", 3.358798342815074], ["valiance", 3.358798342815074], ["desmos", 3.358798342815074], ["mthethwa", 3.358798342815074], ["elyon", 3.358798342815074], ["nobita", 3.3583976463149288], ["creak", 3.3582091373753435], ["gladstone's", 3.35819236779472], ["uniting", 3.3581025765549395], ["trendsetting", 3.357832514881499], ["takeshita", 3.357832514881499], ["shibuya", 3.3577602749478714], ["circles", 3.357675533046472], ["one's", 3.3573867188110884], ["storing", 3.356925547693299], ["mcluhan's", 3.356886313397911], ["assertive", 3.3559917714630645], ["unshakeable", 3.3557916765565863], ["meena", 3.3555503721728], ["credibility", 3.355472948306345], ["puja", 3.3553281114885514], ["gargantuan", 3.3552894689904345], ["slideshows", 3.3549825971521283], ["sluggish", 3.354467452962024], ["sea's", 3.3537438152888432], ["disorganized", 3.3533104997162484], ["enum", 3.3531846429199996], ["lauterbrunnen", 3.3529416761318735], ["checkmating", 3.3529416761318735], ["bmis", 3.3529416761318735], ["vieta", 3.3529416761318735], ["negat", 3.3529416761318735], ["edm's", 3.3529416761318735], ["marble's", 3.3529416761318735], ["sabich", 3.3529416761318735], ["chws", 3.3529416761318735], ["venture", 3.35286061600344], ["deliberated", 3.352399128935089], ["tenfold", 3.352144621850701], ["tomas", 3.35165638182269], ["paulius", 3.351014698157085], ["inheritable", 3.350785443542722], ["disembarking", 3.350748162885806], ["farm's", 3.350354529776834], ["journalistic", 3.3502249336455723], ["gravies", 3.3498726219660777], ["maze", 3.3498662710150637], ["recently", 3.3496832554782308], ["unwell", 3.3495887344557422], ["kiki's", 3.349588734455742], ["nambi", 3.349588734455742], ["vulnerable", 3.349358995859414], ["acupuncturists", 3.3492469745603786], ["humpback", 3.3492046513535225], ["fictional", 3.349074678480534], ["tracking", 3.3489614706477915], ["serengeti", 3.3486319824607533], ["sanitized", 3.3482547684125494], ["jfk's", 3.3482547684125494], ["coherently", 3.34794557098532], ["capitalize", 3.347889298263729], ["resurgences", 3.3476195464872203], ["serums", 3.347333119146344], ["sojourn", 3.346850593630242], ["tedious", 3.346840595550305], ["jellyfish", 3.346800895309295], ["dehydration", 3.3465480765631717], ["jazmin", 3.346324028086877], ["ornate", 3.345586836326165], ["philatelic", 3.3454069757360716], ["cause's", 3.3454069757360716], ["dalmatic", 3.3454069757360716], ["rillettes", 3.3454069757360716], ["specimens", 3.3453762101088187], ["relevance", 3.345173829045889], ["careful", 3.345092000079295], ["tailpipe", 3.345042075324233], ["headaches", 3.3449268155490532], ["tradespeople", 3.3448776391892783], ["flyways", 3.3447236818276287], ["cambiar", 3.3447236818276287], ["erau", 3.3447236818276287], ["inspecting", 3.344455265130905], ["hopscotch", 3.344419616038372], ["champurrado", 3.344194871759356], ["barquentine", 3.344194871759356], ["gaozu", 3.344194871759356], ["salsiccia", 3.344194871759356], ["vigesimal", 3.344194871759356], ["rustic", 3.3437569085489653], ["perfectly", 3.3436448833025993], ["architecture", 3.3435508896731716], ["elvira's", 3.343429784852067], ["citadelle", 3.343429784852067], ["jeckyll", 3.343429784852067], ["sumney", 3.343429784852067], ["wading", 3.3429427250619805], ["modernizations", 3.3429029285985448], ["rushdie's", 3.3429029285985448], ["mediating", 3.342644442239971], ["tempest's", 3.3422245109570228], ["franzia", 3.341993295148586], ["viewing", 3.3418350568296873], ["mersin", 3.3418064454731335], ["konkani", 3.3413638388524127], ["selin", 3.3413638388524127], ["ceramides", 3.3410418831268127], ["carboys", 3.3410418831268127], ["mislabeling", 3.340767606505979], ["pavement", 3.3404443480662915], ["daydreaming", 3.34004574660829], ["iucn's", 3.34004574660829], ["chinua", 3.34004574660829], ["fatimid", 3.34004574660829], ["aldrich's", 3.34004574660829], ["brighid", 3.34004574660829], ["featherlight", 3.34004574660829], ["bellinzona", 3.34004574660829], ["voss's", 3.3400457466082893], ["stylish", 3.3399314593286795], ["skits", 3.3392913602171865], ["domaine", 3.339075273267871], ["elbrus", 3.33900165603417], ["raja's", 3.338363668844344], ["overstepping", 3.3383106579087527], ["waterbird", 3.3383106579087527], ["paso's", 3.3383106579087527], ["comical", 3.337912608148271], ["teenager", 3.337888876184231], ["mukha", 3.337725733355262], ["hydrophones", 3.337725733355262], ["geysers", 3.337357581017458], ["inhalations", 3.3372553992844733], ["outcompeted", 3.337025813745172], ["lorikeets", 3.337025813745172], ["altstadt", 3.337025813745172], ["katka", 3.3367550611436507], ["sighisoara", 3.3367550611436507], ["aodh", 3.3367550611436507], ["nebulon", 3.3367550611436507], ["existence", 3.3365188733531186], ["earning", 3.3360882134127285], ["groundwork", 3.335982280815181], ["bina", 3.3359363260041235], ["saucers", 3.335861973230492], ["jimbo's", 3.335721129758991], ["romansh", 3.335721129758991], ["reluctant", 3.335377659673622], ["cucumbers", 3.334927234944656], ["prik", 3.3349151079806267], ["spectrograms", 3.3349151079806267], ["patellofemoral", 3.3349151079806267], ["imes", 3.3343878486750174], ["dyslexia", 3.3342290683012714], ["changing", 3.3341838625948634], ["unresolved", 3.3322204965733793], ["eavesdroppers", 3.3320360445780537], ["khalifa's", 3.3318660427390445], ["loyal", 3.331608054188738], ["synchronistic", 3.3307626844998732], ["gael's", 3.3307626844998732], ["yajna", 3.330438711862532], ["lunchbreak", 3.330193104287911], ["thea's", 3.33000049624255], ["lined", 3.3298427021013888], ["uncertainties", 3.3298405922571366], ["endorphins", 3.329683020744768], ["thoughtless", 3.3295893241557915], ["diagnosing", 3.3295030171768185], ["swordfish", 3.3294424267777867], ["fanciers", 3.3294424267777867], ["grim", 3.329081048326421], ["tuvo", 3.3284079565504214], ["narrowboats", 3.3284079565504214], ["equilibriums", 3.3284079565504214], ["skyline", 3.328310033397831], ["absorbed", 3.328266120984091], ["demerits", 3.328245277764666], ["mosses", 3.3281431140633275], ["thud", 3.3280268702261075], ["polkas", 3.327894234069089], ["rosaline", 3.327728758926393], ["cela", 3.327728758926393], ["swung", 3.32772073518497], ["encrypting", 3.3275595942845198], ["thrushes", 3.3274060276050417], ["baku's", 3.3274060276050417], ["leavening", 3.327032581361764], ["maracan\u00e3", 3.326859427678852], ["resulted", 3.3262614308286365], ["denominators", 3.326215329438069], ["enjoyed", 3.3258931767131865], ["judgments", 3.3258490795018947], ["novices", 3.325802481164331], ["obligingly", 3.3255563933320533], ["futile", 3.3252833286728434], ["tableau", 3.3251348280966453], ["ivanova", 3.3247769660523665], ["reeked", 3.3240646664809774], ["boastfulness", 3.3240646664809774], ["energ\u00eda", 3.3240646664809774], ["touchups", 3.3240646664809774], ["ginger's", 3.3237856615568138], ["anni", 3.323650986286897], ["synonymous", 3.323285119217986], ["layer", 3.3231459135477612], ["darnell's", 3.3230046968806963], ["tau's", 3.3230046968806963], ["arman's", 3.3230046968806963], ["duplicates", 3.323004696880696], ["danai", 3.322653613447323], ["legislations", 3.3224954241302167], ["saltiness", 3.3224954241302167], ["nokomis", 3.3223188341981618], ["kollam", 3.3223188341981618], ["bambi's", 3.3219992491660575], ["sulfurous", 3.3219992491660575], ["confrontational", 3.321762452134199], ["kohlrabi", 3.321693846901651], ["eldercare", 3.321693846901651], ["ultraman", 3.321575447548705], ["conformity", 3.3205369803125486], ["turntablism", 3.3205340614183316], ["kind's", 3.3205340614183316], ["stickler", 3.319884504677052], ["willow's", 3.319412259370555], ["hours", 3.3190979377863346], ["makerspace", 3.3188391069472845], ["vinyls", 3.3188391069472845], ["stitching", 3.318574023951522], ["flutes", 3.3182455776980735], ["deliverables", 3.318128836719404], ["lexicons", 3.3176189518290258], ["decades", 3.3174640237861075], ["stardew", 3.3172255582774257], ["laborers", 3.317142575774404], ["ascents", 3.3170342865506033], ["mizrahi", 3.3170342865506033], ["parabola", 3.316675164124733], ["wampanoag", 3.316601990335647], ["pfizer's", 3.316601990335647], ["mummification", 3.315893740327701], ["combine", 3.315679317637795], ["councilor", 3.3152447612420293], ["tones", 3.314852258691016], ["wildly", 3.31403970955032], ["classmate's", 3.313805207633814], ["cern's", 3.3135374470320333], ["beseeched", 3.3130723891447302], ["tanit", 3.3125659386706], ["heidegger's", 3.3125659386705997], ["dogmas", 3.3125433857979236], ["margot's", 3.312350508245036], ["shutterbugs", 3.31153084268164], ["olaplex", 3.31153084268164], ["odoo", 3.31153084268164], ["reclaimed", 3.31132759626362], ["koji", 3.3112108192994416], ["unhealed", 3.3111790573900177], ["stedelijk", 3.3111790573900177], ["cobwebs", 3.3106540881307565], ["recognition", 3.3106048203711156], ["hinged", 3.3103564510828827], ["jina", 3.3102239095850012], ["further", 3.3100966697470353], ["impulsivity", 3.3100415926774356], ["defeated", 3.3096887285097725], ["grimy", 3.3096741505024854], ["oromia", 3.3093015847286114], ["enjoyment", 3.309067967171591], ["forgetting", 3.30903103610407], ["genealogical", 3.3089876588865814], ["sprinting", 3.3082357871167822], ["zack's", 3.3082357871167822], ["rapa", 3.3082357871167822], ["lysander's", 3.3082357871167822], ["retrofuturism", 3.3082357871167822], ["corby's", 3.3082357871167822], ["retroarch", 3.3082357871167822], ["herm\u00e9", 3.3082357871167822], ["silesians", 3.3082357871167822], ["gelston", 3.3082357871167822], ["swamiji's", 3.3082357871167822], ["pham's", 3.3082357871167822], ["gurdial", 3.3082357871167822], ["larita", 3.3082357871167822], ["ostreatus", 3.308235787116782], ["wayna", 3.308235787116782], ["mottainai", 3.308235787116782], ["cerina", 3.308235787116782], ["underestimated", 3.3080669995766234], ["solved", 3.3076154929066974], ["glittering", 3.307168614282228], ["robert's", 3.3070195239597537], ["enzo's", 3.3068316123344745], ["soundproof", 3.3066985028290885], ["botticelli", 3.3066985028290885], ["digital", 3.3063516587469004], ["cutting", 3.305147016603012], ["unswayed", 3.3046241759954755], ["mudbrick", 3.3046241759954755], ["expansionism", 3.3046241759954755], ["merguez", 3.3046241759954755], ["addax", 3.3043710256832344], ["fertile", 3.3043682452793255], ["exploratorium", 3.3040797120073386], ["equipo", 3.3040797120073386], ["instructions", 3.3035173097922086], ["equipping", 3.303370734488669], ["scheduling", 3.303256554682091], ["agnetha", 3.3029929411784034], ["envelop", 3.302758575548708], ["qutub", 3.3026850559303442], ["honeywell's", 3.302285722751464], ["thresholding", 3.302285722751464], ["salarymen", 3.3019463654682712], ["crontab", 3.3019463654682712], ["plodded", 3.3016977717272233], ["desta", 3.301565956900821], ["gastropub", 3.301463554185121], ["wedging", 3.301463554185121], ["scripts", 3.301222714176257], ["chalked", 3.3011467104301038], ["discrediting", 3.301123486591654], ["marginalia", 3.3010439701882675], ["choppy", 3.3006060920357934], ["equality", 3.300578675439184], ["kismet", 3.3003213474346844], ["happened", 3.300216456063202], ["invitation", 3.2996955045141707], ["thoughtlessly", 3.2994372876829607], ["kurma", 3.2994372876829607], ["gagarin's", 3.2994372876829607], ["worsen", 3.2988864251097128], ["chopin", 3.29874027682081], ["belgrade's", 3.2986744120095084], ["expand", 3.298525228947009], ["accentuating", 3.2985056818605565], ["baisakhi", 3.29824111706507], ["checkboxes", 3.2981905367510427], ["gaye's", 3.2981497023999626], ["observe", 3.2981312843322192], ["contributor's", 3.2977666865246404], ["siberia's", 3.2977666865246404], ["recomputed", 3.2977666865246404], ["commandingly", 3.2977666865246404], ["yeter", 3.2977666865246404], ["medcity", 3.2977666865246404], ["polytheistic", 3.2975640587712443], ["civilization", 3.296559660809311], ["vicariously", 3.296420659305651], ["airbnb", 3.2955726836254264], ["predated", 3.295312991073357], ["nanotechnology", 3.295239146524538], ["plant's", 3.2950988010836872], ["buddhism", 3.2949679796567413], ["langdon's", 3.293597575669363], ["agendas", 3.293562160641667], ["disregarded", 3.293106660041553], ["giraffes", 3.2929763969548045], ["symon\u00e9", 3.2929198806949453], ["monmouth's", 3.2929198806949453], ["beichuan", 3.2929198806949453], ["scrolls", 3.2926278653584977], ["gustav's", 3.2921763900919436], ["reemerging", 3.291979100202203], ["gasp", 3.2919585631642128], ["joining", 3.2918908594567053], ["bikepacking", 3.2913570331008803], ["talkies", 3.2911477923486165], ["kazimierz", 3.2901943175544344], ["cuando", 3.2899148610805335], ["m\u00e9tis", 3.2895602625120905], ["antivenoms", 3.2894389928718004], ["monocots", 3.2891681456924777], ["sanding", 3.288889379004988], ["instrumentals", 3.2886604274297007], ["jitters", 3.2886604274297007], ["grandma", 3.28861717954202], ["capsaicin", 3.2884496879115384], ["aia's", 3.2883066558690905], ["fluffball", 3.2883066558690905], ["maricar", 3.2883066558690905], ["acquiesced", 3.287795166759121], ["sabbats", 3.287559313447302], ["olufemi", 3.287559313447302], ["froze", 3.2873234601354837], ["fusor", 3.2870291474557773], ["tel\u00e9fono", 3.2870291474557773], ["introspecting", 3.2870291474557773], ["oceania's", 3.2870291474557773], ["allicin", 3.2870291474557773], ["wopr", 3.2870291474557773], ["matua", 3.2870291474557773], ["tyrol's", 3.2870291474557773], ["romani", 3.287029147455777], ["whittling", 3.2869795992322697], ["rasam", 3.2863269408444857], ["ballgowns", 3.2863269408444857], ["todai", 3.2863269408444857], ["shenlong", 3.2863269408444857], ["pickaxes", 3.2859581050486555], ["platformers", 3.2859581050486555], ["sister's", 3.285805268123156], ["disappointed", 3.285719657615113], ["albrecht's", 3.2855766378899545], ["lito", 3.2855143874799912], ["careless", 3.2853773740148444], ["visages", 3.284990047488373], ["pauline's", 3.284990047488373], ["lugging", 3.2848913813609175], ["palermo's", 3.284708946179956], ["joyfulness", 3.284708946179956], ["n\u00famero", 3.284708946179956], ["trajectory", 3.2843587782109944], ["workweek", 3.284294607078437], ["nephrologist", 3.283910523976218], ["hanau", 3.283910523976218], ["omnivore's", 3.2839105239762176], ["yongzheng", 3.2839105239762176], ["lineouts", 3.283910523976217], ["cuchara", 3.283910523976217], ["responsabilidad", 3.283910523976217], ["daniela's", 3.283910523976217], ["eton's", 3.283910523976217], ["\u017ei\u017eek's", 3.283910523976217], ["snowfall", 3.28382260133836], ["summerville", 3.2825138581499647], ["midcourt", 3.282490761536107], ["concernant", 3.282490761536107], ["orleanians", 3.282490761536107], ["organelles", 3.28221903216843], ["virage", 3.2815564662529373], ["nena", 3.281556466252937], ["undercooked", 3.281029900709572], ["regalia", 3.2809973188096824], ["hersheypark", 3.2808949954877176], ["prahlad", 3.2808949954877176], ["interminably", 3.2807468336833696], ["wrasses", 3.2801998906157928], ["adventurer", 3.2797542869627967], ["jenkinson's", 3.2797165130899133], ["nwa's", 3.2797165130899133], ["nafplio", 3.2797165130899133], ["bela's", 3.2797165130899133], ["syntax", 3.279691274794314], ["ales", 3.279677121827446], ["carnaval", 3.2795011107181096], ["copal", 3.279468519402723], ["crumbly", 3.2790273856506027], ["drumbeat", 3.278697967588954], ["ming's", 3.278697967588954], ["solution", 3.277842565994535], ["instills", 3.277500488418907], ["neuch\u00e2tel", 3.2773276563737452], ["surrealists", 3.2768645856872265], ["glaciology", 3.276695403553169], ["groundskeeper", 3.2766129450340484], ["perspiration", 3.2762930094228047], ["layered", 3.2761212641833346], ["flapper", 3.2761169930671046], ["bazaar's", 3.2757109970749037], ["maguro", 3.2757109970749037], ["bajau", 3.2757109970749037], ["scrunched", 3.275385314931627], ["sticking", 3.275153429245614], ["utilize", 3.2749768330292195], ["worked", 3.274657193877547], ["teammates", 3.274633811059156], ["building's", 3.27431707750168], ["anecdote", 3.274211210192322], ["iceland's", 3.2738504042091137], ["zushi", 3.273774997667649], ["breadbox", 3.273774997667649], ["shadow's", 3.273041789381497], ["treating", 3.2730190835765063], ["ceil", 3.2728230421580045], ["murakami's", 3.2728230421580045], ["pristina", 3.272586694583196], ["bracing", 3.272521878051317], ["mistakenly", 3.272161939438115], ["annotators", 3.271881547697916], ["greenheart", 3.271881547697916], ["snoozing", 3.2711025282817983], ["lastpass", 3.2710048377440852], ["etna's", 3.2706421986268186], ["staghorn", 3.2706421986268186], ["zithers", 3.2706421986268186], ["relx", 3.2706421986268186], ["worsening", 3.2704022395513506], ["participate", 3.2700589977103207], ["ember\u00e1", 3.2697679291270525], ["steer's", 3.2697679291270525], ["kiribati's", 3.2697679291270525], ["nizams", 3.2697679291270525], ["barbecues", 3.2693806688117193], ["diwani", 3.269118134228577], ["bury's", 3.269118134228577], ["fishmonger's", 3.269118134228577], ["gaining", 3.268904539425505], ["gurgled", 3.268616196851911], ["expats", 3.2683892338642004], ["broomsticks", 3.268308803479166], ["zolt\u00e1n", 3.268216805821014], ["jinda", 3.268216805821014], ["cubert", 3.268216805821014], ["decrescendo", 3.267621285266018], ["phytoplankton", 3.2675452439686063], ["bicycles", 3.267541850890794], ["marshmallows", 3.267269315921177], ["inner", 3.2672230855470663], ["outsider", 3.267115814129028], ["shriveling", 3.267030003291484], ["tarps", 3.266702259221757], ["uncork", 3.266638147781734], ["wineglass", 3.266638147781734], ["sampo", 3.2660389020770273], ["sarongs", 3.2660389020770273], ["lulled", 3.265787397464208], ["printmaking", 3.264881890713034], ["algae", 3.2648569028764904], ["bowties", 3.264706368865246], ["nuzzled", 3.2647063688652453], ["abruptness", 3.2647063688652453], ["luminar", 3.2647063688652453], ["tamagotchis", 3.2647063688652453], ["seasonality", 3.2640724452984755], ["cathedrals", 3.264000062198904], ["suharto's", 3.2636715966405654], ["mightiest", 3.263557979690268], ["fieldwork", 3.263346074544885], ["riffs", 3.2629444638407787], ["conductor's", 3.2627475450439265], ["smallest", 3.262747545043926], ["darkening", 3.262615097881631], ["confide", 3.262403805179396], ["biofuel", 3.2622880678512716], ["elderberry", 3.262288067851271], ["brewmasters", 3.2621050092087795], ["etymology", 3.2617077866665176], ["warred", 3.261707786666517], ["tarmac", 3.261619737389227], ["kaylin", 3.2614978122373683], ["pickpocketing", 3.261243801504328], ["cathartic", 3.2608655378430513], ["cologne's", 3.2607611043711486], ["gollum's", 3.2607611043711486], ["cashews", 3.2607611043711486], ["yaga", 3.260735780038247], ["blueberries", 3.2606798976769786], ["demoralized", 3.2606352721942384], ["stuffiness", 3.260520847879521], ["uncle's", 3.2605208478795205], ["sighs", 3.2601629858352417], ["breaches", 3.259867607639963], ["tampered", 3.2595033575927554], ["divergent", 3.2591291621517677], ["potholed", 3.258507559304109], ["cookstoves", 3.258507559304109], ["staplers", 3.258111002463498], ["cenotes", 3.2575625912528485], ["aboveboard", 3.2575625912528485], ["payscale", 3.257410935638731], ["summerset", 3.257410935638731], ["abundant", 3.2573344512581306], ["conch", 3.2572986072503163], ["mirthful", 3.2570020470508143], ["orwell's", 3.2569814298515927], ["pseudoscience", 3.2569718028122865], ["calculators", 3.2567069836263505], ["omotesando", 3.256544602943083], ["mbaye", 3.256544602943083], ["events", 3.2563688529168915], ["underdog", 3.2562547458355664], ["feigned", 3.255503840845625], ["mischievousness", 3.2554447905138546], ["kegerator", 3.2554447905138546], ["yatai", 3.2554447905138546], ["emina", 3.2554447905138546], ["tomes", 3.255401554644401], ["faulty", 3.2552802142654524], ["rapidly", 3.254986786433738], ["pickling", 3.254967583764902], ["felix's", 3.254877145389092], ["barotrauma", 3.254630114640353], ["resoundingly", 3.254297160152922], ["mittens", 3.254297160152922], ["malala", 3.2540024135574908], ["franconian", 3.2540024135574903], ["gobies", 3.2540024135574903], ["livonian", 3.2540024135574903], ["zebras", 3.253855466194161], ["python3", 3.2534163180219076], ["angela's", 3.253342494188959], ["yogurts", 3.253098523998169], ["dijo", 3.2529567829618746], ["ramen", 3.2526677641300545], ["canggu", 3.2520278975298464], ["haziq", 3.2511972390630444], ["reprieve", 3.2510281524581424], ["exploit", 3.2508555140340247], ["whether", 3.2508402705422106], ["selene", 3.250801138034894], ["manta", 3.250679491274357], ["environment's", 3.2506296913212163], ["severe", 3.25051254078992], ["lockpick", 3.2504500091758772], ["startups", 3.2504219578273625], ["apothecary", 3.250341660842238], ["chisels", 3.2499175280490182], ["docuseries", 3.249904173741927], ["mirabelle", 3.249904173741927], ["blackwattle", 3.2491601480611254], ["simmon's", 3.2491601480611254], ["refreshable", 3.2491601480611254], ["entangling", 3.249160148061125], ["intrusive", 3.248138746825117], ["oldham's", 3.2480860455328404], ["multiply", 3.2479075949545657], ["piece", 3.247871762078448], ["jellybean", 3.247613665363333], ["azorean", 3.247613665363333], ["wero", 3.247613665363333], ["epipen", 3.2474226292653703], ["astrophysicists", 3.246646291080033], ["jian's", 3.2463996041800196], ["gaveled", 3.2463996041800196], ["cgroups", 3.2463996041800196], ["susannah's", 3.2463996041800196], ["glastonbury's", 3.2463996041800196], ["amilia", 3.2463996041800196], ["user1's", 3.2463996041800196], ["taproots", 3.2463996041800196], ["ordnances", 3.2463996041800196], ["emph", 3.2463996041800196], ["gramicidin", 3.2463996041800196], ["sprout", 3.2457255178834714], ["elvis's", 3.2453537280188796], ["imagining", 3.2449574515913935], ["monastiraki", 3.244615868133767], ["sardinia's", 3.244615868133767], ["birdcages", 3.244615868133767], ["coincidental", 3.2441874511709186], ["marko", 3.2441874511709186], ["sleeplessness", 3.2441387187413944], ["bantered", 3.2440109243716466], ["scalding", 3.243209039458467], ["kelp", 3.2429416597394773], ["task", 3.242598390837158], ["leafing", 3.2424470072593463], ["bold", 3.2419574643762625], ["durian", 3.241853424283189], ["quavering", 3.241650532926932], ["predating", 3.2415374849571696], ["seawater", 3.241507167547097], ["hellos", 3.2409170356347543], ["accommodating", 3.24086300774432], ["proceeded", 3.2407651888269], ["pavs", 3.2407207710531742], ["reverbs", 3.240523358725503], ["fiera", 3.240523358725503], ["signify", 3.240476740874631], ["vinaigrette", 3.239810046915626], ["innocent", 3.239783461521653], ["germinating", 3.239314208218516], ["dict", 3.239314208218516], ["accusations", 3.238671718923701], ["zhuang", 3.2382538762354653], ["pretentiousness", 3.237973870450791], ["metamorphosed", 3.2379357766405508], ["weightlessness", 3.2375207098828733], ["potawatomi", 3.237500482458035], ["slithery", 3.2374436852004003], ["containing", 3.23733623576835], ["amlodipine", 3.2371918257240795], ["footwork", 3.2368079689858513], ["maturity", 3.2365796678411756], ["garb", 3.2364762608521165], ["roseate", 3.236317617831635], ["stagehands", 3.236317617831635], ["j\u00e4rvi", 3.236317617831635], ["phyllotaxis", 3.236317617831635], ["bamar", 3.2363176178316344], ["maximian", 3.2363176178316344], ["yowled", 3.2363176178316344], ["table", 3.2361413541042348], ["microfinance", 3.2353240115454582], ["jagged", 3.2349282895613536], ["regality", 3.2348676547412083], ["greenpeace's", 3.2348676547412083], ["splashed", 3.234805319879377], ["stylesheet", 3.23462321070188], ["scooped", 3.2345523536764533], ["bridge's", 3.2345523536764533], ["longships", 3.2343087304214353], ["unscrambling", 3.2342188385723407], ["dicot", 3.2342188385723407], ["marans", 3.2342188385723407], ["galapagos", 3.234178225175616], ["whovians", 3.234012548303264], ["glaciologist", 3.234012548303264], ["curation", 3.2336648984727567], ["lowbrow", 3.2335336886980164], ["sanaa", 3.233300916099875], ["vigorously", 3.2321126065050643], ["microbrews", 3.232059305176593], ["lello", 3.232059305176593], ["heroism", 3.231664670340796], ["lend", 3.2316433690286366], ["glimpsed", 3.231472830978593], ["ghats", 3.2311903719225223], ["pimsleur", 3.231016524272533], ["obscured", 3.2307612893913817], ["experts", 3.2304772541144606], ["ozzy's", 3.230416236325742], ["pungent", 3.2304128972906954], ["biochar", 3.2302638998783393], ["unbridled", 3.229985044223999], ["resistance", 3.2299331076766444], ["cedric", 3.2296651871727584], ["bound", 3.229320579278267], ["meteoric", 3.2289966065870384], ["protonmail", 3.228928764822886], ["shara", 3.2288162193797074], ["finnegan", 3.228710888388013], ["hamster's", 3.2282970041567114], ["detract", 3.2279624187529192], ["twilight", 3.227594411746254], ["assignment's", 3.2275471093822268], ["sheepherding", 3.2275471093822268], ["toowoomba's", 3.2275471093822268], ["chest's", 3.2275471093822268], ["soufri\u00e8re", 3.2275471093822268], ["daphni", 3.2275471093822268], ["percentages", 3.227460769357413], ["ideals", 3.226764308777339], ["o'reilly", 3.2266628499002037], ["inconsequential", 3.2265580043892785], ["limelight", 3.226491588888606], ["deceiving", 3.2263681278578953], ["crescendo", 3.226263968427128], ["dispel", 3.2255298924388622], ["sequestering", 3.2255298924388622], ["coworker's", 3.2255298924388622], ["pentagons", 3.2255298924388622], ["allende's", 3.2255298924388622], ["deepens", 3.2250884018978776], ["restaurateurs", 3.22465004249548], ["dusted", 3.2245681959891193], ["debunk", 3.2244939104595987], ["cubicle", 3.2244695604558125], ["hunching", 3.224128097613813], ["snowmen", 3.22368515129363], ["milena", 3.2234092284727622], ["paleography", 3.223281091677227], ["anahita", 3.223281091677227], ["cared", 3.2231256513145046], ["scavenged", 3.223097366124806], ["boh\u00e8me", 3.223097366124806], ["comprendre", 3.2230973661248057], ["guzzled", 3.2228809491929504], ["loftiest", 3.2228809491929504], ["helloworld", 3.2228809491929504], ["heartbreak", 3.222491716716], ["shards", 3.2222068877419816], ["immaculately", 3.2220838134939496], ["unreadable", 3.221959853819672], ["niccolo", 3.2216327560404268], ["spicy", 3.2210079378170113], ["phung", 3.2204065184322657], ["squids", 3.2199091034801692], ["microbiome", 3.219424088133781], ["aldrin", 3.219424088133781], ["shawarma", 3.2193269503380186], ["bedridden", 3.219284367211468], ["sorpresa", 3.2192484320835777], ["retablo", 3.2192484320835777], ["jiro's", 3.2192484320835777], ["nakshatras", 3.2192484320835777], ["edenfield", 3.2192484320835777], ["qutb's", 3.2192484320835777], ["unphased", 3.2190917090208213], ["necessities", 3.219028992420511], ["progression", 3.218967585934191], ["geneticist", 3.218824009086599], ["rococo", 3.2178758764789523], ["comprehended", 3.2177762148128077], ["hart's", 3.217711922295179], ["elude", 3.217711922295179], ["staircases", 3.217711922295179], ["lederhosen", 3.217461024665407], ["juzo", 3.217085044197843], ["calculator", 3.2166441351594166], ["amy's", 3.2165771924995004], ["novara", 3.2163403485857605], ["todav\u00eda", 3.2163403485857605], ["chytridiomycosis", 3.2163403485857605], ["ayant", 3.2163403485857605], ["gu\u00eda", 3.2163403485857605], ["atole", 3.2163403485857605], ["spelunkers", 3.2163403485857605], ["northamptonshire's", 3.2163403485857605], ["hoatzin", 3.2163403485857605], ["wittenburg", 3.2163403485857605], ["gopuram", 3.2163403485857605], ["morrie's", 3.2163403485857605], ["vasishtha", 3.2163403485857605], ["raas", 3.21634034858576], ["summarizing", 3.216154325430839], ["pursue", 3.215509755199038], ["fusilli", 3.215402640029321], ["jeepneys", 3.2150460466346193], ["planetarium", 3.214251815891873], ["polluters", 3.21408930758428], ["unsavory", 3.2137147646277313], ["raspy", 3.213606617004746], ["licklider", 3.2135724136902475], ["starz's", 3.2135724136902475], ["osce's", 3.2135724136902475], ["nandi", 3.2134686161316663], ["cin\u00e9ma", 3.2133797996640134], ["hindustani", 3.2128059086422596], ["tomohiro", 3.212679175735919], ["kreuzberg", 3.2125779969570596], ["disruptively", 3.2122378736513397], ["tsitsi", 3.2122378736513397], ["sarala", 3.2122378736513397], ["jahannam", 3.2122378736513397], ["maypoles", 3.2122378736513397], ["mabasa", 3.2122378736513397], ["sandbridge", 3.2122378736513397], ["firstline", 3.2122378736513397], ["yusaf", 3.2122378736513397], ["cluckin", 3.2122378736513397], ["celebrations", 3.2120376172477614], ["commodified", 3.211616751278091], ["quelque", 3.211544135718797], ["storyville", 3.211544135718797], ["the's", 3.2114522933447485], ["wailea", 3.211206525204139], ["vincenzo's", 3.210934734554524], ["saaremaa", 3.210934734554524], ["h\u00f6ch", 3.210934734554524], ["intelligence's", 3.210568022044867], ["kaira", 3.2102945960508245], ["sergei", 3.2102734652652654], ["defiantly", 3.2100225371867523], ["balut", 3.2093891051265304], ["toolset", 3.2093483043497044], ["seabirds", 3.209085445543838], ["setup", 3.208786991808942], ["clammy", 3.208708354538269], ["memento", 3.208619403985328], ["yeasted", 3.208418328022741], ["direkt", 3.208418328022741], ["matea", 3.208418328022741], ["luaus", 3.208418328022741], ["alerted", 3.208315986130301], ["emrs", 3.207928866798024], ["inkblots", 3.2076050407406065], ["doodles", 3.2075122693925384], ["isamu", 3.207374939948587], ["unsightly", 3.207031653434725], ["modestly", 3.206111986731873], ["reupholstering", 3.2060150184137353], ["uma's", 3.2060150184137353], ["luminaria", 3.2060150184137353], ["lexer", 3.2053480712495137], ["abkhazia's", 3.2053480712495137], ["supai", 3.2053480712495137], ["umaid", 3.204853418769383], ["bilang", 3.204853418769383], ["planktons", 3.204853418769383], ["dana\u00eb", 3.204853418769383], ["arroway", 3.204853418769383], ["nebby", 3.204853418769383], ["chuah", 3.204853418769383], ["divulging", 3.2048534187693827], ["toothpicks", 3.2048534187693827], ["cryotherapy", 3.2045209995785555], ["militaria", 3.204282245463596], ["visualizers", 3.2041687673233734], ["cryptanalysts", 3.2041687673233734], ["violinist's", 3.2041687673233734], ["discord's", 3.2041687673233734], ["paw's", 3.203717348787543], ["dayang", 3.203717348787543], ["aozora", 3.203717348787543], ["eruzione", 3.203717348787543], ["ciar\u00e1n", 3.2032698813986347], ["kaap", 3.2032698813986347], ["nikola", 3.2031913228152447], ["eyepiece", 3.2029737393448845], ["abbesses", 3.2029737393448845], ["manipuri", 3.2029737393448845], ["semicircles", 3.2029737393448845], ["demeter", 3.2028879745042227], ["clutter", 3.202556032806107], ["eveline", 3.2024491776450246], ["alco", 3.2020593007059395], ["gloom", 3.2018127608768125], ["m\u00f6nchengladbach", 3.2015185036614024], ["proyectos", 3.201518503661402], ["portafilter", 3.201518503661402], ["empagliflozin", 3.201518503661402], ["necesita", 3.201518503661402], ["viviparity", 3.201518503661402], ["middlegame", 3.2015185036614016], ["choppier", 3.2015185036614016], ["crocodylus", 3.2015185036614016], ["niggled", 3.2015185036614016], ["disciple's", 3.2015185036614016], ["yuca", 3.2008064221436836], ["commitments", 3.2001953477158045], ["acupressure", 3.200022466977448], ["corkboard", 3.1994122414879405], ["utterson", 3.1994122414879405], ["bluegills", 3.1994122414879405], ["ascending", 3.1993802345627556], ["packed", 3.1982247845635787], ["pepper's", 3.19817090826784], ["applicable", 3.198053156318087], ["coldly", 3.197679948312625], ["businesspeople", 3.1974871075112143], ["particularly", 3.1973854029506037], ["bitterness", 3.1973746942506347], ["hangry", 3.197288855231768], ["nikhil", 3.1970450827914827], ["josephine's", 3.1969782626716063], ["elijah", 3.196570912025701], ["hashtable", 3.1962758423562794], ["zippy's", 3.1962758423562794], ["methi", 3.1960922011128234], ["initialized", 3.196014034115144], ["homage", 3.195963043112972], ["mashing", 3.195692955751259], ["honeycutt", 3.195455021646892], ["unsubscribes", 3.1954550216468918], ["stanage", 3.1954550216468918], ["verit\u00e0", 3.1954550216468918], ["maccheroni", 3.1954550216468918], ["radical's", 3.1954550216468918], ["waiyaki", 3.1954550216468918], ["harun", 3.194848673445441], ["culprits", 3.1948008855114476], ["christianity's", 3.194515181934643], ["slumbering", 3.194158691009307], ["vitality", 3.194153142512415], ["chess", 3.194052275807714], ["cellos", 3.19367694651561], ["misgendered", 3.193230548955178], ["educators", 3.1928729506673994], ["unwelcome", 3.1922677391444823], ["unabated", 3.1921573384460182], ["measurer", 3.1916704322015934], ["scatterbrained", 3.1911089420526766], ["suffocated", 3.191041199516491], ["factoids", 3.1900845090054686], ["fr\u00e4ulein", 3.1900845090054686], ["razaq", 3.1900845090054686], ["synagogue's", 3.1900845090054686], ["shopee", 3.1900845090054686], ["merv's", 3.190084509005468], ["hexahedron", 3.190084509005468], ["knope's", 3.190084509005468], ["crowdedness", 3.190084509005468], ["masjed", 3.190084509005468], ["proteger", 3.190084509005468], ["pepijn", 3.190084509005468], ["trainwrecks", 3.190084509005468], ["diverge", 3.1895601690138506], ["madagascar's", 3.189212543853356], ["tamarind", 3.189173316629289], ["alguien", 3.189173316629289], ["marcelo's", 3.189173316629289], ["cato's", 3.189173316629289], ["therapeutic", 3.189083226987576], ["havoc", 3.189018904123768], ["diffuser", 3.1889088743973955], ["chulalongkorn", 3.188428182676805], ["roderick's", 3.188428182676805], ["textures", 3.1881479234867567], ["mythological", 3.18777285356416], ["musical", 3.1877378933429794], ["adele's", 3.1876230240448162], ["junit", 3.1876230240448162], ["rastafarian", 3.1872027705149484], ["slapstick", 3.1868515138332687], ["unprepared", 3.1868448771862874], ["subtlety", 3.1861941620432668], ["transnistrian", 3.1859469156085747], ["snoopy's", 3.1858010789833062], ["janszoon", 3.1852945923252802], ["osprey's", 3.1852945923252802], ["mevlevi", 3.1852945923252802], ["swaziland's", 3.1852945923252802], ["nagual", 3.1852945923252802], ["retrospectives", 3.185056795043285], ["bagpipe", 3.1845461678440006], ["footrests", 3.1838109898137175], ["readonly", 3.1838109898137175], ["pitstops", 3.183284435445105], ["venue's", 3.1832680891144314], ["statuses", 3.18311340485064], ["daisies", 3.182718654765531], ["singledom", 3.1826613920560667], ["repairability", 3.1826613920560667], ["guayas", 3.1826613920560667], ["contextually", 3.182109297732955], ["\u00e9quipe", 3.181912662680767], ["isolationism", 3.1819126626807663], ["exercising", 3.1814046804738267], ["toothbrushes", 3.181368431112011], ["quilts", 3.1810967731111055], ["petey's", 3.180995949150752], ["ayuda", 3.1809959491507516], ["letty's", 3.1809959491507516], ["sapori", 3.1809959491507516], ["fallas", 3.1809959491507516], ["malasadas", 3.1809959491507516], ["themeparks", 3.1809959491507516], ["sistrum", 3.1809959491507516], ["anahi", 3.1809959491507516], ["rejuvenation", 3.1808928374506173], ["lavish", 3.1807177631932224], ["apathy", 3.1805415211580166], ["into", 3.1801387817074307], ["adverb", 3.1800998939538085], ["dazu", 3.180099893953808], ["d'ampezzo", 3.180099893953808], ["frieda", 3.1800658333761467], ["toweling", 3.1799893048630463], ["navbar", 3.1799893048630463], ["taso", 3.1799893048630463], ["inuits", 3.1795401157644125], ["turmoils", 3.1795401157644125], ["wispy", 3.1793026433223064], ["intrusions", 3.178882159297994], ["oslo's", 3.1787318950944172], ["nostalgia's", 3.1783670268787265], ["zwolle", 3.178269381194337], ["lamar's", 3.178136626949268], ["prufrock", 3.17687148114537], ["spaceships", 3.1765244423894905], ["upend", 3.176451669223394], ["willingly", 3.1762328262689974], ["gemeinschaft", 3.1761235264715277], ["elven", 3.1755577125163725], ["layouts", 3.1753784456660816], ["clad", 3.174926943721451], ["uncompromised", 3.1748786492485395], ["jeanette", 3.1747785479774118], ["sk\u00e5ne", 3.174569694708023], ["crooned", 3.173953153691378], ["chole", 3.173953153691378], ["solemn", 3.173878934867633], ["amazonica", 3.1735982841527273], ["kettering's", 3.1735982841527273], ["clickstream", 3.1735982841527273], ["shmulik", 3.1735982841527273], ["cliffhangers", 3.1733892197071305], ["sandalwood", 3.1728109303342236], ["extensive", 3.1725338822568947], ["jem's", 3.1722808917558183], ["biruni", 3.1722808917558183], ["completing", 3.172238528998568], ["furtive", 3.171730912405653], ["sarojini", 3.171591265475002], ["desert's", 3.171591265475002], ["datapoint", 3.171591265475002], ["anderen", 3.1715607916744575], ["digestive", 3.1712541490564776], ["troublemakers", 3.17102349045433], ["tremors", 3.170969982860759], ["noor", 3.170969982860759], ["patricia", 3.1707282474435803], ["reverent", 3.17039262932025], ["tastier", 3.17039262932025], ["weary", 3.1700539483428622], ["sven", 3.1699173080864687], ["auditory", 3.1697731095099284], ["nosql", 3.169703413531267], ["mckinley's", 3.1695672211897317], ["pemberley", 3.1691395097039172], ["jojoba", 3.1687318683829417], ["testers", 3.168365524058536], ["ellos", 3.168335268756122], ["hai's", 3.1681693525815953], ["yoyogi", 3.167974328306275], ["verticillium", 3.1674597961756423], ["yoku", 3.1674597961756423], ["hydroponically", 3.1674597961756423], ["honeyeaters", 3.1674597961756423], ["dask", 3.1674597961756423], ["namun", 3.1674597961756423], ["elon's", 3.1674597961756423], ["encased", 3.1672222366909293], ["underfunded", 3.1668838943945197], ["bulldozers", 3.166841634622915], ["ellipticals", 3.166765177799288], ["melding", 3.1666545097867846], ["competitor", 3.1665731362481786], ["auron", 3.166454253383206], ["spilled", 3.1659700268335693], ["inquisitive", 3.165773800996177], ["margriet", 3.165639416982438], ["yokohama's", 3.165639416982438], ["digidestined", 3.165639416982438], ["tudjman", 3.165639416982438], ["podem", 3.165639416982438], ["nevena", 3.165639416982438], ["reviewing", 3.1651406724667037], ["intuitive", 3.1648344374736155], ["broths", 3.1646306966792372], ["tides", 3.1641312791016953], ["josquin", 3.163916776269405], ["issues", 3.163828627077048], ["warhol's", 3.163771342264521], ["pranks", 3.163500471430423], ["promptness", 3.1634275813939854], ["personifying", 3.1634275813939854], ["swirls", 3.1631377262783267], ["institution's", 3.1631377262783267], ["framerates", 3.1630896805628175], ["rmse", 3.1630896805628175], ["commercialism", 3.1623726637757237], ["footholds", 3.162284208273395], ["decisi\u00f3n", 3.162284208273394], ["paco2", 3.162284208273394], ["kea's", 3.162284208273394], ["guineapigs", 3.162284208273394], ["hablas", 3.162284208273394], ["unflustered", 3.162284208273394], ["shastriji", 3.162284208273394], ["peneus", 3.162284208273394], ["reconciliation", 3.1622008073711987], ["clearcut", 3.162004928558807], ["dictating", 3.1619697806652325], ["lara's", 3.1614995223656344], ["beforehand", 3.1612691972934286], ["stefan's", 3.161203085467147], ["mistook", 3.1608496414727973], ["tailor", 3.1606047490737637], ["genevi\u00e8ve", 3.1605466894776404], ["chided", 3.160434378376774], ["walkmans", 3.160434378376774], ["rebellious", 3.159979150587149], ["sidelined", 3.159562095493379], ["around", 3.159515423398234], ["beatrice", 3.1593973579785026], ["umayyads", 3.1593932601921733], ["urmia", 3.1593932601921733], ["scars", 3.1592932585272417], ["grandparents", 3.15916903623484], ["haydn's", 3.1586937635294], ["gupta's", 3.1586937635294], ["stares", 3.158455016133086], ["cinema", 3.1583872353743216], ["zaman", 3.1581941197807333], ["pasta", 3.158132540766231], ["coronations", 3.1578614331569286], ["sehr", 3.1578614331569286], ["qobuz", 3.1578614331569286], ["patong", 3.1578614331569286], ["gerwig's", 3.1578614331569286], ["ilokano", 3.1578614331569286], ["kadazan", 3.1578614331569286], ["phisher", 3.1578614331569286], ["terius", 3.1578614331569286], ["superstorms", 3.1578614331569286], ["lunch", 3.1578268832506575], ["sharpened", 3.1576686455236467], ["fingers", 3.1573367884567993], ["crawdads", 3.1572815834629964], ["prankster", 3.15717093867446], ["sledding", 3.1569710586926925], ["charcuterie", 3.1568544620366614], ["nightjars", 3.1568544620366614], ["makgeolli", 3.1568544620366614], ["hakon", 3.156764699619133], ["handmaids", 3.15668048273316], ["bono", 3.1566083135405965], ["prognoses", 3.15615693796091], ["numerical", 3.156145752608393], ["commonplace", 3.1555356041414018], ["grilled", 3.1553991630336093], ["spines", 3.1551369539910827], ["recitations", 3.155076722898413], ["griffins", 3.1546391255720745], ["yugoslavia's", 3.1544876495744956], ["brand's", 3.1541020743079318], ["viewfinder", 3.1540924101463665], ["slumberland", 3.154038356361339], ["hampi", 3.154038356361339], ["kebaya", 3.154038356361339], ["polyamorous", 3.1537850199471733], ["parrots", 3.1537703661741974], ["mezze", 3.153534113618515], ["lavishness", 3.153534113618515], ["stagnation", 3.153495726106481], ["keystrokes", 3.1533681755684864], ["microfiber", 3.1533502025381326], ["gower's", 3.153162234595683], ["frank's", 3.1529100824777623], ["sant'angelo", 3.1528766479427897], ["digesting", 3.152242462635297], ["limitless", 3.152161760063289], ["settings", 3.1519702967017955], ["disagreeing", 3.1514695761718587], ["wits", 3.151332020419198], ["shortlisting", 3.151151364730802], ["patpong", 3.1507007496350306], ["bodhi's", 3.1507007496350306], ["datapad", 3.1507007496350306], ["fotini", 3.1507007496350306], ["therapist", 3.1505361229881443], ["budapest's", 3.150015814689458], ["practicalities", 3.149575499367304], ["antipasto", 3.1494814381994134], ["accessorized", 3.149431942954282], ["montalcino", 3.149431942954282], ["grammatical", 3.1491007844106065], ["programmers", 3.148982947364618], ["movements", 3.1488792507641477], ["denuclearization", 3.1485278525663167], ["quetzalcoatl", 3.1484630360344377], ["loop's", 3.148321066289385], ["robby's", 3.1482268954032193], ["tucking", 3.1480544100725902], ["evolves", 3.147672274345511], ["trials", 3.1472362978837127], ["deluges", 3.147215461195169], ["pinterest's", 3.147215461195169], ["plovers", 3.146858431647671], ["seminyak", 3.146858431647671], ["defuse", 3.1464199062252005], ["pmed", 3.1464199062252005], ["tastefulness", 3.1464199062252005], ["documentarians", 3.146160835010196], ["nutritionists", 3.1461608350101957], ["aviana", 3.145989773633782], ["tetzel", 3.145989773633782], ["lazed", 3.145989773633782], ["fleance", 3.145989773633782], ["caffeine's", 3.145989773633782], ["efua", 3.145989773633782], ["readers", 3.1455915880108654], ["solos", 3.1455293307644263], ["readying", 3.1451537412729973], ["anatta", 3.145153741272997], ["colonization", 3.1448103838776613], ["laughingly", 3.1447473906604295], ["fascinates", 3.1446090170705387], ["haptic", 3.1445470372333997], ["utensils", 3.143870907820157], ["incisive", 3.1435431794537947], ["picturing", 3.143488683481661], ["echinoderms", 3.142823997760943], ["drafty", 3.142823997760943], ["abigail", 3.142307085919206], ["albari\u00f1o", 3.1421016755366815], ["trendsetters", 3.142051045474233], ["puffing", 3.1418452297765893], ["preferred", 3.1415686404310104], ["culturally", 3.141499189379375], ["outcome", 3.141291723372937], ["leif", 3.1412051991106638], ["caring", 3.1412012871691606], ["bungee", 3.1409117227394883], ["calathea", 3.140730177642515], ["parishioners", 3.140306861835963], ["allergies", 3.139821793153197], ["zeigen", 3.1397608164765756], ["mudhoney", 3.1397608164765756], ["seatmates", 3.1397608164765756], ["incriminating", 3.139732188614105], ["kgosi", 3.139448246957763], ["cardboards", 3.139448246957763], ["bloodworms", 3.139448246957763], ["stoking", 3.1390104694760823], ["naoko", 3.138838171511405], ["botherin", 3.138838171511405], ["hablando", 3.138838171511405], ["insbesondere", 3.138838171511405], ["sniffly", 3.138838171511405], ["dirnt", 3.138838171511405], ["nilay", 3.138838171511405], ["diminutive", 3.138789563874215], ["vito's", 3.1383306384575587], ["sapwood", 3.1381748498352615], ["thorne's", 3.13795894513283], ["naraka", 3.13795894513283], ["desk", 3.137812450983062], ["jhumpa", 3.137788041644723], ["pompidou", 3.137674992644808], ["bellmont", 3.137674992644808], ["angler", 3.1373262070742607], ["aari", 3.137120142955569], ["parma's", 3.137120142955569], ["misremember", 3.137120142955569], ["cosimo's", 3.137120142955569], ["dillin", 3.137120142955569], ["jetsetters", 3.137120142955569], ["vandalizing", 3.136684258854722], ["expos\u00e9s", 3.1365159210482205], ["efficient", 3.136377340571219], ["dnes", 3.1363190397525673], ["alphonsine", 3.1363190397525673], ["veinticinco", 3.1363190397525673], ["wuthering", 3.1362368026096044], ["character's", 3.136225123185577], ["sourly", 3.136163120337218], ["decolonization", 3.136059946895091], ["elysian", 3.1359318398711165], ["ch\u00e2teaux", 3.1357414487691875], ["hiked", 3.135583751088802], ["go\u00fbt", 3.1355531498771696], ["ebony's", 3.1355531498771696], ["casablanca's", 3.1355531498771696], ["processes", 3.1354580040161912], ["broaching", 3.13530528006295], ["malevolently", 3.1348202015017894], ["goeben", 3.1348202015017894], ["aidra", 3.1348202015017894], ["merzbow", 3.1348202015017894], ["orthotics", 3.1347010973907903], ["mademoiselle", 3.1345967098086893], ["medleys", 3.1345828409958836], ["tikva", 3.1345828409958836], ["mindset", 3.1345419848270355], ["immediately", 3.134181015259523], ["ivanovich", 3.1328478003484315], ["anglophile", 3.1327990408302866], ["newton's", 3.1327201290156554], ["experiment", 3.131555319611077], ["becoming", 3.1315255531706545], ["torontonian", 3.1304567611735905], ["ashesi", 3.1304567611735905], ["ophiocordyceps", 3.1304567611735905], ["overall", 3.129657719313476], ["scholars", 3.129290305904556], ["puth's", 3.128440363903913], ["lightweighting", 3.128440363903913], ["heartwell", 3.128440363903913], ["these", 3.1283861801013724], ["arriving", 3.128079533089052], ["disconnecting", 3.1279793499982396], ["dicey", 3.1279793499982396], ["moshe", 3.12739974730654], ["kakapo", 3.127188057473575], ["adopt", 3.127003160488683], ["oozed", 3.126962593302164], ["predators", 3.1269595034181785], ["cl\u00e9s", 3.126686262214032], ["kunga", 3.126686262214032], ["mauga", 3.126686262214032], ["rainboots", 3.126686262214032], ["brandybuck", 3.126686262214032], ["electronegativities", 3.126686262214032], ["hodan", 3.126686262214032], ["recoiling", 3.1262828188253593], ["flamboyance", 3.1257963135625477], ["enunciation", 3.125513461665564], ["tillandsia", 3.125389850296703], ["punnett", 3.125389850296703], ["manipulation", 3.1253015624478246], ["authenticating", 3.125026526754302], ["n\u00fcrnberger", 3.1244449100547387], ["gansbaai", 3.1244449100547387], ["visigoths", 3.1244449100547387], ["overpromise", 3.1244449100547387], ["fanatic's", 3.1244449100547387], ["kickboard", 3.1244449100547387], ["bernardino's", 3.1244449100547387], ["adaptiveness", 3.1244449100547387], ["visitas", 3.1244449100547387], ["lmms", 3.1244449100547387], ["eyob", 3.1244449100547387], ["siddharta", 3.1244449100547387], ["heyer's", 3.1244449100547387], ["ironwork", 3.1238913230756364], ["dunson", 3.123467299006536], ["remind", 3.1234199224711237], ["commenters", 3.122905300613381], ["waitomo", 3.122762613697191], ["selfishness", 3.122396638232059], ["bilingually", 3.1220105938022615], ["fussiest", 3.1220105938022615], ["booleans", 3.1220105938022615], ["kleinen", 3.1220105938022615], ["flywire", 3.1220105938022615], ["scanf", 3.1220105938022615], ["maluku", 3.1219023485143547], ["chasing", 3.121583620264113], ["ishmael's", 3.121480541069867], ["crisper", 3.121480541069867], ["quails", 3.121480541069867], ["maitake", 3.121480541069867], ["oxenfree", 3.121480541069867], ["liwei", 3.121480541069867], ["waorani", 3.121480541069867], ["whitby's", 3.121480541069867], ["thane's", 3.121480541069867], ["braithwaite's", 3.121480541069867], ["seasickness", 3.1202678446669654], ["jagger's", 3.1198229501286225], ["vacations", 3.1194672404676416], ["mangosteen", 3.119398628748122], ["rearview", 3.119193742138681], ["fiddled", 3.1191937421386804], ["arpanet", 3.1191937421386804], ["tiredly", 3.118797704334955], ["cozied", 3.118797704334955], ["loneliness", 3.1184007805406178], ["lollipops", 3.1178337274266075], ["isolated", 3.117798334802366], ["palooza", 3.1177091187487997], ["namco's", 3.1177091187487997], ["farmacy", 3.1177091187487997], ["guelph's", 3.117376030167737], ["kangchenjunga", 3.117376030167737], ["joni's", 3.117376030167737], ["cuauht\u00e9moc", 3.117376030167737], ["recenter", 3.117376030167737], ["soybeans", 3.1173434713555657], ["tablet's", 3.1172680167229445], ["brocken", 3.117056332082395], ["streaming", 3.1169410385530036], ["classics", 3.1167830989328826], ["indifference", 3.116765976150091], ["monastics", 3.1165511365666605], ["briarwood", 3.1163819689336276], ["preparation", 3.1161325499330688], ["factorio", 3.11563301868875], ["mati\u00e8re", 3.11563301868875], ["daoism", 3.1150739936255887], ["oversized", 3.114816025459655], ["rosedale", 3.114668799572715], ["contradictions", 3.1144615050834368], ["schlieffen", 3.1141301159339094], ["vegetarianism", 3.113939275750313], ["kirtan", 3.113876934623671], ["symbol's", 3.1138275087901985], ["aliyah", 3.113827508790198], ["unsatisfied", 3.1138119823503643], ["semester's", 3.113633681992266], ["capable", 3.1136330843199524], ["forager", 3.112749126968972], ["ewan's", 3.112749126968972], ["beesly", 3.1119845116098546], ["malty", 3.1118092872567233], ["lingua", 3.111180003275478], ["finnegan's", 3.1110731279774693], ["neglect", 3.1109212377393143], ["seguir", 3.110841053595464], ["keypress", 3.1106199325766206], ["shutterbug", 3.110207588592186], ["glycemic", 3.1100862477842663], ["rennard", 3.1100150478060655], ["impasse", 3.1096328163441465], ["carl's", 3.1095568222480368], ["surpass", 3.10942354029486], ["m\u00fcnster", 3.1093228758660896], ["profe", 3.1090913724131917], ["la's", 3.1084094644050304], ["vinho", 3.1084094644050295], ["quadrilaterals", 3.1084094644050295], ["fula", 3.1079459377923837], ["buzz's", 3.1079459377923837], ["ratched", 3.1079459377923837], ["submerged", 3.1075974129907182], ["consumption", 3.1075078422879834], ["glum", 3.1070930523260714], ["voiceless", 3.106864913118369], ["nala", 3.106712973324105], ["emeka", 3.106514092780393], ["pachyderm", 3.106386218489213], ["commemorating", 3.1059659360023053], ["abbasid", 3.1058703001388936], ["unspeakable", 3.105230409270979], ["marinade", 3.1036785031269103], ["pinpointed", 3.1034784750500886], ["bioremediation", 3.1014710504219836], ["austronesian", 3.1014710504219836], ["lehua", 3.1014710504219836], ["bodyboarding", 3.101471050421983], ["teza", 3.101471050421983], ["jesters", 3.101471050421983], ["navan", 3.101471050421983], ["subrahmanyan", 3.101471050421983], ["dreiberg", 3.101471050421983], ["bhairavi", 3.101471050421983], ["germaphobes", 3.101471050421983], ["ccus", 3.101471050421983], ["coiffures", 3.101471050421983], ["distrokid", 3.101471050421983], ["safavids", 3.101471050421983], ["besonders", 3.101471050421983], ["horno", 3.101471050421983], ["breuer's", 3.101471050421983], ["aeon's", 3.101471050421983], ["salle's", 3.101471050421983], ["ginjo", 3.101471050421983], ["outils", 3.101471050421983], ["anderlecht's", 3.101471050421983], ["resnet", 3.101471050421983], ["bunkmate", 3.101471050421983], ["rajasekhar", 3.101471050421983], ["rectifications", 3.101471050421983], ["l'artusi", 3.101471050421983], ["tratamiento", 3.101471050421983], ["backorders", 3.101471050421983], ["isbe", 3.101471050421983], ["ideations", 3.101471050421983], ["piak", 3.101471050421983], ["prototypal", 3.101471050421983], ["dined", 3.1008215276889106], ["posited", 3.1004179193352526], ["earplugs", 3.1000183707730735], ["nearing", 3.099951962560552], ["practical", 3.0999100451250823], ["reliving", 3.0996782925893696], ["himself", 3.0991501743325665], ["oncologist", 3.098385009575792], ["feuding", 3.0982347328041517], ["ryota", 3.0974691522924065], ["dismiss", 3.097440890299966], ["kemal", 3.097308002032155], ["integral", 3.097254138028865], ["est\u00e1", 3.0963019320046135], ["shawls", 3.0959817211292013], ["successful", 3.095916176898839], ["flossed", 3.0955634865164177], ["snapdragons", 3.0954252978870476], ["restocks", 3.095280489443097], ["classrooms", 3.095109058523682], ["bravado", 3.0945898151278617], ["satisfied", 3.0944562049047986], ["sibo", 3.0943738626635806], ["kauravas", 3.0943738626635806], ["caravanserai", 3.0943738626635806], ["skillet", 3.094234284637665], ["unremarkable", 3.094130290539328], ["darjeeling", 3.09349811713041], ["swashbuckling", 3.0925673057796232], ["unattended", 3.0924336513346606], ["privately", 3.0914976925343516], ["gogh", 3.091489304512579], ["herps", 3.091433927281459], ["greater", 3.0910026090931026], ["scriptwriter", 3.0908677305914805], ["yosemite", 3.090429163959748], ["validity", 3.090080418220832], ["partof", 3.089941418264281], ["woodturning", 3.089403069681042], ["tansen", 3.089403069681042], ["astronomer's", 3.089403069681042], ["tamannaah", 3.089403069681042], ["grudging", 3.0894030696810417], ["lutes", 3.089308418851701], ["ewell", 3.0891473243938163], ["rafflesia", 3.0888119849100564], ["honza", 3.0888119849100564], ["tolstoy's", 3.088273301271251], ["placeholders", 3.088273301271251], ["courageous", 3.08807652249214], ["miscount", 3.0879275087170837], ["caelius", 3.0879275087170837], ["f\u00e1cil", 3.0879275087170837], ["sighing", 3.0872402790705196], ["deliberations", 3.0872210482984226], ["kabuki", 3.0871197704971514], ["history", 3.0869485704667015], ["dodos", 3.086725546220294], ["concorde's", 3.0866314760180504], ["belittlement", 3.0866314760180504], ["acm's", 3.0866314760180504], ["craned", 3.086560131910339], ["parabens", 3.086560131910339], ["porcupines", 3.0865242019862142], ["nauseam", 3.0865242019862142], ["confucian", 3.0863909642773057], ["insidiously", 3.0863419233467537], ["letdown", 3.0862065396592797], ["conspired", 3.0860839072260915], ["adventurer's", 3.086040846191028], ["cochon", 3.086040846191028], ["farah's", 3.086040846191028], ["agrochemical", 3.086040846191028], ["petrichor", 3.0858857687615715], ["canidae", 3.0858857687615715], ["arunachalam", 3.0858857687615715], ["explicitness", 3.0858857687615715], ["metallica's", 3.0857673995337707], ["fluctuated", 3.0857520938311507], ["insensitive", 3.085633751441105], ["bouquets", 3.08556607067623], ["balalaika", 3.0854840862445503], ["truthfully", 3.0854163767492104], ["metalworking", 3.0853827279702264], ["doling", 3.08485602693758], ["isda", 3.084335851248381], ["provided", 3.0841382579230534], ["myriad", 3.0838057136946606], ["telemark", 3.083543472095845], ["faltering", 3.083333792817176], ["tragedy", 3.082952121831058], ["stilted", 3.0826742561770013], ["kangaroos", 3.0825668459241733], ["jottings", 3.082046388310781], ["tetrapod", 3.082046388310781], ["resuming", 3.08138533314306], ["hayworth", 3.0809315070417056], ["sebo", 3.080931507041705], ["wiglaf", 3.080931507041705], ["ostriches", 3.080717712538119], ["zorg", 3.080655808472842], ["bitterman", 3.080655808472842], ["astrolabe", 3.0800815949018316], ["wfp's", 3.0800815949018316], ["gothenburg's", 3.0800815949018316], ["cordoning", 3.07947480183743], ["kuih", 3.07947480183743], ["bite's", 3.07947480183743], ["parklets", 3.07947480183743], ["ribo", 3.07947480183743], ["minjae", 3.07947480183743], ["peeling", 3.0793686130511597], ["bigwigs", 3.079204078777928], ["reality", 3.0790413396810448], ["retellings", 3.078914897328005], ["gym's", 3.078914897328005], ["museums", 3.0787695612900285], ["incentivized", 3.07873775476444], ["henri", 3.078233442551752], ["scrooge's", 3.0780106037470816], ["glasnost", 3.077915574089664], ["petals", 3.0777792576757044], ["traceur", 3.0774286391784025], ["rankbrain", 3.0774286391784025], ["hrw's", 3.0774286391784025], ["construire", 3.0774286391784025], ["\u00f6zt\u00fcrk", 3.0774286391784025], ["accordionists", 3.0774286391784025], ["multisensory", 3.0771048356599704], ["sparks", 3.077090791034318], ["shift", 3.0770594718315647], ["insomnia", 3.0770282302476537], ["stagnating", 3.0768235983656496], ["porcini", 3.076659282018608], ["recommending", 3.0766592820186074], ["starless", 3.076545335895055], ["duckweed", 3.0763375735141874], ["shackleton's", 3.0763375735141874], ["sanctorum", 3.0762558386299346], ["jennies", 3.0758390582697355], ["astonishingly", 3.07576825166158], ["stratocaster", 3.075625458335134], ["harsher", 3.0755805094793303], ["maharaj's", 3.075261435911375], ["unclogging", 3.075261435911375], ["davante", 3.0751427732537153], ["plusieurs", 3.0749627508457267], ["oraci\u00f3n", 3.0749627508457267], ["silverstein's", 3.0749627508457267], ["kithara", 3.0749627508457267], ["translation's", 3.0749627508457267], ["sideris", 3.0749627508457267], ["ukes", 3.0749627508457267], ["agoge", 3.0749627508457267], ["ouidah", 3.0749627508457267], ["defiant", 3.074668214183769], ["shudra", 3.0746571796690554], ["reprimanded", 3.074501736940053], ["echinacea", 3.074501736940053], ["agitated", 3.074333178730791], ["alexandria's", 3.074265164014773], ["rasped", 3.074024403958072], ["zucca", 3.074024403958072], ["azuki", 3.074024403958072], ["candid", 3.0733158522337547], ["cliques", 3.0729673777485207], ["distinctive", 3.072689399074067], ["montepulciano", 3.0726648177555256], ["pide", 3.0725932939003635], ["maksym", 3.0725932939003635], ["prompt", 3.0725125638185955], ["standardizing", 3.0724853396703757], ["dynamo's", 3.0724853396703757], ["yacouba", 3.0724853396703757], ["myclass", 3.0724853396703757], ["joki\u0107", 3.0724853396703757], ["summi", 3.0724853396703757], ["aumento", 3.071933230894155], ["mastabas", 3.071933230894155], ["transfermarkt", 3.071933230894155], ["fago", 3.071933230894155], ["coupledom", 3.071933230894155], ["cleo", 3.071064471496278], ["gina's", 3.0706763024036086], ["otras", 3.070518245527553], ["comradery", 3.070518245527553], ["paydirt", 3.070518245527553], ["driftwood", 3.069570205331929], ["mystery", 3.069356527488536], ["cada", 3.069307646936126], ["audiobooks", 3.0688239867333307], ["lashing", 3.068764628435715], ["shankar's", 3.068359259848083], ["pushpins", 3.068121899342177], ["involved", 3.068083468415695], ["shores", 3.0679814818639457], ["caramels", 3.0678116746809696], ["foodporn", 3.0676368207810163], ["multimeters", 3.0676368207810163], ["byward", 3.0676368207810163], ["avalokitesvara", 3.0676368207810163], ["greenlighting", 3.0676368207810163], ["misfits", 3.067636820781016], ["symbolizes", 3.067636820781016], ["wallet's", 3.0675381067630996], ["hairclip", 3.067388950966796], ["spluttered", 3.067200652074779], ["project", 3.066809308899436], ["ocasek", 3.066623061091399], ["clucky", 3.066623061091399], ["mullaghmore", 3.066623061091399], ["pepperjack", 3.066623061091399], ["upwellings", 3.066623061091399], ["hiram's", 3.066623061091399], ["posole", 3.066623061091399], ["papermaker", 3.066623061091399], ["bashrc", 3.066623061091399], ["appeared", 3.0665840890796625], ["deciding", 3.0665614692963032], ["selbst", 3.066426179795746], ["bhaji", 3.066426179795746], ["kerrang", 3.066169753913115], ["gele", 3.066093053649109], ["nesting", 3.0660256669885895], ["fussiness", 3.0659851116299692], ["strands", 3.0658844503215237], ["ciphers", 3.0658650383627584], ["photoshop's", 3.065821957888397], ["photorespiration", 3.065821957888397], ["amazona", 3.065821957888397], ["perl's", 3.065821957888397], ["elsa's", 3.0657473001029865], ["glamorized", 3.0656158937697064], ["replica", 3.0654197629982747], ["capture", 3.065291152388924], ["fermenting", 3.0651540591992434], ["facilitating", 3.0650374128036435], ["hadiza", 3.064983155711136], ["boxplot", 3.064983155711136], ["firepits", 3.064983155711136], ["deoxygenated", 3.0647672510087056], ["apriori", 3.064401675317338], ["gyrated", 3.064401675317338], ["facetimed", 3.064401675317338], ["acrimoniously", 3.064283148138746], ["bitrates", 3.064283148138746], ["playbills", 3.064283148138746], ["recollections", 3.0642533978169193], ["stockholm's", 3.0642533978169193], ["witbier", 3.064103929332562], ["fdis", 3.064103929332562], ["lucys", 3.064103929332562], ["brute", 3.0637345205309177], ["grunge", 3.0636626414263635], ["cheesiness", 3.0634938538862038], ["alta\u00efr", 3.063181284367391], ["baal's", 3.063181284367391], ["boisterousness", 3.063181284367391], ["croissance", 3.063181284367391], ["multimeter", 3.062970030485711], ["aloy", 3.062302070451097], ["speleothems", 3.062211923201452], ["graboids", 3.062211923201452], ["gerda's", 3.062211923201452], ["bridgeport's", 3.062211923201452], ["liva", 3.062211923201452], ["slicing", 3.0619160573151523], ["tableside", 3.061606383964117], ["sinisterly", 3.061606383964117], ["heta", 3.061606383964117], ["regroup", 3.0615564664687267], ["baskets", 3.0605580280547144], ["synergies", 3.0604529447618813], ["soba", 3.0603919636614276], ["maximize", 3.059924412463169], ["footing", 3.05985637556822], ["leis", 3.0595592794703346], ["freelancers", 3.059221909177845], ["ernesto's", 3.0589851456216817], ["bandes", 3.0589851456216817], ["larisa", 3.0586921393816797], ["torchlight", 3.0586143595434248], ["manatees", 3.0584302276814332], ["gnome's", 3.058395063610567], ["poem's", 3.058256555614389], ["tutus", 3.0581947101835367], ["hanno", 3.0581947101835367], ["undercutting", 3.0581171540182353], ["sue's", 3.0580504557160757], ["hitchcock's", 3.057478553252877], ["croaked", 3.057450170996639], ["religiously", 3.0572661756803368], ["detailed", 3.0570932115647107], ["churn", 3.0569371071338725], ["botanical", 3.0569062236780558], ["sentinels", 3.0566521046066364], ["parasaurolophus", 3.056522194618766], ["bugsy's", 3.056522194618766], ["thirstily", 3.056522194618766], ["yorta", 3.056522194618766], ["alphabetic", 3.0562840549853716], ["beachfront", 3.0561864136925623], ["cappuccino", 3.055430319579012], ["candied", 3.055358051590769], ["restorations", 3.055180437729118], ["jessica's", 3.0550838312330635], ["nila", 3.054948984665654], ["szpilman", 3.0547152054407474], ["invaders", 3.053968965932249], ["strings", 3.0537647866282196], ["unknowledgeable", 3.053756111184722], ["bagna", 3.053756111184722], ["megadeth's", 3.053756111184722], ["grainy", 3.053066588941409], ["breadcrumb", 3.052926286154509], ["mohammad's", 3.0529021525406543], ["riad", 3.052820524140854], ["markovi\u0107", 3.0527568454415333], ["baraat", 3.0527568454415333], ["cobwebby", 3.0527568454415333], ["kbos", 3.0527568454415333], ["ribeye", 3.0526838653591652], ["tyler's", 3.0526347260087583], ["winery", 3.052467807825316], ["carax", 3.052241351208936], ["frontman's", 3.052241351208936], ["cyberdefense", 3.052241351208936], ["dufresne's", 3.052241351208936], ["lwazi", 3.052241351208936], ["intercaste", 3.052241351208936], ["shamwari", 3.052241351208936], ["consejos", 3.052241351208936], ["machaca", 3.052241351208936], ["jugando", 3.052241351208936], ["immunomodulator", 3.052241351208936], ["thrifted", 3.0518780323955554], ["poignancy", 3.051625979968773], ["hurd's", 3.0512854347193623], ["riddled", 3.051047133989316], ["dass", 3.050966928932857], ["positives", 3.0508136899317577], ["infrastructure", 3.0507275379581347], ["biergarten", 3.050627262710148], ["desires", 3.0503879978267974], ["amicable", 3.050278495034525], ["whisperers", 3.049779866248284], ["monoplanes", 3.0497798662482833], ["afrobeats", 3.0494910886830677], ["vivaldi", 3.049235748520139], ["scientific", 3.049196683657606], ["teared", 3.0488496906975997], ["fillings", 3.0487140686364547], ["rahul's", 3.048302975271892], ["freyr", 3.048302975271892], ["kiana", 3.0480588050071384], ["tropes", 3.0479554715127413], ["received", 3.047680847679906], ["annelids", 3.0474042016824554], ["sniffles", 3.0472247887035757], ["suzy's", 3.0470592776075627], ["restore", 3.0466715486834013], ["colorants", 3.0464153513020666], ["thumbtacks", 3.0464153513020666], ["lyuba", 3.0462847327988163], ["passionfruit", 3.046087638807305], ["wesleyan's", 3.046087638807305], ["jajaja", 3.046087638807305], ["rongo", 3.046087638807305], ["manuscripts", 3.0456862019982855], ["reconsidered", 3.045222708953373], ["tulong", 3.045080667687038], ["microfiction", 3.045080667687038], ["hydrometers", 3.045080667687038], ["ilze", 3.045080667687038], ["groomer's", 3.045080667687038], ["herrmann's", 3.043822889633471], ["insincerely", 3.043822889633471], ["homeward", 3.0432797772204525], ["auch", 3.0432557362069432], ["whirling", 3.0425604271178788], ["bulbs", 3.0424120523914127], ["idealism", 3.0423954113663263], ["malicious", 3.042357542366932], ["refreshing", 3.0421944143695394], ["kozhikode", 3.0421385607617366], ["oahu's", 3.041663068733405], ["shielding", 3.0416309030961943], ["melaka", 3.041442578478332], ["patroness", 3.041442578478332], ["oren's", 3.041442578478332], ["watney's", 3.041442578478332], ["vortex's", 3.041442578478332], ["ducs", 3.041442578478332], ["remunerations", 3.0411311467172757], ["panpipes", 3.0411311467172757], ["maneuvers", 3.0408049133917507], ["ng\u0169g\u0129", 3.0406578925705716], ["bellefleur", 3.0406578925705716], ["dweck's", 3.0406578925705716], ["chamunda", 3.0406578925705716], ["shortboard", 3.0406578925705716], ["hayyan", 3.0406578925705716], ["netsh", 3.0406578925705716], ["flightless", 3.040492975532331], ["smita", 3.040473889067239], ["audits", 3.040395928198289], ["gillnets", 3.040315283230564], ["logging", 3.040196770191038], ["insa", 3.040177156144869], ["formative", 3.0401086124351395], ["operant", 3.0398524202811497], ["procope", 3.039852420281149], ["greenwich's", 3.039852420281149], ["manfred's", 3.039852420281149], ["neriah", 3.039852420281149], ["dumpsites", 3.039852420281149], ["tiyo", 3.039852420281149], ["impairments", 3.0394416294135436], ["cleopatra's", 3.0394416294135436], ["reproachful", 3.038905351998485], ["nivel", 3.038905351998485], ["kickabout", 3.0381757228623507], ["specialize", 3.0378178225808616], ["wurlitzer", 3.037702486768447], ["letterboxes", 3.0375963605963587], ["automated", 3.0373816117416763], ["fared", 3.0373739820465957], ["sabotage", 3.0369128520582835], ["chopsticks", 3.0365989179924755], ["cinematography", 3.0365897020108563], ["petrovich", 3.036303954420899], ["mockingly", 3.0361769230446782], ["ardently", 3.036058206449447], ["silkworms", 3.0359114997626566], ["europop", 3.0358810282076214], ["edmodo", 3.0358810282076214], ["tempering", 3.035881028207621], ["dissonant", 3.035881028207621], ["tongues", 3.035482304668324], ["winters", 3.0352288584317058], ["flatirons", 3.0347727482623705], ["tragedies", 3.0346701354898173], ["adjusted", 3.034633147164827], ["dynamically", 3.034224201478254], ["improve", 3.034093225527565], ["farthings", 3.03381553956318], ["whirlwinds", 3.03381553956318], ["resultados", 3.033753341897486], ["comment's", 3.033753341897486], ["merina", 3.033753341897486], ["tartans", 3.0334563344039576], ["livia", 3.033374879654235], ["conversions", 3.0331120966575806], ["surpassed", 3.032747807002561], ["korfball", 3.032549471523717], ["drepung", 3.032549471523717], ["cpr's", 3.032549471523717], ["alone's", 3.032549471523717], ["zareh", 3.032549471523717], ["deglazing", 3.032549471523717], ["baos", 3.032549471523717], ["nutritional", 3.0324863854103095], ["populace", 3.0323606452801597], ["farhad", 3.032082206582034], ["pluto's", 3.031636602929038], ["vajrayana", 3.0311606235862207], ["matterhorn", 3.0310830549514134], ["ukulele", 3.030927787314346], ["representative's", 3.029760737117429], ["sextants", 3.0297146507978243], ["nataraja", 3.0297146507978243], ["catastrophizing", 3.0297146507978243], ["pretense", 3.029711524157318], ["particular", 3.0296533867784117], ["grilling", 3.0295528811368357], ["fromage", 3.0295528811368357], ["mushrooms", 3.0291638501041533], ["mercato", 3.029103392578804], ["longevity", 3.0287830528458963], ["blazing", 3.028761066855799], ["edge", 3.028659788234622], ["allowed", 3.0285545964552396], ["pitstop", 3.0283081435915156], ["congestion", 3.028175892118794], ["inventor", 3.0281200641395096], ["absenteeism", 3.028102918046409], ["fruit's", 3.0280634515954277], ["cernunnos", 3.0278768221068857], ["ryoichi", 3.0278768221068857], ["bahian", 3.0278768221068857], ["lidija", 3.0278768221068857], ["escher's", 3.027273178402319], ["heraldry", 3.0271963391821046], ["borders", 3.027103909410766], ["created", 3.0270911286234705], ["equiv", 3.0270357452118555], ["tenses", 3.026818574372439], ["rarities", 3.026791894908137], ["flatbread", 3.0266838052345024], ["uphill", 3.0265188333701185], ["tendonitis", 3.0262838734420563], ["plump", 3.0262353655859404], ["psychodynamic", 3.026029862709016], ["amulet", 3.025918326778132], ["animosities", 3.0258254150458375], ["pembroke's", 3.025825415045837], ["quantifiably", 3.025825415045837], ["gawthorpe", 3.025825415045837], ["outsoles", 3.025825415045837], ["detritivores", 3.025825415045837], ["naqsh", 3.025825415045837], ["sarus", 3.025825415045837], ["montrose's", 3.025825415045837], ["fileid", 3.025825415045837], ["shucker", 3.025825415045837], ["executing", 3.025677175760374], ["fated", 3.025434985960025], ["neighborhoods", 3.025228072308202], ["powerwall", 3.025148496608467], ["pirelli's", 3.025148496608467], ["aksum", 3.0248915183127987], ["viziers", 3.0248915183127987], ["posi", 3.0248610085390495], ["ballgown", 3.0246727196496295], ["lakebed", 3.0246727196496295], ["entrees", 3.0245750739652397], ["motivation", 3.024569515219954], ["upbringing", 3.02445310760853], ["birds", 3.0244174178929066], ["genghis", 3.024382463619587], ["tectonic", 3.024278882055925], ["rossini's", 3.0242102057495495], ["resulting", 3.024119130051856], ["profiting", 3.0239342741614337], ["mistranslation", 3.0239342741614337], ["workout", 3.023018601756071], ["polylactic", 3.0229527959809204], ["nazia", 3.0229527959809204], ["barabas", 3.0229527959809204], ["eudoxus", 3.0229527959809204], ["handoffs", 3.0228762674678196], ["minerva's", 3.0228762674678196], ["insects", 3.022545996643626], ["investigate", 3.022535571530851], ["jabbar's", 3.02252451459306], ["kanya", 3.02252451459306], ["shoebill", 3.02252451459306], ["jo\u00e3o", 3.0223096940874035], ["stall", 3.0222503312086113], ["urgently", 3.02218841297411], ["teem", 3.0219461516932142], ["defacing", 3.0219461516932142], ["virgil's", 3.0219461516932142], ["leclercq", 3.0219461516932142], ["unlv's", 3.0219461516932142], ["geometer", 3.0215736697319553], ["kayla's", 3.021410630489654], ["misjudging", 3.0213592520716004], ["haleakala", 3.0213592520716004], ["digitized", 3.0213592520716], ["bioethical", 3.0209937576964627], ["ramona's", 3.0209937576964627], ["sensation", 3.0208484276503085], ["geode", 3.0205631099761927], ["clayville", 3.0205631099761923], ["confucius's", 3.0205631099761923], ["speedbumps", 3.0205631099761923], ["tcm's", 3.0205631099761923], ["picanha", 3.0205631099761923], ["xxiii's", 3.0205631099761923], ["cstc", 3.0205631099761923], ["philippides", 3.0205631099761923], ["buoyancy", 3.0200182753603957], ["hina", 3.0198533912003516], ["grander", 3.0198286959933256], ["degradation", 3.0198041745214246], ["radians", 3.0196740117295353], ["bellwood", 3.019572274119221], ["tomatoes", 3.0194422191135932], ["paradisiacal", 3.0189223912852765], ["thistlewood", 3.0189223912852765], ["stuttgart's", 3.0189223912852765], ["stretched", 3.0184382549745856], ["fiddlesticks", 3.0183662527150705], ["silverstream", 3.0176475085186865], ["savannas", 3.0176475085186865], ["dolmabah\u00e7e", 3.0176475085186865], ["phere", 3.0176475085186865], ["milaje", 3.0176475085186865], ["kieler", 3.0176475085186865], ["cleansweep", 3.0176475085186865], ["curator", 3.0176002839880365], ["autographed", 3.017098538219068], ["alina", 3.017098538219068], ["juilliard", 3.0168223998288104], ["steep", 3.0165218535854508], ["animations", 3.0164176085152143], ["birdlife", 3.0163326294300075], ["variables", 3.0161058022174423], ["vocabulary", 3.0160750245253416], ["transcendent", 3.015841209972622], ["landed", 3.0156993337861753], ["comique", 3.015536084289384], ["delimiter", 3.015536084289384], ["ib\u00e9rico", 3.0153190767991505], ["mikolaj", 3.0153190767991505], ["beiden", 3.0153190767991505], ["bornemisza", 3.0153190767991505], ["veena", 3.015100970536814], ["puck's", 3.015052352422609], ["ayanna", 3.015052352422609], ["daos", 3.0149758418444774], ["verdi's", 3.014629861010168], ["chianina", 3.014513918167161], ["mosuo", 3.014513918167161], ["unthinkable", 3.014096274592891], ["jakey", 3.013785481457173], ["szechuan", 3.013785481457173], ["leftovers", 3.013596037326694], ["kirpan", 3.0132453969088853], ["niko's", 3.012857591838498], ["outweigh", 3.0120953255281027], ["abyss", 3.012010291836106], ["headpieces", 3.01183315879129], ["rizwan", 3.0116353372373466], ["attractions", 3.011473704072207], ["ananda", 3.01109705954876], ["unanswered", 3.01085944522338], ["sonnets", 3.0107560868918544], ["headlines", 3.0103724911549756], ["olga", 3.0102513136448663], ["businessman", 3.010132553942665], ["jasper", 3.0099417762347227], ["reynold's", 3.009207609562748], ["deserving", 3.008760410420122], ["gothic", 3.008496303049154], ["hurtful", 3.008426918909324], ["crashing", 3.0082476431260625], ["purposeful", 3.0081618359135613], ["grittiness", 3.0080828571446174], ["feta", 3.0079745163249836], ["snapped", 3.0079468150315494], ["bluefin", 3.0077939656337276], ["maddy", 3.0077288386085854], ["marat's", 3.007487079197075], ["loppers", 3.007487079197075], ["kpelle", 3.007487079197075], ["porcupine's", 3.007487079197075], ["culturale", 3.007487079197075], ["broodingly", 3.007487079197075], ["consectetur", 3.007487079197075], ["isochronic", 3.007487079197075], ["mehwish", 3.007487079197075], ["graboid", 3.007487079197075], ["unoriginal", 3.0074870791970745], ["simba's", 3.0074870791970745], ["shinji's", 3.0074870791970745], ["viewers", 3.0070069683079255], ["monopolize", 3.0068498997311432], ["luka", 3.0066025241737813], ["ritualistic", 3.0061364113471356], ["overspent", 3.006099861909622], ["bootcamps", 3.006099861909622], ["unkindness", 3.005549265357386], ["greenberg's", 3.005549265357386], ["gasses", 3.005454993332753], ["subtasks", 3.0053691305497527], ["wisest", 3.0052040677503156], ["turn", 3.0050197526261804], ["shop", 3.004888218079725], ["mystical", 3.0046702074961367], ["miscalculate", 3.0043477398659713], ["eureka's", 3.0043477398659713], ["tilted", 3.004079062296583], ["usernames", 3.003950568939643], ["loungewear", 3.003746423625934], ["lookout", 3.0036261160548623], ["maneuverability", 3.003529859356026], ["topa", 3.003529859356026], ["harlow's", 3.003529859356026], ["crossed", 3.0034977952756177], ["mornings", 3.0034727091351265], ["operettas", 3.0029371743873967], ["checkmates", 3.002337272554614], ["yukon's", 3.002337272554614], ["catwoman's", 3.002337272554614], ["bacteriophages", 3.0022239768084797], ["compartmentalizing", 3.0021356787002826], ["averted", 3.0017998440152596], ["linings", 3.0014739989222314], ["escuchar", 3.0014235971825642], ["surliness", 3.0014235971825642], ["transliterating", 3.0014235971825642], ["zoroastrian", 3.0012488418057273], ["gesture", 3.0011615081362946], ["windfalls", 3.0010424640273667], ["elamite", 3.0010424640273667], ["frontrunners", 3.000882800138027], ["floes", 3.000882800138027], ["azizah", 3.000833346720975], ["layover", 3.0007395120322093], ["spike's", 3.0004929231989417], ["furthermore", 3.0004725578623592], ["trowel", 3.000288306933039], ["workforces", 3.000288306933039], ["penmanship", 3.000288306933039], ["canceling", 3.000182502193012], ["popping", 2.9996180070501373], ["matrix's", 2.9992247520564232], ["haddock's", 2.9992247520564232], ["safiya", 2.998987659190648], ["quilliam", 2.998987659190648], ["place's", 2.9987733335205933], ["tenderized", 2.9987733335205933], ["buick's", 2.9987733335205933], ["rusted", 2.9986960990448974], ["swooping", 2.9986081914632643], ["uxmal", 2.9984010155137], ["pynchon's", 2.9984010155137], ["earworms", 2.9984010155137], ["fomo", 2.9983799000135902], ["cumbia", 2.998088682074584], ["orphanage's", 2.9980886820745836], ["overprotectiveness", 2.9980886820745836], ["gamemakers", 2.9980886820745836], ["eostre", 2.9980886820745836], ["muninn", 2.9980886820745836], ["nrhp", 2.9980886820745836], ["chelonians", 2.9980886820745836], ["eeps", 2.9980886820745836], ["porchetta", 2.9978229176315314], ["ronda's", 2.997704361002661], ["kip's", 2.997704361002661], ["sultanates", 2.997704361002661], ["calen", 2.997704361002661], ["llamo", 2.996927082430231], ["paphiopedilum", 2.996927082430231], ["leaner", 2.9968301141120937], ["desired", 2.9965771605511597], ["arid", 2.9964981841000085], ["academic", 2.9964212245205935], ["psyche", 2.996280473350637], ["objectively", 2.996248367664211], ["owning", 2.996056889864288], ["giacomo", 2.9959104474092415], ["spankin", 2.995910447409241], ["figuring", 2.995894444052517], ["unfollowing", 2.9958481973116373], ["joneses", 2.995831425123767], ["heyday", 2.9955671608953787], ["interstellar", 2.9955395277865655], ["jalebi", 2.9954378521169582], ["malou", 2.9954378521169582], ["shakespeare's", 2.9953484265280266], ["representing", 2.9951813351219316], ["criticism", 2.994836654177054], ["evaluating", 2.994745779145985], ["strength", 2.9947136000017527], ["jie's", 2.9945237728212253], ["naskh", 2.9945237728212253], ["supernature", 2.9945237728212253], ["privado", 2.9945237728212253], ["cedd", 2.9945237728212253], ["pueblito", 2.9945237728212253], ["succ\u00e8s", 2.9945237728212253], ["rokit", 2.9945237728212253], ["artscience", 2.9945237728212253], ["remoras", 2.9945237728212253], ["cottingley", 2.9945237728212253], ["harihara", 2.9945237728212253], ["competing", 2.99445161562188], ["solvable", 2.9937357402494302], ["seek", 2.993140552994959], ["pies", 2.993082798975657], ["puya", 2.9928592237535097], ["mukunda", 2.9928592237535097], ["wetly", 2.9928592237535097], ["dionysios", 2.9928592237535097], ["pigeons", 2.9925809751145915], ["pets", 2.992506034183824], ["alexa's", 2.992007366289443], ["medici's", 2.992007366289443], ["agnolotti", 2.992007366289443], ["cessnas", 2.992007366289443], ["pfds", 2.992007366289443], ["ellenor", 2.992007366289443], ["merida's", 2.991489807499218], ["refolded", 2.991489807499218], ["clawhammer", 2.991489807499218], ["waii", 2.991489807499218], ["wealthier", 2.9913253495658756], ["eurydice", 2.991325349565875], ["persisting", 2.9912790649978085], ["recycle", 2.9912580613087734], ["agave", 2.9909459002614907], ["wassily", 2.9908404717476444], ["kush's", 2.9907042271926265], ["squeakers", 2.9907042271926265], ["belham", 2.9907042271926265], ["panglong", 2.9907042271926265], ["piloxing", 2.9907042271926265], ["panier", 2.990136192201707], ["pipefish", 2.990136192201707], ["coquilles", 2.990136192201707], ["kites", 2.9899269161524207], ["glasses", 2.989818092606792], ["representations", 2.989606026210182], ["h\u00e9lo\u00efse", 2.9893696871537188], ["mellissa", 2.9893696871537188], ["lampshades", 2.9886902849520927], ["empathic", 2.9882553976816433], ["mangrove", 2.98796360476221], ["unveiling", 2.9878750023101417], ["colorism", 2.9878045721342668], ["lawn's", 2.9878045721342668], ["macram\u00e9", 2.9875394608146446], ["kriek", 2.9875394608146446], ["nullable", 2.9875394608146446], ["drifting", 2.986847669628923], ["ladleful", 2.9866017522582062], ["ng\u0101ti", 2.9866017522582062], ["balwinder", 2.9866017522582062], ["paperwork", 2.986375997568173], ["turquoise", 2.986107921630586], ["tanzania's", 2.985967990613165], ["sofrito", 2.985822977355401], ["idolization", 2.985822977355401], ["kailasa", 2.985822977355401], ["slowpoke", 2.985591912274349], ["peppercorns", 2.9853429106344938], ["palace's", 2.984834532286456], ["sloths", 2.984834532286456], ["legroom", 2.984834532286456], ["steeping", 2.984834532286456], ["tweaked", 2.984648974189422], ["neapolitan", 2.9844760570608697], ["months", 2.9844625167036525], ["wrongdoing", 2.984228628296141], ["eyesore", 2.9841180917573675], ["jeered", 2.9837742003034053], ["nairs", 2.983693668760389], ["oscarssowhite", 2.983693668760389], ["groundedness", 2.983693668760389], ["bewitchingly", 2.983693668760389], ["dell'opera", 2.983693668760389], ["meeta", 2.983693668760389], ["magga", 2.983693668760389], ["remote", 2.983156567237792], ["moreau", 2.983149033469468], ["representation", 2.9830203478387434], ["bras\u00edlia", 2.9829878866979973], ["stemming", 2.982865344317934], ["boeuf", 2.982341698816371], ["uttered", 2.9823416988163705], ["cumin", 2.982277261033217], ["spheres", 2.9822394007014807], ["plugging", 2.9822206906538207], ["hakan", 2.981923439023899], ["carta", 2.981684837650458], ["intercom", 2.9815688190912364], ["area's", 2.981547503139], ["sanctuary's", 2.9815246562067683], ["kayakers", 2.9814141065346806], ["naoshima", 2.9812589942040773], ["maduros", 2.9812589942040773], ["ishbel", 2.9812589942040773], ["vitacost", 2.9812589942040773], ["quickfix", 2.9812589942040773], ["waray", 2.9812589942040773], ["skijoring", 2.9812589942040773], ["bukhara", 2.981258994204077], ["simplest", 2.981020183676288], ["exfoliating", 2.9808582873500176], ["reattaching", 2.9808188585291373], ["mithun", 2.9801844730311795], ["ravenously", 2.9801844730311795], ["roberta's", 2.980061151651016], ["decisions", 2.97977523396733], ["quizzing", 2.9796275448696914], ["delays", 2.979620585697551], ["bryn's", 2.9795791750778875], ["dashlane", 2.9795791750778875], ["arindam", 2.9795791750778875], ["unknown", 2.979461558007743], ["participated", 2.9789933505092403], ["smartphone's", 2.9788140032301533], ["critically", 2.978392910713538], ["polonius", 2.9774122084051045], ["affluent", 2.977412208405104], ["marseille's", 2.977412208405104], ["omnichannel", 2.977412208405104], ["authenticator", 2.977412208405104], ["camper's", 2.977412208405104], ["journalism's", 2.977412208405104], ["nnamdi", 2.977412208405104], ["cols", 2.977412208405104], ["musicman", 2.9774122084051036], ["westfjords", 2.9774122084051036], ["entiendo", 2.9774122084051036], ["shushed", 2.9768424777873457], ["whittaker's", 2.9758036658830047], ["cfrp", 2.9758036658830047], ["coastal", 2.9751711454525407], ["chicxulub", 2.9750321826589765], ["climbers", 2.9748141174728135], ["abating", 2.9747442763187193], ["microorganisms", 2.974655345249173], ["algorithmically", 2.9739134866560963], ["finch's", 2.9738928086315757], ["competitive", 2.9738438492395645], ["discourage", 2.9737438017863256], ["pastoralist", 2.97357533700252], ["puppeteers", 2.97347383246806], ["mj's", 2.97347383246806], ["joplin's", 2.973311089660744], ["annaliese", 2.973311089660744], ["boswellia", 2.972243089987734], ["peanut's", 2.972243089987734], ["miras", 2.9717517289214443], ["linguistica", 2.9717517289214443], ["mwana", 2.9717517289214443], ["businessperson", 2.9716420297066444], ["stumbles", 2.971615066254783], ["entertaining", 2.9713535207717214], ["curmudgeon", 2.9712858458363693], ["punchier", 2.9708827956673733], ["contrabass", 2.9708827956673733], ["wellsprings", 2.9704229778689415], ["hanja", 2.9704229778689415], ["connoisseurship", 2.9696411847440913], ["katerina", 2.9696411847440913], ["shakuhachi", 2.9696411847440913], ["melting", 2.969547939497536], ["errands", 2.969348383674007], ["nutrition", 2.96894470014046], ["frost's", 2.968929552540702], ["clusaz", 2.9689295525407013], ["sapote", 2.9689295525407013], ["oldfield's", 2.9689295525407013], ["handline", 2.9689295525407013], ["hereford's", 2.9689295525407013], ["brettanomyces", 2.9689295525407013], ["sorolla", 2.9689295525407013], ["compatibilities", 2.9689295525407013], ["guillotin", 2.9689295525407013], ["miniland", 2.9689295525407013], ["int\u00e9r\u00eat", 2.9689295525407013], ["roselyn", 2.9689295525407013], ["otak", 2.9689295525407013], ["sirpa", 2.9689295525407013], ["hajji", 2.9688564261970436], ["stakeout", 2.968550862546756], ["handrails", 2.9683188901420867], ["mikey's", 2.9681367809645898], ["louder", 2.968028029735143], ["tremolos", 2.9679148807865867], ["csiro's", 2.9679148807865867], ["dinner", 2.967804334570629], ["olio", 2.967682103148878], ["quenching", 2.967618089298508], ["tresses", 2.9674016966573444], ["befuddling", 2.967401696657344], ["eldred", 2.9673533833767083], ["carex", 2.9673533833767083], ["workstations", 2.9672772954415465], ["handling", 2.967082410947372], ["often", 2.966710323084037], ["windrose", 2.9666244830123323], ["bocephus", 2.9666244830123323], ["nowadays", 2.9665450385230048], ["rambled", 2.9660044987943563], ["correcting", 2.9659100641121485], ["regulations", 2.965870025002267], ["mispronouncing", 2.965716425319338], ["breaks", 2.965695539992399], ["physicality", 2.965666992592855], ["wynwood", 2.9654416183859316], ["thrills", 2.965434113313819], ["cl\u00e1sico", 2.965400459707554], ["vindaloo", 2.965006324203416], ["organizer", 2.9647395309947773], ["legos", 2.964466937933777], ["bizarrely", 2.9643819580400925], ["garbled", 2.964320571810358], ["rearranged", 2.964023538887783], ["grazers", 2.9636278926254507], ["versatile", 2.9635675673922792], ["pastures", 2.963152571391669], ["athletically", 2.963012521385288], ["underwent", 2.962702770761044], ["mistake", 2.962620577241553], ["auxerre", 2.9625992123433864], ["jamun", 2.962221329790792], ["multitasker", 2.962221329790792], ["greenroom", 2.962221329790792], ["ellora", 2.962221329790792], ["fetched", 2.9619048531529937], ["scarfs", 2.961600787363737], ["imagenet", 2.961404615886797], ["physiologies", 2.961404615886797], ["coldbrook", 2.961404615886797], ["glissandos", 2.961404615886797], ["climates", 2.961180210384204], ["wintertime", 2.9609210028197785], ["gordy's", 2.9609210028197785], ["generating", 2.960650931899241], ["caloric", 2.9605246173452144], ["blankets", 2.960181813680537], ["queasy", 2.960157021745682], ["mouthfeel", 2.96013819242807], ["beckoned", 2.9597989160199916], ["duomo", 2.9593672859299214], ["eddy's", 2.959335479368567], ["deprecating", 2.9592983537133355], ["schmidt", 2.95916146628344], ["herniated", 2.959068877678873], ["yale's", 2.958971029186378], ["inconclusively", 2.9589526870546714], ["monteverdi's", 2.9589526870546714], ["olan", 2.958619262019743], ["peppered", 2.9584844912151325], ["clementines", 2.958451963775383], ["attended", 2.958403870577433], ["novel's", 2.9580697007788164], ["substituting", 2.9577019250940113], ["derailing", 2.9576942467600373], ["jalopies", 2.9576942467600373], ["lie\u00df", 2.9576942467600373], ["annabelle's", 2.9576942467600373], ["historias", 2.9576942467600373], ["buelow", 2.9576942467600373], ["farmageddon", 2.9576942467600373], ["pocketknives", 2.9576942467600373], ["jumped", 2.9574094625293488], ["otavalo", 2.9565424966639466], ["makeda", 2.9565424966639466], ["doe's", 2.956542496663946], ["touchstones", 2.956542496663946], ["auteurs", 2.956542496663946], ["thora", 2.955969939414532], ["rewrote", 2.9557791718968223], ["lindsey's", 2.9555194715785955], ["bushido", 2.9553284354806335], ["biodynamic", 2.955222822028101], ["bleakness", 2.955222822028101], ["factories", 2.9551345263369777], ["formatting", 2.9549113400000815], ["coalesce", 2.954750405882114], ["pi\u00f1atas", 2.9546558527688718], ["climbing", 2.954574103397451], ["unfortunately", 2.954147456736711], ["statistic", 2.9540961849054774], ["reglas", 2.9537819527828413], ["pequod", 2.9537819527828413], ["karatsu", 2.9537819527828413], ["myspace's", 2.9537819527828413], ["youngstown's", 2.9537819527828413], ["okapis", 2.9537819527828413], ["qunu", 2.9537819527828413], ["resplendence", 2.9537819527828413], ["goldsworthy's", 2.9537819527828413], ["tsosie", 2.9537819527828413], ["sportmanship", 2.953781952782841], ["hurleys", 2.953781952782841], ["sakura", 2.953695076843054], ["pursued", 2.9534317621389543], ["kinsley", 2.9534008196276433], ["zucchini", 2.9531989695026866], ["glared", 2.9531744911987214], ["blistered", 2.953024572794948], ["emphasize", 2.952853621311967], ["rebelliousness", 2.9526004400017283], ["lies", 2.952600440001728], ["poaceae", 2.952548640902765], ["entertained", 2.9524301074955264], ["thuringiensis", 2.9524301074955264], ["fanfictions", 2.9524301074955264], ["remington's", 2.9515858323718507], ["stepkids", 2.9515858323718507], ["hazards", 2.9512945574541822], ["novels", 2.9512220084237626], ["immediate", 2.950913839430502], ["uuid", 2.950327237048592], ["imagery", 2.950067877842212], ["optimizations", 2.9499853178820916], ["morphing", 2.949562264278866], ["detoxifying", 2.9488332081499062], ["michelangelo", 2.9483225488976976], ["wrathful", 2.948311986203614], ["lernen", 2.948311986203614], ["glac\u00e9", 2.948311986203614], ["petar", 2.948064955454875], ["dnieper", 2.9474973103301116], ["shining", 2.947274353590532], ["heightening", 2.946973979508769], ["stagnate", 2.9468993540579107], ["antagonistically", 2.946397497900884], ["lachlan's", 2.946397497900884], ["booths", 2.9462351172176158], ["bragging", 2.945833594073535], ["rugelach", 2.9456893994875455], ["jouni", 2.9456893994875455], ["fossicking", 2.9456893994875455], ["encyclopedias", 2.945618233817864], ["subjectivity", 2.945501118984577], ["khlong", 2.9452818608323867], ["corruptive", 2.9452818608323867], ["topographies", 2.9452818608323867], ["carrom", 2.9452818608323867], ["massenet", 2.9449482123633035], ["thelma's", 2.9445873007897427], ["iaea's", 2.9445873007897427], ["knowledgebase", 2.9444345415398576], ["emil's", 2.9437691326039164], ["urged", 2.943478466324016], ["tama", 2.943202638657185], ["recount", 2.943189079572861], ["annoyance", 2.942629355503175], ["adverbs", 2.9425011774857808], ["glaringly", 2.942421252964446], ["eurocup", 2.942421252964446], ["thara", 2.941887528811903], ["gurpreet", 2.941887528811903], ["jerome's", 2.941698299339639], ["rusty", 2.94113947572695], ["moirai", 2.9406540329926956], ["violists", 2.9406540329926956], ["fishiness", 2.9406540329926956], ["bellflowers", 2.9406540329926956], ["meer", 2.9402236093930183], ["samarkand", 2.94019455580004], ["pilgrimage", 2.9393308550011885], ["nonagenarian", 2.939135100124414], ["rach", 2.9388002345208877], ["entered", 2.9383513958615035], ["tranquilize", 2.938235731978721], ["tarnishes", 2.938235731978721], ["clementine's", 2.938235731978721], ["commentator's", 2.938235731978721], ["shortchanging", 2.938235731978721], ["croissant", 2.9379108855738587], ["handloom", 2.937876973037088], ["grasslands", 2.937869322734179], ["capricious", 2.9376007809831557], ["nosotros", 2.937450946793897], ["sugaring", 2.9374133878052033], ["moshing", 2.9374133878052033], ["poking", 2.937346512345178], ["embarks", 2.9372407738472575], ["courtyards", 2.9371195930491942], ["savitri", 2.9371195930491942], ["outwards", 2.937032769550071], ["cariocas", 2.936903198766939], ["waterford's", 2.936903198766939], ["protegee", 2.936903198766939], ["gemologist", 2.936903198766939], ["elettra", 2.936903198766939], ["unrest", 2.936872095411873], ["gumbo", 2.93680212838361], ["hearing", 2.9361593589719575], ["cozies", 2.935590102830532], ["kamasi", 2.9354768251881307], ["clock's", 2.9353208155779487], ["unies", 2.935320815577948], ["ombres", 2.935320815577948], ["hoysala", 2.935320815577948], ["numbers", 2.935089307118089], ["bourgogne", 2.9350506525944624], ["ushered", 2.934998929083094], ["bubble's", 2.9347252950229517], ["subak", 2.9347252950229517], ["titano", 2.9347252950229517], ["puzzlingly", 2.9347252950229517], ["bolden's", 2.9347252950229517], ["folklorico", 2.9347252950229517], ["fear", 2.9347086765907644], ["dina's", 2.9344128907973803], ["sewing", 2.934371385664554], ["duller", 2.9342995611793796], ["switcheroo", 2.9342613068340158], ["trolleybus", 2.9342613068340158], ["estragon", 2.9342613068340158], ["indifferent", 2.934104394464666], ["carboy", 2.9340394512989456], ["mach's", 2.93382396661539], ["plantable", 2.93382396661539], ["secretos", 2.93382396661539], ["chiara's", 2.93382396661539], ["r\u00e9sultats", 2.93382396661539], ["vivekananda's", 2.93382396661539], ["earthrealm", 2.93382396661539], ["fanar", 2.93382396661539], ["conjured", 2.933546098520683], ["questionably", 2.933353928062661], ["submitting", 2.9322999022171476], ["madonnina", 2.9322999022171476], ["acehnese", 2.9322999022171476], ["dropbox's", 2.9322999022171476], ["cuar\u00f3n's", 2.9322999022171476], ["moutai", 2.9322999022171476], ["converge", 2.9321672189636536], ["guttural", 2.9319822099200494], ["uncorking", 2.931649868986774], ["wulfric", 2.931649868986774], ["shoulders", 2.931485789406802], ["crunchy", 2.931076978254221], ["tac's", 2.93106055314605], ["flareups", 2.93106055314605], ["narmer", 2.93106055314605], ["vasilyevich", 2.93106055314605], ["edible", 2.930926459639997], ["dunnottar", 2.9308399570330717], ["hubspot", 2.930665398369758], ["neurotypical", 2.9301190586859622], ["enology", 2.9300329521574513], ["dobry", 2.9300329521574513], ["cerave", 2.9300329521574513], ["recalibrating", 2.929864690007272], ["critical", 2.9295871577696024], ["lomi", 2.929440168068941], ["goulash", 2.929389430850183], ["osint", 2.929167103176318], ["roti", 2.9284875946519686], ["climbed", 2.928174165175241], ["compostela", 2.9280554648069907], ["tribe", 2.9278826555695767], ["shoeboxes", 2.927863745027712], ["overproduced", 2.927863745027712], ["gnomes", 2.927788671598352], ["brigid", 2.927788671598352], ["grenada's", 2.9276422894838783], ["machines", 2.9274871396906725], ["pretoria's", 2.9272311037690626], ["bizi", 2.9272311037690626], ["hakeem's", 2.9272311037690626], ["conduct", 2.926964111319086], ["ephemeral", 2.926874420152578], ["googoo", 2.926516273218692], ["einen", 2.9265162732186916], ["centerpiece", 2.9262133212235346], ["desaturated", 2.9261442269088493], ["moli\u00e8re", 2.9260775151567406], ["parsed", 2.9257210242314042], ["saree", 2.9256968129039223], ["allergen", 2.9256968129039223], ["kevon", 2.9256144444702215], ["sceptics", 2.9255010617455586], ["materials", 2.9254227069496057], ["bojack's", 2.9254050082582896], ["ebeneezer", 2.9254050082582896], ["slogans", 2.9252379290522112], ["plaintext", 2.92511289265289], ["pasargadae", 2.924964567877643], ["ticwatch", 2.924964567877643], ["sociopolitical", 2.924830977351592], ["brewhouse", 2.924830977351592], ["nast's", 2.9245810665576117], ["blockchains", 2.924244133255013], ["lowering", 2.923920833737793], ["between", 2.9237341169144386], ["catechins", 2.9234409901258576], ["marlborough's", 2.923225587754053], ["household's", 2.922901505094657], ["odyssey", 2.9228962102230387], ["tehsil", 2.9228522988270003], ["chesterfield's", 2.922852298827], ["petronilla", 2.922852298827], ["articulating", 2.922742210228174], ["markings", 2.9227119120691314], ["factors", 2.9223096426488655], ["puedo", 2.9217888485083674], ["entranceway", 2.921759643762018], ["dazzling", 2.9217123201526274], ["epicurean", 2.9215857294975085], ["both", 2.9215817081412028], ["sneaker", 2.920247840024777], ["populating", 2.9201542813203902], ["acadians", 2.919869416011511], ["distinctively", 2.9191584912841346], ["veritable", 2.919031576867749], ["brambleton", 2.919031576867749], ["jelena's", 2.9190315768677486], ["femenina", 2.9190315768677486], ["kachinas", 2.9190315768677486], ["elrond's", 2.9190315768677486], ["luhur", 2.9190315768677486], ["yatsuhashi", 2.9190315768677486], ["yoshua", 2.9190315768677486], ["enchantresses", 2.9190315768677486], ["petai", 2.9190315768677486], ["gimmicky", 2.9187662103607606], ["unconsciously", 2.9187662103607606], ["brutalism", 2.9183365693494467], ["breaking", 2.9181797503803173], ["salieri", 2.9175217329486793], ["familiarize", 2.9174939994569193], ["mental", 2.9171445697685017], ["juanita", 2.917101486719316], ["sundress", 2.916615159999481], ["shadowfax", 2.916615159999481], ["dependency", 2.9163197877140123], ["changes", 2.9160970655779734], ["fenwick's", 2.915912953388189], ["partygoer", 2.915912953388189], ["anaxagoras", 2.915912953388189], ["wearable", 2.9157008869915795], ["watchtowers", 2.9153827873966645], ["quanta", 2.9153827873966645], ["gautama", 2.9153827873966645], ["amaretti", 2.915382787396664], ["panino", 2.915382787396664], ["elain", 2.915382787396664], ["pieterson", 2.915382787396664], ["recommenders", 2.915382787396664], ["viewpoint", 2.9152408436414965], ["performers", 2.9151376118195556], ["amar", 2.915084569026431], ["kusama", 2.914878483160823], ["vaishnavas", 2.9146354449748757], ["envision", 2.914544551977631], ["rhinoceroses", 2.9145435209377557], ["peppering", 2.9139402427220493], ["gzip", 2.913837570503216], ["cryptids", 2.913837570503216], ["sibling", 2.9137063165585984], ["ludzie", 2.9135031079721663], ["mushier", 2.9135031079721663], ["thiago's", 2.9135031079721663], ["kannagi", 2.9135031079721663], ["every", 2.9131509346472515], ["ramekins", 2.91298385951236], ["herbal", 2.912944949889782], ["maharajas", 2.9126858560484714], ["regeln", 2.912685856048471], ["evangeline's", 2.912685856048471], ["updates", 2.9125853821354992], ["mathematically", 2.912507251323426], ["curds", 2.9125072513234254], ["reliable", 2.9122017914475133], ["vacuums", 2.912196986317681], ["decryption", 2.9121180810277987], ["balsamic", 2.911999364432567], ["showmanship", 2.9113808892670874], ["watercolors", 2.9111093617322], ["unmet", 2.9108778573792917], ["microscope", 2.9102791008579953], ["tadpoles", 2.9101987723844314], ["daphne's", 2.910022220149022], ["pedir", 2.9100222201490213], ["skydeck", 2.9100222201490213], ["bryggen", 2.9100222201490213], ["mimar", 2.9100222201490213], ["slack's", 2.9100222201490213], ["vaccine's", 2.9100222201490213], ["pancreases", 2.9100222201490213], ["lemmas", 2.9097437491231695], ["akira's", 2.9097437491231695], ["burrata", 2.909416848353692], ["hapi", 2.9090953876877905], ["oakfield", 2.9090953876877905], ["lorelei's", 2.908965950740619], ["darum", 2.908965950740619], ["hobbys", 2.908965950740619], ["dreidels", 2.908965950740619], ["mushroom's", 2.908965950740619], ["ryback's", 2.908965950740619], ["rasgulla", 2.908965950740619], ["tarea", 2.908965950740619], ["zipf's", 2.908965950740619], ["sentences", 2.908950106699493], ["concealed", 2.9088844671005423], ["racecar", 2.908853374658934], ["springtime", 2.9087367780029028], ["windermere's", 2.908556583170855], ["exude", 2.908448985048289], ["cheeks", 2.9081569698207943], ["redwoods", 2.908045974159107], ["villager", 2.9077267631412145], ["raval", 2.9066417993767995], ["melina", 2.906623589617374], ["antoine's", 2.906521441538316], ["trotsky's", 2.906164309211531], ["f\u00fatbol", 2.906164309211531], ["lurched", 2.9059206723275803], ["cornmeal", 2.9059206723275803], ["motherlode", 2.9056261505675525], ["karnataka's", 2.9054673037410406], ["campsites", 2.9054299305411773], ["bihar's", 2.9051754143193262], ["bayezid", 2.9051754143193262], ["pol\u00edticas", 2.9051754143193262], ["neel's", 2.9051754143193262], ["chokwe", 2.9051754143193262], ["safeguards", 2.9050445505619242], ["colombo's", 2.9049591105880834], ["farmland", 2.9048599391898753], ["caricatures", 2.9047515640929236], ["tano", 2.9044364189834107], ["allocation", 2.9038791493187355], ["monolingualism", 2.9035048131610055], ["spellwork", 2.9035048131610055], ["lamai", 2.9035048131610055], ["kibana", 2.9035048131610055], ["downriver", 2.9033215110894677], ["signups", 2.9029769031949764], ["shakira's", 2.9029769031949764], ["brittany's", 2.9029769031949764], ["known", 2.9026072584820284], ["components", 2.9021648816835928], ["salak", 2.9021477181326527], ["moonscape", 2.9021477181326527], ["grandparent's", 2.9020907686091415], ["tiempo", 2.9020907686091415], ["splurges", 2.9019612167691067], ["nihongo", 2.9019612167691067], ["immutably", 2.901715287852432], ["charmingly", 2.9013761439431454], ["elucidating", 2.9013761439431454], ["realm's", 2.9013761439431454], ["hongi", 2.9013761439431454], ["meats", 2.90100510703047], ["palio", 2.900878395419566], ["decorations", 2.900819358116422], ["heatstroke", 2.9007876295123256], ["benji's", 2.9007495419395584], ["wetsuit", 2.90053067250732], ["schitt's", 2.9005306725073194], ["heston's", 2.9005306725073194], ["rev's", 2.9005306725073194], ["tripods", 2.9003711010338917], ["belching", 2.8998754321445546], ["they", 2.899452218753117], ["thylakoid", 2.8988623888366276], ["rightful", 2.89884160846108], ["pleasantly", 2.8987677639122604], ["telescopes", 2.898598355829675], ["protech", 2.898571075160732], ["nabawi", 2.898571075160732], ["belgrad", 2.898571075160732], ["bjurman", 2.898571075160732], ["thistle's", 2.898571075160732], ["ern\u0151", 2.898571075160732], ["bvb's", 2.898571075160732], ["munky", 2.898571075160732], ["smartphones", 2.8984196396106827], ["frodo", 2.898352039563113], ["artworks", 2.8978077847776067], ["resizing", 2.8973235888752193], ["zippy", 2.8972049812401126], ["plitvice", 2.897182538118739], ["bratwurst", 2.8967739610941323], ["sweeteners", 2.896458557258496], ["sugarcoat", 2.8963941891287748], ["elde", 2.8963153778259767], ["ramc", 2.8963153778259767], ["quran's", 2.8963153778259767], ["percival", 2.896232252743382], ["kerogen", 2.895898041316203], ["gusty", 2.895769371499394], ["wapi", 2.8956526054283045], ["moniker", 2.8954254954200356], ["marine", 2.8952186511278444], ["scenery", 2.895051641053606], ["caters", 2.8947063137271845], ["belle's", 2.8947063137271845], ["d\u00e9j\u00e0", 2.8947063137271845], ["iguanas", 2.8947063137271845], ["scholar's", 2.894706313727184], ["luon", 2.894706313727184], ["sziget", 2.894706313727184], ["grainier", 2.894706313727184], ["democratised", 2.894706313727184], ["fascinators", 2.894706313727184], ["uq's", 2.894706313727184], ["pisa's", 2.894706313727184], ["sanzang", 2.894706313727184], ["hohenschwangau", 2.894706313727184], ["unlike", 2.8940059815545083], ["assuage", 2.8937378606279114], ["unsteadily", 2.8936405161153558], ["desalination", 2.893365861625435], ["bandwagons", 2.8930718098797947], ["mughals", 2.892900508166531], ["punctual", 2.892259393766299], ["directly", 2.89164911220477], ["chompers", 2.891201826664561], ["bijan", 2.891110405262927], ["fractured", 2.8910929493965956], ["skyler's", 2.8910467608653296], ["eris", 2.8910467608653296], ["pulla", 2.8910467608653296], ["granada's", 2.8910467608653296], ["kaveri", 2.891014086286206], ["nematocysts", 2.890691464470975], ["chunnel", 2.890691464470975], ["container's", 2.890691464470975], ["neuroscientific", 2.890118823531103], ["liang", 2.8901115418006333], ["timbre", 2.8899748552215825], ["pyramids", 2.88987536193525], ["adi's", 2.889404653811933], ["contained", 2.8893832215697226], ["gandhiji", 2.889370449554415], ["mallards", 2.889370449554415], ["pupillage", 2.889041526420477], ["encarnaci\u00f3n", 2.889041526420477], ["liling", 2.889041526420477], ["adana", 2.8883443218288827], ["exhaled", 2.888036483511223], ["ahmet", 2.887697339601937], ["naturally", 2.8875722456597677], ["transactional", 2.887245730444383], ["splattered", 2.8871875960291917], ["kanban", 2.8871875960291917], ["legian", 2.8871875960291917], ["delegate's", 2.8871875960291917], ["orchids", 2.8871875960291913], ["worthington", 2.8867209169996606], ["granddad's", 2.8863802169475274], ["tatties", 2.8863802169475274], ["twinges", 2.8863401105083195], ["sigrun", 2.886266936719233], ["apenas", 2.886266936719233], ["perishable", 2.8861866890773196], ["silvering", 2.8860911163649012], ["magique", 2.8860911163649012], ["antinori", 2.8860911163649012], ["landlubbers", 2.885960977431936], ["impractical", 2.885829503041858], ["emi's", 2.885579177642185], ["bioprinting", 2.885579177642185], ["retrained", 2.885413516572362], ["adler's", 2.8853985763840106], ["rebbetzin", 2.8850893492297516], ["lepre", 2.8850893492297516], ["navigli", 2.8850893492297516], ["fum\u00e9", 2.8850893492297516], ["bellic", 2.8850893492297516], ["moca's", 2.8850893492297516], ["alternative", 2.885081475691567], ["alumna", 2.8848937499518383], ["entr\u00e9e", 2.8848937499518383], ["appropriation", 2.884747848857802], ["reassembling", 2.8847417481033384], ["colorfully", 2.884102993896682], ["bastia", 2.884102993896682], ["homeschoolers", 2.8840247406138], ["midnight's", 2.8840247406138], ["casual", 2.884003197945336], ["signage", 2.8837677921730083], ["indecisiveness", 2.883751890591036], ["recursively", 2.8833767391137712], ["bard's", 2.8833456139087894], ["redouble", 2.8833456139087894], ["fasciitis", 2.8833456139087894], ["meenakshi", 2.883298604116437], ["predetermined", 2.883127488472276], ["retired", 2.8831274884722755], ["neurotransmitters", 2.882933232634242], ["clarinetist", 2.8828911859160526], ["reverend", 2.8824508038794563], ["atmospheres", 2.882449641191153], ["arjan", 2.8821751175638632], ["impressionists", 2.8821274704301625], ["oblast", 2.882116423436635], ["galilei", 2.881619937987007], ["luton's", 2.881366653295262], ["doppelbock", 2.881366653295262], ["jamy", 2.881366653295262], ["guangxu", 2.881366653295262], ["ajar", 2.8813666532952618], ["coworking", 2.8811796487097676], ["tearful", 2.881161847863861], ["jonna", 2.880623266645805], ["chilling", 2.880190585273509], ["kubrick's", 2.8799374039632704], ["arsia", 2.8799374039632704], ["buddleja", 2.8799374039632704], ["spud's", 2.8799374039632704], ["standstills", 2.8799374039632704], ["coleen's", 2.8799374039632704], ["stammtisch", 2.8799374039632704], ["nalp", 2.8799374039632704], ["kahekili", 2.8799374039632704], ["doubt's", 2.8799374039632704], ["nadya's", 2.8799374039632704], ["c\u00e9line", 2.8795629808988332], ["mentored", 2.878713461165156], ["havana's", 2.8787134611651557], ["cookies", 2.87835310273405], ["invincible", 2.8777140117297213], ["making", 2.8775716544522694], ["caravaggio", 2.877475919002618], ["isaan", 2.8774104041937787], ["eking", 2.8771396196796353], ["quieted", 2.87700660864047], ["frankenfood", 2.876726771405898], ["mahmud's", 2.876726771405898], ["usera", 2.876726771405898], ["dynasty's", 2.8767267714058975], ["yukata", 2.8767267714058975], ["inca", 2.876462366371761], ["eden's", 2.87632722602098], ["eccentricity", 2.8755835635126368], ["twc's", 2.875536073238925], ["decoding", 2.875506731034096], ["masse", 2.875349614972778], ["councilors", 2.8744469447138608], ["thermometers", 2.8744469447138608], ["transformational", 2.874443369531094], ["resemblance", 2.8744019102779776], ["lina", 2.8738500446344917], ["archrival", 2.873499674066179], ["swathed", 2.8733628570361085], ["reverberations", 2.8733168582070325], ["frailty", 2.872941604601416], ["timurid", 2.872941604601416], ["opened", 2.8725754779267154], ["diagonals", 2.872047164500357], ["recipient's", 2.872047164500357], ["enticed", 2.871895969109609], ["battlefields", 2.871842905342663], ["glowered", 2.871732454094429], ["excelente", 2.871732454094429], ["chettinad", 2.871732454094429], ["menorah", 2.8716409247731827], ["tommy", 2.8714060668472325], ["confining", 2.8713373262874775], ["embracement", 2.871230841438692], ["doorstops", 2.871230841438692], ["ragnhild", 2.871230841438692], ["underlining", 2.8710760581049217], ["sidelining", 2.8710760581049213], ["subversively", 2.870783121051753], ["spiralizer", 2.870783121051753], ["marvelling", 2.8703810505866194], ["earlier", 2.870305909721421], ["remoulade", 2.87025598496636], ["latex", 2.870158149768997], ["acutely", 2.8698945453238087], ["conquistador", 2.86979489966757], ["ambrose", 2.8697543658413034], ["sphagnum", 2.869712774126714], ["bodhidharma", 2.869688516404682], ["disrupts", 2.8695931878128365], ["intimate", 2.868887690953817], ["biotechnological", 2.8688607216403343], ["vojvodina", 2.8687653504518225], ["positive", 2.8684131193700075], ["capybara", 2.8683108154257204], ["salamanders", 2.8682891974909936], ["poltava", 2.8682515662712844], ["coifs", 2.868026992863339], ["dhotis", 2.868026992863339], ["calvino's", 2.868026992863339], ["futuristic", 2.867667848159403], ["harmonized", 2.8675896269475385], ["ammu", 2.867263854394694], ["breathing", 2.8672531528759797], ["darija", 2.866698341425897], ["dukedoms", 2.866698341425897], ["madchester", 2.866698341425897], ["atacama", 2.8666983414258964], ["documentarian", 2.8665111223597117], ["hispano", 2.866187039700316], ["nonviolently", 2.865916287098795], ["airshows", 2.8655845198265086], ["thinkers", 2.865266301548786], ["karakorum", 2.8652410686945102], ["griller", 2.8652410686945102], ["parviz", 2.8646734469447], ["formalities", 2.8645784319996226], ["lucas", 2.8645489204238936], ["danial", 2.8645165559075707], ["cappadocia", 2.8642997348014787], ["filippo", 2.8641772922017776], ["mustafa", 2.863758301525124], ["competition", 2.8636687141377215], ["outings", 2.8634394511050436], ["requesting", 2.8631837025226843], ["yowls", 2.862896354235677], ["unferth", 2.862896354235677], ["brauerei", 2.862896354235677], ["simion", 2.862896354235677], ["powertrains", 2.8628963542356765], ["gesha", 2.8628963542356765], ["m\u00eames", 2.8628963542356765], ["lcvp", 2.8628963542356765], ["gnuplot", 2.8628963542356765], ["wwii's", 2.8628963542356765], ["independente", 2.8628963542356765], ["r\u00eave", 2.8628963542356765], ["crookedest", 2.8628963542356765], ["yogacara", 2.8628963542356765], ["kurung", 2.8628963542356765], ["lamis", 2.8628963542356765], ["rustier", 2.8628963542356765], ["napoleone", 2.8628963542356765], ["mortiz", 2.8628963542356765], ["oviedo's", 2.8628963542356765], ["cinquantenaire", 2.8628963542356765], ["peppers", 2.8628545906127116], ["baker's", 2.862628793828739], ["mournful", 2.86236322642297], ["indirectly", 2.8618174199371347], ["instability", 2.8618174199371347], ["manawatu", 2.86166658776049], ["hyperlocal", 2.861578261991554], ["ceasefires", 2.861419111534833], ["aesthetically", 2.8613571626473777], ["scorsese's", 2.8613571626473777], ["nandini", 2.8609488056953807], ["stari", 2.8609488056953802], ["backstory", 2.8605451833167392], ["none", 2.860540323980977], ["rusty's", 2.8603817329320007], ["nightingale's", 2.8603817329320007], ["hist\u00f3rico", 2.860146213645633], ["minerality", 2.860146213645633], ["excavating", 2.8597311841370194], ["hairstylists", 2.859731184137019], ["saqqara", 2.859731184137019], ["lilypad", 2.8596614431009475], ["malefactor", 2.8596614431009475], ["angles", 2.8596031821069587], ["sighting", 2.8595563084890685], ["unhurried", 2.8595123159918994], ["spacely", 2.85934877511023], ["islamabad's", 2.85934877511023], ["peyto", 2.85934877511023], ["orchard's", 2.85934877511023], ["bheki", 2.85934877511023], ["holographic", 2.8588917075496925], ["arising", 2.858788325181345], ["thirsted", 2.85862656101534], ["nashville's", 2.8585224848055946], ["feasibility", 2.8583157200689002], ["ppes", 2.858218419016337], ["heti", 2.858218419016337], ["parallelizing", 2.8579000255197333], ["recuerdos", 2.8579000255197333], ["garavani", 2.8579000255197333], ["savannahs", 2.857900025519733], ["glamorous", 2.8576718703824833], ["milsim", 2.8575351251078946], ["bient\u00f4t", 2.8575351251078946], ["dilshad", 2.8575351251078946], ["coen's", 2.8575351251078946], ["monopolization", 2.8572894283558576], ["brooklands", 2.8571127252372213], ["ethnicities", 2.8568979047315644], ["jeanne", 2.8567324009956883], ["suspense", 2.8566469639638936], ["sweat", 2.8562300931596307], ["trevor's", 2.8560799343446397], ["moselle", 2.8560799343446397], ["device's", 2.855968189756262], ["deviated", 2.8555555943530213], ["waiting", 2.855403034212851], ["oluf", 2.855322554356746], ["amalia's", 2.855322554356746], ["kanopy", 2.855322554356746], ["barako", 2.855322554356746], ["hamata", 2.855322554356746], ["knitty", 2.855322554356746], ["materialize", 2.8544961806954428], ["freude", 2.854451232246781], ["beta's", 2.854451232246781], ["tinier", 2.854361974859906], ["boater", 2.854361974859906], ["misconception", 2.854100708810013], ["she'd", 2.8540106317763936], ["perfectionist", 2.8536731056717737], ["repetitions", 2.8535591024446374], ["avogadro's", 2.8533533663882245], ["curandero", 2.8533533663882245], ["replant", 2.8521371032311964], ["cardboard", 2.8519960486418694], ["varma's", 2.851927402686881], ["bi\u00e8re", 2.851927402686881], ["septime", 2.851927402686881], ["piast", 2.851927402686881], ["whipped", 2.851867631753292], ["prideful", 2.8516303269157683], ["lingers", 2.8515433047201895], ["pastry", 2.851157634671112], ["milder", 2.8510977510788265], ["refills", 2.8510977510788265], ["assimilation", 2.8509522275104784], ["kenneth's", 2.850797478993265], ["lardons", 2.850542971908878], ["oakley's", 2.8504849538560424], ["shawshank", 2.850230603774527], ["sorrowful", 2.850134274679203], ["esther's", 2.8497574579239675], ["ravana", 2.84938801527353], ["tycoons", 2.84938801527353], ["grimaced", 2.849351468258648], ["crema", 2.8492635144536025], ["luftwaffe's", 2.8491982855551234], ["\u010dech", 2.8491982855551234], ["\u00faltima", 2.8491982855551234], ["solar", 2.8491814761257106], ["eloise", 2.849130942267715], ["lockdown", 2.8488557648649535], ["entranceways", 2.848758594461674], ["intercambio", 2.848758594461674], ["terns", 2.8487585944616733], ["octal", 2.8487585944616733], ["doting", 2.8484809647380147], ["github's", 2.84804345486221], ["kadhi", 2.84804345486221], ["darnay", 2.8476139893554153], ["waitress", 2.8475434481670487], ["hsbc's", 2.8472521118628045], ["retrying", 2.8472521118628045], ["jody's", 2.8472521118628045], ["overcooked", 2.846991374489923], ["endangered", 2.8468419907022553], ["discern", 2.8467855453402717], ["ergonomics", 2.846586520460031], ["another", 2.8465165963854395], ["cushions", 2.8461499497549863], ["sestertius", 2.8460557874460553], ["neurolinguistics", 2.8460557874460553], ["aditya's", 2.8460557874460553], ["apex's", 2.8460557874460553], ["zuba", 2.8460557874460553], ["gediminas", 2.8460557874460553], ["blaney's", 2.8460557874460553], ["spiky", 2.8454065366504886], ["papercraft", 2.844544454529038], ["menu", 2.844219363734235], ["disarmingly", 2.844036694458103], ["true", 2.8439476302783393], ["biotech", 2.8430864277385517], ["cyberbullied", 2.8430151295534847], ["waaf", 2.8430151295534847], ["tr\u00e9sor", 2.8430151295534847], ["genre", 2.8427077765665056], ["forcefulness", 2.8422149564238612], ["rspo", 2.842075289841236], ["fiorentina's", 2.842075289841236], ["dwayne's", 2.842075289841236], ["nayla", 2.842075289841236], ["habanero", 2.841689714574672], ["babbage", 2.8413476719994946], ["latencies", 2.840947482186537], ["overprescribing", 2.840599653657517], ["mismanaged", 2.8402945409127636], ["jeden", 2.8402945409127636], ["bendingly", 2.839901202796033], ["cambios", 2.839901202796033], ["est\u00e3o", 2.839901202796033], ["fant\u00f4me", 2.839901202796033], ["olexandr", 2.839901202796033], ["saraiki", 2.839901202796033], ["hung", 2.8396456301406805], ["simply", 2.8395208055033425], ["natalya", 2.839038884617046], ["manipulated", 2.8390149072606454], ["langston", 2.8390121085568683], ["bukovina", 2.838875103361083], ["greetings", 2.838773801621284], ["getaways", 2.838634520725205], ["shriveled", 2.8385648951544247], ["chopping", 2.838508718625521], ["keffiyeh", 2.838364251800067], ["veeru", 2.838364251800067], ["mailboxes", 2.8383642518000665], ["philosopher's", 2.8378079095261586], ["dima", 2.8375160674073463], ["prototype", 2.837387213643449], ["mercutio", 2.8370045267425894], ["morada", 2.8366071315154224], ["preened", 2.83652126219498], ["talia's", 2.83652126219498], ["remember", 2.8365072094660158], ["benchwarmer", 2.8363880546594205], ["penpals", 2.8363880546594205], ["sidechain", 2.8363880546594205], ["language", 2.836270660761297], ["stiftung", 2.835630674671528], ["bastoni", 2.8356306746715276], ["javed's", 2.8356306746715276], ["chocolatey", 2.8356306746715276], ["spello", 2.8356306746715276], ["gece", 2.8356306746715276], ["hbar", 2.8356306746715276], ["effaclar", 2.8356306746715276], ["kompa", 2.8356306746715276], ["nezahualcoyotl", 2.8356306746715276], ["commas", 2.835422417424563], ["viele", 2.835056195199349], ["pikas", 2.834759780729675], ["energiewende", 2.834759780729675], ["attention", 2.834744540085693], ["yogic", 2.8347445400856928], ["proud", 2.834466796409535], ["rheas", 2.834456986478369], ["nexgen", 2.834456986478369], ["continuums", 2.833442688039837], ["crossfitter", 2.833442688039837], ["lorikeet", 2.833442688039837], ["ath\u00e9n\u00e9e", 2.833442688039837], ["lasalle's", 2.833442688039837], ["jhansi", 2.8332357163314335], ["aiko", 2.8331895325617893], ["urls", 2.8329854968033636], ["flamingos", 2.832800703938322], ["asker", 2.832676892718745], ["cytokinesis", 2.832557375529904], ["uncombed", 2.8323936533534093], ["ceilidhs", 2.8323936533534093], ["espa\u00f1oles", 2.8323936533534093], ["gymnosperm", 2.8323936533534093], ["predictably", 2.8322633632453553], ["ancestry", 2.8321534376891377], ["literacy", 2.8321039929445275], ["reasoned", 2.832068761140231], ["demographer", 2.8319191161968265], ["lately", 2.831775816823421], ["kira's", 2.831639894551084], ["allie's", 2.831136806724346], ["quesadillas", 2.831086394744169], ["adjustability", 2.8308729050831523], ["squanto", 2.8308729050831523], ["exhibit's", 2.8306383671456974], ["brashly", 2.8306383671456974], ["hospitality", 2.8303852936218754], ["splashing", 2.83015537153954], ["delighted", 2.8298527889980347], ["prehistoric", 2.8297680352063086], ["lahti", 2.8294121863498796], ["kimo", 2.8294121863498796], ["tectonics", 2.829412186349879], ["tamalpais", 2.829412186349879], ["painting", 2.8292273503779066], ["evocative", 2.8290053693568744], ["pet's", 2.8287175679735252], ["cynthia's", 2.8285415979848487], ["acidity", 2.8282275249670845], ["commonsense", 2.8280975394899337], ["carnevale", 2.82804161272735], ["platypus", 2.828035235364372], ["persuasive", 2.8277692472174283], ["contacting", 2.8272357154373022], ["activities", 2.826908843237517], ["outperformed", 2.8268134151109816], ["precarious", 2.8266740455243795], ["barrington's", 2.8266571598782635], ["cobblestoned", 2.8266571598782635], ["malalai", 2.8266571598782635], ["allurements", 2.8266571598782635], ["karahi", 2.8266571598782635], ["stationery", 2.8266330004153586], ["recommended", 2.826617900751043], ["crickets", 2.8265505301500102], ["multitalented", 2.8263840529062945], ["tosca", 2.82601836729976], ["fanfare", 2.8258719773338528], ["chandeliers", 2.8256190579565663], ["peranakan", 2.8255394623061485], ["tanvi", 2.8255394623061485], ["jarek", 2.8255394623061485], ["spoonfuls", 2.8254549665088304], ["congratulatory", 2.8253996980752962], ["conviction", 2.8249456895205913], ["symposiums", 2.8248533621411034], ["worried", 2.8247355773047054], ["haywire", 2.824673096459591], ["colchester's", 2.824553992348592], ["orson's", 2.824553992348592], ["joey's", 2.8243943859750624], ["boybands", 2.8243943859750624], ["jokes", 2.8243676556366175], ["gondry's", 2.824103720709448], ["giethoorn", 2.824103720709448], ["schr\u00f6der's", 2.824103720709448], ["ivanovi\u0107", 2.824103720709448], ["imhotep", 2.8238155468603963], ["guests", 2.8237773610861017], ["moderator", 2.8234247741506895], ["grittier", 2.8234081286600126], ["morioh", 2.8234081286600126], ["xavier's", 2.8234081286600126], ["stormier", 2.8234081286600126], ["yinyang", 2.8234081286600126], ["enrico's", 2.8234081286600126], ["alonzo's", 2.8234081286600126], ["huginn", 2.8234081286600126], ["deconstructionism", 2.8234081286600126], ["chamula", 2.8234081286600126], ["markinson", 2.8234081286600126], ["confidence", 2.823268403357733], ["gazetteers", 2.823050893494749], ["olivier's", 2.823010801312889], ["tambourines", 2.822987179965889], ["kristiansand", 2.8228958063721046], ["elucidated", 2.82273124715621], ["spearheading", 2.82273124715621], ["kostroma", 2.8223913124383415], ["heighten", 2.8218008285342204], ["stonework", 2.821515256490087], ["brunt", 2.8213988161717554], ["hellbent", 2.821338181351611], ["denser", 2.8209114918765152], ["socially", 2.8205702732675357], ["shackles", 2.8201233194194177], ["lughnasadh", 2.8195191367472576], ["buyuk", 2.8195191367472576], ["brandenburg's", 2.8195191367472576], ["evenhandedness", 2.8195191367472576], ["xiaolan", 2.8195191367472576], ["xochicalco", 2.8195191367472576], ["buriganga", 2.8195191367472576], ["spitz's", 2.8195191367472576], ["lucrezia's", 2.8195191367472576], ["birdbath", 2.819519136747257], ["smorgasburg", 2.819519136747257], ["toshi", 2.819519136747257], ["ogtt", 2.819519136747257], ["melike", 2.819519136747257], ["jezero", 2.819519136747257], ["lakshmana", 2.819519136747257], ["flyknits", 2.819519136747257], ["grilledcheese", 2.819519136747257], ["kyro", 2.819519136747257], ["erzulie", 2.819519136747257], ["meadowvale", 2.819519136747257], ["macabre", 2.81938660940828], ["baha'i", 2.8190803010839507], ["borealis", 2.8189171041913683], ["launching", 2.8188149784367496], ["deduced", 2.8187422313594053], ["sharpshooting", 2.818583972854472], ["slobbery", 2.818583972854472], ["messiest", 2.818583972854472], ["brashness", 2.8184690365101077], ["toto's", 2.8184690365101077], ["randy's", 2.818362410947566], ["bern's", 2.818198518884613], ["skydiving", 2.818081734834406], ["sporks", 2.817855703628233], ["countenances", 2.817855703628233], ["grenouille", 2.817855703628233], ["quietest", 2.8178189242024856], ["unevenly", 2.8177916414337143], ["refilled", 2.8177916414337143], ["crudit\u00e9s", 2.817607598349463], ["fitbits", 2.817607598349463], ["mousses", 2.817607598349463], ["hardanger", 2.817607598349463], ["vaccinations", 2.817514145361126], ["vada", 2.8174110850562313], ["cognito", 2.8172725079530365], ["kola", 2.817244020440948], ["sunlight's", 2.817056674619094], ["aliah", 2.817056674619094], ["earthworm", 2.816088543120797], ["bernina", 2.8158092431462745], ["disarray", 2.815809243146274], ["sloth's", 2.8155198188227932], ["famosa", 2.8155198188227932], ["demotivate", 2.8155198188227932], ["foxwood", 2.8155198188227932], ["connoisseur's", 2.8155198188227932], ["brubeck's", 2.8155198188227932], ["decrementing", 2.8155198188227932], ["dongdong", 2.8155198188227932], ["cilantro", 2.8153043916459346], ["d\u00edas", 2.8151814149984156], ["unmanaged", 2.8148523464519517], ["nori", 2.814701867870403], ["fogged", 2.814668351135649], ["biogas", 2.814668351135649], ["intertextuality", 2.8146297972037653], ["marjan", 2.8146297972037653], ["athena", 2.814323618337971], ["polynomial", 2.8139238119487513], ["bearable", 2.813705695228191], ["shrewdly", 2.8135833069856844], ["hester's", 2.8135554064932675], ["attend", 2.8135147523321], ["cicadas", 2.813402213874314], ["supporting", 2.8132337525663917], ["etsy's", 2.8132302464794194], ["ruths", 2.8132302464794194], ["siddarth", 2.8132302464794194], ["vecchia", 2.812670102406981], ["hygrometer", 2.812670102406981], ["stationmaster", 2.812557361100968], ["adare", 2.812000419049265], ["fledglings", 2.812000419049265], ["eldridge", 2.8118861843881073], ["doar", 2.8114761344554267], ["ihnen", 2.811346617510309], ["salvaged", 2.8112812373564133], ["ramayana", 2.8111205691058823], ["coop's", 2.8110009520440937], ["extractives", 2.8110009520440937], ["flopsy", 2.8110009520440937], ["enoch's", 2.8110009520440937], ["solzhenitsyn's", 2.8110009520440937], ["meeting", 2.8106806441341914], ["observed", 2.810666997482009], ["ketogenic", 2.810496675509666], ["wittenberg", 2.8104688284070813], ["mechanisms", 2.8103385716328395], ["microdosing", 2.8103125436476746], ["waistlines", 2.8101726644707474], ["hindrances", 2.8100075059244953], ["lucia's", 2.8098797550831645], ["llu\u00eds", 2.809879755083164], ["katrien", 2.809879755083164], ["superheroine", 2.809879755083164], ["hortensio", 2.809879755083164], ["oppositional", 2.8098095344220355], ["swatches", 2.8097537393975727], ["viscerally", 2.809567892735209], ["byway", 2.809567892735209], ["narrative", 2.8092998184067204], ["olav", 2.8087479400450768], ["screeched", 2.807998520919688], ["subcategories", 2.8079389688641885], ["carys", 2.807727789803911], ["riches", 2.8073315378980923], ["anomaly", 2.807094137161591], ["fading", 2.807021808681675], ["zeynep", 2.806721319388972], ["recognitions", 2.8066544106480307], ["tactful", 2.806472518633774], ["retribution", 2.806314388790158], ["rodriquez", 2.8062440210573527], ["joon's", 2.805923292459842], ["thumbing", 2.8058168007555886], ["forcefully", 2.805638427150963], ["prowled", 2.805617970585374], ["bricks", 2.805535020161904], ["dresden's", 2.8053506988741557], ["gadget's", 2.8053506988741557], ["pratfalls", 2.8053506988741557], ["lugh", 2.80480860212075], ["m\u00e9thodes", 2.8048086021207497], ["treemap", 2.8048086021207497], ["diversas", 2.8048086021207497], ["carrhae", 2.8048086021207497], ["amarcord", 2.8048086021207497], ["kitaab", 2.8048086021207497], ["prompts", 2.8046361955505454], ["fathom", 2.804566181325407], ["steps", 2.8040763626597323], ["sines", 2.803931551275809], ["bright", 2.803516460565719], ["whistlers", 2.8034649352568817], ["whispery", 2.803119249150965], ["divvying", 2.803119249150965], ["sixties", 2.8029713912459986], ["vouching", 2.8028108751961627], ["reminiscence", 2.8026814450010487], ["morisot", 2.8024311419790924], ["pulao", 2.8024311419790924], ["paprika", 2.802206299942669], ["heroes", 2.8021790940562616], ["accomplished", 2.802146099683385], ["ma\u00eetre", 2.802018673139861], ["screenings", 2.8018879079096064], ["adrift", 2.8015722845029183], ["theater's", 2.801474214272075], ["confined", 2.801365016556643], ["mediterranean's", 2.801328690703727], ["yawar", 2.8013286907037265], ["sarajevo's", 2.8013286907037265], ["mud\u00e9jar", 2.8013286907037265], ["guamanian", 2.8013286907037265], ["preventative", 2.8009729664255425], ["boxcars", 2.800722342502276], ["glitch", 2.800122444813606], ["classroom's", 2.7999391427420686], ["qipao", 2.7999391427420686], ["karaoke", 2.7998558731764844], ["hockney's", 2.7997715318984437], ["banfi", 2.7997715318984437], ["expressionist", 2.7995579393189582], ["bertolli", 2.799276435252662], ["pouco", 2.799276435252662], ["coalminer", 2.799276435252662], ["monopolistic", 2.799175770823891], ["finesse", 2.7991289718642904], ["scriptura", 2.798977399205738], ["fishermen", 2.798956200761136], ["brian's", 2.7988639884521236], ["jia's", 2.79813356245656], ["bins", 2.7980111320312524], ["kosovan", 2.7980103672131986], ["anemones", 2.7979228199551507], ["reeds", 2.7978222656759075], ["haben", 2.797526887480629], ["thorny", 2.797519598476985], ["logoed", 2.797405261164926], ["cowspiracy", 2.797405261164926], ["fastq", 2.797405261164926], ["positiva", 2.797405261164926], ["piemontese", 2.797405261164926], ["coolheaded", 2.797405261164926], ["pruney", 2.797405261164926], ["somaly", 2.797405261164926], ["nyai", 2.797405261164926], ["keola", 2.797405261164926], ["planning", 2.7973550175483637], ["without", 2.797271174357288], ["ricotta", 2.7971513440131455], ["bambu", 2.7971513440131455], ["monstrosity", 2.797116166356804], ["welche", 2.7969629836532794], ["circumambulation", 2.7969629836532794], ["gonzo's", 2.7969629836532794], ["feelings", 2.796861854701786], ["flora's", 2.7967925341336777], ["watching", 2.796404450338571], ["curator's", 2.796074557492029], ["viktor's", 2.7959280184640822], ["desensitization", 2.7957758177009646], ["sfmoma", 2.795692214464886], ["fads", 2.795657338065005], ["incantation", 2.795595944071826], ["revisit", 2.7953870078912546], ["missy's", 2.795366831292812], ["wishlists", 2.795366831292812], ["karad\u017ei\u0107", 2.7951529219852445], ["macher", 2.7951529219852445], ["flouting", 2.795060657488727], ["pommes", 2.7950015790364118], ["hanne", 2.7948888546331436], ["perspectiva", 2.7947321553253035], ["hedgies", 2.7947321553253035], ["richtig", 2.7947321553253035], ["tsuburaya", 2.7947321553253035], ["jupiter's", 2.7947073683438814], ["dreamcatcher", 2.7946588604877656], ["clinic's", 2.7946588604877656], ["weekends", 2.794425416430207], ["aisles", 2.794403420181622], ["hadrian's", 2.7942225164549455], ["reusability", 2.794020876727978], ["precisely", 2.7936794930130167], ["machinery", 2.7935688196639], ["wonder's", 2.7935460843104862], ["serving", 2.793436218229473], ["bolten", 2.793377899717813], ["kassapa", 2.793377899717813], ["nebula's", 2.793377899717813], ["soho's", 2.793377899717813], ["lockton", 2.793377899717813], ["postgis", 2.793377899717813], ["metate", 2.793377899717813], ["masp", 2.793377899717813], ["rayhan", 2.793377899717813], ["autotomy", 2.793377899717813], ["tr's", 2.793377899717813], ["harbingers", 2.793324894444573], ["wwii", 2.793305229097499], ["fiasco", 2.793184828010038], ["reproductions", 2.7931461962907265], ["fale", 2.793037465297145], ["conmebol", 2.793037465297145], ["lefse", 2.792947752212467], ["antwort", 2.792947752212467], ["stipe's", 2.792947752212467], ["monarchia", 2.792947752212467], ["yakata", 2.792947752212467], ["garlic", 2.792815037230949], ["sione", 2.792321202952268], ["posture", 2.7917402502187674], ["multiplications", 2.7916800270158837], ["bobbleheads", 2.7913239453797853], ["livability", 2.7913239453797853], ["kipling's", 2.7913239453797853], ["bergamot", 2.791323945379785], ["photophores", 2.791323945379785], ["thenardier", 2.791323945379785], ["lifeguard's", 2.791323945379785], ["cooktops", 2.791323945379785], ["namdev", 2.791323945379785], ["menchie's", 2.791323945379785], ["voudrais", 2.791323945379785], ["sedan's", 2.791323945379785], ["speedy's", 2.791323945379785], ["refinements", 2.7906142266039446], ["vision", 2.7903651768115045], ["cookstove", 2.789839987939392], ["messiaen's", 2.789839987939392], ["crustaceans", 2.7897809249566894], ["forged", 2.7893153736518927], ["friend", 2.7893146162719047], ["vinnie's", 2.7891449259064207], ["fountain's", 2.7891449259064207], ["wishful", 2.788919704255427], ["biskit", 2.788919704255427], ["muskoxen", 2.788919704255427], ["fleadh", 2.7886425583044923], ["flagons", 2.7886425583044923], ["inevitable", 2.7885326214344053], ["optimist", 2.7883701634270026], ["ravel's", 2.788341761677456], ["nowruz", 2.788341761677456], ["renovations", 2.788111266204049], ["surges", 2.7880009406829043], ["neels", 2.7878391464467263], ["jari", 2.787587233270843], ["mejores", 2.787398032657732], ["modify", 2.787202055166906], ["nirguna", 2.7868290597994636], ["hypertime", 2.7868290597994636], ["wittedness", 2.7868290597994636], ["entailments", 2.7868290597994636], ["jayan", 2.7868290597994636], ["vritra", 2.7868290597994636], ["outmuscle", 2.7868290597994636], ["sauce's", 2.7868290597994636], ["determiner", 2.786829059799463], ["yellowstone", 2.78645252488174], ["separating", 2.786260319175015], ["shaking", 2.786258209330762], ["brash", 2.7861026136450677], ["muralist", 2.786067214785849], ["v\u00e9lez", 2.786067214785849], ["slipped", 2.785927322262367], ["scapa", 2.7856849071062904], ["underscore", 2.7856728191357147], ["vicariate", 2.7849944126238215], ["spoonbill", 2.7849944126238215], ["lumpur's", 2.7849944126238215], ["archaic", 2.7845521745273354], ["schema", 2.7844317874899582], ["yearns", 2.784148305417369], ["verses", 2.783347393510318], ["swallowed", 2.7832331513352058], ["hylian", 2.783069079466114], ["caponata", 2.783069079466114], ["gorged", 2.7829415911894553], ["reanalyzing", 2.7826282321543028], ["laforgue", 2.7826282321543028], ["tamariki", 2.7826282321543028], ["yulara", 2.7826282321543028], ["explosion's", 2.7826282321543028], ["pontmercy", 2.7826282321543028], ["dorayaki", 2.7826282321543028], ["pwas", 2.7826282321543028], ["commencer", 2.7826282321543028], ["compa\u00f1eros", 2.7826282321543028], ["change", 2.782625525707535], ["help", 2.782167431559749], ["queasiness", 2.782097601293861], ["zilker", 2.782097601293861], ["gesticulating", 2.7819255482572944], ["remedies", 2.7815974385352664], ["nectars", 2.781260175009684], ["periapsis", 2.781260175009684], ["blacktip", 2.781260175009684], ["guam's", 2.781260175009684], ["suwon", 2.780985708545045], ["glittered", 2.78086954858061], ["evolutions", 2.7807206255492822], ["caapi", 2.780629217619709], ["easy's", 2.780629217619709], ["nerdcore", 2.780629217619709], ["glamorization", 2.780629217619709], ["msf's", 2.780629217619709], ["manufacturing's", 2.780629217619709], ["th\u00e9nardier", 2.780629217619709], ["sifa", 2.780629217619709], ["circe", 2.7804627627129377], ["tito's", 2.780190459557757], ["aqu\u00ed", 2.7801138090529585], ["sibelius", 2.7799307864190514], ["trickiest", 2.779546207213625], ["dockworkers", 2.7791859345211836], ["sequined", 2.7791613138095026], ["affected", 2.779145171186366], ["mystifying", 2.7791347578453753], ["diversely", 2.7787746405514877], ["cathars", 2.7787746405514877], ["longboard", 2.778069795591657], ["groupers", 2.777716886308342], ["rei's", 2.777716886308342], ["imparts", 2.7768453335964884], ["seating", 2.7766854232742277], ["upscale", 2.7763732644187766], ["lakota", 2.7758475429325693], ["breadlines", 2.775745780286341], ["aktiv", 2.775745780286341], ["noelle's", 2.775745780286341], ["cheatgrass", 2.775745780286341], ["sirree", 2.775745780286341], ["aerodynamics", 2.7756525875456903], ["stitch", 2.775568188556131], ["alterations", 2.7750568637896755], ["k\u00f6lsch", 2.7750004135354587], ["mataji", 2.7750004135354587], ["heartedly", 2.774768050092835], ["yanomami", 2.77473585062064], ["demurely", 2.77473585062064], ["gl\u00fcck", 2.774580785467481], ["educator's", 2.774580785467481], ["gilmour's", 2.7744068305593013], ["mori's", 2.7744068305593013], ["upliftment", 2.7744068305593013], ["f\u00eates", 2.7741801355533315], ["nitya", 2.7737972048000046], ["bayanihan", 2.7737972048000046], ["even", 2.7737741270019414], ["visitor", 2.7736881296028755], ["healings", 2.7736732971253506], ["tane", 2.7736732971253506], ["schoolyard", 2.773580975983778], ["gravity's", 2.7734993991129917], ["chatty", 2.7734308431658117], ["mosel", 2.7734308431658117], ["tajine", 2.7730799980243614], ["antiquarianism", 2.7730799980243614], ["paisleys", 2.7730799980243614], ["authy", 2.7730799980243614], ["wieliczka", 2.7730799980243614], ["encourages", 2.7730378154871236], ["floyd's", 2.7727681356764053], ["bottlenose", 2.7727681356764053], ["heaping", 2.772590227625558], ["merge", 2.7725605973879084], ["stonemason", 2.7725271511348035], ["tomsk", 2.772445425942434], ["generalizability", 2.7722646260755157], ["mayans", 2.772179062352489], ["drafted", 2.7721734107265026], ["packaging", 2.771599677734557], ["westernization", 2.771527321653687], ["donn\u00e9es", 2.771527321653687], ["bipin", 2.771136855702482], ["sous", 2.7706474717103053], ["m\u00fasica", 2.7706474717103053], ["optometrist", 2.7702815164241192], ["shadowboxing", 2.7702459867846843], ["wto's", 2.7702459867846843], ["admire", 2.7697166979759875], ["reasoning", 2.769674461184682], ["smooth", 2.7695765786517264], ["mysteriously", 2.7695114017284657], ["bathhouses", 2.7691705807339138], ["sourness", 2.7691705807339138], ["bioderma", 2.7691705807339138], ["seront", 2.7691705807339138], ["vientos", 2.7691705807339138], ["mattancherry", 2.7691705807339138], ["sackboy", 2.7691705807339138], ["biofilters", 2.7691705807339138], ["elodea", 2.7691705807339138], ["larysa", 2.7691705807339138], ["amaro's", 2.7691705807339138], ["romeo's", 2.7688495174781766], ["yayoi", 2.768730447661479], ["could", 2.76870532154876], ["wounds", 2.768526807744205], ["industries", 2.7683518328085985], ["stopwatch", 2.768202438834223], ["grantham's", 2.76797953962392], ["suggestion", 2.767971692764843], ["feathers", 2.7678995449017196], ["mangalsutra", 2.767846674480176], ["primatologists", 2.767846674480176], ["unhealthier", 2.767846674480176], ["suppliers", 2.7677481875795293], ["yamada", 2.7676399846311077], ["regularly", 2.7674491772937575], ["showed", 2.7672147201425266], ["straddled", 2.7671117178486098], ["fortnite's", 2.7671117178486093], ["hollow's", 2.7671117178486093], ["amphibian", 2.767102933366966], ["housed", 2.767018539596886], ["yani", 2.7667090957732614], ["bivalves", 2.766369580606275], ["drained", 2.766258995666171], ["choreographing", 2.766176882808796], ["torgo", 2.766176882808796], ["madeira's", 2.766176882808796], ["activerecord", 2.766176882808796], ["inclining", 2.7655434555400054], ["ardha", 2.76516696061719], ["riceville", 2.76516696061719], ["abrahms", 2.76516696061719], ["hellworld", 2.76516696061719], ["before", 2.7649946064177775], ["pavel", 2.7648772930118457], ["frosting", 2.764739907804739], ["windfall", 2.7646178226723626], ["speculating", 2.7642854798120036], ["scaly", 2.7634779204688753], ["nonstop", 2.7632251478289533], ["butterfly's", 2.7631287540123126], ["spacesuits", 2.763128754012312], ["conservators", 2.76290409113289], ["conjunctions", 2.76290409113289], ["isixhosa", 2.762882507799496], ["rasim", 2.762882507799496], ["llegar", 2.762882507799496], ["delphi's", 2.762882507799496], ["chemica", 2.762882507799496], ["pilkington's", 2.762882507799496], ["unaccustomed", 2.7627288222198656], ["stew", 2.762568584647396], ["cathar", 2.762443850578285], ["sabbatical", 2.7624325764476834], ["retrieving", 2.7623768822425134], ["superfood", 2.7622162882722647], ["especially", 2.761986326628756], ["severity", 2.7616997579586005], ["crossword", 2.761657700549662], ["dorje", 2.761657700549662], ["arrival", 2.760918251009102], ["srpska", 2.760917366454079], ["karson", 2.7607460617840753], ["minting", 2.760471615558833], ["wsop", 2.7601608391315255], ["preventing", 2.760121266933115], ["dartmouth's", 2.7600913329282286], ["intermarriages", 2.7600913329282286], ["kuban", 2.760068810763129], ["seahorse", 2.7599166436033595], ["wristwatches", 2.759859746317533], ["kaliningrad", 2.759676281599969], ["laneways", 2.759676281599969], ["bea's", 2.759439663553017], ["sturdily", 2.759315881158348], ["unsweetened", 2.7593028578385552], ["phonographs", 2.7592034472344977], ["multiple", 2.7590418068020752], ["installations", 2.758920516494779], ["hub's", 2.758767066949941], ["petabytes", 2.758767066949941], ["virus's", 2.758767066949941], ["wardrobes", 2.758713399659464], ["applewood", 2.7585199246541676], ["divine", 2.7583128856936843], ["invested", 2.758188228866889], ["condors", 2.758093898410978], ["tendai", 2.757794528618466], ["appearances", 2.757333839884103], ["brink", 2.757246053591198], ["inflations", 2.756863155930652], ["seekin", 2.756863155930652], ["angola's", 2.756863155930652], ["spicier", 2.756863155930652], ["newline", 2.756863155930652], ["procrastinator", 2.756863155930652], ["qila", 2.756863155930652], ["anthills", 2.756863155930652], ["chevre", 2.756863155930652], ["karnstein", 2.756863155930652], ["spaatz", 2.756863155930652], ["silhouetting", 2.7568631559306516], ["cuenta", 2.7568631559306516], ["reggy", 2.7568631559306516], ["parasitological", 2.7568631559306516], ["csat", 2.7568631559306516], ["buckett", 2.7568631559306516], ["mbapp\u00e9", 2.7568631559306516], ["medication's", 2.7568631559306516], ["ezio's", 2.7568631559306516], ["skunk's", 2.7568631559306516], ["rennaissance", 2.7568631559306516], ["bu\u00f1uel's", 2.7568631559306516], ["clis", 2.7568631559306516], ["farukh", 2.7568631559306516], ["szcz\u0119sny", 2.7568631559306516], ["mirus", 2.7568631559306516], ["zoom", 2.756822373931304], ["sweets", 2.756664820451808], ["composers", 2.7565698726161907], ["overpriced", 2.7565498760265688], ["validate", 2.7554902161119608], ["wisden", 2.7554522740698895], ["flatbreads", 2.7552433773431484], ["carpentry", 2.7550902535795707], ["accountable", 2.75460627803222], ["playland", 2.7541062927747215], ["bulgur", 2.7541062927747215], ["labeling", 2.754106292774721], ["paulo's", 2.7539763882281063], ["riskier", 2.7537828619016786], ["growls", 2.7537828619016786], ["tark", 2.7537088502602507], ["about", 2.753581390423239], ["beakers", 2.7533059131488056], ["sightedness", 2.7533059131488056], ["tenaciously", 2.7533059131488056], ["eitan", 2.752924779993608], ["precolonial", 2.7527849559958435], ["monetarily", 2.7522988129572568], ["bushwhacking", 2.7522988129572568], ["consciousness", 2.752084593127039], ["monterosso", 2.751850677465323], ["aulani", 2.751850677465323], ["tlingit", 2.751850677465323], ["strategic", 2.751569773700026], ["benin's", 2.7514996089346777], ["durban's", 2.7514996089346777], ["makenna", 2.7514996089346777], ["grownups", 2.7513049640840173], ["protectant", 2.751022344159612], ["nugget", 2.7506839798914964], ["vocabularies", 2.750321446747088], ["shuddered", 2.750321446747088], ["rheumatologist", 2.7499709980408253], ["flowerbed", 2.749918916243169], ["rosa", 2.7497416235310213], ["elder", 2.7493596936019014], ["replenishing", 2.7492052027197333], ["luisa", 2.749154821448609], ["william's", 2.7487802063635254], ["weightlifters", 2.748754734883797], ["fulda", 2.748754734883797], ["amass", 2.748672136367191], ["judaism's", 2.7486090746254703], ["yacht's", 2.7486090746254703], ["accidental", 2.748438248858065], ["demise", 2.74829697282032], ["petanque", 2.7481389054372007], ["bingo's", 2.7481389054372007], ["redemption's", 2.7481389054372007], ["malpighian", 2.7481389054372007], ["heavyweights", 2.7479421999980995], ["songbird", 2.7479421999980995], ["everett's", 2.7477945271282485], ["reddened", 2.74767361207755], ["worrying", 2.7476213987602023], ["dears", 2.7473394468465275], ["ethicist", 2.7470172160880426], ["appease", 2.7465685403483793], ["granth", 2.7464060887874666], ["overindulge", 2.7461941808690007], ["cubesat", 2.7461941808690007], ["oppression", 2.746184220853384], ["tara", 2.746125264426152], ["burglaries", 2.746071164928788], ["unites", 2.7460116732956052], ["clicky", 2.7459664636147205], ["elderberries", 2.7457017666353862], ["lulu's", 2.7453021168896523], ["despised", 2.7449887152289025], ["moment's", 2.7449617492652294], ["aman's", 2.744664646391136], ["abyssal", 2.744485402985657], ["acquired", 2.7443199777470944], ["almond", 2.7439327358187646], ["kameez", 2.7434150429748927], ["shreya", 2.7434150429748927], ["bu's", 2.7433159168351695], ["swathes", 2.74327298544367], ["hauptbahnhof", 2.743009572232508], ["hailstorms", 2.7429194838992106], ["definitive", 2.7428603882368994], ["npcs", 2.742762290101779], ["carrying", 2.7420357027406466], ["instruments", 2.7419972000611117], ["underlying", 2.741951879198743], ["blossom", 2.741789659538513], ["guidepost", 2.7417818476378146], ["devoutly", 2.7417570564461005], ["bucuresti", 2.7416318677763387], ["reveal", 2.7416081715157716], ["contrast", 2.7414419526521647], ["tacos", 2.741285341815356], ["axios", 2.7411096521824763], ["neocolonialism", 2.7411096521824763], ["bashkir", 2.7411096521824763], ["powerpc", 2.7410298202378067], ["step", 2.740689558749192], ["hogged", 2.7406653700556185], ["rommel's", 2.7406653700556185], ["teapot", 2.740586473091209], ["grosbeak", 2.7400822497641175], ["interesting", 2.7397774965217714], ["grammaticus", 2.739632761206085], ["hedgie", 2.739632761206085], ["medication", 2.739588768708916], ["bogged", 2.7389449562989636], ["hundredth", 2.7385125188246118], ["siddhi", 2.7381408254998836], ["nutcrackers", 2.7381408254998836], ["perceive", 2.7380377189515825], ["musically", 2.737965303652095], ["amazonian", 2.737924704685563], ["existen", 2.7378503065794058], ["puces", 2.7378503065794058], ["atok", 2.7378503065794058], ["bushes", 2.737782760930395], ["twine", 2.7377497685981127], ["crowd's", 2.7377182729033556], ["summoned", 2.737493195669852], ["costumes", 2.7372880064600618], ["persuasiveness", 2.7372413541090816], ["roosevelt's", 2.7372196592422027], ["empires", 2.7372026577270026], ["manyara", 2.7370296080462584], ["cyrillic", 2.7368484585456754], ["habilitation", 2.736592103313515], ["chordate", 2.7365921033135145], ["trishul", 2.7365921033135145], ["apprendre", 2.7365921033135145], ["cornelis", 2.7363333239184024], ["stale", 2.7362123036659667], ["embolisms", 2.7360409266562513], ["scrutinize", 2.7357671596070086], ["reinventing", 2.735748452568755], ["flattered", 2.7357113610273904], ["arahant", 2.7356565162696467], ["incans", 2.7356565162696467], ["hastening", 2.7354974664721894], ["vienna's", 2.7352732637456527], ["electronegativity", 2.7347069652400493], ["harz", 2.7347069652400493], ["makemake", 2.734630388544114], ["elisa's", 2.734630388544114], ["campur", 2.734630388544114], ["hemlines", 2.734433056289204], ["lazzaro", 2.734433056289204], ["incidents", 2.7343976540429615], ["foodways", 2.7343581505761154], ["biohazardous", 2.7343581505761154], ["filtered", 2.7342721939584425], ["mistaking", 2.7341595769994584], ["dadaists", 2.7340791629064314], ["encomienda", 2.7340791629064314], ["finagling", 2.7340791629064314], ["yeshe", 2.7340791629064314], ["caressed", 2.7338892962978965], ["messerschmitt", 2.7333289582580726], ["hooves", 2.7331713631843724], ["casita", 2.7328334453797507], ["chignon", 2.7326801457909093], ["mendel's", 2.7326801457909093], ["caipirinha", 2.7326801457909093], ["garifuna", 2.7326801457909093], ["nyang", 2.7326801457909093], ["fortunately", 2.7324857486032905], ["concerning", 2.7324568085796184], ["difference", 2.7322743691060642], ["walmart's", 2.732190839749171], ["undiagnosed", 2.7321748291611234], ["denying", 2.732079234629861], ["skill", 2.7316530574895372], ["masks", 2.7312559289882126], ["latest", 2.7312180328593474], ["verbier", 2.7312179172708317], ["footwear", 2.7307096442422605], ["lover", 2.730657808349553], ["bastante", 2.73048169032366], ["liberland", 2.73048169032366], ["vishnu", 2.7304607649543478], ["hamish", 2.7304539528014096], ["voyagers", 2.730354856354395], ["unruly", 2.7303283480548193], ["rescheduling", 2.7300974942225875], ["tenderizing", 2.7300974942225875], ["doubloons", 2.7300564005761574], ["narrow", 2.7296489782056796], ["jonah's", 2.729294524371346], ["oversampling", 2.7292945243713453], ["brunches", 2.7292945243713453], ["wallonia", 2.7292945243713453], ["stimme", 2.7292945243713453], ["ahead", 2.729124247529361], ["detractor", 2.728959682692487], ["oscilloscopes", 2.728959682692487], ["borobudur", 2.7286980876308795], ["ganges", 2.728698087630879], ["middlemen", 2.7284115432893747], ["trees", 2.728383071654487], ["classical", 2.72820389718878], ["heartened", 2.7279954789052], ["speculoos", 2.7279954789052], ["jeremiah's", 2.7279954789052], ["rakishly", 2.7279954789052], ["cupcakes", 2.727846366682432], ["intersection", 2.7278234709086946], ["netizens", 2.72770402639677], ["wailua", 2.727534824484581], ["hongik", 2.727534824484581], ["leidseplein", 2.727534824484581], ["bolshaya", 2.727534824484581], ["lidi", 2.727534824484581], ["fedexfield", 2.727534824484581], ["ganymede's", 2.727534824484581], ["pawns", 2.727516813226784], ["questions", 2.72698051902025], ["analyst", 2.726975667511217], ["heal", 2.7267569571482726], ["bhagavad", 2.7267099651626605], ["frail", 2.726696433439055], ["sourest", 2.72656795641493], ["reinheitsgebot", 2.72656795641493], ["misterio", 2.72656795641493], ["rajiv's", 2.72656795641493], ["purls", 2.72656795641493], ["anticipate", 2.72652225974527], ["encroach", 2.7260826449663775], ["alvin's", 2.7260602156409237], ["veces", 2.725799909103264], ["phoniness", 2.725799909103264], ["stormborn", 2.725799909103264], ["starlight", 2.725698615907088], ["villandry", 2.7255351655223485], ["einigen", 2.7255351655223485], ["kichijoji", 2.7255351655223485], ["jotaro's", 2.7255351655223485], ["meridius", 2.7255351655223485], ["joydeep", 2.7255351655223485], ["zenefits", 2.7255351655223485], ["kantele", 2.7255351655223485], ["quilly", 2.7255351655223485], ["besnik", 2.7255351655223485], ["cancha", 2.7255351655223485], ["unaffordable", 2.7254940346815504], ["evaporation", 2.7251592296374496], ["antiseptic", 2.725069610246931], ["tonya", 2.725069610246931], ["sagas", 2.7250271440562686], ["cnidarian", 2.724656810651088], ["ffmpeg", 2.724656810651088], ["quora", 2.7245834291048814], ["potos\u00ed", 2.7244966686027365], ["owl's", 2.7244294717432322], ["physiques", 2.7242651118571475], ["marge", 2.7241254059539752], ["sparingly", 2.72338696046578], ["venerable", 2.72329167717698], ["ryogoku", 2.7232428735412535], ["chasselas", 2.7232428735412535], ["warisan", 2.7232428735412535], ["martorana", 2.7232428735412535], ["merzouga", 2.7232428735412535], ["frown", 2.7231597464694115], ["plexiglass", 2.7229325324730436], ["cookware", 2.7227651116336147], ["flour", 2.7227194057444506], ["increased", 2.7224277987984555], ["sailboats", 2.7217472183931317], ["scenario", 2.7216909179251494], ["piet\u00e0", 2.721411814288578], ["aquapark", 2.721290728112192], ["nakula", 2.721290728112192], ["tanaya", 2.721290728112192], ["balconette", 2.721290728112192], ["massages", 2.7210239349035534], ["haiku", 2.7210239349035534], ["ayushman", 2.720993189217799], ["macchiatos", 2.720993189217799], ["commandline", 2.720993189217799], ["insidious", 2.7207387421632845], ["behaviorism", 2.7207387421632845], ["sluggishly", 2.720463699796091], ["toward", 2.7199133128893065], ["nowheresville", 2.7190979072192727], ["cloudfront", 2.7190979072192727], ["reinventions", 2.7190979072192727], ["effluents", 2.7185067989785736], ["cozying", 2.7185067989785736], ["adir", 2.718420841007316], ["r\u00e9gional", 2.718420841007316], ["doncaster's", 2.718420841007316], ["jolokia", 2.7181431677855583], ["seekh", 2.7181431677855583], ["hinglish", 2.7181431677855583], ["tambopata", 2.7181431677855583], ["sawadee", 2.7181431677855583], ["misconstrue", 2.7180164478243563], ["natasha's", 2.7180164478243563], ["expose", 2.717733019722652], ["saab's", 2.717479396560214], ["peafowl", 2.717479396560214], ["dorothea's", 2.717479396560214], ["gentoos", 2.717479396560214], ["samuli", 2.717479396560214], ["infinitely", 2.717424696894422], ["tingler", 2.7171389029633515], ["articulately", 2.717078203765269], ["hangouts", 2.7168193335540054], ["triggers", 2.716797196305763], ["pressured", 2.7167316036607754], ["flippers", 2.7164181575352613], ["psyllium", 2.7161714488320445], ["kimiko", 2.716136768399858], ["edutainment", 2.7160811565825655], ["buonarroti", 2.7160811565825655], ["beautiful", 2.7160441432330362], ["sejm", 2.7159774735333753], ["kahu", 2.7159774735333753], ["sneaking", 2.715615032285834], ["sumptuous", 2.715116370997988], ["rollouts", 2.7150925020529146], ["groundhogs", 2.7150050242437467], ["rauf", 2.7150050242437467], ["opposers", 2.7146354927107073], ["moved", 2.7144561138556007], ["symphony's", 2.71441550604193], ["tandoor", 2.71441550604193], ["nellie's", 2.71441550604193], ["entrusted", 2.714337075333849], ["addi", 2.714125757661858], ["overburdened", 2.714038097197749], ["software", 2.7138766852616074], ["moviegoers", 2.713787169119236], ["scootering", 2.7137871691192355], ["banteay", 2.7137871691192355], ["ko\u0161ice", 2.7137871691192355], ["istiklal", 2.7137871691192355], ["globex", 2.7137871691192355], ["renfield", 2.713787169119235], ["rousseau", 2.7134831033299784], ["manually", 2.7129514433870856], ["swishing", 2.7129351166328557], ["bees", 2.7128349630949824], ["allocations", 2.7127533454357615], ["embroideries", 2.7119839882759664], ["congregant", 2.7118390089116837], ["strudels", 2.7118390089116837], ["backrest", 2.7115093554805587], ["noodles", 2.711465708752153], ["chordata", 2.711219726196701], ["charmant", 2.711219726196701], ["blacula", 2.711219726196701], ["redirections", 2.711219726196701], ["dragneel", 2.711219726196701], ["classifying", 2.710915436665141], ["courteously", 2.710915436665141], ["proofread", 2.7107973191348953], ["gimmickry", 2.7107973191348953], ["conj", 2.709706286158154], ["woodwind", 2.7094795704380936], ["mississauga's", 2.7093310325525373], ["cyberlink", 2.7093310325525373], ["revolve", 2.709331032552537], ["different", 2.7093015968493965], ["instantaneously", 2.7092951140713946], ["shelley's", 2.7090400195522633], ["ampera", 2.7088797782166694], ["mountaineering", 2.708857085079547], ["spelunker", 2.7087083409798978], ["zur\u00fcck", 2.7087083409798978], ["sambar", 2.7086180507018653], ["querido", 2.7084971356511662], ["porch", 2.708442826348734], ["serif", 2.7082868486064733], ["aunt's", 2.7082739641506333], ["volcanologists", 2.7080691177725873], ["reintegrate", 2.7079510676802694], ["disbelieving", 2.7079510676802694], ["pertained", 2.7077683326058595], ["tempered", 2.707703162486401], ["gloria's", 2.7076334567176046], ["vermeer's", 2.7073614694291344], ["pandering", 2.7072218901003895], ["kiedis", 2.7070643272469335], ["propolis", 2.7069856975174327], ["shinjuku", 2.7067383712773676], ["rashid", 2.706738371277367], ["xaxis", 2.706738371277367], ["stroopwafel", 2.706738371277367], ["prasert", 2.706738371277367], ["creativo", 2.706738371277367], ["timelimit", 2.706738371277367], ["hilleman", 2.706738371277367], ["comedor", 2.706738371277367], ["kalam's", 2.706738371277367], ["wellingham", 2.706738371277367], ["latia", 2.706738371277367], ["romantique", 2.7067383712773667], ["guitarra", 2.7067383712773667], ["upfront", 2.7067010266530396], ["thickened", 2.7065930352393903], ["fanatics", 2.706524230583437], ["disassembled", 2.7065029208275346], ["gavel", 2.7064915862325756], ["trajectories", 2.706331709863413], ["burst", 2.706330440849072], ["professions", 2.70623934134166], ["ii's", 2.706078834637192], ["audience", 2.7060306037175286], ["unwieldy", 2.705865455135959], ["hypothetical", 2.704909493999477], ["preserved", 2.7049068887611893], ["bourges", 2.7046605270824258], ["polynomials", 2.7044406307245], ["garam", 2.7042039720533246], ["flooding", 2.7038976981921126], ["submersibles", 2.703846556778139], ["dunsmore", 2.703846556778139], ["altruistically", 2.703846556778139], ["hemis", 2.703846556778139], ["daylilies", 2.703846556778139], ["incentives", 2.7036328139673267], ["hydrangeas", 2.7032574834542222], ["hobby's", 2.7032276859709112], ["croydon's", 2.7032276859709112], ["custer's", 2.7030452983526607], ["snowfields", 2.702958010423628], ["aichi", 2.702367023778534], ["adelina", 2.70220121404582], ["homogenized", 2.7021035452991526], ["noses", 2.7020191761264996], ["nestorian", 2.702014569582293], ["skype's", 2.701984146971845], ["monogamy", 2.7019221108302545], ["minestrone", 2.701796945986161], ["helena's", 2.7017258928120387], ["shooed", 2.701468242049802], ["nevadas", 2.701281237464308], ["rajat", 2.7010279527725625], ["emdr", 2.7010279527725625], ["green's", 2.70093821762463], ["hepburn's", 2.700912058670067], ["volunteers", 2.7006019409100053], ["blinking", 2.7001406964973786], ["clafoutis", 2.7001042086026676], ["shantha", 2.7001042086026676], ["grandniece", 2.699903173370101], ["varios", 2.699903173370101], ["assignments", 2.6998734551183303], ["slipping", 2.699419129763199], ["cubes", 2.6993174807613185], ["scene's", 2.6993022768507813], ["hildesheim", 2.6992438295205257], ["unshared", 2.6990845867943807], ["whatsapp's", 2.6990845867943807], ["gifted", 2.6989192924342222], ["hine", 2.698896000168534], ["judicious", 2.6987330559830536], ["corroboree", 2.69856091396233], ["djokovic's", 2.698455038220257], ["honorably", 2.698330080820941], ["curators", 2.698279813867125], ["coupled", 2.698180088109877], ["stammering", 2.6981618872112154], ["literature", 2.6980269361316], ["incorrect", 2.6979593403946365], ["filtration", 2.697671682288611], ["crevasses", 2.697279339334731], ["bessie", 2.697172145634832], ["cake's", 2.696931348193029], ["gammy's", 2.696931348193029], ["hechos", 2.696931348193029], ["karamchand", 2.696931348193029], ["noua", 2.696931348193029], ["parfois", 2.696931348193029], ["wriggler", 2.696931348193029], ["vladim\u00edr", 2.696931348193029], ["dartboards", 2.696931348193029], ["stitchin", 2.696931348193029], ["wubbzy", 2.696931348193029], ["merveilles", 2.696931348193029], ["conservationist", 2.696608362402826], ["pinball", 2.695818328906473], ["citro\u00ebn", 2.6956710998994806], ["symbology", 2.695599530243304], ["squirmed", 2.695599530243304], ["averts", 2.69547551480669], ["sharpeners", 2.69547551480669], ["newfield", 2.695460294730378], ["dwarven", 2.695071395539103], ["tuples", 2.695071395539103], ["complacent", 2.6948782004569853], ["corroborating", 2.694851452551749], ["joyce's", 2.694851452551749], ["saskia", 2.6947880914924784], ["widget", 2.694743048634188], ["glaciers", 2.694631024396041], ["dragonflies", 2.694348371690534], ["syncretic", 2.693849140937951], ["pages", 2.693772640110397], ["iterative", 2.693653310090253], ["claimed", 2.6936382530731304], ["conversationalists", 2.6935531460867326], ["headcount", 2.69320467942098], ["nonchalant", 2.69320467942098], ["tabular", 2.69320467942098], ["paralegal", 2.693092300010548], ["alienated", 2.693031170551027], ["mosh", 2.6929184087690223], ["amelie's", 2.6927500592811016], ["narek", 2.6927500592811016], ["underconsumption", 2.6927500592811016], ["bonjour", 2.6926162580359203], ["nighttime", 2.6926104581835584], ["hanh", 2.6925363489589365], ["ventures", 2.692184749020209], ["tricep", 2.6919771689081737], ["swabian", 2.6919771689081737], ["thuds", 2.6919625903355597], ["grandiosity", 2.691633804473364], ["communality", 2.6914460640950093], ["hillwood", 2.6914460640950093], ["aruban", 2.6914460640950093], ["harpists", 2.6914460640950093], ["straightaways", 2.6914460640950093], ["chet's", 2.6914460640950093], ["amarone", 2.6914460640950093], ["sense", 2.6910363782525915], ["cloudberry", 2.6908537564224533], ["arrondissements", 2.6908537564224533], ["revamp", 2.6908205796952247], ["encompassing", 2.69052613624107], ["vintners", 2.690432718438347], ["msg's", 2.690432718438347], ["alworth", 2.690432718438347], ["verdadero", 2.690432718438347], ["salaria", 2.690432718438347], ["citing", 2.6902679770548987], ["sulking", 2.6901180479449622], ["aviary", 2.689949001660491], ["lycaon", 2.689949001660491], ["fruitier", 2.689949001660491], ["encircling", 2.6891507275393725], ["classicists", 2.6891367489207947], ["ndas", 2.6891367489207947], ["spinach", 2.6890370193460003], ["melanistic", 2.688727754738449], ["seedling", 2.688677394600694], ["adoringly", 2.6885817155360847], ["castmates", 2.6885817155360847], ["alliteration", 2.6885374408268947], ["restaurateur", 2.6883732361904125], ["cleansers", 2.687941577032386], ["ootheca", 2.6879415770323853], ["fpic", 2.6879415770323853], ["jimi's", 2.6879415770323853], ["sahadeva", 2.6879415770323853], ["roommates", 2.6877727894922265], ["refinement", 2.6876798495175818], ["downturn", 2.687674783823747], ["rapunzel's", 2.6874891377836008], ["canarian", 2.6874891377836008], ["lofty", 2.687375098301714], ["nelly's", 2.6872453994677565], ["trips", 2.687187493875028], ["lifecycle", 2.6871044728352405], ["labels", 2.687011135717259], ["lydia's", 2.6865722741403673], ["traipsing", 2.6865722741403673], ["evolution", 2.6864290567795206], ["encrypted", 2.686369221620258], ["platform", 2.6863510790578102], ["keyword", 2.686032979462895], ["spots", 2.6859898725084443], ["woodshop", 2.685652668508089], ["kaiden", 2.685652668508089], ["wesley's", 2.6855090507183292], ["sauvignon", 2.6854302644409507], ["sumac", 2.685109183379032], ["biophilia", 2.684314125511424], ["peel's", 2.684314125511424], ["mariadb", 2.684314125511424], ["waiter", 2.684182218183389], ["gera", 2.684182218183389], ["geneva's", 2.684182218183389], ["glenorchy", 2.684182218183389], ["zakat", 2.684076815598838], ["spring's", 2.684010688502066], ["specimen's", 2.6837218885284098], ["quacked", 2.6837218885284098], ["baozi", 2.6837218885284098], ["represented", 2.683623304655397], ["cargill's", 2.6835423273154744], ["sogno", 2.6835423273154744], ["ix's", 2.6835423273154744], ["eines", 2.6829592942204625], ["popularity", 2.6829507040776837], ["nutrition's", 2.6823533409054994], ["h\u00f6ren", 2.6823533409054994], ["sanzio", 2.6823533409054994], ["coziest", 2.682353340905499], ["hashimoto's", 2.682353340905499], ["vaxxer", 2.682353340905499], ["bronner's", 2.682353340905499], ["value", 2.682345731392475], ["archaeology", 2.6822722213488186], ["appropriateness", 2.6818285326431477], ["sympathetic", 2.681668219696135], ["power", 2.681589964214522], ["deadlift", 2.681387075292781], ["hive's", 2.681387075292781], ["vaisakhi", 2.681387075292781], ["limiting", 2.681119036899022], ["moldova's", 2.680932602907138], ["pediments", 2.680932602907138], ["sacagawea", 2.680932602907138], ["safety", 2.6808809934192057], ["charcoals", 2.68066844559086], ["parathas", 2.68066844559086], ["aveyron", 2.68066844559086], ["siestas", 2.68066844559086], ["doubting", 2.680643998090216], ["jakarta's", 2.680586309719132], ["metaphase", 2.680586309719132], ["meddling", 2.680291281774678], ["geworden", 2.680283623821467], ["noctuid", 2.680283623821467], ["rynek", 2.680283623821467], ["ismailis", 2.680283623821467], ["rajaram", 2.680283623821467], ["tannat", 2.680283623821467], ["kaurava", 2.680283623821467], ["volunteer's", 2.6798558052063317], ["coax", 2.679779811110222], ["superhumans", 2.6796709875645934], ["intending", 2.679510393594345], ["reenact", 2.679293622265683], ["tossing", 2.6791646249441166], ["homeland", 2.6787520331792836], ["himalayas", 2.678752033179283], ["nanomedicine", 2.6787334308637916], ["chocolates", 2.678385915894299], ["combined", 2.678194713244142], ["echo's", 2.6780956371897764], ["pruned", 2.6779368317084433], ["queequeg", 2.677528964393079], ["z\u00f3calo", 2.677528964393079], ["killmonger", 2.677338257201883], ["drew", 2.6770864283559086], ["europ\u00e4ischen", 2.6770592224695013], ["plaka", 2.6770592224695013], ["caimans", 2.6770592224695013], ["educational", 2.6766786134857945], ["maxine", 2.6763255805888573], ["carpathian", 2.6762038408881392], ["malts", 2.6762038408881392], ["jen's", 2.6760336602593866], ["estrangement", 2.67597998069467], ["lacemaker", 2.675778945462103], ["comunidades", 2.675778945462103], ["powertech", 2.675778945462103], ["frivolousness", 2.675778945462103], ["qrcode", 2.675778945462103], ["consulat", 2.675778945462103], ["yessiree", 2.675778945462103], ["manoela", 2.675778945462103], ["microphones", 2.6756236778250355], ["amaterasu", 2.675355897407485], ["bert's", 2.675355897407485], ["chromatids", 2.675355897407485], ["wearables", 2.6753558974074845], ["attempts", 2.6750384105525704], ["independiente", 2.674568733365558], ["ruthless", 2.674342285625613], ["brilliantly", 2.6741572612527325], ["exercisers", 2.6738514285729935], ["stack", 2.673578812286979], ["penelope", 2.6734977246125577], ["leopards", 2.6734451563785067], ["macau's", 2.6730910710819304], ["spiderweb", 2.6729041416364003], ["kabbalistic", 2.6726502942543133], ["deveraux", 2.6726502942543133], ["plath", 2.672324905076585], ["prosperous", 2.6722523934942375], ["seattleites", 2.672036597286632], ["ruisdael", 2.672036597286632], ["mudejar", 2.672036597286632], ["familia", 2.6717962197589875], ["needing", 2.6716367006530244], ["serenissima", 2.6714735891540387], ["eram", 2.6714735891540387], ["autonomy", 2.671206199172834], ["benzer", 2.671123392707928], ["eklavya", 2.671123392707928], ["mooncake", 2.6710177729612807], ["ch\u00e8vre", 2.6707111823078193], ["confectioneries", 2.6707111823078193], ["khichdi", 2.6707111823078193], ["napoletana", 2.6707111823078193], ["devi's", 2.6702188853156885], ["nathaniel's", 2.6702188853156885], ["khalili", 2.6702188853156885], ["saba's", 2.670140639436145], ["binkie", 2.670140639436145], ["unauthorized", 2.66993303544929], ["hamfest", 2.6696206509961375], ["wirklich", 2.6696206509961375], ["vaporetto", 2.6696206509961375], ["colanders", 2.6696206509961375], ["edwina's", 2.6696206509961375], ["cocoas", 2.6696206509961375], ["oldrich", 2.6696206509961375], ["batty's", 2.6696206509961375], ["mathletics", 2.6696206509961375], ["keen", 2.669103012133526], ["linger", 2.669089498098448], ["rachelle", 2.6689950075344555], ["qufu", 2.6689950075344555], ["oakdale", 2.6689950075344555], ["monets", 2.6689950075344555], ["calaway", 2.6689950075344555], ["biology", 2.66890925669084], ["bartender", 2.6688354684517273], ["outages", 2.668778016070428], ["lifecycles", 2.668542597159418], ["microtonal", 2.668542597159418], ["unvisited", 2.668542597159418], ["grimmer", 2.6682496973471665], ["scaling", 2.6679914431873604], ["crimefighting", 2.6679320863845017], ["fifthly", 2.6679320863845017], ["badshahi", 2.6679320863845017], ["hanim", 2.6679320863845017], ["nasheeds", 2.6679320863845017], ["finnish", 2.6677952693544307], ["societies", 2.6677820152046428], ["tulips", 2.667752427658146], ["might", 2.6677293060321587], ["chigurh", 2.66765359660513], ["drills", 2.667424562028737], ["biting", 2.6673704164715786], ["emitting", 2.667265103362906], ["seagull's", 2.6672651033629053], ["lumosity", 2.6672651033629053], ["contagiously", 2.6672651033629053], ["abihu", 2.6672651033629053], ["figures", 2.667080285721167], ["merovingians", 2.6666853891478732], ["conures", 2.6666853891478732], ["moritz's", 2.6666853891478732], ["ilahi", 2.6666853891478732], ["dhritarashtra", 2.6666853891478732], ["oasis's", 2.6666853891478732], ["kamut", 2.6666853891478732], ["feito", 2.6666853891478732], ["opoly", 2.6666853891478732], ["festspiele", 2.6666853891478732], ["melony", 2.6666853891478732], ["andalusi", 2.6666853891478732], ["hetty's", 2.6666853891478732], ["mixxx", 2.6666853891478732], ["a\u00f1os", 2.666566716841828], ["terracotta", 2.66653102737464], ["minesweeping", 2.665727183833771], ["bloco", 2.665727183833771], ["hustle", 2.665655557508395], ["rolling", 2.6653306265043373], ["pompey's", 2.6651067422688146], ["gettext", 2.66496771739963], ["miyagi's", 2.66496771739963], ["nonprofit", 2.664523471495129], ["outspoken", 2.664495932782171], ["uncooperative", 2.6643509694900254], ["newsroom", 2.664185628561068], ["brother's", 2.6641463056585706], ["performer's", 2.6638991657888043], ["cartographers", 2.6638991657888043], ["mot\u00f6rhead", 2.6636163138918207], ["bluegrass", 2.6634150013692786], ["entrepreneurial", 2.6630639075177123], ["pindus", 2.6630432055979334], ["durres", 2.6630432055979334], ["standoffs", 2.663043205597933], ["pescara", 2.663043205597933], ["purist", 2.662959719385456], ["defusing", 2.662450034152205], ["spre", 2.6624500341522044], ["samat", 2.6624500341522044], ["hakurei", 2.6624500341522044], ["fumed", 2.6623863848566236], ["dafydd", 2.6623863848566236], ["leur", 2.6619248220840337], ["invest", 2.661685423387027], ["concurred", 2.6616260650894112], ["fagioli", 2.661626065089411], ["soundboard", 2.6610621612620613], ["fireplace", 2.661008533520556], ["brims", 2.6608482667068603], ["jumbles", 2.6603427828564934], ["partings", 2.6603427828564934], ["mitsubishi's", 2.6603427828564934], ["sandesh", 2.6601311153716813], ["nakamura's", 2.6601311153716813], ["self", 2.660051569571475], ["helplessness", 2.6597117052327266], ["hollowness", 2.6593982968825984], ["althea", 2.6588919859265046], ["tight", 2.6587944189886348], ["pained", 2.6586167706261516], ["customize", 2.6585943240821557], ["mol\u00e9", 2.658403757504557], ["phaedra's", 2.658403757504557], ["ujima", 2.658403757504557], ["watersport", 2.658403757504557], ["diskussion", 2.658403757504557], ["ali'i", 2.658403757504557], ["pileups", 2.658403757504557], ["argc", 2.658403757504557], ["zhiyong", 2.658403757504557], ["klauss", 2.658403757504557], ["yahs", 2.658403757504557], ["nomvula", 2.658403757504557], ["ibizan", 2.6584037575045567], ["keybindings", 2.6584037575045567], ["kadare", 2.6584037575045567], ["hppd", 2.6584037575045567], ["devacurl", 2.6584037575045567], ["hykeham", 2.6584037575045567], ["reels", 2.6578667062404144], ["o'keeffe", 2.6573097641887116], ["shiitake", 2.65720627833451], ["bagpipes", 2.6571148344706152], ["jdbc", 2.656171680462152], ["monocultures", 2.6559558719083283], ["ifugao", 2.655907603741642], ["paramountcy", 2.655907603741642], ["unhealthy", 2.655331824273663], ["adoration", 2.6552438977411152], ["ligurian", 2.6550911048472305], ["unwearable", 2.654955763007145], ["miscalculation", 2.6549201817371477], ["hostility", 2.6549201817371473], ["jumeirah", 2.654592425952579], ["pikachus", 2.6545341741603], ["orbs", 2.6541907563040747], ["squirrel's", 2.6541907563040743], ["transportation", 2.654058839535302], ["ceph", 2.653995127724284], ["feria", 2.6539170001079246], ["powdery", 2.6538903385614256], ["ugric", 2.6538903385614256], ["caver", 2.6538903385614256], ["carlota", 2.6538687800949625], ["fumbling", 2.6538457278648617], ["pilfer", 2.6537804466219406], ["disruption", 2.653601702633951], ["hatchlings", 2.6535976038186733], ["flock", 2.6533401312997866], ["zamindar", 2.6532815922685176], ["aztecs", 2.652896706406148], ["condescension", 2.6528762123297587], ["ingl\u00e9s", 2.6525739247030122], ["brts", 2.6522924844987994], ["samovars", 2.6522924844987994], ["progressiva", 2.6522924844987994], ["apr\u00e9s", 2.6522924844987994], ["erinsborough", 2.6522924844987994], ["plexippus", 2.6522924844987994], ["bunau", 2.6522924844987994], ["vaishnavite", 2.6522924844987994], ["sandwich's", 2.6522924844987994], ["teatre", 2.6522924844987994], ["kidzone", 2.6522924844987994], ["archidamus", 2.6522924844987994], ["rashes", 2.6521331589455763], ["zahra", 2.652113866916168], ["sarcastically", 2.6520960256650894], ["romania's", 2.651872052940664], ["decipherable", 2.651872052940664], ["clarification", 2.6517130583262074], ["cans", 2.651634868909115], ["clapping", 2.6512837241511242], ["silicon", 2.651137453900711], ["murali's", 2.6508299576256262], ["curonian", 2.6508299576256262], ["thibodeau's", 2.6508299576256262], ["goldfinches", 2.6508299576256262], ["microagressions", 2.6508299576256262], ["pptx", 2.6508299576256262], ["nazar\u00e9", 2.6508299576256262], ["tsuite", 2.6508299576256262], ["kizomba", 2.6508299576256262], ["jaydon", 2.6508299576256262], ["topazes", 2.6508299576256262], ["failover", 2.6503479885424217], ["savoured", 2.6503479885424217], ["fiume", 2.6503479885424217], ["usvi", 2.6501734053141406], ["kurosawa's", 2.6501734053141406], ["dayak", 2.6501734053141406], ["garnered", 2.6499875020774497], ["archery", 2.649841108697614], ["robuchon", 2.6498005090983936], ["warfare's", 2.6498005090983936], ["rainfall", 2.649716689250079], ["cnidarians", 2.649497882270036], ["nighter", 2.6494842408856503], ["tiefling", 2.649392219343525], ["overstretching", 2.649392219343525], ["chocoholics", 2.649392219343525], ["starmaker", 2.649392219343525], ["tabrizi", 2.649392219343525], ["biomes", 2.649293962696909], ["overspend", 2.6490211560074823], ["pilaf", 2.6488239241441804], ["warming", 2.6485455102371445], ["uncatalogued", 2.6484471891243904], ["flagstick", 2.6484471891243904], ["j'\u00e9tais", 2.6484471891243904], ["\u043c\u0438\u0440\u0430", 2.6484471891243904], ["lights", 2.648443361427622], ["greg's", 2.6479225957366177], ["intangibles", 2.6479225957366177], ["cabernet", 2.6478962327713376], ["wilbur's", 2.647778642566489], ["kazoos", 2.647778642566489], ["misto", 2.647778642566489], ["margarethe", 2.647778642566489], ["cogito", 2.6473270751816216], ["inconspicuous", 2.647307811386277], ["carved", 2.6470789808476427], ["unhatched", 2.64701385280231], ["conquests", 2.646964565578325], ["albania's", 2.6465886296934262], ["sparing", 2.646588629693426], ["tumblr's", 2.6465886296934253], ["trialing", 2.6463383849288933], ["is\u00e8re", 2.6460567589881014], ["pricey", 2.6460221509627546], ["cope", 2.6458367579236266], ["lethargy", 2.645744691992631], ["decarbonization", 2.6456352476222107], ["tuvan", 2.6456352476222107], ["mechanics", 2.6455777798695843], ["semicolon", 2.645493187379811], ["huggers", 2.645493187379811], ["kura", 2.645493187379811], ["quantities", 2.645332048420695], ["showstoppers", 2.645249262977994], ["homophone", 2.645249262977994], ["magritte", 2.6451626659920824], ["chen's", 2.644923410337494], ["untreated", 2.644656248976652], ["iconography", 2.644627225076558], ["cherryville", 2.644412158780849], ["fuzzball", 2.644412158780849], ["myra's", 2.644412158780849], ["oregano", 2.644360420169615], ["elbows", 2.644282090798025], ["priorities", 2.644179371022373], ["meddle", 2.644135488749589], ["wimbledon's", 2.643876961015461], ["delimiters", 2.643876961015461], ["vimy", 2.643736702290739], ["giggles", 2.6436993407963802], ["eloquent", 2.6435254484090582], ["reckless", 2.643523131327905], ["slammed", 2.643316094292501], ["per\u00f3n", 2.6425541958066976], ["happier", 2.642345457705602], ["muffins", 2.642310738589395], ["unlabeled", 2.6419938577668747], ["woche", 2.6419938577668747], ["oxalates", 2.6419938577668747], ["oppositions", 2.6413852787037557], ["steed", 2.6412976802022454], ["lemmy's", 2.641252765520657], ["solitaria", 2.641252765520657], ["shakti's", 2.641252765520657], ["herbalism", 2.6411708433480965], ["restaurant's", 2.6408383677853458], ["future", 2.64083593229959], ["sonam", 2.6404741861180154], ["creaking", 2.6404282857158314], ["armida", 2.640153846125807], ["naruto's", 2.639874498022156], ["ignacio's", 2.6395498301463682], ["niter", 2.6395498301463682], ["wampanoags", 2.6395498301463682], ["displayname", 2.6395498301463682], ["makaha", 2.6395498301463682], ["csula", 2.6395498301463682], ["outdoing", 2.6392910507512566], ["certain", 2.639270583978131], ["pals", 2.639069911995433], ["readership", 2.639069911995433], ["responsiveness", 2.6390699119954326], ["feasible", 2.6390269478943016], ["catalonia's", 2.638970981499407], ["estamos", 2.638820672734726], ["cr\u00eapes", 2.638820672734726], ["thermostat", 2.6387118778193384], ["retesting", 2.6386193190429093], ["terengganu", 2.6386193190429093], ["revolutionary", 2.638597452031978], ["impress", 2.6381819788242837], ["versus", 2.638179537531197], ["overpacked", 2.638032847485365], ["reiterating", 2.6375758078374982], ["resurrecting", 2.637446332852422], ["eric's", 2.636814296686035], ["unpreparedness", 2.636597313557838], ["carnivore", 2.636574137227206], ["elderflower", 2.636250392858686], ["republika", 2.636250392858686], ["shivaya", 2.6362503928586856], ["berringer", 2.6362503928586856], ["elion", 2.6362503928586856], ["jugar", 2.6362503928586856], ["woolens", 2.635948105231939], ["bougainvillea", 2.6350482722965065], ["protagoras", 2.6350041071323393], ["pancit", 2.6348780605354905], ["montresor", 2.6348780605354905], ["passageway", 2.63482020199222], ["gear", 2.634613505359852], ["ge's", 2.6345829353046955], ["banarasi", 2.6345829353046955], ["stonefish", 2.6345829353046955], ["petitioning", 2.6345586813766375], ["nilo", 2.6345300840351555], ["minimalists", 2.6345300840351555], ["anthems", 2.634335904555956], ["critics", 2.634199624245754], ["banded", 2.6337232682990828], ["pipsqueak", 2.633421522083011], ["gorgo", 2.633421522083011], ["nero's", 2.633248969261503], ["heavens", 2.633119383988736], ["subtopic", 2.633085626480704], ["arlo's", 2.633085626480704], ["felicity's", 2.632759991354996], ["fields", 2.632509925260187], ["olleh", 2.632204995836396], ["mindless", 2.6321400095388494], ["refined", 2.6320294559261592], ["skitter", 2.6319025362459443], ["outside", 2.631831667811568], ["pistachios", 2.6315511942974403], ["sensationalistic", 2.6315511942974403], ["atropos", 2.6315511942974403], ["manipulatively", 2.63155119429744], ["gharial", 2.63155119429744], ["mizrahim", 2.63155119429744], ["nsbe", 2.63155119429744], ["magdalena's", 2.63155119429744], ["fujisan", 2.63155119429744], ["candles", 2.6311360154835572], ["silverchair", 2.6309692502031687], ["robusta", 2.630903028978648], ["fascinate", 2.630506927950497], ["disenchanted", 2.6304817924366755], ["channels", 2.6302542154945363], ["prakash", 2.630117540499094], ["crave", 2.629626912310327], ["cinema's", 2.629623317964622], ["skipping", 2.6295805626427247], ["symbol", 2.6293176928636246], ["motown", 2.6292680234781547], ["temptations", 2.6291918725370356], ["playbacks", 2.6290615526755885], ["wattana", 2.6290615526755885], ["lesotho's", 2.6290615526755885], ["loadouts", 2.6290615526755885], ["capitalizations", 2.6290615526755885], ["necesario", 2.6290615526755885], ["venomous", 2.629013627074368], ["join", 2.6283230328538068], ["vendettas", 2.6282095419872213], ["tourist", 2.628163012651256], ["baffled", 2.6279888324186973], ["potions", 2.6279689780120092], ["grandest", 2.6278899056792873], ["m\u0101ori", 2.6277593337869254], ["setlist", 2.627666985892573], ["maecenas", 2.6275700275889724], ["continuer", 2.6271284191809743], ["hebron's", 2.6271284191809743], ["multitaskers", 2.6271284191809743], ["yawara", 2.6271284191809743], ["seismology", 2.627128419180974], ["resounded", 2.627128419180974], ["radley", 2.6269534172878766], ["cohabitating", 2.6268687188627475], ["flutter", 2.62681113314001], ["frustrating", 2.6266434782437664], ["reprimands", 2.6261053940956227], ["facts", 2.6259121560239453], ["rebeca", 2.625431307799074], ["imminent", 2.625381990032421], ["difficulties", 2.6253760992991815], ["senora", 2.62524517300235], ["ayaan", 2.6251737105357504], ["pens", 2.625102756698806], ["peoplesoft", 2.6249226018684797], ["lengthy", 2.6248608099051585], ["henry's", 2.6246665853209654], ["actions", 2.624623395486605], ["rigveda", 2.6243216580493707], ["jawline", 2.6239783851052176], ["dripping", 2.6237212713967164], ["miuccia", 2.6236008885744875], ["jiho", 2.6236008885744875], ["answered", 2.6232806048296484], ["vistula", 2.6231719004773884], ["caltech", 2.6229583740711635], ["rediscovery", 2.6228489747395782], ["brings", 2.6227014603583165], ["imbue", 2.6226888274290836], ["celle", 2.6223820263730593], ["tillie's", 2.622382026373059], ["llevar", 2.622382026373059], ["aella", 2.622382026373059], ["kicc", 2.622382026373059], ["introvert's", 2.622382026373059], ["d\u00e9sert", 2.622382026373059], ["noin", 2.622382026373059], ["dacic", 2.622382026373059], ["bluntly", 2.62220099408327], ["happiness", 2.6221076848687614], ["desc", 2.6214443752812944], ["smoldered", 2.6213907649758283], ["nonbelievers", 2.6208457353234773], ["store", 2.6204063064639582], ["cooed", 2.6203082519453016], ["gandhi's", 2.620190143786042], ["timely", 2.6199545280174856], ["pulled", 2.619841755420983], ["logistics", 2.6198006016520274], ["doorbell", 2.619635369374282], ["menagerie", 2.6196040793112227], ["bennets", 2.619569174061595], ["recommit", 2.6190199981341196], ["quilt", 2.618477918300088], ["profits", 2.6183458307248086], ["slightly", 2.618259653313533], ["ladainian", 2.6180283207398993], ["winemakers", 2.6179083597647925], ["exhalation", 2.6178947478663925], ["marketability", 2.6178947478663925], ["malayalam", 2.617851835779911], ["rebuilding", 2.617810847627132], ["imbalance", 2.6177277185297765], ["noisily", 2.617681715048716], ["goldsmith's", 2.617641566556154], ["braising", 2.617641566556154], ["fugues", 2.617627643004861], ["radios", 2.6173036630954076], ["attainable", 2.617050810165597], ["wilderness", 2.616652994414381], ["guesswork", 2.616513758901455], ["alcott's", 2.6165137589014544], ["summarization", 2.6165137589014544], ["greasiness", 2.6165137589014544], ["expedite", 2.6163123646774014], ["sally", 2.6160060641372818], ["applauded", 2.6159568168497516], ["ceto", 2.615698476259504], ["framework's", 2.615698476259504], ["saola", 2.615698476259504], ["zingerman's", 2.615698476259504], ["stackexchange", 2.615698476259504], ["usyd", 2.615698476259504], ["mfana", 2.615698476259504], ["vytas", 2.615698476259504], ["vesta's", 2.615698476259504], ["compass", 2.6153859512467563], ["khomeini's", 2.615206155823543], ["muffy", 2.615206155823543], ["immy", 2.615122119636017], ["decir", 2.615122119636017], ["monohull", 2.615122119636017], ["janessa", 2.615122119636017], ["espa\u00f1ola", 2.6149657876106915], ["pargeter", 2.6149657876106915], ["minified", 2.6149657876106915], ["prevenient", 2.6149657876106915], ["castiron", 2.6149657876106915], ["species", 2.6149080666473075], ["sanity", 2.614871320569369], ["ricardo's", 2.6148694117662363], ["redness", 2.6148429327463454], ["grandnephew", 2.6148233680372694], ["quirk", 2.614425226207568], ["foodborne", 2.6140970282128144], ["ensemble's", 2.6138815984107477], ["quadrilateral", 2.613718338218868], ["followed", 2.613698267649189], ["platter", 2.613599199793806], ["designers", 2.6128785931572915], ["comprehensive", 2.61271171737557], ["d'oeuvres", 2.6122249128906283], ["intervention", 2.6121884306433243], ["jazz", 2.611925129718121], ["autographs", 2.6118145603402096], ["dagwood", 2.611765095092197], ["puffball", 2.611765095092197], ["derrick's", 2.611765095092197], ["forgetful", 2.6117650950921965], ["singeing", 2.6117650950921965], ["silphium", 2.6117650950921965], ["avatars", 2.611765095092196], ["hygiene", 2.611669852442861], ["anglers", 2.6114851631420257], ["safavid", 2.6111432462600317], ["jalen", 2.6111432462600317], ["breakdance", 2.6110813869521197], ["tortillas", 2.610921105196603], ["outright", 2.6105577332693914], ["venture's", 2.6105491895544444], ["specter", 2.610404800771836], ["binge", 2.6102229724857975], ["carbs", 2.6101113573682295], ["resolutions", 2.6099405086984193], ["bapuji", 2.60988341995164], ["extolled", 2.6096840073670236], ["tavola", 2.609652016536297], ["moonshadow", 2.609652016536297], ["stitchers", 2.6092980625716438], ["maxwell's", 2.6092611905025063], ["fusion's", 2.60884365762565], ["airboat", 2.6088016702490506], ["groovers", 2.608713967644659], ["palauan", 2.608713967644659], ["mazdas", 2.608416678303617], ["mercury's", 2.607767495456851], ["quill's", 2.607767495456851], ["skateparks", 2.607767495456851], ["depth", 2.607472236126038], ["disaster's", 2.6073383745920404], ["geriatrician", 2.6073383745920404], ["negligence", 2.607178438269719], ["befitting", 2.607033636586595], ["bisogno", 2.6070336365865945], ["mosaic's", 2.6070336365865945], ["tecnol\u00f3gico", 2.6070336365865945], ["oberlin's", 2.6070336365865945], ["neads", 2.6070336365865945], ["njeru", 2.6070336365865945], ["rahima", 2.6070336365865945], ["inhabited", 2.6067348828622223], ["paygrade", 2.6066838400433787], ["semblances", 2.6066838400433787], ["corncobs", 2.6066838400433787], ["zhou's", 2.606517576244422], ["taranto", 2.6061973788042088], ["spectrums", 2.6061973788042088], ["shrugging", 2.606107290143812], ["truces", 2.6058751609215633], ["vasquez's", 2.6058021610851365], ["toulouse's", 2.6058021610851365], ["hagar's", 2.6058021610851365], ["c\u00e9zanne's", 2.6058021610851365], ["exhibit", 2.6056634714648688], ["incongruent", 2.6052356823544662], ["halting", 2.605235682354466], ["torta", 2.605235682354466], ["mwale", 2.605235682354466], ["unnervingly", 2.6047941918134803], ["sensitive", 2.604632618539106], ["galata", 2.6045502743875217], ["humidifiers", 2.6045502743875217], ["eliyahu", 2.6045502743875217], ["puccini's", 2.604414100619255], ["soothed", 2.6043558324110836], ["acme", 2.60427646450382], ["sauntering", 2.6037040917122822], ["karanja", 2.6037040917122822], ["chci", 2.6037040917122822], ["rabbie", 2.6037040917122822], ["patate", 2.6037040917122822], ["airbus's", 2.6037040917122822], ["jarama", 2.6037040917122822], ["weill's", 2.6037040917122822], ["cielos", 2.6037040917122822], ["commodo", 2.6037040917122822], ["ranged", 2.6036567516378875], ["uninspiring", 2.6034883747204254], ["acrid", 2.6031150183883653], ["room", 2.603073513965029], ["lecter's", 2.603020345889879], ["slickers", 2.603020345889879], ["trata", 2.603020345889879], ["trustless", 2.603020345889879], ["summy", 2.603020345889879], ["akiko", 2.6028031560404097], ["parishioner", 2.602703869252081], ["sufficiency", 2.601757397064273], ["context", 2.6012553401164533], ["larry's", 2.601233784224889], ["ashima", 2.601233784224889], ["aba's", 2.601233784224889], ["fransokyo", 2.601233784224889], ["tufail", 2.601233784224889], ["entender", 2.601233784224889], ["medea's", 2.601233784224889], ["viewmodel", 2.601233784224889], ["circumventing", 2.6011128809581345], ["eutrophication", 2.600957791250436], ["freedivers", 2.600360007340702], ["weightroom", 2.600360007340702], ["cheesy", 2.600338622801168], ["squeezed", 2.600112308347869], ["rigors", 2.599993759452344], ["civilize", 2.59984181965808], ["antti", 2.5996600040615725], ["transporting", 2.5994951703845333], ["pantoliano", 2.5993281184489003], ["epiphyllum", 2.5993281184489003], ["hanibal", 2.5993281184489003], ["albiceleste", 2.5993281184489003], ["acarina", 2.5993281184489003], ["tumbuka", 2.5993281184489003], ["murad's", 2.5993281184489003], ["schwarzenegger's", 2.5993281184489], ["loitered", 2.5993281184489], ["breadbasket", 2.5993281184489], ["palenque", 2.5993281184489], ["heftier", 2.5993281184489], ["bryansk", 2.5993281184489], ["dynamic", 2.5992759928850275], ["extinguishing", 2.59917625305287], ["adaptive", 2.599139424761499], ["buttercream", 2.598954221999181], ["solanum", 2.598954221999181], ["destructive", 2.598681027041718], ["caerlaverock", 2.5985297990022023], ["tailfins", 2.5985297990022023], ["condita", 2.5985297990022023], ["pencak", 2.5985297990022023], ["mus\u00e9es", 2.5985297990022023], ["amaravathi", 2.5985297990022023], ["citizens", 2.5982883872015203], ["eaps", 2.598282531916942], ["spokane's", 2.598282531916942], ["swim's", 2.598282531916942], ["wisdoms", 2.5981621518885913], ["saarland", 2.5981621518885913], ["moroccanoil", 2.598090932290667], ["balakirev", 2.598090932290667], ["advantages", 2.598048146269361], ["harasser", 2.5980438654259514], ["consequence", 2.5979193916714194], ["themistocles", 2.5978133584731142], ["desensitized", 2.5978133584731142], ["halil", 2.5978133584731142], ["tunga", 2.597813358473114], ["zakopane", 2.597813358473114], ["ash's", 2.59764024304731], ["greedily", 2.59764024304731], ["bonsoir", 2.59737520476111], ["freshwater", 2.597364879688742], ["hyperbola", 2.5971196833440158], ["lyla", 2.5970559784852214], ["underfeeding", 2.5965804143067768], ["unfathomed", 2.5965804143067768], ["soient", 2.5965804143067768], ["bridgette's", 2.5965804143067768], ["saffron's", 2.5965804143067768], ["3dprinting", 2.5965804143067768], ["tethered", 2.5965472904336515], ["microscopic", 2.5956926945070133], ["planned", 2.595614792565402], ["civico", 2.595557332977262], ["archipelago's", 2.595557332977262], ["snoozes", 2.595557332977262], ["anastasiya", 2.595557332977262], ["chanted", 2.5954582710593797], ["sultry", 2.5954582710593797], ["blacksmiths", 2.5954415632478702], ["reggaeton", 2.594897445519726], ["brighten", 2.5947812855552908], ["krewe", 2.594499821026082], ["pluma", 2.5943864300107844], ["fruitarian", 2.5943864300107844], ["ayen", 2.5943864300107844], ["optimized", 2.5943619989904363], ["tours", 2.594088366115268], ["anzali", 2.593957605807477], ["shishmaref", 2.593957605807477], ["kindlier", 2.593957605807477], ["strach", 2.593957605807477], ["sikri", 2.5939576058074767], ["accustomed", 2.5938801819410218], ["rectangle", 2.593827972743718], ["getaway", 2.5937487525380885], ["cenote", 2.5933204263415455], ["petri", 2.592894750426085], ["beseeching", 2.5928696884556777], ["guaran\u00ed", 2.5925609399812246], ["desensitizing", 2.5925609399812246], ["bestsellers", 2.5924186869804027], ["gregarious", 2.5924186869804027], ["deceptively", 2.592327825959646], ["although", 2.5923025480742057], ["lathering", 2.5922743108004638], ["khrushchev's", 2.5922743108004633], ["glares", 2.592090374591088], ["depletion", 2.591751025613501], ["despairing", 2.5915804936547158], ["reared", 2.591451366574813], ["javanese", 2.591253031095969], ["dotted", 2.5912368246949735], ["hendrik", 2.591229038900947], ["hedgehog's", 2.591102396555075], ["treffen", 2.591102396555075], ["nightshades", 2.591102396555075], ["personnelle", 2.591102396555075], ["disraeli's", 2.591102396555075], ["energy", 2.5907062026806065], ["terese", 2.590702473357582], ["talent", 2.5904451430659545], ["constante", 2.5902395585941838], ["murrelets", 2.5902395585941838], ["armonia", 2.5902395585941838], ["woodford's", 2.5902395585941838], ["adverse", 2.5900480663859047], ["paintings", 2.5900031409928994], ["mask's", 2.58995494816032], ["waiter's", 2.5898881967441305], ["fluffier", 2.5898881967441305], ["overuse", 2.5898760161999954], ["estoy", 2.589482178606291], ["hyperpigmentation", 2.589482178606291], ["nico's", 2.589482178606291], ["zipped", 2.5893420228789936], ["rhythmical", 2.589054094265308], ["greatschools", 2.5890540942653075], ["kiedy", 2.5890540942653075], ["reel's", 2.5890540942653075], ["tadashi's", 2.5890540942653075], ["perahu", 2.5890540942653075], ["kapono", 2.5890540942653075], ["anilao", 2.5890540942653075], ["mikasa's", 2.5890540942653075], ["expanded", 2.588867709481035], ["forefathers", 2.5885354536214247], ["ufology", 2.5882779989133096], ["bloops", 2.5882779989133096], ["bungy", 2.5882779989133096], ["morrison's", 2.588159589672209], ["graptolites", 2.5879824659150326], ["battled", 2.587894123792967], ["wince", 2.5878613482145103], ["slid", 2.5877626060140604], ["nevertheless", 2.587463207795868], ["suthep", 2.587446984337148], ["mairi", 2.5874150752139196], ["unadulterated", 2.5872100386426116], ["pup's", 2.5872100386426116], ["tourism's", 2.587156755628388], ["keisha", 2.587156755628388], ["seville's", 2.587156755628388], ["mahogany", 2.586785905849392], ["gastropods", 2.586758833543441], ["arlberg", 2.5861208758805665], ["sdgs", 2.58601529838002], ["hitchcockian", 2.58601529838002], ["acord", 2.58601529838002], ["blackmore's", 2.58601529838002], ["\u0440\u0443\u0441\u0441\u043a\u0438\u0439", 2.58601529838002], ["druidism", 2.5857697750590773], ["jaws", 2.5857553683187575], ["confused", 2.5856385907002744], ["promote", 2.585618664360612], ["giotto", 2.5851004776292132], ["rudely", 2.5850921074909006], ["buttresses", 2.5849351445698856], ["dialect", 2.5848571405822414], ["plagiarized", 2.584775038263999], ["crippling", 2.5846494989630187], ["waterlogged", 2.584559208684986], ["endlessness", 2.584559208684986], ["toivo", 2.584559208684986], ["buchanans", 2.584559208684986], ["leafa", 2.584559208684986], ["conventioneers", 2.584559208684986], ["dalang", 2.584559208684986], ["pogson", 2.584559208684986], ["lucuma", 2.584559208684986], ["refrigerator's", 2.584559208684986], ["elcano", 2.584559208684986], ["virupaksha", 2.584559208684986], ["isfahani", 2.584559208684986], ["pemberton", 2.5842987996967053], ["waning", 2.584189985940888], ["devaki", 2.5833763440585855], ["stalemates", 2.5833763440585855], ["stray", 2.5833562138533073], ["kayak", 2.5831332449836424], ["stackoverflow", 2.582679529260488], ["romesco", 2.582637603325369], ["prohm", 2.582637603325369], ["wary", 2.582558259620198], ["alexandru", 2.582516078875749], ["sekhmet", 2.582449364432998], ["crimefighter", 2.582449364432998], ["blockages", 2.5821060677411496], ["fiba", 2.5821060677411496], ["monaco's", 2.5820376875057813], ["daughter's", 2.581781771923414], ["substantial", 2.581525370959651], ["adjectives", 2.5814098120333435], ["script", 2.5813448747870233], ["prokaryotic", 2.5812242935770056], ["collection", 2.5810768552248633], ["here's", 2.5809732519969213], ["physical", 2.580666049463637], ["straddling", 2.580512212059287], ["arguments", 2.579786228314555], ["gdynia", 2.579588902514438], ["elsie's", 2.579118031403544], ["splendens", 2.578903718075172], ["teppanyaki", 2.578903718075172], ["absentminded", 2.578903718075172], ["ahlan", 2.578751210463222], ["nasira", 2.578751210463222], ["tiana's", 2.578751210463222], ["gr\u00e2ce", 2.5784778928998446], ["saima", 2.5780489839779714], ["ebert's", 2.5780489839779714], ["irises", 2.5777067750481266], ["revisions", 2.577520409137929], ["zigzagging", 2.577346485311912], ["dslr", 2.5772616297428166], ["bandanas", 2.5772223954474285], ["bhakti", 2.5763252147458124], ["robotic", 2.576276701941304], ["pieter", 2.5760973066494337], ["baggins", 2.5759126833322945], ["impassable", 2.5759126833322945], ["polygons", 2.5758958817285], ["bagpiper", 2.5756978628266376], ["dublin's", 2.5756587398868658], ["arboriculture", 2.575548067228398], ["squeals", 2.575398745666862], ["prokaryotes", 2.575290444626254], ["toothaches", 2.575273367216657], ["unanimously", 2.5751878573096105], ["wriggled", 2.5751608115624953], ["intolerance", 2.5750824915864747], ["bringing", 2.5746683575378317], ["glute", 2.5742209718502465], ["yeni", 2.5742209718502465], ["jeffrey's", 2.5742209718502465], ["cormac", 2.5741483378678005], ["metafiction", 2.57413159228107], ["cohabitation", 2.5739558888544836], ["dryly", 2.57393111474273], ["sharper", 2.5738724919569402], ["exponents", 2.5736680353582746], ["collin's", 2.5736680353582746], ["petitioned", 2.5736570680228965], ["illustrates", 2.573567266008237], ["unagi", 2.5733567848702528], ["wanted", 2.573164575597264], ["reinterpretation", 2.5730722788686085], ["sakra", 2.573072278868608], ["scheidegg", 2.573072278868608], ["spine's", 2.573072278868608], ["muckross", 2.573072278868608], ["tumo", 2.573072278868608], ["marjo", 2.573072278868608], ["exchangerate", 2.573072278868608], ["shahrzad", 2.573072278868608], ["fonder", 2.572396577114704], ["stating", 2.572241309477636], ["kiko", 2.572020843645141], ["sequences", 2.5717678309064094], ["problemas", 2.57170070515919], ["bleak", 2.5715691564539744], ["pescatarian", 2.5714014527134985], ["hoverflies", 2.5714014527134985], ["crossbills", 2.5714014527134985], ["candyland", 2.5714014527134985], ["kovalam", 2.5714014527134985], ["bashkortostan", 2.5714014527134985], ["trail", 2.571317467037085], ["belize's", 2.571305058896858], ["belaying", 2.571305058896858], ["kapalua", 2.5712649555692084], ["discipline", 2.5711559886945348], ["diminishing", 2.571119500799824], ["classico", 2.570857930952198], ["thought", 2.570416288083507], ["glamour", 2.5703919952447873], ["grooves", 2.570387693023882], ["quranic", 2.5702447269138076], ["sneeringly", 2.570143340588735], ["fashioneds", 2.570143340588735], ["erfahrung", 2.570143340588735], ["cambrai", 2.569790298921072], ["fortified", 2.569724415308742], ["unbridgeable", 2.5696398516229673], ["danishes", 2.5695035239742], ["riebeeck", 2.5695035239742], ["zinni", 2.5695035239742], ["dari", 2.569243302263149], ["clarified", 2.5691020390485786], ["criticality", 2.5690518534328763], ["vinci", 2.5689723285341475], ["spiced", 2.5686087861399587], ["confidentiality", 2.5682819847324168], ["tarkovsky's", 2.567884633145083], ["perfect", 2.5677510676843025], ["scour", 2.5676058299527758], ["knotweed", 2.5674786434797636], ["swiftness", 2.5672584858186864], ["smoothness", 2.5670367733718673], ["kojo", 2.5667346624181935], ["sadhya", 2.566734662418193], ["taleggio", 2.566734662418193], ["readathon", 2.566734662418193], ["teutoburg", 2.566734662418193], ["batfish", 2.566734662418193], ["mindbenders", 2.5667346624181926], ["semilinear", 2.5667346624181926], ["avare", 2.5667346624181926], ["kadhai", 2.5667346624181926], ["hogtown", 2.5667346624181926], ["disfluency", 2.5667346624181926], ["peels", 2.565717731093939], ["indiscretions", 2.56548867471799], ["surprised", 2.565350376152797], ["triaging", 2.5652845298405555], ["milpa", 2.5652845298405555], ["those", 2.564763438213871], ["gallina", 2.5643447791198524], ["confidant", 2.5642714818765073], ["portland's", 2.564095346569948], ["doubloon", 2.564055398678717], ["rachmaninoff's", 2.564055398678717], ["tugboats", 2.5638827350155062], ["arachnophobia", 2.563787779912546], ["togas", 2.563643700637824], ["chowed", 2.5631992152247793], ["tokens", 2.563150824443135], ["flashing", 2.5630194166994524], ["footstep", 2.5629738790300345], ["soft", 2.5628899189520444], ["catrina", 2.562424444681135], ["r\u00e9union", 2.5623368304420873], ["alvarez's", 2.5619334378345453], ["accommodate", 2.561874163287614], ["karimi", 2.561865518072142], ["mouthwashes", 2.5616870032983936], ["lavash", 2.5616870032983936], ["businessman's", 2.5615388432635005], ["jovi's", 2.5615388432635005], ["linguini", 2.5615388432635005], ["jittery", 2.561465443730557], ["seaweeds", 2.56145020870145], ["returning", 2.5612730829989703], ["ostrum", 2.561214802929122], ["brownie's", 2.561214802929122], ["database's", 2.561214802929122], ["ushanka", 2.561214802929122], ["adonia", 2.561214802929122], ["asano's", 2.561214802929122], ["vasque", 2.561214802929122], ["bedrich", 2.561214802929122], ["backpacks", 2.5607017390663556], ["stonecutter", 2.5606451377149466], ["conform", 2.5605891374398366], ["jerry's", 2.5605744992283896], ["exhibits", 2.5605669918854534], ["cannelloni", 2.560428189046158], ["dislodging", 2.560363335241978], ["swatting", 2.560363335241978], ["futuro", 2.560363335241978], ["biked", 2.560233945544421], ["destructively", 2.5599443590784627], ["gatekeeping", 2.5599443590784627], ["t\u00f3rshavn", 2.5599443590784623], ["salerno's", 2.5599443590784623], ["setzen", 2.5599443590784623], ["vexing", 2.5595297931903525], ["invention", 2.5593339108082205], ["rallies", 2.5592459056800165], ["sweeten", 2.5591085407556604], ["roars", 2.5589997301636735], ["emails", 2.5585353711354686], ["moviegoing", 2.55852767708672], ["chaga", 2.55852767708672], ["herb's", 2.55852767708672], ["fartlek", 2.5584119170796127], ["crop's", 2.5584119170796127], ["pestles", 2.5584119170796127], ["blouses", 2.55805090910873], ["miko", 2.5580274504365383], ["expressionists", 2.557914268389265], ["unfortunate", 2.557867760857112], ["unkempt", 2.557826468128336], ["gemstone", 2.5578205228401614], ["grapevine", 2.5577370828920842], ["herbivore", 2.5571312334091454], ["philosophes", 2.5571312334091454], ["complimentary", 2.5570521142496956], ["crispier", 2.5567161820808857], ["treetop", 2.5566180280505537], ["obediently", 2.5565396882917657], ["dread", 2.5564070305766555], ["woodpecker", 2.5562502696972715], ["condition's", 2.555986192307564], ["echoey", 2.555986192307564], ["seawall", 2.555315590546877], ["guitars", 2.5552791016765277], ["assembling", 2.5548005456466694], ["carnatic", 2.554693191162404], ["cliffs", 2.5542005350915042], ["hunkered", 2.5541526297592805], ["saman", 2.5541526297592805], ["koenji", 2.5541526297592805], ["\u00eeles", 2.5541526297592805], ["cynic's", 2.55415262975928], ["geothermally", 2.55415262975928], ["pixlr", 2.55415262975928], ["intruder's", 2.55415262975928], ["loudoun's", 2.55415262975928], ["amsterdams", 2.55415262975928], ["wiemer", 2.55415262975928], ["resending", 2.55415262975928], ["salander's", 2.55415262975928], ["nritya", 2.55415262975928], ["quebecker", 2.55415262975928], ["asianbeauty", 2.55415262975928], ["jayasurya", 2.55415262975928], ["snezana", 2.55415262975928], ["mbili", 2.55415262975928], ["darcus", 2.55415262975928], ["chewy", 2.5538174941017755], ["ecthr", 2.553561253832038], ["bushehr", 2.5535444981807665], ["debris", 2.553494795119061], ["plantas", 2.5533363457612746], ["fearfulness", 2.5533363457612746], ["intrusion", 2.5528762471306123], ["yielding", 2.552844298873234], ["foldable", 2.552577612721402], ["goodwill", 2.5524841661977162], ["barques", 2.552406797476465], ["firehole", 2.552406797476465], ["kannst", 2.552406797476465], ["lillie's", 2.552406797476465], ["ruthlessness", 2.552388864456037], ["riverfront", 2.5523758279298825], ["subterranean", 2.5523423776185874], ["powerfully", 2.5521960325066697], ["radiolab", 2.5520676072043744], ["flea", 2.5519328338833356], ["prey", 2.5518449136958927], ["rhizobia", 2.5518432693345434], ["learning's", 2.5518432693345434], ["botty", 2.5518432693345434], ["farsighted", 2.551843269334543], ["peterson's", 2.5518153268952406], ["tamarins", 2.551757363148065], ["tweety", 2.5515648358081564], ["mainyu", 2.551564835808156], ["per's", 2.551564835808156], ["protist", 2.551564835808156], ["radames", 2.551564835808156], ["calamansi", 2.551564835808156], ["\u043f\u0435\u0440\u0435\u0432\u043e\u0434", 2.551564835808156], ["peque\u00f1as", 2.551564835808156], ["electroacupuncture", 2.551564835808156], ["nibelheim", 2.551564835808156], ["granddaughter", 2.5513106078094405], ["featherstone", 2.551283802656708], ["unsupervised", 2.551258157342314], ["unseasonable", 2.5511512753177303], ["siqueiros", 2.5510691456992087], ["denomination's", 2.5510691456992087], ["camazotz", 2.550742546141444], ["geekiest", 2.550742546141444], ["semipalmated", 2.550742546141444], ["peart's", 2.550742546141444], ["cougar's", 2.550742546141444], ["amoc", 2.550742546141444], ["keepie", 2.550742546141444], ["nsync's", 2.550742546141444], ["huambo", 2.550742546141444], ["quote's", 2.550742546141444], ["rocamadour", 2.550742546141444], ["rosy's", 2.550742546141444], ["gavarnie", 2.550742546141444], ["moonfish", 2.550742546141444], ["nishani", 2.550742546141444], ["satyavati", 2.550742546141444], ["blissful", 2.550649791867039], ["clash's", 2.550511123301112], ["necessary", 2.550362915331994], ["ielts", 2.550098419235853], ["racers", 2.5499323431421224], ["error", 2.5499159226607415], ["counteroffer", 2.5498260019279546], ["firewood", 2.5496230980020718], ["pallor", 2.5496230980020718], ["kievan", 2.5495445144685878], ["crucian", 2.549154288018068], ["lux's", 2.5488452996195208], ["backhands", 2.5488452996195208], ["developments", 2.548288189814188], ["ratios", 2.548000104187428], ["scout's", 2.5478751424971993], ["oinks", 2.5476369342752005], ["demasiado", 2.5476369342752005], ["clovers", 2.5476369342752005], ["layer's", 2.5476369342752005], ["lobsterman", 2.5476369342752005], ["puppyhood", 2.5476369342752005], ["rediscoveries", 2.5476369342752005], ["thornier", 2.5476369342752005], ["obtenir", 2.5476369342752005], ["tenor's", 2.5476369342752005], ["cataloochee", 2.5476369342752005], ["unsatisfactoriness", 2.5476369342752005], ["kefi", 2.5476369342752005], ["siggi", 2.547215767646952], ["scaley", 2.547215767646952], ["calidad", 2.547215767646952], ["numark", 2.547215767646952], ["kabukicho", 2.547215767646952], ["rhombus", 2.5469656201950226], ["nocked", 2.5469033791544766], ["mannschaft", 2.5469033791544766], ["prism's", 2.5469033791544766], ["apixaban", 2.5469033791544766], ["meeple", 2.5469033791544766], ["friedreich's", 2.5469033791544766], ["varilla", 2.5469033791544766], ["gagauz", 2.5469033791544766], ["condescendingly", 2.546903379154476], ["megan's", 2.546903379154476], ["passages", 2.5467027005260428], ["edibility", 2.5464709677148916], ["spookiness", 2.5464709677148916], ["patrimoine", 2.546185840521104], ["landscape's", 2.546185840521104], ["avocets", 2.546185840521104], ["pamplemousse", 2.546185840521104], ["w\u00fcrzburg", 2.546129113644853], ["sentiment", 2.545757003892168], ["accepted", 2.545592008307889], ["recite", 2.5454795909117807], ["snowy", 2.545360060686597], ["entrance", 2.5448266217315525], ["eltz", 2.544796759320602], ["ir\u00e8ne", 2.5447967593206013], ["etwa", 2.5447967593206013], ["qizilbash", 2.5447967593206013], ["massacre's", 2.5447967593206013], ["quels", 2.5447967593206013], ["curanderos", 2.5447967593206013], ["ganks", 2.5447967593206013], ["shiganshina", 2.5447967593206013], ["karna's", 2.5447967593206013], ["regular", 2.544320145449433], ["aging", 2.544221288235255], ["circle's", 2.5441052384620906], ["artemisia", 2.543930340190483], ["gruesome", 2.5438732443676226], ["satchel", 2.5436749975249313], ["excused", 2.5435310227125627], ["liu's", 2.543489500711362], ["microphone", 2.5433244126241377], ["restocked", 2.5432062613460267], ["homesteading", 2.5432062613460262], ["handblown", 2.5432062613460262], ["busuttil", 2.5432062613460262], ["imogen's", 2.5432062613460262], ["tebogo", 2.5432062613460262], ["branzino", 2.5432062613460262], ["angered", 2.542888161751111], ["puede", 2.542766336374335], ["monet", 2.5426760953545013], ["searing", 2.5424404660249347], ["pinegrove", 2.5414832218735692], ["mehendi", 2.5414832218735692], ["takia", 2.5414832218735692], ["gardens", 2.5414054071877166], ["rarebit", 2.541242573902556], ["socotra", 2.541242573902556], ["roughriders", 2.5410673157940113], ["ciutadella", 2.5409642340806613], ["conservational", 2.5409642340806613], ["zindel", 2.5409642340806613], ["lactose", 2.5409088753827507], ["breed's", 2.5407531204021896], ["kodansha", 2.5407531204021896], ["extending", 2.540725084505689], ["oughta", 2.540585299894966], ["disseminating", 2.5399029193988314], ["nifty", 2.5398533196698945], ["merciless", 2.5398228383819297], ["risto", 2.5396826179232534], ["dangerously", 2.53963168454215], ["piacenza", 2.5395873493731695], ["morpho", 2.5395873493731695], ["glossier", 2.5395873493731695], ["wrestling", 2.539250762035343], ["animalia", 2.539216064672969], ["flaxseed", 2.539103147894106], ["f\u00e1il", 2.5388786273221817], ["nationalistic", 2.5386950307272307], ["microgravity", 2.5386875618023654], ["kovind", 2.5383797646104993], ["bassoonist", 2.5383797646104993], ["minecraft's", 2.5383797646104993], ["crossplay", 2.5383797646104993], ["sankranti", 2.5383797646104993], ["vaan", 2.5381558491971137], ["overbooking", 2.5381558491971137], ["waistline", 2.5381300013433394], ["loaf", 2.537786214850143], ["farmhand", 2.5375672230725317], ["outmaneuvered", 2.5375672230725317], ["syrahs", 2.5375672230725312], ["carr\u00e9e", 2.5375672230725312], ["cattle's", 2.5375672230725312], ["thesauruses", 2.5375672230725312], ["pollywog", 2.5375672230725312], ["innen", 2.5375672230725312], ["som's", 2.5375672230725312], ["boca's", 2.5375672230725312], ["cerdo", 2.5375672230725312], ["bellucci", 2.5372034141516613], ["projectionist", 2.5372034141516613], ["sullivan's", 2.5368767285900633], ["curate", 2.536829404980673], ["decreased", 2.536755201561148], ["hashing", 2.5366768486082956], ["luiza", 2.5365732680302], ["kaeding", 2.5365190746945587], ["athlete's", 2.5363631579607535], ["arm's", 2.5361106449207065], ["taking", 2.5357424763201943], ["powerhouse", 2.535311607763134], ["pesaro", 2.535301249024629], ["thwack", 2.535115467301447], ["marion's", 2.535115467301447], ["indigestion", 2.5349595753160026], ["gpio", 2.5349484746483086], ["presenting", 2.534832289509887], ["crusts", 2.5347274196254483], ["dehumanization", 2.534599308191745], ["sofort", 2.5345354820652766], ["jotunheimen", 2.5345354820652766], ["hydrogeologist", 2.5345354820652766], ["dearies", 2.5345354820652766], ["unidas", 2.5345354820652766], ["slowpokes", 2.5345354820652766], ["untethered", 2.534193439490099], ["unam", 2.534193439490099], ["working", 2.5341921986760765], ["billy", 2.5337569196410024], ["bluto", 2.5337282408893658], ["meteorology", 2.533490809862777], ["carbon", 2.5331417693739526], ["grizelda", 2.5331386589834], ["lightness", 2.5330342341067005], ["saprophytic", 2.532868024511286], ["impacto", 2.532868024511286], ["dupatta", 2.532562159516176], ["foods", 2.5325381663710336], ["autoimmune", 2.532309200898598], ["molting", 2.5322598929327724], ["poutine", 2.5322137057242773], ["rivera's", 2.5320727755239987], ["starfish", 2.5320409655645073], ["farc's", 2.5318131023852923], ["meal's", 2.5318131023852923], ["creek's", 2.5314709654795644], ["frontwoman", 2.5311753660874623], ["thermos", 2.5311334210155243], ["defenses", 2.5309938412254565], ["katharina", 2.530953536208557], ["outpaced", 2.530800377144338], ["threateningly", 2.530800377144338], ["complied", 2.530660985187016], ["response", 2.530591680026927], ["chimamanda", 2.530406539550634], ["blacksmith", 2.530255063553055], ["mesmo", 2.5299469010813227], ["karlee", 2.5299469010813227], ["perseveres", 2.5298273666187154], ["briarcliff", 2.5298273666187154], ["narrate", 2.529433116000554], ["gazpacho", 2.5291996178925076], ["garage", 2.5291276736564705], ["lunches", 2.528966044071715], ["prodigious", 2.5288917795748476], ["volumizing", 2.52877735526013], ["warned", 2.528582106459268], ["musubi", 2.528437351582112], ["pivot", 2.5282877930022245], ["glassdoor", 2.527865651849315], ["naively", 2.527796029404472], ["timp", 2.527788543985547], ["elitism", 2.5276625891026936], ["adversary", 2.5276193811951875], ["riggan", 2.5275146081537385], ["curio", 2.527295368968397], ["being's", 2.527124559603098], ["psyche's", 2.527124559603098], ["gohonzon", 2.527124559603098], ["j\u00f3zef", 2.527124559603098], ["tastiness", 2.527124559603098], ["taka's", 2.527124559603098], ["grime's", 2.527124559603098], ["smythe's", 2.527124559603098], ["miramare", 2.527124559603098], ["quieting", 2.5271245596030973], ["apartment's", 2.5271245596030973], ["pele's", 2.5265988419913636], ["chuseok", 2.5265988419913636], ["pixel", 2.526546931132331], ["improvising", 2.5264852869872296], ["telescope", 2.5264296207885835], ["sleepovers", 2.526363835774198], ["laois", 2.5262891465255426], ["griddle", 2.526152855570292], ["seniors", 2.5261256961408036], ["fetching", 2.5260665529094837], ["gastroenterologist", 2.5260173856448924], ["paraphrased", 2.5258252667703966], ["journaled", 2.525748879385567], ["bento", 2.5257488793855667], ["nanomaterial", 2.5255826898290867], ["bedsheets", 2.525483569629329], ["balloons", 2.525425398868771], ["jade", 2.5254100924663248], ["otec", 2.525091474679845], ["epitomizing", 2.525091474679845], ["tienda", 2.5248156289867403], ["websites", 2.524440398534612], ["mountaineer", 2.524327741908675], ["whirlpools", 2.524327741908675], ["daydreamed", 2.524327741908675], ["constancy", 2.5242528271490032], ["mohan", 2.5239066366694622], ["tragic", 2.5236669937283676], ["versailles", 2.5233958179873097], ["aare", 2.523230685089071], ["urbanity", 2.523230685089071], ["inhabit", 2.5229162638199014], ["organic", 2.5225956887878436], ["mission", 2.52237910892106], ["calculus", 2.522321983921074], ["repertoire", 2.5222223308190075], ["generalife", 2.5220753596838104], ["stardoll", 2.5220753596838104], ["norte\u00f1o", 2.5220753596838104], ["irgun", 2.5219952607426706], ["icfs", 2.5215211792048646], ["turri", 2.5215211792048646], ["telangana's", 2.5215211792048646], ["tubeworms", 2.5215211792048646], ["mathema", 2.5215211792048646], ["muffuletta", 2.5215211792048646], ["brach's", 2.5215211792048646], ["namath's", 2.5215211792048646], ["halde", 2.5215211792048646], ["secci\u00f3n", 2.5215211792048646], ["biotech's", 2.5215211792048646], ["beatboxers", 2.5215211792048646], ["arctangent", 2.5215211792048646], ["vladan", 2.5215211792048646], ["pooled", 2.5214472497880918], ["childhoods", 2.5211958216333543], ["shelly's", 2.5210579835806706], ["cohesiveness", 2.520830351484534], ["spools", 2.5207175071158745], ["attuning", 2.5207175071158745], ["widespread", 2.5207155938578047], ["amarna", 2.5205605997080247], ["hesitates", 2.5203533167639693], ["turtles", 2.5201667621143202], ["tidier", 2.5199452284678614], ["weirdness", 2.519945228467861], ["denali", 2.5198522587121532], ["midsummer", 2.5196446983273164], ["stress", 2.519526500998298], ["vowels", 2.519466606392179], ["circumferences", 2.5194469201371383], ["zoom's", 2.5194469201371383], ["mirrors", 2.519393109008421], ["recriminations", 2.5193487917273956], ["sobre", 2.519154036873366], ["steadied", 2.5190987571113053], ["absorb", 2.5190090259062017], ["rely", 2.5189670722134974], ["comrades", 2.518946511209236], ["garcia's", 2.518656853031508], ["sensei", 2.5185914117395027], ["trilobites", 2.518543961427008], ["shopkeepers", 2.5185321176485327], ["hypochondriac", 2.5183944929426505], ["conviviality", 2.5183944929426505], ["stella's", 2.5183944929426505], ["protozoa", 2.5183944929426505], ["peugeot's", 2.5182884597443453], ["fillets", 2.51814036868526], ["itineraries", 2.5180822166812806], ["traveler's", 2.5178609065253736], ["molds", 2.5178305891153006], ["faction's", 2.517799376726334], ["satyagraha", 2.5176647350484336], ["khalid's", 2.5176647350484336], ["liszt", 2.517436209611831], ["councilman", 2.5172209633560043], ["byzantine", 2.517176057887127], ["chattier", 2.51713592498016], ["sundance's", 2.51713592498016], ["mignons", 2.51713592498016], ["sepkoski", 2.51713592498016], ["maxville", 2.51713592498016], ["postino", 2.51713592498016], ["kyouka", 2.51713592498016], ["macaco", 2.51713592498016], ["sambal", 2.5163708380728713], ["guide", 2.5163415619270686], ["involuntarily", 2.5162734377101352], ["krill", 2.5160411519114168], ["automatism", 2.5157443195373363], ["parrot's", 2.5157443195373363], ["musk's", 2.5149088517621156], ["cuttings", 2.5148002648094514], ["vectorize", 2.514706257098905], ["etag", 2.514706257098905], ["pagode", 2.514706257098905], ["chhattisgarh's", 2.514706257098905], ["favorable", 2.5142942430793807], ["shrewdness", 2.514191475879438], ["antica", 2.514191475879438], ["kilauea", 2.513681340429345], ["unabashedly", 2.5133534852411], ["petr", 2.51322155967774], ["thien", 2.5129867998290942], ["enrolled", 2.5126388783971554], ["npos", 2.512584142114012], ["hongwu", 2.512584142114012], ["mbuti", 2.512584142114012], ["gregorios", 2.512584142114012], ["t\u00f8nsberg", 2.512584142114012], ["steinbrenner's", 2.512584142114012], ["affably", 2.5122562997155105], ["precocious", 2.5120201988500703], ["scenes", 2.51186508020492], ["sufi", 2.511695620661542], ["wat's", 2.5111608504927236], ["wsu's", 2.5111608504927236], ["proposition", 2.5109649803698884], ["burnouts", 2.510714659865415], ["hutus", 2.510714659865415], ["wrinkles", 2.510367156106264], ["archway", 2.5103359698714685], ["photoshoot", 2.510332854912646], ["outburst", 2.5102118884691964], ["serap", 2.510139994115846], ["jugendstil", 2.510139994115846], ["sikhi", 2.510139994115846], ["chowing", 2.510027733829884], ["rapid", 2.50986544755399], ["bewildering", 2.5098609656594086], ["entire", 2.5093371888669465], ["modernized", 2.5091000523671094], ["gutzon", 2.509055231802054], ["chocolaterie", 2.509055231802054], ["dataquest", 2.509055231802054], ["kamaka", 2.509055231802054], ["malaparte", 2.509055231802054], ["woodwinds", 2.5090552318020536], ["policies", 2.50901148500843], ["galactic", 2.5088793002054333], ["negotiable", 2.5088285099437586], ["disrespecting", 2.5086437426291464], ["sunburnt", 2.5082639237473856], ["antes", 2.5075723386390503], ["morans", 2.50757233863905], ["prokaryote", 2.50757233863905], ["nabataeans", 2.50757233863905], ["cognacs", 2.50757233863905], ["overexploited", 2.50757233863905], ["freestyling", 2.50741725345514], ["pesto", 2.5073289018958222], ["discouragement", 2.5067937103705704], ["counteracting", 2.506624807930792], ["gopi", 2.506624807930792], ["johannesburg's", 2.5064949713614397], ["congaree", 2.5064949713614397], ["invincibility", 2.5064091471884775], ["prominent", 2.5063944086914853], ["mumbling", 2.5062899662519347], ["heartbeats", 2.5061551306765555], ["poses", 2.50610924943012], ["mothballs", 2.505988608740963], ["mendeleev", 2.505988608740963], ["cactus", 2.5058897570740166], ["antihistamines", 2.5058242923939202], ["mulan's", 2.5058242923939202], ["brasov", 2.5057082497759526], ["waikiki", 2.505608559771983], ["insoles", 2.5055021034781513], ["cohabiting", 2.5055021034781513], ["berating", 2.5051000330130178], ["decoherence", 2.505034309956217], ["yaroslavl", 2.505034309956217], ["guitarist", 2.505000786579704], ["agriculture", 2.504928276757912], ["professors", 2.504885200771101], ["warbirds", 2.504807095959849], ["hongdae", 2.504807095959849], ["semaphores", 2.504807095959849], ["entangle", 2.504732316669905], ["halfway", 2.5046940816938297], ["flushed", 2.5045212460934514], ["plato's", 2.503887841164475], ["aztec", 2.5038225019755886], ["jordan's", 2.5034795535091607], ["neuroscience", 2.5031730889221393], ["meekly", 2.5031341575087156], ["truffles", 2.502941549463355], ["bradbury's", 2.502941549463355], ["leviathans", 2.502941549463355], ["jos\u00e9phine", 2.5027522911231306], ["mindedly", 2.502566295857738], ["trespassed", 2.502566295857738], ["quantifying", 2.5023834799985916], ["betsy's", 2.501739079345909], ["peacemaker", 2.501431345156669], ["translation", 2.5012288001084784], ["maximalism", 2.5011863309854703], ["telstra's", 2.5011863309854703], ["particle's", 2.5011863309854703], ["detrimental", 2.501096147365649], ["calavera", 2.5009866354700225], ["aerosmith's", 2.5009866354700225], ["dishonoring", 2.500790886506263], ["litter", 2.5007650785507782], ["brille", 2.5004107693324515], ["zeigt", 2.5004107693324515], ["ujiji", 2.5004107693324515], ["guardrails", 2.5004107693324515], ["thoran", 2.5004107693324515], ["dauphinois", 2.5004107693324515], ["bioretention", 2.5004107693324515], ["ariss", 2.5004107693324515], ["stormwater", 2.500081044835397], ["tulane's", 2.500045105283119], ["dmc's", 2.500045105283119], ["okechukwu", 2.500045105283119], ["ceili", 2.500045105283119], ["gaeilge", 2.49992628749379], ["audioslave", 2.49992628749379], ["concerts", 2.499925282700266], ["bono's", 2.499792315023935], ["purposefully", 2.499785666640119], ["sigurd", 2.4994746046468608], ["acknowledgment", 2.4994490728827086], ["after", 2.499229806092167], ["exemplifies", 2.4990699425515595], ["teakwood", 2.4990270334335403], ["basotho", 2.4990270334335403], ["garfunkel's", 2.4990270334335403], ["\u00f6mer", 2.4990270334335403], ["vlogger", 2.498984329622306], ["sketcher", 2.498608996124502], ["matka", 2.498407235062153], ["montenegrin", 2.498288404753708], ["redistribute", 2.498026937725278], ["teta", 2.497829033896899], ["instinctually", 2.497464653293851], ["galerie", 2.4972362373624257], ["dribbled", 2.4970818200833405], ["meows", 2.4970818200833405], ["band's", 2.4968977955242986], ["dona", 2.496844432325941], ["transported", 2.4965411765030496], ["leah's", 2.4964927467594236], ["petite", 2.4963959547857537], ["amalric", 2.4961915262095236], ["provoke", 2.4960953086813884], ["garlands", 2.4960520012508813], ["bastet", 2.495858360102898], ["rideshare", 2.4957719982219255], ["envy", 2.495698178781759], ["voyages", 2.495436477351021], ["polyrhythm", 2.495436477351021], ["nahanni", 2.495436477351021], ["streamlabs", 2.495436477351021], ["deallocation", 2.495436477351021], ["platyhelminthes", 2.495436477351021], ["\u0161ibenik", 2.495436477351021], ["cam\u00f5es", 2.495436477351021], ["amadu", 2.495436477351021], ["athletes", 2.4951688649851524], ["extinguish", 2.4947105321939733], ["underperforming", 2.4947105321939733], ["tarantino's", 2.4945165007695094], ["overcrowding", 2.494468859125109], ["verbosity", 2.49416729499904], ["intentionally", 2.4941270069021324], ["keychain", 2.494122110808913], ["stride", 2.4940900283542136], ["intensity", 2.4940322130973067], ["preloaded", 2.4939008241341893], ["berklee", 2.4939008241341893], ["ramani", 2.493645065665916], ["salesian", 2.4935827245392743], ["paisa", 2.4935827245392743], ["mateus", 2.4935827245392743], ["unrestrained", 2.4935579622953585], ["imbolc", 2.493339471907869], ["honkai", 2.493339471907869], ["conqueror's", 2.4932409092533288], ["yassa", 2.493048499860733], ["balat", 2.493048499860733], ["meininger", 2.493048499860733], ["imagemagick", 2.493048499860733], ["pyle's", 2.493048499860733], ["microfilaments", 2.493048499860733], ["benjy's", 2.493048499860733], ["bolzano", 2.492991968148718], ["damselfish", 2.4927711246382294], ["northeastern's", 2.4927711246382294], ["pavla", 2.492506414951], ["lansing's", 2.492506414951], ["kawagoe", 2.492506414951], ["uncracked", 2.492506414951], ["sources", 2.4924712146830537], ["microbeads", 2.4924549168845758], ["wondrous", 2.4924549168845753], ["iframe", 2.49231568137165], ["ruggedly", 2.4922535226605227], ["redtail", 2.4922535226605222], ["eluned", 2.4920116737015063], ["mondrian's", 2.4920116737015063], ["jawlines", 2.4920116737015063], ["secretariat's", 2.4920116737015063], ["tibias", 2.4920116737015063], ["kuznetsova", 2.491964565730359], ["upwork", 2.491964565730359], ["catedral", 2.49193338733327], ["tostadas", 2.491780160168089], ["helene's", 2.491780160168089], ["healer's", 2.491780160168089], ["soured", 2.4917081512048935], ["verify", 2.4914742145761415], ["anabaptist", 2.491262925054406], ["op\u00e9ra", 2.4910227801801965], ["retelling", 2.4909233262423918], ["teepees", 2.4908500443934836], ["romcoms", 2.4908500443934836], ["torte", 2.490793804835019], ["mishandling", 2.490528009333834], ["ducale", 2.4904005460637486], ["freelance", 2.4902416826561478], ["focus", 2.490220729920617], ["capitoline", 2.4901233914445733], ["downhearted", 2.4901233914445733], ["astronomical", 2.490009508701248], ["marcellinus", 2.489406282594096], ["sakurajima", 2.489206538850006], ["committed", 2.4890033713104636], ["plankton", 2.488930517963641], ["settler's", 2.4888347935485053], ["cornhole", 2.488834793548505], ["antony's", 2.4887956023857063], ["disorientation", 2.488772279644743], ["choma", 2.4884959460612963], ["lovable", 2.488474419279756], ["formatted", 2.488474419279756], ["preservationist", 2.488185814462834], ["hasan's", 2.488185814462834], ["filmfare", 2.488146438203703], ["hamza", 2.4875716260085596], ["monitoring", 2.4874081063749642], ["sephardi", 2.487280596402378], ["gazelles", 2.487258156122728], ["minimal", 2.486850849391072], ["overselling", 2.4863567502339072], ["therapy", 2.4861991620385724], ["diagnoses", 2.4861677132922884], ["potting", 2.486013442365535], ["squeezing", 2.4859483342613125], ["chronos", 2.485576090054497], ["janie", 2.4854919304946956], ["steampunk", 2.4853539057253604], ["banishing", 2.4850058169430462], ["unleashing", 2.48468132740021], ["sensorineural", 2.484603880172307], ["handhold", 2.484603880172307], ["banat", 2.484445848032366], ["est\u00e1dio", 2.4843017481969536], ["lennon's", 2.48422496913407], ["undermining", 2.4841447552183733], ["bites", 2.483790721935997], ["indiscriminately", 2.483774931269877], ["lisa", 2.4835943972589414], ["dosages", 2.4831429075644786], ["impressions", 2.4831114811487662], ["rewatching", 2.4830972558486835], ["fats", 2.4824827228851327], ["posters", 2.482192675277336], ["sirens", 2.482142278796862], ["dizziness", 2.4811768403375867], ["malfunction", 2.4811768403375867], ["silken", 2.4811768403375867], ["permafrost", 2.4811768403375867], ["picadillo", 2.4811768403375867], ["iucn", 2.4811768403375867], ["maidservant", 2.4811768403375867], ["cookouts", 2.4811768403375867], ["leap's", 2.4811768403375867], ["camotes", 2.4811768403375867], ["expletives", 2.4811768403375867], ["woodfire", 2.4811768403375867], ["\u00e9cole", 2.4811768403375867], ["foamy", 2.4811768403375867], ["elementos", 2.4811768403375867], ["quo's", 2.4811768403375867], ["slea", 2.4811768403375867], ["fleshing", 2.4811768403375867], ["dirndl", 2.4811768403375867], ["arkwright's", 2.4811768403375867], ["leapster", 2.4811768403375867], ["prepper", 2.4811768403375867], ["haredi", 2.4811768403375867], ["golding's", 2.4811768403375867], ["trogons", 2.4811768403375867], ["inupiat", 2.4811768403375867], ["espa\u00e7o", 2.4811768403375867], ["ay\u015fe", 2.4811768403375867], ["nehmen", 2.4811768403375867], ["snowman's", 2.4811768403375867], ["antiguo", 2.4811768403375867], ["coef", 2.4811768403375867], ["culturelles", 2.4811768403375867], ["mangiare", 2.4811768403375867], ["newari", 2.4811768403375867], ["gelang", 2.4811768403375867], ["hammarby", 2.4811768403375867], ["bittersweetness", 2.4811768403375867], ["tekke", 2.4811768403375867], ["photon's", 2.4811768403375867], ["hauptschule", 2.4811768403375867], ["shesha", 2.4811768403375867], ["joannou", 2.4811768403375867], ["wikus", 2.4811768403375867], ["duvernay's", 2.4811768403375867], ["ceol", 2.4811768403375867], ["shipibo", 2.4811768403375867], ["mustelid", 2.4811768403375867], ["hubo", 2.4811768403375867], ["shuki", 2.4811768403375867], ["ludmila", 2.4811768403375867], ["riona", 2.4811768403375867], ["holmenkollen", 2.4811768403375867], ["lucayan", 2.4811768403375867], ["kabhie", 2.4811768403375867], ["clunkiness", 2.4811768403375867], ["uncrushed", 2.4811768403375867], ["mpow", 2.4811768403375867], ["parken", 2.4811768403375867], ["azat", 2.4811768403375867], ["csfb", 2.4811768403375867], ["gaiseric", 2.4811768403375867], ["pelu", 2.4811768403375862], ["sebum", 2.4811768403375862], ["blackrock's", 2.4811768403375862], ["plinian", 2.4811768403375862], ["jeopardizes", 2.4811768403375862], ["numerology", 2.4811768403375862], ["knowledge's", 2.4811768403375862], ["kosovo's", 2.4811768403375862], ["zebrawood", 2.4811768403375862], ["wa\u0142\u0119sa", 2.4811768403375862], ["priestley's", 2.4811768403375862], ["tanmay", 2.4811768403375862], ["ivarsson", 2.4811768403375862], ["makana", 2.4811768403375862], ["significado", 2.4811768403375862], ["\u00e1rea", 2.4811768403375862], ["entro", 2.4811768403375862], ["chicharron", 2.4811768403375862], ["rangle", 2.4811768403375862], ["hickories", 2.4811768403375862], ["quaran", 2.4811768403375862], ["caseta", 2.4811768403375862], ["icin", 2.4811768403375862], ["gandhari", 2.4811768403375862], ["mortdecai", 2.4811768403375862], ["reconsiderations", 2.4811768403375862], ["reddits", 2.4811768403375862], ["assimil", 2.4811768403375862], ["chipo", 2.4811768403375862], ["recuva", 2.4811768403375862], ["playground's", 2.4811768403375862], ["dune's", 2.4811768403375862], ["zooropa", 2.4811768403375862], ["flabbergast", 2.4811768403375862], ["emotional", 2.480715352303826], ["continue", 2.4797811783648966], ["unreachable", 2.478462204844876], ["rowdy", 2.478113659053219], ["cryptographer", 2.4779794475021], ["norgay", 2.477506460396259], ["yitzhak", 2.4774288390681947], ["quiz", 2.477047801722804], ["custodians", 2.4763402383096382], ["items", 2.476321469406246], ["longtail", 2.4762243915544775], ["marula", 2.4762243915544775], ["evenings", 2.4759471936015123], ["verbatim", 2.475875180422335], ["swot", 2.475875180422335], ["scattering", 2.4757399986555715], ["rain", 2.475421283924328], ["figurehead", 2.475415593804295], ["glucagon", 2.474973898236743], ["instructor", 2.4749148225976874], ["cashier", 2.4748405016324235], ["lemurs", 2.4747984937043284], ["declining", 2.4742031184490636], ["sugarless", 2.4740058090071315], ["alphonse", 2.474005809007131], ["investigating", 2.4739400745532687], ["cleaning", 2.473885218602717], ["samba", 2.4737826047206863], ["monocot", 2.4736808377988027], ["snowmelt", 2.4735188871266685], ["guides", 2.4734310989219983], ["rocketeers", 2.4733250148934807], ["kiddo's", 2.4733250148934807], ["ihren", 2.473134127727092], ["intelli", 2.473134127727092], ["rsvp'd", 2.473134127727092], ["stroking", 2.4730860462930075], ["beading", 2.4729610229854755], ["brightest", 2.472789763694192], ["religions", 2.4727841757484987], ["kaluga", 2.4723470295178798], ["baser", 2.472187069176943], ["funtastic", 2.4719359582506684], ["sprinkled", 2.4719080762788543], ["puppeteer", 2.4716093203876985], ["truffle", 2.4714678874840916], ["chateaus", 2.4712916736430146], ["gacaca", 2.4712916736430146], ["unmuted", 2.4712916736430146], ["venue", 2.4710709234317174], ["victory", 2.470803950195021], ["genetics", 2.4704202933419035], ["izuku", 2.4703182983667435], ["dobrynin", 2.4703182983667435], ["starfall", 2.4703182983667435], ["sentimental", 2.470198181752022], ["milan's", 2.470198181752022], ["muggsy", 2.470075377964711], ["ronald's", 2.470075377964711], ["laurence's", 2.470075377964711], ["namm", 2.4698987637905976], ["castellan", 2.4698987637905976], ["lautaro", 2.469821339924142], ["uspto", 2.469582556036944], ["pediatrician's", 2.46955540314631], ["spectroscope", 2.46955540314631], ["scalable", 2.4695144345075626], ["vandalized", 2.4694573815775573], ["fresher", 2.4693617125264553], ["smokey's", 2.4692767116069505], ["caes", 2.4692767116069505], ["cacao", 2.469182200916065], ["tyrolean", 2.4690142087673044], ["aldersgate", 2.468832676952822], ["newest", 2.4687709561358986], ["purloined", 2.4687709561358986], ["unbiased", 2.4684231276068798], ["amusingly", 2.4682540442941616], ["jarocho", 2.4681863856761335], ["agoda", 2.4681863856761335], ["lcas", 2.4681863856761335], ["andaluz", 2.4681863856761335], ["hagupit", 2.4681863856761335], ["jamaicensis", 2.4681863856761335], ["aurangzeb", 2.468186385676133], ["chatroom", 2.468118014862126], ["fumiko", 2.468100809558469], ["impressionistic", 2.4679790911868547], ["oven's", 2.467837179905664], ["unconsumed", 2.467837179905664], ["diamox", 2.467837179905664], ["windir", 2.467837179905664], ["chants", 2.4678318652999143], ["lovers", 2.46771513620384], ["random", 2.467595661843107], ["headlamps", 2.467506444523054], ["eccentrically", 2.467468680998705], ["metamorphoses", 2.4670256416284366], ["kensuke", 2.4668761092693585], ["humans", 2.466820153706149], ["fitness", 2.4667455056539906], ["ieps", 2.466667034253741], ["integrals", 2.466650043848491], ["gorgon's", 2.4662299919018182], ["cuvier's", 2.4662299919018182], ["pioneering", 2.466161072715947], ["vinegar", 2.465563027269101], ["pickpockets", 2.465271860591833], ["openshift", 2.464745205633364], ["antiguos", 2.464745205633364], ["satoshi's", 2.464745205633364], ["anping", 2.464745205633364], ["vocoders", 2.464745205633364], ["greenfingers", 2.464745205633364], ["alphorn", 2.464745205633364], ["occasionally", 2.4643656013377484], ["affirming", 2.463579841469944], ["fianna", 2.463579841469944], ["sz\u00e9chenyi", 2.4635798414699437], ["adventism", 2.4635798414699437], ["dominica's", 2.4635798414699437], ["kanpai", 2.4635798414699437], ["shuf", 2.4635798414699437], ["wettin", 2.4635798414699437], ["younger", 2.4628465134763173], ["aldwin", 2.4624745023450925], ["embellishing", 2.462090864642682], ["javon", 2.4619924833246674], ["unexpressed", 2.461942911342722], ["stagecraft", 2.461942911342722], ["frivolity", 2.46186807115208], ["hovercrafts", 2.46186807115208], ["pints", 2.4617459373710875], ["immer", 2.461484960652368], ["twloha", 2.4614849606523674], ["findet", 2.4614849606523674], ["dakshineswar", 2.4614849606523674], ["kindley", 2.4614849606523674], ["noobie", 2.4614849606523674], ["itu's", 2.4614849606523674], ["etiolation", 2.4614849606523674], ["achieng", 2.4614849606523674], ["clowder", 2.4614849606523674], ["persist", 2.4614152548835704], ["instigating", 2.4613670452051504], ["andalusia", 2.4613047146140588], ["amoebas", 2.461086339606108], ["multinational", 2.461013006511152], ["decaf", 2.460839325252852], ["riptides", 2.4606712466157883], ["convincing", 2.460566850506272], ["terabytes", 2.460500366668107], ["raju", 2.4603015544212847], ["frequent", 2.4601668751573085], ["chicha", 2.4596013895520423], ["linewidth", 2.4596013895520423], ["searched", 2.459503109095624], ["shavings", 2.459459756395901], ["maxime", 2.459459756395901], ["ulaanbaatar", 2.4586206872436085], ["always", 2.458615391451143], ["visual", 2.4584458056768987], ["silverware", 2.458432719301159], ["scallions", 2.4579882717363004], ["merovingian", 2.4579882717363004], ["tool", 2.4575954075575184], ["sabbath's", 2.4575840336148835], ["aerating", 2.4575840336148835], ["harmon's", 2.4575465847153235], ["kaifeng", 2.4575465847153235], ["vacuuming", 2.457319370718956], ["wealth", 2.4569777081910837], ["religion's", 2.456851577197022], ["logi", 2.4567618160784224], ["desire", 2.4566394035258945], ["quality", 2.4565285603306672], ["clumsy", 2.456365071934211], ["sinatra's", 2.456365071934211], ["sembilan", 2.4560782306809927], ["voyage", 2.456077306579017], ["sour", 2.456005481087785], ["luwak", 2.4558587093137336], ["santo's", 2.4558587093137336], ["urmila", 2.4558587093137336], ["buckingham's", 2.4558587093137336], ["crewmate", 2.4556634538045525], ["aminos", 2.4556634538045525], ["telephoto", 2.4555599703046025], ["proposal", 2.455439887349678], ["calisthenics", 2.455331248250737], ["ligier", 2.4553312482507366], ["disturb", 2.454519568499249], ["initiating", 2.454297424567263], ["socioemotional", 2.4539111607734374], ["amoxicillin", 2.4539111607734374], ["torrance's", 2.4539111607734374], ["stiflingly", 2.4539111607734374], ["poetic", 2.453580077521587], ["emotive", 2.4533902749319614], ["coasters", 2.453243061340409], ["myofascial", 2.453140943836597], ["signboards", 2.453140943836597], ["chandragupta", 2.453140943836597], ["gustave", 2.4531409438365968], ["salutations", 2.453072895544119], ["wronged", 2.452935803130492], ["augmented", 2.452910268738804], ["soundstages", 2.452874442919287], ["juana", 2.4524742804879227], ["sandinistas", 2.452325946845289], ["additions", 2.452275652102276], ["broke", 2.4522297772003148], ["kleine", 2.4521911295859793], ["eurotrip", 2.451900417442748], ["shochu", 2.451900417442748], ["seumas", 2.451900417442748], ["jordaan", 2.4517557710845717], ["assistant's", 2.4516390208097585], ["burgers", 2.4506938105848675], ["vandals", 2.4505942747814426], ["catnaps", 2.450545027493913], ["minored", 2.450545027493913], ["fezziwig", 2.450545027493913], ["wardenclyffe", 2.450545027493913], ["ajeet", 2.450545027493913], ["aziz's", 2.450272502468613], ["chasseurs", 2.450272502468613], ["hubs", 2.4500385652496846], ["storyboarding", 2.4498416563608774], ["reliant", 2.4498000904111383], ["pathways", 2.449665326690279], ["wizardry", 2.449598226006017], ["hike", 2.449570378903432], ["introductions", 2.4495418356232825], ["romagna", 2.449366880846079], ["quaife", 2.449058046287909], ["hairdryer", 2.4488938702960192], ["canids", 2.448833753033372], ["sunflower", 2.4486723402458277], ["cousin's", 2.4479467933687795], ["livestreams", 2.4476474235762677], ["swerved", 2.4473426106966194], ["eccentrics", 2.4470947408824], ["virologist", 2.446953711505344], ["surface", 2.4468220840867585], ["avidly", 2.446716050888454], ["excretory", 2.446605244491511], ["festival", 2.4465112553397272], ["hobbiton", 2.446005519147234], ["congregation's", 2.446005519147234], ["churchgoers", 2.4459828426023016], ["definitively", 2.4459709527382016], ["ellen's", 2.445843625839108], ["visualized", 2.4457876302535326], ["role", 2.4457644577784237], ["improper", 2.445591540916955], ["overproduction", 2.444993011415997], ["apiece", 2.4448147659533284], ["dictated", 2.4447187316632384], ["ingolstadt", 2.4446889456267393], ["yekaterinburg", 2.444418664925178], ["cassandra", 2.4442379716471674], ["nickels", 2.4441443501832945], ["kiki", 2.443959187732523], ["wrens", 2.443809719248165], ["lured", 2.4436865309368816], ["shyness", 2.4436779460251796], ["fixies", 2.443583251847623], ["ayten", 2.443583251847623], ["demander", 2.443583251847623], ["ganden", 2.443583251847623], ["logstash", 2.443583251847623], ["supachai", 2.443583251847623], ["speleology", 2.443583251847623], ["karsts", 2.443583251847623], ["lou's", 2.4434403104465203], ["barrettes", 2.4432641212152806], ["terra's", 2.4431719434396366], ["distressing", 2.442702895661466], ["lake's", 2.44250142196302], ["querying", 2.442408452207312], ["boccia", 2.4423217220325233], ["adamant", 2.4422054763532266], ["trent's", 2.441793080967149], ["rashly", 2.4415233291906393], ["zosia", 2.4415024259164446], ["shrimps", 2.4414426916489065], ["bundeswehr", 2.4408325014703087], ["perestroika", 2.440667830699422], ["fairytales", 2.4406551080205157], ["jingu", 2.440501810168118], ["blackbirds", 2.4400977535770307], ["designer", 2.4399968520939606], ["whitehead's", 2.43988645418976], ["pralines", 2.43988645418976], ["unregulated", 2.4398238929986267], ["surging", 2.4398238929986267], ["listicles", 2.4395263897803607], ["montespan", 2.4395263897803607], ["sayur", 2.4393592531408856], ["amasya", 2.4393592531408856], ["heavy", 2.4393205841000936], ["milks", 2.4392414571206134], ["bristling", 2.4391229955861022], ["ngaio", 2.439122995586102], ["possessively", 2.439122995586102], ["ivanna", 2.439122995586102], ["olaf's", 2.438927699093448], ["uneven", 2.438891995593805], ["coloane", 2.4387635610155765], ["yoda's", 2.4387635610155765], ["rivermen", 2.4387635610155765], ["pinetop", 2.4387635610155765], ["assef", 2.4387635610155765], ["maharaja's", 2.4387635610155765], ["backtesting", 2.4387635610155765], ["steganographic", 2.4387635610155765], ["philoxenus", 2.4387635610155765], ["oval's", 2.4387635610155765], ["ximending", 2.4387635610155765], ["bellevue's", 2.4387635610155765], ["seahaven", 2.4387635610155765], ["djemaa", 2.4387635610155765], ["budrys", 2.4387635610155765], ["desgrange", 2.4387635610155765], ["fareeda", 2.4387635610155765], ["sanitize", 2.4387232442101374], ["pr\u00e9s", 2.4385030371868175], ["musa's", 2.4385030371868175], ["debra's", 2.4385030371868175], ["interjection", 2.4383979292972837], ["musicology", 2.4383979292972837], ["cadences", 2.4383979292972837], ["bedecked", 2.4383979292972837], ["buskers", 2.4383979292972837], ["finland's", 2.438136017597037], ["obliviously", 2.4380259387664984], ["bengal's", 2.4379233613459648], ["conchas", 2.4379233613459648], ["qsos", 2.437837506881908], ["plasmodesmata", 2.437837506881908], ["basie", 2.437732109670197], ["artist's", 2.4377000256126378], ["imagination", 2.4376815113895], ["soliloquy", 2.4371843431684805], ["floe", 2.4371843431684805], ["maritza", 2.4371843431684805], ["marit", 2.4371843431684805], ["r\u00e9my", 2.43701349851928], ["floodlights", 2.43701349851928], ["human", 2.436925494457459], ["reappearing", 2.436870111045844], ["unep's", 2.436870111045844], ["mulaney's", 2.436870111045844], ["rasoi", 2.436870111045844], ["vulcania", 2.436870111045844], ["settignano", 2.436870111045844], ["langka", 2.436870111045844], ["caffeinate", 2.436870111045844], ["fennecs", 2.436870111045844], ["blackpink's", 2.436870111045844], ["lopezes", 2.436870111045844], ["asbjorn", 2.436870111045844], ["handknit", 2.436870111045844], ["poho", 2.436870111045844], ["verticale", 2.436870111045844], ["nerida", 2.436870111045844], ["confiture", 2.436870111045844], ["lieselotte", 2.436870111045844], ["zabi", 2.436870111045844], ["larman", 2.436870111045844], ["settlers", 2.4365862956980675], ["organization's", 2.436283951981284], ["hopefuls", 2.4362793546552877], ["wheelbarrows", 2.4362279845343693], ["veterinarian's", 2.4362008251048812], ["bountifully", 2.4362008251048812], ["cyanobacteria", 2.4360645341496303], ["yakovlev", 2.4360645341496303], ["carbonating", 2.435858541883932], ["chantal's", 2.435858541883932], ["solopreneurs", 2.435858541883932], ["coucou", 2.435858541883932], ["strangely", 2.435582565066426], ["contestant's", 2.4354107326106744], ["webpages", 2.4350583117067766], ["nanomaterials", 2.4350583117067766], ["panelist", 2.435028036961505], ["interessant", 2.434799703135015], ["dagang", 2.434799703135015], ["swallowtail", 2.434799703135015], ["caecilian", 2.434799703135015], ["actuelles", 2.434799703135015], ["culturales", 2.434799703135015], ["madhan", 2.434799703135015], ["merseyside's", 2.434799703135015], ["aurelius's", 2.434799703135015], ["unblinkingly", 2.434799703135015], ["vector3", 2.434799703135015], ["adelaja", 2.434799703135015], ["kilig", 2.434799703135015], ["murugesan", 2.434799703135015], ["deluge", 2.4346547745812566], ["unprofessionalism", 2.434557651270701], ["hostal", 2.434557651270701], ["knead", 2.4344880288258577], ["celine's", 2.4344880288258577], ["distribute", 2.434126971566193], ["facepalms", 2.4340658876729493], ["aruba's", 2.4340658876729493], ["washboards", 2.4340658876729493], ["lumberyards", 2.4340658876729493], ["cardamoms", 2.4340658876729493], ["j\u00f3venes", 2.4340658876729493], ["irrigations", 2.4340658876729493], ["lavinia's", 2.4340658876729493], ["kitchenettes", 2.433916329093061], ["mountainside", 2.433710848609389], ["amylases", 2.4336902022450015], ["tijuana's", 2.4336902022450015], ["funimation's", 2.4336902022450015], ["darle", 2.4336902022450015], ["fiddleheads", 2.4336902022450015], ["zazen", 2.433308476858405], ["aube", 2.433308476858405], ["longhouse", 2.433092017850424], ["itself", 2.433040040447069], ["flyers", 2.4328192100820276], ["thumper", 2.4327850934515696], ["hostel", 2.432752385275236], ["rearrange", 2.4325263140564575], ["eggplant", 2.4325263140564575], ["alighieri", 2.4325263140564575], ["digivice", 2.4325263140564575], ["cholesterols", 2.4325263140564575], ["seltzers", 2.4325263140564575], ["kautokeino", 2.4325263140564575], ["barcade", 2.4325263140564575], ["rza's", 2.4325263140564575], ["merrells", 2.4325263140564575], ["heliconius", 2.4325263140564575], ["pagination", 2.432389655274769], ["artificiality", 2.4315533035308348], ["picked", 2.4315296732752123], ["jumpscares", 2.4313039390242683], ["cale's", 2.4311242815987884], ["heavenly", 2.431121794515124], ["rendre", 2.430882715195608], ["genet's", 2.430882715195608], ["karakura", 2.430882715195608], ["mirzakhani", 2.430882715195608], ["yusheng", 2.430882715195608], ["lenca", 2.430882715195608], ["conocer", 2.4305405782898806], ["midwesterner", 2.4305405782898806], ["brayer", 2.4305405782898806], ["chidinma", 2.4305405782898806], ["este", 2.4304797173979966], ["overpopulation", 2.4303925190441276], ["dozing", 2.430150067271518], ["unbearably", 2.4300770884770873], ["snowfalls", 2.429842147089223], ["grammar", 2.4297091856089623], ["werden", 2.429485656163887], ["narvik", 2.429485656163887], ["backslapping", 2.429485656163887], ["contacto", 2.429485656163887], ["vinos", 2.429485656163887], ["huaorani", 2.429485656163887], ["romare", 2.429485656163887], ["grow", 2.429381492568574], ["downsize", 2.42844489406643], ["morbid", 2.4280087651874953], ["desecrating", 2.4278181986098963], ["interfaces", 2.427768457503201], ["publius", 2.427754372483428], ["laggy", 2.4275711678611573], ["mestre", 2.4272382133737262], ["multiples", 2.4272382133737262], ["microinsurance", 2.427238213373726], ["hemen", 2.427238213373726], ["blipped", 2.427238213373726], ["mukwege", 2.427238213373726], ["uderzo", 2.427238213373726], ["kaktovik", 2.427238213373726], ["lisdoonvarna", 2.427238213373726], ["intercommunity", 2.427238213373726], ["abandon", 2.4269454036844937], ["tubers", 2.426884130483373], ["geologist", 2.4268053725400653], ["d\u00e9partement", 2.426699903723388], ["taylor", 2.426557645463041], ["hailstorm", 2.4264449982713163], ["chapel's", 2.4262430357544664], ["longyearbyen", 2.4257315478160764], ["serena's", 2.425350361429991], ["deserts", 2.4247864576026417], ["comicon", 2.424786457602641], ["texts", 2.424755558762787], ["urban", 2.424725904171517], ["furrows", 2.4243165377465172], ["sprouts", 2.424255724588665], ["caulerpa", 2.4241382922838492], ["tianyi", 2.4241382922838492], ["stantz", 2.4241382922838492], ["konev", 2.4241382922838492], ["atiq", 2.4241382922838492], ["neshat", 2.4241382922838492], ["kale", 2.42379962590478], ["thudded", 2.4237717206382596], ["turbulences", 2.423666118872808], ["ashoke", 2.423666118872808], ["baldachin", 2.423666118872808], ["soupe", 2.423666118872808], ["canaima", 2.423666118872808], ["gazette's", 2.423666118872808], ["slowed", 2.423621276204999], ["shoelaces", 2.4230776581258247], ["foreboding", 2.4228671739115004], ["extracurriculars", 2.4227962088002317], ["oprah's", 2.4225308422932432], ["stoker's", 2.42210120128193], ["fridtjof", 2.4218658401701543], ["cubist", 2.4218658401701543], ["nana", 2.42182880640444], ["glitchy", 2.4218185427218546], ["exerciser", 2.421803591009609], ["monopolized", 2.4215852568360288], ["ganache", 2.4215852568360288], ["fertilizing", 2.4215852568360288], ["encrusted", 2.4215756173045664], ["conclusions", 2.4215045373274675], ["rafiki", 2.421389446594512], ["quite", 2.4211132748257223], ["extensively", 2.42107275898258], ["irrespective", 2.420836936632879], ["microtrauma", 2.42066033203667], ["nov\u00e9", 2.42066033203667], ["bioart", 2.42066033203667], ["melburnian", 2.42066033203667], ["amusa", 2.42066033203667], ["sputnik's", 2.42066033203667], ["hydrosols", 2.42066033203667], ["macro's", 2.42066033203667], ["popovich's", 2.42066033203667], ["relic's", 2.42066033203667], ["gadgeteer", 2.42066033203667], ["medya", 2.42066033203667], ["besondere", 2.42066033203667], ["semillas", 2.42066033203667], ["rival", 2.420626266452527], ["superbug", 2.420363682486175], ["cairo's", 2.420363682486175], ["stressful", 2.4202098468299504], ["squaring", 2.4201642950833837], ["sequencers", 2.420011709225299], ["druidic", 2.419949209729961], ["loops", 2.4198412607274515], ["indulgent", 2.4196517235649875], ["malec\u00f3n", 2.4195873443008376], ["vasuki", 2.4195873443008376], ["playcalling", 2.4195873443008376], ["linkedin", 2.419292687294038], ["frenzied", 2.4192412611158467], ["endurance", 2.419147419329147], ["verge's", 2.419147419329147], ["leyland's", 2.419147419329147], ["dropoffs", 2.419147419329147], ["milenko", 2.419147419329147], ["foraging", 2.4186879421364917], ["ruffled", 2.4186133406734776], ["ipscs", 2.4182626890770456], ["fraternizing", 2.4182626890770456], ["fernwood", 2.4182626890770456], ["hatchets", 2.4182484422130828], ["satine", 2.41809607321036], ["heights", 2.4179787490782636], ["wines", 2.4178694756316674], ["cognition", 2.41773122250247], ["lens", 2.4176385955748874], ["shkoder", 2.4175569213545716], ["verrazzano", 2.4175569213545716], ["sanath", 2.4175569213545716], ["persona's", 2.4175569213545716], ["unordered", 2.417556921354571], ["cuticles", 2.4171464702643584], ["imperfect", 2.4169477944706914], ["monte's", 2.4164504879809536], ["alfred's", 2.4162245670303197], ["jozef", 2.4162245670303197], ["morty's", 2.4160255885125297], ["icebergs", 2.4160255885125297], ["apollo's", 2.415882712960282], ["existences", 2.415450301520697], ["frowns", 2.4153496996755695], ["kales", 2.4153048888241995], ["gametophytes", 2.4153048888241995], ["maserati's", 2.4153048888241995], ["pashto", 2.4151880945839275], ["destined", 2.414824461301006], ["rotorua", 2.414716746399973], ["constantly", 2.4146827648376843], ["magwitch", 2.414568066100336], ["chak", 2.4141180068149493], ["enclosures", 2.413978300911777], ["vanishing", 2.413811555156378], ["filk", 2.4135961481883137], ["silkies", 2.4135961481883137], ["unharvested", 2.4135961481883137], ["touchless", 2.4135083810556526], ["yilmaz", 2.4135083810556526], ["armoring", 2.4135083810556526], ["crusades", 2.4133155934223702], ["bagan", 2.4130126414272137], ["impressionable", 2.412839342616424], ["museo", 2.4128178865731837], ["feodor", 2.412813783148382], ["wanaka", 2.412607981904511], ["plowing", 2.4125276787472187], ["dispersed", 2.412473367701657], ["misdeeds", 2.4123644873963697], ["vielen", 2.4122552614393205], ["sienna's", 2.4122552614393205], ["monetization", 2.411870224685699], ["aargau", 2.411448217189926], ["student's", 2.411134049151023], ["metamorphose", 2.410983645592489], ["drafting", 2.4108677810763792], ["tesla's", 2.410866413501824], ["eifel", 2.4108600877774125], ["alla", 2.410834197956882], ["testimonies", 2.4107888448670165], ["sahrawi", 2.4102860734707985], ["tripped", 2.410154306246139], ["life", 2.410151554586505], ["jumble", 2.41012146642699], ["inventions", 2.4100907821020194], ["ginza", 2.4099468831987085], ["adulting", 2.409142674005205], ["safaris", 2.4089095537258123], ["inside", 2.4088465335436164], ["underpaying", 2.4087743736161755], ["felipe's", 2.4086581017890976], ["clammed", 2.4086581017890976], ["ambika", 2.4086581017890976], ["kiryas", 2.408201050915893], ["tefnut", 2.408201050915893], ["savannah's", 2.408201050915893], ["tikanga", 2.408201050915893], ["tiefer", 2.408201050915893], ["zugspitze", 2.408201050915893], ["cimbalom", 2.408201050915893], ["haneke's", 2.408201050915893], ["helvering", 2.408201050915893], ["eduardo", 2.408101084081068], ["homonyms", 2.407720947038119], ["lancia", 2.407234484168586], ["instagram's", 2.407234484168586], ["losing", 2.4071620735749737], ["grapplers", 2.407111860029002], ["humanness", 2.407111860029002], ["maddie's", 2.406953601524069], ["arching", 2.4062095089043845], ["syllabi", 2.406196221536176], ["redstart", 2.4061234483700797], ["tavern", 2.406042612073843], ["oilskin", 2.4057037805554544], ["keala", 2.4057037805554544], ["contradicted", 2.405662762588182], ["blindfolded", 2.405662762588182], ["guitar's", 2.4053712024424314], ["behold", 2.4052312535724707], ["vr's", 2.4052224472660275], ["beldon", 2.4052224472660275], ["anatoly", 2.40493956312051], ["milo's", 2.4048329375579685], ["pots", 2.40479974455676], ["criticisms", 2.4046241194431444], ["zoologist", 2.404597308228402], ["peckham's", 2.4042411243581263], ["weymouth's", 2.4042411243581263], ["epss", 2.4042411243581263], ["thylacines", 2.4042411243581263], ["v\u00e4sterbotten", 2.4042411243581263], ["waterfall's", 2.4042411243581263], ["partisi", 2.4042411243581263], ["puddicombe", 2.4042411243581263], ["sweltered", 2.4042411243581263], ["ingr", 2.4042411243581263], ["cooked", 2.404144954713152], ["dimensionality", 2.4039417635955607], ["saturn's", 2.403478191892777], ["pontiac's", 2.403234112368867], ["especiales", 2.403234112368867], ["kindergartner", 2.4032341123688665], ["basics", 2.402912790230115], ["physicist", 2.4028735408136246], ["nouns", 2.4027880115906575], ["beth's", 2.4027753416652096], ["ideation", 2.4027321392964684], ["jasna", 2.4027206556628804], ["unpatched", 2.4027206556628804], ["gary's", 2.4026906341636427], ["benefactor", 2.402505379546395], ["socrates", 2.4024728437892438], ["hematologist", 2.4023497395099174], ["intertribal", 2.4023497395099174], ["backups", 2.402230304508663], ["toppled", 2.402148972225872], ["potent", 2.4020047429777236], ["structured", 2.4017866887897195], ["undermined", 2.401636721532506], ["karim", 2.401483412022108], ["bothering", 2.4012794596475593], ["mussolini's", 2.401138877746052], ["scorekeeper", 2.4011388777460514], ["diversidad", 2.4011388777460514], ["mudskippers", 2.4011388777460514], ["heer's", 2.4011388777460514], ["quba", 2.4011388777460514], ["mat's", 2.4011388777460514], ["intimidations", 2.4011388777460514], ["redshanks", 2.4011388777460514], ["cady's", 2.4011388777460514], ["sarlat", 2.4011388777460514], ["sinx", 2.4011388777460514], ["conquerors", 2.4009397783366198], ["munich's", 2.4009034719737237], ["budge", 2.4009034719737237], ["perching", 2.4009034719737237], ["valentino's", 2.4005736378972413], ["counterfeiters", 2.400538593026615], ["roskilde", 2.4004881626030308], ["robi", 2.4004881626030308], ["fruition", 2.4004638587844362], ["consequently", 2.4002750926159018], ["tundra", 2.399504769343141], ["coincidence", 2.3994231516839193], ["crackdowns", 2.399379801645139], ["bahasa", 2.3993324653958954], ["chivalric", 2.398969173940859], ["ones", 2.3989268884123787], ["bounds", 2.3988162729860885], ["tautly", 2.3987208124109602], ["perplexities", 2.3987208124109602], ["biofuels", 2.398709980037349], ["shelter", 2.3986568016027636], ["bruised", 2.3986397331998264], ["mandu", 2.3986020168081263], ["langue", 2.398470945659667], ["despondent", 2.3982982819964564], ["invisible", 2.3982692239653307], ["reuter's", 2.3981634888021284], ["relief's", 2.3981634888021284], ["argentinosaurus", 2.3981634888021284], ["biff's", 2.3981634888021284], ["buscar", 2.3981634888021284], ["nummer", 2.3981634888021284], ["deplete", 2.3979158725410232], ["vegetation", 2.3977688582685133], ["associating", 2.397551991274357], ["furloughed", 2.3975416659441846], ["trento", 2.3973532984342896], ["misgivings", 2.396804328134671], ["glyphs", 2.3963502816935667], ["catriona", 2.396269648082688], ["inbox", 2.3961735596963916], ["europcar", 2.39603841934561], ["shortages", 2.3959845342690334], ["consistent", 2.3958966562226722], ["signs", 2.395810814544295], ["camping", 2.3957002981879563], ["spotlights", 2.3956986810335916], ["misbehaviors", 2.39561901825698], ["sengen", 2.39561901825698], ["tarta", 2.39561901825698], ["cavatelli", 2.39561901825698], ["mariology", 2.39561901825698], ["quilled", 2.3953724539282177], ["al's", 2.3953724539282173], ["yoshi", 2.3951244643721212], ["bulky", 2.3949559451358553], ["daringly", 2.3948750371954097], ["spectrum's", 2.3948750371954097], ["mode's", 2.3948750371954097], ["alcuin", 2.3948508223235967], ["paiutes", 2.3948067604524175], ["dunant", 2.3948067604524175], ["andorran", 2.3948067604524175], ["underestimation", 2.394806760452417], ["stacked", 2.394597133262575], ["folks", 2.394495109591603], ["salvageable", 2.3944690475300905], ["contrasts", 2.3943356509257714], ["doilies", 2.394335650925771], ["skirmishes", 2.394292921129439], ["effective", 2.3940638725347543], ["parser", 2.3940160614030113], ["rejoined", 2.3937448945352147], ["whitewater", 2.393679420275304], ["honked", 2.3935516359390476], ["thyme", 2.393169285744416], ["beauvais", 2.393135081486898], ["vandalize", 2.393135081486898], ["involvement", 2.3930330330845937], ["configurations", 2.392751921027683], ["scrunchies", 2.3927070016221457], ["jeroen", 2.3925633817541017], ["vanquish", 2.391925874857817], ["kamala's", 2.3918799874719667], ["ignorance", 2.391628912554494], ["mauchly", 2.391495749722975], ["frein", 2.391495749722975], ["unproductively", 2.391495749722975], ["pidi", 2.391495749722975], ["tagger", 2.3912791287311523], ["gouda", 2.391224809872176], ["jogged", 2.39113413242211], ["disregard", 2.391050941979591], ["bhut", 2.390952227961674], ["ccc's", 2.390952227961674], ["tainan", 2.390952227961674], ["razvan", 2.390952227961674], ["congregationalist", 2.390793195154901], ["equalizers", 2.3905545124159366], ["thinker", 2.3905379312721933], ["chasse", 2.3904020778862116], ["persistently", 2.3903480452895143], ["disdain", 2.3901617085775397], ["bumblebees", 2.390031568651716], ["imperialistic", 2.3899571035604694], ["seem", 2.3895076943239473], ["embodiment", 2.3893489719819554], ["vergessen", 2.389281401806565], ["grimsel", 2.389281401806565], ["devoutness", 2.389281401806565], ["recente", 2.389281401806565], ["aram's", 2.389281401806565], ["woodchopper", 2.389281401806565], ["contr\u00f4le", 2.389281401806565], ["mariamman", 2.389281401806565], ["compos\u00e9", 2.389281401806565], ["tirthankaras", 2.389281401806565], ["revolves", 2.3892447900780662], ["intersections", 2.3891331833573215], ["joysticks", 2.38897094424396], ["classifier", 2.3888792335854223], ["sonically", 2.3887925962824634], ["arthritic", 2.3887925962824634], ["enlisting", 2.3885956149518557], ["rereading", 2.38839779182069], ["juncture", 2.387263137502379], ["micaela", 2.3872121084107794], ["teotihuacan", 2.387192869112678], ["centeredness", 2.3866558178485358], ["poodle", 2.3866558178485358], ["subconsciousness", 2.3866558178485358], ["northwest's", 2.3866558178485358], ["apprenticing", 2.386538254387728], ["bottles", 2.386251816704026], ["mani\u00e8re", 2.385746961863064], ["webp", 2.385746961863064], ["buxton's", 2.385746961863064], ["blobfish", 2.385746961863064], ["stoneground", 2.385746961863064], ["regardez", 2.385746961863064], ["sukuma", 2.385746961863064], ["thabang", 2.385746961863064], ["hairnets", 2.385746961863064], ["sailplanes", 2.385746961863064], ["khasi", 2.385746961863064], ["aletheia", 2.385746961863064], ["underbanked", 2.385746961863064], ["fidget", 2.385501640838708], ["brownstone", 2.385501640838708], ["sava", 2.384967942446946], ["seawolf", 2.38465508682331], ["unlocked", 2.3845188208183483], ["uprooted", 2.384367917376438], ["attitudes", 2.3842406667224365], ["valor", 2.3841190404067336], ["mona's", 2.384087311802638], ["unstructured", 2.3840873118026376], ["identically", 2.3840873118026376], ["ronit", 2.3840489213208125], ["kublai", 2.3840223255050907], ["beanies", 2.3837484303766865], ["softhearted", 2.38366334006231], ["\u0438\u0441\u0442\u043e\u0440\u0438\u0438", 2.38366334006231], ["tomu", 2.38366334006231], ["blender's", 2.38366334006231], ["botanico", 2.38366334006231], ["sawhorses", 2.38366334006231], ["churchgoer", 2.383581382169996], ["improv", 2.3834086138474633], ["sensory", 2.3832765432971263], ["bacteria's", 2.3832356492716293], ["tartlets", 2.3827585523074943], ["kristan", 2.3827585523074943], ["overgeneralization", 2.382587032112252], ["googleplex", 2.382587032112252], ["adipiscing", 2.382587032112252], ["manderley", 2.382587032112252], ["manipur's", 2.382587032112252], ["hiwa", 2.382587032112252], ["luxuria", 2.382587032112252], ["tanoa", 2.382587032112252], ["funan", 2.382587032112252], ["whirligigs", 2.382587032112252], ["hydrating", 2.382423533259474], ["ionescu", 2.3823946242117344], ["sicily's", 2.3823946242117344], ["perplexity", 2.382289357570509], ["chance's", 2.3819297667240833], ["thundering", 2.3818753549512683], ["rarest", 2.381505633930863], ["ribosomes", 2.3814866994311656], ["penwith", 2.3814866994311656], ["umit", 2.3814866994311656], ["such", 2.381388353858634], ["fashioned", 2.381331549575437], ["shape", 2.38117359854417], ["react", 2.380903072859116], ["guesthouse", 2.3807243366801947], ["alma's", 2.3805150606309082], ["quickbooks", 2.380445301947813], ["suffragettes", 2.380445301947813], ["brenda's", 2.3803159931693916], ["jrpgs", 2.3800762560114372], ["mackerels", 2.379904316242175], ["overstory", 2.379904316242175], ["perfekt", 2.379904316242175], ["olla", 2.3796741514146853], ["candidate's", 2.379617334908151], ["infatuation", 2.379617334908151], ["thutmose", 2.3795271373814844], ["avifauna", 2.3795271373814844], ["turbofans", 2.3792106688168637], ["jacquetta", 2.3792106688168637], ["zookeeper's", 2.3792106688168637], ["unzen", 2.3792106688168637], ["thunderheads", 2.3792106688168637], ["readied", 2.3791232942054497], ["cinematographers", 2.3790107351472156], ["galas", 2.378930541972027], ["resolvers", 2.3787093602056513], ["snris", 2.3787093602056513], ["antivenin", 2.3787093602056513], ["goalball", 2.3787093602056513], ["magnificence", 2.3786322545537804], ["reading", 2.378378807115629], ["apparent", 2.3780714773790605], ["decaying", 2.3777944719901876], ["enumerating", 2.3777944719901876], ["mesures", 2.377794471990187], ["mombi", 2.377794471990187], ["samarth", 2.377794471990187], ["apulian", 2.377794471990187], ["kingfisher's", 2.377794471990187], ["minwoo", 2.377794471990187], ["sargis", 2.377794471990187], ["galleries", 2.377619247637056], ["greens", 2.377508885337294], ["creams", 2.377296243708995], ["mussels", 2.3772901677543463], ["script's", 2.37725320304596], ["redressal", 2.3771754158922986], ["retested", 2.377127488968591], ["scholar", 2.3771143248300066], ["telegu", 2.3768282816318003], ["akane's", 2.3768282816318003], ["wrexham's", 2.3768282816318003], ["barnet's", 2.3768282816318003], ["euna", 2.3768282816318003], ["peahen", 2.3768282816318003], ["lobstermen", 2.3768282816318003], ["isn't", 2.3765667560828083], ["daily", 2.3765538835700184], ["mesoamerica", 2.3765489012872063], ["recreated", 2.376474697075114], ["pescado", 2.3763383822951534], ["elisabeth's", 2.3763383822951534], ["cyberbully", 2.3763383822951534], ["guanacos", 2.3763383822951534], ["skewering", 2.376215187867371], ["concept", 2.3762007336122677], ["irony", 2.3761380694034053], ["deny", 2.376038387925108], ["kristjan", 2.3755948471317314], ["vuejs", 2.3755948471317314], ["redditt", 2.3755948471317314], ["jesmyn", 2.3755948471317314], ["bengaluru's", 2.3755948471317314], ["external", 2.3751833644391778], ["longitudes", 2.3751436420325613], ["monochromatic", 2.374840690037405], ["kilimanjaro", 2.374840690037405], ["leurs", 2.374840690037404], ["s'mores", 2.3744595568822064], ["biles", 2.3743319046292695], ["silverwork", 2.3743319046292695], ["tearing", 2.374208762618008], ["documentation", 2.373965495384728], ["jolla's", 2.373965495384728], ["microtargeting", 2.373965495384728], ["peele's", 2.373965495384728], ["sunyata", 2.373965495384728], ["zorbing", 2.373965495384728], ["guernsey's", 2.373965495384728], ["nonessentials", 2.373965495384728], ["kino's", 2.373965495384728], ["tarana", 2.373965495384728], ["calotype", 2.373965495384728], ["geotextiles", 2.373965495384728], ["tidepools", 2.373965495384728], ["snuggles", 2.3739213573258753], ["reappeared", 2.3738182270537487], ["skilling", 2.373545881510157], ["politician's", 2.3734868733090457], ["appliances", 2.3733206151143347], ["danuel", 2.3732995864098654], ["leguminous", 2.3732995864098654], ["dystopias", 2.3732995864098654], ["manah", 2.3732995864098654], ["cimeti\u00e8re", 2.3732995864098654], ["livvy", 2.3732995864098654], ["risques", 2.3732995864098654], ["lovely's", 2.3732995864098654], ["luogo", 2.3732995864098654], ["guaido", 2.3732995864098654], ["bromley's", 2.3732995864098654], ["\u00e1kos", 2.3732995864098654], ["staggering", 2.37282164920892], ["tacticians", 2.372600220095909], ["downtrodden", 2.372492812074936], ["ribfest", 2.372440006697294], ["formaggio", 2.372440006697294], ["witness", 2.3723348834744398], ["expecting", 2.372023927627678], ["shetland's", 2.371713156205046], ["nesrin", 2.371713156205046], ["commenced", 2.371519522269599], ["guacamole", 2.3713092311713466], ["analytica", 2.3711955974148213], ["listener", 2.370985603017458], ["mighty", 2.370705395303508], ["pepperoni", 2.370496893048018], ["crowdsourced", 2.37041001710823], ["upperclassman", 2.370030578715086], ["belleau", 2.370030578715086], ["bekah", 2.370030578715086], ["caused", 2.369981543599664], ["kanga", 2.3699516716327986], ["pradesh's", 2.3699349936947707], ["ticktock", 2.3699349936947707], ["shuttered", 2.369746742717635], ["overbearingly", 2.3696632744797177], ["lambics", 2.3696632744797177], ["gymnast's", 2.3696632744797177], ["palahniuk's", 2.3696632744797177], ["technotronic", 2.3696632744797177], ["nuisances", 2.3696632744797173], ["celebrate", 2.369443585537271], ["altered", 2.369287579966173], ["typos", 2.36928062989099], ["multiculturalism", 2.3692212999808904], ["technologists", 2.3691110084047478], ["interactively", 2.369075477069322], ["intensifying", 2.3690048779201653], ["campus's", 2.3689628123826205], ["sattva", 2.3689628123826205], ["productos", 2.368761061163715], ["triptans", 2.368761061163715], ["stickiest", 2.368761061163715], ["handwrote", 2.3683960748676967], ["cloudlike", 2.3683960748676967], ["terreiro", 2.3683960748676967], ["shipment's", 2.3683960748676967], ["pascu", 2.3683960748676967], ["meseret", 2.3683960748676967], ["vorrei", 2.3683960748676967], ["uro\u0161", 2.3683960748676967], ["morana", 2.3683960748676967], ["ottway", 2.3683960748676967], ["raghuvir", 2.3683960748676967], ["languidly", 2.368396074867696], ["balances", 2.3678281141782795], ["yonas", 2.367789726666245], ["ngoma", 2.367789726666245], ["avery's", 2.3676314595085786], ["semolina", 2.3670997442301114], ["dogon", 2.3670037197384386], ["chandon", 2.366824638692875], ["equipped", 2.3667309347701826], ["divinity", 2.3663564227262195], ["perlite", 2.3663075421738093], ["dull", 2.3660912555504012], ["dhyana", 2.3659646487464547], ["mutually", 2.365923845144452], ["mockups", 2.3657732663683966], ["flagging", 2.3655598721857616], ["superstition", 2.365541746852718], ["nalanda", 2.3655287672710044], ["farms", 2.365388587788499], ["breweries", 2.365121794579861], ["moisture", 2.3649302679691253], ["handel's", 2.3647134376278633], ["hamar", 2.364657485957092], ["faith's", 2.364540322202059], ["cocoa", 2.3644990735894464], ["vlad's", 2.364382063697126], ["lopsided", 2.3636795360903577], ["prithvi", 2.363399458675992], ["m\u00fcntzer", 2.363025562226273], ["dussehra", 2.363025562226273], ["preamps", 2.363025562226273], ["fulcrums", 2.363025562226273], ["messagebox", 2.363025562226273], ["tousling", 2.363025562226273], ["fortschritte", 2.363025562226273], ["tekakwitha", 2.363025562226273], ["scheint", 2.363025562226273], ["edenia", 2.363025562226273], ["cattedrale", 2.363025562226273], ["pe\u00f1as", 2.363025562226273], ["fanboy's", 2.363025562226273], ["calabashes", 2.363025562226273], ["accidentes", 2.363025562226273], ["snowshill", 2.363025562226273], ["naruhito", 2.363025562226273], ["novacane", 2.363025562226273], ["odette's", 2.3630255622262726], ["moruga", 2.3630255622262726], ["group2", 2.3630255622262726], ["afrofuturist", 2.3630255622262726], ["ultracapacitors", 2.3630255622262726], ["earthshine", 2.3630255622262726], ["manyatta", 2.3630255622262726], ["achebe", 2.362693676613601], ["pani", 2.362554840002722], ["saris", 2.362554840002722], ["hyponatremia", 2.362511861017093], ["carthaginian", 2.362460245106123], ["selina's", 2.3617328786583154], ["intoned", 2.361602052851438], ["chef's", 2.3614342992214064], ["herculean", 2.361240799717039], ["project's", 2.361043911042761], ["sparring", 2.3609647841196804], ["tables", 2.360871573540705], ["tagging", 2.360829160261383], ["disorienting", 2.360731362651296], ["tiniest", 2.3602989942698582], ["immutable", 2.3602720366770686], ["darwin's", 2.3601906631384626], ["disturbance", 2.360166127117191], ["exchange", 2.360061767453311], ["lucille's", 2.359988254305674], ["clarabelle", 2.3596206262576476], ["biomechanics", 2.359620626257647], ["complex's", 2.3595505246347637], ["obsolete", 2.359338290795282], ["petroglyph", 2.359280672270288], ["valjean", 2.3592386622868076], ["swaggered", 2.3592386622868076], ["snively", 2.3592386622868076], ["hanifa", 2.3592386622868076], ["tchotchkes", 2.3590607542359603], ["neural", 2.358844034383551], ["accused", 2.358797961806353], ["effervescent", 2.358649588962891], ["riah", 2.3585331181916223], ["donburi", 2.3585331181916223], ["suburbia", 2.3585254730275436], ["vocalization", 2.358481282298915], ["borscht", 2.358481282298915], ["jams", 2.358471367506346], ["testing", 2.3581783303037573], ["bacopa", 2.3581432780067972], ["weifang", 2.3581432780067972], ["revised", 2.358016975436772], ["curragh", 2.3579978482640893], ["assignment", 2.3577752637088762], ["developing", 2.3576337790643223], ["consistently", 2.3573318928759086], ["rescuers", 2.357260430975147], ["soaps", 2.357117998320707], ["incline", 2.357117998320707], ["federer's", 2.357117998320707], ["eyespots", 2.357117998320707], ["aquanaut", 2.357117998320707], ["cheeseheads", 2.357117998320707], ["khoikhoi", 2.357117998320707], ["gauguin's", 2.357117998320707], ["desalinated", 2.356859002825891], ["integration", 2.356370655898919], ["ankara's", 2.356266335789699], ["raden", 2.356266335789699], ["unifying", 2.356238148377325], ["capsized", 2.35603922578143], ["soundlessly", 2.355547633231886], ["toweled", 2.355547633231886], ["d'orcia", 2.355547633231886], ["fname", 2.355547633231886], ["entropia", 2.355547633231886], ["hhof", 2.355547633231886], ["salep", 2.355547633231886], ["blazor", 2.355547633231886], ["wheaty", 2.355547633231886], ["nwsa", 2.355547633231886], ["analytica's", 2.355547633231886], ["daniyal", 2.355547633231886], ["lassie's", 2.355547633231886], ["messaging", 2.3555451445391937], ["swift", 2.3555342684226193], ["jalapenos", 2.3554415274826415], ["overpowering", 2.355169430121489], ["maaya", 2.355015306083133], ["clamshells", 2.355015306083133], ["necesito", 2.355015306083133], ["beaks", 2.3548623830113096], ["foxgloves", 2.354586185218322], ["routes", 2.354556783517778], ["archaeologists", 2.354467168363082], ["beans", 2.3542412715493013], ["breonna", 2.3539370023715565], ["malachi's", 2.3539370023715565], ["past's", 2.3539370023715565], ["tanzimat", 2.3539370023715565], ["phaidon", 2.353685540241189], ["ayana", 2.3536397130305144], ["faris", 2.3535889721801833], ["breezing", 2.353527871059769], ["brianna's", 2.3533410312335423], ["snarks", 2.3533410312335423], ["hicham", 2.3533410312335423], ["decimated", 2.3528766703885062], ["constructiveness", 2.352840107216677], ["seamlessness", 2.352840107216677], ["rink's", 2.352840107216677], ["hammertime", 2.352840107216677], ["ivanisevic", 2.352840107216677], ["hostess", 2.352687325391533], ["wren's", 2.352493438569673], ["sowie", 2.352493438569673], ["footrest", 2.352493438569673], ["jobim", 2.3524131719767536], ["panned", 2.352391947196255], ["subaqueous", 2.3520449601713183], ["crewe's", 2.3520449601713183], ["klieg", 2.3520449601713183], ["remaining", 2.352004565956545], ["csas", 2.3517241356243215], ["montesquieu", 2.3516648624078775], ["endpoint", 2.351432575478571], ["disputes", 2.3514047621751932], ["nadph", 2.3512722937230532], ["hermione's", 2.3512722937230532], ["sneezing", 2.3512104344151417], ["unnerved", 2.350991635751973], ["fidgety", 2.3509150562198635], ["fernanda", 2.350842648335418], ["uprooting", 2.3508099216079845], ["bulldoze", 2.350588585582977], ["boded", 2.350588585582977], ["speargun", 2.350588585582977], ["unfairness", 2.3505885855829765], ["artemis", 2.3504719889269454], ["quiche", 2.3502754243005914], ["forehands", 2.3502754243005914], ["anaphase", 2.3501387982531674], ["digits", 2.3500599460396594], ["cindy's", 2.3500577390189337], ["leakages", 2.3500577390189337], ["nira", 2.349946084944578], ["oppressors", 2.349758575489197], ["theresienwiese", 2.3495992806227144], ["sorgen", 2.3495992806227144], ["organica", 2.3495992806227144], ["mawashi", 2.3495992806227144], ["chuckwalla", 2.3495992806227144], ["sambu", 2.3495992806227144], ["sahle", 2.3495992806227144], ["tonality", 2.349199348830268], ["crewmember", 2.349199348830268], ["maslow's", 2.349118298170182], ["healthy", 2.3486639092450723], ["sneering", 2.3483814601505046], ["person's", 2.3482083178485857], ["pedestrians", 2.348095537083116], ["wagner's", 2.347749710479616], ["timeframes", 2.3476958077877796], ["scarcities", 2.347059173292312], ["dishwasher's", 2.3470591732923114], ["lpga's", 2.3470591732923114], ["lasst", 2.3470591732923114], ["pizarro's", 2.3470591732923114], ["bifidus", 2.3470591732923114], ["consider", 2.3470231386069487], ["tall", 2.3469983948933484], ["exceeded", 2.3469775364515018], ["journalism", 2.3467797614859673], ["composing", 2.3467421678974776], ["strategy", 2.3466068309789136], ["tribesman", 2.3465393373735313], ["uyuni", 2.346044667490154], ["ensemble", 2.346011778178244], ["tournaments", 2.3460028997585014], ["tavern's", 2.345839921773718], ["syncopations", 2.345839921773718], ["jatra", 2.345839921773718], ["tawhid", 2.345839921773718], ["feuilles", 2.345839921773718], ["selassie", 2.34571028870996], ["landslides", 2.345549810479905], ["hairdryers", 2.345391043105181], ["treaty's", 2.345391043105181], ["bryson's", 2.345391043105181], ["pointillism", 2.345391043105181], ["virtuoso", 2.3452433484551114], ["coincidences", 2.3451474083015347], ["curcumin", 2.3451123039319772], ["tricycles", 2.3451123039319772], ["knockoff", 2.3446719656264574], ["mapuche", 2.3446527558214036], ["weighed", 2.3446352974985754], ["mela", 2.344472882192265], ["dietitian", 2.3444589328087813], ["gains", 2.3441545462897304], ["checklists", 2.3440591728452462], ["grouchy", 2.3440423877225274], ["handfuls", 2.344026361725961], ["rescuing", 2.343825979533184], ["fatou", 2.3436808945254284], ["lands", 2.3436517821359693], ["rainbows", 2.343511544680146], ["ferments", 2.3433336825410542], ["chowder", 2.3433336825410542], ["exercise", 2.34310634205116], ["goldfish", 2.342828498193109], ["undervalue", 2.342718311300891], ["clich\u00e9", 2.342711165054231], ["pricking", 2.3425228404363687], ["dilutes", 2.3425228404363687], ["jennifer's", 2.3425007933700783], ["delay's", 2.3421138315871026], ["immortan", 2.3421138315871026], ["gaped", 2.3420454287298713], ["negotiations", 2.341648932834652], ["kindi", 2.341507945351696], ["foday", 2.341507945351696], ["hartebeest", 2.341507945351696], ["madhyamaka", 2.341507945351696], ["piezoelectricity", 2.341507945351696], ["constance's", 2.3413922296143426], ["fraudulence", 2.3413922296143426], ["homebuilt", 2.3413922296143426], ["romanticizes", 2.3413922296143426], ["balzac's", 2.3413922296143426], ["docker", 2.3408799299938905], ["alexander's", 2.3406682071140494], ["emir", 2.3405029748363035], ["endocrinologist", 2.3404008493964468], ["kagoshima", 2.3404008493964468], ["lacto", 2.3402311034266625], ["neurologist", 2.3402008835002235], ["bamboo", 2.3401882682020725], ["scut", 2.339899867222279], ["heavenward", 2.339899867222279], ["rehydrate", 2.339843982343673], ["smartly", 2.3397701696485464], ["adjust", 2.339410454203768], ["pairing", 2.3393507772278377], ["outdoor", 2.338436196228754], ["set\u00fabal", 2.3382818390616], ["speck", 2.3382137938228973], ["unserer", 2.337951012542208], ["zollverein", 2.3377922490617964], ["volatile", 2.337741188710452], ["alpaca", 2.337653932774477], ["amur", 2.3374000366756404], ["oulu", 2.337340501767292], ["riverport", 2.3373405017672915], ["suomalainen", 2.3373405017672915], ["kardelj", 2.3373405017672915], ["rotonde", 2.3373405017672915], ["siu's", 2.3373405017672915], ["janmabhoomi", 2.3373405017672915], ["puno", 2.336922372876099], ["droves", 2.3368545681910406], ["miserly", 2.3362482865795497], ["stabilize", 2.336010764783113], ["noticeably", 2.335990073261064], ["transcendence", 2.3356954663229472], ["camellias", 2.3356757293301356], ["moodier", 2.3356757293301356], ["clopping", 2.3356757293301356], ["\u00e9conomique", 2.3356757293301356], ["senses", 2.3354971664788233], ["vacation", 2.3353706315926486], ["ectotherms", 2.3352252614941995], ["ruinas", 2.3352252614941995], ["kigo", 2.3352252614941995], ["remi's", 2.3352252614941995], ["milosevic's", 2.3352252614941995], ["reinforcement", 2.3349811045779183], ["bellissimo", 2.3345865237312085], ["sophie", 2.33453410088098], ["underlined", 2.3344723366827056], ["bangalter", 2.334440575586439], ["katherina", 2.334440575586439], ["bastilla", 2.334440575586439], ["foresighted", 2.334440575586439], ["paititi", 2.334440575586439], ["deregulations", 2.334440575586439], ["hartwood", 2.334440575586439], ["mazes", 2.3343316803992398], ["echo", 2.3342235029303193], ["paramedics", 2.3341671125475845], ["ironclads", 2.334096769905204], ["orenburg", 2.334096769905204], ["headset", 2.3338793103348365], ["kendra's", 2.333322726946893], ["pickled", 2.332809171168751], ["atsume", 2.3327303627105516], ["gameforge", 2.3327303627105516], ["futrelle", 2.3327303627105516], ["erasures", 2.3327303627105516], ["seyhan", 2.3327303627105516], ["pebblebrook", 2.3327303627105516], ["i\u00f1\u00e1rritu's", 2.3327303627105516], ["skyrider", 2.3327303627105516], ["hinckley's", 2.3327303627105516], ["hepatologist", 2.3327303627105516], ["gel\u00e9e", 2.3327303627105516], ["jumprope", 2.3327303627105516], ["buzek", 2.3327303627105516], ["natalia", 2.332482199906008], ["allmusic", 2.332386657043436], ["puducherry", 2.3323062299173314], ["replicants", 2.3323062299173314], ["activist", 2.3322143344788007], ["conquered", 2.3318718882384974], ["ruth's", 2.331818529309894], ["gorgonzola", 2.3317083559799006], ["henriette", 2.331641628977955], ["lactase", 2.3316023499626257], ["ember's", 2.3316023499626253], ["akili", 2.3316023499626253], ["houghton's", 2.3316023499626253], ["bheema", 2.3316023499626253], ["crochet", 2.3313304862160753], ["nobleman", 2.3311247171362184], ["relinquishing", 2.3307164598594494], ["overfilled", 2.330585182398022], ["efendi", 2.330483896644767], ["steinbeck's", 2.330451144429228], ["schoolbooks", 2.330451144429228], ["guavas", 2.3301486848387767], ["dunes", 2.330079532752926], ["neither", 2.32992615366132], ["broth", 2.329774416814946], ["conflict", 2.329524132316038], ["qadar", 2.3294891732427123], ["durness", 2.3294891732427123], ["raccoon's", 2.3294891732427123], ["ciriaco", 2.3294891732427123], ["deduplication", 2.329268054194469], ["fao's", 2.329268054194469], ["genshin", 2.329268054194469], ["thrillseekers", 2.329268054194469], ["gd's", 2.329268054194469], ["displacement", 2.3289360131318615], ["animals", 2.328684859095451], ["datasource", 2.3286125686016184], ["heterotrophs", 2.3286125686016184], ["quinny", 2.3286125686016184], ["stealthy", 2.3283653007381893], ["noirs", 2.3283507306066484], ["puffins", 2.3282275830565027], ["waterdeep", 2.328096521139381], ["juleps", 2.328096521139381], ["lhotse", 2.328096521139381], ["lacson", 2.328096521139381], ["hydrologist", 2.328056355228592], ["filets", 2.328056355228592], ["aloes", 2.328056355228592], ["mediocrity", 2.3279829672409855], ["stone's", 2.3279829672409855], ["faceless", 2.3278894753324373], ["jockeyed", 2.327877584756088], ["vuvuzelas", 2.327877584756088], ["yudhishthira", 2.327877584756088], ["unpacking", 2.327701984234231], ["spurts", 2.3274439813541337], ["purpose", 2.327419063413636], ["adichie", 2.3271727605924952], ["vaxxers", 2.3271496680629187], ["standup", 2.3269343357935566], ["tbis", 2.3269343357935566], ["herding", 2.3266254209899593], ["coriander", 2.3266254209899593], ["legends", 2.32658043720886], ["shelters", 2.3262975344375993], ["weights", 2.3262746398082235], ["hexa", 2.326103287816488], ["fished", 2.3261032878164873], ["obasi", 2.3261032878164873], ["\u0434\u0435\u043d\u044c", 2.3261032878164873], ["kenkey", 2.3261032878164873], ["amed", 2.3261032878164873], ["pseudopodia", 2.3261032878164873], ["stablecoins", 2.3261032878164873], ["lyka", 2.3261032878164873], ["tarkov", 2.3261032878164873], ["homecooking", 2.3261032878164873], ["planetariums", 2.3261032878164873], ["metallers", 2.3261032878164873], ["fcvs", 2.3261032878164873], ["costumbres", 2.3261032878164873], ["algirdas", 2.3261032878164873], ["gouden", 2.3261032878164873], ["jayceon", 2.3261032878164873], ["putera", 2.3261032878164873], ["aquaventure", 2.3261032878164873], ["pensar", 2.3261032878164873], ["decimeters", 2.3261032878164873], ["roddick's", 2.3261032878164873], ["colormap", 2.3261032878164873], ["tibbie", 2.3261032878164873], ["studebakers", 2.3261032878164873], ["hermocrates", 2.3261032878164873], ["complained", 2.326008053290886], ["suspicion", 2.3256046590302457], ["fufu", 2.3252330658607017], ["walkie", 2.3252271660508312], ["condiment", 2.325174703669654], ["sylvia", 2.3250123181002587], ["utilized", 2.3250021738340894], ["boxes", 2.3249420064186124], ["wangari", 2.3248174374639077], ["fishing", 2.3243580532593007], ["togo's", 2.3242874382553507], ["hyperledger", 2.3242874382553507], ["rotoscoping", 2.3242874382553507], ["city", 2.32400312707571], ["dimness", 2.3239672884704667], ["yilan", 2.3239672884704667], ["saravanan", 2.3239672884704667], ["grandpappy", 2.3239672884704667], ["atan", 2.3239672884704667], ["stir", 2.3239268169039105], ["enough", 2.3239117007802816], ["luciferin", 2.323868798587077], ["resistencia", 2.323760787023118], ["spinner's", 2.323760787023118], ["jaron", 2.323641802855835], ["enrolling", 2.3235885815593886], ["locales", 2.323368993439541], ["unreturned", 2.3232837686797403], ["watermarking", 2.3232837686797403], ["workstation", 2.32328376867974], ["junkie", 2.323101864219305], ["tkts", 2.3230141732244336], ["harmlessness", 2.3230141732244336], ["gleam", 2.322945305503625], ["norse", 2.3228193772925114], ["inspected", 2.3226572088715742], ["fermi's", 2.3225627957497954], ["tauern", 2.3225627957497954], ["rh\u00f4ne", 2.322127042880049], ["reggie's", 2.322127042880049], ["lola's", 2.322127042880049], ["tanah", 2.322127042880049], ["salacious", 2.321795689135346], ["usma", 2.3215689734152853], ["frenzy", 2.3214792764322216], ["required", 2.3214244972032794], ["plage", 2.321404089255242], ["critter's", 2.3211009151545166], ["haman's", 2.3211009151545166], ["ortona", 2.3211009151545166], ["pardus", 2.3211009151545166], ["metella", 2.3211009151545166], ["bhangra", 2.320909436775111], ["countryside", 2.3207353571522953], ["bahnhof", 2.3205251024740017], ["cobs", 2.3203598229082987], ["pyrrhus", 2.3203598229082987], ["converging", 2.320244085580174], ["homelessness", 2.320223175416059], ["kamil", 2.3202144187334075], ["prosciutto", 2.3201565144159733], ["dozed", 2.3201565144159733], ["controversial", 2.3199733215050653], ["osaa", 2.3197994848684753], ["thierry's", 2.3197994848684753], ["frutas", 2.3197994848684753], ["fourier's", 2.3197994848684753], ["darabont's", 2.3197994848684753], ["grna", 2.3197994848684753], ["biofabrication", 2.3197994848684753], ["fotograf\u00eda", 2.3197994848684753], ["\u00e9l\u00e9ments", 2.3197994848684753], ["ale\u0161", 2.3197994848684753], ["huib", 2.3197994848684753], ["after's", 2.3197994848684753], ["giggler", 2.3197994848684753], ["abcd", 2.3197619901297513], ["fertilization", 2.319457278422726], ["unfathomable", 2.319427304620622], ["osteoarthritis", 2.319360959446005], ["petrovna", 2.319360959446005], ["autumn's", 2.319360959446005], ["concluded", 2.318958827718194], ["laughably", 2.3188568601285855], ["enfoque", 2.3188568601285855], ["preuve", 2.3188568601285855], ["sunbirds", 2.3188568601285855], ["externe", 2.3188568601285855], ["anglicization", 2.3188568601285855], ["mutuo", 2.3188568601285855], ["ratchett", 2.3188568601285855], ["adso", 2.3188568601285855], ["salavat", 2.3188568601285855], ["scripto", 2.3188568601285855], ["biofiltration", 2.3188568601285855], ["andreescu", 2.3188568601285855], ["arpoador", 2.3188568601285855], ["roschmann", 2.3188568601285855], ["tiferet", 2.3188568601285855], ["smiths", 2.3188282322661147], ["letting", 2.318656097742613], ["glazes", 2.3184046008118937], ["debrief", 2.3184046008118937], ["tolkien's", 2.318298710774944], ["sc\u00e8ne", 2.318109805727771], ["agglutinative", 2.3179415218943245], ["abloh", 2.3179415218943245], ["itched", 2.3179031587172516], ["fodmap", 2.3179031587172516], ["sprinklings", 2.3175827629526906], ["rezaei", 2.3175827629526906], ["pinebrook", 2.3175827629526906], ["gastropubs", 2.3175827629526906], ["midgame", 2.3175827629526906], ["neuron's", 2.3175827629526906], ["jeremy's", 2.317488090454204], ["collections", 2.3174498006881494], ["saly", 2.317052302260781], ["liegt", 2.317052302260781], ["curfews", 2.3169813141387756], ["frenzies", 2.3168883908075935], ["boudicca", 2.3168883908075935], ["determining", 2.316828792392396], ["soohyun", 2.316761507544132], ["swagger's", 2.316761507544132], ["bomas", 2.316761507544132], ["manikarnika", 2.316761507544132], ["inchi", 2.316761507544132], ["taima", 2.316761507544132], ["calculon", 2.316761507544132], ["worthy", 2.3165284900095435], ["gracias", 2.316301231822447], ["juniper's", 2.3162885060113547], ["ladakhi", 2.3162885060113547], ["calmed", 2.3162194789744834], ["saunas", 2.3160795220641726], ["undermine", 2.316042587541069], ["bl\u00fccher", 2.3157650509817476], ["patches", 2.315479464328854], ["kakadu", 2.315026605493552], ["cgas", 2.314973607013538], ["fuzzy's", 2.314973607013538], ["pataliputra", 2.314973607013538], ["smoothened", 2.314973607013538], ["wiele", 2.314973607013538], ["tortoise's", 2.314973607013538], ["uottawa", 2.314973607013538], ["strutting", 2.314314070373363], ["hideout", 2.3142613074421488], ["patricio", 2.3140771755801572], ["nova's", 2.313529756530993], ["replaying", 2.313410434763159], ["timbaland", 2.3131290065298615], ["jayda", 2.3129614613316485], ["ichthyologist", 2.3129614613316485], ["regimens", 2.3128883242103417], ["neurodegenerative", 2.3128297687071657], ["flinching", 2.312752584619307], ["swift's", 2.3126834611560074], ["buddha's", 2.312666001064074], ["negatives", 2.312558087718726], ["spiritual", 2.3125358965281873], ["shushing", 2.3125003446128822], ["roasting", 2.312366178597723], ["downwards", 2.3123625932893015], ["fatigued", 2.312300929738693], ["vitruvian", 2.3122798755172], ["apr\u00e8s", 2.312225109964405], ["monarchs", 2.31216914251749], ["statue's", 2.312005692132751], ["bima", 2.3120056921327508], ["spoleto", 2.3120056921327508], ["hurled", 2.3116297562478514], ["slouching", 2.3115237230495467], ["lush's", 2.3113624558277897], ["infinitives", 2.3113095145681224], ["villainess", 2.311192369304843], ["hand", 2.3111583004669054], ["hummingbird", 2.3111400502925217], ["einar", 2.3110961698305745], ["kurhaus", 2.3108999983536345], ["herbivorous", 2.3108999983536345], ["vermi", 2.3108999983536345], ["banquo", 2.3108999983536345], ["medidas", 2.3108999983536345], ["ipc's", 2.3108999983536345], ["chinchillas", 2.3108999983536345], ["lindenhof", 2.3108999983536345], ["eutychius", 2.3108999983536345], ["vrubel", 2.3108999983536345], ["mahalaya", 2.3108999983536345], ["gesamtkunstwerk", 2.3108999983536345], ["mussorgsky's", 2.3108999983536345], ["deked", 2.3108999983536345], ["mahane", 2.3108999983536345], ["bahadur's", 2.3108999983536345], ["stronglifts", 2.3108999983536345], ["soapdish", 2.3108999983536345], ["citoyens", 2.3108999983536345], ["endothermy", 2.3108999983536345], ["dressed", 2.31074923721706], ["asuka", 2.310504173065844], ["deontology", 2.3102205217295966], ["olympiads", 2.3102205217295966], ["mae's", 2.310061196176374], ["averageness", 2.3100611961763735], ["motaung", 2.3100611961763735], ["teppan", 2.3100611961763735], ["geraes", 2.3100611961763735], ["inteligente", 2.3100611961763735], ["cl\u00ednica", 2.3100611961763735], ["aldeia", 2.3100611961763735], ["journalist", 2.309975638354293], ["lumberyard", 2.3097134001516557], ["homeroom", 2.3095680675188484], ["standoff", 2.309332370284576], ["fashion's", 2.3092880833262477], ["members", 2.3092224239563333], ["rubbed", 2.309095220765786], ["dosa", 2.309037777251964], ["dwarfed", 2.3090135493753867], ["veneers", 2.308872893091921], ["yehuda", 2.308872893091921], ["refraining", 2.3086446759432513], ["stinginess", 2.308573234053233], ["cryptographers", 2.308507952810311], ["seismometers", 2.308507952810311], ["counterpart", 2.308336377807084], ["catastrophe", 2.308246333283755], ["mushiness", 2.3080714793838015], ["swooshed", 2.3080714793838015], ["wunderland", 2.3080714793838015], ["allergist", 2.308071479383801], ["filial", 2.3080054590325605], ["handy", 2.3075210550492216], ["runtime", 2.3074944615139557], ["syd's", 2.3074401688041015], ["abkhazian", 2.3074401688041015], ["alc\u00e1zar", 2.3074401688041015], ["anwr", 2.3074401688041015], ["iterated", 2.307397917668028], ["fortresses", 2.3072793305740418], ["ciphertext", 2.3072625758279424], ["dubose", 2.3072625758279424], ["servicemembers", 2.306936893684666], ["pushed", 2.306664327998601], ["buzzes", 2.3066453895914063], ["condoning", 2.3066216857409727], ["megalopolis", 2.3065755071286453], ["ease", 2.3065314476718153], ["ebikes", 2.3064460769335313], ["schoenberg's", 2.3064460769335313], ["floofy", 2.3064460769335313], ["cirrus", 2.3063288079648037], ["kickstart", 2.3063011923867616], ["predicting", 2.306244947997239], ["educator", 2.306096615406723], ["secure", 2.306046635211241], ["caseback", 2.305926431540508], ["riyadh's", 2.305926431540508], ["ooda", 2.305926431540508], ["kaylan", 2.305926431540508], ["manzana", 2.305926431540508], ["polynices", 2.305926431540508], ["snapshots", 2.305919111139138], ["suspecting", 2.30569887301108], ["garuda", 2.305518302968554], ["mlms", 2.305518302968554], ["constants", 2.305426814147008], ["finalize", 2.305320234385825], ["indifferently", 2.3052068516611617], ["hyphens", 2.305147402340663], ["malleability", 2.3050933226362096], ["aftertaste", 2.3050599038626847], ["lenina", 2.3050599038626847], ["krak\u00f3w", 2.305043916486462], ["copacabana", 2.304882696418863], ["harps", 2.304882696418863], ["dishwashing", 2.3048186825684933], ["albini", 2.3046504248194974], ["recollecting", 2.3046504248194974], ["whipping", 2.304569056505697], ["backpack", 2.3044495678790473], ["minor", 2.3044004280439907], ["goodies", 2.3042868564732153], ["brera", 2.304243831822634], ["playground", 2.304237162906705], ["rote", 2.3041679144586444], ["crepes", 2.303949923170616], ["kadar", 2.303949923170616], ["brindisi", 2.303949923170616], ["puntland", 2.303949923170616], ["stakeholder", 2.3036024194114653], ["hachiko", 2.3035534423715847], ["frasier's", 2.3035534423715847], ["quizzed", 2.3031019474903913], ["reheating", 2.3029313776696565], ["enshrouded", 2.3028886841456644], ["stratus", 2.302570725371652], ["sophisticated", 2.3025099544686345], ["dignity", 2.302295805277058], ["oysters", 2.302169561445651], ["distributing", 2.3019444606868844], ["apus", 2.3013814171247184], ["fran\u00e7ais", 2.301337241345059], ["zugzwang", 2.3010914245066325], ["unmistakably", 2.301044563872797], ["descents", 2.3007276155857617], ["tarsiers", 2.3007276155857617], ["runtimes", 2.3007276155857617], ["cocker's", 2.3007276155857617], ["reallocate", 2.300584855123142], ["suricata", 2.3004288585911397], ["meta's", 2.3004288585911397], ["haast", 2.3004288585911397], ["faso's", 2.3004288585911397], ["slavophiles", 2.3004288585911397], ["estava", 2.3004288585911397], ["quirkily", 2.3004288585911397], ["openmind", 2.3004288585911397], ["sarabjit", 2.3004288585911397], ["evaluations", 2.3003922081048107], ["heya", 2.3003453079062033], ["skye's", 2.3003408005273447], ["sitara", 2.300298655463531], ["stefanos", 2.300298655463531], ["wracked", 2.3002576957296377], ["filigree", 2.3000440957950357], ["trustpilot", 2.2999672958185493], ["millones", 2.2999672958185493], ["alva's", 2.2999672958185493], ["lanfang", 2.2999672958185493], ["doudou", 2.299488833722477], ["ch\u00e2telet", 2.299488833722477], ["squeaking", 2.299393961289717], ["debaters", 2.29931047507724], ["restraint", 2.299204946212141], ["palms", 2.299107384870561], ["bowing", 2.299047902057487], ["ouzo", 2.298897401409277], ["chalking", 2.2987373667833526], ["screen", 2.298575541939578], ["eug\u00e8ne", 2.2984545148863686], ["marina's", 2.2984545148863686], ["fossil", 2.298453530550187], ["repainting", 2.298301256488003], ["gpus", 2.2982695732614182], ["neoclassical", 2.2982051284488345], ["yangtze", 2.2979362353625548], ["narrowing", 2.297876416458714], ["lulling", 2.297785508660461], ["litterbug", 2.297385963275543], ["maxi's", 2.297385963275543], ["muspelheim", 2.297385963275543], ["macrocytic", 2.297385963275543], ["nonmetals", 2.297385963275543], ["rebeccah", 2.297385963275543], ["quadricycle", 2.297385963275543], ["sunniva", 2.297385963275543], ["encanta", 2.297385963275543], ["dharma's", 2.297385963275543], ["amik", 2.297385963275543], ["jaccard", 2.297385963275543], ["mihailovic", 2.297385963275543], ["reznor", 2.297166117728818], ["campervan", 2.297133503279579], ["eragon", 2.297089526377056], ["chitchat", 2.2966790752868427], ["higgins", 2.296615028388538], ["blown", 2.296467008890233], ["wildflower", 2.2963544838634604], ["misted", 2.2962814507931992], ["janissary", 2.296204787973088], ["pianist's", 2.296204787973088], ["skittered", 2.296204787973088], ["empty", 2.2960772210404228], ["crop", 2.2958804422613115], ["irritably", 2.2958257690729793], ["gelderland", 2.2958257690729793], ["m\u00fcller's", 2.2958257690729793], ["mongolia's", 2.2956682915272997], ["fisherman's", 2.2954600109710004], ["bhaiya", 2.29543902601853], ["yeon's", 2.2952338452771586], ["antlered", 2.2952338452771586], ["caty", 2.2952338452771586], ["\u00faltimo", 2.2950885773122676], ["wil's", 2.2950885773122676], ["runoff", 2.2945396443829895], ["chiron", 2.294153460412643], ["camille's", 2.2939874633298696], ["continent's", 2.2938226707610747], ["gallons", 2.293527196005419], ["resource", 2.293468216926741], ["homesteader", 2.293165736477333], ["sobek", 2.293165736477333], ["mozzarella", 2.2931193893437456], ["adhere", 2.2930876282474792], ["photorealistic", 2.2929496317602527], ["simile", 2.292916340778898], ["farewell", 2.292850402327427], ["numeric", 2.292656050998211], ["feast", 2.2925127203231406], ["sparkle", 2.2925040181035827], ["wind's", 2.2923916459640745], ["vented", 2.292362739947481], ["optogenetics", 2.292134795359485], ["dismayed", 2.2920959298074743], ["fond", 2.2920510541185553], ["sandpaper", 2.29173875755576], ["loyally", 2.2917051543481706], ["xhosa", 2.29129085765869], ["wringing", 2.290507562187891], ["completist", 2.290317083388542], ["trombones", 2.290317083388542], ["panchami", 2.290317083388542], ["gramma's", 2.290317083388542], ["herc's", 2.290317083388542], ["weaponry", 2.2899420314644057], ["injustices", 2.2899194588948975], ["emollients", 2.2897912989341367], ["freedonia", 2.2897912989341367], ["triggering", 2.2897482960333955], ["spitefully", 2.28973959547039], ["shoemaking", 2.28973959547039], ["farida", 2.289449539038773], ["instrumental", 2.289184036724975], ["keiko", 2.289144726159125], ["irked", 2.289128455008966], ["etchings", 2.2887590445563046], ["mileva", 2.2884640760395216], ["numerics", 2.2884640760395216], ["stratego", 2.2884640760395216], ["peda", 2.2884640760395216], ["gerontius", 2.2884640760395216], ["noongar", 2.2884640760395216], ["inglenook", 2.2883053863456007], ["aino", 2.2883053863456007], ["samosa", 2.288196419422441], ["tekapo", 2.2881169709165063], ["beignet", 2.2881169709165063], ["bizet's", 2.2881169709165063], ["ayahuasca", 2.2881051325629733], ["synths", 2.2880150468463927], ["striding", 2.287799748464753], ["waterways", 2.2877463193153504], ["passing", 2.2877089646958066], ["cabrito", 2.287609852793519], ["kamakhya", 2.287609852793519], ["esquina", 2.287609852793519], ["backslashes", 2.287609852793519], ["razorbills", 2.287609852793519], ["ctdna", 2.287609852793519], ["doohickeys", 2.287609852793519], ["blablacar", 2.287609852793519], ["abcdefg", 2.287609852793519], ["\u00e1frica", 2.287609852793519], ["roh's", 2.287609852793519], ["home's", 2.2874684302997497], ["waterfall", 2.2874019343630954], ["nicolette", 2.2873348996862126], ["healers", 2.2872893434134394], ["rahel", 2.287118316512301], ["binti", 2.287118316512301], ["tommi", 2.287118316512301], ["russification", 2.2869977832676884], ["sagged", 2.286867810672595], ["serve", 2.286572673750092], ["github", 2.2864619690925507], ["jarring", 2.286392864086786], ["gabe's", 2.2863200204157863], ["bessarabia", 2.2863200204157863], ["s\u00e9curit\u00e9", 2.286280774329196], ["stallion's", 2.286280774329196], ["carnations", 2.2860965973626413], ["ended", 2.286082382531234], ["covert", 2.285971861256272], ["parisian", 2.285670497682714], ["multispectral", 2.2856286144358524], ["bumps", 2.285425702682812], ["momo", 2.285358472056042], ["discounted", 2.285294458205672], ["teenager's", 2.285294458205672], ["deerstalker", 2.285294458205672], ["kipchoge", 2.285294458205672], ["slowing", 2.284810798002877], ["commandeering", 2.2847503404775282], ["dogmatic", 2.2843737419559336], ["intermingle", 2.2842580434853974], ["adulthood", 2.2841788801499794], ["walruses", 2.2839972238869177], ["symphonic", 2.2839734787500614], ["stretches", 2.2838217563306946], ["fangirl", 2.2834735064461453], ["sahara's", 2.283027301005071], ["umaine", 2.283027301005071], ["strokes", 2.282947587435171], ["skein", 2.282909284602848], ["fundraising", 2.282797918041245], ["cocooned", 2.28268269311058], ["dorm", 2.282356222473693], ["predisposition", 2.2822322525344214], ["maryam", 2.2816128656172334], ["travailler", 2.281541922149505], ["zafar's", 2.281541922149505], ["donaldo", 2.281541922149505], ["archerfish", 2.281541922149505], ["floated", 2.2813310792675945], ["convinced", 2.2812493439398667], ["bagus", 2.2811827373380837], ["algy", 2.2811827373380837], ["vivek", 2.280707048788063], ["trams", 2.2806888902924514], ["atum", 2.280637983194612], ["captchas", 2.280637983194612], ["let's", 2.2805437475585646], ["carnivores", 2.280387963084834], ["ubers", 2.280319667548353], ["concatenated", 2.280319667548353], ["klosters", 2.280293813876994], ["tumbling", 2.2801159584481616], ["linoleum", 2.2800003397696744], ["antihero", 2.27982479308506], ["confit", 2.27982479308506], ["unbidden", 2.2797137635580387], ["habitat's", 2.2797137635580387], ["seneca's", 2.2797137635580387], ["fuelwood", 2.2797137635580387], ["paragons", 2.2794551460011974], ["thuringia", 2.2793943864547104], ["asanas", 2.2792205858915042], ["underactive", 2.2791603543810814], ["seismologists", 2.2789224710333746], ["sumerians", 2.278811353359698], ["supremes", 2.278811353359698], ["flatfooted", 2.278631792146763], ["scrimmaging", 2.278631792146763], ["pmrc", 2.278631792146763], ["greuze", 2.278631792146763], ["students", 2.2784800255973114], ["sprigs", 2.278255314362022], ["graphic", 2.2782311699558564], ["alice", 2.2781838559824954], ["porche", 2.2781264402301193], ["caregiving", 2.2775791783842836], ["reykjavik", 2.277498144488979], ["glassware", 2.2772679701717213], ["setar", 2.277029378790823], ["reloj", 2.277029378790823], ["keyboard's", 2.277029378790823], ["egoistic", 2.277029378790823], ["bradykinesia", 2.277029378790823], ["kibale", 2.277029378790823], ["antworten", 2.277029378790823], ["engrossment", 2.277029378790823], ["kahoolawe", 2.277029378790823], ["malapropisms", 2.277029378790823], ["pefc", 2.277029378790823], ["persephone", 2.2769522109731906], ["cryptanalysis", 2.2769234162342227], ["greenhouses", 2.2768024474196062], ["decentralized", 2.276320701212278], ["polka", 2.2762441962464126], ["strs", 2.275799786439397], ["salinity", 2.275772397963148], ["dances", 2.275685533139449], ["deindustrialization", 2.2756355636232306], ["gullah", 2.2756355636232306], ["logographic", 2.2753150064231145], ["raluca", 2.2753150064231145], ["otoscope", 2.2753150064231145], ["anger", 2.275276899119458], ["accents", 2.275079086664384], ["adversarial", 2.2750096895869922], ["compromised", 2.274412103642788], ["peeled", 2.2744121036427876], ["asexually", 2.2744121036427876], ["landscapers", 2.2744121036427876], ["malted", 2.2744121036427876], ["slayer's", 2.2744121036427876], ["cocoons", 2.2744121036427876], ["cellblocks", 2.2744121036427876], ["museu", 2.2744121036427876], ["shortcutting", 2.2744121036427876], ["veneziana", 2.2744121036427876], ["saratov", 2.2744121036427876], ["cartomancy", 2.2744121036427876], ["oishii", 2.2744121036427876], ["konferenz", 2.2744121036427876], ["frbs", 2.2744121036427876], ["erano", 2.2744121036427876], ["verdicchio", 2.2744121036427876], ["semilla", 2.2744121036427876], ["queremos", 2.2744121036427876], ["tenoch", 2.2744121036427876], ["vevo's", 2.2744121036427876], ["herlev", 2.2744121036427876], ["bikram", 2.2735883397515733], ["eora", 2.2735883397515733], ["legwear", 2.2735883397515733], ["painted", 2.273544430194158], ["holbein's", 2.2733295657543335], ["pleco", 2.2733295657543335], ["zooniverse", 2.2733295657543335], ["jamberry", 2.2733295657543335], ["gungnir", 2.2733295657543335], ["homomorphic", 2.2732029531358005], ["hector's", 2.2730781375995956], ["sweaty", 2.273045064061334], ["traces", 2.27302656674741], ["shoestring", 2.2730244208461783], ["purifier", 2.2722356327302107], ["crocheted", 2.2722356327302107], ["admirably", 2.2721231951184913], ["casseroles", 2.272088904354082], ["waste", 2.271928366248435], ["samhain", 2.2719209622368264], ["revolution's", 2.2716179855793444], ["disharmony", 2.2715797099894344], ["metaprogramming", 2.27149992425272], ["couchbase", 2.27149992425272], ["whistled", 2.2714046165635904], ["playability", 2.271231107693637], ["lorenzo", 2.2712002242378198], ["\u00e9glise", 2.2711000826888665], ["waterland", 2.2711000826888665], ["keiki", 2.2709712381304685], ["d\u00e9veloppement", 2.2709712381304685], ["muffling", 2.2709712381304685], ["napoleonic", 2.270932889323404], ["marmalades", 2.270907616580164], ["symptoms", 2.270810837036779], ["noises", 2.270806597558074], ["reviews", 2.2707665148642215], ["goghs", 2.2707198762018095], ["whizzy", 2.2707198762018095], ["polow", 2.2707198762018095], ["nazneen", 2.2707198762018095], ["preteens", 2.270719876201809], ["above", 2.270276808908892], ["pachuca", 2.270276808908892], ["syntactical", 2.270276808908892], ["eirik", 2.2701489131954724], ["compared", 2.26917340093903], ["slivers", 2.269149001254193], ["weekly", 2.2690762394700186], ["unbroken", 2.2689315925496727], ["goffredo", 2.2689227743500058], ["dilara", 2.268780379297015], ["galicians", 2.268780379297015], ["muddying", 2.268504539737222], ["repairing", 2.268354770511447], ["equal", 2.268278545681774], ["affinity", 2.26780782458374], ["waterbody", 2.2677422734268267], ["taita", 2.2677422734268267], ["morgana's", 2.2677422734268267], ["libert\u00e0", 2.2677422734268267], ["microbots", 2.2677422734268267], ["kaval", 2.2677422734268267], ["alila", 2.2677422734268267], ["aniela", 2.2677422734268267], ["edf's", 2.2677422734268267], ["cooperatives", 2.267579877560699], ["recordkeeping", 2.2675655891826954], ["notification", 2.267205159695493], ["enroll", 2.2669333791240396], ["kurta", 2.2667541504318693], ["biweekly", 2.266720636413441], ["confetti", 2.2666825807756923], ["craning", 2.2664596137699107], ["nyla", 2.2664596137699107], ["notice", 2.266417547702573], ["telemann", 2.266180252298967], ["embroiderer", 2.266180252298967], ["praise", 2.2661415141749957], ["bloodlines", 2.265948050093878], ["inventing", 2.2657413243620383], ["charlie", 2.2656649560458106], ["whisks", 2.2654223324821445], ["preproduction", 2.2654223324821445], ["surinamese", 2.2654223324821445], ["wuxia", 2.2654223324821445], ["alpha's", 2.2654223324821445], ["totp", 2.2654223324821445], ["mister's", 2.265422332482144], ["milanesa", 2.265422332482144], ["tutee", 2.265422332482144], ["aucune", 2.265422332482144], ["keflav\u00edk", 2.265422332482144], ["aviculture", 2.265422332482144], ["husain's", 2.265422332482144], ["rosee", 2.265422332482144], ["guix", 2.265422332482144], ["objeto", 2.265422332482144], ["farnaz", 2.265422332482144], ["azai", 2.265422332482144], ["eclectica", 2.265422332482144], ["ableton", 2.265251640624663], ["truman's", 2.265074341340442], ["peephole", 2.265074341340442], ["lutheranism", 2.265013706520297], ["eigenvectors", 2.265013706520297], ["themed", 2.264964070898295], ["slavic", 2.2647337542655843], ["roped", 2.2646244237992468], ["suresh", 2.2646244237992468], ["ajwain", 2.264566163800178], ["titus's", 2.264566163800178], ["councilor's", 2.264566163800178], ["lehrer's", 2.264566163800178], ["oenologist", 2.264566163800178], ["atsuta", 2.264566163800178], ["cyfarthfa", 2.264566163800178], ["abierta", 2.264566163800178], ["seaglass", 2.264566163800178], ["eggbert", 2.264566163800178], ["contenido", 2.264566163800178], ["priding", 2.264313054806795], ["endures", 2.264236247491316], ["autohotkey", 2.264191793463957], ["envying", 2.264073866808048], ["neela", 2.264073866808048], ["tulsa's", 2.2640161112950046], ["odaiba", 2.2640161112950046], ["effort", 2.2639993122154896], ["overthinking", 2.2638986424549166], ["reflexes", 2.263717375882712], ["taste", 2.26371578262718], ["binomial", 2.2632690938807922], ["kaena", 2.2632356313890147], ["torquay's", 2.2632356313890147], ["babbino", 2.2632356313890147], ["hazelton", 2.2632356313890147], ["pobres", 2.2632356313890147], ["akari's", 2.2632356313890147], ["jemma's", 2.2632356313890147], ["misdirections", 2.2632356313890147], ["janiculum", 2.2632356313890147], ["proptech", 2.2632356313890147], ["longyi", 2.2632356313890147], ["squishier", 2.2632356313890147], ["pronged", 2.2630966205400873], ["zigzagged", 2.263096620540087], ["fern", 2.2630733922756536], ["chrysalis", 2.262982897091819], ["sender", 2.26292517382641], ["medicinal", 2.2627015344848522], ["illogical", 2.2623088019826043], ["como's", 2.2622494720725057], ["naina's", 2.2622494720725057], ["\u010desk\u00e9", 2.2622494720725057], ["soutien", 2.2622494720725057], ["unclenching", 2.2622494720725057], ["erion", 2.2622494720725057], ["arae", 2.2622494720725057], ["swifty", 2.2622494720725053], ["orchestrations", 2.2622494720725053], ["roasted", 2.2621679348776205], ["saito", 2.2620129217016083], ["sunset", 2.26193103226412], ["textiles", 2.2615645860229163], ["rudra", 2.261551139159379], ["spiel", 2.26129136542802], ["bonfires", 2.2611114480659293], ["yoko's", 2.261024458820822], ["vidas", 2.261024458820822], ["roleplayers", 2.2609773109256928], ["minoans", 2.2609773109256928], ["coparenting", 2.260885438625371], ["urgent", 2.2607744295203647], ["surfers", 2.2606277878631342], ["venusian", 2.2604504976280264], ["detecting", 2.2604195655703903], ["crostini", 2.260394155392293], ["jardins", 2.260394155392293], ["wholeness", 2.260369533703328], ["oblivion", 2.2603639732070646], ["intake", 2.26033911109288], ["zeros", 2.260028469785758], ["onstage", 2.259973616944778], ["diverting", 2.2599385720741516], ["movesets", 2.2598079029096976], ["saimaa", 2.2598079029096976], ["pedestrianized", 2.2598079029096976], ["django's", 2.2598079029096976], ["gottlieb's", 2.2598079029096976], ["fatos", 2.2598079029096976], ["supplements", 2.259598059689442], ["communicator", 2.259448866118822], ["paz's", 2.2593497718305837], ["bayes", 2.259096197220951], ["watch's", 2.259096197220951], ["angler's", 2.2590961972209507], ["recovering", 2.258874869093468], ["subjects", 2.258853057206504], ["consensus", 2.258757059293039], ["cura\u00e7ao", 2.25872650292801], ["ottoman", 2.258644433074695], ["anne's", 2.2585071238970342], ["dialectal", 2.2585071238970342], ["geocentric", 2.2581497086218483], ["rivka", 2.258088571798462], ["steelseries", 2.258088571798462], ["vials", 2.257870924707204], ["pandy", 2.2572836639165916], ["dikembe", 2.2572836639165916], ["lifesaver", 2.257249813245196], ["romanian", 2.257217983433431], ["andrei", 2.257141166812987], ["kutch", 2.2570705450812882], ["collies", 2.2570705450812882], ["strategist", 2.2569166259224587], ["depiction", 2.256815104775145], ["vieja", 2.2566894119260907], ["flower's", 2.256432561321501], ["advisors", 2.2563020306863817], ["roomful", 2.256234983933355], ["brooding", 2.256139871097666], ["meowing", 2.255940326012993], ["skateboarding", 2.2559352548317633], ["gitanjali", 2.255615309397806], ["haulover", 2.255615309397806], ["unscrambled", 2.255615309397806], ["crosscurrents", 2.255615309397806], ["califa", 2.255615309397806], ["havasupai", 2.255615309397806], ["moray's", 2.255615309397806], ["reboot's", 2.255615309397806], ["arjumand", 2.255615309397806], ["disincentivize", 2.255615309397806], ["grainger's", 2.255615309397806], ["fattiness", 2.255615309397806], ["kennings", 2.255615309397806], ["fermentors", 2.255615309397806], ["eel's", 2.255615309397806], ["kovalenko", 2.255615309397806], ["madhu's", 2.255615309397806], ["babbitt's", 2.255615309397806], ["upgradability", 2.255615309397806], ["algea", 2.255615309397806], ["\u043e\u0440\u0433\u0430\u043d\u0438\u0437\u0430\u0446\u0438\u0438", 2.255615309397806], ["hongqi", 2.255615309397806], ["memling", 2.2556153093978057], ["sora's", 2.2556153093978057], ["caramelize", 2.2556153093978057], ["gummies", 2.2556153093978057], ["warburg's", 2.2556153093978057], ["zamindari", 2.2556153093978057], ["innerspring", 2.2556153093978057], ["serling's", 2.2556153093978057], ["hundertwasser", 2.2556153093978057], ["diversions", 2.2553378659403154], ["rajkumar", 2.254720223957569], ["sugars", 2.254298802407885], ["indentations", 2.254104852717406], ["dawa", 2.254104852717406], ["cinematographer", 2.2539808055504165], ["announcing", 2.2538193403930222], ["decolonizing", 2.25370052560375], ["misjudged", 2.25370052560375], ["nasm", 2.25370052560375], ["peaks", 2.2536519195535933], ["mezcal", 2.253473902458504], ["winery's", 2.2533641963145747], ["unadvertised", 2.2533641963145747], ["dhaba", 2.2533641963145747], ["luminary", 2.25331366112291], ["revolutionizes", 2.25331366112291], ["checkmated", 2.25331366112291], ["purple's", 2.25331366112291], ["ningaloo", 2.25331366112291], ["moma", 2.2532786913286786], ["tajikistan's", 2.2532608048994787], ["urbanite", 2.2532054639817827], ["spending", 2.2532036037502334], ["canopic", 2.253147458949887], ["pineapples", 2.2528098177194504], ["nurturers", 2.2526473945170196], ["merlion", 2.2526473945170196], ["recoleta", 2.2526473945170196], ["captured", 2.252502560270837], ["lumi\u00e8re", 2.2524158550582345], ["murugan", 2.2524158550582345], ["owasp", 2.252365143245936], ["prepped", 2.25228598790792], ["debriefing", 2.252197710444173], ["abbasids", 2.25195953093849], ["kohen", 2.25195953093849], ["chaplin's", 2.251180110755732], ["misdiagnosis", 2.251180110755732], ["retaliated", 2.251146974791234], ["emerald", 2.2511380830299057], ["tempting", 2.2510990326597495], ["calm", 2.251006573794454], ["suppress", 2.2509757505557184], ["maple", 2.2508797634443742], ["cycles", 2.2508443251946146], ["breezes", 2.2507600488917223], ["intrude", 2.250726979005922], ["varese", 2.2504359401454574], ["fresh's", 2.250086840502223], ["halophytes", 2.250086840502223], ["preguntas", 2.250086840502223], ["thabiso", 2.250086840502223], ["commodification", 2.249679479636233], ["boleyn's", 2.2494185640423177], ["krakens", 2.2494185640423177], ["oyun", 2.2494185640423177], ["buscema", 2.2494185640423177], ["ideea", 2.2494185640423177], ["underequipped", 2.2494185640423177], ["adopter", 2.2493826881481542], ["temptation", 2.2493640326831894], ["egyptologist", 2.249291154324728], ["homesteaders", 2.249066748822135], ["recruitments", 2.249019942996058], ["toyota's", 2.248800761635275], ["finery", 2.248566511555938], ["boughs", 2.248566511555938], ["willy's", 2.248566511555938], ["troublemaker", 2.2484117475314656], ["greeting", 2.248339890497974], ["fizzled", 2.248315583477425], ["segueing", 2.248315583477425], ["numbly", 2.248315583477425], ["discordia", 2.248315583477425], ["ossi", 2.248315583477425], ["snacking", 2.2481183220977843], ["chomsky's", 2.247534804195255], ["siciliana", 2.2474427901608576], ["alligators", 2.2474427901608576], ["malgudi", 2.2474427901608576], ["matador's", 2.2474427901608576], ["hammams", 2.2474427901608576], ["fortezza", 2.2474427901608576], ["margutta", 2.2474427901608576], ["rolanda", 2.2474427901608576], ["toytown", 2.2474427901608576], ["achillas", 2.2474427901608576], ["gunsmoke", 2.2473486564314267], ["frivolously", 2.2471537299949205], ["lighting", 2.2471191583990744], ["lambda", 2.247083199314432], ["soothe", 2.247026045802417], ["sorrows", 2.2469245562939495], ["pigments", 2.246843472083481], ["moonlight", 2.246775735888004], ["spice", 2.24630539316671], ["siempre", 2.2462169122753153], ["ensue", 2.246182231843129], ["skimmed", 2.2461629605122697], ["rivals", 2.2461472945188525], ["crucially", 2.2458688234930007], ["arose", 2.245534347121168], ["powered", 2.2453256485481936], ["jesse's", 2.2451780149070193], ["ev's", 2.2448742841149594], ["etymologies", 2.2448742841149594], ["canoe's", 2.2448742841149594], ["fatiha", 2.2448742841149594], ["organ's", 2.2448742841149594], ["d\u00edaz's", 2.2448742841149594], ["bunu", 2.2448742841149594], ["entrec\u00f4te", 2.2448742841149594], ["marionberry", 2.2448742841149594], ["springtail", 2.2448742841149594], ["vectorworks", 2.2448742841149594], ["terrafirma", 2.2448742841149594], ["blackbriar", 2.2448742841149594], ["nida's", 2.2448742841149594], ["stheno", 2.2448742841149594], ["kaiman", 2.2448742841149594], ["countermeasures", 2.2445697705115797], ["keycard", 2.2444856285948567], ["fetch", 2.2444381552360246], ["homer's", 2.2442805590993244], ["diverging", 2.2441833409681093], ["brilliant", 2.244172032856326], ["goa's", 2.2438468816966], ["cammy", 2.243780290799114], ["frankish", 2.243780290799114], ["anglicisms", 2.243617355624413], ["mckinsey's", 2.243617355624413], ["dashain", 2.243617355624413], ["vase", 2.2434271863282937], ["epic", 2.243415854275773], ["hawk's", 2.2432557734559007], ["fencer", 2.243186408713369], ["cursor", 2.243145241020647], ["sabi", 2.24302817039447], ["macgyver", 2.242918137126362], ["nuthatches", 2.2427113316552556], ["gametes", 2.2426021441512805], ["padma", 2.242495399320772], ["carrots", 2.2420657701270676], ["nymphenburg", 2.242027265365289], ["econ\u00f3mico", 2.242027265365289], ["montaner", 2.242027265365289], ["yeji", 2.242027265365289], ["flippant", 2.24198884159039], ["braque", 2.241614938511889], ["joon", 2.2416149385118884], ["fumes", 2.241556337263888], ["consumers", 2.241515412065795], ["haphazard", 2.2410629525629813], ["ruvo", 2.2410629525629813], ["charli's", 2.2410629525629813], ["pq's", 2.2410629525629813], ["normande", 2.2410629525629813], ["graph's", 2.2410629525629813], ["h\u00e9ireann", 2.2410629525629813], ["synthesia", 2.2410629525629813], ["mapungubwe", 2.2410629525629813], ["poblanos", 2.2410629525629813], ["cartagena's", 2.2410629525629813], ["spacecrafts", 2.240913906077709], ["tereshkova", 2.240913906077709], ["barcelona's", 2.240654595610984], ["unbearable", 2.240628848359095], ["scholarly", 2.240228385867617], ["quipu", 2.2399513141936547], ["lezama", 2.2399513141936547], ["humayun's", 2.2399513141936547], ["joyland", 2.2399513141936547], ["expenses", 2.239783841198201], ["anonymous", 2.2390944513208653], ["nneka", 2.238931764209952], ["blizzards", 2.2387996682737605], ["orbited", 2.2387630111092016], ["basil", 2.238609887490265], ["anthropomorphic", 2.238603685555979], ["scarcer", 2.238453019000214], ["philippe's", 2.2381749848406067], ["innovated", 2.2379242089319407], ["rochester's", 2.2379242089319407], ["mortals", 2.23786143405932], ["divisi\u00f3n", 2.2377702515702915], ["reposado", 2.2377702515702915], ["viajes", 2.2377702515702915], ["everglades", 2.2377241186358376], ["shucking", 2.237696870024085], ["hieroglyphic", 2.2374898292330023], ["mashhad", 2.2374898292330023], ["polynesian", 2.237434027306942], ["compiling", 2.237401215774419], ["knitted", 2.237337047407858], ["proxima", 2.237300484236029], ["ritual", 2.237194451037724], ["company's", 2.2371944510377237], ["advanced", 2.237097718997165], ["stettin", 2.2366377767466226], ["futures", 2.2364653961857943], ["layla", 2.2362663179070847], ["metoo", 2.2361495900907253], ["caverns", 2.235657247236119], ["graffiti", 2.2355569853377246], ["images", 2.2354873542792473], ["johnathan", 2.2354221818660545], ["canines", 2.235418436976098], ["contemporary", 2.2353074718873054], ["battleground", 2.2351798202699285], ["cit\u00e9", 2.23502834427235], ["rosy", 2.235007724503046], ["ruins", 2.2349495767536087], ["poly's", 2.2343931223470204], ["qigong", 2.2343931223470204], ["bargello", 2.2343931223470204], ["navigability", 2.2343931223470204], ["rebozo", 2.2343931223470204], ["caliphs", 2.2343931223470204], ["helpful", 2.234073763933706], ["homelands", 2.23398324451252], ["yikes", 2.233961402427587], ["bactrian", 2.2338569623618145], ["filmmakers", 2.2338249516249196], ["gargoyles", 2.2338087565577065], ["existentialist", 2.233059156303828], ["etosha", 2.233059156303828], ["erode", 2.233059156303828], ["sicherheit", 2.233059156303828], ["invulnerability", 2.233059156303828], ["warby", 2.233059156303828], ["casa's", 2.233059156303828], ["galvanization", 2.233059156303828], ["medtech", 2.233059156303828], ["qiong", 2.233059156303828], ["tide", 2.232837622657369], ["haughtiness", 2.2326379038691355], ["proceso", 2.2326379038691355], ["perovskites", 2.232421321640502], ["hurling", 2.2321601791877637], ["enron", 2.2319835016042595], ["jutting", 2.231812333770995], ["straws", 2.231665236730605], ["fragmented", 2.2313200136587317], ["grady's", 2.2312741369942324], ["scatting", 2.2312741369942324], ["moons", 2.23099150893688], ["nouveau", 2.230938492337727], ["alors", 2.230882685391251], ["unfounded", 2.230869882621177], ["aftermath", 2.2307154754093026], ["wormholes", 2.2305944110981946], ["eruptions", 2.230544450046729], ["permeates", 2.2305273432014427], ["podgorica", 2.2304351149920723], ["elwood's", 2.2302713171573814], ["construcci\u00f3n", 2.2302713171573814], ["nopales", 2.2302713171573814], ["gl\u00f3ria", 2.2302713171573814], ["saucisson", 2.2302713171573814], ["huascar", 2.2302713171573814], ["paperbark", 2.2302713171573814], ["calvary's", 2.2302713171573814], ["arranged", 2.230018498411257], ["sich", 2.2299512103660355], ["seize", 2.2299134487408008], ["correspondence", 2.2299051179474665], ["noah's", 2.229822838685996], ["homemakers", 2.229706214627696], ["mariachi", 2.229602408383853], ["virtuality", 2.229527231975945], ["illuminates", 2.2295202352954226], ["turnarounds", 2.229463247839571], ["outstretched", 2.2293969026649534], ["medell\u00edn", 2.2291050896259987], ["watered", 2.229093805189133], ["alighting", 2.2290247224171003], ["earned", 2.229005402592854], ["ungraceful", 2.2288537718286796], ["drum's", 2.2288537718286796], ["somaliland's", 2.2288537718286796], ["kov\u00e1cs", 2.2288537718286796], ["chemotherapies", 2.2288537718286796], ["kubert", 2.2288537718286796], ["fiascos", 2.2286019523990297], ["netsuke", 2.2286019523990297], ["diesen", 2.2286019523990297], ["petra", 2.2283902751526554], ["seine", 2.22802974378963], ["agamemnon's", 2.227995530099057], ["nimbler", 2.227995530099057], ["suffered", 2.227961874577455], ["arced", 2.2278709636487557], ["kernels", 2.2278629744392466], ["earthporn", 2.2276819928727556], ["wilhelm", 2.227585972097201], ["tameka", 2.2275758925682756], ["pakeha", 2.2275758925682756], ["orographic", 2.2273612554178754], ["flawed", 2.2269152898420397], ["email", 2.2267137666808354], ["damien's", 2.2266971644055267], ["gibbous", 2.2266971644055267], ["travelmate", 2.2266971644055267], ["deboning", 2.2266971644055267], ["selfimprovement", 2.2266971644055267], ["macrocosmic", 2.2266971644055267], ["danielewski", 2.2266971644055267], ["regrew", 2.2266971644055267], ["xico", 2.2266971644055267], ["phaon", 2.2266971644055267], ["impetuously", 2.2266971644055262], ["muholi", 2.2266971644055262], ["casio's", 2.2266971644055262], ["rsc's", 2.2266971644055262], ["sylar's", 2.2266971644055262], ["magicband", 2.2266971644055262], ["elija", 2.2266971644055262], ["orpingtons", 2.2266971644055262], ["prometaphase", 2.2266971644055262], ["vivobook", 2.2266971644055262], ["furney", 2.2266971644055262], ["metatarsalgia", 2.2266971644055262], ["grimaldis", 2.2266971644055262], ["dagget", 2.2266971644055262], ["autotune", 2.2261025857234418], ["kazimir", 2.2260011040227803], ["baronets", 2.2260011040227803], ["piave", 2.2260011040227803], ["targeting", 2.225567373623177], ["parks", 2.2253489572439005], ["hamstrings", 2.2251340214329707], ["yaro", 2.2251340214329707], ["nonny", 2.2251340214329707], ["shibori", 2.2251340214329707], ["brave", 2.224885185871875], ["bronchodilators", 2.224836932856315], ["karter", 2.224836932856315], ["unfccc", 2.2245033740957676], ["bahia's", 2.224503374095767], ["espartero", 2.224503374095767], ["mesopotamian", 2.2242264879401668], ["things", 2.224196843753204], ["rowdiest", 2.224162506278713], ["sidequest", 2.224162506278713], ["bouazizi", 2.224162506278713], ["string's", 2.224162506278713], ["headstands", 2.224162506278713], ["smrs", 2.224162506278713], ["weightlifter", 2.224036695066237], ["muddy", 2.224019692940789], ["newer", 2.2236586011108153], ["saving", 2.223651095019772], ["fanatic", 2.2235723036790076], ["halftime", 2.223180934691243], ["hobart's", 2.2230197124411903], ["alvaro", 2.2230197124411903], ["bartender's", 2.2230197124411903], ["obliviousness", 2.2230197124411903], ["quakerism", 2.2229252324500117], ["visit", 2.2227509943398798], ["hiding", 2.2227308125186904], ["jogging", 2.2227209194690882], ["rivia", 2.222720919469088], ["neurorehabilitation", 2.222720919469088], ["pulse's", 2.222720919469088], ["brunhilda", 2.222720919469088], ["mosasaurs", 2.222720919469088], ["sagada", 2.222720919469088], ["oatcake", 2.222720919469088], ["humanizes", 2.222188020663174], ["houseboats", 2.2221389307015236], ["determiners", 2.2219494092575403], ["hawksbill", 2.22194940925754], ["architecture's", 2.22194940925754], ["clockmakers", 2.22194940925754], ["apprentice", 2.2219161465418327], ["monolithic", 2.221893860522309], ["inquired", 2.221889871492019], ["cognitive", 2.2217111381968575], ["housefly", 2.221655966387855], ["falconers", 2.221655966387855], ["limits", 2.221620510531424], ["niseko", 2.2214393198614757], ["sagebrush", 2.2214393198614757], ["maldivian", 2.221323860437366], ["propose", 2.2212500751292428], ["deckhand", 2.2212440284926966], ["multilateralism", 2.2212440284926966], ["possessing", 2.2211714539975422], ["zagreb's", 2.2211408396035166], ["czechoslovakia's", 2.2211408396035166], ["dadjokes", 2.2211408396035166], ["krio", 2.2211408396035166], ["qilin", 2.2211408396035166], ["sonoran", 2.221140839603516], ["advised", 2.221064062969746], ["ignore", 2.2208586675359427], ["jenolan", 2.2208064311663582], ["uuids", 2.2208064311663582], ["moonlight's", 2.2208064311663582], ["muestra", 2.2208064311663582], ["synthesist", 2.2208064311663582], ["bertalan", 2.2208064311663582], ["gilley's", 2.2208064311663582], ["b\u00e1thory", 2.2208064311663582], ["monarch's", 2.2206532721021404], ["hamsa", 2.2206532721021404], ["bottle's", 2.2205798645033137], ["workhorses", 2.2205601947287086], ["harlem's", 2.2205601947287086], ["wegener's", 2.2205084563332487], ["longshanks", 2.2205084563332487], ["impostors", 2.2203054435731207], ["mannerisms", 2.2202966733126814], ["neuromorphic", 2.220292474575021], ["presenter's", 2.220292474575021], ["partnered", 2.2202507212873357], ["jalape\u00f1o", 2.2201884983424223], ["tribulation", 2.2201219213821424], ["goliath's", 2.220000330828367], ["gerizim", 2.220000330828367], ["pilau", 2.220000330828367], ["alarm's", 2.220000330828367], ["noctilucent", 2.220000330828367], ["celled", 2.2200003308283667], ["ayurvedic", 2.2200003308283667], ["painkillers", 2.2199014003323407], ["corporations", 2.21989886022501], ["reduced", 2.2197448320690647], ["prosthetics", 2.2196898991609633], ["cushioned", 2.2193740082635967], ["strollers", 2.2193441084426655], ["hue's", 2.2189386377002807], ["angklung", 2.2189386377002807], ["waiwai", 2.2189386377002807], ["transcriptionists", 2.2189386377002807], ["ilulissat", 2.2189386377002807], ["atalaya", 2.2189386377002807], ["wernigerode", 2.2189386377002807], ["muresan", 2.2189386377002807], ["melaina", 2.2189386377002807], ["crosspost", 2.2189386377002807], ["plakat", 2.2189386377002807], ["ambrosio's", 2.2189386377002807], ["keeping", 2.218717093642917], ["sean's", 2.218340310640808], ["merengue", 2.218270685070028], ["iii's", 2.218270685070028], ["cosplays", 2.218270685070028], ["cremonese", 2.218270685070028], ["fridrich", 2.218270685070028], ["iver's", 2.218270685070028], ["miracoli", 2.218270685070028], ["skene's", 2.218270685070028], ["kraken's", 2.218270685070028], ["packages", 2.2180991194723805], ["fruity", 2.217381390554442], ["fogging", 2.21737355558905], ["tarragona", 2.21722185732295], ["leapers", 2.217021131100125], ["willibald", 2.217021131100125], ["ofcom's", 2.217021131100125], ["tucano", 2.217021131100125], ["auctions", 2.2167819323512585], ["preamble", 2.2166519140397876], ["monic", 2.2166133155753656], ["minangkabau", 2.2166133155753656], ["quintet", 2.2165880671094964], ["sanctum", 2.2165430018446974], ["missive", 2.216093844575024], ["colons", 2.2159475918877067], ["conrail", 2.2159475918877067], ["famagusta", 2.215857107704543], ["valora", 2.215336464587131], ["dhokla", 2.215336464587131], ["estaci\u00f3n", 2.215336464587131], ["putdowns", 2.215336464587131], ["kipa", 2.215336464587131], ["troponins", 2.215336464587131], ["homeliness", 2.215336464587131], ["newbie's", 2.215336464587131], ["guiliana", 2.215336464587131], ["lusciousness", 2.215336464587131], ["rowdier", 2.215336464587131], ["screwdrivers", 2.2153364645871307], ["aave", 2.2153364645871307], ["kimono", 2.2152370106493264], ["manchester's", 2.2150687502107456], ["morocco's", 2.2149318369223665], ["eldritch", 2.2149318369223665], ["erase", 2.214871098502666], ["vegetable", 2.214827776535561], ["dominance", 2.214707256597189], ["busybody", 2.2146800685976236], ["oaktree", 2.214666166110705], ["alphabetized", 2.214508301422799], ["accurate", 2.214450330001296], ["quebecois", 2.2143836316991363], ["splotch", 2.2143225806216447], ["pigeonhole", 2.2142147752379726], ["accumulating", 2.2140854510541876], ["arabesque", 2.214064501066794], ["volc\u00e1n", 2.2136893742788875], ["haad", 2.2136893742788875], ["blackwoods", 2.2136893742788875], ["zakarian", 2.2136893742788875], ["creditworthiness", 2.213598945791376], ["unbox", 2.213598945791376], ["santorini", 2.213423943898279], ["iteration", 2.213311014105223], ["lilt", 2.21314847795544], ["dendrochronology", 2.2131099957785008], ["willemstad", 2.2131099957785008], ["singhs", 2.2131099957785008], ["affection", 2.213078525498225], ["knocked", 2.2129530681148855], ["trapped", 2.212729513805578], ["manche", 2.212611573486778], ["caber", 2.212611573486778], ["dumpsters", 2.2120914646953413], ["deconstructed", 2.2120914646953413], ["warten", 2.2119018344094763], ["squiddy", 2.2119018344094763], ["mudbugs", 2.2119018344094763], ["fondants", 2.2119018344094763], ["asistencia", 2.2119018344094763], ["viande", 2.2119018344094763], ["kaar", 2.2119018344094763], ["quintessa", 2.2119018344094763], ["dianabol", 2.2119018344094763], ["machine's", 2.2118676103365322], ["substances", 2.2116294855911587], ["leeuwenhoek", 2.2115752821345214], ["warzone", 2.2114837055182837], ["divulged", 2.211320748941354], ["voller", 2.2113207489413536], ["matteo's", 2.2113207489413536], ["chemex", 2.2113207489413536], ["turabi", 2.2113207489413536], ["photographs", 2.2109098379346044], ["lookups", 2.21050300320985], ["highfalutin", 2.21050300320985], ["arbor's", 2.21050300320985], ["tikki", 2.210440693394673], ["scarfing", 2.210440693394673], ["workaround", 2.21032712677247], ["maang", 2.210243737082333], ["interviewee's", 2.210243737082333], ["sideloading", 2.210243737082333], ["fullwood", 2.210243737082333], ["covadonga", 2.210243737082333], ["rinascimento", 2.210243737082333], ["pushiness", 2.210243737082333], ["reversi", 2.210243737082333], ["masego", 2.210243737082333], ["vlogging", 2.2100548677179166], ["skimming", 2.209950156320292], ["siphoning", 2.209798123425663], ["exquisitely", 2.2097981234256627], ["ventriloquism", 2.2094289006815653], ["prynne", 2.2094289006815653], ["drones", 2.2093995098163632], ["monotheistic", 2.2093195013499805], ["ifeoma", 2.2092670496156592], ["kesel", 2.2092670496156592], ["codepen", 2.2092670496156592], ["sibelius's", 2.2092670496156592], ["longbowmen", 2.2092670496156592], ["yousafzai", 2.208711159272478], ["school's", 2.2086776497802796], ["dejection", 2.2084233578891284], ["ramone's", 2.208377292447067], ["awkwafina", 2.208377292447067], ["kbbq", 2.208377292447067], ["hualalai", 2.208377292447067], ["capri's", 2.208377292447067], ["harika", 2.208377292447067], ["rowena's", 2.208377292447067], ["sum's", 2.208377292447067], ["misguidedly", 2.2081670909153277], ["begawan", 2.2081670909153277], ["gurbaksh", 2.2081670909153277], ["idealized", 2.2077061438135814], ["unrequited", 2.207547885308649], ["dna's", 2.2074527049266783], ["starters", 2.2071846864604563], ["protests", 2.207069614083836], ["lovelace", 2.207065875119339], ["tagalog", 2.2068848919476656], ["publicly", 2.206818437887692], ["plasterwork", 2.206815939723334], ["fort's", 2.206815939723334], ["suppleness", 2.206815939723334], ["bassoon", 2.206710375698473], ["tanya", 2.206696652375241], ["reach", 2.2066094123743967], ["swings", 2.206455728475263], ["exploited", 2.2063826203377768], ["brad's", 2.2063629497938666], ["esha", 2.2063629497938666], ["cooperative", 2.206328245593008], ["unfollowed", 2.205490524744522], ["quacking", 2.2054905247445213], ["dam's", 2.2054905247445213], ["bacolod", 2.2054905247445213], ["grain's", 2.2054905247445213], ["vemos", 2.2054905247445213], ["cauchy's", 2.2054905247445213], ["andam", 2.2054905247445213], ["genti", 2.2054905247445213], ["calangute", 2.2054905247445213], ["qinling", 2.2054905247445213], ["yuxi", 2.2054905247445213], ["ruchika", 2.2054905247445213], ["glossy", 2.2053880391253418], ["controversies", 2.205380689559026], ["interestingly", 2.2053173549482947], ["circleville", 2.2050921341150516], ["comic", 2.205086953522075], ["ciara", 2.2050110702826204], ["jahangir", 2.2048885895576373], ["folkloric", 2.2046601442758678], ["simplistic", 2.2045671734482766], ["fungible", 2.204559152056707], ["automatons", 2.204559152056707], ["rustin", 2.204559152056707], ["consoling", 2.2044988473503015], ["sunken", 2.204430192761471], ["frankenstein's", 2.204024108172218], ["hula", 2.2038688405351503], ["organizations", 2.2038470101707937], ["manera", 2.203839708483485], ["annatto", 2.203839708483485], ["validator", 2.203676798984041], ["grids", 2.203652615973901], ["aber", 2.2036214649777888], ["bosch's", 2.2030600408539023], ["smalltown", 2.2030600408539023], ["slipperier", 2.2029140171221564], ["vecinos", 2.2029140171221564], ["f\u00e9minin", 2.2029140171221564], ["spacewar", 2.2029140171221564], ["pendulum's", 2.2029140171221564], ["mexicanum", 2.2029140171221564], ["gaboon", 2.2029140171221564], ["kumis", 2.2029140171221564], ["helicases", 2.2029140171221564], ["exhaustedly", 2.2029140171221564], ["boora", 2.2029140171221564], ["preterist", 2.2029140171221564], ["satyabhama", 2.2029140171221564], ["acclimate", 2.202914017122156], ["evergreens", 2.202914017122156], ["mochas", 2.2026773990752044], ["seljuks", 2.2026773990752044], ["baptist's", 2.2026773990752044], ["sullying", 2.2026773990752044], ["bags", 2.2024825718216006], ["volcanoes", 2.202469304847611], ["brno", 2.202437037071813], ["manuscript", 2.202342664169841], ["factual", 2.2022730554593446], ["steganography", 2.2022279647966743], ["rosewater", 2.2020444457996087], ["thefts", 2.2020088376359985], ["crackly", 2.2019952941189356], ["conglomerates", 2.201873093807872], ["renegotiating", 2.2018390503658054], ["global", 2.201828630754573], ["oystercatchers", 2.201607618891098], ["bumbles", 2.201607618891098], ["downvote", 2.201607618891098], ["prefabrication", 2.2015238151676426], ["kitzb\u00fchel", 2.2015238151676426], ["harpreet", 2.2015238151676426], ["fairgrounds", 2.201375803616267], ["stigmatization", 2.201043971267214], ["outrageousness", 2.201043971267214], ["schopenhauer's", 2.201043971267214], ["runestones", 2.201043971267214], ["beardie", 2.201043971267214], ["alb\u00e1n", 2.201043971267214], ["youssef's", 2.201043971267214], ["cetacean", 2.2010439712672136], ["lyre", 2.2007489910611024], ["destructiveness", 2.2006959801255115], ["zimmer's", 2.2006959801255115], ["sorbets", 2.200653922716573], ["tempest", 2.20061299762249], ["pinetree", 2.2005295884742093], ["sped", 2.2003403408268487], ["virtuous", 2.2002200510640657], ["castling", 2.2001888648292702], ["terrified", 2.1998723478194044], ["collaborators", 2.199818509639018], ["gemacht", 2.199624858455307], ["volkov's", 2.199624858455307], ["permute", 2.199624858455307], ["oymyakon", 2.199624858455307], ["mimouna", 2.199624858455307], ["juscelino", 2.199624858455307], ["zoomies", 2.199624858455307], ["nyongo", 2.199624858455307], ["mediator's", 2.199624858455307], ["tombaugh", 2.199624858455307], ["foulsham", 2.199624858455307], ["emcee", 2.199329936686576], ["aleksandra", 2.199224926662861], ["varieties", 2.1989464556370097], ["campers", 2.1989081267598554], ["hoarder", 2.198854749095673], ["frothed", 2.198854749095673], ["imitate", 2.198704400053], ["curtsying", 2.19851112434976], ["skeptic's", 2.19851112434976], ["xenakis", 2.19851112434976], ["wildlife's", 2.19851112434976], ["pitha", 2.19851112434976], ["harmfully", 2.19851112434976], ["tantos", 2.19851112434976], ["utopic", 2.19851112434976], ["nathalie's", 2.19851112434976], ["instanceof", 2.19851112434976], ["skyguard", 2.19851112434976], ["shoba", 2.19851112434976], ["klee's", 2.19851112434976], ["discredit", 2.198387689539285], ["precedence", 2.1982763034018995], ["amazed", 2.1982677467345186], ["earthly", 2.1980377607105464], ["vindicated", 2.198012850882478], ["asakura", 2.197613772870434], ["shoegaze", 2.1971867200579833], ["synth", 2.1970599387542866], ["literal", 2.197042073137637], ["climber", 2.1970150332854104], ["humidifier", 2.196875327382238], ["d\u00f6ner", 2.196875327382238], ["grazing", 2.1965240280723197], ["recharge", 2.196484039533911], ["halten", 2.1962570117820746], ["dustier", 2.1962570117820746], ["benu", 2.1962570117820746], ["trouv\u00e9", 2.1962570117820746], ["lenition", 2.1962570117820746], ["ambidexterity", 2.1962570117820746], ["java", 2.195841503698764], ["kaiserslautern", 2.195841503698764], ["platen", 2.195841503698764], ["impossibilities", 2.195841503698764], ["fixes", 2.1957458284992653], ["obscenities", 2.1957317171129085], ["blackcaps", 2.1957317171129085], ["sammies", 2.1957317171129085], ["cbgb's", 2.1957317171129085], ["wickremesinghe", 2.1957317171129085], ["mubarak's", 2.1953868377856134], ["spotify's", 2.1953868377856134], ["calamitous", 2.1952419628265636], ["sparklers", 2.1948872049140187], ["desolation", 2.1947980181117064], ["over", 2.194662944699604], ["reverends", 2.1946527334681356], ["clearing", 2.1943753537476556], ["complementary", 2.194098528232411], ["khabarovsk", 2.1939666594624962], ["shaky", 2.1939058338233233], ["emulators", 2.193623233013694], ["translated", 2.19355389384597], ["guinea's", 2.1935041631969967], ["brochure", 2.1932967069394436], ["sumitra", 2.1932129965197293], ["labradors", 2.1931830999412596], ["counter", 2.1931074245358575], ["garden", 2.1929137992776524], ["elise", 2.1928422789467077], ["curled", 2.1928391664262095], ["needles", 2.1926966987214898], ["keyboard", 2.1926679054146114], ["multigenerational", 2.192667905414611], ["penalizing", 2.1925501466656634], ["speedruns", 2.192538818642834], ["lotions", 2.192538818642834], ["interventions", 2.1924580807346676], ["trust", 2.192404614297704], ["skates", 2.1923424081546985], ["sundarbans", 2.19190502121169], ["heute", 2.19190502121169], ["fedoras", 2.191706208964868], ["satins", 2.191639186359781], ["dancefloor", 2.1914585865257123], ["lampposts", 2.1913197328215133], ["theo", 2.191253769716083], ["open", 2.1911297324473282], ["shakily", 2.1908263590214863], ["dolmen", 2.1903044141398187], ["vibrance", 2.1903044141398187], ["wort", 2.1902351555022177], ["samoyed", 2.1902351555022177], ["isto", 2.1902140923083078], ["bodhisattva", 2.1901746183226845], ["noncompliant", 2.1900738557804353], ["financial", 2.190010738123971], ["lectionary", 2.1898265295403703], ["videoconferencing", 2.189713295840099], ["chopstick", 2.189713295840099], ["cr\u00e8me", 2.189628508207157], ["wong's", 2.1895644228477105], ["peddling", 2.189273682650812], ["orania", 2.189273682650812], ["teashops", 2.189273682650812], ["fftf", 2.189273682650812], ["budu", 2.189273682650812], ["animatics", 2.189273682650812], ["massimo's", 2.189273682650812], ["setelah", 2.189273682650812], ["boukman", 2.189273682650812], ["talkbox", 2.189273682650812], ["yemisi", 2.189273682650812], ["typecasting", 2.1892736826508115], ["aeson", 2.1892736826508115], ["lakotas", 2.1892736826508115], ["decisis", 2.1892736826508115], ["ranchera", 2.1892736826508115], ["eigentlich", 2.1892736826508115], ["hirohito's", 2.1892736826508115], ["kivy", 2.1892736826508115], ["geometry", 2.1890910028090445], ["glory", 2.188961384512675], ["temperatures", 2.1889429045398816], ["retell", 2.1888542815621816], ["coughing", 2.188690889888069], ["near", 2.1885124890319205], ["kitchen", 2.1884312775648698], ["downfall", 2.188369371715615], ["cassava", 2.1882328235306927], ["draconic", 2.1879468501158716], ["releasing", 2.1876608119425787], ["inhibitions", 2.1875464450313635], ["interweave", 2.1874355015570663], ["kilmister", 2.1873532671397147], ["precarity", 2.1873532671397147], ["vibes", 2.1872796962919656], ["halva", 2.187263330325145], ["mindhunter", 2.187263330325145], ["subtrees", 2.187263330325145], ["almir", 2.187263330325145], ["starlink", 2.187164555788913], ["ponderously", 2.1867999270771947], ["daniel's", 2.186663166606458], ["ihre", 2.186607900388832], ["photography", 2.1864271093045753], ["tweezers", 2.1863001360529837], ["spoiler", 2.1862313117115413], ["sudo", 2.1860182571004017], ["misprints", 2.1859975529349107], ["gatz", 2.1859338295880337], ["magically", 2.185927631170778], ["inflating", 2.18560715190191], ["improbability", 2.185407415661583], ["luego", 2.185378177877412], ["exup\u00e9ry", 2.185378177877412], ["passwords", 2.185369870259705], ["gender", 2.1852215565543887], ["stained", 2.18480288064086], ["predict", 2.184779590285592], ["aalborg", 2.1845143920363537], ["readmissions", 2.1842838850835165], ["meerkat", 2.18407799537807], ["tikkun", 2.1838760650900357], ["unrestored", 2.1838760650900357], ["gibt", 2.1838760650900357], ["anemone", 2.183845731371512], ["insomniac", 2.183673622071689], ["unmanageable", 2.18333202129915], ["arcs", 2.1832992910992552], ["devon's", 2.183050940917179], ["magnum", 2.1829292568765992], ["augustus's", 2.182516665111766], ["chers", 2.182516665111766], ["careened", 2.1823963831241597], ["estoril", 2.1823963831241597], ["massaging", 2.182334621850017], ["voiceovers", 2.182239871622215], ["reap", 2.1821913715240453], ["photocopied", 2.182163221117416], ["palabras", 2.1820927931628225], ["maestra", 2.1820927931628225], ["craftsmen", 2.1820222909171374], ["podcaster", 2.181724463055464], ["resonant", 2.181597710726456], ["teaches", 2.1815420165212855], ["unmasking", 2.1814064042373205], ["ninguna", 2.1812543651319443], ["sansevieria", 2.1812543651319443], ["homegoing", 2.1812543651319443], ["dunedin's", 2.1812543651319443], ["heatless", 2.1812543651319443], ["boccaccio's", 2.1812543651319443], ["displays", 2.181208922332671], ["husband's", 2.181017523423866], ["prepositions", 2.180861487904795], ["hirta", 2.180861487904795], ["ridership", 2.1807420663509895], ["modern", 2.180477293264366], ["cracks", 2.1797794773016164], ["quincea\u00f1era", 2.17972544852087], ["percussion", 2.1795123911597902], ["alphazero", 2.1794120894857176], ["aayush", 2.1794120894857176], ["yorick's", 2.1794120894857176], ["vulnicura", 2.1794120894857176], ["vacaciones", 2.1794120894857176], ["hessa", 2.1794120894857176], ["stefi", 2.1794120894857176], ["lior", 2.1792196129256642], ["khalil", 2.179120529339967], ["p\u00e9tanque", 2.1790774645679405], ["yamamoto's", 2.1790774645679405], ["devin's", 2.1790774645679405], ["steinberg's", 2.1790774645679405], ["sourpuss", 2.1790774645679405], ["retaliations", 2.1790774645679405], ["applauding", 2.1790409151304266], ["errant", 2.1790409151304266], ["helvellyn", 2.179019473364391], ["dispensers", 2.17866199067674], ["obsessive", 2.1786392925194717], ["artist", 2.17863841546346], ["maclean's", 2.1785942988330027], ["inciting", 2.178573854066685], ["girona", 2.178449522497355], ["readings", 2.178389158126898], ["dissimilar", 2.1783273142375577], ["propagating", 2.1780607779789647], ["vandalism", 2.1779863058757742], ["cathedral", 2.177696152087493], ["secretiveness", 2.1773592680513514], ["manures", 2.1773592680513514], ["numerologist", 2.1773592680513514], ["carbon's", 2.1773592680513514], ["lembongan", 2.1773592680513514], ["kinderman", 2.1773592680513514], ["waggles", 2.1773592680513514], ["gusto", 2.1772326773962325], ["bruna", 2.177191598309472], ["ann\u00e9es", 2.1771915983094714], ["prosperously", 2.1771915983094714], ["biren", 2.1771915983094714], ["furiosa", 2.1771294514308623], ["zines", 2.1771110510805296], ["thymine", 2.1771110510805296], ["epitomized", 2.177063725665859], ["shock", 2.176758841541307], ["femininity", 2.176668773568883], ["entendres", 2.1766687735688826], ["dad's", 2.17663302689203], ["procrastinate", 2.1764709125768307], ["pip's", 2.1764709125768307], ["ncds", 2.1764709125768307], ["veja", 2.1764709125768302], ["navigational", 2.1764263584128063], ["dependencies", 2.176232202863838], ["knocking", 2.1761784224146656], ["dadaist", 2.176170295047469], ["emme", 2.175952705216693], ["spirituals", 2.1758012292191142], ["sensors", 2.1756218075909604], ["island's", 2.1756155679769464], ["saudade", 2.175562156454669], ["enoki", 2.175562156454669], ["interviews", 2.1752553203073166], ["sealing", 2.175137643971444], ["bubbly", 2.1749556480174417], ["skylights", 2.1749556480174412], ["bobbie's", 2.1747366728058526], ["ephraim's", 2.1747366728058526], ["wagyu", 2.174437945240907], ["tone", 2.1744022086706045], ["crosswalks", 2.1743646504033687], ["fade", 2.1741140711466365], ["caressing", 2.173983517248171], ["mladic", 2.173983517248171], ["lactobacilli", 2.173983517248171], ["eleven's", 2.173983517248171], ["seasoning", 2.1739525875942154], ["fascial", 2.1738901170283205], ["charrington", 2.173680565253014], ["vandelay", 2.173680565253014], ["pareidolia", 2.173680565253014], ["datasheets", 2.173680565253014], ["fela's", 2.173680565253014], ["warrington's", 2.173680565253014], ["unversed", 2.173680565253014], ["reselling", 2.173592934510613], ["toys", 2.1735387905271906], ["bongo", 2.1731224957882507], ["maliciously", 2.1727819788266998], ["winemaker", 2.172295641846581], ["silvery", 2.172270323715557], ["tickled", 2.1722459984524165], ["aqua's", 2.1722365333694826], ["ollantaytambo", 2.1722365333694826], ["ventus", 2.1721230858419527], ["animated", 2.1720900672784387], ["humoring", 2.17181096226023], ["busch's", 2.17181096226023], ["youngsville", 2.171773493341053], ["differentiators", 2.171773493341053], ["stephanos", 2.171773493341053], ["compensators", 2.171708394168566], ["ajahn", 2.1716072904258206], ["badassery", 2.171532404833544], ["teach", 2.1713478348903035], ["barnyard", 2.1710297352953885], ["skidded", 2.1710297352953885], ["rosehip", 2.1710297352953885], ["reenacted", 2.1710297352953885], ["mehtab", 2.1710297352953885], ["salzburger", 2.1710297352953885], ["monteverdi", 2.1710297352953885], ["odisha's", 2.1710297352953885], ["shindigs", 2.1710297352953885], ["troopships", 2.1710297352953885], ["mekonnen", 2.1710297352953885], ["hagop", 2.1710297352953885], ["shanaya", 2.1710297352953885], ["kristine's", 2.1710297352953885], ["bisporus", 2.1710297352953885], ["cocoa's", 2.1710297352953885], ["adrena", 2.1710297352953885], ["traversal", 2.171029735295388], ["resistant", 2.1708392662441853], ["dryness", 2.1706652851132002], ["disposing", 2.1704389789048317], ["hope's", 2.1703358938970836], ["ebike", 2.17006952753984], ["bossa", 2.169978389176601], ["deflating", 2.169901927640689], ["destress", 2.169450038665292], ["dahlias", 2.1694059284627065], ["admirers", 2.1693532644573224], ["matriarchs", 2.1692369774627744], ["belched", 2.1692369774627744], ["grove's", 2.1692369774627744], ["tamara", 2.1689757809573607], ["imitating", 2.1686741876621567], ["tiny's", 2.1686378553079164], ["flus", 2.1686378553079164], ["picnic", 2.168178937816395], ["circulatory", 2.1680998706053813], ["vermentino", 2.168018598353231], ["piaget's", 2.168018598353231], ["zakes", 2.168018598353231], ["bookish", 2.167694820187408], ["muromachi", 2.167694820187408], ["offending", 2.1675708456480773], ["unhurriedly", 2.1675708456480773], ["milovan", 2.1675708456480773], ["misidentify", 2.1675708456480773], ["chagall's", 2.1675708456480773], ["millipedes", 2.167373124196284], ["autocompletion", 2.167293023186446], ["igloolik", 2.167293023186446], ["overeating", 2.1672320156418103], ["hypnotherapy", 2.1672320156418103], ["restorers", 2.167200758689929], ["resort's", 2.1671038225733352], ["unbefitting", 2.1669666727839183], ["puttered", 2.1669666727839183], ["metalsmith", 2.1669666727839183], ["woodhouse's", 2.1669666727839183], ["jannet", 2.1669666727839183], ["affogato", 2.1669666727839183], ["pharoahs", 2.1669666727839183], ["trei", 2.1669666727839183], ["chartbuster", 2.1669666727839183], ["idun", 2.1669666727839183], ["ounces", 2.166918275113273], ["conservancies", 2.166661466210287], ["step's", 2.166661466210287], ["schicksal", 2.1661067653740833], ["kabelo", 2.1661067653740833], ["winterlude", 2.1661067653740833], ["karpas", 2.1661067653740833], ["kallax", 2.1661067653740833], ["urgente", 2.1661067653740833], ["ulpan", 2.1661067653740833], ["bannion", 2.1661067653740833], ["chaminda", 2.1661067653740833], ["endearing", 2.1659754861761824], ["inventors", 2.1657983383428694], ["kindergartener", 2.1655241653833963], ["somnus", 2.1653906970218935], ["pode", 2.1653906970218935], ["genel", 2.1653906970218935], ["fin's", 2.1653906970218935], ["dhabi's", 2.1653906970218935], ["zadar", 2.1653906970218935], ["goldcrest", 2.1653906970218935], ["papermakers", 2.1653906970218935], ["sapphira", 2.1653906970218935], ["volunteer", 2.1651190784244267], ["grayscale", 2.164948419510247], ["sirena", 2.164948419510247], ["hopps", 2.164948419510247], ["popsicles", 2.164867872281305], ["nahm", 2.164785162710646], ["plymouth's", 2.164785162710646], ["paranoia", 2.164661695876271], ["photosynthetic", 2.164492676875295], ["willow", 2.1644470212291864], ["doubled", 2.163995471057318], ["personally", 2.1638596678132296], ["background", 2.1636367550665385], ["tally", 2.163443403229608], ["assonance", 2.1633036508281447], ["villefranche", 2.1633036508281447], ["materiales", 2.1630772454225116], ["moonshining", 2.1630772454225116], ["cidery", 2.1630772454225116], ["polygon's", 2.1630772454225116], ["masa", 2.163077245422511], ["aleksander", 2.163077245422511], ["mai's", 2.162867969373225], ["eine", 2.1624485900965924], ["shadowing", 2.162065946132319], ["eagle's", 2.1620097971174275], ["wallachia", 2.1616313381728975], ["empezar", 2.161631338172897], ["regenerist", 2.161631338172897], ["retropie", 2.161631338172897], ["whitemore", 2.161631338172897], ["ypsilon", 2.161631338172897], ["flyes", 2.161631338172897], ["philz", 2.161631338172897], ["cora\u00e7\u00e3o", 2.161631338172897], ["plenti", 2.161631338172897], ["shopify", 2.1611233167068167], ["marionettes", 2.1610249899714464], ["dojo's", 2.1610249899714464], ["ragione", 2.1610249899714464], ["cryptex", 2.1610249899714464], ["stanishev", 2.1610249899714464], ["gulper", 2.1610249899714464], ["gelert", 2.1610249899714464], ["unwilling", 2.1610010265095925], ["occurred", 2.1609916408203667], ["bespoke", 2.1608223622180502], ["kevin's", 2.1607486157912272], ["raices", 2.160575338496213], ["goblets", 2.160575338496213], ["earaches", 2.160575338496213], ["aspectos", 2.160575338496213], ["saskatoon's", 2.160575338496213], ["mayores", 2.160575338496213], ["landscaping", 2.1605669413903503], ["almonds", 2.1603126101996897], ["holofernes", 2.1602285923337194], ["sugarcane", 2.160147380732504], ["involve", 2.1601177533797675], ["cramps", 2.1600484502364776], ["mold", 2.1599718588847154], ["ichiro's", 2.159953052972453], ["aussi", 2.159953052972453], ["bandaging", 2.159953052972453], ["hikurangi", 2.159953052972453], ["tobio", 2.159953052972453], ["passporting", 2.159953052972453], ["wollte", 2.159953052972453], ["unoriginality", 2.159953052972453], ["sociologist", 2.159751658748399], ["reapplied", 2.1595428054790107], ["process", 2.159264594563458], ["pachelbel's", 2.1592519971292288], ["strain", 2.1591517610597295], ["yodels", 2.15913618849436], ["brontes", 2.15913618849436], ["counterfeit", 2.1591302137141777], ["cones", 2.1591302137141777], ["sweeping", 2.1590662002872776], ["bioengineered", 2.159035095873867], ["owner", 2.1589541852884198], ["giulia", 2.1586238510937004], ["malnourishment", 2.1586238510937004], ["kibera", 2.1586238510937004], ["pronghorn", 2.1586238510937004], ["flavorless", 2.1586238510937004], ["ikari", 2.1586238510937004], ["subtext", 2.1583610940909415], ["lea's", 2.158240163953442], ["stupas", 2.1581504503497526], ["blueberry", 2.158003556883616], ["chronic", 2.1576816320404073], ["yor\u00f9b\u00e1", 2.1575450785544232], ["tadaa", 2.1575450785544232], ["perimenopause", 2.1575450785544232], ["schwerdt", 2.1575450785544232], ["brooded", 2.1575450785544232], ["principios", 2.1575450785544232], ["sitemaps", 2.1575450785544232], ["zion's", 2.1575450785544232], ["tatang", 2.1575450785544232], ["soarer", 2.1575450785544232], ["whiteboarding", 2.1575450785544232], ["rehydrated", 2.1575450785544232], ["challengingly", 2.1575450785544232], ["regeneratively", 2.1575450785544232], ["akkad", 2.1575450785544232], ["enfermedad", 2.1575450785544232], ["\u00f8rsted", 2.1575450785544232], ["log4j", 2.1575450785544232], ["cdls", 2.1575450785544232], ["piaf's", 2.1575450785544232], ["channapatna", 2.1575450785544232], ["openldap", 2.1575450785544232], ["taleem", 2.1575450785544232], ["hwoarang", 2.1575450785544232], ["draconia", 2.1575450785544232], ["r\u00e8gles", 2.1575450785544232], ["genoise", 2.1575450785544232], ["khagan", 2.1575450785544232], ["abdi's", 2.1575450785544232], ["footbag", 2.1575450785544232], ["maximo", 2.1571047632322693], ["swore", 2.1569404567772468], ["student", 2.1569224726888976], ["periods", 2.156884708448604], ["shanties", 2.15671525352421], ["word's", 2.156652054929194], ["typesetting", 2.156504939742448], ["neonicotinoids", 2.1564154738012533], ["redlining", 2.1562608255314744], ["asa's", 2.156186422963394], ["debrecen", 2.156186422963394], ["streptomyces", 2.1560571164312825], ["influence", 2.1559287087546934], ["museum", 2.15586698793777], ["pendulums", 2.155632361286201], ["sequesters", 2.1556323612862007], ["flashlight", 2.1555223800432786], ["blushing", 2.1555223800432786], ["micronutrients", 2.1555223800432786], ["diagramming", 2.1552595435135817], ["alwin", 2.1552595435135817], ["down", 2.155230621106519], ["jefferson's", 2.15502705911385], ["lilies", 2.154752511342819], ["monolinguals", 2.1547062034510622], ["wyrm", 2.1543152206523954], ["netherlandish", 2.1543152206523954], ["skara", 2.1543152206523954], ["steaks", 2.1540020510969935], ["marineris", 2.1537993405708216], ["bleiben", 2.1537993405708216], ["bugattis", 2.1537993405708216], ["vail's", 2.1537993405708216], ["researchers", 2.153661497413025], ["snowshoes", 2.153605740630096], ["bespectacled", 2.1532051890285953], ["ros\u00e9s", 2.153087340788815], ["pansori", 2.153087340788815], ["chilis", 2.1530211937122927], ["hills", 2.1527224409005363], ["strombolian", 2.152619176548591], ["segreto", 2.152619176548591], ["ballarat's", 2.152619176548591], ["nujoma", 2.152619176548591], ["chronosphere", 2.152619176548591], ["stiffly", 2.1523461747506776], ["shamanic", 2.1520411370274988], ["advocates", 2.152024087781018], ["pepsico", 2.1519789303173664], ["rajinikanth", 2.151884852268092], ["botswana's", 2.1515740128388665], ["acacia", 2.1515695247829365], ["shoulder", 2.151517949066784], ["zain", 2.151441497082197], ["wanderings", 2.15123311156929], ["octopus", 2.1512147813621367], ["bedouin", 2.1508653724288678], ["amiga", 2.150815305730254], ["tran's", 2.150768869136853], ["peak's", 2.150768869136853], ["gradle", 2.1503532616259085], ["voronezh", 2.1503532616259085], ["nordvpn", 2.1503532616259085], ["aardvarks", 2.1496626069730658], ["tarte", 2.1495028154081406], ["plugins", 2.1493194379424345], ["alfons", 2.14926502616962], ["literary", 2.1490153721531775], ["portobello", 2.148930801520883], ["involves", 2.148370586068561], ["shade", 2.1483569124440276], ["ndebele", 2.148336044682545], ["exhibitions", 2.1481086340041693], ["them", 2.147896362967557], ["finley", 2.147836125700928], ["alibis", 2.1476853295395237], ["guntur", 2.1475013342232216], ["harvest", 2.1474933381650834], ["drummer", 2.1473316475197213], ["connotations", 2.1473274362108623], ["ottomans", 2.147322155538236], ["procuring", 2.1471722656767573], ["marshallese", 2.1471722656767573], ["living", 2.147148408207139], ["aboard", 2.1470270462964702], ["donato", 2.146977510822728], ["dietician", 2.146977510822728], ["maronite", 2.1468338618523797], ["plateaued", 2.1468338618523797], ["soir\u00e9e", 2.1468338618523797], ["apart", 2.146485423765349], ["unita", 2.1460753705218782], ["firm", 2.1460168990767503], ["knife's", 2.1459513102203283], ["hamburg's", 2.1459513102203283], ["vlogs", 2.1456937746018006], ["thunder's", 2.1454882089977954], ["shinkansen", 2.1454882089977954], ["thrifty", 2.145326936534985], ["futility", 2.1451841432085383], ["whirl", 2.1451841432085383], ["glaswegians", 2.145109660234225], ["tibs", 2.145109660234225], ["dana", 2.1450355603795974], ["paws", 2.1449727666189804], ["accompaniments", 2.1449425956189416], ["hoca", 2.1449425956189416], ["vercors", 2.1449425956189416], ["higashiyama", 2.1447460823257103], ["vrchat", 2.1447460823257103], ["elemento", 2.1447460823257103], ["artemio", 2.1447460823257103], ["embroiderers", 2.144562039241459], ["interjections", 2.144328328901027], ["glutes", 2.144226899057174], ["flowerbeds", 2.144226899057174], ["vaishno", 2.1442268990571733], ["eher", 2.1442268990571733], ["farinata", 2.1442268990571733], ["uncompassionate", 2.1442268990571733], ["sarawakian", 2.1442268990571733], ["ramanand", 2.1442268990571733], ["fibromyalgia", 2.1441551617905605], ["hyperloop", 2.1439912697276067], ["lama's", 2.1435483462410163], ["phishers", 2.1434250191398], ["programas", 2.1434250191398], ["graeme's", 2.1434250191398], ["subtotal", 2.1434250191398], ["danica's", 2.1434250191398], ["lunaire", 2.1434250191398], ["antwerp's", 2.1434250191398], ["eids", 2.1434250191398], ["rise's", 2.1434250191398], ["zaharias", 2.1434250191398], ["insync", 2.1434250191398], ["confianza", 2.1434250191398], ["tatar", 2.1433829106968134], ["disappearances", 2.1433464082190046], ["nasi", 2.143124468774882], ["freeing", 2.143104139383222], ["salaryman", 2.1430132773121784], ["gar\u00e7ons", 2.1430132773121784], ["wolof", 2.1429867446910333], ["swahili", 2.142986744691033], ["anke", 2.1429670712668933], ["sickled", 2.1428345439279157], ["mutualism", 2.1428345439279157], ["capheus", 2.1428345439279157], ["skylarks", 2.1428345439279157], ["baitcasting", 2.1428345439279157], ["zagallo", 2.1428345439279157], ["th\u00e9o", 2.1428345439279157], ["vaishyas", 2.1428345439279157], ["anini", 2.1428345439279157], ["zendaya's", 2.1428345439279157], ["liutprand", 2.1428345439279157], ["aurait", 2.1428345439279157], ["voltan", 2.1428345439279157], ["apprentices", 2.142434612135469], ["tanabata", 2.142023171514463], ["unclipped", 2.142023171514463], ["oughtta", 2.142023171514463], ["weave", 2.1419362900436423], ["mgm's", 2.1418998967002647], ["seedy", 2.141666695764766], ["everything", 2.1414093460703207], ["uw's", 2.1412291237788783], ["secrecy", 2.1410154993235624], ["vitriolic", 2.1410154993235624], ["weakened", 2.140881357486817], ["appalachians", 2.140876544527397], ["mocked", 2.1408220990422078], ["jungfrau", 2.140779588930072], ["clouseau", 2.1406231563696827], ["dividers", 2.1403979224517142], ["pronunciation", 2.1403979224517142], ["deducting", 2.1403979224517142], ["tracery", 2.14035584578572], ["gawking", 2.14035584578572], ["peppermint", 2.140267176909089], ["fingal", 2.1400150247911687], ["added", 2.139843909147007], ["repetition", 2.139809629357366], ["pulleys", 2.1397864151314323], ["chlorofluorocarbons", 2.1396549866924457], ["laurels", 2.1394893517317746], ["vines", 2.1393480417695723], ["manuals", 2.1389812011076943], ["sneha", 2.1389455520151612], ["wiesel's", 2.138945552015161], ["tetum", 2.138945552015161], ["knez", 2.138945552015161], ["abello", 2.138945552015161], ["huron's", 2.138945552015161], ["maxxi", 2.138945552015161], ["caddisflies", 2.138945552015161], ["kueh", 2.138945552015161], ["untilled", 2.138945552015161], ["balance's", 2.138945552015161], ["gatka", 2.138945552015161], ["roten", 2.138945552015161], ["salsa's", 2.138945552015161], ["siona", 2.138945552015161], ["asterius", 2.138945552015161], ["pond's", 2.138538133814777], ["taunted", 2.13832331330912], ["consoles", 2.1383168129768166], ["gamesmanship", 2.1382499599657248], ["tartu", 2.1382499599657248], ["omnivore", 2.138190630055626], ["telenovela", 2.138190630055626], ["therapists", 2.138054987324359], ["seething", 2.1379641195081636], ["kannan", 2.1377996883265813], ["superpowers", 2.1377675820010076], ["dermatologist", 2.1370484003081565], ["librettos", 2.137031506778458], ["arag\u00f3n", 2.137031506778458], ["outage", 2.136947635840202], ["groundcovers", 2.136898714166343], ["kolache", 2.136898714166343], ["fushi", 2.1367461221416235], ["swaddled", 2.1365689458462556], ["akari", 2.136568945846255], ["held", 2.136378817352743], ["msmes", 2.136360723553511], ["sceptically", 2.136360723553511], ["plassey", 2.136360723553511], ["tfl's", 2.136360723553511], ["grasses", 2.13634661817239], ["strongarm", 2.136314622676889], ["campmates", 2.1361125115489155], ["oguz", 2.1361125115489155], ["taraweeh", 2.1361125115489155], ["comun", 2.1361125115489155], ["friederike", 2.1361125115489155], ["lrrk2", 2.1361125115489155], ["ysleta", 2.1361125115489155], ["leaky", 2.135969627768879], ["britney's", 2.135969627768879], ["weeknd's", 2.135969627768879], ["research", 2.1359084239390995], ["ravaging", 2.1358998043035538], ["geoffrey's", 2.135662082966015], ["awed", 2.1354390838971034], ["coins", 2.1349708740718008], ["valid", 2.1349492887421455], ["callbacks", 2.1348459063737986], ["marking", 2.13475888417822], ["danny's", 2.134606765664098], ["theater", 2.1344766836297007], ["svalbard", 2.1343456691076015], ["bosphorus", 2.1343456691076015], ["unanswerable", 2.1343456691076015], ["namsan", 2.1343456691076015], ["vraiment", 2.1343456691076015], ["tabitha's", 2.1343456691076015], ["ethologist", 2.1343456691076015], ["tertre", 2.1343456691076015], ["selvaraj", 2.1343456691076015], ["dajuan", 2.1343456691076015], ["msgbox", 2.1343456691076015], ["budai", 2.1343456691076015], ["morpheus", 2.134318445310802], ["unveil", 2.134126155708089], ["radishes", 2.1339181158886293], ["inspection", 2.133901980401931], ["alfie's", 2.1338120826903246], ["perception", 2.1337299245168033], ["deda", 2.133690157538465], ["margaux", 2.133539425894683], ["jenna's", 2.133539425894683], ["sandra's", 2.133436146805425], ["paddlers", 2.133436146805425], ["barbi", 2.1333818208104973], ["bitdefender", 2.1333818208104973], ["motorboats", 2.1333483113182985], ["consuelo", 2.1333483113182985], ["disconnect", 2.133122866901342], ["sift", 2.1326305699092116], ["restorer", 2.1322613471651137], ["observatory's", 2.1322613471651133], ["infringed", 2.1321888828882347], ["googling", 2.132109314270485], ["clumsily", 2.1320665311443583], ["poha", 2.1317153135294755], ["makua", 2.1317153135294755], ["envelopes", 2.1316091686732026], ["stonemasons", 2.1315564673809266], ["ernakulam", 2.1315564673809266], ["stunted", 2.131471851586898], ["imputation", 2.131267285931004], ["wafts", 2.1312672859310036], ["fondo", 2.1312672859310036], ["event", 2.131125510416184], ["cozumel", 2.130962884404769], ["magnolias", 2.130962884404769], ["cobain's", 2.130893051113457], ["families", 2.130165806736671], ["humidity", 2.1301049989495833], ["bring", 2.1300836475996023], ["destruction", 2.129965716846133], ["employee", 2.129886169968271], ["prank", 2.129552976736851], ["abolitionist", 2.1293008520715286], ["artistically", 2.1292791634627846], ["larnaca", 2.12911796434374], ["tortellini", 2.12911796434374], ["problem", 2.1289139825931644], ["bluey", 2.1289109926353365], ["merchandise", 2.1288264041979894], ["epiphytes", 2.128820365970119], ["adewale", 2.128820365970119], ["cultura", 2.128791802702093], ["hormonal", 2.1287105975980416], ["lucas's", 2.1284605247994004], ["nyfw", 2.1284605247994004], ["shape's", 2.1284605247994004], ["zaur", 2.1284605247994004], ["explicitly", 2.128316493636068], ["devotees", 2.128209425308917], ["villanelle", 2.128104892297345], ["dishcloth", 2.128104892297345], ["poked", 2.127974019536589], ["cafeteria", 2.12794748288589], ["famous", 2.1277346336377634], ["lensing", 2.1276091405894806], ["facilitators", 2.127593899945498], ["damit", 2.127482821296966], ["monument", 2.1274372706572153], ["mutex", 2.1273247951960634], ["stakes", 2.1273137623942024], ["geyser", 2.1272847394970436], ["awaits", 2.1271723136809704], ["boars", 2.127045236762131], ["creole", 2.1270259579988027], ["sounds", 2.126964247359275], ["reinterpret", 2.1267230060036457], ["volt's", 2.1267230060036457], ["katrina's", 2.1267230060036457], ["ivar's", 2.1267230060036457], ["hilarion", 2.1267230060036457], ["trainability", 2.1267230060036457], ["photoperiods", 2.1267230060036457], ["nassar's", 2.1267230060036457], ["hoa's", 2.1267230060036457], ["dauphinoise", 2.1267230060036457], ["\u0438\u0441\u0442\u043e\u0440\u0438\u044f", 2.1267230060036457], ["laduma", 2.1267230060036457], ["soundex", 2.1267230060036457], ["oberon's", 2.1267230060036457], ["shrek's", 2.1267230060036457], ["presentes", 2.1267230060036457], ["internes", 2.1267230060036457], ["caramelizes", 2.1267230060036457], ["logotherapy", 2.1267230060036457], ["cbrt", 2.1267230060036457], ["melquiades", 2.1267230060036457], ["r\u00e9alit\u00e9", 2.1267230060036457], ["mirella", 2.1267230060036457], ["tuiles", 2.1267230060036457], ["sampaguita", 2.1267230060036457], ["caribou's", 2.1267230060036457], ["ashford's", 2.1267230060036457], ["miad", 2.1267230060036457], ["beveridge's", 2.1267230060036457], ["starflower", 2.1267230060036457], ["bisceglie", 2.1267230060036457], ["havoc's", 2.1267230060036457], ["tina", 2.1262606749153843], ["libraries", 2.126260674915384], ["aside", 2.1260623915309154], ["boredom", 2.125799949143401], ["eleusinian", 2.1254028799912286], ["shorebird", 2.1254028799912286], ["shoyu", 2.1254028799912286], ["centimeters", 2.1253988968523982], ["burcu", 2.1253108393329927], ["offhanded", 2.1253108393329927], ["sharbat", 2.1253108393329927], ["ublock", 2.1253108393329927], ["hint", 2.1252233936276177], ["aleksandr", 2.125218533022602], ["thunderbolt", 2.1251380498338595], ["bolt's", 2.124789621452733], ["pmcs", 2.1247646975266625], ["koffi", 2.1247646975266625], ["fredrick", 2.124726082993314], ["naturel", 2.1246859150017263], ["profited", 2.124667539180833], ["stewing", 2.124507669539059], ["apartment", 2.124486854297109], ["contaminating", 2.1243599804414193], ["hardscape", 2.123964610405716], ["ew's", 2.123964610405716], ["offseasons", 2.123964610405716], ["niamh", 2.1239187826307506], ["perks", 2.1238754752002436], ["outlandish", 2.123844962049652], ["kofta", 2.1237630574914625], ["sarovar", 2.1237630574914625], ["math's", 2.1237630574914625], ["merit", 2.123680610592279], ["euclidean", 2.1232761291705766], ["desperately", 2.123159909867933], ["closeted", 2.123154678812364], ["surrogacy", 2.123092659469499], ["landlocked", 2.1230243572975525], ["scheherazade", 2.122784630066602], ["cuttlefish", 2.1225406598758116], ["bifacial", 2.1225406598758116], ["pooja's", 2.1225406598758116], ["ramasamy", 2.1225406598758116], ["kaffir", 2.122379209014259], ["equestrians", 2.12224655114983], ["lawnmower", 2.122124303562187], ["cassatt", 2.12205913976241], ["mestiza", 2.12205913976241], ["tartine", 2.12205913976241], ["cobbler's", 2.12205913976241], ["illustrations", 2.1217242980835516], ["morally", 2.121493359267571], ["outliers", 2.121479606087463], ["giorgio's", 2.121452317017267], ["alula", 2.121452317017267], ["passeig", 2.121452317017267], ["ragdolls", 2.121452317017267], ["meristems", 2.121452317017267], ["interact", 2.1214061984886365], ["andere", 2.1214061984886365], ["fairs", 2.1211530171783983], ["cohabited", 2.1209122874080073], ["underneath", 2.1208892866468996], ["worksheets", 2.120800255558477], ["rombauer", 2.1206639661005013], ["cmap", 2.1206639661005013], ["xylophone", 2.1206639661005013], ["stellen", 2.1206639661005013], ["psychoeducation", 2.1206639661005013], ["nakdong", 2.1206639661005013], ["claytonia", 2.1206639661005013], ["haylee", 2.1206639661005013], ["romanoff's", 2.1206639661005013], ["respuesta", 2.1206639661005013], ["viejos", 2.1206639661005013], ["marvelstudios", 2.1206639661005013], ["arcsine", 2.1206639661005013], ["dojo", 2.1205554367265966], ["munch's", 2.1205554367265966], ["provenance", 2.120438363550916], ["denarii", 2.1202783908339375], ["koblenz", 2.1202783908339375], ["campanas", 2.1202783908339375], ["stemmer", 2.1202783908339375], ["typename", 2.1202783908339375], ["dinkley", 2.1199928699086974], ["poisonwood", 2.1199928699086974], ["forro", 2.1199928699086974], ["sprout's", 2.1199928699086974], ["niu's", 2.1199928699086974], ["hausser", 2.1199928699086974], ["manaia", 2.1199928699086974], ["andermatt", 2.119598305816029], ["goombas", 2.119598305816029], ["envelope", 2.119452785782846], ["prickly", 2.1194351701575274], ["natural", 2.1194154868376684], ["ceremonies", 2.119396306634732], ["map's", 2.1193385511216887], ["changement", 2.1193385511216887], ["arvn", 2.1193385511216887], ["pronghorns", 2.1193385511216887], ["suelo", 2.1193385511216887], ["terraza", 2.1193385511216887], ["gioconda", 2.119154596729611], ["hikmet", 2.119154596729611], ["conked", 2.119154596729611], ["skyward", 2.11912756669649], ["gaelic", 2.1189820601963527], ["awash", 2.1187092671404435], ["secured", 2.118414191122347], ["runes", 2.1182727535098596], ["harpsichord", 2.1182229140531916], ["flappers", 2.118077790532086], ["totta", 2.118077790532086], ["cotinga", 2.118077790532086], ["vadai", 2.118077790532086], ["viva's", 2.118077790532086], ["giuliano's", 2.118077790532086], ["trichocereus", 2.118077790532086], ["spiderling", 2.118077790532086], ["nuncius", 2.118077790532086], ["palavras", 2.118077790532086], ["kven", 2.118077790532086], ["laroche's", 2.118077790532086], ["drita", 2.118077790532086], ["cache", 2.118006928110188], ["whose", 2.1177480531471136], ["magnetically", 2.117671093567698], ["languished", 2.1176207172065507], ["glossing", 2.1175560964950093], ["dunkers", 2.1174156843143517], ["acquaintance", 2.117387390930343], ["raita", 2.1173504286534692], ["naja", 2.1173504286534692], ["m\u00e9rida", 2.1173504286534692], ["pianist", 2.11721613163906], ["ticino", 2.1168770661610363], ["face", 2.1168177810338986], ["pandan", 2.116297893229118], ["durrani", 2.116297893229118], ["pelourinho", 2.1162978932291177], ["c\u00f3digo", 2.1162978932291177], ["yarning", 2.1162978932291177], ["arianna's", 2.1162978932291177], ["seline", 2.1162978932291177], ["aquinas's", 2.1162978932291177], ["hove's", 2.1162978932291177], ["carmela's", 2.1162978932291177], ["smoothed", 2.1161970979398617], ["storks", 2.116066957186644], ["regurgitate", 2.1159568848673342], ["bearing", 2.1158758645674025], ["annoyances", 2.115825558022505], ["halocline", 2.1157321894351515], ["aravinda", 2.1157321894351515], ["tarina", 2.1157321894351515], ["polysemous", 2.1157321894351515], ["ecobuild", 2.1157321894351515], ["kulon", 2.1157321894351515], ["salzkammergut", 2.1157321894351515], ["meissen", 2.11564632368071], ["encourage", 2.1155991888623142], ["sunrise", 2.1153815557825215], ["descending", 2.1151266768556836], ["plots", 2.1151084745899866], ["biplane", 2.115030952440547], ["mandarin", 2.114997116317976], ["solstice", 2.1149078781925144], ["evapotranspiration", 2.1149078781925144], ["astounding", 2.1147193389189325], ["university's", 2.1145551878100926], ["developer's", 2.1144219089252183], ["meteoroids", 2.1143941769833345], ["viator", 2.1138110566918336], ["tweak", 2.1136691954381215], ["karthik", 2.1135950862135], ["pyqt", 2.1135950862134996], ["telemachus", 2.1135950862134996], ["aquella", 2.1135950862134996], ["mitad", 2.1135950862134996], ["ashaninka", 2.1135950862134996], ["sdcs", 2.1135950862134996], ["beothuk", 2.1135950862134996], ["inconvenienced", 2.113364193151864], ["reinvesting", 2.113278205390979], ["disciplined", 2.113212188552954], ["container", 2.11317425663518], ["matched", 2.1131356090208446], ["juneau's", 2.113090166221571], ["kvothe", 2.113090166221571], ["riku's", 2.113090166221571], ["stankonia", 2.113090166221571], ["controversy", 2.113022002022661], ["okayama", 2.1130022124165255], ["silverback", 2.1129656654016435], ["latynina", 2.1127925059643204], ["bananas", 2.1127146508742802], ["clarify", 2.112680025702703], ["pastels", 2.1125448526302884], ["eileen's", 2.1124570622069236], ["rosaries", 2.1123532559630807], ["chris's", 2.1122728496878738], ["barstools", 2.1122728496878733], ["literaria", 2.1122728496878733], ["transferable", 2.1122366319047803], ["exacerbates", 2.112141044464591], ["blackpink", 2.112141044464591], ["sorts", 2.111757177442879], ["overexposing", 2.111639864117095], ["opal's", 2.111639864117095], ["midville", 2.111639864117095], ["mazovia", 2.111639864117095], ["assurer", 2.111639864117095], ["emilian", 2.111639864117095], ["palinka", 2.111639864117095], ["menya", 2.111639864117095], ["trivialised", 2.111639864117095], ["baoli", 2.111639864117095], ["couponer", 2.111639864117095], ["erlebnis", 2.111639864117095], ["rohmer's", 2.111639864117095], ["r\u00e9ponse", 2.111639864117095], ["pardos", 2.111639864117095], ["pitfall", 2.1115620012902467], ["fifi", 2.111541556860851], ["string", 2.111372754293646], ["employees", 2.1112856087451544], ["affluence", 2.1112559295963464], ["bedford's", 2.1112120602548345], ["consonants", 2.110810375954984], ["arles", 2.1107233537594055], ["investigaci\u00f3n", 2.110723353759405], ["jaspal", 2.110723353759405], ["biopesticides", 2.110723353759405], ["backseat", 2.1106010735387795], ["tree", 2.1105948559004424], ["theropods", 2.11046846626348], ["bevo", 2.11046846626348], ["dunder", 2.1104262779882923], ["bullfrogs", 2.1102792714211436], ["specimen", 2.110181827068062], ["hibiscus", 2.1100790868262256], ["proprioception", 2.1100606462699987], ["protective", 2.11001116411079], ["brushes", 2.1099797261976083], ["noodle", 2.109943727534225], ["fame", 2.109906680256022], ["grover's", 2.109502576805236], ["sazerac", 2.109502576805236], ["hyatt's", 2.109357832563654], ["zamindars", 2.1093128050728853], ["maximized", 2.1092908502167305], ["leela's", 2.1092778508462704], ["insect", 2.109216068794804], ["redefined", 2.109127293654417], ["inventory", 2.109090211998555], ["ordering", 2.109000314286949], ["underperformance", 2.109000314286949], ["nfcc", 2.1090003142869485], ["shirataki", 2.1090003142869485], ["gefunden", 2.1090003142869485], ["sweatier", 2.1090003142869485], ["inverses", 2.1085099631327315], ["relativism", 2.108223327802501], ["grieving", 2.1081204643435663], ["vial", 2.1080157203026877], ["klezmer", 2.107991705815267], ["headlamp", 2.1079024484283924], ["bysshe", 2.1078574999797306], ["vrindavan", 2.1076294983530603], ["jeering", 2.1076167584280245], ["violet's", 2.107505629443372], ["cookbook", 2.107375089282361], ["server's", 2.1073750892823604], ["mufasa", 2.1073008780949367], ["sultan's", 2.1072195653584767], ["faith", 2.107198551982316], ["counted", 2.106744698977551], ["restrooms", 2.1066814574268204], ["pranksters", 2.106468501184563], ["unread", 2.106340183171577], ["hiro's", 2.106262173461449], ["quitclaim", 2.106262173461449], ["senseless", 2.1062311437062142], ["squares", 2.105534983504354], ["stances", 2.1052785490264423], ["slurp", 2.105240955437952], ["lawns", 2.1051713376814893], ["boss's", 2.10515463261708], ["agra", 2.1050642523193153], ["tweetie", 2.1048543889753306], ["kiem", 2.1048543889753306], ["busloads", 2.1047907328213062], ["kleines", 2.1045696413577746], ["esperantist", 2.1045696413577746], ["rochelle's", 2.1045696413577746], ["torchlit", 2.1045696413577746], ["siena's", 2.1045696413577746], ["ashok's", 2.1045696413577746], ["maitri", 2.1045696413577746], ["specter's", 2.1045696413577746], ["coatis", 2.1045696413577746], ["hervor", 2.1045696413577746], ["cort's", 2.1045696413577746], ["formar", 2.1045696413577746], ["sfsr", 2.1045696413577746], ["aedui", 2.1045696413577746], ["easttown", 2.1045696413577746], ["sticht", 2.1045696413577746], ["harbans", 2.1045696413577746], ["loupe", 2.104569641357774], ["marketing", 2.104515268257151], ["haris", 2.1043626005666916], ["irritation", 2.1042832480505655], ["executives", 2.1041307148806787], ["houdini", 2.10393285780517], ["cavern", 2.103831195869579], ["coopted", 2.1036919327743404], ["forno", 2.1036919327743404], ["casey's", 2.1036064515905624], ["switching", 2.1032828198287796], ["ambrosial", 2.10320886957363], ["researcher", 2.1031850560674075], ["soulfully", 2.103092750381383], ["typographer", 2.103092750381383], ["ahas", 2.103092750381383], ["vocalist", 2.103069214270205], ["recordings", 2.102750498568214], ["gorton's", 2.102692237574226], ["constrictors", 2.102692237574226], ["pattern's", 2.102692237574226], ["squats", 2.102505086956222], ["tamari", 2.102439510141825], ["intact", 2.1023751824096073], ["congregation", 2.102243705644577], ["understand", 2.1020657431178003], ["usaaf", 2.1020560220711775], ["biologists", 2.1018704957802314], ["gladly", 2.101577990405597], ["lauded", 2.101456195515652], ["characters", 2.1014162164350605], ["darcie", 2.1014048749797927], ["crooning", 2.100996518027795], ["\u00e9tats", 2.100996518027795], ["grohl's", 2.100996518027795], ["scruffy", 2.100939994042914], ["conjunto", 2.1007675764872014], ["meaning", 2.1006125302315564], ["precise", 2.100454038503564], ["data", 2.1003955023680607], ["adapts", 2.1003374438957834], ["exhibiting", 2.100103664261834], ["phoebe's", 2.1000322052254874], ["purifying", 2.099943386941064], ["computing", 2.099881459232716], ["slippery", 2.099855927897498], ["tintin", 2.0998382840381966], ["invoicing", 2.099828648923833], ["object's", 2.099828648923833], ["squandering", 2.0997661609976666], ["levels", 2.099533015732177], ["provocative", 2.099526729987478], ["suprematist", 2.0994573264394965], ["poliziano", 2.0994573264394965], ["underpinnings", 2.0990852801296542], ["reserved", 2.099059175290894], ["offer", 2.0990534535609524], ["interrogative", 2.0990313001963066], ["salata", 2.0988807108595897], ["unpackaged", 2.0988807108595897], ["conflict's", 2.0988807108595897], ["jimmy's", 2.0987029400483936], ["action", 2.098685309445096], ["spacex's", 2.0986620774522087], ["tender", 2.0985107861814565], ["alchemist's", 2.098507775409899], ["thomas", 2.0982616948695916], ["lyrics", 2.098255932433701], ["bertie's", 2.0978635080098593], ["regrow", 2.0977222377399594], ["smilingly", 2.0977222377399594], ["carolyn's", 2.0977222377399594], ["brachiosaurus", 2.0977222377399594], ["timekeepers", 2.09752666415822], ["plantings", 2.09743550698029], ["refracting", 2.0971851864758175], ["luminaries", 2.097133816354899], ["mcing", 2.0967691608486647], ["bipedalism", 2.0967691608486647], ["nabataean", 2.0967691608486647], ["timers", 2.0966593823585677], ["barkeep", 2.096594430085261], ["speakerphone", 2.096540255918441], ["spluttering", 2.096498260440286], ["cloves", 2.096276330490346], ["busker", 2.0961666409748574], ["acronyms", 2.0961089096509165], ["westvleteren", 2.095971303481523], ["turbine's", 2.095971303481523], ["fermentor", 2.095971303481523], ["nachricht", 2.095971303481523], ["dingo's", 2.095971303481523], ["clodia", 2.095971303481523], ["anwen", 2.095971303481523], ["liviu", 2.095971303481523], ["equinoxes", 2.0958425583154616], ["caretakers", 2.095827663866773], ["lin's", 2.095751311741457], ["shias", 2.095751311741457], ["sitcoms", 2.0956348589315392], ["deram", 2.095588547582421], ["northland's", 2.095588547582421], ["shichi", 2.095588547582421], ["variabilities", 2.095588547582421], ["centara", 2.095588547582421], ["ovoviviparous", 2.095588547582421], ["srhr", 2.095588547582421], ["demand's", 2.095588547582421], ["lausche", 2.095588547582421], ["sattvic", 2.095588547582421], ["dylan's", 2.095570578977511], ["fahrenheit", 2.095453659124186], ["refreshingly", 2.095273553061699], ["patrolled", 2.0952507197057324], ["paiute", 2.095215998507295], ["resetting", 2.0951733226070486], ["manipulative", 2.095061118554715], ["misconstruing", 2.0950334247883595], ["nextcloud", 2.0950334247883595], ["coopt", 2.0950334247883595], ["geoduck", 2.0950334247883595], ["runner's", 2.094988158577053], ["paperbacks", 2.094885835255088], ["sahitya", 2.094853253355199], ["cogs", 2.0943930542816447], ["refracts", 2.094155666524245], ["castles", 2.093923718902953], ["reinvent", 2.0938974987370678], ["roundworms", 2.0937600202619127], ["marathoners", 2.0937600202619127], ["unable", 2.0937596703045314], ["choux", 2.093492959034839], ["brittle", 2.0934117686932048], ["floury", 2.093252907095952], ["hulu's", 2.093252907095952], ["stalactite", 2.093252907095952], ["peregrines", 2.0928187261977906], ["imperiously", 2.0928187261977906], ["fuerza", 2.0928187261977906], ["insignificantly", 2.0928187261977906], ["copperplate", 2.092760492862337], ["lim's", 2.092760492862337], ["intrigues", 2.0926936108259673], ["melty", 2.0925587810076034], ["material's", 2.0925587810076034], ["antimalware", 2.092558781007603], ["amateurishly", 2.092558781007603], ["milu", 2.092558781007603], ["tsukimi", 2.092558781007603], ["outmigration", 2.092558781007603], ["abyssus", 2.092558781007603], ["waylander", 2.092558781007603], ["limpieza", 2.092558781007603], ["overseeing", 2.092459135351365], ["village", 2.092334012969099], ["benevento", 2.0921649246588347], ["arafat's", 2.092116566556841], ["measuring", 2.0920350025581445], ["trendiest", 2.0919726300885535], ["audre", 2.0919726300885535], ["eco's", 2.0918586944298156], ["hyperinflation", 2.09157055962342], ["clams", 2.0915401809660543], ["hoard", 2.091486124825742], ["moths", 2.091387021901836], ["caleb", 2.0910099081938665], ["poster", 2.090991772703853], ["elks", 2.090991772703853], ["happily", 2.0909204210317758], ["sumiyoshi", 2.0908793598350446], ["dxracer", 2.0908793598350446], ["callisthenes", 2.0908793598350446], ["komnenos", 2.0908793598350446], ["campy", 2.0906212265807445], ["theresa's", 2.0906212265807445], ["nectarines", 2.0906212265807445], ["slieve", 2.090494299179458], ["goatskin", 2.090494299179458], ["cantabrian", 2.0902693293872447], ["seekers", 2.090257611536197], ["electromagnetism", 2.09017426433108], ["brands", 2.090124375645146], ["mushroom", 2.089824942921342], ["bookshop", 2.0897663852920836], ["visits", 2.089585105511307], ["goer", 2.088908522392184], ["vega's", 2.088854006608994], ["teenagers", 2.088688719564577], ["isaac's", 2.088585568132272], ["armors", 2.088585568132272], ["zane's", 2.0885326938868576], ["bellini's", 2.088427742495205], ["wrought", 2.088248653440489], ["emin's", 2.087721613229037], ["moxibustion", 2.087721613229037], ["kubera", 2.087721613229037], ["enslaved", 2.087631327145285], ["disbandment", 2.087610858766797], ["mistreatments", 2.0869711741157273], ["imta", 2.0869711741157273], ["bikila", 2.0869711741157273], ["undissolved", 2.0869711741157273], ["vark", 2.0869711741157273], ["elio's", 2.0869711741157273], ["interface's", 2.0869711741157273], ["costanera", 2.0869711741157273], ["microtransaction", 2.0869711741157273], ["immaturely", 2.0869711741157273], ["kaeng", 2.0869711741157273], ["plan\u00e8te", 2.0869711741157273], ["talao", 2.0869711741157273], ["mower's", 2.0869711741157273], ["smartmoney", 2.0869711741157273], ["timbuktu's", 2.0869711741157273], ["conti's", 2.0869711741157273], ["nasza", 2.0869711741157273], ["intercept's", 2.0869711741157273], ["latavia", 2.0869711741157273], ["azules", 2.0869711741157273], ["steeleye", 2.0869711741157273], ["kapuas", 2.0869711741157273], ["wordlessly", 2.086971174115727], ["cloche", 2.0868812959428538], ["technique", 2.0868379381029163], ["beatboxing", 2.0868176074362483], ["conforming", 2.0868109279099456], ["ondrej", 2.0864441611929707], ["greenies", 2.0864441611929707], ["flick", 2.086047975834177], ["epochal", 2.086026454654193], ["rubik", 2.086026454654193], ["kago", 2.085945131257263], ["descendant", 2.0854291343037414], ["fulfilled", 2.0854101779441803], ["radiate", 2.0853311931126752], ["percussive", 2.0853106861631225], ["resident", 2.0852740607512206], ["needlessly", 2.08526731842285], ["mites", 2.08526731842285], ["ricordi", 2.0851995517268342], ["radicchio", 2.0851698022611074], ["manchego", 2.0851698022611074], ["indiewire", 2.0851698022611074], ["nutty", 2.0849889255094882], ["longwinded", 2.0847353617161537], ["hollywoodland", 2.0847353617161537], ["braj", 2.0847353617161537], ["pluralization", 2.0847353617161537], ["awkward", 2.084570265397471], ["craggy", 2.084331553194255], ["alluring", 2.084188545883573], ["orcas", 2.0841885458835727], ["balaji", 2.0841885458835727], ["eyewear", 2.084106251461008], ["passcodes", 2.084031260505421], ["arta", 2.084031260505421], ["paperclip", 2.083476246312184], ["accompany", 2.083458787989356], ["foggy", 2.083388166257657], ["neons", 2.0832876647312566], ["beltane", 2.0832876647312566], ["muito", 2.0832876647312566], ["magie", 2.0832876647312566], ["cissp", 2.0832876647312566], ["psychedelics", 2.083154722200099], ["referrals", 2.083089290321398], ["inadequacy", 2.0830625661181035], ["rendezvous", 2.0828826633360267], ["anmol", 2.0825225278612836], ["dissemination", 2.082416276711903], ["intoxicating", 2.0822738113444057], ["clever", 2.082197478048734], ["ranger", 2.0821944511897574], ["mercado", 2.0821936298310404], ["volcanic", 2.0821571730318342], ["pedantic", 2.0821254229091126], ["conceding", 2.0820310008050185], ["refusing", 2.0819537090161244], ["petitions", 2.0817903823947708], ["faintest", 2.081627900745858], ["unrealistic", 2.0814714069806097], ["implicitly", 2.081269514306705], ["doraemon", 2.0811503701607106], ["papaya", 2.0810410341023076], ["rehashing", 2.0809870273799116], ["govind", 2.0809870273799116], ["rucellai", 2.080987027379911], ["agvs", 2.080987027379911], ["sm\u00f6rg\u00e5sbord", 2.080987027379911], ["paleobotanist", 2.080987027379911], ["hbot", 2.080987027379911], ["hikawa", 2.080987027379911], ["assyriology", 2.080987027379911], ["hangnails", 2.080987027379911], ["meadows", 2.0809468072982074], ["monuments", 2.080897188850472], ["disclaimers", 2.0808451160987205], ["impatient", 2.080693930615492], ["foes", 2.0804068265358464], ["huish", 2.080028488905761], ["posso", 2.080028488905761], ["luhrmann's", 2.080028488905761], ["jessa", 2.0799456412495383], ["spirals", 2.0798548790775713], ["snail's", 2.079809998518271], ["toho", 2.079809998518271], ["relaying", 2.0798099985182708], ["mainstream", 2.079757872954399], ["neema", 2.07973887201786], ["uninterrupted", 2.079728003249258], ["granger's", 2.079599085832081], ["informaci\u00f3n", 2.079599085832081], ["transcendentalist", 2.079599085832081], ["serpents", 2.079533993263272], ["novak's", 2.0794624947591203], ["kyrgyzstan's", 2.0794624947591203], ["bcaas", 2.0794624947591203], ["caterpillar's", 2.079198469556637], ["monetizing", 2.0789769415614026], ["heard", 2.07882600238994], ["upvote", 2.0788238392017617], ["lumbered", 2.078601790084137], ["arifa", 2.0784727458325336], ["galbatorix", 2.0784727458325336], ["karaage", 2.0784727458325336], ["eisenstein's", 2.0784727458325336], ["columbus's", 2.078250686778491], ["bhanu", 2.078250686778491], ["fencing", 2.0782419237042515], ["roadmaps", 2.078063726479213], ["intermittently", 2.0779856037827287], ["desist", 2.0779856037827287], ["eniac", 2.0779341697686258], ["promisingly", 2.0779341697686258], ["skimped", 2.0779341697686258], ["waitangi", 2.0777334516648085], ["sophomore", 2.077605831817371], ["douaumont", 2.077540416550611], ["melanism", 2.077540416550611], ["mevlana", 2.077540416550611], ["buried", 2.077367780864318], ["epitome", 2.077173700366388], ["grieved", 2.0771438684245767], ["kicker", 2.077140845130863], ["doch", 2.07704576407048], ["unite", 2.076970008728434], ["subjugation", 2.076888249034907], ["fusing", 2.0768030010451755], ["mcvie", 2.0766371381086324], ["quarantines", 2.07644586638181], ["excruciatingly", 2.076262564310272], ["museum's", 2.0762031491560493], ["stigma", 2.07608674395594], ["exfiltrate", 2.0760867439559396], ["spacefaring", 2.0760867439559396], ["beninese", 2.0760867439559396], ["shaggy's", 2.0760867439559396], ["defacement", 2.0760184089194387], ["anemometer", 2.0760184089194387], ["olympiad", 2.075959517166875], ["dreadfuls", 2.0758850058601324], ["parn", 2.0758850058601324], ["argan", 2.07569268744195], ["jumpstarted", 2.07569268744195], ["swami's", 2.07569268744195], ["njord", 2.07569268744195], ["coroutines", 2.07569268744195], ["yochanan", 2.07569268744195], ["investments", 2.075651163207142], ["folly", 2.075578069177378], ["jested", 2.0755091440086275], ["multifold", 2.0755091440086275], ["shyamalan's", 2.0755091440086275], ["quintic", 2.0755091440086275], ["gleeful", 2.0753337883864575], ["varaha", 2.075333788386457], ["nepomuk", 2.075333788386457], ["whipple's", 2.075333788386457], ["hbase", 2.075333788386457], ["kachi", 2.075333788386457], ["tirana", 2.075305320158907], ["laur", 2.075277062028978], ["haab", 2.075166084645981], ["buko", 2.075166084645981], ["airspaces", 2.075166084645981], ["jed's", 2.075166084645981], ["cuthbert's", 2.075005542631078], ["moisturized", 2.074870589190602], ["abundance", 2.0743872119562723], ["attracted", 2.0741543748380895], ["impersonations", 2.0740093588462902], ["abrupt", 2.073850309048833], ["generate", 2.073800486220714], ["georgetown's", 2.073764666850202], ["frankfurt's", 2.073764666850202], ["groundstrokes", 2.0736060048930836], ["maidenhair", 2.0736060048930836], ["tighter", 2.0734150148663173], ["geology", 2.0734008204908005], ["handmaid", 2.072983231120758], ["runestone", 2.0729626558090377], ["kawi", 2.0729626558090377], ["balloon's", 2.0729626558090377], ["perovskite", 2.0728819172440596], ["psychographic", 2.072727581608795], ["lala's", 2.072727581608795], ["conure", 2.072727581608795], ["helga's", 2.072378825453591], ["downspouts", 2.072378825453591], ["unscripted", 2.072306853972097], ["diodorus", 2.072191646875347], ["swimsuits", 2.071963958528256], ["bugs", 2.0715571001509447], ["quelling", 2.0713395943889674], ["workington", 2.0713395943889674], ["casper's", 2.0712432754122463], ["europa's", 2.0712432754122463], ["senselessness", 2.0712432754122463], ["spearheaded", 2.071243275412246], ["stones", 2.0709920906298165], ["figment", 2.0709822820559696], ["courtrooms", 2.0708654951455734], ["teasing", 2.070210084811248], ["traps", 2.070164502872969], ["cheetahs", 2.0701498145933117], ["jumping", 2.0697393395780774], ["maglev", 2.0695797476647626], ["tray", 2.0692661655982683], ["bending", 2.0691150080852796], ["grief", 2.068474425894768], ["sustaining", 2.0684627771772077], ["buildup", 2.067647366947989], ["alaikum", 2.067647366947989], ["shieldmaiden", 2.067647366947989], ["oraz", 2.067647366947989], ["kendi", 2.067647366947989], ["kulturen", 2.067647366947989], ["polenta", 2.067647366947989], ["evet", 2.067647366947989], ["protostar", 2.067647366947989], ["schoolyards", 2.067647366947989], ["ulrich's", 2.067647366947989], ["chacun", 2.067647366947989], ["parching", 2.067647366947989], ["roominess", 2.067647366947989], ["flowrate", 2.067647366947989], ["zaha's", 2.067647366947989], ["reptile's", 2.067647366947989], ["m\u00eame", 2.067647366947989], ["kanzashi", 2.067647366947989], ["lightstream", 2.067647366947989], ["felting", 2.067647366947989], ["outlandishly", 2.067647366947989], ["hornsby's", 2.067647366947989], ["warsan", 2.067647366947989], ["gonz\u00e1lez's", 2.067647366947989], ["mystique's", 2.067647366947989], ["geographia", 2.067647366947989], ["blanket's", 2.067647366947989], ["restring", 2.067647366947989], ["stockier", 2.067647366947989], ["werft", 2.067647366947989], ["sulcata", 2.067647366947989], ["batsy", 2.067647366947989], ["anicca", 2.067647366947989], ["eilis", 2.067647366947989], ["vallenato", 2.067647366947989], ["tinha", 2.067647366947989], ["kainji", 2.067647366947989], ["stessa", 2.067647366947989], ["iyer's", 2.067647366947989], ["m\u00e9dicos", 2.067647366947989], ["minkes", 2.067647366947989], ["croagh", 2.067647366947989], ["tearjerking", 2.067647366947989], ["shopaholics", 2.067647366947989], ["spalted", 2.067647366947989], ["instituciones", 2.067647366947989], ["brochet", 2.067647366947989], ["checkride", 2.067647366947989], ["del\u00e0", 2.067647366947989], ["alhazen", 2.067647366947989], ["molti", 2.067647366947989], ["brezhnev's", 2.067647366947989], ["showpieces", 2.067647366947989], ["lynx's", 2.067647366947989], ["\u043d\u043e\u0432\u044b\u0439", 2.067647366947989], ["gule", 2.067647366947989], ["bromelain", 2.067647366947989], ["kyla's", 2.067647366947989], ["mariehamn", 2.067647366947989], ["j\u00f3hann", 2.067647366947989], ["mixins", 2.067647366947989], ["rhiana", 2.067647366947989], ["mazzoli", 2.067647366947989], ["meister's", 2.067647366947989], ["mim\u00ec", 2.067647366947989], ["crostata", 2.067647366947989], ["tempelhofer", 2.067647366947989], ["iciest", 2.067647366947989], ["protonvpn", 2.067647366947989], ["mehldau", 2.067647366947989], ["nijinsky's", 2.067647366947989], ["pantalla", 2.067647366947989], ["neuroptera", 2.067647366947989], ["elyn", 2.067647366947989], ["insolite", 2.067647366947989], ["parvanov", 2.067647366947989], ["shemp", 2.067647366947989], ["meletus", 2.067647366947989], ["kampala's", 2.067647366947989], ["herge", 2.067647366947989], ["clam's", 2.067647366947989], ["fieldcraft", 2.067647366947989], ["plotholes", 2.067647366947989], ["linet", 2.067647366947989], ["cannaregio", 2.067647366947989], ["xenophanes", 2.067647366947989], ["mon\u00e9gasque", 2.067647366947989], ["choropleth", 2.067647366947989], ["hippocampi", 2.067647366947989], ["placentals", 2.067647366947989], ["jasbir", 2.067647366947989], ["unremarkably", 2.067647366947989], ["sottero", 2.067647366947989], ["rcp's", 2.067647366947989], ["farke", 2.067647366947989], ["circulus", 2.067647366947989], ["sebagai", 2.067647366947989], ["hayri", 2.067647366947989], ["b\u00f6rje", 2.067647366947989], ["spmc", 2.067647366947989], ["rsas", 2.067647366947989], ["polonaises", 2.0676473669479885], ["precociousness", 2.0676473669479885], ["bimah", 2.0676473669479885], ["patwa", 2.0676473669479885], ["surgeonfish", 2.0676473669479885], ["conditionalities", 2.0676473669479885], ["poquelin", 2.0676473669479885], ["zuberi", 2.0676473669479885], ["underdog's", 2.0676473669479885], ["staley's", 2.0676473669479885], ["jenni's", 2.0676473669479885], ["diaspora's", 2.0676473669479885], ["kfcs", 2.0676473669479885], ["artboard", 2.0676473669479885], ["westlb", 2.0676473669479885], ["elam's", 2.0676473669479885], ["scrimmaged", 2.0676473669479885], ["establishing", 2.0667818401432196], ["spoons", 2.0667813366372254], ["nomadic", 2.0667284125626786], ["fondue", 2.066584309175779], ["dramas", 2.066366008016359], ["warmer", 2.066280921731571], ["penny's", 2.0662780640559704], ["interpretation", 2.0661775935184714], ["marsupial", 2.0660383028491967], ["ageism", 2.0659998391655603], ["hypocrisies", 2.06548229117108], ["otto's", 2.06537522698431], ["hordes", 2.065365825025839], ["rdbms", 2.0652734893739955], ["disorient", 2.0652734893739955], ["sabotaged", 2.0651622138627146], ["pesticide", 2.0650918477304128], ["shampoos", 2.065030091799953], ["barbershop", 2.064684515040387], ["vii's", 2.0645193073913353], ["dissonances", 2.0643705882365024], ["persuasively", 2.064035755826682], ["rimini", 2.064035755826682], ["address", 2.063969853918952], ["spinning", 2.0639234366117054], ["cadence", 2.0638880080989925], ["underwhelming", 2.0638491628296], ["obsessing", 2.0634147213657004], ["trulli", 2.063122974460138], ["shaniqua", 2.063122974460138], ["parlor", 2.063115537102623], ["etsy", 2.0629522724902905], ["bookmarked", 2.0628804565919125], ["word", 2.062873600884299], ["jabs", 2.0627438554058197], ["details", 2.062520211853564], ["evading", 2.0623286598947463], ["precariousness", 2.0623183788888446], ["postcards", 2.0622346775057165], ["fraudulent", 2.0620591308211025], ["virtuosic", 2.0620591308211025], ["nurseries", 2.0618579543205344], ["varangian", 2.0618230081678535], ["pigeonholing", 2.0618230081678535], ["question", 2.0615754989212998], ["sweetened", 2.0615660511628477], ["armitage's", 2.0614006981656687], ["knoxville's", 2.0614006981656687], ["hando", 2.0614006981656687], ["peter's", 2.0613485736447035], ["blowfish", 2.061285375049687], ["obliterating", 2.061057255818274], ["espa\u00f1a", 2.0608121690407226], ["catalyzed", 2.0607167053828], ["lessened", 2.0606855913016995], ["sanitas", 2.0606383928227414], ["adnan's", 2.0606383928227414], ["gruevski", 2.0606383928227414], ["l'hopital", 2.0606383928227414], ["reincarnation", 2.060608567400932], ["restoration", 2.0605308132198887], ["ephemera", 2.0603282612242793], ["understaffed", 2.0602059625425206], ["anabelle", 2.060017671867], ["disposal", 2.0598730128482643], ["reclined", 2.059732927265891], ["cataclysmic", 2.059694877075112], ["citt\u00e0", 2.0594748477110403], ["wise's", 2.0594748477110403], ["overeager", 2.0594748477110403], ["sanur", 2.0594748477110403], ["marty's", 2.059376777480197], ["crawfish", 2.0593212701683323], ["purchases", 2.0592806078352224], ["borers", 2.0592763249765396], ["cartons", 2.059120986053358], ["spacetime", 2.0590831470848907], ["pizzazz", 2.058987063840353], ["shoshone", 2.0588488675141674], ["outreach", 2.0587551882926385], ["preposition", 2.058722270400012], ["jaggery", 2.058657595787346], ["columbines", 2.0586183391447226], ["multifaith", 2.0586183391447226], ["orby", 2.0586183391447226], ["efficiencies", 2.0585763978542815], ["kudzu", 2.0584578230948862], ["unresponsive", 2.058335546666954], ["gonzales", 2.058248969825498], ["gullibility", 2.058248969825498], ["radian", 2.058248969825498], ["umbrella", 2.058161768202712], ["props", 2.057958962142861], ["employee's", 2.0578403438636506], ["compile", 2.057826041954986], ["stopover", 2.0577424693817705], ["michelin", 2.0576656210385846], ["vividness", 2.0575612822311693], ["bodhi", 2.0574869376263774], ["ginny's", 2.057360564127352], ["postman's", 2.057360564127352], ["phileas", 2.057360564127352], ["makeba", 2.057360564127352], ["id\u00e9es", 2.057257179174381], ["handyman", 2.057133905760118], ["galette", 2.0570657529922673], ["microwaving", 2.0570440471174862], ["pollinate", 2.0568863769341332], ["yelp", 2.056728330291073], ["feat", 2.0566848148898615], ["outperforming", 2.056433008347593], ["little", 2.0564209983083], ["damsels", 2.0563177923345752], ["bluebirds", 2.056160437131611], ["maira", 2.056096264339341], ["doodlebug", 2.056096264339341], ["alone", 2.055911685685932], ["waitlisted", 2.0558322391368575], ["harkening", 2.0558322391368575], ["dapps", 2.0558322391368575], ["undulated", 2.0558322391368575], ["ages", 2.055644033561105], ["experiments", 2.0556079012670256], ["mailchimp", 2.055499027541714], ["punjab's", 2.055499027541714], ["retinoids", 2.0554847353777066], ["adom", 2.055412767143563], ["seguridad", 2.0552662449902166], ["brutality", 2.055075239741923], ["aubrey", 2.0548766037991926], ["devices", 2.0546886927047523], ["odysseus", 2.054571336168871], ["piety", 2.054535456816123], ["waders", 2.054449617797257], ["sukarno", 2.054307706516066], ["heroics", 2.05425369979367], ["flowing", 2.054059969965188], ["slash's", 2.0539543380278036], ["vaccinology", 2.0539543380278036], ["naidoc", 2.0539543380278036], ["seward's", 2.0539543380278036], ["spongey", 2.0539543380278036], ["psychonaut", 2.0539543380278036], ["yupik", 2.0539543380278036], ["pastafarian", 2.0539543380278036], ["vincentio", 2.0539543380278036], ["lyris", 2.0539543380278036], ["stinging", 2.0538630511683356], ["gnaeus", 2.0537705389818948], ["leones", 2.0537705389818948], ["beatboxer", 2.0537705389818948], ["leaping", 2.053538714326461], ["refused", 2.053441974086882], ["waimea", 2.0533877299345544], ["tarkhan", 2.0533877299345544], ["profiterole", 2.0533877299345544], ["catadioptric", 2.0533877299345544], ["hevs", 2.0533877299345544], ["w\u00fcrttemberg", 2.0533877299345544], ["vanillas", 2.0533877299345544], ["jcvd", 2.0533877299345544], ["egpu", 2.0533877299345544], ["moinuddin", 2.0533877299345544], ["echolocating", 2.0533877299345544], ["multifunctionality", 2.0533877299345544], ["yuyuan", 2.0533877299345544], ["subplot", 2.0533004260344723], ["articulate", 2.053227463120859], ["puma's", 2.052931015581811], ["brogues", 2.0526099315520034], ["burges", 2.0526099315520034], ["newseum", 2.0526099315520034], ["uninteresting", 2.05257949139938], ["dedi", 2.0524918365096267], ["timey", 2.0522276238724446], ["mosaic", 2.052194423468693], ["hvac", 2.0521883212137983], ["questlove", 2.0521883212137983], ["attract", 2.052074579817178], ["discounts", 2.0518970547524367], ["ammonite", 2.051742387202235], ["checkouts", 2.051742387202235], ["foragers", 2.051742387202235], ["taxis", 2.0517076854282443], ["devoted", 2.05148959282217], ["gowns", 2.051152651998179], ["manny", 2.050896806000562], ["babylon's", 2.050768612932087], ["overexposure", 2.050619682749594], ["lund's", 2.050559372179823], ["keukenhof", 2.050559372179823], ["stallholder", 2.050559372179823], ["overfilling", 2.050559372179823], ["reptile", 2.0504556922115675], ["syntactically", 2.0500503680803464], ["meddlesome", 2.0500503680803464], ["fertilizer", 2.050036986332158], ["yeast", 2.049886806231897], ["anthocyanins", 2.049667824626702], ["sixthly", 2.049667824626702], ["chronicle's", 2.049667824626702], ["olimpia", 2.049667824626702], ["treacy's", 2.049667824626702], ["lascaux", 2.049667824626702], ["assunta", 2.049667824626702], ["leafy's", 2.049667824626702], ["myron's", 2.049667824626702], ["zvonimir", 2.049667824626702], ["faible", 2.049667824626702], ["dfsp", 2.049667824626702], ["muthoni", 2.049667824626702], ["interloper", 2.0494034195925654], ["greenlit", 2.0494034195925654], ["salma", 2.0493930947779897], ["somersaulted", 2.0493496026387144], ["aronofsky's", 2.0493496026387144], ["ethnobotanical", 2.0493496026387144], ["multitask", 2.0493264409117407], ["sugarloaf", 2.0493264409117407], ["supervisor's", 2.0492682792417845], ["garonne", 2.0491862297430963], ["waggy", 2.0491862297430963], ["comunicaci\u00f3n", 2.0491862297430963], ["r\u00fcgen", 2.0491862297430963], ["monetize", 2.0490833503407155], ["kula", 2.049061772638344], ["toners", 2.0488505727030075], ["teacher", 2.0486217421643724], ["hypotheses", 2.0485412583673304], ["loire", 2.0485161101719576], ["matted", 2.0485161101719576], ["tchaikovsky", 2.0485161101719576], ["irreversibly", 2.048502483920693], ["simplifies", 2.048372688103558], ["youngsters", 2.0483048593217013], ["preventable", 2.0482906681935824], ["hitchhiker's", 2.0476626712079713], ["fondant", 2.0476378763001053], ["claves", 2.0475731206669403], ["markku", 2.0475731206669403], ["clashes", 2.047519826207787], ["fairey", 2.0474751975143497], ["promotions", 2.047073761306715], ["umpteenth", 2.0469708932785093], ["cora's", 2.0469708932785093], ["limoges", 2.0469708932785093], ["soreness", 2.046970893278509], ["transcriptions", 2.046970893278509], ["meniere's", 2.046970893278509], ["criterion's", 2.046970893278509], ["pile", 2.0468809955669025], ["syriac", 2.0468351615038736], ["radicalization", 2.046828949523341], ["matzah", 2.0468146478855407], ["loudspeaker", 2.04676204000912], ["halwa", 2.04676204000912], ["shahada", 2.04676204000912], ["temporarily", 2.0467551387706537], ["podcasting", 2.0464407272869836], ["tweeds", 2.0464407272869836], ["zhuge", 2.0464407272869836], ["tuscan", 2.0463801368879526], ["rupa", 2.0461093735422806], ["smirks", 2.0459966091789], ["niro's", 2.045750950051414], ["onur", 2.045750950051414], ["rival's", 2.0457385206756924], ["roosting", 2.0457049867273165], ["disposable", 2.0456369272353165], ["plausible", 2.0454095119716604], ["yearnings", 2.045294422440443], ["dichotomies", 2.045294422440443], ["pasteurization", 2.045294422440443], ["lineages", 2.045242519318521], ["addiction", 2.0449931436231674], ["enforcing", 2.0449699442137206], ["hop's", 2.044925967311198], ["topeng", 2.0449259673111975], ["gourmet's", 2.0449259673111975], ["jiddu", 2.0449259673111975], ["pobeda", 2.0449259673111975], ["nocturno", 2.0449259673111975], ["delicacy", 2.044916938940481], ["pyrotechnics", 2.0447889538460515], ["diluted", 2.044657931817177], ["notes", 2.044586525361132], ["dolly's", 2.0443678162454013], ["grandmasters", 2.0443678162454013], ["mackie's", 2.04399007441998], ["epidaurus", 2.04399007441998], ["rustem", 2.04399007441998], ["booth", 2.043721733130448], ["erupt", 2.043713185208398], ["trek", 2.0435701574776073], ["decorating", 2.043449572435235], ["catapult", 2.0433221038074243], ["unsurprisingly", 2.0433221038074243], ["calcutta's", 2.0433221038074243], ["amoako", 2.0433221038074243], ["walpole's", 2.0433221038074243], ["chemosynthetic", 2.0433221038074243], ["nebe", 2.0433221038074243], ["personale", 2.0433221038074243], ["cuddlers", 2.0433221038074243], ["moorish", 2.0431951893910387], ["historic", 2.0431567864851097], ["sonars", 2.0430743082576055], ["tinsel", 2.0426690766090196], ["liberties", 2.0424136296942157], ["violins", 2.042378820982937], ["recurring", 2.042255206301259], ["flossing", 2.04224629364396], ["eurail", 2.0422462936439594], ["jest", 2.0421164168517616], ["pushy", 2.0420085395978336], ["though", 2.0419535768197252], ["aleksandar", 2.041671897513969], ["financi\u00e8re", 2.0414746154676346], ["munchy", 2.0414746154676346], ["zippity", 2.0414746154676346], ["mirabell", 2.0414746154676346], ["rukhsana", 2.0414746154676346], ["eon's", 2.0414746154676346], ["sawasdee", 2.0414746154676346], ["verb", 2.0414177832072773], ["wideout", 2.0413318550050144], ["graub\u00fcnden", 2.0411390673717325], ["einige", 2.0411390673717325], ["iroquoian", 2.0411390673717325], ["monocultural", 2.041070922642745], ["lyonnaise", 2.041070922642745], ["optimizers", 2.041070922642745], ["mort's", 2.041070922642745], ["brownies", 2.040968046084144], ["liguria", 2.040968046084144], ["vengeful", 2.0407679511776653], ["animism", 2.0406780534660585], ["volcanos", 2.0406780534660585], ["carousels", 2.040594036913155], ["moonbeams", 2.0404414805407787], ["detective", 2.040385053517119], ["summers", 2.040127789893236], ["zappa", 2.039893711015801], ["debunked", 2.039800264362831], ["appropriately", 2.039752924288436], ["passively", 2.0396733143363397], ["overhears", 2.0394521755805166], ["hedwig's", 2.0393234304144547], ["toolbox", 2.0392886415466553], ["coin", 2.0391271729445384], ["salsa", 2.0391028546255208], ["alcatraz", 2.038827646760383], ["thawing", 2.0384882374141067], ["amalfi", 2.03837094405315], ["worries", 2.0383295532483676], ["ascii", 2.038322560541755], ["caving", 2.0382436071154455], ["mejor", 2.0381095474201607], ["dnipro", 2.0381095474201607], ["barnacles", 2.0378817493261154], ["trinomial", 2.0378541772225134], ["rootedness", 2.0378541772225134], ["building", 2.0378223489287515], ["tuscany", 2.0377324688729965], ["outwit", 2.037522703323581], ["mugabe's", 2.037522703323581], ["decibel", 2.037360701291286], ["condescending", 2.0371249534358995], ["acadia", 2.0369280346390477], ["emilia", 2.0368526189296143], ["fractals", 2.0368526189296143], ["trailheads", 2.0367869584860787], ["restock", 2.0367869584860787], ["expr", 2.0367869584860783], ["supplies", 2.036783279761891], ["reruns", 2.036739236204952], ["tita", 2.036632656443769], ["decor", 2.036563272303938], ["unsuspected", 2.036276165518433], ["moonflower", 2.035976619533031], ["cime", 2.035976619533031], ["bavaria's", 2.035976619533031], ["ji\u0159\u00ed", 2.0358374074564813], ["freundschaft", 2.0358374074564813], ["meatiness", 2.0358374074564813], ["telecommuters", 2.0358374074564813], ["invercargill", 2.0358374074564813], ["amatrice", 2.0358374074564813], ["rashidun", 2.0358374074564813], ["deliberateness", 2.0358374074564813], ["snri", 2.0358374074564813], ["comprar", 2.0358374074564813], ["harmen", 2.0358374074564813], ["yogurt", 2.035785259981905], ["slice", 2.035681538654973], ["animators", 2.034957290000195], ["enclosure", 2.034930784532399], ["turnaround", 2.0348615640776577], ["autumn", 2.0346331732757315], ["huddles", 2.0345650090768213], ["strobes", 2.0345650090768213], ["contrails", 2.0344054478652236], ["hayrides", 2.034298215868182], ["manet's", 2.034298215868182], ["ritmo", 2.034298215868182], ["revising", 2.034236376382737], ["destiny", 2.0339247053760885], ["energies", 2.033921476178751], ["ledgers", 2.033912067803048], ["spiked", 2.033835251182604], ["insulins", 2.033751508473432], ["exactly", 2.0337379501300417], ["common", 2.033669900976193], ["composer's", 2.0334147946475256], ["climber's", 2.032994059122045], ["yoga's", 2.032994059122045], ["belted", 2.032913512606232], ["florentines", 2.0328384887165414], ["pastureland", 2.0328384887165414], ["tawa", 2.0327713872645288], ["shostakovich", 2.0322479567745604], ["orla", 2.0322479567745604], ["quadcopters", 2.032201983514595], ["rebellion", 2.0320866808895723], ["golden", 2.031675604423009], ["pianos", 2.0316698794985357], ["smithers", 2.0315676142361445], ["dabbed", 2.0314635380263995], ["biometrics", 2.031372851738375], ["grunting", 2.031372851738375], ["halloumi", 2.031159472237142], ["malwa", 2.031159472237142], ["reunite", 2.031065913532755], ["cumulus", 2.030963300760202], ["hens", 2.0309408181864406], ["built", 2.030864450455976], ["okeke", 2.030856488532544], ["gamla", 2.030856488532544], ["jason's", 2.0308147822736116], ["sericulture", 2.0307849341498887], ["plastic", 2.0307743222374164], ["gases", 2.030683838600315], ["scar's", 2.0305040010746715], ["inspires", 2.0303596246016724], ["grytviken", 2.0300537784580257], ["tomo's", 2.0300537784580257], ["tanesha", 2.0300537784580257], ["cecina", 2.0300537784580257], ["babalawo", 2.0300537784580257], ["nanotechnological", 2.0300537784580257], ["eastsiders", 2.0300537784580257], ["thebutton", 2.0300537784580257], ["pearse's", 2.0300537784580257], ["permet", 2.0300537784580257], ["nobility's", 2.0300537784580257], ["yuuki's", 2.0300537784580257], ["vidhana", 2.0300537784580257], ["yakan", 2.0300537784580257], ["sodden", 2.0300537784580253], ["ansible", 2.0300537784580253], ["prowling", 2.0300537784580253], ["readouts", 2.0300537784580253], ["excluder", 2.0300537784580253], ["farman", 2.0300537784580253], ["aperitivo", 2.0300537784580253], ["hydronium", 2.0300537784580253], ["ruel", 2.0300537784580253], ["simulated", 2.0295062990139967], ["hablo", 2.0293576008933965], ["stickered", 2.0293576008933965], ["chiropractic", 2.02925861442416], ["oiled", 2.0292482015618116], ["pisin", 2.029215259755647], ["xian's", 2.029215259755647], ["seedpods", 2.029215259755647], ["lancelot's", 2.029215259755647], ["utilis", 2.029215259755647], ["critiqued", 2.0292036721856395], ["nonrefundable", 2.0291795089582587], ["dogmatism", 2.0291795089582587], ["fiercest", 2.029140573879322], ["infringe", 2.0291302921491625], ["hymns", 2.0291268132623825], ["absolution", 2.029098009259128], ["turbojets", 2.0289997526125125], ["kapa", 2.0289997526125125], ["nicoli", 2.0289997526125125], ["raff's", 2.0289997526125125], ["sweatbands", 2.0289997526125125], ["kerouac", 2.028971948573422], ["expos", 2.028971948573422], ["native", 2.0289697021205497], ["hunts", 2.028694367445665], ["cicchetti", 2.028338481644795], ["micronation", 2.028338481644795], ["sgraffito", 2.028338481644795], ["italy's", 2.028126512212654], ["aviaries", 2.0280923738411576], ["phytoestrogens", 2.0280923738411576], ["lengua", 2.0278849175836045], ["hoarse", 2.0278327130548983], ["lobsters", 2.0278099726528653], ["amiable", 2.027609108686279], ["jusqu'\u00e0", 2.0274207644781836], ["kinjo", 2.0274207644781836], ["unasked", 2.027303028080711], ["sunny's", 2.0270328650972247], ["hawking", 2.027007401459701], ["troupe's", 2.0269825927114926], ["restored", 2.0269523074076035], ["squid's", 2.026876855487042], ["hussam", 2.026876855487042], ["roller", 2.026844663534727], ["jazeera's", 2.026838537337173], ["konstantinos", 2.026838537337173], ["stumptown", 2.0267847707632463], ["narrowly", 2.0265719561683504], ["claire", 2.02651092718672], ["myrddin", 2.026459172387272], ["beverley's", 2.026459172387272], ["agronomists", 2.026343012966654], ["volcano's", 2.0262944196090293], ["cube's", 2.0262944196090293], ["jackbox", 2.026086615853557], ["bergen's", 2.026086615853557], ["raci", 2.026086615853557], ["abrasiveness", 2.026016883049707], ["camila's", 2.026016883049707], ["hanukkah", 2.0259348287626033], ["popcorn", 2.0258323755046828], ["gaia's", 2.0258163508536655], ["emblematic", 2.0257921570907422], ["chars", 2.0257709645794217], ["prendre", 2.025450481908234], ["nephron", 2.025450481908234], ["alliums", 2.025450481908234], ["hofburg", 2.025450481908234], ["alfa's", 2.025450481908234], ["cygnets", 2.025450481908234], ["jummah", 2.025450481908234], ["delicioso", 2.025450481908234], ["rascal's", 2.025450481908234], ["prayagraj", 2.025450481908234], ["porthcurno", 2.025450481908234], ["favore", 2.025450481908234], ["arcand", 2.025450481908234], ["appetizer", 2.02538054231977], ["vineyard", 2.025341037537814], ["tulum", 2.0252979630466443], ["silverfish", 2.0252340876259787], ["galvanize", 2.025078156452001], ["flavorings", 2.025078156452001], ["noisemakers", 2.025049243698664], ["jun's", 2.024960453565837], ["humblest", 2.024960453565837], ["fast", 2.024612733083911], ["koala's", 2.0245713801365723], ["flaunting", 2.0241179486964525], ["yangon's", 2.0241179486964525], ["bai's", 2.0241179486964525], ["buang", 2.0241179486964525], ["clear", 2.0240130379075203], ["rendering", 2.0237475606157056], ["fusions", 2.0237327326057306], ["physicist's", 2.0236548697788828], ["einkorn", 2.0236548697788828], ["haworthia", 2.0236548697788828], ["whaler's", 2.0236548697788828], ["hustler's", 2.0236548697788828], ["huntresses", 2.0236548697788828], ["heliconia", 2.0236548697788828], ["outpace", 2.0236214154084116], ["burden", 2.0235456648741454], ["variations", 2.0234703158883196], ["polishing", 2.0232633135213565], ["hands", 2.0232200418689326], ["trekkers", 2.023202610462191], ["amalgamation", 2.023192948558607], ["xanthe", 2.023021884208104], ["avestan", 2.023021884208104], ["chemicals", 2.0228297132103594], ["mariah", 2.0226302344017792], ["kuta", 2.022593572181807], ["train's", 2.022403442069477], ["antiquity", 2.022297480461025], ["aboveground", 2.022159124875133], ["n\u00fcrburgring", 2.022159124875133], ["assistantships", 2.022011686522991], ["dewdrop", 2.022011686522991], ["vinous", 2.022011686522991], ["dharmic", 2.022011686522991], ["volleyed", 2.02195350248505], ["retrieve", 2.021905943564895], ["barroso's", 2.021699647682478], ["kunas", 2.021699647682478], ["monemvasia", 2.021699647682478], ["hushpuppy", 2.021699647682478], ["farhan's", 2.021699647682478], ["distraction", 2.021562490311536], ["exosuits", 2.0211833811738766], ["mythologist", 2.0211833811738766], ["marketplace's", 2.0211833811738766], ["valgrind", 2.0211833811738766], ["sindh's", 2.0211833811738766], ["corp's", 2.021091068619359], ["tiles", 2.020974157378286], ["bartolomeo", 2.0209585554362604], ["ietf", 2.0209585554362604], ["aarhus", 2.020853242327587], ["cracked", 2.020622674528762], ["enterprise's", 2.0203868557034634], ["days", 2.020257092506127], ["synergistic", 2.0201648595650497], ["kallio", 2.0201648595650497], ["mussorgsky", 2.0201648595650497], ["ji's", 2.020147359869454], ["emptier", 2.020147359869454], ["soccer", 2.020146249623866], ["favelas", 2.0200156998559935], ["team's", 2.0199401862374327], ["terrifying", 2.019750350501517], ["grounding", 2.0197052137143596], ["amenemhat", 2.0195625444608263], ["desayuno", 2.0195625444608263], ["cashmeres", 2.0195625444608263], ["ruggers", 2.0195625444608263], ["neuroplastic", 2.0195625444608263], ["novruz", 2.0195625444608263], ["stopsmoking", 2.0195625444608263], ["lionhearts", 2.0195625444608263], ["brainchild", 2.01946219352694], ["churchill's", 2.0192813466685036], ["novelties", 2.0192247119843354], ["decomposing", 2.0191510420179886], ["wools", 2.0189968406668597], ["pictionary", 2.0189968406668597], ["surfacing", 2.018938366476618], ["ibrahim's", 2.018775701911036], ["silverman's", 2.018708967730285], ["syncing", 2.0186904771512353], ["stasia", 2.018584548071257], ["neumann's", 2.018584548071257], ["indispensability", 2.018584548071257], ["hotspurs", 2.018584548071257], ["rundown", 2.0183090228815055], ["isolate", 2.0182939750233513], ["seismic", 2.0181906720037], ["shabaka", 2.018182118934879], ["produit", 2.018182118934879], ["freq", 2.0181821189348788], ["bursts", 2.0179191391353157], ["calicut", 2.0179191391353157], ["cassia", 2.0178706710770187], ["makossa", 2.01782453882876], ["tui's", 2.01782453882876], ["pottered", 2.01782453882876], ["tabouli", 2.01782453882876], ["pico's", 2.01782453882876], ["plates", 2.017749804586581], ["malevolence", 2.0174054683118694], ["mezes", 2.017216943363892], ["mayombe", 2.017216943363892], ["japonisme", 2.017216943363892], ["concatenates", 2.017216943363892], ["musicmaster", 2.017216943363892], ["push's", 2.017216943363892], ["regierung", 2.017216943363892], ["savoy's", 2.017216943363892], ["paddleball", 2.017216943363892], ["capitole", 2.017216943363892], ["turbulently", 2.017216943363892], ["snowscape", 2.017216943363892], ["patternmaking", 2.017216943363892], ["marceline's", 2.017216943363892], ["jahnavi", 2.017216943363892], ["feiyu", 2.017216943363892], ["dierker", 2.017216943363892], ["marble", 2.0169083887985413], ["intonation", 2.016852561690596], ["aurelius", 2.016710187081523], ["shimmery", 2.0166642811876274], ["starlings", 2.016413626882021], ["titus", 2.0164013700064074], ["sitti", 2.016383382643493], ["neuk\u00f6lln", 2.016383382643493], ["streetball", 2.016383382643493], ["adapted", 2.0162914769419022], ["hatten", 2.016213352844805], ["ustedes", 2.016213352844805], ["santiago", 2.0161597804362423], ["triceratops", 2.015956182774289], ["bloodedness", 2.015956182774289], ["linens", 2.01572157830754], ["preying", 2.0156929000026067], ["transfrontier", 2.015601323386369], ["sand's", 2.015601323386369], ["marnie's", 2.015601323386369], ["willed", 2.0148563703450617], ["pest", 2.014749660855361], ["cesky", 2.0146307677954765], ["giray", 2.0146307677954765], ["havan", 2.0146307677954765], ["glagolitic", 2.0146307677954765], ["bacchanalia", 2.0146307677954765], ["erza's", 2.0146307677954765], ["mohu", 2.0146307677954765], ["\u00f6tzi", 2.0146307677954765], ["fortuity", 2.0146307677954765], ["boxwoods", 2.0146307677954765], ["trebek's", 2.0146307677954765], ["gurdwaras", 2.0146307677954765], ["tetradrachms", 2.0146307677954765], ["motherwell's", 2.0146307677954765], ["zondi", 2.0146307677954765], ["alge", 2.0146307677954765], ["hypochromic", 2.0146307677954765], ["mirela", 2.0146307677954765], ["rebecca", 2.0144598590218923], ["continuity", 2.0144291426985292], ["farouk", 2.0143791635961086], ["gandalf's", 2.014357486356546], ["isambard", 2.014357486356546], ["koro", 2.0142887252202986], ["happyness", 2.0142395013810668], ["dollop", 2.0140967876601272], ["maturely", 2.013708739984128], ["fail2ban", 2.0135204725252667], ["bicorne", 2.0135204725252667], ["macedonia's", 2.0135204725252667], ["mediana", 2.0135204725252667], ["carr\u00e9's", 2.0135204725252667], ["leyenda", 2.0135204725252667], ["baug", 2.0135204725252667], ["bowstrings", 2.0135204725252667], ["predictable", 2.0134210673069877], ["tyler", 2.0132738294333836], ["lovestruck", 2.013235594133568], ["mayor", 2.0132340283271564], ["iodized", 2.01297399907196], ["conceptualization", 2.0128752512672476], ["lanois", 2.012875251267247], ["letzten", 2.012875251267247], ["magomed", 2.012875251267247], ["oceanica", 2.012875251267247], ["nkem", 2.012875251267247], ["superpositions", 2.012754074020166], ["saverin", 2.012754074020166], ["geben", 2.012754074020166], ["streusel", 2.012754074020166], ["thando", 2.012754074020166], ["mostar", 2.0125101038293756], ["tease", 2.012368362793081], ["cumbersome", 2.012319975335863], ["objects", 2.012280365233048], ["strudel", 2.012246289173306], ["helios", 2.0121400550836133], ["vedas", 2.01209863171655], ["dissenters", 2.0120742361367387], ["motty", 2.0117650056791243], ["popper's", 2.0117650056791243], ["grandmaster's", 2.0117650056791243], ["timber's", 2.0117650056791243], ["amtrak's", 2.0117650056791243], ["format's", 2.0117650056791243], ["ibirapuera", 2.0117650056791243], ["salvar", 2.0117650056791243], ["chimborazo", 2.0117650056791243], ["tokusatsu", 2.0117650056791243], ["ecatepec", 2.0117650056791243], ["nationalgalerie", 2.0117650056791243], ["cronulla's", 2.0117650056791243], ["nyos", 2.0117650056791243], ["taua", 2.0117650056791243], ["mediator", 2.0116618382429357], ["operas", 2.0116376787800307], ["parentheses", 2.0115911494440657], ["competitively", 2.0114497718463253], ["carly's", 2.011256984213044], ["geometrical", 2.011160093521059], ["variegated", 2.0111217554774683], ["tutorial", 2.011090166387732], ["cycling", 2.0110332128529844], ["manifesting", 2.010922381592976], ["mick's", 2.0108438678560114], ["defused", 2.010767848303323], ["peelings", 2.010678152623652], ["cistercians", 2.010678152623652], ["barrenness", 2.010678152623652], ["digested", 2.010482998567662], ["hypothesizing", 2.010439732289349], ["passage", 2.0103535253928815], ["drinks", 2.0103066630170146], ["rockstars", 2.0100673136912093], ["synchronization", 2.009945579963394], ["braniff", 2.009945579963394], ["sulfites", 2.009891853904749], ["snowcapped", 2.009891853904749], ["kilda's", 2.009891853904749], ["cursing", 2.009874866989148], ["escaping", 2.009660831852576], ["twisted", 2.009493431580216], ["ngos", 2.0093327022259304], ["effectiveness", 2.009310173380528], ["psychokinetic", 2.0091791894764524], ["bridgeville", 2.0091791894764524], ["burundi's", 2.0091791894764524], ["agneta", 2.0091791894764524], ["economist's", 2.0090500731478436], ["allies", 2.009019559349689], ["matchstick", 2.0089528610475296], ["rejuvenates", 2.0088302646059235], ["unschooling", 2.0088302646059235], ["geli", 2.0088302646059235], ["austin's", 2.0088052284419593], ["vinegary", 2.0087819614832774], ["fanfares", 2.0087819614832774], ["unrepaired", 2.0087819614832774], ["diagon", 2.008571727892332], ["getan", 2.008571727892332], ["omnipresence", 2.008571727892332], ["borgia's", 2.008571727892332], ["informativeness", 2.008571727892332], ["dehumidifiers", 2.008571727892332], ["linocut", 2.008571727892332], ["kadane", 2.008571727892332], ["farfisa", 2.008571727892332], ["oyin", 2.008571727892332], ["koontz's", 2.008571727892332], ["bioprinted", 2.008571727892332], ["couloirs", 2.008571727892332], ["daehyun", 2.008571727892332], ["bardem's", 2.008571727892332], ["klomp", 2.008571727892332], ["devotedly", 2.008300427598161], ["edric", 2.008300427598161], ["vonnegut's", 2.0082142353715717], ["spectral", 2.008202505148234], ["reconciled", 2.0079457608559808], ["hinders", 2.0078887725275267], ["souq", 2.0078119125990117], ["deity", 2.0076564715125964], ["squashes", 2.0076188950043377], ["presley's", 2.0076188950043377], ["build", 2.0075540157467824], ["bosniaks", 2.007497625364047], ["wargame", 2.007244185441643], ["outputs", 2.00722142757106], ["hypothyroidism", 2.007215084824248], ["mortadella", 2.0071898415986325], ["thermocline", 2.0071898415986325], ["ginsburg's", 2.0071898415986325], ["ruminating", 2.00702524519454], ["fare", 2.0068761010537655], ["faroese", 2.0068342090965774], ["tuvalu", 2.0068342090965774], ["bioenergy", 2.0068342090965774], ["abisko", 2.0068342090965774], ["madge's", 2.0068342090965774], ["chargepoint", 2.0068342090965774], ["astley's", 2.0068342090965774], ["eunice's", 2.0068342090965774], ["oluwa", 2.0068342090965774], ["agape", 2.0065893238971757], ["cubesats", 2.0065169230556137], ["trentino", 2.0065169230556137], ["adelphia", 2.0065169230556137], ["feverish", 2.0063373996375797], ["form's", 2.006274006004804], ["bellboy", 2.006274006004804], ["spelunking", 2.006274006004804], ["dissonance", 2.006208702330106], ["twirls", 2.00605787091124], ["risky", 2.006021505127128], ["drooping", 2.0059900161598225], ["choruses", 2.005926550024168], ["anthropomorphism", 2.0058111840112267], ["terraforming", 2.0058111840112267], ["sherwani", 2.005741757159127], ["h\u00f6lderlin", 2.005741757159127], ["tamarindo", 2.005741757159127], ["fatih", 2.0056179459395493], ["whittaker", 2.005617945939549], ["pleaded", 2.0055727020146703], ["behaviorists", 2.005388571429745], ["forest", 2.0052217500369003], ["muhammad's", 2.005180679124787], ["mika", 2.005178020967858], ["emit", 2.0051533060818083], ["reinvigorating", 2.0049913861313833], ["sandpipers", 2.0049913861313833], ["fernando's", 2.0049913861313833], ["madrasas", 2.0049913861313833], ["pressurizing", 2.0049913861313833], ["exfoliator", 2.0048183033114477], ["angle's", 2.0048183033114477], ["raffaello", 2.0048183033114477], ["rebooking", 2.00469791549524], ["nahua", 2.00469791549524], ["pcos", 2.0046251298760875], ["heartbeat", 2.0045576139564987], ["frontliners", 2.0044441723251247], ["linode", 2.0044441723251247], ["xerophytic", 2.0044441723251247], ["igreja", 2.0044441723251247], ["ducatis", 2.0044441723251247], ["ambedkar's", 2.0044441723251247], ["wearability", 2.0044441723251247], ["tafari", 2.0044441723251247], ["gesticulations", 2.0044441723251247], ["taleo", 2.0044441723251247], ["breadfruit", 2.004027447964974], ["huan", 2.004027447964974], ["astronomically", 2.003479000387534], ["kerman", 2.003479000387534], ["juiciest", 2.003320559976274], ["father's", 2.0030731491802287], ["coconut", 2.002430663437139], ["sasha", 2.0024262755073474], ["crabtree's", 2.0022154882471033], ["profit's", 2.0021023495519428], ["conversos", 2.0021023495519428], ["nicaea", 2.0020530366861906], ["specialties", 2.0019691564684643], ["afresh", 2.0018118288959137], ["spuds", 2.0017630101706634], ["soundstage", 2.0017462556508416], ["replies", 2.0016497338211643], ["schleswig", 2.001544142577161], ["drove", 2.0014645931500463], ["holding", 2.00144001930118], ["seclusion", 2.0010452642626446], ["heroines", 2.0009929452503235], ["thousands", 2.0009911900318453], ["doled", 2.0009490647883763], ["concision", 2.0009490647883763], ["pounced", 2.0009490647883763], ["vocabulario", 2.0009490647883763], ["xiaoyun", 2.0009490647883763], ["phpmyadmin", 2.0009490647883763], ["farnesina", 2.0009490647883763], ["wienermobile", 2.0009490647883763], ["canaan's", 2.0009490647883763], ["congresswoman's", 2.0009490647883763], ["transmilenio", 2.0009490647883763], ["oumou", 2.0009490647883763], ["makani", 2.000949064788376], ["syscall", 2.000949064788376], ["gere's", 2.000949064788376], ["mohamed's", 2.000949064788376], ["hellbenders", 2.000949064788376], ["hoverfly", 2.000949064788376], ["zuzanna", 2.000949064788376], ["prepend", 2.000949064788376], ["phenomenon", 2.000563696820343], ["cena's", 2.0005300178638135], ["geographically", 2.0004488275221792], ["interdependency", 2.000357068023646], ["exonerating", 2.000328615465961], ["gondoliers", 2.000132350884381], ["samanta", 2.000132350884381], ["shain", 2.000132350884381], ["aerate", 1.9999410299950322], ["gehry", 1.999669919267507], ["heracles", 1.9993949295924245], ["sawdust", 1.9993098692268263], ["twisting", 1.999239779485574], ["dieticians", 1.998980111219089], ["leaf's", 1.998980111219089], ["applesauce", 1.9988871968996014], ["magda", 1.9985218780529823], ["grandkids", 1.9984052225664748], ["sunpower", 1.998370522179113], ["hairstylist", 1.9982895755250498], ["leblanc's", 1.9982632271175194], ["tanay", 1.9982632271175194], ["reynald", 1.9982632271175194], ["shellfish", 1.998139221420801], ["siren's", 1.9980630805603161], ["catamaran", 1.99800029774553], ["pancetta", 1.997665456066672], ["constructs", 1.9976141496803956], ["neuromancer", 1.9975576256955145], ["regretted", 1.9974921722406485], ["osteoporosis", 1.9973854502819977], ["bermudian", 1.9973473564717572], ["namibia's", 1.9973473564717572], ["bickle", 1.9973473564717572], ["winded", 1.9973306143878145], ["footfalls", 1.997259371477419], ["palawan", 1.99693228971408], ["lenin's", 1.9968036723035558], ["unappealing", 1.9965243616967068], ["diatomaceous", 1.996443616563025], ["mcgreggor", 1.9963491818808166], ["clonmacnoise", 1.9963491818808166], ["utnapishtim", 1.9963491818808166], ["targaryen's", 1.9963491818808166], ["flakier", 1.9963491818808166], ["immunomodulators", 1.9963491818808166], ["admirer", 1.9961467870328844], ["instructors", 1.9959466921264053], ["tyranny", 1.9958106759513063], ["chloe's", 1.9957291976628415], ["col\u00f3n", 1.9957291976628415], ["chemnitz", 1.9957291976628415], ["aten", 1.9957291976628415], ["lamplighter", 1.9956397472035314], ["mari's", 1.9956397472035314], ["mbti", 1.995404266175107], ["pskov", 1.9952797091048091], ["nutmeg", 1.9950917411623592], ["linny", 1.9949160525327332], ["daruma", 1.9949160525327332], ["espoir", 1.9949160525327332], ["imagine", 1.9948125003247776], ["shed", 1.9947202280525762], ["neologisms", 1.994671577526295], ["anyone's", 1.9946406181004797], ["polyester", 1.994558436767502], ["mesmerize", 1.9945151449659058], ["dexterity", 1.9944844601175216], ["discriminatory", 1.9944181893685808], ["elixir", 1.9943528326989357], ["unesco's", 1.9942792345724152], ["superuser", 1.9942653100834464], ["wakeboarding", 1.9942168997105836], ["udine", 1.9941310161231716], ["typewritten", 1.9941310161231716], ["yari", 1.9940946938154516], ["steins", 1.9940051319608003], ["catalyze", 1.9939366753166103], ["stabilized", 1.993845216912429], ["karahan", 1.993802818128418], ["sv's", 1.993802818128418], ["k\u00e1lm\u00e1n", 1.993802818128418], ["hadary", 1.993802818128418], ["pawprints", 1.993802818128418], ["tarsals", 1.993802818128418], ["apoe4", 1.993802818128418], ["crypto's", 1.993802818128418], ["tanjiro", 1.993802818128418], ["repr\u00e9sentant", 1.993802818128418], ["spinneret", 1.993802818128418], ["hasis", 1.993802818128418], ["yasodhara", 1.993802818128418], ["beanstalks", 1.993802818128418], ["netherland's", 1.993802818128418], ["rajamani", 1.993802818128418], ["ni\u00f1as", 1.993802818128418], ["byutv", 1.993802818128418], ["signe", 1.9936093389612048], ["isabel's", 1.993576069257938], ["ragu", 1.9934972544781302], ["finalized", 1.993455314369267], ["passphrase", 1.9934241281344711], ["hispania", 1.9934241281344711], ["corrected", 1.993295603045617], ["chives", 1.9932715623815145], ["wongs", 1.9932715623815145], ["asafoetida", 1.9932715623815145], ["jabulani", 1.9932715623815145], ["noether", 1.9932715623815145], ["heed", 1.9928601217605082], ["contours", 1.9926350438680154], ["synthetics", 1.9924601899680618], ["typo", 1.992385002791082], ["l\u00e9on", 1.9923204368509988], ["perseid", 1.992307989204945], ["thumbelina", 1.992307989204945], ["limped", 1.9917989962006506], ["ranks", 1.9914363175286], ["wonderland", 1.9913130744280358], ["nara", 1.9911101439725662], ["winelands", 1.991067834838804], ["uncaptured", 1.991067834838804], ["schaffen", 1.991067834838804], ["schilt", 1.991067834838804], ["corkhill", 1.991067834838804], ["fivesome", 1.991067834838804], ["ambivert", 1.991067834838804], ["gameshows", 1.991067834838804], ["anthracnose", 1.991067834838804], ["hammerfest", 1.991067834838804], ["amantes", 1.991067834838804], ["avionic", 1.991067834838804], ["qbert", 1.991067834838804], ["sanni", 1.991067834838804], ["mabior", 1.991067834838804], ["scarff", 1.991067834838804], ["strung", 1.9910227880552107], ["spaceship", 1.9909006306202248], ["mimi's", 1.9907116509685288], ["dribbling", 1.9906076213044939], ["keyframes", 1.9905713710974156], ["dragons", 1.9905536579803567], ["signature", 1.9904474545843014], ["feuds", 1.9903770400903966], ["clothing", 1.9903243643982593], ["pricing", 1.9896782644198048], ["grabbing", 1.9895752490516845], ["freemasonry", 1.9890998692091824], ["strauss's", 1.9888798482071133], ["dusting", 1.988879848207113], ["saladin", 1.9886819398685682], ["ashbury", 1.9884608720435977], ["screenwriter", 1.988294413946201], ["oxford's", 1.9881919222792452], ["tandoori", 1.9881919222792452], ["slavery's", 1.98812246821922], ["oblige", 1.9878952542228523], ["incrementally", 1.9878434334868382], ["mobility", 1.9878214096740328], ["appalachian", 1.9877293114165162], ["semana", 1.9876094043564536], ["persona", 1.9874732853724546], ["queenstown", 1.987410304947022], ["swiping", 1.9873900020467181], ["opponents", 1.9872766975315634], ["offsite", 1.987238858233345], ["bargained", 1.9870441645076433], ["andy's", 1.9869823932903234], ["achieve", 1.9866756281262192], ["speakers", 1.9866526287116815], ["tricksters", 1.9866179431081352], ["afterlife", 1.9865666972746572], ["anita's", 1.9865118373588904], ["matzo", 1.9865118373588904], ["urquell", 1.986440672777826], ["oils", 1.9859695842204854], ["prolonged", 1.985901680025618], ["giants", 1.9857928564799887], ["quartic", 1.985727899160826], ["cronenberg's", 1.985727899160826], ["lurks", 1.985692206563666], ["preservatives", 1.985692206563666], ["tinge", 1.985675548258335], ["dismantling", 1.985654454120741], ["gimmicks", 1.9855809508371667], ["verification", 1.98543671714838], ["son's", 1.985268373039547], ["dimitrios", 1.9849414722700696], ["breezed", 1.9849414722700696], ["albuquerque's", 1.9849414722700693], ["haute", 1.9849414722700693], ["perpetuates", 1.9849414722700693], ["einem", 1.9849414722700693], ["ramanujan", 1.9849414722700693], ["honeyeater", 1.9849414722700693], ["walkouts", 1.9849414722700693], ["listicle", 1.9849414722700693], ["jatte", 1.9849414722700693], ["klammer", 1.9849414722700693], ["meteora", 1.9849414722700693], ["mangueira", 1.9849414722700693], ["chouchou", 1.9849414722700693], ["stallholders", 1.9849414722700691], ["gavrilo", 1.9849414722700691], ["kroger's", 1.9849414722700691], ["brunello", 1.9849414722700691], ["ginsberg's", 1.9849414722700691], ["taal", 1.9849414722700691], ["entertainment", 1.9847544488901445], ["ability", 1.9844517663147394], ["francesco", 1.9843333406915553], ["todos", 1.9843318465107727], ["adequate", 1.9840553376842345], ["older", 1.983942440729789], ["acrobatic", 1.9837892229634106], ["peshwa", 1.9837207210571113], ["exclusionary", 1.9834557376351365], ["serbo", 1.9834098816278856], ["sukhumvit", 1.9831521620486718], ["shanice", 1.9831521620486718], ["kima", 1.9831521620486718], ["suprematism", 1.9830967311248369], ["jungle", 1.983061792845571], ["darryl's", 1.9830068314588898], ["partook", 1.982820808303969], ["bacon's", 1.9827745055972854], ["apron", 1.9827745055972852], ["bigwig", 1.9827261358054822], ["havisham", 1.9827261358054822], ["zacatecas", 1.9827261358054822], ["doorframes", 1.9826755573473867], ["overindulgent", 1.9826755573473867], ["elaborative", 1.9826755573473867], ["bune", 1.9826755573473867], ["kacper", 1.9826755573473867], ["hypostyle", 1.9826755573473867], ["spacesuit", 1.9825671403654401], ["priciest", 1.9823835580016804], ["extremadura", 1.9823835580016804], ["scarred", 1.9822735401836848], ["magnifiers", 1.9822073931071627], ["bongani", 1.9822073931071627], ["luffy's", 1.9822073931071627], ["chosin", 1.9822073931071627], ["gayatri", 1.9819759222616977], ["workbenches", 1.9816839096788166], ["homebuying", 1.9816839096788166], ["tohono", 1.9816839096788166], ["esperanza", 1.9816110335582064], ["fortress", 1.9815714527925308], ["glaube", 1.981495393325156], ["setrakian", 1.981495393325156], ["adage", 1.9813213489339987], ["tenaya", 1.9812837177142892], ["yasur", 1.9812837177142892], ["farrukh", 1.9812837177142892], ["akshardham", 1.9811678192809625], ["wheelz", 1.9811678192809625], ["paramecia", 1.9811678192809625], ["intentional", 1.9811606313705072], ["karnak", 1.9810443358716332], ["lowood", 1.9809795132435821], ["kanagawa", 1.9809395741404925], ["lulls", 1.9809395741404925], ["modders", 1.9808781325110723], ["granting", 1.9807975443530128], ["lettuce", 1.9804985936303463], ["shades", 1.9804753539574613], ["remain", 1.9801883302460497], ["athletic", 1.9801482897375988], ["unaccounted", 1.980035190382396], ["moss's", 1.980035190382396], ["unstoppable", 1.9800119818588025], ["dismantle", 1.9799505993153672], ["freunde", 1.9798864430537617], ["contain", 1.9798770448586207], ["rested", 1.9798387949634626], ["salwar", 1.9796623726097766], ["knightley's", 1.9796623726097764], ["tartlet", 1.9796623726097764], ["hariharan", 1.9796623726097764], ["gillnet", 1.9796623726097764], ["linnie", 1.9796623726097764], ["freedoms", 1.9795864143412831], ["valentin's", 1.9791488415144174], ["fancypants", 1.9791488415144174], ["centuria", 1.9791488415144174], ["roquefort", 1.9790339083645037], ["rejection", 1.9790041473372715], ["utensil", 1.978944670239042], ["stagnated", 1.9788356679649879], ["communicators", 1.9786959470426748], ["curdled", 1.9786600119147844], ["jangling", 1.9785794803717678], ["bedfellows", 1.9785794803717678], ["paraphernalia", 1.9785590238061783], ["scampi", 1.9782355889178054], ["klingon", 1.9780666749794613], ["herbed", 1.9780279764301196], ["competitiveness", 1.9779026727230122], ["greenleaf", 1.9778636302532586], ["spectacularly", 1.9778523955833907], ["hypnagogia", 1.9777496553415546], ["destructuring", 1.9777496553415546], ["spiegelman's", 1.9777496553415546], ["disparately", 1.9777496553415546], ["cometa", 1.9777496553415546], ["steeplechasing", 1.9777496553415546], ["pignut", 1.9777496553415546], ["bantham", 1.9777496553415546], ["propiedad", 1.9777496553415546], ["excellente", 1.9777496553415546], ["briar's", 1.9777496553415546], ["marvelous", 1.9776223665463595], ["bloom", 1.9773609084805537], ["hanuman", 1.9773460329629133], ["tallit", 1.9772874563101592], ["leverage", 1.9772593041873376], ["kyoto", 1.9772179645958667], ["skatepark", 1.9771877946440144], ["afflicting", 1.9771877946440142], ["mies", 1.9771877946440142], ["finklestein", 1.9770333389144117], ["underreported", 1.9769376760109156], ["alighted", 1.9769376760109156], ["barolo", 1.9768726044966136], ["godforsaken", 1.9767617684008245], ["\u00e1ine", 1.9767617684008243], ["mahakali", 1.9767617684008243], ["coches", 1.9767617684008243], ["undersupplied", 1.9767617684008243], ["humor's", 1.9767617684008243], ["retiree's", 1.9767617684008243], ["\u00e9milien", 1.9767617684008243], ["lembas", 1.9767617684008243], ["coaster's", 1.9767617684008243], ["vigeland", 1.9767617684008243], ["contrada", 1.9767617684008243], ["misidentified", 1.9766313106038667], ["zilch", 1.9765307033197723], ["erfahrungen", 1.9765307033197723], ["kapha", 1.9765307033197723], ["soup", 1.9764069278192626], ["mjolnir", 1.9763856900620087], ["joggers", 1.976254638693395], ["misjudgments", 1.9761585454016175], ["hornworm", 1.9761585454016175], ["similarly", 1.9759472062238455], ["inclinations", 1.975919011032478], ["sincere", 1.9757984073237385], ["thermopylae", 1.9757519284169671], ["toshiro", 1.9757519284169671], ["bentos", 1.9756443225170481], ["be\u015fikta\u015f", 1.975565905262046], ["sasanian", 1.975565905262046], ["plotted", 1.9752113670138436], ["core", 1.975076830642284], ["decongestants", 1.9750661415622577], ["investopedia", 1.9750661415622577], ["auditions", 1.9750563055754973], ["abhaya", 1.974814219860528], ["pokimane", 1.974814219860528], ["limburger", 1.974814219860528], ["cleisthenes", 1.974814219860528], ["overclocking", 1.9745491085409066], ["seamanship", 1.9742697439245314], ["interwebs", 1.9742697439245314], ["plowed", 1.9738621867315465], ["heteronormative", 1.9736633957230805], ["bermuda's", 1.9736633957230802], ["isostasy", 1.97366339572308], ["unamir", 1.97366339572308], ["nerdfighter", 1.97366339572308], ["aquarion", 1.97366339572308], ["subcontinent's", 1.97366339572308], ["psybient", 1.97366339572308], ["sifakas", 1.97366339572308], ["fairytale", 1.9734545424536916], ["sightlines", 1.973126344458938], ["instrument", 1.9730809016596644], ["mounds", 1.9729839935214546], ["durch", 1.972880195962873], ["protectors", 1.9726235730627195], ["milestone", 1.9726081370988007], ["parrying", 1.9724267645227525], ["marconi's", 1.9722174884734665], ["materialistically", 1.9722174884734665], ["stift", 1.9722174884734665], ["concertinas", 1.9722174884734665], ["ropes", 1.9721826854543287], ["salamander", 1.9719962017987427], ["moroccan", 1.9719047803971088], ["michoac\u00e1n", 1.9717961645066913], ["roas", 1.9717961645066913], ["misi\u00f3n", 1.9717961645066913], ["\u00e5lesund", 1.9717961645066913], ["aceite", 1.9717961645066913], ["\u0153uvre", 1.9717961645066913], ["electron's", 1.9717961645066913], ["yage", 1.9717961645066913], ["kancha", 1.9717961645066913], ["pesty", 1.9717961645066913], ["startx", 1.9717961645066913], ["bucketfuls", 1.9717961645066913], ["nikolova", 1.9717961645066913], ["chelone", 1.9717961645066913], ["biofeedback", 1.9716494534825466], ["cantankerous", 1.971345982733973], ["inflammation", 1.9710801491396914], ["apicius", 1.971028331109298], ["fogg's", 1.971028331109298], ["ziran", 1.971028331109298], ["measurability", 1.971028331109298], ["st\u00fcck", 1.971028331109298], ["handwritings", 1.971028331109298], ["quetzales", 1.971028331109298], ["negativities", 1.971028331109298], ["dorcy", 1.971028331109298], ["brenne", 1.971028331109298], ["pinocytosis", 1.971028331109298], ["nossos", 1.971028331109298], ["inicio", 1.971028331109298], ["geofilters", 1.971028331109298], ["evernote's", 1.971028331109298], ["franzini", 1.971028331109298], ["haily", 1.971028331109298], ["droid's", 1.971028331109298], ["jacob's", 1.9709465852740407], ["hauntings", 1.9708638731759551], ["admonished", 1.9708253677079737], ["getup", 1.9708081864706777], ["copernicus", 1.970763318896712], ["literati", 1.9706954808422936], ["signposts", 1.9706954808422936], ["ngorongoro", 1.970677520691729], ["flesch", 1.970677520691729], ["brando's", 1.9705479049642436], ["outbidding", 1.9705095711853315], ["abashed", 1.9704645895711788], ["misato", 1.9703463143857307], ["endotherms", 1.9703463143857305], ["nebulizer", 1.9703463143857305], ["objectif", 1.9703463143857305], ["nanocarriers", 1.9703463143857305], ["diada", 1.9703463143857305], ["ardern", 1.9703463143857305], ["yusuke's", 1.9703463143857305], ["teslamotors", 1.9703463143857305], ["tsking", 1.9703463143857305], ["jorrit", 1.9703463143857305], ["freekeh", 1.9703463143857305], ["rogue", 1.970233465226831], ["featured", 1.9700995235342316], ["patrolling", 1.9700230039262674], ["squirrel", 1.9700037749659962], ["cobbled", 1.969999361345691], ["massaged", 1.9699796018760738], ["folds", 1.9692611725355937], ["paints", 1.9691965864123468], ["eqip", 1.969187968521894], ["inovative", 1.969187968521894], ["thammarat", 1.969187968521894], ["nyou", 1.969187968521894], ["reanalyze", 1.969187968521894], ["kaldi", 1.969187968521894], ["evita's", 1.969187968521894], ["encyclopedia's", 1.969187968521894], ["gasohol", 1.969187968521894], ["meltem", 1.969187968521894], ["corningware", 1.969187968521894], ["boolean", 1.9687901527706775], ["monounsaturated", 1.968759884180911], ["jalandhar", 1.968759884180911], ["freud's", 1.9686179404257431], ["anubis", 1.9685983912858096], ["bullfighting", 1.9685725972817312], ["adhan", 1.9683657606018432], ["eilean", 1.9682412435370278], ["beatlemania", 1.9681801957642064], ["vinaya", 1.9681801957642064], ["kielbasa", 1.9681801957642064], ["bingen", 1.9679216096996937], ["genk", 1.9678299078539483], ["kapoor's", 1.967829907853948], ["swordsmanship", 1.967829907853948], ["tino's", 1.9678299078539478], ["conquest's", 1.9678299078539478], ["tawakkul", 1.9678299078539478], ["bisschen", 1.9678299078539478], ["scattergories", 1.9678299078539478], ["vidur", 1.9678299078539478], ["olda", 1.9678299078539478], ["aircrafts", 1.967688255830636], ["thao", 1.9675999137085702], ["irma's", 1.9675999137085698], ["lilja", 1.9675999137085698], ["watery", 1.9673331204999316], ["ziel", 1.9672761355427466], ["buscando", 1.9672761355427466], ["metronomes", 1.9672761355427466], ["birch's", 1.9672761355427466], ["today's", 1.9670096632469805], ["jellies", 1.9669282446235903], ["nagarjuna", 1.9667865197797942], ["dichotomy", 1.966786519779794], ["circle", 1.9666995805706857], ["goggles", 1.9666649464372254], ["courtship", 1.966532740874016], ["deliveries", 1.9664819509196363], ["hyrule", 1.9664646234590448], ["odometer", 1.9664646234590448], ["seafoam", 1.9663965938242474], ["maunganui", 1.9663505218813726], ["lolcats", 1.9663505218813726], ["zappy", 1.9663505218813726], ["huffing", 1.9663505218813724], ["canoeing", 1.9663081284098267], ["idli", 1.9660672268919068], ["mozambique's", 1.9660672268919068], ["ichthyology", 1.9660672268919068], ["cilician", 1.9660672268919068], ["schmo", 1.9660672268919068], ["everest", 1.966032875531437], ["allergy", 1.9659274286443666], ["governance", 1.9658787721552804], ["systematically", 1.9658266657961694], ["criticizing", 1.9655989646437817], ["bonbons", 1.9655706312662855], ["wordsworth", 1.965520654896307], ["gluttonous", 1.96551056930357], ["samsara", 1.9652311889589762], ["presence", 1.9651428823988468], ["plein", 1.964649319672513], ["behance", 1.9646329073847437], ["followers", 1.9645830981955046], ["squabbling", 1.964530763043642], ["hostels", 1.9643708147401466], ["obgyn", 1.9642649986005898], ["timmins", 1.9642649986005896], ["phazon", 1.9642649986005893], ["furrow", 1.9642649986005893], ["sian's", 1.9642649986005893], ["frita", 1.9642649986005893], ["sammi's", 1.9642649986005893], ["overbaked", 1.9642649986005893], ["dorm's", 1.9642649986005893], ["fondues", 1.9642649986005893], ["derinkuyu", 1.9642649986005893], ["ceba", 1.9642649986005893], ["unutilized", 1.9642649986005893], ["chemehuevi", 1.9642649986005893], ["richtersveld", 1.9642649986005893], ["drummle", 1.9642649986005893], ["brogan's", 1.9642649986005893], ["snowshoers", 1.9642649986005893], ["nymphets", 1.9642649986005893], ["giftee", 1.9642649986005893], ["serapio", 1.9642649986005893], ["hoolihan", 1.9642649986005893], ["vidia", 1.9642649986005893], ["viajar", 1.9642649986005893], ["peashooters", 1.9642649986005893], ["wiri", 1.9642649986005893], ["slighted", 1.9639737806615827], ["doorway", 1.9638821009400436], ["apprentice's", 1.9637454892119093], ["tongue's", 1.9637454892119093], ["hoopoe", 1.963687443470157], ["infallible", 1.963674242210033], ["nucleosynthesis", 1.9636532686103683], ["adorable", 1.9635817759923806], ["tutsis", 1.9635803471545803], ["echinoderm", 1.9635212405549247], ["metalworker", 1.9635212405549247], ["eline", 1.9635212405549247], ["docx", 1.963366021484525], ["moonless", 1.963366021484525], ["taza", 1.963366021484525], ["taxila", 1.963366021484525], ["poinsettias", 1.963366021484525], ["mantas", 1.963220732253646], ["isso", 1.963220732253646], ["idea", 1.9632122139587285], ["physiotherapists", 1.96312892861875], ["boland's", 1.9629563610265717], ["modigliani's", 1.9629563610265717], ["zamorin", 1.9629563610265717], ["melzi", 1.9629563610265717], ["shus", 1.9629563610265717], ["shanzhai", 1.9629563610265717], ["montenapoleone", 1.9629563610265717], ["bengio", 1.9629563610265717], ["denitrifying", 1.9629563610265717], ["niffler", 1.9629563610265717], ["sedums", 1.9629563610265717], ["replacing", 1.9625932837081632], ["lafite", 1.9625127550692774], ["oumar", 1.9623853191760912], ["technical", 1.9623371565716201], ["colloquial", 1.9621997809015228], ["zhen", 1.9621551543486015], ["sonatas", 1.9620885276880125], ["suspicious", 1.9618443382685526], ["softness", 1.9618312273907736], ["whimpering", 1.9617017993853645], ["tilburg", 1.9616141686429638], ["mattarella", 1.9616141686429638], ["cryonic", 1.9616141686429638], ["teraz", 1.9616141686429638], ["chekhov's", 1.9614708805371461], ["supplier", 1.9614563446628277], ["yan's", 1.9614046168676573], ["wintry", 1.9613772229484274], ["porcelain", 1.9613600394900013], ["choli", 1.9613112166478066], ["capote's", 1.9613112166478066], ["pebbled", 1.9613112166478066], ["kiyomi", 1.9613112166478066], ["pap\u00e1", 1.9613112166478066], ["archaeological", 1.9608548912521786], ["fearless", 1.9608286269203097], ["politica", 1.9607000893472306], ["plausibility", 1.9604791653935014], ["savoie", 1.9604360219951305], ["libertadores", 1.9604360219951305], ["ants", 1.9603660064229163], ["repetitive", 1.9603347598039114], ["afford", 1.9602297513199327], ["stevie's", 1.9601297038666936], ["chickens", 1.9599795570079739], ["phobias", 1.9599668257371308], ["antioxidant", 1.9598733250887022], ["sampling", 1.959776511433905], ["vibrations", 1.9594693145936], ["foreseeing", 1.9592741256458877], ["stashed", 1.9590436668658722], ["endzone", 1.9590304483045182], ["compute", 1.9588746338696355], ["cruciferous", 1.9588238213191476], ["orthotic", 1.9588238213191476], ["gannett's", 1.9588238213191476], ["unroasted", 1.9588238213191476], ["nasreddin", 1.9588238213191476], ["goodnights", 1.9588238213191476], ["lao's", 1.9588238213191476], ["uniworld", 1.9588238213191476], ["hoisin", 1.9588238213191473], ["neo's", 1.9588238213191473], ["losartan", 1.9588238213191473], ["collages", 1.9588238213191471], ["gilman's", 1.958823821319147], ["insides", 1.9586259603270948], ["patch", 1.958559338777872], ["critique", 1.9585147366925093], ["c\u00f3rdoba", 1.9584570003788253], ["scorched", 1.958433172693813], ["athlete", 1.9583024806121812], ["psychedelia", 1.9581836828154482], ["dendrite", 1.9581836828154482], ["gambles", 1.9578740885572956], ["enterprising", 1.9576513117366143], ["befriended", 1.9576513117366143], ["vaguely", 1.9576064731816043], ["educations", 1.9575951683846282], ["scrupulous", 1.9575951683846282], ["greyson", 1.957372840710763], ["inherit", 1.9572062628765674], ["unbeatable", 1.9570076932728913], ["resentfully", 1.9569845501254202], ["redhawk", 1.9569845501254202], ["sliced", 1.9565946938522338], ["extract", 1.9565851655233542], ["raza", 1.9563685779865105], ["trample", 1.9563125087277127], ["altruistic", 1.9562589566073563], ["r\u00e9volution", 1.9562172693280373], ["railroader", 1.9562172693280373], ["telltale", 1.9561923199638906], ["targeted", 1.9557203835781416], ["lada", 1.9556874221874863], ["tetons", 1.9554822590173406], ["algonquian", 1.9554822590173406], ["chainsaws", 1.955364529802469], ["amateurs", 1.9552859721863531], ["infringing", 1.9552521767446625], ["folder", 1.9551673501860183], ["magdeburg", 1.9550634265487266], ["entrenching", 1.9548666014780987], ["negi", 1.9548666014780987], ["slowness", 1.954596144006948], ["mesopotamians", 1.9545375547517094], ["cristoforo", 1.9545375547517094], ["taunting", 1.9545348933443636], ["astute", 1.95441905875798], ["powhatan", 1.9542602532766475], ["mysterio", 1.9542012282304884], ["gusts", 1.9541467242516948], ["fraying", 1.9539267617658496], ["hamsters", 1.9538011961362776], ["perceiving", 1.9537859990858], ["frostbite", 1.9537515374127181], ["foundational", 1.9536570686310302], ["finno", 1.9533303250253835], ["erodes", 1.9532668743083128], ["robotech", 1.9530674121045235], ["icc's", 1.9530674121045235], ["edmond's", 1.9527780687842116], ["timurids", 1.9527780687842116], ["meinem", 1.9527780687842116], ["octagons", 1.9527780687842116], ["kinyarwanda", 1.9527780687842116], ["pankration", 1.9527780687842116], ["mauna", 1.952708661353297], ["rinsed", 1.952505034798763], ["complexion", 1.9524953135887315], ["mark", 1.9524646396992218], ["admirable", 1.951667411994019], ["jillian", 1.951499523552476], ["aldonza", 1.9514874025127082], ["sidechains", 1.9514874025127082], ["tenosynovitis", 1.9514874025127082], ["monogramming", 1.9514874025127082], ["lamingtons", 1.9514874025127082], ["fl\u00e5m", 1.9514874025127082], ["orlok", 1.9514874025127082], ["mrinal", 1.9514874025127082], ["frontman", 1.9514120555087506], ["emirati", 1.9512566211264133], ["ramadan", 1.951246478142028], ["relevant", 1.9512291677809614], ["musty", 1.950806938815721], ["unpicked", 1.950802429749454], ["pent", 1.9506895360903247], ["intermediaries", 1.950662055396984], ["flexible", 1.9505725782594672], ["masculinity", 1.9504806828209362], ["valerie's", 1.9504096296468143], ["basey", 1.9502700055055266], ["interrobang", 1.9502700055055266], ["surjeet", 1.9502700055055266], ["bubbles", 1.9499556724144176], ["past", 1.9498041076708779], ["skyscraper", 1.949608257771591], ["veer", 1.9494960888366752], ["beneficence", 1.9494960888366752], ["interlibrary", 1.9494960888366752], ["dachshunds", 1.9494960888366752], ["skittish", 1.949496088836675], ["wastage", 1.949496088836675], ["madeleines", 1.949496088836675], ["unternehmen", 1.949496088836675], ["mojo's", 1.949496088836675], ["beyonc\u00e9", 1.9494203508378858], ["aerobics", 1.949180738450257], ["arjuna", 1.9490297022125516], ["ariadne", 1.9490297022125516], ["branches", 1.9490256717108616], ["constructions", 1.9489644080851742], ["pitchblende", 1.948960660083905], ["maricel", 1.948960660083905], ["johannes", 1.9489243891038786], ["sparse", 1.9486447805009386], ["reserve's", 1.948568199218], ["kaufen", 1.948568199218], ["kafkaesque", 1.948568199218], ["paddleboat", 1.948568199218], ["viticulturist", 1.948568199218], ["baixa", 1.948568199218], ["mydas", 1.948568199218], ["syreeta", 1.948568199218], ["raycom", 1.948568199218], ["handfasting", 1.948568199218], ["signifies", 1.9480419192599205], ["beijing's", 1.9480314035708324], ["suchen", 1.9480314035708322], ["dirges", 1.9480314035708322], ["fiddlehead", 1.9480314035708322], ["hajar", 1.9478397625080117], ["regulus", 1.9477238196650055], ["suit", 1.9475504839575968], ["nibs", 1.9470930119937333], ["globetrotters", 1.947034603876023], ["cobweb", 1.94676952088026], ["histamines", 1.9465907295388563], ["ping's", 1.9465907295388563], ["payslips", 1.9465907295388563], ["anau", 1.9465907295388563], ["memory", 1.9464947537379034], ["mary's", 1.9464179371174595], ["asafa", 1.9464065546229878], ["cortney", 1.9464065546229878], ["plimoth", 1.9460210512451661], ["bbox", 1.9460210512451661], ["loxy", 1.9460210512451661], ["gddr6", 1.9460210512451661], ["capaz", 1.9460210512451661], ["jellyfishes", 1.9460210512451661], ["hillshire", 1.9460210512451661], ["deckard's", 1.9460210512451661], ["tatupu", 1.9460210512451661], ["episodio", 1.9460210512451661], ["rechargeables", 1.9460210512451661], ["hist\u00f3rica", 1.9460210512451661], ["ybas", 1.9460210512451661], ["igualdad", 1.9460210512451661], ["claridon", 1.9460210512451661], ["lagering", 1.9460210512451661], ["haryana's", 1.9460210512451661], ["smax", 1.9460210512451661], ["tiamat's", 1.9460210512451661], ["intellectuelle", 1.9460210512451661], ["c\u00e1ncer", 1.9460210512451661], ["aberhart", 1.9460210512451661], ["cpio", 1.9460210512451661], ["bifilar", 1.9460210512451661], ["hypersexualization", 1.9460210512451661], ["spragge", 1.9460210512451661], ["naturalia", 1.9460210512451661], ["funkadelic", 1.946021051245166], ["unpasteurized", 1.946021051245166], ["deploying", 1.9457649958436865], ["corsets", 1.9457649958436862], ["bazaar", 1.94570918889721], ["hairstyles", 1.9456561722980574], ["acoustics", 1.9455577128996313], ["obor", 1.945395723143866], ["exposure", 1.9453328674370436], ["crucibles", 1.945301368903729], ["arabesques", 1.9450595783937603], ["mariner's", 1.944990319756159], ["vaishnava", 1.944990319756159], ["maps", 1.9447048972737204], ["aretha", 1.9444404873960963], ["melanie", 1.9439544802172948], ["greece's", 1.9439481157775353], ["cornerstone", 1.9434197373909508], ["enlisted", 1.943220616146955], ["discouraging", 1.943090296649917], ["intrapreneurship", 1.943090296649917], ["regi\u00f3n", 1.943090296649917], ["richelieu's", 1.943090296649917], ["anthias", 1.943090296649917], ["philodemus", 1.943090296649917], ["pejeta", 1.943090296649917], ["wireframe", 1.9427843222704746], ["landholdings", 1.9427843222704746], ["muscles", 1.9427794455266516], ["konark", 1.9427559152531437], ["manifested", 1.942722494620346], ["usna", 1.942558380335477], ["endgames", 1.942558380335477], ["tussles", 1.9424662111202402], ["igor's", 1.9424662111202402], ["elsewhere", 1.94238239730039], ["tightened", 1.9423161265514493], ["streams", 1.9421787653627358], ["olya", 1.9421625612297662], ["irate", 1.9421483725362203], ["verbs", 1.942084781391511], ["unmonitored", 1.9420817373762487], ["backpacker", 1.9417905706989809], ["implausible", 1.9417905706989809], ["sqli", 1.9417905706989809], ["lancashire's", 1.9417905706989809], ["stockhausen's", 1.9417905706989809], ["mandla", 1.9417905706989809], ["gregorio's", 1.9417905706989809], ["parula", 1.9417905706989809], ["ravis", 1.9417905706989809], ["uo's", 1.9417905706989809], ["chaiwala", 1.9417905706989809], ["roomster", 1.9417905706989809], ["l\u00e8ve", 1.9417905706989809], ["lifetimes", 1.9416193369625878], ["beaked", 1.9415208775641615], ["immaculate", 1.941467860965009], ["scrapes", 1.94141806636912], ["dependence", 1.9409643772881555], ["cranes", 1.9408831919276264], ["cure's", 1.940585238252861], ["velvets", 1.9404075289819587], ["quarantine", 1.9403711331473006], ["unnecessarily", 1.9401293430244286], ["haunts", 1.9401009269627636], ["bilingualism", 1.939980549659925], ["requires", 1.9399303795895861], ["tamarin", 1.939829166082113], ["barometers", 1.939829166082113], ["clunkier", 1.939829166082113], ["syncretistic", 1.939829166082113], ["deplaned", 1.939829166082113], ["evento", 1.939829166082113], ["luces", 1.939829166082113], ["brie's", 1.939829166082113], ["kogoro", 1.939829166082113], ["tripod", 1.9397544768334576], ["walnuts", 1.939723763497562], ["adas", 1.9396501489940656], ["jnana", 1.9396196662081755], ["coron", 1.9396196662081755], ["talismans", 1.939435172141607], ["veiled", 1.9392714590001192], ["insider", 1.9392355831059556], ["independently", 1.9392015166047154], ["necklaces", 1.9391892752141988], ["physique", 1.9391351306022986], ["reprimand", 1.9390053267373708], ["eddie", 1.9389138439275748], ["krautrock", 1.9388749692414748], ["torvald", 1.9388749692414748], ["katarina", 1.9387713132138629], ["craftsman", 1.9386062421191865], ["devlet", 1.9384194065137397], ["overstone", 1.9384194065137397], ["zaatar", 1.9384194065137397], ["dvo\u0159\u00e1k's", 1.9384194065137397], ["thamel", 1.9384194065137397], ["siddartha", 1.9384194065137397], ["amod", 1.9384194065137397], ["seadog", 1.9384194065137397], ["quietcomfort", 1.9384194065137395], ["atomicity", 1.9384194065137395], ["grace", 1.9379836626471514], ["instagrammers", 1.9377611995845327], ["landmass", 1.937749059744916], ["samuelson", 1.9377166562757286], ["talavera", 1.9376809610255439], ["boom's", 1.9376809610255439], ["springs", 1.9375281792004], ["kiowa", 1.9374926721361232], ["courtesies", 1.937462162362375], ["purcell's", 1.9374428728076876], ["eure", 1.9374428728076876], ["canopies", 1.9372051840667062], ["surfer's", 1.9370841003783683], ["bellerose", 1.9369782024940267], ["tripoli's", 1.9369782024940267], ["davuto\u011flu", 1.9369782024940267], ["bielefeld", 1.9369186947151482], ["navigation", 1.9368251924224011], ["quantity", 1.9366963670412831], ["fright", 1.9365585238834866], ["resurgence", 1.9362909459889401], ["shanghainese", 1.9362775066170392], ["julien's", 1.936245478207369], ["bushcraft", 1.936245478207369], ["handymen", 1.936178459222972], ["silat", 1.936178459222972], ["westphalia", 1.9361128204922586], ["timgad", 1.9356698754406703], ["ghos", 1.9356698754406703], ["showrooming", 1.9356698754406703], ["dmoz", 1.9356698754406703], ["hemu", 1.9356698754406703], ["pepperpot", 1.9356698754406703], ["diagoras", 1.9356698754406703], ["ciconia", 1.9356698754406703], ["cyprinids", 1.9356698754406703], ["despondency", 1.9356168724320328], ["shucked", 1.9355777445565465], ["blueish", 1.9353736296684878], ["vins", 1.9353736296684878], ["cloud", 1.9352317834896948], ["eola", 1.9351058690667076], ["jicama", 1.9351058690667076], ["qatar's", 1.9351058690667073], ["owain", 1.9350805022728543], ["juegos", 1.9346408111794047], ["lcdr", 1.9346408111794047], ["flatworm", 1.9346408111794047], ["isi's", 1.9344768585682879], ["p\u00e9p\u00e9", 1.9344768585682879], ["bagpipers", 1.9344768585682879], ["pluralsight", 1.9344768585682879], ["reemerge", 1.9344226654693815], ["johnny's", 1.934290229079746], ["tabby", 1.9342723828239532], ["hacer", 1.9342507626287637], ["cumbria's", 1.9342507626287635], ["fosbury", 1.9342507626287635], ["hyperion's", 1.9342507626287635], ["aeronca", 1.9342507626287635], ["farrar's", 1.9342507626287635], ["faultline", 1.9342507626287635], ["qualtrics", 1.9342507626287635], ["yucatec", 1.9342507626287635], ["jestem", 1.9342507626287635], ["organizaci\u00f3n", 1.9342507626287635], ["messy", 1.9340455370836571], ["satu", 1.9340177554596472], ["stitches", 1.9340120529157714], ["bossiness", 1.9339971569171301], ["arief", 1.9339971569171301], ["solskj\u00e6r", 1.9339971569171301], ["skyr", 1.9339971569171301], ["art\u00edculo", 1.9339971569171301], ["eutropius", 1.9339971569171301], ["halimah", 1.9339971569171301], ["awakening", 1.9339001201259818], ["abandoned", 1.9338845862926044], ["entrancing", 1.9338584196748838], ["reddening", 1.9338584196748836], ["calendula", 1.9338584196748836], ["danie", 1.9338584196748836], ["crumpling", 1.9338584196748836], ["cliquey", 1.933770918728335], ["brainers", 1.933770918728335], ["drivetrains", 1.933770918728335], ["hohe", 1.933770918728335], ["luring", 1.9337106954047993], ["hive", 1.9335120156111731], ["paradise", 1.9333501055554636], ["safekeeping", 1.9332739453888976], ["tuzla", 1.9332502880963698], ["sayan", 1.9331408060118866], ["mcmurphy", 1.9331408060118866], ["knobs", 1.9329549099010914], ["disinfectants", 1.9328810653522717], ["pantheon", 1.9326275027448792], ["algona", 1.9325749258910851], ["euryale", 1.9325749258910851], ["economically", 1.932455039109082], ["blockbuster", 1.9323602412165575], ["infusions", 1.9322225101654305], ["scene", 1.9321267112313718], ["mixing", 1.9320481675341905], ["booklover", 1.9308769185506511], ["greentown", 1.9308769185506511], ["jaydee", 1.9308769185506511], ["disinterestedly", 1.9307170777461353], ["message", 1.930603699466676], ["xv's", 1.9305986942108022], ["wrenchingly", 1.9305986942108022], ["frosty's", 1.9305986942108022], ["siddhant", 1.9305986942108022], ["secret", 1.930486797865545], ["salade", 1.9304350702855366], ["yakushima", 1.9304350702855366], ["bimal", 1.9304350702855366], ["mijo", 1.9302510265187058], ["rebuild", 1.9300993393381742], ["zydeco", 1.930086385523609], ["viennese", 1.9300175886526894], ["negotiating", 1.930009945207869], ["malice", 1.9299930353950128], ["rink", 1.9299114800913757], ["jiangshi", 1.9298042091514562], ["electroclash", 1.9298042091514562], ["hibernators", 1.9298042091514562], ["marilyn's", 1.9298042091514562], ["thermes", 1.9298042091514562], ["relev\u00e9", 1.9298042091514562], ["larroquette", 1.9298042091514562], ["levitch", 1.9298042091514562], ["crisanto", 1.9298042091514562], ["statman", 1.9298042091514562], ["chto", 1.9298042091514562], ["sunnism", 1.9298042091514562], ["ruen", 1.9298042091514562], ["opah", 1.9298042091514562], ["coelacanths", 1.9298042091514562], ["eufemia", 1.9298042091514562], ["rushes", 1.9296014986252847], ["nothingness", 1.929360983563686], ["persian", 1.9293352161589534], ["racked", 1.9292889076269832], ["jingles", 1.9292415840175932], ["conditions", 1.9292384026817575], ["overly", 1.929151697161887], ["raincoats", 1.9291149933624738], ["diane's", 1.928914898455995], ["bayer's", 1.928714539919863], ["fatehpur", 1.928714539919863], ["mahalia", 1.928714539919863], ["attaching", 1.9283651871744596], ["custom", 1.9283540376400652], ["mixed", 1.928220747704915], ["ulsan", 1.9281434482141486], ["afcon", 1.9280313068003754], ["borage", 1.9280313068003754], ["checkered", 1.9280278797984083], ["imitated", 1.9279817635555885], ["horsetooth", 1.9279414637758274], ["ishikawa's", 1.9279414637758274], ["stepsiblings", 1.9279414637758274], ["claras", 1.9279414637758274], ["buraq", 1.9279414637758274], ["aglianico", 1.9279414637758274], ["coquito", 1.9279414637758274], ["jokhang", 1.9279414637758274], ["cheeps", 1.9279414637758274], ["gorchakov", 1.9279414637758274], ["fondos", 1.9279414637758274], ["midco", 1.9279414637758274], ["osbern", 1.9279414637758274], ["blistering", 1.9277834717596747], ["whitney's", 1.9277771038897424], ["copycats", 1.9276075054415513], ["giustiniani", 1.9276075054415513], ["pracha", 1.9276075054415513], ["harrypotter", 1.9276075054415513], ["chokeholds", 1.9276075054415513], ["skiffle", 1.9274678844430404], ["estevan", 1.9274678844430404], ["guideposts", 1.9274678844430404], ["manang", 1.9274678844430404], ["curies", 1.9273911998246025], ["ogami", 1.9273911998246025], ["villainous", 1.927214670059903], ["thrums", 1.9271276429806499], ["lovina", 1.9271276429806499], ["blindside", 1.9270235114437453], ["gallery's", 1.9270235114437453], ["amalie", 1.926871376006849], ["mildred", 1.9268379639836828], ["grapeseed", 1.9267458823540686], ["qinghai", 1.926671410110626], ["ambiguous", 1.9265804578804084], ["narrations", 1.9265608407327146], ["salty", 1.9265210357566662], ["ranjit", 1.9264726524675522], ["bali", 1.9264565896049692], ["briefcase", 1.9264421809125163], ["commandeered", 1.9264173993775857], ["normalizing", 1.9262296702134587], ["mycorrhiza", 1.9261767576304945], ["ogre's", 1.9261767576304945], ["ernest's", 1.9261767576304945], ["cristine", 1.9261767576304945], ["pinewood", 1.9261037216938313], ["cruising", 1.926097852340613], ["acai", 1.9260276842803183], ["landmark's", 1.9260276842803183], ["copperfield's", 1.9260276842803183], ["margalla", 1.9260276842803183], ["pettigrew's", 1.9260276842803183], ["mvvm", 1.9260276842803183], ["designed", 1.9259046987664228], ["relayed", 1.9258658332144127], ["edge's", 1.9258658332144123], ["browsers", 1.9257133799523334], ["irreverent", 1.9256334630759182], ["pulling", 1.925459688195905], ["curling", 1.9252094372249053], ["guagua", 1.9250509968136447], ["timelier", 1.9250509968136447], ["wurundjeri", 1.9250509968136447], ["sabahan", 1.9250509968136447], ["rolfe's", 1.9250509968136447], ["ukay", 1.9250509968136447], ["linzy", 1.9250509968136447], ["dazzled", 1.9247798151271769], ["haggis", 1.9246765643318915], ["cara's", 1.924651193906726], ["whereas", 1.9246498404883252], ["jenga", 1.9245630244443634], ["touchpoint", 1.9241779578128222], ["palazzos", 1.9241779578128222], ["minut", 1.9241779578128222], ["portfolio's", 1.9241779578128222], ["theano", 1.9241779578128222], ["horrorcore", 1.9241779578128222], ["fabricating", 1.9239633973804167], ["parkas", 1.9238110283776941], ["erard", 1.9236972338060403], ["trisha's", 1.9236972338060403], ["haeundae", 1.9236972338060403], ["n'golo", 1.9236972338060403], ["gucci's", 1.9235084880020952], ["nation's", 1.9234834584152387], ["faucet", 1.9234422172531547], ["overcrowded", 1.9234163554974706], ["rasters", 1.9233928994865013], ["conciencia", 1.9233928994865013], ["balagtas", 1.9233928994865013], ["l'apprentissage", 1.9233928994865013], ["birdville", 1.9233928994865013], ["buchheim", 1.9233928994865013], ["raos", 1.9233928994865013], ["chassepot", 1.9233928994865013], ["volta's", 1.9233928994865013], ["yantras", 1.9233928994865013], ["dismaland", 1.9233928994865013], ["macrs", 1.9233928994865013], ["syllabaries", 1.9233928994865013], ["cansado", 1.9233928994865013], ["maravilloso", 1.9233928994865013], ["viceroyalties", 1.9233928994865013], ["sisteron", 1.9233928994865013], ["jamesons", 1.9233928994865013], ["esto", 1.9232088427497083], ["flavius", 1.9232088427497083], ["m\u00fctter", 1.9229120512616298], ["kokeshi", 1.9229120512616298], ["francophones", 1.9229120512616298], ["changchun", 1.9229120512616298], ["opini\u00f3n", 1.9229120512616298], ["charlottesville's", 1.9229120512616298], ["hydrophone", 1.9229120512616296], ["cays", 1.9229120512616296], ["document's", 1.9228408349344097], ["heures", 1.9226831604092003], ["caret", 1.9226831604092003], ["boringly", 1.9226831604092003], ["ernie's", 1.9226831604092003], ["unfairly", 1.9226411804275316], ["unequal", 1.9222917570515452], ["shenanigans", 1.9221462559405378], ["cryptographic", 1.9221034740790242], ["meditatively", 1.9220383974446091], ["th\u00e9odore", 1.9220383974446091], ["lines", 1.9218399378121012], ["website's", 1.9216957881046013], ["misappropriated", 1.9216957881046013], ["mali's", 1.9216957881046013], ["widen", 1.9215181316884062], ["siento", 1.921450078375909], ["ihrer", 1.921450078375909], ["calamity", 1.9214173664180527], ["professional", 1.9214166991480335], ["henk", 1.921321553287054], ["lapped", 1.92113978208996], ["connects", 1.9210768612909657], ["burkitt's", 1.9209111021968412], ["thermochromic", 1.9209111021968412], ["wirehaired", 1.9209111021968412], ["zeder", 1.9209111021968412], ["postnasal", 1.9209111021968412], ["rimbaud's", 1.9209111021968412], ["amali", 1.9209111021968412], ["fixing", 1.920592251776761], ["ifan", 1.9204838532862103], ["djing", 1.9204424958584834], ["fences", 1.9204184061959637], ["mapmakers", 1.9204155110972032], ["impala's", 1.9203211417846187], ["tomales", 1.9203211417846187], ["role's", 1.9203211417846187], ["hayy", 1.9203211417846187], ["sundew", 1.9203211417846187], ["graduating", 1.9202151199134365], ["appropriating", 1.9201976357555879], ["tracts", 1.920124212115183], ["tuba", 1.9200436387294473], ["rousing", 1.9199891989628026], ["innocence", 1.919792228311501], ["stomped", 1.9196312823768045], ["veteran", 1.9195425518488254], ["campground", 1.9195286282975328], ["astronauts", 1.9194659723167165], ["yuzu", 1.919114331608052], ["wobbling", 1.91889575781504], ["livable", 1.9188816134465012], ["maintained", 1.9184149182385175], ["goldens", 1.9182508808968337], ["kampung", 1.918072876573028], ["aage", 1.9180149917083318], ["original", 1.9177700261545851], ["atari's", 1.9175371365606644], ["kishor", 1.9175371365606644], ["foals", 1.9175371365606642], ["implementation", 1.917499680212854], ["olympe", 1.9174830330355652], ["matti's", 1.9172730129881352], ["gibberd", 1.9172730129881352], ["purikura", 1.9172730129881352], ["havva", 1.9172730129881352], ["michaelson's", 1.9172730129881352], ["pol\u00edticos", 1.9172730129881352], ["kodjo", 1.9172730129881352], ["hatao", 1.9172730129881352], ["calabaza", 1.9172730129881352], ["ng\u1ecdc", 1.9172730129881352], ["sardarji", 1.9172730129881352], ["kawawa", 1.9172730129881352], ["campa\u00f1a", 1.9172730129881352], ["crossmolina", 1.9172730129881352], ["guianese", 1.9172730129881352], ["rakib", 1.9172730129881352], ["hnwis", 1.9172730129881352], ["sarani", 1.9172730129881352], ["fullwidth", 1.9172730129881352], ["keepalive", 1.9172730129881352], ["boasts", 1.917273012988135], ["davenport's", 1.917273012988135], ["calibrating", 1.917273012988135], ["sandwiched", 1.917273012988135], ["unaccomplished", 1.917273012988135], ["neuroanatomy", 1.917273012988135], ["classify", 1.9170888811261437], ["tannins", 1.916989644230673], ["thinking", 1.9168944844419604], ["imprinted", 1.9168370048742054], ["demarcating", 1.9166985778499184], ["transcribe", 1.9165825185056664], ["dreaming", 1.916573798139068], ["ostentatious", 1.916566661638637], ["sele\u00e7\u00e3o", 1.916566661638637], ["cryptogram", 1.916566661638637], ["gajah", 1.916566661638637], ["agrabah", 1.916566661638637], ["arnel", 1.916566661638637], ["transylvanian", 1.9164952146055843], ["\u00e1lvares", 1.9163560961956971], ["unprescribed", 1.9163560961956971], ["crockpots", 1.9163560961956971], ["venustiano", 1.9163560961956971], ["braincell", 1.9163560961956971], ["bestand", 1.9163560961956971], ["gajar", 1.9163560961956971], ["h\u00fctte", 1.9163560961956971], ["rota's", 1.9163560961956971], ["gan's", 1.9163560961956971], ["ssids", 1.9163560961956971], ["pusser", 1.9163560961956971], ["greenhorn", 1.916356096195697], ["adaptions", 1.9162660418678683], ["lovecraftian", 1.9162660418678683], ["endothermic", 1.9162660418678683], ["drought", 1.91619349936399], ["tarts", 1.9160920625643667], ["monarchies", 1.916068947876357], ["hissing", 1.9156385091407455], ["speculated", 1.9154897273802778], ["aphex", 1.9154139893814888], ["sociability", 1.9154139893814888], ["anemia", 1.9153679324903328], ["artsy", 1.915238308456462], ["parque", 1.915228648659376], ["sitting", 1.915184076434335], ["panoramas", 1.9151288919715972], ["scandinavia's", 1.9150022593209561], ["bonham's", 1.9150022593209561], ["filioque", 1.9150022593209561], ["avar", 1.9150022593209561], ["stylishly", 1.9149595613887527], ["ballplayers", 1.9148212572170509], ["instantaneous", 1.9148212572170504], ["skyrim", 1.9146653652316057], ["changed", 1.914588445194685], ["restarting", 1.9145619368951088], ["pouncing", 1.9144883027296193], ["gaslights", 1.9144883027296193], ["clover's", 1.9144883027296193], ["uncrossable", 1.9144883027296193], ["brienz", 1.9144883027296193], ["ironfist", 1.9144883027296193], ["haproxy", 1.9144883027296193], ["arbaeen", 1.9144883027296193], ["bossypants", 1.9144883027296193], ["eirini", 1.9144883027296193], ["wontons", 1.9143724486719653], ["windsurfing", 1.9142854430419132], ["knacks", 1.91424127198088], ["nguyen", 1.9141549565310263], ["ramps", 1.9141434943651487], ["logs", 1.9140781399105824], ["interviewees", 1.9140507054032811], ["ethnographer", 1.9140507054032811], ["edith's", 1.9140507054032811], ["patties", 1.914006063862433], ["shoppers", 1.9140007823280227], ["helmed", 1.9139461467500851], ["wicca", 1.9138725878985907], ["observatories", 1.9138180223610488], ["suffering", 1.9137631691069925], ["henceforward", 1.9133453246384373], ["rapturously", 1.9133453246384373], ["fellini's", 1.9133453246384373], ["finnmark", 1.9133453246384373], ["tomato", 1.9132747118394142], ["prepacked", 1.9130569096060832], ["kahanamoku", 1.9130569096060832], ["sandier", 1.9130569096060832], ["roombas", 1.9130569096060832], ["crawshay", 1.9130569096060832], ["dillo", 1.9130569096060832], ["getafix", 1.9130569096060832], ["pose", 1.9128691926221681], ["socialism's", 1.9125738144268898], ["lorient", 1.9125738144268898], ["h\u00fcsker", 1.9125738144268898], ["retributions", 1.9125738144268898], ["mayotte", 1.9125738144268898], ["walk\u00fcre", 1.9125738144268898], ["serpens", 1.9125738144268898], ["lorises", 1.9125738144268898], ["comforts", 1.9124771953910513], ["posing", 1.9123924418508416], ["martijn", 1.912369501445966], ["difficult", 1.912351144710449], ["pamphlets", 1.912190916766344], ["identical", 1.9116548600415795], ["civically", 1.9114806473899837], ["discipline's", 1.9114806473899837], ["lionhearted", 1.9114806473899837], ["natalija", 1.9114806473899837], ["philanthropic", 1.911425121445252], ["booked", 1.9113923016457766], ["investigative", 1.9113867517423162], ["fallacies", 1.9112879640743101], ["okavango", 1.911176755395168], ["pema", 1.911176755395168], ["astrakhan", 1.911176755395168], ["postponing", 1.911079849566378], ["un's", 1.911079849566378], ["dionysian", 1.9105061670599417], ["inequitable", 1.9105061670599417], ["teems", 1.9105061670599417], ["food", 1.910268923059272], ["fully", 1.910186427776393], ["adept", 1.9101583385309224], ["submersible", 1.9101442449924693], ["youngster's", 1.9101442449924693], ["arm\u00e9e", 1.9101442449924693], ["scientists", 1.9100350575332952], ["expected", 1.9098940346157796], ["larger", 1.909755432766345], ["trysts", 1.909718954666415], ["gondolas", 1.9095968352336399], ["violist", 1.9095968352336399], ["pampering", 1.9094515281345183], ["bobby's", 1.909446589761577], ["flashcard", 1.9094273945206646], ["impoverished", 1.9090661983579598], ["volunteerism", 1.9090333223601978], ["undaunted", 1.9089870791985106], ["mics", 1.908861187939276], ["altercation", 1.9088144555778932], ["pixels", 1.9086593364020852], ["homogenous", 1.9085975694904516], ["gendarmenmarkt", 1.9085975694904511], ["libbed", 1.9085975694904511], ["lungis", 1.9085975694904511], ["rorschach's", 1.9085975694904511], ["spongebob's", 1.9085975694904511], ["mellifera", 1.9085975694904511], ["fynn", 1.9085975694904511], ["celestite", 1.9085975694904511], ["aztl\u00e1n", 1.9085975694904511], ["tzitzit", 1.9085975694904511], ["paddings", 1.9085975694904511], ["ideias", 1.9085975694904511], ["g\u00e1lvez", 1.9085975694904511], ["moldavite", 1.9085975694904511], ["pediococcus", 1.9085975694904511], ["icj's", 1.9085975694904511], ["sandgrouse", 1.9085975694904511], ["igors", 1.9085975694904511], ["otsuka's", 1.9085975694904511], ["gamsat", 1.9085975694904511], ["gegeben", 1.9085975694904511], ["burrow's", 1.9085975694904511], ["oakview", 1.9085975694904511], ["schikaneder", 1.9085975694904511], ["timestamping", 1.9085975694904511], ["hamet", 1.9085975694904511], ["l\u00e9once", 1.9085975694904511], ["priorat", 1.908597569490451], ["rodent's", 1.908597569490451], ["stoneridge", 1.908597569490451], ["synesthetes", 1.908597569490451], ["salk's", 1.908597569490451], ["conferencia", 1.908597569490451], ["saranda", 1.908597569490451], ["alvarado's", 1.908597569490451], ["thylakoids", 1.908597569490451], ["aguda", 1.908597569490451], ["unkind", 1.908499893053937], ["lee's", 1.9084385196929938], ["fists", 1.9083847147429243], ["rising", 1.9079200461840613], ["adjourned", 1.9079003648988562], ["chel", 1.9078746158656443], ["migrating", 1.9075789359561726], ["glare", 1.9074724137180443], ["profession's", 1.9074237702472223], ["perk", 1.9073046485562803], ["favored", 1.9072203980693416], ["barks", 1.9071006302202627], ["speeches", 1.9070949133546595], ["objectives", 1.9070945489044773], ["aeration", 1.9070533966996015], ["blew", 1.90702876012462], ["monolingual", 1.9069306720498482], ["hovering", 1.906887784167783], ["doorknobs", 1.906830349518701], ["filmography", 1.906740957068768], ["beaches", 1.9067245488493394], ["epicenters", 1.9066927814969479], ["gbbo", 1.9066927814969479], ["marais", 1.9066927814969477], ["logic", 1.9064431045484718], ["excellence", 1.9063708723260457], ["bodegas", 1.9062700114788775], ["negeri", 1.9062700114788775], ["baghdad's", 1.9061937942140148], ["collectivism", 1.9060696256898015], ["jeepney", 1.9059949364411461], ["flaring", 1.905994936441146], ["aladdin's", 1.9058314860564072], ["ideal", 1.9056791502978303], ["reverb", 1.9056153857881222], ["engraved", 1.9054838453589562], ["retinol", 1.9050391672422486], ["stalin's", 1.9049882700020049], ["hanako", 1.9046681039062063], ["ironclad", 1.9045986031543762], ["planted", 1.9045287864900373], ["questo", 1.9041589704916362], ["tenderize", 1.9040205969017452], ["kias", 1.9040205969017452], ["navarino", 1.9040205969017452], ["boyishly", 1.9040205969017452], ["superhero", 1.9039878211234615], ["rita's", 1.9039586170646066], ["dabbing", 1.9039364700276853], ["leticia", 1.9037393211317482], ["amari", 1.903602617173603], ["sailboat", 1.903349461695556], ["dante's", 1.9032028979977393], ["bartered", 1.9031016079029133], ["medico", 1.903038547443392], ["sapsucker", 1.9029674881645207], ["pontiff's", 1.9029674881645207], ["anonymizing", 1.9029674881645207], ["assist", 1.9027701784322733], ["subordinating", 1.9024487371145156], ["nonpolar", 1.9024487371145156], ["scholarships", 1.9023074957614348], ["votre", 1.9022355775921498], ["accreditations", 1.9022355775921496], ["haganah", 1.9022355775921496], ["nimbyism", 1.9020463192519257], ["hijabi", 1.9020463192519257], ["wakandan", 1.9020463192519257], ["listo", 1.9020463192519257], ["dyscalculia", 1.902007108269835], ["documented", 1.901946106960777], ["essentials", 1.9019020860813516], ["longbow", 1.901781149599414], ["storms", 1.901475936768472], ["astoundingly", 1.90146262530544], ["bowstring", 1.9014377715341635], ["unmoderated", 1.901404362570261], ["stingless", 1.901404362570261], ["deescalation", 1.901404362570261], ["warring", 1.9013366004760857], ["glitterati", 1.9012113559862314], ["abiy", 1.9012113559862314], ["argentina's", 1.9011870084044742], ["scooters", 1.9010540648110368], ["sumerian", 1.9010269319496749], ["ideological", 1.9009016115489572], ["clumsiest", 1.9009016115489572], ["birdwatch", 1.9009016115489572], ["octopus's", 1.9009016115489572], ["jeder", 1.9009016115489572], ["backstreets", 1.9004758777053852], ["shovels", 1.900331165543645], ["spiros", 1.9002663896236278], ["snoozed", 1.9002448448974953], ["tailwinds", 1.9002448448974953], ["logins", 1.9000833795893788], ["hema", 1.9000833795893788], ["napa", 1.9000591066795731], ["artifice", 1.900059106679573], ["bots", 1.900034280015171], ["pancakes", 1.8997905447160672], ["grungy", 1.8996510183834647], ["darwaza", 1.899465523703416], ["saad's", 1.899465523703416], ["neben", 1.899465523703416], ["dodo's", 1.899465523703416], ["suja", 1.899465523703416], ["saucerful", 1.899465523703416], ["idol's", 1.899383650189463], ["flames", 1.8993756541313245], ["moby", 1.8993408712784228], ["engrossing", 1.8992280905129524], ["shouting", 1.8991773210365355], ["krab", 1.8991723963077825], ["karlheinz", 1.8988598267889694], ["disruptive", 1.8987894986472365], ["countercultural", 1.8986396691278924], ["adeptly", 1.8986396691278924], ["gamble's", 1.8986396691278924], ["respectably", 1.8984762187431536], ["rwanda's", 1.8984762187431536], ["garba", 1.8984762187431536], ["alter", 1.8984325833279418], ["geek's", 1.8984093262369435], ["entryways", 1.8984093262369435], ["shipwreck", 1.8983887917931768], ["comprehensible", 1.8983662959697911], ["sauk", 1.8981002828582538], ["contestants", 1.89807189410722], ["valleys", 1.8977880065968842], ["intersecting", 1.8976526478612858], ["linus", 1.8975608965886153], ["measured", 1.8974730190503033], ["outfits", 1.8974029106694015], ["tages", 1.897370524964037], ["spaying", 1.897370524964037], ["siam's", 1.8973705249640367], ["griswold's", 1.8973705249640367], ["subtleness", 1.8973705249640367], ["sawsan", 1.8973705249640367], ["promotional", 1.89721295240928], ["based", 1.8971392097302835], ["righteously", 1.89706645917478], ["sunbathing", 1.8968864401254188], ["twang", 1.8967674192663369], ["narwhal", 1.8966063805721867], ["treeline", 1.896531722786776], ["genetic", 1.8965141424123757], ["leadership", 1.8964995236064228], ["franco's", 1.8963685690452035], ["tooltip", 1.896183926762058], ["performance's", 1.8960958386859288], ["lfos", 1.8960958386859288], ["nokias", 1.8960958386859288], ["quilmes", 1.8960958386859288], ["boulders", 1.8960885178525746], ["endlessly", 1.8957741795704373], ["clickable", 1.8956654831551187], ["opec's", 1.8956359561153038], ["actium", 1.8956040913320442], ["astronomers", 1.895480713684192], ["newsworthiness", 1.8953434197023231], ["caff\u00e9", 1.8953434197023231], ["improvised", 1.895343419702323], ["destabilize", 1.8952888067237985], ["enraged", 1.8950857689400868], ["confession", 1.8950437606636348], ["ni\u00f1a", 1.8949647297083767], ["gmat", 1.8949647297083767], ["airiness", 1.8948468665402318], ["htet", 1.8948468665402318], ["horror's", 1.8948468665402318], ["iras", 1.8948155739120738], ["constable's", 1.8947168598941568], ["bludger", 1.8947168598941568], ["sergio's", 1.8947168598941568], ["troublemaking", 1.8947168598941568], ["anantara", 1.8947168598941568], ["dugongs", 1.8947168598941568], ["manual's", 1.8947168598941568], ["zander's", 1.8947168598941568], ["khosrow", 1.8947168598941568], ["alika", 1.8947168598941568], ["owls", 1.8946729763278263], ["discrimination", 1.8946339328607231], ["brewery's", 1.8945420059942035], ["borghese", 1.8945420059942035], ["surveying", 1.8945060547250547], ["orchid", 1.8943785175977474], ["tsar's", 1.8943543040464925], ["iihf", 1.8941126772219972], ["hbcus", 1.8941126772219972], ["deshpande", 1.8941126772219972], ["basswood", 1.8941126772219972], ["dawn's", 1.8937458996616352], ["happening", 1.893656970925478], ["diacritics", 1.8936029757907509], ["quotient", 1.8936029757907507], ["clinched", 1.8935738115954757], ["longhouses", 1.893529693941842], ["humanism", 1.893262368957698], ["lunchbox", 1.892783475343245], ["waterslides", 1.8927717189971638], ["upheavals", 1.8926925897446973], ["vesuvius", 1.8926925897446973], ["arctic's", 1.8924230138168032], ["anoushka", 1.8924230138168032], ["vaishya", 1.8924230138168032], ["mainzer", 1.8924230138168032], ["vincenti", 1.8924230138168032], ["domesday", 1.8922899320296152], ["piped", 1.892169399621482], ["inhabitant", 1.892167033892229], ["torchbearer", 1.8921643800852448], ["juicier", 1.8919925752830113], ["bilingual", 1.8919910407891445], ["partnership", 1.8919384198441704], ["phalaenopsis", 1.8918554855475527], ["fallen", 1.8917201138402429], ["crixus", 1.8916773782715641], ["refolding", 1.8916773782715641], ["tetrameter", 1.8916773782715641], ["jorvik", 1.8916773782715641], ["tenjin", 1.8916773782715641], ["tingling", 1.8914050437652792], ["lifesaving", 1.8912578621903124], ["geography", 1.8911503896143764], ["madonna's", 1.8911113929278682], ["alphabetically", 1.8910277694348618], ["groan", 1.8909575010451607], ["tumnus", 1.8908596738632166], ["humectants", 1.8908596738632166], ["trouve", 1.8908596738632166], ["teochew", 1.8908596738632166], ["solvability", 1.8908596738632166], ["troubadours", 1.8908278679814026], ["conscientious", 1.8908278679814021], ["pews", 1.8907143584330368], ["sincerest", 1.8906908188384812], ["flurries", 1.8906784220039687], ["committing", 1.8906306002756827], ["mango", 1.8906157411497972], ["campout", 1.8905692549171789], ["younglings", 1.8905692549171789], ["escalation", 1.8905617790610558], ["buffalonian", 1.8904204497810186], ["parashurama", 1.8904204497810186], ["overprotection", 1.8904204497810186], ["ima's", 1.8904204497810186], ["kiras", 1.8904204497810186], ["ingenier\u00eda", 1.8904204497810186], ["liliana's", 1.8904204497810186], ["shekou", 1.8904204497810186], ["usct", 1.8904204497810186], ["bertrando", 1.8904204497810186], ["marinha", 1.8904204497810186], ["lyran", 1.8904204497810186], ["hachijo", 1.8904204497810186], ["zl\u00edn", 1.8904204497810186], ["plts", 1.8904204497810186], ["zwerg", 1.8904204497810186], ["stowe's", 1.8904204497810184], ["ramones", 1.8904204497810184], ["lecce", 1.8904204497810184], ["striders", 1.8902456697838124], ["laurent's", 1.8902456697838124], ["dungeon", 1.8902298832034194], ["decentralization", 1.890181277153263], ["clarissa", 1.89011115324146], ["folding", 1.8901020780735327], ["client", 1.8900918734600456], ["evoo", 1.8900374634662327], ["wakeup", 1.8899322213590706], ["oilcloth", 1.8898398538197836], ["rounded", 1.8897214396016162], ["dynamism", 1.8896888629196478], ["hums", 1.8896379909855794], ["metrocards", 1.8896379909855792], ["nobels", 1.8896379909855792], ["karalis", 1.8896379909855792], ["envelope's", 1.8896379909855792], ["releasedate", 1.8896379909855792], ["yup'ik", 1.8896379909855792], ["skyline's", 1.8896379909855792], ["riveras", 1.8896379909855792], ["lounge's", 1.8896379909855792], ["mogao", 1.8896379909855792], ["indigos", 1.8896379909855792], ["kosho", 1.8896379909855792], ["yawa", 1.8896379909855792], ["transcendental", 1.8894161639170721], ["coiled", 1.8890068009874952], ["hanna's", 1.888895917160227], ["firebase", 1.888761021717657], ["hedgehogs", 1.8887325743794283], ["nicias", 1.8884835435824725], ["yali", 1.8884835435824725], ["digit", 1.8884746647255348], ["elicit", 1.8884512618124965], ["fritters", 1.8883292767171012], ["fmri", 1.8880771572088735], ["ruining", 1.887912210357426], ["edgar's", 1.8878519437351204], ["financement", 1.8878519437351202], ["stoffe", 1.8878519437351202], ["materialien", 1.8878519437351202], ["wrestlemanias", 1.8878519437351202], ["sighthounds", 1.8878519437351202], ["stitch's", 1.8878519437351202], ["micrometeoroid", 1.8878519437351202], ["shipping's", 1.8878519437351202], ["rodinia", 1.8878519437351202], ["ranjini", 1.8878519437351202], ["iom's", 1.8878519437351202], ["tapaj\u00f3s", 1.8878519437351202], ["pero", 1.8877684474085508], ["acuity", 1.8877684474085508], ["muggle", 1.8876965010061177], ["complement", 1.8876321598561503], ["shops", 1.8876204263408463], ["annoyed", 1.887615848734942], ["vouched", 1.8874321878832616], ["trav\u00e9s", 1.8872486033887683], ["laveau", 1.8872486033887683], ["bosna", 1.887092244763798], ["dai's", 1.887092244763798], ["misattribution", 1.886979150418553], ["mahabharata", 1.886979150418553], ["deplaning", 1.886979150418553], ["wctu", 1.886979150418553], ["debriefs", 1.8868264945532978], ["queste", 1.8868264945532978], ["tweet's", 1.8868264945532978], ["gallerists", 1.8868264945532978], ["drumnadrochit", 1.8868264945532978], ["chaucerian", 1.8868264945532978], ["asatru", 1.8868264945532978], ["morgause", 1.8868264945532978], ["exaggerations", 1.8868264945532975], ["vitamins", 1.886714934117887], ["uavs", 1.8866716674339417], ["rizal", 1.8866716674339417], ["udacity", 1.8864617729370825], ["mouthpieces", 1.8863741731333707], ["ernst's", 1.8862747908999196], ["mycelial", 1.8862747908999196], ["bitlocker", 1.8862747908999196], ["alberich", 1.8862747908999196], ["paradigms", 1.8862311917229848], ["cardio", 1.8861455217184453], ["avicenna", 1.8860978420452386], ["kalinga", 1.8860978420452386], ["puffed", 1.8858818931281278], ["groaning", 1.8858818931281278], ["rumi", 1.8856943986565662], ["introversion", 1.885694398656566], ["virology", 1.885578184284653], ["battlegrounds", 1.885578184284653], ["politici", 1.8852078933937546], ["connacht's", 1.8852078933937546], ["susquehannock", 1.8852078933937546], ["urushiol", 1.8852078933937546], ["deveined", 1.8852078933937546], ["direwolves", 1.8852078933937546], ["kupu", 1.8852078933937546], ["fungo", 1.8852078933937546], ["popularizing", 1.8852078933937544], ["bake", 1.8851450606107347], ["swapo", 1.884925041496771], ["perking", 1.8848719090962327], ["elton's", 1.8848719090962327], ["garbed", 1.8848719090962327], ["activists", 1.8845916533941935], ["yardages", 1.884438106585509], ["paystub", 1.884438106585509], ["xcaret", 1.884438106585509], ["groaner", 1.884438106585509], ["lavera", 1.884438106585509], ["deafeningly", 1.884438106585509], ["wei\u00dfwurst", 1.884438106585509], ["informant's", 1.884438106585509], ["zoonotic", 1.8844381065855085], ["simplified", 1.8841753107951347], ["gerhard", 1.8841519736111254], ["unceremoniously", 1.8841213182390606], ["sprinkles", 1.8838564898859453], ["horticulturist", 1.8836159259107332], ["paramahamsa", 1.8836159259107332], ["fontana's", 1.8836159259107332], ["vendel", 1.8836159259107332], ["conscientiousness", 1.8836159259107332], ["anoeta", 1.8836159259107332], ["bendu", 1.8836159259107332], ["helper's", 1.8836159259107332], ["oli's", 1.8836159259107332], ["fa\u00e7on", 1.8836159259107332], ["beautifying", 1.8833029029068429], ["replaced", 1.883248167968909], ["tendencies", 1.883206843469595], ["quips", 1.8831953716231324], ["spiccato", 1.8830359948990614], ["morrisseau", 1.8830359948990614], ["overpopulating", 1.8830359948990614], ["blackletter", 1.8830359948990614], ["tutul", 1.8830359948990614], ["sanzo", 1.8830359948990614], ["thetans", 1.8830359948990614], ["eboracum", 1.8830359948990614], ["roguishly", 1.8830359948990614], ["venado", 1.8830359948990614], ["reshoring", 1.8830359948990614], ["trueba", 1.8830359948990614], ["karibu", 1.8830359948990614], ["swee'pea", 1.8830359948990614], ["tagalogs", 1.8830359948990614], ["sargon's", 1.8830359948990614], ["treant", 1.8830359948990614], ["katelyn's", 1.8830359948990614], ["safezone", 1.8830359948990614], ["mellody", 1.8830359948990614], ["chards", 1.8830359948990614], ["ronia", 1.8830359948990614], ["akot", 1.8830359948990614], ["barrier", 1.8829668638320811], ["buenas", 1.882878878837743], ["studebaker", 1.882878878837743], ["ascap", 1.8826049955868849], ["babelsberg", 1.8826049955868849], ["pitas", 1.8823957242445564], ["collie's", 1.8822720857733415], ["mezzaluna", 1.8822720857733415], ["perceptrons", 1.8822720857733415], ["balduin", 1.8822720857733415], ["m\u00fcnchner", 1.8822720857733415], ["quadtree", 1.8822720857733415], ["rustiness", 1.8822720857733415], ["thereby", 1.882216113366373], ["massive", 1.8821498603132265], ["portly", 1.8821094283169146], ["anagram", 1.882089269914195], ["hengist", 1.882089269914195], ["validators", 1.8820072008752593], ["nikon's", 1.8820072008752593], ["renovating", 1.8818342258855187], ["vidura", 1.8817914238515403], ["babacan", 1.8817914238515403], ["sendgrid", 1.8817914238515403], ["legalzoom", 1.8817914238515403], ["curlyhair", 1.8816122568112803], ["hypervigilance", 1.8816122568112803], ["wallachian", 1.8816122568112803], ["lydgate", 1.8816122568112803], ["dinners", 1.881376126279577], ["watermelon", 1.8813744925506208], ["grisly", 1.8811831680377702], ["mountains", 1.8811010807717634], ["somersaults", 1.8808921209010736], ["aguas", 1.8808921209010736], ["intelligently", 1.8808206584344742], ["comparisons", 1.880788343964164], ["hawa", 1.880709385826664], ["eyestrain", 1.8805629400913553], ["vaults", 1.880512193863456], ["venues", 1.8804159335034436], ["functional", 1.8802757365914609], ["folded", 1.8800160834995743], ["oaxaca", 1.8799840727031147], ["umbria", 1.8798435886400493], ["goldstein's", 1.8796794244981718], ["spa's", 1.8796794244981718], ["weaponizing", 1.8796794244981718], ["knotting", 1.8796794244981716], ["clonezilla", 1.8796794244981716], ["brahe", 1.8796794244981716], ["complejo", 1.8796794244981716], ["hogarth's", 1.8796794244981716], ["rivi", 1.8796794244981716], ["timmys", 1.8796794244981716], ["lolo's", 1.8796794244981716], ["mountaineer's", 1.8796794244981716], ["rala", 1.8796794244981716], ["strummer's", 1.8796794244981716], ["unintimidated", 1.8796794244981716], ["hoteles", 1.8796794244981716], ["neurostimulator", 1.8796794244981716], ["nureyev's", 1.8796794244981716], ["zaire's", 1.8796794244981716], ["portishead's", 1.8796794244981716], ["rewi", 1.8796794244981716], ["behemoths", 1.8794914565557221], ["peacekeeping", 1.8793594790642143], ["pervasiveness", 1.8788621725744772], ["greatly", 1.8788537709191866], ["dubbing", 1.8787858587276123], ["abhiyan", 1.878775732467163], ["visa's", 1.8786053219698868], ["prevalent", 1.8785759311046852], ["jardin", 1.8785422518926034], ["afflictions", 1.8784230834946511], ["cryptocurrency", 1.878250868135553], ["gorillas", 1.877847941981994], ["generous", 1.8777640549580183], ["salinger", 1.877717337416232], ["sweaters", 1.877557205793093], ["faceted", 1.8772623855137422], ["pockmarked", 1.8772061620975165], ["harrington's", 1.877064218342348], ["ama's", 1.877064218342348], ["pflag", 1.877064218342348], ["aubree", 1.877064218342348], ["brunei's", 1.876835733992123], ["sandboxes", 1.876835733992123], ["ravenwood", 1.876835733992123], ["corruption", 1.8768330527982175], ["lifespan", 1.8767876099989438], ["rewound", 1.8767876099989438], ["bioengineering", 1.876608398959555], ["twilio", 1.8763347991876766], ["qi's", 1.876198536675027], ["iduna", 1.876198536675027], ["irregularities", 1.8760859197160427], ["oboe", 1.8760735088675018], ["thwart", 1.8759854458030711], ["arthouse", 1.8758294786889584], ["disappoint", 1.8757272780158933], ["delta's", 1.875722204657123], ["tessa", 1.8756711014869902], ["entering", 1.8756470475730698], ["bruges", 1.8756281344548795], ["auteur", 1.8756014759162096], ["stains", 1.8755284657690714], ["colony's", 1.8751422672666243], ["preoccupation", 1.875113401604654], ["farmhands", 1.8751134016046538], ["generated", 1.8750445985199569], ["theropod", 1.8749447156203425], ["stis", 1.8749346026499238], ["deserted", 1.8747931182596596], ["sauce", 1.8747772205590802], ["stunts", 1.8747284831568594], ["hypothesis", 1.874611139086367], ["jazzy", 1.8745291028750466], ["ramblings", 1.8744163221095766], ["processions", 1.8743255388308702], ["readjusting", 1.8740803794039218], ["soar", 1.8740480583855936], ["legoland", 1.8740006346960625], ["kazan's", 1.8740006346960623], ["bumba", 1.8740006346960623], ["all\u00e1", 1.8740006346960623], ["drc's", 1.8740006346960623], ["levain", 1.8740006346960623], ["osmosis", 1.873952038245244], ["redcoats", 1.873716303565281], ["goreng", 1.873716303565281], ["suomi", 1.873716303565281], ["dolci", 1.873657568218315], ["archdale", 1.873657568218315], ["moonglow", 1.873657568218315], ["lysimachus", 1.873657568218315], ["drenthe", 1.873657568218315], ["accessing", 1.8736171471489622], ["harringtons", 1.8735416957651163], ["prezi", 1.8735416957651163], ["cliente", 1.8735416957651163], ["allurement", 1.8735416957651163], ["umeboshi", 1.8735416957651163], ["conventionality", 1.8735416957651163], ["eliza", 1.8732063562813568], ["anaya", 1.8732063562813568], ["paralyzing", 1.8730984899703882], ["constanze", 1.8730984899703882], ["hemophilia", 1.8729506109941318], ["aromatics", 1.872675295844148], ["void", 1.8725587565606607], ["emus", 1.872476914553833], ["hussein's", 1.8723695600695678], ["spaghetti", 1.8722616291853866], ["glimpsing", 1.8721981133558903], ["nabucco", 1.8721981133558903], ["lathered", 1.8721607068001789], ["syllabary", 1.8721607068001789], ["perimeters", 1.8719593125761258], ["husbandry", 1.8718161357188374], ["fancy", 1.871737778929667], ["emphasis", 1.8715276792778641], ["abroad", 1.8715162452832084], ["binary", 1.87146452923555], ["yazdi", 1.8712786226009732], ["stressed", 1.8712345601168503], ["coraz\u00f3n", 1.8712208670879302], ["refrigerating", 1.8712208670879302], ["kims", 1.8712208670879302], ["declarative", 1.871051387795557], ["rehearsal", 1.8709834545510153], ["nightscapes", 1.8707285700957994], ["vizier's", 1.8707285700957994], ["pennywise's", 1.8707285700957994], ["stube", 1.8707285700957994], ["biofertilizers", 1.8707285700957994], ["rockmore", 1.8707285700957994], ["spasibo", 1.8707285700957994], ["irc's", 1.8707285700957994], ["kamerad", 1.8707285700957994], ["thanu", 1.8707285700957994], ["raipur", 1.870425618100642], ["gurus", 1.8704060828133395], ["underrated", 1.8703318295222888], ["chapatis", 1.8702337992494371], ["brill's", 1.8702337992494371], ["fionnuala", 1.8702337992494371], ["centering", 1.8699329385933663], ["alberto's", 1.8699306819399961], ["compound's", 1.8699306819399961], ["miniver", 1.8699306819399961], ["boatbuilding", 1.8699306819399961], ["guiseppe", 1.8699306819399961], ["contamination", 1.8697970380627857], ["immunizations", 1.869681129687011], ["topmost", 1.869681129687011], ["pace", 1.869618440378545], ["universally", 1.8695987081466476], ["drums", 1.8694836545528326], ["zeuxis", 1.8693798112132503], ["kebap", 1.8693798112132503], ["bahnhofstrasse", 1.8693798112132503], ["jiuzhaigou", 1.8693798112132503], ["banjo's", 1.8693798112132503], ["ichiba", 1.8693798112132503], ["lelouch's", 1.8693798112132503], ["eleftherios", 1.8693798112132503], ["automattic", 1.8693798112132503], ["khorasani", 1.8693798112132503], ["kesey's", 1.8693798112132503], ["grading", 1.8692933992034868], ["anchored", 1.869067067747359], ["carrie's", 1.8690087552761296], ["rightfully", 1.8687254306105787], ["owned", 1.8686066416344147], ["altitudes", 1.8685828342404445], ["sticky", 1.8685502924380684], ["dawn", 1.8684862366993857], ["lateness", 1.8684780695603458], ["colorblind", 1.8684780695603458], ["thizz", 1.8683560544710742], ["stork's", 1.8683560544710742], ["louvin", 1.8683560544710742], ["desde", 1.8678522281193068], ["outshined", 1.8677638655473114], ["stratofortress", 1.867552460469151], ["sarcopenia", 1.867552460469151], ["pslf", 1.867552460469151], ["lihtc", 1.867552460469151], ["gradkowski", 1.867552460469151], ["huwa", 1.867552460469151], ["wirework", 1.867552460469151], ["patrias", 1.867552460469151], ["stratocasters", 1.867552460469151], ["sol\u00e9", 1.867552460469151], ["droney", 1.867552460469151], ["caesar's", 1.8674427335325783], ["vasco", 1.8673575176235908], ["indignation", 1.867085572354034], ["stupa", 1.867085572354034], ["grocer", 1.8669639460383312], ["tupi", 1.8667901941587557], ["turbaned", 1.8667901941587557], ["esotericism", 1.8667901941587557], ["pond", 1.8667283348508439], ["designating", 1.8666033937190543], ["flow's", 1.8663719595459722], ["noblemen", 1.8661842901684411], ["cetaceans", 1.8661842901684411], ["posting", 1.8661488015061514], ["harrison's", 1.8661482856868434], ["recalcitrant", 1.8661245531553114], ["stamp", 1.8660757910818035], ["subtractions", 1.8659849328038256], ["translator", 1.8659192071726785], ["sasi", 1.8658677731585753], ["muller's", 1.8657558732055928], ["fame's", 1.8657558732055928], ["demonstrate", 1.8657507620285356], ["painfully", 1.865508553175853], ["stadio", 1.865477402179741], ["fountain", 1.8653078534001635], ["sausages", 1.865224689723781], ["anish", 1.8651605213572204], ["identify", 1.8651204529556948], ["physiotherapy", 1.8650681512389147], ["swanky", 1.865045678643018], ["nostrils", 1.8648210061902335], ["hardhat", 1.8647675626754305], ["trapping", 1.864501013145349], ["pantsuits", 1.8644681459184176], ["gibberish", 1.8642068221614665], ["ravages", 1.8641189478710214], ["coconuts", 1.8640075381125571], ["advisor", 1.8639232881457604], ["shanti", 1.8638986474821255], ["coefficients", 1.8638719999403437], ["mazda's", 1.8638648139555187], ["paperweight", 1.8637150239065434], ["agricultural", 1.863431784541208], ["kaikoura", 1.8633343860242744], ["deontological", 1.8633343860242744], ["mechanization", 1.8632962264013782], ["destitution", 1.8631885566847306], ["tuple", 1.8631382455625876], ["genderfluid", 1.8628030457642875], ["speckled", 1.8628030457642872], ["citrus", 1.8625149834376225], ["seriousness", 1.8622496698346431], ["roshan", 1.8622399479995455], ["myrtles", 1.862091780760177], ["rexy", 1.862091780760177], ["misfit", 1.8619614027924671], ["breeds", 1.8616596167376382], ["tartuffe", 1.8616455997612888], ["laughs", 1.8613995420949268], ["softer", 1.8612401485298957], ["lioness", 1.8608826302531902], ["marmalade", 1.8608826302531902], ["francophone", 1.8608826302531902], ["arcturus", 1.8608826302531902], ["scoville", 1.86088263025319], ["sunt", 1.86088263025319], ["elke", 1.86088263025319], ["elize", 1.86088263025319], ["mahmoud's", 1.86088263025319], ["rekordbox", 1.86088263025319], ["admonishing", 1.86088263025319], ["connell's", 1.86088263025319], ["sarson", 1.86088263025319], ["rauparaha", 1.86088263025319], ["modernism's", 1.86088263025319], ["vinda", 1.86088263025319], ["oulipo", 1.86088263025319], ["recontextualizing", 1.86088263025319], ["kloten", 1.86088263025319], ["kesi", 1.86088263025319], ["solito", 1.86088263025319], ["chariot's", 1.86088263025319], ["jinhao", 1.86088263025319], ["nr's", 1.86088263025319], ["frankie's", 1.8608826302531898], ["darya", 1.8608826302531898], ["shoji", 1.8608826302531898], ["apomixis", 1.8608826302531898], ["progr\u00e8s", 1.8608826302531898], ["d\u00e9couverte", 1.8608826302531898], ["redentor", 1.8608826302531898], ["labialized", 1.8608826302531898], ["hermit's", 1.8608826302531896], ["hiring", 1.8606637028849249], ["situation", 1.8605195312033844], ["fortune", 1.8603863948851225], ["priestess", 1.860186452688561], ["reunited", 1.860173722584522], ["animal's", 1.8601537298888136], ["bird", 1.860127236835757], ["clogging", 1.8601196607450912], ["rides", 1.8599117349678402], ["huck", 1.8598294991664592], ["flaw", 1.859635807720357], ["maraca", 1.8594949474565805], ["chloris", 1.8594949474565805], ["bleats", 1.8594299506042806], ["francoist", 1.859358565854948], ["node's", 1.859358565854948], ["blackbeard's", 1.8593201763235063], ["comebacks", 1.8591691103358299], ["dayu", 1.8590950388696614], ["stems", 1.859047440282526], ["chili", 1.8588034317836333], ["mystique", 1.8584014534128523], ["muchos", 1.8580673766522018], ["kushan", 1.8580673766522018], ["chest", 1.857959189263064], ["miami's", 1.8578666130242545], ["talons", 1.857723236992998], ["underlines", 1.8576463126353582], ["windblown", 1.8576350165878266], ["economies", 1.8573654981341545], ["preimplantation", 1.8571682936658582], ["borris", 1.8571682936658582], ["hogging", 1.8569976978309495], ["letterpress", 1.8564833805362793], ["illicitly", 1.8564833805362793], ["rattle", 1.8564519573240155], ["textbooks", 1.8562677499155815], ["personal", 1.8562657568920555], ["complicity", 1.8562449165329329], ["baguette", 1.8561905409182098], ["shawl", 1.8558532177389921], ["dwindles", 1.8558395878947802], ["immigrating", 1.8557562318227403], ["glade", 1.8557392288428716], ["corbusier", 1.8557135118358201], ["marathi", 1.8556258996592545], ["biodynamics", 1.8555809703379387], ["sezs", 1.8555809703379387], ["mckagan", 1.8555809703379387], ["bayoumi", 1.8555809703379387], ["gedung", 1.8555809703379387], ["muscadet", 1.8555809703379387], ["unplanted", 1.8555809703379387], ["kayvon", 1.8555809703379387], ["colored", 1.855520086759557], ["recyclers", 1.855488767556804], ["joad", 1.855488767556804], ["fascinator", 1.855488767556804], ["tunas", 1.855488767556804], ["pairs", 1.8554009604896533], ["jackpot", 1.8553863524423158], ["sage", 1.8553770603411985], ["hajj", 1.8552435919796955], ["brazil's", 1.8551315581067123], ["nasl's", 1.8550854881028687], ["kuciak", 1.8550854881028687], ["verran", 1.8550854881028687], ["lispro", 1.8550854881028687], ["didrikson", 1.8550854881028687], ["omb's", 1.8550854881028687], ["ghostemane", 1.8550854881028687], ["transparente", 1.8550854881028687], ["lido's", 1.8550854881028687], ["kulan", 1.8550854881028687], ["napoletano", 1.8550854881028687], ["likha", 1.8550854881028687], ["booktopia", 1.8550854881028687], ["sagan's", 1.8550854881028684], ["seva", 1.8550854881028684], ["chair", 1.8549835057246322], ["monocle", 1.8545745535404674], ["shaq's", 1.8544878445822168], ["gravitational", 1.8544588714432544], ["starbound", 1.8542602721384098], ["norma's", 1.8542602721384098], ["undp", 1.8541157843249965], ["franca", 1.8540960415214134], ["disturbed", 1.8540013949590683], ["divided", 1.8539016920244435], ["zoroastrians", 1.853898765042414], ["undervaluing", 1.853898765042414], ["firehose", 1.853898765042414], ["midpoint", 1.8538736561279423], ["postmarked", 1.853807030898615], ["measurable", 1.8537596345393024], ["tatlin", 1.8537528117464726], ["campiglio", 1.8537528117464726], ["weez", 1.8537528117464726], ["synergized", 1.8537528117464726], ["hookeri", 1.8537528117464726], ["messerschmitts", 1.8537528117464726], ["obstinacy", 1.8535850513110208], ["paddler", 1.8535418703705346], ["coru\u00f1a", 1.8535418703705346], ["they'd", 1.8534897359942248], ["plucky", 1.8534450497965425], ["plushie", 1.8534092060353058], ["windhoek", 1.8532871909460342], ["throwers", 1.8532623082619812], ["soren", 1.8532623082619812], ["establishment", 1.8531466416255216], ["dioxide", 1.8530089266576915], ["emoji", 1.8528552934168037], ["peka", 1.8522674328909066], ["airtrain", 1.8522674328909066], ["orcish", 1.8522674328909066], ["liara", 1.852146092082987], ["toothbrushing", 1.852146092082987], ["globes", 1.8520212843948414], ["krakow", 1.8519188410901204], ["sharma's", 1.8518928590925468], ["ruggedness", 1.8518928590925468], ["porches", 1.8518601690155987], ["chanting", 1.851660030486947], ["baseboards", 1.851624507714617], ["mandoline", 1.8516245077146167], ["kwanzaa", 1.851484233130699], ["amok", 1.8514044504726135], ["compliments", 1.8512997969449967], ["demands", 1.8512282456604365], ["genderqueer", 1.8510366904105806], ["counts", 1.8508150278905637], ["filleting", 1.8506015107490286], ["idols", 1.8504440221870446], ["mozarts", 1.8503691690653188], ["geotagged", 1.8503691690653188], ["insurtech", 1.8503691690653188], ["cash's", 1.8502793104226873], ["unjust", 1.8502753686681035], ["paraphrasing", 1.8500692605052844], ["exploits", 1.8499039716676253], ["pomelo", 1.8498239004137598], ["lima's", 1.8498059479302544], ["toodles", 1.8498059479302544], ["multifactor", 1.8497205236449874], ["johanna's", 1.8497205236449874], ["lilya", 1.8497205236449874], ["earmarked", 1.8497173344716709], ["compose", 1.8496828736822217], ["croplands", 1.849604553706201], ["ghostlike", 1.849604553706201], ["weatherproofing", 1.849604553706201], ["veracrypt", 1.849604553706201], ["francine's", 1.849604553706201], ["handclaps", 1.849604553706201], ["buba", 1.849604553706201], ["andreea", 1.849604553706201], ["cassata", 1.849604553706201], ["discernible", 1.84958037776505], ["expert", 1.8491944418317647], ["explanation", 1.8490180937257574], ["schemas", 1.8488146495122484], ["tastemakers", 1.8487199986829077], ["sluggers", 1.8487199986829077], ["metalloids", 1.8485589042250232], ["ilocano", 1.8485589042250232], ["transporte", 1.8485589042250232], ["parkfield", 1.8485589042250232], ["weavers", 1.8484767460515021], ["niece's", 1.848476746051502], ["whimsically", 1.848476746051502], ["wall's", 1.8483694905684902], ["ducklings", 1.8483694905684902], ["skaters", 1.8482969506282894], ["pollen", 1.8482664768277446], ["finny", 1.8482507010017393], ["tripitaka", 1.8482235647412635], ["bellavista", 1.8482235647412635], ["jetlagged", 1.8481764173836772], ["hospital's", 1.8481289175224827], ["norwich's", 1.8481031932783925], ["stewarding", 1.8481031932783925], ["moderating", 1.8480930589112436], ["producing", 1.8476848811024582], ["sag's", 1.847684881102458], ["verse's", 1.847684881102458], ["pilea", 1.847684881102458], ["gelu", 1.847684881102458], ["zophar", 1.847684881102458], ["drinkability", 1.847684881102458], ["tiradentes", 1.847684881102458], ["caramello", 1.847684881102458], ["thersites", 1.847684881102458], ["sampled", 1.8473664918233355], ["glamping", 1.8473390885482903], ["gal\u00e1pagos", 1.8472497904711154], ["iterates", 1.8469893475503774], ["glutinous", 1.8469434345209563], ["munnetra", 1.8468306578564562], ["mimir", 1.8468306578564562], ["morio", 1.8468306578564562], ["tuxedos", 1.846720661986423], ["necessarily", 1.8467103453207983], ["eurobasket", 1.8465405675922792], ["barb's", 1.8465405675922792], ["mausoleums", 1.8464571835070411], ["khulna", 1.846386209599311], ["yana", 1.8461637845901706], ["parlament", 1.8459521350432813], ["headcanons", 1.8459521350432813], ["hace", 1.8459357818174211], ["favorites", 1.8458774179330533], ["throbbing", 1.8458451948572043], ["lazlo", 1.8455667238313531], ["kshv", 1.845503434961841], ["uf's", 1.845503434961841], ["facit", 1.845503434961841], ["walpurgis", 1.845503434961841], ["monologues", 1.845460950996948], ["nots", 1.8454304613187034], ["mackintosh's", 1.845268817184704], ["tardigrade", 1.845268817184704], ["osman's", 1.8451789793649773], ["essentiality", 1.8451789793649773], ["alignments", 1.845085663952229], ["ossificans", 1.844977650507436], ["asiya", 1.844977650507436], ["\u00e9nergie", 1.844977650507436], ["malama", 1.8447411452324065], ["prerna", 1.8447411452324065], ["tebe", 1.8447411452324065], ["nyu's", 1.844701042164032], ["currie's", 1.8445199532485657], ["houseflies", 1.8445199532485657], ["congresswoman", 1.8444806198903811], ["newsfeed", 1.8444146423748433], ["architectures", 1.8443414513176064], ["virtuousness", 1.8441179218725305], ["oopsie", 1.8441179218725305], ["damascena", 1.8441179218725305], ["nezalezhnosti", 1.8441179218725305], ["tracheae", 1.8441179218725305], ["whitetip", 1.8441179218725305], ["juez", 1.8441179218725305], ["inout", 1.8441179218725305], ["lightstick", 1.8441179218725305], ["bhaer", 1.8441179218725305], ["hallway", 1.843868846205161], ["aetius", 1.8436956370266702], ["granda", 1.8436956370266702], ["ursula's", 1.8435988906533152], ["goldie's", 1.8435988906533152], ["ballad", 1.8434294851785722], ["endure", 1.8432403743536638], ["dancers", 1.843226900847765], ["temperamental", 1.8432230086137764], ["lurk", 1.843159938536493], ["besoin", 1.843159938536493], ["kratos", 1.843159938536493], ["numbingly", 1.843159938536493], ["spotless", 1.843025675720457], ["stimulating", 1.8430067164179047], ["cling", 1.8428158086002464], ["coordinates", 1.8426773236334664], ["arthritis", 1.8422890445946403], ["vegetarians", 1.842273803950658], ["erik", 1.8422349546264356], ["alerts", 1.8420858360082084], ["stroganoff", 1.8420858360082082], ["alaskan", 1.8420382491873348], ["japan's", 1.842010144219302], ["dictionaries", 1.8419862503433342], ["combines", 1.8419388219750072], ["taki's", 1.8419216718663307], ["westenra", 1.8419216718663307], ["vodun", 1.8419216718663307], ["bucatini", 1.8419216718663307], ["tcpdump", 1.8419216718663307], ["redfern's", 1.8419216718663307], ["ohrid", 1.8416292337188565], ["cataloging", 1.8414579681419876], ["herrings", 1.8412530666429243], ["medgar", 1.8412008282955479], ["scooping", 1.8411758550462727], ["dingy", 1.8411758550462727], ["dissent", 1.8410605563221185], ["hopping", 1.8410516629419413], ["dreamland", 1.84101555590885], ["surviving", 1.8409404612560842], ["settlin", 1.8408731396053062], ["laravel", 1.8408731396053062], ["uday's", 1.8408731396053062], ["ipms", 1.8408731396053062], ["plin", 1.8408731396053062], ["ihres", 1.8408731396053062], ["hornpipes", 1.8408731396053062], ["pawprint", 1.8408731396053062], ["panang", 1.8408731396053062], ["kualoa", 1.8408731396053062], ["cyoa", 1.8408731396053062], ["kimbal", 1.8408731396053062], ["arches", 1.8406460815554009], ["blemishes", 1.8405228640578597], ["dewi", 1.840433370580078], ["genug", 1.8404333705800777], ["beautyblender", 1.8404333705800777], ["elating", 1.8404333705800777], ["overlanding", 1.8404333705800777], ["litha", 1.8404333705800777], ["mogadishu's", 1.8404333705800777], ["hollywood's", 1.84020615658371], ["cbas", 1.840107226087684], ["colecci\u00f3n", 1.840107226087684], ["flixbus", 1.840107226087684], ["hidalgo's", 1.840107226087684], ["hypercars", 1.840107226087684], ["orienteering", 1.8401072260876836], ["endorphin", 1.8399738366548395], ["nadia's", 1.8398557078774478], ["cakes", 1.8396804835243163], ["pedagogical", 1.839675990592185], ["prove", 1.8396725701664332], ["cowhands", 1.8396558321894654], ["crisping", 1.8396558321894654], ["predatory", 1.8396154001931535], ["chidi", 1.8394931747330385], ["objectified", 1.8393446368474817], ["artie's", 1.8392990193122805], ["paycheck", 1.8392901102818977], ["fitting", 1.839221562599449], ["teus", 1.8388680131938273], ["valentina's", 1.8388680131938273], ["jabiru", 1.8388680131938273], ["delhomme", 1.8388680131938273], ["elena", 1.838808103203923], ["suddenness", 1.8387292656073186], ["statuettes", 1.8387292656073186], ["orne", 1.8387292656073186], ["flowy", 1.838462357599537], ["photographer", 1.8383441914155685], ["namespaces", 1.8383264771592116], ["secunderabad", 1.8383264771592116], ["concealing", 1.8382369686151883], ["ravenous", 1.8382284069283688], ["coriolis", 1.8381493345956466], ["proteges", 1.8379087706204347], ["seymour's", 1.8379087706204347], ["brevifolia", 1.8379087706204345], ["ricocheted", 1.8379087706204345], ["bookstall", 1.8379087706204345], ["furano", 1.8379087706204345], ["smoo", 1.8379087706204345], ["nice's", 1.8379087706204345], ["ghostbusting", 1.8379087706204345], ["readville", 1.8379087706204345], ["amrs", 1.8379087706204345], ["brodersen", 1.8379087706204345], ["naiman", 1.8379087706204345], ["denyin", 1.8379087706204345], ["nogai", 1.8379087706204345], ["baumgartner's", 1.8379087706204345], ["chichicastenango", 1.8379087706204345], ["p\u00e4rt", 1.8379087706204345], ["feist's", 1.8379087706204345], ["strim", 1.8379087706204345], ["pageid", 1.8379087706204345], ["lombardic", 1.8379087706204345], ["giove", 1.8379087706204345], ["periander", 1.8379087706204345], ["spondylus", 1.8379087706204345], ["abeni", 1.8379087706204345], ["fassa", 1.8379087706204345], ["necromancer's", 1.8379087706204345], ["nantou", 1.8379087706204345], ["sprung", 1.8377121072305822], ["altai", 1.837694061651904], ["granulated", 1.8376439422673192], ["blisters", 1.8375553266260845], ["profiles", 1.8375416625541379], ["barbershops", 1.8373812422706581], ["kesha's", 1.8373812422706581], ["tortoises", 1.8372229837657252], ["reinforce", 1.837055044730115], ["productions", 1.8370251606345593], ["suited", 1.8369984045938657], ["yama", 1.8369638303784703], ["mating", 1.836934562648666], ["potentialities", 1.8368402190096087], ["mult", 1.8368402190096087], ["greenlandic", 1.8368289182287658], ["atorvastatin", 1.8368289182287658], ["pleas", 1.836760077638797], ["nossa", 1.8365573671126252], ["maude's", 1.836520622002141], ["persone", 1.836520622002141], ["tacular", 1.836520622002141], ["toby's", 1.8365042551843709], ["geoengineering", 1.8365042551843709], ["headers", 1.8364467977347136], ["music", 1.836198293892973], ["traverse", 1.8361697135167596], ["futebol", 1.8360708618498143], ["appointments", 1.8359594734394735], ["figured", 1.8357670442775278], ["heian", 1.835735567682201], ["against", 1.835648699193264], ["haunt", 1.8354988326070851], ["lack", 1.835191117657627], ["decoded", 1.835165623201598], ["aisne", 1.8349539333686333], ["sydney's", 1.834889349068701], ["repentant", 1.834786110087827], ["polly", 1.8346869385125737], ["retention", 1.8345729377511582], ["weightless", 1.8345166571176115], ["jam's", 1.8343743306769338], ["sprucing", 1.8343743306769338], ["gigot", 1.8343330726231186], ["huashan", 1.8343330726231186], ["mandarin's", 1.8343330726231186], ["undertrained", 1.8343330726231186], ["herbie's", 1.8343330726231186], ["lavender's", 1.8343330726231186], ["extra\u00f1o", 1.8343330726231186], ["patrimonio", 1.8343330726231186], ["meneer", 1.8343330726231186], ["blowholes", 1.8342033093893446], ["afterlives", 1.8342033093893446], ["wapello", 1.8342033093893446], ["lingo", 1.834124840798569], ["maters", 1.8340429769322304], ["halles", 1.8340429769322304], ["martelli", 1.8340429769322304], ["mihir", 1.8340429769322304], ["kermis", 1.8339133167712598], ["coutume", 1.8339133167712598], ["numerologists", 1.8339133167712598], ["muni's", 1.8339133167712598], ["entasis", 1.8339133167712598], ["cornermen", 1.8339133167712598], ["slithy", 1.8339133167712598], ["skyspace", 1.8339133167712598], ["l\u00e9man", 1.8339133167712598], ["loch's", 1.8339133167712598], ["cyberia", 1.8339133167712598], ["riyadi", 1.8339133167712598], ["chorizos", 1.8339133167712598], ["gyaltsen", 1.8339133167712598], ["shifra", 1.8339133167712598], ["scuffles", 1.8339133167712596], ["weirdly", 1.8339133167712596], ["darpa's", 1.8339133167712596], ["puis", 1.8339133167712596], ["lbj's", 1.8339133167712596], ["gandhara", 1.8339133167712596], ["elazar", 1.8339133167712596], ["bird's", 1.8338815881671633], ["chen", 1.8338127732780156], ["counting", 1.8338031195763869], ["sociable", 1.8336242566053225], ["odors", 1.8336242566053225], ["secant", 1.8334663115754266], ["recommendation", 1.833448479823441], ["hardened", 1.8331474208596883], ["nathan's", 1.833072168592486], ["crisis", 1.8328875774143691], ["cancellations", 1.8326415930460953], ["slurred", 1.8326198245179297], ["aesir", 1.8324480909114604], ["winehouse's", 1.8324480909114604], ["fickleness", 1.8324480909114604], ["bridgers", 1.8324060497503136], ["edirne", 1.832185745089143], ["milly's", 1.832185745089143], ["handlebars", 1.83200487373157], ["bleached", 1.8318969195015826], ["gerund", 1.8318969195015826], ["buchanan's", 1.8318969195015826], ["moscow's", 1.831741291524393], ["hutong", 1.8317412915243927], ["curley's", 1.8317412915243927], ["gossiping", 1.831727797040147], ["simvastatin", 1.83165410726492], ["moisturizing", 1.8315596312310183], ["become", 1.8314544890034492], ["uruguay's", 1.8313448107253616], ["infighting", 1.8311960055892011], ["hellenistic", 1.8310944902208883], ["praia", 1.8310744416678986], ["jyrki", 1.8310509020733432], ["befalling", 1.8310509020733432], ["segment's", 1.8310509020733432], ["kaido", 1.8310509020733432], ["viral", 1.8307983610640968], ["playwright's", 1.8306443090764801], ["goopy", 1.8306443090764801], ["inculcating", 1.8306443090764801], ["squished", 1.8304935767332051], ["canals", 1.8304841060503785], ["pillowcases", 1.8304760513274843], ["investors", 1.8304060745431072], ["inconsistent", 1.8303203223333102], ["nian", 1.8300878822348154], ["pups", 1.8300163154930096], ["urbanites", 1.8298679197489705], ["storm", 1.8298000538616523], ["fashion", 1.8297834621409406], ["caleta", 1.8297764309274236], ["montfaucon", 1.8297764309274236], ["phoenix's", 1.8297263000663027], ["parsi", 1.8297263000663027], ["splints", 1.829712066932366], ["eliminating", 1.8293149832569984], ["acci\u00f3n", 1.8290726707616825], ["cordilleras", 1.8290726707616825], ["squeaky", 1.8290726707616822], ["bliss", 1.828993541509216], ["gogol's", 1.8288772778901128], ["taupin", 1.8288772778901128], ["nyonya", 1.8288772778901128], ["hattin", 1.828755838495831], ["baronetage", 1.828755838495831], ["frogfish", 1.828755838495831], ["nadal's", 1.8287192267673322], ["laughing", 1.8286334275863945], ["awakened", 1.8286273313288015], ["graduation", 1.8285425047701573], ["disappointing", 1.82850918005069], ["daintree", 1.8285028915486823], ["digesters", 1.8284425210094732], ["fran\u00e7aise", 1.828415440193676], ["lola", 1.8283281823480513], ["graininess", 1.8282355665645371], ["nevado", 1.8282355665645371], ["regression", 1.8279049633879438], ["codes", 1.827772886324182], ["appleton's", 1.8276525832843833], ["mislabel", 1.8276525832843833], ["bandha", 1.8276525832843833], ["acceso", 1.8276525832843833], ["verulamium", 1.8276525832843833], ["suivre", 1.8276525832843833], ["shimin", 1.8276525832843833], ["riverfest", 1.8276525832843833], ["daulah", 1.8276525832843833], ["t'challa", 1.827652583284383], ["succeed", 1.827609412625073], ["spain's", 1.8275150796417532], ["ears", 1.8274029951586783], ["marbling", 1.8273532134918713], ["restrictive", 1.827275912242854], ["paprikash", 1.8272232545121763], ["phytotherapy", 1.8272232545121763], ["mckellen's", 1.8272232545121763], ["bioinformaticians", 1.8272232545121763], ["mandovi", 1.8272232545121763], ["listos", 1.8272232545121763], ["b\u0119dzie", 1.8272232545121763], ["actividad", 1.8272232545121763], ["acacia's", 1.8272232545121763], ["ljajic", 1.8272232545121763], ["phae", 1.8272232545121763], ["biscayan", 1.8272232545121763], ["igloos", 1.8270484006122227], ["daylong", 1.8270484006122227], ["summations", 1.8270484006122227], ["containerized", 1.8270484006122227], ["sharp", 1.8269513045889707], ["marlin", 1.8268272618563997], ["devour", 1.8265149780728382], ["oats", 1.8262845468221878], ["helm", 1.8261905603379551], ["pantries", 1.8260346409226058], ["soda", 1.8259205929575242], ["rubbery", 1.8258291490936296], ["pilgrimages", 1.8258291490936296], ["mycelium", 1.8256886325179051], ["forms", 1.8255131540655098], ["adriatic", 1.8251665123987417], ["choreography", 1.8251504718729237], ["javafx", 1.8250742121663812], ["whistles", 1.8250028671208574], ["thwarting", 1.8249773765756498], ["complicated", 1.8249686468354223], ["armadillos", 1.8249235456106163], ["raku", 1.8249235456106163], ["blackboards", 1.824887149775958], ["masquerading", 1.8248871497759578], ["magic", 1.8248732028921166], ["oatmeal", 1.8248313428294816], ["orchestrating", 1.82456563903045], ["dial's", 1.824394735542343], ["muddied", 1.824394735542343], ["subfields", 1.824394735542343], ["seger's", 1.824394735542343], ["lizo", 1.824394735542343], ["padr\u00f3n", 1.8243947355423429], ["f\u00edsica", 1.8243947355423429], ["linkups", 1.8243947355423429], ["arriv\u00e9e", 1.8243947355423429], ["culprit's", 1.8243947355423429], ["geffrye", 1.8243947355423429], ["itadakimasu", 1.8243947355423429], ["kenge", 1.8243947355423429], ["roqueforti", 1.8243947355423429], ["tet's", 1.8243947355423429], ["ock's", 1.8243947355423429], ["mehdi's", 1.8243947355423429], ["rimuru", 1.8243947355423429], ["rumack", 1.8243947355423429], ["iwmi", 1.8243947355423429], ["ekundayo", 1.8243947355423429], ["noam", 1.824178830839912], ["spoelstra", 1.8239852193278554], ["eluding", 1.8238132551485446], ["suitable", 1.8237683600164734], ["joana", 1.8236649776481262], ["mathilda", 1.8236649776481262], ["letters", 1.8235835385833448], ["takeoffs", 1.8235155091637683], ["narrators", 1.8234779542682011], ["alignment", 1.8234386679860723], ["alzheimer's", 1.823420660453782], ["deductive", 1.8234151947447366], ["makerspaces", 1.823343211199091], ["interjects", 1.8231379768266998], ["charmeuse", 1.8229054337174107], ["lazio's", 1.8229054337174107], ["lemonis", 1.8229054337174107], ["tengwar", 1.8229054337174107], ["funkiness", 1.8229054337174107], ["schuss", 1.8229054337174107], ["strategists", 1.822821459816834], ["puffiness", 1.8227943892830956], ["bounty", 1.822762731524367], ["ghibli", 1.8225371699934272], ["tenuously", 1.822513916227351], ["unsporting", 1.822333272564329], ["peacemaking", 1.8222997368029643], ["homework", 1.822010859754568], ["thereon", 1.8219167800441534], ["obligations", 1.8218999644403118], ["articles", 1.8217545198470864], ["thrum", 1.8216571657355272], ["uncoordinated", 1.8216235030326582], ["mina's", 1.8214988708827522], ["rosetta", 1.8214093623387282], ["bj\u00f6rn", 1.8208636489574224], ["shrill", 1.820837285992142], ["takashi", 1.8207640992527065], ["peterson", 1.8207562533861485], ["asimov", 1.8207196957872938], ["mento", 1.8205306619723594], ["fessed", 1.8205306619723594], ["sangeet", 1.8203026352009395], ["spandex", 1.8202588280814158], ["bruckner", 1.8202520051384914], ["space", 1.8199359148086776], ["lata", 1.8198035434926338], ["blasphemous", 1.819649200103071], ["bridge", 1.8195367037881418], ["treaded", 1.8195296829142302], ["frivolities", 1.8195296829142302], ["andouille", 1.8195296829142302], ["cowpox", 1.8195296829142302], ["herk", 1.8195296829142302], ["mood", 1.8191086347231427], ["blatant", 1.819089757942539], ["danza", 1.8189708593015417], ["woocommerce", 1.8189708593015417], ["mitchell's", 1.818919307307751], ["katanas", 1.818805780116174], ["grotte", 1.818805780116174], ["lantern's", 1.818805780116174], ["sung's", 1.8186636526034667], ["fermina", 1.8186636526034665], ["bonspiel", 1.8186636526034665], ["jungfraujoch", 1.8186636526034665], ["sabda", 1.8186636526034665], ["zo\u00eb's", 1.8186636526034665], ["t\u00e9cnica", 1.8186636526034665], ["gamechangers", 1.8186636526034665], ["hawaii's", 1.8185623625086405], ["wayward", 1.8185210744425482], ["hydrotherapy", 1.81846935093118], ["present", 1.8183904229080707], ["backroad", 1.8178287904786945], ["bifidobacterium", 1.8178287904786945], ["pepperonis", 1.8178287904786945], ["alcyone", 1.8178287904786945], ["bolster", 1.817705288178688], ["fizzle", 1.8176975903106054], ["shabbat", 1.8176530499717816], ["malting", 1.8176062900147436], ["overworked", 1.8174620355472824], ["lost", 1.817249363246682], ["allocated", 1.8172051089513892], ["garc\u00eda's", 1.817199939402176], ["hmac", 1.817199939402176], ["succop", 1.817199939402176], ["pointillist", 1.817199939402176], ["subby", 1.817199939402176], ["smitten", 1.8170971566001737], ["goslar", 1.8168800661510933], ["tsunamis", 1.816880066151093], ["lindisfarne", 1.8168617508278457], ["mellower", 1.8168380709572567], ["chugging", 1.8167769905043751], ["cynicism", 1.8167728197582995], ["worksheet", 1.8164197080262237], ["clawed", 1.8163311124018247], ["undersea", 1.8163311124018247], ["antico", 1.8163311124018244], ["lighthouse", 1.8161515548217517], ["lifeguards", 1.8160911446234633], ["marvin", 1.816091144623463], ["rioja", 1.816070501007549], ["restaurant", 1.8158951128036451], ["ikat", 1.8158674292753556], ["dapp", 1.8158674292753556], ["liven", 1.8158495611369927], ["yumiko", 1.8158049548018163], ["synonyms", 1.8157007063087711], ["parakeet", 1.8156956348165785], ["recordar", 1.8154952490275025], ["sforzesco", 1.8154952490275025], ["czas", 1.8154952490275025], ["maalaea", 1.8154952490275025], ["kirito's", 1.8154952490275025], ["coliseums", 1.8154952490275025], ["laja", 1.8154952490275025], ["biesbosch", 1.8154952490275025], ["identidad", 1.8154952490275025], ["saguna", 1.8154952490275025], ["barsana", 1.8154952490275025], ["meazza", 1.8154952490275025], ["harlem", 1.8154285641698704], ["tact", 1.8153943881803343], ["display", 1.8149198461616902], ["quakes", 1.8148036317897778], ["dimming", 1.814769631421976], ["coronaviruses", 1.8147696314219754], ["sleep's", 1.8147696314219754], ["whistleblower's", 1.8147696314219754], ["powderfinger", 1.8147696314219754], ["misplacement", 1.8147696314219754], ["serdar", 1.8147696314219754], ["ryoji", 1.8147696314219754], ["predestined", 1.8147118638067787], ["scratched", 1.8147118638067785], ["roses", 1.8146646302861171], ["woefully", 1.814658782867093], ["sinking", 1.8146358429924598], ["invoking", 1.8146067129929255], ["bohol", 1.8145054930506181], ["invoices", 1.8144866405558204], ["militarization", 1.8141938187414612], ["cardiologist", 1.8140090123626813], ["festive", 1.8138708585415304], ["classwork", 1.8138258281088564], ["yaks", 1.813771677588552], ["brocades", 1.813725760480692], ["experientially", 1.813725760480692], ["aquavit", 1.813725760480692], ["filmmaker", 1.8135796185489124], ["nepotism", 1.8135041065687179], ["formats", 1.8134166385249928], ["jemison", 1.8131676910159287], ["harrow's", 1.812731938146182], ["segmentations", 1.812731938146182], ["di\u00e1logo", 1.812731938146182], ["flavanols", 1.812731938146182], ["tripes", 1.812731938146182], ["flextime", 1.812731938146182], ["webpack", 1.8124465584329976], ["romanticism", 1.812349428392353], ["counterpoints", 1.8123378659857154], ["megaliths", 1.8123378659857154], ["effects", 1.812218311297084], ["straightened", 1.8121780478317486], ["silverfox", 1.8120954451903721], ["shkodra", 1.8120954451903721], ["lauras", 1.8120954451903721], ["rastro", 1.8120954451903721], ["swearer", 1.8120954451903721], ["jedem", 1.8120954451903721], ["kadyrov's", 1.8120954451903721], ["satem", 1.8120954451903721], ["canine's", 1.8120954451903721], ["sadra", 1.8120954451903721], ["cynewulf", 1.8120954451903721], ["processed", 1.811639975856139], ["commoners", 1.8113854691768458], ["rehana", 1.8113854691768458], ["silverbacks", 1.8113274454255108], ["borglum", 1.8113274454255108], ["yowl", 1.8113274454255108], ["stockpot", 1.8113274454255108], ["poppyseed", 1.8113274454255108], ["avraham", 1.8112590934464383], ["systems", 1.8107289274549132], ["gira", 1.810685410940002], ["speyer", 1.8105885051112116], ["contented", 1.8105517403998508], ["copays", 1.8104772964320697], ["pungency", 1.8104772964320697], ["centricity", 1.8104772964320697], ["eintracht", 1.8103401390611278], ["meuse", 1.8101971111412354], ["alfredo", 1.8101919206118844], ["buicks", 1.8101112180398764], ["risorgimento", 1.8101112180398764], ["coelacanth", 1.8101112180398764], ["trances", 1.8100616680352766], ["inflammations", 1.8100616680352766], ["aesop's", 1.8100616680352766], ["bookbinding", 1.8097995776580047], ["strawberries", 1.8093999984075688], ["nacho's", 1.8091914460794902], ["gordes", 1.8091914460794902], ["habibti", 1.8091914460794902], ["eucerin", 1.8091914460794902], ["familiarities", 1.8091914460794902], ["marquesan", 1.8091914460794902], ["hamsterdam", 1.8091914460794902], ["aeternum", 1.8091914460794902], ["gnomeo", 1.8091914460794902], ["mccallum's", 1.8091914460794902], ["quadrivium", 1.8091914460794902], ["frankincense", 1.8090034781370403], ["ceaselessly", 1.8089855051066865], ["marmosets", 1.808894711382741], ["burrow", 1.8086991490873598], ["laureates", 1.808647328351346], ["ocean", 1.8085352466157283], ["weather", 1.8084464233440314], ["smithing", 1.8083603981024212], ["flamboyantly", 1.8083603981024212], ["locksmithing", 1.8083153243138343], ["tzus", 1.8083153243138343], ["hogsmeade", 1.8081450658330591], ["excavate", 1.8079472381453312], ["mikl\u00f3s", 1.807892672607789], ["sportiva", 1.807892672607789], ["intelligence", 1.807891428273924], ["typographic", 1.8078660311006773], ["spouse's", 1.8078463223277583], ["decadence", 1.8077635128150233], ["marybeth", 1.807672903653706], ["reptilia", 1.807672903653706], ["fand", 1.8076438058347688], ["flippancy", 1.8076438058347688], ["misdirection", 1.8075403763147282], ["encompass", 1.8075239885255], ["rhododendrons", 1.8075239885255], ["photojournalistic", 1.807479817464467], ["wale's", 1.807479817464467], ["odysseus's", 1.807479817464467], ["riggings", 1.807479817464467], ["acrisius", 1.807479817464467], ["diversity's", 1.807479817464467], ["naturkunde", 1.807479817464467], ["overdubbing", 1.807479817464467], ["winona's", 1.807479817464467], ["tamilians", 1.807479817464467], ["j\u00f3zsef", 1.807479817464467], ["blythburgh", 1.807479817464467], ["cybil", 1.807479817464467], ["punctually", 1.8072769577767605], ["root", 1.8068663484567355], ["curriculum", 1.8068418467988674], ["jussi", 1.8068056991176271], ["weight", 1.8067220843669636], ["labrador's", 1.8066821652943592], ["kamiyama", 1.8066821652943592], ["maytals", 1.8066821652943592], ["grishin", 1.8066821652943592], ["portions", 1.8065825297683151], ["banksy", 1.8063625753038153], ["coddling", 1.8061507881869197], ["motherload", 1.805919852144446], ["dialectology", 1.805919852144446], ["estou", 1.805919852144446], ["rosings", 1.805919852144446], ["tourmalines", 1.805919852144446], ["semipermeable", 1.805919852144446], ["mench\u00fa", 1.805919852144446], ["olmecs", 1.805919852144446], ["finless", 1.805919852144446], ["escuelas", 1.805919852144446], ["speedwork", 1.805919852144446], ["entelechy", 1.805919852144446], ["pediatrician", 1.805745367134577], ["receding", 1.8057010445757997], ["topically", 1.805673197942013], ["dreamily", 1.8054992186385117], ["eyeshadows", 1.8054992186385117], ["\u00e9tait", 1.8054091643106829], ["interviewer's", 1.805190580431371], ["turo", 1.805190580431371], ["marduk", 1.8051258473242553], ["vosges", 1.8051258473242553], ["inky", 1.804775616275707], ["assembled", 1.8045664453902641], ["guayabera", 1.804492247518245], ["sonores", 1.804492247518245], ["kollege", 1.804492247518245], ["motovun", 1.804492247518245], ["burndown", 1.804492247518245], ["renseignements", 1.804492247518245], ["segeyaro", 1.804492247518245], ["flautists", 1.804492247518245], ["fisco", 1.804492247518245], ["hacemos", 1.804492247518245], ["daac", 1.804492247518245], ["tabs", 1.8044922475182448], ["inn's", 1.8044922475182448], ["oshun", 1.8044922475182448], ["lewises", 1.8044922475182448], ["ancestries", 1.8044922475182448], ["plutonian", 1.8044922475182448], ["beanery", 1.8044922475182448], ["violin", 1.804266685987305], ["karmic", 1.8042281239457159], ["halt", 1.8041741819046986], ["surfboards", 1.8039997550926121], ["hors", 1.8039917411626094], ["teide", 1.803987635592205], ["tomate", 1.803987635592205], ["content", 1.803914809999039], ["adzuki", 1.8036923839333519], ["gardel", 1.8036923839333519], ["eventos", 1.8036923839333519], ["spyridon", 1.8036923839333519], ["marginalizing", 1.8036923839333516], ["sorted", 1.8032576651743437], ["exaggerating", 1.8032552971872846], ["hopi", 1.8029885039786464], ["braised", 1.8029885039786464], ["distract", 1.8028434059178076], ["incongruities", 1.8025643711854262], ["mycologia", 1.8025643711854262], ["xinghai", 1.8025643711854262], ["deadheading", 1.8025643711854262], ["philosophus", 1.8025643711854262], ["colonizations", 1.8025643711854262], ["kody's", 1.8025643711854262], ["vladikavkaz", 1.8025643711854262], ["ambiances", 1.8025643711854262], ["medjool", 1.8025643711854262], ["millau", 1.8025643711854262], ["tub's", 1.8025643711854262], ["belger", 1.8025643711854262], ["eitri", 1.8025643711854262], ["rameau's", 1.8025643711854262], ["cordoba's", 1.8025643711854262], ["chutki", 1.8025643711854262], ["mwila", 1.8025643711854262], ["anteros", 1.8025643711854262], ["izrael", 1.8025643711854262], ["spire's", 1.8025643711854262], ["malaga's", 1.8025643711854262], ["denzel's", 1.8025643711854262], ["cr\u00e9ole", 1.8025643711854262], ["bigorre", 1.8025643711854262], ["facteur", 1.8025643711854262], ["xserve", 1.8025643711854262], ["derailed", 1.8022191468188518], ["stalks", 1.8018477330175715], ["animator", 1.801841700738694], ["gymnosperms", 1.801762304632438], ["pounce", 1.801712014285868], ["pete's", 1.801712014285868], ["hyperopia", 1.8013227646478969], ["n'dour", 1.8013227646478969], ["tj's", 1.8011505952080258], ["pandavas", 1.8011505952080258], ["scribe", 1.8009208566116985], ["actividades", 1.8008541583095385], ["instantiating", 1.8008541583095385], ["populaces", 1.8008541583095385], ["paramahansa", 1.8008541583095385], ["bottlecap", 1.8008541583095385], ["fabian's", 1.8008541583095385], ["armoires", 1.8008541583095385], ["bioblitz", 1.8008541583095385], ["nuggets", 1.800816165605777], ["kottayam", 1.8006080391133776], ["vesalius", 1.8006080391133776], ["cinderella's", 1.8004437072193253], ["mudslides", 1.8000694724017785], ["ladino", 1.8000694724017785], ["sacs", 1.7999810168994492], ["unused", 1.7998468155309266], ["bantering", 1.7996190045658422], ["smoothen", 1.7996190045658422], ["brik", 1.7996190045658422], ["zooming", 1.79908210009718], ["unsalted", 1.7990406093082199], ["giuseppina", 1.7990406093082199], ["goldbach", 1.7990406093082199], ["funniness", 1.7990406093082199], ["dactylic", 1.7990406093082199], ["diagnostics", 1.7988532092447502], ["keven", 1.7988532092447502], ["reread", 1.7988532092447502], ["moringa", 1.7988532092447502], ["scotswoman", 1.79885320924475], ["obis", 1.79885320924475], ["elfman's", 1.79885320924475], ["demille's", 1.79885320924475], ["cousteau's", 1.79885320924475], ["cler", 1.79885320924475], ["greenest", 1.7986226166015964], ["peque\u00f1os", 1.7986226166015964], ["impartially", 1.7986226166015964], ["exxonmobil's", 1.7986226166015964], ["b\u00e1lint", 1.7986226166015964], ["murmuration", 1.7986226166015964], ["gump's", 1.7986226166015964], ["correctness", 1.798600027934512], ["rabindranath", 1.7985564177566717], ["equalizing", 1.7985564177566717], ["wrangle", 1.798541503611542], ["valide", 1.7983959899252775], ["catfishing", 1.7983723610198787], ["udinese", 1.7981562494581385], ["inconsistency", 1.7981234513505333], ["sivaraman", 1.797954232128686], ["adriano's", 1.797954232128686], ["dress's", 1.797954232128686], ["bizim", 1.797954232128686], ["shlokas", 1.797954232128686], ["europan", 1.797954232128686], ["microphotography", 1.797954232128686], ["kanaks", 1.797954232128686], ["zworykin", 1.797954232128686], ["jigoro", 1.797954232128686], ["kapua", 1.797954232128686], ["ngurah", 1.7979542321286859], ["portrayed", 1.797626638772832], ["epipens", 1.7975873026935576], ["tiergarten", 1.7975873026935576], ["gama", 1.797519243201558], ["suburban", 1.7974035671793624], ["reduce", 1.7972129526743221], ["gbif", 1.7971140665996541], ["lifelessness", 1.7971140665996541], ["gloucestershire's", 1.7971140665996541], ["togaf", 1.7971140665996541], ["notion", 1.797092914864781], ["malm\u00f6", 1.7970886961744885], ["waterwheel", 1.7968220191361617], ["concrete", 1.7967780431601044], ["verbalize", 1.7967142636927353], ["sitios", 1.796714263692735], ["chebbi", 1.796714263692735], ["calphalon", 1.796714263692735], ["perdus", 1.796714263692735], ["recession's", 1.796714263692735], ["gesti\u00f3n", 1.796714263692735], ["icrc's", 1.796714263692735], ["rishi's", 1.796714263692735], ["untampered", 1.796714263692735], ["hanashi", 1.796714263692735], ["hydropneumatic", 1.796714263692735], ["hanussen", 1.796714263692735], ["trainset", 1.796714263692735], ["parenthood", 1.7965558232814747], ["philosopher", 1.796372032404413], ["tinkling", 1.7962686500360654], ["multithreaded", 1.7961381167426973], ["catacombs", 1.7961317974304618], ["reciprocate", 1.7960757724831784], ["planet", 1.7960301583512555], ["jahren", 1.7958994272919675], ["seuss's", 1.7958994272919675], ["newsweek's", 1.7958994272919675], ["dentro", 1.7955885028758851], ["bilbo's", 1.7955885028758851], ["bleaker", 1.7955885028758851], ["claddagh", 1.7955885028758851], ["console", 1.7955449734576334], ["interns", 1.7955302045478694], ["compresses", 1.7955120460522582], ["carb", 1.7953749239545573], ["kili", 1.795093850395754], ["ridgemont", 1.795063630469315], ["heavyset", 1.795063630469315], ["neurochemistry", 1.795063630469315], ["pali", 1.7950554895911723], ["maelstrom", 1.7950554895911723], ["clinic", 1.7949476531071817], ["tammy's", 1.7946514459387826], ["emerita", 1.7946514459387826], ["gigi's", 1.7946514459387826], ["combed", 1.794552697094929], ["aria", 1.7945491269706952], ["sisterhood", 1.7942282085555243], ["blinding", 1.7941421139782865], ["dentist's", 1.7941421139782863], ["birthplace", 1.7940817153210242], ["choreographed", 1.7940817153210242], ["ningbo", 1.794081715321024], ["infallibility", 1.7940107637385152], ["ballrooms", 1.7938237967305526], ["where", 1.7938176608472851], ["demanda", 1.7936218122922312], ["gelatins", 1.7936218122922312], ["lothl\u00f3rien", 1.7936218122922312], ["hulme's", 1.7936218122922312], ["avions", 1.7936218122922312], ["shikara", 1.7936218122922312], ["microbursts", 1.7936218122922312], ["aadil", 1.7936218122922312], ["incendies", 1.7936218122922312], ["wallets", 1.793551549329499], ["engagements", 1.793512927965878], ["blubber", 1.7931809023088754], ["misma", 1.7931809023088752], ["peirce's", 1.7931809023088752], ["almagest", 1.7931809023088752], ["shek's", 1.7931809023088752], ["bragi", 1.7931809023088752], ["hamlet", 1.7931637097786515], ["intensities", 1.7927864594854417], ["arya's", 1.7927578326138631], ["surgeries", 1.7927268466760156], ["godavari", 1.7927184313428166], ["arrivals", 1.792691346230667], ["lightsabers", 1.79268274328056], ["bayous", 1.79268274328056], ["coded", 1.7926502671439064], ["outpacing", 1.7926205879950983], ["high's", 1.7925236764887873], ["beads", 1.7925236764887873], ["cartloads", 1.792485169825633], ["dolomiti", 1.792485169825633], ["rajma", 1.792485169825633], ["tsarist", 1.7919610513549238], ["gentrifying", 1.7919610513549238], ["absurdist", 1.7919610513549238], ["ebrahim", 1.7919610513549236], ["marie", 1.791809868536695], ["picante", 1.791659425189067], ["ethereum's", 1.791659425189067], ["compliances", 1.791659425189067], ["specialist's", 1.791659425189067], ["drummed", 1.7915446067694356], ["indulgently", 1.7914029818901605], ["demystified", 1.7914029818901605], ["aged", 1.7913547159456287], ["cooler", 1.7912355610507311], ["protect", 1.7907864023299507], ["intersects", 1.7907624152001713], ["pharaonic", 1.7907624152001713], ["iterators", 1.7907624152001713], ["alders", 1.7905399878724853], ["flailed", 1.7904090813080846], ["versions", 1.7903241638560898], ["watchfulness", 1.7903228474422315], ["bennett's", 1.7902441436955365], ["nanoparticle", 1.790216201256233], ["sana", 1.790157496019268], ["wider", 1.7900835324814883], ["demographic", 1.7899066581377638], ["ademola", 1.7899036907907961], ["negations", 1.7899036907907961], ["baju", 1.7899036907907961], ["aviv's", 1.7899036907907961], ["toothsome", 1.7895427503409493], ["baijiu", 1.7895427503409493], ["gnocchi", 1.7895213494470203], ["ostracize", 1.7895035558350503], ["renderings", 1.789310221397298], ["uzbekistan's", 1.7893102213972978], ["racecars", 1.7893102213972978], ["thermosphere", 1.7891211764716353], ["silk", 1.7888515661674205], ["buzzers", 1.7885149724100107], ["grug", 1.7884913151174824], ["heavier", 1.788339748056879], ["charts", 1.7882440393067551], ["zoologists", 1.7881584814846747], ["railroaders", 1.7880366909906096], ["quest's", 1.7880366909906096], ["m\u00e1rquez", 1.7877681035892496], ["gerardus", 1.7877475050467329], ["newsted", 1.7877475050467329], ["goofily", 1.7877475050467329], ["maven's", 1.7877475050467329], ["sinuously", 1.7877475050467329], ["soulfire", 1.7877475050467329], ["engadin", 1.7877475050467329], ["holyfield's", 1.7877475050467329], ["wina", 1.7875913835285204], ["anacondas", 1.787288401938092], ["dharamshala", 1.787288401938092], ["trotted", 1.7871244493269751], ["jennie's", 1.7870689142306209], ["snafus", 1.7870689142306209], ["taskrabbit", 1.7870689142306209], ["jeanine", 1.7867959681588006], ["peloponnesian", 1.7866847582335252], ["smolensk", 1.7866847582335252], ["indicating", 1.7866490467373988], ["rattlesnakes", 1.7866079190133106], ["embolden", 1.7864473250430626], ["breaching", 1.7861875159498335], ["mishmash", 1.786187515949833], ["software's", 1.786187515949833], ["confidences", 1.786148388074347], ["pacific's", 1.786148388074347], ["p\u00e2t\u00e9", 1.7860953850657095], ["wayfinder", 1.7860611496593524], ["calliope", 1.7860611496593524], ["motorcity", 1.7860611496593524], ["hooved", 1.7860611496593524], ["geotech", 1.7860611496593524], ["novos", 1.7860611496593524], ["kongu", 1.7860611496593524], ["nwobhm", 1.7860611496593524], ["soul", 1.785986425320399], ["abbaye", 1.785986425320399], ["merapi", 1.785986425320399], ["ramji", 1.785986425320399], ["neuromarketing", 1.7859370573124016], ["workshopping", 1.7859370573124016], ["typographers", 1.7859370573124016], ["croisette", 1.7859370573124016], ["viro", 1.7859370573124016], ["rewatch", 1.7857558931261408], ["gunfighting", 1.785695453273263], ["donorschoose", 1.785695453273263], ["semble", 1.785695453273263], ["mineral's", 1.785695453273263], ["catharism", 1.785695453273263], ["cowpokes", 1.785695453273263], ["alesmith", 1.785695453273263], ["nga's", 1.785695453273263], ["ummi", 1.785695453273263], ["pleasing", 1.78560849301519], ["waking", 1.7855562177603372], ["dismounting", 1.7855197822990108], ["germline", 1.785441442540223], ["fran's", 1.7853486489514], ["assessments", 1.7852811317145405], ["garageband", 1.7852369948770441], ["meets", 1.7851471450393919], ["grown", 1.7850954017646734], ["estrellas", 1.7850193095953857], ["rimworld", 1.7850193095953857], ["starkness", 1.7850193095953857], ["kermit's", 1.7850193095953857], ["shakeup", 1.7847870580123533], ["postmodernism", 1.7847061483130007], ["vide", 1.7846811601280113], ["documents", 1.7846020562258778], ["unobstructed", 1.7845600764396736], ["regretting", 1.7844535222249336], ["gouges", 1.7843030981440053], ["bandages", 1.784289779964508], ["calamari", 1.7841795827696356], ["busily", 1.7841795827696356], ["vira", 1.784084299480836], ["merrily", 1.7837940434507937], ["leonardo", 1.7837019727980123], ["andes", 1.7835705982716568], ["showrunners", 1.7833458539926403], ["edgier", 1.7833458539926403], ["scuttling", 1.7833458539926403], ["rehome", 1.7833458539926403], ["squawk", 1.7830510373528663], ["janmashtami", 1.782881561919224], ["lombardi's", 1.782881561919224], ["potbellied", 1.782881561919224], ["stampeders", 1.7828323654081337], ["lugged", 1.7827715075018216], ["marathon", 1.7825972230494351], ["kraftwerk", 1.7824546266793007], ["checkpointing", 1.7824546266793007], ["ippudo", 1.7824546266793007], ["midge's", 1.7824546266793007], ["gazelle's", 1.7824546266793007], ["falar", 1.7824546266793007], ["miniaturists", 1.7824546266793007], ["concept2", 1.7824546266793007], ["meridionalis", 1.7824546266793007], ["embera", 1.7824546266793007], ["therme", 1.7824546266793007], ["ccne", 1.7824546266793007], ["infrasonic", 1.7822537867213648], ["acuerdo", 1.7822537867213648], ["rebreathers", 1.7822537867213648], ["aya's", 1.7822537867213648], ["janissaries", 1.7822145654501185], ["drummer's", 1.7822145654501185], ["ilyich", 1.7821151115123144], ["doppelgangers", 1.7821151115123144], ["unfurls", 1.78206071405462], ["scavenging", 1.7820316713441562], ["forehead", 1.782027414882137], ["hakim", 1.78200128561818], ["airbrushed", 1.7819360944242666], ["unsurprised", 1.7819360944242666], ["style's", 1.7819360944242666], ["parece", 1.7819360944242666], ["preppers", 1.781825525046355], ["dartboard", 1.781766862836209], ["ruler's", 1.781766862836209], ["universelle", 1.781766862836209], ["seagull", 1.7817212739186101], ["wireshark", 1.7816091936547889], ["green", 1.781549298169359], ["singular", 1.781484971362387], ["chroma", 1.7814143328758294], ["truth", 1.7813742045391578], ["conspires", 1.7813577315244211], ["avoir", 1.7813577315244211], ["paleomagnetism", 1.7813577315244211], ["sarabeth", 1.7813577315244211], ["culturel", 1.7813577315244211], ["mycket", 1.7813577315244211], ["santino's", 1.7813577315244211], ["barbera's", 1.7813577315244211], ["biocontrol", 1.7811974798393253], ["cybernetic", 1.7809852495631622], ["motives", 1.7809490002013457], ["evaporated", 1.7808082867695676], ["seishin", 1.780748928471952], ["naped", 1.780748928471952], ["dehydrators", 1.780748928471952], ["scale's", 1.780748928471952], ["overhyping", 1.780748928471952], ["redragon", 1.780748928471952], ["mayhem's", 1.780748928471952], ["isu's", 1.780748928471952], ["tinariwen", 1.780748928471952], ["manthan", 1.780748928471952], ["kimber's", 1.780748928471952], ["northrop's", 1.780748928471952], ["assisting", 1.7806565719635064], ["florist", 1.780609261889327], ["bochum", 1.7806092618893268], ["anastasia", 1.780474121538546], ["recommender", 1.7804081804948237], ["gracie's", 1.7804081804948237], ["pair", 1.7803743493481408], ["irritants", 1.780175205149272], ["filters", 1.7801381401921381], ["deoxyribonucleic", 1.7801342715439787], ["rugs", 1.7800442880357394], ["oise", 1.7799746898073994], ["positano", 1.7799746898073991], ["sorceress", 1.7799746898073991], ["whimpered", 1.7799746898073991], ["misappropriation", 1.7798391656190478], ["convair", 1.7796922886191064], ["enjoyably", 1.7796922886191064], ["suzdal", 1.7793300050229706], ["horia", 1.7793300050229706], ["yasaka", 1.7793300050229706], ["toolmaking", 1.7793300050229706], ["skin's", 1.7792649710315587], ["lexicon", 1.7791613295595312], ["javert", 1.7791384320250136], ["grammarian", 1.7791384320250136], ["coltrane", 1.7788698017317894], ["agates", 1.7788698017317894], ["kehinde", 1.7788698017317894], ["spectrogram", 1.7788698017317894], ["loki's", 1.7786749638564627], ["sonic", 1.7783711090057708], ["collapsing", 1.7783455231154297], ["rhapsody", 1.7781767355752705], ["watermarks", 1.7781767355752705], ["popolo", 1.7781767355752705], ["manchukuo", 1.7781767355752705], ["princip", 1.7781767355752705], ["oven", 1.7780290464776312], ["changin", 1.7778075128311726], ["doodled", 1.7777111445612188], ["samples", 1.7776669047176668], ["access", 1.7776635159609744], ["vacationers", 1.7773327978744755], ["squared", 1.7773327978744753], ["maybe", 1.7772924863821713], ["shutdowns", 1.7771514558891806], ["kawakubo", 1.7771514558891803], ["crusader's", 1.7771514558891803], ["floria", 1.7767221092869652], ["gopis", 1.7767221092869652], ["alban's", 1.7767221092869652], ["squeaks", 1.7766451449330867], ["watchmakers", 1.7766451449330867], ["leashes", 1.7764067650665991], ["hoon's", 1.7762339796600173], ["vervet", 1.7762339796600173], ["kyiv's", 1.7762339796600173], ["datos", 1.77605607160917], ["neuen", 1.775744209261214], ["leonidas", 1.7754528887445602], ["decay", 1.7753627776202525], ["tracks", 1.7753612157564476], ["radii", 1.7752645561852027], ["ceramics", 1.7751692484960735], ["stomachache", 1.7751509101602245], ["meringues", 1.7751509101602245], ["layups", 1.7749957396261193], ["geralt", 1.7749957396261193], ["intimacy", 1.7748647146213492], ["counterbalancing", 1.7748302542877248], ["musher", 1.7748302542877248], ["lorene", 1.7748302542877248], ["sticklers", 1.7746608979472618], ["rappelling", 1.7746165480560223], ["doughs", 1.7745530236950524], ["sidekicks", 1.7745125503680208], ["medley", 1.7744961067545253], ["constituents", 1.7744803827405031], ["zack", 1.774417376726274], ["longsword", 1.7743993028856537], ["umbrian", 1.7743993028856537], ["derived", 1.7741050607603577], ["topiary", 1.7740414408413745], ["mouthguard", 1.7740414408413745], ["caddo", 1.7740414408413745], ["deco", 1.7739931689184105], ["emitted", 1.7736259327580643], ["menschen", 1.7735819636487191], ["hybrid", 1.7735605926165028], ["spatula", 1.7734931144678192], ["promises", 1.773344481054763], ["dancing", 1.7733347765514047], ["alcove", 1.7731191808647502], ["slights", 1.7730784269992343], ["gantt", 1.7730784269992343], ["performing", 1.7729819851657482], ["unfavorably", 1.7728161683276276], ["showing", 1.7726521125710932], ["guaran\u00e1", 1.772269171669705], ["bocce", 1.772269171669705], ["accepta", 1.772269171669705], ["phototropic", 1.772269171669705], ["brilla", 1.772269171669705], ["oberkampf", 1.772269171669705], ["volvox", 1.772269171669705], ["fornarina", 1.772269171669705], ["torvald's", 1.772269171669705], ["nitpickers", 1.772269171669705], ["agentes", 1.772269171669705], ["yozakura", 1.772269171669705], ["praw", 1.772269171669705], ["d\u00f6blin", 1.772269171669705], ["heine's", 1.772269171669705], ["birthrights", 1.772269171669705], ["backpack's", 1.772269171669705], ["webmoney", 1.772269171669705], ["gravelotte", 1.772269171669705], ["longfin", 1.772269171669705], ["demme's", 1.772269171669705], ["harari's", 1.772269171669705], ["sentia", 1.772269171669705], ["oaklyn", 1.772269171669705], ["skyscraper's", 1.7722691716697048], ["barehanded", 1.7722691716697048], ["brayden's", 1.7722691716697048], ["hospitaller", 1.7722691716697048], ["wensum", 1.7722691716697048], ["mecca's", 1.7722691716697048], ["pubblico", 1.7722691716697048], ["freewheelin", 1.7722691716697048], ["keralan", 1.7722691716697048], ["savez", 1.7722691716697048], ["egyptological", 1.7722691716697048], ["changemaker", 1.7722691716697048], ["kelabit", 1.7722691716697048], ["instrumente", 1.7722691716697048], ["mintages", 1.7722691716697048], ["winterreise", 1.7722691716697048], ["vihuela", 1.7722691716697048], ["prochaine", 1.7722691716697048], ["pawnee's", 1.7722691716697048], ["thomasin", 1.7722691716697048], ["soudha", 1.7722691716697048], ["esdeath", 1.7722691716697048], ["renaissances", 1.7722691716697045], ["tardily", 1.7722691716697045], ["thredup", 1.7722691716697045], ["tightfisted", 1.7722691716697045], ["m\u00fcrren", 1.7722691716697045], ["greenbrook", 1.7722691716697045], ["anaheim's", 1.7722691716697045], ["ad\u00e8s", 1.7722691716697045], ["catl", 1.7722691716697045], ["marktplatz", 1.7722691716697045], ["terrible's", 1.7722691716697045], ["omul", 1.7722691716697045], ["lycosa", 1.7722691716697045], ["yalie", 1.7722691716697045], ["perdre", 1.7722691716697045], ["archytas", 1.7722691716697045], ["tzur", 1.7722691716697045], ["arcite", 1.7722691716697045], ["ugli", 1.7722691716697045], ["cafeteria's", 1.7722691716697045], ["realismo", 1.7722691716697045], ["innovaci\u00f3n", 1.7722691716697045], ["gazza's", 1.7722691716697045], ["potencia", 1.7722691716697045], ["m\u00e1ire", 1.7722691716697045], ["unadoptable", 1.7722691716697045], ["polita", 1.7722691716697045], ["neuesten", 1.7722691716697045], ["void's", 1.7722691716697045], ["sonobe", 1.7722691716697045], ["onewheel", 1.7722691716697045], ["abrahim", 1.7722691716697045], ["ebere", 1.7722691716697045], ["joggling", 1.7722691716697045], ["harte's", 1.7722691716697045], ["matrices", 1.771330221115178], ["flagella", 1.7712015396385785], ["pomerania", 1.7710469170685534], ["shillong", 1.7710469170685534], ["goblins", 1.770879156633101], ["gulliver's", 1.7707544116939187], ["riku", 1.7707544116939187], ["insistence", 1.7706580178772777], ["certainties", 1.7705086393799667], ["adulation", 1.7704879966228007], ["claude", 1.7704814914617597], ["workplaces", 1.7704397325247552], ["cathy's", 1.7702778579936713], ["streisand's", 1.7700892587894221], ["voltaire", 1.7700538352051174], ["cidade", 1.7697659666249737], ["dundee's", 1.7697659666249737], ["transgression", 1.7697569344418227], ["queues", 1.76975298704326], ["pausanias", 1.769662893476073], ["gaudi", 1.769521467527581], ["jerrie", 1.769460504202716], ["webex", 1.769460504202716], ["sacrifice", 1.7693971867556144], ["verbalizing", 1.7693967743088137], ["okra", 1.7693638123718853], ["then", 1.769304937873089], ["barbecue", 1.7692516390272937], ["katy's", 1.769186964414627], ["reveries", 1.769186964414627], ["panem", 1.769186964414627], ["reconciliations", 1.769186964414627], ["nigel's", 1.769186964414627], ["zoning", 1.7690565991530829], ["rewiring", 1.7688772689488057], ["distracted", 1.768860063645149], ["boycotts", 1.7685331628756713], ["yanni", 1.7684983861980668], ["firestorm", 1.768434935480996], ["arc's", 1.768382616468675], ["gnaws", 1.7682910141575392], ["ruby's", 1.7679147019604426], ["troop's", 1.7679147019604424], ["tatras", 1.7679147019604424], ["hafsa", 1.7679147019604424], ["jetbrains", 1.7679147019604424], ["jenkins's", 1.7679147019604424], ["bodhisattvas", 1.7678384987405307], ["barajas", 1.7678384987405307], ["slimy", 1.7678384987405305], ["cellars", 1.7678384987405305], ["podiatrist", 1.7675050072297325], ["song's", 1.7672303671620164], ["center's", 1.7672223124391222], ["infestation", 1.7670002633214974], ["went", 1.766990501592567], ["penny", 1.766937216554938], ["montblanc", 1.7668986590282814], ["akkadian", 1.766730830508237], ["troy's", 1.7667018339576535], ["stiffness", 1.7666607249239146], ["automotives", 1.7666607249239146], ["viagem", 1.7666607249239146], ["mastermind", 1.7666195799870894], ["beehive", 1.766559541606336], ["cappadocian", 1.7665336727322623], ["isle's", 1.7665336727322623], ["rufina", 1.7665336727322623], ["mesfin", 1.7665336727322623], ["waterline", 1.7664584530740663], ["empiricism", 1.7664007307039111], ["samaa", 1.766400730703911], ["arpeggiated", 1.766400730703911], ["paleobotany", 1.766400730703911], ["dishcloths", 1.766400730703911], ["janine's", 1.766400730703911], ["sharko", 1.766400730703911], ["thematic", 1.7663892132029502], ["catnip", 1.7663492746453382], ["pizza", 1.7663396465600292], ["parmi", 1.7662614795623497], ["healthwise", 1.7662614795623497], ["kindnesses", 1.7661649150508654], ["victrola", 1.7661649150508654], ["diplomat's", 1.7661649150508654], ["sparrow's", 1.7661649150508654], ["yeasty", 1.7661649150508654], ["gaulish", 1.7661649150508654], ["vasile", 1.7661649150508654], ["hrothgar", 1.7661649150508654], ["agrippa", 1.7661402869261782], ["farnsworth's", 1.766115459268074], ["vorenus", 1.766115459268074], ["cooperations", 1.7659621639413428], ["donating", 1.765845265417637], ["snacks", 1.7657822748396983], ["nicki's", 1.7655729405424894], ["grunts", 1.7654527517786673], ["elina", 1.7654527517786673], ["skewed", 1.7653430622631794], ["syllable", 1.7653157710918619], ["asteroid's", 1.7652641551808916], ["titanic's", 1.7652641551808916], ["typical", 1.7651755390211452], ["predictions", 1.7651678864697882], ["boiling", 1.765079182859372], ["smoothing", 1.765018979603783], ["ussr's", 1.7650132078143528], ["m\u00e9lange", 1.7650132078143528], ["halong", 1.7650132078143528], ["maoris", 1.7649255148810181], ["penguins", 1.7649225857871425], ["fretting", 1.76486185469833], ["principled", 1.76479784084796], ["tried", 1.7646300804125077], ["steer", 1.764471187314358], ["synthesizing", 1.7641700921217518], ["determinism", 1.7637547112364163], ["fixate", 1.7635815776909316], ["respekt", 1.7635815776909314], ["corporal's", 1.7635815776909314], ["hoppiness", 1.7635815776909314], ["tongatapu", 1.7635815776909314], ["kamogawa", 1.7635815776909314], ["marsten", 1.7635815776909314], ["group1", 1.7635815776909314], ["nmcc", 1.7635815776909314], ["commoner's", 1.7635815776909314], ["micky's", 1.7635815776909314], ["ezee", 1.7635815776909314], ["bucharest", 1.7631144037630666], ["downfield", 1.7629414391872325], ["toothy", 1.7629414391872325], ["karneval", 1.7629414391872322], ["alegr\u00eda", 1.7629414391872322], ["lennox's", 1.7629414391872322], ["hijri", 1.7629414391872322], ["comisi\u00f3n", 1.7629414391872322], ["riba", 1.7627348122018611], ["stockpiling", 1.762698258079682], ["venoms", 1.7624232318270954], ["apologies", 1.7624065437934637], ["pays", 1.7623171986287902], ["clicker", 1.7618184443674756], ["obergefell", 1.7617823718373395], ["gruppe", 1.7617823718373395], ["associate's", 1.7616355566396866], ["yunque", 1.7615605966142986], ["blogs", 1.7614684740241755], ["flamb\u00e9ed", 1.7613292385112498], ["mikel's", 1.7613292385112498], ["aengus", 1.7613292385112498], ["tomatometer", 1.7613292385112498], ["\u00e9conomiques", 1.7613292385112498], ["psyllids", 1.7613292385112498], ["sognefjord", 1.7613292385112498], ["echocardiograms", 1.7613292385112498], ["piccolos", 1.7613292385112498], ["bj\u00f6rkman", 1.7613292385112498], ["tripit", 1.7613292385112498], ["unproven", 1.7613292385112496], ["dariush", 1.7611693647490179], ["schoolbook", 1.7611693647490179], ["sizing", 1.7611644471130403], ["helps", 1.761049096659243], ["bikes", 1.760964104144234], ["dystopian", 1.7608351770137711], ["npca", 1.7608351770137711], ["moscatel", 1.7608351770137711], ["lateralus", 1.7608351770137711], ["corazones", 1.7608351770137711], ["optimates", 1.7608351770137711], ["waldos", 1.7608351770137711], ["roglic", 1.7608351770137711], ["sabina's", 1.7608351770137711], ["nakatsu", 1.7608351770137711], ["barcodes", 1.7606604216369337], ["shockwave", 1.7605955423952335], ["carson's", 1.7605955423952335], ["migrated", 1.760540919798361], ["unproductive", 1.7604540438585734], ["mudflats", 1.7604540438585732], ["headset's", 1.7602943799692337], ["scammer's", 1.7602943799692337], ["downstrokes", 1.7602943799692337], ["drawbars", 1.7602943799692337], ["dweller's", 1.7602943799692337], ["tombow", 1.7602943799692337], ["chick'n", 1.7602943799692337], ["indico", 1.7602943799692337], ["kiter", 1.7602943799692337], ["vezes", 1.7602943799692337], ["buccinum", 1.7602943799692337], ["nidaa", 1.7602943799692337], ["locali", 1.7602943799692337], ["canonica", 1.7602943799692337], ["sweated", 1.7602449265521816], ["dorothy's", 1.7602066066269264], ["mangas", 1.760151091863416], ["posts", 1.7601091055634162], ["hairballs", 1.7601025640838717], ["honolulu's", 1.7601025640838717], ["cousy", 1.7601025640838717], ["galway's", 1.7601025640838717], ["moisturizer", 1.7599340331156066], ["headscarves", 1.7599045030301486], ["drakos", 1.7596998867642457], ["jam\u00e1s", 1.7596998867642457], ["pulldowns", 1.7596998867642457], ["eynsford", 1.7596998867642457], ["alisdair", 1.7596998867642457], ["hlsl", 1.7596998867642457], ["pavarotti's", 1.7596998867642457], ["kaori's", 1.7596998867642457], ["sociais", 1.7596998867642457], ["orangetown", 1.7596998867642457], ["dor\u00e9e", 1.7596998867642457], ["reticuli", 1.7596998867642457], ["sanshin", 1.7596998867642457], ["zuza", 1.7596998867642457], ["bikeability", 1.7596998867642457], ["chaeronea", 1.7596998867642457], ["sensoria", 1.7596998867642457], ["rainy", 1.7593799413302886], ["extraversion", 1.7593799413302886], ["raz\u00f3n", 1.7593799413302886], ["namaste", 1.7593799413302886], ["luberon", 1.7593799413302886], ["archduke's", 1.7593799413302886], ["glassmakers", 1.7593799413302886], ["scams", 1.7590907598803658], ["parc", 1.759043282328886], ["nervous", 1.7590411702096758], ["ribbons", 1.7590351966323459], ["raga", 1.7589899213920641], ["multivitamins", 1.758716525062819], ["discogs", 1.758716525062819], ["nightingales", 1.758716525062819], ["idling", 1.7585950827506192], ["whippy", 1.7585660595176194], ["mountain's", 1.758503974219843], ["waterborne", 1.7581849133518], ["remains", 1.758155617648088], ["beaumont's", 1.7581193180036792], ["compressing", 1.7581048371592842], ["octavian", 1.758019771294471], ["selvedge", 1.7579188140043631], ["blustery", 1.7578507106120531], ["classicism", 1.7578507106120531], ["hypothermia", 1.757608515694636], ["sertanejo", 1.7575002619057907], ["crayolas", 1.7575002619057907], ["mir\u00f3", 1.7575002619057907], ["vegfest", 1.7575002619057907], ["bricklink", 1.7575002619057907], ["stardust's", 1.7575002619057907], ["chirac's", 1.7575002619057907], ["militant's", 1.7575002619057907], ["makefiles", 1.7575002619057907], ["alethiometer", 1.7575002619057907], ["mechanizing", 1.7575002619057907], ["sobie", 1.7575002619057907], ["kaurism\u00e4ki", 1.7575002619057907], ["valhalla's", 1.7575002619057907], ["precrime", 1.7575002619057907], ["pufnstuf", 1.7575002619057907], ["versi\u00f3n", 1.7575002619057907], ["generosa", 1.7575002619057907], ["curteis", 1.7575002619057907], ["n\u01b0\u1edbc", 1.7575002619057907], ["fray's", 1.7575002619057907], ["brex", 1.7575002619057907], ["documenter", 1.7575002619057907], ["ikumi", 1.7575002619057907], ["educa\u00e7\u00e3o", 1.7575002619057907], ["bazargan", 1.7575002619057907], ["dormancy", 1.7575002619057905], ["crackles", 1.7575002619057905], ["peonies", 1.7570782930553932], ["perusal", 1.7570056094256594], ["playgrounds", 1.7569959576699496], ["gyms", 1.756941438293102], ["social", 1.7569373586682486], ["gaping", 1.7568332788841943], ["plant", 1.7568074934581017], ["deciphers", 1.7565853736903267], ["pineview", 1.7565853736903267], ["graceville", 1.7565853736903267], ["headgear", 1.756357915294217], ["hijabs", 1.7563386622614379], ["pureed", 1.7563386622614379], ["recent", 1.7562897483564137], ["addison", 1.7561685568219934], ["wu's", 1.7560840650791139], ["peeta's", 1.7560840650791139], ["bereich", 1.7560840650791139], ["chitons", 1.7560840650791139], ["beide", 1.7560840650791139], ["lunaria", 1.7560840650791139], ["coisa", 1.7560840650791139], ["gifter", 1.7560840650791139], ["chalukyas", 1.7560840650791139], ["dilations", 1.7560840650791139], ["avalokiteshvara", 1.7560840650791139], ["microevolution", 1.7560840650791139], ["lenses", 1.7559907740148095], ["yvonne's", 1.7557675965144934], ["curtains", 1.7552833095867433], ["bus's", 1.755268987768796], ["rushd", 1.755268987768796], ["demarcations", 1.755268987768796], ["jean", 1.7552430049549737], ["wealthy", 1.755202875942515], ["slumbers", 1.755202875942515], ["outlets", 1.7551844968548087], ["grrrl", 1.7551733275699972], ["timeliness", 1.754849431948165], ["picard's", 1.7547796732650696], ["mekong", 1.754441611954684], ["rubble", 1.7544175876496138], ["revolutions", 1.7543987908553687], ["upending", 1.7543674628649604], ["anxiousness", 1.7543674628649604], ["inanna", 1.7543674628649604], ["volleys", 1.7541653467978098], ["ciders", 1.7541653467978098], ["lemur", 1.7541653467978098], ["tailcoat", 1.7540453241461087], ["greenspan's", 1.7540453241461087], ["delian", 1.7540453241461087], ["jinyoung", 1.7540453241461087], ["rostock", 1.753935352652432], ["louisianan", 1.7537113792962], ["garowe", 1.7537113792962], ["blearily", 1.7537113792962], ["taipa", 1.7537113792962], ["woodstock's", 1.7537113792962], ["redzepi", 1.7537113792962], ["samara's", 1.7537113792962], ["arogya", 1.7537113792962], ["weissbier", 1.7537113792962], ["meting", 1.7537113792961998], ["karoo", 1.7537113792961998], ["riotous", 1.7536712853003313], ["aglow", 1.7536712853003313], ["dander", 1.7536712853003313], ["izmir", 1.7536453261369045], ["desktops", 1.7533113490878247], ["vanish", 1.7530273920915764], ["ridiculed", 1.7528313807546176], ["tabriz", 1.7528313807546176], ["fungal", 1.7526352092776776], ["galatasaray's", 1.752504087970786], ["folgen", 1.752504087970786], ["folium", 1.752504087970786], ["dismal", 1.7525015539856965], ["fiction", 1.7524326461773434], ["staggeringly", 1.7523311434884208], ["purity", 1.7522678481608611], ["shalwar", 1.752243531311855], ["acidified", 1.752243531311855], ["exhale", 1.7522109314322027], ["monetary", 1.7520849949923554], ["maris", 1.7520590846243482], ["sonnet", 1.7520590846243482], ["overloading", 1.7519266037861916], ["interpretable", 1.7519216496069066], ["philco", 1.7519216496069066], ["rabaul", 1.7519120257789043], ["tuti", 1.75183955585795], ["pasadena's", 1.75183955585795], ["banjo", 1.7514974147114253], ["guffaw", 1.7514189461206495], ["cylinder's", 1.7514189461206493], ["traer", 1.7514189461206493], ["anderer", 1.7514189461206493], ["corso's", 1.7514189461206493], ["tagalong", 1.7514189461206493], ["zorro's", 1.7514189461206493], ["nigo", 1.7514189461206493], ["mighell", 1.7514189461206493], ["krapp's", 1.7514189461206493], ["later", 1.7512732307094094], ["simulate", 1.7512597262164566], ["hotbed", 1.751255689321048], ["rotations", 1.751230621502787], ["decimate", 1.7509570740357017], ["meltwater", 1.7509570740357017], ["eggy", 1.7509013873304244], ["rectification", 1.7508022493086353], ["specs", 1.7507670010278333], ["replanted", 1.750736929397082], ["tamper", 1.750572577428903], ["ralph's", 1.7503574800926975], ["anderson's", 1.7502634961214725], ["argonauts", 1.7501158070238334], ["hobbling", 1.7501158070238334], ["dahomey", 1.7499690959996888], ["copywriters", 1.749948401245653], ["subaru's", 1.749948401245653], ["jigs", 1.7498826137117716], ["practice", 1.7498445727853866], ["disentangled", 1.749799831933797], ["bafflement", 1.749799831933797], ["vivian's", 1.749799831933797], ["gorditas", 1.7495477720329136], ["journ\u00e9e", 1.7495477720329136], ["\u00f8resund", 1.7495477720329136], ["psychoacoustic", 1.7495477720329136], ["impersonality", 1.7495477720329136], ["lucinda's", 1.7495477720329136], ["intrusiveness", 1.7495477720329136], ["mawlid", 1.7495477720329136], ["vivah", 1.7495477720329136], ["laal", 1.7492899927825771], ["belugas", 1.7492296724379985], ["katniss", 1.749072996518115], ["kachina", 1.7489191167069946], ["celestron", 1.7489191167069946], ["grup", 1.748756479508249], ["mushing", 1.748756479508249], ["skittering", 1.748638916047442], ["northwich", 1.748638916047442], ["dnssec", 1.748638916047442], ["confirmations", 1.7484803237278295], ["tonearm", 1.7484803237278295], ["ruthlessly", 1.7483859473159693], ["bean", 1.748385292885004], ["mammals", 1.7482144204374612], ["wissen", 1.7481976039730025], ["ayutthaya", 1.7481018647832998], ["ishiguro", 1.7481018647832995], ["sigmoid", 1.7481018647832995], ["toll", 1.748092466386177], ["rfid", 1.747790315707416], ["alaya", 1.7476115136290828], ["hart", 1.7475379609559503], ["biscotti", 1.7474955165818487], ["gossipy", 1.7474955165818487], ["carlotta", 1.7474955165818487], ["bureaucracies", 1.747307634040554], ["solaria", 1.7473076340405538], ["amaranth", 1.747162025071051], ["rano", 1.7469510406458517], ["jaiden", 1.7469510406458517], ["coherence", 1.74690987295313], ["lubin", 1.7468055341457147], ["wander", 1.7466318590795253], ["rob's", 1.746577800385058], ["bread", 1.7465207926597819], ["squeaked", 1.7465021376135137], ["busier", 1.7464452730460802], ["strong", 1.7461207279633586], ["portraying", 1.7460992156768251], ["euclid", 1.7460767957900558], ["canoa", 1.746013332089413], ["asymptomatically", 1.746013332089413], ["rototiller", 1.746013332089413], ["pranzo", 1.746013332089413], ["nanodegree", 1.746013332089413], ["dragonslayers", 1.746013332089413], ["ivana's", 1.746013332089413], ["anthropomorphization", 1.746013332089413], ["usery", 1.746013332089413], ["brooms", 1.7460133320894127], ["vermicelli", 1.7460133320894127], ["left", 1.745867886071594], ["plazas", 1.7458611873236332], ["country's", 1.7457929072199634], ["fynbos", 1.7455515459661413], ["ravena", 1.7455515459661413], ["kaiya", 1.7455515459661413], ["saccone", 1.7455515459661413], ["acela", 1.7454572326126743], ["petras", 1.7454572326126743], ["juego", 1.7454572326126743], ["harrises", 1.7453145074618004], ["chawal", 1.7453145074618004], ["skulked", 1.7453145074618004], ["dockless", 1.7453145074618004], ["hoffmann's", 1.7453145074618004], ["launched", 1.7452493301999863], ["displaced", 1.7450812705892567], ["compote", 1.7450034921055557], ["dharma", 1.7448275844954642], ["veterinarian", 1.7448275844954642], ["cubase", 1.7445774658623656], ["farmville", 1.744477804196221], ["eradicate", 1.7444581785142725], ["archie's", 1.7444081715910542], ["sprains", 1.7442461121176622], ["stavanger", 1.7442461121176622], ["implementations", 1.7440272048695489], ["avani", 1.743887636892076], ["defaced", 1.743590634164504], ["cecilia's", 1.7434304721307006], ["beacon", 1.7433650726335645], ["smu's", 1.7432155786915127], ["mahalo", 1.743185780134612], ["uncluttered", 1.7431052485915957], ["flavonoids", 1.7430267303371547], ["manometer", 1.74297546635285], ["skiable", 1.74297546635285], ["dangled", 1.74297546635285], ["kultura", 1.74297546635285], ["clacks", 1.74297546635285], ["encyclop\u00e9die", 1.74297546635285], ["dreadlocked", 1.74297546635285], ["locations", 1.7428975023767206], ["sports", 1.7427347433955533], ["fragmentation", 1.742661851390046], ["jerusalem's", 1.7426164189919626], ["kayla", 1.74259396693477], ["bart\u00f3k", 1.7425790953401867], ["reacting", 1.7424763493833626], ["bushwalk", 1.742399466529204], ["ronny's", 1.742399466529204], ["breslauer", 1.742399466529204], ["mern", 1.742399466529204], ["bartlet's", 1.742399466529204], ["malfoy's", 1.742399466529204], ["lacandon", 1.742399466529204], ["lagoon's", 1.742399466529204], ["jagjivan", 1.742399466529204], ["carloman", 1.742399466529204], ["impervious", 1.7423811013174357], ["painter", 1.742363431005572], ["acrobats", 1.7421843554839538], ["titular", 1.7421361315972348], ["impish", 1.7421028878966034], ["inventoried", 1.7421028878966034], ["accidents", 1.7420428153832277], ["pinged", 1.7419471622429075], ["reinvented", 1.7418788174526183], ["cryptos", 1.741876559936891], ["zealotry", 1.741876559936891], ["haystack", 1.7418168685651634], ["rivera", 1.7417861419169858], ["paleontologists", 1.7417532786475773], ["popsicle", 1.74170356114469], ["eternally", 1.741595282160037], ["helen's", 1.7415498393607634], ["backache", 1.7413078427670412], ["curr", 1.7413078427670412], ["misspellings", 1.7413078427670412], ["asserting", 1.7412544611654848], ["unknowns", 1.7411767300614642], ["rinko", 1.7411767300614642], ["rains", 1.7411264070345838], ["fruit", 1.7411258287256057], ["cringed", 1.7408256863658875], ["contingencies", 1.7407678554680612], ["pecking", 1.740746597470046], ["patty's", 1.7407209246347466], ["andras", 1.7406705740352837], ["osei", 1.7406705740352837], ["freya", 1.7405270372517399], ["seljuk", 1.7404439115723813], ["photovoltaic", 1.7403683265796503], ["aesthetic", 1.7403683265796501], ["disassemble", 1.7400884946051758], ["toolkit", 1.7399252592867327], ["philosophers", 1.7397988204189696], ["logged", 1.7395991538295295], ["renoir", 1.7395960528623862], ["enmities", 1.7395704784654666], ["trickster", 1.7395025216671745], ["grand", 1.739494181514657], ["nirvana", 1.7394746181939362], ["whirr", 1.739256314550367], ["pickpocket", 1.7391793358695422], ["numerals", 1.7391426450964393], ["misidentification", 1.7391426450964391], ["nor'easters", 1.7391426450964391], ["mosaico", 1.7391426450964391], ["uint", 1.7391426450964391], ["biospheres", 1.7391426450964391], ["osetra", 1.7391426450964391], ["midoriya's", 1.7391426450964391], ["zep's", 1.7391426450964391], ["notetaker", 1.7391426450964391], ["zaporozhian", 1.7391426450964391], ["hankyu", 1.7391426450964391], ["llegado", 1.7391426450964391], ["helsingin", 1.7391426450964391], ["kynance", 1.7391426450964391], ["pustak", 1.7391426450964391], ["aykroyd's", 1.7391426450964391], ["danzon", 1.7391426450964391], ["thoa", 1.7391426450964391], ["xbee", 1.7391426450964391], ["tsukiyomi", 1.7391426450964391], ["lysette", 1.7391426450964391], ["sundae", 1.738981333314865], ["croaks", 1.7389750976354474], ["blenders", 1.7389444522024111], ["away", 1.738828959927423], ["socio", 1.7387826120576297], ["nerv", 1.7387034676608089], ["cooing", 1.7386685293815431], ["thinning", 1.7386217424684396], ["bard", 1.7385232244283224], ["escape", 1.73847033579626], ["harnesses", 1.7384696766876124], ["roaster", 1.7384297279387946], ["mascarpone", 1.738187072214518], ["group", 1.7378342675140288], ["beatmaker", 1.737789226695586], ["assynt", 1.737789226695586], ["bacterias", 1.737789226695586], ["needlecraft", 1.737789226695586], ["tenho", 1.737789226695586], ["browncoats", 1.737789226695586], ["lauritsen", 1.737789226695586], ["larissa's", 1.737789226695586], ["flew", 1.7377649242791973], ["bernardin", 1.737678384162948], ["jayavarman", 1.7374334139956071], ["exoskeleton", 1.7373915632569827], ["nancy's", 1.7373788613549543], ["paragliders", 1.7373667153348529], ["bitter", 1.737359844961075], ["humorous", 1.737135493869519], ["lighter", 1.737126370777815], ["livestock", 1.7368237882363107], ["pierced", 1.7368237882363107], ["neutrogena", 1.7368237882363107], ["quenched", 1.7368237882363107], ["skewered", 1.7368237882363107], ["iterator", 1.7368237882363107], ["lomo", 1.7368237882363107], ["sergeyevich", 1.7368237882363105], ["batignolles", 1.7368237882363105], ["cailleach", 1.7368237882363105], ["hyperventilation", 1.7368237882363105], ["odenwald", 1.7368237882363105], ["drachma", 1.7368237882363105], ["furever", 1.7368237882363105], ["ienumerable", 1.7368237882363105], ["magpie's", 1.7368237882363105], ["brosque", 1.7368237882363105], ["ndugu", 1.7368237882363105], ["transparent", 1.736458909289202], ["robes", 1.7363174256158336], ["lute", 1.736154104878595], ["waterproof", 1.7361248651826944], ["cipher", 1.7358839485240614], ["emmanuelle", 1.735880374989034], ["heeded", 1.7358332984796885], ["carried", 1.7357917197713144], ["inhaling", 1.7356643598062464], ["humanos", 1.7352698152674146], ["jauntily", 1.7352698152674146], ["yitzchak", 1.7352698152674146], ["favorably", 1.7352534231474894], ["outbreaks", 1.7351622858878701], ["sledges", 1.7351473173982446], ["arequipa", 1.7351473173982446], ["proximity", 1.735025834004182], ["bienvenue", 1.7347828672160563], ["winfrey's", 1.7347210959987365], ["placemats", 1.7347210959987365], ["consisting", 1.7347057104457784], ["squidward", 1.7346280565191978], ["engineers", 1.7344101920881223], ["skywalker's", 1.734323861948817], ["oireachtas", 1.734323861948817], ["orl\u00e9ans", 1.7341558561499262], ["pugilists", 1.734155856149926], ["keflavik", 1.734155856149926], ["dobby's", 1.734155856149926], ["mastermind's", 1.734155856149926], ["hallward", 1.734155856149926], ["microplane", 1.734155856149926], ["viento", 1.734155856149926], ["granville's", 1.734155856149926], ["patryk", 1.734155856149926], ["mystery's", 1.734155856149926], ["giornata", 1.734155856149926], ["suncatcher", 1.734155856149926], ["aisyah", 1.734155856149926], ["restrictions", 1.7340263829751454], ["strove", 1.7336427922871598], ["twelve", 1.7336115503626595], ["asgard's", 1.7335733382271348], ["bart\u00f3k's", 1.7335733382271348], ["datapoints", 1.7335733382271348], ["m\u00e9canique", 1.7335733382271348], ["shibani", 1.7335733382271348], ["whale's", 1.7334249158522865], ["royales", 1.7333570740881346], ["objectifying", 1.7333455029461178], ["burrowed", 1.7333455029461178], ["velociraptor", 1.7333455029461178], ["olive", 1.7332936585854237], ["carbo", 1.7331749987652258], ["nonfunctional", 1.732885412299267], ["scraping", 1.7327657887310857], ["fallible", 1.7327226694919509], ["ridiculousness", 1.7327226694919509], ["jubilantly", 1.7326653912971974], ["feuille", 1.7326653912971974], ["recaptcha", 1.7326653912971974], ["mmorpg", 1.7326653912971972], ["aprons", 1.7326653912971972], ["goals", 1.7326049521946592], ["olimpico", 1.7325740782330974], ["pushkar", 1.7325458971322802], ["omar", 1.7324088828264572], ["interviewing", 1.7323612399623223], ["pharaoh's", 1.732312557617515], ["amazon's", 1.7322290163097596], ["burrowing", 1.7320977371118582], ["fiddling", 1.7319810779853648], ["aldridge", 1.7318614345556353], ["bands", 1.7318402279159746], ["concerned", 1.7317687215420288], ["mill's", 1.7317601620315402], ["pavo", 1.7317138130502605], ["baldwin's", 1.7315765864591617], ["troyes", 1.7313706523234809], ["smartphone", 1.731322048285997], ["kandinsky", 1.7311633087526506], ["abraj", 1.7310536095378513], ["khaja", 1.7310536095378513], ["mist's", 1.7310536095378513], ["pangnirtung", 1.7310536095378513], ["agr\u00edcola", 1.7310536095378513], ["t\u00e8ne", 1.7310536095378513], ["khushal", 1.7310536095378513], ["rescuer's", 1.7310536095378513], ["wolvie", 1.7310536095378513], ["automan", 1.7310536095378513], ["screenless", 1.7310536095378513], ["kandersteg", 1.7310536095378513], ["quintessential", 1.7310266460859893], ["susceptibility", 1.730916224330745], ["strikeforce", 1.730697425667576], ["birthplaces", 1.730655116533814], ["lochs", 1.7306208461354666], ["dream", 1.7305216600994138], ["roadie", 1.7304000294263753], ["impurities", 1.7303511530006475], ["glacier's", 1.7302943754985798], ["healthline", 1.7302943754985798], ["montu", 1.7302943754985798], ["prejudgment", 1.7302943754985798], ["geometers", 1.7302943754985798], ["fortunes", 1.730278717434461], ["matter's", 1.730139159664878], ["until", 1.7300356054065418], ["achieved", 1.7299583960081921], ["ong's", 1.7299041715501102], ["prado's", 1.7299041715501102], ["bewitchment", 1.7299041715501102], ["dieselgate", 1.7299041715501102], ["nnenna", 1.7299041715501102], ["asantehene", 1.7299041715501102], ["ludwig's", 1.7296665473507211], ["attr", 1.7296665473507211], ["probleme", 1.7296665473507211], ["paperless", 1.729444823655381], ["musique", 1.729444823655381], ["sunburned", 1.729444823655381], ["stabilizing", 1.729305070538318], ["basel's", 1.7291832112563406], ["finishing", 1.7290701106102553], ["sidewalks", 1.7289180777156272], ["divinities", 1.728819991977157], ["falcon's", 1.728819991977157], ["locate", 1.7287051269332543], ["adolescence", 1.7286976632133708], ["livestreamed", 1.7285797923037851], ["botanica", 1.7285797923037851], ["funding", 1.7285144606445475], ["kishan", 1.7284602707969705], ["refinishing", 1.7283411323719085], ["opeth", 1.7283411323719085], ["dominating", 1.728335551677261], ["graduated", 1.728193607922093], ["viacom's", 1.7281828738669756], ["ingesting", 1.7281730924379863], ["inconsolable", 1.7281348421028329], ["reverting", 1.728074046230082], ["democracy's", 1.7279624423779623], ["pagodas", 1.7279624423779623], ["bichon", 1.7279624423779623], ["hakone", 1.7279624423779623], ["obtaining", 1.7279312037909174], ["perilously", 1.7277601285455797], ["sanya", 1.7277601285455797], ["malcolm's", 1.7277601285455797], ["justicia", 1.7277304699732932], ["francisco's", 1.727708098352177], ["ryukyu", 1.7276342443832082], ["encodings", 1.7275737868578591], ["christine's", 1.7274860259339648], ["essaying", 1.7274015977033832], ["hardrada", 1.7274015977033832], ["rudraksha", 1.7274015977033832], ["p\u00e2t\u00e9s", 1.7274015977033832], ["l'\u00e2me", 1.7274015977033832], ["fleuri", 1.7274015977033832], ["umd's", 1.7274015977033832], ["uscgc", 1.7274015977033832], ["chuppah", 1.7274015977033832], ["cityview", 1.7274015977033832], ["zygo", 1.7274015977033832], ["submatrix", 1.7274015977033832], ["mohandas", 1.7274015977033828], ["taut", 1.7273629663358843], ["rubato", 1.727134982238388], ["coiffure", 1.727093682980085], ["steamed", 1.7266271710842382], ["cristobal", 1.72650817336401], ["wiggly", 1.7263851801701655], ["bocuse", 1.7263851801701655], ["barnwood", 1.7263851801701655], ["bequia", 1.7263851801701655], ["andon", 1.7263851801701655], ["sport's", 1.7260360628435385], ["laozi", 1.7260360628435385], ["atpl", 1.7260360628435385], ["shukriya", 1.7260360628435385], ["mesma", 1.7260360628435385], ["folkwang", 1.7260360628435385], ["stylophone", 1.7260360628435385], ["hosius", 1.7260360628435385], ["arora's", 1.7260360628435385], ["blakey's", 1.7260360628435385], ["brackens", 1.7260360628435385], ["bytedance", 1.7260360628435385], ["alimentaire", 1.7260360628435385], ["ambur", 1.7260360628435385], ["martynas", 1.7260360628435385], ["whisker", 1.7258874715846848], ["slamming", 1.7257471059133749], ["metronome", 1.7255186659422064], ["firefox's", 1.7255186659422062], ["letterboxd", 1.7255186659422062], ["brewery", 1.7254223191669256], ["impasses", 1.725321643943355], ["ploughman's", 1.725321643943355], ["thunderstrike", 1.725321643943355], ["pridefully", 1.725321643943355], ["whiteacre", 1.725321643943355], ["dachstein", 1.725321643943355], ["molas", 1.725321643943355], ["reshared", 1.725321643943355], ["imanol", 1.725321643943355], ["cn's", 1.725321643943355], ["factorized", 1.725321643943355], ["skaldic", 1.725321643943355], ["downspout", 1.725321643943355], ["delicata", 1.725321643943355], ["plock", 1.725321643943355], ["tenkara", 1.725321643943355], ["stuffie", 1.725321643943355], ["manh\u00e3", 1.725321643943355], ["saga", 1.7252717503157105], ["wind", 1.7252246450592987], ["multistep", 1.7250785724240618], ["carioca", 1.7250086604251793], ["thundercloud", 1.7250086604251793], ["gaston's", 1.7250086604251793], ["experiment's", 1.7250086604251793], ["epona", 1.7250086604251793], ["trianon", 1.7250086604251793], ["balletic", 1.7250086604251793], ["rufinus", 1.7250086604251793], ["unblinking", 1.7247204865761274], ["bethany's", 1.7244346461185658], ["dargah", 1.7244346461185658], ["huddle", 1.7244179040346228], ["yurts", 1.7244179040346228], ["thespians", 1.7244179040346228], ["sprawl", 1.7243798102243821], ["swamped", 1.7243798102243821], ["oversaw", 1.7242785120773003], ["fordham's", 1.7242076348108653], ["orthorexia", 1.7242076348108653], ["aspern", 1.7242076348108653], ["shortcoming", 1.7241511108259844], ["joel's", 1.7240805839203774], ["zhangjiajie", 1.7240805839203774], ["sinaiticus", 1.7240805839203774], ["noncancerous", 1.7240805839203774], ["wknd", 1.7240805839203774], ["cove", 1.724048850271186], ["galicia", 1.7240205625477358], ["collectives", 1.7237321516415645], ["sais", 1.7235020333888604], ["savings", 1.7234254332984875], ["zooplankton", 1.7233076498064561], ["novel", 1.7231976531295385], ["preinstalled", 1.7230394724566576], ["michaelson", 1.7230394724566576], ["cpap", 1.7230394724566573], ["nuevos", 1.7230394724566573], ["lani", 1.7230394724566573], ["spellchecker", 1.7230394724566573], ["electrotherapy", 1.7230394724566573], ["amway", 1.7230394724566573], ["kedi", 1.7230394724566573], ["simeone's", 1.7230394724566573], ["neowise", 1.7230394724566573], ["pullan", 1.7230394724566573], ["bizen", 1.7230394724566573], ["instant", 1.722945059334879], ["ammonites", 1.7227385048632151], ["orientations", 1.7222110880948993], ["mambazo", 1.7221535909798158], ["quotidien", 1.7221535909798158], ["demande", 1.7221535909798158], ["domes", 1.7219367271942851], ["arthur's", 1.7218402228750092], ["complicate", 1.7218276644760286], ["tasty", 1.7217646224423542], ["huguenots", 1.721729176279884], ["aphid", 1.7216329096219989], ["warlpiri", 1.7216329096219989], ["duvalle", 1.7216329096219989], ["harjo", 1.7216329096219989], ["compatibilism", 1.7216329096219989], ["measurements", 1.7215379666306594], ["mantra", 1.7215127286203034], ["agreeably", 1.7214366450404186], ["chronotype", 1.7213906308562203], ["speedier", 1.7213906308562203], ["texturizing", 1.7213906308562203], ["spritzes", 1.7213906308562203], ["katsushika", 1.7213906308562203], ["hasharon", 1.7213906308562203], ["superscalar", 1.7213906308562203], ["yerevan's", 1.7213906308562203], ["rowboats", 1.7210475193093087], ["quentin's", 1.7210475193093087], ["organizational", 1.7208576354915348], ["mona", 1.720735465784554], ["bastions", 1.7207266677956417], ["nephthys", 1.7207266677956417], ["packaged", 1.7207073827779245], ["surfing", 1.7205189118569495], ["garment", 1.7202400869898382], ["moldings", 1.720221117929219], ["jabbing", 1.720221117929219], ["document", 1.7202171774220258], ["sixths", 1.7202048599285265], ["boboli", 1.7199898450717783], ["vojislav", 1.7199898450717783], ["asha's", 1.7199898450717783], ["ecdc", 1.7199898450717783], ["feudalism", 1.7199534316104665], ["popular", 1.7197630466290321], ["granddad", 1.7197300855784525], ["ataturk", 1.7195497722592765], ["electronics", 1.7193755123913559], ["uncovers", 1.7192547614766946], ["delilah", 1.7192289043925704], ["doughnuts", 1.7192267893686681], ["complicit", 1.719215436466173], ["ayurveda", 1.7191446412167062], ["inga", 1.7191446412167062], ["surveyed", 1.719063227520219], ["initialization", 1.7190522736774188], ["d\u00e1il", 1.7188875701133883], ["zeffirelli's", 1.7188875701133883], ["asiento", 1.7188875701133883], ["rosellas", 1.7188875701133883], ["mathilde's", 1.7188875701133883], ["chinyere", 1.7188875701133883], ["paciencia", 1.7188875701133883], ["preassigned", 1.7188875701133883], ["tradizione", 1.7188875701133883], ["thorgrim", 1.7188875701133883], ["tinku", 1.7188875701133883], ["cultivation", 1.7188105605898316], ["politics", 1.7187590796619228], ["maritime", 1.7185958443434797], ["sicknesses", 1.7184447005301062], ["gwen's", 1.7184447005301062], ["snowmobiling", 1.718382609017585], ["juxtaposing", 1.718286260118846], ["moonwalk", 1.7182149619337788], ["barrage", 1.7180883712786597], ["tombstones", 1.717737812541406], ["ethology", 1.717737812541406], ["oceanography", 1.717737812541406], ["vignerons", 1.717737812541406], ["rca's", 1.717737812541406], ["tagout", 1.717737812541406], ["akkadians", 1.717737812541406], ["merimbula", 1.717737812541406], ["bellinis", 1.717737812541406], ["patrice's", 1.717737812541406], ["hypermobile", 1.717737812541406], ["ssbm", 1.717737812541406], ["parmesan", 1.7175443735985523], ["mozart", 1.7174838166671393], ["heretical", 1.7171986606856178], ["niente", 1.7171986606856178], ["dinoflagellates", 1.7171986606856178], ["appian", 1.717105826591244], ["requested", 1.7169975620822113], ["ecsc", 1.7169937199217218], ["festively", 1.7169937199217218], ["espacio", 1.7169937199217218], ["animator's", 1.7168857600550267], ["wear's", 1.7168857600550267], ["chimi", 1.7168857600550267], ["binaria", 1.7168857600550267], ["tiare", 1.7168857600550267], ["habesha", 1.7168857600550267], ["becket's", 1.7168857600550267], ["caprices", 1.7168857600550265], ["capitalism's", 1.7168857600550265], ["maniacally", 1.7165692834172284], ["fara", 1.7164305539321663], ["actress's", 1.7162290397592004], ["sizes", 1.7162020866825116], ["bhutanese", 1.716147314566831], ["dogfights", 1.716147314566831], ["burnside's", 1.7160877281873796], ["sarangi", 1.7160877281873796], ["directv's", 1.7160877281873796], ["moonwalker", 1.7160877281873796], ["appetites", 1.7160636041471162], ["sirene", 1.7157073895951396], ["curlicues", 1.7157073895951396], ["outra", 1.7157073895951396], ["estacado", 1.7157073895951396], ["raindrop", 1.715454801094647], ["arius", 1.715381519245739], ["bofors", 1.7152483374507663], ["madrasa", 1.7149310514098026], ["hermia", 1.7149310514098024], ["stuffed", 1.71478702024647], ["cheesesteaks", 1.7147556288390773], ["kurukshetra", 1.7147556288390773], ["shopper's", 1.7147556288390773], ["mutterings", 1.71474001531184], ["mismo", 1.71474001531184], ["tags", 1.7146771265752037], ["nbut", 1.714634401859308], ["swordsmiths", 1.714634401859308], ["appalachia's", 1.714634401859308], ["crankiest", 1.714634401859308], ["rohinton", 1.714634401859308], ["handwerker", 1.714634401859308], ["bodybuilder's", 1.714634401859308], ["bharatnatyam", 1.714634401859308], ["stojanovski", 1.714634401859308], ["abrir", 1.714634401859308], ["boldo", 1.714634401859308], ["llanwrtyd", 1.714634401859308], ["fatio", 1.714634401859308], ["gehazi", 1.714634401859308], ["nadella", 1.7146344018593078], ["motorbikes", 1.7145754189077473], ["animatronics", 1.7145210750647368], ["breakdowns", 1.7144131587106808], ["architect", 1.7143898143049248], ["sashayed", 1.7143809826453174], ["shortcrust", 1.7143809826453174], ["loaches", 1.7143809826453174], ["ruminated", 1.7143809826453174], ["raghavendra", 1.7143809826453174], ["snails", 1.7143296025958874], ["jarringly", 1.7142676351423325], ["preplanned", 1.7142676351423325], ["binos", 1.7142676351423325], ["joven", 1.7142676351423325], ["dobsonian", 1.7142676351423325], ["slacktivism", 1.7142676351423325], ["ostrava", 1.7142676351423325], ["raskolnikov", 1.7142676351423325], ["backblaze", 1.7142676351423325], ["parallelepiped", 1.7142676351423325], ["lucentio", 1.7142676351423325], ["winegrowers", 1.7141619857133208], ["swifties", 1.7141619857133208], ["robespierre", 1.713970843654254], ["circularity", 1.713970843654254], ["akademi", 1.713970843654254], ["park", 1.7136975089279227], ["concurrently", 1.7136479606067838], ["downplay", 1.7136295151901044], ["stragglers", 1.7136185372115706], ["raimundo", 1.7135199173602116], ["lows", 1.7135077647366845], ["soulmate", 1.713264536987913], ["selling", 1.713202677449815], ["baleen", 1.713193532614048], ["effervescence", 1.713193532614048], ["westmeath", 1.713193532614048], ["arabic", 1.7130555054199927], ["gulp", 1.7129158562847617], ["min's", 1.7128123994588502], ["writer's", 1.712648219022765], ["torontonians", 1.7125968089872232], ["osu's", 1.7125968089872232], ["estimating", 1.7124757912049606], ["viewfinders", 1.7123614813597428], ["thermohaline", 1.7123614813597428], ["olly's", 1.7123614813597428], ["fizzed", 1.7123614813597428], ["sisler", 1.7123614813597428], ["pugilism", 1.7123614813597428], ["soufriere", 1.7123614813597428], ["fatah's", 1.7123614813597428], ["endorsements", 1.7122846766285762], ["lovebirds", 1.7122170757701527], ["kati", 1.712012019832935], ["calorie", 1.711948236366602], ["fissures", 1.7118734066016308], ["leashed", 1.711819680542986], ["examined", 1.7117166416376564], ["dreamweaver", 1.7116979468151705], ["hydrothermal", 1.7115056572124578], ["vloggers", 1.7114141458955041], ["define", 1.7114038882544205], ["materialism", 1.7113474188935585], ["scouting", 1.7113053555429654], ["tossed", 1.711217336859161], ["serializer", 1.7111564416121285], ["harker's", 1.7111564416121285], ["setosa", 1.7111564416121285], ["erfurt", 1.7111564416121285], ["minton's", 1.7111564416121285], ["aften", 1.7111564416121285], ["patanjali's", 1.7111564416121285], ["pollina", 1.7111564416121285], ["atitl\u00e1n", 1.7111564416121285], ["emeriti", 1.7111564416121285], ["lecun", 1.7111564416121285], ["glocke", 1.7111564416121285], ["tridacna", 1.7111564416121285], ["weronika", 1.7111564416121285], ["xin's", 1.7111564416121285], ["navarrese", 1.7111564416121285], ["gobos", 1.7111564416121285], ["kalari", 1.7111564416121285], ["ensslin", 1.7111564416121285], ["meskel", 1.7111564416121285], ["cesareo", 1.7111564416121285], ["husserl's", 1.7111564416121285], ["sayulita", 1.7111564416121285], ["takotsubo", 1.7111564416121285], ["logical", 1.711023145005264], ["mahatma", 1.7109332472936576], ["issue", 1.7108444072021876], ["debussy", 1.7108114503940617], ["paula's", 1.7107657666254592], ["square's", 1.7107657666254592], ["deckard", 1.7107657666254592], ["linking", 1.7107061372853887], ["science", 1.710696244235786], ["greenspace", 1.7106051669853208], ["finca", 1.7105082762933363], ["elitist", 1.7103154886600545], ["next", 1.710303425002762], ["tokyo", 1.7102778639767846], ["stemware", 1.710220360407746], ["dhammapada", 1.710220360407746], ["f\u00e9in's", 1.710220360407746], ["scoreboard", 1.710020951998096], ["adama", 1.709756404523537], ["j\u00fcrgen", 1.7095631860799014], ["weka", 1.7095631860799014], ["medici", 1.7095308429925973], ["reactants", 1.709518971333074], ["damming", 1.709518971333074], ["flairs", 1.7093702657858114], ["flore", 1.709255156677004], ["metabolisms", 1.7091918532750308], ["hotspot", 1.7091620194082227], ["disrepair", 1.7090764297301042], ["deel", 1.7090764297301042], ["wang's", 1.7090111864862136], ["insulation", 1.7089900736812416], ["alliterative", 1.7088900092391326], ["underway", 1.7088645080516698], ["nazca", 1.7088497356246615], ["vigils", 1.7088260729173264], ["pedometers", 1.7087994768165196], ["sardou", 1.7087994768165196], ["ru's", 1.7087994768165196], ["ramil", 1.7087994768165196], ["polypharmacy", 1.7087994768165196], ["stove", 1.7087714964742093], ["desiderius", 1.7085948301725897], ["performativity", 1.7085948301725897], ["grader's", 1.7085948301725897], ["boiled", 1.7085932793013634], ["curls", 1.7085213908180112], ["undamaged", 1.7085124201690673], ["fuming", 1.7084197435994493], ["college's", 1.7081948246939536], ["dismounted", 1.7081468699811526], ["fairhair", 1.7080565205222518], ["pupusa", 1.7080565205222518], ["mathematischen", 1.7080565205222518], ["javac", 1.7080565205222518], ["dialyzer", 1.7080565205222518], ["fineries", 1.7080565205222518], ["monopoly's", 1.7080565205222518], ["pariksha", 1.7080565205222518], ["skywatcher", 1.7080565205222518], ["gouldian", 1.7080565205222518], ["temazcal", 1.7080565205222518], ["clothing's", 1.7080565205222518], ["otwell", 1.7080565205222518], ["cirno", 1.7080565205222518], ["epigraphy", 1.7079205179328318], ["ecos", 1.7079205179328318], ["breakup", 1.7078672621820277], ["sharpen", 1.7077887401047005], ["audaciously", 1.707692562175667], ["senegal's", 1.707692562175667], ["barbacoa", 1.707692562175667], ["plushy", 1.707609268225702], ["teresa's", 1.7075779775707345], ["jingling", 1.70755147719862], ["assamese", 1.7074765352860812], ["developed", 1.7073337065986283], ["sprinkling", 1.7073049640908151], ["cocktails", 1.7072117530815278], ["unsupportive", 1.7071344927109036], ["books", 1.7070848601499946], ["mercantilism", 1.7069088901613185], ["couch", 1.7068149602349303], ["flowerpots", 1.7067266727174228], ["condone", 1.7066473131511237], ["lighthouses", 1.7066181571365486], ["asad", 1.706442031007687], ["moldy", 1.706442031007687], ["subterfuge", 1.706442031007687], ["maffeo", 1.706385559339976], ["homebodies", 1.706385559339976], ["sinai's", 1.706385559339976], ["grail", 1.7062398376002048], ["rialto", 1.7062077243966693], ["stitched", 1.706182748942985], ["khalifa", 1.7061498987266426], ["humankind", 1.7060495018445268], ["dozens", 1.7059469208898874], ["pillars", 1.7058834535366574], ["strains", 1.7058681533711466], ["bikaner", 1.7058090777320911], ["toefl", 1.7058090777320911], ["nbcuniversal's", 1.705809077732091], ["piteously", 1.705809077732091], ["adjani", 1.705809077732091], ["fenghuang", 1.705809077732091], ["montargis", 1.705809077732091], ["athlone", 1.7058090777320907], ["convened", 1.7056855131484088], ["spoken", 1.7053691527603996], ["convincingly", 1.7053167807399603], ["partway", 1.7051912548136798], ["suzy", 1.7051491902745541], ["bounding", 1.704942745036442], ["wenn", 1.7047821005630772], ["janitor", 1.7046313039154748], ["netherfield", 1.7044606120579944], ["yokosuka", 1.7043597921945102], ["blobs", 1.7041767245476582], ["marched", 1.7041534882033957], ["decompose", 1.7041534882033957], ["sel\u00e7uk", 1.7041049727593314], ["mondai", 1.7041049727593314], ["stateswoman", 1.7041049727593314], ["cellaring", 1.7041049727593314], ["cdjs", 1.7041049727593314], ["collinearity", 1.7041049727593314], ["cdmx", 1.7041049727593314], ["mammillaria", 1.7041049727593314], ["neni", 1.7041049727593314], ["reliability", 1.7040197675106934], ["na\u00efve", 1.703953496761753], ["austen", 1.7039481951018378], ["whimpers", 1.7039407216776195], ["aviators", 1.703910217905302], ["nsaids", 1.7039039972342886], ["washi", 1.7035831415715006], ["decommissioned", 1.7034085583432192], ["fiber", 1.703381839518717], ["swamiji", 1.7033475927714385], ["gilded", 1.7033279008917532], ["pharmacogenomics", 1.7033250620166496], ["ihrem", 1.7033250620166496], ["spinny", 1.7031199409831699], ["ifad", 1.7031199409831699], ["erythritol", 1.7031199409831699], ["multidimensional", 1.7030443952722334], ["friedan", 1.7029895585953434], ["woolsthorpe", 1.70276841983952], ["fragment's", 1.70276841983952], ["rainier's", 1.70276841983952], ["gyrfalcons", 1.70276841983952], ["windchime", 1.70276841983952], ["reservedly", 1.70276841983952], ["pyrethrin", 1.70276841983952], ["achievability", 1.70276841983952], ["earthshattering", 1.70276841983952], ["lycanthropic", 1.70276841983952], ["tamga", 1.70276841983952], ["watchfully", 1.70276841983952], ["pastafarians", 1.70276841983952], ["alphanumerics", 1.70276841983952], ["pietri", 1.70276841983952], ["teamworking", 1.70276841983952], ["invertibility", 1.70276841983952], ["beaulieu's", 1.70276841983952], ["stockwell's", 1.70276841983952], ["sanguinaria", 1.70276841983952], ["fauset", 1.70276841983952], ["cahuita", 1.70276841983952], ["straker's", 1.70276841983952], ["mahuta", 1.70276841983952], ["kafala", 1.70276841983952], ["lightyear's", 1.70276841983952], ["theguardian", 1.70276841983952], ["zibo", 1.70276841983952], ["yoik", 1.70276841983952], ["geometrid", 1.70276841983952], ["elding", 1.70276841983952], ["gourmand", 1.7026006594040681], ["fonts", 1.702600659404068], ["swelling", 1.7023773874257786], ["timbered", 1.7022960846329074], ["chiding", 1.7022027160455535], ["usda's", 1.7022027160455535], ["restaurants", 1.702155289919265], ["workforce", 1.701927787898612], ["truthful", 1.7019174980870746], ["prepare", 1.7018776355554994], ["ohne", 1.7018328327956522], ["scarecrows", 1.7016179006369259], ["peepers", 1.7016179006369259], ["frolic", 1.7016004009413304], ["citywide", 1.7014881428321218], ["chico's", 1.7013784048029166], ["alden's", 1.7013784048029166], ["swarms", 1.7012534504547376], ["fatburger", 1.7008067050701197], ["reggae", 1.7006574817500337], ["bloopers", 1.700639959314721], ["ironing", 1.7003938108186556], ["antiseptics", 1.7003679004506487], ["grandad", 1.7002537304053371], ["storytellers", 1.7001455219008852], ["trimmings", 1.7001455219008852], ["catherine's", 1.700065612823902], ["exhausting", 1.7000425235177383], ["lochner", 1.699844710327433], ["puttering", 1.699844710327433], ["entries", 1.6998078573255833], ["treasure's", 1.6997385827935425], ["sehen", 1.6997385827935425], ["nemeses", 1.6997385827935425], ["embellished", 1.6995052747840784], ["ebro", 1.6994361920120458], ["harpa", 1.6994361920120453], ["overijssel", 1.6994361920120453], ["mpge", 1.6994361920120453], ["ratio's", 1.6994361920120453], ["avocadoes", 1.6994361920120453], ["fena", 1.6994361920120453], ["envers", 1.6994361920120453], ["myoelectric", 1.6994361920120453], ["thundersnow", 1.6994361920120453], ["sabio", 1.6994361920120453], ["namesake", 1.6994092168343946], ["roasts", 1.6993798259691928], ["anaerobic", 1.6993269035109515], ["hunting", 1.6989457463581532], ["firecrackers", 1.6988465917005313], ["primal", 1.6988344777622424], ["bogdan", 1.6987433762982758], ["consciousnesses", 1.698673257694644], ["mnemosyne", 1.698673257694644], ["browser's", 1.698673257694644], ["profuse", 1.6986518368465013], ["affect", 1.6984899249704268], ["gymnasts", 1.6984783279765479], ["blackouts", 1.6983874215660935], ["displacing", 1.6983874215660935], ["metformin", 1.6983830202874184], ["hitched", 1.6982494974379134], ["seeker", 1.6982062498652977], ["jellystone", 1.698143845097968], ["drupes", 1.698143845097968], ["elyssa", 1.698143845097968], ["spielen", 1.698143845097968], ["nestorianism", 1.698143845097968], ["sarsen", 1.698143845097968], ["korakuen", 1.698143845097968], ["keepass", 1.698143845097968], ["antwone", 1.698143845097968], ["alexi", 1.698110390727497], ["queso", 1.6980325279006492], ["groundwater", 1.6980272049146683], ["bias", 1.6977164976402612], ["grumpier", 1.6976473118099273], ["hirst's", 1.6976473118099273], ["topicals", 1.6976473118099273], ["wasteful", 1.6974717899621392], ["telepathically", 1.6971938803698075], ["duets", 1.6970123608143624], ["replacements", 1.6969346620420926], ["archivist", 1.6967160168077158], ["respond", 1.6967150888703817], ["borrowed", 1.696691592110309], ["concurrency", 1.6966402044467814], ["mmos", 1.6965311728804011], ["neater", 1.6965311728804011], ["riri's", 1.6965311728804011], ["ludicrousness", 1.6965311728804011], ["expediting", 1.6965311728804011], ["konzept", 1.6965311728804011], ["perahera", 1.6965311728804011], ["polpo", 1.6965311728804011], ["trench's", 1.6965311728804011], ["silloth", 1.6965311728804011], ["intaglios", 1.6965311728804011], ["muggs", 1.6965311728804011], ["starshot", 1.6965311728804011], ["glauco", 1.6965311728804011], ["raystown", 1.6965311728804011], ["graphviz", 1.6965311728804011], ["kobato", 1.6965311728804011], ["baburao", 1.6965311728804011], ["ugochukwu", 1.6965311728804011], ["kingkiller", 1.6965311728804011], ["aerosoles", 1.6965311728804011], ["ramna", 1.6965311728804011], ["stiva", 1.6965311728804011], ["panchos", 1.6965311728804011], ["outcropping", 1.6964672974597352], ["disabilities", 1.6964427080498528], ["mixdown", 1.696314778598146], ["loincloths", 1.696314778598146], ["candelabras", 1.696314778598146], ["supercalifragilisticexpialidocious", 1.696314778598146], ["mermaids", 1.6959731034156378], ["mixtec", 1.6957683441156164], ["sousaphone", 1.6957683441156164], ["atlases", 1.6956514214534162], ["seaworthy", 1.6956514214534162], ["prometheus", 1.695470840897351], ["dingo", 1.695323677383689], ["porthole", 1.69525433331966], ["cherries", 1.6948038578757547], ["hammerheads", 1.694732395409155], ["recouping", 1.694732395409155], ["extended", 1.694552937947264], ["undersides", 1.6944622324256688], ["crimson", 1.69443182211999], ["sectors", 1.6943044757159957], ["assimilating", 1.694266386120488], ["zeroed", 1.694266386120488], ["landmines", 1.6941368748541585], ["lateran", 1.6941368748541585], ["types", 1.6941216133782406], ["slumps", 1.6940965737652405], ["counterbalance", 1.6939670973577523], ["bookings", 1.693924936323932], ["cajun", 1.693904762031904], ["eventual", 1.6938536865321399], ["supercontinent", 1.6937923079795334], ["semantics", 1.6937829654957608], ["assessing", 1.6937415358268124], ["eurogamer", 1.6936728866652222], ["tari", 1.6936122814663863], ["budget", 1.6936002612824723], ["estas", 1.6935016529288291], ["trying", 1.6934410419906791], ["saul's", 1.693403193530403], ["lullaby", 1.6934031935304028], ["doted", 1.6932355464465962], ["undergone", 1.6930767228935164], ["credentials", 1.6930648512339932], ["handicraft", 1.6929730118634538], ["meio", 1.6929730118634536], ["confounders", 1.6928107097819791], ["kibbles", 1.6928107097819791], ["uiuc", 1.6927655078938675], ["bakewell", 1.6924583745349102], ["turtle's", 1.6924076596129836], ["upstart", 1.6923633939874867], ["slovene", 1.6922898449482], ["quintessentially", 1.6920683832049048], ["harpy", 1.6920683832049048], ["hustled", 1.6919695616032053], ["shortcut", 1.691963224828421], ["prearranged", 1.6917114820483545], ["omitting", 1.6917114820483545], ["vieille", 1.6917114820483545], ["fionn", 1.6917114820483545], ["fingersmith", 1.6917114820483545], ["teranga", 1.6917114820483545], ["reynard's", 1.6917114820483545], ["fishier", 1.6917114820483545], ["zarina", 1.6917114820483545], ["hexi", 1.6917114820483545], ["halophytic", 1.6917114820483545], ["lifeguarding", 1.6917114820483545], ["satsangs", 1.6917114820483545], ["magictcg", 1.6917114820483545], ["unspooled", 1.6917114820483545], ["nizari", 1.6917114820483545], ["jellie", 1.6917114820483545], ["mesclun", 1.6917114820483545], ["alphago", 1.6917114820483545], ["m\u00e9dica", 1.6917114820483545], ["activo", 1.6917114820483545], ["tre'davious", 1.6917114820483545], ["harmonizations", 1.6917114820483545], ["oeko", 1.6917114820483545], ["sarathi", 1.6917114820483545], ["leit\u00e3o", 1.6917114820483545], ["almod\u00f3var's", 1.6917114820483545], ["mavis's", 1.6917114820483545], ["linus's", 1.6917114820483545], ["friedrick", 1.6917114820483545], ["sharkfin", 1.6917114820483545], ["waalwijk", 1.6917114820483545], ["courr\u00e8ges", 1.6917114820483545], ["merezhkovsky", 1.6917114820483545], ["shivaji's", 1.6917114820483545], ["anania", 1.6917114820483545], ["hlynur", 1.6917114820483545], ["rheumatoid", 1.6917114820483543], ["regal", 1.691505489782656], ["haruki", 1.6912511523933755], ["portugal's", 1.6912179119587707], ["wager", 1.6911664448918087], ["importing", 1.6911664448918087], ["chosen", 1.6911628188649874], ["juxtaposed", 1.6909087719738358], ["begonia", 1.6908760689707996], ["wxyz", 1.6908178151270399], ["quants", 1.6908178151270399], ["prospects", 1.6906993469736247], ["diyarbakir", 1.6906057882692378], ["proceed", 1.6905687365439244], ["wonder", 1.6904803030692532], ["rebelling", 1.690392408768005], ["syrups", 1.6903485724958482], ["rothko", 1.690301722479981], ["sapporo", 1.6902618578135231], ["puttanesca", 1.690261857813523], ["sandhi", 1.690261857813523], ["snowmobiles", 1.6902340047714564], ["motorsports", 1.6900953810078445], ["stollen", 1.6899656497655389], ["ciso", 1.6899656497655389], ["habitability", 1.6897669860919773], ["septimus", 1.6897669860919773], ["galileo", 1.689712835571673], ["ajaccio", 1.689563276991785], ["franklin's", 1.689530638517169], ["bulgaria's", 1.6895173037318199], ["chais", 1.6895173037318199], ["ampullae", 1.6895173037318199], ["lorenzini", 1.6895173037318199], ["padano", 1.6895173037318199], ["orthognathic", 1.6895173037318199], ["estimations", 1.6894445319886582], ["thunderstorm", 1.6894305227467163], ["dravidian", 1.6891940542476873], ["nehisi", 1.6891521605929105], ["affability", 1.6891521605929105], ["tomaso", 1.6891521605929105], ["wolfing", 1.6891521605929105], ["plurals", 1.688993873241851], ["friesland", 1.688993873241851], ["roadside", 1.6888719663219849], ["fries", 1.6888134004588804], ["carries", 1.6888057187968362], ["molde", 1.6888010106813895], ["unsanitary", 1.6887850350401534], ["millinery", 1.6887591059889333], ["sunrise's", 1.6887591059889333], ["plague's", 1.6887591059889333], ["alpenglow", 1.6887591059889333], ["yasunari", 1.6887591059889333], ["amihan", 1.6887591059889333], ["orcid", 1.6887591059889333], ["govindan", 1.6887591059889333], ["krause's", 1.6887591059889333], ["michala", 1.6887591059889333], ["kosrae", 1.6887591059889333], ["archeologist", 1.6887097126252015], ["growling", 1.6886894894044668], ["bumble", 1.6886894894044668], ["scalpels", 1.6886894894044668], ["temporary", 1.688679098552939], ["outing", 1.688535061755057], ["d'artagnan", 1.6883747730107839], ["gurudwara", 1.6883747730107839], ["ribbit", 1.6883747730107839], ["doolin", 1.6883747730107839], ["blueprint", 1.6883504658588544], ["deceive", 1.688234075113033], ["angering", 1.6881424704828516], ["bandleaders", 1.6881424704828516], ["stroud's", 1.6881424704828516], ["paddlewheel", 1.6881424704828516], ["mcqs", 1.6881424704828516], ["sidekick", 1.6876280405399617], ["guitar", 1.687569963210404], ["tener", 1.6874502440583081], ["snowdrift", 1.6874502440583081], ["sportsmanlike", 1.6874502440583081], ["spillage", 1.6872002514295588], ["primavera", 1.6872002514295588], ["intellectuals", 1.6870396574593103], ["gotthard", 1.6870356625844285], ["humblebrag", 1.6870356625844285], ["multigrain", 1.6870356625844285], ["amandine", 1.6870356625844285], ["fernandes", 1.686985120489645], ["broach", 1.6868432475676398], ["antibiotics", 1.6867491283676792], ["haleem", 1.6866005712264562], ["makalu", 1.6866005712264562], ["theosis", 1.6866005712264562], ["tango's", 1.6866005712264562], ["ascenders", 1.6866005712264562], ["lifetime", 1.686491343760891], ["dance's", 1.6864609808626054], ["comics", 1.6863620160105257], ["ever", 1.686328853436399], ["semisweet", 1.6862366876080686], ["honorine", 1.6862366876080686], ["barnaby's", 1.6862366876080686], ["waitress's", 1.6862366876080686], ["ketubah", 1.6862366876080686], ["leakproof", 1.6862366876080686], ["aleksandrs", 1.6862366876080686], ["adheres", 1.6861035986824484], ["internalized", 1.6861035986824484], ["apparition", 1.6861035986824484], ["straighter", 1.6860606712523685], ["patina", 1.6860606712523685], ["talkative", 1.6860301358778456], ["esa's", 1.6859278530498985], ["microwaved", 1.6859278530498985], ["squiggly", 1.6857407356411251], ["lu\u00eds", 1.6857407356411251], ["society", 1.6855656923624447], ["watt's", 1.6854890949879466], ["expense", 1.6853758566940165], ["undesirable", 1.6853583297576924], ["kosova", 1.6851923626570313], ["splotches", 1.685132604062611], ["supervision", 1.685120734674741], ["unbaked", 1.685108392735129], ["rangefinders", 1.685108392735129], ["widening", 1.6849446361201612], ["scornfully", 1.684933791815789], ["manufacturer's", 1.6849196134463642], ["mojitos", 1.6848531922562664], ["christabel", 1.6848531922562664], ["pronation", 1.6847497064020651], ["leeloo", 1.684749706402065], ["burkinabe", 1.684749706402065], ["townshend's", 1.684749706402065], ["shenzi", 1.684749706402065], ["asterism", 1.684749706402065], ["danios", 1.684749706402065], ["plinius", 1.684749706402065], ["vessel's", 1.6846901114354431], ["diagnose", 1.684690111435443], ["dashboard", 1.6846495088834175], ["negating", 1.684404559215094], ["clarifications", 1.684404559215094], ["mixtures", 1.6843761477088492], ["fans", 1.6843480574916005], ["interwar", 1.6842425571827988], ["archivists", 1.6842425571827988], ["voluntourism", 1.6842425571827988], ["gryffindors", 1.6842425571827988], ["mahabali", 1.6842425571827988], ["deepest", 1.6839566076824282], ["enjoy", 1.6839121878118757], ["sequester", 1.6839110832838664], ["khoisan", 1.683832586257045], ["margo's", 1.683832586257045], ["hargeisa", 1.683832586257045], ["beaten", 1.6837541724846454], ["freer", 1.6837464126344115], ["lawnmowers", 1.6836557130862198], ["rhcp", 1.6836557130862195], ["slovenia's", 1.6836557130862195], ["berkeley's", 1.6836557130862195], ["overacting", 1.6836557130862195], ["untranslatable", 1.6835389628114061], ["lexicographical", 1.6834707163042082], ["chador", 1.6834707163042082], ["homophonic", 1.6834707163042082], ["sculptors", 1.6833996048244175], ["calculative", 1.68339431645323], ["niketan", 1.68339431645323], ["podcast's", 1.68339431645323], ["bisquick", 1.68339431645323], ["processus", 1.68339431645323], ["develop", 1.6832268365606908], ["fontina", 1.6832104193244934], ["lapwings", 1.6832104193244934], ["ptsd", 1.6831960467304552], ["blurriness", 1.6830740054806388], ["skip's", 1.6830740054806388], ["sweetheart", 1.6829145823417029], ["naptime", 1.68288516127245], ["turners", 1.682852569957064], ["warhammer", 1.6826890208107632], ["christmases", 1.6826890208107632], ["foxes", 1.6825775265179035], ["clicking", 1.6823477653580499], ["rinses", 1.6823130849258638], ["speedsters", 1.6823130849258638], ["chomped", 1.6821537900593808], ["reverso", 1.6821537900593808], ["sanitation", 1.6821311804654284], ["mausoleum", 1.6821296489315096], ["benito", 1.6821296489315096], ["downright", 1.682031133012189], ["parrot", 1.681820353426238], ["acts", 1.6816360562269712], ["product", 1.6816169073612348], ["retreating", 1.6814641974438322], ["napping", 1.6812926875239933], ["shady", 1.6812028649149027], ["basements", 1.6812004283395736], ["cher's", 1.6811712235932244], ["ringo's", 1.6811712235932244], ["tirthankara", 1.6811712235932244], ["dronacharya", 1.6811712235932244], ["broome's", 1.6811712235932244], ["arcadia's", 1.6811712235932244], ["anticipatory", 1.6811238999838343], ["ferdinand's", 1.6810872070403216], ["protested", 1.681013255195298], ["fyodorovich", 1.680797214422236], ["ceau\u0219escu", 1.680797214422236], ["depoe", 1.680797214422236], ["locomotive's", 1.680797214422236], ["sorenson's", 1.680797214422236], ["kaliya", 1.680797214422236], ["mullum", 1.680797214422236], ["deeds", 1.6807258677781107], ["fantine", 1.6807053223871136], ["forefront", 1.6806037691954288], ["sylhet", 1.6805133918598547], ["pails", 1.6805133918598547], ["deadpanned", 1.68047833807725], ["alimentos", 1.68047833807725], ["ungreased", 1.68047833807725], ["h\u00e5kon", 1.68047833807725], ["boyband", 1.68047833807725], ["feldberg", 1.68047833807725], ["polyphemus", 1.6804334055013652], ["congee", 1.6804334055013652], ["reshoots", 1.6804334055013652], ["penniless", 1.6804015465280688], ["discoloration", 1.6803635722100527], ["depilatory", 1.6803359005168093], ["fotis", 1.6803359005168093], ["sukhoi", 1.6802906450387454], ["horseback", 1.68011117474288], ["closures", 1.6800960271431222], ["raghu", 1.679963485645241], ["griddles", 1.679963485645241], ["gecko's", 1.679963485645241], ["marcel", 1.6798980537665402], ["opposition's", 1.6798629191390664], ["lars", 1.6794898214077925], ["nursery's", 1.679399171407537], ["viraat", 1.679399171407537], ["sant'agata", 1.679399171407537], ["goodweather", 1.679399171407537], ["dreariest", 1.679399171407537], ["hamer's", 1.679399171407537], ["phylum", 1.6793991714075367], ["edmund", 1.679297674139491], ["buildings", 1.6792503582880693], ["unreliable", 1.679002853019004], ["bonfire", 1.6788523667330961], ["hippolyta", 1.6786312386304552], ["shoutout", 1.678631238630455], ["lineups", 1.6786031913248802], ["llamas", 1.678570071115341], ["viserion", 1.6784431566989555], ["offenbach's", 1.6784431566989555], ["wakatobi", 1.6784431566989555], ["mystic's", 1.6784431566989555], ["ferret's", 1.6784431566989555], ["guajillo", 1.6784431566989555], ["e\u00e4rendil", 1.6784431566989555], ["cholent", 1.6784431566989555], ["antoinette", 1.6783667422597708], ["hyaluronic", 1.6781777901919674], ["uncharacteristic", 1.6780590735967362], ["externally", 1.677895838278293], ["insignia", 1.6778738420297084], ["winnie's", 1.6777481491806538], ["pessoa", 1.6777481491806538], ["thermopolis", 1.6777481491806538], ["boulder's", 1.6776640557442757], ["deviate", 1.6774912985760642], ["grandfatherly", 1.6774153286789317], ["chakri", 1.6774153286789317], ["handovers", 1.6774153286789317], ["workaholism", 1.6774153286789317], ["portlanders", 1.6774153286789317], ["weaves", 1.67724289700452], ["juxtaposition", 1.67724289700452], ["triangle", 1.6771997384303134], ["malnourished", 1.6770553445387968], ["nunca", 1.6768392931951823], ["reactant", 1.6768392931951823], ["macchiato", 1.6767770427852184], ["pablo", 1.6766621310934184], ["disappeared", 1.6765666363995408], ["amounts", 1.6765321564545665], ["mcclintock's", 1.6764708380659368], ["ligo", 1.6764708380659368], ["microcar", 1.6764708380659368], ["peut", 1.6764708380659368], ["darkwave", 1.6764708380659368], ["barra's", 1.6764708380659368], ["mediacityuk", 1.6764708380659368], ["sealskin", 1.6764708380659368], ["camote", 1.6764708380659368], ["barware", 1.6764708380659368], ["damavand", 1.6764708380659368], ["allegra's", 1.6764708380659368], ["visakha", 1.6764708380659368], ["lyngstad", 1.6764708380659368], ["conking", 1.6764708380659368], ["lampo", 1.6764708380659368], ["houndsditch", 1.6764708380659368], ["kibi", 1.6764708380659368], ["daikanyama", 1.6764708380659368], ["reza", 1.6763848652024465], ["hazara", 1.676364732316692], ["nearer", 1.6763451027530818], ["looms", 1.6762538830163047], ["par\u00e1", 1.676127650586292], ["thatcher's", 1.6760267398306876], ["yaki", 1.6757686513274799], ["cello", 1.6757486660126164], ["woman", 1.6755475816258305], ["hairstyling", 1.6755073490785426], ["yugoslavian", 1.6755073490785426], ["stocked", 1.6754710518206903], ["comedies", 1.6754146614379555], ["pompoms", 1.675392721771361], ["scripting", 1.675324533219396], ["maples", 1.674936310983039], ["manpreet", 1.6749363109830389], ["webassembly", 1.6749363109830389], ["noga", 1.6749363109830389], ["whales", 1.674899680336544], ["dreading", 1.6748578569116053], ["sedimentary", 1.6747943672278711], ["yelps", 1.6747943672278711], ["hotpot", 1.6747943672278711], ["reader", 1.674794367227871], ["diaphragms", 1.674794367227871], ["docg", 1.674794367227871], ["hollywoo", 1.674794367227871], ["unii", 1.674794367227871], ["mobike", 1.674794367227871], ["honkytonk", 1.674794367227871], ["lotte's", 1.674794367227871], ["paychex", 1.674794367227871], ["jomon", 1.6746234872801893], ["tipples", 1.6746234872801893], ["salim's", 1.6746234872801893], ["waterlilies", 1.6746234872801893], ["bobsledding", 1.6746234872801893], ["leblon", 1.6746234872801893], ["beppu", 1.6746234872801893], ["martti", 1.6742083942898696], ["disbanding", 1.6741921398394397], ["relapses", 1.6741921398394397], ["d\u00e9tente", 1.6741273842062747], ["imposed", 1.6739387890070647], ["squandered", 1.6738351493772252], ["khartoum's", 1.6738097732436101], ["arcilla", 1.6738097732436101], ["pel\u00edcula", 1.6738097732436101], ["malory's", 1.6738097732436101], ["sirloins", 1.6738097732436101], ["topknots", 1.6738097732436101], ["pnds", 1.6738097732436101], ["golshifteh", 1.6738097732436101], ["ossietzky", 1.6738097732436101], ["vinalhaven", 1.6738097732436101], ["eumolpus", 1.6738097732436101], ["secr\u00e9taire", 1.6738097732436101], ["mummifying", 1.6738097732436101], ["bextra", 1.6738097732436101], ["zabrze", 1.6738097732436101], ["snowsuits", 1.6738097732436101], ["parboiling", 1.6738097732436101], ["purrr", 1.6738097732436101], ["fanling", 1.6738097732436101], ["computational", 1.6737531059639403], ["excusing", 1.6735454527109226], ["shaka", 1.6734869555438523], ["abound", 1.6734604011846785], ["find", 1.6734437440086958], ["limbs", 1.6734409980422325], ["kilograms", 1.6733684035265275], ["prehensile", 1.673351822553256], ["parasols", 1.673351822553256], ["neil's", 1.6732627765856871], ["hallyu", 1.6731620140434385], ["ofelia", 1.672914687803373], ["coimbra", 1.6727934181630828], ["recreational", 1.672734349556004], ["tagalongs", 1.6727034878680358], ["willa's", 1.6727034878680358], ["natales", 1.6727034878680358], ["novecento", 1.6727034878680358], ["svgs", 1.6727034878680358], ["gaulois", 1.6727034878680358], ["racking", 1.6725947423694156], ["dolphins", 1.6725877870338948], ["choreograph", 1.6725790307632837], ["genetically", 1.6724869270590668], ["epics", 1.6724291617527645], ["pucks", 1.6724156578676652], ["longings", 1.6723618409138143], ["winged", 1.6723131903875335], ["when", 1.6721949633714233], ["wally's", 1.672097435879678], ["wole", 1.672097435879678], ["regrowth", 1.6720974358796779], ["cassoulet", 1.6718257047614722], ["akane", 1.6717148924260334], ["reentered", 1.6717148924260334], ["hopper's", 1.6717148924260334], ["compl\u00e8te", 1.6717148924260334], ["heraclitus", 1.6717148924260334], ["truffaut's", 1.6717148924260334], ["gurudwaras", 1.6717148924260334], ["caliban's", 1.6717148924260334], ["psychology's", 1.6717148924260334], ["stenotype", 1.6717148924260334], ["eixample", 1.6717148924260334], ["franc\u00e9s", 1.6717148924260334], ["nin's", 1.6717148924260334], ["minmax", 1.6717148924260334], ["slatina", 1.6717148924260334], ["unit\u00e9", 1.6717148924260334], ["apft", 1.6717148924260334], ["petrifaction", 1.6717148924260334], ["lingonberries", 1.6717148924260334], ["elbphilharmonie", 1.6717148924260334], ["conciergerie", 1.6717148924260334], ["unique's", 1.6717148924260334], ["vizcarra", 1.6717148924260334], ["tarkas", 1.6717148924260334], ["delacruz", 1.6717148924260334], ["wildebeest", 1.6716085661488878], ["profiteering", 1.6715296608590058], ["legible", 1.6711571428581211], ["zedong's", 1.671157142858121], ["electromagnet", 1.6711455777567865], ["microfilm", 1.671046000656211], ["normans", 1.671046000656211], ["monitored", 1.6710416976294953], ["hairband", 1.670996647574293], ["analise", 1.670996647574293], ["phragmites", 1.670996647574293], ["vector's", 1.670996647574293], ["turntablist", 1.670996647574293], ["lectures", 1.6709855431308613], ["scraped", 1.670868454505818], ["herbalists", 1.6708261551094858], ["fermentable", 1.6708261551094858], ["anki", 1.6708261551094858], ["unsatisfying", 1.6707924690982943], ["realistic", 1.6707731495900824], ["langues", 1.6707614946545668], ["surya", 1.67057677807141], ["logarithmic", 1.6705209415634064], ["nordland", 1.6703838032529894], ["pales", 1.6702894372104982], ["deviations", 1.6702831366090753], ["wardrobe", 1.670174109244675], ["mashup", 1.670022873304145], ["jealously", 1.670022873304145], ["coulis", 1.670022873304145], ["detangle", 1.670022873304145], ["scafell", 1.670022873304145], ["vices", 1.669929314599758], ["perfected", 1.669832013547196], ["rowboat", 1.6696252488105012], ["hillside", 1.669625248810501], ["sido", 1.6695769392925817], ["painter's", 1.6695376366339354], ["canyon", 1.669353189946429], ["locomotives", 1.6692734239967535], ["shambhala", 1.6692734239967533], ["unpublicized", 1.6691553289543763], ["chronobiology", 1.6691553289543763], ["permis", 1.6691553289543763], ["zile", 1.6691553289543763], ["pardoner", 1.6691553289543763], ["gomti", 1.6691553289543763], ["warfare", 1.6690344924199443], ["mirth", 1.668993054471686], ["slower", 1.6688837103569099], ["abcde", 1.6686276996422367], ["bluesky", 1.6686276996422367], ["azaleas", 1.6685643380436173], ["aggrandizement", 1.6683775305718256], ["meddled", 1.6679681630020617], ["intramural", 1.667930986615246], ["hathor", 1.6678109224785767], ["montana's", 1.667788289372923], ["headphones", 1.6675208251069868], ["tallahassee's", 1.6674575539903134], ["incandescents", 1.6674575539903134], ["urantia", 1.6674575539903134], ["ceinture", 1.6674575539903134], ["bioinspired", 1.6674575539903134], ["telescreen", 1.6674575539903134], ["regales", 1.6674575539903134], ["jarnail", 1.6674575539903134], ["huaca", 1.6674575539903134], ["retyping", 1.6674575539903134], ["orli", 1.6674575539903134], ["karolin", 1.6674575539903134], ["mattel's", 1.6672647843308928], ["chhatrapati", 1.6672647843308928], ["wayfinding", 1.6672647843308928], ["faster", 1.667214570701897], ["noch", 1.667040689601816], ["requests", 1.6668701898843845], ["places", 1.6667943110604273], ["accumulated", 1.6667466379553189], ["quotas", 1.666678368217421], ["godsend", 1.6664620569431552], ["oktoberfest", 1.6664620569431552], ["complicating", 1.666462056943155], ["serialize", 1.666462056943155], ["utsav", 1.666462056943155], ["natron", 1.666462056943155], ["admittedly", 1.666392687725287], ["strikingly", 1.666335809817629], ["turku", 1.6662805251286732], ["comped", 1.6662805251286732], ["heists", 1.666035169448581], ["eigenvalues", 1.666035169448581], ["hooke's", 1.6660180222890268], ["sonographer", 1.6660180222890268], ["draping", 1.6659330213695225], ["tea's", 1.6659330213695223], ["fiestas", 1.665733890001919], ["willing", 1.6653895254902102], ["triomphe", 1.6653322521587872], ["intrapersonal", 1.665219355931266], ["quello", 1.665219355931266], ["nomadism", 1.665219355931266], ["opencourseware", 1.665000248121275], ["cantine", 1.665000248121275], ["piggy's", 1.665000248121275], ["asahi's", 1.665000248121275], ["impresarios", 1.665000248121275], ["bertrand's", 1.665000248121275], ["aftermaths", 1.665000248121275], ["tulu", 1.665000248121275], ["unusable", 1.6647553076648758], ["rohe", 1.6647212133888933], ["enabled", 1.664696554505567], ["conjugating", 1.6646313547462621], ["unequipped", 1.6646313547462621], ["doppelganger", 1.6642783228800029], ["mentally", 1.6642729211327685], ["scribes", 1.6642429708707653], ["smudged", 1.664122638882333], ["pineapple", 1.6640208362106155], ["baitfish", 1.6639471170345446], ["bedsores", 1.6639471170345446], ["bennu", 1.6639471170345446], ["sugarhill", 1.6639471170345446], ["extroverts", 1.6638479988146169], ["loman", 1.6638479988146169], ["closest", 1.6638434941362574], ["gazebo", 1.663791097614288], ["merchants", 1.6637552329806065], ["siouxsie", 1.6637348580558236], ["pentathlon", 1.663516290680882], ["dormitory", 1.6635162906808818], ["anton's", 1.663429713839426], ["vocals", 1.6634261215571975], ["webcomic", 1.6634012082671372], ["curlews", 1.6633587756453097], ["sukhothai", 1.6633587756453097], ["misattributed", 1.6633587756453097], ["aislinn", 1.6633587756453097], ["felicitas", 1.6633587756453097], ["damage", 1.6632634572268057], ["duchenne", 1.6632064534131075], ["onions", 1.6631702800405515], ["presentable", 1.6630798326101779], ["como", 1.6630166290617117], ["bennet", 1.6629792394167398], ["classes", 1.662936111191283], ["counterweights", 1.6627781966660267], ["trackable", 1.662644274453022], ["concepci\u00f3n", 1.662644274453022], ["giraffe", 1.662625916216646], ["triggered", 1.6624138788996767], ["roebling", 1.6623884830261833], ["nemo's", 1.6621143198394055], ["otro", 1.6620959541382547], ["chana", 1.6620323332404887], ["worktime", 1.6620323332404887], ["abberline", 1.6620323332404887], ["mcgarity", 1.6620323332404887], ["ferragosto", 1.6620323332404887], ["capilano", 1.6618928307784069], ["hymnals", 1.6618928307784069], ["antonio", 1.661885541774763], ["cowering", 1.6618655183525755], ["dua's", 1.6615023484403482], ["montaigu", 1.6615023484403482], ["cyprus's", 1.6615023484403482], ["cristofori", 1.6615023484403482], ["storyland", 1.6615023484403482], ["contexte", 1.6615023484403482], ["angono", 1.6615023484403482], ["investitures", 1.6615023484403482], ["barbera", 1.6615023484403482], ["pourover", 1.6615023484403482], ["hashcode", 1.6615023484403482], ["welten", 1.6615023484403482], ["preposterousness", 1.6615023484403482], ["smartsheet", 1.6615023484403482], ["ne's", 1.6615023484403482], ["petrit", 1.6615023484403482], ["sangmin", 1.6615023484403482], ["glockenspiels", 1.6615023484403482], ["rivermouth", 1.6615023484403482], ["mattie's", 1.6615023484403482], ["iberia's", 1.6615023484403482], ["gretel's", 1.6615023484403482], ["puiu", 1.6615023484403482], ["ushabti", 1.6615023484403482], ["maquillaje", 1.6615023484403482], ["utonium", 1.6615023484403482], ["orujo", 1.6615023484403482], ["menthon", 1.6615023484403482], ["crabbing", 1.661309710486906], ["encrypt", 1.661309710486906], ["baronetcy", 1.661309710486906], ["kojiro", 1.661309710486906], ["serkan", 1.661309710486906], ["window", 1.661208842690048], ["scallops", 1.6610796692046805], ["songwriter", 1.6610513218547436], ["existing", 1.66097571329503], ["zaid", 1.660869395164752], ["markets", 1.6608644186279413], ["motorboat", 1.6606610814284797], ["prides", 1.66065590894795], ["li\u00e8ge", 1.6604798854566925], ["alarms", 1.6604542322635543], ["sequestration", 1.6604350300496542], ["produce", 1.6603516960676967], ["delle", 1.660289975250773], ["creamiest", 1.6602670307091658], ["waterloo's", 1.6602670307091658], ["raimo", 1.6602670307091658], ["michigander", 1.6602670307091658], ["nea's", 1.6602670307091658], ["citadel's", 1.6602670307091658], ["unison", 1.660120740752756], ["alertness", 1.6600044341048623], ["partizan", 1.659782680865098], ["hanging", 1.6596796032479664], ["formulating", 1.6595590708398331], ["vantage", 1.6595305830006635], ["sympathizing", 1.6594195534736422], ["crevasse", 1.6594195534736422], ["incensed", 1.6593607394967695], ["carby", 1.6590703423415], ["movie's", 1.6590408634796958], ["basket", 1.6589954206804223], ["dimes", 1.6589772293209009], ["bacterial", 1.658969356654023], ["floodgates", 1.6589544955863396], ["condense", 1.6589544955863396], ["ferrying", 1.6588848039144668], ["splintered", 1.6588439446828438], ["bristol's", 1.6587642921358023], ["junkers", 1.6587642921358023], ["administering", 1.6586667177656766], ["laundromat", 1.6586422860462424], ["celsius", 1.658571287887202], ["shine", 1.6585326000986313], ["encuentro", 1.6583701246472302], ["startle", 1.6583375820623665], ["silverstein", 1.658182065287036], ["maddie", 1.6580673323491906], ["volvo's", 1.6579826549919385], ["guardians", 1.657804136838201], ["rebirths", 1.657571166822605], ["molted", 1.657571166822605], ["peer's", 1.657571166822605], ["goddess's", 1.657571166822605], ["modals", 1.657571166822605], ["grimoires", 1.657571166822605], ["brisket", 1.657466310346971], ["monologue", 1.6574528086663718], ["ruckus", 1.6573869012531706], ["philanthropy", 1.65728366943123], ["demonstrations", 1.6569981983481694], ["sardonically", 1.656902603816907], ["foreseen", 1.656838482199112], ["class's", 1.6567858256447756], ["samson's", 1.6567858256447756], ["refried", 1.6567351687064265], ["undergoing", 1.6565361945723653], ["mushy", 1.656491771132384], ["nestl\u00e9", 1.6564809191206173], ["modernist", 1.6564152795216667], ["tormentors", 1.6563900335220703], ["diplomats", 1.656313625275504], ["coincidentally", 1.656256839110406], ["mysterium", 1.6562011654646358], ["dustpan", 1.6562011654646358], ["sphinx", 1.656185540925339], ["scoff", 1.6561752541225185], ["deze", 1.656054799756937], ["cartwheels", 1.6559640072788804], ["quadrupedal", 1.6559276505535314], ["deadlier", 1.6557654213408197], ["ennui", 1.6557269576571834], ["chains", 1.6556566078779802], ["cafeterias", 1.6556551778460848], ["hipparchus", 1.6555895286950089], ["ashen", 1.6554871964504096], ["speaking", 1.6554454605024833], ["scold", 1.6553981705657892], ["folders", 1.6553634642613717], ["deadline", 1.6552457012130897], ["christened", 1.6551119547925006], ["faithfully", 1.6547848765799873], ["church's", 1.6546656147151986], ["denim", 1.6543677603096838], ["rodents", 1.6543157545504437], ["impossibly", 1.6541178935583911], ["drag\u00e3o", 1.6541178935583911], ["bright's", 1.6541178935583911], ["qui\u00e9n", 1.6541178935583911], ["stitchery", 1.6541178935583911], ["oppressiveness", 1.6541178935583911], ["kulcha", 1.6541178935583911], ["liaising", 1.6541178935583911], ["marthy", 1.6541178935583911], ["trane", 1.6541178935583911], ["le\u00f3n's", 1.6541178935583911], ["heroscape", 1.6541178935583911], ["naach", 1.6541178935583911], ["hal's", 1.6541178935583911], ["ueno", 1.6541178935583911], ["fasching", 1.6541178935583911], ["optimizer", 1.6541178935583911], ["codependency", 1.6541178935583911], ["arista", 1.6541178935583911], ["achraf", 1.6541178935583911], ["qiantang", 1.6541178935583911], ["schepisi", 1.6541178935583911], ["sala's", 1.6541178935583911], ["disjointedness", 1.6541178935583911], ["tryline", 1.6541178935583911], ["pontefract", 1.6541178935583911], ["martinican", 1.6541178935583911], ["mikael's", 1.6541178935583911], ["snackbar", 1.6541178935583911], ["kagame's", 1.6541178935583911], ["echter", 1.6541178935583911], ["feliway", 1.6541178935583911], ["junglee", 1.6541178935583911], ["memsaab", 1.6541178935583911], ["toussaint's", 1.6541178935583911], ["winterthur", 1.6541178935583911], ["sofabed", 1.6541178935583911], ["lombardo's", 1.6541178935583911], ["\u010desk\u00e1", 1.6541178935583911], ["tataki", 1.6541178935583911], ["artaxerxes", 1.6541178935583911], ["nishimura's", 1.6541178935583911], ["tajo", 1.6541178935583911], ["chiropractor's", 1.6541178935583911], ["grupa", 1.6541178935583911], ["innerspeaker", 1.6541178935583911], ["opensecrets", 1.6541178935583911], ["melany", 1.6541178935583911], ["hillslopes", 1.6541178935583911], ["overstocks", 1.6541178935583911], ["majnun", 1.6541178935583911], ["zillow's", 1.6541178935583911], ["pafa", 1.6541178935583911], ["antlion", 1.6541178935583911], ["graupel", 1.6541178935583911], ["veer's", 1.6541178935583911], ["cyberlaw", 1.6541178935583911], ["p\u0101keh\u0101", 1.6541178935583911], ["tpus", 1.6541178935583911], ["icewine", 1.6541178935583911], ["zosima", 1.6541178935583911], ["janey's", 1.6541178935583911], ["alodia", 1.6541178935583911], ["hooks", 1.654117893558391], ["typhoons", 1.654117893558391], ["coldplay's", 1.654117893558391], ["ladened", 1.654117893558391], ["howrah", 1.654117893558391], ["table's", 1.654117893558391], ["definity", 1.654117893558391], ["kiji", 1.654117893558391], ["malfurion", 1.654117893558391], ["illustratively", 1.654117893558391], ["uplighting", 1.654117893558391], ["lungomare", 1.654117893558391], ["vandalization", 1.654117893558391], ["sax's", 1.654117893558391], ["fruitdale", 1.654117893558391], ["francesi", 1.654117893558391], ["aicardi", 1.654117893558391], ["rinascente", 1.654117893558391], ["sezen", 1.654117893558391], ["kairon", 1.654117893558391], ["gulaab", 1.654117893558391], ["josif", 1.654117893558391], ["ignota", 1.654117893558391], ["asara", 1.654117893558391], ["shinsekai", 1.654117893558391], ["ncts", 1.654117893558391], ["pleading", 1.6537519382722057], ["injecting", 1.6536313882955798], ["rephrase", 1.6535061635681698], ["statues", 1.6532852838804253], ["yoruba", 1.6531561971086477], ["tents", 1.652974972903294], ["physics", 1.6527675932371189], ["expressive", 1.652426880841928], ["peppery", 1.6523120879977378], ["bystanders", 1.6521855128416172], ["mila", 1.6521855128416172], ["conus", 1.6521855128416172], ["gabriel's", 1.6520398560036695], ["groundings", 1.6520398560036693], ["sandbags", 1.6519917780139715], ["corleone", 1.6519414226458142], ["symmetrical", 1.6517363102037914], ["chortled", 1.651670973597506], ["appearing", 1.651654313716921], ["junos", 1.651615445913068], ["malvolio", 1.651615445913068], ["fulham's", 1.651615445913068], ["inactions", 1.651557339543595], ["lors", 1.651557339543595], ["wundt", 1.651557339543595], ["caitriona", 1.651557339543595], ["wale", 1.6514670636007653], ["corrupting", 1.651453399528819], ["deva's", 1.6509731827341356], ["ozturk", 1.6509731827341356], ["yasmin's", 1.6509731827341356], ["grazie", 1.6508997653608068], ["scissoring", 1.6508488858636117], ["suffragan", 1.6508488858636117], ["bront\u00eb", 1.6508228379935337], ["conditioners", 1.6507667633040184], ["headstrong", 1.6507421427552107], ["auggie", 1.6507317770925605], ["cafe", 1.650587763907504], ["circled", 1.6504855400759149], ["hostname", 1.6504256661174124], ["farooq", 1.650288916952932], ["termite", 1.6500031724301365], ["mustaine", 1.6499825988244952], ["lest", 1.6499468469968246], ["mandy's", 1.6497877420045732], ["frightened", 1.6497570372935553], ["pretentious", 1.6496473046568818], ["lean", 1.6495331102664628], ["kannada", 1.64953040336231], ["crusted", 1.6494584065342828], ["danu", 1.6493372060047542], ["snoop's", 1.6493372060047542], ["doubtfully", 1.6493372060047542], ["kathleen's", 1.6493372060047542], ["kitna", 1.6493372060047542], ["ut's", 1.6493372060047542], ["fossils", 1.6492318104434132], ["fair", 1.649220722619683], ["asmaa", 1.649120558532535], ["knapping", 1.649120558532535], ["kwek", 1.649120558532535], ["irresistibly", 1.6491205585325348], ["porpoises", 1.6489487751410212], ["garbs", 1.6488833432623202], ["modernista", 1.6488833432623202], ["superfeet", 1.6488833432623202], ["endtime", 1.6488833432623202], ["kabul's", 1.6488833432623202], ["escamillo", 1.6488833432623202], ["cardinalis", 1.6488833432623202], ["hire's", 1.6488833432623202], ["bica", 1.6488833432623202], ["ellena", 1.6488833432623202], ["\u0433\u043e\u0434\u0430", 1.6488833432623202], ["hypersphere", 1.6488833432623202], ["alguns", 1.6488833432623202], ["dragonfish", 1.6488833432623202], ["clerval", 1.6488833432623202], ["makonde", 1.6488833432623202], ["dorival", 1.6488833432623202], ["multidimensionality", 1.6488833432623202], ["broz", 1.6487991865051488], ["warthogs", 1.6487991865051488], ["gus's", 1.6487991865051488], ["dojos", 1.6487560851513945], ["crust", 1.6484225774578156], ["carne", 1.6483644400155792], ["velvet", 1.6481501507335834], ["pickles", 1.6480929608335095], ["innovator", 1.6480365777732497], ["retiro", 1.6480141374936], ["collegeville", 1.6480141374936], ["leaps", 1.64797815465217], ["exited", 1.647914951457547], ["headdress", 1.6477837440753995], ["violinists", 1.6477559016600893], ["drip", 1.6476973043662895], ["kamakura", 1.6474480633424298], ["parfait", 1.6472713790982985], ["confusing", 1.6472651194279349], ["feeding", 1.64725460334559], ["jonny's", 1.6471089194331436], ["meld", 1.6470288168717124], ["alienation", 1.6469880750516739], ["eleazar", 1.646894671315778], ["tangibles", 1.6467987878346815], ["ghiberti", 1.6467987878346815], ["maligne", 1.6467987878346815], ["istrian", 1.6465991758603984], ["rupali", 1.6465991758603984], ["disinfecting", 1.6465991758603982], ["lanfranc", 1.646547559949428], ["clockwork", 1.6465277956290885], ["guin's", 1.6464599403474727], ["unformatted", 1.6464599403474727], ["dessner", 1.6464599403474727], ["speeds", 1.646438060481156], ["talmudic", 1.6464314721199227], ["shunned", 1.6464243219604453], ["volga", 1.6462068949370248], ["applications", 1.6460933399328903], ["acropora", 1.6460881950459718], ["geoglyphs", 1.6460881950459718], ["dunsinane", 1.6460881950459718], ["zikr", 1.6460881950459718], ["andong", 1.6460881950459718], ["texto", 1.6460881950459718], ["otter's", 1.6459895501011015], ["sly's", 1.6459895501011015], ["immerses", 1.645956127636228], ["plans", 1.6459446051431612], ["diplomat", 1.6459316978606788], ["kansai", 1.6458802546462477], ["oatcakes", 1.6456785165504402], ["benchwarmers", 1.6456785165504402], ["tintype", 1.6456785165504402], ["zehra", 1.6456785165504402], ["camellia", 1.645372557263143], ["jousting", 1.645372557263143], ["lauryn", 1.645192797010414], ["inconvenient", 1.6451464066780743], ["explosiveness", 1.6451281223977479], ["mechanicsburg", 1.6451281223977479], ["revving", 1.645069108582689], ["aspiration", 1.6450202451438198], ["quarrymen", 1.6449791206658033], ["type's", 1.6449791206658033], ["hissed", 1.6449574305402668], ["kylie's", 1.6448873249559448], ["results", 1.644772127459786], ["wintery", 1.6447194964359004], ["whinnying", 1.6445840728462389], ["egle", 1.6445840728462389], ["jacqueline's", 1.6445840728462389], ["selfcare", 1.6445840728462389], ["showoff", 1.6445009290609587], ["antiguan", 1.644444689502494], ["braille", 1.6443877883021654], ["haarlem", 1.6442248439557692], ["mocking", 1.6442164836321613], ["photinus", 1.6441533279345453], ["protectin", 1.6441533279345453], ["reabsorbing", 1.6441533279345453], ["realschule", 1.6441533279345453], ["keanu's", 1.6441533279345453], ["bananagrams", 1.6441533279345453], ["yippies", 1.6441533279345453], ["toungoo", 1.6441533279345453], ["lactea", 1.6441533279345453], ["equiangular", 1.6441533279345453], ["amniotes", 1.6441533279345453], ["heartfield", 1.6441533279345453], ["lonelier", 1.6440318088415715], ["repertoires", 1.6440318088415715], ["oricon", 1.6437796567236511], ["hoops", 1.6435596942378055], ["impoliteness", 1.6435145737278887], ["deservingly", 1.6435145737278887], ["solidaire", 1.6435145737278887], ["flamenca", 1.6435145737278887], ["necho", 1.6435145737278887], ["lekha", 1.6435145737278887], ["faps", 1.6435145737278887], ["sichuan", 1.643484278528373], ["throat", 1.643469509618609], ["danielle", 1.643407777700819], ["gymnasium", 1.6431793720042147], ["localizing", 1.6431634704222429], ["more's", 1.6431634704222429], ["barbro", 1.6431634704222429], ["clanks", 1.6431634704222429], ["reindeer's", 1.6431634704222429], ["hungnam", 1.6431634704222429], ["olek", 1.6431634704222429], ["keas", 1.6431634704222429], ["nyjer", 1.6431634704222429], ["pressy", 1.6431634704222429], ["flughafen", 1.6431634704222429], ["leipziger", 1.6431634704222429], ["sifaka", 1.6431634704222429], ["sliabh", 1.6431634704222429], ["crispino", 1.6431634704222429], ["omurice", 1.6431634704222429], ["birdie's", 1.6431634704222429], ["podium", 1.642980682916751], ["policymaker", 1.6429790525243277], ["subtree", 1.6429790525243277], ["ramat", 1.642941421304618], ["bowed", 1.642910085401103], ["bradgate", 1.6427883189449772], ["fixatives", 1.6427883189449772], ["secam", 1.6427883189449772], ["knee", 1.6427361711365933], ["palaces", 1.6426663081414483], ["gomen", 1.6424691759981207], ["populares", 1.6424691759981207], ["superiority", 1.6424242580640838], ["prophetic", 1.6423789923782992], ["stigmatized", 1.6423027657472598], ["drachmas", 1.6423027657472595], ["sanda", 1.6423027657472595], ["engendered", 1.6422177648277554], ["apricots", 1.6420879452416026], ["playthrough", 1.6420314371336429], ["tvnz", 1.6419552619881088], ["jacinda", 1.6419552619881088], ["hellofresh", 1.6419552619881086], ["acrostics", 1.6419552619881086], ["cone's", 1.6419552619881086], ["stellarium", 1.6419552619881086], ["biljana", 1.6419552619881086], ["thingyan", 1.6419552619881086], ["skyrail", 1.6419552619881086], ["swaps", 1.6417999943510413], ["smorgasbord", 1.641539050261369], ["regia", 1.641393909761788], ["colan", 1.6412453807680536], ["malamutes", 1.6412453807680536], ["sunbathed", 1.6412453807680536], ["acerola", 1.6412453807680536], ["asturias", 1.6412453807680534], ["scratch", 1.641084585328112], ["farmhouse", 1.641067343637731], ["tatarstan", 1.641031517818214], ["debater", 1.6409376712989616], ["caste", 1.6408184532081729], ["apps", 1.6407889524964585], ["jared's", 1.6407782331264686], ["aerobic", 1.6407782331264686], ["tutankhamun", 1.6407782331264686], ["porky's", 1.6407242264040722], ["raspberries", 1.6406944237415562], ["negate", 1.6406944237415562], ["sediment", 1.6405956370140926], ["tribes", 1.6405912285409745], ["ascend", 1.6405712383956284], ["meted", 1.6404474977438588], ["samoas", 1.6404474977438588], ["pidgins", 1.6404474977438588], ["blaugrana", 1.6404474977438588], ["gaddi", 1.6404474977438588], ["montecarlo", 1.6402565704280136], ["coyote's", 1.6402565704280136], ["singlish", 1.6402565704280136], ["transcendentalism", 1.6402010362808563], ["bine", 1.6402010362808563], ["trickery", 1.640194342265812], ["clock", 1.6401593522261324], ["ssris", 1.6400999453078964], ["reveler", 1.6400999453078964], ["lexie's", 1.6400999453078964], ["afrikaner", 1.6400999453078964], ["baux", 1.6400999453078964], ["vorontsov", 1.6400999453078964], ["calibri", 1.6400999453078964], ["initialisms", 1.6400999453078964], ["elrick", 1.6400999453078964], ["shined", 1.6400348464770114], ["margins", 1.640014865275162], ["hogwarts", 1.6398771434813983], ["otacon", 1.6398582565449566], ["grayson's", 1.6398582565449566], ["tinguely", 1.6398582565449566], ["barbless", 1.6398582565449566], ["diophantus", 1.6398582565449566], ["femininely", 1.6398582565449566], ["savitsky", 1.6398582565449566], ["timba", 1.6398582565449566], ["ijssel", 1.6398582565449566], ["berlaymont", 1.6398582565449566], ["understandin", 1.6398582565449566], ["cymatics", 1.6398582565449566], ["cesario", 1.6397342597013616], ["michiganders", 1.6397342597013616], ["timed", 1.639540235863362], ["princeton's", 1.639348983794477], ["cusp", 1.6393489837944768], ["waddle", 1.6388825971703531], ["minced", 1.638817303042976], ["teatro", 1.638622643970022], ["respiratory", 1.6384985140681336], ["balkan", 1.6383770942487226], ["hangul", 1.638289014194196], ["asano", 1.6382129138126373], ["parsis", 1.6380338403456944], ["warfarin", 1.638030934639345], ["alexanderplatz", 1.637975526055238], ["eclairs", 1.637975526055238], ["doric", 1.6378765547243583], ["buzz", 1.637798248269266], ["screeching", 1.6377690539127558], ["molten", 1.6376926574658137], ["biloba", 1.6375767146228073], ["equipment", 1.6375603695843333], ["dribbble", 1.6374656999990782], ["aama", 1.6374656999990782], ["miramax", 1.6371235316382706], ["dodgeball", 1.6369252021045568], ["protections", 1.6368953845980005], ["performance", 1.6368922520461624], ["sper", 1.6368874988338245], ["fibbed", 1.6368874988338245], ["darwins", 1.6368874988338245], ["laem", 1.6368874988338245], ["pascal's", 1.63679728734312], ["orthopedist", 1.63679728734312], ["thailand's", 1.6365954582452724], ["silverton", 1.6365208946907486], ["scales", 1.6364075737032764], ["myer's", 1.636267700462437], ["sentido", 1.636267700462437], ["sue\u00f1o", 1.636267700462437], ["departments", 1.6362419042973089], ["softwares", 1.6361383512371042], ["reuse", 1.636092249846537], ["leaden", 1.6359950879516756], ["whitman's", 1.6359407738489582], ["honeyed", 1.6359407738489582], ["sevillian", 1.635940773848958], ["terrell's", 1.635940773848958], ["clear's", 1.635940773848958], ["carni", 1.635940773848958], ["danakil", 1.635940773848958], ["upanishadic", 1.635940773848958], ["lumin", 1.635940773848958], ["fritas", 1.635940773848958], ["natraj", 1.635940773848958], ["negocio", 1.635940773848958], ["saben", 1.635940773848958], ["bumblefoot", 1.635940773848958], ["blowguns", 1.635940773848958], ["opobo", 1.635940773848958], ["zamosc", 1.635940773848958], ["alive", 1.6357643767568215], ["krabby", 1.6357388058521867], ["refuting", 1.635717284672203], ["lightning", 1.635708358568336], ["luanda", 1.6355512641408991], ["equalization", 1.6355512641408991], ["discomforting", 1.6355024188720426], ["calculates", 1.6354106078574329], ["boardgame", 1.6353210993134095], ["bulged", 1.6353210993134093], ["hdds", 1.6353210993134093], ["thumping", 1.635009979960389], ["racing", 1.6349463530990314], ["rsvping", 1.6348839645635262], ["unfelt", 1.6348839645635262], ["nauruans", 1.6348839645635262], ["qalam", 1.6348839645635262], ["sleepiest", 1.6348839645635262], ["hyacinth's", 1.6348839645635262], ["d\u00e9claration", 1.6348839645635262], ["nazan", 1.6348839645635262], ["azad's", 1.6348839645635262], ["supervolcanoes", 1.6348839645635262], ["scifres", 1.6348839645635262], ["tile's", 1.6348839645635262], ["earthships", 1.6348839645635262], ["schnoor", 1.6348839645635262], ["acarnania", 1.6348839645635262], ["aperto", 1.6348839645635262], ["comarca", 1.6348839645635262], ["triskele", 1.6348839645635262], ["pandoc", 1.6348839645635262], ["statistiques", 1.6348839645635262], ["porte\u00f1o", 1.6348839645635262], ["sambac", 1.6348839645635262], ["elspeth's", 1.6348839645635262], ["caeruleus", 1.6348839645635262], ["animotion", 1.6348839645635262], ["cabarete", 1.6348839645635262], ["am's", 1.634883964563526], ["globalizing", 1.6347940863906527], ["expansions", 1.6347753859321035], ["relatives", 1.6346715392084876], ["underfunding", 1.6346576830459394], ["afterparty", 1.6346576830459394], ["jaber", 1.6345590671142884], ["telescope's", 1.6345038473897144], ["jord", 1.6345038473897144], ["blackcap", 1.6343080984259555], ["hailstones", 1.634154401739583], ["coequal", 1.634108402910507], ["temas", 1.634108402910507], ["unchallenging", 1.634108402910507], ["intimidation", 1.6340321762794678], ["smelled", 1.6339648749185183], ["checks", 1.6339279251517222], ["railings", 1.633854949362301], ["memorize", 1.6336905340295074], ["alarmist", 1.6336122998365927], ["solana", 1.6334414198889113], ["fintan", 1.633319073890867], ["longboards", 1.633319073890867], ["diseases", 1.6330802587767803], ["beethoven", 1.633039153864213], ["subconscious", 1.6330194510385136], ["pressure", 1.6328697762051079], ["chobe", 1.632847088573147], ["proyecto", 1.632847088573147], ["bheem", 1.6327547473316224], ["precum", 1.6327547473316224], ["hosier", 1.632736816203347], ["tao's", 1.632736816203347], ["code", 1.6325936844684625], ["waterway", 1.6324829078644982], ["checked", 1.6323926996105744], ["starbursts", 1.632353184432623], ["terek", 1.632353184432623], ["abdoulaye", 1.6323531844326227], ["phrased", 1.6323531844326227], ["synchronize", 1.6323531844326227], ["coords", 1.6323531844326227], ["yun's", 1.6323531844326227], ["ranjha", 1.6323531844326227], ["bisque", 1.6321216449738378], ["bill's", 1.6318509219143358], ["jackhammers", 1.6318357983202827], ["puft", 1.6318357983202827], ["mystica", 1.6316716173410017], ["thumb's", 1.6316716173410017], ["arbuscular", 1.6316716173410017], ["swirl", 1.6315876670771645], ["backroom", 1.6314869273529902], ["troupes", 1.631458744331564], ["volkov", 1.6313737725219633], ["dropped", 1.6312005059828336], ["radiations", 1.6311440339256358], ["laudato", 1.6310525046043898], ["panglao", 1.6310525046043898], ["ringu", 1.6310525046043898], ["umrao", 1.6310525046043898], ["breathings", 1.6310525046043898], ["unenthusiastically", 1.6310525046043898], ["carota", 1.6310525046043898], ["saac", 1.6310525046043898], ["yaksha", 1.6310525046043898], ["frizz", 1.6310525046043896], ["prerequisites", 1.6309928901122623], ["nursed", 1.6308538933332588], ["codenamed", 1.6307734878025266], ["stint", 1.630748906118684], ["anthropologist", 1.6307320888563586], ["standardized", 1.6306498593706062], ["anchovies", 1.6304876379361284], ["swigs", 1.6304876379361284], ["weigh", 1.6304150782975746], ["hamilton's", 1.6303020600123934], ["judy's", 1.6303020600123932], ["soupy", 1.6303020600123932], ["fashionista", 1.6302833417780105], ["behavioral", 1.6302428493765055], ["lincoln's", 1.6301563726703958], ["gnome", 1.6300389621964653], ["diet", 1.630006457327691], ["astound", 1.6297926304178265], ["daewoo", 1.6297926304178265], ["emotionality", 1.6297926304178265], ["nosy", 1.629586484120025], ["backcountry", 1.6291000962358886], ["pastoralism", 1.629055501231749], ["epiphytic", 1.629055501231749], ["clutches", 1.6290209324147464], ["interviewers", 1.6290047676440431], ["snippets", 1.6289465343085894], ["theodore", 1.6289350089609476], ["regionale", 1.6287812956264243], ["granja", 1.6287812956264243], ["pra\u00e7a", 1.6287812956264243], ["reconstructing", 1.6286493036251581], ["jungler", 1.6286493036251581], ["scrooge", 1.6282723014715412], ["uneventful", 1.628176930283029], ["screamed", 1.6281246123739022], ["rash", 1.6278746769778973], ["anthropomorphized", 1.6278439214922011], ["spas", 1.6278241120133876], ["larb", 1.6277223952569273], ["xinjiang's", 1.6277223952569273], ["saalbach", 1.6277223952569273], ["shaila", 1.6277223952569273], ["pressuring", 1.627468216384395], ["designated", 1.6274011017382768], ["shorebirds", 1.627148580076461], ["weller's", 1.6270012067787452], ["fathomless", 1.6270012067787452], ["mileages", 1.6270012067787452], ["tough", 1.6269400906468268], ["homeownership", 1.6268794252512917], ["rose", 1.6268412142243724], ["associated", 1.6268174877313946], ["acknowledge", 1.6267318357180205], ["dissections", 1.6267318357180205], ["san's", 1.6267318357180205], ["broadcasted", 1.6266754342259007], ["avoided", 1.6266552951973894], ["reunions", 1.6265809137000713], ["padova", 1.6265492619990845], ["sleepy", 1.6264917074446812], ["ideen", 1.6264783914809373], ["froggie", 1.6264783914809373], ["frother", 1.6264783914809373], ["revive", 1.626232711949865], ["specifying", 1.6261767129239588], ["lookalikes", 1.6261767129239588], ["foreheads", 1.6260666601337137], ["goby", 1.6259227021909186], ["headshake", 1.6255986195315222], ["waterers", 1.6255986195315222], ["edmon", 1.6255986195315222], ["aeonium", 1.6255986195315222], ["poct", 1.6255986195315222], ["maraming", 1.6255986195315222], ["ravs", 1.6255986195315222], ["maiti", 1.6255986195315222], ["laad", 1.6255986195315222], ["fish", 1.6255458193642431], ["wastelands", 1.6254240117313579], ["adventureland", 1.6253359035988286], ["underreporting", 1.6253359035988286], ["strictness", 1.6253278669300013], ["sextus", 1.6253278669300013], ["timetables", 1.6253148456606081], ["dedalus", 1.6253148456606081], ["timespan", 1.6252670000660936], ["acceptances", 1.6252250045879388], ["sibylle", 1.6252250045879386], ["poleis", 1.6252250045879386], ["atharvaveda", 1.6252250045879386], ["hypermasculinity", 1.6252250045879386], ["tumultuously", 1.6252250045879386], ["tuca", 1.6252250045879386], ["naab", 1.6252250045879386], ["tadka", 1.6252250045879386], ["darkman", 1.6252250045879386], ["confucianist", 1.6252250045879386], ["mopsy", 1.6252250045879386], ["extracted", 1.625196729970601], ["sardinian", 1.6251708304211192], ["svensson", 1.6251708304211192], ["ume\u00e5", 1.6251708304211192], ["silos", 1.6251565871556752], ["kabbalah", 1.6249960996577153], ["verbalized", 1.6249520868709635], ["razi", 1.6249520868709635], ["koine", 1.6249520868709635], ["fanciest", 1.6247706406081615], ["preclinical", 1.6246526485257171], ["unearths", 1.624580074030563], ["shampooing", 1.624580074030563], ["funds", 1.6244941458282998], ["camper", 1.6243528600341948], ["expelliarmus", 1.6240430227664202], ["weniger", 1.6240430227664202], ["ainsworth's", 1.6240430227664202], ["krumping", 1.6240430227664202], ["superhero's", 1.6240430227664202], ["hognose", 1.6240430227664202], ["frannie", 1.6240430227664202], ["roosted", 1.6240430227664202], ["emfs", 1.6240430227664202], ["qualit\u00e9", 1.6240430227664202], ["gr\u00e1inne", 1.6238068326816666], ["ceridwen", 1.6238068326816666], ["mumbaikars", 1.6238068326816666], ["sakis", 1.6238068326816666], ["optionals", 1.6238068326816666], ["itaipu", 1.6238068326816666], ["avril's", 1.6238068326816666], ["goizueta", 1.6238068326816666], ["planeteers", 1.6238068326816666], ["nottingham's", 1.6237322606112792], ["latency", 1.6237113146326854], ["sisyphus", 1.6237113146326851], ["stravinsky", 1.6235888720329843], ["grigio", 1.6234860807147171], ["bluesy", 1.6233722078045176], ["swimmers", 1.6233231455400166], ["broomstick", 1.6232784074073023], ["cortisol", 1.62319980209001], ["clustering", 1.62319980209001], ["tiktokers", 1.6231998020900098], ["contraltos", 1.6231998020900098], ["\u00e1ngela", 1.6231998020900098], ["memers", 1.6231998020900098], ["unlimited's", 1.6231998020900098], ["jodelle", 1.6231998020900098], ["sacramento's", 1.6231998020900098], ["korsakov's", 1.6231998020900098], ["jyotirlinga", 1.6231998020900098], ["vrienden", 1.6231998020900098], ["amager", 1.6231998020900098], ["ssa's", 1.6231998020900098], ["tisanes", 1.6231998020900098], ["unics", 1.6231998020900098], ["jondalar", 1.6231998020900098], ["gekommen", 1.6231998020900098], ["fillo", 1.6231998020900098], ["pbkdf2", 1.6231998020900098], ["crabber", 1.6231998020900098], ["comice", 1.6231998020900098], ["mahadevi", 1.6231998020900098], ["sybylla", 1.6231998020900098], ["pitahaya", 1.6231998020900098], ["jord\u00e3o", 1.6231998020900098], ["vovinam", 1.6231998020900098], ["erised", 1.6231998020900098], ["labaki", 1.6231998020900098], ["aroids", 1.6231998020900098], ["hairclips", 1.6231998020900098], ["atim", 1.6231998020900098], ["aadam", 1.6231998020900098], ["chartres", 1.6231329762438966], ["festivity", 1.6231329762438966], ["sarge", 1.6231329762438966], ["megafauna", 1.6231031830541713], ["hierarchical", 1.6229895760559871], ["sinuous", 1.62288454964679], ["iyer", 1.62288454964679], ["sumatran", 1.622623927041965], ["chameleon", 1.6226108860620407], ["wilma", 1.6225775099947777], ["perimeter", 1.6223477795276247], ["fluffing", 1.6223079340668833], ["sorter", 1.6223079340668833], ["floaties", 1.6222056512389365], ["sizzle", 1.6220054621858233], ["records", 1.6219053017080804], ["views", 1.621867488473924], ["sanja", 1.6218218735653156], ["tightness", 1.6214708298697385], ["projections", 1.6212289997683411], ["kann", 1.6211673379098175], ["bombarded", 1.621147300409761], ["illusions", 1.6208851613332633], ["eurystheus", 1.6208692926828454], ["casemates", 1.6208692926828454], ["skater's", 1.6208692926828454], ["carbohydrates", 1.6208287709505285], ["scandals", 1.6208057970908958], ["sextet", 1.6207687424785848], ["priestesses", 1.6207687424785848], ["dots", 1.6207478630100827], ["historial", 1.6203603855265871], ["aestheticians", 1.6203603855265871], ["spirogyra", 1.6203603855265871], ["overate", 1.6203603855265871], ["suraya", 1.6203603855265871], ["kells", 1.6201291697181501], ["raven's", 1.6201000448476786], ["compacted", 1.6194507520766284], ["bingley", 1.6192943589571618], ["pattern", 1.6191900322522614], ["buddy's", 1.619142763968226], ["skimp", 1.6191365483773978], ["danube", 1.6190577860318818], ["acidic", 1.6189962122568087], ["barque", 1.618923895823106], ["moishe", 1.6187502412290176], ["xslt", 1.6187502412290176], ["squarespace", 1.618672510124997], ["promenade", 1.618632594152662], ["animatronic", 1.618516018365909], ["bottega", 1.618516018365909], ["nkvd", 1.618468801024805], ["correlating", 1.6184175073664835], ["sorbonne", 1.6184175073664835], ["lyric", 1.618285128885288], ["knock", 1.6182479039692845], ["josie's", 1.6181588089158174], ["prebiotic", 1.6181588089158174], ["mst3k", 1.6181588089158174], ["uncirculated", 1.6181588089158174], ["refract", 1.6181588089158174], ["howlers", 1.6181588089158174], ["percolated", 1.6181588089158174], ["arigato", 1.6181588089158174], ["minsoo", 1.6181588089158174], ["jame's", 1.6181588089158174], ["diem's", 1.6181588089158174], ["rcn's", 1.6181588089158174], ["velit", 1.6181588089158174], ["mustelidae", 1.6181588089158174], ["ridvan", 1.6181588089158174], ["kakaako", 1.6181588089158174], ["arnar", 1.6181588089158174], ["wizzair", 1.6181588089158174], ["cogging", 1.6181588089158172], ["gokstad", 1.6181588089158172], ["werde", 1.6181588089158172], ["aspirin's", 1.6181588089158172], ["goatskins", 1.6181588089158172], ["motores", 1.6181588089158172], ["systemverilog", 1.6181588089158172], ["ciobanu", 1.6181588089158172], ["nigella's", 1.6181588089158172], ["pitch's", 1.6181588089158172], ["caramelo", 1.6181588089158172], ["seedpod", 1.6181588089158172], ["regata", 1.6181588089158172], ["menjadi", 1.6181588089158172], ["celf", 1.6181588089158172], ["encryptor", 1.6181588089158172], ["associ\u00e9s", 1.6181588089158172], ["bhisma", 1.6181588089158172], ["chacal", 1.6181588089158172], ["formality", 1.6180440458797238], ["seder", 1.618006869121553], ["stepney", 1.6179340646368015], ["termites", 1.6179166601976855], ["gupta", 1.6177494532647523], ["swarming", 1.6177272999001064], ["conferences", 1.6176345456257015], ["kingston's", 1.6176299988475442], ["colorings", 1.6176299988475442], ["swiped", 1.6174953571696444], ["korma", 1.6174953571696444], ["brittleness", 1.6174953571696444], ["crist\u00f3bal", 1.6173597181459822], ["explainable", 1.61731160535094], ["roadsides", 1.6171956191486057], ["distros", 1.6171956191486057], ["piggybacking", 1.6171956191486057], ["mathura", 1.6171956191486057], ["hardshell", 1.6171543652107176], ["stdout", 1.617006274151632], ["neutralizing", 1.6169745276850738], ["istanbul", 1.6168250537763473], ["benevolent", 1.616730297334126], ["earnest", 1.616565435909226], ["handprints", 1.6165243050684277], ["eglin", 1.6165243050684277], ["antibiotic", 1.616298267669279], ["symmetry", 1.6162110251643447], ["prague", 1.6160953849312938], ["moods", 1.615945942168582], ["patronage", 1.6158444210212688], ["alte", 1.6158281275037987], ["kickstarted", 1.6158281275037987], ["weiwei", 1.6158281275037987], ["keiji", 1.615650035568661], ["syllogisms", 1.6155082621362664], ["providence's", 1.6155082621362664], ["uga's", 1.6155082621362664], ["activity", 1.6154551724896868], ["viewer's", 1.6154424751838243], ["thwarted", 1.6154044960495775], ["skopje", 1.6153797695874685], ["roni", 1.6153495054281164], ["flabbergasted", 1.6152967593218168], ["restroom", 1.6152967593218166], ["dodging", 1.6151974725334877], ["chaotically", 1.6151319499391696], ["jafari", 1.6151319499391696], ["talat", 1.6151319499391696], ["finning", 1.6151319499391696], ["hecho", 1.614967174184228], ["winter's", 1.6148856101855318], ["equine", 1.6148856101855318], ["buddies", 1.6148774032816382], ["scribble", 1.6148441446889878], ["toiletries", 1.6148441446889878], ["dimitri", 1.6146883391189177], ["trundled", 1.6146833819207234], ["zayd", 1.6146833819207234], ["lobbying", 1.6146784502094087], ["checkpoints", 1.6146028549900515], ["impassive", 1.6145456951479034], ["acapella", 1.6145456951479034], ["retaining", 1.6142050986143206], ["dealings", 1.6141950193262542], ["louverture", 1.6141485020783557], ["trnc", 1.6141485020783557], ["cltv", 1.6141485020783557], ["straitlaced", 1.6141485020783557], ["cavaco", 1.6141485020783557], ["plantae", 1.6141485020783557], ["lacking", 1.6141089170079475], ["tactician", 1.6140989122626235], ["bramante", 1.6140787312332256], ["cryptid", 1.6140787312332256], ["lorem", 1.6140787312332256], ["tero", 1.6140787312332256], ["spewing", 1.6138927538741301], ["nsstring", 1.6137735546911134], ["precisa", 1.6137735546911134], ["saap", 1.6137735546911134], ["widthwise", 1.6137735546911134], ["bidayuh", 1.6137735546911134], ["fariz", 1.6137735546911134], ["tailcoats", 1.6137735546911134], ["quente", 1.6137735546911134], ["wolframalpha", 1.6137735546911134], ["pyrokinetic", 1.6137735546911134], ["quijada", 1.6137735546911134], ["totoaba", 1.6137735546911134], ["soundgarden's", 1.6137735546911134], ["rver", 1.6137735546911134], ["kiin", 1.6137735546911134], ["touto", 1.6137735546911134], ["foer's", 1.6137735546911134], ["cogon", 1.6137735546911134], ["teor\u00eda", 1.6137735546911134], ["fruitwood", 1.6137735546911134], ["philolaus", 1.6137735546911134], ["m\u00e4rta", 1.6137735546911134], ["huevos", 1.6137735546911132], ["ambiente", 1.6134820493524769], ["alkane", 1.6134820493524769], ["woodworks", 1.6134820493524769], ["stoplights", 1.6134820493524769], ["nusa", 1.6132366528430697], ["shah's", 1.6132366528430697], ["abie", 1.6131490293216508], ["jetstream", 1.6127649462194316], ["outsider's", 1.6127649462194316], ["neos", 1.6127649462194316], ["sculpted", 1.6127649462194316], ["eagleton", 1.6127649462194316], ["jeremey", 1.6127649462194311], ["olympique", 1.6127649462194311], ["imperiali", 1.6127649462194311], ["furikake", 1.6127649462194311], ["easeus", 1.6127649462194311], ["beguines", 1.6127649462194311], ["yangshuo", 1.6127649462194311], ["saleem's", 1.6127649462194311], ["proximities", 1.6127649462194311], ["ectomorph", 1.6127649462194311], ["balay", 1.6127649462194311], ["fredrich", 1.6127649462194311], ["wolfsburg's", 1.6127649462194311], ["extremist", 1.6123341863513172], ["motivators", 1.6122822269897938], ["trapezium", 1.6121751260513637], ["equines", 1.6120640488069065], ["etcetera", 1.6120220789019648], ["accessories", 1.6119295331418768], ["preflight", 1.6117046142363811], ["kuiper", 1.6117046142363811], ["stockbroker", 1.611686173680154], ["boogaard", 1.6116405458386136], ["berg\u00e8re", 1.6116405458386136], ["jabr", 1.6116405458386136], ["cfo's", 1.6116405458386136], ["subfolder", 1.6116405458386136], ["planarian", 1.6116405458386136], ["tresco", 1.6116405458386136], ["vaduz", 1.6116405458386136], ["classiness", 1.6116405458386136], ["sare", 1.611605517789367], ["plan", 1.611556167758754], ["emory's", 1.6114404745964752], ["digitech", 1.6114404745964752], ["laid", 1.6112270266902964], ["instinct", 1.611222778293807], ["koto", 1.6110419067469748], ["drone", 1.6110281224311946], ["cobras", 1.611017638585391], ["slumping", 1.611017638585391], ["concierge", 1.610927037448811], ["b\u00e9la", 1.6108163780202134], ["topi", 1.6108163780202134], ["sisu", 1.6108163780202134], ["declared", 1.6107031232112632], ["olwen", 1.610379199257568], ["vibed", 1.6102032592161328], ["godrich", 1.6102032592161328], ["urbi", 1.6102032592161328], ["lolita's", 1.6102032592161328], ["tugendhat", 1.6102032592161328], ["dimmu", 1.6102032592161328], ["chitra", 1.610125396389285], ["nests", 1.6099978345015826], ["noise", 1.609829469394243], ["spot", 1.6098073578466061], ["parenting", 1.6097513711097093], ["glenfinnan", 1.6097120440668904], ["cushiony", 1.6096242160417888], ["kamilah", 1.6096242160417888], ["mispriced", 1.6096242160417888], ["tipos", 1.6096242160417888], ["kooper", 1.6096242160417888], ["bodies", 1.6095495435794018], ["initializes", 1.6095458545104107], ["pikachu's", 1.6095458545104107], ["elucidate", 1.6095458545104102], ["grayish", 1.6094924108185065], ["estados", 1.6094889951673208], ["arachne", 1.6094889951673208], ["tolle", 1.6094120045432994], ["keiser", 1.6094120045432994], ["emden", 1.6094120045432994], ["konnichiwa", 1.6093275475741275], ["pretzels", 1.609228180986494], ["ducked", 1.6091161567483465], ["midwestern", 1.609005587370435], ["whistleblowers", 1.6089142669125647], ["africanism", 1.6089088836058307], ["panthera", 1.6089088836058307], ["backsplash", 1.6089088836058307], ["skadi", 1.6088322451756685], ["tyto", 1.6088322451756685], ["holmium", 1.6088322451756685], ["ardor", 1.6087995951047367], ["biosecurity", 1.6087995951047367], ["minerals", 1.6086831041789231], ["considered", 1.6086710044328743], ["scarf", 1.6086210183023748], ["workbook", 1.6085595617442827], ["violators", 1.6085425220989216], ["byzantines", 1.6085195866066861], ["aberdare", 1.6084870551154011], ["auntie", 1.6084228867488406], ["stuck", 1.608362022807559], ["linnaeus", 1.6083620228075588], ["outdoorsmen", 1.6081701742928802], ["luis's", 1.6081701742928802], ["nibbed", 1.6081701742928802], ["tippa", 1.6081701742928802], ["opies", 1.6081701742928802], ["joint's", 1.6081701742928802], ["heptagonal", 1.6081701742928802], ["groundskeeping", 1.6081701742928802], ["songlines", 1.6081701742928802], ["pyrrolizidine", 1.6081701742928802], ["precompiled", 1.6081701742928802], ["sulu's", 1.6081701742928802], ["belowdecks", 1.6081701742928802], ["burde", 1.6081701742928802], ["webbrowser", 1.6081701742928802], ["chatrier", 1.6081701742928802], ["franglais", 1.6081701742928802], ["wash's", 1.6081701742928802], ["cigale", 1.6081701742928802], ["sunidhi", 1.6081701742928802], ["perkinson", 1.6081701742928802], ["nemechek", 1.6081701742928802], ["claudius", 1.6079542038145465], ["lifters", 1.6078925445692214], ["earthenware", 1.6078925445692214], ["package", 1.60755275181525], ["punnet", 1.6075230233173097], ["cubists", 1.6075230233173097], ["rapturous", 1.607455034693417], ["ashdod", 1.6074290820466623], ["toro's", 1.6073710835230453], ["threw", 1.607337371881193], ["tequilas", 1.6072400585182747], ["spyros", 1.6072400585182747], ["tamika", 1.607125907945937], ["announced", 1.6070789159603243], ["salad", 1.6069579365931093], ["clocks", 1.606888474755474], ["cyberwarfare", 1.6068573823138657], ["nonnegative", 1.6068573823138657], ["goneril", 1.6068573823138657], ["greenhills", 1.6068573823138657], ["daejeon", 1.6067862068451237], ["zaragoza", 1.6067343080658332], ["veterinarians", 1.606689899832007], ["droppings", 1.6065713882972574], ["productive", 1.6065620041185873], ["coiling", 1.6065620041185873], ["mikie", 1.6065173786358473], ["biodegradability", 1.6065173786358473], ["lahori", 1.6065173786358473], ["stand's", 1.6065173786358473], ["midfoot", 1.6065173786358473], ["overanalyze", 1.6065173786358473], ["sta\u00ebl", 1.6065173786358473], ["rosario's", 1.6065173786358473], ["parallelized", 1.6065173786358473], ["sultans", 1.6064568695067087], ["thurgood", 1.6063395878679594], ["replayed", 1.6062769243706883], ["homeschooler", 1.6062715082075285], ["stepfamily", 1.6062715082075285], ["petaled", 1.6062715082075285], ["trunchbull", 1.6062715082075285], ["sanfrancisco", 1.6062715082075285], ["basque", 1.6061922923377423], ["b\u00fcrger", 1.6061189368256699], ["balloonists", 1.6061189368256699], ["tiktaalik", 1.6061189368256699], ["familiares", 1.6061189368256699], ["manitoban", 1.6061189368256699], ["semantically", 1.6059981002912378], ["automobiles", 1.6059397024838749], ["angellist", 1.6059397024838749], ["extrusive", 1.6059397024838749], ["g\u00f6del's", 1.6059397024838749], ["profesor", 1.6057720678593985], ["o'connell", 1.6057667551312995], ["blackboard", 1.6056758695327527], ["faire", 1.6055731292909166], ["terrapins", 1.605467367277262], ["tycho's", 1.605467367277262], ["invader's", 1.605467367277262], ["tswift", 1.605467367277262], ["kippy", 1.605467367277262], ["aalia", 1.605467367277262], ["alexa", 1.605268943680647], ["chocolate", 1.6050162871549467], ["drowning", 1.6049717761776239], ["alhaji", 1.6049247963521935], ["reshuffling", 1.6048881943453437], ["alvarez", 1.6048568128637737], ["evms", 1.6047412400193346], ["playgroups", 1.6047412400193346], ["sdlc", 1.6047412400193346], ["disproportionately", 1.6047011214883342], ["rumble", 1.6044636110658854], ["emperors", 1.604341197687421], ["repeating", 1.6043389844061462], ["mistreatment", 1.6043211506894868], ["subverting", 1.6040284080492284], ["tarik", 1.603964433399012], ["auspiciousness", 1.6038025203703032], ["aukerman", 1.6038025203703032], ["laughable", 1.6037275471321089], ["vi's", 1.6036874699742936], ["displace", 1.6035901765525882], ["interethnic", 1.6035808016282873], ["than", 1.6035165892856693], ["devops", 1.603302831150347], ["evan's", 1.6032219583719791], ["networks", 1.6030865968422279], ["upwards", 1.6030521543177538], ["homes", 1.6030427839062718], ["moreland's", 1.6030075092068676], ["ngubane", 1.6030075092068676], ["iacuc", 1.6030075092068676], ["suitemates", 1.6030075092068676], ["infrastructure's", 1.6030075092068676], ["rickrolling", 1.6030075092068676], ["eukarya", 1.6030075092068676], ["noum\u00e9a", 1.6030075092068676], ["stilettos", 1.6029708271128353], ["highlights", 1.6026973438568048], ["kensington's", 1.6026740356247569], ["chachapoyas", 1.6026740356247569], ["probl\u00e8mes", 1.6026740356247569], ["runde", 1.6026740356247569], ["xiaomei", 1.6026740356247569], ["mercados", 1.6026740356247569], ["continuously", 1.6026594028182706], ["world", 1.602566861770647], ["punjabi", 1.6024980075697586], ["sistemi", 1.6024267093846916], ["bhutia", 1.6024267093846916], ["rojak", 1.6024267093846916], ["shudras", 1.6024267093846916], ["bakhtiar", 1.6024267093846916], ["whistleblowing", 1.6024267093846913], ["robot's", 1.6024267093846913], ["bumbling", 1.6023447899169043], ["otherwise", 1.602124326172924], ["dislodged", 1.6021171813357469], ["figc", 1.6020843836616867], ["borgir", 1.6020843836616867], ["supervisor", 1.6020438117241456], ["webpage", 1.6019301128555117], ["akka", 1.6018586743937717], ["problema", 1.6018586743937717], ["aristarchus", 1.6018137704419202], ["upstarts", 1.6017723905976822], ["odysseys", 1.6016349406993173], ["ocelots", 1.6016349406993173], ["information", 1.601590426732186], ["fam\u00edlia", 1.6014868696724422], ["dimensions", 1.6012451966035781], ["giulietta", 1.6012083986465908], ["oceanographers", 1.6012083986465908], ["coop", 1.6011796847081274], ["melodically", 1.6011012944058785], ["import", 1.601048277806726], ["hoover's", 1.6010296503529697], ["hunger", 1.6008513590098739], ["opening", 1.600800603246506], ["warum", 1.6007592518307012], ["basant", 1.6007592518307012], ["philosophized", 1.600759251830701], ["horseshoeing", 1.600759251830701], ["clangor", 1.600759251830701], ["wszystko", 1.600759251830701], ["fellowship's", 1.600759251830701], ["mahto", 1.600759251830701], ["gerberas", 1.600759251830701], ["lumper", 1.600759251830701], ["cassowaries", 1.6007592518307008], ["concordia's", 1.6007592518307008], ["naturaleza", 1.6007592518307008], ["gemeinde", 1.6007592518307008], ["friedl\u00e4nder", 1.6007592518307008], ["h\u00e9lio", 1.6007592518307008], ["careem", 1.6007592518307008], ["lagosians", 1.6007592518307008], ["didion's", 1.6007592518307008], ["mgnrega", 1.6007592518307008], ["cyanotype", 1.6007592518307008], ["rahs", 1.6007592518307008], ["valeriu", 1.6007592518307008], ["shoveling", 1.6006265188910798], ["valais", 1.600512221081962], ["chamberlin", 1.6003965420304374], ["convert", 1.6003590620177433], ["kiosks", 1.600285654418917], ["horsehair", 1.6001792665945305], ["dove's", 1.600011233488724], ["juliette", 1.599965224424039], ["reel", 1.59981517147863], ["turkmenistan's", 1.5997061207439702], ["subhadra", 1.5997061207439702], ["hermanas", 1.5997061207439702], ["amylase", 1.5993860514921208], ["maratha", 1.5993860514921208], ["switzerland's", 1.5992246006305681], ["kingfishers", 1.59908545413392], ["unconditionally", 1.599075694686534], ["olives", 1.5989190933157615], ["imperialism", 1.5988083264925324], ["reminiscences", 1.5986529896572397], ["lidar", 1.5986105816940155], ["mar\u00eda", 1.5983652591996147], ["fishnets", 1.5981697883350927], ["curses", 1.5981697883350925], ["breda", 1.59813792117814], ["dispersers", 1.5980461005564117], ["nauset", 1.5980461005564117], ["guiltier", 1.5980461005564117], ["sitemap", 1.5980461005564117], ["campervans", 1.5980461005564117], ["bayani", 1.5980461005564117], ["burgenland", 1.5980461005564117], ["istoria", 1.5980461005564117], ["ast\u00e9rix", 1.5980461005564117], ["uproot", 1.5979476138005533], ["geocoding", 1.5979476138005533], ["medusa's", 1.59782592335876], ["pasilla", 1.5977275108234459], ["insatiability", 1.5977275108234459], ["punga", 1.5977275108234459], ["sangrias", 1.5977275108234459], ["aiguilles", 1.5977275108234459], ["strahinja", 1.5977275108234459], ["osi's", 1.5977275108234459], ["aronnax", 1.5977275108234459], ["blasket", 1.5977275108234459], ["counseling", 1.5975773302508698], ["magma", 1.5975577333270394], ["segregating", 1.5972575909673215], ["langley", 1.5970677049846425], ["flattery", 1.5969855321032493], ["steamships", 1.5969756390536467], ["rewire", 1.596927647238553], ["denominations", 1.5968599664736776], ["jolts", 1.5968599664736773], ["muster", 1.5967269210157284], ["shortwave", 1.5966832444765027], ["interlaced", 1.5966485407610453], ["swells", 1.5966272106725201], ["julio's", 1.596615274477589], ["chainlink", 1.596615274477589], ["initiative's", 1.5965833581302733], ["ovule", 1.5965833581302733], ["aaron's", 1.596583358130273], ["particles", 1.5965339143888897], ["honeydew", 1.5964544211733873], ["erol", 1.596416106573796], ["plantations", 1.596396071231093], ["herd", 1.5963361012664319], ["clueless", 1.5962761127868084], ["fleetwood", 1.5962237672838473], ["lyrical", 1.5961622759123395], ["bah\u00eda", 1.596078669223009], ["ludacris", 1.5960187113466293], ["lightly", 1.5957612672149246], ["koby", 1.5956121159727243], ["trackball", 1.5956121159727243], ["malai", 1.5955544710263383], ["macbeth", 1.5955510399108692], ["collisions", 1.5955486171232114], ["extroversion", 1.595468280745924], ["aila", 1.595468280745924], ["rentals", 1.5953118011448133], ["mavens", 1.5953103738933045], ["rajshahi", 1.595245729723362], ["allama", 1.595245729723362], ["cbgb", 1.595245729723362], ["transnistria", 1.5950422545027345], ["expectancies", 1.5950422545027343], ["dyers", 1.5950422545027343], ["mindy's", 1.5950422545027343], ["datatypes", 1.5950422545027343], ["galina", 1.5950422545027343], ["raynaud's", 1.5950422545027343], ["realidad", 1.5945866583146444], ["bilgi", 1.5945866583146444], ["touching", 1.5945297911277936], ["pennies", 1.5944884203865874], ["slopes", 1.594467775030556], ["throwaway", 1.5944404653103166], ["piazza", 1.594323767000706], ["highlight", 1.5943092789811175], ["breakfasts", 1.5942649434625282], ["tasteful", 1.5941561199168996], ["turnouts", 1.5941561199168994], ["damaged", 1.594080165523828], ["roleplayer", 1.5940521599934216], ["punter", 1.5939345862704406], ["giving", 1.5938921649338846], ["eske", 1.5938115120224081], ["mirasol", 1.5938115120224081], ["freediver", 1.5938115120224081], ["endlich", 1.5938115120224081], ["othered", 1.5938115120224081], ["unclogged", 1.5938115120224081], ["chipotles", 1.5938115120224081], ["ferdowsi", 1.5938115120224081], ["huldah", 1.5938115120224081], ["deliberate", 1.5938089053061109], ["yixing", 1.5937030082772912], ["coherent", 1.5935895748538247], ["hosted", 1.5935288561089491], ["assigned", 1.5934725303792552], ["s\u00e9ance", 1.5933047357069796], ["immune", 1.593257142800835], ["pickups", 1.5929874496660157], ["torchy's", 1.5929734578042873], ["itse", 1.5929734578042873], ["riddler's", 1.5929734578042873], ["kunnen", 1.5929734578042873], ["gamefish", 1.5929734578042873], ["lignans", 1.5929734578042873], ["kopi", 1.5928542678710431], ["kite", 1.592845639191369], ["richly", 1.5927554555715475], ["wird", 1.59272050449618], ["worldbank", 1.5926472961626401], ["sahib's", 1.5926472961626401], ["inang", 1.5926472961626401], ["judson's", 1.5926472961626401], ["huxtables", 1.5926472961626401], ["tisquantum", 1.5926472961626401], ["ieyasu's", 1.5926472961626401], ["equids", 1.5926472961626401], ["sris", 1.5926472961626401], ["netminders", 1.5926472961626401], ["mirabai", 1.5926472961626401], ["tranquila", 1.5926472961626401], ["leech's", 1.5926472961626401], ["lcfc", 1.5926472961626401], ["plushies", 1.59264729616264], ["herders", 1.5926089329855675], ["lichens", 1.5926011123929964], ["charro", 1.5925144537824445], ["vietnam's", 1.592464408434851], ["siobhan", 1.5923342031368553], ["multiplayer", 1.5923250454752318], ["multitudes", 1.592212283769529], ["pulsing", 1.592172182969666], ["catamarans", 1.5920884725499516], ["photographers", 1.5920272995509295], ["annapurna", 1.5919054949068585], ["vilifying", 1.5918393584093553], ["yerba", 1.5918052331846162], ["malagasy", 1.5916422177225527], ["neuropsychologist", 1.5915443548218073], ["professor", 1.591515572775734], ["tapu", 1.5913754906992799], ["volumes", 1.591329462757021], ["glide", 1.5913155202632419], ["inflate", 1.5908722093929233], ["raspberry", 1.5907798349759337], ["frontiers", 1.5907545065067592], ["strummer", 1.590497974575376], ["dully", 1.590497974575376], ["whiteside's", 1.590497974575376], ["rsync", 1.590497974575376], ["tamatoa", 1.590497974575376], ["forechecking", 1.590497974575376], ["prototype's", 1.590497974575376], ["smokier", 1.590497974575376], ["pinz\u00f3n", 1.590497974575376], ["gahan", 1.590497974575376], ["andromache", 1.590497974575376], ["theorbo", 1.590497974575376], ["m\u00e9tropole", 1.590497974575376], ["sera's", 1.590497974575376], ["zvenigorod", 1.5904979745753758], ["openstax", 1.5904979745753758], ["melchor", 1.5904979745753758], ["changeovers", 1.5904979745753758], ["gyrfalcon", 1.5904979745753758], ["fazenda", 1.5904979745753758], ["numbat", 1.5904979745753758], ["vista's", 1.5904979745753758], ["tomates", 1.5904979745753758], ["unrecognizably", 1.5904979745753758], ["kiara's", 1.5904979745753758], ["solanaceous", 1.5904979745753758], ["spiritedly", 1.5904979745753758], ["enzyme's", 1.5904979745753758], ["endospores", 1.5904979745753758], ["pandyas", 1.5904979745753758], ["lunisolar", 1.5904979745753758], ["dimos", 1.5904979745753758], ["decarbonizing", 1.5904979745753758], ["juani", 1.5904979745753758], ["phytohormones", 1.5904979745753758], ["murry's", 1.5904979745753758], ["rbge", 1.5904979745753758], ["ostrich's", 1.5904979745753758], ["sumati", 1.5904979745753758], ["vasanthi", 1.5904979745753758], ["edy's", 1.5904979745753758], ["grunt's", 1.5904979745753758], ["pichit", 1.5904979745753758], ["upturned", 1.5902844849143591], ["chalk", 1.5902318096082024], ["astrological", 1.5902279409633937], ["inconveniencing", 1.5902087931254532], ["pedals", 1.5901424514987064], ["disciplinarian", 1.5899786282979638], ["meghalaya", 1.5899495269979362], ["cookout", 1.5898803054590362], ["alderney", 1.5897910865866758], ["twilight's", 1.5897910865866758], ["relationally", 1.5897700578822063], ["conflagrations", 1.5897700578822063], ["quenya", 1.5897700578822063], ["baum's", 1.5897700578822063], ["alejandro's", 1.5896987293620215], ["telford's", 1.5896987293620215], ["suppertime", 1.5896987293620215], ["wrigleyville", 1.5896987293620215], ["havilland", 1.5896513216611532], ["rocky's", 1.5896513216611532], ["soak", 1.5895991673661565], ["molding", 1.5895539370678862], ["disturbing", 1.5895039133412667], ["transplanting", 1.5895039133412665], ["team", 1.589464150891902], ["uralic", 1.5893659475472086], ["mahout", 1.5893659475472086], ["avonlea", 1.5893659475472086], ["specific", 1.5891126062461196], ["projector", 1.5891117039671856], ["tongo", 1.5890027943407576], ["vizcaya", 1.5890027943407576], ["vegans", 1.5889870014995295], ["tenure", 1.5888570142678822], ["horror", 1.5887371928027791], ["leia's", 1.588435740572685], ["nurmi", 1.588435740572685], ["acclimatize", 1.588435740572685], ["eroica", 1.588435740572685], ["phrasal", 1.588416949188081], ["deceptive", 1.5884063608005918], ["guesses", 1.5882594959444922], ["plateaus", 1.5880617632357639], ["dramatic", 1.5880497216619827], ["irkutsk", 1.5879531778160556], ["stepsisters", 1.5879531778160556], ["swimming", 1.5877225851729013], ["balm", 1.5876743939014109], ["digitalis", 1.5874195913987785], ["reordering", 1.5874195913987785], ["clarinets", 1.5874195913987785], ["regurgitated", 1.5874195913987785], ["hotel's", 1.5873892739887059], ["soften", 1.5873030878142202], ["usain", 1.5872848473540115], ["buttoning", 1.5872848473540115], ["rainbow's", 1.587242093358231], ["purr", 1.5871584948422819], ["waterfowl", 1.587103235815413], ["draven", 1.5867991420763636], ["soundgarden", 1.5865692544788152], ["akila", 1.5861404458779091], ["suena", 1.5861404458779091], ["wooden's", 1.5861404458779091], ["interrogatives", 1.5861404458779091], ["pennington's", 1.5861404458779091], ["horseracing", 1.58606873970947], ["wallflower", 1.58606873970947], ["watersports", 1.58606873970947], ["increasing", 1.5860625107038986], ["crusty", 1.5860071567648102], ["accompanying", 1.58595562019036], ["cavers", 1.585795632737501], ["fira", 1.585795632737501], ["sargasso", 1.585795632737501], ["decrypting", 1.585795632737501], ["rays", 1.5857502637989334], ["standardization", 1.5855325174840187], ["choir's", 1.5854687319680232], ["argument", 1.5851994689657953], ["sniffle", 1.585196314660125], ["veles", 1.5850269741714311], ["ijtihad", 1.5850269741714311], ["kyi's", 1.5850269741714311], ["texcoco", 1.5850269741714311], ["chillwave", 1.5850269741714311], ["unflinching", 1.5850191383647565], ["ironworks", 1.58493425542477], ["imposters", 1.5848665463400373], ["predicted", 1.5848517067656334], ["gaddafi's", 1.5848354686973065], ["semester", 1.584822721054882], ["concierges", 1.5847836105948658], ["zlatko", 1.5847836105948658], ["extravehicular", 1.5847836105948658], ["narbonne", 1.5843659341914709], ["cursive", 1.5843282846991893], ["mncs", 1.5842649419723103], ["craven's", 1.584167634378061], ["toolboxes", 1.584167634378061], ["extractive", 1.5840846762907979], ["braver", 1.5838011409982167], ["tansu", 1.5837298980878212], ["sperms", 1.5837298980878212], ["symbioses", 1.5837298980878212], ["hiang", 1.5837298980878212], ["labra", 1.5837298980878212], ["tripa", 1.5837298980878212], ["mogis", 1.5837298980878212], ["andrius", 1.5837298980878212], ["distractibility", 1.5837298980878212], ["thanon", 1.5837298980878212], ["spaceball", 1.5837298980878212], ["wordlists", 1.5837298980878212], ["trepidatious", 1.5837298980878212], ["hurtfully", 1.5837298980878212], ["loremaster", 1.5837298980878212], ["handywork", 1.5837298980878212], ["stiegler", 1.5837298980878212], ["giovanna's", 1.5837298980878212], ["zojirushi", 1.5837298980878212], ["stubing", 1.5837298980878212], ["skynyrd's", 1.5837298980878212], ["wenonah", 1.5837298980878212], ["mundra", 1.5837298980878212], ["smug", 1.5835746304507539], ["adores", 1.5835395459366086], ["mccartney's", 1.5834419471972598], ["garish", 1.5833825888996442], ["oppressions", 1.583227126691603], ["slipperiness", 1.583227126691603], ["dodge's", 1.583227126691603], ["newspeak", 1.583227126691603], ["bonaparte", 1.5830756506940242], ["delegate", 1.5830502535207773], ["dowels", 1.5828197084912192], ["chic", 1.5827214688207716], ["rewilding", 1.5827094535658577], ["mispronounce", 1.5827094535658577], ["fashionistas", 1.5826198070377595], ["support", 1.5824858058404445], ["realism", 1.582383188990808], ["hyundai's", 1.5823153483396186], ["with", 1.5822243928147846], ["triathlons", 1.5820987156085171], ["coolers", 1.5820484540854443], ["clavichord", 1.5819583881347026], ["posie", 1.5819583881347026], ["resisting", 1.5817502357152116], ["vasudha", 1.5816266711315294], ["heathlands", 1.5816266711315294], ["bloomsday", 1.5816266711315294], ["torah's", 1.5816266711315294], ["kowalski", 1.5815688631391633], ["widower", 1.5814603786076955], ["mashed", 1.5813704637498538], ["nextgen", 1.581224562655818], ["night", 1.5811771875014724], ["zither", 1.5811421041366975], ["antivenom", 1.5811421041366975], ["icpr", 1.581142104136697], ["heutigen", 1.581142104136697], ["montignac", 1.581142104136697], ["muzungu", 1.581142104136697], ["stupors", 1.581142104136697], ["restitutions", 1.581142104136697], ["greyhound's", 1.581142104136697], ["espa\u00f1olas", 1.581142104136697], ["donn\u00e9", 1.581142104136697], ["repurposes", 1.581142104136697], ["azem", 1.581142104136697], ["omniture", 1.581142104136697], ["brocante", 1.581142104136697], ["crowdie", 1.581142104136697], ["jearim", 1.581142104136697], ["bson", 1.581142104136697], ["gonatas", 1.581142104136697], ["boryana", 1.581142104136697], ["patka", 1.581142104136697], ["photophone", 1.581142104136697], ["tulipe", 1.581142104136697], ["boyet", 1.581142104136697], ["yevon", 1.581142104136697], ["parmele", 1.581142104136697], ["dpes", 1.581142104136697], ["bethulia", 1.581142104136697], ["mrkh", 1.581142104136697], ["question's", 1.5810024642794036], ["mendoza's", 1.5810024642794036], ["giverny", 1.5810024642794036], ["malawi's", 1.5810024642794036], ["jukeboxes", 1.5810024642794036], ["cacophonous", 1.5810024642794036], ["mysticism", 1.5809549867279238], ["palazzo", 1.5808931249847982], ["serbia's", 1.5808887159789833], ["splayed", 1.5808887159789833], ["tenuous", 1.5808641011293767], ["gravitas", 1.5807497611828174], ["fribourg", 1.5806015427335736], ["aeneid", 1.5804756585712023], ["factory", 1.5804048088370002], ["adoptions", 1.5802956880213674], ["slight", 1.580031348145098], ["ruby", 1.580029000472569], ["buzzcut", 1.579872518377795], ["saphira", 1.579872518377795], ["joy's", 1.5798513758884225], ["keyblade", 1.5796966300960866], ["adad", 1.5796966300960866], ["kapoor", 1.5796825883482635], ["possible", 1.5792582240415098], ["handwriting", 1.5790823036288268], ["mahua", 1.5789307165784643], ["spa\u00df", 1.5789307165784643], ["mississippiensis", 1.5789307165784643], ["sphygmomanometer", 1.5789307165784643], ["lucescu", 1.5789307165784643], ["elodie", 1.5789307165784643], ["husks", 1.5789307165784643], ["kaftan", 1.5789307165784643], ["tecla", 1.5789307165784643], ["maybellene", 1.5789307165784643], ["pulcher", 1.5789307165784643], ["demel", 1.5789307165784643], ["jotuns", 1.5789307165784643], ["vandusen", 1.5789307165784643], ["delphinidae", 1.5789307165784643], ["epilation", 1.5789307165784643], ["tirer", 1.5789307165784643], ["anathem", 1.5789307165784643], ["parallelize", 1.5789307165784643], ["abera", 1.5789307165784643], ["shoma", 1.5789307165784643], ["espacios", 1.5789307165784643], ["objc", 1.5789307165784643], ["debraj", 1.5789307165784643], ["holmesian", 1.5789307165784643], ["vikernes", 1.5789307165784643], ["barleys", 1.5789307165784643], ["freda's", 1.5789307165784643], ["voorburg", 1.5789307165784643], ["siakam", 1.5789307165784643], ["algumas", 1.5789307165784643], ["dasheen", 1.5789307165784643], ["d\u017eeko", 1.5789307165784643], ["arabi's", 1.5789307165784643], ["contr\u00f4l\u00e9e", 1.5789307165784643], ["reservas", 1.5789307165784643], ["thabor", 1.5789307165784643], ["lafortune", 1.5789307165784643], ["morshed", 1.5789307165784643], ["baudrillard's", 1.5789307165784643], ["sandinista", 1.578930716578464], ["sans", 1.5788119999832328], ["offsetting", 1.578778515815347], ["labors", 1.578581550121901], ["akio", 1.5784272310183307], ["omnipresent", 1.578382349809567], ["rodent", 1.578216914265364], ["beanbags", 1.578078468980455], ["belgorod", 1.578078468980455], ["overlapped", 1.577995552685679], ["qualcomm's", 1.577941411618202], ["unfettered", 1.5779133643126277], ["saut\u00e9", 1.5779085344140844], ["shortbread", 1.5779085344140844], ["anonymity", 1.5778259254065146], ["seller", 1.5776149409813154], ["lepidoptera", 1.5775383614492064], ["fingerless", 1.5775383614492064], ["encores", 1.5775383614492064], ["dre's", 1.5774369316053531], ["bicycle", 1.5773010049936635], ["pomegranates", 1.5772536791737075], ["sarcophagi", 1.577182177578931], ["phages", 1.577182177578931], ["brightman", 1.577182177578931], ["acclimated", 1.5770191781806695], ["hominem", 1.5770191781806695], ["verdean", 1.5770191781806695], ["magnavox", 1.5770191781806695], ["tharpe", 1.5770191781806695], ["chickadee", 1.5768925099735867], ["painters", 1.5768801572062845], ["downsized", 1.5768531761619133], ["mulhouse", 1.5768226648874382], ["quaintness", 1.5767670568092576], ["vit\u00f3ria", 1.5767670568092576], ["gachet", 1.5767670568092576], ["prospero", 1.576655600272155], ["headstand", 1.5765420165533832], ["seit", 1.5765420165533832], ["kylian", 1.5765420165533832], ["tatanka", 1.5765420165533832], ["integrated", 1.5765015923991128], ["anastasius", 1.5764383239713948], ["phong", 1.5762770515085844], ["comfortable", 1.576198806132618], ["smith's", 1.5757705357465315], ["intermolecular", 1.575689889984502], ["abruzzo", 1.575689889984502], ["powerlifters", 1.5755472936143675], ["nclex", 1.5755472936143675], ["huguenot", 1.5755001229520036], ["edges", 1.5754464327671993], ["lad's", 1.5753503748175155], ["haystacks", 1.5753503748175155], ["cardsharp", 1.5753503748175153], ["draftsperson", 1.5753503748175153], ["subordinate's", 1.5753503748175153], ["categoryid", 1.5753503748175153], ["est\u00e9vez", 1.5753503748175153], ["oumuamua", 1.5753503748175153], ["gestapo's", 1.5753503748175153], ["stiers", 1.5753503748175153], ["hakkas", 1.5753503748175153], ["materie", 1.5753503748175153], ["ebin", 1.5753503748175153], ["phyu", 1.5753503748175153], ["sharpening", 1.5750607883515564], ["instrumentalists", 1.5750607883515562], ["steeps", 1.575007907344729], ["deloris", 1.575007907344729], ["instigate", 1.5750079073447287], ["dwarfing", 1.574931398654], ["obsidian", 1.5747536511906903], ["schoolhouse", 1.5745929948296222], ["stressing", 1.574561395532114], ["cochineal", 1.5745441566113654], ["karin's", 1.5744885857722102], ["liveability", 1.5744885857722102], ["gatun", 1.5744885857722102], ["jola", 1.5744885857722102], ["ugliness", 1.5744431049681442], ["aberdeen's", 1.5744247870532275], ["danlos", 1.5744247870532275], ["muses", 1.5744194859596685], ["glazed", 1.5742443925338248], ["cnidaria", 1.5741605331446924], ["chimes", 1.5739291348054516], ["bergman's", 1.5737558906324773], ["abandonment", 1.5737185377758716], ["burano", 1.5736077305975844], ["desegregating", 1.5736077305975844], ["velha", 1.5736077305975844], ["sveti", 1.5736077305975844], ["springboards", 1.5736077305975844], ["shinjiro", 1.5736077305975844], ["flaubert's", 1.5736077305975844], ["valerian", 1.5735884697930484], ["rahul", 1.5732178044411247], ["catchup", 1.5732099531126003], ["fife's", 1.573080417505098], ["du's", 1.573080417505098], ["kato's", 1.573080417505098], ["leto's", 1.572888889856863], ["unscramble", 1.572888889856863], ["potlatch", 1.572888889856863], ["cholerae", 1.5728537815135255], ["camden's", 1.5728537815135255], ["thimbles", 1.5728537815135255], ["vais", 1.5728537815135255], ["beavers", 1.5728503622661745], ["renting", 1.572602011753535], ["doong", 1.5725768706364984], ["iroko", 1.5725768706364984], ["kunsthaus", 1.5725768706364984], ["bolu", 1.5725768706364984], ["cranberries", 1.572350315351696], ["pedrito", 1.572095518671198], ["nimi", 1.572095518671198], ["psaltery", 1.572095518671198], ["congregationalism", 1.572095518671198], ["silk's", 1.572095518671198], ["miguel", 1.5719034600542792], ["parvati", 1.5718956590832662], ["were", 1.5718663148224694], ["tentative", 1.5718519238521627], ["foucault's", 1.5718450140358533], ["leutnant", 1.5718450140358533], ["dayton's", 1.5717171866837112], ["hurricane's", 1.5717171866837112], ["asks", 1.5717127475883912], ["arabella", 1.571706325907795], ["javier", 1.5715521783629764], ["apply", 1.5714568085757303], ["phytochemicals", 1.5714119988804716], ["linville", 1.5714119988804716], ["exaggerated", 1.5713256670488662], ["magnificently", 1.5712715814022409], ["quitting", 1.571164376441316], ["floral", 1.5710609551848944], ["chorro", 1.5710508377683408], ["davian", 1.5710508377683408], ["annelida", 1.5710508377683408], ["naismith's", 1.5710508377683408], ["carabiniere", 1.5710508377683408], ["spiegel's", 1.5710508377683408], ["cupping", 1.5709963913695273], ["fenny", 1.5709499547761256], ["upside", 1.5709254936176602], ["expansionist", 1.5707450158588754], ["cameras", 1.5706189286575325], ["negronis", 1.5703650888212575], ["persin", 1.5703650888212575], ["macrophotography", 1.5703650888212575], ["suivant", 1.5703650888212575], ["zoomie", 1.5703650888212575], ["mthembu", 1.5703650888212575], ["alaskas", 1.5703650888212575], ["profesionales", 1.5703650888212575], ["welwitschia", 1.5703650888212575], ["summitville", 1.5703650888212575], ["shorebreak", 1.5703650888212575], ["pleshette", 1.5703650888212575], ["dragonfly's", 1.5703650888212575], ["agrobiodiversity", 1.5703650888212575], ["twist's", 1.5703650888212575], ["cendol", 1.5703650888212575], ["glorfindel", 1.5703650888212575], ["uniflora", 1.5703650888212575], ["sigillata", 1.5703650888212575], ["minelaying", 1.5703650888212575], ["follett's", 1.5703650888212575], ["vanity's", 1.5703650888212575], ["potholes", 1.5700513848669686], ["perpetually", 1.5699860351791282], ["pears", 1.5699860351791282], ["earthlings", 1.5699481777357294], ["sultanate", 1.569908255340873], ["ploys", 1.5698804082382878], ["popups", 1.5698804082382878], ["bodyboard", 1.5698804082382878], ["volatilities", 1.5698804082382878], ["defragmentation", 1.5698804082382878], ["pramila", 1.5698804082382878], ["ohia", 1.5697241234788815], ["am\u00e9ricain", 1.5697241234788815], ["quippy", 1.5697241234788815], ["dreamville", 1.5697241234788815], ["amable", 1.5697241234788815], ["proxying", 1.5697241234788815], ["swankiest", 1.5697241234788815], ["youssou", 1.5697241234788815], ["astyanax", 1.5697241234788815], ["trainline", 1.5697241234788815], ["santuario", 1.5697241234788815], ["turm", 1.5697241234788815], ["listentothis", 1.5697241234788815], ["eberron", 1.5697241234788815], ["ipfs", 1.5697241234788815], ["convivial", 1.5697241234788812], ["detaching", 1.5697241234788812], ["icty", 1.5695323194559736], ["campgrounds", 1.569512698585154], ["palau", 1.569512698585154], ["feynman", 1.569512698585154], ["programs", 1.5694224742100078], ["llah", 1.5694190857557024], ["fmnr", 1.5694190857557024], ["senlac", 1.5694190857557024], ["lupita's", 1.5694190857557024], ["jeszcze", 1.5694190857557024], ["periodo", 1.5694190857557024], ["unvalidated", 1.5694190857557024], ["cas's", 1.5694190857557024], ["norrbotten", 1.5694190857557024], ["childeric", 1.5694190857557024], ["gorguts", 1.5694190857557024], ["ninhydrin", 1.5694190857557024], ["agriculture's", 1.5692913349143711], ["jung's", 1.5692913349143711], ["mon\u00e1e", 1.5692913349143711], ["jamison", 1.569123693494126], ["akihiko", 1.569123693494126], ["gratified", 1.56907327178161], ["coolness", 1.569034673617415], ["paralympians", 1.5690185807065125], ["adah", 1.5690185807065125], ["moyo", 1.5689794725664152], ["gabriela", 1.568979472566415], ["metrocard", 1.568979472566415], ["nevsky", 1.568979472566415], ["jiggles", 1.568979472566415], ["robin's", 1.568921647059742], ["yakiniku", 1.5688940208223614], ["antonie", 1.5688940208223614], ["recursos", 1.5688940208223614], ["salish", 1.568744066794087], ["daimyo", 1.568744066794087], ["duisburg", 1.568744066794087], ["ringan", 1.5685600714777845], ["honeysuckles", 1.5685600714777845], ["magots", 1.5685600714777845], ["kalyan's", 1.5685600714777845], ["gaekwad", 1.5685600714777845], ["keam", 1.5685600714777845], ["skywards", 1.5683289590464484], ["mapreduce", 1.5683289590464484], ["elric", 1.5681595198762837], ["calendars", 1.5681037630933548], ["johnnie's", 1.5680299734475456], ["dakotan", 1.5680299734475456], ["lukashenko's", 1.5680299734475456], ["cartographic", 1.5679659199355582], ["frog's", 1.567776574938585], ["aqis", 1.5677765749385848], ["bo\u00eete", 1.5677765749385848], ["fiddlin", 1.5677765749385848], ["boubou", 1.5677765749385848], ["grettir", 1.5677765749385848], ["importants", 1.5677765749385848], ["gallbladders", 1.5677765749385848], ["clowny", 1.5677765749385848], ["surdas", 1.5677765749385848], ["flagged", 1.567763209409387], ["leaves", 1.5676654352560828], ["digester", 1.567652640031475], ["algeria's", 1.5675652130814983], ["gotland", 1.567500233591651], ["unconscious", 1.567471362107643], ["sundanese", 1.5674101007508947], ["honeypots", 1.5674101007508947], ["gibbon's", 1.5674101007508947], ["soapy", 1.5672914204979418], ["qajar", 1.5672914204979418], ["linework", 1.5672914204979418], ["shimmied", 1.5672914204979418], ["fortify", 1.5671412914726695], ["cyberterrorism", 1.5670590570553178], ["putonghua", 1.5670590570553178], ["ephialtes", 1.5670590570553178], ["lambrusco", 1.5670590570553178], ["perito", 1.5670590570553178], ["jignesh", 1.5670590570553178], ["redistricting", 1.5669555798328498], ["langdon", 1.5669253945632455], ["hossein", 1.5667382013925302], ["hierarchy", 1.566474333526566], ["late", 1.5664609747744098], ["monitor", 1.5664284385601355], ["marlowe", 1.5663878090168597], ["stencils", 1.5657912099217781], ["farrah's", 1.5657912099217781], ["undistracted", 1.5657912099217781], ["younger's", 1.5657912099217781], ["solarwinds", 1.5657912099217781], ["longlines", 1.5657912099217781], ["transmat", 1.5657912099217781], ["machine", 1.5657507074752512], ["locating", 1.5657240470160048], ["cappy", 1.565708144075098], ["devotee", 1.5656779826807097], ["beanie", 1.5656556008885592], ["swachh", 1.5656464437226358], ["batter", 1.5656020806592954], ["aurelia", 1.5655876401003364], ["thing's", 1.565417044976303], ["microcontroller", 1.5653970247220772], ["parton's", 1.5652283805867953], ["frappes", 1.5652283805867953], ["itms", 1.5652283805867953], ["sayaji", 1.5652283805867953], ["kanina", 1.5652283805867953], ["phytoestrogen", 1.5652283805867953], ["ghoda", 1.5652283805867953], ["homerun", 1.5651132055771864], ["dell's", 1.56505000698217], ["sondra", 1.56505000698217], ["tutsi", 1.5648669280786218], ["fancier", 1.5647622787053623], ["imgur", 1.564403024755224], ["ida's", 1.564338468414597], ["cult's", 1.564338468414597], ["euler", 1.5643076027412013], ["panning", 1.5642201819519568], ["phew", 1.5642201819519568], ["alongs", 1.5642201819519568], ["tupac", 1.5640507412389812], ["undivided", 1.5640275439985143], ["playlist", 1.563994885475695], ["xochimilco", 1.5639896749991196], ["sarva", 1.5639896749991196], ["fluorescents", 1.5639896749991196], ["junho", 1.5639896749991196], ["voynich", 1.5639896749991196], ["mahershala", 1.5639896749991196], ["ishan", 1.5639896749991196], ["soundly", 1.56369987365327], ["terrifyingly", 1.5636583212544166], ["perturbation", 1.5636300438013677], ["combinatorial", 1.5636300438013677], ["hannah's", 1.5635940287367875], ["existentialists", 1.5635940287367875], ["mesosphere", 1.5635940287367875], ["mugwort", 1.5635940287367875], ["jaipur", 1.563535247006384], ["health", 1.5633805046354756], ["shopaholic", 1.5633431311070158], ["mara's", 1.5633431311070158], ["cao's", 1.5633431311070158], ["heinous", 1.5632808013700021], ["stimulation", 1.5632404583883417], ["seas", 1.5631997214418296], ["appendages", 1.5630234453790661], ["boast", 1.5629882503484613], ["profession", 1.5627738276585557], ["timur", 1.5626290720120197], ["ultralight", 1.5625308289821715], ["generation's", 1.5622969245885776], ["clich\u00e9s", 1.5622969245885776], ["roguish", 1.5622224550273693], ["sahana", 1.5622224550273693], ["madrasahs", 1.5622224550273693], ["intellects", 1.5622224550273693], ["neuroengineering", 1.5622224550273693], ["vissi", 1.5622224550273693], ["proboscidea", 1.5622224550273693], ["conditioner's", 1.5622224550273693], ["columnist's", 1.5622224550273693], ["celebrator", 1.5622224550273693], ["remdesivir", 1.5622224550273693], ["bunna", 1.5622224550273693], ["sernin", 1.5622224550273693], ["crystalis", 1.5622224550273693], ["tumon", 1.5622224550273693], ["tali's", 1.5622224550273693], ["dioses", 1.5622224550273693], ["ulen", 1.5622224550273693], ["bhonsle", 1.5622224550273693], ["editing", 1.5622076331824448], ["recidivism", 1.5621286841717255], ["warsaw's", 1.5618462198644993], ["imf's", 1.5618122075339274], ["back", 1.56180946857248], ["briefcases", 1.5616818936242458], ["moderna", 1.5616818936242458], ["compensator", 1.5614302529710675], ["icefield", 1.5613813068485956], ["ragnar's", 1.5613813068485956], ["quagmires", 1.5613813068485956], ["industry", 1.561284528680556], ["rickshaw", 1.5612489863988628], ["preening", 1.5611232272937445], ["gunung", 1.5610737620457316], ["bouncing", 1.5610218111068634], ["crawlers", 1.5610052969011308], ["bonasera", 1.5610052969011308], ["cooldowns", 1.5610052969011308], ["laysan", 1.5610052969011308], ["contrario", 1.5610052969011308], ["crossbreeds", 1.5610052969011308], ["manzarek", 1.5610052969011308], ["mortal's", 1.5610052969011308], ["valencians", 1.5610052969011308], ["heartstring", 1.5610052969011308], ["mosul's", 1.5610052969011308], ["calypso's", 1.5610052969011308], ["apparmor", 1.5610052969011308], ["xiang's", 1.5610052969011308], ["hydroplaned", 1.5610052969011308], ["okemo", 1.5610052969011308], ["minjung", 1.5610052969011308], ["hollygrove", 1.5610052969011308], ["pajitnov", 1.5610052969011308], ["brahmagupta", 1.5610052969011308], ["dock's", 1.5610052969011308], ["fsms", 1.5610052969011308], ["sunitha", 1.5610052969011308], ["klis", 1.5610052969011308], ["lawmaking", 1.5609601550475478], ["career", 1.5609041089284434], ["undocumented", 1.5608857941032819], ["timekeeper", 1.5607402705349336], ["monsoon", 1.5605906668665384], ["correct", 1.560527527814269], ["reign", 1.5604905794550539], ["body's", 1.5604043798046334], ["stickers", 1.5603564150000557], ["buckwheat", 1.5602862464359541], ["trace", 1.5602600427593842], ["kursk", 1.5602160044044213], ["palette", 1.5602091095104624], ["mom's", 1.5601455612153678], ["brasileiro", 1.5601339223334827], ["mandrill", 1.560021366679321], ["forthrightness", 1.560021366679321], ["gazers", 1.5599114750643113], ["cerca", 1.5599114750643113], ["extricating", 1.559699314374061], ["halakhic", 1.559699314374061], ["participation", 1.5596051527009833], ["habe", 1.5595968710693402], ["congruent", 1.5595968710693402], ["inklings", 1.5595968710693402], ["echocardiogram", 1.5595968710693402], ["ceva", 1.5595968710693402], ["regattas", 1.5595968710693402], ["vili", 1.5594967428675508], ["patr\u00f3n", 1.5594377447688534], ["pontus", 1.5594312384364737], ["bailey's", 1.5593773950109289], ["scrubbing", 1.559308697220288], ["julianne", 1.5593031247977929], ["leonel", 1.5593031247977927], ["justice", 1.55920556619328], ["cyclical", 1.558863518308649], ["ostend", 1.5588545593743999], ["landesmuseum", 1.5588545593743999], ["shiflett", 1.5588545593743999], ["megumin", 1.5588545593743999], ["unpleasantries", 1.5588545593743999], ["vaughan's", 1.5588545593743999], ["mellowness", 1.5588545593743999], ["parthenos", 1.5588545593743999], ["equidae", 1.5588545593743999], ["ronen", 1.5588545593743999], ["okan", 1.5588545593743999], ["y\u0131lmaz", 1.5588545593743999], ["cyclists", 1.5588373271386247], ["pestle", 1.55877968643687], ["sang", 1.5587107364835053], ["hiragana", 1.5586880150838685], ["skyrockets", 1.5586880150838685], ["donc", 1.5586340083614723], ["hadamard", 1.5586340083614723], ["bhutto's", 1.5586340083614723], ["herbology", 1.5586340083614723], ["reptilian", 1.5584616096782349], ["apprehend", 1.5584153582882272], ["eyesight", 1.5582050366413274], ["alps", 1.5580833629613027], ["disrespectful", 1.5579746567245625], ["coppola's", 1.5579482485840659], ["compartments", 1.5578653437177088], ["tallgrass", 1.5577736592134621], ["ballerinas", 1.5577615114093588], ["railing", 1.557551945102029], ["supposedly", 1.557400967380758], ["bedside", 1.557381534604753], ["ptolemy", 1.5570138066895787], ["criollo", 1.5570138066895787], ["macy", 1.5568168409961327], ["chances", 1.5566704389494532], ["bjork's", 1.5566318580064706], ["bowerbird", 1.5566318580064706], ["miyo", 1.5566318580064706], ["tusker", 1.5564577965217334], ["placed", 1.5564478391230683], ["dishonors", 1.556374563484486], ["gamely", 1.556374563484486], ["webtoons", 1.556374563484486], ["avenue's", 1.556374563484486], ["eartha", 1.556374563484486], ["stovl", 1.5559922558049273], ["dabu", 1.5559922558049273], ["duane's", 1.5559922558049273], ["namak", 1.5559922558049273], ["pollute", 1.5558760849630722], ["faders", 1.5557218130734065], ["larson's", 1.5556584951322965], ["renge", 1.5555365639887346], ["speedboat", 1.555420526797732], ["sound", 1.5553459389143138], ["nets", 1.5551524808865151], ["jamil", 1.5551037942960928], ["instructing", 1.5548708199448877], ["likened", 1.5548708199448877], ["leanings", 1.5546416600352004], ["combat", 1.554597412028597], ["john", 1.5545865184319323], ["dehydrating", 1.554564501816451], ["crocodilian", 1.5545456862065958], ["rowdiness", 1.5545456862065958], ["gowri", 1.5545456862065958], ["carrara", 1.5545332448645697], ["hashes", 1.5545332448645697], ["sheeran's", 1.5544722373199336], ["grapes", 1.5543163670196725], ["mordechai", 1.5543004344643503], ["moats", 1.55422032414405], ["thalassa", 1.5542047322025148], ["singalong", 1.5541437351565102], ["proteus", 1.5541189481750883], ["decathlon", 1.5540704403189725], ["ethos", 1.554000231579857], ["sabar", 1.5539729688961919], ["vodafone's", 1.5539729688961919], ["fahri", 1.5539729688961919], ["achievement", 1.5539318010533478], ["contribution", 1.553737862483546], ["unrelenting", 1.5537177089133203], ["antiquarian", 1.5536161484776374], ["aminah", 1.5534324565591848], ["evasiveness", 1.5534324565591848], ["boudin", 1.5534324565591848], ["discus", 1.5534324565591846], ["utm_medium", 1.5531931092446065], ["calvin's", 1.5529351500694468], ["collard", 1.5528646815385125], ["muted", 1.552858205017364], ["inflatable", 1.5528264045708269], ["festers", 1.5527786550202287], ["mahavira", 1.5527786550202287], ["rocks", 1.5526997902095923], ["kauri", 1.5525159333914864], ["greasy", 1.5524952250977557], ["structure", 1.5524565727303008], ["thinners", 1.5523260231855667], ["balancer", 1.5523260231855667], ["marmot", 1.551971171047813], ["kala", 1.5516957329665402], ["mirage", 1.5516898239957368], ["extinctions", 1.5516725255283397], ["overstepped", 1.5516504134264553], ["telugu", 1.5515737606300246], ["sauna", 1.5514324849976036], ["internally", 1.5511770157519769], ["drummers", 1.5511441511728392], ["reformation", 1.5509185028540842], ["trackers", 1.5507355252109918], ["hoplite", 1.5507355252109918], ["rastafari", 1.5507355252109918], ["esport", 1.5507355252109918], ["strays", 1.5507355252109918], ["dido's", 1.5507355252109918], ["thomson's", 1.5507355252109918], ["brasseries", 1.5507355252109918], ["boudreaux", 1.5507355252109918], ["marwah", 1.5507355252109918], ["breadmaker", 1.5507355252109918], ["funafuti", 1.5507355252109918], ["disseminators", 1.5507355252109916], ["m\u00e9dias", 1.5507355252109916], ["esas", 1.5507355252109916], ["posty", 1.5507355252109916], ["p\u00e9pin", 1.5507355252109916], ["arboles", 1.5507355252109916], ["heartedness", 1.5507355252109916], ["hardangervidda", 1.5507355252109916], ["kannadiga", 1.5507355252109916], ["perches", 1.5507355252109916], ["matobo", 1.5507355252109916], ["accuracies", 1.5507355252109916], ["charu", 1.5507355252109916], ["zorilla", 1.5507355252109916], ["inoue's", 1.5507355252109916], ["battlecruisers", 1.5507355252109916], ["p\u00e2te", 1.5507355252109916], ["euroskeptic", 1.5507355252109916], ["mitochondrion", 1.5507355252109916], ["greenbelts", 1.5507355252109916], ["prueba", 1.5507355252109916], ["stubai", 1.5507355252109916], ["epicures", 1.5507355252109916], ["constantine's", 1.5507355252109916], ["socialbakers", 1.5507355252109916], ["ranji", 1.5507355252109916], ["greenworks", 1.5507355252109916], ["basket's", 1.5507355252109916], ["tantas", 1.5507355252109916], ["quaternions", 1.5507355252109916], ["otowa", 1.5507355252109916], ["y\u0131ld\u0131z", 1.5507355252109916], ["callias", 1.5507355252109916], ["ritual's", 1.5507355252109916], ["liliuokalani", 1.5507355252109916], ["humains", 1.5507355252109916], ["irukandji", 1.5507355252109916], ["idei", 1.5507355252109916], ["grimani", 1.5507355252109916], ["mesocyclone", 1.5507355252109916], ["codington", 1.5507355252109916], ["kr\u00e1lov\u00e9", 1.5507355252109916], ["counterpunching", 1.5507355252109916], ["jitsi", 1.5507355252109916], ["krupke", 1.5507355252109916], ["mtukudzi", 1.5507355252109916], ["deewar", 1.5507355252109916], ["animais", 1.5507355252109916], ["hodrick", 1.5507355252109916], ["cedrik", 1.5507355252109916], ["haboob", 1.5507355252109916], ["nanosatellite", 1.5507355252109916], ["jeyne", 1.5507355252109916], ["elands", 1.5507355252109916], ["harnam", 1.5507355252109916], ["gardos", 1.5507355252109916], ["nashiri", 1.5507355252109916], ["cupit", 1.5507355252109916], ["norteno", 1.5507355252109916], ["soins", 1.5507355252109916], ["akinator", 1.5507355252109916], ["appaji", 1.5507355252109916], ["tildes", 1.5507355252109916], ["karez", 1.5507355252109916], ["sleepwell", 1.5507355252109916], ["hakman", 1.5507355252109916], ["zeppeli", 1.5507355252109916], ["hormisdas", 1.5507355252109916], ["chisako", 1.5507355252109916], ["mrunal", 1.5507355252109916], ["kiwis", 1.5503169731124193], ["pivoting", 1.549991767165327], ["mukesh", 1.5499382318818342], ["films", 1.5499122514310169], ["hollow", 1.5495520233511224], ["spiral", 1.5495089547390732], ["else", 1.549484598553988], ["intelligences", 1.549469618659799], ["impossible", 1.54941881444995], ["luau", 1.5493811710405017], ["predominantly", 1.5492609747908226], ["undiluted", 1.5492515677705982], ["towels", 1.5492090741347213], ["flowers", 1.5491457551531094], ["rabbi", 1.5487100747290834], ["aotearoa", 1.5485359003525363], ["stilled", 1.5485359003525363], ["name's", 1.5484621956452373], ["pheromones", 1.5483894805435015], ["duckie", 1.5483312840866335], ["would", 1.5482483455234595], ["pranced", 1.548222177520212], ["crustacean", 1.5481723259957667], ["platformer", 1.5481723259957667], ["mura", 1.548084695253366], ["glycolysis", 1.548084695253366], ["migration", 1.5480385938627985], ["cleats", 1.5478751434780595], ["quadratus", 1.5478477495588299], ["diversified", 1.5478032253087746], ["invasively", 1.5477126489435238], ["bleue", 1.5477126489435238], ["rastafarians", 1.5477126489435238], ["koechlin", 1.5477126489435238], ["foreignness", 1.5477126489435238], ["gordy", 1.5476402447215485], ["unsolicited", 1.5475947950333493], ["mario's", 1.5474708188421264], ["schnauzer", 1.5474006101030109], ["khun", 1.5474006101030109], ["wrangling", 1.5473422308451033], ["pavers", 1.547327315265473], ["printer's", 1.5471706159576333], ["problems", 1.5471338168943727], ["fodmaps", 1.5470167349826678], ["mounded", 1.5470167349826678], ["koans", 1.5470167349826678], ["surreptitiously", 1.546937805557414], ["grid's", 1.5468293903867827], ["mulch", 1.5468293903867825], ["burritos", 1.546818627213903], ["casbah", 1.5468096124889381], ["eater", 1.546726018910066], ["socialization", 1.5466002304770958], ["pennywise", 1.5464869347309615], ["rarer", 1.5463048522818172], ["markups", 1.54624063963067], ["gobbles", 1.5458125552896869], ["mildew", 1.545594965458911], ["nycc", 1.5455141934762746], ["cyclist", 1.5455119950208072], ["dutifully", 1.5454338652957402], ["cardiovascular", 1.545426117567263], ["homogeneity", 1.5454012998171454], ["interpret", 1.5453583617799196], ["baptistery", 1.5452687724781677], ["fingerlings", 1.545216893448675], ["announcer's", 1.545216893448675], ["lunchtimes", 1.545216893448675], ["lan's", 1.545216893448675], ["move's", 1.545216893448675], ["biagio", 1.545216893448675], ["protege", 1.545164020329994], ["dashi", 1.545096486937497], ["painful", 1.5449777703422658], ["vardaman", 1.544701534840521], ["pcmag", 1.544701534840521], ["polli", 1.544701534840521], ["ciabatta", 1.5446542094258504], ["dickensian", 1.5445736621969082], ["overdid", 1.5445736621969082], ["infinitum", 1.5445736621969082], ["partridges", 1.5445736621969082], ["prayerful", 1.5445325831101477], ["laotian", 1.5444059924550284], ["shiva", 1.544358668845638], ["condensing", 1.5443480562284293], ["strictly", 1.544147572910785], ["showering", 1.5441366506356256], ["breads", 1.5441218383470166], ["perch", 1.5440554337177752], ["booking", 1.5438814455415508], ["whacky", 1.543843367321165], ["switchbacks", 1.543843367321165], ["invading", 1.543668882311296], ["didn't", 1.5436390486530323], ["siddharth", 1.5433761023794819], ["adrienne's", 1.5433157380090252], ["figueroa's", 1.5433157380090252], ["velout\u00e9", 1.5433157380090252], ["mossad's", 1.5433157380090252], ["danum", 1.5433157380090252], ["puyi", 1.5433157380090252], ["fude", 1.5433157380090252], ["impera", 1.5433157380090252], ["thums", 1.5433157380090252], ["factored", 1.543266013780656], ["dani's", 1.5431709616733769], ["ramses", 1.5430775720000731], ["sunbird", 1.543020423095514], ["crunches", 1.5430094407437482], ["distinguishing", 1.5429818475849364], ["kauai", 1.5429716319358904], ["descendants", 1.5428760469155753], ["mussel", 1.5428408716280992], ["justinian", 1.542783035338115], ["backpackers", 1.542783035338115], ["tunisia's", 1.5426447311664124], ["condensation", 1.5422769678007497], ["libertas", 1.5421679256241905], ["surinder", 1.5421679256241905], ["ferociously", 1.5421203278487086], ["costuming", 1.5420398119855092], ["sensei's", 1.5420398119855092], ["tia's", 1.5419743075544323], ["believer's", 1.5419743075544323], ["caldo", 1.5419743075544323], ["encyclop\u00e6dia", 1.5419743075544323], ["spontaneously", 1.5419551048855957], ["unproduced", 1.5417717360479224], ["selkie", 1.5417717360479224], ["investor", 1.5417575826966121], ["vases", 1.5415994089384308], ["kanak", 1.5415595753576723], ["afterglow", 1.541509181125068], ["molly", 1.5414436686226827], ["daoist", 1.5413371280885009], ["performer", 1.541200099160566], ["stirling's", 1.5411984063597557], ["planters", 1.541153744730013], ["barometric", 1.5411253332237687], ["dynasties", 1.5410953686769897], ["ceilings", 1.5410571758337883], ["rex's", 1.540822147424255], ["originated", 1.5408068857151875], ["percy's", 1.5407307798870498], ["edgewater", 1.5406717548408906], ["dandelion", 1.5405795746680482], ["azeri", 1.5404657535208528], ["markus", 1.5403972883762516], ["studio", 1.5403656176288367], ["nicht", 1.54027172274668], ["skyview", 1.54027172274668], ["refrained", 1.5402220640231203], ["imaginary", 1.5401794649097174], ["feng", 1.5401089710143268], ["unorthodox", 1.5400407974509158], ["nerve's", 1.5400407974509158], ["jota", 1.5400407974509158], ["taylorswift", 1.5400407974509158], ["garbo's", 1.5400407974509158], ["matherson", 1.5400407974509158], ["shafie", 1.5400407974509158], ["baybayin", 1.5400407974509158], ["septuagenarians", 1.5400407974509158], ["tankful", 1.5400407974509158], ["cottage's", 1.5400407974509158], ["qualche", 1.5400407974509158], ["intermingles", 1.5400407974509158], ["copii", 1.5400407974509158], ["koreana", 1.5400407974509158], ["unlistenable", 1.5398404278441695], ["grec", 1.5398404278441695], ["variegation", 1.5398404278441695], ["chioma", 1.5398404278441695], ["siamese", 1.5397656919136793], ["giardiasis", 1.539737400918715], ["yalda", 1.539737400918715], ["prophesized", 1.539737400918715], ["pcms", 1.539737400918715], ["aburi", 1.539737400918715], ["a\u00f1ejo", 1.539737400918715], ["ziro", 1.539737400918715], ["challa", 1.539737400918715], ["vince's", 1.539658842888056], ["quarantining", 1.539658842888056], ["purchased", 1.5394650948175939], ["hyperlinks", 1.53936887738222], ["made", 1.5393271814577327], ["nephew's", 1.5392485953946138], ["contents", 1.5390690645709937], ["surveymonkey", 1.5390053018735408], ["competencies", 1.538989933720141], ["kinabalu", 1.538957787044832], ["emphasizes", 1.5388225900504304], ["tariqa", 1.5387143195892012], ["makola", 1.5387143195892012], ["nautiloids", 1.5387143195892012], ["loly", 1.5387143195892012], ["christo's", 1.5387143195892012], ["r\u00e9cit", 1.5387143195892012], ["vicente's", 1.5387143195892012], ["gontier", 1.5387143195892012], ["fasal", 1.5387143195892012], ["eagleton's", 1.5387143195892012], ["charizard's", 1.5387143195892012], ["jivamukti", 1.5387143195892012], ["cochem", 1.5387143195892012], ["yekini", 1.5387143195892012], ["guccio", 1.5387143195892012], ["ffviii", 1.5387143195892012], ["nutshells", 1.5387143195892012], ["mathcore", 1.5387143195892012], ["timehop", 1.5387143195892012], ["devereaux's", 1.5387143195892012], ["weylin", 1.5387143195892012], ["stanyan", 1.5387143195892012], ["ngunnawal", 1.5387143195892012], ["musu", 1.5387143195892012], ["hammerstone", 1.5387143195892012], ["examples", 1.53870733771802], ["trains", 1.5385297359157823], ["diabetes", 1.5383296410093035], ["oboes", 1.5382726679475276], ["frequently", 1.5381929370511582], ["countenance", 1.53814652832736], ["depleted", 1.5379777010319506], ["sarabi", 1.5379195291348677], ["jots", 1.5379195291348677], ["geopark", 1.5379195291348677], ["thetan", 1.5379195291348677], ["wide", 1.5379079497699921], ["scoffing", 1.5377635930609375], ["betcha", 1.5376766997355307], ["platinum's", 1.5376307179556874], ["lexemes", 1.5376307179556874], ["soup's", 1.5376307179556874], ["fervid", 1.5375713866268577], ["gon\u00e7alves", 1.5375713866268577], ["manco", 1.5375713866268577], ["nutri", 1.5374813754228633], ["histograms", 1.5374813754228633], ["gabon's", 1.5374813754228633], ["pat's", 1.5374813754228633], ["leicestershire's", 1.5374813754228633], ["excelsa", 1.5374813754228633], ["napa's", 1.5374813754228633], ["babushkas", 1.5374813754228633], ["fuerzas", 1.5374813754228633], ["surahs", 1.5374813754228633], ["aquarians", 1.5374813754228633], ["tubthumping", 1.5374813754228633], ["turian", 1.5374813754228633], ["strana", 1.5374813754228633], ["concertos", 1.5373958647790693], ["beachcombers", 1.5373285898345277], ["bligh's", 1.5373285898345277], ["fragaria", 1.5373285898345277], ["yelp's", 1.5373285898345277], ["binondo", 1.5373285898345277], ["factuality", 1.5373285898345277], ["lannes", 1.5373285898345277], ["bianca's", 1.5372508684700266], ["northwestern's", 1.5371325820073864], ["sashaying", 1.537012201979036], ["its's", 1.537012201979036], ["mealybug", 1.537012201979036], ["divisions", 1.5368870963346888], ["bettering", 1.536848341403132], ["mediocre", 1.5366805204507712], ["yasemin", 1.5366805204507712], ["mysteriousness", 1.5366805204507712], ["culshaw", 1.5366805204507712], ["teeters", 1.5364503463707677], ["laserdisc", 1.5364503463707677], ["elida", 1.5363324088777628], ["tastemaker", 1.5363324088777628], ["vata", 1.5363324088777628], ["slow", 1.5362444712070427], ["logically", 1.5362081886922863], ["fitter", 1.536140367326653], ["yoshiro", 1.5359666154470775], ["armadillo's", 1.5359666154470775], ["maribel", 1.5359666154470775], ["capsulatum", 1.5359666154470775], ["sinan's", 1.5359666154470775], ["smartcity", 1.5359666154470775], ["rashtrakuta", 1.5359666154470775], ["snowcaps", 1.5359666154470775], ["jeetan", 1.5359666154470775], ["foxhunt", 1.5359666154470775], ["westerplatte", 1.5359666154470775], ["zeus's", 1.5359666154470775], ["nino's", 1.5359666154470775], ["dscovr", 1.5359666154470775], ["cioccolata", 1.5359666154470775], ["gryph", 1.5359666154470775], ["eavan", 1.5359666154470775], ["monash's", 1.5359666154470775], ["gurage", 1.5359666154470775], ["seamster", 1.5359666154470775], ["trysil", 1.5359666154470775], ["brushtail", 1.5359666154470775], ["yakshas", 1.5359666154470775], ["glasswing", 1.5359666154470775], ["bookstalls", 1.5359666154470775], ["furless", 1.5359666154470775], ["m\u00e9ni\u00e8re's", 1.5359666154470775], ["poncha", 1.5359666154470775], ["bektas", 1.5359666154470775], ["\u00e1sgeir", 1.5359666154470775], ["screwface", 1.5359666154470775], ["recliners", 1.5358246066993475], ["tetrapods", 1.5358246066993475], ["mousse", 1.535772287687026], ["statisticians", 1.5354985042025793], ["satirical", 1.535295887667406], ["vernaculars", 1.5352591227637764], ["interviewee", 1.535259122763776], ["leeks", 1.535259122763776], ["showstopper", 1.535228169958882], ["climatology", 1.535228169958882], ["flakes", 1.5352281699588817], ["joost", 1.5351573601175479], ["radioactivity", 1.5350946502949645], ["pollo", 1.5349653334291848], ["some", 1.5348724822662636], ["floating", 1.5347146813743748], ["goji", 1.534653823468063], ["espionage", 1.5345447949816702], ["betray", 1.5344616466954697], ["played", 1.534432733854185], ["wingspan", 1.5344119933666653], ["rerunning", 1.5344119933666653], ["mauritshuis", 1.5344119933666653], ["annelid", 1.5344119933666653], ["rusko", 1.5344119933666653], ["primitivo", 1.5344119933666653], ["shamelessly", 1.5343753524422112], ["cantonese", 1.5343003794737127], ["polycystic", 1.5342787400454874], ["kobe's", 1.5342787400454874], ["consciously", 1.5342466917783686], ["design", 1.534063022726426], ["matriarchal", 1.5340609496710886], ["kazhagam", 1.5340609496710884], ["caminito", 1.5340609496710884], ["chihuahuan", 1.5340609496710884], ["simians", 1.5340609496710884], ["cybercafe", 1.5340609496710884], ["jinling", 1.5340609496710884], ["nerdery", 1.5340609496710884], ["diadora", 1.5340609496710884], ["mirvis", 1.5340609496710884], ["mincio", 1.5340609496710884], ["timer", 1.5340303157953576], ["dignitaries", 1.5339974274785557], ["mockery", 1.5338762466804927], ["lipan", 1.5338184103905081], ["oscillated", 1.5338184103905081], ["bjorn's", 1.5338184103905081], ["sifu", 1.5338184103905081], ["ivybridge", 1.5338184103905081], ["windstorms", 1.5338184103905081], ["adeola", 1.5338184103905081], ["abdali", 1.5338184103905081], ["hibernation", 1.5337078410125964], ["transcutaneous", 1.5335324369756873], ["misallocation", 1.5335324369756873], ["ventoux", 1.5335324369756873], ["bundesrat", 1.5335324369756873], ["bulb", 1.5334975898322787], ["phcs", 1.5333115305456995], ["insectivore", 1.5333115305456995], ["grampa's", 1.5333115305456995], ["albula", 1.5333115305456995], ["peacebuilders", 1.5333115305456995], ["nk's", 1.5333115305456995], ["plagueis", 1.5333115305456995], ["pommier", 1.5333115305456995], ["superflat", 1.5333115305456995], ["tangerines", 1.5332400474906624], ["tangles", 1.5332087846312115], ["circulate", 1.5331325814112993], ["handshake", 1.5331289209946222], ["porque", 1.5331289209946222], ["inflated", 1.5331222328258791], ["chamonix", 1.5330465268245543], ["famines", 1.5329925558149848], ["triggerfish", 1.5329925558149846], ["rotifers", 1.5329925558149846], ["bronchodilator", 1.5329925558149846], ["pelias", 1.5329925558149846], ["bhuvan", 1.5329925558149846], ["utada", 1.5329925558149846], ["plugged", 1.5329161642667488], ["provider's", 1.5329109789441986], ["invasiveness", 1.5328736500514932], ["wheelchairs", 1.5328424229970186], ["draining", 1.532738256580374], ["navajo", 1.5326145707590881], ["option", 1.532588724706779], ["preemptively", 1.5324915778555683], ["science's", 1.5324915778555683], ["compactness", 1.5324915778555683], ["spacebar", 1.5324915778555683], ["fresca", 1.5324915778555683], ["aliya", 1.5324915778555683], ["arida", 1.532491577855568], ["todes", 1.532491577855568], ["mehari", 1.532491577855568], ["matsutake", 1.532491577855568], ["patchworks", 1.532491577855568], ["mesomorph", 1.532491577855568], ["melrose's", 1.532491577855568], ["verita", 1.532491577855568], ["ixelles", 1.532491577855568], ["goodness", 1.5322645420662562], ["jewelry", 1.5322206828796845], ["zapata's", 1.532200837658669], ["bhindi", 1.532200837658669], ["dwarvish", 1.532200837658669], ["plyo", 1.532200837658669], ["allegory", 1.5321797155076122], ["templates", 1.5321266989084599], ["tico", 1.5320519646662807], ["vanquished", 1.5320427722625687], ["jafar", 1.532001807456765], ["distorted", 1.531995957740058], ["receiving", 1.5318875109097128], ["autumnal", 1.5318570057736405], ["dialing", 1.5318570057736405], ["bianchi's", 1.5318570057736405], ["biographies", 1.5317618199613512], ["ahoy", 1.5316762059067224], ["alchemy", 1.5315906421836956], ["saung", 1.5315906421836956], ["herpetofauna", 1.5315906421836956], ["tintypes", 1.5315906421836956], ["toomer's", 1.5315906421836956], ["foren", 1.5315906421836956], ["sa\u00fade", 1.5315906421836956], ["traduce", 1.5315906421836956], ["strategia", 1.5315906421836956], ["honne", 1.5315906421836956], ["electrocutions", 1.5315906421836956], ["offroading", 1.5315906421836956], ["synesthete", 1.5315906421836956], ["braggadocious", 1.5315906421836956], ["tensest", 1.5315906421836956], ["midna", 1.5315906421836956], ["ori's", 1.5315906421836956], ["sandage", 1.5315906421836956], ["subspecies", 1.5315906421836953], ["odia", 1.5315906421836953], ["tantrums", 1.531574604585348], ["studio's", 1.5314979589375723], ["explorer's", 1.5314850152428554], ["fresheners", 1.5314850152428554], ["breathe", 1.5313087285215352], ["pollens", 1.5312048510899463], ["analects", 1.5312048510899463], ["liv's", 1.5312048510899463], ["railway's", 1.5312048510899463], ["autocross", 1.5312048510899463], ["piano's", 1.5312048510899463], ["features", 1.5311225486946607], ["libertador", 1.531105961600726], ["erskine's", 1.531105961600726], ["aligners", 1.531105961600726], ["tsilhqot'in", 1.531105961600726], ["gon\u00e7alo", 1.531105961600726], ["teshuvah", 1.531105961600726], ["titicaca", 1.5311059616007259], ["stourbridge", 1.5311059616007259], ["chivalry", 1.5310928752249857], ["maung", 1.5309389014848938], ["turbans", 1.5309389014848938], ["uchis", 1.5308031919820586], ["ghibli's", 1.5308031919820586], ["yoo's", 1.5308031919820586], ["pavane", 1.5308031919820586], ["futurism", 1.5307599489540364], ["chapped", 1.5307599489540364], ["highbury", 1.5307260345631077], ["dissimilarity", 1.530690765190638], ["petty's", 1.530690765190638], ["sumida", 1.530690765190638], ["rerouted", 1.5306414874195253], ["madison's", 1.5306414874195253], ["vivienne", 1.5305711623444709], ["zari", 1.5304455276848665], ["benches", 1.5303394105065213], ["practitioner", 1.5302527421613663], ["icicles", 1.530219957951391], ["mind", 1.5302151307590945], ["paganism", 1.5301260741465992], ["boombox", 1.530059051541512], ["nessie", 1.530059051541512], ["nizhny", 1.530059051541512], ["weeds", 1.5299341181658652], ["peregrine", 1.5296211232035803], ["bullseye", 1.5296003025219036], ["beach's", 1.529492572810841], ["bike's", 1.529492572810841], ["anegada", 1.5294925728108408], ["sexagesimal", 1.5294925728108408], ["demisexual", 1.5294925728108408], ["bibtex", 1.5294925728108408], ["galaxia", 1.5294925728108408], ["rrms", 1.5294925728108408], ["sterndale", 1.5294925728108408], ["lamme", 1.5294925728108408], ["marcellina", 1.5294925728108408], ["sheets", 1.5294316275129205], ["operatic", 1.529232964461885], ["parkside", 1.5292151138407166], ["commented", 1.5292107859550714], ["submitted", 1.529188463176481], ["aclu's", 1.5291282778071944], ["haaland", 1.5291282778071944], ["arrangements", 1.5287698125950617], ["ming", 1.528748151193591], ["susie", 1.5286330878401684], ["daikon", 1.5285821605651204], ["inferno", 1.5284049336479535], ["matchups", 1.5283691474435253], ["ground", 1.5282817546558798], ["sossusvlei", 1.528261097309383], ["frequentative", 1.528261097309383], ["p\u00e2tissier", 1.528261097309383], ["shuffleboard", 1.528261097309383], ["kairat", 1.528261097309383], ["glissandi", 1.528261097309383], ["vajpayee's", 1.528261097309383], ["musicbox", 1.528261097309383], ["adidas's", 1.528261097309383], ["bankart", 1.528261097309383], ["extraordinaires", 1.528261097309383], ["revati", 1.528261097309383], ["keoki", 1.528261097309383], ["armonica", 1.528261097309383], ["scusa", 1.528261097309383], ["appa's", 1.528261097309383], ["kadal", 1.528261097309383], ["tzeltal", 1.528261097309383], ["silas's", 1.528261097309383], ["diagnosis", 1.5282007861864064], ["hakkasan", 1.5280734194145886], ["prose", 1.5279323285355082], ["bunnies", 1.5277616655782362], ["sandbox", 1.5276960259792856], ["transient", 1.527408221583886], ["ultraboost", 1.5273576027203737], ["hildur", 1.5273576027203737], ["kova", 1.5273576027203737], ["anyone", 1.5273556026092272], ["radiologist", 1.5273148056082626], ["carbonation", 1.5272582543113828], ["submissions", 1.5269152602233451], ["tigerfish", 1.526878055592361], ["enorme", 1.526878055592361], ["sur's", 1.526878055592361], ["contar", 1.526878055592361], ["amorgos", 1.526878055592361], ["suffocatingly", 1.526878055592361], ["southend's", 1.526878055592361], ["euglena", 1.526878055592361], ["rohith", 1.526878055592361], ["fauzia", 1.526878055592361], ["countrymen", 1.5266535147018325], ["caius", 1.5266030415333565], ["payslip", 1.5264510762703272], ["newgrange", 1.5264510762703272], ["coober", 1.5264510762703272], ["claudy", 1.5264510762703272], ["clamming", 1.5264510762703272], ["verrocchio", 1.5264510762703272], ["bicameral", 1.5263784227207666], ["wrinkle", 1.5262996926925154], ["enrichment", 1.5261861168964732], ["pritchett", 1.5260335433934713], ["aakash", 1.5259390161977777], ["abbey's", 1.5259237568076158], ["guggenheim", 1.5259237568076158], ["entertainingly", 1.5259237568076158], ["express's", 1.5256427173596812], ["adeptness", 1.5256427173596812], ["adjei", 1.5256427173596812], ["convent's", 1.5256427173596812], ["fsas", 1.5256427173596812], ["hoffa's", 1.5256427173596812], ["seasonal", 1.5255989954410794], ["contrary", 1.5255884626400025], ["snapchat's", 1.5255884626400025], ["arts", 1.525527560904967], ["denominator", 1.5254642796149607], ["anticipations", 1.5254642796149607], ["downstream", 1.5252990000492574], ["rachid", 1.5251939989133991], ["rebellions", 1.5250428656216972], ["dermatologists", 1.524889933124142], ["trumpeting", 1.524889933124142], ["plus", 1.5248377198067948], ["ted's", 1.5247959491529168], ["conscience", 1.5247539036921056], ["ioc's", 1.5247455443415339], ["trnas", 1.5247455443415339], ["socioeconomics", 1.5247455443415339], ["pumpkins", 1.5247231873687428], ["extra", 1.5246896636147775], ["bergamo", 1.524648385534545], ["sftp", 1.5244518722413138], ["golfer's", 1.5244518722413138], ["sedona", 1.5244248453487999], ["empirical", 1.5244064903944596], ["rabbi's", 1.524151487635946], ["timewise", 1.5240643982417608], ["lewellen", 1.5240643982417608], ["rationed", 1.5240341188166302], ["albertina", 1.5238441577217836], ["nafld", 1.5238441577217836], ["tempranillo", 1.5238441577217836], ["frappe", 1.5237524096239017], ["dyes", 1.5236970596432104], ["wherever", 1.5236560145341889], ["groundcover", 1.5235296388037813], ["espa\u00f1ol", 1.5235296388037813], ["balladry", 1.5235296388037813], ["maharaja", 1.5235296388037813], ["jai's", 1.5235296388037813], ["nrel", 1.5235296388037813], ["kumba", 1.5235296388037813], ["jataka", 1.5235296388037813], ["tesfaye", 1.5235296388037813], ["olafur", 1.5235296388037811], ["pyromaniac", 1.5235296388037811], ["libation", 1.5232541361566196], ["miyu", 1.5232541361566196], ["charting", 1.5232125370814191], ["abramovi\u0107", 1.5232076756510333], ["lovegood", 1.5232076756510333], ["molehills", 1.5231423663994081], ["dashiki", 1.5231423663994081], ["peruvian", 1.52314068772815], ["juxtapositions", 1.522998792239738], ["connie's", 1.522998792239738], ["javan", 1.522998792239738], ["shrinking", 1.5229914569053986], ["designer's", 1.5229105114107069], ["organization", 1.5227924639201704], ["headings", 1.5226050759591734], ["charybdis", 1.5225403338435188], ["arbeit", 1.5225403338435188], ["hypertension", 1.5224964502771883], ["zumba", 1.5224468174542407], ["sneezy", 1.5224468174542405], ["chittering", 1.5224468174542405], ["mediation", 1.5224010983305933], ["extraction", 1.5221292470597145], ["hardwood", 1.522128349312122], ["cucumber", 1.5220137310475508], ["antibacterial", 1.5219589896916126], ["overproducing", 1.5219114448167355], ["illustrator's", 1.5219114448167355], ["bedframe", 1.5219114448167355], ["donata", 1.5219114448167355], ["ollivander", 1.5219114448167355], ["nyiragongo", 1.5219114448167355], ["afi's", 1.5219114448167355], ["pneumococcus", 1.5219114448167355], ["vasily", 1.5216885153732027], ["desecrated", 1.5216370554015406], ["faux", 1.5215554877225144], ["clipping", 1.5215216688650814], ["understated", 1.5214037834938223], ["thawed", 1.521111337789807], ["bud's", 1.5210692803808683], ["telmo", 1.5209136881877032], ["relenting", 1.5209136881877032], ["dhol", 1.5209136881877032], ["pw's", 1.520721289239166], ["perfumery", 1.520721289239166], ["meilin", 1.520721289239166], ["g\u00e9ricault", 1.520721289239166], ["pencil's", 1.520721289239166], ["varosha", 1.520721289239166], ["burrower", 1.520721289239166], ["meridiana", 1.520721289239166], ["bathyscaphe", 1.520721289239166], ["pharsalus", 1.520721289239166], ["gwtw", 1.520721289239166], ["panthenol", 1.520721289239166], ["dodong", 1.520721289239166], ["jianye", 1.520721289239166], ["senado", 1.520721289239166], ["korg", 1.520582334443], ["tenures", 1.5205655733586378], ["washing", 1.5205019259342856], ["many", 1.5204547913735629], ["superhuman", 1.5204420870440793], ["library", 1.5203376818770213], ["alsatian", 1.5203289462852858], ["fctc", 1.5203289462852858], ["presbyterianism", 1.5203289462852858], ["uswnt", 1.5203289462852858], ["cataphracts", 1.5203289462852858], ["kaieteur", 1.5203289462852858], ["readerships", 1.5203289462852858], ["supergroups", 1.5203289462852858], ["quieren", 1.5203289462852858], ["loofahs", 1.5203289462852858], ["nkunda", 1.5203289462852858], ["mandola", 1.5203289462852858], ["whitening", 1.5202676735821612], ["benevolently", 1.5202369445336195], ["sitges", 1.5202369445336195], ["hamm's", 1.5202369445336195], ["leopold's", 1.5202369445336195], ["media", 1.5202356589011419], ["shrieks", 1.5201958759179963], ["careworn", 1.520195875917996], ["unresponsiveness", 1.520195875917996], ["anamika", 1.520195875917996], ["spiderlings", 1.520195875917996], ["interquartile", 1.520195875917996], ["suffragette", 1.5199262101405748], ["conches", 1.5199262101405746], ["jayenge", 1.5199262101405746], ["liberdade", 1.5199262101405746], ["splendiferous", 1.5199262101405746], ["hanon", 1.5199262101405746], ["reata", 1.5199262101405746], ["zone", 1.519921671879561], ["appreciates", 1.519720814706772], ["embedding", 1.519720814706772], ["ticketing", 1.5197208147067718], ["metta", 1.5197208147067718], ["teammate's", 1.5197208147067718], ["underexplored", 1.5190878614311754], ["leute", 1.5190878614311754], ["verity's", 1.5190878614311754], ["thelonius", 1.5190878614311754], ["bhaiyya", 1.5190878614311754], ["stetsons", 1.5190878614311754], ["bubbie", 1.5190878614311754], ["mirch", 1.5190878614311754], ["franklinville", 1.5190878614311754], ["cliches", 1.5190486692778875], ["ibuprofen", 1.5190283593606142], ["uprights", 1.5188953281154045], ["misalignment", 1.5188953281154045], ["unicameral", 1.5188512433842234], ["norway's", 1.5187615968561259], ["decentralizing", 1.5187615968561259], ["speaker's", 1.518727460989555], ["momentum", 1.5187134993228741], ["nostril", 1.5186150270657046], ["cheetah", 1.5185430408901561], ["rental", 1.5185393019256586], ["shelf", 1.5183620750084919], ["loom", 1.5181629981974274], ["nibbles", 1.5180884615223391], ["radiance", 1.5180291032247235], ["journeying", 1.5179405141038105], ["lightweight", 1.5179206242131604], ["beaux", 1.517814882776345], ["variable", 1.5177471513110488], ["substantiating", 1.5176670908911165], ["unc's", 1.5176670908911165], ["hozier", 1.5175054782421846], ["rodin", 1.5174812404252855], ["timezone", 1.5174139354130696], ["buzzer", 1.5173730876168108], ["sponges", 1.5173350677449087], ["beaut\u00e9", 1.517266413156078], ["arlene's", 1.517266413156078], ["zwirner", 1.517266413156078], ["sumeria", 1.517266413156078], ["couleurs", 1.517266413156078], ["cave's", 1.5172061084496729], ["algebraically", 1.5172061084496729], ["aggressive", 1.5171387287604874], ["cordyceps", 1.5171282537667907], ["mortimer's", 1.5171282537667907], ["postcard", 1.5171268571009644], ["lecture", 1.5171137810701851], ["domaines", 1.516876670948743], ["piha", 1.516876670948743], ["sadri", 1.516876670948743], ["xiaoling", 1.516876670948743], ["powergen", 1.516876670948743], ["koyasan", 1.516876670948743], ["badea", 1.516876670948743], ["amaya's", 1.516876670948743], ["wilmington's", 1.516876670948743], ["gr\u00fcn", 1.516876670948743], ["nizwa", 1.516876670948743], ["veil", 1.516666849777472], ["officeholder", 1.5165087648922433], ["arbs", 1.5165087648922433], ["kele", 1.5165087648922433], ["lengths", 1.51644041263421], ["angelou", 1.51644041263421], ["informality", 1.5162747357618587], ["pomp", 1.5162747357618584], ["dervishes", 1.5162747357618584], ["jeff's", 1.5162747357618584], ["alicia's", 1.5162747357618584], ["quizlet", 1.5162747357618584], ["gradients", 1.5161127579031082], ["indistinguishable", 1.5160846072683458], ["jatin", 1.5155803117931104], ["manitoba's", 1.5155803117931104], ["immanuel", 1.51544932763134], ["detection", 1.5149202769069883], ["crassus", 1.514889377392044], ["menial", 1.5148803685587142], ["marx's", 1.514857325398912], ["permanence", 1.514596645145205], ["mehta", 1.5145018334107778], ["expounding", 1.5143734508267341], ["egypt's", 1.5143055477933365], ["thermodynamics", 1.5143055477933365], ["piri", 1.5141540717957578], ["meatpacking", 1.5141540717957578], ["overstuffed", 1.5141540717957578], ["interested", 1.5139912733498755], ["wuyi", 1.5139384110534428], ["preapproval", 1.5139384110534428], ["dilts", 1.5139384110534428], ["mauritania's", 1.5139384110534428], ["thrus", 1.5138717647067683], ["aphrodite", 1.5137315040681956], ["cleared", 1.5136749091148098], ["fajitas", 1.5136365892011592], ["fought", 1.513598169591052], ["subways", 1.5135505626828758], ["medicine's", 1.5135178726059277], ["dhruv", 1.5135178726059277], ["wielder", 1.5133879680593134], ["cambridge's", 1.5133879680593134], ["wheelchair", 1.513380791012539], ["chance", 1.5132179501307221], ["original's", 1.5129127075229187], ["nightguard", 1.5129127075229185], ["lectionaries", 1.5129127075229185], ["unbias", 1.5129127075229185], ["buong", 1.5129127075229185], ["shtetls", 1.5129127075229185], ["huji", 1.5129127075229185], ["svaneti", 1.5129127075229185], ["eichhornia", 1.5129127075229185], ["chuquisaca", 1.5129127075229185], ["lens's", 1.5129127075229185], ["moutarde", 1.5129127075229185], ["ruffins", 1.5129127075229185], ["trumbauer", 1.5129127075229185], ["colorfast", 1.5129127075229185], ["lugal", 1.5129127075229185], ["household", 1.5127188495556496], ["functionalism", 1.5127174929800125], ["simba", 1.5126347901484023], ["cantina", 1.512533689725627], ["protruding", 1.5124391000666506], ["ornithological", 1.5124027369473492], ["okinawan", 1.5124027369473492], ["oversensitive", 1.5123363598248147], ["reshot", 1.5123363598248147], ["adeno", 1.5123363598248147], ["nikolaos", 1.5123363598248147], ["amanda's", 1.5123023104651285], ["democratization", 1.5122962405453202], ["endorsing", 1.5122345052747188], ["accountant", 1.512223345558795], ["chastised", 1.5120519694398642], ["castaways", 1.5119671370807168], ["fawns", 1.5117103927884634], ["kahuna", 1.5117103927884634], ["cauldrons", 1.5117103927884634], ["comes", 1.5116976206007893], ["helsinki", 1.511459254266783], ["godfrey's", 1.51126225729653], ["canyons", 1.51126225729653], ["giddily", 1.51126225729653], ["canid", 1.51126225729653], ["zelda's", 1.5110544728770359], ["barging", 1.5109730758466071], ["flitwick", 1.5109730758466071], ["doga", 1.5109730758466071], ["forces", 1.5108808103096527], ["substitutions", 1.5107898771270227], ["reforest", 1.5107868348659776], ["jambon", 1.5107868348659776], ["fidel's", 1.5107868348659776], ["summoner's", 1.5107868348659776], ["festival's", 1.5107681147674095], ["sol's", 1.5106796260358002], ["congregating", 1.5105988410290603], ["recapping", 1.5105988410290603], ["dongdaemun", 1.5104968788482314], ["gaja", 1.5104968788482314], ["exoticism", 1.5102815549880964], ["appendicitis", 1.5102815549880964], ["ribeira", 1.5102815549880964], ["sabarmati", 1.5102815549880964], ["pooh's", 1.5102815549880964], ["perch\u00e9", 1.5102815549880964], ["liver's", 1.5102815549880964], ["shadier", 1.5102815549880964], ["lossless", 1.5102815549880961], ["osin", 1.5102815549880961], ["deccani", 1.5102815549880961], ["zhukov's", 1.5102815549880961], ["campamento", 1.5102815549880961], ["resolv", 1.5102815549880961], ["mumme", 1.5102815549880961], ["knizia", 1.5102815549880961], ["wirtschaftliche", 1.5102815549880961], ["pobreza", 1.5102815549880961], ["valli\u00e8re", 1.5102815549880961], ["tasawwuf", 1.5102815549880961], ["zampano", 1.5102815549880961], ["potahto", 1.5102815549880961], ["mcvay's", 1.5102815549880961], ["moona", 1.5102815549880961], ["prater's", 1.5102815549880961], ["todaiji", 1.5102815549880961], ["tezuka's", 1.5102815549880961], ["infoseek", 1.5102815549880961], ["bulles", 1.5102815549880961], ["smithwick's", 1.5102815549880961], ["seluruh", 1.5102815549880961], ["eternalism", 1.5102815549880961], ["system's", 1.5102322959680379], ["there", 1.5101050381873065], ["gaga's", 1.5099733342625885], ["animal", 1.5099458997552873], ["vertices", 1.5099266956001762], ["reversing", 1.5099206273727248], ["perspiring", 1.5098634260969033], ["fleetingly", 1.5098634260969033], ["antiques", 1.5096564384504356], ["stacy's", 1.509631692012628], ["tasters", 1.509631692012628], ["interlocking", 1.5094908316608773], ["anonymously", 1.5094630310769714], ["wangchuk", 1.5094611956426383], ["baikal", 1.5094611956426383], ["wspu", 1.5094611956426383], ["ambystoma", 1.5094611956426383], ["\u00e9ire", 1.5094611956426383], ["packie", 1.5094611956426383], ["nostoc", 1.5094611956426383], ["surfboard", 1.5094465917224018], ["celestia", 1.509382577872032], ["overstep", 1.509382577872032], ["taluka", 1.509382577872032], ["tallest", 1.5091208503572284], ["samplers", 1.5089426529003407], ["tisket", 1.5088237542593432], ["fastback", 1.5088237542593432], ["claudiu", 1.5088237542593432], ["wsib", 1.5088237542593432], ["bansa", 1.5088237542593432], ["buchhandlung", 1.5088237542593432], ["highchairs", 1.5088237542593432], ["hcws", 1.5088237542593432], ["toolbelt", 1.5088237542593432], ["r\u00e9gions", 1.5088237542593432], ["exotique", 1.5088237542593432], ["sleeth", 1.5088237542593432], ["atmospheric", 1.5087849919278271], ["opry", 1.5087271276980714], ["aida", 1.5086470511407066], ["improperly", 1.508644132383836], ["fifties", 1.50861678255094], ["howler", 1.50861678255094], ["luster", 1.5085664012798154], ["utmost", 1.5085189413796949], ["function", 1.5084567656181744], ["internal", 1.5083600651697775], ["springsteen's", 1.5081663147150037], ["hubble's", 1.5081663147150037], ["jess's", 1.5081663147150037], ["shui", 1.5081663147150035], ["babylonians", 1.5080758862274923], ["haggadah", 1.5078975272769863], ["onigiri", 1.5078975272769863], ["rabbits", 1.5078717708189384], ["thrillers", 1.5078534495942246], ["lauper's", 1.5076595383995752], ["intracytoplasmic", 1.5076595383995752], ["pyrrho", 1.5076595383995752], ["civics", 1.5075707219319228], ["symbolic", 1.5075548232935143], ["celebrity's", 1.507550485268407], ["thermometer", 1.5073896647472627], ["recreations", 1.5073816288072435], ["aqueduct", 1.5073659535305903], ["warmup", 1.5073537798293064], ["prudently", 1.5072569590835807], ["monaghan's", 1.5072569590835805], ["planchets", 1.5072569590835805], ["grasshopper's", 1.5072569590835805], ["foolhardiness", 1.5072569590835805], ["ordinateur", 1.5072569590835805], ["merinos", 1.5072569590835805], ["paoay", 1.5072569590835805], ["waipio", 1.5072569590835805], ["grimaldi's", 1.5072569590835805], ["plum's", 1.5072569590835805], ["arwu", 1.5072569590835805], ["bastila", 1.5072569590835805], ["vaslav", 1.5072569590835805], ["seyda", 1.5072569590835805], ["kazbek", 1.5072569590835805], ["wonderbolts", 1.5072569590835805], ["nomophobia", 1.5072569590835805], ["bocelli's", 1.5072569590835805], ["gerecht", 1.5072569590835805], ["browncoat", 1.5072569590835805], ["streetfood", 1.5072569590835805], ["bunbun", 1.5072569590835805], ["ruter", 1.5072569590835805], ["andric", 1.5072569590835805], ["col\u00e1iste", 1.5072569590835805], ["oberst's", 1.5072569590835805], ["screecher", 1.5072569590835805], ["thrip", 1.5072569590835805], ["eardrops", 1.5072569590835805], ["culpepper's", 1.5072569590835805], ["ra\u00fal's", 1.5072569590835805], ["savitribai", 1.5072569590835805], ["sikma", 1.5072569590835805], ["instrumentos", 1.5072569590835805], ["soeun", 1.5072569590835805], ["chatree", 1.5072569590835805], ["zazou", 1.5072569590835805], ["malnutrition", 1.5072436323200167], ["dash", 1.5071908716630669], ["winds", 1.5070907277785472], ["wouldn't", 1.5070273768977598], ["teton", 1.507023517117796], ["yucca", 1.507023517117796], ["katsuragi", 1.5069294369281954], ["noodling", 1.5068927407285866], ["pagerank", 1.5068927407285866], ["space's", 1.5067875548608827], ["curry", 1.5066619892313107], ["incubators", 1.5066015317059613], ["kitten", 1.506591017353727], ["collectivization", 1.5064287959192493], ["ananya", 1.506428795919249], ["glacier", 1.5063355185944243], ["boost", 1.5062770841613626], ["polyphonic", 1.5060839964695079], ["sohrab", 1.5060027696760592], ["cheongsam", 1.505916579395645], ["cabin's", 1.505916579395645], ["sociolinguistics", 1.5058176686186733], ["luminous", 1.5058176686186733], ["medicating", 1.505671415931356], ["modularity", 1.505671415931356], ["supernovas", 1.505671415931356], ["equated", 1.5055936952624414], ["nowhere", 1.5055603134167972], ["colonial", 1.5055546361251146], ["trina", 1.5053780434459272], ["urchins", 1.5053662844254423], ["utilization", 1.5051949376351752], ["dutiful", 1.5049761162703392], ["weaponization", 1.5048557759443943], ["disneyland's", 1.5048557759443943], ["patagonian", 1.504713696720859], ["png's", 1.504603804781374], ["puppeteering", 1.504603804781374], ["liga's", 1.504603804781374], ["abiogenesis", 1.504603804781374], ["hooping", 1.5045434031834302], ["unimpressed", 1.5045116778248948], ["abaya", 1.5043917049173299], ["purl", 1.5043917049173299], ["caseworker", 1.5042134594546621], ["earpiece", 1.5041832306919873], ["autonomous", 1.5041258472780963], ["fergus", 1.504093247398444], ["quick", 1.5040738120670512], ["piste", 1.5038969828168638], ["ticaret", 1.5037435395985375], ["lackadaisically", 1.5037435395985375], ["sole's", 1.5037435395985375], ["panpipe", 1.5037435395985375], ["preserve's", 1.5037435395985375], ["anfield's", 1.5037435395985375], ["unice", 1.5037435395985375], ["laroi", 1.5037435395985375], ["muladhara", 1.5037435395985375], ["palamon", 1.5037435395985375], ["madalina", 1.5037435395985375], ["shorthands", 1.5037435395985375], ["oomen", 1.5037435395985375], ["quasiperiodic", 1.5037435395985375], ["dolenz", 1.5037435395985372], ["detouring", 1.5037435395985372], ["d\u00f3nde", 1.5037435395985372], ["dorji", 1.5037435395985372], ["monotheism", 1.5036245725463537], ["melted", 1.5035191002642687], ["astronaut", 1.503356268963339], ["outgrew", 1.5033519397184336], ["ellington", 1.5033012620868906], ["trespassing", 1.5030897380595816], ["sailors", 1.503054546784575], ["maimonides", 1.5029485090262422], ["musicologist", 1.5028266228060991], ["trappings", 1.502774632678693], ["handcrafts", 1.502774632678693], ["hmcs", 1.502774632678693], ["grimace", 1.5027274966663764], ["innovator's", 1.5026845652748764], ["yellowhammer", 1.5026845652748764], ["entourages", 1.5026845652748764], ["armillaria", 1.5026845652748764], ["exuberant", 1.5025347425409823], ["aridity", 1.5025347425409823], ["unabashed", 1.5024211520637145], ["chairs", 1.5024116524634643], ["wisconsinites", 1.5021738699329856], ["jeonju", 1.5021738699329856], ["dicing", 1.5021582677947436], ["frittata", 1.5021582677947436], ["britten's", 1.5021582677947436], ["mikko", 1.5020849989298624], ["spills", 1.5019981229900747], ["grasshoppers", 1.501873211812967], ["marlene's", 1.501764929678013], ["tarsier", 1.501764929678013], ["kilmainham", 1.501764929678013], ["derry's", 1.501764929678013], ["seattle's", 1.5016965588640054], ["seborrheic", 1.5016433391242379], ["kiev's", 1.5016433391242379], ["deepti", 1.5016433391242379], ["pontoise", 1.5015767621639573], ["alus", 1.5015767621639573], ["okinawa's", 1.5015767621639573], ["klaudia", 1.5015767621639573], ["nawabs", 1.5015767621639573], ["bandleader", 1.501553621822423], ["attempted", 1.5014893537031506], ["sommes", 1.5014685871936984], ["fabrica", 1.5014685871936984], ["sermons", 1.501444288873928], ["robins", 1.501430087999155], ["rigel", 1.5012917838274529], ["ultrasounds", 1.5012568041731313], ["crumb", 1.5012050657776717], ["resident's", 1.5011686362773071], ["gulps", 1.5011686362773071], ["penicillin", 1.5011119884042399], ["whitelisting", 1.5010678393501413], ["schwarzenberg", 1.5010678393501413], ["laius", 1.5010678393501413], ["morwenna", 1.5010678393501413], ["cabin", 1.5010200929657087], ["sands", 1.5009967634735741], ["magna", 1.5009756600064192], ["cosimo", 1.5009588293400216], ["gala", 1.5008999220076298], ["genomics", 1.5008961234795433], ["deli", 1.500890454757781], ["dishonestly", 1.500890454757781], ["croaky", 1.5007117985912821], ["finlay's", 1.5007117985912821], ["roc\u00edo", 1.5007117985912821], ["hatchling", 1.5004414000690136], ["quoll", 1.5004166733604876], ["annette's", 1.5004166733604876], ["mateu", 1.5004166733604876], ["obree", 1.5004166733604876], ["unvetted", 1.5004166733604876], ["ceramica", 1.5004166733604876], ["entomological", 1.5004166733604873], ["thrice", 1.5003928067113887], ["subcategory", 1.5003697560161047], ["campeche", 1.5002464615994708], ["chell", 1.5002464615994708], ["antonov", 1.5000270631698716], ["maxim's", 1.4999841807495409], ["scratchers", 1.4999841807495409], ["anhalt", 1.4999841807495409], ["skyler", 1.4999659018572444], ["electropop", 1.499955772496274], ["disentangling", 1.499955772496274], ["empanada", 1.499955772496274], ["antonina", 1.499955772496274], ["encapsulation", 1.4996793678698928], ["pengo", 1.4996123760282116], ["sanitisers", 1.4996123760282116], ["ravikant", 1.4996123760282116], ["wollstonecraft", 1.4996123760282116], ["ototoxicity", 1.4996123760282116], ["foros", 1.4996123760282116], ["cixin", 1.4996123760282116], ["birchbark", 1.4996123760282116], ["bersatu", 1.4996123760282116], ["larvik", 1.4996123760282116], ["gusteau", 1.4996123760282116], ["consiste", 1.4996123760282116], ["lehne", 1.4996123760282116], ["voitures", 1.4996123760282116], ["nautical", 1.499546197194318], ["busyness", 1.499493829595324], ["recalibrated", 1.499493829595324], ["capac", 1.4993466286640387], ["fireside", 1.499333118602508], ["edibles", 1.4993299276901855], ["trunks", 1.4993133657876383], ["fuertes", 1.49920050775685], ["dursleys", 1.49920050775685], ["needle's", 1.49920050775685], ["ymca's", 1.49920050775685], ["blasko", 1.49920050775685], ["nonresponse", 1.49920050775685], ["odyssey's", 1.49920050775685], ["rocinha", 1.49920050775685], ["clergymen", 1.499165397674701], ["provide", 1.4990531395367257], ["swingset", 1.4990443410372918], ["canci\u00f3n", 1.4990443410372918], ["kalachakra", 1.4990443410372918], ["mercier's", 1.4990443410372918], ["zorb", 1.4990443410372918], ["bwca", 1.4990443410372918], ["romanticising", 1.4989114588157657], ["montserrat", 1.4989114588157657], ["pitaka", 1.4989114588157657], ["germanotta", 1.4989114588157657], ["ayame", 1.4989114588157657], ["ellie's", 1.4988333566120933], ["treachery", 1.4988146024409645], ["gearing", 1.4987792580415293], ["feira", 1.4986974203381394], ["graystone", 1.4986974203381394], ["chocolate's", 1.4986974203381394], ["dilys", 1.4986974203381394], ["digitalocean", 1.4986974203381394], ["kiosk", 1.4986308115639022], ["fenrir", 1.49835653613017], ["deadlocks", 1.49835653613017], ["researcher's", 1.49835653613017], ["motions", 1.498141565426371], ["outnumbered", 1.498121284177047], ["bowels", 1.498010517353818], ["nausea", 1.4980015793969002], ["agni", 1.4980015793969002], ["donuts", 1.4979445286080644], ["accordionist", 1.4979240986558187], ["disease's", 1.4979240986558187], ["sepal", 1.4979240986558187], ["crowdfund", 1.497895648055654], ["abbreviations", 1.497728565440143], ["domains", 1.4976856013390119], ["trance", 1.497538772566213], ["raul", 1.4972618864106126], ["siglos", 1.4972618864106126], ["cvpr", 1.4972618864106126], ["sportscasting", 1.4972618864106126], ["sogdiana", 1.4972618864106126], ["ridleys", 1.4972618864106126], ["suki's", 1.4972618864106126], ["treno", 1.4972618864106126], ["iea's", 1.4972618864106126], ["klinikum", 1.4972618864106126], ["durmitor", 1.4972618864106126], ["duang", 1.4972618864106126], ["vulfpeck", 1.4972618864106126], ["elpida", 1.4972618864106126], ["camaron", 1.4972618864106126], ["ind\u00edgenas", 1.4972618864106126], ["encoded", 1.4972199463017497], ["hazardous", 1.4971701577514618], ["kira", 1.4971298527345622], ["matter", 1.4968216201178228], ["televisa", 1.4967099004617055], ["addai", 1.4964296118767548], ["cleansings", 1.4964296118767548], ["karateka", 1.4964296118767548], ["giralda", 1.4964296118767548], ["adfa", 1.4964296118767548], ["marisa's", 1.4964296118767548], ["nanna's", 1.4964296118767548], ["hulagu", 1.4964296118767548], ["dipesh", 1.4964296118767548], ["copyeditor", 1.4964296118767548], ["kline", 1.4963237523965707], ["blige's", 1.4961870393995498], ["digitale", 1.4961870393995498], ["midlife", 1.4960036831447214], ["amerigo", 1.4960036831447214], ["toggling", 1.4960036831447214], ["disturbingly", 1.4959220547449208], ["bongos", 1.4958290999164399], ["underpricing", 1.495744903749609], ["dwalin", 1.495744903749609], ["denialism", 1.495744903749609], ["perspectivas", 1.495744903749609], ["tifa", 1.495744903749609], ["razor's", 1.495744903749609], ["koothrappali", 1.495744903749609], ["aragorn's", 1.495744903749609], ["mallorcan", 1.495744903749609], ["elix", 1.495744903749609], ["rosada", 1.495744903749609], ["pyeongtaek", 1.495744903749609], ["marcella's", 1.495744903749609], ["copaiba", 1.495744903749609], ["arp\u00e8ge", 1.495744903749609], ["faial", 1.495744903749609], ["gallica", 1.495744903749609], ["whirley", 1.495744903749609], ["garibi", 1.495744903749609], ["mitarbeiter", 1.495744903749609], ["konichiwa", 1.495744903749609], ["auggie's", 1.495744903749609], ["dovre", 1.495744903749609], ["voisins", 1.495744903749609], ["supernatural", 1.4957403684406225], ["illegible", 1.4956953347387143], ["blowin", 1.4956953347387143], ["fixations", 1.4956626747829376], ["alaa", 1.4956626747829376], ["wristwatch", 1.4956395324989042], ["windmills", 1.4955710203615888], ["romantics", 1.4954729501307453], ["formulaic", 1.4954729501307453], ["activist's", 1.4954461431576287], ["hawaiian", 1.4954399870677688], ["damir", 1.4954202358738222], ["breguet", 1.4954202358738222], ["passo", 1.4954202358738222], ["baji", 1.4954202358738222], ["gittes", 1.4953521135963133], ["bonifacio", 1.4953521135963133], ["mccardell", 1.4953521135963133], ["handstands", 1.4953521135963133], ["kale's", 1.4953521135963133], ["quacker", 1.4953521135963133], ["havelis", 1.4953521135963133], ["odessa's", 1.4953521135963133], ["sadie's", 1.4953521135963133], ["wattages", 1.4953521135963133], ["miyamoto's", 1.4953521135963133], ["maqam", 1.4953521135963133], ["misheard", 1.4953099969334884], ["croaking", 1.495278758084241], ["cobb's", 1.495278758084241], ["couture", 1.4952355169402824], ["discontinuing", 1.4952355169402824], ["yawned", 1.4952070042209038], ["imperiled", 1.4952070042209038], ["wanting", 1.4951700115050495], ["peril", 1.4951314625540237], ["hola", 1.4950680961008536], ["backbones", 1.4950680961008533], ["lublin", 1.4950680961008533], ["puffs", 1.494909046303396], ["tuned", 1.4949006752614244], ["elephants", 1.4948514338380632], ["matatus", 1.4946848435768594], ["alap", 1.4946848435768594], ["ampersands", 1.4946848435768594], ["pureology", 1.4946848435768594], ["owlbear", 1.4946848435768594], ["wadud", 1.4946848435768594], ["sutras", 1.4942403276382492], ["dreamer's", 1.4942110924378493], ["jasmina", 1.4942110924378493], ["kallie", 1.4942110924378493], ["maximilian", 1.4941893679381046], ["wang", 1.4941362892066579], ["adversely", 1.494041968375321], ["sufis", 1.4940419683753208], ["archetypes", 1.49396124425773], ["technophobia", 1.4939022860671334], ["angelinos", 1.4939022860671334], ["mercian", 1.4939022860671334], ["wario's", 1.4939022860671334], ["satie's", 1.4939022860671334], ["flamingo's", 1.4939022860671334], ["shady's", 1.4939022860671334], ["nottinghamshire's", 1.4939022860671334], ["anjaneya", 1.4939022860671334], ["synyster", 1.4939022860671334], ["schneerson", 1.4939022860671334], ["houseparty", 1.4939022860671334], ["valer", 1.4939022860671334], ["countercurrent", 1.4937697304073223], ["softwoods", 1.4937697304073223], ["barranco", 1.4937697304073223], ["sections", 1.4936901275499548], ["classic", 1.4936211973719826], ["ziplines", 1.4935831102769255], ["osmanthus", 1.4935831102769255], ["layout", 1.4935650755148797], ["checkout", 1.4935321918782658], ["katie's", 1.493490742737638], ["emanate", 1.4933008761291031], ["strahm", 1.4933008761291031], ["ladas", 1.4933008761291031], ["veltliner", 1.4933008761291031], ["crunchier", 1.4933008761291031], ["chessmaster", 1.4933008761291031], ["dinant", 1.493049520990889], ["salesforce", 1.4929114886777004], ["sides", 1.4928413989364482], ["infidelity", 1.4927959561371744], ["tinctures", 1.4927959561371744], ["deducing", 1.4927405380892798], ["herc", 1.4926601708803806], ["partner's", 1.4926032406009881], ["courtly", 1.4925331121825072], ["chill", 1.4924853801845233], ["wirelessly", 1.492417266143228], ["firework", 1.4922428694354892], ["rungs", 1.492200719470633], ["thanks", 1.4921511582912734], ["smaller", 1.4920287578159992], ["levees", 1.4919708105714171], ["feisty", 1.4918871001517027], ["audition", 1.4918654974627439], ["domingo's", 1.4918468343801945], ["dancehalls", 1.4918468343801945], ["harpal", 1.4918468343801945], ["yelper", 1.4918468343801945], ["spatialization", 1.4918468343801945], ["langrisser", 1.4918468343801945], ["saddar", 1.4918468343801945], ["otaru", 1.4918468343801945], ["curve's", 1.4918468343801945], ["hungarica", 1.4918468343801945], ["swasthya", 1.4918468343801945], ["domantas", 1.4918468343801945], ["ccamlr", 1.4918468343801945], ["fecha", 1.4918468343801945], ["overbudget", 1.4918468343801945], ["bunch's", 1.4918468343801945], ["fangio's", 1.4918468343801945], ["globacom", 1.4918468343801945], ["camogli", 1.4918468343801945], ["echota", 1.4918468343801945], ["interdit", 1.4918468343801945], ["atomico", 1.4918468343801945], ["revel", 1.4915911935517818], ["carpets", 1.4913569341601776], ["nachos", 1.4913301455143073], ["westwood", 1.4911386305166086], ["masterclass", 1.491091851164415], ["succeeded", 1.4907737515694999], ["fr\u00e9d\u00e9ric", 1.4905929306971053], ["kilts", 1.4905929306971053], ["elyse", 1.4904291436750088], ["newbie", 1.4902764692913733], ["gansu", 1.4901320679038956], ["za'atar", 1.4900922365155898], ["lyc\u00e9e", 1.490045470917984], ["kawaii", 1.4899529267353848], ["landforms", 1.4899223673595803], ["\u00fcber", 1.489912511094126], ["aina", 1.4898932701548664], ["pearl's", 1.4898137724348457], ["humiliated", 1.4896715426618274], ["monasteries", 1.489625058587862], ["tripadvisor", 1.489316479809031], ["france's", 1.4892599383186986], ["strike", 1.4892490313010942], ["formulae", 1.4891273566372443], ["procedures", 1.4890311492034694], ["goods", 1.4889554687091184], ["gladiators", 1.4887061042025522], ["pergamon", 1.4887061042025522], ["flask", 1.4887061042025522], ["durable", 1.488706104202552], ["aarti", 1.488706104202552], ["ajax's", 1.488706104202552], ["moping", 1.488706104202552], ["florist's", 1.488706104202552], ["worcestershire's", 1.488706104202552], ["swimmer's", 1.488706104202552], ["florenz", 1.488706104202552], ["wrasse", 1.488706104202552], ["shrimping", 1.488706104202552], ["enumerable", 1.488706104202552], ["callum's", 1.488706104202552], ["salchow", 1.488706104202552], ["spirometer", 1.488706104202552], ["flamboyant", 1.4887061042025518], ["spem", 1.4887061042025518], ["snet", 1.4887061042025518], ["kandar", 1.4887061042025518], ["lutwidge", 1.4887061042025518], ["kermani", 1.4887061042025518], ["vinden", 1.4887061042025518], ["dumbo's", 1.4887061042025518], ["sauger", 1.4887061042025518], ["sherron", 1.4887061042025518], ["trainee's", 1.4887061042025518], ["dharmapala", 1.4887061042025518], ["institutions", 1.4884775747567311], ["doctrinal", 1.4884145440568015], ["violently", 1.4884086009842863], ["people's", 1.4883953096087725], ["stubs", 1.4883390662084193], ["yato", 1.4879890010695065], ["cataracts", 1.4879018329415026], ["stardust", 1.487867868783519], ["elsa", 1.4877837336299358], ["sloth", 1.487721510218291], ["sprain", 1.487654758083765], ["davor", 1.4875160913294883], ["wasi", 1.4875160913294883], ["zeba", 1.4875160913294883], ["mantar", 1.4873985011246402], ["dispersants", 1.4873985011246402], ["pel\u00e9", 1.4870629407321299], ["hopelessness", 1.486996912349277], ["bloodstream", 1.4869949477609397], ["cftr", 1.4869567433280482], ["minoan", 1.4869464043157878], ["parades", 1.4867975066330614], ["distorting", 1.48678766850126], ["marne", 1.486655544830372], ["posadas", 1.4865038762377554], ["transferwise", 1.4865038762377554], ["m\u00e9li\u00e8s", 1.4865038762377554], ["gentlemen", 1.4862249273622146], ["linguistically", 1.4861742911001667], ["squabble", 1.4861547655492486], ["posturing", 1.4858936871291215], ["pbuh", 1.485875864460722], ["antillean", 1.4857346349326865], ["harn", 1.4857346349326865], ["turfs", 1.4857346349326865], ["grendel's", 1.4857346349326865], ["luce", 1.4856585993833042], ["viel", 1.4856585993833042], ["entail", 1.4856429229181847], ["copywriting", 1.4854942247975844], ["salada", 1.4854942247975844], ["schlocky", 1.4854942247975844], ["contradictory", 1.4854197772617075], ["gummy", 1.4852489446179444], ["charleroi", 1.4849467453535556], ["monastery", 1.4848205923720454], ["vashem", 1.484524345482882], ["tlaquepaque", 1.4844647762703507], ["filetype", 1.4844647762703507], ["quavered", 1.4844647762703507], ["marathoned", 1.4844647762703507], ["monterey's", 1.4844647762703507], ["comunitaria", 1.4844647762703507], ["httr", 1.4844647762703507], ["shanhaiguan", 1.4844647762703507], ["pterodactyls", 1.4844647762703507], ["thada", 1.4844647762703507], ["larmes", 1.4844647762703507], ["chillingworth", 1.4844647762703507], ["flourless", 1.4844647762703507], ["tsaritsyn", 1.4844647762703507], ["pohamba", 1.4844647762703507], ["bafokeng", 1.4844647762703507], ["dentelles", 1.4844647762703507], ["scorpio's", 1.4844647762703507], ["fragmentations", 1.4844647762703507], ["flagellants", 1.4844647762703507], ["pedology", 1.4844647762703507], ["royalty's", 1.4844647762703507], ["fillmore's", 1.4844647762703507], ["rollergirl", 1.4844647762703507], ["rheingau", 1.4844647762703507], ["coadjutors", 1.4844647762703507], ["bourdain's", 1.4844647762703507], ["overtrading", 1.4844647762703507], ["danao", 1.4844647762703507], ["tresvant", 1.4844647762703507], ["ranthambore", 1.4844647762703507], ["uncustomary", 1.4844647762703507], ["sigatoka", 1.4844647762703507], ["becontree", 1.4844647762703507], ["grotta", 1.4844647762703507], ["setoff", 1.4844647762703507], ["takato", 1.4844647762703507], ["gl\u00f6gg", 1.4844647762703507], ["albondigas", 1.4844647762703507], ["cpcb", 1.4844647762703507], ["supernova's", 1.4844647762703507], ["r\u00e9forme", 1.4844647762703507], ["individuelle", 1.4844647762703507], ["kirkstone", 1.4844647762703507], ["masz", 1.4844647762703507], ["tekle", 1.4844647762703507], ["indra", 1.4844034854042787], ["fluctuate", 1.4843888057086176], ["derechos", 1.4843312331994378], ["sombreros", 1.4843312331994378], ["hirshhorn", 1.4843312331994378], ["wave", 1.484213805606737], ["funneling", 1.4842085026792815], ["delhi's", 1.4842085026792815], ["bleeps", 1.4840683904822949], ["contort", 1.4840683904822949], ["monk's", 1.484037223051379], ["shepherdess", 1.483995008936088], ["messina's", 1.4839574403932934], ["erin's", 1.4839574403932934], ["scorebook", 1.4839574403932934], ["tagovailoa", 1.4839574403932934], ["arteria", 1.4839574403932934], ["kaikan", 1.4839574403932934], ["legale", 1.4839574403932934], ["linlin", 1.4839574403932934], ["turbografx", 1.4839574403932934], ["ngram", 1.4839574403932934], ["pixelart", 1.4839574403932934], ["maruja", 1.4839574403932934], ["hydrus", 1.4839574403932934], ["elian", 1.483841051574439], ["pompeii", 1.483743750521877], ["lazing", 1.4836766916883541], ["loggerhead", 1.4836766916883541], ["fresco", 1.4836630618441422], ["overhyped", 1.4836424779977815], ["files", 1.4835800617719936], ["taichung", 1.4833122415061661], ["th\u00e9\u00e2tre", 1.4833122415061661], ["cholesterol", 1.4832711454094316], ["extinguished", 1.4831034468211444], ["collect", 1.4828742782957756], ["shortcakes", 1.4827750041858085], ["sundiata", 1.4827750041858085], ["derni\u00e8re", 1.4827750041858085], ["complexe", 1.4827750041858085], ["nayeli", 1.4827750041858085], ["seaquarium", 1.4827750041858085], ["strategos", 1.4827750041858085], ["ahlul", 1.4827750041858085], ["blessings", 1.4827459818992497], ["endemol", 1.4827208091929658], ["edgemont", 1.482700229853777], ["pripyat", 1.4826544533724602], ["taekwondo", 1.482610727571665], ["prescribed", 1.4826023481377608], ["tortoise", 1.4825713263006184], ["grooving", 1.482547154598877], ["bonnets", 1.482547154598877], ["backlinks", 1.482418521566219], ["terry's", 1.482414689076498], ["member", 1.4824045722934827], ["swerving", 1.4821909790200463], ["endpoints", 1.4819392582743585], ["agony", 1.4818485802719708], ["knut", 1.4818139463127253], ["performative", 1.4817332418877858], ["l\u00e9onard", 1.4817332418877858], ["ffion", 1.4817332418877858], ["pickle", 1.481617027515873], ["suzhou", 1.4814793755413747], ["sassi", 1.4812996061716934], ["thibodeaux", 1.4812996061716934], ["bartolome", 1.4812996061716932], ["robotically", 1.4812996061716932], ["subnets", 1.4812996061716932], ["ono's", 1.4812996061716932], ["excellent", 1.4812902963278], ["deduce", 1.4812599143280125], ["satisfy", 1.481171640498909], ["hestia", 1.481012532604606], ["models", 1.4809467875382234], ["gushing", 1.48091183140568], ["bizet", 1.4809118314056797], ["trumpeter", 1.4807821861495776], ["tart", 1.4807536143296751], ["interference", 1.4807064040520264], ["misjudge", 1.4807023079433985], ["spry", 1.4804605014098893], ["elvish", 1.480404769049006], ["misgendering", 1.4801608457118447], ["dumbledore's", 1.4801503219944916], ["reassure", 1.4800219852613705], ["watercolor", 1.4799490094719485], ["aquaculture", 1.4798447583442034], ["jains", 1.479737995141091], ["amritsar", 1.4796836429649607], ["liza", 1.4796127859395347], ["righteousness", 1.479257914247624], ["seaside", 1.4792384557328426], ["love", 1.4790591685298322], ["fibers", 1.4790539288511244], ["bouquet", 1.4790517196097988], ["describing", 1.478927348420045], ["saviors", 1.478864275399543], ["repelling", 1.478864275399543], ["patchwork", 1.4788601643599426], ["fowls", 1.4788471233800187], ["melongena", 1.4788471233800187], ["tasket", 1.4788471233800187], ["paletas", 1.4788471233800187], ["ushmm", 1.4788471233800187], ["dini", 1.4788471233800187], ["swashbucklers", 1.4788471233800187], ["malapascua", 1.4788471233800187], ["misguiding", 1.4788471233800187], ["untappd", 1.4788471233800187], ["nonagon", 1.4788471233800187], ["dodik", 1.4788471233800187], ["sigiriya", 1.4788471233800187], ["guids", 1.4788471233800187], ["eskrima", 1.4788471233800187], ["drako", 1.4788471233800187], ["joueur", 1.4788471233800187], ["marenghi", 1.4788471233800187], ["draymond's", 1.4788471233800187], ["haruka's", 1.4788471233800187], ["bactrians", 1.4788471233800187], ["vraie", 1.4788471233800187], ["cleg", 1.4788471233800187], ["nitori", 1.4788471233800187], ["peripherals", 1.4787676886818628], ["cruelly", 1.4786811472718948], ["diminished", 1.4786329503635744], ["sdks", 1.4785411339069419], ["sunshine", 1.4784391655528792], ["agonized", 1.478271248331973], ["redwood", 1.478243310297514], ["simmers", 1.4781479048819663], ["depict", 1.4780862811505235], ["denuded", 1.477984969707266], ["potatoes", 1.4778131327083868], ["rethink", 1.477772104567166], ["spindly", 1.4777597357892978], ["specialty", 1.477675098920123], ["tetsuo", 1.477554747616765], ["virtuosity", 1.4774847014070553], ["whizz", 1.477464526285165], ["brainiac", 1.4773673928354178], ["catch", 1.4772377767016662], ["figure", 1.4771272789476433], ["duress", 1.4768909763914209], ["earpieces", 1.4768909763914206], ["thant", 1.4768909763914206], ["martine's", 1.4768909763914204], ["autorickshaws", 1.4768909763914204], ["kabak", 1.4768909763914204], ["gesamte", 1.4768909763914204], ["suppe", 1.4768909763914204], ["dench's", 1.4768909763914204], ["anmer", 1.4768909763914204], ["zonas", 1.4768909763914204], ["turistas", 1.4768909763914204], ["hazare's", 1.4768909763914204], ["maboneng", 1.4768909763914204], ["sledders", 1.4768909763914204], ["bbb's", 1.4768909763914204], ["sumfest", 1.4768909763914204], ["straphangers", 1.4768909763914204], ["jordy's", 1.4768909763914204], ["sarama", 1.4768909763914204], ["lightblue", 1.4768909763914204], ["yuma's", 1.4768909763914204], ["idul", 1.4768909763914204], ["ho\u00e0ng", 1.4768909763914204], ["mulatu", 1.4768909763914204], ["asom", 1.4768909763914204], ["vaccinated", 1.4768344446794057], ["negative", 1.4767827827574938], ["grates", 1.476761708034625], ["marshy", 1.4764944955923893], ["introduce", 1.4764592697983216], ["padi", 1.4764417319879555], ["tetrahedron", 1.4763002200008641], ["neurotoxins", 1.4763002200008641], ["mayumi", 1.4763002200008641], ["amadeus", 1.4762743204513824], ["mahalaxmi", 1.4762743204513824], ["bsod", 1.4761809326527708], ["abraham's", 1.4761431834919818], ["hazelnuts", 1.4760899507771068], ["buttons", 1.4759754586343277], ["rocky", 1.4759579887125416], ["flicka", 1.4759412072747702], ["processing", 1.4759125361195613], ["indiscernible", 1.4758964370133791], ["quicker", 1.4758827142825381], ["technically", 1.4758079230087335], ["underworlds", 1.4757929161859253], ["oldman's", 1.4757929161859253], ["fatphobia", 1.4757929161859253], ["mikrotik", 1.4757929161859253], ["djilas", 1.4757929161859253], ["intrinsically", 1.4757608337312256], ["grazed", 1.475596793577057], ["authorities", 1.4754576658044964], ["herein", 1.4754043902882694], ["crabs", 1.4753611291648796], ["expediency", 1.4752943374980245], ["prepositional", 1.4752943374980245], ["compete", 1.4752368584978623], ["backoff", 1.475163618524296], ["beetle's", 1.475163618524296], ["peierls", 1.475163618524296], ["eldest", 1.474781132139433], ["tightening", 1.474439337370611], ["michael's", 1.4744077969870473], ["kevins", 1.4743835214230143], ["cacha\u00e7a", 1.4743835214230143], ["samurai's", 1.4743835214230143], ["vivarium", 1.4743835214230143], ["gideon's", 1.4743224703455227], ["ankit", 1.4743224703455227], ["crammed", 1.4742992709360756], ["instagram", 1.474039334562239], ["blinky", 1.4740324777274374], ["imitation", 1.4739872585395322], ["luc\u00eda", 1.4738614564398484], ["site's", 1.4738190431605265], ["catchphrases", 1.4737486551650558], ["succinctly", 1.473657546443108], ["extravagances", 1.473627128929782], ["au's", 1.473627128929782], ["exceeding", 1.4734654602201342], ["stockpiled", 1.4734496770289554], ["puree", 1.4733033869750851], ["counterfeiter", 1.4733033869750851], ["raleigh's", 1.4733033869750851], ["untuk", 1.4733033869750851], ["gostkowski", 1.4733033869750851], ["parenthood's", 1.4733033869750851], ["pedalboard", 1.4733033869750851], ["conda", 1.4733033869750851], ["cracking", 1.4731987489504421], ["ni\u00f1os", 1.4731987489504421], ["shrieked", 1.4731987489504421], ["greendale", 1.4731987489504421], ["tartare", 1.4731987489504421], ["hitch", 1.4731397856224875], ["dispersing", 1.4730701639151842], ["mahjong", 1.4730171639943281], ["doom", 1.472958697011555], ["losses", 1.4728688052216736], ["kotte", 1.4728446997437727], ["sciency", 1.4728446997437727], ["brosnan's", 1.4728446997437727], ["nerdgasm", 1.4728446997437727], ["solidarit\u00e9", 1.4728446997437727], ["pinballs", 1.4728446997437727], ["taklamakan", 1.4728446997437727], ["mechanic", 1.472821746796825], ["under", 1.4728210278951392], ["peace", 1.47270842114628], ["criteria", 1.4727009819917323], ["oomph", 1.4726409304161936], ["ecologically", 1.4725843529420384], ["usted", 1.4725683686556408], ["futura", 1.4723466964640624], ["diatonic", 1.4722645709232063], ["luxury", 1.4722572309529924], ["fruiting", 1.4722462087015409], ["coordination", 1.4721440925479057], ["child's", 1.471944827696689], ["chechnya's", 1.4718845663019582], ["paramore's", 1.4718845663019582], ["proctored", 1.4718845663019582], ["marisela", 1.4718845663019582], ["ripening", 1.471610472341753], ["wisbech", 1.4713955681071733], ["hemingway", 1.4713378663201888], ["chiara", 1.4712505887428904], ["quickening", 1.4711810633228442], ["overslept", 1.4711810633228442], ["castleford", 1.4711810633228442], ["funerary", 1.4711402504656486], ["premise", 1.4710638483030258], ["porcupine", 1.4710534626507432], ["jaro", 1.471053462650743], ["zephyrs", 1.471053462650743], ["hume's", 1.471053462650743], ["phonetic", 1.470983412485855], ["ramekin", 1.4709437644059515], ["yakisoba", 1.4709437644059515], ["philosophy", 1.470840025012419], ["dreidel", 1.4708007043238271], ["barramundi", 1.4708007043238271], ["invalides", 1.4708007043238271], ["arapaho", 1.4706313060279075], ["classifications", 1.4706313060279073], ["impressively", 1.4706225002215276], ["circadian", 1.4706225002215276], ["andr\u00e9", 1.4705851497506484], ["growlers", 1.4705851497506481], ["realpolitik", 1.4705851497506481], ["royalty", 1.4704216789924571], ["riff", 1.4704185458175936], ["grazalema", 1.4703270164963478], ["oversexualized", 1.4703270164963478], ["jasmin's", 1.4703270164963478], ["paddleboats", 1.4703270164963478], ["innervisions", 1.4703270164963478], ["reshare", 1.4703270164963478], ["latt\u00e9", 1.4703270164963478], ["akhal", 1.4703270164963478], ["rooky", 1.4703270164963478], ["meridiem", 1.4703270164963478], ["gruta", 1.4703270164963478], ["hlhs", 1.4703270164963478], ["balthazar's", 1.4703270164963478], ["durrell's", 1.4703270164963478], ["kasongo", 1.4703270164963478], ["nytt", 1.4703270164963478], ["natal's", 1.4703270164963478], ["arezzo", 1.4703270164963476], ["adopted", 1.4702121471981837], ["suzie", 1.4701709970760912], ["bogot\u00e1", 1.4701118046965092], ["downplaying", 1.4701118046965092], ["naps", 1.47009727790002], ["belleza", 1.470061422743368], ["hayek's", 1.470061422743368], ["neagh", 1.470061422743368], ["wojtek", 1.470061422743368], ["ruffle", 1.4698057295678897], ["scuffle", 1.4698057295678897], ["benefactors", 1.4698057295678897], ["electricity", 1.4697975074639198], ["volver", 1.469567550062207], ["eyre's", 1.469567550062207], ["marauding", 1.4694001172604438], ["cabinetry", 1.4693494054481453], ["titles", 1.4693490630940818], ["bellini", 1.4691980799050575], ["family", 1.4691770164679911], ["liesel", 1.4691178659893604], ["lightings", 1.4691178659893604], ["ipcc's", 1.4691178659893604], ["catus", 1.4691178659893604], ["tutted", 1.4691178659893604], ["vernazza", 1.4691178659893604], ["zainal", 1.4691178659893604], ["iruka", 1.4691178659893604], ["whistleblower", 1.468988142557485], ["hristo", 1.4687066939026017], ["portability", 1.4687066939026014], ["excuse", 1.4686789743342623], ["chagrin", 1.4684865691777014], ["skit", 1.468469555504763], ["brining", 1.4684515993834697], ["lysander", 1.4684515993834697], ["bunratty", 1.4684515993834695], ["schumi", 1.4684515993834695], ["longfellow's", 1.4684515993834695], ["fumarole", 1.4684515993834695], ["slamet", 1.4684515993834695], ["corduroys", 1.4684515993834695], ["primeiro", 1.4684515993834695], ["clouzot", 1.4684515993834695], ["petg", 1.4684515993834695], ["eargasm", 1.4684515993834695], ["quadrifoglio", 1.4684515993834695], ["logica", 1.4681519765311164], ["gentleness", 1.46806473490263], ["medicis", 1.4680296305330722], ["krasnodar", 1.4680296305330722], ["michel's", 1.4679213767442267], ["costs", 1.4679208069874432], ["tepid", 1.4679087154823733], ["tangent", 1.467896233928753], ["solarization", 1.4677384125940653], ["multiplies", 1.4677384125940653], ["triumvirate", 1.4677384125940653], ["responders", 1.467711530938157], ["unrefined", 1.4676791818268098], ["rosie's", 1.4676604077889743], ["hops", 1.4675781791429088], ["pacha", 1.4674664580984949], ["farming's", 1.4673626475114758], ["salamander's", 1.4673626475114758], ["ligero", 1.4673626475114758], ["laurelwood", 1.4673626475114758], ["chargrilled", 1.4673626475114758], ["gamma's", 1.4673626475114758], ["pevensie", 1.4673626475114758], ["nordictrack", 1.4673626475114758], ["stefano's", 1.4673626475114758], ["bewick's", 1.4673626475114758], ["jacky's", 1.4673626475114758], ["selvaggio", 1.4673626475114758], ["niran", 1.4673626475114758], ["laurasia", 1.4673626475114758], ["incensing", 1.4673626475114758], ["iacob", 1.4673626475114758], ["orlando's", 1.4671799124370661], ["podemos", 1.4671799124370661], ["trekker", 1.4671306534170079], ["cratchit", 1.4671306534170079], ["microalgae", 1.4671306534170079], ["roppongi", 1.4671306534170079], ["pasado", 1.4671306534170079], ["satire", 1.4670582525754452], ["kadhim", 1.4670197256494728], ["oceanview", 1.4670197256494728], ["counteracted", 1.466911983307695], ["underperformed", 1.4664980398908884], ["investigation", 1.4664329064098418], ["having", 1.4663244858664475], ["bytes", 1.4662119185621287], ["webcomics", 1.4661499511085738], ["vernal", 1.4661499511085738], ["cluj", 1.4661499511085738], ["prospector's", 1.4661499511085738], ["starker", 1.4661499511085738], ["am\u00e9ricains", 1.4661499511085738], ["bita", 1.4661499511085738], ["slope's", 1.4661499511085738], ["gardon", 1.4661499511085738], ["authentique", 1.4661499511085738], ["chugach", 1.4661499511085738], ["equipos", 1.4661499511085738], ["hiero", 1.4661499511085738], ["babka", 1.4661499511085738], ["botta", 1.4661499511085738], ["powerboating", 1.4661499511085738], ["improvisor", 1.4661499511085738], ["erminio", 1.4661499511085738], ["nimbo", 1.4661499511085738], ["book2", 1.4661499511085738], ["linderhof", 1.4661499511085738], ["qeshm", 1.4661499511085738], ["mantequilla", 1.4661499511085738], ["evangelio", 1.4661499511085738], ["homestead's", 1.4661499511085738], ["paczki", 1.4661499511085738], ["s\u016brah", 1.4661499511085738], ["chronometry", 1.4661499511085738], ["stef\u00e1nsson", 1.4661499511085738], ["glucomannan", 1.4661499511085738], ["h\u00e4yh\u00e4", 1.4661499511085738], ["geirr", 1.4661499511085738], ["flumps", 1.4661499511085738], ["spells", 1.4659963918212473], ["bernardo's", 1.465903166063782], ["chimay", 1.465903166063782], ["shiism", 1.465903166063782], ["moderator's", 1.465824934493387], ["aoraki", 1.465824934493387], ["stylists", 1.4657843732626423], ["fifteen", 1.4656023923196981], ["legato", 1.4655302765730251], ["krabi", 1.465332699184879], ["carpal", 1.4650758485802893], ["maxy", 1.4650758485802893], ["frappuccinos", 1.4650758485802893], ["nishiki", 1.4650758485802893], ["bran's", 1.4650758485802893], ["shrine's", 1.4650758485802893], ["immersions", 1.4650164760787256], ["sanat", 1.4650164760787256], ["tractor's", 1.4650164760787256], ["r\u00e9gion", 1.4650164760787256], ["guanaco", 1.4650164760787256], ["darcy", 1.4649627851562592], ["exacted", 1.4649501557312345], ["architects", 1.4649014555498667], ["elle's", 1.4649014555498667], ["lyanna", 1.4649014555498667], ["overbooked", 1.4646947154250916], ["resistors", 1.4646831492455263], ["praises", 1.464583551588159], ["kontinental", 1.464583551588159], ["adventurously", 1.464583551588159], ["grammatica", 1.464583551588159], ["inkheart", 1.464583551588159], ["albigensian", 1.464583551588159], ["gamesa", 1.464583551588159], ["aeta", 1.464583551588159], ["hammond's", 1.4643654453258224], ["hemoglobin", 1.4642521978434553], ["break", 1.4642229684932604], ["donations", 1.4640912545960283], ["nora", 1.4639692958245638], ["carrey's", 1.463931872513856], ["storming", 1.463894335799176], ["ssri", 1.463894335799176], ["hyacinths", 1.463894335799176], ["pashmina", 1.463894335799176], ["bustards", 1.4638211447419391], ["orenda", 1.4638211447419391], ["valedictorians", 1.4638211447419391], ["legwarmers", 1.4638211447419391], ["thenceforward", 1.4638211447419391], ["scottie's", 1.4638211447419391], ["praxiteles", 1.4638211447419391], ["pcaa", 1.4638211447419391], ["critic", 1.4637829473888355], ["frogs", 1.4637576318410308], ["monstrously", 1.4636155518845315], ["llama's", 1.4636155518845313], ["dpos", 1.4636155518845313], ["yelpers", 1.4636155518845313], ["carpetbag", 1.4636155518845313], ["ottonian", 1.4636155518845313], ["serica", 1.4636155518845313], ["overpopulate", 1.4636155518845313], ["flagg's", 1.4636155518845313], ["ayora", 1.4636155518845313], ["adore", 1.463448615408696], ["harvested", 1.4634183306499506], ["oder", 1.4633566194560688], ["gesehen", 1.463258136609346], ["alium", 1.463258136609346], ["diverters", 1.463258136609346], ["lawang", 1.463258136609346], ["lejos", 1.463258136609346], ["wakrah", 1.463258136609346], ["zwinger", 1.463258136609346], ["estadual", 1.463258136609346], ["batwa", 1.463258136609346], ["kasowitz", 1.463258136609346], ["glories", 1.4631233486453987], ["lincei", 1.4631018223120904], ["frizell", 1.4631018223120904], ["tituba", 1.4631018223120904], ["tania's", 1.4631018223120904], ["carafe", 1.4630387575783703], ["melisma", 1.4627025041291448], ["palpating", 1.4627025041291448], ["tramp's", 1.4627025041291448], ["mentalhealth", 1.4627025041291448], ["buzkashi", 1.4627025041291448], ["vishwamitra", 1.4627025041291448], ["cricketer's", 1.4627025041291448], ["atalanta", 1.4626690632854291], ["skimmers", 1.4626690632854291], ["trabajo", 1.4626690632854291], ["headed", 1.4625314358446244], ["firmware", 1.4624348200107422], ["balsa", 1.462415488675796], ["fickle", 1.4623905922595775], ["pantiles", 1.4622905332788059], ["handiness", 1.4622905332788059], ["duma's", 1.4622905332788059], ["kasuri", 1.4622905332788059], ["savviest", 1.4622905332788059], ["fougasse", 1.4622905332788059], ["statesman's", 1.4622905332788059], ["stderr", 1.4622905332788059], ["boomboxes", 1.4622905332788059], ["goncalo", 1.4622905332788059], ["thiebaud", 1.4622905332788059], ["meridians", 1.4622905332788056], ["archives", 1.4621220666275065], ["charring", 1.4621220666275065], ["buckets", 1.4620098976925906], ["folky", 1.4619728857208003], ["frise", 1.4619728857208003], ["lashed", 1.4619483147479309], ["steeper", 1.4618495775174711], ["lunar", 1.4617459483822453], ["borromini", 1.4617204950629288], ["voldermort", 1.4617204950629288], ["animales", 1.4617204950629288], ["bairro", 1.4617204950629288], ["active", 1.461605588655034], ["landfill", 1.4615682325113595], ["chafed", 1.4614261494135001], ["tomorrowland", 1.461405129260326], ["rejoining", 1.4613957268030515], ["personality", 1.4613782128061577], ["prairies", 1.461371105114087], ["misters", 1.4612700141411266], ["pulmonology", 1.4609723910171417], ["hefeweizen", 1.4609723910171417], ["chuma", 1.4609723910171417], ["cued", 1.460879821881009], ["degas", 1.4608561600763137], ["sematary", 1.46072600053042], ["juneteenth", 1.46072600053042], ["assumed", 1.4607042512901707], ["riesling", 1.4606928172955147], ["accounted", 1.4603658846192709], ["schindler's", 1.4602759529070173], ["scuola", 1.4602627258895726], ["liqueurs", 1.4602141213470008], ["unleashed", 1.4601094897715425], ["moon's", 1.4600771406601951], ["debug", 1.4599189695909005], ["competed", 1.4596058820010618], ["basilico", 1.4595157884338745], ["funicular", 1.4595157884338745], ["ganondorf", 1.4595157884338745], ["mellor's", 1.4595157884338745], ["singita", 1.4595157884338745], ["zaroff", 1.4595157884338745], ["odra", 1.4595157884338745], ["insect's", 1.4595157884338745], ["projector's", 1.4595157884338745], ["windchimes", 1.4595157884338745], ["laney's", 1.4595157884338745], ["cplp", 1.4595157884338745], ["adland", 1.4595157884338745], ["tipitaka", 1.4595157884338745], ["technovation", 1.4595157884338745], ["m\u00fchle", 1.4595157884338745], ["medlars", 1.4595157884338745], ["mfcs", 1.4595157884338745], ["einhard", 1.4595157884338745], ["greenway's", 1.4595157884338745], ["cpted", 1.4595157884338743], ["v\u00e9lodrome", 1.4595157884338743], ["kotagiri", 1.4595157884338743], ["earliness", 1.4595157884338743], ["sagrado", 1.4595157884338743], ["donck", 1.4595157884338743], ["superpowered", 1.4595157884338743], ["cholita", 1.4595157884338743], ["cyropaedia", 1.4595157884338743], ["temprano", 1.4595157884338743], ["upticks", 1.4595157884338743], ["fjord's", 1.4595157884338743], ["lqts", 1.4595157884338743], ["menteri", 1.4595157884338743], ["toaster's", 1.4595157884338743], ["towson's", 1.4595157884338743], ["dibon", 1.4595157884338743], ["brollies", 1.4595157884338743], ["historischen", 1.4595157884338743], ["gdzie", 1.4595157884338743], ["khayy\u00e1m", 1.4595157884338743], ["constituci\u00f3n", 1.4595157884338743], ["cascata", 1.4595157884338743], ["endtroducing", 1.4595157884338743], ["beatmakers", 1.4595157884338743], ["virtuelle", 1.4595157884338743], ["faucibus", 1.4595157884338743], ["bikeable", 1.4595157884338743], ["walachia", 1.4595157884338743], ["peninnah", 1.4595157884338743], ["callisto's", 1.4595157884338743], ["karyo", 1.4595157884338743], ["dreamer", 1.459477581214282], ["miffed", 1.4593237468827647], ["roaming", 1.4592617587871535], ["badges", 1.4592617587871535], ["actual", 1.4592508904411996], ["monica's", 1.4591818031275967], ["caregiver", 1.4588935142605113], ["tangling", 1.458713856681988], ["dust", 1.4586326961961256], ["saccharine", 1.4583075799997405], ["eclipses", 1.4582054496735402], ["vieux", 1.4580620874493786], ["albums", 1.4579715265674025], ["reproduction", 1.45794272846741], ["curtain", 1.457862745690068], ["brca1", 1.4578120735057416], ["aveeno", 1.4578007287529886], ["altenberg", 1.4577657695028985], ["timoth\u00e9e", 1.4577657695028985], ["ajay's", 1.4577657695028985], ["raspbian", 1.4577657695028985], ["m\u00e9thode", 1.4577657695028985], ["niclas", 1.4577657695028985], ["squirming", 1.45775925958565], ["unchanging", 1.4577491492113752], ["newsboy", 1.4576913936983322], ["poder", 1.4576913936983322], ["holst", 1.4573902800041165], ["salicylic", 1.457357902187534], ["nate's", 1.457345504733601], ["eggs", 1.4573380615533473], ["oskar", 1.4573257325884217], ["showboating", 1.4573177224874378], ["estonia's", 1.4573177224874378], ["ezio", 1.4572850874471852], ["tubman", 1.457254566789822], ["doro", 1.4571990967062018], ["markers", 1.45704156166872], ["beetles", 1.4569832952849937], ["diagnostician", 1.456918785251608], ["hoan", 1.456918785251608], ["midsole", 1.456918785251608], ["unenthusiastic", 1.4566459540184147], ["employer's", 1.4566112794021653], ["bookcases", 1.4565837254660385], ["conjuring", 1.4565613030321864], ["entreaty", 1.4565613030321864], ["parrilla", 1.4565613030321864], ["pamela's", 1.4565613030321864], ["monopod", 1.45649192975618], ["huac", 1.45649192975618], ["spot's", 1.45649192975618], ["maison", 1.4564306331087296], ["sweethearts", 1.4564245295778875], ["skirmish", 1.4563429280242357], ["showtimes", 1.4563429280242357], ["electromagnets", 1.4563429280242357], ["highlife", 1.4563429280242357], ["papyrus", 1.4562070624651327], ["autocratic", 1.4562070624651327], ["wrestlers", 1.4561583471715076], ["nagorno", 1.4561516562974135], ["interface", 1.4559996822162835], ["summits", 1.4559996822162835], ["surfaces", 1.4559782001657182], ["bradley's", 1.4558971542476749], ["reintroduce", 1.455817210412502], ["transforms", 1.4557117313257222], ["giselle", 1.4556749574116798], ["lips", 1.455642207468589], ["earhart", 1.4555850082074133], ["rollbacks", 1.4555149227857553], ["homeruns", 1.4555149227857553], ["squealing", 1.4554389286896459], ["reapplication", 1.4554389286896459], ["kombu", 1.4554389286896459], ["agribusiness", 1.455427161861861], ["mongolian", 1.4554220246370477], ["called", 1.4552070276846778], ["lizards", 1.4551959572209812], ["correlation", 1.455121113822292], ["prin", 1.4550826797306873], ["sheryl's", 1.4550111100745107], ["c\u00e9vennes", 1.4550111100745107], ["carnivals", 1.4550111100745107], ["marvelettes", 1.4550111100745107], ["murkiness", 1.4550111100745107], ["jame", 1.4550111100745107], ["hartono", 1.4550111100745107], ["burrowers", 1.4550111100745107], ["bruny", 1.4550111100745107], ["langle", 1.4550111100745107], ["deconstructivist", 1.4550111100745107], ["tirano", 1.4550111100745107], ["selenites", 1.4550111100745107], ["earthship", 1.4550111100745107], ["gauda", 1.4550111100745107], ["bollea", 1.4550111100745107], ["kleo", 1.4550111100745107], ["jmsdf", 1.4550111100745107], ["b\u00e9dard", 1.4550111100745107], ["torlonia", 1.4550111100745107], ["counselor's", 1.454930922082773], ["biscay", 1.454930922082773], ["vermiculite", 1.4548718745615847], ["cosas", 1.4548718745615847], ["reductions", 1.4547134004180675], ["perpetrator's", 1.454708424176846], ["africanized", 1.454708424176846], ["arminianism", 1.454708424176846], ["lyricists", 1.4544829753703095], ["tempsford", 1.4544829753703095], ["participat", 1.4544829753703095], ["acc\u00e8s", 1.4544829753703095], ["lembang", 1.4544829753703095], ["yono", 1.4544829753703095], ["fluss", 1.4544829753703095], ["riverwest", 1.4544829753703095], ["nspe", 1.4544829753703095], ["jet\u00e9e", 1.4544829753703095], ["broche", 1.4544829753703095], ["platyrhynchos", 1.4544829753703095], ["pavement's", 1.4544829753703095], ["zond", 1.4544829753703095], ["drink's", 1.4543621309888397], ["hugos", 1.4543621309888397], ["dropshipping", 1.4543621309888397], ["dwelt", 1.454291570846639], ["acharya", 1.4542244925241445], ["buddy", 1.4542068724384956], ["waterless", 1.454199671323685], ["shipment", 1.4541853829456812], ["news", 1.4540869204339144], ["retakes", 1.4540229870795536], ["civility", 1.4539232513006233], ["installation", 1.4537985127936643], ["nematodes", 1.4535040529208747], ["liquids", 1.4533780895884738], ["planeta", 1.4533208996883997], ["natgeo", 1.4533208996883997], ["kukri", 1.4533208996883997], ["kookaburras", 1.4533208996883997], ["unschooled", 1.4533208996883997], ["shakespearian", 1.453260720769158], ["interconnections", 1.453260720769158], ["waterskiing", 1.453260720769158], ["cruelty", 1.4531066104064039], ["anticonvulsants", 1.4530821546057655], ["gujarat's", 1.4530821546057655], ["local's", 1.4530821546057655], ["figurative", 1.4529921951370868], ["kabobs", 1.4528130752837762], ["pavlov's", 1.4528130752837762], ["ties", 1.4527068866530113], ["lightshow", 1.4526796489095941], ["germy", 1.4526796489095941], ["fundraiser", 1.4526684457246408], ["jose's", 1.452616259252187], ["reminds", 1.4526094652301709], ["kurtz's", 1.4525877071596631], ["intraplate", 1.4525877071596631], ["greensward", 1.4525877071596631], ["incongruity", 1.452587707159663], ["motes", 1.4525222752809623], ["merlot", 1.4525038798417544], ["solomon's", 1.4524795552940417], ["lycopene", 1.452396199222002], ["electrolytes", 1.4523328311504826], ["politicos", 1.4522182094917053], ["eastwood's", 1.4522182094917053], ["stateful", 1.4521338444172291], ["honestly", 1.4519948142179653], ["exothermic", 1.4518974367909505], ["clicks", 1.451842905431822], ["infringements", 1.4518157051375806], ["marjorie", 1.451767235512133], ["memphis's", 1.4517524065805028], ["outputting", 1.4517524065805028], ["havel's", 1.4517524065805028], ["pappalardo", 1.4517524065805028], ["sviatoslav", 1.4517524065805028], ["quelled", 1.4516607555447338], ["progressivism", 1.4515242034251585], ["geospatial", 1.4510738590663264], ["theoretical", 1.4510019463346768], ["impostor", 1.4509576134462274], ["seesaw", 1.450857643926216], ["respectability", 1.450794022375911], ["refurbish", 1.4506880719715727], ["motherhood", 1.450534152812743], ["ranger's", 1.4505341528127427], ["raja", 1.4503938147561628], ["mullioned", 1.4503747709697792], ["talitha", 1.4503747709697792], ["samudra", 1.4503747709697792], ["dene", 1.4500917626476293], ["airplanes", 1.4500334404874138], ["spikes", 1.4499148438668905], ["eyeliners", 1.4498000768244774], ["reprieves", 1.4498000768244774], ["baize", 1.4498000768244774], ["scriabin", 1.4498000768244774], ["avez", 1.4498000768244774], ["cogburn", 1.4498000768244774], ["teenage", 1.4496904973653595], ["workmanship", 1.4493703738069559], ["irina", 1.449322344832114], ["eroded", 1.4492855375803662], ["lothbrok", 1.449285537580366], ["tomislav", 1.449285537580366], ["pseudoscientific", 1.449285537580366], ["pragmatically", 1.449285537580366], ["antirheumatic", 1.449285537580366], ["dendera", 1.449285537580366], ["ethologists", 1.449285537580366], ["husna", 1.449285537580366], ["tranh", 1.449285537580366], ["identifiers", 1.4490539243804708], ["oahu", 1.4488345828173932], ["kaunas", 1.4486871229067846], ["trainees", 1.4486871229067844], ["infamously", 1.4486617944376097], ["nurse", 1.4485723274225892], ["embezzlement", 1.4485346696447055], ["hindu", 1.448511039389083], ["controlling", 1.4484708040889693], ["laziness", 1.4484708040889693], ["toluca", 1.4483424618238543], ["deface", 1.4483424618238543], ["pain", 1.4482420519933081], ["paddy's", 1.4481769207548065], ["defining", 1.4481693334558083], ["pleurotus", 1.4481576889129884], ["changsha", 1.4481576889129884], ["bouncier", 1.4481576889129884], ["vechain", 1.4481576889129884], ["pi\u00f1as", 1.4481576889129884], ["jamieson's", 1.4481576889129884], ["kogen", 1.4481576889129884], ["detailers", 1.4481576889129884], ["jeera", 1.4481576889129884], ["locke's", 1.448121799007439], ["bodega", 1.4481050286333914], ["spidery", 1.448049334428221], ["lair", 1.4478396621264036], ["whenua", 1.4478263027141522], ["unfinished", 1.4478257619760375], ["suzuki's", 1.4473531568635924], ["azeem", 1.4473531568635924], ["disinformation", 1.4473531568635922], ["raccoon", 1.4473531568635922], ["ledger's", 1.4473531568635922], ["untranslated", 1.4473531568635922], ["laertes", 1.4473531568635922], ["malbec", 1.4473531568635922], ["plebeians", 1.4473531568635922], ["naivet\u00e9", 1.4473531568635922], ["valencia's", 1.4473531568635922], ["misrouted", 1.447353156863592], ["dragonheart", 1.447353156863592], ["shylock's", 1.447353156863592], ["tunelessly", 1.447353156863592], ["streamer's", 1.447353156863592], ["verti", 1.447353156863592], ["candomble", 1.447353156863592], ["rokon", 1.447353156863592], ["meilleure", 1.447353156863592], ["pescador", 1.447353156863592], ["codeigniter", 1.447353156863592], ["redlined", 1.447353156863592], ["twizzles", 1.447353156863592], ["lachen", 1.447353156863592], ["gutes", 1.447353156863592], ["tokaji", 1.447353156863592], ["brannstrom", 1.447353156863592], ["epsxe", 1.447353156863592], ["chieftain's", 1.447353156863592], ["tolani", 1.447353156863592], ["stressfully", 1.447353156863592], ["rostova", 1.447353156863592], ["lerp", 1.447353156863592], ["asuna's", 1.447353156863592], ["taotie", 1.447353156863592], ["mollie's", 1.447353156863592], ["maramures", 1.447353156863592], ["planification", 1.447353156863592], ["huri", 1.447353156863592], ["aaditya", 1.447353156863592], ["keefe's", 1.447353156863592], ["optician's", 1.447353156863592], ["calvinball", 1.447353156863592], ["puoi", 1.447353156863592], ["pfdj", 1.447353156863592], ["aksumite", 1.447353156863592], ["seiss", 1.447353156863592], ["adji", 1.447353156863592], ["highs", 1.4472468510863712], ["buses", 1.447235005585481], ["villages", 1.4472134509604198], ["guarded", 1.4471594662437375], ["knuckles", 1.44715529587154], ["czech", 1.4469354503248149], ["woods", 1.446869239394732], ["interchangeably", 1.446832338887031], ["buddhist", 1.4468274838041986], ["opportunists", 1.4466522594510676], ["somewhat", 1.4466309363748553], ["fuss", 1.4464606472087944], ["dethrone", 1.4464502540832656], ["thermidor", 1.4463590956294825], ["overachieving", 1.4463590956294825], ["videoconference", 1.4463590956294825], ["injuries", 1.4463456486920612], ["pianists", 1.4462649214073036], ["walking", 1.4460565934594563], ["from", 1.4459432828416703], ["cymbals", 1.4458815217269745], ["loot", 1.4457472171611083], ["phoenician", 1.4456652814620021], ["kavin", 1.4455552026314635], ["bellagio", 1.4455233804326648], ["misbehavior", 1.4455233804326648], ["masking", 1.4455152480929716], ["shoelace", 1.445507043143103], ["raging", 1.4454104009483388], ["gossip", 1.4453917522467243], ["midler's", 1.4453457322354875], ["broodmares", 1.4453457322354875], ["fectly", 1.4453457322354875], ["phagocyte", 1.4453457322354875], ["bookseller's", 1.4453457322354875], ["sruti", 1.4453457322354875], ["hammett's", 1.4453457322354875], ["tangency", 1.4453457322354875], ["jorah", 1.4452855094966444], ["lifeguard", 1.4452324928974916], ["gatekeepers", 1.4451948610734586], ["fala", 1.4451635151739346], ["tapenade", 1.4451635151739346], ["gifts", 1.445130435944123], ["batley", 1.445081016899913], ["scans", 1.4449731311174652], ["lillehammer", 1.444920630549536], ["recrimination", 1.4447358817155567], ["staircase", 1.4443060975859845], ["enfants", 1.4441721609144413], ["honor\u00e9", 1.4441721609144413], ["disengagement", 1.443808618520253], ["oman's", 1.4435937980145959], ["limone", 1.4435937980145959], ["morse's", 1.4435937980145959], ["samadi", 1.4435937980145959], ["kaffe", 1.4435937980145959], ["rugova", 1.4435937980145959], ["stern's", 1.4434883954300446], ["mikkel", 1.4434883954300446], ["dubois", 1.4434413915747715], ["idealists", 1.4434222683332727], ["biscuit", 1.4433938321183726], ["interplanetary", 1.4432866794399424], ["master's", 1.4432647779532575], ["jantar", 1.4432227214654587], ["mouthparts", 1.4432227214654587], ["sanchi", 1.4431901084737637], ["span's", 1.4431901084737637], ["consumptions", 1.4431901084737637], ["regurgitation", 1.4431334683596169], ["spotlight", 1.4430824494557903], ["rossini", 1.442980488715968], ["doktor", 1.4429539071466815], ["camouflaged", 1.4427895888210924], ["hetfield", 1.442758384937041], ["adler", 1.4426580624058938], ["dance", 1.4426104451593824], ["overlapping", 1.442568518328506], ["scoreboards", 1.442544674614876], ["borgu", 1.4425446746148758], ["reefton", 1.4425446746148758], ["uncurled", 1.4425446746148758], ["kidul", 1.4425446746148758], ["obols", 1.4425446746148758], ["thinset", 1.4425446746148758], ["belloq", 1.4425446746148758], ["deallocate", 1.4425446746148758], ["ranya", 1.4425446746148758], ["sorenstam", 1.4425446746148758], ["backpain", 1.4425446746148758], ["sergey's", 1.4425446746148758], ["sonata's", 1.4425446746148758], ["turid", 1.4425446746148758], ["nesterenko", 1.4425446746148758], ["etrusco", 1.4425446746148758], ["brillantes", 1.4425446746148758], ["margueritte", 1.4425446746148758], ["mirar", 1.4425446746148758], ["theyear", 1.4425446746148758], ["coevolved", 1.4425446746148758], ["kecap", 1.4425446746148758], ["bernini", 1.442406632062281], ["tonbridge", 1.442406632062281], ["period's", 1.4423101145051824], ["subjective", 1.4422594388769454], ["flute", 1.4422308884620896], ["folk", 1.442071937082954], ["recirculating", 1.442051496948341], ["prabang", 1.442051496948341], ["blurts", 1.442051496948341], ["bidding", 1.4420400072828898], ["lamps", 1.4420396496636139], ["janusz", 1.4419404674213199], ["mirin", 1.4419404674213199], ["peggy's", 1.4419404674213199], ["collapsed", 1.4418858200759892], ["tank's", 1.441875945295518], ["fabrications", 1.4418638275708098], ["christendom", 1.4415776055592682], ["monarchy", 1.4415213309568158], ["kitten's", 1.4414455929580263], ["longhaired", 1.4414455929580263], ["colmar", 1.4414455929580263], ["axel's", 1.4414455929580263], ["pedra", 1.4414455929580263], ["lumpia", 1.4414455929580263], ["cardcaptor", 1.4414455929580263], ["subic", 1.441271841078451], ["dumbbell", 1.4410582117183175], ["cordially", 1.4410582117183175], ["toucan", 1.4410328374013057], ["wartime", 1.4408685044531127], ["aunt", 1.4407937238374124], ["compiled", 1.4407713265017892], ["driest", 1.440683326647631], ["ponente", 1.440683326647631], ["nanp", 1.440683326647631], ["zelf", 1.440683326647631], ["q'eqchi", 1.440683326647631], ["torrone", 1.440683326647631], ["momoyama", 1.440683326647631], ["\u0447\u0430\u0441\u0442\u044c", 1.440683326647631], ["longhua", 1.440683326647631], ["watermans", 1.440683326647631], ["aphotic", 1.440683326647631], ["cubbyholes", 1.440683326647631], ["cctlds", 1.440683326647631], ["sleepy's", 1.440683326647631], ["hexagons", 1.4406833266476309], ["static", 1.4406333029210112], ["improbable", 1.4403549657754604], ["telegraph's", 1.4403116333229025], ["sweet's", 1.4403116333229025], ["grindelwald", 1.440223338356875], ["stalemate", 1.440210375050499], ["perform", 1.4401684215928845], ["pompously", 1.4401523948891464], ["cuyp", 1.4399687019816352], ["jeevan", 1.4399687019816352], ["callanish", 1.4399687019816352], ["fokker", 1.4399687019816352], ["recompute", 1.4399687019816352], ["lahars", 1.4399687019816352], ["turda", 1.4399687019816352], ["ethnobiology", 1.4399687019816352], ["young'un", 1.4399687019816352], ["keinen", 1.4399687019816352], ["kairouan", 1.4399687019816352], ["lythrum", 1.4399687019816352], ["karachay", 1.4399687019816352], ["serranos", 1.4399687019816352], ["schisandra", 1.4399687019816352], ["maravillas", 1.4399687019816352], ["headbanger's", 1.4399687019816352], ["maysoon", 1.4399687019816352], ["futurelearn", 1.4399687019816352], ["pourtant", 1.4399687019816352], ["iguana's", 1.4399687019816352], ["alekos", 1.4399687019816352], ["parinirvana", 1.4399687019816352], ["suominen", 1.4399687019816352], ["debby's", 1.4399687019816352], ["hallorann", 1.4399687019816352], ["goatfish", 1.4399687019816352], ["umai", 1.4399687019816352], ["rezzonico", 1.4399687019816352], ["home2", 1.4399687019816352], ["weissenfels", 1.4399687019816352], ["carracks", 1.4399687019816352], ["radiohead", 1.439968701981635], ["singed", 1.4397355086695731], ["rodeos", 1.4396952036526738], ["beersheba", 1.4396952036526738], ["detroiters", 1.4393567305825778], ["tune", 1.439282662302279], ["dissenting", 1.4392396039046824], ["eliot", 1.4392396039046824], ["levitation", 1.4392324874465758], ["misty", 1.4390825673958003], ["rada", 1.4390825673958003], ["aligarh", 1.4390825673958], ["sacraments", 1.4388493192791998], ["fixie", 1.4388267759689615], ["facer", 1.4387568111366749], ["falsettos", 1.4387568111366749], ["rohs", 1.4387568111366749], ["alder", 1.4387102579020734], ["corvus", 1.4387052020968896], ["abby's", 1.4387052020968896], ["universe", 1.4385722996651396], ["redhawks", 1.4385698098398176], ["elbowed", 1.438554657429771], ["rush", 1.438384222920871], ["tongva", 1.438363385702949], ["rahim's", 1.438363385702949], ["brandings", 1.438363385702949], ["osrd", 1.438363385702949], ["sensationalizes", 1.438363385702949], ["poderosa", 1.438363385702949], ["endomorph", 1.438363385702949], ["galibier", 1.438363385702949], ["radleys", 1.438363385702949], ["fivefingers", 1.438363385702949], ["coccolithophores", 1.438363385702949], ["inteligencia", 1.438363385702949], ["montreaux", 1.438363385702949], ["silymarin", 1.438363385702949], ["cursi", 1.438363385702949], ["cailyn", 1.438363385702949], ["choicest", 1.4383633857029487], ["pectoris", 1.4383633857029487], ["junto", 1.4383633857029487], ["lizette", 1.4383633857029487], ["paralyzed", 1.438180321272041], ["sequentially", 1.438145010694998], ["fishbowl", 1.4380698421548463], ["personalize", 1.4380698421548463], ["sunnydale", 1.4379547597411013], ["crestview", 1.4379547597411013], ["jaya's", 1.4377680366194625], ["woofs", 1.4377680366194625], ["superglue", 1.4377680366194625], ["armada's", 1.4377680366194625], ["warwickshire's", 1.4377680366194625], ["taren", 1.4377680366194625], ["hinging", 1.437691253279723], ["nicolaus", 1.437691253279723], ["marshmallow", 1.437674807486389], ["halcyon", 1.4376393772873264], ["absences", 1.4376230516073663], ["deanery", 1.4376230516073663], ["jordyn", 1.4375072170209828], ["arrogant", 1.4375036366792218], ["handspring", 1.4374576153089293], ["viceroyalty", 1.4374576153089293], ["nabokov's", 1.4374576153089293], ["foie", 1.4374249962289778], ["desdemona", 1.4373160337230682], ["billboards", 1.4371739698263097], ["cart", 1.4370040946499676], ["permeable", 1.4369250223172458], ["herbicides", 1.4369250223172458], ["ballplayer", 1.4368840562714504], ["rivers", 1.4368495082394963], ["daintily", 1.436839695675721], ["alternated", 1.4367498370330896], ["macaroons", 1.4367498370330896], ["sami", 1.436590060980849], ["david's", 1.4365724274016844], ["mardi", 1.4365654314708203], ["slacking", 1.436470802300708], ["trims", 1.436470802300708], ["recognizer", 1.436470802300708], ["generationally", 1.436470802300708], ["locavore", 1.436470802300708], ["willkommen", 1.436210147101597], ["traf", 1.436210147101597], ["bouchon", 1.436210147101597], ["kano's", 1.436210147101597], ["stubbing", 1.4360509043754521], ["repartee", 1.4357371604200642], ["venus", 1.4357026360962692], ["contributors", 1.4356895050500393], ["nailah", 1.4353915605258765], ["peterhof", 1.4353915605258765], ["catalin", 1.4353915605258765], ["franky's", 1.4353915605258765], ["gunpla", 1.4351905252933097], ["unprocessed", 1.43512799248318], ["therapy's", 1.43512799248318], ["ecologists", 1.43512799248318], ["verstehen", 1.43512799248318], ["kandyan", 1.43512799248318], ["monarch", 1.4350316664319838], ["express", 1.434912739929262], ["slovakia's", 1.4348668523958408], ["boardwalks", 1.4348668523958408], ["misrepresenting", 1.4348640116941076], ["teleportation", 1.434844258202341], ["siculus", 1.434844258202341], ["jace's", 1.434844258202341], ["jibe", 1.4345713367770045], ["horrifying", 1.4344303608201674], ["quasars", 1.4344303608201672], ["maite", 1.4343826752259112], ["pdes", 1.4343826752259112], ["glockenspiel", 1.4343826752259112], ["retrenched", 1.4343826752259112], ["fortifies", 1.4342062660910906], ["microwavable", 1.4342062660910906], ["voicings", 1.4342062660910906], ["cyclist's", 1.4342062660910906], ["breeder's", 1.4342062660910906], ["antiquing", 1.4342062660910906], ["stowing", 1.4341554077128602], ["stephenie", 1.4341554077128602], ["disguising", 1.4340525012867338], ["tracklist", 1.4340361060256221], ["ivy's", 1.4340134964316698], ["yahweh's", 1.4340134964316695], ["rhody", 1.4340134964316695], ["mahabalipuram", 1.4340134964316695], ["maleficent's", 1.4340134964316695], ["pixelation", 1.4340134964316695], ["valenciano", 1.4340134964316695], ["stefanski", 1.4340134964316695], ["helpme", 1.4340134964316695], ["formulate", 1.4339742621362817], ["stalingrad", 1.433902500769982], ["miltiades", 1.4338459127579017], ["earthrise", 1.4338459127579017], ["um's", 1.4338459127579017], ["kayaked", 1.4338459127579017], ["tangata", 1.4338459127579017], ["mutable", 1.4336761120238584], ["ambushes", 1.4336482895898734], ["matilda", 1.4334931030851497], ["vincente", 1.4333825665463762], ["ui's", 1.4333491707129484], ["treadstone", 1.4333491707129484], ["vanderlip", 1.4333491707129484], ["physiatrist", 1.4333491707129484], ["limbu", 1.4333491707129484], ["sardonic", 1.433206095931843], ["crafty", 1.4328030457628471], ["obliqua", 1.4327922599132543], ["beiges", 1.4327922599132543], ["purveyor", 1.432792259913254], ["uranus", 1.4327579989792534], ["hoofbeats", 1.4326205343472551], ["stalkerish", 1.4326205343472551], ["gunther's", 1.4326205343472551], ["daven", 1.4326205343472551], ["enix's", 1.4326205343472551], ["mantids", 1.4326205343472551], ["defilements", 1.4326205343472551], ["misrepresented", 1.4323899193396266], ["childcare", 1.4323606540434912], ["lister's", 1.43233672347235], ["lilith's", 1.43233672347235], ["gregorian", 1.4323157214676068], ["tallies", 1.4322582779537851], ["rasputin", 1.432245086332657], ["ravers", 1.432224029788363], ["volkswagen's", 1.432224029788363], ["rail's", 1.432224029788363], ["bartleby", 1.432224029788363], ["stingrays", 1.4320372504417553], ["alphabetical", 1.4319080801466313], ["subcommittees", 1.4318941111294015], ["dehumanizing", 1.4317364849077918], ["hilda", 1.4317309323133187], ["pastel", 1.431555401700394], ["benefit", 1.4315001510304721], ["turbine", 1.431470526035514], ["breathers", 1.4314481771178387], ["replicated", 1.4314481771178384], ["jostein", 1.4314481771178384], ["greentree", 1.4314481771178384], ["dulcimers", 1.4314481771178384], ["rollergirls", 1.4314481771178384], ["glados", 1.4314481771178384], ["stickied", 1.4314481771178384], ["limmat", 1.4314481771178384], ["newmarket's", 1.4314481771178384], ["kwani", 1.4314481771178384], ["o'connells", 1.4314481771178384], ["dushkin", 1.4314481771178384], ["riam", 1.4314481771178384], ["sittard", 1.4314481771178384], ["lelani", 1.4314481771178384], ["moroder", 1.4314481771178382], ["lakehouse", 1.4314481771178382], ["m\u00e9dico", 1.4314481771178382], ["pedy", 1.4314481771178382], ["niveau", 1.4314481771178382], ["kommunikation", 1.4314481771178382], ["izzy's", 1.4314481771178382], ["kaur's", 1.4314481771178382], ["sounion", 1.4314481771178382], ["scdp", 1.4314481771178382], ["menk", 1.4314481771178382], ["nyree", 1.4314481771178382], ["hope", 1.4313412956539469], ["item", 1.4312788660431255], ["sidewalk", 1.4312091043533928], ["warning", 1.4310458506378947], ["schedule", 1.4309975360250422], ["maggie's", 1.4309904798589679], ["quintus", 1.4308851689852455], ["myths", 1.430870843333473], ["plantar", 1.430793052093986], ["accumulate", 1.4307845918701678], ["colds", 1.4307321973222096], ["minnows", 1.4307288564258194], ["haji", 1.4307095557674163], ["employability", 1.4306785813236889], ["flimsy", 1.430615059131156], ["fukuoka", 1.4304744028476903], ["norovirus", 1.430325472665197], ["polite", 1.430255303636907], ["awnings", 1.4301980040373863], ["conquest", 1.4301520157803236], ["snuffbox", 1.4300731068228165], ["adventured", 1.4300731068228165], ["amarilla", 1.4300731068228165], ["uninitialized", 1.4300731068228165], ["tamati", 1.4300731068228165], ["riaa", 1.4300731068228163], ["crushing", 1.4299800251413075], ["novgorod", 1.4299413895629776], ["tiffanie", 1.4298307215504737], ["gigantes", 1.4298307215504737], ["europ\u00e9en", 1.4298307215504737], ["woodburning", 1.4298307215504737], ["piazzale", 1.4298307215504737], ["brannock", 1.4298307215504737], ["entrar", 1.4298307215504737], ["loas", 1.4298307215504737], ["plea", 1.4298094822349428], ["device", 1.4298046625441105], ["glasgow's", 1.4297561579959497], ["handle", 1.4297128072045742], ["emerald's", 1.429674387555115], ["iain's", 1.429674387555115], ["bumpier", 1.429674387555115], ["buggles", 1.429674387555115], ["bakshi's", 1.429674387555115], ["numbing", 1.429645345660511], ["eventbrite", 1.4296304651468952], ["irresponsible", 1.4295925961475004], ["centralized", 1.4294101831029589], ["posterity", 1.4294025205588992], ["defeatism", 1.4293736145423053], ["rehoming", 1.4293736145423053], ["unicorns", 1.4291982043733171], ["congested", 1.4291913894512112], ["nomads", 1.4289500127598667], ["raoul's", 1.4289500127598667], ["hydrator", 1.4289500127598667], ["crenellations", 1.4289500127598667], ["potencial", 1.4289500127598667], ["boccioni", 1.4289500127598667], ["coop\u00e9ration", 1.4289500127598667], ["kurzweil's", 1.4289500127598667], ["danyal", 1.4289500127598667], ["snape's", 1.4289500127598664], ["russes", 1.4289500127598664], ["unfixed", 1.4288920196586994], ["apulia", 1.4288920196586994], ["vulgate", 1.4288489197274818], ["averages", 1.4288058395216332], ["camouflage", 1.428799363000485], ["medium's", 1.4286447141779288], ["hackers", 1.428490408954593], ["institution", 1.4284144724208079], ["constricting", 1.4284065873656624], ["westernized", 1.4284065873656624], ["butterfly", 1.428353154032178], ["course's", 1.4283090363785451], ["guanabara", 1.4283090363785451], ["s'more", 1.4283090363785451], ["bitterroot", 1.4283090363785451], ["gecko", 1.42827891916112], ["proclaiming", 1.4281713680376893], ["elizabeth", 1.4281065139156628], ["papers", 1.4280788224404948], ["archdeacons", 1.4280154476763087], ["tenpin", 1.4280154476763087], ["prenzlauer", 1.4280154476763087], ["entice", 1.4279964581091853], ["maladies", 1.4279884836383996], ["ahab", 1.427942589745305], ["tradeoff", 1.4278982691658322], ["bookcase", 1.4278082315043823], ["lana's", 1.4275264012901185], ["pricy", 1.4274240256159008], ["shizuoka", 1.4274240256159008], ["gain", 1.4273104203127744], ["crossbill", 1.4272256161233905], ["zeina", 1.4272256161233905], ["quigley's", 1.4272256161233905], ["whanau", 1.4272256161233905], ["tabbies", 1.4272256161233905], ["snakeheads", 1.4272256161233905], ["hitherto", 1.4271166081658033], ["roar", 1.4270481168528453], ["contests", 1.4270481168528453], ["hanshin", 1.426797598244811], ["bitbucket", 1.426797598244811], ["pavlovna", 1.426797598244811], ["angeleno", 1.426797598244811], ["intarsia", 1.426797598244811], ["isolations", 1.426797598244811], ["varanasi", 1.4267431670322779], ["dulcie", 1.4266766831941124], ["aeroflot", 1.4266766831941124], ["aller", 1.4266766831941124], ["europe's", 1.4266766831941122], ["translators", 1.4266766831941122], ["opponent's", 1.4266766831941122], ["patinas", 1.4266766831941122], ["inplace", 1.4266766831941122], ["bario", 1.4266766831941122], ["osian", 1.4266766831941122], ["haemon", 1.4266766831941122], ["glamorizes", 1.4266766831941122], ["longtemps", 1.4266766831941122], ["foghorns", 1.4266766831941122], ["croma", 1.4266766831941122], ["corneliu", 1.4266766831941122], ["products", 1.4266654051175653], ["arawak", 1.4263951578340135], ["hendersons", 1.4263951578340135], ["perfecta", 1.4263951578340135], ["sola", 1.426305249535379], ["failed", 1.4262262426179544], ["piranhas", 1.4261465172025871], ["tamarama", 1.4259637013434405], ["arantxa", 1.4259637013434405], ["xitsonga", 1.4259637013434405], ["fjallraven", 1.4259637013434405], ["decembrists", 1.4259637013434405], ["gorgan", 1.4259637013434405], ["takeback", 1.4259637013434405], ["devito's", 1.4259637013434405], ["clarity's", 1.4259637013434405], ["barre's", 1.4259637013434405], ["compa\u00f1ero", 1.4259637013434405], ["mihal", 1.4259637013434405], ["tomoko's", 1.4259637013434405], ["dirch", 1.4259637013434405], ["bilbies", 1.4259637013434405], ["meret", 1.4259637013434405], ["scouted", 1.4259394914334007], ["bland", 1.4258915006497017], ["outlooks", 1.425803029377092], ["fevers", 1.4255488755394132], ["subfield", 1.4253569082790392], ["euphorbia", 1.4253569082790392], ["humanistic", 1.4252873549238312], ["positioning", 1.4251750398549878], ["zeppelin", 1.425086185219537], ["blog's", 1.4250861852195367], ["grump", 1.4250861852195367], ["pronouns", 1.4250257100113783], ["uncannily", 1.4250002123560463], ["tullie", 1.4250002123560463], ["pteridium", 1.4250002123560463], ["alofa", 1.4250002123560463], ["hagos", 1.4250002123560463], ["daikichi", 1.4250002123560463], ["gathering's", 1.4250002123560463], ["coconutty", 1.4250002123560463], ["rasheed's", 1.4250002123560463], ["uncarved", 1.4250002123560463], ["ch\u00e2tillon", 1.4250002123560463], ["moodswings", 1.4250002123560463], ["huanglong", 1.4250002123560463], ["petr\u00f3leo", 1.4250002123560463], ["producci\u00f3n", 1.4250002123560463], ["elizabethan", 1.424937540549016], ["scatter", 1.4249044140224427], ["situational", 1.4248787289619835], ["regional", 1.4248335232555758], ["brand", 1.424749733724915], ["bollywood", 1.4247178593727932], ["shrews", 1.4246317572268012], ["gwalior", 1.4246317572268012], ["harald's", 1.4245991427775617], ["chaffinches", 1.4245991427775617], ["grenzen", 1.4245991427775617], ["soundbox", 1.4245991427775617], ["pecci", 1.4245991427775617], ["battlefield", 1.4245628721374204], ["refill", 1.4245377376420971], ["rationalism", 1.4244947437616546], ["unpleasant", 1.4244157058831866], ["unmoving", 1.4243792972308367], ["sanjay", 1.4242724420697541], ["spqr", 1.4242404823690076], ["dpmne", 1.4242404823690076], ["megaplex", 1.4242404823690076], ["purling", 1.4242404823690076], ["snowbanks", 1.4242404823690076], ["geotag", 1.4242404823690076], ["delos", 1.4242404823690074], ["label's", 1.4242404823690074], ["eczema", 1.4240258532364867], ["personnes", 1.4239797518459194], ["vino", 1.4238332292599394], ["biggie", 1.4238011471208736], ["horde", 1.4237576516172445], ["inspections", 1.4236260559314022], ["oilier", 1.4236260559314022], ["bethlehem's", 1.4236260559314022], ["maja", 1.4236260559314022], ["innu", 1.4236260559314022], ["rotonda", 1.4236260559314022], ["sicher", 1.4236260559314022], ["taller", 1.4235578854897772], ["ricky's", 1.4234640040183764], ["rabbit's", 1.4234640040183764], ["maddening", 1.4234326722697408], ["forging", 1.4234119768252471], ["comedian's", 1.423310745620011], ["teabags", 1.423310745620011], ["aviation", 1.423239917976077], ["consistencies", 1.4230771351463891], ["earmarking", 1.4230771351463891], ["easy", 1.423043985820182], ["haves", 1.4229869421015788], ["neurological", 1.4229227412887024], ["stevia", 1.4229227412887022], ["dishwater", 1.4228398117915284], ["shabbily", 1.4228398117915284], ["mammalia", 1.4227541367423462], ["sickly", 1.4226186836888872], ["stayed", 1.4223950063457422], ["moxie", 1.422308414109011], ["prophethood", 1.4222215314089757], ["pawed", 1.422168915648975], ["subdomains", 1.422168915648975], ["bleus", 1.422168915648975], ["tarpaulins", 1.422168915648975], ["balconies", 1.4221654525753167], ["malini", 1.4221379450715435], ["charmers", 1.4221379450715435], ["incoming", 1.4220846108985403], ["tapioca", 1.4220653833109909], ["rewrites", 1.4220653833109909], ["foam", 1.4220476219248257], ["yukos", 1.4220183472290515], ["famitsu", 1.4220183472290515], ["petit", 1.4220078020429394], ["mint", 1.421905189270386], ["peak", 1.4218826689450825], ["stanislaw", 1.4218616139834115], ["sleeves", 1.4218480456305809], ["cinnamon", 1.4218480456305809], ["tristan's", 1.4218330760120932], ["colonoscopies", 1.4218330760120932], ["pedometer", 1.4217979646878305], ["broccoli", 1.4215729966554433], ["gendered", 1.4215075647767423], ["drakon", 1.4215075647767423], ["creative's", 1.4215075647767423], ["reykjanes", 1.4215075647767423], ["kriging", 1.4215075647767423], ["extension's", 1.4215075647767423], ["l\u00e9vesque", 1.4215075647767423], ["medios", 1.4215075647767423], ["displeased", 1.4214244599790355], ["binocular", 1.421377687429572], ["contenders", 1.4213730524015689], ["blurt", 1.4213364019152401], ["disquiet", 1.4211804053832378], ["judah's", 1.421037644920618], ["swiftest", 1.421037644920618], ["kleinschmidt", 1.421037644920618], ["possesses", 1.4210376449206177], ["neutralize", 1.4210376449206177], ["meadowlark", 1.420844857287336], ["kolhapur", 1.420844857287336], ["leaders", 1.420761803419933], ["pennine", 1.4206738359997473], ["cricketing", 1.4205503206253776], ["diffuse", 1.420506217229922], ["kanye's", 1.4204280191613212], ["akira", 1.4203952228388275], ["tipo", 1.420383843381662], ["plumper", 1.420383843381662], ["pact", 1.4203235767593558], ["castilla", 1.4202998268287588], ["out's", 1.420259846538067], ["toggled", 1.420202231843063], ["leans", 1.420006852978151], ["nursing", 1.4199691955635347], ["mileageplus", 1.4199506013980165], ["desto", 1.4199506013980165], ["nafta's", 1.4199506013980165], ["qualquer", 1.4199506013980165], ["sazon", 1.4199506013980165], ["rhaegal", 1.4199506013980165], ["gadsby's", 1.4199506013980165], ["germophobe", 1.4199506013980165], ["alaminos", 1.4199506013980165], ["kreisky", 1.4199506013980165], ["altricial", 1.4199506013980165], ["iora", 1.4199506013980165], ["weathervanes", 1.4199506013980165], ["chital", 1.4199506013980165], ["petal", 1.4199098372659187], ["aliases", 1.4198642034301796], ["earthen", 1.4198642034301796], ["entity", 1.4198158532946938], ["rattling", 1.4196100402944167], ["buckle", 1.4196100402944167], ["cleanout", 1.4195787892478728], ["venerated", 1.4195426952028882], ["maneuver", 1.4195405551134952], ["treadmill", 1.4194768396842041], ["knot", 1.419469455229322], ["greedier", 1.4194375516805415], ["makis", 1.4194375516805415], ["handsaw", 1.4194375516805415], ["zipporah", 1.4194375516805415], ["pitted", 1.4194042683172607], ["bioaccumulation", 1.4193172603626025], ["harbors", 1.4192331526730995], ["frequents", 1.4191945351347286], ["demoralizing", 1.4191821259000332], ["matera", 1.4191579654961197], ["siders", 1.4191579654961197], ["cheng's", 1.4191232850639333], ["calf's", 1.4191232850639333], ["velma", 1.4190903511283322], ["plying", 1.4188765763607156], ["robie", 1.4188338828367235], ["hokkaido", 1.4187944915742554], ["igbo", 1.4187493395342723], ["molded", 1.4186493463577259], ["stoop", 1.4185858891495333], ["conclusively", 1.4185576490307041], ["denmark's", 1.4184754003419724], ["blackbeard", 1.4183035657577114], ["bushels", 1.41826063109749], ["packets", 1.4182436803379799], ["token", 1.418137080604448], ["upstream", 1.417961805862348], ["filosof\u00eda", 1.4178153373357638], ["vitesse", 1.4178153373357638], ["frite", 1.4178153373357638], ["niggling", 1.4178153373357638], ["chiffonade", 1.4178153373357638], ["teuscher", 1.4178153373357638], ["routemasters", 1.4178153373357638], ["degrom's", 1.4178153373357638], ["johans", 1.4178153373357638], ["warmbloods", 1.4178153373357638], ["rankine's", 1.4178153373357638], ["lorex", 1.4178153373357638], ["charmion", 1.4178153373357638], ["kante's", 1.4178153373357638], ["rayearth", 1.4178153373357638], ["heartwork", 1.4178153373357638], ["vulcanus", 1.4178153373357638], ["lourey", 1.4178153373357638], ["overpromised", 1.4178153373357638], ["porte\u00f1os", 1.4178153373357638], ["yohannan", 1.4178153373357638], ["darkrooms", 1.4178153373357636], ["waldner", 1.4178153373357636], ["hagrid's", 1.4178153373357636], ["groundskeepers", 1.4178153373357636], ["playout", 1.4178153373357636], ["heartthrobs", 1.4178153373357636], ["glaciations", 1.4178153373357636], ["craftier", 1.4178153373357636], ["bar\u00e7a's", 1.4178153373357636], ["karsa", 1.4178153373357636], ["telogen", 1.4178153373357636], ["kiesha", 1.4178153373357636], ["char", 1.417635106572543], ["agreement", 1.417507116610256], ["wright's", 1.4174266818156607], ["perished", 1.4173722700428464], ["shadow", 1.4172933235070173], ["brahms", 1.4171457142230421], ["leather", 1.4169845861615438], ["yggdrasil", 1.4169465779378865], ["interferences", 1.4167213440199184], ["naturalness", 1.4167213440199184], ["nim's", 1.4167213440199184], ["librettists", 1.4167213440199184], ["phrase", 1.4166899464117533], ["swooned", 1.4166178581657167], ["adoptable", 1.4166178581657167], ["meatier", 1.4165372586061942], ["oxygen", 1.4164313743309207], ["ancients", 1.4163384463593722], ["chain", 1.4163245695314062], ["na'vi", 1.4161968266767047], ["desynchronization", 1.4161968266767047], ["freedos", 1.4161968266767047], ["vava'u", 1.4161968266767047], ["woba", 1.4161968266767047], ["microfractures", 1.4161968266767047], ["vladek", 1.4161968266767047], ["udolpho", 1.4161968266767047], ["chalukya", 1.4161968266767047], ["preethi", 1.4161968266767047], ["boucherie", 1.4161968266767047], ["better", 1.416078821768253], ["regensburg", 1.4159595580984132], ["slurring", 1.415838828965872], ["descent", 1.4158160677649025], ["verifiable", 1.415780585129484], ["mesopotamia", 1.415780585129484], ["homeschooled", 1.4157723757545884], ["differ", 1.4156621131085017], ["homesick", 1.415524507619152], ["heroine", 1.415515046965455], ["covering", 1.4155025889131592], ["secreto", 1.4154364525415761], ["eryn", 1.4154364525415761], ["oiseaux", 1.4154364525415761], ["hoards", 1.4151435747654075], ["collaborator", 1.4151263772132303], ["categorized", 1.4151221832023442], ["unripened", 1.4151198709149733], ["mca's", 1.4151198709149733], ["cracklings", 1.4151198709149733], ["pamphlet", 1.4151018151590447], ["masashi", 1.4147060931749398], ["dealership", 1.414648164291335], ["dissatisfied", 1.41451405162383], ["reexamined", 1.414502684678437], ["keenly", 1.4142707989924246], ["fava", 1.4142707989924244], ["genovia", 1.4142707989924244], ["skateboards", 1.4141154016955348], ["dogen", 1.414066706890399], ["blazer", 1.414004005783786], ["lululemon", 1.414004005783786], ["renewal", 1.4137688267992925], ["deeming", 1.4137598014738193], ["lotus", 1.4137352931995457], ["columns", 1.4137123347518337], ["tenderloin", 1.4136178577186513], ["distance", 1.4135699015798995], ["qualitative", 1.4134789340433802], ["tiye", 1.4133285799391317], ["oyster's", 1.4133285799391317], ["cooperi", 1.4133285799391317], ["hdfs", 1.4133285799391317], ["undercharging", 1.4133285799391317], ["mycoskie", 1.4133285799391317], ["laners", 1.4133285799391317], ["mcgonagall's", 1.4133285799391317], ["chandrachud", 1.4133285799391317], ["taleb's", 1.4133285799391317], ["tribemates", 1.4133285799391317], ["zainuddin", 1.4133285799391317], ["totalmente", 1.4133285799391317], ["sherfield", 1.4133285799391317], ["rockatansky", 1.4133285799391317], ["paulina's", 1.4133285799391317], ["cinnamaldehyde", 1.4133285799391317], ["grogan's", 1.4133285799391317], ["brownhills", 1.4133285799391317], ["thile", 1.4133285799391317], ["gaylords", 1.4133285799391317], ["mercanti", 1.4133285799391317], ["karanga", 1.4133285799391317], ["god\u00edn", 1.4133285799391317], ["ruhul", 1.4133285799391317], ["mamah", 1.4133285799391317], ["pawnshop", 1.4133285799391315], ["terraced", 1.4132977884668019], ["spacecraft's", 1.4131920264531472], ["sequence", 1.413163130760131], ["cerebrum", 1.413120584563129], ["mooing", 1.4130962774111995], ["headpiece", 1.4128923674144591], ["epicureans", 1.4127697311530742], ["rovinj", 1.4127697311530742], ["floridian", 1.412751711130993], ["gore's", 1.412751711130993], ["anse", 1.4127274832544152], ["honey", 1.4126272844186964], ["cornfield", 1.412587404675971], ["melbourne's", 1.4124583524339913], ["adrianne", 1.4123622014229338], ["orville", 1.412308286237355], ["procedurally", 1.4122769961742958], ["gobi", 1.4121971929875312], ["einer", 1.4121971929875312], ["busy", 1.4121295698686456], ["discourses", 1.412116722635861], ["stoer", 1.412051860354724], ["exoduses", 1.412051860354724], ["ricciuto", 1.412051860354724], ["peshawari", 1.412051860354724], ["outcasted", 1.412051860354724], ["dekes", 1.412051860354724], ["litterers", 1.412051860354724], ["dmitry's", 1.412051860354724], ["retranslate", 1.412051860354724], ["presbycusis", 1.412051860354724], ["riccardo's", 1.412051860354724], ["stealthiest", 1.412051860354724], ["tenaciousness", 1.412051860354724], ["faleomavaega", 1.412051860354724], ["particulares", 1.412051860354724], ["backboned", 1.412051860354724], ["chiemsee", 1.412051860354724], ["bempton", 1.412051860354724], ["bestie's", 1.412051860354724], ["hunkpapa", 1.412051860354724], ["yamabushi", 1.412051860354724], ["savannah", 1.4119502738899503], ["playroom", 1.4118245683047677], ["numbness", 1.4116489368673253], ["radium", 1.4116362689083892], ["goya", 1.411610456209568], ["tyrell's", 1.4116104562095677], ["inveraray", 1.4116104562095677], ["espero", 1.4116104562095677], ["denisovans", 1.4116104562095677], ["wudu", 1.4114479031860523], ["hijos", 1.4114479031860523], ["crudo", 1.4114479031860523], ["nefertiti", 1.4113940722210185], ["forgettable", 1.41135767638636], ["storyline", 1.4113384990902074], ["standards", 1.4113287733857223], ["creameries", 1.4111956562191708], ["zhao's", 1.4111956562191708], ["ghirardelli", 1.4111956562191708], ["tonga's", 1.4111956562191708], ["dandi", 1.4111956562191708], ["danger", 1.4110816256757952], ["susceptible", 1.4110367864441211], ["patient's", 1.4110060926235592], ["frequencies", 1.4108873760283276], ["swampland", 1.410865262152745], ["translator's", 1.410865262152745], ["multimedia", 1.4107848480597187], ["nightstand", 1.4107262606490851], ["strives", 1.4106994080858777], ["goal", 1.4106569109858456], ["apparently", 1.4105942532781364], ["photograph", 1.410549033731918], ["ringing", 1.4104894647681103], ["blasting", 1.4104267221975353], ["pilsner", 1.4104028046785901], ["richard", 1.4103097630317558], ["ruin", 1.4101963174163796], ["pipettes", 1.4100832789519004], ["underground", 1.4099970015640917], ["terminology", 1.409971473778478], ["scuffs", 1.4097595683736286], ["hokusai", 1.4097595683736286], ["hoovervilles", 1.4097595683736286], ["utuado", 1.4097595683736286], ["greenland's", 1.4097595683736286], ["spoonerisms", 1.4097595683736286], ["atlantan", 1.4097595683736286], ["auel", 1.4097595683736286], ["tainui", 1.4097595683736286], ["depthless", 1.4097595683736286], ["spida", 1.4097595683736286], ["kantara", 1.4097595683736286], ["alteryx", 1.4097595683736286], ["solide", 1.4097595683736286], ["micanopy", 1.4097595683736286], ["salpingo", 1.4097595683736286], ["timucua", 1.4097595683736286], ["phycocyanin", 1.4097595683736286], ["bammy", 1.4097595683736286], ["natureza", 1.4097595683736286], ["pax's", 1.4097595683736286], ["divisoria", 1.4097595683736286], ["zentropa", 1.4097595683736286], ["algor", 1.4097595683736286], ["aukey", 1.4097595683736286], ["spielt", 1.4097595683736286], ["vergel", 1.4097595683736286], ["southminster", 1.4097595683736286], ["afas", 1.4097595683736286], ["aliana", 1.4097595683736286], ["screenwriting", 1.4096702016814973], ["harmonics", 1.409584985145347], ["cessna", 1.4095036362280844], ["eurasian", 1.4094709789021094], ["davon", 1.40945736910924], ["comit\u00e9", 1.40945736910924], ["harmed", 1.4094234667546788], ["logistically", 1.4094026672170783], ["warbles", 1.4092345182550539], ["submarine's", 1.4092345182550539], ["unfeasible", 1.4092120889296003], ["laura", 1.409066031482521], ["vizier", 1.4089094621012428], ["eggshell", 1.4088037991747315], ["llanelli", 1.4088037991747315], ["claustrophobic", 1.408803799174731], ["management", 1.408757072680563], ["minh", 1.408757072680563], ["commands", 1.4087207981653533], ["jabbed", 1.4087055425281156], ["kabaddi", 1.4087055425281156], ["blurry", 1.408668141611017], ["statue", 1.4086225915510053], ["mit's", 1.4085847687333175], ["pivo", 1.408528337309547], ["pinots", 1.408528337309547], ["madsen's", 1.408528337309547], ["godmother's", 1.408528337309547], ["muizenberg", 1.408528337309547], ["puesto", 1.408528337309547], ["lolitas", 1.408528337309547], ["stonemasonry", 1.408528337309547], ["kodomo", 1.408528337309547], ["bodhgaya", 1.408528337309547], ["costner's", 1.408528337309547], ["moqtada", 1.408528337309547], ["ebla", 1.408528337309547], ["underdevelopment", 1.4084327358386888], ["fend", 1.408297595487908], ["universal", 1.4082886736006721], ["humbert", 1.408235503975387], ["prisms", 1.408235503975387], ["leached", 1.4082355039753869], ["iridescence", 1.4082355039753869], ["nobles", 1.4081419956417895], ["eu's", 1.4080678568915803], ["danang", 1.4079693974931544], ["paintwork", 1.407858768955597], ["ludovico", 1.407858768955597], ["augie's", 1.4077599094113966], ["hatpins", 1.4077599094113966], ["donezo", 1.4077599094113966], ["melodyne", 1.4077599094113966], ["anglicize", 1.4077599094113966], ["murres", 1.4077599094113966], ["hattie's", 1.4077599094113966], ["borsalino", 1.4077599094113966], ["lerici", 1.4077599094113966], ["watchpoint", 1.4077599094113966], ["qaqa", 1.4077599094113966], ["fraises", 1.4077599094113966], ["farz", 1.4077599094113966], ["douglass's", 1.4077599094113966], ["maa's", 1.4077599094113966], ["kagu", 1.4077599094113966], ["lukie", 1.4077599094113966], ["boghosian", 1.4077599094113966], ["svan", 1.4077599094113966], ["eithne", 1.4077599094113966], ["aufgabe", 1.4077599094113966], ["cmms", 1.4077599094113966], ["chuc", 1.4077599094113966], ["montone", 1.4077599094113966], ["neopaganism", 1.4077599094113966], ["maotai", 1.4077599094113966], ["iason", 1.4077599094113966], ["melvin", 1.4076996457166444], ["isar", 1.4076268004940922], ["contingency", 1.407503953064231], ["ambient", 1.407487725615962], ["resupplying", 1.4074747068365565], ["bournemouth's", 1.4074747068365565], ["adelia", 1.4074747068365565], ["codger", 1.4074747068365565], ["prenatal", 1.4074261732259759], ["socialites", 1.4073341755678246], ["allspice", 1.4073341755678246], ["randomly", 1.4073298862879429], ["drab", 1.4072164726816605], ["rained", 1.4072164726816605], ["improvise", 1.4071817223057457], ["pint", 1.4069955758719597], ["songs", 1.4069204918787075], ["matchday", 1.406915772934351], ["panipat", 1.4068528476140956], ["tiberius", 1.4067998521527338], ["campania", 1.4067386550128282], ["videographers", 1.4067386550128282], ["imbues", 1.4067386550128282], ["bako", 1.4063350512034904], ["colab", 1.4063350512034904], ["monsoonal", 1.4063350512034904], ["antikythera", 1.4063350512034904], ["renko", 1.4063350512034904], ["folklorist", 1.406278680550484], ["mustachioed", 1.4062085367152568], ["mimicry", 1.406193900144487], ["sedimentation", 1.4061811852241464], ["conduits", 1.4060002095246327], ["boeing's", 1.4060002095246327], ["mare's", 1.4060002095246327], ["obstructing", 1.4060002095246324], ["oceanographer", 1.4060002095246324], ["katsu", 1.4060002095246324], ["unos", 1.4060002095246324], ["colloquialism", 1.4060002095246324], ["cloverleaf", 1.4060002095246324], ["figureheads", 1.4060002095246322], ["parsnips", 1.4056857384422068], ["compressions", 1.4055004760220469], ["london's", 1.405480700135952], ["primates", 1.4054767544950253], ["transcranial", 1.4054767544950253], ["crumbs", 1.4054337307939615], ["kiwi", 1.405227257237923], ["lauderdale's", 1.4051972396733905], ["henrik's", 1.4051972396733905], ["sona's", 1.4051972396733905], ["lindenberg", 1.4051972396733905], ["paperweights", 1.4050863322353737], ["peacocks", 1.4050328891190427], ["incremental", 1.4049663858411585], ["teju", 1.404939877541582], ["aslan's", 1.404939877541582], ["cork's", 1.404939877541582], ["storyboards", 1.4048672520632912], ["civic", 1.404346091631074], ["salting", 1.4042771700521757], ["keine", 1.4040768166251458], ["numeracy", 1.403999260459844], ["geocities", 1.4038638949019555], ["mindblown", 1.4038237386120556], ["favorite's", 1.4038237386120556], ["pagosa", 1.4038237386120556], ["tuatara", 1.4038237386120556], ["lauder's", 1.4038237386120556], ["hindu's", 1.4038237386120556], ["shapur", 1.4038237386120556], ["murrelet", 1.4038237386120556], ["boulevards", 1.4034665497314358], ["krupp", 1.403360659694486], ["nooyi", 1.403236259316887], ["keenan's", 1.403236259316887], ["comedians", 1.4031898150452855], ["agung", 1.4031482821219456], ["metadata", 1.4031109206275871], ["grozny", 1.4031109206275871], ["sinkholes", 1.4031109206275871], ["farm", 1.4031010883054953], ["sportswear", 1.4030836288558897], ["unworn", 1.4029691034369598], ["dooming", 1.4029691034369598], ["boggling", 1.4029084003777943], ["lisbeth", 1.4029084003777943], ["gameplay", 1.402709294343731], ["zander", 1.402675349437078], ["ucl's", 1.4026753494370778], ["isomerism", 1.4026753494370778], ["reyn", 1.4026753494370778], ["arul", 1.4026753494370778], ["azul", 1.402624458721452], ["pleyel", 1.4026145881050682], ["aperol", 1.4026145881050682], ["manufacturing", 1.402445906612854], ["earthquakes", 1.4024301709054419], ["frontex", 1.402404301060375], ["konami's", 1.402404301060375], ["picos", 1.402404301060375], ["fido", 1.4024043010603748], ["dredd's", 1.4024043010603748], ["cuerdas", 1.4024043010603748], ["draupadi's", 1.4024043010603748], ["tenshu", 1.4024043010603748], ["noryangjin", 1.4024043010603748], ["bluets", 1.4024043010603748], ["suzani", 1.4024043010603748], ["textboxes", 1.4024043010603748], ["jara's", 1.4024043010603748], ["holzer's", 1.4024043010603748], ["tremaine's", 1.4024043010603748], ["rainshadow", 1.4024043010603748], ["barograph", 1.4024043010603748], ["goldeneyes", 1.4024043010603748], ["reinhard's", 1.4024043010603748], ["santosa", 1.4024043010603748], ["bisti", 1.4024043010603748], ["cabanne", 1.4024043010603748], ["anunoby", 1.4024043010603748], ["iggle", 1.4024043010603748], ["mortar", 1.4023947034399187], ["pronoun", 1.4023678560421562], ["steppes", 1.40213924638242], ["fossett", 1.4021392463824198], ["ziggy's", 1.4021392463824198], ["erroneous", 1.4020143832750942], ["alphaville", 1.4019657756379045], ["rovaniemi", 1.4019033231009586], ["caracol", 1.4019033231009586], ["tego", 1.4019033231009586], ["judaism", 1.4018649147907365], ["overpowered", 1.4018649147907365], ["beaver's", 1.401794825049484], ["humanae", 1.401794825049484], ["roxy's", 1.401794825049484], ["ortega's", 1.401794825049484], ["academy's", 1.4016592924006532], ["soundproofed", 1.4016263400945501], ["hasa", 1.4016263400945501], ["idiom", 1.4014793085293764], ["wowed", 1.4012401580323781], ["mart\u00ednez's", 1.4012155709895462], ["nonnie", 1.4012155709895462], ["japantown", 1.4012155709895462], ["oldtimer", 1.4012155709895462], ["bluewave", 1.4012155709895462], ["\u00eates", 1.4012155709895462], ["croutons", 1.4011823904171807], ["nebo", 1.4011351568965196], ["preordained", 1.4011351568965196], ["beeped", 1.4011351568965196], ["shamanistic", 1.4011031499713347], ["blacklivesmatter", 1.4011031499713347], ["madama", 1.4011031499713347], ["analogy", 1.4009217773952864], ["homebound", 1.40088917108948], ["basquiat", 1.400855823731269], ["krishnan", 1.400796858534962], ["reforms", 1.4007861422514591], ["lahaina", 1.4006643453518635], ["microeconomics", 1.4006643453518635], ["kimberley's", 1.4006643453518632], ["liddell's", 1.4006643453518632], ["lutenist", 1.4006643453518632], ["lauenburg", 1.4006643453518632], ["trapezes", 1.4006643453518632], ["abstractionism", 1.4006643453518632], ["blakely's", 1.4006643453518632], ["antrel", 1.4006643453518632], ["godwits", 1.4006643453518632], ["tenderizer", 1.4006643453518632], ["bakeoff", 1.4006643453518632], ["sterkfontein", 1.4006643453518632], ["seydlitz", 1.4006643453518632], ["lujo", 1.4006643453518632], ["hal\u00e9vy", 1.4006643453518632], ["rochfort", 1.4006643453518632], ["ga\u00eblle", 1.4006643453518632], ["kitchens", 1.4005990082395439], ["baffling", 1.4004864832127712], ["since", 1.4002380221472055], ["lassie", 1.4001962870910942], ["refurbished", 1.4000926456190668], ["visceral", 1.4000926456190665], ["underpaid", 1.400042581518172], ["grip", 1.4000376915362243], ["surveys", 1.4000040321604832], ["trainspotting", 1.399969571371034], ["handled", 1.3999134273882239], ["klaus's", 1.3998857659492219], ["porites", 1.3998857659492219], ["miia", 1.3998857659492219], ["drago's", 1.3998857659492219], ["desalegn", 1.3998857659492219], ["blotchy", 1.3998531497850573], ["psychedelic", 1.3997689434872547], ["tipsy", 1.3996382176263311], ["silla", 1.399638217626331], ["crumb's", 1.399638217626331], ["shira", 1.399638217626331], ["grizedale", 1.399638217626331], ["gloppy", 1.399638217626331], ["friendy", 1.399638217626331], ["zabbix", 1.399638217626331], ["brasidas", 1.399638217626331], ["midwich", 1.399638217626331], ["natasa", 1.399638217626331], ["wohlfahrt", 1.399638217626331], ["incorrectly", 1.399330379308671], ["earbuds", 1.3991679834425435], ["sobs", 1.3991536950645398], ["hipaa", 1.3990667812282798], ["esra", 1.3990051836065993], ["manji", 1.3990051836065993], ["showerthoughts", 1.3990051836065993], ["halley's", 1.3989614099775753], ["cheesecloth", 1.3989614099775753], ["dependable", 1.398798291729645], ["clubface", 1.398702630582463], ["bellbird", 1.398702630582463], ["dunnock", 1.398702630582463], ["portaging", 1.398702630582463], ["wool's", 1.398702630582463], ["entheogens", 1.398702630582463], ["wastebaskets", 1.398702630582463], ["griselda", 1.398702630582463], ["fulfillments", 1.398702630582463], ["adharma", 1.398702630582463], ["prinsengracht", 1.398702630582463], ["liberations", 1.398702630582463], ["nanoscopic", 1.398702630582463], ["microbead", 1.398702630582463], ["frankies", 1.398702630582463], ["kharak", 1.398702630582463], ["reddie", 1.398702630582463], ["seneviratne", 1.398702630582463], ["regimes", 1.3986811038009228], ["geo's", 1.3985756720065727], ["nitrox", 1.3985756720065727], ["hopis", 1.3984814918266397], ["shikha", 1.3984814918266397], ["cleitus", 1.3984814918266397], ["schulz's", 1.3984814918266397], ["louisa's", 1.3984814918266397], ["noninvasively", 1.3984814918266397], ["oxygene", 1.3984814918266397], ["capitol's", 1.3983511094388132], ["seleucids", 1.398265125849719], ["brahmacharya", 1.398265125849719], ["fluffiest", 1.398265125849719], ["undervaluation", 1.398265125849719], ["toraja", 1.398265125849719], ["freelancer", 1.3981431495302301], ["overflow", 1.3980671951371586], ["marbled", 1.3980372787460142], ["blackberries", 1.397873798320146], ["berets", 1.397846107232443], ["hailu", 1.397846107232443], ["emmaline", 1.397846107232443], ["ancestress", 1.397846107232443], ["lamelo", 1.397846107232443], ["rajdeep", 1.397846107232443], ["post", 1.3978018724183623], ["aggression", 1.3977712854949653], ["mamadou", 1.3976935039456273], ["mats", 1.397662921754681], ["buon", 1.3976249290509188], ["placebos", 1.3976249290509188], ["applied", 1.3976235868585354], ["ucla's", 1.3975608325166815], ["clarinet", 1.3975416521143906], ["tsars", 1.3975303287443634], ["herzegovina", 1.3975175536602304], ["rumpled", 1.3974444273165718], ["narration", 1.3972943258743253], ["everdeen", 1.3971112582274727], ["parkinson's", 1.3969365498339013], ["huang's", 1.3967730421552431], ["parked", 1.3966919815258259], ["caucusing", 1.3966889498589066], ["vosotros", 1.3966889498589066], ["photoaging", 1.3966889498589066], ["stanze", 1.3966889498589066], ["lendale", 1.3966889498589066], ["sentir", 1.3966889498589066], ["relaci\u00f3n", 1.3966889498589066], ["bend's", 1.3966889498589066], ["krayzie", 1.3966889498589066], ["particulier", 1.3966889498589066], ["waan", 1.3966889498589066], ["lemp", 1.3966889498589066], ["autoharp", 1.3966889498589066], ["mahavidyalaya", 1.3966889498589066], ["motortrend", 1.3966889498589066], ["katja", 1.3966889498589063], ["spicing", 1.3966889498589063], ["passengers", 1.3966533104685661], ["militaries", 1.3965905568367252], ["trendsetter", 1.3964738761062332], ["apology", 1.3962158068060393], ["vasudeva", 1.396160601476134], ["harmon", 1.3960448703504384], ["habsburg", 1.396042988789453], ["cyberpunk", 1.3959316658247116], ["essere", 1.395907731093254], ["amigas", 1.395907731093254], ["devs", 1.3958662856708162], ["locked", 1.3957311504828387], ["patras", 1.3956619726898927], ["constituted", 1.3956619726898924], ["tumblers", 1.3956619726898924], ["marginalize", 1.3956619726898924], ["bobblehead", 1.3956619726898924], ["burpee", 1.3956619726898924], ["varangians", 1.3956619726898924], ["volca", 1.3956619726898924], ["helixes", 1.3956619726898924], ["lepcha", 1.3956619726898924], ["bicycle's", 1.3956619726898924], ["rotherham's", 1.3956619726898924], ["nimal", 1.3956619726898924], ["temple", 1.3955739876955542], ["streamers", 1.3955032484805567], ["invite", 1.39535182558485], ["mopping", 1.3951101095136609], ["harbor's", 1.3947443777045605], ["enjolras", 1.3947443777045605], ["nicomachean", 1.3945724629532104], ["tam\u00e1s", 1.3945724629532104], ["disunity", 1.3945621602606648], ["parameterized", 1.3944598521277134], ["backtracking", 1.3943514919643623], ["kaum", 1.3943212791522461], ["berlioz's", 1.3943212791522461], ["investment's", 1.3943212791522461], ["tamilian", 1.3943212791522461], ["ohlone", 1.3943212791522461], ["transcribers", 1.3943212791522461], ["g\u00f6tz", 1.3943212791522461], ["curlers", 1.394321279152246], ["mononucleosis", 1.3941850817135009], ["frogging", 1.3941850817135009], ["bresse", 1.3941850817135009], ["socratic", 1.3939945151359023], ["tasmania's", 1.3939945151359023], ["comma", 1.3939945151359021], ["cheesesteak", 1.3939195732233631], ["floundered", 1.3939195732233631], ["bawk", 1.3939195732233631], ["mombo", 1.3939195732233631], ["artan", 1.3939195732233631], ["kedai", 1.3939195732233631], ["d'olonne", 1.3939195732233631], ["leppard's", 1.3939195732233631], ["distastefully", 1.3939195732233631], ["quli", 1.3939195732233631], ["bangus", 1.3939195732233631], ["bhuna", 1.3939195732233631], ["presumptuousness", 1.3939195732233631], ["marzocco", 1.3939195732233631], ["mileycyrus", 1.3939195732233631], ["matic's", 1.3939195732233631], ["shukra", 1.3939195732233631], ["convince", 1.393889703518223], ["awardees", 1.393747484387163], ["junius", 1.393747484387163], ["smarts", 1.3937474843871627], ["permission", 1.3937382754560863], ["scurry", 1.393668739366999], ["chopped", 1.3936560461398684], ["epoch", 1.3934609287186253], ["poverty", 1.3932497174284533], ["bundled", 1.3931392450412239], ["mapping", 1.3929241787455147], ["aptly", 1.3928424535531452], ["rushmore", 1.3928331019142173], ["abysmal", 1.3926797889875635], ["shikoku", 1.39266054909271], ["jumpsuits", 1.392581041182851], ["wondering", 1.3925696505216252], ["ebonite", 1.3924972063119108], ["timisoara", 1.3924972063119108], ["hippias", 1.3924972063119108], ["studied", 1.392470167531206], ["interventionism", 1.3922760872636677], ["stationed", 1.3922158937449791], ["alternating", 1.3921286512400448], ["holds", 1.3921085284713186], ["precancerous", 1.391879690921085], ["moving", 1.3918456864068969], ["unaffected", 1.3918382412578651], ["ferrets", 1.3918220561512749], ["headway", 1.3917146458984462], ["parables", 1.391685727753454], ["bluebird", 1.3916563827723731], ["idee", 1.3916165756676029], ["adolphe", 1.3915345397330572], ["herds", 1.3914500984238873], ["adults", 1.3914044263121925], ["bacchus", 1.3913892816155673], ["athenian", 1.3913141160771516], ["yagura", 1.3913141160771514], ["vertrag", 1.3913141160771514], ["shortfin", 1.3913141160771514], ["krokus", 1.3913141160771514], ["gaben", 1.3913141160771514], ["trogdor", 1.3913141160771514], ["milione", 1.3913141160771514], ["nowak's", 1.3913141160771514], ["lumet's", 1.3913141160771514], ["mudville", 1.3913141160771514], ["mmap", 1.3913141160771514], ["gashapon", 1.3913141160771514], ["ejercicio", 1.3913141160771514], ["mondrag\u00f3n", 1.3913141160771514], ["paleta", 1.3913141160771514], ["wohnung", 1.3913141160771514], ["jamdani", 1.3913141160771514], ["sandanski", 1.3913141160771514], ["eternalize", 1.3913141160771514], ["castoreum", 1.3913141160771514], ["angelicus", 1.3913141160771514], ["barsham", 1.3913141160771514], ["chido", 1.3913141160771514], ["downhaul", 1.3913141160771514], ["tsas", 1.3913141160771514], ["monads", 1.3913141160771512], ["donna's", 1.3913141160771512], ["adhd", 1.3911452672766564], ["sunglasses", 1.391113148482607], ["trevally", 1.3910488006469306], ["dise\u00f1o", 1.3910488006469306], ["hakea", 1.3910488006469306], ["savimbi", 1.3910488006469306], ["centraal", 1.3910488006469306], ["thorn's", 1.3910488006469306], ["ownership", 1.3909418055932343], ["guillemots", 1.3908658834366627], ["digitalized", 1.3908658834366627], ["hermosillo", 1.3908658834366627], ["trombone", 1.3908418944228913], ["pines", 1.3908123997746873], ["postcolonial", 1.390805807162092], ["cognizant", 1.390735278888163], ["abdulhamid", 1.390630087504842], ["vrbo", 1.390630087504842], ["sokha", 1.390630087504842], ["exosphere", 1.390630087504842], ["erikson's", 1.390630087504842], ["unida", 1.390630087504842], ["kainan", 1.390630087504842], ["ilha", 1.3905839974184206], ["groundnuts", 1.3905839974184206], ["nhat", 1.3905496577716145], ["tarpaulin", 1.3904928542725226], ["ramakrishna", 1.3904928542725226], ["blips", 1.3904928542725226], ["cartoonish", 1.39047956605037], ["establish", 1.3904067022988997], ["jessy", 1.3902313813564688], ["reinvest", 1.3902071241087484], ["appeased", 1.390162910543754], ["northumbrian", 1.390162910543754], ["rebooked", 1.3901330344845564], ["kiwi's", 1.3901330344845564], ["ersten", 1.3901330344845564], ["owing", 1.3900596879246983], ["praha", 1.3900569045264792], ["location", 1.3900244921617773], ["whiskeys", 1.3899786487755066], ["headwinds", 1.3899786487755066], ["goku's", 1.3899786487755066], ["cistern", 1.3899653932095257], ["dbms", 1.3897731036068126], ["someone", 1.3897379450816392], ["pendant", 1.3895512676463102], ["pedestals", 1.3894590305890484], ["chihiro", 1.3894590305890484], ["illusionary", 1.3894590305890484], ["obedience", 1.3893805123346077], ["rutherford's", 1.3893282702812573], ["riddle's", 1.3893282702812573], ["blank", 1.3892792351658356], ["meadow", 1.3892720072091236], ["stellar", 1.3892432760811932], ["thickets", 1.389137842648552], ["allegations", 1.3890455011156588], ["syrupy", 1.3890348977958284], ["spheroid", 1.3890348977958284], ["renaming", 1.389021543488737], ["twister", 1.389015963296131], ["dufresne", 1.3889841642081227], ["acra", 1.388858443154171], ["kerk", 1.388858443154171], ["seismometer", 1.388858443154171], ["rollerblade", 1.388858443154171], ["pane", 1.3888538655060392], ["toxic", 1.3886747505533097], ["mermaid's", 1.3886045047399778], ["gondola", 1.3886045047399778], ["gallery", 1.3885757637526825], ["unofficial", 1.3885360545215266], ["flouted", 1.3884731325067954], ["archiving", 1.3884425335594874], ["gamedev", 1.3882775178079354], ["wingsuit", 1.3882775178079354], ["carolers", 1.3879511330617715], ["mercantilist", 1.3879511330617715], ["grandmothers", 1.3878844864324371], ["yipping", 1.3877768767989893], ["patissier", 1.3877768767989893], ["aliz\u00e9", 1.3877768767989893], ["kezar", 1.3877768767989893], ["chitter", 1.3877768767989893], ["splinted", 1.3877768767989893], ["syra", 1.3877768767989893], ["kinetoscope", 1.3877768767989893], ["asheton", 1.3876827966093883], ["ducati's", 1.3876827966093883], ["insectivores", 1.3876827966093883], ["consultation", 1.3875686101392675], ["poco", 1.3875002067677293], ["chisel", 1.3875002067677293], ["chrono", 1.3874335801071402], ["evelyn", 1.387401773409874], ["trabant", 1.38736792531032], ["madrid's", 1.38736792531032], ["interweaves", 1.38736792531032], ["fra\u00eeche", 1.38736792531032], ["broaches", 1.38736792531032], ["birthing", 1.3872940542415668], ["greets", 1.3872034152796506], ["fuzzies", 1.3872034152796506], ["thorne", 1.386977853748711], ["ingrid", 1.3869499753572465], ["preconfigured", 1.3867604696750861], ["kh\u00f4ng", 1.3867604696750861], ["isizulu", 1.3867604696750861], ["molise", 1.3867604696750861], ["malamute", 1.3867154215829058], ["flaherty's", 1.3866856592705075], ["sportspersons", 1.3866856592705075], ["eurocentric", 1.3865399990121807], ["forefather", 1.3865399990121807], ["ahmadiyya", 1.3865399990121807], ["yuan's", 1.3865399990121807], ["ringcentral", 1.3865399990121807], ["boogied", 1.3865399990121807], ["doordash", 1.3865399990121807], ["nodier", 1.3865399990121807], ["jokowi's", 1.3865399990121807], ["makiling", 1.3865399990121807], ["ptolemaeus", 1.3865399990121807], ["fahren", 1.3865399990121807], ["besart", 1.3865399990121807], ["tunay", 1.3865399990121807], ["dulces", 1.3865399990121807], ["shehnai", 1.3865399990121807], ["mikey", 1.386246792715397], ["synthetic", 1.3862287666891702], ["graffitied", 1.3861323130377579], ["thula", 1.3861323130377579], ["netty", 1.3861323130377579], ["energia", 1.3860895311762444], ["amigos", 1.3860592625864778], ["foresee", 1.3860083873011617], ["psychologist", 1.3858807882361053], ["acquisitions", 1.3856613109354707], ["long", 1.3856220403978763], ["propriety", 1.3853572471576963], ["dogma", 1.3853237358551527], ["televisions", 1.3853237358551527], ["outback", 1.3853237358551527], ["virtualbox", 1.3853237358551527], ["advancing", 1.385209501193995], ["portraits", 1.385209501193995], ["training", 1.3850142734015563], ["mismatches", 1.3849477999702529], ["impassively", 1.3849106923153391], ["tiresias", 1.3849106923153391], ["nightly", 1.384776014698345], ["player's", 1.384739054080437], ["viking", 1.384596422711], ["lusail", 1.3845852903669569], ["endeavor's", 1.3845852903669569], ["ecevit", 1.3845852903669569], ["altissimo", 1.3845852903669569], ["manarola", 1.3845852903669569], ["velika", 1.3845852903669569], ["commerce", 1.3844754702687123], ["igual", 1.3844641012824608], ["stjepan", 1.3844641012824608], ["latha", 1.3844641012824608], ["everlastingly", 1.3844641012824608], ["darkthrone", 1.3844641012824608], ["observational", 1.3844247587390883], ["masterstroke", 1.3844247587390883], ["contrite", 1.3843652238307396], ["complete", 1.3843391418708917], ["lambic", 1.3842723897363653], ["driss", 1.3842723897363653], ["paraguay's", 1.3842723897363653], ["pecans", 1.384235500398864], ["chargebacks", 1.3840751054083775], ["destabilizing", 1.3840728459890275], ["charging", 1.3839266768234308], ["glauben", 1.383923337240088], ["bolls", 1.383923337240088], ["dynegy", 1.383923337240088], ["zaandam", 1.383923337240088], ["dadaji", 1.383923337240088], ["nordby", 1.383923337240088], ["procumbens", 1.383923337240088], ["vaes", 1.383923337240088], ["circula", 1.383923337240088], ["lichen", 1.383814247964172], ["tome", 1.383792145212969], ["eventuality", 1.3837332378805771], ["supermassive", 1.3835714923238407], ["schnoz", 1.3835558589243047], ["ramotswe", 1.3835558589243047], ["ledezma", 1.3835558589243047], ["spellings", 1.3833465690050726], ["mirrorless", 1.3833109817811324], ["rehydration", 1.3832560884882046], ["contextualize", 1.3831847903031376], ["kept", 1.3831320296461875], ["blackthorn", 1.383088441404398], ["zinfandel", 1.383088441404398], ["antietam", 1.3830483344465496], ["runny", 1.3830210746083793], ["lara", 1.3828806942976637], ["bullies", 1.382730415428811], ["midsummer's", 1.3826991679899865], ["closeness", 1.3826268045069594], ["monosyllabic", 1.382583480308595], ["prospers", 1.382583480308595], ["chasers", 1.3825201462050536], ["giorgi", 1.3822081028364637], ["lagged", 1.3821843759786347], ["disguise", 1.382133651346147], ["dandruff", 1.382132065422951], ["dynasty", 1.3820590294862873], ["ireland's", 1.3820155000680359], ["outbursts", 1.3820155000680359], ["nihilistic", 1.3819212781627064], ["ebay's", 1.3817936062042657], ["disparagingly", 1.3817936062042657], ["pharmacist", 1.381641624105793], ["tapa", 1.3815643770061563], ["capitalization", 1.381564377006156], ["minaret", 1.381441253899748], ["erica's", 1.3813644111099328], ["creates", 1.3812750441523327], ["nasa's", 1.3812675855180776], ["hardline", 1.3812162882238417], ["sparkles", 1.381209273840319], ["oscar's", 1.3810824079229513], ["kilometers", 1.380975733071404], ["coca", 1.380828850274831], ["himalayan", 1.380828850274831], ["wget", 1.380828850274831], ["sweetpea", 1.380828850274831], ["alaykum", 1.380828850274831], ["s\u00e9rie", 1.380828850274831], ["mushers", 1.380828850274831], ["commemorative", 1.3807019358584451], ["annecy", 1.3806548547039796], ["excerpts", 1.3805769967637156], ["clearings", 1.3805522419314264], ["ceremony", 1.3805437754022842], ["indoor", 1.3804824151910793], ["flapping", 1.3804586832270396], ["hoteliers", 1.3804586832270396], ["luxor", 1.3804459873694506], ["gerrymandering", 1.380436569351457], ["embellish", 1.3803730308920377], ["unexciting", 1.380257315154684], ["freebsd", 1.380257315154684], ["hilltop", 1.3801546174377826], ["notifications", 1.3801206844941236], ["sarcasm", 1.3801174295354994], ["stirrings", 1.3799803774911297], ["antlers", 1.3798239330945836], ["acacias", 1.3797198317765085], ["exasperation", 1.3796933002334864], ["gelatinous", 1.3796514289192776], ["harper", 1.379647103993168], ["loosened", 1.3795862850463438], ["appraising", 1.3792424200700115], ["guarani", 1.3792424200700115], ["phone's", 1.378780547985064], ["being", 1.378437147385843], ["messieurs", 1.378431577965326], ["undulations", 1.378431577965326], ["nina's", 1.378431577965326], ["heydays", 1.378431577965326], ["arboretums", 1.378431577965326], ["wh\u0101nau", 1.378431577965326], ["roboticists", 1.378431577965326], ["oliva", 1.378431577965326], ["nastase", 1.378431577965326], ["sudamericana", 1.378431577965326], ["fruitlessly", 1.378431577965326], ["ren\u00e9's", 1.378431577965326], ["veil's", 1.378431577965326], ["fursuits", 1.378431577965326], ["sparging", 1.378431577965326], ["adzes", 1.378431577965326], ["pac\u00edfico", 1.378431577965326], ["zakariya", 1.378431577965326], ["kaia", 1.378431577965326], ["stehen", 1.378431577965326], ["pixabay", 1.378431577965326], ["alchemy's", 1.378431577965326], ["runnier", 1.378431577965326], ["sipi", 1.378431577965326], ["adnoc", 1.378431577965326], ["jaak", 1.378431577965326], ["enuma", 1.378431577965326], ["cwtch", 1.378431577965326], ["concerne", 1.3784315779653258], ["niger's", 1.3784315779653258], ["webseite", 1.3784315779653258], ["edificio", 1.3784315779653258], ["underutilization", 1.3784315779653258], ["f\u00e9ile", 1.3784315779653258], ["micronutrient", 1.3784315779653258], ["leechers", 1.3784315779653258], ["suavecito", 1.3784315779653258], ["siey\u00e8s", 1.3784315779653258], ["tweedy's", 1.3784315779653258], ["crawdaddy", 1.3784315779653258], ["parcel's", 1.3784315779653258], ["venkateswara", 1.3784315779653258], ["contritely", 1.3784315779653258], ["peppier", 1.3784315779653258], ["desideri", 1.3784315779653258], ["eataly", 1.3784315779653258], ["theatro", 1.3784315779653258], ["navette", 1.3784315779653258], ["rinconada", 1.3784315779653258], ["toyoshima", 1.3784315779653258], ["cheeping", 1.3784315779653258], ["mouratoglou", 1.3784315779653258], ["eudo", 1.3784315779653258], ["areco", 1.3784315779653258], ["isidorus", 1.3784315779653258], ["cullors", 1.3784315779653258], ["marshmallowy", 1.3784315779653258], ["sharov", 1.3784315779653258], ["br\u00fcder", 1.3784315779653258], ["spreewald", 1.3784315779653258], ["poultices", 1.3784315779653258], ["kayah", 1.3784315779653258], ["emmett's", 1.3784315779653258], ["fa'afafine", 1.3784315779653258], ["nita's", 1.3784315779653258], ["gwenny", 1.3784315779653258], ["huerto", 1.3784315779653258], ["dbsa", 1.3784315779653258], ["cittadella", 1.3784315779653258], ["cvetkovic", 1.3784315779653258], ["yuna's", 1.3784315779653258], ["suitor's", 1.3784315779653258], ["aitana", 1.3784315779653258], ["multiplicand", 1.3784315779653258], ["minervois", 1.3784315779653258], ["faisant", 1.3784315779653258], ["bork's", 1.3784315779653258], ["kaitlin's", 1.3784315779653258], ["gemeinden", 1.3784315779653258], ["meeropol", 1.3784315779653258], ["hissa", 1.3784315779653258], ["volumetrics", 1.3784315779653258], ["migratorius", 1.3784315779653258], ["nitrifying", 1.3784315779653258], ["boogying", 1.3784315779653258], ["millet's", 1.3784315779653258], ["d\u00fchring", 1.3784315779653258], ["tiotropium", 1.3784315779653258], ["flnc", 1.3784315779653258], ["solanine", 1.3784315779653258], ["vanderlei", 1.3784315779653258], ["stadtmuseum", 1.3784315779653258], ["windtalkers", 1.3784315779653258], ["millonarios", 1.3784315779653258], ["afram", 1.3784315779653258], ["kosai", 1.3784315779653258], ["virtualenv", 1.3784315779653258], ["cornville", 1.3784315779653258], ["nmpa", 1.3784315779653258], ["moivre", 1.3784315779653258], ["miege", 1.3784315779653258], ["buruk", 1.3784315779653258], ["diggles", 1.3784315779653258], ["finanzas", 1.3784315779653258], ["yechiel", 1.3784315779653258], ["varada", 1.3784315779653258], ["sextillion", 1.3784315779653258], ["verna's", 1.3784315779653258], ["gunnhild", 1.3784315779653258], ["somnia", 1.3784315779653258], ["vorse", 1.3784315779653258], ["wiesler", 1.3784315779653258], ["redpolls", 1.3784315779653258], ["broen", 1.3784315779653258], ["allegoria", 1.3784315779653258], ["staes", 1.3784315779653258], ["ineson", 1.3784315779653258], ["harg", 1.3784315779653258], ["shaden", 1.3784315779653258], ["suitcase", 1.378276261731189], ["cheese", 1.3779535734665151], ["mowing", 1.3777591723175378], ["lifting", 1.3777004099109267], ["fishers", 1.3776216886715744], ["yogi", 1.3774809354977635], ["whiskey", 1.3774028976832624], ["pastrami", 1.3773833410239074], ["yada", 1.3773063276975992], ["gagarin", 1.3773063276975992], ["strapped", 1.3772787079183004], ["jordan", 1.3772063054515788], ["bighorn", 1.3771403071662953], ["stavros", 1.3770932948799228], ["malware", 1.3770932948799226], ["santana's", 1.3770531463873608], ["p\u00e8re", 1.3770531463873608], ["jaume", 1.3769881941140532], ["costly", 1.3768927883071977], ["modes", 1.3768899110689172], ["defended", 1.3766809698613098], ["dingle", 1.3765813342365134], ["hyperthyroidism", 1.376543315529757], ["confusions", 1.376543315529757], ["atom's", 1.3763924779979217], ["sorghum", 1.3763924779979215], ["tintern", 1.376247058824335], ["palmistry", 1.376247058824335], ["dried", 1.3761509002636014], ["fuzhou", 1.3760549717964201], ["fertility", 1.3759627452883731], ["trail's", 1.3759253387326615], ["pango", 1.3759253387326615], ["extinction", 1.3758664965941554], ["availability", 1.3758585056864574], ["demons", 1.375564440283158], ["stadium's", 1.37551646586691], ["proofreader", 1.3754349875784448], ["playing", 1.3754255367933785], ["aristocrats", 1.3752575578844846], ["unsaved", 1.3751100560907106], ["dixieland", 1.3751100560907106], ["factually", 1.3750175059455976], ["flavian", 1.3749594581215845], ["sandbars", 1.3749594581215845], ["fulfill", 1.374928381689834], ["cursors", 1.37487891925923], ["batak", 1.37487891925923], ["shepherd's", 1.3748710535941977], ["pervasive", 1.374774514595214], ["mayhem", 1.374765536534567], ["shrubs", 1.37472041602465], ["sardines", 1.3747060872140682], ["paneling", 1.3746218424483363], ["posted", 1.3746198379105021], ["contaminated", 1.3745521597518906], ["algernon", 1.3744413812922682], ["minefields", 1.3743773674418984], ["hypnotherapist", 1.3743773674418984], ["ncsu", 1.3743773674418984], ["unblemished", 1.3743321531981427], ["retweets", 1.3743168568370714], ["solidity", 1.3742349478684759], ["sculpture", 1.374190250033125], ["fragment", 1.3740827234094917], ["invigoration", 1.3740694527186004], ["waimanalo", 1.3740694527186004], ["ridwan", 1.3740694527186004], ["casamance", 1.3740694527186004], ["mariko", 1.374038170545516], ["montague", 1.3739516753369385], ["beartooth", 1.3738972635641242], ["scrimshaw", 1.3738972635641242], ["reconsideration", 1.3738059686432946], ["door", 1.3736932194160703], ["geological", 1.3736560737088348], ["hohenzollern", 1.3735086080440213], ["baseload", 1.373345114578], ["trip's", 1.373345114578], ["uc's", 1.373345114578], ["netflix's", 1.3732829370968644], ["deliciously", 1.3732802228414864], ["vita", 1.3732676651052746], ["installed", 1.3730990797095317], ["scandinavian", 1.3730784456237128], ["cp's", 1.3729832318073603], ["absorption", 1.3727911686638046], ["wailer", 1.3726761225249695], ["theodosius", 1.3726662747875242], ["merchant", 1.3726077045484224], ["cognates", 1.3725659116761117], ["retiring", 1.3725115310932403], ["elbe", 1.3723323231955677], ["bereft", 1.372314924370551], ["abstaining", 1.3722427014928285], ["beeping", 1.3722427014928285], ["jyoti", 1.3722427014928285], ["canap\u00e9s", 1.3721659798836654], ["slurs", 1.3721659798836652], ["psilocybin", 1.372041497802573], ["locarno", 1.371944841127842], ["purslane", 1.3718045030712618], ["steakhouses", 1.3718045030712618], ["simd", 1.3718045030712618], ["strabo", 1.3717684645975177], ["furloughs", 1.3717075214874463], ["moneybags", 1.3717075214874463], ["lawn", 1.3716621077028575], ["guevara's", 1.3716579584175848], ["glomerulus", 1.3716579584175848], ["nih's", 1.3716579584175848], ["studded", 1.3716168600517895], ["blender", 1.371565041480294], ["happenstance", 1.371414108113866], ["infrasound", 1.3713771686615115], ["cockatiels", 1.371344526150594], ["csikszentmihalyi", 1.371344526150594], ["mazatl\u00e1n", 1.371344526150594], ["salford's", 1.371344526150594], ["blur", 1.3712870311950267], ["handouts", 1.371226140171416], ["adequately", 1.3711766749234033], ["dancer's", 1.3711766749234031], ["nuget", 1.3711766749234031], ["sportscasters", 1.3711766749234031], ["greasepaint", 1.3711766749234031], ["castoff", 1.3711766749234031], ["yosi", 1.3711766749234031], ["anise", 1.3710306030500756], ["poitiers", 1.3708785282230502], ["odin's", 1.3708502042865167], ["gardener's", 1.3708502042865167], ["regrets", 1.3708193438780545], ["huangpu", 1.3708159338881694], ["shamim", 1.3708159338881694], ["alarmism", 1.3708159338881694], ["corte", 1.3706064205358284], ["umoja", 1.3705548260912381], ["frumpy", 1.3704929667833265], ["drug's", 1.3704929667833265], ["dulce", 1.3704790880924491], ["raise", 1.3704307882725806], ["zhou", 1.3703332924447797], ["pesce", 1.3703326850278092], ["crystallizing", 1.3703326850278092], ["grades", 1.3702331576900977], ["proofing", 1.3702021357088165], ["walt's", 1.370143539996547], ["equalized", 1.370143539996547], ["gourd", 1.3700900858142497], ["faye's", 1.3700900858142497], ["skyhawk", 1.3700900858142497], ["singh's", 1.3700861675768092], ["knuckled", 1.3700411248820588], ["tangentially", 1.3698163806030428], ["jalisco", 1.3698163806030426], ["conditioner", 1.3697108883904432], ["spat", 1.369609615866348], ["quahog", 1.369567066209922], ["heartburn", 1.3695024665291133], ["racetrack", 1.3693733133101254], ["preferential", 1.3692420341122238], ["lymphedema", 1.3692420341122236], ["farmworkers", 1.36909558719823], ["anjana", 1.3690704127499418], ["skidding", 1.3690635963675033], ["preoperational", 1.3689251532897029], ["veidt's", 1.3689251532897029], ["taihang", 1.3689251532897029], ["m\u00e9zi\u00e8res", 1.3689251532897029], ["pau's", 1.3689251532897029], ["labyrinthitis", 1.3689251532897029], ["snufkin", 1.3689251532897029], ["kristofferson's", 1.3689251532897029], ["gmcs", 1.3689251532897029], ["dongting", 1.3689251532897029], ["cordova's", 1.3689251532897029], ["munduruku", 1.3689251532897029], ["kuropatkin", 1.3689251532897029], ["eraser", 1.3687778091644207], ["wizarding", 1.3687583739094287], ["lemon", 1.3686840975211425], ["emmental", 1.3686206770545408], ["growths", 1.3686206770545406], ["pangasinan", 1.3685856381227164], ["typeface", 1.3685021979545586], ["hezekiah", 1.3683877297841716], ["changers", 1.3683827899167673], ["kafka", 1.3682373894466113], ["casa", 1.3682224449527296], ["quantify", 1.3682121714390312], ["nocturne", 1.3681553528339647], ["construct", 1.3680247803976506], ["nerves", 1.3679554979727895], ["mel's", 1.3679492085511409], ["wannacry", 1.3679492085511407], ["darlene's", 1.3679492085511407], ["snoozy", 1.3679492085511407], ["kyara", 1.3679492085511407], ["modbus", 1.3679492085511407], ["flailing", 1.3678282581348236], ["fishmonger", 1.3678282581348233], ["refueled", 1.3678282581348233], ["garland's", 1.3678282581348233], ["unidentifiable", 1.3678282581348233], ["finer", 1.3676128121940443], ["bifocals", 1.3673534826200247], ["deschutes", 1.367315194272057], ["philo's", 1.367315194272057], ["jotunheim", 1.367315194272057], ["mahalakshmi", 1.367315194272057], ["sandboxing", 1.367315194272057], ["baobabs", 1.367315194272057], ["pinkel", 1.367315194272057], ["graders", 1.367286812015819], ["storico", 1.3672611437840834], ["devotionals", 1.3672611437840834], ["herries", 1.3672611437840834], ["muscogee", 1.3672611437840834], ["regret", 1.3672450755441456], ["batangas", 1.367179075288058], ["dieta", 1.3671790752880577], ["pedicabs", 1.3671790752880577], ["manutius", 1.3671790752880577], ["cdfi", 1.3671790752880577], ["hardbodies", 1.3671790752880577], ["hassanal", 1.3671790752880577], ["lingala", 1.3671790752880577], ["boltz", 1.3671790752880577], ["piccata", 1.3671790752880577], ["ishani", 1.3671790752880577], ["amigaos", 1.3671790752880577], ["poetry", 1.367069128025843], ["cogwheel", 1.3670395814532157], ["contagions", 1.3670395814532157], ["samkhya", 1.3670395814532157], ["formula", 1.3670228570328042], ["refraction", 1.3669641505764802], ["translucent", 1.3669446481489482], ["arises", 1.366780952343211], ["supervolcano", 1.3667499544232469], ["anche", 1.3667499544232469], ["postfix", 1.3667499544232469], ["missy", 1.3666921881529839], ["crossing", 1.366618290516782], ["juices", 1.366534088680226], ["doner", 1.3664452164178014], ["cower", 1.3663945698197133], ["tesla", 1.3663322719244417], ["universalis", 1.3663237330237386], ["boo's", 1.3663237330237386], ["arwen", 1.3663237330237386], ["doorways", 1.3659291560756288], ["insistent", 1.3657549304179661], ["digress", 1.36572603472495], ["grandmaster", 1.3657154194970111], ["riverside", 1.3657075941687227], ["tickles", 1.3656753741360887], ["sandwich", 1.365658112009514], ["audrey's", 1.3656089586354159], ["ember", 1.3655780359199898], ["chloe", 1.3655302432676433], ["genially", 1.3655188699750185], ["floristry", 1.3655188699750185], ["pl\u00e1cido", 1.3655188699750185], ["downturned", 1.3655188699750185], ["procol", 1.3655188699750185], ["spritzing", 1.3655188699750185], ["silkie", 1.3655188699750185], ["denoising", 1.3655188699750185], ["konjac", 1.3655188699750185], ["autonomously", 1.3654989247166809], ["vinny", 1.3652967849187454], ["unnaturally", 1.3652706734520892], ["quinary", 1.36524084516183], ["coupole", 1.36524084516183], ["footlight", 1.36524084516183], ["neom", 1.36524084516183], ["prasat", 1.36524084516183], ["blackie's", 1.36524084516183], ["sucrase", 1.36524084516183], ["omer's", 1.36524084516183], ["queenie's", 1.36524084516183], ["hughesy", 1.36524084516183], ["jehanne", 1.36524084516183], ["camelus", 1.36524084516183], ["shipbuilder", 1.3652408451618299], ["ghat", 1.3651890038538688], ["ladybug", 1.3651774281771976], ["fritz", 1.365118968809311], ["poetically", 1.3651084474719808], ["teeter", 1.3651084474719808], ["nikolas", 1.3651084474719808], ["imdb's", 1.3650487471112938], ["taina", 1.3650487471112938], ["candle's", 1.3650487471112938], ["revitalizes", 1.3650487471112938], ["cider", 1.365039853457878], ["reside", 1.3649123451814351], ["asynchronous", 1.3648107125704116], ["lanes", 1.3647802298620035], ["manchus", 1.364647262185673], ["roomba", 1.364647262185673], ["mctavish", 1.364647262185673], ["preventive", 1.3646472621856727], ["resumes", 1.3646472621856727], ["gluing", 1.3646472621856727], ["tortilla", 1.3646472621856727], ["gravesite", 1.3646472621856727], ["siem", 1.3646472621856727], ["cawdor", 1.3646472621856727], ["khalid", 1.3646472621856727], ["khanates", 1.3646472621856727], ["xylitol", 1.3646472621856727], ["georgey", 1.3646472621856727], ["adelaida", 1.3646472621856727], ["mpla", 1.3646472621856727], ["prima's", 1.3646472621856727], ["diko", 1.3646472621856727], ["milgram's", 1.3646472621856727], ["sorbian", 1.3646472621856727], ["factfinder", 1.3646472621856727], ["radka", 1.3646472621856727], ["nalu", 1.3645080266727467], ["arctic", 1.3644153764996598], ["recovery", 1.3643534386124747], ["lilacs", 1.364153003850944], ["shrimp", 1.3640440587398854], ["tiptoeing", 1.3639977394526], ["dexterous", 1.3639977394526], ["errigal", 1.3639977394526], ["wascana", 1.3639977394526], ["hieroglyph", 1.3639977394526], ["posad", 1.3639977394526], ["pamir", 1.3639977394526], ["stewardson", 1.3639977394526], ["superstrong", 1.3639977394526], ["markievicz", 1.3639977394526], ["pashminas", 1.3639977394526], ["gillan's", 1.3639977394526], ["panela", 1.3639977394526], ["dehumanize", 1.3637674122422905], ["flayer", 1.3637674122422905], ["piraeus", 1.3637674122422905], ["unexpectedness", 1.3637674122422905], ["diop", 1.3637674122422905], ["sefa", 1.3637674122422905], ["sexto", 1.3637674122422905], ["carafes", 1.3637674122422905], ["homeowners", 1.3636467876532783], ["gyatso", 1.3634950128790144], ["balloon", 1.3633972990419256], ["terraces", 1.3633748638060124], ["cobwebbed", 1.363283978207465], ["eatting", 1.363283978207465], ["annacone", 1.363283978207465], ["erich's", 1.363283978207465], ["pr\u00e9sence", 1.363283978207465], ["pulmo", 1.363283978207465], ["ruku", 1.363283978207465], ["kalighat", 1.363283978207465], ["ikura", 1.363283978207465], ["salesians", 1.363283978207465], ["hilsa", 1.363283978207465], ["jampa", 1.363283978207465], ["olfa", 1.363283978207465], ["ecdis", 1.363283978207465], ["lasha", 1.363283978207465], ["korbut", 1.363283978207465], ["getresponse", 1.363283978207465], ["anthemic", 1.363115671543489], ["detergent", 1.3630768970968516], ["allows", 1.3630567642110971], ["oreos", 1.363030107820462], ["mactan", 1.3630301078204619], ["pullman's", 1.3630301078204619], ["raghunath", 1.3630301078204619], ["snell's", 1.362899954551632], ["emirate's", 1.362899954551632], ["highrises", 1.362899954551632], ["liveness", 1.362899954551632], ["skald", 1.362899954551632], ["zuri", 1.3627675827611745], ["bavarian", 1.3627675827611743], ["samatha", 1.3626600496133199], ["laddu", 1.3626600496133199], ["hawkeye's", 1.3626600496133199], ["gem's", 1.3626600496133199], ["leonard's", 1.3626135106771993], ["ouch", 1.3625967028134929], ["ankh", 1.3625265982195722], ["stampedes", 1.3624959527865361], ["metoprolol", 1.3624959527865361], ["farmhouses", 1.3624319257210855], ["priced", 1.3623868116880447], ["kali", 1.3623498762223971], ["intervening", 1.3623284053255442], ["rochefort", 1.362328405325544], ["starches", 1.3622917145524407], ["wanderer", 1.3622147358716163], ["posies", 1.3622147358716161], ["itemize", 1.3622147358716161], ["barebones", 1.3622147358716161], ["stereotypical", 1.3621325559285737], ["sloshing", 1.3621154490832874], ["properly", 1.362063885841614], ["introverts", 1.3619983189397606], ["parsley", 1.3619793300992882], ["backends", 1.3619234153549626], ["solera", 1.3619234153549626], ["petrescu", 1.3619234153549626], ["land's", 1.3619234153549624], ["howling", 1.3619206942292579], ["excels", 1.3615858806552914], ["valley's", 1.361574597367917], ["incognito", 1.3614109445678408], ["corsica", 1.3612291093262123], ["misleading", 1.3612098635403687], ["taoiseach", 1.3612011832407593], ["simmons", 1.3611497492266562], ["pyrenees", 1.3611314245576638], ["trajan", 1.361033897855084], ["sulla", 1.361033897855084], ["greats", 1.3608300670466917], ["summarized", 1.360770423372645], ["geduld", 1.360645364056096], ["\u043e\u0431\u043b\u0430\u0441\u0442\u0438", 1.360645364056096], ["ilmari", 1.360645364056096], ["yoyo's", 1.360645364056096], ["malayalis", 1.360645364056096], ["orkan", 1.360645364056096], ["sirius's", 1.360645364056096], ["sikelel", 1.360645364056096], ["presentation's", 1.360645364056096], ["lexico", 1.360645364056096], ["wolfsheim", 1.360645364056096], ["stanny", 1.360645364056096], ["rmhc", 1.360645364056096], ["cso's", 1.360645364056096], ["raveonettes", 1.360645364056096], ["ponders", 1.3606453640560958], ["concatenation", 1.3606453640560958], ["peloponnese", 1.3603693710816422], ["kabat", 1.360294320360519], ["anchoring", 1.3602593288667475], ["brief", 1.36022462068779], ["giant's", 1.360136031566877], ["eukaryotic", 1.360136031566877], ["relativity", 1.3601154323403069], ["astutely", 1.3600524902591216], ["chipp", 1.3599291406772005], ["webkinz", 1.3599291406772005], ["matchbox", 1.3598757682619464], ["ginger", 1.3598365197807878], ["trawling", 1.3598211745099589], ["room's", 1.359790866234834], ["acclaimed", 1.3596948134025637], ["finality", 1.3595897074751524], ["kickball", 1.3595489536096363], ["bucketloads", 1.3595489536096363], ["inkpot", 1.3595489536096363], ["escapology", 1.3595489536096363], ["sinabi", 1.3595489536096363], ["reba's", 1.3595489536096363], ["shepherdesses", 1.3595489536096363], ["\u00e9poca", 1.3595489536096363], ["euphorically", 1.3595489536096363], ["truthy", 1.3595489536096363], ["indonesia's", 1.359457216163374], ["enterprises", 1.3594367908209637], ["puebla", 1.359405339283551], ["carrefour", 1.3593681625253797], ["aitor", 1.3593349188247483], ["urania", 1.3593349188247483], ["flutie", 1.3593349188247483], ["phantom's", 1.3593349188247483], ["titan's", 1.3592866949380298], ["ffvii", 1.3592533994892868], ["cotopaxi", 1.3592533994892868], ["tuna", 1.3592533994892866], ["sunblock", 1.359210420503658], ["couple", 1.3591886731369298], ["inefficiencies", 1.3590851698535318], ["halted", 1.3590252602815844], ["motorized", 1.3590082239121781], ["typesetter", 1.3588919550817968], ["childrearing", 1.3588919550817968], ["hollowed", 1.358829071410297], ["microbiologists", 1.358739698280107], ["karen", 1.358696472202749], ["jumpstart", 1.358587832884077], ["meltdowns", 1.358587832884077], ["futhark", 1.3585694514816757], ["boleros", 1.3585694514816757], ["flirtatiously", 1.3585694514816757], ["makki", 1.3585694514816757], ["krauser", 1.3585694514816757], ["cycad", 1.3585694514816757], ["clockworks", 1.3585694514816757], ["jaakko", 1.3585694514816757], ["hecklers", 1.3583658018303875], ["ville", 1.3583544223732222], ["uncaring", 1.358117849447942], ["highborn", 1.3580857244160223], ["giardino", 1.3580857244160223], ["vishwanath", 1.3580857244160223], ["threading", 1.358085724416022], ["overshare", 1.3580405782912828], ["hrant", 1.3580405782912828], ["borne", 1.3580201872916084], ["paleocene", 1.357941378833409], ["isla", 1.3579413788334087], ["petronas", 1.3578804162574791], ["exponential", 1.357690691605287], ["revise", 1.3576638174363782], ["campus", 1.3574870759334638], ["googler", 1.3573177463553536], ["safecracker", 1.3573177463553536], ["mocap", 1.3573177463553536], ["scriptwriters", 1.357224938304321], ["unvaccinated", 1.357224938304321], ["prefabricated", 1.3572077530209652], ["scratcher", 1.357051822878517], ["unwarranted", 1.3569536905877266], ["curbing", 1.3569522356763049], ["eaters", 1.3569306834956276], ["telekinetic", 1.3568935845596177], ["judith's", 1.3568935845596175], ["turtle", 1.356864973203064], ["antarctica", 1.356596245889592], ["fugit", 1.3565863865554157], ["cepheid", 1.3565863865554157], ["hoka", 1.3565863865554157], ["usns", 1.3565863865554157], ["palabra", 1.3565863865554157], ["stoa", 1.3565863865554157], ["grandchildren", 1.3564034817599155], ["acorn", 1.3563766727178808], ["neurotransmission", 1.3562702985768158], ["straightedge", 1.3562702985768158], ["pouches", 1.3562100109653812], ["scroll", 1.3560796625988716], ["ducks", 1.3558975201794443], ["worse", 1.3557585101030198], ["timothy", 1.3556384506625103], ["regina's", 1.3556098630983504], ["nuoc", 1.3556098630983504], ["surtsey", 1.3556098630983504], ["paper", 1.3555284071314404], ["lagging", 1.3554577183325704], ["olam", 1.3554577183325704], ["verne", 1.3554024927084531], ["goalposts", 1.3553986595674477], ["bestselling", 1.3551604095608525], ["limping", 1.354853143079077], ["grumet", 1.3546655162762686], ["unvalued", 1.3546655162762686], ["vergangenheit", 1.3546655162762686], ["bipm", 1.3546655162762686], ["philosophicus", 1.3546655162762686], ["colasanto", 1.3546655162762686], ["viardot", 1.3546655162762686], ["bienes", 1.3546655162762686], ["drewery", 1.3546655162762686], ["magnetospheres", 1.3546655162762686], ["iccat", 1.3546655162762686], ["pleiotropy", 1.3546655162762686], ["tbilisi's", 1.3546655162762686], ["ayyappan", 1.3546655162762686], ["esmee", 1.3546655162762686], ["hoisting", 1.3545464378878473], ["quinn's", 1.3544947421802993], ["qwerty", 1.3544398891083342], ["reprisal", 1.3544345909805044], ["macros", 1.3544345909805044], ["valdo", 1.3543541704899489], ["realsense", 1.3543541704899489], ["outras", 1.3543541704899489], ["condor's", 1.3543541704899489], ["azelaic", 1.3543541704899489], ["freezy", 1.3543541704899489], ["epicondylitis", 1.3543541704899489], ["hackery", 1.3543541704899489], ["ticketholders", 1.3543541704899489], ["charm's", 1.3543541704899489], ["rotifer", 1.3543541704899489], ["curiosa", 1.3543541704899489], ["hilkiah", 1.3543541704899489], ["gans", 1.3543090253509327], ["electric's", 1.3543090253509327], ["naturopathic", 1.3543090253509327], ["galloping", 1.3542968770115265], ["vegemite", 1.354267837156372], ["goats", 1.3542499497118772], ["hesi", 1.3542485678255831], ["barbeques", 1.3542485678255831], ["nkurunziza", 1.3542485678255831], ["database", 1.3542482121930812], ["gratifying", 1.35404394235517], ["pappy's", 1.3540345588862936], ["danaus", 1.3540345588862936], ["cante", 1.3540345588862936], ["insiders", 1.353962768614841], ["multiracial", 1.3538974327837416], ["bakes", 1.3537892257335435], ["rollo", 1.3537892257335435], ["childfree", 1.3536807347145672], ["maathai", 1.3536807347145672], ["faraday's", 1.3536807347145672], ["fark", 1.3536807347145672], ["nikka", 1.3536807347145672], ["ladytron", 1.3533691856386836], ["effleurage", 1.3533691856386836], ["juszczyk", 1.3533691856386836], ["ilmen", 1.3533691856386836], ["pereire", 1.3533691856386836], ["rubirosa", 1.3533691856386836], ["seductresses", 1.3533691856386836], ["veramente", 1.3533691856386836], ["assun\u00e7\u00e3o", 1.3533691856386836], ["kdei", 1.3533691856386836], ["picturephone", 1.3533691856386836], ["suebi", 1.3533691856386836], ["berom", 1.3533691856386836], ["vistavision", 1.3533691856386836], ["nypl's", 1.3533691856386836], ["lyndell", 1.3533691856386836], ["artaphernes", 1.3533691856386836], ["basu's", 1.3533691856386836], ["nakamoto's", 1.3533691856386836], ["a'dam", 1.3533691856386836], ["kasbahs", 1.3533691856386836], ["laiho", 1.3533691856386836], ["ollas", 1.3533691856386836], ["canova's", 1.3533691856386836], ["picketts", 1.3533691856386836], ["atenas", 1.3533691856386836], ["whittington's", 1.3533691856386836], ["yongping", 1.3533691856386836], ["godiva's", 1.3533691856386836], ["leroux's", 1.3533691856386836], ["ntu's", 1.3533691856386836], ["damis", 1.3533691856386836], ["murni", 1.3533691856386836], ["dossie", 1.3533691856386836], ["jianwen", 1.3533691856386836], ["poison's", 1.3533691856386834], ["watchlists", 1.3533691856386834], ["muscatel", 1.3533691856386834], ["baldur's", 1.3533691856386834], ["jar's", 1.3533691856386834], ["jianhua", 1.3533691856386834], ["stroller", 1.3532752016674587], ["unpronounceable", 1.353110217176066], ["shinra", 1.353110217176066], ["bohemian", 1.3528695240195132], ["posthumously", 1.3527603825862147], ["amharic", 1.3526897834370577], ["jacek", 1.3526897834370577], ["infanta", 1.3526897834370577], ["baba's", 1.3526415677969426], ["stretford", 1.3526415677969426], ["volatility", 1.3526027144170436], ["recognize", 1.3525412180855363], ["bebop", 1.352535008379291], ["capers", 1.352535008379291], ["rationing", 1.3524846306153904], ["teddy's", 1.3522413779839848], ["homeowner's", 1.3522413779839848], ["macrame", 1.3522413779839848], ["zeitgeist", 1.35222031536225], ["huntington's", 1.3522094862508183], ["paleontological", 1.3521019860266623], ["cfcs", 1.3521019860266623], ["work", 1.3520876647257838], ["reits", 1.3519881966737461], ["leered", 1.3519232783890696], ["graphology", 1.3519232783890696], ["distantly", 1.3519232783890696], ["mcts", 1.3519232783890696], ["comrade", 1.3517171856937726], ["gull's", 1.3516138429854556], ["bobst", 1.3516138429854556], ["daula", 1.3516138429854556], ["chaton", 1.3516138429854556], ["flimsier", 1.3516138429854556], ["metal's", 1.3515353520538074], ["benders", 1.3515019544222948], ["tecumseh", 1.3514190859475614], ["anya's", 1.3513217284743286], ["mycelia", 1.3513217284743286], ["todd's", 1.3509920734859224], ["aramaic", 1.3509797626414402], ["entrepreneur's", 1.3508629464060193], ["anand", 1.3508629464060193], ["downtown", 1.3508574546467447], ["groupthink", 1.350734121024901], ["soul's", 1.3506646109271754], ["outerwear", 1.3506406187321538], ["gills", 1.350623219175069], ["stickier", 1.350584475380168], ["hilltops", 1.3505139763862812], ["sops", 1.3505139763862812], ["primarily", 1.3503106609062387], ["provencal", 1.3500521043013338], ["biomedicine", 1.3500521043013338], ["faneuil", 1.3500521043013338], ["dracula's", 1.3500521043013338], ["r\u00e9publique", 1.3500521043013338], ["rama", 1.3500111066668274], ["endings", 1.3499890128407388], ["crunch", 1.3499882305250348], ["proportions", 1.3499882305250348], ["palais", 1.3499601952466465], ["checklist", 1.3496898131481765], ["james's", 1.349574692594837], ["cube", 1.3494448927403364], ["deoxyribose", 1.349280456981165], ["microfibers", 1.349280456981165], ["technique's", 1.349280456981165], ["compa\u00f1\u00eda", 1.349280456981165], ["lupe's", 1.349280456981165], ["dracul", 1.349280456981165], ["placards", 1.349272448431444], ["pinwheels", 1.3492275191101284], ["venetian", 1.3491399069335628], ["oldenburg", 1.3490809436056077], ["decrease", 1.348972259849756], ["loafers", 1.3488646284379633], ["quagmire", 1.3487536027102007], ["elem", 1.3487422824399187], ["anthropogenic", 1.3487221176478483], ["buen", 1.3484656740965146], ["recalibration", 1.3484656740965146], ["flirtatiousness", 1.3484656740965144], ["komplett", 1.3484656740965144], ["smithfield's", 1.3484656740965144], ["miyu's", 1.3484656740965144], ["jemima's", 1.3484656740965144], ["fredricksen", 1.3484656740965144], ["entity's", 1.3484656740965144], ["carmenere", 1.3484656740965144], ["chillida", 1.3484656740965144], ["trumbo's", 1.3484656740965144], ["steeplechases", 1.3484656740965144], ["laniakea", 1.3484656740965144], ["wolven", 1.3484656740965144], ["killua's", 1.3484656740965144], ["enr\u00edquez", 1.3484656740965144], ["turpin's", 1.3484656740965144], ["francolin", 1.3484656740965144], ["azmuth", 1.3484656740965144], ["anora", 1.3484656740965144], ["morozov's", 1.3484656740965144], ["zadina", 1.3484656740965144], ["stma", 1.3484656740965144], ["clearpath", 1.3484656740965144], ["penjing", 1.3484656740965144], ["sassicaia", 1.3484656740965144], ["towns", 1.3484436583304067], ["poacher", 1.3483237303413462], ["character", 1.348240720753053], ["customary", 1.3481792743073258], ["glass", 1.3481236056854018], ["obstructive", 1.3480516714772743], ["deworming", 1.3480077910153274], ["edwin's", 1.3480077910153274], ["simplon", 1.3480077910153274], ["departmental", 1.3479726866457695], ["antidotes", 1.347799765121652], ["disliked", 1.3477520891483543], ["inconsiderate", 1.347730147365189], ["altars", 1.347701080157594], ["exhibition", 1.3475759172585635], ["kana", 1.3475680570063444], ["proteins", 1.347416867461106], ["revved", 1.3473940457462392], ["eels", 1.347394045746239], ["circling", 1.3472558357347084], ["nakshatra", 1.3471543622578468], ["saquon", 1.3471543622578468], ["sherry's", 1.3471543622578468], ["tranquilized", 1.3471543622578468], ["winglets", 1.3471543622578468], ["entails", 1.3471517844653433], ["chests", 1.347038910415535], ["refueling", 1.3470078733677615], ["legitimizing", 1.3469245704689756], ["talas", 1.3469245704689754], ["montini", 1.3469245704689754], ["durell", 1.3469245704689754], ["preheat", 1.3468352873872793], ["achondroplasia", 1.3467765730433663], ["elastica", 1.3467765730433663], ["lambdas", 1.3467765730433663], ["desplat", 1.3467765730433663], ["hohner", 1.3467765730433663], ["gruffudd", 1.3467765730433663], ["haverstock", 1.3467765730433663], ["teambuilding", 1.346691377156914], ["unflinchingly", 1.3466216184738182], ["guitarist's", 1.3465461500052738], ["intramuros", 1.3465461500052738], ["leatherwood", 1.3465461500052738], ["priam's", 1.3463750296405508], ["estudiante", 1.3463750296405508], ["telly's", 1.3463750296405508], ["horticulturally", 1.3463750296405508], ["bothrops", 1.3463750296405508], ["sarv", 1.3463750296405508], ["apartamento", 1.3463750296405508], ["tuula", 1.3463750296405508], ["orogenies", 1.3463750296405508], ["microenterprises", 1.3463750296405508], ["niyamas", 1.3463750296405508], ["stuccos", 1.3463750296405508], ["athas", 1.3463750296405508], ["pinnacle's", 1.3463750296405508], ["georgen", 1.3463750296405508], ["cha\u00eene", 1.3463750296405508], ["dynamax", 1.3463750296405508], ["labelle's", 1.3463750296405508], ["cnsa", 1.3463750296405508], ["bogdana", 1.3463750296405508], ["deindividuation", 1.3463750296405508], ["siva's", 1.3463750296405508], ["glip", 1.3463750296405508], ["marvolo", 1.3463750296405508], ["grinderman", 1.3463750296405508], ["multiwavelength", 1.3463750296405508], ["giru", 1.3463750296405508], ["closets", 1.3463081290179602], ["barking", 1.3462681744794682], ["edged", 1.3461258012648427], ["formal", 1.34608242838031], ["marvelously", 1.346038435883141], ["pileup", 1.346038435883141], ["tarot", 1.3458504679406909], ["patois", 1.345816902236682], ["puff's", 1.3457230320475047], ["servlets", 1.3457230320475047], ["ma\u00edz", 1.3457230320475047], ["potage", 1.3457230320475047], ["lepidopteran", 1.3457230320475047], ["miking", 1.3457230320475047], ["hinata's", 1.3457230320475047], ["elevator's", 1.3457230320475047], ["googlemaps", 1.3457230320475047], ["mool", 1.3457230320475047], ["twelver", 1.3457230320475047], ["overgrowth", 1.345532455711773], ["jonesy's", 1.3452163592191735], ["undigested", 1.3452163592191735], ["yeerks", 1.3452163592191735], ["boldrini", 1.3452163592191735], ["flimsiness", 1.3452163592191735], ["trilliums", 1.3452163592191735], ["artiodactyla", 1.3452163592191735], ["koichi's", 1.3452163592191735], ["necip", 1.3452163592191735], ["pienso", 1.3452163592191735], ["wovens", 1.3452163592191735], ["digitales", 1.3452163592191735], ["renard's", 1.3452163592191735], ["finisher's", 1.3452163592191735], ["tobor", 1.3452163592191735], ["vasistha", 1.3452163592191735], ["sekiro", 1.3452163592191735], ["chryse", 1.3452163592191735], ["ophelia", 1.345196666401735], ["tailors", 1.3451942484966353], ["tagore", 1.3451813548902842], ["orbiting", 1.3451615801934929], ["florets", 1.3449745008302454], ["deer's", 1.3449369788745795], ["basketry", 1.3449369788745795], ["caretaking", 1.3449369788745795], ["guru", 1.3447528034053562], ["carpaccio", 1.3446377715377889], ["corporation's", 1.34451205746042], ["thylacine", 1.3442908577680424], ["pathfinding", 1.3442908577680424], ["officials", 1.3442384171941693], ["peanuts", 1.344033255204016], ["springfield's", 1.343970788516193], ["politicizing", 1.343970788516193], ["airflow", 1.3439707885161927], ["mmm's", 1.3439707885161927], ["gyrating", 1.3439707885161927], ["manot", 1.3439707885161927], ["perfumer", 1.3439707885161927], ["flosser", 1.3439707885161927], ["istria", 1.3439707885161927], ["anaerobe", 1.3439707885161927], ["nccaa", 1.3439707885161927], ["seagle", 1.3439707885161927], ["shastri's", 1.3439707885161927], ["polydorus", 1.3439707885161927], ["chiriqu\u00ed", 1.3439707885161927], ["stopmotion", 1.3439707885161927], ["rickroll", 1.3439707885161927], ["saging", 1.3439707885161927], ["ba\u00eda", 1.3439707885161927], ["xingjian", 1.3439707885161927], ["cowpeas", 1.3439707885161927], ["nosara", 1.3439707885161927], ["mania's", 1.3439707885161927], ["janning", 1.3439707885161927], ["junagarh", 1.3439707885161927], ["samlesbury", 1.3439707885161927], ["kranks", 1.3439707885161927], ["alario", 1.3439707885161927], ["mopti", 1.3439707885161927], ["yotel", 1.3439707885161927], ["balmedie", 1.3439707885161927], ["coldspring", 1.3439707885161927], ["especies", 1.3439707885161927], ["jewelweed", 1.3439707885161927], ["marburger", 1.3439707885161927], ["dustiest", 1.3439707885161927], ["kamchatsky", 1.3439707885161927], ["chimurenga", 1.3439707885161927], ["sagredo", 1.3439707885161927], ["mor\u00f3n", 1.3439707885161927], ["iima", 1.3439707885161927], ["eleri", 1.3439707885161927], ["toy's", 1.3437103795279122], ["wwf's", 1.3437103795279122], ["leah", 1.3436203398099305], ["juniper", 1.3435308635445018], ["spinoffs", 1.3435193371260294], ["tiered", 1.3434664842803516], ["microsystems", 1.3432861370701836], ["rattlers", 1.343257806665521], ["replenish", 1.3432189167463935], ["ftas", 1.3429735309437099], ["redbrick", 1.342904990904364], ["crisped", 1.342904990904364], ["wouter", 1.342904990904364], ["r\u00e9mi", 1.342904990904364], ["pawing", 1.3428347185343532], ["narrator's", 1.3428347185343532], ["pumping", 1.3427545253591644], ["schoolchildren", 1.3426344711815024], ["tomoko", 1.342554591689516], ["gran", 1.342542232153574], ["rick's", 1.3422185449848807], ["mongols", 1.3420911090916945], ["champagnes", 1.3420911090916945], ["predawn", 1.3420911090916945], ["romanovs", 1.3419826660479734], ["watermill", 1.3419826660479734], ["portugu\u00eas", 1.3419826660479734], ["dell'arte", 1.3419826660479734], ["m\u00f6bius", 1.3419826660479734], ["chapultepec", 1.3419826660479734], ["subjecting", 1.3419276587069557], ["charlemagne", 1.3419276587069557], ["glittery", 1.341860944264205], ["distractors", 1.3418609442642049], ["khanum", 1.3418609442642049], ["georgie's", 1.3418609442642049], ["worry", 1.341682034349889], ["ancien", 1.3414975261155373], ["rural", 1.3414619721737777], ["hazelnut", 1.3413990878110336], ["affections", 1.3413199585585671], ["pooch's", 1.3411766704527497], ["masinissa", 1.3411766704527497], ["tempters", 1.3411766704527497], ["dunaliella", 1.3411766704527497], ["formaci\u00f3n", 1.3411766704527497], ["vbox", 1.3411766704527497], ["shoya", 1.3411766704527497], ["spis", 1.3411766704527497], ["kao's", 1.3411766704527497], ["esperar", 1.3411766704527497], ["soninke", 1.3411766704527497], ["ubba", 1.3411766704527497], ["gbas", 1.3411766704527497], ["ateeq", 1.3411766704527497], ["tonguing", 1.3411766704527495], ["huawei's", 1.3411766704527495], ["prea", 1.3411766704527495], ["baqarah", 1.3411766704527495], ["katas", 1.3411766704527495], ["yussuf", 1.3411766704527495], ["iggy's", 1.3411766704527495], ["teals", 1.3411766704527495], ["inventively", 1.3411766704527495], ["bacteria", 1.3409560252585782], ["directories", 1.340635873408212], ["jawad", 1.340635873408212], ["caved", 1.340614921061984], ["reportage", 1.340501581524669], ["recharging", 1.340501581524669], ["rover's", 1.3404058792628342], ["asthma", 1.3403899467075118], ["caves", 1.3403418564027738], ["chuba", 1.34033422279543], ["contender's", 1.34033422279543], ["staple", 1.3403016856729657], ["sours", 1.3402785610752141], ["libros", 1.3402785610752141], ["triviality", 1.3402785610752141], ["minha", 1.3402785610752141], ["centripetal", 1.3401418119107336], ["mightiness", 1.3401418119107336], ["kabocha", 1.3401418119107336], ["resignedly", 1.3401418119107336], ["ferrara's", 1.3401418119107336], ["hussites", 1.3401418119107336], ["brighton's", 1.3401418119107336], ["grieux", 1.3401418119107336], ["ssts", 1.3401418119107336], ["overcautious", 1.3401418119107336], ["syrup", 1.340051248290152], ["incandescent", 1.339835493782297], ["friar", 1.3398354937822967], ["entombed", 1.3398354937822967], ["olli", 1.3398354937822967], ["sangiovese", 1.3398354937822967], ["enlightens", 1.3398354937822967], ["ottavio", 1.3398354937822967], ["bandaged", 1.3397387617417378], ["romanov", 1.3396631898350513], ["tadeusz", 1.339475902935871], ["patch's", 1.3394527929843014], ["allures", 1.3394527929843014], ["lockean", 1.3394527929843014], ["unsafely", 1.3394527929843014], ["earthquake's", 1.3394527929843014], ["artichokes", 1.3394527929843012], ["couches", 1.3394456343461045], ["linz", 1.3393963474388741], ["budgies", 1.3393667154318958], ["thea", 1.3393158755958388], ["classism", 1.3391782283941278], ["punishing", 1.3391288294923274], ["chipping", 1.3389799155614905], ["login", 1.3389522269459306], ["multitrack", 1.3388669517321075], ["slurps", 1.3388669517321075], ["compulsions", 1.3388016700988228], ["unnatural", 1.3387674375265224], ["masseuse", 1.3387644821965397], ["soprano's", 1.3387644821965394], ["waistbands", 1.3387644821965394], ["rubisco", 1.3387644821965394], ["ebby", 1.3387644821965394], ["keto", 1.3386539810011837], ["marwari", 1.3385296112347507], ["sunkissed", 1.3385296112347507], ["yoki", 1.3385296112347507], ["paris's", 1.3381627902944286], ["unsanctioned", 1.3381019204440696], ["grenadiers", 1.3381019204440696], ["hopes", 1.3380561637772739], ["sphynx", 1.3380168301296933], ["nile", 1.3379618362140786], ["separated", 1.337912581731035], ["hoaxing", 1.3378894727310515], ["particuliers", 1.3378894727310515], ["fring's", 1.3378894727310515], ["hotsauce", 1.3378894727310515], ["bisons", 1.3378894727310515], ["kalamata", 1.3378894727310515], ["kamis", 1.3378894727310515], ["nioxin", 1.3378894727310515], ["threema", 1.3378894727310515], ["drizella", 1.3378894727310515], ["trabi", 1.3378894727310515], ["nurdles", 1.3378894727310515], ["wielder's", 1.3378894727310515], ["champignon", 1.3378894727310515], ["moshed", 1.3378894727310515], ["nappers", 1.3378894727310515], ["asperatus", 1.3378894727310515], ["yuzawa", 1.3378894727310515], ["braucht", 1.3378894727310515], ["greyson's", 1.3378894727310515], ["husin", 1.3378894727310515], ["pirarucu", 1.3378894727310515], ["mosquitofish", 1.3378894727310515], ["creux", 1.3378894727310515], ["wonderbolt", 1.3378894727310515], ["duanwu", 1.3378894727310515], ["shankaran", 1.3378894727310515], ["oppikoppi", 1.3378894727310515], ["dg's", 1.3378894727310515], ["gorgonia", 1.3378894727310515], ["bahs", 1.3378894727310515], ["im\u00e1genes", 1.3378894727310515], ["parvana", 1.3378894727310515], ["zabludowicz", 1.3378894727310515], ["vidigal", 1.3378894727310515], ["luhring", 1.3378894727310515], ["cerumen", 1.3378894727310515], ["kaushambi", 1.3378894727310515], ["akvavit", 1.3378894727310515], ["irreal", 1.3378894727310515], ["etoys", 1.3378894727310515], ["mmvas", 1.3378894727310515], ["kirsi", 1.3378894727310515], ["cgsc", 1.3378894727310515], ["viridula", 1.3378894727310515], ["transcellular", 1.3378894727310515], ["silla's", 1.3378894727310515], ["tibur\u00f3n", 1.3378894727310515], ["hsieh's", 1.3378894727310515], ["rupak", 1.3378894727310515], ["barracas", 1.3378894727310515], ["harkonnens", 1.3378894727310515], ["ruido", 1.3378894727310515], ["teotihuac\u00e1n", 1.3378894727310515], ["ftps", 1.3378894727310515], ["jhabua", 1.3378894727310515], ["gepa", 1.3378894727310515], ["prop", 1.337687106365366], ["maillard", 1.3376779487037425], ["kunsthalle", 1.3376779487037425], ["masaccio", 1.3376779487037425], ["micah's", 1.3376779487037425], ["comune", 1.3376779487037425], ["mendez", 1.3374605638091777], ["vespertine", 1.3374373302934945], ["pumpkin's", 1.3374373302934945], ["candleholders", 1.3374373302934945], ["txiki", 1.3374373302934945], ["nationalizations", 1.3374373302934945], ["sorrow's", 1.3374373302934945], ["lenzerheide", 1.3374373302934945], ["bacchic", 1.3374373302934945], ["pap\u00e0", 1.3374373302934945], ["eridani", 1.3374373302934945], ["candling", 1.3374373302934945], ["delacroix", 1.3373719139408269], ["recede", 1.3373204607277636], ["oceanic", 1.3373036806799115], ["patronizing", 1.3372843666827787], ["etch", 1.3372843666827787], ["parents", 1.3372241317373734], ["eaglets", 1.3371611714394178], ["grbs", 1.3371611714394178], ["approvals", 1.3370786306263662], ["twenty", 1.3369768672212814], ["orchard", 1.3369674667894333], ["euphoric", 1.3369618143909452], ["simulation", 1.336730461915514], ["alma", 1.3366339752786354], ["impermanent", 1.3366339752786354], ["rootsy", 1.3364652769968266], ["hikma", 1.3364652769968266], ["digger's", 1.3364652769968266], ["dinge", 1.3364652769968266], ["luta", 1.3364652769968266], ["ultimate", 1.3364321096207719], ["bordered", 1.3364131808990722], ["sangria", 1.3363128353052742], ["uyghur", 1.3362811908705186], ["bloodshot", 1.3362636196676718], ["unclos", 1.336018298643316], ["rodney's", 1.336018298643316], ["valmont", 1.336018298643316], ["jadranka", 1.336018298643316], ["galveston's", 1.336018298643316], ["songdo", 1.336018298643316], ["aumont", 1.336018298643316], ["phonebooks", 1.336018298643316], ["columnaris", 1.336018298643316], ["pushcarts", 1.336018298643316], ["shrewsbury's", 1.336018298643316], ["amaranthine", 1.336018298643316], ["williamsburg's", 1.336018298643316], ["valrico", 1.336018298643316], ["hallmarks", 1.3360182986433158], ["equip", 1.3359880993146849], ["temperament", 1.3359349537712857], ["intermittent", 1.3357845928184804], ["backed", 1.335677477648764], ["chinatown", 1.335561696353964], ["andrew's", 1.335520403625188], ["indicators", 1.3354677416521168], ["maypole", 1.3354569464169952], ["christiansborg", 1.3353555911539097], ["hsu's", 1.3353555911539097], ["indeterminism", 1.3353555911539097], ["kaju", 1.3353555911539097], ["bouck", 1.3353555911539097], ["actualization", 1.3353555911539094], ["yogyakarta", 1.3353242631635012], ["mandatory", 1.3349650355401528], ["ni\u00f1o", 1.3349596470596048], ["cornwallis", 1.3349508503382317], ["visitor's", 1.3349508503382317], ["waisted", 1.3349021597137893], ["pdmp", 1.3348103254980688], ["shammah", 1.3348103254980688], ["leverkusen's", 1.3348103254980688], ["tahiti's", 1.3348103254980688], ["sahrawis", 1.3348103254980688], ["jacko's", 1.3348103254980688], ["bretonne", 1.3348103254980688], ["kahneman's", 1.3348103254980688], ["bovidae", 1.3348103254980688], ["przed", 1.3348103254980688], ["diaghilev's", 1.3348103254980688], ["kurstin", 1.3348103254980688], ["js's", 1.3348103254980688], ["villaggio", 1.3348103254980688], ["agyeman", 1.3348103254980688], ["glamorgan's", 1.3348103254980688], ["altdorf", 1.3348103254980688], ["kigurumi", 1.3348103254980688], ["soneji", 1.3348103254980688], ["intervalometer", 1.3348103254980688], ["shank's", 1.3348103254980688], ["ey\u00fcp", 1.3348103254980688], ["umeed", 1.3348103254980688], ["broederbond", 1.3348103254980688], ["rtrs", 1.3348103254980688], ["athens", 1.3346763159772679], ["darrel", 1.3344975037672278], ["lolcat", 1.3344975037672278], ["rhonda's", 1.3344975037672278], ["anim\u00e9", 1.3344975037672278], ["morissette's", 1.3344975037672278], ["reenergized", 1.3344975037672278], ["nume", 1.3344975037672278], ["standover", 1.3344975037672278], ["unfair", 1.3344707871004855], ["verona", 1.3343217674704355], ["logarithm", 1.3342884519035965], ["englishes", 1.334271298579709], ["spear's", 1.334271298579709], ["exosuit", 1.334271298579709], ["horsefly", 1.334271298579709], ["ioniq", 1.334271298579709], ["methodology", 1.3342425232950004], ["encoding", 1.3341494635231899], ["oddest", 1.3341001101312653], ["fending", 1.3341001101312653], ["color", 1.3340674443010692], ["communism's", 1.3339660431922509], ["fleur's", 1.3339660431922509], ["rotini", 1.3339660431922509], ["baltra", 1.3339660431922509], ["ketchups", 1.3339660431922509], ["teletypes", 1.3339660431922509], ["mansart", 1.3339660431922509], ["uros", 1.3339660431922509], ["micrographia", 1.3339660431922509], ["precocial", 1.3339660431922509], ["pantera's", 1.3339660431922509], ["tokyu", 1.3339660431922509], ["poreless", 1.3339660431922509], ["talens", 1.3339660431922509], ["jianxin", 1.3339660431922509], ["hadia", 1.3339660431922509], ["celebration's", 1.3339660431922509], ["dorridge", 1.3339660431922509], ["soot", 1.3339279298767313], ["never", 1.33385839065585], ["sockeye", 1.333632551681453], ["travel", 1.333620145797251], ["elon", 1.333490931998785], ["times", 1.3334480013379566], ["daffy", 1.3333674686856902], ["elroy", 1.3333426945739366], ["baraka's", 1.3333426945739366], ["penh's", 1.3333426945739366], ["amsel", 1.3333426945739366], ["bandmembers", 1.3333426945739366], ["lekin", 1.3333426945739366], ["overstimulating", 1.3333426945739366], ["mitsotakis", 1.3333426945739366], ["sommet", 1.3333426945739366], ["junkie's", 1.3333426945739366], ["relex", 1.3333426945739366], ["khumalo", 1.333283358420914], ["parthenogenesis", 1.333283358420914], ["numeral", 1.3332639558010972], ["mortality", 1.333241938199032], ["saved", 1.333234927187809], ["reschedule", 1.333036114940987], ["inhaler", 1.333036114940987], ["contemporaries", 1.333014728763042], ["benjamin", 1.3329908680158484], ["krabs", 1.3329726642239161], ["ogham", 1.3328635919168854], ["dover's", 1.3328635919168854], ["govardhan", 1.3328635919168854], ["medievalists", 1.3328635919168854], ["biotope", 1.3328635919168854], ["masum", 1.3328635919168854], ["supination", 1.3328635919168854], ["tian's", 1.3328635919168854], ["voguing", 1.3328635919168854], ["noria", 1.3328635919168854], ["toula", 1.3328635919168854], ["miranda's", 1.3328254774304136], ["baritone", 1.332820648265112], ["picasso", 1.3327710319452244], ["fungicides", 1.3326845037621098], ["clarence's", 1.3325533711344073], ["paperboy", 1.3324838586998151], ["tails", 1.3322728253336686], ["playacting", 1.3321754847450127], ["sipper", 1.3321754847450127], ["sideways", 1.3321130990054537], ["facial", 1.3319587419082093], ["enshrined", 1.3319408401994044], ["buzzkill", 1.3318081569459104], ["empire", 1.3317555807287478], ["condemning", 1.3317336918546638], ["hsas", 1.33170508379701], ["cova", 1.33170508379701], ["pettiness", 1.3314484171389023], ["action's", 1.3313631826201684], ["culex", 1.3313631826201684], ["composer", 1.3313546350907475], ["articulated", 1.3312273292341867], ["becky's", 1.3312250170761024], ["hera", 1.3310064400739143], ["styling", 1.3309624110420166], ["knows", 1.3308544818858612], ["lightening", 1.3308130325447056], ["manageability", 1.3308130325447054], ["mineshaft", 1.3308130325447054], ["cloth", 1.3307919520277953], ["hyenas", 1.330676535400415], ["bruno", 1.3306559076392521], ["domestically", 1.330565162730486], ["antiquities", 1.330565162730486], ["rumbles", 1.3305404506342045], ["raves", 1.330512728731324], ["reinforces", 1.33042413335343], ["danke", 1.3302695107834048], ["sites", 1.330249128717356], ["instrumentation", 1.3301864727365393], ["quebeckers", 1.3301713914282467], ["anja", 1.3301713914282467], ["susceptibilities", 1.3301713914282467], ["frequentist", 1.3301713914282467], ["splotchy", 1.3301713914282467], ["kyrenia", 1.3301713914282467], ["riyaz", 1.3301713914282467], ["kavitha", 1.3300655576858407], ["abstraction", 1.3299107864209465], ["insular", 1.329864409283744], ["paranormal", 1.3297425710419406], ["horseless", 1.3296767633106037], ["rosi", 1.3296767633106037], ["minar", 1.3296767633106037], ["vasa", 1.3296563067450144], ["veronica's", 1.3296563067450144], ["bayreuth", 1.3296258665923908], ["rift", 1.3294452109039951], ["eeriest", 1.3292018787522786], ["highroller", 1.3292018787522786], ["mirages", 1.3292018787522786], ["beauchamp's", 1.3292018787522786], ["smallville's", 1.3292018787522786], ["shant\u00e9", 1.3292018787522786], ["yeti's", 1.3292018787522786], ["perrault's", 1.3292018787522786], ["florens", 1.3292018787522786], ["yolky", 1.3292018787522786], ["blast's", 1.3292018787522786], ["deodato", 1.3292018787522786], ["selecci\u00f3n", 1.3292018787522786], ["nordhavn", 1.3292018787522786], ["chantant", 1.3292018787522786], ["crpgs", 1.3292018787522786], ["wentworth's", 1.3292018787522786], ["nonlinearly", 1.3292018787522786], ["gravas", 1.3292018787522786], ["alveo", 1.3292018787522786], ["clayne", 1.3292018787522786], ["phad", 1.3292018787522786], ["heliophysics", 1.3292018787522786], ["greenstick", 1.3292018787522786], ["stinkhorn", 1.3292018787522786], ["hexy", 1.3292018787522786], ["l'endroit", 1.3292018787522786], ["budke", 1.3292018787522786], ["pi\u00e8ces", 1.3292018787522786], ["sensibles", 1.3292018787522786], ["agrario", 1.3292018787522786], ["rttt", 1.3292018787522786], ["quickpay", 1.3292018787522786], ["astaire's", 1.3292018787522786], ["niinist\u00f6", 1.3292018787522786], ["ilps", 1.3292018787522786], ["andropause", 1.3292018787522786], ["koes", 1.3292018787522786], ["tesson", 1.3292018787522786], ["raichand", 1.3292018787522786], ["pickaxe", 1.3292018787522784], ["westmorland", 1.3292018787522784], ["slingshots", 1.3292018787522784], ["moco", 1.3292018787522784], ["fleeing", 1.3291613233936592], ["dapper", 1.3290928831082645], ["cady", 1.3290514314541741], ["mammoths", 1.3289841552668156], ["batu", 1.3289014941469108], ["wild", 1.3288844993285087], ["noisy", 1.3288610577577267], ["groups", 1.3288581659432273], ["observatory", 1.3288564071788536], ["scopes", 1.328851627927838], ["parallelism", 1.328735492128155], ["electrocardiogram", 1.3286301790194819], ["paces", 1.3286031391672548], ["perce", 1.3285734145070054], ["directx", 1.3284176888533097], ["gait", 1.3283373572051225], ["movies", 1.3281936672786059], ["mesas", 1.3281593674748258], ["schr\u00f6der", 1.3281593674748258], ["cockatiel", 1.328085840231076], ["randomize", 1.328085840231076], ["beograd", 1.328085840231076], ["pharmacologist", 1.328085840231076], ["urination", 1.3280203659711656], ["goodhearted", 1.327953801870821], ["together's", 1.327953801870821], ["arcachon", 1.327953801870821], ["loren's", 1.327953801870821], ["yin's", 1.327953801870821], ["innsbruck", 1.3278659974671005], ["wastewater", 1.3278088287483458], ["handheld", 1.3278088287483458], ["miracles", 1.327753241342392], ["hiro", 1.3276472566718664], ["amboise", 1.3275669440920668], ["encroaches", 1.3275669440920668], ["brokenness", 1.3275455524236153], ["damper", 1.3274723393769408], ["established", 1.3273671114234213], ["idealization", 1.327141100645686], ["padm\u00e9", 1.3270717475363292], ["seamount", 1.3270717475363292], ["whistle", 1.3270536736957093], ["lexi's", 1.3270475807316104], ["childproof", 1.3270475807316104], ["stalagmite", 1.3270475807316104], ["preached", 1.3270054767873887], ["barrow's", 1.3269451692807128], ["inkblot", 1.3269451692807128], ["dressings", 1.3269144381827833], ["event's", 1.32689022331097], ["powders", 1.3268137146202414], ["gurgling", 1.3267924493648122], ["responsive", 1.326761028994889], ["puglia", 1.3266700656498933], ["kyla", 1.3266700656498933], ["roughneck", 1.3266407961342588], ["dunvegan", 1.3266407961342588], ["parries", 1.3266407961342588], ["lizard's", 1.3266407961342588], ["nereus", 1.3266407961342588], ["broca", 1.3266407961342588], ["rapinoe", 1.3265986631652196], ["survivalists", 1.3265986631652196], ["euphemia", 1.3265986631652196], ["odes", 1.3263928065781623], ["waveforms", 1.3263928065781623], ["lombards", 1.3261462422493997], ["chitin", 1.3261462422493997], ["armstrong's", 1.326056933458084], ["microbes", 1.32603292063075], ["authorship", 1.325933864676859], ["subsequent", 1.3259105217191776], ["embroidering", 1.3257268411607694], ["climax", 1.3256840580346427], ["soulless", 1.3255826202330587], ["kaizen", 1.3254149788128133], ["vientiane", 1.3254149788128133], ["ambas", 1.3254149788128131], ["branly", 1.3254149788128131], ["adages", 1.3254149788128131], ["wiseacre", 1.3254149788128131], ["keeffe", 1.3254149788128131], ["britannian", 1.3254149788128131], ["canadarm2", 1.3254149788128131], ["kittridge", 1.3254149788128131], ["corrib", 1.3254149788128131], ["talkeetna", 1.3254149788128131], ["celestia's", 1.3254149788128131], ["indecisively", 1.3254149788128131], ["pescadero", 1.3254149788128131], ["cornerman", 1.3254149788128131], ["freedom", 1.325184898488867], ["fermat", 1.325173994271211], ["klink", 1.325173994271211], ["maynooth", 1.325173994271211], ["severing", 1.3250460281984389], ["styria", 1.3250263754682399], ["anganwadi", 1.3249002545491968], ["proselytization", 1.3249002545491968], ["stylist's", 1.3249002545491968], ["novarum", 1.3249002545491968], ["bohemianism", 1.3249002545491968], ["buford's", 1.3249002545491968], ["bev's", 1.3249002545491968], ["salisbury's", 1.3249002545491968], ["sache", 1.3249002545491968], ["bordeaux's", 1.3249002545491968], ["saira", 1.3248259054888967], ["decca", 1.3248259054888967], ["riles", 1.3248259054888967], ["nota", 1.3247581359914549], ["nuits", 1.3246236862881489], ["mat\u00e9", 1.324471622956007], ["ou's", 1.324471622956007], ["ferrata", 1.324471622956007], ["tiene", 1.3244119620720902], ["vanlife", 1.3242235945621952], ["persson's", 1.3242235945621952], ["takatsuki", 1.3242235945621952], ["subtask", 1.3242235945621952], ["kahunas", 1.3242235945621952], ["bashu", 1.3242235945621952], ["purply", 1.3242235945621952], ["delvers", 1.3242235945621952], ["jeniffer", 1.3242235945621952], ["fingerspelling", 1.3242235945621952], ["nebuchadnezzar's", 1.3242235945621952], ["storica", 1.3242235945621952], ["lucanus", 1.3242235945621952], ["photoelectrons", 1.3242235945621952], ["artoria", 1.3242235945621952], ["lone", 1.3242098782564313], ["italian", 1.3241991933442832], ["beowulf", 1.3240895638340004], ["druze", 1.3239612978683089], ["taoists", 1.3239481163856688], ["datatype", 1.3239481163856688], ["tintagel", 1.3239481163856688], ["corrs", 1.3239481163856688], ["jaylen", 1.3238893212832445], ["rotator", 1.3238493879653566], ["mini", 1.3237304004731965], ["rounds", 1.323645814899094], ["arti", 1.3236017717042516], ["ironies", 1.3236017717042516], ["frankenstein", 1.3234875529183903], ["gogo", 1.3232943148467131], ["learnin", 1.323294314846713], ["altarpiece", 1.3232943148467127], ["sim\u00f3n", 1.3232943148467127], ["bosnia's", 1.3232943148467127], ["libras", 1.3232943148467127], ["millions", 1.3231145194235001], ["urdu", 1.323067100850345], ["tidal", 1.3230466924075572], ["rise", 1.3230318580601963], ["courtyard", 1.323016778287391], ["decried", 1.3229157981662647], ["guggenheim's", 1.3228176238111053], ["pollutions", 1.3228176238111053], ["consul's", 1.3228176238111053], ["intermarrying", 1.3228176238111053], ["etas", 1.3228176238111053], ["alison's", 1.3227960865655206], ["vaccines", 1.3227908876617167], ["frights", 1.3227373727950096], ["dodecahedron", 1.3227373727950096], ["generative", 1.3227326449337897], ["plena", 1.322624631488997], ["mint's", 1.322624631488997], ["springboks", 1.3226107950559862], ["madam", 1.3225501744061658], ["transactions", 1.3224971495968052], ["multivitamin", 1.3224327951104846], ["croatia's", 1.3224327951104846], ["kirkus", 1.3223854588612414], ["feels", 1.3223672588484339], ["ingested", 1.3223326183969697], ["asha", 1.322313612538279], ["rosemary", 1.3222938403143187], ["laborer", 1.3221219374737012], ["commodores", 1.3218496703981901], ["pixar's", 1.3217484102732937], ["seldom", 1.3216479445779485], ["yucatan", 1.3216318848029356], ["scissorhands", 1.3214963606145842], ["yelping", 1.3214963606145842], ["herta", 1.3213367788780048], ["verdi", 1.3212923567304546], ["dolphin's", 1.3212770979033488], ["scalpers", 1.321226667479765], ["remark", 1.3212130943023013], ["exact", 1.3211883777137101], ["inflammatory", 1.3211675918407093], ["cubed", 1.3210985831295998], ["peds", 1.320964571334659], ["dishonorable", 1.3208770235617633], ["volcanically", 1.3208770235617633], ["saponification", 1.3208770235617633], ["sorceresses", 1.3208770235617633], ["mahadev", 1.3208617885326566], ["impenetrability", 1.3207785461873083], ["louisianans", 1.3207785461873083], ["mudflat", 1.3207785461873083], ["chai's", 1.3207785461873083], ["pet\u00e9n", 1.3207785461873083], ["bandari", 1.3207785461873083], ["petrova", 1.3206263827603284], ["flocks", 1.3205889818432297], ["gail's", 1.3205854216740482], ["element", 1.3205374516907824], ["busking", 1.3204907251966138], ["switched", 1.3204175880753068], ["home", 1.320078259004129], ["missed", 1.3200154420991266], ["rube", 1.3198276006985366], ["embraces", 1.319786698777783], ["sendoff", 1.3197749150731843], ["ximo", 1.3197749150731841], ["fleck's", 1.3197749150731841], ["planetology", 1.3197749150731841], ["ketupat", 1.3197749150731841], ["chorusing", 1.3197749150731841], ["diks", 1.3197749150731841], ["stavo", 1.3197749150731841], ["creatin", 1.3197749150731841], ["skitty", 1.3197749150731841], ["oskari", 1.3197749150731841], ["unaccepting", 1.3197749150731841], ["terrier's", 1.3197749150731841], ["burundians", 1.3197749150731841], ["gul's", 1.3197749150731841], ["shack's", 1.3197749150731841], ["tadi", 1.3197749150731841], ["franjo", 1.3197749150731841], ["wvu's", 1.3197749150731841], ["lethargically", 1.3197749150731841], ["kuwa", 1.3197749150731841], ["tannu", 1.3197749150731841], ["pralidoxime", 1.3197749150731841], ["kriol", 1.3197749150731841], ["viktoras", 1.3197749150731841], ["zecora", 1.3197749150731841], ["varchar2", 1.3197749150731841], ["izettle", 1.3197749150731841], ["passado", 1.3197749150731841], ["climacteric", 1.3197749150731841], ["aamna", 1.3197749150731841], ["smugness", 1.3196455253756283], ["courtroom", 1.3195943142953324], ["miletus", 1.3195349559977165], ["stringently", 1.3195349559977165], ["jokinen", 1.3194854907497036], ["cutbacks", 1.3194697648038032], ["walkways", 1.3194697648038032], ["mingle", 1.3194653382412536], ["overstimulation", 1.319410336367363], ["latecomer", 1.319410336367363], ["smartthings", 1.319410336367363], ["groomed", 1.3193016164829512], ["sealant", 1.319193196102353], ["rinks", 1.319193196102353], ["cooperated", 1.319159020112817], ["cashless", 1.319106674609856], ["guanajuato", 1.319106674609856], ["unsinkable", 1.3190164237426827], ["firma", 1.3189899188411707], ["fundraise", 1.3189899188411707], ["billie", 1.3189867161655713], ["transition", 1.3189436401787598], ["repellent", 1.3189263801105655], ["polar", 1.3188053435895228], ["fela", 1.3186386850138498], ["font", 1.3182480756310235], ["more", 1.3181799123378308], ["curried", 1.318125196429343], ["nobility", 1.317967494511525], ["looking", 1.3177416811919251], ["reaffirm", 1.317666398936677], ["loving", 1.31754695606401], ["overwriting", 1.3175241361482533], ["octogenarians", 1.3174390302677452], ["spidey's", 1.3174390302677452], ["kavinsky", 1.3174390302677452], ["hailemariam", 1.3174390302677452], ["rooster's", 1.3172650420175778], ["redoubling", 1.3172650420175778], ["quinten", 1.3172650420175778], ["charlee", 1.3172650420175778], ["strongbox", 1.3172650420175778], ["bathtime", 1.3172650420175778], ["mitzvot", 1.3172650420175778], ["tinker", 1.3172321026713678], ["newts", 1.317167952277978], ["albino", 1.317017528197049], ["pedantry", 1.3169323229484113], ["pulsars", 1.3169323229484113], ["vocal", 1.3169059974646942], ["banding", 1.3168853709786823], ["vanderbilts", 1.3168256973858699], ["akureyri", 1.3168256973858699], ["marr's", 1.3168256973858699], ["piatti", 1.3167107610415054], ["coventry's", 1.3167107610415054], ["thankfulness", 1.3165428132403523], ["fishtank", 1.316542813240352], ["dobs", 1.316542813240352], ["tranquilly", 1.316542813240352], ["stratford's", 1.316542813240352], ["remeasured", 1.316542813240352], ["tradesperson", 1.316542813240352], ["procedure's", 1.316542813240352], ["froome's", 1.316542813240352], ["hirosaki", 1.316542813240352], ["charkha", 1.316542813240352], ["syst\u00e8me", 1.316542813240352], ["fanboy", 1.3163940580639488], ["consecutively", 1.3163940580639488], ["mixture", 1.3161739398082166], ["musicologists", 1.316102497918198], ["uofl", 1.316102497918198], ["aquitaine", 1.3160614799509252], ["shyam", 1.3160614799509252], ["narita", 1.3160614799509252], ["ardennes", 1.3160614799509252], ["holography", 1.3160296078817604], ["remixing", 1.3159512681229721], ["pompadour", 1.3159512681229721], ["brahma", 1.3159277979118376], ["civilizing", 1.3157755971487204], ["belligerence", 1.3157755971487204], ["cuny", 1.3157755971487202], ["madagascariensis", 1.31577559714872], ["bioleaching", 1.31577559714872], ["roie", 1.31577559714872], ["knuth's", 1.31577559714872], ["rubinstein's", 1.31577559714872], ["bioko", 1.31577559714872], ["sweatsuits", 1.31577559714872], ["formfitting", 1.31577559714872], ["baken", 1.31577559714872], ["calatrava's", 1.31577559714872], ["cska's", 1.31577559714872], ["covic", 1.31577559714872], ["sunpu", 1.31577559714872], ["destressing", 1.31577559714872], ["aiuto", 1.31577559714872], ["broma", 1.31577559714872], ["tabletalk", 1.31577559714872], ["nutrigenomics", 1.31577559714872], ["adesola", 1.31577559714872], ["aqualand", 1.31577559714872], ["swayambhu", 1.31577559714872], ["escapist", 1.315667258276991], ["america's", 1.315664029079653], ["epigenetic", 1.3156618495284782], ["decoys", 1.3153977721387708], ["firenze", 1.3153756653562738], ["persecution", 1.3152600279351436], ["viability", 1.315023725378921], ["youngest", 1.315023725378921], ["overestimation", 1.315023725378921], ["einsteins", 1.315023725378921], ["matadors", 1.315023725378921], ["birthdays", 1.3149415672054], ["rois", 1.314811658982311], ["bartholomew's", 1.314811658982311], ["penn's", 1.3147271703780836], ["huffily", 1.3145307763377942], ["selick", 1.3145307763377942], ["napata", 1.3145307763377942], ["suceava", 1.3145307763377942], ["unshaded", 1.3145307763377942], ["kirito", 1.3145307763377942], ["chudley", 1.3145307763377942], ["ittf", 1.3145307763377942], ["soapboxes", 1.3145307763377942], ["tischer", 1.3145307763377942], ["animoto", 1.3145307763377942], ["teshome", 1.3145307763377942], ["krones", 1.3145307763377942], ["lviv", 1.3145154768641403], ["entertain", 1.314363056989482], ["jackson's", 1.3142483576163155], ["wanes", 1.314230655155982], ["dimmers", 1.3141826484838912], ["kila", 1.3141826484838912], ["quelques", 1.3141047709936107], ["defeat", 1.3141047709936104], ["confuse", 1.3140164099950231], ["selfies", 1.3140029880711872], ["bilbo", 1.3139960189953492], ["morrie", 1.313956552544367], ["manual", 1.3139552760218378], ["rhone", 1.3138835784719824], ["klar", 1.3137850137944862], ["boaters", 1.3137794774058902], ["cabbages", 1.313743511284398], ["implying", 1.3137178428313747], ["macroeconomic", 1.3136945235001685], ["rife", 1.3136159654695094], ["keys", 1.3136059100415853], ["linting", 1.3135642095904871], ["tapasya", 1.3135642095904871], ["mapmyrun", 1.3135642095904871], ["cierto", 1.3135642095904871], ["click's", 1.3135642095904871], ["fusidic", 1.3135642095904871], ["filt", 1.3135642095904871], ["loiret", 1.3135642095904871], ["sickling", 1.3135642095904871], ["vasilievna", 1.3135642095904871], ["iceboxes", 1.3135642095904871], ["gedmatch", 1.3135642095904871], ["tempy", 1.3135642095904871], ["arriva", 1.313564209590487], ["rebook", 1.313564209590487], ["castanets", 1.313564209590487], ["probability", 1.313487758763474], ["tolerance", 1.313405566570005], ["snooze", 1.313334966272932], ["tusks", 1.313332907563794], ["organism", 1.313301571787941], ["excess", 1.3131858166083004], ["pathogens", 1.3130526970478116], ["lactobacillus", 1.3129560780119731], ["verbose", 1.3127681100695232], ["potentiality", 1.312622586501175], ["diocletian", 1.3125868552678752], ["mosque", 1.3125297589672413], ["vents", 1.312411960283829], ["deadpool's", 1.3123989275238732], ["corporate", 1.31218485027556], ["problem's", 1.3121608290246851], ["devika", 1.3121608290246851], ["wanda's", 1.3120359208067867], ["bilbao", 1.31202065587769], ["kfor", 1.3120206558776897], ["exclaim", 1.311908178060494], ["arielle", 1.3118183773076715], ["psychology", 1.311800628012796], ["deoxygenation", 1.3118004442872437], ["wilayat", 1.3118004442872437], ["hizo", 1.3118004442872437], ["adobes", 1.3118004442872437], ["paraiba", 1.3118004442872437], ["swar", 1.3118004442872437], ["pollutant", 1.3114791870355815], ["molasses", 1.3113019601184146], ["allergenic", 1.3111910131865296], ["rellik", 1.3111910131865294], ["declaration's", 1.3111910131865294], ["svensson's", 1.3111910131865294], ["form1", 1.3111910131865294], ["terakhir", 1.3111910131865294], ["recontextualize", 1.3111910131865294], ["wingsuits", 1.3111910131865294], ["radiuses", 1.3111910131865294], ["gambrinus", 1.3111910131865294], ["est\u00e9", 1.3111910131865294], ["keisei", 1.3111910131865294], ["keypoints", 1.3111910131865294], ["laberinto", 1.3111910131865294], ["orava", 1.3111910131865294], ["tayrona", 1.3111910131865294], ["catastrophize", 1.3111910131865294], ["pentathlete", 1.3111910131865294], ["knesset's", 1.3111910131865294], ["gynecologist's", 1.3111910131865294], ["summerisle", 1.3111910131865294], ["fanie", 1.3111910131865294], ["jamshidi", 1.3111910131865294], ["tomari", 1.3111910131865294], ["southampton's", 1.310976415639363], ["conversationally", 1.310931062549498], ["donny's", 1.310931062549498], ["souleymane", 1.310931062549498], ["submission", 1.3107823974467199], ["faceoff", 1.3106953824879144], ["godot", 1.3106953824879144], ["readmore", 1.3106953824879142], ["philosophy's", 1.3106953824879142], ["unsalvageable", 1.3106953824879142], ["aaliya", 1.3106953824879142], ["southpaws", 1.3106953824879142], ["mediterraneo", 1.3106953824879142], ["hourglass", 1.3106697371735208], ["yossi", 1.3105703310501098], ["nightshade", 1.3104807255304154], ["unloading", 1.3104606415346218], ["disappearance", 1.3102869332291855], ["stickiness", 1.3102843988299613], ["comets", 1.3102214476234288], ["suitcases", 1.310154125972651], ["weeknights", 1.3101541259726508], ["onegin", 1.3101541259726508], ["reims", 1.3101199245440354], ["lymphatic", 1.3100313423265018], ["outperform", 1.3099154201194023], ["trumps", 1.309556725561228], ["plagiarize", 1.3095099990670596], ["bestow", 1.3094378298744962], ["apprenticeships", 1.3093323661317549], ["cranked", 1.3092375817591615], ["goblet", 1.3092375817591615], ["nagano", 1.3091636594746061], ["arbeiten", 1.3091636594746061], ["hwange", 1.3091636594746061], ["seinem", 1.3091636594746061], ["m\u00e9tiers", 1.3091636594746061], ["armyworm", 1.3091636594746061], ["mandela", 1.3091345507773415], ["driveways", 1.3090141603699497], ["followership", 1.3090141603699497], ["piccola", 1.3090141603699497], ["autotrophs", 1.3090141603699497], ["barn's", 1.3090141603699497], ["clusters", 1.3089632308002257], ["centralization", 1.3089473739331963], ["substitutes", 1.3088504624268849], ["overhauls", 1.308840857524358], ["tent", 1.308820783278077], ["horticulture", 1.3087797041913165], ["retailed", 1.3087526190791665], ["igloo", 1.3087214226494595], ["sagen", 1.3087132178081202], ["amethysts", 1.3087132178081202], ["juelz", 1.3087132178081202], ["deprivation", 1.308708002512607], ["sanskrit", 1.3086991569623743], ["descartes", 1.3086031361868191], ["technicality", 1.3084904490833575], ["scrawny", 1.3083008485600724], ["culminate", 1.3082568794507274], ["huckleberry", 1.3082568794507274], ["clase", 1.3082568794507272], ["anjuna", 1.3082568794507272], ["lugosi's", 1.3082568794507272], ["demining", 1.3082568794507272], ["daten", 1.3082568794507272], ["lovett's", 1.3082568794507272], ["anabaptism", 1.3082568794507272], ["cottony", 1.3082568794507272], ["cort\u00e1zar", 1.3082568794507272], ["biohazards", 1.3082568794507272], ["aerith", 1.3082568794507272], ["sukie", 1.3082568794507272], ["sulo", 1.3082568794507272], ["swilly", 1.3082568794507272], ["nkosazana", 1.3082568794507272], ["thuan", 1.3082568794507272], ["alanya", 1.3082568794507272], ["analysis", 1.3082027779765169], ["gilbert's", 1.308169112318066], ["unprofessional", 1.308070977090063], ["chlo", 1.3078874849271824], ["codices", 1.307849238129752], ["makassar", 1.307849238129752], ["yearlong", 1.307786959594603], ["solid", 1.3077465307354728], ["samhsa", 1.3076030779117715], ["multinomial", 1.3076030779117715], ["excreting", 1.3076030779117715], ["discriminations", 1.3076030779117715], ["chronographs", 1.3075610598180085], ["shopfronts", 1.3075610598180085], ["propre", 1.3075610598180085], ["urbe", 1.3075610598180085], ["verifiability", 1.3075610598180085], ["seabright", 1.3075610598180085], ["polyphony", 1.3074828938053458], ["rulers", 1.3074464787407643], ["sized", 1.307420118932725], ["nobel's", 1.3074116840186347], ["bread's", 1.3074116840186347], ["chile's", 1.3074116840186347], ["distrust", 1.3073132953571547], ["what", 1.3071175587595536], ["cameroon's", 1.3070485141064072], ["imprudent", 1.3070485141064072], ["batching", 1.3070485141064072], ["interdenominational", 1.3070485141064072], ["apartheid", 1.307037627932134], ["pr\u00e9sident", 1.3069407992053739], ["calories", 1.3065598978394517], ["piecewise", 1.3063772000262293], ["grandfathers", 1.3062776708585175], ["temporal", 1.3062495402995071], ["sheaf", 1.3060466031425235], ["desecration", 1.3060012641413297], ["unilever", 1.305968574064382], ["xander's", 1.3058825475460984], ["kelvin's", 1.3058825475460984], ["janikowski", 1.3058825475460984], ["weiter", 1.3058825475460984], ["estudiantes", 1.3058825475460984], ["bombarding", 1.3058825475460982], ["deliveryman", 1.3058825475460982], ["conservations", 1.3058825475460982], ["superteam", 1.3058825475460982], ["avi's", 1.3058825475460982], ["jessicas", 1.3058825475460982], ["paratha", 1.3058825475460982], ["keyloggers", 1.3058825475460982], ["decembrist", 1.3058825475460982], ["replantation", 1.3058825475460982], ["harinder", 1.3058825475460982], ["fez's", 1.3058825475460982], ["xianxia", 1.3058825475460982], ["compris", 1.3058825475460982], ["velella", 1.3058825475460982], ["ramba", 1.3058825475460982], ["aryabhata", 1.3058825475460982], ["asala", 1.3058825475460982], ["larue", 1.305882547546098], ["sporophyte", 1.305882547546098], ["limassol", 1.305882547546098], ["villefort", 1.305882547546098], ["orinoco", 1.305882547546098], ["zebra's", 1.305882547546098], ["s\u00f3lo", 1.305882547546098], ["paint", 1.3058411222689104], ["statistics", 1.305797238635957], ["forecasting", 1.3057718795335942], ["tidy", 1.3057284611098094], ["camels", 1.305719312227655], ["removing", 1.3056852301282], ["reproductive", 1.3056652453789155], ["lunging", 1.3055974203523109], ["humps", 1.305522800287821], ["dialectical", 1.3053372938728014], ["senator's", 1.305314772525426], ["sekou", 1.305314772525426], ["mountaintop", 1.305274594777222], ["brandon's", 1.3052024003859182], ["interrogations", 1.3049301210527502], ["frenchwoman", 1.3049152271405084], ["arman", 1.3048420036835118], ["uncle", 1.3048174660356882], ["floats", 1.3047716405178038], ["margaret", 1.304734139070462], ["ferocity", 1.3046490312858214], ["vincent's", 1.3046490312858214], ["steakhouse", 1.3046490312858214], ["prato", 1.3046187902420212], ["splitting", 1.304572333713154], ["divination", 1.304536276878525], ["band", 1.3045115278282118], ["shading", 1.3044787238074862], ["timo", 1.304421828812825], ["budenholzer", 1.304421828812825], ["palearctic", 1.304421828812825], ["peacekeeper", 1.3043718865013791], ["patiala", 1.3043569838223295], ["rebranding", 1.3043340702169923], ["piping", 1.3042688143599672], ["serpentine", 1.3042359999861488], ["quip", 1.3042083391518082], ["tydfil", 1.3042083391518082], ["compares", 1.3041630580706816], ["infrequent", 1.3040781039031226], ["microbiology", 1.3040117607504564], ["upbeat", 1.303898684678882], ["resolution", 1.3038616187707186], ["jimbo", 1.303776285372637], ["chutney", 1.303745648831932], ["larches", 1.3036691872960202], ["hapu", 1.3036691872960202], ["plowshare", 1.3036691872960202], ["souffles", 1.3036691872960202], ["logisticians", 1.3036691872960202], ["regtech", 1.3036691872960202], ["animale", 1.3036691872960202], ["confucius", 1.3036024351614939], ["rules", 1.3035132223848331], ["swales", 1.3034030237216434], ["desk's", 1.3033419200216894], ["riverdale's", 1.3033419200216894], ["metafictional", 1.3033419200216894], ["warehouse's", 1.3033419200216894], ["bhandar", 1.3033419200216894], ["masak", 1.3033419200216894], ["measurers", 1.3033419200216894], ["culzean", 1.3033419200216894], ["perun", 1.3033419200216894], ["anshul", 1.3033419200216894], ["halk", 1.3033419200216894], ["union's", 1.3033308230279046], ["zipline", 1.3033148009638447], ["masterclasses", 1.3033148009638447], ["silky", 1.3032994831663367], ["parka", 1.3031572274468715], ["turboprops", 1.3030250365011395], ["dealership's", 1.3030250365011395], ["lenience", 1.3030250365011395], ["belichick's", 1.3030250365011395], ["track", 1.302995925267189], ["salento", 1.3029848791713659], ["backward", 1.3029805863293291], ["conversationalist", 1.3029010805425685], ["endedness", 1.3029010805425683], ["hario", 1.3029010805425683], ["psychoacoustics", 1.3029010805425683], ["derin", 1.3029010805425683], ["belacqua", 1.3029010805425683], ["lipi", 1.3029010805425683], ["pomfrey", 1.3029010805425683], ["buttermaker", 1.3029010805425683], ["portfolios", 1.3028887120113308], ["astrid", 1.302808310228436], ["bison", 1.3026178411772331], ["pomeranian", 1.3026178411772331], ["gegen", 1.3026178411772331], ["unclimbed", 1.302617841177233], ["tealights", 1.302617841177233], ["shagbark", 1.302617841177233], ["coroutine", 1.302617841177233], ["simonon", 1.302617841177233], ["burle", 1.302617841177233], ["bfi's", 1.302617841177233], ["o'lantern", 1.302617841177233], ["progressive's", 1.302617841177233], ["pitu", 1.302617841177233], ["lind's", 1.302617841177233], ["vaucouleurs", 1.302617841177233], ["traders", 1.3025482234207701], ["mainz", 1.3023970959067404], ["boudica", 1.3023970959067401], ["pockets", 1.3023076940721907], ["mewling", 1.3022751371937609], ["bumpy", 1.3022070503096272], ["haida", 1.3021779162055418], ["slobodan", 1.3019361991881293], ["starly", 1.3018520458561411], ["cfps", 1.3018520458561411], ["tomme", 1.3018520458561411], ["pagnol", 1.3018520458561411], ["nabal", 1.3018520458561411], ["durbar", 1.3018520458561411], ["quich\u00e9", 1.3018520458561411], ["doo's", 1.3018520458561411], ["whatsit", 1.3018520458561411], ["nostalgics", 1.3018520458561411], ["whiteread", 1.3018520458561411], ["essentiel", 1.3018520458561411], ["maurier's", 1.3018520458561411], ["mice's", 1.3018520458561411], ["mandolinist", 1.3018520458561411], ["rauschenberg's", 1.3018520458561411], ["oinking", 1.3018520458561411], ["glucophage", 1.3018520458561411], ["rug's", 1.3018520458561411], ["astp", 1.3018520458561411], ["iafrika", 1.3018520458561411], ["erden", 1.3018520458561411], ["egalit\u00e9", 1.3018520458561411], ["coap", 1.3018520458561411], ["popeil", 1.3018520458561411], ["favicon", 1.3018520458561411], ["crystal", 1.3017776020081944], ["hybrids", 1.3015575091941827], ["montreal's", 1.3015509960984637], ["tick", 1.3015365042024423], ["olof", 1.3015101729449394], ["awaken", 1.3015101729449392], ["truckloads", 1.3014015780202048], ["sleepily", 1.3014015780202048], ["mito", 1.3013519346260405], ["mediterranean", 1.3012845578188121], ["browse", 1.3011827131847498], ["indecisive", 1.301149689200702], ["acclimation", 1.30110492846971], ["anymore", 1.3009086493239579], ["stem", 1.3008810173889964], ["entrust", 1.3008599223632287], ["obnoxiously", 1.3006168921124446], ["coasted", 1.3006168921124446], ["undergarments", 1.3006168921124446], ["wola", 1.3006168921124446], ["alia's", 1.3006168921124446], ["bandwagon", 1.300572915209922], ["biochemist", 1.3005564404790673], ["hawkish", 1.3005425073888466], ["baker", 1.300501543284004], ["pangea", 1.300478895625218], ["outcry", 1.3003204552139576], ["bengali", 1.3003204552139576], ["otaku", 1.300310401008429], ["casings", 1.3002320942153698], ["saigon's", 1.300180003670351], ["visi\u00f3n", 1.300180003670351], ["deviled", 1.300180003670351], ["fairphone", 1.300180003670351], ["brouwerij", 1.300180003670351], ["addictively", 1.300180003670351], ["tyra's", 1.300180003670351], ["jude's", 1.300180003670351], ["lempicka", 1.300180003670351], ["fitzroy's", 1.300180003670351], ["guidant", 1.300180003670351], ["lamplighters", 1.300180003670351], ["backbend", 1.300180003670351], ["benigna", 1.300180003670351], ["villains", 1.2997037073043531], ["illness", 1.2996732538764433], ["breakups", 1.2996640592244502], ["daltrey's", 1.2996640592244502], ["monolids", 1.2996640592244502], ["ramen's", 1.2996640592244502], ["bhimrao", 1.2996640592244502], ["hilft", 1.2996640592244502], ["mandibularis", 1.2996640592244502], ["superflu", 1.2996640592244502], ["tatarian", 1.2996640592244502], ["eyzies", 1.2996640592244502], ["firewalking", 1.2996640592244502], ["postgraduation", 1.2996640592244502], ["onboarded", 1.2996640592244502], ["chichimeca", 1.2996640592244502], ["ozette", 1.2996640592244502], ["palmiers", 1.2996640592244502], ["sidespin", 1.2996640592244502], ["omps", 1.2996640592244502], ["playscripts", 1.2996640592244502], ["geles", 1.2996640592244502], ["nui's", 1.2996640592244502], ["arifi", 1.2996640592244502], ["ruhig", 1.2996640592244502], ["panaro", 1.2996640592244502], ["tarfon", 1.2996640592244502], ["axitinib", 1.2996640592244502], ["ritva", 1.2996640592244502], ["shevington", 1.2996640592244502], ["alcatel's", 1.2996640592244502], ["heightmap", 1.2996640592244502], ["sofian", 1.2996640592244502], ["sculpturing", 1.29966405922445], ["insignias", 1.29966405922445], ["composition", 1.2996612986805691], ["categories", 1.2992731174954053], ["diehards", 1.299264899501101], ["roland's", 1.299264899501101], ["weasels", 1.299141265958471], ["furniture", 1.299055125714184], ["procrustes", 1.2990454661453334], ["coven's", 1.2990454661453334], ["disorientate", 1.2990454661453334], ["internacionales", 1.2990454661453334], ["pogo's", 1.2990454661453334], ["winegrower", 1.2990454661453334], ["tianfu", 1.2990454661453334], ["tokaj", 1.2990454661453334], ["floreana", 1.2990454661453334], ["kikwit", 1.2990454661453334], ["antichi", 1.2990454661453334], ["rokko", 1.2990454661453334], ["wangfujing", 1.2990454661453334], ["yertle", 1.2990454661453334], ["bricky", 1.2990454661453334], ["vadi", 1.2990454661453334], ["bosatsu", 1.2990454661453334], ["cabled", 1.2990454661453332], ["bagheera", 1.2990454661453332], ["cowbells", 1.2990454661453332], ["compte", 1.2990454661453332], ["segovia", 1.2989066792365571], ["fema's", 1.298906679236557], ["unserved", 1.298906679236557], ["decors", 1.298906679236557], ["snakebites", 1.298906679236557], ["festering", 1.298820121523655], ["heist", 1.298820121523655], ["uhuru", 1.298695606125177], ["dobre", 1.298687602380555], ["miren", 1.298687602380555], ["meltingly", 1.298687602380555], ["bulrushes", 1.298687602380555], ["bajra", 1.298687602380555], ["ministro", 1.298687602380555], ["dbas", 1.298687602380555], ["ratzenberger", 1.298687602380555], ["weiskopf", 1.298687602380555], ["forget", 1.2985724441549433], ["tendril", 1.2985598416720079], ["grottoes", 1.2985598416720079], ["accentuated", 1.2983439332120332], ["inflaming", 1.2982902071533882], ["naive", 1.2982029357673794], ["emulation", 1.2981066105584371], ["hospitalizations", 1.2979935398681206], ["disparaging", 1.2977960548274332], ["amit", 1.29775278854912], ["insecurities", 1.297746420679188], ["pasig", 1.297626968222531], ["rukmini", 1.2976043697486586], ["beirut's", 1.2976043697486586], ["porcelains", 1.2976043697486586], ["ohchr", 1.2976043697486586], ["cornets", 1.2976043697486586], ["shells", 1.29744264859016], ["ovens", 1.297386444224714], ["sorbet", 1.297365922236015], ["aquatics", 1.297281977004464], ["kanti", 1.297249910513041], ["pancake", 1.297079500015765], ["habilis", 1.2970481637681113], ["durham's", 1.2969788029037386], ["isthmian", 1.2969788029037386], ["antivirals", 1.2969788029037383], ["csos", 1.2969788029037383], ["imprudence", 1.2968367616374037], ["sangha", 1.2967963097557291], ["defensive", 1.296776737365605], ["rose's", 1.2966845922112082], ["seatbelts", 1.2966845922112082], ["sounding", 1.2966805162444353], ["sundresses", 1.2966602131707727], ["massasoit", 1.2966602131707727], ["coffee's", 1.296614993982868], ["dishwashers", 1.2965872030236347], ["sentinel", 1.2965838508067622], ["survive", 1.2965835591335118], ["racer's", 1.2961371554002319], ["blur's", 1.2961371554002319], ["horribly", 1.2959933399044867], ["thronged", 1.2959718317834716], ["revolutionaries", 1.2957256832874064], ["wyoming's", 1.2957256832874062], ["extends", 1.2956974656501912], ["htaccess", 1.2955511982775374], ["conseils", 1.2955511982775374], ["susanna's", 1.2955511982775374], ["crick's", 1.2955511982775374], ["maquiladoras", 1.2955511982775374], ["neogaf", 1.2955511982775374], ["bike", 1.2955183086252346], ["unfocused", 1.295481713096616], ["han's", 1.2952584183457232], ["angiotensin", 1.2952584183457232], ["mask", 1.2951986359193606], ["fence", 1.295189626562642], ["prosthetic", 1.2951444169593487], ["teams", 1.295131760999794], ["ottokar", 1.2951197792970919], ["fanno", 1.2951197792970919], ["westmoreland's", 1.2951197792970919], ["powermac", 1.2951197792970919], ["macbook's", 1.2951197792970919], ["shinbashi", 1.2951197792970919], ["kastel", 1.2951197792970919], ["safesearch", 1.2951197792970919], ["chisti", 1.2951197792970919], ["splendide", 1.2951197792970919], ["harilal", 1.2951197792970919], ["ghez", 1.2951197792970919], ["besa", 1.2950166564545649], ["laundromats", 1.2950166564545649], ["brith", 1.2950166564545649], ["trey", 1.2950082839531492], ["p\u00e9ter", 1.29497747408016], ["ldrs", 1.29497747408016], ["caryatids", 1.29497747408016], ["familias", 1.29497747408016], ["stifler", 1.29497747408016], ["presences", 1.2949568369889395], ["perfecto", 1.2949568369889395], ["samus", 1.2949568369889395], ["unladylike", 1.2949440983720653], ["blondie's", 1.2949440983720653], ["puskas", 1.2949440983720653], ["aurelie", 1.2949440983720653], ["interphase", 1.2948902702098515], ["calzones", 1.2948902702098515], ["sarcastic", 1.294825200347027], ["airport's", 1.2947231875943406], ["bushfires", 1.2947231875943406], ["diced", 1.2947231875943406], ["clearview", 1.294527047132654], ["bridget's", 1.2945270471326538], ["mythbusting", 1.2945270471326538], ["emals", 1.2945270471326538], ["cyanobacterial", 1.2945270471326538], ["alarmists", 1.2945270471326538], ["eyjafjallaj\u00f6kull", 1.2945270471326538], ["passmark", 1.2945270471326538], ["interlaken", 1.2945270471326538], ["holocaust's", 1.2945270471326538], ["pandawa", 1.2945270471326538], ["successo", 1.2945270471326538], ["plateau's", 1.2945270471326538], ["elber", 1.2945270471326538], ["firekeepers", 1.2945270471326538], ["sarthak", 1.2945270471326538], ["polymathic", 1.2945270471326538], ["osmand", 1.2945270471326538], ["bangarra", 1.2945270471326538], ["netero", 1.2945270471326538], ["particip", 1.2945270471326538], ["saramago's", 1.2945270471326538], ["gaffigan's", 1.2945270471326538], ["dxing", 1.2945270471326538], ["boutonni\u00e8re", 1.2945270471326538], ["taieb", 1.2945270471326538], ["uwaterloo", 1.2945270471326538], ["gombo", 1.2945270471326538], ["kilmeny", 1.2945270471326538], ["hallicrafters", 1.2945270471326538], ["amants", 1.2945270471326538], ["ummagumma", 1.2945270471326538], ["populo", 1.2945270471326538], ["fruitport", 1.2945270471326538], ["affin", 1.2945270471326538], ["malique", 1.2945270471326538], ["transloading", 1.2945270471326538], ["preakness", 1.2941940926452227], ["krasnoyarsk", 1.2941940926452227], ["donatello", 1.2941389994566548], ["ethic", 1.2938586433979433], ["mammalogy", 1.293723492168574], ["seatback", 1.293723492168574], ["roatan", 1.293723492168574], ["digimon", 1.2936722352948749], ["hires", 1.2935804420899106], ["currencies", 1.2935439930201231], ["premonitions", 1.2933794167717205], ["synchronously", 1.2933794167717205], ["fc's", 1.2933794167717205], ["breadwinners", 1.2933794167717205], ["transcontinental", 1.2932449745053296], ["cerberus", 1.2932449745053296], ["ceo's", 1.2931398207205729], ["laila's", 1.29300764919001], ["craftsman's", 1.29300764919001], ["subarus", 1.29300764919001], ["lamp's", 1.29300764919001], ["stuffings", 1.29300764919001], ["notw", 1.29300764919001], ["serpent's", 1.2929023896939833], ["stigmatize", 1.2929023896939833], ["believers", 1.2927094216288388], ["fist", 1.2927059233872245], ["intelligent", 1.2926295138968997], ["moulin", 1.2925749825377715], ["misplace", 1.2925502388146066], ["tremble", 1.2924675722849428], ["riveter", 1.2922796043424931], ["olmec", 1.2922796043424931], ["aristophanes", 1.2922796043424931], ["casy", 1.292279604342493], ["gong's", 1.292279604342493], ["shrimp's", 1.292279604342493], ["bakan", 1.292279604342493], ["rinus", 1.292279604342493], ["nas's", 1.292279604342493], ["kwashiorkor", 1.292279604342493], ["innkeeper's", 1.292279604342493], ["vegie", 1.292279604342493], ["muddler", 1.292279604342493], ["civica", 1.292279604342493], ["liliya", 1.292279604342493], ["superheroines", 1.292279604342493], ["azaan", 1.292279604342493], ["llave", 1.292279604342493], ["wilt", 1.292155047272195], ["biotechnology", 1.292004442145371], ["toast", 1.2918715160463847], ["sobriety", 1.2918127162273756], ["paralleled", 1.2916881720292928], ["bared", 1.2916881720292928], ["marc's", 1.291671472763979], ["trumpet", 1.2915178395230913], ["drives", 1.2915110024416903], ["concert", 1.291504964624735], ["mestizo", 1.2914843553552053], ["feel", 1.2914419420758834], ["bivvy", 1.2913188016626844], ["ciudadanos", 1.2913188016626844], ["winegrowing", 1.2913188016626844], ["thelin", 1.2913188016626844], ["enfield's", 1.2913188016626844], ["stationarity", 1.2913188016626844], ["lanczos", 1.2913188016626844], ["aerosystems", 1.2913188016626844], ["cobbler", 1.2912629849865807], ["fruitfully", 1.2912580394378745], ["constrictive", 1.2912580394378745], ["nanak", 1.2912246822164992], ["electrician's", 1.291224682216499], ["waxen", 1.291224682216499], ["assemblymember", 1.291224682216499], ["colaba", 1.291224682216499], ["abysmally", 1.2910188437528907], ["roiled", 1.2907856394819817], ["heroku", 1.2907856394819817], ["enchant", 1.2906121467885028], ["downs", 1.2904452050921456], ["nakamura", 1.2904334906220036], ["petting", 1.2904334906220036], ["loch", 1.2902999419698833], ["caper", 1.2902804977722395], ["detached", 1.2902521705385974], ["densities", 1.290211956975545], ["miriam", 1.290211956975545], ["hautes", 1.290211956975545], ["santeria", 1.290211956975545], ["paradox", 1.2899795282317197], ["radish", 1.2899650737078499], ["indigestible", 1.2898181193818408], ["oakridge", 1.2898181193818408], ["neva", 1.2897975641838395], ["corrupts", 1.2897975641838395], ["messier", 1.289740100841091], ["hundreds", 1.2896328125042131], ["edinburgh's", 1.289559015701772], ["kickoff", 1.289519995306925], ["pursuers", 1.2895130339219287], ["treatises", 1.2893861492525884], ["pilar", 1.2893861492525884], ["radix", 1.289375605231611], ["businesswomen", 1.289375605231611], ["gurkha", 1.289375605231611], ["miro", 1.2893584809125298], ["pocketed", 1.2891523796771802], ["costumed", 1.2891030511251147], ["hassle", 1.2889446892345124], ["religious", 1.2889152967623743], ["unchangeable", 1.2888603431325596], ["shoring", 1.2887666112433096], ["lumi", 1.288673242655956], ["proprietor", 1.2886308148714083], ["feeds", 1.288549312701092], ["blanket", 1.2885154258026648], ["gnaw", 1.288434805519656], ["wiggled", 1.2883033594060547], ["guayaquil", 1.2883033594060544], ["halen", 1.2883033594060544], ["blurs", 1.288188935091337], ["proceeding", 1.2880945412912306], ["subscriptions", 1.2879636413920676], ["insularity", 1.2878489314133188], ["dollhouse", 1.287774033614199], ["shrine", 1.287716918812077], ["unscented", 1.287699372833431], ["retreats", 1.2876452223131267], ["formas", 1.2876366836083284], ["teruel", 1.2876366836083284], ["benz's", 1.2876366836083284], ["attachments", 1.2876273977668602], ["aren't", 1.2875867832650092], ["pharaoh", 1.2874814752852215], ["mitigated", 1.2873540569500252], ["order's", 1.2872772316805219], ["destitute", 1.2872430274230042], ["ibm's", 1.2872001238167536], ["cow's", 1.2872001238167536], ["procured", 1.287110485925123], ["aimee's", 1.287065796140535], ["pageant's", 1.287065796140535], ["riordan's", 1.287065796140535], ["supermarket's", 1.287065796140535], ["ghirlandaio", 1.287065796140535], ["everlong", 1.287065796140535], ["jensens", 1.287065796140535], ["selima", 1.287065796140535], ["tagline", 1.2869756393577134], ["maddeningly", 1.286965557371365], ["ambitiously", 1.286965557371365], ["necklines", 1.286965557371365], ["vitamin", 1.2869280464515687], ["racial", 1.2869243976620977], ["devoting", 1.286838427061051], ["nauruan", 1.2868340176791373], ["overpricing", 1.2868340176791373], ["loitering", 1.2867928306033853], ["olivers", 1.2866783924351264], ["corkscrews", 1.2866783924351264], ["oscorp", 1.2866783924351264], ["cruised", 1.2866783924351262], ["birthright", 1.2866674186322058], ["niko", 1.2866674186322058], ["attribution", 1.2866364619654513], ["infertility", 1.2865969973406162], ["abel's", 1.2865361394343042], ["pumbaa", 1.2865361394343042], ["seismograms", 1.286536139434304], ["shrinkwrapped", 1.286536139434304], ["biguanide", 1.286536139434304], ["capturer", 1.286536139434304], ["lembah", 1.286536139434304], ["bedsore", 1.286536139434304], ["vasarely", 1.286536139434304], ["danxia", 1.286536139434304], ["ezeiza", 1.286536139434304], ["miscasting", 1.286536139434304], ["daydreamers", 1.286536139434304], ["preprocessors", 1.286536139434304], ["ml's", 1.286536139434304], ["perpendicularity", 1.286536139434304], ["turbinado", 1.286536139434304], ["lusitanians", 1.286536139434304], ["iyengar's", 1.286536139434304], ["oilskins", 1.286536139434304], ["conques", 1.286536139434304], ["kinnersley", 1.286536139434304], ["hoban's", 1.286536139434304], ["sa\u0161a", 1.286536139434304], ["securedrop", 1.286536139434304], ["slewfoot", 1.286536139434304], ["afsana", 1.286536139434304], ["ciancia", 1.286536139434304], ["jagd", 1.286536139434304], ["mandans", 1.286536139434304], ["pierrefonds", 1.286536139434304], ["misko", 1.286536139434304], ["avramov", 1.286536139434304], ["wfirst", 1.286536139434304], ["arli", 1.286536139434304], ["blinds", 1.2864901917150389], ["austro", 1.2864681694353908], ["kolkata", 1.286394761836564], ["vests", 1.2862100588459684], ["goan", 1.2861610560117287], ["albin", 1.2861610560117287], ["gymnastics", 1.2860974426171963], ["statecraft", 1.28605501148388], ["orvieto", 1.28605501148388], ["maui's", 1.28605501148388], ["quebec's", 1.2860184186538477], ["pecked", 1.285906718622448], ["mountaineers", 1.2858503525795952], ["retrievers", 1.2857007263567493], ["fragen", 1.2857007263567493], ["vectors", 1.2853874464526664], ["lassos", 1.2850716702943674], ["patwardhan", 1.2850716702943674], ["r\u00e9serve", 1.2850716702943674], ["nami's", 1.2850716702943674], ["tunicate", 1.2850716702943674], ["socialmedia", 1.2850716702943674], ["ubuntu's", 1.2850716702943674], ["potholders", 1.2850716702943674], ["fehler", 1.2850716702943674], ["peterborough's", 1.2850716702943674], ["piki", 1.2850716702943674], ["avory", 1.2850716702943674], ["viracocha", 1.2850716702943674], ["millie's", 1.2848951494605358], ["fianc\u00e9's", 1.2848951494605358], ["prime's", 1.2848951494605358], ["chillingly", 1.2848951494605358], ["crypto", 1.2848506200843632], ["levers", 1.284826456856983], ["youngers", 1.2848199258114836], ["horseshoes", 1.2847517619871], ["merak", 1.2847517619871], ["kabyle", 1.2847517619871], ["songcraft", 1.2847517619871], ["gulistan", 1.2847517619871], ["dysgraphia", 1.2847517619871], ["lieben", 1.2847517619871], ["ercot", 1.2847517619871], ["songhai", 1.2846329794647269], ["cryonics", 1.2846329794647269], ["placements", 1.2846092995941374], ["akan", 1.2845809173378995], ["asterix", 1.2845809173378995], ["story", 1.284554521839598], ["instructed", 1.284510987067833], ["lookalike", 1.2844826100816555], ["inadvertently", 1.2843473571711108], ["cygnus", 1.2841688096360813], ["shelving", 1.28411783842033], ["deviation", 1.2840831536249102], ["murcia", 1.284033150836183], ["fraudsters", 1.2840225880521647], ["apnea", 1.2840090148747012], ["acrylics", 1.2839423165725414], ["ken's", 1.2839423165725414], ["livestream", 1.2838300838525611], ["analog", 1.2837895439251366], ["coincided", 1.2837874812282424], ["poland's", 1.2837532234478621], ["yamamoto", 1.2837532234478621], ["spanglish", 1.2837393217398818], ["bills", 1.283652954754109], ["unleash", 1.2835504316065454], ["college", 1.2834554189667764], ["emission", 1.2834271021426857], ["calcu", 1.2833673312090965], ["apothecary's", 1.2833673312090965], ["unflatteringly", 1.2833673312090965], ["tampopo", 1.2833673312090965], ["trancelike", 1.2833673312090965], ["deodorizers", 1.2833673312090965], ["shushi", 1.2833673312090965], ["teargassed", 1.2833673312090965], ["koca", 1.2833673312090965], ["siuc", 1.2833673312090965], ["dixwell", 1.2833673312090965], ["mordants", 1.2833673312090965], ["masurian", 1.2833673312090965], ["terno", 1.2833673312090965], ["entablatures", 1.2833673312090965], ["sublett", 1.2833673312090965], ["baudot", 1.2833673312090965], ["flexibles", 1.2833673312090963], ["asmodee", 1.2833673312090963], ["artesanal", 1.2833673312090963], ["chiado", 1.2833673312090963], ["sadiq's", 1.2833673312090963], ["stereo's", 1.2833673312090963], ["antik", 1.2833673312090963], ["sakamoto's", 1.2833673312090963], ["digivolve", 1.2833673312090963], ["lightsticks", 1.2833673312090963], ["shikoh", 1.2833673312090963], ["moosonee", 1.2833673312090963], ["grebel", 1.2833673312090963], ["mpaa's", 1.2833673312090963], ["hlas", 1.2833673312090963], ["amiran", 1.2833673312090963], ["nstar", 1.2833673312090963], ["sterculia", 1.2833673312090963], ["powassan", 1.2833673312090963], ["beegees", 1.2833673312090963], ["waterbus", 1.2833673312090963], ["water", 1.2833456186500185], ["determinants", 1.283314712622331], ["illumination", 1.2832336471120955], ["redrawing", 1.2831576306647814], ["seeps", 1.2830016994908033], ["mist", 1.2830016994908033], ["entrapped", 1.2830016994908033], ["waistcoats", 1.2830016994908033], ["buna", 1.2830016994908033], ["husband", 1.2829545147175576], ["cohen's", 1.282774676270251], ["rainbow", 1.2826835998958883], ["wallet", 1.2826452474624586], ["distributive", 1.2825238033857667], ["regolith", 1.2825001911204417], ["draper's", 1.2825001911204417], ["sliver", 1.2824916114334506], ["conceptualizing", 1.2824060073654941], ["halsey's", 1.2823389920013968], ["sidney's", 1.2823389920013968], ["office's", 1.2821723895599262], ["cassette", 1.2821061202859958], ["analyses", 1.2820083901128405], ["riverbend", 1.2819413675077533], ["hermit", 1.2819413675077531], ["veracity", 1.2818938899562733], ["gironde", 1.281893889956273], ["demetrius", 1.2818758319335233], ["promotes", 1.281807322297578], ["l\u00e1szl\u00f3", 1.2817887736061333], ["karachi's", 1.2817887736061333], ["smothering", 1.2816995374063558], ["stu's", 1.2816995374063556], ["regrowing", 1.2816995374063556], ["kippah", 1.2816995374063556], ["elda", 1.2816995374063556], ["gangly", 1.2816675069293495], ["lumberjacks", 1.2814869395150172], ["squishing", 1.2814869395150172], ["phoebe", 1.2814244556660161], ["encircled", 1.281406840573877], ["hardware", 1.2812797203503294], ["centerfield", 1.2812122223405675], ["seeker's", 1.2812122223405675], ["streep's", 1.2812122223405675], ["gras", 1.2811684152210436], ["rewriting", 1.2810423903916888], ["darabont", 1.2809667189172727], ["yolks", 1.280962599523399], ["leaderboard", 1.280932759036071], ["writhed", 1.280932759036071], ["deceivers", 1.280932759036071], ["unsustainability", 1.2808435016491968], ["derrida's", 1.2808435016491968], ["kozel", 1.2808435016491968], ["psychohistory", 1.2808435016491968], ["hanga", 1.2808435016491968], ["sixty", 1.2808237202823758], ["rates", 1.2806337644298411], ["undecided", 1.2806327299337354], ["uh's", 1.280607401464561], ["remedy's", 1.280607401464561], ["informa\u00e7\u00e3o", 1.280607401464561], ["espelette", 1.280607401464561], ["phonotactics", 1.280607401464561], ["baz's", 1.280607401464561], ["chupke", 1.280607401464561], ["moushumi", 1.280607401464561], ["ioannes", 1.280607401464561], ["catina", 1.280607401464561], ["kauder", 1.280607401464561], ["royksopp", 1.280607401464561], ["acetobacter", 1.280607401464561], ["chongo", 1.280607401464561], ["modou", 1.280607401464561], ["behaviorist", 1.2806074014645608], ["chip's", 1.2806074014645608], ["uae's", 1.2806074014645608], ["he'd", 1.2804406557091617], ["unfavorable", 1.2804097768655696], ["stanley's", 1.2803326716158483], ["bloodied", 1.2803225688574735], ["revoluci\u00f3n", 1.2803225688574733], ["demetrious", 1.2803225688574733], ["distracting", 1.28025035479129], ["umbrage", 1.2802419313157412], ["reinstalling", 1.2802419313157412], ["amami", 1.280214094523079], ["cryengine", 1.280214094523079], ["layman's", 1.280181667620989], ["odense", 1.2799721795392314], ["speedrun", 1.2799721795392314], ["cathedrale", 1.2799721795392311], ["descriptivism", 1.2799721795392311], ["shennong", 1.2799721795392311], ["stravaganza", 1.2799721795392311], ["taloned", 1.2799721795392311], ["katutura", 1.2799721795392311], ["fisherwomen", 1.2799721795392311], ["decimalization", 1.2799721795392311], ["reacher's", 1.2799721795392311], ["sapsuckers", 1.2799721795392311], ["deinococcus", 1.2799721795392311], ["estatal", 1.2799721795392311], ["levassor", 1.2799721795392311], ["mapplethorpe's", 1.2799721795392311], ["shiroi", 1.2799721795392311], ["kedua", 1.2799721795392311], ["cancro", 1.2799721795392311], ["liceu", 1.2799721795392311], ["sombat", 1.2799721795392311], ["tinashe's", 1.2799721795392311], ["duloc", 1.2799721795392311], ["ziziphus", 1.2799721795392311], ["harvey's", 1.2799097647066315], ["kiribati", 1.2798475473893245], ["newlyweds", 1.2798165125061465], ["derogatory", 1.2798024219557378], ["head", 1.2797358769830085], ["regions", 1.2795626581475474], ["snowboarding", 1.2793018176776068], ["mehreen", 1.27926383854336], ["driving", 1.2792435303891658], ["philos", 1.2791377778576543], ["limes", 1.279137777857654], ["almaty", 1.2790291881176856], ["stimulate", 1.279025454554365], ["tapirs", 1.2789571341946324], ["persuading", 1.2789155420927072], ["culturing", 1.2788781862233858], ["deck's", 1.2786766611388878], ["hooghly", 1.2786766611388878], ["marielle", 1.2786766611388878], ["wires", 1.2786121047982606], ["confidential", 1.2785560506601052], ["critic's", 1.2785103369425117], ["cataloged", 1.2784368804451294], ["koda", 1.2784368804451294], ["id\u00e9e", 1.2783580910404428], ["crockett's", 1.2783580910404428], ["gdps", 1.2783580910404428], ["bankole", 1.2783580910404428], ["mezuzah", 1.2783580910404428], ["grails", 1.278182008658757], ["josh's", 1.278115116152547], ["pathway", 1.278079886856407], ["rearing", 1.2780029915707094], ["sating", 1.277993096153782], ["miffy", 1.277993096153782], ["samui", 1.2779555412582149], ["hanji", 1.2779436974797396], ["socon", 1.2779436974797396], ["viola's", 1.2779436974797396], ["tarantella", 1.2779436974797396], ["epee", 1.2779436974797396], ["johari", 1.2779436974797396], ["baring", 1.2778193695414903], ["trolls", 1.2777763224520307], ["explain", 1.2777101509887925], ["hashed", 1.277700039575552], ["typecast", 1.277700039575552], ["hysterically", 1.2776209103230858], ["vehicle's", 1.2776209103230858], ["longwell", 1.2774937965124873], ["thermae", 1.2774937965124873], ["neretva", 1.2774937965124873], ["troopship", 1.2774937965124873], ["sniffed", 1.2773855343263423], ["kiddos", 1.2773855343263423], ["strapping", 1.277358437776889], ["noaa's", 1.277346595581202], ["akiva", 1.277346595581202], ["wishers", 1.2771839487873418], ["shortsighted", 1.2771659657265957], ["selections", 1.277106784937854], ["emptied", 1.2770763148796402], ["luddites", 1.2770763148796402], ["greystone", 1.2770763148796402], ["partage", 1.27707631487964], ["otis's", 1.27707631487964], ["udaya", 1.27707631487964], ["nkrumah's", 1.27707631487964], ["institutionen", 1.27707631487964], ["bulb's", 1.27707631487964], ["chaudron", 1.27707631487964], ["dollman", 1.27707631487964], ["endoscopies", 1.27707631487964], ["vincula", 1.27707631487964], ["thalaivar", 1.27707631487964], ["martorell", 1.27707631487964], ["zheng's", 1.27707631487964], ["alisha's", 1.27707631487964], ["celyn", 1.27707631487964], ["dominated", 1.276984491038646], ["eternal", 1.276844376324114], ["poetry's", 1.2766681728806373], ["thiong'o", 1.2766681728806373], ["duero", 1.2766681728806373], ["partenkirchen", 1.2766681728806373], ["marpol", 1.2766681728806373], ["basin's", 1.2766681728806373], ["provisionals", 1.2766681728806373], ["juro", 1.2766681728806373], ["unicode", 1.2765723384333578], ["pangolins", 1.2765475048113673], ["blacklight", 1.2765475048113673], ["concentration", 1.2765388050328244], ["vinatieri", 1.2764896927781797], ["misclassification", 1.2764896927781797], ["peninsula's", 1.2764896927781797], ["throwing", 1.2763232977696712], ["schumann's", 1.2762482991146435], ["roll's", 1.2762482991146435], ["rashida", 1.2762375127023793], ["voyeuristic", 1.2762375127023793], ["bruises", 1.2760996872145172], ["bacterium", 1.2760338036021874], ["divinely", 1.2760338036021874], ["cary's", 1.2760338036021872], ["adventure's", 1.2760338036021872], ["reformulate", 1.2760338036021872], ["outraged", 1.275938520313388], ["undo", 1.275938520313388], ["shoichi", 1.2758786815740324], ["flik", 1.2758786815740324], ["oakenshield", 1.2758786815740324], ["tuya", 1.2758786815740324], ["leka", 1.2758786815740324], ["janne", 1.2757824179040782], ["brahmaputra", 1.2757824179040782], ["cornell's", 1.2757824179040782], ["numerus", 1.275782417904078], ["euclidian", 1.275782417904078], ["llena", 1.275782417904078], ["mahouts", 1.275782417904078], ["boty", 1.275782417904078], ["shatteringly", 1.275782417904078], ["inny", 1.275782417904078], ["remediating", 1.2756693266517156], ["undecipherable", 1.2756693266517156], ["dulhania", 1.2756693266517156], ["relearned", 1.2756693266517156], ["frogged", 1.2756693266517156], ["vishu", 1.2756693266517156], ["alster", 1.2756693266517156], ["cannellini", 1.2756693266517156], ["flip", 1.2756075616495073], ["fortifications", 1.2755793756094516], ["alley's", 1.2755635497589584], ["casteism", 1.2755635497589584], ["shoto", 1.2755635497589584], ["condition", 1.2754585504870737], ["eyck", 1.2754364094993773], ["conceptos", 1.275371273070722], ["unnoticeably", 1.275371273070722], ["alderwoman", 1.275371273070722], ["troyens", 1.275371273070722], ["dhl's", 1.275371273070722], ["vidisha", 1.275371273070722], ["pervak", 1.275371273070722], ["azin", 1.275371273070722], ["zip's", 1.275371273070722], ["marram", 1.275371273070722], ["bagoong", 1.275371273070722], ["mecanique", 1.275371273070722], ["argenteum", 1.275371273070722], ["rubygems", 1.275371273070722], ["ceros", 1.275371273070722], ["caminhos", 1.275371273070722], ["linocuts", 1.275371273070722], ["weeders", 1.275371273070722], ["mayfair's", 1.275371273070722], ["steampunks", 1.275371273070722], ["kontext", 1.275371273070722], ["kudumbashree", 1.275371273070722], ["lgbtq2", 1.275371273070722], ["parpola", 1.275371273070722], ["consadole", 1.275371273070722], ["brigantia", 1.275371273070722], ["occidental's", 1.275371273070722], ["kasteel", 1.275371273070722], ["chladni", 1.275371273070722], ["moken", 1.275371273070722], ["rokh", 1.275371273070722], ["medley's", 1.275371273070722], ["icimod", 1.275371273070722], ["graciella", 1.275371273070722], ["malthe", 1.275371273070722], ["sylvatic", 1.275371273070722], ["sopho", 1.275371273070722], ["fhir", 1.275371273070722], ["cadr", 1.275371273070722], ["sphingidae", 1.275371273070722], ["escena", 1.275371273070722], ["silly", 1.2753248959335197], ["farthest", 1.2751091414256641], ["collectivist", 1.2750492096179264], ["wrongs", 1.2750162327859178], ["constraint", 1.2750131250216448], ["skiing", 1.2750093705549794], ["bookseller", 1.2749624066218328], ["output", 1.2749467239987466], ["constructed", 1.2749431764196215], ["biopics", 1.2749130009639773], ["consequentialism", 1.2749130009639773], ["wakefield's", 1.2749130009639773], ["taming", 1.2748835327455748], ["anglicanism", 1.2748115490010359], ["jute", 1.2747699487633954], ["claustrophobia", 1.2747644097051238], ["stylistic", 1.2747495505792383], ["departure", 1.2747419688445245], ["shadowed", 1.2746508189884567], ["fixation", 1.2746045542701958], ["reachable", 1.274444634364433], ["sulfonylureas", 1.2743204315933225], ["nguesso", 1.2743204315933225], ["disaccharides", 1.2743204315933225], ["youtuber's", 1.2743204315933225], ["donne's", 1.2743204315933225], ["muteness", 1.2743204315933225], ["comaneci", 1.2743204315933225], ["knowingness", 1.2743204315933225], ["turing", 1.274204364457238], ["stage", 1.2741926128516194], ["courses", 1.2741876898816982], ["ready", 1.2740063647853819], ["spores", 1.2739888776348762], ["lungs", 1.2739633069278895], ["zygote", 1.2738184671376003], ["sangeetha", 1.2738184671376003], ["beyond's", 1.2738184671376003], ["adroitly", 1.2738184671376003], ["handsprings", 1.2738184671376003], ["orca's", 1.2738184671376003], ["converso", 1.2738184671376003], ["dispatcher's", 1.2738184671376003], ["documento", 1.2738184671376003], ["shindig", 1.273607883823476], ["robicheaux", 1.273607883823476], ["deconsecrated", 1.273607883823476], ["hoarfrost", 1.273607883823476], ["gurdeep", 1.273607883823476], ["bombcast", 1.273607883823476], ["crono", 1.273607883823476], ["porcaro", 1.273607883823476], ["operations", 1.2736038097041895], ["transatlantic", 1.27356608703404], ["storylines", 1.2734714867274843], ["wisnu", 1.2734516895772383], ["microlensing", 1.2734516895772383], ["kierra", 1.2734516895772383], ["fuddle", 1.2734516895772383], ["guadeloupean", 1.2734516895772383], ["gunfights", 1.273451689577238], ["rhineland", 1.273451689577238], ["napoleon", 1.2734461100747918], ["carnival", 1.2733166505903735], ["mansion", 1.2733163242056271], ["bpmn", 1.2732354838574458], ["literarily", 1.2732354838574458], ["mcafee's", 1.2732354838574458], ["garishly", 1.2732354838574458], ["trochaic", 1.2732354838574458], ["stethoscope", 1.273164046081322], ["hank", 1.2731625295251805], ["overtone", 1.2730645568224266], ["educaci\u00f3n", 1.2729515963471096], ["anglicized", 1.2729515963471096], ["fiumicino", 1.2728580611558429], ["nonthreatening", 1.2728580611558429], ["winterfest", 1.2728580611558429], ["coevolution", 1.2728580611558429], ["pythons", 1.2727592706107718], ["recall", 1.2726163602592067], ["germinate", 1.2725473724448044], ["packing", 1.2725200873313989], ["interludes", 1.2724801533350862], ["tested", 1.2724657914949848], ["mitigation", 1.2724646504092414], ["tuna's", 1.272398379660301], ["harrar", 1.272398379660301], ["rustica", 1.272398379660301], ["elsevier's", 1.272398379660301], ["mavis", 1.272398379660301], ["gesagt", 1.272398379660301], ["laurelhurst", 1.272398379660301], ["innodb", 1.272398379660301], ["dendrology", 1.272398379660301], ["talamanca", 1.272398379660301], ["perhentian", 1.272398379660301], ["goodway", 1.272398379660301], ["renshaw's", 1.2723983796603007], ["citygate", 1.2723983796603007], ["onomatopoeic", 1.2723983796603007], ["kapilavastu", 1.2723983796603007], ["themen", 1.2723983796603007], ["hildebrand's", 1.2723983796603007], ["ziauddin", 1.2723983796603007], ["elephanta", 1.2723983796603007], ["lkab", 1.2723983796603007], ["grazings", 1.2723983796603007], ["preventively", 1.2723983796603007], ["jintao's", 1.2723983796603007], ["zdena", 1.2723983796603007], ["quartz's", 1.2723983796603007], ["sonido", 1.2723983796603007], ["rythme", 1.2723983796603007], ["pollan's", 1.2723983796603007], ["azuchi", 1.2723983796603007], ["dovedale", 1.2723983796603007], ["porpoising", 1.2723983796603007], ["lightfoot's", 1.2723983796603007], ["pechersk", 1.2723983796603007], ["dragonzord", 1.2723983796603007], ["menageries", 1.2723983796603007], ["mellea", 1.2723983796603007], ["bandlimited", 1.2723983796603007], ["kubat", 1.2723983796603007], ["blissfulness", 1.2723983796603007], ["navman", 1.2723983796603007], ["parentheticals", 1.2723983796603007], ["botswanan", 1.2723983796603007], ["farberware", 1.2723983796603007], ["kindergarten", 1.272357134980247], ["eating", 1.2722689832149117], ["touristic", 1.2721670345003624], ["demand", 1.272123030444471], ["blackmore", 1.2719957219452183], ["herbert's", 1.2719957219452183], ["predation", 1.2718374987624657], ["zoos", 1.2718328692693408], ["desert", 1.271825546923641], ["upriver", 1.271816046305765], ["mikasa", 1.2713831681871675], ["extenuating", 1.2713468107514905], ["labor", 1.2713398479013303], ["film's", 1.2712766600361265], ["polarization", 1.2712623096784612], ["steelhead", 1.2711174416942], ["automotive", 1.2710564553997308], ["foosball", 1.270970177397417], ["pantanal", 1.2708466743192517], ["slade's", 1.2708466743192517], ["represent", 1.2708196011444854], ["capacities", 1.270813665314724], ["startling", 1.270738178587238], ["anaerobes", 1.270621286951278], ["celestino", 1.270621286951278], ["siri's", 1.270621286951278], ["kulu", 1.270621286951278], ["masseuses", 1.270621286951278], ["bounties", 1.2706177919126673], ["bottling", 1.2705336578970055], ["nonato", 1.2704821170403306], ["clergy's", 1.2704821170403306], ["caru", 1.2704821170403306], ["mizzle", 1.2704821170403306], ["actuelle", 1.2704821170403306], ["perspectivism", 1.2704821170403306], ["sevis", 1.2704821170403306], ["isaacson's", 1.2704821170403306], ["bunkbed", 1.2704821170403306], ["samuda", 1.2704821170403306], ["capco", 1.2704821170403306], ["cullins", 1.2704821170403306], ["ripleys", 1.2704821170403306], ["kennell", 1.2704821170403306], ["pathshala", 1.2704821170403306], ["krisflyer", 1.2704821170403306], ["fierceness", 1.2704821170403304], ["structuring", 1.270466465890136], ["definitions", 1.270430108339268], ["dragon's", 1.2701262396966218], ["hezbollah's", 1.2701262396966218], ["wampum", 1.2701262396966218], ["lab's", 1.2699707774885804], ["aires", 1.2699431884206747], ["flapjacks", 1.2697936746865848], ["dyeing", 1.2697444347438531], ["salvador's", 1.2694393136610909], ["abstractions", 1.2692853053328401], ["dordogne", 1.269107694195662], ["enfp", 1.2690779245985568], ["laxer", 1.2690779245985568], ["enameling", 1.2690779245985568], ["concept's", 1.2690779245985568], ["attributes", 1.2690682745260335], ["padding", 1.2685221991659714], ["wailing", 1.2684776172113477], ["plantes", 1.2682866340151], ["crandon", 1.2682866340151], ["detoxify", 1.2681570517280998], ["satyajit", 1.2680047388465567], ["farah", 1.2679669232345872], ["pushups", 1.2679292117978576], ["pi\u00f1ata", 1.2679292117978576], ["leisure", 1.267875401045102], ["foraged", 1.2678540997329426], ["oort", 1.2678540997329426], ["concluding", 1.267759809198427], ["ayodhya", 1.2676558038815668], ["worth", 1.2676332477284729], ["diplomatic", 1.2676120904065786], ["coco's", 1.2675577336507236], ["blackwell's", 1.2674742373241543], ["erika's", 1.2674742373241543], ["waybill", 1.2674742373241543], ["proctor's", 1.2674742373241543], ["guestroom", 1.2674742373241543], ["brie", 1.2674678359391172], ["dissolving", 1.2674678359391172], ["computers", 1.2674291883247817], ["underpriced", 1.2674280734897527], ["shtetl", 1.2674280734897527], ["manure", 1.2672846266787545], ["brioche", 1.2672677410326383], ["condesa", 1.2672677410326383], ["ranjith", 1.2672677410326383], ["deportiva", 1.2672677410326383], ["cheesehead", 1.2672677410326383], ["guru's", 1.2672677410326383], ["pronounceable", 1.2672677410326383], ["dungog", 1.2672677410326383], ["powderhorn", 1.2672677410326383], ["ayesha's", 1.2672677410326383], ["blockbuster's", 1.2672677410326383], ["aoc's", 1.2672677410326383], ["wozniak's", 1.2672677410326383], ["preobrazhensky", 1.2672677410326383], ["chattanooga's", 1.2672677410326383], ["debes", 1.2672677410326383], ["bombo", 1.2672677410326383], ["mopane", 1.2672677410326383], ["priss", 1.2672677410326383], ["orte", 1.2672677410326383], ["blocksize", 1.2672677410326383], ["sneaky", 1.2671992291814775], ["mine", 1.2670693439011236], ["everywhere", 1.2670561162227227], ["rainsford", 1.2670431676246936], ["ramesses", 1.266983918470257], ["limber", 1.266983918470257], ["contextualized", 1.266983918470257], ["readily", 1.266828833286347], ["time", 1.2667953733954667], ["sets", 1.266792685778481], ["choose", 1.2667277555162613], ["happy's", 1.2667272361529007], ["updrafts", 1.2667272361529007], ["tailed", 1.2665920202422003], ["carolingian", 1.2665693294917], ["ropa", 1.2665293834441556], ["tunnels", 1.2664887532978844], ["shallots", 1.2664879696503966], ["arugula", 1.2664340122556432], ["honorific", 1.2664340122556432], ["casualness", 1.266281079758088], ["carpathians", 1.2662557667929755], ["overheating", 1.266187863759578], ["relate", 1.2660418722377738], ["wallabies", 1.2659065511926464], ["pujo", 1.2659065511926462], ["kigoma", 1.2659065511926462], ["codebooks", 1.2659065511926462], ["gr\u00fcner", 1.2659065511926462], ["elastane", 1.2659065511926462], ["coden", 1.2659065511926462], ["shrinkwrap", 1.2659065511926462], ["bici", 1.2659065511926462], ["ciutat", 1.2659065511926462], ["encik", 1.2659065511926462], ["nyingma", 1.2659065511926462], ["sociocultural", 1.2658207270196842], ["brca2", 1.2657354827397822], ["regrouping", 1.2656146995381925], ["haile", 1.2656146995381925], ["sentimentally", 1.2655876830437311], ["utm_source", 1.2655876830437311], ["superior", 1.2655294165326034], ["sales", 1.2653174826774911], ["privileged", 1.2651697387108374], ["mexico's", 1.2651275317765276], ["confectionery", 1.2650737179352827], ["amontillado", 1.2650737179352827], ["leela", 1.2650034444279576], ["wrenches", 1.2649734505406616], ["pi\u00f1a", 1.2647243816506766], ["unaltered", 1.264716558000277], ["ambassadors", 1.264701265518553], ["bele", 1.2645116128442246], ["celui", 1.2645116128442246], ["kamal's", 1.2645116128442246], ["turgor", 1.2645116128442246], ["souchong", 1.2645116128442246], ["titrations", 1.2645116128442246], ["javelina", 1.2645116128442246], ["langtang", 1.2645116128442246], ["annotator", 1.2645116128442246], ["rattler", 1.2643317392150861], ["chemo", 1.2642850529135905], ["erased", 1.2642760307804304], ["chan's", 1.264218675791056], ["zinnia", 1.264218675791056], ["harbor", 1.264168398651434], ["gargling", 1.264143896501112], ["stores", 1.2641218304811286], ["crocodiles", 1.2640844129750204], ["practitioners", 1.263984620168807], ["starchy", 1.2639328259246576], ["fins", 1.2638833119012205], ["villi", 1.2637769887700792], ["watcher", 1.2637769887700792], ["position's", 1.2637573040768693], ["assim", 1.2637573040768693], ["rundgren", 1.2637573040768693], ["univision's", 1.2637573040768693], ["sprig", 1.2636476844841984], ["mmhg", 1.2636476844841984], ["springing", 1.2633776448791272], ["capitan", 1.2632994209956818], ["hitchhiking", 1.2632994209956818], ["paper's", 1.2631929251450735], ["tortugas", 1.2631929251450735], ["reformulating", 1.2631445732627713], ["paavo", 1.2631445732627713], ["divvied", 1.2631445732627713], ["chamb\u00e9ry", 1.2631445732627713], ["myplate", 1.2631445732627713], ["gram's", 1.2631445732627713], ["tso's", 1.2631445732627713], ["tilla", 1.2631445732627713], ["synthy", 1.2631445732627713], ["instructables", 1.2631445732627713], ["sulis", 1.2631445732627713], ["peniche", 1.2631445732627713], ["yuzo", 1.2631445732627713], ["memet", 1.2631445732627713], ["basic", 1.2631140526947962], ["mailbox", 1.2630892885738156], ["decomposition", 1.2630826058092164], ["problematic", 1.2630228126522776], ["burrs", 1.2630086928224462], ["ebbets", 1.2629701700171996], ["fastpass", 1.2629701700171996], ["empresses", 1.2628744396927836], ["rapiers", 1.2628744396927836], ["randomizing", 1.2628744396927836], ["buffet's", 1.2628744396927836], ["googlers", 1.2628744396927836], ["stairmaster", 1.2628744396927836], ["agatha", 1.2628334539097508], ["originating", 1.2627711270194937], ["bougainville", 1.2627417848146645], ["pyotr", 1.2625846687533462], ["perpetrator", 1.262550990286614], ["bonaventure", 1.2625457373399223], ["thorp", 1.2625457373399223], ["silversmith", 1.2624811569953016], ["river", 1.2624320517912047], ["substandard", 1.2623531292945616], ["intercity", 1.262305504110479], ["rolle", 1.2622580868966327], ["westward", 1.2621879506985176], ["guarding", 1.2621797820537368], ["illuminations", 1.2621264135745016], ["novosibirsk", 1.2621264135745016], ["reiterated", 1.2620226703601913], ["textile", 1.2620007114065819], ["karabakh", 1.261925321744178], ["categorical", 1.2618369701848595], ["dimmer", 1.2617950598297982], ["coraline", 1.2617950598297982], ["bedouins", 1.2617950598297982], ["annual", 1.26169693955674], ["helplines", 1.2616153425445356], ["pietra", 1.2616153425445356], ["kafe", 1.2616153425445356], ["skywatchers", 1.2616153425445356], ["camiguin", 1.2616153425445356], ["stapes", 1.2616153425445356], ["imaginarium", 1.2616153425445356], ["vilakazi", 1.2616153425445356], ["helicase", 1.2616153425445356], ["tempio", 1.2616153425445356], ["nidaros", 1.2616153425445356], ["piccolo's", 1.2616153425445356], ["cid's", 1.2616153425445356], ["asifa", 1.2616153425445356], ["frailties", 1.2614972137671439], ["important", 1.2614799291644356], ["hutu", 1.261413645447406], ["specializes", 1.261395757595675], ["computed", 1.2612648938382733], ["sine", 1.2612648938382731], ["joshua's", 1.2612648938382731], ["retaliation", 1.261206650250472], ["sumo", 1.2610850984150603], ["woke", 1.2610656025257962], ["racetracks", 1.2610376798419054], ["cremona", 1.2610376798419054], ["gryffindor", 1.2609259352535278], ["harder", 1.2608647040253156], ["philanthropists", 1.2608429249878759], ["kaleb", 1.2608429249878759], ["vivacity", 1.2605979108166772], ["unexamined", 1.2605979108166772], ["caldera", 1.2604605458923208], ["additional", 1.2602949398902181], ["revelled", 1.2602802998540124], ["atreus", 1.2602802998540124], ["topographical", 1.2602490924377125], ["magnitude", 1.26018971720746], ["obelix", 1.2600741021609734], ["ecologies", 1.2600741021609734], ["sepsis", 1.2600486306812448], ["makings", 1.2600063258757832], ["replace", 1.2599154818724756], ["overcharging", 1.2598971893542998], ["caregivers", 1.2598389301369297], ["fjord", 1.2597925752797652], ["brahmin", 1.2597925752797652], ["nary", 1.259723460068569], ["dilution", 1.2596743958636978], ["oyster", 1.2595368888341956], ["easily", 1.2595253233598078], ["ruse", 1.2594111410265265], ["jealousies", 1.2593852144137752], ["paar", 1.2593852144137752], ["midgard", 1.2593852144137752], ["marinus", 1.2593852144137752], ["menos", 1.2593852144137752], ["condos", 1.259385214413775], ["popularized", 1.2592038948551416], ["desecrate", 1.2591972464713252], ["wendi", 1.2591972464713252], ["mandala", 1.2591046652459394], ["manhattan's", 1.2591046652459394], ["circumvent", 1.2590652689798179], ["marv", 1.2590047125059818], ["patient", 1.2589339636996424], ["fairways", 1.2588861844780674], ["montag", 1.2588323675242168], ["morelos", 1.2588323675242168], ["codecs", 1.2586333426439755], ["infosec", 1.2585679624900803], ["gastroparesis", 1.2585679624900803], ["maiden's", 1.2585679624900803], ["illuminators", 1.2585679624900803], ["ludwigia", 1.25856796249008], ["ginori", 1.25856796249008], ["sharky's", 1.25856796249008], ["flams", 1.25856796249008], ["homograph", 1.25856796249008], ["kinabatangan", 1.25856796249008], ["patronato", 1.25856796249008], ["terreur", 1.25856796249008], ["psychadelic", 1.25856796249008], ["xandria", 1.25856796249008], ["devotee's", 1.25856796249008], ["kaupapa", 1.25856796249008], ["feitan", 1.25856796249008], ["obidos", 1.25856796249008], ["tr\u1ea7n", 1.25856796249008], ["dakshin", 1.25856796249008], ["maqui", 1.25856796249008], ["tsitsikamma", 1.25856796249008], ["unifications", 1.25856796249008], ["glorieux", 1.25856796249008], ["westmalle", 1.25856796249008], ["peddler's", 1.25856796249008], ["cpat", 1.25856796249008], ["nutkin", 1.25856796249008], ["bewitching", 1.2584643340040784], ["maryann", 1.2583959094535127], ["matt's", 1.2583655316144056], ["colobus", 1.2582830252108206], ["manette", 1.2582830252108206], ["cabella", 1.2582830252108206], ["ua's", 1.2582480418082067], ["victoriana", 1.2582480418082067], ["shiri", 1.2582480418082067], ["rhyolite", 1.2581854190364357], ["sesotho", 1.258061496509199], ["radioing", 1.258061496509199], ["qurans", 1.258061496509199], ["cuong", 1.258061496509199], ["cont\u00e9", 1.258061496509199], ["aerogels", 1.258061496509199], ["karne", 1.258061496509199], ["millerton", 1.258061496509199], ["behzad", 1.258061496509199], ["plums", 1.2578742180918312], ["fluke", 1.257668985374016], ["asin", 1.2575827820889138], ["volleying", 1.257516527266613], ["comiskey", 1.2574799417747224], ["module's", 1.2574799417747224], ["agatha's", 1.2574799417747224], ["jcpoa", 1.2574799417747224], ["sardinas", 1.2573531285494526], ["grassington", 1.2573531285494526], ["solitario", 1.2573531285494526], ["euobserver", 1.2573531285494526], ["trantor", 1.2573531285494526], ["olet", 1.2573531285494526], ["hemocytes", 1.2573531285494526], ["kahe", 1.2573531285494526], ["d\u00e9co", 1.2573531285494526], ["gumline", 1.2573531285494526], ["tralles", 1.2573531285494526], ["rossner", 1.2573531285494526], ["reengaging", 1.2573531285494526], ["zandt's", 1.2573531285494526], ["bansky", 1.2573531285494526], ["powerpop", 1.2573531285494526], ["hornwort", 1.2573531285494526], ["majjhima", 1.2573531285494526], ["eville", 1.2573531285494526], ["cswe", 1.2573531285494526], ["amex's", 1.2573531285494526], ["adria's", 1.2573531285494526], ["shojiro", 1.2573531285494526], ["tiden", 1.2573531285494526], ["bielsa's", 1.2573531285494526], ["lempira", 1.2573531285494526], ["kamenka", 1.2573531285494526], ["revolution", 1.2573346550415951], ["poseidon", 1.2573265179012294], ["johan", 1.2572629957086963], ["banana", 1.2572123049990551], ["inhalers", 1.2572034436531967], ["guilbert", 1.257095737939719], ["gave", 1.257092427915758], ["wildness", 1.2570200548730157], ["viruses", 1.2569909858730732], ["colonies", 1.2569681754293458], ["bambi", 1.2569119520131196], ["decorative", 1.2569067781519963], ["kochi", 1.2567599638209006], ["mannequins", 1.256723240662318], ["liberation", 1.25661077983887], ["guitarists", 1.2565753843462264], ["emulate", 1.2565181547468363], ["rory's", 1.2564933999145471], ["courtiers", 1.256377727334578], ["birdy's", 1.256292071057006], ["an\u00edbal", 1.256292071057006], ["wrightbus", 1.256292071057006], ["reroutes", 1.256292071057006], ["monegasque", 1.256292071057006], ["merg", 1.256292071057006], ["recutting", 1.256292071057006], ["zapotecs", 1.256292071057006], ["echte", 1.256292071057006], ["zulu's", 1.256292071057006], ["petropoulos", 1.256292071057006], ["dragana", 1.256292071057006], ["sergiu", 1.256292071057006], ["septoplasty", 1.256292071057006], ["corbet's", 1.256292071057006], ["mumbi", 1.256292071057006], ["techno", 1.256246332268011], ["journals", 1.2562448846060197], ["sapphires", 1.2561737018292052], ["manifest", 1.2561281498577552], ["hitomi", 1.2559043265906302], ["luxe", 1.2558259442650326], ["beatriz", 1.2555804252463618], ["oscilloscope", 1.2554161702106512], ["idealist", 1.2552699399341045], ["readiness", 1.255258617425288], ["chomp", 1.2552266316162126], ["rennes", 1.2552266316162126], ["aquifers", 1.255098226252639], ["rakhimov", 1.255069997057923], ["superfluidity", 1.255069997057923], ["iskender", 1.255069997057923], ["massiveness", 1.255069997057923], ["hodja", 1.255069997057923], ["pickford's", 1.255069997057923], ["jaejoong", 1.255069997057923], ["yildiz", 1.2549899441176846], ["janitor's", 1.2549899441176846], ["lear's", 1.2549899441176846], ["supercell", 1.2545668812411177], ["edutopia", 1.254527615901027], ["starfinder", 1.254527615901027], ["bugling", 1.254527615901027], ["autoworker", 1.254527615901027], ["abu's", 1.254527615901027], ["countermovement", 1.254527615901027], ["endodontist", 1.254527615901027], ["as's", 1.254527615901027], ["essing", 1.254527615901027], ["ammal", 1.254527615901027], ["vergers", 1.254527615901027], ["georgievski", 1.254527615901027], ["epigenomics", 1.254527615901027], ["erek", 1.254527615901027], ["murashige", 1.254527615901027], ["thelonious", 1.2545276159010268], ["providers", 1.2544646313869776], ["temperature", 1.254328575669061], ["antennae", 1.253967314896104], ["finn's", 1.2539280806007158], ["prismatic", 1.2538331363983535], ["zambezi", 1.2537861693195251], ["crackers", 1.253786169319525], ["adaptively", 1.25370862672757], ["spiritualism", 1.2536472456442542], ["dieting", 1.2534537815631364], ["armor", 1.253353317096575], ["ballet", 1.2532588518450403], ["shears", 1.2530880516062621], ["lantern", 1.2530857482343756], ["disrespect", 1.253081465149416], ["hen's", 1.2529943043704814], ["gerry's", 1.2529943043704814], ["updo", 1.2527510517390756], ["naina", 1.2527510517390756], ["periodization", 1.2527510517390756], ["germany's", 1.252664944612914], ["positioned", 1.2526524890845352], ["luchador", 1.2525171549781084], ["illusionists", 1.2525171549781084], ["aliyu", 1.2525171549781084], ["longform", 1.2524035479799245], ["picassos", 1.2524035479799245], ["cauliflower", 1.2523661583349528], ["regatta", 1.2523078789288224], ["compaction", 1.2520753499851711], ["philip's", 1.2518324239769698], ["bianchi", 1.2517792950929965], ["asser", 1.251665102491729], ["flashpoints", 1.2514707747316776], ["yeboah", 1.2514707747316776], ["nonconformists", 1.2514707747316776], ["rosacea", 1.2514232535327128], ["sarajevo", 1.2514189539956637], ["outspend", 1.2513761455615653], ["implausibility", 1.2513761455615653], ["maximalist", 1.2513761455615653], ["principio", 1.2513761455615653], ["concei\u00e7\u00e3o", 1.2513139684699646], ["feminity", 1.2513139684699646], ["mildewed", 1.2513139684699646], ["beamon", 1.2513139684699646], ["loretta's", 1.2513139684699646], ["epicycles", 1.2513139684699646], ["dovid", 1.2513139684699646], ["surrey's", 1.2512699932821778], ["taira", 1.2512699932821778], ["ricocheting", 1.2511917399992958], ["wash", 1.2509697329903449], ["cele", 1.2509266570035333], ["durum", 1.2509266570035333], ["bootcamp", 1.2508695396729546], ["kosovar", 1.2508695396729546], ["ragout", 1.2508695396729546], ["bakugo", 1.2508695396729546], ["writing", 1.2507251939267536], ["retreat", 1.2506969184072059], ["encrypts", 1.250674504885613], ["chemotherapy", 1.250499457134329], ["geologists", 1.2504771684455012], ["deprecation", 1.2504735868633654], ["agrippina", 1.250434360011403], ["gauging", 1.2502616242246904], ["indices", 1.2502428047615466], ["j\u00falia", 1.2502053846662258], ["aavs", 1.2502053846662258], ["creditcards", 1.2502053846662258], ["d'apprentissage", 1.2502053846662258], ["convention's", 1.2502053846662258], ["grinchy", 1.2502053846662258], ["narcis", 1.2502053846662258], ["cyrodiil", 1.2502053846662258], ["cerastium", 1.2502053846662258], ["wembley's", 1.2502053846662258], ["druzy", 1.2502053846662258], ["riverways", 1.2502053846662258], ["hinaus", 1.2502053846662258], ["nyungwe", 1.2502053846662258], ["siente", 1.2502053846662258], ["indyref2", 1.2502053846662258], ["longwy", 1.2502053846662258], ["zweli", 1.2502053846662258], ["matylda", 1.2502053846662258], ["kinako", 1.2502053846662258], ["tresemm\u00e9", 1.2502053846662258], ["bisc", 1.2502053846662258], ["geertje", 1.2502053846662258], ["fireflight", 1.2502053846662258], ["hajia", 1.2502053846662258], ["ultrasparc", 1.2502053846662258], ["xcalibur", 1.2502053846662258], ["ilya's", 1.2502053846662258], ["elsley", 1.2502053846662258], ["rassie", 1.2502053846662258], ["dinagat", 1.2502053846662258], ["filos", 1.2502053846662258], ["sunchokes", 1.2502053846662258], ["ccca", 1.2502053846662258], ["granadilla", 1.2502053846662258], ["cymbal", 1.250155940118681], ["servers", 1.250114366966518], ["potion", 1.2500857477873104], ["teacher's", 1.2500225526415598], ["spreadable", 1.249963143746895], ["phototherapy", 1.249963143746895], ["guard", 1.249944813281798], ["banners", 1.2499395891650404], ["robyn's", 1.2498961575119676], ["protoss", 1.2498961575119676], ["computer", 1.249856267209068], ["detective's", 1.2498121258949553], ["cloistered", 1.2498121258949553], ["alpena", 1.2497779640218956], ["toughest", 1.2497328065582438], ["org's", 1.2497328065582438], ["erwin's", 1.2497328065582438], ["stoneheart", 1.2497328065582438], ["typhi", 1.2497328065582438], ["havarti", 1.2497328065582438], ["tests", 1.249654258623873], ["thicker", 1.2496278835318322], ["provence", 1.2496108814063844], ["spendings", 1.2495135167167701], ["malti", 1.2495135167167701], ["kaunitz", 1.2495135167167701], ["chimichangas", 1.2495135167167701], ["belleek", 1.2495135167167701], ["waterfronts", 1.2495135167167701], ["powerscourt", 1.2495135167167701], ["matignon", 1.2495135167167701], ["jayanthi", 1.2495135167167701], ["checkers", 1.2494497660271418], ["disclosing", 1.2493925960538623], ["unawares", 1.2493869196026146], ["cinemascore", 1.2493869196026146], ["y'see", 1.249304498062251], ["skybridge", 1.249304498062251], ["precalculus", 1.249304498062251], ["rastas", 1.249304498062251], ["soichiro", 1.249304498062251], ["kin's", 1.2492036175310766], ["kuip", 1.2492036175310766], ["refreezing", 1.2492036175310766], ["dvina", 1.2492036175310766], ["starfleet's", 1.2492036175310766], ["sleekest", 1.2492036175310766], ["politicians", 1.24916885880973], ["wenceslas", 1.249144202376854], ["advaita", 1.249144202376854], ["cando", 1.2491050454788766], ["devons", 1.2491050454788766], ["trish's", 1.2491050454788766], ["estar", 1.248970774359123], ["close", 1.2489474941090573], ["dessalines", 1.2489145169484495], ["noncommittal", 1.2488042375209045], ["embarrassments", 1.2488042375209045], ["promoted", 1.248771024642247], ["thor's", 1.2487323266469255], ["venkman", 1.2487323266469255], ["alata", 1.2487323266469255], ["scourging", 1.2487323266469255], ["yuna", 1.2487323266469255], ["encyclicals", 1.2487323266469255], ["zalando", 1.2487323266469255], ["redes", 1.2487323266469255], ["crab's", 1.2487323266469255], ["saxes", 1.2487323266469255], ["only", 1.248665402655862], ["alcoves", 1.2485922164279468], ["vanilla", 1.2484765546322945], ["chewed", 1.248431220526182], ["snorkel", 1.2483582850132744], ["kimberly's", 1.2482700822131823], ["pepsin", 1.2482700822131823], ["fragmenting", 1.2482463733797118], ["snes", 1.2480957631047618], ["noting", 1.2480319506898059], ["novoselic", 1.247929180051449], ["platonically", 1.247929180051449], ["rosalind", 1.2478774238125583], ["beverages", 1.2477576497325829], ["candide", 1.2475580180349102], ["tower", 1.2475372856868272], ["janez", 1.247519081733982], ["ranjeet", 1.247519081733982], ["sunnis", 1.247519081733982], ["demetria", 1.247519081733982], ["streaks", 1.2472778675324487], ["hamming", 1.2472582503847547], ["symbolically", 1.2472368039853463], ["toothpastes", 1.24708364749952], ["catfishes", 1.2469832058397665], ["insulin", 1.246925791355414], ["delphiniums", 1.246822532832958], ["calea", 1.246822532832958], ["kyoung", 1.246822532832958], ["daunte", 1.246822532832958], ["lingnan", 1.246822532832958], ["polarized", 1.2467913622696374], ["sending", 1.2467151590497252], ["skateboard", 1.2467147827375282], ["desu", 1.2466697359539343], ["zarathustra", 1.2466697359539343], ["postcolonialism", 1.2466204546266644], ["vespasian", 1.2465242499303666], ["ansu", 1.2465242499303666], ["mancunians", 1.2465242499303666], ["march\u00e9s", 1.2465242499303666], ["mastaba", 1.2465242499303666], ["verwendung", 1.2465242499303666], ["chicharrones", 1.2465242499303666], ["nanoengineering", 1.2465242499303666], ["longwear", 1.2465242499303666], ["lyngen", 1.2465242499303666], ["tounes", 1.2465242499303666], ["djiboutian", 1.2465242499303666], ["minack", 1.2465242499303666], ["miniaturizing", 1.2465242499303666], ["essanay", 1.2465242499303666], ["naras", 1.2465242499303666], ["cristiani", 1.2465242499303666], ["skyblue", 1.2465242499303666], ["oula", 1.2465242499303666], ["interiors", 1.2464530199732275], ["thin", 1.2464339045892496], ["mari", 1.2464264833225287], ["creeds", 1.2463991387644315], ["syllables", 1.246292274974167], ["overreach", 1.2462661703755156], ["clambered", 1.2462532074755002], ["castiel's", 1.2462532074755], ["ciphered", 1.2462532074755], ["retie", 1.2462532074755], ["snomed", 1.2462532074755], ["jaeden", 1.2462532074755], ["seok's", 1.2462532074755], ["keats", 1.246157840685825], ["coloring", 1.246010014807063], ["laham", 1.2460058368507532], ["unvented", 1.2460058368507532], ["blastula", 1.2460058368507532], ["serge's", 1.2460058368507532], ["chanderi", 1.2460058368507532], ["hangtime", 1.2460058368507532], ["statt", 1.2458900800840444], ["debuggers", 1.2458900800840444], ["bienvenido", 1.2458900800840444], ["missing", 1.245816614225219], ["invulnerable", 1.245768330065114], ["suffrage", 1.2457575385861632], ["accomplish", 1.2456836940373437], ["metaphor", 1.2456709053165984], ["beauties", 1.2456520463735639], ["mandal", 1.2456520463735639], ["pearls", 1.2455936760426733], ["reunion", 1.2455449426465535], ["regulating", 1.245480881544107], ["dehydrated", 1.2453599140925193], ["gliders", 1.2452088612122525], ["analisa", 1.2452002730318743], ["crampon", 1.2452002730318743], ["nightlights", 1.2452002730318743], ["laxmi", 1.2452002730318743], ["restrung", 1.2452002730318743], ["anglia's", 1.2452002730318743], ["baldomero", 1.2452002730318743], ["yingying", 1.2452002730318743], ["param", 1.2450996507875889], ["passenger's", 1.2450996507875889], ["scraper", 1.244903510325902], ["apprehended", 1.2448453412183922], ["punk", 1.2448284059086105], ["cocoon", 1.2447237149026893], ["intervals", 1.2446393211326099], ["recitals", 1.2446097764740407], ["activity's", 1.244603269425003], ["derailleurs", 1.244603269425003], ["knapweed", 1.244603269425003], ["lewin's", 1.244603269425003], ["presonus", 1.244603269425003], ["chelonia", 1.244603269425003], ["yuuka", 1.244603269425003], ["sp\u00e9cial", 1.244603269425003], ["bushier", 1.244603269425003], ["florentina", 1.244603269425003], ["mayor's", 1.2445537712834878], ["germination", 1.2445205546225773], ["achiever", 1.2445143328908463], ["bedrock", 1.2444855565672295], ["whoopsie", 1.2443978011928531], ["swadeshi", 1.2443978011928531], ["pedal", 1.2442789578031628], ["handlebar", 1.2442479730306482], ["boomy", 1.2442479730306482], ["infosphere", 1.2442479730306482], ["tipps", 1.2442479730306482], ["barley", 1.244201784499382], ["loophole", 1.2441533294221516], ["utter", 1.2440929072314169], ["midsize", 1.2440830354368744], ["selena", 1.2440732191018518], ["given", 1.2440012451223315], ["francisco", 1.2439918043303184], ["transit", 1.243886186855318], ["radioed", 1.2438531265376584], ["spouting", 1.2437858130042798], ["pippy", 1.2437775934854227], ["communication's", 1.2437775934854227], ["hualien", 1.2437775934854227], ["harpsichordist", 1.2437775934854227], ["tunicates", 1.2437775934854227], ["combien", 1.2437775934854227], ["lei's", 1.2437775934854227], ["dripper", 1.2437775934854227], ["emissaries", 1.243705476500876], ["shines", 1.2437054765008757], ["anthropology", 1.2436413223213338], ["actors", 1.243504544343692], ["spycraft", 1.2431783751169536], ["fasttrack", 1.2431783751169536], ["ovine", 1.2431783751169536], ["ranking", 1.2429895590465392], ["safe", 1.2429529604909955], ["imperfective", 1.2426946823422547], ["hcmc", 1.2426946823422547], ["nicanor", 1.2426946823422547], ["ramsar", 1.2426946823422547], ["practice's", 1.2426946823422547], ["handful", 1.2425118130682797], ["aware", 1.2424194710708667], ["relationship", 1.2423745155538588], ["acclimatized", 1.242222924016183], ["transoceanic", 1.242222924016183], ["result", 1.2415496707989901], ["relocating", 1.241373602713204], ["purely", 1.241277635957776], ["standout", 1.241237942901866], ["defunct", 1.241237942901866], ["examination", 1.2411337337600763], ["backing", 1.2410992506947451], ["bounced", 1.2410191800369075], ["leaching", 1.2405884201687936], ["mailbags", 1.2405884201687933], ["hampers", 1.2405884201687933], ["treacherously", 1.2405884201687933], ["bindi", 1.2405884201687933], ["vadodara", 1.2405884201687933], ["rationality", 1.2405884201687933], ["blandness", 1.2405884201687933], ["callout", 1.2405884201687933], ["twig", 1.2405884201687933], ["utilitarianism", 1.2405884201687933], ["orda", 1.2405884201687933], ["philosophizing", 1.2405884201687933], ["interloping", 1.2405884201687933], ["agreeableness", 1.2405884201687933], ["griswald", 1.2405884201687933], ["misperceived", 1.2405884201687933], ["facetime", 1.2405884201687933], ["pagoda", 1.2405884201687933], ["ptarmigan", 1.2405884201687933], ["alkalinity", 1.2405884201687933], ["trespassers", 1.2405884201687933], ["nonsteroidal", 1.2405884201687933], ["payitforward", 1.2405884201687933], ["mousepad", 1.2405884201687933], ["understudied", 1.2405884201687933], ["bluer", 1.2405884201687933], ["bloom's", 1.2405884201687933], ["facials", 1.2405884201687933], ["econometrician", 1.2405884201687933], ["deseo", 1.2405884201687933], ["theban", 1.2405884201687933], ["livestreaming", 1.2405884201687933], ["thermoregulatory", 1.2405884201687933], ["monadology", 1.2405884201687933], ["polytheism", 1.2405884201687933], ["drumstick", 1.2405884201687933], ["brielle", 1.2405884201687933], ["pillowcase", 1.2405884201687933], ["languish", 1.2405884201687933], ["underpainting", 1.2405884201687933], ["clienteles", 1.2405884201687933], ["meadowfoam", 1.2405884201687933], ["magnifique", 1.2405884201687933], ["baubles", 1.2405884201687933], ["segundos", 1.2405884201687933], ["bahrain's", 1.2405884201687933], ["spinnerets", 1.2405884201687933], ["pleasers", 1.2405884201687933], ["unencumbered", 1.2405884201687933], ["gojek", 1.2405884201687933], ["teve", 1.2405884201687933], ["watchmaker's", 1.2405884201687933], ["anabaptists", 1.2405884201687933], ["lightboxes", 1.2405884201687933], ["marcelle", 1.2405884201687933], ["yiruma", 1.2405884201687933], ["ungroomed", 1.2405884201687933], ["loups", 1.2405884201687933], ["siyah", 1.2405884201687933], ["arils", 1.2405884201687933], ["gamespot's", 1.2405884201687933], ["bipartisanship", 1.2405884201687933], ["mcnealy", 1.2405884201687933], ["akai", 1.2405884201687933], ["jibes", 1.2405884201687933], ["fujifilm's", 1.2405884201687933], ["poblado", 1.2405884201687933], ["orienteers", 1.2405884201687933], ["krob", 1.2405884201687933], ["pitcairn", 1.2405884201687933], ["karera", 1.2405884201687933], ["cardrona", 1.2405884201687933], ["elrond", 1.2405884201687933], ["ngoni", 1.2405884201687933], ["duodecimal", 1.2405884201687933], ["arc'teryx", 1.2405884201687933], ["acteurs", 1.2405884201687933], ["eufor", 1.2405884201687933], ["sild", 1.2405884201687933], ["reincorporating", 1.2405884201687933], ["commerford", 1.2405884201687933], ["stoutest", 1.2405884201687933], ["mangena", 1.2405884201687933], ["unrushed", 1.2405884201687933], ["kape", 1.2405884201687933], ["chueca", 1.2405884201687933], ["weihenstephan", 1.2405884201687933], ["huangdi", 1.2405884201687933], ["cluster's", 1.2405884201687933], ["mueck", 1.2405884201687933], ["shaftoe", 1.2405884201687933], ["bleek", 1.2405884201687933], ["chemtech", 1.2405884201687933], ["etymologists", 1.2405884201687933], ["broflovski", 1.2405884201687933], ["aerotropolis", 1.2405884201687933], ["juicio", 1.2405884201687933], ["emeril's", 1.2405884201687933], ["gleann", 1.2405884201687933], ["folarin", 1.2405884201687933], ["catal\u00e1n", 1.2405884201687933], ["microlepidoptera", 1.2405884201687933], ["lontano", 1.2405884201687933], ["dagomba", 1.2405884201687933], ["becchetti", 1.2405884201687933], ["rockfeller", 1.2405884201687933], ["vuoi", 1.2405884201687933], ["sameer's", 1.2405884201687933], ["hideyoshi's", 1.2405884201687933], ["autoinjector", 1.2405884201687933], ["fris\u00e9e", 1.2405884201687933], ["auli'i", 1.2405884201687933], ["wildwater", 1.2405884201687933], ["aetolia", 1.2405884201687933], ["sandpapered", 1.2405884201687933], ["mahabodhi", 1.2405884201687933], ["dougie's", 1.2405884201687933], ["virbhadra", 1.2405884201687933], ["puntos", 1.2405884201687933], ["serer", 1.2405884201687933], ["tredway", 1.2405884201687933], ["jacquemus", 1.2405884201687933], ["unposted", 1.2405884201687933], ["burgoyne's", 1.2405884201687933], ["juridiques", 1.2405884201687933], ["arverni", 1.2405884201687933], ["cassadine", 1.2405884201687933], ["caldey", 1.2405884201687933], ["stadtholder", 1.2405884201687933], ["parmenion", 1.2405884201687933], ["pellentesque", 1.2405884201687933], ["jiten", 1.2405884201687933], ["bovids", 1.2405884201687933], ["sharded", 1.2405884201687933], ["origen's", 1.2405884201687933], ["tpwd", 1.2405884201687933], ["aguecheek", 1.2405884201687933], ["westfield's", 1.2405884201687933], ["laging", 1.2405884201687933], ["synchronizations", 1.2405884201687933], ["senufo", 1.2405884201687933], ["kourosh", 1.2405884201687933], ["bonavita", 1.2405884201687933], ["iglu", 1.2405884201687933], ["turnbow", 1.2405884201687933], ["smallie", 1.2405884201687933], ["shavasana", 1.2405884201687933], ["dhanushkodi", 1.2405884201687933], ["abott", 1.2405884201687933], ["lucretia's", 1.2405884201687933], ["brahmani", 1.2405884201687933], ["hirudo", 1.2405884201687933], ["lusia", 1.2405884201687933], ["kulikovo", 1.2405884201687933], ["afa's", 1.2405884201687933], ["pmps", 1.2405884201687933], ["enchant\u00e9e", 1.2405884201687933], ["gabagool", 1.2405884201687933], ["chunkin", 1.2405884201687933], ["mahdist", 1.2405884201687933], ["sake's", 1.2405884201687933], ["thoko", 1.2405884201687933], ["manele", 1.2405884201687933], ["vorw\u00e4rts", 1.2405884201687933], ["visaya", 1.2405884201687933], ["kalana", 1.2405884201687933], ["jossy", 1.2405884201687933], ["comorbidities", 1.2405884201687931], ["cooperstown", 1.2405884201687931], ["heartthrob", 1.2405884201687931], ["snowballs", 1.2405884201687931], ["bajo", 1.2405884201687931], ["organelle", 1.2405884201687931], ["tacitus", 1.2405884201687931], ["marjane", 1.2405884201687931], ["electroconvulsive", 1.2405884201687931], ["malevich", 1.2405884201687931], ["cowhide", 1.2405884201687931], ["voici", 1.2405884201687931], ["yosef", 1.2405884201687931], ["tilia", 1.2405884201687931], ["delux", 1.2405884201687931], ["faruq", 1.2405884201687931], ["falling", 1.240262663909668], ["detect", 1.2402561196991053], ["duplicate", 1.2399505855054673], ["glucose", 1.2399261843573153], ["concerted", 1.2399214371471972], ["exclusive", 1.239836548398994], ["modular", 1.2398226248477016], ["overload", 1.2398032376243826], ["stealing", 1.2397856864851546], ["appetite", 1.2397513159716482], ["convention", 1.2396462011155005], ["kanji", 1.239372157011765], ["completely", 1.2392113670224059], ["overlaps", 1.2391960650395353], ["rehearse", 1.2391640936243973], ["jesting", 1.2391306194400402], ["obliques", 1.2391306194400402], ["asynchronously", 1.2391306194400402], ["bushmeat", 1.2391306194400402], ["outranked", 1.2391306194400402], ["pepita", 1.2391306194400402], ["hadley's", 1.2391306194400402], ["everett", 1.2391245990240511], ["afterthought", 1.2389539163214036], ["fluffs", 1.2389142117745036], ["dome's", 1.2389142117745036], ["photosensitivity", 1.2389142117745036], ["jester's", 1.2387115844348011], ["haymitch", 1.2387115844348011], ["pompeian", 1.2386223529419014], ["tienen", 1.2386223529419014], ["cauliflowers", 1.2386223529419014], ["goers", 1.2385712032254295], ["bandits", 1.2385629696868852], ["dropout", 1.2385207728018452], ["sputnik", 1.2384821579953318], ["chorizo", 1.2384821579953318], ["footage", 1.2383991464861424], ["rayan", 1.2381121957772387], ["monetarism", 1.2381121957772387], ["bogside", 1.2381121957772387], ["monocacy", 1.2381121957772387], ["cascades", 1.2380488328194201], ["arnold's", 1.237998465220633], ["soldering", 1.2378678315280722], ["shuttlecock", 1.2378678315280722], ["vale's", 1.2378678315280722], ["personifications", 1.2378678315280722], ["alida", 1.2378678315280722], ["personified", 1.2375365421363977], ["signatures", 1.237497986381896], ["landmark", 1.2372311576915322], ["offspring", 1.237218400691255], ["rations", 1.2370235109154346], ["upgrading", 1.2370183815496025], ["hansberry", 1.2370029045035658], ["kion", 1.2370029045035658], ["pigeon", 1.236930665613013], ["sorenson", 1.2369288673069385], ["demonizing", 1.2368404188994013], ["chelo", 1.2368404188994013], ["bloodline", 1.2366912837703574], ["impersonating", 1.2365993898788614], ["conor's", 1.2365993898788614], ["glassy", 1.2364736990405385], ["plentiful", 1.2362544868232253], ["longer", 1.2362249712426823], ["derek's", 1.2361891704518826], ["hijab", 1.2361891704518826], ["hoots", 1.2361891704518826], ["kabab", 1.2361735147589399], ["helpmate", 1.2361735147589399], ["appliance", 1.2361577472396188], ["gagosian", 1.2360106031202], ["tinted", 1.2359195390176203], ["adonis", 1.2359010583797423], ["graphene", 1.2358623690443409], ["ignites", 1.235751818140845], ["aquanauts", 1.2356458368215073], ["glassful", 1.2356458368215073], ["resmi", 1.2356458368215073], ["belayer", 1.2356458368215073], ["exhaustingly", 1.2356458368215073], ["processo", 1.2356458368215073], ["soubirous", 1.2356458368215073], ["lampang", 1.2356458368215073], ["harjit", 1.2356458368215073], ["haloumi", 1.2356458368215073], ["camtasia", 1.2356458368215073], ["nanban", 1.2356458368215073], ["track's", 1.2355247939640226], ["dishwasher", 1.235498826650152], ["talked", 1.2354118641709668], ["dishonest", 1.2353412183916443], ["examine", 1.235330297063442], ["redirecting", 1.235286760253542], ["oksana", 1.235286760253542], ["upward", 1.23526815793805], ["comoros", 1.2351710034868335], ["collabs", 1.2351710034868335], ["colin's", 1.2350500790073256], ["modified", 1.2349558635484867], ["salix", 1.2349236328620865], ["somewhere", 1.2348591572829228], ["counterpoint", 1.234818241470334], ["bulging", 1.2347912780184718], ["gloves", 1.234723820364359], ["niue", 1.23465259040722], ["delphine", 1.2346383558034755], ["weimar", 1.2346240527641357], ["charity", 1.2346054405537947], ["deductibles", 1.2344921625758263], ["frage", 1.234416338476411], ["hyphal", 1.234416338476411], ["landscaper", 1.2343854780679493], ["hydrometer", 1.2343386044500588], ["miseducation", 1.2343386044500588], ["fixings", 1.2342264282704918], ["carvers", 1.2342264282704918], ["host", 1.234206445783205], ["starting", 1.2341734116713392], ["impaired", 1.2341135327983925], ["holes", 1.234108561009876], ["terreno", 1.2340931928380667], ["proton's", 1.2340931928380667], ["tradestation", 1.2340931928380667], ["labine", 1.2340931928380667], ["ouseburn", 1.2340931928380667], ["skyliner", 1.2340931928380667], ["durmstrang", 1.2340931928380667], ["rye's", 1.2340931928380667], ["daco", 1.2340931928380667], ["henschke", 1.2340931928380667], ["zentangle", 1.2340931928380667], ["kawah", 1.2340931928380667], ["elmore's", 1.2340931928380667], ["ndcs", 1.2340931928380667], ["f\u00fcrstenberg", 1.2340931928380667], ["kappa's", 1.2340931928380667], ["roxie's", 1.2340931928380667], ["diaphoresis", 1.2340931928380667], ["dehydrator", 1.2340931928380665], ["einmal", 1.233918589952832], ["shepherding", 1.233918589952832], ["spookier", 1.233918589952832], ["slytherins", 1.233918589952832], ["demonstrates", 1.2339056924910885], ["eruption", 1.2338420178838874], ["convolutions", 1.2337343404993526], ["rojos", 1.2337343404993526], ["jonathan's", 1.2334993434821147], ["ari's", 1.233458601662076], ["avery", 1.2334191906050038], ["bahri", 1.2333335171268704], ["heartwrenching", 1.2333335171268704], ["vercingetorix", 1.2333335171268704], ["lothair", 1.2333335171268704], ["classica", 1.2333335171268704], ["tony", 1.2332925111531554], ["salem's", 1.2332908412266241], ["douro", 1.2332908412266241], ["overlords", 1.2331597469941298], ["virtually", 1.2331585224875627], ["fortran", 1.2329929808616376], ["fried", 1.2328212578859974], ["rigs", 1.2327456198114044], ["banknotes", 1.2327241512612257], ["carpeted", 1.2324445136906612], ["conspicuously", 1.232415900931845], ["shoplocal", 1.232372602816682], ["dancefloors", 1.232372602816682], ["gebiet", 1.232372602816682], ["downdrafts", 1.232372602816682], ["skittishness", 1.232372602816682], ["sandworms", 1.232372602816682], ["solfeggio", 1.232372602816682], ["dbpedia", 1.232372602816682], ["bassist's", 1.232372602816682], ["caderousse", 1.232372602816682], ["hackman's", 1.232372602816682], ["fische", 1.232372602816682], ["almendras", 1.232372602816682], ["sanuki", 1.232372602816682], ["fusee", 1.232372602816682], ["chumphon", 1.232372602816682], ["estrategia", 1.232372602816682], ["niharika", 1.232372602816682], ["anfisa", 1.232372602816682], ["ghadames", 1.232372602816682], ["accha", 1.232372602816682], ["mooed", 1.232372602816682], ["virgile", 1.232372602816682], ["epistasis", 1.232372602816682], ["clementi's", 1.232372602816682], ["vicenzo", 1.232372602816682], ["g\u00f6teborgs", 1.232372602816682], ["noble", 1.2323178307010012], ["snowflake", 1.2322249027294532], ["leeches", 1.232217378197344], ["lifted", 1.2321209119041492], ["monks", 1.2320815281447788], ["progressive", 1.232034591335666], ["tickling", 1.2318416157962753], ["vinnie", 1.2318416157962753], ["bora's", 1.2317899207349718], ["metalsmiths", 1.2317899207349718], ["symlink", 1.2317899207349718], ["troisi\u00e8me", 1.2317899207349718], ["ziehen", 1.2317899207349718], ["yennefer", 1.2317899207349718], ["enthusing", 1.2317899207349718], ["usno", 1.2317899207349718], ["lalaurie", 1.2317899207349718], ["neung", 1.2317899207349718], ["murgia", 1.2317899207349718], ["kokua", 1.2317899207349718], ["croc's", 1.2317899207349718], ["amp\u00e8re", 1.2317899207349718], ["barq's", 1.2317899207349718], ["swarmers", 1.2317899207349718], ["trist\u00e1n", 1.2317899207349718], ["sam\u0101dhi", 1.2317899207349718], ["skymaster", 1.2317899207349718], ["bentong", 1.2317899207349718], ["tungusic", 1.2317899207349718], ["ashhurst", 1.2317899207349718], ["ototoxic", 1.2317899207349718], ["robonaut", 1.2317899207349718], ["pachyrhinosaurus", 1.2317899207349718], ["xlri", 1.2317899207349718], ["lumpy", 1.231624631005724], ["whiff", 1.2316203111073323], ["polygon", 1.231565958931202], ["tunics", 1.231565958931202], ["concretely", 1.231565958931202], ["precipitation", 1.2315040198380378], ["liberally", 1.2314327860716066], ["absolve", 1.2314124703154739], ["sedans", 1.2314124703154739], ["bunsen", 1.2313647144426312], ["artichoke", 1.231347538081875], ["lsat", 1.231237251172546], ["oracle's", 1.2311542876960269], ["article", 1.2310671561961155], ["gabby's", 1.2309962416623337], ["afrikaans", 1.2309863426133072], ["unnoticeable", 1.230971455671361], ["slacked", 1.230971455671361], ["reshma", 1.230971455671361], ["falters", 1.23093403557604], ["daredevils", 1.23093403557604], ["roundtrip", 1.23093403557604], ["crested", 1.230781397084455], ["wuppertal", 1.230742480326184], ["yella", 1.230742480326184], ["kyle's", 1.2306835226025752], ["muddled", 1.2305431698030542], ["laci", 1.2305023354519737], ["uhura", 1.2305023354519737], ["teaspoons", 1.2303838150675224], ["clangs", 1.2303356233078941], ["wnba's", 1.2303356233078941], ["naacp's", 1.2303356233078941], ["georgina's", 1.2303356233078941], ["amigo", 1.2303073006646321], ["heroine's", 1.2302501833340536], ["seurat", 1.2302501833340536], ["feeder", 1.2302501833340533], ["brood", 1.2301340233696183], ["clam", 1.2301256262637552], ["brush", 1.229933954913226], ["codex", 1.2299078576110487], ["internalize", 1.229862871867622], ["stylized", 1.2298006947760212], ["amin's", 1.2298006947760212], ["levothyroxine", 1.2298006947760212], ["westerfield", 1.2298006947760212], ["ranil", 1.2298006947760212], ["lescaut", 1.2298006947760212], ["peptidoglycan", 1.2298006947760212], ["supported", 1.2297624084854832], ["easier", 1.2297359684236644], ["volodymyr", 1.2295117378458578], ["soybean", 1.229493726980292], ["parts", 1.2293489011484606], ["hercule", 1.2293444163606169], ["comments", 1.2293377397591654], ["alfalfa", 1.2293274057648405], ["downtown's", 1.2293103436218042], ["scarlett's", 1.2293103436218042], ["shannen", 1.2293103436218042], ["arial", 1.22925884555538], ["functions", 1.2292425324020595], ["calvinist", 1.2292329197553489], ["masdar", 1.2290659271022102], ["poznan", 1.2289941358681502], ["cystic", 1.2289247683552404], ["abbess", 1.2288106820026337], ["demotic", 1.2287732923576618], ["orontes", 1.2287732923576618], ["lagaan", 1.2287732923576618], ["curvaceous", 1.2287167606456468], ["leathery", 1.2286882914381574], ["vancouver's", 1.228496915098922], ["limited", 1.2284542839067614], ["titan", 1.228435717277344], ["lista", 1.228425788598511], ["paran\u00e1", 1.228425788598511], ["irritated", 1.2284180907304287], ["tremor", 1.2284040696135639], ["seti", 1.2283959049828592], ["grills", 1.2283809080392112], ["recover", 1.2282461558860884], ["niels", 1.2282072982110208], ["ccgs", 1.2281825359671055], ["resolver", 1.2281825359671055], ["rahman's", 1.2281825359671055], ["healthiness", 1.2281825359671055], ["note's", 1.2281825359671055], ["imam", 1.2280254994582234], ["telemundo", 1.22786443637219], ["conserves", 1.22786443637219], ["frisian", 1.22786443637219], ["we'd", 1.2276863005990377], ["peru's", 1.2276656241253685], ["cast", 1.2275994393028444], ["sengoku", 1.22759796550734], ["nailing", 1.2275917795765487], ["lonely", 1.2274158802018327], ["stigmatizing", 1.2272487597368709], ["interbreeding", 1.2272487597368709], ["rebuilt", 1.227200775346828], ["uppies", 1.2269555803867187], ["wanli", 1.2269555803867187], ["lycopersicum", 1.2269555803867187], ["witt's", 1.2269555803867187], ["beane's", 1.2269555803867187], ["jahres", 1.2269555803867187], ["yardi", 1.2269555803867187], ["mutabilis", 1.2269555803867187], ["kralove", 1.2269555803867187], ["kenard", 1.2269555803867187], ["playfields", 1.2269555803867187], ["ovolo", 1.2269555803867187], ["tucan", 1.2269555803867187], ["fars", 1.2269180243542612], ["counselor", 1.2269158087309524], ["pinks", 1.22682594958055], ["fiddle", 1.2267528615795502], ["cubbies", 1.2266492244365597], ["begin", 1.2266407642310575], ["coach", 1.226618197498691], ["hygienist", 1.226609959096469], ["dharavi", 1.2265704719182986], ["afrocentric", 1.2265704719182986], ["mazy", 1.2265704719182986], ["debe", 1.2265136731004276], ["electrostatics", 1.2265136731004276], ["dancewear", 1.2264372214596435], ["grigoriy", 1.2264372214596435], ["skippable", 1.2264372214596435], ["solidaridad", 1.2264372214596435], ["beefeaters", 1.2264372214596435], ["milfoil", 1.2264372214596435], ["theodotus", 1.2264372214596435], ["nyong", 1.2264372214596435], ["verloren", 1.2264372214596435], ["nail's", 1.2264372214596435], ["prosa", 1.2264372214596435], ["abandonware", 1.2264372214596435], ["axler", 1.2264372214596435], ["muzio", 1.2264372214596435], ["kamehameha", 1.2262876891005652], ["vitus", 1.2259932622844547], ["milton's", 1.2259932622844547], ["impulses", 1.2259558388026999], ["purer", 1.2259502087213738], ["dilation", 1.2258778855422858], ["geht", 1.2258778855422858], ["millipede", 1.2258778855422858], ["papercut", 1.2258778855422858], ["herp", 1.2258778855422858], ["carlisle's", 1.2258778855422858], ["laugh", 1.22579365673352], ["levitating", 1.2257606701269352], ["intruder", 1.2257437724060898], ["car's", 1.2257334779208175], ["bratislava", 1.2257310738194662], ["sergius", 1.2256415717330247], ["zendaya", 1.2256415717330247], ["forceful", 1.225615801304687], ["flexi", 1.2253975415544816], ["sullivan", 1.225281512516887], ["bevor", 1.2252725137469564], ["armillary", 1.2252725137469564], ["alchemic", 1.2252725137469564], ["disfellowshipped", 1.2252725137469564], ["syncretized", 1.2252725137469564], ["squelchy", 1.2252725137469564], ["booksmart", 1.2252725137469564], ["prespa", 1.2252725137469564], ["autorickshaw", 1.2252725137469564], ["demitasse", 1.2252725137469564], ["emmerdale's", 1.2252725137469564], ["apophenia", 1.2252725137469564], ["tricorne", 1.2252725137469564], ["noul", 1.2252725137469564], ["fouquet's", 1.2252725137469564], ["avobenzone", 1.2252725137469564], ["tepeyac", 1.2252725137469564], ["inuvialuit", 1.2252725137469564], ["olympia's", 1.2252725137469564], ["desai's", 1.2252725137469564], ["dmx's", 1.2252725137469564], ["charak", 1.2252725137469564], ["prayers", 1.2250810649166832], ["parties", 1.2249851431316188], ["scorsese", 1.224963880871957], ["fitbit", 1.224920828180439], ["impatiens", 1.2249208281804387], ["brined", 1.2249208281804387], ["manifestations", 1.2248942293112364], ["paid", 1.2248743001799884], ["raisin", 1.2247769991274262], ["ghee", 1.2246834404230396], ["disarmament", 1.224661947207167], ["playwrights", 1.2246014559913605], ["backlighting", 1.2245526121525103], ["falsies", 1.2245526121525103], ["tatra", 1.2245526121525103], ["symmetrically", 1.2245290231439547], ["margaritas", 1.2245029949478041], ["risk", 1.2244275406596303], ["sorkin's", 1.224416876516686], ["estilo", 1.224416876516686], ["choirboys", 1.224416876516686], ["minoring", 1.224416876516686], ["conduit", 1.2243503518419765], ["elephant", 1.224254005969904], ["rishikesh", 1.224156785464571], ["shootouts", 1.224156785464571], ["pursuer", 1.22397339668439], ["pula", 1.22397339668439], ["settle", 1.2238881914357518], ["buda", 1.2237668822681997], ["muse", 1.223685801590658], ["deet", 1.223476855752672], ["omsk", 1.223476855752672], ["soph", 1.223358025444227], ["frei", 1.223358025444227], ["twin's", 1.2233026222457555], ["morelia", 1.2233026222457555], ["bacillus", 1.2232548494279119], ["ptolemaic", 1.2231969937178289], ["education", 1.2231942013771073], ["misdiagnosed", 1.223115343828388], ["ba\u00f1o", 1.2231153438283877], ["naza", 1.2231153438283877], ["negocios", 1.2231153438283877], ["alf's", 1.2231153438283877], ["hilmi", 1.2231153438283877], ["rockfalls", 1.2231153438283877], ["rewatches", 1.2231153438283877], ["silversmithing", 1.2231153438283877], ["permadeath", 1.2231153438283877], ["jowar", 1.2231153438283877], ["stand", 1.2230224073208962], ["smart's", 1.2229914213011508], ["milliliter", 1.2229914213011508], ["floured", 1.2227809308840738], ["tethering", 1.2227809308840738], ["mitchell", 1.2226925757169331], ["faisal", 1.2224878034884405], ["hammerhead", 1.222458735531671], ["jamming", 1.2223444728133699], ["hard", 1.2222504007780388], ["mustards", 1.222172127831605], ["bataan", 1.2221410087164692], ["wacky", 1.222138643663719], ["fanaticism", 1.2221066559949565], ["conquers", 1.2221066559949565], ["staff's", 1.222032610465414], ["shouts", 1.2219974697800968], ["flags", 1.221828302595509], ["cambio", 1.2217916259238117], ["dortmund's", 1.2217916259238117], ["ibram", 1.2217916259238115], ["ioctl", 1.2217916259238115], ["comunes", 1.2217916259238115], ["psychoville", 1.2217916259238115], ["lushest", 1.2217916259238115], ["mamounia", 1.2217916259238115], ["mahseer", 1.2217916259238115], ["yibin", 1.2217916259238115], ["svidrigailov", 1.2217916259238115], ["lasta", 1.2217916259238115], ["quisiera", 1.2217916259238115], ["takie", 1.2217916259238115], ["trellising", 1.2217916259238115], ["bardini", 1.2217916259238115], ["garganta", 1.2217916259238115], ["kalila", 1.2217916259238115], ["mbuna", 1.2217916259238115], ["technomancer", 1.2217916259238115], ["carabaos", 1.2217916259238115], ["heliskiing", 1.2217916259238115], ["host's", 1.2216842156709833], ["finlandia", 1.2216320606076403], ["quantic", 1.2216320606076403], ["hermoso", 1.2216320606076403], ["rasht", 1.2216320606076403], ["saltaire", 1.2216320606076403], ["davion", 1.2216320606076403], ["urubamba", 1.2215859717198183], ["robbi", 1.2215859717198183], ["prussian", 1.2215024444738887], ["wrinkling", 1.2215024444738887], ["dostoevsky", 1.2214435371414971], ["sterling's", 1.2212796509832868], ["author's", 1.221218638143853], ["joseph's", 1.2211705144618035], ["austria's", 1.2211608610162616], ["sassou", 1.2211608610162616], ["khmers", 1.2211608610162616], ["playmat", 1.2211608610162616], ["refractometer", 1.2211608610162616], ["regenerations", 1.2211608610162616], ["greenberg", 1.2210735461436664], ["martian", 1.2208402779701881], ["mojave", 1.2207390054460927], ["principal's", 1.2207162944452656], ["headstones", 1.2207162944452656], ["fenice", 1.2207162944452656], ["whatcha", 1.2206478485472512], ["stomach", 1.2205408886641644], ["bottle", 1.22047888206767], ["panther's", 1.2204162507351544], ["prophecies", 1.2203714236919685], ["hearsay", 1.220287882384213], ["eternia", 1.220250905084059], ["hasten", 1.2201792099662516], ["paychecks", 1.2201792099662516], ["unimportant", 1.2201551991307191], ["flores", 1.2201187112360081], ["broodmare", 1.21994322709488], ["pearlescent", 1.21994322709488], ["farquaad", 1.21994322709488], ["tori's", 1.21994322709488], ["personification", 1.2199432270948798], ["binds", 1.2199119464993136], ["femi", 1.2199119464993133], ["untamable", 1.2197631948901804], ["bierstadt", 1.2197631948901804], ["chuuk", 1.2197631948901804], ["asthenosphere", 1.2197631948901804], ["transylvania", 1.2197131342524914], ["gesticulated", 1.219561497793051], ["benjamina", 1.219561497793051], ["operative's", 1.219561497793051], ["bromate", 1.219561497793051], ["cerb", 1.219561497793051], ["nefesh", 1.219561497793051], ["akhmed", 1.219561497793051], ["voysey", 1.219561497793051], ["bay's", 1.219525798434179], ["require", 1.2194593271752054], ["flatiron", 1.2193817805077882], ["sensitize", 1.2193817805077882], ["tyndale's", 1.2193817805077882], ["perspectival", 1.2193817805077882], ["lilah", 1.2193817805077882], ["ilva", 1.2193817805077882], ["interservice", 1.2193817805077882], ["zeno's", 1.2193817805077882], ["reinserting", 1.2193817805077882], ["porchlight", 1.2193817805077882], ["takashima", 1.2193817805077882], ["inaki", 1.2193817805077882], ["clima", 1.2193817805077882], ["bhavesh", 1.2193817805077882], ["palatial", 1.2192515185934087], ["memorialization", 1.2192515185934087], ["lindiwe", 1.2192515185934087], ["blaze's", 1.2192515185934087], ["flicked", 1.2192252739420244], ["callings", 1.2191989646486419], ["sport", 1.21917267593236], ["signori", 1.219075326177427], ["daria", 1.219075326177427], ["unhealthily", 1.219075326177427], ["lockets", 1.219075326177427], ["arrakis", 1.219075326177427], ["crabby", 1.219050426763085], ["deci", 1.2190129693832492], ["philanthropist", 1.2189721067870642], ["doja", 1.2189616806729824], ["tricorder", 1.2189616806729824], ["ant\u00f4nio", 1.2189616806729824], ["viewed", 1.2189389359736906], ["g\u00f6ttingen", 1.2189187534409542], ["insubordination", 1.2188713362271077], ["colonizing", 1.2187690258442168], ["inkscape", 1.2187140651532227], ["anaphora", 1.2187140651532227], ["cuckoos", 1.2186956833422853], ["atman", 1.2186956833422853], ["competency", 1.218577980456121], ["wrath", 1.2185500230839585], ["scratchy", 1.2184709604899333], ["gatecrashers", 1.218435055522922], ["slint", 1.218435055522922], ["annuus", 1.218435055522922], ["khruangbin", 1.218435055522922], ["oversensitivity", 1.218435055522922], ["deuxi\u00e8me", 1.218435055522922], ["patillo", 1.218435055522922], ["babi\u0161", 1.218435055522922], ["m\u00e1laga's", 1.218435055522922], ["broomball", 1.218435055522922], ["buttercup's", 1.218435055522922], ["parfaite", 1.218435055522922], ["steppingstones", 1.218435055522922], ["nabopolassar", 1.218435055522922], ["anssi", 1.218435055522922], ["weav", 1.218435055522922], ["equinor", 1.218435055522922], ["jamendo", 1.218435055522922], ["chillstep", 1.218435055522922], ["mokra", 1.218435055522922], ["kiva's", 1.218435055522922], ["saracino", 1.218435055522922], ["vi\u00f1oly", 1.218435055522922], ["fiind", 1.218435055522922], ["ugra", 1.218435055522922], ["cheeni", 1.218435055522922], ["raheem's", 1.218435055522922], ["redfield's", 1.218435055522922], ["lien's", 1.218435055522922], ["caam", 1.218435055522922], ["membrane's", 1.218435055522922], ["sandi's", 1.218435055522922], ["rubber's", 1.218435055522922], ["murgh", 1.218435055522922], ["castelvecchio", 1.218435055522922], ["mercan", 1.218435055522922], ["tatio", 1.218435055522922], ["lev's", 1.218435055522922], ["hellboy's", 1.218435055522922], ["staraya", 1.218435055522922], ["whernside", 1.218435055522922], ["imfc", 1.218435055522922], ["firebird's", 1.218435055522922], ["abcb", 1.218435055522922], ["xingtai", 1.218435055522922], ["laleh", 1.218435055522922], ["mangaung", 1.218435055522922], ["brummy", 1.218435055522922], ["helon", 1.218435055522922], ["algebraist", 1.218435055522922], ["zumo", 1.218435055522922], ["lilts", 1.218435055522922], ["sabalan", 1.218435055522922], ["pyun", 1.218435055522922], ["maddux's", 1.218435055522922], ["kristiansund", 1.218435055522922], ["staud", 1.218435055522922], ["wrington", 1.218435055522922], ["boetie", 1.218435055522922], ["eudemus", 1.218435055522922], ["kareem", 1.2184350555229218], ["tyrannosaurus", 1.2182642069523884], ["fracking", 1.218087551763771], ["notre", 1.2180563655289753], ["philharmonic's", 1.2180322670748152], ["alcal\u00e1", 1.2180322670748152], ["keyset", 1.2180322670748152], ["pyromaniacs", 1.2180322670748152], ["boutonnieres", 1.2180322670748152], ["moltisanti", 1.2180322670748152], ["polydipsia", 1.2180322670748152], ["pacino's", 1.2180322670748152], ["yanda", 1.2180322670748152], ["svein", 1.2180322670748152], ["anshuman", 1.2180322670748152], ["johanssen", 1.2180322670748152], ["pains", 1.2179996714778587], ["protestantism", 1.2179839151925131], ["hearth", 1.2179427585307914], ["ahmad", 1.2178774193419046], ["objectification", 1.2178774193419044], ["coleslaw", 1.2178774193419044], ["captioning", 1.2177354755867367], ["explorer", 1.217634029908608], ["joblessness", 1.2176145605360378], ["antigone", 1.2176145605360378], ["sacrosanct", 1.2175291558533883], ["taunts", 1.2174634167226643], ["swans", 1.2173998515675075], ["einsatz", 1.2173998515675075], ["repas", 1.2173998515675075], ["shimbashi", 1.2173998515675075], ["remingtons", 1.2173998515675075], ["reticulocyte", 1.2173998515675075], ["monumento", 1.2173998515675075], ["lodestones", 1.2173998515675075], ["toothfish", 1.2173998515675075], ["catulus", 1.2173998515675075], ["emei", 1.2173998515675075], ["guinness's", 1.2173998515675075], ["protista", 1.2173998515675075], ["waldensians", 1.2173998515675075], ["happyland", 1.2173998515675075], ["foucauldian", 1.2173998515675075], ["ingvild", 1.2173998515675075], ["picking", 1.217310470175808], ["medievalist", 1.2172788256353504], ["peste", 1.2172788256353504], ["vacantly", 1.2172788256353504], ["connelly's", 1.2172788256353504], ["esos", 1.2172788256353504], ["push", 1.2172593519952712], ["pancreas", 1.2172150151511205], ["affirmation", 1.2170600190966265], ["watchband", 1.2170329438364746], ["harvick's", 1.2170329438364746], ["deathadder", 1.2170329438364746], ["pluie", 1.2170329438364746], ["litt\u00e9raire", 1.2170329438364746], ["tsugaru", 1.2170329438364746], ["hradec", 1.2170329438364746], ["ebbing", 1.217003088986877], ["qu\u00e9b\u00e9cois", 1.217003088986877], ["indoctrinating", 1.2169581645465306], ["chez", 1.2169183829714088], ["premiered", 1.2168451011225008], ["roman", 1.216640352564269], ["hyperkalemia", 1.2165991413257504], ["enka", 1.2165991413257504], ["reconstructionist", 1.2165991413257504], ["thai", 1.216531063642547], ["kendo", 1.2164993246315354], ["high", 1.2164300577660758], ["tech's", 1.2164054100290507], ["angrier", 1.2163064404145998], ["hannahan", 1.2162631570282287], ["yoshizaki", 1.2162631570282287], ["eski\u015fehir", 1.2162631570282287], ["alda's", 1.2162631570282287], ["fieldy", 1.2162631570282287], ["k\u00fcnstler", 1.2162631570282287], ["alfar", 1.2162631570282287], ["belfast's", 1.2162631570282287], ["niepce", 1.2162631570282287], ["humbucker", 1.2162631570282287], ["malong", 1.2162631570282287], ["azzurro", 1.2162631570282287], ["theek", 1.2162631570282287], ["densifying", 1.2162631570282287], ["ropework", 1.2162631570282287], ["grief's", 1.2162631570282287], ["spontan", 1.2162631570282287], ["beagle's", 1.2162631570282287], ["alarma", 1.2162631570282287], ["saludo", 1.2162631570282287], ["bleibt", 1.2162631570282287], ["provencale", 1.2162631570282287], ["pickelhaube", 1.2162631570282287], ["gorak", 1.2162631570282287], ["nekos", 1.2162631570282287], ["fenves", 1.2162631570282287], ["devaraja", 1.2162631570282287], ["kcom", 1.2162631570282287], ["pandoran", 1.2162631570282287], ["n\u00e5gon", 1.2162631570282287], ["bharatha", 1.2162631570282287], ["roja's", 1.2162631570282287], ["xu\u00e2n", 1.2162631570282287], ["settimana", 1.2162631570282287], ["pidd", 1.2162631570282287], ["herdwick", 1.2162631570282287], ["clodoaldo", 1.2162631570282287], ["ag\u00fcero's", 1.2162631570282287], ["hillz", 1.2162631570282287], ["ishiro", 1.2162631570282287], ["arki", 1.2162631570282287], ["foire", 1.2162631570282287], ["mccartneys", 1.2162631570282287], ["muumuus", 1.2162631570282287], ["syna", 1.2162631570282287], ["morchella", 1.2162631570282287], ["d\u00e9tails", 1.2162631570282287], ["m\u00fcnch", 1.2162631570282287], ["taffarel", 1.2162631570282287], ["haca", 1.2162631570282287], ["knidos", 1.2162631570282287], ["slopers", 1.2162631570282287], ["dyllan", 1.2162631570282287], ["proffessor", 1.2162631570282287], ["destorm", 1.2162631570282287], ["dependability", 1.2161261132922256], ["charted", 1.216103122402304], ["unchanged", 1.2160022132963573], ["cronus", 1.2159409681124598], ["simmer", 1.2159100483697367], ["hold", 1.2158887049130456], ["steinem", 1.2158755034323632], ["arcade", 1.2158690605862865], ["teachers", 1.2158217640716051], ["souped", 1.2157766517654174], ["superhot", 1.2157766517654174], ["tickers", 1.2157766517654174], ["dizzy", 1.2156844147081558], ["prada's", 1.2156519695121342], ["maharashtra's", 1.2156519695121342], ["nosso", 1.2156519695121342], ["mta's", 1.2156519695121342], ["minshew", 1.2156519695121342], ["bitterns", 1.2156519695121342], ["rookeries", 1.2156519695121342], ["vipul", 1.2156519695121342], ["depeche", 1.2156101298298243], ["densely", 1.2155879691159621], ["nikolai", 1.2155843124754688], ["dessus", 1.2155621407993942], ["tiziano", 1.2155621407993942], ["malaita", 1.2155621407993942], ["rubdown", 1.2155621407993942], ["nasp", 1.2155621407993942], ["ashore", 1.215476811663866], ["wheezing", 1.2154752942544453], ["supple", 1.2154752942544453], ["detectives", 1.2154055355713498], ["polonium", 1.2153561133179027], ["anhui", 1.2153561133179027], ["proboscis", 1.2152702891449403], ["runic", 1.2152702891449403], ["kamchatka", 1.2152702891449403], ["gehen", 1.2152702891449403], ["bisher", 1.2152702891449403], ["supercooling", 1.2152702891449403], ["uraraka", 1.2152702891449403], ["chens", 1.2152702891449403], ["gozen", 1.2152702891449403], ["che's", 1.2152702891449403], ["micaiah", 1.2152702891449403], ["hydrofoils", 1.2152702891449403], ["alexisonfire", 1.2152702891449403], ["colder", 1.2152046225226885], ["furthering", 1.2150809947073788], ["tonight's", 1.2150708465373126], ["beaker", 1.2150295462191985], ["pumpernickel", 1.2150092774849008], ["printmakers", 1.2150092774849008], ["bioreactors", 1.2150092774849008], ["thoth", 1.2147428280819434], ["assyrian", 1.2147428280819432], ["mythos", 1.2146978792261403], ["embargoes", 1.2146075108458867], ["christina's", 1.2146075108458867], ["superpower", 1.2144314354061982], ["speculation", 1.2144306704702497], ["tumbler", 1.2143047671991154], ["sunburn", 1.214133672712893], ["riverboats", 1.214098987709673], ["colorists", 1.214098987709673], ["slushies", 1.214098987709673], ["tempus", 1.2140801205925371], ["besides", 1.214062489779704], ["facilitate", 1.2140497323678436], ["subsidies", 1.2139721595178992], ["portfolio", 1.213928966884315], ["diecast", 1.2139090993049482], ["freewheeling", 1.2139090993049482], ["mele", 1.2139090993049482], ["healer", 1.2138771862417141], ["norm's", 1.2138427328351376], ["bided", 1.2138427328351376], ["nonbeliever", 1.2138427328351376], ["custards", 1.2138427328351376], ["impoverishing", 1.2138427328351376], ["taxonomy", 1.2137090043984695], ["janeiro", 1.2136028110291701], ["firefighter", 1.213549954601012], ["taverns", 1.2135210364560196], ["hoarders", 1.2135210364560196], ["supercells", 1.213349951861694], ["canalside", 1.213349951861694], ["salesgirl", 1.213349951861694], ["elsinore", 1.213322740604644], ["aldrich", 1.21329547492508], ["fountains", 1.2132318550060968], ["overruns", 1.2131552610004417], ["disappear", 1.2131542697390443], ["eamonn", 1.2131070311144212], ["orangutan", 1.2130634789923065], ["hesitancy", 1.2128967143614542], ["normalization", 1.2126529593256885], ["clenching", 1.2125349131800516], ["meringue", 1.2124389260320623], ["tyrone", 1.2122644836352592], ["hints", 1.2122020749615412], ["flenderson", 1.2120691461419246], ["vivoactive", 1.2120691461419246], ["taong", 1.2120691461419246], ["della's", 1.2120691461419246], ["calabarzon", 1.2120691461419246], ["younguns", 1.2120691461419246], ["catholicism's", 1.2120691461419246], ["joosten", 1.2120691461419246], ["lakm\u00e9", 1.2120691461419246], ["fumiya", 1.2120691461419246], ["unknot", 1.2120691461419246], ["vintner's", 1.2120691461419246], ["elderslie", 1.2120691461419246], ["mariza", 1.2120691461419246], ["brainiac's", 1.2120691461419246], ["criddle", 1.2120691461419246], ["beauxbatons", 1.2120691461419246], ["brdc", 1.2120691461419246], ["marium", 1.2120691461419246], ["pouch", 1.211994932417691], ["offcuts", 1.2118858603191298], ["arnolfini", 1.2118858603191298], ["flink", 1.2118858603191298], ["escondida", 1.2118858603191298], ["tellis", 1.2118858603191298], ["supercomputers", 1.2118330594364042], ["motivational", 1.211833059436404], ["delaying", 1.211725750801601], ["irrelevant", 1.2116197062737524], ["boating", 1.211602709417186], ["attendance", 1.2115016511283492], ["hustles", 1.2113981044001159], ["brechin", 1.2113981044001159], ["trikes", 1.2113489624543774], ["meetinghouse", 1.2113489624543774], ["pinot", 1.2113387744900168], ["flanked", 1.2112411672185637], ["touches", 1.2112299926339087], ["salespeople", 1.2112253806381712], ["glissando", 1.2112253806381712], ["biwa", 1.2112253806381712], ["eelgrass", 1.2112253806381712], ["hanzi", 1.2112253806381712], ["gruppo", 1.2112253806381712], ["listings", 1.2112142862907411], ["guilty", 1.2112044585706923], ["vaud", 1.211050600640965], ["iglesia", 1.211050600640965], ["minimally", 1.2108991246433862], ["sapphire", 1.2108142980847423], ["development", 1.2107656475584612], ["matching", 1.2107552224456866], ["kittens", 1.2107301903952392], ["cornhuskers", 1.210694723297256], ["raphaelite", 1.210694723297256], ["chora", 1.21067756273648], ["azurite", 1.21067756273648], ["burren", 1.210661945120862], ["mainspring", 1.210661945120862], ["paulie's", 1.210661945120862], ["shopping", 1.21064854096806], ["winnings", 1.210643182440581], ["singapore's", 1.2106075333480475], ["johson", 1.210330166018335], ["drachms", 1.210330166018335], ["arm\u00e9", 1.210330166018335], ["despacho", 1.210330166018335], ["parda", 1.210330166018335], ["kabinet", 1.210330166018335], ["gaultier's", 1.210330166018335], ["huapi", 1.210330166018335], ["jetsetting", 1.210330166018335], ["guney", 1.210330166018335], ["xanny", 1.210330166018335], ["abenakis", 1.210330166018335], ["olman", 1.210330166018335], ["raquelle", 1.210330166018335], ["halc\u00f3n", 1.210330166018335], ["whiskys", 1.210330166018335], ["mladost", 1.210330166018335], ["chitrangada", 1.210330166018335], ["gatehouses", 1.210330166018335], ["ayala's", 1.210330166018335], ["brightstone", 1.210330166018335], ["squirtles", 1.210330166018335], ["dirt", 1.2102866915152453], ["zeus", 1.2101993666488085], ["presents", 1.2101771087016204], ["predator", 1.2101178273927178], ["exudes", 1.210107132203958], ["alumnus", 1.2100696398448378], ["charleston's", 1.2100058546126495], ["d'is\u00e8re", 1.2100058546126495], ["ludus", 1.2099746048649804], ["endanger", 1.2099385886116818], ["liqueur", 1.2098617719912381], ["seaman's", 1.2098617719912381], ["sejanus", 1.2098617719912381], ["chemistry", 1.209850626722647], ["shapeshifting", 1.2095737096645736], ["roofs", 1.2095737096645733], ["mesdames", 1.2095328470683888], ["collective's", 1.2095328470683888], ["wainwright's", 1.2095328470683888], ["cobbling", 1.2095328470683888], ["camargue", 1.2095328470683888], ["job's", 1.2094178568479694], ["sponge", 1.2093345216658262], ["wheat", 1.2093304570331678], ["climatic", 1.2091811183923682], ["knope", 1.209181118392368], ["unconvincingly", 1.2091505069871278], ["wheezed", 1.2091505069871278], ["shoguns", 1.2091505069871278], ["strigoi", 1.2091505069871278], ["trawlers", 1.2091368827560633], ["follow", 1.2091242211065414], ["dealbreaker", 1.20904803660518], ["tablet", 1.2090198755484267], ["exceptionally", 1.209009805837224], ["copying", 1.2089909698012373], ["deductions", 1.2089534154544892], ["spermatophores", 1.2087784606772858], ["sweetgreen", 1.2087784606772858], ["matsuhisa", 1.2087784606772858], ["grange's", 1.2087784606772858], ["pollocks", 1.2087784606772858], ["jabbawockeez", 1.2087784606772858], ["passings", 1.2087784606772858], ["pragmatists", 1.2087784606772856], ["excavation", 1.2087121899283453], ["marinate", 1.2086966870025004], ["stolen", 1.2086806790897437], ["cassettes", 1.2086144918139277], ["accessorize", 1.2085732351321792], ["anniversaries", 1.2085437008286402], ["overbearing", 1.2085437008286402], ["debacle", 1.2084250166829358], ["paring", 1.2084117752652799], ["aloha", 1.2083129490749709], ["sprints", 1.208225243990477], ["shiori", 1.2082252439904768], ["preciously", 1.2082252439904768], ["pasay", 1.2082252439904768], ["bonhomme", 1.2082252439904768], ["sanjana", 1.2082252439904768], ["grandstands", 1.2081122835151599], ["karel", 1.2080127942562648], ["suprachiasmatic", 1.207941356480141], ["graduate's", 1.207941356480141], ["seachange", 1.207941356480141], ["transcendentalists", 1.207941356480141], ["emulator", 1.2078289096476884], ["daydreamer", 1.207784399443176], ["quartier", 1.207784399443176], ["observer's", 1.2077251507603486], ["replicant", 1.2077251507603486], ["leuven", 1.2076524444120997], ["tantallon", 1.2076524444120997], ["stephan's", 1.2076524444120997], ["signy", 1.2076524444120997], ["leider", 1.2076524444120997], ["memetics", 1.2076524444120997], ["vegeta's", 1.2076524444120997], ["dulcis", 1.2076524444120997], ["leaf", 1.2076306193679236], ["hurricanes", 1.2076148858432543], ["eardrums", 1.2075689565141106], ["satisfactorily", 1.2075550006968432], ["powershell", 1.2075060622976257], ["immobility", 1.2073583731999864], ["paywall", 1.2073583731999864], ["bitte", 1.2073583731999864], ["saltier", 1.2073583731999864], ["unprompted", 1.2073583731999864], ["pragmatics", 1.207239269088987], ["capitalized", 1.2072208695573567], ["foolishness", 1.2071464714511997], ["astrology", 1.2069891504558885], ["marnie", 1.2068882253934101], ["aaas", 1.206798074094157], ["leptis", 1.2067980740941568], ["thomsons", 1.2067980740941568], ["maquiladora", 1.2067980740941568], ["piglet's", 1.2067980740941568], ["inquisition's", 1.2067980740941568], ["andalou", 1.2067980740941568], ["brijesh", 1.2067980740941568], ["strawberry", 1.2067971546289578], ["beatles", 1.2067541905278263], ["enjoys", 1.2066216631888487], ["squarepants", 1.2065219189284604], ["auguste", 1.2064068915741912], ["christel", 1.206406891574191], ["opioid", 1.2063852814818967], ["classically", 1.2063039909522555], ["mals", 1.2063039909522555], ["refugio", 1.2063039909522555], ["doorbells", 1.2063039909522555], ["tropical", 1.2061641454634473], ["sizable", 1.2061276307196602], ["nonhuman", 1.2061276307196602], ["blocky", 1.2061276307196602], ["craftsy", 1.20612763071966], ["kingside", 1.20612763071966], ["machinery's", 1.20612763071966], ["ncac", 1.20612763071966], ["perder", 1.20612763071966], ["brosnahan", 1.20612763071966], ["hoja", 1.20612763071966], ["andalusians", 1.20612763071966], ["shine's", 1.20612763071966], ["europeo", 1.20612763071966], ["houdin", 1.20612763071966], ["joliot", 1.20612763071966], ["arowana", 1.20612763071966], ["manawa", 1.20612763071966], ["changwon", 1.20612763071966], ["comuna", 1.20612763071966], ["relaxed", 1.2060465465091916], ["gestational", 1.2060209409690434], ["vocalists", 1.2060209409690434], ["outsized", 1.2059903367377909], ["logos", 1.205822667981172], ["dollywood", 1.2058055672668644], ["prone", 1.205746982751597], ["snapshot", 1.2056493993558763], ["bollywood's", 1.2054918227962446], ["emet", 1.2054918227962446], ["gabonese", 1.2054300438887062], ["immunosuppressants", 1.2054300438887062], ["intj", 1.2054300438887062], ["tamped", 1.2054300438887062], ["petrovsky", 1.2054300438887062], ["backgammon", 1.2053944224335083], ["asteroids", 1.205255205670315], ["panorama", 1.2051430367353992], ["ageless", 1.2051430367353992], ["phangan", 1.2051430367353992], ["embraer", 1.2051430367353992], ["rila", 1.2051430367353992], ["horus", 1.204980256558897], ["george", 1.2049722544334018], ["theft", 1.204841094580226], ["doctor's", 1.2047975651619744], ["substance", 1.2047573075627416], ["twofold", 1.204733263516516], ["other", 1.2046182006096648], ["afro", 1.2046159678367614], ["jackie's", 1.2044547768629061], ["phantasmal", 1.2044547768629061], ["answerer", 1.2044547768629061], ["refracted", 1.2044045912472037], ["relocations", 1.2044045912472037], ["taiping", 1.2041005254579467], ["vignettes", 1.2041005254579464], ["summarize", 1.2041005254579464], ["le's", 1.2041005254579464], ["canyonlands", 1.2041005254579464], ["granny", 1.2040108746109912], ["caithness", 1.2039928915502447], ["marksmanship", 1.2039043539810064], ["repair", 1.2038534867933672], ["outsell", 1.2038302447563844], ["kava", 1.2036935972641452], ["tongue", 1.2036888671586448], ["deteriorate", 1.203634722546744], ["scramble", 1.2036239488739682], ["stormtroopers", 1.203617242084293], ["heimlich", 1.2034768007620344], ["gentoo", 1.2034768007620344], ["haussmann", 1.2033707675637297], ["cast's", 1.2033707675637297], ["swabbing", 1.2033707675637297], ["crucible", 1.2033707675637293], ["remarkably", 1.2033075796985255], ["upheld", 1.2032750923642306], ["manson's", 1.2032212990793718], ["glinda", 1.2032212990793718], ["pans", 1.2032064512166873], ["iced", 1.203112311642861], ["flinched", 1.2031084074748721], ["mock", 1.2030904085987197], ["acquiescing", 1.2030895258563865], ["superficially", 1.2030895258563865], ["woodcutter", 1.20299483167883], ["linear", 1.2029948316788297], ["damning", 1.2028996580370832], ["schloss", 1.2028518902777272], ["weirder", 1.202780011173173], ["uncommon", 1.2027513104578422], ["verdun", 1.2025835232708435], ["outrun", 1.202500179198699], ["extreme", 1.2023421389741422], ["holly's", 1.202298654114201], ["chin", 1.2022542379855776], ["kilometer", 1.2021701981248565], ["valenciennes", 1.2021205621790632], ["fluff", 1.2021144754926727], ["freddie", 1.2019683632626692], ["safari", 1.2019471415077982], ["gloat", 1.2019130017942268], ["placard", 1.2018956039452051], ["agincourt", 1.2018200320385186], ["matsuda", 1.2018200320385186], ["moldovan", 1.2018200320385184], ["narcissus", 1.2017390959463885], ["rajah", 1.2017333018637304], ["shamefully", 1.2017333018637304], ["headscarf", 1.2017333018637304], ["overkill", 1.2016170561844333], ["thiruvananthapuram", 1.2016170561844333], ["here", 1.2013975272469766], ["hira", 1.2013292929482617], ["southern's", 1.20131467129939], ["shearer's", 1.20131467129939], ["orchards", 1.2012318220117142], ["rimsky", 1.2012046607983553], ["quicken", 1.2011774620695084], ["mal's", 1.2011748697549587], ["catman", 1.2011748697549587], ["shanta", 1.2011477478512884], ["squirrelly", 1.2011477478512884], ["bu\u00f1uel", 1.2011477478512884], ["scorn", 1.201073381600454], ["cartridges", 1.2010054433385051], ["radiologists", 1.2009951727165977], ["conservator", 1.2009627458145076], ["corgis", 1.2009349090218455], ["annuals", 1.2008441687217382], ["berated", 1.2008259708044087], ["asada", 1.2008259708044087], ["hendaye", 1.2005694388730257], ["3doodler", 1.2005694388730257], ["shinshu", 1.2005694388730257], ["unabsorbed", 1.2005694388730257], ["rebellion's", 1.2005694388730257], ["fathomed", 1.2005694388730257], ["catlike", 1.2005694388730257], ["skalds", 1.2005694388730257], ["merseybeat", 1.2005694388730257], ["kilopascals", 1.2005694388730257], ["merriman's", 1.2005694388730257], ["tealeaf", 1.2005694388730257], ["esteghlal", 1.2005694388730257], ["josey", 1.2005694388730257], ["troll's", 1.2005694388730257], ["monotonously", 1.2005694388730257], ["hohhot", 1.2005694388730257], ["encouragements", 1.2005694388730257], ["darenth", 1.2005694388730257], ["shaiva", 1.2005694388730257], ["t\u00e9l\u00e9vision", 1.2005694388730257], ["durley", 1.2005694388730257], ["skyways", 1.2005694388730257], ["catenaccio", 1.2005694388730257], ["prehypertension", 1.2005694388730257], ["ncate", 1.2005694388730257], ["crypsis", 1.2005694388730257], ["fantasma", 1.2005694388730257], ["enrobed", 1.2005694388730257], ["guillermo's", 1.2005694388730257], ["coreldraw", 1.2005694388730257], ["topologists", 1.2005694388730257], ["outdrink", 1.2005694388730257], ["umbellata", 1.2005694388730257], ["mithraism", 1.2005694388730257], ["sunrun", 1.2005694388730257], ["jrtc", 1.2005694388730257], ["schoodic", 1.2005694388730257], ["indominus", 1.2005694388730257], ["meroe", 1.2005694388730257], ["roughrider", 1.2005694388730257], ["klarna", 1.2005694388730257], ["moeraki", 1.2005694388730257], ["srivijaya", 1.2005694388730257], ["chandrakanta", 1.2005694388730257], ["monroe's", 1.2004399276066966], ["dubious", 1.2004399276066964], ["roadmap", 1.2003762437909082], ["templars", 1.2003748571163204], ["meters", 1.200307586279362], ["roosters", 1.2002958048128836], ["sprinklers", 1.2002868189486204], ["dancehall", 1.200259694435752], ["disinfectant", 1.200259694435752], ["initiate", 1.2002129061305726], ["casing", 1.200158529690078], ["marianna", 1.2000794105306287], ["albinism", 1.200008425116543], ["entrench", 1.200008425116543], ["listing", 1.199512478218707], ["streetcars", 1.1995093334082374], ["aurora's", 1.1995093334082374], ["jaime", 1.1994738183476374], ["reno's", 1.1994639421521482], ["fiction's", 1.1994639421521482], ["patsy's", 1.1994639421521482], ["guestbook", 1.1994639421521482], ["applicability", 1.1993880667314534], ["clinics", 1.1993695180400084], ["srdjan", 1.1993604062054801], ["cdata", 1.1993604062054801], ["huizhou", 1.1993604062054801], ["deceivingly", 1.1993604062054801], ["ilang", 1.1993604062054801], ["palmach", 1.1993604062054801], ["periodontist", 1.1993604062054801], ["sauternes", 1.1993010084040632], ["scrutinizes", 1.1993010084040632], ["trier's", 1.1993010084040632], ["aviation's", 1.1993010084040632], ["lookahead", 1.1993010084040632], ["kunti", 1.1993010084040632], ["thorvald", 1.1993010084040632], ["chah", 1.1993010084040632], ["dicaprio's", 1.1992354728298336], ["microchips", 1.1992354728298336], ["potato", 1.1991393031848594], ["energizing", 1.1991291670526125], ["skybar", 1.1990817444010642], ["trabalho", 1.1990817444010642], ["ibcs", 1.1990817444010642], ["valeurs", 1.1990817444010642], ["guilhem", 1.1990817444010642], ["taskmasters", 1.1990817444010642], ["snoek", 1.1990817444010642], ["centavo", 1.1990817444010642], ["uyghurs", 1.199037611837781], ["microorganism", 1.199037611837781], ["inflexible", 1.1990223133074678], ["acquisition", 1.1989400946345554], ["tangier", 1.1989379696115676], ["hoon", 1.198685228904136], ["ethiopia's", 1.1984696775087418], ["ignis", 1.1984696775087418], ["eastside", 1.1983790508435237], ["first", 1.1983372483457733], ["rethinking", 1.1982956331175842], ["whale", 1.1982395379724553], ["ballooning", 1.1981751408467833], ["wieder", 1.1981751408467833], ["filmmaker's", 1.1981751408467833], ["catalan", 1.1981348240413443], ["wagers", 1.1980539600487203], ["request", 1.1980200557689145], ["nato's", 1.198005293618377], ["redundancy", 1.1978504458854666], ["talkback", 1.1978095091284902], ["o'reillys", 1.19780950912849], ["melbournians", 1.19780950912849], ["grizzly's", 1.19780950912849], ["kingsfield", 1.19780950912849], ["allergenicity", 1.19780950912849], ["vanakkam", 1.19780950912849], ["himation", 1.19780950912849], ["burette", 1.19780950912849], ["fatberg", 1.19780950912849], ["nasugbu", 1.19780950912849], ["leprechaun's", 1.19780950912849], ["kuzey", 1.19780950912849], ["urewera", 1.19780950912849], ["\u00eatre", 1.1976862269641086], ["mustapha", 1.1976862269641086], ["distilled", 1.197644974855258], ["scripture", 1.1976206718301796], ["fourths", 1.1974375185977049], ["seismically", 1.1974375185977049], ["almanacs", 1.1974375185977049], ["waymo", 1.1974254111617983], ["garages", 1.1973975640592132], ["cat's", 1.197389359109344], ["sporadically", 1.197346756485025], ["curves", 1.19723636625958], ["verticality", 1.1970590019172567], ["brexit", 1.19695670425686], ["maintenance", 1.1968707831650987], ["deteriorated", 1.1968355249931975], ["klaas", 1.1968029465157772], ["theanine", 1.1967090869795434], ["forced", 1.196690676070513], ["amstel", 1.1965959229996872], ["lures", 1.1964342972925217], ["aider", 1.1964342972925215], ["capcom's", 1.1964342972925215], ["tripling", 1.1964250783504868], ["unacknowledged", 1.1962816908770506], ["rice", 1.1962766985695248], ["fighting", 1.196273170352187], ["agreements", 1.1962279857506364], ["americano", 1.1961825706772928], ["considerably", 1.1961753547267504], ["metacritic", 1.1961596337715639], ["nest's", 1.1961596337715639], ["hotfixes", 1.1961596337715639], ["kawhi's", 1.1961596337715639], ["lapsang", 1.1961596337715639], ["igtv", 1.1961596337715639], ["tutu's", 1.1961596337715639], ["noshing", 1.1961596337715639], ["email's", 1.1961596337715639], ["adventurousness", 1.1961596337715639], ["xiong", 1.1960544768806827], ["router", 1.1958330151373875], ["venison", 1.1958249204719813], ["propulsion", 1.1957893938849202], ["swamper", 1.1957478748614876], ["limu", 1.1957478748614876], ["zocdoc", 1.1957478748614876], ["kaffa", 1.1957478748614876], ["tiru", 1.1957478748614876], ["fritjof", 1.1957478748614876], ["carlita", 1.1957478748614876], ["kulas", 1.1957478748614876], ["unamplified", 1.1957478748614876], ["placida", 1.1957478748614876], ["hanazono", 1.1957478748614876], ["wirecard", 1.1957478748614876], ["shubha", 1.1957478748614876], ["chronicles", 1.1957341935356884], ["takeoff", 1.1957278567609038], ["reckoned", 1.1956000488879468], ["nolan's", 1.1955243108891576], ["rosales", 1.1954761139808372], ["gond", 1.195476113980837], ["eliya", 1.195476113980837], ["adia", 1.195476113980837], ["horner's", 1.195476113980837], ["seabiscuit", 1.195476113980837], ["games", 1.1954670915195995], ["cashier's", 1.1952701217151387], ["nonzero", 1.195158421683739], ["shankar", 1.1951218288537067], ["chievo", 1.1951218288537067], ["archive's", 1.1951218288537067], ["relievers", 1.195015784325858], ["headwear", 1.195015784325858], ["gush", 1.194708670902196], ["physicists", 1.1946866486225478], ["sonar", 1.1946866486225478], ["photosystems", 1.1946407009032824], ["edda", 1.1946407009032824], ["illegibility", 1.1946407009032824], ["vertigo's", 1.1946407009032824], ["metabolization", 1.1946407009032824], ["rewrapped", 1.1946407009032824], ["furat", 1.1946407009032824], ["jackfish", 1.1946407009032824], ["klaip\u0117da", 1.1946407009032824], ["thanos's", 1.1946407009032824], ["mulcahy's", 1.1946407009032824], ["dahshur", 1.1946407009032824], ["quartiers", 1.1946407009032824], ["caballero's", 1.1946407009032824], ["himmelman", 1.1946407009032824], ["abinbev", 1.1946407009032824], ["chenrezig", 1.1946407009032824], ["hotlines", 1.1944698915379832], ["lugar", 1.1944698915379832], ["misguide", 1.1944396167927114], ["obsessives", 1.1944396167927114], ["ecgs", 1.1944396167927114], ["downhill", 1.1944396167927112], ["liberty's", 1.1942112829662215], ["kadi", 1.1942112829662215], ["ravenscroft", 1.1942112829662215], ["bitterly", 1.1941458793111923], ["bayan", 1.1940663544124637], ["rotund", 1.1938996086570646], ["graver", 1.1938996086570646], ["markt", 1.193596434556339], ["billow", 1.193596434556339], ["croquettes", 1.1934774675041555], ["moonlighting", 1.1934774675041555], ["steeler", 1.1934774675041555], ["oblate", 1.1934774675041555], ["ruslan", 1.1934177578049607], ["donovan's", 1.1934177578049607], ["vionnet", 1.193407205084074], ["huddersfield's", 1.193407205084074], ["kesey", 1.1933279089242679], ["riverdance", 1.1933279089242679], ["lindbergh's", 1.193269127193864], ["redacting", 1.193269127193864], ["tidus", 1.193269127193864], ["mascots", 1.1932483457740244], ["saucony", 1.1931878085944916], ["junipers", 1.1931878085944916], ["broadsheets", 1.1931878085944916], ["peristalsis", 1.1931878085944916], ["erno", 1.1931878085944916], ["fakes", 1.1931541570446924], ["kebab", 1.1930518545361573], ["webbed", 1.1930142329646802], ["playwright", 1.1929903338439498], ["sauropods", 1.1928734809315322], ["hoplites", 1.192873480931532], ["dagda", 1.192873480931532], ["redding's", 1.192873480931532], ["hadza", 1.192873480931532], ["bellezza", 1.192873480931532], ["rasmussen's", 1.192873480931532], ["possibility", 1.192597236546474], ["helga", 1.192578032391363], ["moodle", 1.1925035976816307], ["haze", 1.1924743445576842], ["trimming", 1.1924243991504755], ["felicity", 1.192373411699351], ["paco", 1.1923362605307033], ["coldness", 1.192327543411655], ["entrapping", 1.192327543411655], ["glorification", 1.192327543411655], ["comedically", 1.192327543411655], ["yellow's", 1.192327543411655], ["bnsf", 1.192316497205027], ["confirmed", 1.1922902983408543], ["catalogued", 1.19214439017918], ["prevented", 1.1920743526808917], ["erbe", 1.1920244606604062], ["cmes", 1.1920244606604062], ["greenacres", 1.1920244606604062], ["blouse", 1.1919987070455156], ["chirag", 1.1919378938876641], ["tormentor", 1.1919378938876641], ["chemistries", 1.1919378938876641], ["misnomers", 1.191831670031155], ["fantasia's", 1.191831670031155], ["interspaced", 1.191831670031155], ["armando's", 1.191831670031155], ["vango", 1.191831670031155], ["jpl's", 1.191831670031155], ["maggiano's", 1.191831670031155], ["higgens", 1.191831670031155], ["muser", 1.191831670031155], ["miniato", 1.191831670031155], ["namita", 1.191831670031155], ["hikaru's", 1.191831670031155], ["likethis", 1.191831670031155], ["dumitrescu", 1.191831670031155], ["pattu", 1.191831670031155], ["britanny", 1.191831670031155], ["hillenburg", 1.191831670031155], ["martial", 1.1918279013934634], ["indesign", 1.1918012351059757], ["messily", 1.1915256012920614], ["clar", 1.1915256012920614], ["erecting", 1.1915256012920612], ["part", 1.1914874529453299], ["europ\u00e9enne", 1.1913792761537472], ["ordnung", 1.1913792761537472], ["faqih", 1.1913792761537472], ["saeko", 1.1913792761537472], ["iceni", 1.1913792761537472], ["gefilte", 1.1913792761537472], ["gluey", 1.1913792761537472], ["panama's", 1.191293516056126], ["amaka", 1.191293516056126], ["comunale", 1.191293516056126], ["fiorella", 1.191293516056126], ["modul", 1.191293516056126], ["wai's", 1.191293516056126], ["kunsthistorisches", 1.191293516056126], ["sba's", 1.191293516056126], ["ekgs", 1.191293516056126], ["zubaida", 1.191293516056126], ["densa", 1.191293516056126], ["pench", 1.191293516056126], ["astatine", 1.191293516056126], ["staples", 1.1912316765706799], ["disadvantaged", 1.191205774317414], ["binx", 1.1911973121058672], ["perilla", 1.1911973121058672], ["hijras", 1.1911973121058672], ["contortionists", 1.1911973121058672], ["tess's", 1.1911973121058672], ["tads", 1.1911973121058672], ["derbyshire's", 1.1911973121058672], ["sorcerer", 1.1911446787852544], ["twitching", 1.1910664668560167], ["dishing", 1.1909648833620419], ["marionette", 1.1909648833620416], ["natalie's", 1.1909648833620416], ["enneagram", 1.1909648833620416], ["toutes", 1.1909648833620416], ["guest", 1.190834295107287], ["incentivizes", 1.1906576168802663], ["visby", 1.1906576168802663], ["fantastique", 1.1906576168802663], ["bel\u00e9m", 1.1906576168802663], ["alise", 1.1906576168802663], ["determinant", 1.1905898944189175], ["electric", 1.190563169016463], ["upright", 1.190513760300162], ["earmuffs", 1.1902942950268152], ["amann", 1.1902324326342668], ["d'alembert", 1.1902324326342668], ["sheaffer", 1.1902324326342668], ["intan", 1.1902324326342668], ["reshoot", 1.1901579965846958], ["adornment", 1.1901238064670119], ["trivialise", 1.1900128730635906], ["ulvaeus", 1.1900128730635906], ["counterproposal", 1.1900128730635906], ["marchesi", 1.1900128730635906], ["jigar", 1.1900128730635906], ["contemporain", 1.1899521581210875], ["dahil", 1.1899521581210875], ["chiang's", 1.1899521581210875], ["gandalf", 1.1899310596785675], ["emptying", 1.1899310596785675], ["pharmacological", 1.1898912972292033], ["panna", 1.1898580386749023], ["electrolyte", 1.1896924849823811], ["needle", 1.1896658378958962], ["zoetis", 1.1896053344084319], ["sinologist", 1.1896053344084319], ["ecdysis", 1.1896053344084319], ["anaximenes", 1.1896053344084319], ["protego", 1.1896053344084319], ["nonslip", 1.1896053344084319], ["riffling", 1.1896053344084319], ["lark's", 1.1896053344084319], ["disciplinarians", 1.1896053344084319], ["pwrs", 1.1896053344084319], ["hyperrealism", 1.1896053344084319], ["ottman", 1.1896053344084319], ["havenhurst", 1.1896053344084319], ["atlantia", 1.1896053344084319], ["zhongli", 1.1896053344084319], ["skulking", 1.1895053675736078], ["tuolumne", 1.1895053675736078], ["vuelta", 1.1894886683082937], ["organics", 1.1894706722696038], ["coaster", 1.1894457955006104], ["alessandro", 1.1893641241102109], ["alchemists", 1.1893546801028254], ["password", 1.1893467245531257], ["iuds", 1.1892537269204295], ["captive", 1.18908295282446], ["exponent", 1.1890712803862507], ["spieler", 1.1888972359950936], ["slawomir", 1.1888972359950936], ["defun", 1.1888972359950936], ["sorella", 1.1888972359950936], ["kanani", 1.1888972359950936], ["shailendra", 1.1888972359950936], ["thinq", 1.1888972359950936], ["kete", 1.1888972359950936], ["macroeconomics", 1.188789321205587], ["mazu", 1.1887702306285735], ["raya", 1.1887372013691688], ["prevent", 1.1887330306126294], ["fei's", 1.1885877079461493], ["suburbanite", 1.1885877079461493], ["uncaused", 1.1885877079461493], ["parent's", 1.1885637444842956], ["warms", 1.1885637444842956], ["columbian", 1.1885268795066435], ["sporadic", 1.188420086602721], ["pageantry", 1.1883531182669493], ["alleviation", 1.188291952808048], ["consubstantial", 1.1881691911475767], ["chigger", 1.1881691911475767], ["spotify", 1.1881019870078058], ["elevators", 1.1879267076391793], ["grandson", 1.1878458898763065], ["budgets", 1.1878070219288848], ["paduan", 1.1877974235658657], ["buchla", 1.1877974235658657], ["mowgli's", 1.1877974235658657], ["vasilievich", 1.1877974235658657], ["leona's", 1.1877974235658657], ["greet", 1.1876843702211854], ["brunch", 1.1876049563907511], ["chaperones", 1.1874893049086506], ["kera", 1.1874649865595504], ["weisse", 1.1874649865595504], ["wail", 1.1874649865595501], ["westerns", 1.1873679110195403], ["qualifications", 1.1873035947438886], ["echidna", 1.1872297784411032], ["englischer", 1.1871659523146347], ["coulombs", 1.1871659523146347], ["usc's", 1.1871659523146347], ["obando", 1.1871659523146347], ["mawson's", 1.1871659523146347], ["barbari", 1.1871659523146347], ["untaught", 1.1871659523146347], ["celeb's", 1.1871659523146347], ["vanney", 1.1871659523146347], ["avalanche's", 1.1871659523146347], ["udi's", 1.1871659523146347], ["diff\u00e9rence", 1.1871659523146347], ["medinet", 1.1871659523146347], ["vibha", 1.1871659523146347], ["comedy", 1.1871359067565823], ["scissors", 1.187047235719403], ["honor", 1.1870405730533444], ["amos's", 1.186982747692364], ["stapleton's", 1.186982747692364], ["trudy's", 1.186982747692364], ["bookmarklet", 1.186982747692364], ["triangulum", 1.186982747692364], ["yathrib", 1.186982747692364], ["pringle's", 1.186982747692364], ["dalt", 1.186982747692364], ["favela", 1.1869606786629376], ["refrigerator", 1.1869413533506832], ["dwell", 1.1868962869303052], ["merged", 1.1867765720289931], ["shrines", 1.1866869922580112], ["darks", 1.1866497932049327], ["gaugin", 1.1866497932049327], ["taiaha", 1.1866497932049327], ["baciu", 1.1866497932049327], ["hoje", 1.1866497932049327], ["isopods", 1.1866497932049327], ["pietersen's", 1.1866497932049327], ["bh's", 1.1866497932049327], ["rione", 1.1866497932049327], ["musst", 1.1866497932049327], ["starscape", 1.1866497932049327], ["ibadat", 1.1866497932049327], ["crone's", 1.1866497932049327], ["piriformis", 1.1866497932049327], ["maniples", 1.1866497932049327], ["graveyard's", 1.1866497932049327], ["yuria", 1.1866497932049327], ["bodyboarders", 1.1866497932049327], ["sled's", 1.1866497932049327], ["nomnom", 1.1866497932049327], ["kuumba", 1.1866497932049327], ["unli", 1.1866497932049327], ["liliha", 1.1866497932049327], ["bellocq", 1.1866497932049327], ["3po's", 1.1866497932049327], ["khak", 1.1866497932049327], ["lamassu", 1.1866497932049327], ["megathread", 1.1866497932049327], ["tomen", 1.1866497932049327], ["sport1", 1.1866497932049327], ["finex", 1.1866497932049327], ["penderyn", 1.1866497932049327], ["skylift", 1.1866497932049327], ["adps", 1.1866497932049327], ["veintiuno", 1.1866497932049327], ["valmir", 1.1866497932049327], ["raghunandan", 1.1866497932049327], ["regent's", 1.1864536527432459], ["operational", 1.1864466909675997], ["ouro", 1.1863001100479544], ["dsps", 1.1863001100479544], ["thomases", 1.1863001100479544], ["kenia", 1.1863001100479544], ["lalande", 1.1863001100479544], ["hallucinating", 1.186238832237589], ["insulating", 1.186238832237589], ["macos", 1.186238832237589], ["elpis", 1.186220003348647], ["mixte", 1.186220003348647], ["frolicsome", 1.186220003348647], ["footgear", 1.186220003348647], ["marro", 1.186220003348647], ["tailfin", 1.186220003348647], ["ecovillage", 1.186220003348647], ["snowmageddon", 1.186220003348647], ["csny", 1.186220003348647], ["picky", 1.1862064620244077], ["blarney", 1.1860570610404948], ["oxfords", 1.1860570610404948], ["reconfiguring", 1.1860570610404948], ["promise", 1.186031549293534], ["randomization", 1.1859184219918635], ["robots", 1.1858603127898015], ["declarations", 1.185856578102523], ["printing", 1.1858336537707674], ["drafts", 1.185741353171857], ["charizard", 1.1856951272409706], ["freeloaders", 1.185625642060049], ["papier", 1.1853865935102286], ["extinct", 1.1852098561174946], ["diaghilev", 1.185205008554115], ["sintra", 1.185205008554115], ["misunderstand", 1.1851578737357196], ["hastened", 1.185143127647283], ["trainers", 1.185057319456476], ["esta", 1.1850129273577634], ["tripwires", 1.1849674968473853], ["unalaska", 1.1849674968473853], ["unwto", 1.1849674968473853], ["rigi", 1.1849674968473853], ["yolngu", 1.1849674968473853], ["mcgahee", 1.1849674968473853], ["bracelets", 1.1849209910586551], ["elapsed", 1.184894214998477], ["kits", 1.184833948880942], ["xm's", 1.1848316372398588], ["daybeds", 1.1848316372398588], ["sukiya", 1.1848316372398588], ["skloot", 1.1848316372398588], ["tatlong", 1.1848316372398588], ["rainproof", 1.1848316372398588], ["bowe's", 1.1848316372398588], ["arrhythmogenic", 1.1848316372398588], ["healthworks", 1.1848316372398588], ["jttf", 1.1848316372398588], ["breedings", 1.1848316372398588], ["tercio", 1.1848316372398588], ["dilmun", 1.1848316372398588], ["ohim", 1.1848316372398588], ["hablan", 1.1848316372398588], ["glaciologists", 1.1848316372398588], ["kaitaia", 1.1848316372398588], ["laddoos", 1.1848316372398588], ["joyousness", 1.1848316372398588], ["floatable", 1.1848316372398588], ["prosumers", 1.1848316372398588], ["cleto", 1.1848316372398588], ["yuri's", 1.1848316372398586], ["famously", 1.1847820155074786], ["rework", 1.1846340455477782], ["barbecued", 1.184589637313952], ["extravagantly", 1.184589637313952], ["bhaktapur", 1.1844814061913103], ["ar's", 1.1844814061913103], ["shrubberies", 1.1844814061913103], ["tutor's", 1.1844814061913103], ["shebeen", 1.1844814061913103], ["negreanu", 1.1844814061913103], ["devi", 1.18439082506713], ["lecter", 1.1843684011279114], ["meet", 1.1843417419575921], ["pinterest", 1.1843100713068315], ["stone", 1.1842016121015335], ["appellations", 1.1841980374338483], ["panjabi", 1.184198037433848], ["shish", 1.184198037433848], ["demonetization", 1.184198037433848], ["korner", 1.184198037433848], ["hawaiians", 1.1840628086982727], ["adenine", 1.1839405471017252], ["sympathy", 1.1839105570512778], ["lure", 1.183860659075605], ["lulu", 1.183776427095643], ["marcia", 1.1837614667288034], ["veneration", 1.1836653347629378], ["dahl's", 1.1835498721150557], ["salves", 1.1835018598692193], ["stirrers", 1.1834560587136516], ["gustatory", 1.1834123193464374], ["pozzo", 1.1834123193464374], ["librairie", 1.1834123193464374], ["messiaen", 1.1834123193464374], ["bathwater", 1.1833304930840796], ["f\u00e9lix", 1.1833304930840796], ["inserting", 1.1832278803115266], ["auscultation", 1.1832201695251499], ["overemphasized", 1.1832201695251499], ["c\u00e1ceres", 1.1832201695251499], ["nakia", 1.1832201695251499], ["stanford's", 1.1831537710869047], ["dearest", 1.183040212084918], ["moondance", 1.183035349130035], ["kwabena", 1.183035349130035], ["toni's", 1.183035349130035], ["enduringly", 1.183035349130035], ["opioids", 1.1829896720895277], ["turnover", 1.1829823243732274], ["nuneaton", 1.1826169986655786], ["pothole", 1.1825491958334111], ["celery", 1.1824172885053763], ["beetroot", 1.1823347030478237], ["jaren", 1.182328742978546], ["goslings", 1.182328742978546], ["amoebae", 1.182328742978546], ["tuva", 1.182328742978546], ["uni\u00f3n", 1.182328742978546], ["yoast", 1.182328742978546], ["edition's", 1.182328742978546], ["metamorphic", 1.1822701611010296], ["recheck", 1.1822701611010296], ["sirs", 1.1822701611010296], ["sahara", 1.1822659366708181], ["hogmanay", 1.182191031848563], ["fanciful", 1.1820826425831268], ["farmed", 1.182043798093412], ["chiapas", 1.1820355743791158], ["dine", 1.181971621028132], ["neurotransmitter", 1.1818623411075486], ["mikhail", 1.1816604702107756], ["dostana", 1.1815127811131365], ["infuser", 1.1815127811131365], ["laune", 1.1815127811131365], ["bogyoke", 1.1815127811131365], ["kippot", 1.1815127811131365], ["tuon", 1.1815127811131365], ["plisetskaya", 1.1815127811131365], ["madrasah", 1.1815127811131365], ["textview", 1.1815127811131365], ["patang", 1.1815127811131365], ["vida's", 1.1815127811131365], ["leehom", 1.1815127811131365], ["estoria", 1.1815127811131365], ["khodro", 1.1815127811131365], ["scorcho", 1.1815127811131365], ["subintervals", 1.1815127811131365], ["charan's", 1.1815127811131365], ["hisoka's", 1.1815127811131365], ["ramsden's", 1.1815127811131365], ["corgan's", 1.1815127811131365], ["zhongyang", 1.1815127811131365], ["hambourg", 1.1815127811131365], ["tinmouth", 1.1815127811131365], ["starla", 1.1815127811131365], ["kongens", 1.1815127811131365], ["lighttpd", 1.1815127811131365], ["l'entrec\u00f4te", 1.1815127811131365], ["werris", 1.1815127811131365], ["murabaha", 1.1815127811131365], ["olafson", 1.1815127811131365], ["alpino", 1.1815127811131365], ["vinte", 1.1815127811131365], ["beilinson", 1.1815127811131365], ["hamama", 1.1815127811131365], ["loehmann", 1.1815127811131365], ["atlantida", 1.1815127811131365], ["enset", 1.1815127811131365], ["nestmates", 1.1815127811131363], ["berk's", 1.1815127811131363], ["councilman's", 1.1815127811131363], ["beginning's", 1.1815127811131363], ["gorefest", 1.1815127811131363], ["absinthium", 1.1815127811131363], ["bombes", 1.1815127811131363], ["ditchburn", 1.1815127811131363], ["fanlights", 1.1815127811131363], ["caball\u00e9", 1.1815127811131363], ["cartola", 1.1815127811131363], ["vaut", 1.1815127811131363], ["premierleague", 1.1815127811131363], ["questbridge", 1.1815127811131363], ["shotoku", 1.1815127811131363], ["wobbegong", 1.1815127811131363], ["kooning's", 1.1815127811131363], ["mhfa", 1.1815127811131363], ["crociere", 1.1815127811131363], ["coracles", 1.1815127811131363], ["beckford's", 1.1815127811131363], ["smartpen", 1.1815127811131363], ["carrabba", 1.1815127811131363], ["noreaga", 1.1815127811131363], ["ambitiousness", 1.1815127811131363], ["kevi", 1.1815127811131363], ["donja", 1.1815127811131363], ["mouthpiece", 1.1813785182971008], ["lenny", 1.1812098291179793], ["resorts", 1.1812079424062656], ["stages", 1.1810955182350482], ["repairs", 1.1810401760006912], ["unbound", 1.1810121401041902], ["robbie's", 1.1809968366672356], ["elephantine", 1.1809081174585239], ["toyotas", 1.1809081174585239], ["salvation", 1.1808563851236293], ["brooch", 1.1808146144697513], ["subsisting", 1.180801026425719], ["wonton", 1.180801026425719], ["read", 1.1806111540686444], ["caricatured", 1.180559948225142], ["talkie", 1.1805296766810454], ["totaling", 1.1804198817906069], ["belton", 1.180365681325648], ["sont", 1.1802820386328103], ["clogged", 1.1801828059930355], ["verde", 1.1801494971349291], ["gymnast", 1.1801494971349291], ["boat's", 1.180130894819437], ["clapton", 1.1801117778153738], ["vera's", 1.1800719118678766], ["hannibal's", 1.1800719118678766], ["grays", 1.180027151136885], ["dystrophy", 1.180027151136885], ["vanir", 1.179994127152837], ["clari", 1.179994127152837], ["kosovars", 1.179994127152837], ["herodes", 1.179994127152837], ["ayer's", 1.179994127152837], ["horatio's", 1.179994127152837], ["showtunes", 1.179994127152837], ["macgregor's", 1.179994127152837], ["kudus", 1.179994127152837], ["floorplans", 1.179994127152837], ["calamba", 1.179994127152837], ["oxygenating", 1.179994127152837], ["fallout", 1.1798249057115464], ["unhygienic", 1.1798249057115464], ["renegotiated", 1.1798249057115464], ["dhana", 1.1798103131288238], ["poppier", 1.1798103131288238], ["pelican's", 1.1798103131288238], ["dodger's", 1.1798103131288238], ["kenjutsu", 1.1798103131288238], ["jsdf", 1.1798103131288238], ["consolatory", 1.1798103131288238], ["manjaro", 1.1798103131288238], ["taverne", 1.1798103131288238], ["amiens", 1.1798103131288236], ["uniformly", 1.1797595685992266], ["recalibrate", 1.1796666673926473], ["oxymoronic", 1.179640336135144], ["geographic's", 1.179640336135144], ["tuli", 1.179640336135144], ["aicha", 1.179640336135144], ["tirzah", 1.179640336135144], ["repaint", 1.179625844239123], ["fishy", 1.1795332832599625], ["auntie's", 1.1795068171384113], ["rajdhani", 1.1795068171384113], ["screenshots", 1.1794738873758173], ["growl", 1.1794579762764181], ["misread", 1.17937517575257], ["rurally", 1.1792665590958111], ["teetotalers", 1.1792665590958111], ["zarkon", 1.1792665590958111], ["recommitting", 1.1792665590958111], ["itcz", 1.1792665590958111], ["nocturna", 1.1792665590958111], ["mantel's", 1.1792665590958111], ["kobuk", 1.1792665590958111], ["jerkily", 1.1792665590958111], ["challinor", 1.1792665590958111], ["saturnia", 1.1792665590958111], ["wavers", 1.1792406411494576], ["levantine", 1.1789989241320447], ["speckles", 1.1789989241320447], ["malta's", 1.1789989241320447], ["resize", 1.1789697900279592], ["virtualization", 1.1788676032449728], ["helper", 1.1788003587751723], ["provolone", 1.1785589991603536], ["honored", 1.1785589991603536], ["lingcod", 1.1785589991603536], ["growth's", 1.1785589991603536], ["flowstone", 1.1785589991603536], ["leptospirosis", 1.1785589991603536], ["bhavani", 1.1785589991603536], ["champenoise", 1.1785589991603536], ["stream's", 1.1785589991603536], ["reforesting", 1.1785589991603536], ["oscura", 1.1785589991603536], ["dao's", 1.1785589991603536], ["arteta's", 1.1785589991603536], ["pertuan", 1.1785589991603536], ["ohel", 1.1785589991603536], ["patronages", 1.1785589991603536], ["paraglide", 1.1785589991603536], ["gypo", 1.1785589991603536], ["leucippus", 1.1785589991603536], ["rougarou", 1.1785589991603536], ["typee", 1.1785589991603536], ["yiren", 1.1785589991603536], ["mirror", 1.1784748914708505], ["reporting", 1.1784517436773418], ["christianity", 1.1783989232351706], ["bojack", 1.1783760215172607], ["buggers", 1.1782472935271455], ["aegon's", 1.1782124660820943], ["denny's", 1.178212466082094], ["bondi", 1.1780148814322093], ["audio", 1.1780122991785844], ["osiris", 1.177968242769797], ["diplodocus", 1.1779625624198877], ["judd's", 1.1779625624198877], ["leopard's", 1.1779625624198877], ["klang", 1.177862039373742], ["santi's", 1.177773816615943], ["novice's", 1.177773816615943], ["reinstating", 1.177773816615943], ["compusa", 1.177773816615943], ["pasar", 1.177773816615943], ["wanjiku", 1.177773816615943], ["brunhilde", 1.177773816615943], ["shrimpy", 1.177773816615943], ["readme", 1.177773816615943], ["barkcloth", 1.177773816615943], ["fakt", 1.177773816615943], ["aron's", 1.177773816615943], ["leymah", 1.177773816615943], ["ipeds", 1.177773816615943], ["pirjo", 1.177773816615943], ["nobilities", 1.177773816615943], ["matoaka", 1.177773816615943], ["budtenders", 1.177773816615943], ["boombastic", 1.177773816615943], ["inat", 1.177773816615943], ["murat's", 1.177773816615943], ["subshells", 1.177773816615943], ["virginals", 1.177773816615943], ["vestals", 1.177773816615943], ["compactum", 1.177773816615943], ["podo", 1.177773816615943], ["teku", 1.177773816615943], ["malinke", 1.177773816615943], ["fubotv", 1.177773816615943], ["eurotas", 1.177773816615943], ["eustacia", 1.177773816615943], ["sailed", 1.1777125938054627], ["handkerchiefs", 1.1776853453433334], ["undergraduate", 1.1776828773946977], ["guwahati", 1.1775076530415667], ["powercore", 1.1775076530415665], ["iolani", 1.1775076530415665], ["unplowed", 1.1775076530415665], ["petrarchan", 1.1775076530415665], ["waterpower", 1.1775076530415665], ["formicidae", 1.1775076530415665], ["aquel", 1.1775076530415665], ["solamente", 1.1775076530415665], ["rikku", 1.1775076530415665], ["mike", 1.1774383408410254], ["powdered", 1.1773427360033255], ["rhinos", 1.177326626557537], ["krishna", 1.177326626557537], ["indignant", 1.1771881832264655], ["additives", 1.1771650795871305], ["stirs", 1.177098736434464], ["roscommon", 1.1769685011857782], ["modules", 1.176949453305843], ["frangipani", 1.1768427701205946], ["leontes", 1.1768427701205946], ["riverwalk", 1.1768198565152572], ["unachievable", 1.1768198565152572], ["herm\u00e8s", 1.1768198565152572], ["bletchley", 1.1766705156167712], ["kedah", 1.1765580500955652], ["velocities", 1.1764989814884865], ["chippenham", 1.1764200536083387], ["istvan", 1.1764200536083387], ["tutors", 1.1763593743018983], ["sinclair's", 1.176334836654075], ["cross", 1.1763011282356464], ["logician", 1.1762616131970782], ["untouchable", 1.1761547580359955], ["heron", 1.1760374779811487], ["schadenfreude", 1.1760224800856591], ["lodge's", 1.1758620678121607], ["quicksand", 1.1758395464647853], ["embezzled", 1.1756540400497655], ["scores", 1.1756074133279817], ["beauty's", 1.1755794199852758], ["eminem's", 1.1754958178759864], ["climb", 1.175494895231021], ["omnivorous", 1.1752942927914884], ["utd's", 1.1752942927914884], ["sisera", 1.1752942927914884], ["pousada", 1.1752942927914884], ["rutabagas", 1.1752942927914884], ["ray's", 1.1752942927914882], ["languishing", 1.1751377121502957], ["hoosiers", 1.1751302371950632], ["nawab", 1.1751302371950632], ["migrate", 1.175100109513043], ["sun's", 1.175100109513043], ["actor's", 1.1750693991265837], ["subheading", 1.174973042472289], ["flexors", 1.174910209689269], ["viable", 1.1748862044953803], ["viewer", 1.174873040356796], ["castel", 1.1748618813519036], ["caden", 1.1747996403113574], ["monkees", 1.1747612795067757], ["afterthoughts", 1.1745215812248933], ["purism", 1.1745215812248933], ["nebraskan", 1.1745215812248933], ["liverpudlian", 1.1745215812248933], ["inevitably", 1.174340373550208], ["muscular", 1.1743176712281524], ["drowsiness", 1.1743176712281524], ["hares", 1.1742360411322128], ["fuel", 1.1741637716146127], ["crossbreed", 1.1741283262311792], ["pitti", 1.1741283262311792], ["vibranium", 1.1741283262311792], ["lestrade", 1.1741283262311792], ["tienes", 1.174003165978943], ["quarterback's", 1.174003165978943], ["inter's", 1.174003165978943], ["collards", 1.174003165978943], ["razing", 1.1738901180091807], ["hadiths", 1.1738901180091807], ["suppressed", 1.17388341466223], ["wrench", 1.1737875052366276], ["centroid", 1.173529586646156], ["schumacher's", 1.173529586646156], ["clausewitz", 1.173529586646156], ["tibidabo", 1.1735295866461557], ["breiter", 1.1735295866461557], ["xishan", 1.1735295866461557], ["chernihiv", 1.1735295866461557], ["keery", 1.1735295866461557], ["techno's", 1.1735295866461557], ["dreamier", 1.1735295866461557], ["counterproposals", 1.1735295866461557], ["recv", 1.1735295866461557], ["chelonian", 1.1735295866461557], ["wesker's", 1.1735295866461557], ["tulliver", 1.1735295866461557], ["ravenclaws", 1.1735295866461557], ["lev\u00e9e", 1.1735295866461557], ["kiriath", 1.1735295866461557], ["tachinid", 1.1735295866461557], ["ensi", 1.1735295866461557], ["muise", 1.1735295866461557], ["olaz\u00e1bal", 1.1735295866461557], ["valleyview", 1.1735295866461557], ["pawz", 1.1735295866461557], ["patthar", 1.1735295866461557], ["dhoop", 1.1735295866461557], ["laureles", 1.1735295866461557], ["liferay", 1.1735295866461557], ["kirstine", 1.1735295866461557], ["clairaut", 1.1735295866461557], ["prudent", 1.173490447249158], ["nikes", 1.1733898807429837], ["fringing", 1.1733276022078345], ["cytosine", 1.1733276022078345], ["hellbender", 1.172919960886859], ["jacobsen's", 1.172919960886859], ["baudelaire's", 1.172919960886859], ["sudeley", 1.172919960886859], ["malgorzata", 1.172919960886859], ["jazzman", 1.172919960886859], ["hostnames", 1.172919960886859], ["gable's", 1.172919960886859], ["avere", 1.172919960886859], ["biomimetics", 1.172919960886859], ["hardcourts", 1.172919960886859], ["contemporaneity", 1.172919960886859], ["lumbee", 1.172919960886859], ["serval", 1.172919960886859], ["yirrkala", 1.172919960886859], ["abijah", 1.172919960886859], ["matamata", 1.172919960886859], ["hermie", 1.1726955215525905], ["dera", 1.1726599475947095], ["india's", 1.1725616794495928], ["following", 1.1725111306120308], ["indentation", 1.1724554176881272], ["recruiter", 1.1724242212584202], ["sewn", 1.1723971361462702], ["supplementation", 1.172363018835156], ["bolsonaro", 1.1723560570595097], ["hollandaise", 1.1723560570595097], ["isfahan", 1.1723359828132287], ["bottled", 1.1723359828132285], ["ghana's", 1.1722294664043906], ["relaxing", 1.1721529380486506], ["folktale", 1.1721421625043082], ["occurrence", 1.1721348995312961], ["glowingly", 1.1719458760029087], ["figura", 1.1718404472627142], ["impassible", 1.1718404472627142], ["maat", 1.1718404472627142], ["sculpt", 1.1717760672275765], ["gertrude", 1.1717425792693164], ["assorted", 1.1717390104630907], ["prestige", 1.1717158957750808], ["courthouse", 1.1717007093682656], ["birchwood", 1.1716668412705271], ["epicurus", 1.171666841270527], ["label", 1.171641219865732], ["pavements", 1.171453461769294], ["cadillac's", 1.171453461769294], ["lofi", 1.171453461769294], ["autofill", 1.171453461769294], ["borodino", 1.171453461769294], ["amna", 1.171453461769294], ["pria", 1.171453461769294], ["degenerative", 1.1712614202181841], ["shutter", 1.1711647635434537], ["horrific", 1.1711154686393408], ["lavigne's", 1.1710569157935513], ["weyden", 1.1710569157935513], ["tautologies", 1.1710569157935513], ["herstory", 1.1710227143649357], ["skis", 1.1709553927141578], ["rehabilitation", 1.1708767474285], ["archibald", 1.1707161988029646], ["irena", 1.1705552029012], ["arms", 1.1705449105762076], ["archipelago", 1.1704430001499038], ["arduino", 1.1702779933330303], ["kisumu", 1.1702004246982234], ["taken", 1.1701183601894856], ["fozzie", 1.1701155517133313], ["sensitizing", 1.1701155517133313], ["jpegs", 1.1701155517133313], ["misconduct", 1.1700078973583148], ["olsen's", 1.169985827151057], ["dukedom", 1.169985827151057], ["vastu", 1.1699499336111394], ["testudo", 1.1699499336111394], ["dominate", 1.169936825929761], ["duchamp", 1.1699219911718366], ["capes", 1.169874880219172], ["flyway", 1.1698531155100464], ["mamluk", 1.1698531155100464], ["predator's", 1.1698531155100464], ["walthamstow", 1.1698324266230447], ["postoperative", 1.1696976533020051], ["locks", 1.1696569582578795], ["dysmorphia", 1.1695783085766402], ["funtime", 1.1695783085766402], ["overboard", 1.1695658860773457], ["smes", 1.1695461773044264], ["blackened", 1.1694799743863447], ["photos", 1.169474808805567], ["castle", 1.1693645672520105], ["eeyore", 1.169358463029915], ["cannonballs", 1.1692902351016212], ["hominids", 1.1692902351016212], ["tester", 1.169290235101621], ["rijn", 1.1691409195308], ["authenticators", 1.1691409195308], ["commerciales", 1.1691409195308], ["icsi", 1.1691409195308], ["cucurbits", 1.1691409195308], ["wei\u00dfbier", 1.1691409195308], ["epididymitis", 1.1691409195308], ["jianghu", 1.1691409195308], ["reoccurred", 1.1691409195308], ["swea", 1.1691409195308], ["whannell", 1.1691409195308], ["sinofsky", 1.1691409195308], ["asamblea", 1.1691409195308], ["phacoemulsification", 1.1691409195308], ["altamura", 1.1691409195308], ["metalsmithing", 1.1691409195308], ["scoti", 1.1691409195308], ["rinnegan", 1.1691409195308], ["pamplona", 1.1690847648276523], ["nevis", 1.1690160113129013], ["ares", 1.168977218679375], ["lotr", 1.1688188421424994], ["expreso", 1.1686702508836457], ["doru", 1.1686702508836457], ["mizuki's", 1.1686702508836457], ["roster's", 1.1686702508836457], ["saler", 1.1686702508836457], ["papoulias", 1.1686702508836457], ["demeco", 1.1686702508836457], ["adultos", 1.1686702508836457], ["cdfis", 1.1686702508836457], ["taare", 1.1686702508836457], ["insomnium", 1.1686702508836457], ["ahwazi", 1.1686702508836457], ["ustaad", 1.1686702508836457], ["nippori", 1.1686702508836457], ["administraci\u00f3n", 1.1686702508836457], ["violino", 1.1686702508836457], ["babban", 1.1686702508836457], ["konzerthaus", 1.1686702508836457], ["etfe", 1.1686702508836457], ["yoginis", 1.1686702508836457], ["nvgs", 1.1686702508836457], ["taani", 1.1686702508836457], ["plaisirs", 1.1686702508836457], ["zorawar", 1.1686702508836457], ["tour\u00e9's", 1.1686702508836457], ["mcpd", 1.1686702508836457], ["meridor", 1.1686702508836457], ["dairy", 1.1685737752907024], ["poets", 1.1685195552977161], ["parameters", 1.1685102329569863], ["bash", 1.1683807969515385], ["julie's", 1.1683211335570192], ["porto's", 1.1682207623256138], ["greco", 1.168210180711658], ["pentecostalism", 1.168210180711658], ["real's", 1.168210180711658], ["callback", 1.168204006674342], ["l'arm\u00e9e", 1.168086499509578], ["immigrant", 1.1680703879716539], ["raincoat", 1.1680696816203042], ["liftoff", 1.1680696816203042], ["bundling", 1.1680696816203042], ["dabble", 1.1680696816203042], ["omaha's", 1.1679477664046045], ["jericho's", 1.1679477664046045], ["bakehouse", 1.1679477664046045], ["sprach", 1.1679477664046045], ["tallinn", 1.167742480906213], ["toolbag", 1.1676126307470998], ["gnarl", 1.1676126307470998], ["howlin", 1.1676126307470998], ["ca\u00f1o", 1.1676126307470998], ["cifar", 1.1676126307470998], ["witchers", 1.1676126307470998], ["wand's", 1.1676126307470998], ["tuguegarao", 1.1676126307470998], ["oberndorf", 1.1676126307470998], ["scvs", 1.1676126307470998], ["cipp", 1.1676126307470998], ["berm\u00fadez", 1.1676126307470998], ["darbari", 1.1676126307470998], ["lleva", 1.1676126307470998], ["osim", 1.1676126307470998], ["vijayakumar", 1.1676126307470998], ["langton's", 1.1676126307470998], ["rydges", 1.1676126307470998], ["esfp", 1.1676126307470998], ["thinsulate", 1.1676126307470998], ["myuran", 1.1676126307470998], ["distributism", 1.1676126307470998], ["starveling", 1.1676126307470998], ["mahna", 1.1676126307470998], ["trocad\u00e9ro", 1.1676126307470998], ["ruza", 1.1676126307470998], ["heartmate", 1.1676126307470998], ["tr\u00e8s", 1.1676126307470995], ["wilson's", 1.167544429074743], ["blogger", 1.167519864913089], ["circumference", 1.1674676457350035], ["orchestras", 1.1674561981867102], ["sylvie", 1.1673918265431005], ["region", 1.1673440798420276], ["cornbread", 1.167334627739779], ["t\u00fcrk\u00e7e", 1.1672932618656042], ["diminuendo", 1.1672932618656042], ["suleiman's", 1.1672932618656042], ["sarnath", 1.1672932618656042], ["multiline", 1.1672932618656042], ["boson", 1.167231887497943], ["oromo", 1.1672202877932196], ["readjustments", 1.1672202877932194], ["christiansted", 1.1672202877932194], ["underperforms", 1.1672202877932194], ["rosenbluth", 1.1672202877932194], ["duped", 1.1671869386421398], ["violas", 1.1670720693439758], ["jeweler's", 1.167048384952602], ["breastmilk", 1.1669461851816048], ["valley", 1.1669329133942004], ["alpine", 1.1668076220388506], ["padang", 1.1666460639997924], ["kairos", 1.1666460639997924], ["roam", 1.166497722853157], ["exhibited", 1.166393007747096], ["formentor", 1.1663651813552758], ["kahili", 1.1663651813552758], ["cornhusker", 1.1663651813552758], ["okwu", 1.1663651813552758], ["mahakala", 1.1663651813552758], ["thermokarst", 1.1663651813552758], ["fertilizations", 1.1663651813552758], ["deniro's", 1.1663651813552758], ["duvall's", 1.1663651813552758], ["kinesio", 1.1663651813552758], ["sglt2", 1.1663651813552758], ["dragoness", 1.1663651813552758], ["kumarakom", 1.1663651813552758], ["metamask", 1.1663651813552758], ["puuc", 1.1663651813552758], ["wuornos", 1.1663651813552758], ["cossington", 1.1663651813552758], ["weathersby", 1.1663651813552758], ["shediac", 1.1663651813552758], ["manzanas", 1.1663651813552758], ["dynos", 1.1663651813552758], ["regla", 1.1663651813552758], ["traceback", 1.1663651813552758], ["taho", 1.1663651813552758], ["inator", 1.1663651813552758], ["acaso", 1.1663651813552758], ["syllabics", 1.1663651813552758], ["facta", 1.1663651813552758], ["mbare", 1.1663651813552758], ["hunyadi", 1.1663651813552758], ["d'int\u00e9r\u00eat", 1.1663651813552758], ["phalaropes", 1.1663651813552758], ["maxent", 1.1663651813552758], ["omi's", 1.1663651813552758], ["melodicism", 1.1663651813552758], ["diaconal", 1.1663651813552758], ["loiterers", 1.1663651813552758], ["acoustica", 1.1663651813552758], ["rowell's", 1.1663651813552758], ["gatwick's", 1.1663651813552758], ["ariq", 1.1663651813552758], ["harzianum", 1.1663651813552758], ["gebracht", 1.1663651813552758], ["tineye", 1.1663651813552758], ["cashbacks", 1.1663651813552758], ["colchagua", 1.1663651813552758], ["nagaraja", 1.1663651813552758], ["limburgish", 1.1663651813552758], ["anakinra", 1.1663651813552758], ["senthil", 1.1663651813552758], ["dementieva", 1.1663651813552758], ["pular", 1.1663651813552758], ["saiyaman", 1.1663651813552758], ["tera's", 1.1663651813552758], ["franciscana", 1.1663651813552758], ["looey", 1.1663651813552758], ["namine", 1.1663651813552758], ["tethys", 1.1663651813552756], ["mitm", 1.166193328521718], ["rousers", 1.166193328521718], ["oecd's", 1.166193328521718], ["agnostic", 1.166153114958666], ["nitrogen", 1.1661531149586657], ["phonograph", 1.1661531149586657], ["koala", 1.1661531149586657], ["vacuoles", 1.1661531149586657], ["lian", 1.1661531149586657], ["lysosomes", 1.1661531149586657], ["decorated", 1.16609614189689], ["mato", 1.1659960784497834], ["conwy", 1.1659960784497834], ["dinnerware", 1.1658541779899503], ["enzo", 1.1658244822645814], ["loud", 1.1658237552010988], ["effusive", 1.1658011749813129], ["winston's", 1.1658011749813129], ["henderson's", 1.1657790681988158], ["altitude", 1.1657356092403397], ["nippur", 1.1656535491518862], ["likelihoods", 1.1656535491518862], ["io's", 1.1656535491518862], ["pierluigi", 1.1656535491518862], ["recitatives", 1.1656535491518862], ["snake's", 1.165628394780951], ["snowboarder", 1.165628394780951], ["chester's", 1.165607801367383], ["lelouch", 1.165607801367383], ["gigabytes", 1.1655095820605952], ["viollet", 1.165479726672144], ["refractors", 1.165479726672144], ["parlez", 1.165479726672144], ["wayanad", 1.165479726672144], ["yaku", 1.165479726672144], ["udaipur", 1.1654012431888663], ["sinkers", 1.165292591199011], ["tullamore", 1.165292591199011], ["bloatware", 1.165292591199011], ["orsino", 1.165292591199011], ["zeeland", 1.165225572214614], ["expensive", 1.1652163441107832], ["chits", 1.165193543804944], ["pyres", 1.165193543804944], ["pasi", 1.165193543804944], ["glazing", 1.1651801044330432], ["previously", 1.1651437604748005], ["cringeworthy", 1.1651153603866613], ["gerais", 1.1651153603866613], ["primi", 1.1650743424193883], ["rocketman", 1.1650743424193883], ["angiosperm", 1.1650743424193883], ["noun", 1.1649948260160021], ["marauders", 1.1649716021775527], ["penne", 1.1648914996161213], ["vultures", 1.1647746833807002], ["antonius", 1.1646340270972346], ["trinidad's", 1.1646340270972344], ["canibus", 1.1646340270972344], ["adri\u00e0", 1.1646340270972344], ["agroecosystems", 1.1646340270972344], ["ina's", 1.1646340270972344], ["raquel's", 1.1646340270972344], ["linton's", 1.1646340270972344], ["conglomerate's", 1.1646340270972344], ["spotlight's", 1.1646340270972344], ["sangoma", 1.1646340270972344], ["thale", 1.1646340270972344], ["fixins", 1.1646340270972344], ["izotope", 1.1646340270972344], ["nway", 1.1646340270972344], ["wormlike", 1.1646340270972344], ["cuhk", 1.1646340270972344], ["demos", 1.1644726366205884], ["embattled", 1.1644660792960813], ["drying", 1.1643486227111475], ["potstickers", 1.1643062843008092], ["minify", 1.1643062843008092], ["kubitschek", 1.1643062843008092], ["masqueraders", 1.1643062843008092], ["direcci\u00f3n", 1.1643062843008092], ["lil's", 1.1643062843008092], ["albena", 1.1643062843008092], ["plethora", 1.164299362146919], ["lending", 1.1642445173891751], ["most", 1.1641321964222107], ["rounding", 1.1641276644767575], ["answer", 1.1640994381820349], ["prospered", 1.164028177614296], ["rod's", 1.1640088880596087], ["sangam", 1.1640088880596087], ["kaposi's", 1.163938792378044], ["forwardness", 1.163938792378044], ["remigius", 1.163938792378044], ["justine's", 1.163938792378044], ["tenley", 1.163938792378044], ["authoritatively", 1.1639228436415083], ["minhaj", 1.1638254640306045], ["underflow", 1.1638254640306045], ["caliphates", 1.1638254640306045], ["bonzi", 1.1638254640306045], ["stoves", 1.1637900893964395], ["stiffened", 1.1637378100698414], ["unesco", 1.163706884271009], ["tying", 1.163666758208432], ["geophysicist", 1.1636527041893332], ["giallo", 1.1636381550751163], ["segways", 1.1636381550751163], ["zipcode", 1.1636381550751163], ["giada", 1.1635863802962476], ["volcano", 1.1635640608644005], ["coinciding", 1.1634897047910715], ["handel", 1.1634671678967783], ["sdcc", 1.163422633268342], ["journey's", 1.1633533434267673], ["junkyard", 1.1633275754963597], ["overstated", 1.163324981544885], ["raman", 1.163184185406125], ["troubles", 1.1631310060129116], ["sketchup", 1.163051643908244], ["fifths", 1.1630516439082437], ["lillet", 1.1630516439082437], ["lemmy", 1.1630516439082437], ["melon's", 1.1630516439082437], ["kufi", 1.1630516439082437], ["dissociatives", 1.1630516439082437], ["kolo", 1.1630516439082437], ["thabit", 1.1630516439082437], ["roark's", 1.1630516439082437], ["algonquins", 1.1630516439082437], ["fitzwalter", 1.1630516439082437], ["sippers", 1.1630516439082437], ["militare", 1.1630516439082437], ["dreamhost", 1.1630516439082437], ["khaos", 1.1630516439082437], ["surreality", 1.1630516439082437], ["hinamatsuri", 1.1630516439082437], ["ducky", 1.1628897717239837], ["crystalline", 1.162873398445576], ["emerges", 1.1628611358584635], ["skier", 1.162855348272141], ["coloration", 1.1627475781189867], ["locale", 1.1626456922000734], ["rotational", 1.162504325487581], ["decline", 1.162495603281674], ["motivating", 1.1624143553362392], ["floods", 1.1624087187319538], ["lidia", 1.1621869214964162], ["samsungs", 1.1621437191276753], ["toka", 1.1621437191276753], ["shielded", 1.1620701657277306], ["enhancers", 1.1620701657277304], ["cinematographic", 1.1620701657277304], ["chaudhary", 1.1620701657277304], ["aeneas", 1.1619836442352334], ["astrometry", 1.1619836442352334], ["overreactions", 1.1619836442352334], ["millwall's", 1.1619836442352334], ["braylon", 1.1619836442352334], ["religiosa", 1.1619836442352334], ["serpentina", 1.1619836442352334], ["liquid's", 1.1619836442352334], ["swingline", 1.1619836442352334], ["ye've", 1.161880393511559], ["crohn", 1.161880393511559], ["deirdre's", 1.161880393511559], ["asare", 1.161880393511559], ["merrivale", 1.161880393511559], ["diary's", 1.161880393511559], ["adley", 1.161880393511559], ["khmer", 1.1618752238546353], ["phonology", 1.1618639908801829], ["alle", 1.1618513842138078], ["anos", 1.1618209014279175], ["paderborn", 1.1618209014279175], ["cleaned", 1.1616633663904359], ["masterminding", 1.1616418843398701], ["recuperated", 1.1616418843398701], ["incremented", 1.1616418843398701], ["taboo", 1.1615271920360024], ["ancona", 1.1615070866122168], ["shirley's", 1.1614019252644023], ["wails", 1.1614019252644023], ["galois", 1.1614019252644023], ["guild's", 1.1614019252644023], ["ryman", 1.1612813978748977], ["oiling", 1.1612813978748977], ["storyteller", 1.1609839965412958], ["esteem", 1.1609032697549286], ["stoles", 1.1607844867076427], ["bookstagram", 1.1607844867076427], ["ally", 1.1607378480452304], ["grassy", 1.1607344528935606], ["whitened", 1.1605504575772583], ["fibber", 1.1604977316466973], ["gateau", 1.1604977316466973], ["areas", 1.160469284324417], ["windshield", 1.1604570280489412], ["raghavan", 1.1603150518049303], ["galle", 1.1602914350445996], ["sporran", 1.1602625512370008], ["huesca", 1.1602625512370008], ["enrichments", 1.1602625512370008], ["alana's", 1.1602625512370008], ["bhai's", 1.1602625512370008], ["bapa", 1.1602625512370008], ["chalybeate", 1.1602625512370008], ["complying", 1.1601799114541493], ["trophic", 1.1601799114541493], ["ringo", 1.1601640674130231], ["scriptures", 1.16014680791001], ["octavius", 1.1600782572079866], ["overpower", 1.159985217728448], ["bechet", 1.159979663866225], ["sholay", 1.159979663866225], ["paisley's", 1.159979663866225], ["squeeze", 1.1599715377559878], ["print", 1.159968048771657], ["archaea", 1.1599501728578219], ["stranraer", 1.1599501728578219], ["guatemala's", 1.1598997424342377], ["colorado's", 1.159884483607132], ["outsourcing", 1.1598689116075924], ["timing", 1.1597275031413812], ["inclined", 1.1597241547245927], ["elongating", 1.159713652310311], ["dextromethorphan", 1.159713652310311], ["lycurgus", 1.159713652310311], ["profiling", 1.1595819616828857], ["townhouses", 1.159570400892464], ["lagom", 1.1594284300642927], ["p\u00e1draig", 1.1594284300642927], ["incinerating", 1.1594284300642927], ["f\u00f3rum", 1.1594284300642927], ["goldfrapp's", 1.1594284300642927], ["moso", 1.1594284300642927], ["amestris", 1.1594284300642927], ["diarmuid", 1.1594284300642927], ["tuxen", 1.1594284300642927], ["jaguar's", 1.1594284300642927], ["winslet's", 1.1594284300642927], ["schaden", 1.1594284300642927], ["rancilio", 1.1594284300642927], ["dunwall", 1.1594284300642927], ["kandiyohi", 1.1594284300642927], ["cavelier", 1.1594284300642927], ["lincolnville", 1.1594284300642927], ["shotspotter", 1.1594284300642927], ["flintstone's", 1.1594284300642927], ["blazer's", 1.1594284300642927], ["astrov", 1.1594284300642927], ["srinivasan's", 1.1594284300642927], ["poiana", 1.1594284300642927], ["solovetsky", 1.1594284300642927], ["sandsend", 1.1594284300642927], ["kulig", 1.1594284300642927], ["mammal's", 1.1594284300642927], ["orosi", 1.1594284300642927], ["mcu's", 1.1594284300642927], ["inglesa", 1.1594284300642927], ["coupang", 1.1594284300642927], ["xining", 1.1594284300642927], ["bjcp", 1.1594284300642927], ["schmetterling", 1.1594284300642927], ["nerita", 1.1594284300642927], ["hebdige", 1.1594284300642927], ["fashionability", 1.1594284300642927], ["micronesians", 1.1594284300642927], ["wetherington", 1.1594284300642927], ["crate's", 1.1594284300642927], ["mixings", 1.1594284300642927], ["verbenas", 1.1594284300642927], ["geser", 1.1594284300642927], ["sentri", 1.1594284300642927], ["thenetherlands", 1.1594284300642927], ["lavine's", 1.1594284300642927], ["spocks", 1.1594284300642927], ["pluvialis", 1.1594284300642927], ["rouf", 1.1594284300642927], ["helix's", 1.1594284300642927], ["ulong", 1.1594284300642927], ["sch\u00f6nborn", 1.1594284300642927], ["forta", 1.1594284300642927], ["eaeu", 1.1594284300642927], ["yharnam", 1.1594284300642927], ["tumaco", 1.1594284300642927], ["pantokrator", 1.1594284300642927], ["skeres", 1.1594284300642927], ["odland", 1.1594284300642927], ["crate", 1.159424499798428], ["anomalous", 1.1594018071382295], ["sainthood", 1.1593862690304721], ["balliol", 1.1593271699393963], ["boils", 1.159154923870534], ["mouths", 1.1590640382719868], ["burke's", 1.1589707609471622], ["catholicism", 1.158942857473924], ["failing", 1.1588956727006783], ["cats", 1.1588732731875362], ["causation", 1.1588671194751348], ["outrageous", 1.158865447838489], ["jovan", 1.1587913814763455], ["dismorphia", 1.1587913814763453], ["mugg", 1.1587913814763453], ["skulle", 1.1587913814763453], ["grayton", 1.1587913814763453], ["norvig", 1.1587913814763453], ["bioelectricity", 1.1587913814763453], ["similarweb", 1.1587913814763453], ["equilibrio", 1.1587913814763453], ["desiccants", 1.1587913814763453], ["showier", 1.1587913814763453], ["trasimeno", 1.1587913814763453], ["catalan's", 1.1587913814763453], ["grumeti", 1.1587913814763453], ["horntail", 1.1587913814763453], ["profonde", 1.1587913814763453], ["porvoo", 1.1587913814763453], ["natsu's", 1.1587913814763453], ["dasara", 1.1587913814763453], ["matvey", 1.1587913814763453], ["hayata", 1.1587913814763453], ["valtellina", 1.1587913814763453], ["texta", 1.1587913814763453], ["ling", 1.1586483208119656], ["jingle", 1.1586483208119656], ["knitwear", 1.1585660452816005], ["compartment", 1.1585449296816581], ["defa", 1.1585261511303906], ["stesso", 1.1585261511303906], ["oceanarium", 1.1585261511303906], ["monarchial", 1.1585261511303906], ["sigmaringen", 1.1585261511303906], ["transhumance", 1.1585261511303906], ["caddy's", 1.1585261511303906], ["certifiers", 1.1585261511303906], ["nakaya", 1.1585261511303906], ["kealoha", 1.1585261511303906], ["phis", 1.1585261511303906], ["faten", 1.1585261511303906], ["mansingh", 1.1585261511303906], ["quispe", 1.1585261511303906], ["appalachia", 1.1584441954037967], ["tigris", 1.1584441954037967], ["stenography", 1.1584441954037967], ["thorens", 1.1584441954037967], ["sant\u00e9", 1.1584302466476812], ["impartial", 1.1583994373326107], ["levitt's", 1.1582889426637382], ["attractants", 1.1582889426637382], ["sambhar", 1.1582889426637382], ["samina", 1.1582889426637382], ["maverick's", 1.1582889426637382], ["mah\u0101y\u0101na", 1.1582889426637382], ["pngs", 1.1582889426637382], ["muscovado", 1.1582889426637382], ["derision", 1.1580135966393332], ["brahman", 1.1580105531916136], ["stars", 1.1579967601520311], ["requirements", 1.157948765909952], ["wrenching", 1.1578825254908738], ["somnolent", 1.1578825254908738], ["desiccated", 1.1578825254908738], ["simplifications", 1.1578825254908738], ["carlo's", 1.1578825254908738], ["statement's", 1.1578825254908738], ["rinat", 1.1578825254908738], ["anoop", 1.1578825254908738], ["cooling", 1.1578465664062312], ["minefield", 1.157087276503586], ["furred", 1.1569532457753915], ["nautilus", 1.1568780004542998], ["interpreters", 1.1568739170191917], ["breakwaters", 1.1568187840802253], ["gladius", 1.1568187840802253], ["rennet", 1.1567648782654965], ["flagship", 1.1567052173815793], ["enclaves", 1.156625944218143], ["meatballs", 1.1565738879168561], ["salmon", 1.1565446360181426], ["supporters", 1.1565040939129083], ["thomasina", 1.1564807306658245], ["savors", 1.1564807306658245], ["precedent", 1.1564423730959847], ["vestments", 1.1564187043461318], ["laplace", 1.156333000532037], ["refuel", 1.1563288020359417], ["scoop", 1.156279043859363], ["prophet's", 1.15627658578839], ["maisel", 1.1562501723064411], ["seguro", 1.1562501723064411], ["ornette", 1.1562501723064411], ["artificially", 1.1562284075973155], ["hasek", 1.156215907965878], ["yarmulke", 1.156215907965878], ["pyrenean", 1.156215907965878], ["narcosis", 1.156215907965878], ["ppis", 1.1561696815182543], ["gamete", 1.1561504648693466], ["terrapin", 1.1561504648693466], ["fretless", 1.1561504648693466], ["bullfights", 1.1561504648693466], ["biomimetic", 1.1561504648693466], ["craft's", 1.1561504648693466], ["exams", 1.1561043487552984], ["fundamentals", 1.1560753272781172], ["porthos", 1.1560028460663756], ["coahuila", 1.1560028460663756], ["victorian", 1.1560028460663754], ["eyelids", 1.1558861763089927], ["wraps", 1.1558815764260855], ["mirador", 1.1557618615247733], ["caseloads", 1.1557618615247733], ["braai", 1.1557618615247733], ["peatlands", 1.1557618615247733], ["subway's", 1.1557618615247733], ["flowchart", 1.155706054578297], ["visto", 1.1556547572840612], ["ba's", 1.1556547572840612], ["sega's", 1.1556547572840612], ["surfer", 1.155608113387231], ["florentine", 1.1555961846524214], ["rudolph's", 1.1555122439647922], ["zits", 1.1554499991768172], ["handcraft", 1.1554499991768172], ["kilobytes", 1.1554499991768172], ["woff", 1.1554499991768172], ["satoyama", 1.1554499991768172], ["everglade", 1.1554499991768172], ["blendtec", 1.1554499991768172], ["subotica", 1.1554499991768172], ["process's", 1.1554499991768172], ["cassowary", 1.1554499991768172], ["walken's", 1.1554499991768172], ["chihuahua's", 1.1554499991768172], ["buntingford", 1.1554499991768172], ["monospace", 1.1554499991768172], ["plank's", 1.1554499991768172], ["shipyard's", 1.1554499991768172], ["pyrokinesis", 1.1554499991768172], ["genderbent", 1.1554499991768172], ["gamle", 1.1554499991768172], ["alfas", 1.1554499991768172], ["bal\u00e1zs", 1.1554499991768172], ["gaita", 1.1554499991768172], ["federaci\u00f3n", 1.1554499991768172], ["dalibor", 1.1554499991768172], ["adobe", 1.1553208384875753], ["muffin", 1.1552979662821887], ["evangelion", 1.1552408138672547], ["fincher's", 1.15520379206001], ["rattlesnake", 1.15520379206001], ["suburbanization", 1.15520379206001], ["plagiarist", 1.15520379206001], ["guptas", 1.15520379206001], ["pentax", 1.1551484463279673], ["gloomier", 1.1551102608647983], ["arnie's", 1.1551102608647983], ["ridge's", 1.1551102608647983], ["saluki", 1.1551102608647983], ["singapura", 1.1551102608647983], ["infinitesimally", 1.155030598088187], ["prawns", 1.1548898779860808], ["adelaide's", 1.1548750384116766], ["vocational", 1.154829623338333], ["mise", 1.1547840337594242], ["garcia", 1.1547384225359785], ["jamboree", 1.154552086779011], ["hotelier", 1.1545067746876934], ["sweet", 1.1544491627044289], ["thrust", 1.1543983683255747], ["disbanded", 1.1543631257173452], ["sanded", 1.154218340283624], ["chocolat", 1.154218340283624], ["alchemical", 1.154218340283624], ["stroke", 1.154214118872095], ["stanwyck's", 1.1540357396919008], ["unknotting", 1.1540357396919008], ["chasubles", 1.1540357396919008], ["amathus", 1.1540357396919008], ["frogmouth", 1.1540357396919008], ["maternelle", 1.1540357396919008], ["masnavi", 1.1540357396919008], ["bixby's", 1.1540357396919008], ["uninflated", 1.1540357396919008], ["timbavati", 1.1540357396919008], ["arach", 1.1540357396919008], ["apprentissage", 1.1540357396919008], ["whiteman's", 1.1540357396919008], ["googlies", 1.1540357396919008], ["mcgm", 1.1540357396919008], ["sestertii", 1.1540357396919008], ["shao's", 1.1540357396919008], ["mindshift", 1.1540357396919008], ["interoception", 1.1540357396919008], ["truesight", 1.1540357396919008], ["bashaud", 1.1540357396919008], ["narv\u00e1ez", 1.1540357396919008], ["mediocracy", 1.1540357396919008], ["malvan", 1.1540357396919008], ["yosa", 1.1540357396919008], ["fargo's", 1.1540357396919005], ["debian", 1.1539597162571384], ["evesham", 1.1539097532574367], ["burgas", 1.1539097532574367], ["ignored", 1.153880627361297], ["nanjing", 1.1537472307569778], ["epistolary", 1.1537472307569778], ["tailgates", 1.1537472307569778], ["animist", 1.1537472307569778], ["nubians", 1.153704500960646], ["stashing", 1.1536312879139712], ["grigor", 1.1536312879139712], ["searches", 1.1535416083668946], ["germs", 1.1534940494467394], ["economic", 1.1534715444413848], ["ballets", 1.1534276412342175], ["coped", 1.1534276412342175], ["theron's", 1.1533108428704864], ["horniman", 1.1533108428704864], ["thurman's", 1.1533108428704864], ["luni", 1.1533108428704864], ["pharmacare", 1.1533108428704864], ["gajendra", 1.1533108428704864], ["maenads", 1.1532230384667657], ["vigors", 1.1532230384667657], ["flyhalf", 1.1532230384667657], ["brissot", 1.1532230384667657], ["unrefrigerated", 1.1532230384667657], ["framboise", 1.1532230384667657], ["booch", 1.1532230384667657], ["eastview", 1.1532230384667657], ["msgt", 1.1532230384667657], ["biodome", 1.1532230384667657], ["zeroes", 1.153156474366421], ["tetris", 1.153120376443614], ["disenfranchised", 1.1530910001065104], ["counterproductive", 1.1530826298176016], ["depletes", 1.153017472862761], ["abierto", 1.152963215770254], ["tricolored", 1.152963215770254], ["balachandran", 1.152963215770254], ["pedipalps", 1.152963215770254], ["participial", 1.152963215770254], ["zartan", 1.152963215770254], ["tugboat", 1.152759151484277], ["begun", 1.1526637474205859], ["creaks", 1.1526034258305808], ["gulags", 1.1526034258305808], ["hbcu", 1.1526034258305808], ["kuba", 1.1525466613181048], ["uneasiness", 1.152521958243231], ["attached", 1.1525103903380784], ["thumbs", 1.1524813242057852], ["pencil", 1.1523131049738422], ["advertisements", 1.152290311971726], ["ghulam", 1.1522617365321801], ["latched", 1.152249306658322], ["biltong", 1.152249306658322], ["goteborg", 1.152249306658322], ["unidad", 1.152249306658322], ["loss", 1.1521779394220633], ["sleepaway", 1.152121915911317], ["tobago's", 1.152121915911317], ["archbishop's", 1.151974961585308], ["mithras", 1.151974961585308], ["petrovic", 1.151974961585308], ["talbot's", 1.151974961585308], ["trotsky", 1.1518553752714302], ["sleepiness", 1.1517308473743342], ["spectacled", 1.151662575714861], ["ja'far", 1.151662575714861], ["today", 1.151559601599159], ["egret", 1.1515509737451957], ["moba", 1.1515509737451957], ["recitation", 1.151528546176827], ["england's", 1.1514587666809382], ["grape", 1.1514443420728322], ["scented", 1.1512660539166402], ["depressive", 1.1512660539166402], ["veils", 1.1512240000718887], ["blotting", 1.151208850992367], ["owner's", 1.1510847807228255], ["heiden", 1.1510614207751693], ["hamid", 1.1509073295541818], ["doctrines", 1.1508843344027422], ["ababa", 1.1508089950249991], ["yeon", 1.150793448804195], ["ilsa", 1.1506907085623592], ["gauntlet", 1.150690708562359], ["juliet", 1.1506537326398474], ["specializations", 1.1505457122533163], ["dravida", 1.1503638077928808], ["makeups", 1.1503638077928808], ["rockefeller's", 1.1503638077928808], ["tamo", 1.1503638077928808], ["stool", 1.150261589576443], ["snowball", 1.15021442929557], ["evernote", 1.15021442929557], ["torcida", 1.1502144292955698], ["piquillo", 1.1502144292955698], ["authoritativeness", 1.1502144292955698], ["molecule's", 1.1502144292955698], ["fulmars", 1.1502144292955698], ["borat's", 1.1502144292955698], ["homewards", 1.1502144292955698], ["batin", 1.1502144292955698], ["jonben\u00e9t", 1.1502144292955698], ["ficar", 1.1502144292955698], ["taliyah", 1.1502144292955698], ["senja", 1.1502144292955698], ["ripple's", 1.1502144292955698], ["lended", 1.1502144292955698], ["pratfall", 1.1502144292955698], ["ccri", 1.1502144292955698], ["leekie", 1.1502144292955698], ["komplete", 1.1502144292955698], ["abu'l", 1.1502144292955698], ["cherimoya", 1.1502144292955698], ["stenmark", 1.1502144292955698], ["menina", 1.1502144292955698], ["mehra's", 1.1502144292955698], ["thunks", 1.1502144292955698], ["hassabis", 1.1502144292955698], ["gstreamer", 1.1502144292955698], ["raazi", 1.1502144292955698], ["subclassing", 1.1502144292955698], ["stingo", 1.1502144292955698], ["dizygotic", 1.1502144292955698], ["hypnic", 1.1502144292955698], ["impis", 1.1502144292955698], ["kuning", 1.1502144292955698], ["motherboard's", 1.1502144292955698], ["franz", 1.1501974644809785], ["amulets", 1.1501937865384038], ["symmetries", 1.1501288478648188], ["jehovah's", 1.1500254654964714], ["bravest", 1.1500254654964714], ["teepee", 1.1499836479092747], ["bertolt", 1.149813657717418], ["kristoffer", 1.149813657717418], ["tear", 1.1494827080626475], ["warding", 1.1494431484829228], ["julian", 1.1494172343262965], ["africa's", 1.1493928475603588], ["abhorred", 1.149368683391676], ["palatinate", 1.149368683391676], ["dorian", 1.1492207594941999], ["biodiesel", 1.1490013555925735], ["expedia's", 1.1486929816377716], ["amalia", 1.1486929816377716], ["kookaburra", 1.1486929816377716], ["jaisalmer", 1.1486929816377716], ["discman", 1.1486929816377716], ["unwomanly", 1.1486929816377716], ["graters", 1.1486929816377716], ["dispirited", 1.1486929816377716], ["poblaci\u00f3n", 1.1486929816377716], ["sncb", 1.1486929816377716], ["wackier", 1.1486929816377716], ["baymax", 1.1486929816377716], ["ricci's", 1.1486929816377716], ["lillywhite", 1.1486929816377716], ["seita", 1.1486929816377716], ["temptresses", 1.1486929816377716], ["stubb's", 1.1486929816377716], ["kiyomori", 1.1486929816377716], ["dorset's", 1.1486929816377716], ["vuze", 1.1486929816377716], ["phonak", 1.1486929816377716], ["nightcall", 1.1486929816377716], ["wunderman", 1.1486929816377716], ["esdi", 1.1486929816377716], ["kharijites", 1.1486929816377716], ["xorn", 1.1486929816377716], ["agneepath", 1.1486929816377716], ["tashman", 1.1486929816377716], ["solinus", 1.1486929816377716], ["undergo", 1.1486005171872373], ["buenos", 1.148534930514991], ["bench", 1.1484810595040165], ["cilicia", 1.1483825240751668], ["tomasz", 1.148351362907173], ["carriages", 1.1483395376434216], ["mammogram", 1.1482468872759706], ["functioning", 1.148194120685746], ["uproar", 1.1481859723217382], ["scalp", 1.148160474957542], ["falsehood", 1.1481407253965996], ["rashomon", 1.1481407253965996], ["lipases", 1.148102393986544], ["grill's", 1.148102393986544], ["scrawls", 1.148102393986544], ["iturbide", 1.148102393986544], ["crematoriums", 1.148102393986544], ["hamnet", 1.148102393986544], ["castilians", 1.148102393986544], ["scepters", 1.148102393986544], ["nightcrawlers", 1.148102393986544], ["yuzuki", 1.148102393986544], ["waists", 1.1480699617155274], ["valuables", 1.1480365856482642], ["impersonate", 1.147875406561873], ["c\u00f4te", 1.147875406561873], ["neuromodulation", 1.1478754065618728], ["karni", 1.1478754065618728], ["apa's", 1.1478754065618728], ["f\u00e1bio", 1.1478754065618728], ["noiseless", 1.1478341457636498], ["stereotypically", 1.1478341457636498], ["katakana", 1.1478341457636498], ["unseasoned", 1.1478341457636498], ["argument's", 1.1478341457636498], ["synapse", 1.1478139817909532], ["collapsible", 1.1478093716518964], ["conrad's", 1.147544288656134], ["kindergarteners", 1.147544288656134], ["superstorm", 1.147544288656134], ["sand", 1.1474315078906638], ["crueler", 1.1473650128728725], ["wiley's", 1.1473650128728725], ["sarpanch", 1.1473650128728725], ["ellipse", 1.1473500420350218], ["sleds", 1.1472107971453358], ["sarong", 1.1472107971453358], ["brutal", 1.1470519916640034], ["bastille", 1.1469803848287843], ["wiring", 1.1469038669053093], ["mammograms", 1.1466793365703587], ["aman", 1.146649633545435], ["guzzle", 1.1464748158801263], ["liquid", 1.1464049727600605], ["scooted", 1.146366514839518], ["braiding", 1.146366514839518], ["harpist", 1.146366514839518], ["alcibiades", 1.146366514839518], ["spill", 1.146289358173304], ["investor's", 1.1462470954411283], ["drawl", 1.1461065206195142], ["titian", 1.146096427477804], ["simona", 1.146096427477804], ["procedurals", 1.1460865469954815], ["beaky", 1.1460865469954815], ["herero", 1.1460865469954815], ["napolean", 1.1460865469954815], ["airline's", 1.1460673976797426], ["chima", 1.1460673976797424], ["prabhakar", 1.1460673976797424], ["asaf", 1.1460673976797424], ["affects", 1.1460470267697234], ["evade", 1.1458986101395834], ["guido", 1.145867817818068], ["suva", 1.1458525771740853], ["pomegranate", 1.145831372799849], ["dams", 1.1457639880531725], ["mitch's", 1.1457198939205915], ["executive's", 1.1457198939205915], ["belong", 1.145424950105012], ["touch", 1.145415603728538], ["object", 1.1452374093624316], ["expert's", 1.145158541694271], ["uri's", 1.145158541694271], ["ganzen", 1.145158541694271], ["sbrt", 1.145158541694271], ["kollontai", 1.145158541694271], ["jieun", 1.145158541694271], ["brunnhilde", 1.145158541694271], ["personen", 1.145158541694271], ["kenobi's", 1.145158541694271], ["maxtor", 1.145158541694271], ["dejar", 1.145158541694271], ["kildare", 1.1451585416942707], ["mangled", 1.1451585416942707], ["jimi", 1.145104929402993], ["studies", 1.1450955879524498], ["permits", 1.1450161090398312], ["bells", 1.1449662396721558], ["hopewell", 1.1449132206699146], ["pol\u00edtica", 1.144869797735195], ["eternities", 1.144869797735195], ["shuttlesworth", 1.144869797735195], ["crowne", 1.1448640050323124], ["punk's", 1.1448640050323124], ["embroidered", 1.144853514685448], ["hospitalization", 1.1447706153590083], ["intuitively", 1.1447247695193865], ["louie's", 1.1445428650589513], ["image's", 1.1444542621483333], ["hussain's", 1.1444542621483333], ["ritsuko", 1.1444542621483333], ["balcony", 1.1443039457676332], ["vet's", 1.1442870359547774], ["wearer's", 1.1442870359547774], ["vehicles", 1.1442866284161222], ["stature", 1.1442448513684509], ["rekindles", 1.1441526931728003], ["wackiest", 1.1441526931728003], ["tenterhooks", 1.1441526931728003], ["barbering", 1.1441526931728003], ["koki", 1.1441526931728003], ["kingdoms", 1.144137038769755], ["surah", 1.1440584854582532], ["chelm", 1.1440584854582532], ["curatorship", 1.1440584854582532], ["depression's", 1.1440584854582532], ["na\u00efvely", 1.1440584854582532], ["desalinate", 1.1440584854582532], ["loyola's", 1.1440584854582532], ["bayport", 1.1440584854582532], ["patho", 1.1440584854582532], ["common's", 1.1440584854582532], ["admonishments", 1.1440584854582532], ["kyriakos", 1.1440584854582532], ["zorak", 1.1440584854582532], ["googol", 1.1440584854582532], ["nimitz", 1.1440445742412608], ["pickup", 1.1439458968296774], ["megaphone", 1.1438924736548077], ["herder", 1.1438563796098231], ["lofoten", 1.1438049263967598], ["megabytes", 1.1438049263967598], ["skrein", 1.1438049263967596], ["trithemius", 1.1438049263967596], ["khash", 1.1438049263967596], ["astronomia", 1.1438049263967596], ["precios", 1.1438049263967596], ["baseplates", 1.1438049263967596], ["pucci's", 1.1438049263967596], ["palavra", 1.1438049263967596], ["bhil", 1.1438049263967596], ["ironer", 1.1438049263967596], ["pussyhat", 1.1438049263967596], ["bahauddin", 1.1438049263967596], ["abbacchio", 1.1438049263967596], ["nintendoswitch", 1.1438049263967596], ["hsms", 1.1438049263967596], ["agnesi", 1.1438049263967596], ["trakai", 1.1438049263967596], ["farrukhabad", 1.1438049263967596], ["nieuwendyk", 1.1438049263967596], ["neda's", 1.1438049263967596], ["olorun", 1.1438049263967596], ["kondabolu", 1.1438049263967596], ["dequan", 1.1438049263967596], ["loutre", 1.1438049263967596], ["betazoid", 1.1438049263967596], ["dhanmondi", 1.1438049263967596], ["paquete", 1.1438049263967596], ["berrian", 1.1438049263967596], ["gendreau", 1.1438049263967596], ["aniko", 1.1438049263967596], ["tupaia", 1.1438049263967596], ["suplee", 1.1438049263967596], ["dalessandro", 1.1438049263967596], ["tonal", 1.1437620068873264], ["quasar", 1.1437395100440921], ["skateboarder", 1.1437395100440921], ["fanned", 1.1435969618646875], ["translates", 1.1435680437196953], ["sunnybrook", 1.1435220164471194], ["blore", 1.1435220164471194], ["sudha", 1.1435220164471194], ["brawn", 1.1434988916338442], ["andersons", 1.1434988916338442], ["stefani's", 1.1434988916338442], ["doublethink", 1.1434988916338442], ["battletoads", 1.1434988916338442], ["rapporteurs", 1.1434988916338442], ["flukes", 1.1434339053362974], ["maca", 1.1434339053362974], ["willie's", 1.143287367606535], ["roomier", 1.143287367606535], ["hummers", 1.143287367606535], ["barkley's", 1.143287367606535], ["harmonic", 1.143196432043393], ["ojos", 1.1431600102078932], ["breeding", 1.1430988468171956], ["puncturing", 1.143074919893517], ["carmelite", 1.1430140500431578], ["entre", 1.142992962001203], ["casas", 1.1428615424312079], ["tessellation", 1.1428143072179262], ["bathrobes", 1.1428143072179262], ["gino's", 1.1428143072179262], ["freeform", 1.1427391926625223], ["bubblegum", 1.142647229102836], ["blakes", 1.142647229102836], ["rosses", 1.142647229102836], ["dogg's", 1.142647229102836], ["tang's", 1.142647229102836], ["transplantations", 1.142647229102836], ["monitors", 1.1424299112810052], ["scorecards", 1.1423751702387641], ["eigen", 1.1421290217426987], ["readymade", 1.1419986119434589], ["eschewing", 1.1417367532230727], ["abdomens", 1.1417312059914821], ["poulet", 1.1417312059914821], ["expressively", 1.1417312059914821], ["atreides", 1.1417312059914821], ["ana\u00efs", 1.1417312059914821], ["cockatoos", 1.1417009374017155], ["basketball", 1.1416489304611168], ["flirtatious", 1.1415940980161052], ["caked", 1.1415394445066143], ["greek", 1.1414198648097307], ["soliciting", 1.14134134655529], ["limpets", 1.14134134655529], ["irretrievable", 1.14134134655529], ["hessen", 1.14134134655529], ["bada", 1.14134134655529], ["nicaragua's", 1.14134134655529], ["orchestration", 1.1413413465552897], ["jester", 1.1412869347824752], ["pools", 1.1411404415479751], ["movement", 1.141115301505527], ["afforded", 1.1410828906344213], ["grated", 1.1409654106703901], ["streetwise", 1.1408982792623725], ["chondroitin", 1.1408982792623725], ["reintroduced", 1.1408836854780207], ["liminal", 1.1407709610747525], ["bartholemew", 1.1407709610747525], ["pocos", 1.1407709610747525], ["xianfeng", 1.1407709610747525], ["latifah's", 1.1407709610747525], ["practicums", 1.1407709610747525], ["fmod", 1.1407709610747525], ["songkick", 1.1407709610747525], ["estructura", 1.1407709610747525], ["charon's", 1.1407709610747525], ["sariel", 1.1407709610747525], ["innateness", 1.1407709610747525], ["kasu", 1.1407709610747525], ["bibury", 1.1407709610747525], ["elektra's", 1.1407709610747525], ["twinkletoes", 1.1407709610747525], ["unnerving", 1.1407105388840175], ["chronometers", 1.1405913686690419], ["fuerte", 1.1405913686690419], ["restitution", 1.1405409669293745], ["crohn's", 1.140318991597306], ["dasein", 1.140318991597306], ["cholecystectomy", 1.140318991597306], ["hermione", 1.1402728971982068], ["carthage", 1.1402625740160126], ["quinault", 1.140146906938497], ["giana", 1.140146906938497], ["qing", 1.140025570958141], ["hieronymus", 1.1400001698848372], ["jaap", 1.1400001698848372], ["maradona's", 1.1398568817790193], ["chlorosis", 1.1398568817790193], ["altarpieces", 1.1398568817790193], ["hfpa", 1.1398568817790193], ["corpuscular", 1.1398568817790193], ["heritage's", 1.1398568817790193], ["bisection", 1.1398568817790193], ["errand", 1.1397906110300788], ["pampanga", 1.1396793142220494], ["timon", 1.1395801771905407], ["persnickety", 1.1395801771905407], ["disponible", 1.1395801771905407], ["ramli", 1.1395801771905407], ["linguistique", 1.1395801771905407], ["yonder", 1.1395329861571875], ["lorax", 1.1395034377846696], ["anatolia", 1.139487835842644], ["muskets", 1.139487835842644], ["movimiento", 1.1394612355166873], ["ocarina", 1.139405676679849], ["barbara's", 1.139392385895206], ["fish's", 1.139392385895206], ["survivalist", 1.139392385895206], ["retracement", 1.1393158960733816], ["tonalities", 1.1393158960733816], ["montenegrins", 1.1393158960733816], ["brazilians", 1.1392736991883419], ["outdid", 1.139217381944884], ["sein", 1.139085731245892], ["ser\u00e1", 1.139085731245892], ["katz's", 1.139085731245892], ["smas", 1.1390632201150597], ["thrombolytic", 1.1390632201150597], ["tortas", 1.1390632201150597], ["leed", 1.1389783209930637], ["amboseli", 1.1389582953527058], ["norfolk's", 1.1389582953527058], ["geelong's", 1.1389582953527058], ["soymilk", 1.1389582953527058], ["albrecht", 1.1389582953527055], ["silesia", 1.1389008447451217], ["sharks", 1.1388384050058264], ["cascade", 1.1388181158702007], ["mandating", 1.1388063980806413], ["drugstore", 1.1387965497959691], ["brogue", 1.1387965497959691], ["downside", 1.1386957113076677], ["pitches", 1.1386728039581306], ["vetting", 1.138648596496009], ["exhales", 1.1386222486480706], ["ipanema", 1.1386222486480706], ["shabby", 1.1386133510763847], ["harolds", 1.1385146893954114], ["boccaccio", 1.1385146893954114], ["tika", 1.1385146893954114], ["plumpness", 1.1385146893954114], ["sapphire's", 1.1385146893954114], ["tiff's", 1.1385146893954114], ["engel's", 1.1385146893954114], ["radioman", 1.1384617081171113], ["codebook", 1.1384617081171113], ["opentable", 1.1384617081171113], ["kickflip", 1.1384617081171113], ["reisen", 1.1384617081171113], ["overachievers", 1.1384617081171113], ["fluctuation", 1.1382718494332227], ["rest", 1.1381304118207354], ["sharpener", 1.1381209400368577], ["syllabus", 1.1380557699173999], ["theorem", 1.138052457176285], ["pale", 1.138033110768173], ["rodrigo", 1.137963828343312], ["opossums", 1.1379190336720657], ["nootropics", 1.1379190336720657], ["eakins", 1.1379190336720657], ["beerfest", 1.1378998932196984], ["didja", 1.1378998932196984], ["databanks", 1.1378998932196984], ["tahani", 1.1378998932196984], ["marketa", 1.1378998932196984], ["bartending", 1.1378907032674033], ["flow", 1.1378449073752734], ["seances", 1.1378177818116153], ["mellark", 1.1378177818116153], ["matings", 1.1378177818116153], ["lucha", 1.1377255612100743], ["liquors", 1.1377255612100743], ["diana's", 1.1377203919624257], ["bronchitis", 1.1377030824384486], ["galenic", 1.1376575032115572], ["microneedling", 1.1376575032115572], ["rutaceae", 1.1376575032115572], ["stinkiest", 1.1376575032115572], ["aming", 1.1376575032115572], ["oura", 1.1376575032115572], ["shaggs", 1.1376575032115572], ["chazelle's", 1.1376575032115572], ["zhangzhou", 1.1376575032115572], ["bette's", 1.1376575032115572], ["craftiest", 1.1376575032115572], ["binky", 1.1376575032115572], ["scms", 1.1376575032115572], ["loys", 1.1376575032115572], ["jochi", 1.1376575032115572], ["climate's", 1.1376575032115572], ["qbasic", 1.1376575032115572], ["selu", 1.1376575032115572], ["rolls", 1.1376195812947836], ["olympians", 1.1376195812947836], ["follies", 1.1374664608096743], ["cheer", 1.137457765413892], ["verbal", 1.1374349426738235], ["erkki", 1.1373815736691824], ["gyres", 1.1373815736691824], ["crooners", 1.1373815736691824], ["diamantina", 1.1373815736691824], ["utils", 1.1373815736691824], ["grainne", 1.1373815736691824], ["bioinformatics", 1.1373624547992265], ["defeats", 1.1373269668720927], ["fines", 1.1373089198496003], ["refrigerators", 1.137206051821394], ["juche", 1.1372060518213938], ["oedo", 1.1372060518213938], ["newsstands", 1.1372060518213938], ["divya's", 1.1372060518213938], ["overthought", 1.1372060518213938], ["jumbotrons", 1.1372060518213938], ["escritoire", 1.1372060518213938], ["winfield's", 1.1372060518213938], ["sunderbans", 1.1372060518213938], ["ezrin", 1.1372060518213938], ["emmeline", 1.1372060518213938], ["suavemente", 1.1372060518213938], ["maybell", 1.1372060518213938], ["xuanwu", 1.1372060518213938], ["pratiques", 1.1372060518213938], ["floofs", 1.1372060518213938], ["chuckanut", 1.1372060518213938], ["haplo", 1.1372060518213938], ["stimmen", 1.1372060518213938], ["senua's", 1.1372060518213938], ["devenir", 1.1372060518213938], ["xia's", 1.1372060518213938], ["surefooted", 1.1372060518213938], ["microgroove", 1.1372060518213938], ["hashtagging", 1.1372060518213938], ["territoires", 1.1372060518213938], ["combina", 1.1372060518213938], ["krakower", 1.1372060518213938], ["kelsang", 1.1372060518213938], ["tempo's", 1.1372060518213938], ["fallout's", 1.1372060518213938], ["diabeetus", 1.1372060518213938], ["damehood", 1.1372060518213938], ["grignan", 1.1372060518213938], ["gutterson", 1.1372060518213938], ["vitalogy", 1.1372060518213938], ["barge's", 1.1372060518213938], ["footcare", 1.1372060518213938], ["pvris", 1.1372060518213938], ["khine", 1.1372060518213938], ["electroforming", 1.1372060518213938], ["pridi", 1.1372060518213938], ["lemn", 1.1372060518213938], ["bethanne", 1.1372060518213938], ["m\u00fclheim", 1.1372060518213938], ["tangs", 1.1370845684273312], ["igneous", 1.1368859825695443], ["legitimacy", 1.1368670932366483], ["dinosaurs", 1.13685159798706], ["getters", 1.1368301159364942], ["shed's", 1.1367941698757866], ["jeannie's", 1.1367941698757866], ["pressingly", 1.1367941698757866], ["rainmaking", 1.1367941698757866], ["cryptomeria", 1.1367941698757866], ["hitcher", 1.1367941698757866], ["starling's", 1.1367941698757866], ["multani", 1.1367941698757866], ["doris's", 1.1367941698757866], ["treliving", 1.1367941698757866], ["thornbush", 1.1367941698757866], ["propellers", 1.1367874997228213], ["comment", 1.1367647357825985], ["collection's", 1.1366647828771668], ["superstar's", 1.1366014765679002], ["bayberry", 1.1366014765679002], ["lapdance", 1.1366014765679002], ["elevated", 1.1365876899920262], ["clover", 1.136574019875165], ["neatness", 1.1365390687997978], ["zerg", 1.1365390687997978], ["gotham's", 1.1365390687997978], ["special", 1.1365092829826688], ["kissed", 1.1364402565003022], ["removers", 1.136239861463007], ["southwards", 1.1361178163651053], ["hostiles", 1.1359604811184132], ["anu's", 1.1355500413444333], ["luminosities", 1.1355500413444333], ["diaoyu", 1.1355500413444333], ["patti's", 1.1355500413444333], ["ebadi", 1.1355500413444333], ["crayton", 1.135453808290082], ["formlessness", 1.135453808290082], ["mcdonagh's", 1.135453808290082], ["brigada", 1.135453808290082], ["misdiagnose", 1.135453808290082], ["ujamaa", 1.135453808290082], ["diggler", 1.135453808290082], ["palsy", 1.1353739592177692], ["prodded", 1.1353457007200267], ["hats", 1.1352600307701486], ["interlocked", 1.135138404454446], ["compartmentalized", 1.135138404454446], ["clocking", 1.1351132913285316], ["coltan", 1.1350744565977362], ["sorters", 1.1350744565977362], ["foreigner", 1.1350151671941642], ["cottontail", 1.1350064269629385], ["frankl", 1.1350064269629385], ["aryan", 1.1349086658581182], ["dijon", 1.1348687113196265], ["loquacious", 1.1348018106970357], ["spare", 1.1347811669512557], ["soundtrack", 1.1347520872613264], ["romana", 1.1346428526061691], ["durante", 1.134608504877992], ["leathers", 1.1345552218637682], ["nonconformist", 1.1345552218637682], ["porky", 1.1345552218637682], ["generation", 1.1345472011982383], ["goosebump", 1.1344613871750029], ["p\u00e9rigord", 1.1344613871750029], ["scummvm", 1.1344613871750029], ["senda", 1.1344613871750029], ["reuptake", 1.1343901896485076], ["streaking", 1.134252269868611], ["piloting", 1.134252269868611], ["artikel", 1.134252269868611], ["antifungals", 1.134252269868611], ["metonymy", 1.134252269868611], ["yowling", 1.134252269868611], ["holkham", 1.134252269868611], ["degraded", 1.1341653939288232], ["mutations", 1.134014381389192], ["onboard", 1.1340026544923196], ["bucket", 1.1339190226739382], ["bar's", 1.1339039122918702], ["coneflower", 1.1338711367134133], ["compy", 1.1338711367134133], ["olgas", 1.1338711367134133], ["winnfield", 1.1338711367134133], ["bat'leth", 1.1338711367134133], ["orian", 1.1338711367134133], ["resubmission", 1.1338711367134133], ["cheshire's", 1.1338711367134133], ["mswati", 1.1338711367134133], ["allelopathy", 1.1338711367134133], ["viaggio", 1.1338711367134133], ["aleena", 1.1338711367134133], ["nocardia", 1.1338711367134133], ["momi", 1.1338711367134133], ["skeeter's", 1.1338711367134133], ["zelkova", 1.1338711367134133], ["bastiaan", 1.1338711367134133], ["zeri", 1.1338711367134133], ["keshava", 1.1338711367134133], ["unwillingness", 1.1338401142472925], ["lent", 1.1337477092503419], ["alfresco", 1.1336411425680353], ["overruling", 1.1336411425680353], ["liebe", 1.1336411425680353], ["prefrontal", 1.1336087833312423], ["exemplary", 1.133587668929235], ["sterilized", 1.133475760179993], ["unchallenged", 1.1334390829342416], ["petrograd", 1.1333770752159347], ["scavenge", 1.1333770752159347], ["leftfield", 1.1332298068849553], ["rancheros", 1.1332298068849553], ["pickett's", 1.1332298068849553], ["polanski's", 1.1332298068849553], ["headline", 1.1332253475448397], ["wasabi", 1.1331690493222595], ["years", 1.1331047644455068], ["lofa", 1.1330901261494835], ["aspen's", 1.1330901261494835], ["galvanized", 1.133070757087498], ["downloaded", 1.133046287887889], ["gargamel", 1.1330451515473499], ["longlegs", 1.1330451515473499], ["again", 1.133036089946016], ["adfs", 1.1329574613413638], ["grupos", 1.1329574613413638], ["lifepo4", 1.1329574613413638], ["whisp", 1.1329574613413638], ["abagail", 1.1329574613413638], ["xim\u00e9nez", 1.1329574613413638], ["shuma", 1.1329574613413638], ["skall", 1.1329574613413638], ["lobola", 1.1329574613413638], ["assalamualaikum", 1.1329574613413638], ["amazonians", 1.1329574613413638], ["tithed", 1.1329574613413638], ["piecrust", 1.1329574613413638], ["mbugua", 1.1329574613413638], ["xanth", 1.1329574613413638], ["adoption", 1.1328751688230567], ["soils", 1.1328706621810192], ["clerk's", 1.1328531099962402], ["braga", 1.1328531099962402], ["outbid", 1.1327111662410723], ["teabag", 1.1327111662410723], ["candelaria", 1.1327111662410723], ["honey's", 1.1327111662410723], ["myeong", 1.1327111662410723], ["jour", 1.1326559578204336], ["haploid", 1.1324345578976676], ["connemara", 1.1324345578976676], ["paolo", 1.132345844863229], ["brayden", 1.1323188853176984], ["recollected", 1.1322830819000893], ["comptia", 1.1322830819000893], ["handkerchief", 1.1321396311209253], ["frictional", 1.1320958967319785], ["bulldozed", 1.1320958967319785], ["buff's", 1.1320958967319785], ["shagun", 1.1320958967319785], ["wonderland's", 1.1320958967319785], ["aliments", 1.1320958967319785], ["hoffnung", 1.1320958967319785], ["vlach", 1.1320958967319785], ["etienne's", 1.1320958967319785], ["madaba", 1.1320958967319785], ["haircutting", 1.1320958967319785], ["interoceptive", 1.1320958967319785], ["vecna", 1.1320958967319785], ["swappers", 1.1320958967319785], ["varric", 1.1320958967319785], ["cream", 1.1320889964672205], ["hardcore", 1.132086796282648], ["duties", 1.1320814521272355], ["still", 1.132074210700303], ["judging", 1.132036933404024], ["instigated", 1.132036933404024], ["slink", 1.132036933404024], ["route's", 1.1320080556475023], ["maguey", 1.1320080556475023], ["ondes", 1.1320080556475023], ["amasses", 1.1320080556475023], ["kamba", 1.1320080556475023], ["northernmost", 1.1318892443063848], ["dubrovnik", 1.1318103123460599], ["mouthwash", 1.1317648745399518], ["samaritans", 1.1317648745399518], ["japanese", 1.1317584731549148], ["ziggy", 1.131738715033791], ["seville", 1.1317367910443057], ["drum", 1.131628212412883], ["leftmost", 1.1316178156945074], ["heforshe", 1.1315483102700434], ["bogart's", 1.1315483102700434], ["bhumi", 1.1315483102700434], ["aylin", 1.1315483102700434], ["disciplinary", 1.1315402565454153], ["taluk", 1.1315257019121963], ["travancore", 1.1315257019121963], ["jewels", 1.1315186927400718], ["piglets", 1.1314914485459096], ["rana", 1.13148702718941], ["exiting", 1.1312580251493516], ["graduations", 1.1312484238149336], ["shanxi", 1.1312484238149336], ["helmets", 1.1311797429127013], ["voids", 1.1311247360362526], ["pastoralists", 1.1311247360362526], ["sapa", 1.1311247360362526], ["baggie", 1.1311247360362526], ["ayatollahs", 1.1311247360362526], ["admonishment", 1.1311247360362526], ["mahomes", 1.1311247360362526], ["mountainous", 1.13100310972055], ["whisk", 1.1309550248980627], ["pontic", 1.1309231454577398], ["sephardim", 1.1309231454577398], ["teetotaler", 1.1309231454577398], ["eoghan", 1.1309231454577398], ["ivies", 1.1309231454577398], ["reproach", 1.130915276250312], ["indiscretion", 1.1308557834953865], ["zoo's", 1.1308018343131478], ["stapler", 1.1308018343131478], ["bloodstains", 1.1306628639513052], ["waltzed", 1.1306628639513052], ["jutland", 1.1306628639513052], ["dotty", 1.1306628639513052], ["summaries", 1.1305166044577386], ["budva", 1.130512229410411], ["uofa", 1.130512229410411], ["trabajadores", 1.130512229410411], ["clorinda", 1.130512229410411], ["nick's", 1.1304427193126856], ["waffen", 1.1304427193126854], ["meaninglessness", 1.1303138939315671], ["album's", 1.1302252488140132], ["stara", 1.1302252488140132], ["chiropractor", 1.1301970776961463], ["predicate", 1.1301970776961463], ["licinius", 1.13016955554644], ["wasteland", 1.130136031792475], ["sinterklaas", 1.1301163704710604], ["physiology", 1.1299413448564415], ["tibetan", 1.1299309962710213], ["nablus", 1.129903951454849], ["moneylender", 1.1298215969394367], ["aujourd'hui", 1.1298215969394367], ["languid", 1.129757754390025], ["bridgette", 1.129757754390025], ["propagate", 1.1296414882837793], ["cricketers", 1.1296414882837793], ["judaica", 1.129641488283779], ["moche", 1.129641488283779], ["belgrade", 1.129588403627375], ["shackleton", 1.1295737964400423], ["bushwick", 1.1295629973521308], ["spree", 1.1295164541757197], ["humbuckers", 1.129437434546734], ["knossos", 1.129363251464005], ["zoology", 1.1293356521665596], ["crystals", 1.1292852136495035], ["obfuscating", 1.1291948719863556], ["sox's", 1.1291948719863556], ["bonafide", 1.1291886436638405], ["archdeacon", 1.1291583225488415], ["rome", 1.129067439845109], ["droits", 1.129044285899231], ["sycamores", 1.129044285899231], ["talmud", 1.129044285899231], ["wiccan", 1.1289673540867682], ["normalize", 1.128935462353602], ["particulate", 1.1289354623536019], ["napped", 1.1289354623536019], ["pandered", 1.1289354623536019], ["reigning", 1.1288474773592636], ["canned", 1.1287272101008157], ["stalled", 1.1287233959569918], ["antiviral", 1.1286418319582956], ["transcripts", 1.1285975588928556], ["subtractive", 1.1285352725406441], ["uncontrollable", 1.1285076732431987], ["portuguese", 1.12835574813857], ["decreasing", 1.1281843754186354], ["configure", 1.1281238624525571], ["sphere", 1.1279741207733973], ["backcombing", 1.127807654698903], ["tampoco", 1.127807654698903], ["relationship's", 1.127807654698903], ["wirtanen", 1.127807654698903], ["virtutis", 1.127807654698903], ["brazilwood", 1.127807654698903], ["admiralty's", 1.127807654698903], ["explainers", 1.127807654698903], ["intangibility", 1.127807654698903], ["deceits", 1.127807654698903], ["projets", 1.127807654698903], ["brayan", 1.127807654698903], ["wahre", 1.127807654698903], ["yangming", 1.127807654698903], ["bennie's", 1.127807654698903], ["barlaston", 1.127807654698903], ["schwer", 1.127807654698903], ["hyperglycemic", 1.127807654698903], ["fishman's", 1.127807654698903], ["conflit", 1.127807654698903], ["deodorize", 1.127807654698903], ["carraig", 1.127807654698903], ["veren", 1.127807654698903], ["homura", 1.127807654698903], ["altas", 1.127807654698903], ["stoppered", 1.127807654698903], ["rawhides", 1.127807654698903], ["insight's", 1.127807654698903], ["cryoprotectants", 1.127807654698903], ["fishless", 1.127807654698903], ["adjara", 1.127807654698903], ["rondas", 1.127807654698903], ["aspekte", 1.127807654698903], ["anjouan", 1.127807654698903], ["broccoli's", 1.127807654698903], ["googoosh", 1.127807654698903], ["cloacal", 1.127807654698903], ["trackpads", 1.127807654698903], ["shukran", 1.127807654698903], ["premi\u00e8res", 1.127807654698903], ["singletrack", 1.127807654698903], ["torino's", 1.127807654698903], ["kururugi", 1.127807654698903], ["continente", 1.127807654698903], ["resultat", 1.127807654698903], ["arwen's", 1.127807654698903], ["milou", 1.127807654698903], ["maiori", 1.127807654698903], ["wookieepedia", 1.127807654698903], ["kakariko", 1.127807654698903], ["quotidienne", 1.127807654698903], ["elba's", 1.127807654698903], ["mikkeli", 1.127807654698903], ["burgs", 1.127807654698903], ["bacharach's", 1.127807654698903], ["mawlana", 1.127807654698903], ["mangkhut", 1.127807654698903], ["cuito", 1.127807654698903], ["drumlines", 1.127807654698903], ["gatien", 1.127807654698903], ["organizat", 1.127807654698903], ["scansnap", 1.127807654698903], ["taramasalata", 1.127807654698903], ["ayna", 1.127807654698903], ["mica's", 1.127807654698903], ["umani", 1.127807654698903], ["deene", 1.127807654698903], ["dioxides", 1.127807654698903], ["scalaris", 1.127807654698903], ["socko", 1.1278076546989029], ["savoyard", 1.1278076546989029], ["whitelighter", 1.1278076546989029], ["easterlies", 1.1278076546989029], ["kod\u00e1ly", 1.1278076546989029], ["eked", 1.1278076546989029], ["leafminer", 1.1278076546989029], ["bijeljina", 1.1278076546989029], ["chiasmus", 1.1278076546989029], ["kathy's", 1.1278076546989029], ["childhelp", 1.1278076546989029], ["saaz", 1.1278076546989029], ["zibi", 1.1278076546989029], ["errno", 1.1278076546989029], ["argenteus", 1.1278076546989029], ["almod\u00f3var", 1.1278076546989029], ["tarballs", 1.1278076546989029], ["technopark", 1.1278076546989029], ["paulsen's", 1.1278076546989029], ["contemporanea", 1.1278076546989029], ["cab's", 1.1278076546989029], ["torbj\u00f8rn", 1.1278076546989029], ["d'aubign\u00e9", 1.1278076546989029], ["maheshwar", 1.1278076546989029], ["marten's", 1.1278076546989029], ["desire's", 1.1278076546989029], ["ando's", 1.1278076546989029], ["qemu", 1.1278076546989029], ["hoglan", 1.1278076546989029], ["pr\u00e9sentation", 1.1278076546989029], ["maglione", 1.1278076546989029], ["heteroscedasticity", 1.1278076546989029], ["dfw's", 1.1278076546989029], ["maddaddam", 1.1278076546989029], ["tenebrous", 1.1278076546989029], ["tamami", 1.1278076546989029], ["juru", 1.1278076546989029], ["alveolus", 1.1278076546989029], ["prepending", 1.1278076546989029], ["satyananda", 1.1278076546989029], ["racetams", 1.1278076546989029], ["nightwatcher", 1.1278076546989029], ["chiwa", 1.1278076546989029], ["vrtx", 1.1278076546989029], ["alley", 1.1275793535542271], ["gown", 1.1275668416765472], ["sonata", 1.1275570307756366], ["caretaker", 1.1275277024441905], ["intended", 1.1274744899625848], ["daffodils", 1.127387614604043], ["greasing", 1.127292674034657], ["thrown", 1.1272549840155246], ["droning", 1.127142281451293], ["cavan", 1.127142281451293], ["synced", 1.1270809745090198], ["plate", 1.1269784778034764], ["sappho", 1.1269090031812146], ["alten", 1.126867814986654], ["reallocated", 1.126867814986654], ["corresponding", 1.1267299718288575], ["representatives", 1.1266866615919668], ["resides", 1.1265400898996738], ["excavations", 1.1262870151644777], ["goliath", 1.1262079275291172], ["trumpeters", 1.1262079275291172], ["onomatopoeia", 1.1262079275291172], ["chlorinated", 1.1261825716229679], ["iiot", 1.1261014403952736], ["p\u00fablico", 1.1261014403952736], ["shabnam", 1.1261014403952736], ["aldo's", 1.1261014403952736], ["maintainers", 1.1260725659993662], ["rhizomes", 1.1260725659993662], ["marcela", 1.1260725659993662], ["katherine's", 1.1260725659993662], ["snl's", 1.125979765469245], ["lamotta", 1.125979765469245], ["inaudibly", 1.125979765469245], ["streetscapes", 1.125979765469245], ["protectionist", 1.125851918881506], ["veritas", 1.125851918881506], ["civita", 1.125800879512606], ["exponentials", 1.125800879512606], ["dalio", 1.125800879512606], ["trapdoors", 1.125800879512606], ["windows", 1.125627226566485], ["fatma", 1.1255669772392363], ["em's", 1.1255669772392363], ["art's", 1.125552039389505], ["iguana", 1.125552039389505], ["chases", 1.125497687213375], ["injustice", 1.1253634895029612], ["implicit", 1.1253280775857353], ["impulse", 1.1252727727321512], ["oxygenated", 1.1252661163221167], ["stadion", 1.1252179700727287], ["froyo", 1.1251848461996032], ["brokenhearted", 1.1250434202511117], ["youngins", 1.1249668293722055], ["indulges", 1.1247358050557543], ["elio", 1.1247358050557543], ["kenichi", 1.1247092820211588], ["chessington", 1.124645577162364], ["paul's", 1.1245781383722484], ["fortuna", 1.124509971498029], ["throated", 1.124509971498029], ["tapetum", 1.1244003808176375], ["clay's", 1.1244003808176375], ["hospitably", 1.1244003808176375], ["alp's", 1.1244003808176375], ["bleated", 1.1244003808176375], ["russos", 1.1244003808176375], ["knots", 1.1243740482560258], ["patriarchal", 1.1243723785093027], ["inflight", 1.124283255777969], ["disdainful", 1.124283255777969], ["updated", 1.1242177317416922], ["stammer", 1.1241577917387124], ["microvilli", 1.1241577917387124], ["blaue", 1.1241577917387124], ["operons", 1.1241577917387124], ["samoyeds", 1.1241577917387124], ["coachella's", 1.1241577917387124], ["kapi", 1.1241577917387124], ["maputo", 1.12412501745907], ["doug's", 1.12412501745907], ["checkbox", 1.12412501745907], ["ventilation", 1.1240105133489118], ["adventist", 1.1239690873166215], ["interlocutor", 1.1239519020332656], ["youth's", 1.1239519020332656], ["magnets", 1.123766343936232], ["supply", 1.1237503404736395], ["chinks", 1.1236743282157133], ["assign", 1.1235876304329617], ["muggy", 1.1234827285894382], ["luang", 1.1232964240801073], ["shuri", 1.1232354615041777], ["rhymes", 1.1229533369618494], ["shopper", 1.122932615159237], ["spotlighting", 1.1228110385071988], ["scotland's", 1.122802216231419], ["kilkenny", 1.1227671735605838], ["videos", 1.1226822532989424], ["ballymena", 1.122661764259212], ["neri", 1.122661764259212], ["twittersphere", 1.1225065611398013], ["shredding", 1.122497608422941], ["storybooks", 1.1224371420574797], ["asmara", 1.1224371420574797], ["kumara", 1.1224371420574797], ["daguerreotype", 1.1224371420574797], ["tearjerker", 1.1224371420574797], ["haleigh", 1.1224371420574797], ["philae", 1.1224371420574797], ["democritus", 1.1224371420574797], ["fizz", 1.1223752827495679], ["doctors", 1.1223306394969286], ["dill", 1.1222997568503734], ["lough", 1.1222664032162784], ["side", 1.122226077511613], ["exam", 1.122187206661475], ["cowhand", 1.1221402795496624], ["faubus", 1.1221402795496624], ["piercingly", 1.1221402795496624], ["compi\u00e8gne", 1.1221402795496624], ["merry's", 1.1221402795496624], ["sabai", 1.1221402795496624], ["earnshaw", 1.1221052564448073], ["worker", 1.1220176830589805], ["bodybuilding", 1.1219234408483], ["nutria", 1.1219234408483], ["gloucester's", 1.1219234408483], ["nazari", 1.1219234408483], ["rothenburg", 1.1219234408483], ["alien", 1.1218736513464873], ["orientation", 1.1217356188436938], ["lennie", 1.1217296493741782], ["scandalous", 1.1217257600164954], ["kestrels", 1.1217196241742127], ["sickbed", 1.1217196241742127], ["ecology", 1.121717493363529], ["jiangxi", 1.1216986965692841], ["penguin", 1.1216729795622324], ["stalwart", 1.1215651769053667], ["banner", 1.121510728626834], ["symptomatology", 1.1214358600395873], ["airdate", 1.1214358600395873], ["uncrowded", 1.1214358600395873], ["nibble", 1.121435860039587], ["heartland", 1.1214082966581957], ["evidence", 1.1214020341209827], ["flaunt", 1.12130107207564], ["misspelled", 1.1212278147482544], ["servitude", 1.1211672918721713], ["splint", 1.121144458489522], ["ballparks", 1.121124350078465], ["inflict", 1.1210614079972794], ["hendrix", 1.1210614079972794], ["reserving", 1.1210487604110482], ["emerson's", 1.1210136326826445], ["vujicic", 1.1210136326826445], ["hirohito", 1.1210136326826445], ["ptms", 1.1210136326826445], ["nebulization", 1.1210136326826445], ["valcartier", 1.1210136326826445], ["fragilities", 1.1210136326826445], ["whackers", 1.1210136326826445], ["beetee", 1.1210136326826445], ["guybrush", 1.1210136326826445], ["spaza", 1.1210136326826445], ["trenchtown", 1.1210136326826445], ["mistretta", 1.1210136326826445], ["cvts", 1.1210136326826445], ["sporades", 1.1210136326826445], ["broderbund", 1.1210136326826445], ["rhinoviruses", 1.1210136326826445], ["book1", 1.1210136326826445], ["immortalizes", 1.1210136326826445], ["stored", 1.1209661195220542], ["saxony", 1.1208458790526135], ["enver", 1.1208074692559444], ["administrators", 1.1207349626270626], ["dayo", 1.1206583193283868], ["plate's", 1.1206583193283868], ["intellij", 1.1206583193283868], ["hide", 1.1206410383340293], ["production's", 1.1205314762814909], ["despondence", 1.1205314762814906], ["egnatia", 1.1205314762814906], ["derecha", 1.1205314762814906], ["akagera", 1.1205314762814906], ["preorders", 1.1205314762814906], ["zippers", 1.1205314762814906], ["polyphenols", 1.1205314762814906], ["h\u00e9ros", 1.1205314762814906], ["andr\u00e1ssy", 1.1205314762814906], ["paiement", 1.1205314762814906], ["retouches", 1.1205314762814906], ["mirnyi", 1.1205314762814906], ["rnli's", 1.1205314762814906], ["safta", 1.1205314762814906], ["gravenstein", 1.1205314762814906], ["tochter", 1.1205314762814906], ["barding", 1.1205314762814906], ["usonian", 1.1205314762814906], ["kaiping", 1.1205314762814906], ["valign", 1.1205314762814906], ["civic's", 1.1205314762814906], ["heugh", 1.1205314762814906], ["plectrums", 1.1205314762814906], ["milliner's", 1.1205314762814906], ["valujet", 1.1205314762814906], ["amalienborg", 1.1205314762814906], ["truus", 1.1205314762814906], ["wwoof", 1.1205314762814906], ["mordred's", 1.1205314762814906], ["zindzi", 1.1205314762814906], ["katowice", 1.1201429424825027], ["swap", 1.1200903945129037], ["aureum", 1.1199756570968273], ["inaugural", 1.1199756570968273], ["sundial", 1.1199756570968273], ["bootup", 1.1199756570968273], ["karenin", 1.1199756570968273], ["avms", 1.1199756570968273], ["aquaphor", 1.1199756570968273], ["furigana", 1.1199756570968273], ["jackal's", 1.1199756570968273], ["democracies", 1.1199377880974326], ["light", 1.1198745291726495], ["dualistic", 1.1197105741010647], ["ghillie", 1.119465882104976], ["headstart", 1.1193815055546008], ["production", 1.1191815606443039], ["galleria", 1.119178521397826], ["assuming", 1.1191567065710948], ["clan's", 1.1190613912542995], ["mourning", 1.1190613912542993], ["titans", 1.1190295044394076], ["valiant", 1.1189621044659706], ["chloroplast", 1.1189621044659703], ["ti's", 1.1189621044659703], ["aidan's", 1.1189621044659703], ["amputees", 1.1188851257851458], ["pitch", 1.1188355045834544], ["xiamen", 1.1187851934613118], ["sidon", 1.1187851934613118]];


In [3]:

word_penalties = {}
for el in word_penalties_list[:500]:
    word_penalties[el[0]] = el[1]

In [4]:
# Load the over-represented words and their penalty values from a JSON file
if os.path.exists('over_represented_words.json'):
    with open('over_represented_words.json', 'r') as f:
        over_represented_words = json.load(f)
        # Use the top 500 words
        for el in over_represented_words[:500]:
            word_penalties[el[0]] = el[1]

In [5]:

def precompute_starting_tokens(
    tokenizer: PreTrainedTokenizer, word_penalties: Dict[str, float]
) -> Dict[Tuple[int, ...], Set[int]]:
    """
    Precompute all starting token IDs for each target word variant.

    Args:
        tokenizer (PreTrainedTokenizer): The tokenizer used by the model.
        word_penalties (Dict[str, float]): Dictionary of target words with their respective penalty.

    Returns:
        Dict[Tuple[int, ...], Set[int]]: A mapping from each token sequence (word variant) to a set of starting token IDs.
    """
    starting_tokens_lookup = {}

    for word in word_penalties.keys():
        variants = [
            word.lower(),
            word.capitalize(),
            word.upper(),
            f" {word.lower()}",
            f" {word.capitalize()}",
            f" {word.upper()}",
        ]

        for variant in variants:
            # Encode the full variant
            token_ids = tokenizer.encode(variant, add_special_tokens=False)
            starting_tokens = set()
            if token_ids:
                starting_tokens.add(token_ids[0])
                first_token_decoded = tokenizer.decode(token_ids[0], skip_special_tokens=True)

                # Iterate over all possible prefixes of the first token
                for i in range(len(first_token_decoded) - 1):
                    prefix = first_token_decoded[:-(i + 1)]
                    encoded_prefix = tokenizer.encode(prefix, add_special_tokens=False)
                    if encoded_prefix:
                        starting_tokens.add(encoded_prefix[0])  # Add the first token of the prefix

                starting_tokens_lookup[tuple(token_ids)] = starting_tokens

    return starting_tokens_lookup

# Precompute starting tokens
starting_tokens_lookup = precompute_starting_tokens(tokenizer, word_penalties)

class AdvancedCustomWordSampler:
    """
    A sampler that generates text while downregulating specified words or phrases.
    It uses backtracking and custom adjustments to avoid overrepresented words.
    """
    def __init__(
        self,
        model: PreTrainedModel,
        tokenizer: PreTrainedTokenizer,
        word_penalties: Dict[str, float],
        starting_tokens_lookup: Dict[Tuple[int, ...], Set[int]],
        max_backtrack: int = 5,
        adjustment_strength: float = 1.0,
        device: torch.device = torch.device('cuda'),
        output_every_n_tokens: int = 5,
        slow_debug: bool = False,
        debug_delay: float = 2.0,
    ):
        """
        Initializes the AdvancedCustomWordSampler with the necessary parameters.

        Args:
            model (PreTrainedModel): The language model to use for Generation.
            tokenizer (PreTrainedTokenizer): The tokenizer associated with the model.
            word_penalties (Dict[str, float]): Dictionary of target words with their respective penalty.
            starting_tokens_lookup (Dict[Tuple[int, ...], Set[int]]): Mapping from token sequences to starting token IDs.
            max_backtrack (int): Maximum number of tokens to backtrack.
            adjustment_strength (float): Strength of the downregulation adjustment.
            device (torch.device): Device to run the model on.
            output_every_n_tokens (int): Frequency of updating the inference output display.
            slow_debug (bool): Enables slow debug mode when set to True.
            debug_delay (float): Time in seconds to pause during slow debug steps.
        """
        self.model = model.to(device)
        self.tokenizer = tokenizer
        self.word_penalties = word_penalties
        self.starting_tokens_lookup = starting_tokens_lookup
        self.max_backtrack = max_backtrack
        self.adjustment_strength = adjustment_strength
        self.device = device
        self.output_every_n_tokens = output_every_n_tokens
        self.slow_debug = slow_debug
        self.debug_delay = debug_delay

        # Prepare token sequences for downregulation
        self.token_sequences = self._prepare_token_sequences()
        self.max_sequence_length = max(len(seq) for seq in self.token_sequences.keys())

        # Initialize a cache to store logits along with their positions
        self.logit_cache = {}

        # Record of downregulated sequences at specific positions
        self.downregulated_positions = {}  # Key: position, Value: set of sequences

    def _prepare_token_sequences(self) -> Dict[Tuple[int, ...], float]:
        """
        Prepares the token sequences from word_penalties for efficient lookup.

        Returns:
            Dict[Tuple[int, ...], float]: Mapping from token ID sequences to their adjustment factors.
        """
        token_sequences = {}
        for word, penalty in self.word_penalties.items():
            variants = [
                word.lower(),
                word.capitalize(),
                word.upper(),
                f" {word.lower()}",
                f" {word.capitalize()}",
                f" {word.upper()}",
            ]
            for variant in variants:
                token_ids = tuple(self.tokenizer.encode(variant, add_special_tokens=False))
                if token_ids:
                    token_sequences[token_ids] = 1 / penalty  # Inverse of the over-representation penalty
        return token_sequences

    def _adjust_logits(self, logits: torch.FloatTensor, adjustment: float) -> torch.FloatTensor:
        """
        Adjusts the logits by applying the downregulation factor.

        Args:
            logits (torch.FloatTensor): The original logits.
            adjustment (float): The adjustment factor to apply.

        Returns:
            torch.FloatTensor: The adjusted logits.
        """
        log_adjustment = torch.log(torch.tensor(adjustment ** self.adjustment_strength, device=self.device))
        return logits + log_adjustment  # Lowering the logit for disallowed tokens

    @torch.no_grad()
    def generate_stream(
        self,
        prompt: str,
        max_length: int,
        temperature: float = 1.0,
        top_k: int = 50,
        top_p: float = 0.95,
    ) -> Generator[str, None, None]:
        """
        Generates text in a streaming fashion with custom downregulation and backtracking.

        Args:
            prompt (str): The initial text prompt.
            max_length (int): The maximum length of the generated text.
            temperature (float): Sampling temperature.
            top_k (int): Top-k filtering.
            top_p (float): Top-p (nucleus) filtering.

        Yields:
            Generator[str, None, None]: Yields generated text chunks.
        """
        # Encode the prompt
        input_ids = self.tokenizer.encode(prompt, return_tensors="pt").to(self.device)
        generated_sequence = input_ids[0].tolist()
        current_position = len(generated_sequence)  # Tracks the current position in the sequence

        # Initialize display (if using in Jupyter)
        with inference_output:
            inference_output.clear_output(wait=True)
            display(HTML(f"<div style='white-space: pre-wrap;'>{self.tokenizer.decode(generated_sequence)}</div>"))

        past_key_values = None
        output_tokens_counter = 0

        while len(generated_sequence) < max_length:
            current_input_ids = torch.tensor([generated_sequence], device=self.device)

            regenerating = False
            if current_position in self.logit_cache:
                # We backtracked and want to use the cached logits
                next_token_logits = self.logit_cache[current_position]
                past_key_values = None
                regenerating = True
            else:
                if past_key_values is None:
                    outputs = self.model(current_input_ids, use_cache=True)
                else:
                    outputs = self.model(current_input_ids[:, -1:], past_key_values=past_key_values, use_cache=True)

                next_token_logits = outputs.logits[:, -1, :] / temperature
                past_key_values = outputs.past_key_values
                self.logit_cache[current_position] = next_token_logits.clone()

            # Apply top-k and top-p filtering
            filtered_logits = self._filter_logits(next_token_logits, top_k, top_p)

            # Sample the next token
            probs = torch.softmax(filtered_logits, dim=-1)
            next_token_index = torch.multinomial(probs, num_samples=1)
            next_token = next_token_index.item()

            if regenerating:
                with debug_output:
                    debug_output.clear_output(wait=True)
                alt_token = tokenizer.decode(next_token, skip_special_tokens=True)
                debug_info = f"Alternate token: {alt_token}"
                self._display_debug(debug_info)
                if self.slow_debug:
                    time.sleep(self.debug_delay)

            # Append the new token to the sequence
            generated_sequence.append(next_token)
            current_position += 1
            output_tokens_counter += 1

            # Yield the current text chunk
            current_text = self.tokenizer.decode(generated_sequence)
            if output_tokens_counter >= self.output_every_n_tokens:
                output_tokens_counter = 0
                with inference_output:
                    inference_output.clear_output(wait=True)
                    display(HTML(f"<div style='white-space: pre-wrap;'>{current_text}</div>"))
                yield current_text  # Yield the generated text chunk

            # Check for end-of-sequence token
            if next_token == self.tokenizer.eos_token_id:
                break

            # After adding the token, check for disallowed sequences
            matched_sequence, start_pos = self._detect_disallowed_sequence(generated_sequence)

            if matched_sequence:
                # Downregulate the relevant tokens at the start_pos
                adjustment = self.token_sequences[matched_sequence]
                word = self.tokenizer.decode(torch.tensor(matched_sequence))

                # Display debug information
                debug_info = f"Replacing '{word}'"
                self._display_debug(debug_info)

                if self.slow_debug:
                    time.sleep(self.debug_delay)
                    with debug_output:
                        debug_output.clear_output(wait=True)

                # Identify starting tokens to downregulate
                starting_tokens = self.starting_tokens_lookup.get(matched_sequence, set())

                # Find the cached logits corresponding to start_pos
                for token_id in starting_tokens:
                    # Apply downregulation
                    self.logit_cache[start_pos][:, token_id] = self._adjust_logits(
                        self.logit_cache[start_pos][:, token_id], adjustment
                    )

                # Record that this sequence has been downregulated at this position
                if start_pos not in self.downregulated_positions:
                    self.downregulated_positions[start_pos] = set()
                self.downregulated_positions[start_pos].add(matched_sequence)

                # Backtrack: remove tokens from the generated_sequence that are part of the disallowed sequence
                for _ in range(len(matched_sequence)):
                    generated_sequence.pop()
                    current_position -= 1

                # Update the model's past_key_values by re-encoding up to start_pos
                # This is necessary because we've modified the generated_sequence
                new_input_ids = torch.tensor([generated_sequence], device=self.device)
                outputs = self.model(new_input_ids, use_cache=True)
                past_key_values = outputs.past_key_values

                # Clear the logit_cache ahead of start_pos since we've backtracked
                to_del = [key for key in self.logit_cache if key > start_pos]
                for key in to_del:
                    del self.logit_cache[key]

                continue  # Continue to the next iteration

        # Final display of the generated text
        final_text = self.tokenizer.decode(generated_sequence)
        with inference_output:
            inference_output.clear_output(wait=True)
            display(HTML(f"<div style='white-space: pre-wrap;'>{final_text}</div>"))
        yield final_text

        # Clear variables to free up memory
        del outputs, next_token_logits, filtered_logits, past_key_values

    def _filter_logits(self, logits: torch.FloatTensor, top_k: int, top_p: float) -> torch.FloatTensor:
        """
        Applies top-k and top-p (nucleus) filtering to the logits.

        Args:
            logits (torch.FloatTensor): The original logits.
            top_k (int): The number of top tokens to keep.
            top_p (float): The cumulative probability threshold.

        Returns:
            torch.FloatTensor: The filtered logits.
        """
        # Apply top-k
        if top_k > 0:
            top_k = min(top_k, logits.size(-1))
            top_k_logits, _ = torch.topk(logits, top_k)
            min_top_k = top_k_logits[:, -1].unsqueeze(-1)
            logits = torch.where(logits < min_top_k, float('-inf'), logits)

        # Apply top-p
        if top_p < 1.0:
            sorted_logits, sorted_indices = torch.sort(logits, descending=True)
            cumulative_probs = torch.cumsum(torch.softmax(sorted_logits, dim=-1), dim=-1)

            # Remove tokens with cumulative probability above the threshold
            sorted_indices_to_remove = cumulative_probs > top_p
            # Shift the mask right to keep the first token above the threshold
            sorted_indices_to_remove[:, 1:] = sorted_indices_to_remove[:, :-1].clone()
            sorted_indices_to_remove[:, 0] = False
            indices_to_remove = sorted_indices_to_remove.scatter(
                dim=1, index=sorted_indices, src=sorted_indices_to_remove
            )
            logits = logits.masked_fill(indices_to_remove, float('-inf'))

        return logits

    def _detect_disallowed_sequence(self, generated_sequence: List[int]) -> Tuple[Tuple[int, ...], int]:
        """
        Detects if the recent tokens in the generated_sequence match any disallowed sequence.

        Args:
            generated_sequence (List[int]): The list of generated token IDs.

        Returns:
            Tuple[Tuple[int, ...], int]: The matched disallowed sequence and its start position.
                                         Returns (None, -1) if no match is found.
        """
        # Start checking from the longest possible sequence to the shortest
        for seq_length in range(self.max_sequence_length, 0, -1):
            if len(generated_sequence) < seq_length:
                continue
            candidate_sequence = tuple(generated_sequence[-seq_length:])
            if candidate_sequence in self.token_sequences:
                start_pos = len(generated_sequence) - seq_length
                return candidate_sequence, start_pos
        return None, -1

    def _display_debug(self, message: str):
        """
        Displays debug information in the debug_output widget.

        Args:
            message (str): The debug message to display.
        """
        with debug_output:
            debug_output.clear_output(wait=True)
            display(HTML(f"<pre>{message}</pre>"))


In [6]:

# Setup separate output widgets for prompt, inference, and debug
prompt_output = Output()
inference_output = Output()
debug_output = Output()

# Display the output widgets
display(HTML("<h2>Prompt</h2>"))
display(prompt_output)
display(HTML("<h2>Inference Output</h2>"))
display(inference_output)
display(HTML("<h2>Debug Information</h2>"))
display(debug_output)

# Enable slow debug mode
SLOW_DEBUG = True

# Initialize the sampler
sampler = AdvancedCustomWordSampler(
    model=model,
    tokenizer=tokenizer,
    word_penalties=word_penalties,
    starting_tokens_lookup=starting_tokens_lookup,
    max_backtrack=5,
    adjustment_strength=1.0,
    device=device,
    output_every_n_tokens=5,
    slow_debug=SLOW_DEBUG,          # Enable slow debug
    debug_delay=1.5                # Set delay to 1.5 seconds per debug step
)

# Define the prompt
prompt = "Write a story about Elara, the weaver of tapestries in future Technopolis. In the bustling city, a group of "

# Display the prompt
with prompt_output:
    prompt_output.clear_output(wait=True)
    display(HTML(f"<div style='white-space: pre-wrap;'>{prompt}</div>"))

# Start generating
try:
    for text_chunk in sampler.generate_stream(prompt, max_length=300, temperature=1.0, top_k=50, top_p=0.95):
        pass  # The text_chunk is already being displayed via the inference_output widget
    print("\nGeneration complete.")
except Exception as e:
    with debug_output:
        debug_output.clear_output(wait=True)
        debug_output.append_stdout(f"\n\nAn error occurred: {str(e)}\n")
        traceback.print_exc(file=sys.stdout)


Output()

Output()

Output()

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)



Generation complete.
